# Inhibitor csv File Generation by Web Scraping and Parsing KIDFamMap Data in Python

KIDFamMap is a database listing thousands of known kinase inhibitors. 

Kinases are listed on the web page at URL http://gemdock.life.nctu.edu.tw/kidfammap/browse.php and each can be clicked on individually to navigate to a page of information about that kinase. This page includes a link to another page, which contains a table of inhibitors for that kinase.

Here we use the scrapy package to create and run a spider to scrape the website's HTML code for all of the relevant information.
- First, the URLs for each kinase's web page are obtained from the main page.
- These URLs are then followed, taking us to each kinase's web page.
- From there we extract the URLs for the web pages containing the individual kinase's table of inhibitors, and we also extract information about the inhibitor type.
- We then follow these URLs and extract all of the relevant information about each inhibitor.

Once the data has been extracted from the website, it is cleaned and inserted into a Pandas data frame.

After that, the following steps take place:
- Using information in the data frame, we generate a column of URLs that our web app can use to display images of the inhibitors' chemical structures.
- Convert kinases and inhibitors to uppercase.
- Translate kinase names to match "Entry name" in kinase table.
- Remove duplicate rows.
- Remove unnecessary columns.
- Use this data frame to make two data frames: one listing the kinase-inhibitor pairs, and one listing each inhibitor alongside all of its information. These two data sets can then be linked via the inhibitor name in our relational database.
- Make primary key columns.
- Export as csv files.

Import packages

In [1]:
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
import urllib.parse 
import urllib.request 

Make a spider

In [2]:
### In scraping spider, get the 'ACC_ID' (div > tr > td > a, in parse3) and
### ENSURE YOU GET CORRESPONDING KINASE NAME FOR TRANSLATION - make kin_dict
### 'INHIB_TYPE' (td 7 (think about it later), in parse2) and representative
### inhibitor (td > span::attr(title), in parse2). Use dictionary
### inh_types and make the keys the types and make the values a list of representative
### inhibitors (append as you go along)

# Use the scrapy.Spider class to make your own spider

class InhibitorSpider( scrapy.Spider ):
    
    name = "inhibitor_spider"
    
    # Define the first action to take
        
    def start_requests( self ): 
        
        # Define which URL to start at
        
        url = 'http://gemdock.life.nctu.edu.tw/kidfammap/browse.php'

        # Go to the website at the above URL and get a response object
        # which contains the HTML code for that web page
        # Define what to do with the response object
        # i.e. send it to the parse method defined below
        
        yield scrapy.Request( url = url, callback = self.parse )
            
    # Using the HTML in the previous response object, get the URLs for each 
    # kinase's web page, and go to those websites
    
    def parse( self, response ):
        
        # Each kinase listed on the page is a hyperlink, leading to
        # a page of information for that kinase
        # Define a CSS locator to point to the hyperlinks' URLs in the HTML
        # code and extract them as strings
        
        links = response.css( 'table > tr > td > a::attr(href)' ).extract()
        
        # Go to the kinases' web pages using the new URLs
        # and send the response objects to the parse2 method below
        
        for link in links:
            yield response.follow( url = link, callback = self.parse2 )
    
    # Using the previous response objects, get the URLs for each 
    # inhibitor table
    
    def parse2( self, response ):
        
        # Each kinase's web page has a hyperlink to another page 
        # containing a table of inhibitors for that kinase
        # Define a CSS locator to point to the URLs in those
        # hyperlinks and extract them as strings
        
        inhib_links = response.css( 'a.show_inhibitor::attr(href)' ).extract()
        
        # Go to the kinases' web pages using the new URLs
        # and send the response objects to the parse3 method below
        
        for ilink in inhib_links:
            yield response.follow( url = ilink, callback = self.parse3 )

    # Using the previous response objects, get information from the 
    # table of inhibitors for each kinase
    
    def parse3( self, response ):
        
        # Each inhibitor list web page has information we'd like to
        # extract and place into a list "inhibs" (which we must initialise
        # in the next cell rather than here)
        # Define a CSS locator to point to the data in the rows
        # of the inhibitor table and extract the text
       
        raw = response.css( 'div.result tbody > tr' ).extract()
        
        # Save the data in a list "fields"
        
        fields = [ field for field in raw ]
        
        # For each kinase, append the information about its
        # inhibitors to "inhibs"
        
        inhibs.append( fields )

Run the spider: crawl KIDFamMap for inhibitors

In [3]:
# Our UniProt accession IDs and kinase names will be returned to "kin_dict"

kin_dict = {}

# Our inhibitor types will be returned to "inh_types"

inh_types = {}

# Our inhibitor table data will be returned to "inhibs"

inhibs = [] 

# Run the spider

process = CrawlerProcess()
process.crawl( InhibitorSpider )
process.start()

# N.B. kernel needs to be cleared before repeating

2020-01-16 09:31:37 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-01-16 09:31:37 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-01-16 09:31:37 [scrapy.crawler] INFO: Overridden settings: {}
2020-01-16 09:31:37 [scrapy.extensions.telnet] INFO: Telnet Password: 41bb7577b40e4e34
2020-01-16 09:31:37 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-01-16 09:31:37 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defau

2020-01-16 09:31:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MAP3K12> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CASK&Query_Pid=CASK&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=CASK)
2020-01-16 09:31:43 [scrapy.dupefilters] DEBUG: Filtered duplicate request: <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAP3K9&Query_Pid=MAP3K9&Query_Cid=&Query_Family=> - no more duplicates will be shown (see DUPEFILTER_DEBUG to show all duplicates)
2020-01-16 09:31:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&Que

2020-01-16 09:31:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=SRMS> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=IRAK4&Query_Pid=IRAK4&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=IRAK1)
2020-01-16 09:31:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=RYK> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=CHEK1&Query_Pid=CHEK1&Query_Cid=&Query_Family=> (referer: http

2020-01-16 09:31:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ALK&Query_Pid=ALK&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=ROS1)
2020-01-16 09:31:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FRK> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FLT4> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MATK> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/br

2020-01-16 09:31:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=NRK> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MYO3B> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:31:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=BTK&Query_Pid=BTK&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=BMX)
2020-01-16 09:31:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MINK1> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/b

...

2020-01-16 09:32:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FYN> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FLT1> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FGFR2> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=FGFR1> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET 

2020-01-16 09:32:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=ERBB2&Query_Pid=ERBB2&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=ERBB2)
2020-01-16 09:32:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PAK7> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PAK4> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PAK1> (referer: http://gemdock.life.nctu.edu.tw/kidfam

2020-01-16 09:32:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=CAMKK2> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=EIF2AK2> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=BUB1> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=AURKA> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:10 [scrapy.core.engine] DEBUG: Crawled (200) 

2020-01-16 09:32:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK12&Query_Pid=MAPK12&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MAPK12)
2020-01-16 09:32:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=CDK6> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=MAPK9&Query_Pid=MAPK9&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=MAPK9)
2020-01-16 09:32:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?Prot

2020-01-16 09:32:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PRKCQ> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/show_inhibitor.php?QueryType=Protein&QueryName=STK11&Query_Pid=STK11&Query_Cid=&Query_Family=> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=STK11)
2020-01-16 09:32:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PRKCI> (referer: http://gemdock.life.nctu.edu.tw/kidfammap/browse.php)
2020-01-16 09:32:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://gemdock.life.nctu.edu.tw/kidfammap/ProcessQuery.php?ProteinQueryType=x&Query_ProteinName=PRKCB> (referer: http://gemdock.life.nctu.edu.tw/kid

Clean up the data in "inhibs" and store in "inhibitors"

In [35]:
inhibitors = []

for i in inhibs: # for each kinase
    for j in i: # for each kinase-inhibitor relationship
        chemical = []
        inh = j.split("</td>") # Split row into individual fields
        for k in inh: # for each field, remove unnecessary characters
            field = k.replace("<tr>","") 
            field = field.replace("\r","")
            field = field.replace("\t","")
            field = field.replace("<td>","")
            field = field.replace("\n","")
            chemical.append(field) # Make a row of cleaned, separate fields
        inhibitors.append(chemical) # Add this row to "inhibitors"

Define column names, based on those on the KIDFamMap website

In [36]:
headers = ["Index","Kinase","Inhibitor","Partial_Img_URL",
           "Ki_nM","IC50_nM","Kd_nM","EC50_nM","POC","Source","Link","To_Remove"]

Save the inhibitors information in a Pandas data frame

In [37]:
inhibitors_df = pd.DataFrame(inhibitors, columns = headers)

Using information in the data frame, generate a column of URLs for the inhibitors' chemical structure images. Our web app can subsequently use these to display images.

In [38]:
IMG_URL = []

for n,i in enumerate(inhibitors_df.Partial_Img_URL):
    URL = 'http://gemdock.life.nctu.edu.tw/kidfammap/data/png/'
    URL += str(inhibitors_df.Source[n])+"/"
    URL += str(i)+".png"
    IMG_URL.append(URL)

IMG_URL = pd.Series(IMG_URL)
inhibitors_df = inhibitors_df.assign(IMG_URL = IMG_URL)

Make a column listing the inhibitor type

In [ ]:
### INHIB_TYPE

Make "kinase" and "inhibitor" entries uppercase.

In [39]:
uppercase_kinase = []
uppercase_inhib = []

for n,i in enumerate(inhibitors_df.Kinase):
    uppercase_kinase.append(str(i).upper())
    uppercase_inhib.append(inhibitors_df.Inhibitor[n].upper())

uppercase_kinase = pd.Series(uppercase_kinase)
uppercase_inhib = pd.Series(uppercase_inhib)

inhibitors_df = inhibitors_df.assign(Kinase = uppercase_kinase)
inhibitors_df = inhibitors_df.assign(Inhibitor = uppercase_inhib)

Translate kinase names using UniProt

In [12]:
# Get a list of unique kinase names appearing in KIDFamMap

kinase_list = list(inhibitors_df.Kinase.drop_duplicates())

In [13]:
# This cell takes several minutes to run

# Create partial URL

url = "https://www.uniprot.org/uploadlists/"

# Define parameters

params = {
"from": "GENENAME", # Assume kinase names are in format "GENENAME"
"to": "ID", # Retrieve IDs in "ID" format
"format": "tab", # Produce tab-delimited output
"query": "", # The query protein ID will be defined during the loop
}

# Create an empty list to store the results in

results=[]

for i in kinase_list: 
    params["query"]=str(i) # Enter the kinase ID
    data = urllib.parse.urlencode(params) 
    data = data.encode("utf-8")
    req = urllib.request.Request(url, data) # Run query in Uniprot
    with urllib.request.urlopen(req) as f:
       response = f.read()
    line=response.decode("utf-8") 
    results.append(line) # Store results

If the above cell fails to run, un-comment and run the cell below to obtain a snapshot of the data from 16th January 2020

In [40]:
# results = ['From\tTo\nADRBK1\tA0A0A9Y2P2_LYGHE\nADRBK1\tA0A0P6J9Z0_HETGA\nADRBK1\tA0A0S2Z392_HUMAN\nADRBK1\tA0A0S2Z3I6_HUMAN\nADRBK1\tA0A151NSD8_ALLMI\nADRBK1\tA0A1S3FIZ2_DIPOR\nADRBK1\tA0A1S3FKL1_DIPOR\nADRBK1\tA0A1S3W6T8_ERIEU\nADRBK1\tA0A1U7Q2D0_MESAU\nADRBK1\tA0A1V4JPF8_PATFA\nADRBK1\tA0A1V4JPV4_PATFA\nADRBK1\tA0A218UUU4_9PASE\nADRBK1\tA0A250YKW7_CASCN\nADRBK1\tA0A2U3VGL8_ODORO\nADRBK1\tA0A2U3YJB9_LEPWE\nADRBK1\tA0A340WTX2_LIPVE\nADRBK1\tA0A384DK66_URSMA\nADRBK1\tA0A3R7JGX0_CLOSI\nADRBK1\tARBK1_BOVIN\nADRBK1\tARBK1_DIDVI\nADRBK1\tARBK1_HUMAN\nADRBK1\tARBK1_MESAU\nADRBK1\tARBK1_MOUSE\nADRBK1\tARBK1_RAT\nADRBK1\tD1LUD8_BALMY\nADRBK1\tF1LMA4_RAT\nADRBK1\tH2Q482_PANTR\nADRBK1\tH9FQI5_MACMU\nADRBK1\tJ9Q9K2_TURTR\nADRBK1\tQ3U1V3_MOUSE\nADRBK1\tQ3U5J8_MOUSE\nADRBK1\tQ7TS64_MOUSE\nADRBK1\tQ9N1W8_HORSE\nADRBK1\tT2MGH5_HYDVU\nADRBK1\tU3FHE2_CALJA\nADRBK1\tV8P629_OPHHA\n','From\tTo\nCAMK2G\tA0A096P4K7_PAPAN\nCAMK2G\tA0A0D9R9D2_CHLSB\nCAMK2G\tA0A0G3FMJ5_RAT\nCAMK2G\tA0A151M788_ALLMI\nCAMK2G\tA0A151M7C2_ALLMI\nCAMK2G\tA0A1A7X516_9TELE\nCAMK2G\tA0A1A7XAP2_9TELE\nCAMK2G\tA0A1A7XCW8_9TELE\nCAMK2G\tA0A1A7XLZ2_9TELE\nCAMK2G\tA0A1A7XNQ4_9TELE\nCAMK2G\tA0A1A7YCM8_9TELE\nCAMK2G\tA0A1A7YZV2_9TELE\nCAMK2G\tA0A1A7YZY9_9TELE\nCAMK2G\tA0A1A7ZAD9_NOTFU\nCAMK2G\tA0A1A7ZBX4_NOTFU\nCAMK2G\tA0A1A7ZLI1_NOTFU\nCAMK2G\tA0A1A8A8Y2_NOTFU\nCAMK2G\tA0A1A8A969_NOTFU\nCAMK2G\tA0A1A8AAW4_NOTFU\nCAMK2G\tA0A1A8AP45_NOTFU\nCAMK2G\tA0A1A8B9N8_NOTFU\nCAMK2G\tA0A1A8BXW3_9TELE\nCAMK2G\tA0A1A8BY04_9TELE\nCAMK2G\tA0A1A8BYX7_9TELE\nCAMK2G\tA0A1A8CVM4_9TELE\nCAMK2G\tA0A1A8DAL9_9TELE\nCAMK2G\tA0A1A8DKL0_9TELE\nCAMK2G\tA0A1A8ELM4_9TELE\nCAMK2G\tA0A1A8FDJ3_9TELE\nCAMK2G\tA0A1A8FXJ2_9TELE\nCAMK2G\tA0A1A8FXS8_9TELE\nCAMK2G\tA0A1A8GCE1_9TELE\nCAMK2G\tA0A1A8GVY3_9TELE\nCAMK2G\tA0A1A8HG96_9TELE\nCAMK2G\tA0A1A8HLT1_NOTKU\nCAMK2G\tA0A1A8IH25_NOTKU\nCAMK2G\tA0A1A8IIW2_NOTKU\nCAMK2G\tA0A1A8IJM5_NOTKU\nCAMK2G\tA0A1A8IMF5_NOTKU\nCAMK2G\tA0A1A8JGR2_NOTKU\nCAMK2G\tA0A1A8JIZ1_NOTKU\nCAMK2G\tA0A1A8JRA9_NOTKU\nCAMK2G\tA0A1A8KUN9_NOTKU\nCAMK2G\tA0A1A8L6U9_9TELE\nCAMK2G\tA0A1A8MCY1_9TELE\nCAMK2G\tA0A1A8NKM5_9TELE\nCAMK2G\tA0A1A8NLY4_9TELE\nCAMK2G\tA0A1A8PBH2_9TELE\nCAMK2G\tA0A1A8PFL9_9TELE\nCAMK2G\tA0A1A8Q586_9TELE\nCAMK2G\tA0A1A8QGM1_9TELE\nCAMK2G\tA0A1A8QL20_9TELE\nCAMK2G\tA0A1A8R7E8_9TELE\nCAMK2G\tA0A1A8RC28_9TELE\nCAMK2G\tA0A1A8S936_9TELE\nCAMK2G\tA0A1A8UA30_NOTFU\nCAMK2G\tA0A1D5Q485_MACMU\nCAMK2G\tA0A1D5RJS6_MACMU\nCAMK2G\tA0A1Q9F671_SYMMI\nCAMK2G\tA0A1S2ZJP8_ERIEU\nCAMK2G\tA0A1S2ZJQ0_ERIEU\nCAMK2G\tA0A1S2ZJQ4_ERIEU\nCAMK2G\tA0A1S2ZJQ5_ERIEU\nCAMK2G\tA0A1S2ZJQ8_ERIEU\nCAMK2G\tA0A1S2ZJQ9_ERIEU\nCAMK2G\tA0A1S3FWW4_DIPOR\nCAMK2G\tA0A1S3W7H6_ERIEU\nCAMK2G\tA0A1S3W7V0_ERIEU\nCAMK2G\tA0A1S3W7V5_ERIEU\nCAMK2G\tA0A1S3W7X0_ERIEU\nCAMK2G\tA0A1S3W7X5_ERIEU\nCAMK2G\tA0A1S3W807_ERIEU\nCAMK2G\tA0A1S3W812_ERIEU\nCAMK2G\tA0A1S3W8X4_ERIEU\nCAMK2G\tA0A1S3W8X8_ERIEU\nCAMK2G\tA0A1U7QPX6_MESAU\nCAMK2G\tA0A1U7QY78_MESAU\nCAMK2G\tA0A1U7R175_MESAU\nCAMK2G\tA0A1U7R180_MESAU\nCAMK2G\tA0A1U7R8T8_MESAU\nCAMK2G\tA0A1U7RBB2_MESAU\nCAMK2G\tA0A1U8CGL7_MESAU\nCAMK2G\tA0A1U8CGL9_MESAU\nCAMK2G\tA0A1U8CGM2_MESAU\nCAMK2G\tA0A1U8CGM4_MESAU\nCAMK2G\tA0A1U8CJC2_MESAU\nCAMK2G\tA0A1U8CJC7_MESAU\nCAMK2G\tA0A1U8CS14_MESAU\nCAMK2G\tA0A1U8CS19_MESAU\nCAMK2G\tA0A1U8CT32_MESAU\nCAMK2G\tA0A1U8CT37_MESAU\nCAMK2G\tA0A1V4KJB9_PATFA\nCAMK2G\tA0A1V4KJC7_PATFA\nCAMK2G\tA0A1V4KJG2_PATFA\nCAMK2G\tA0A1V4KJG7_PATFA\nCAMK2G\tA0A1V4KL86_PATFA\nCAMK2G\tA0A218UWM8_9PASE\nCAMK2G\tA0A250Y2C8_CASCN\nCAMK2G\tA0A286XBZ6_CAVPO\nCAMK2G\tA0A286XJB7_CAVPO\nCAMK2G\tA0A286XQP7_CAVPO\nCAMK2G\tA0A286YCB8_MOUSE\nCAMK2G\tA0A286YCH4_MOUSE\nCAMK2G\tA0A286YCI7_MOUSE\nCAMK2G\tA0A286YCW8_MOUSE\nCAMK2G\tA0A286YDK9_MOUSE\nCAMK2G\tA0A286YDL6_MOUSE\nCAMK2G\tA0A286ZKT1_PIG\nCAMK2G\tA0A286ZVC8_PIG\nCAMK2G\tA0A287B7M7_PIG\nCAMK2G\tA0A287BCC5_PIG\nCAMK2G\tA0A287BS43_PIG\nCAMK2G\tA0A287CSL4_ICTTR\nCAMK2G\tA0A287DFE9_ICTTR\nCAMK2G\tA0A2D0QMU5_ICTPU\nCAMK2G\tA0A2D0QMU9_ICTPU\nCAMK2G\tA0A2D0QMV4_ICTPU\nCAMK2G\tA0A2D0QN59_ICTPU\nCAMK2G\tA0A2D0QN66_ICTPU\nCAMK2G\tA0A2D0QNF0_ICTPU\nCAMK2G\tA0A2D0QNF5_ICTPU\nCAMK2G\tA0A2D0QNF9_ICTPU\nCAMK2G\tA0A2D0QQ22_ICTPU\nCAMK2G\tA0A2D0QQT6_ICTPU\nCAMK2G\tA0A2I0MFJ8_COLLI\nCAMK2G\tA0A2I2U9U9_FELCA\nCAMK2G\tA0A2I3BQP6_MOUSE\nCAMK2G\tA0A2I3G8J1_NOMLE\nCAMK2G\tA0A2I3GNX8_NOMLE\nCAMK2G\tA0A2I3GSG1_NOMLE\nCAMK2G\tA0A2I3H8R6_NOMLE\nCAMK2G\tA0A2I3HJ50_NOMLE\nCAMK2G\tA0A2I3LCB9_PAPAN\nCAMK2G\tA0A2I3LER3_PAPAN\nCAMK2G\tA0A2I3LKN4_PAPAN\nCAMK2G\tA0A2I3MHT9_PAPAN\nCAMK2G\tA0A2I3MIT3_PAPAN\nCAMK2G\tA0A2I3MKM4_PAPAN\nCAMK2G\tA0A2I3N097_PAPAN\nCAMK2G\tA0A2I3N8S0_PAPAN\nCAMK2G\tA0A2I3S956_PANTR\nCAMK2G\tA0A2I3SZW8_PANTR\nCAMK2G\tA0A2I3T1W6_PANTR\nCAMK2G\tA0A2I3T5B9_PANTR\nCAMK2G\tA0A2I3TF83_PANTR\nCAMK2G\tA0A2I3TRJ8_PANTR\nCAMK2G\tA0A2I3TUE7_PANTR\nCAMK2G\tA0A2K5F2W6_AOTNA\nCAMK2G\tA0A2K5F2W7_AOTNA\nCAMK2G\tA0A2K5F2X0_AOTNA\nCAMK2G\tA0A2K5F2X1_AOTNA\nCAMK2G\tA0A2K5F2X8_AOTNA\nCAMK2G\tA0A2K5F2Z6_AOTNA\nCAMK2G\tA0A2K5F309_AOTNA\nCAMK2G\tA0A2K5F317_AOTNA\nCAMK2G\tA0A2K5F326_AOTNA\nCAMK2G\tA0A2K5K0Q6_COLAP\nCAMK2G\tA0A2K5K0R1_COLAP\nCAMK2G\tA0A2K5K0R2_COLAP\nCAMK2G\tA0A2K5K0R4_COLAP\nCAMK2G\tA0A2K5K0U4_COLAP\nCAMK2G\tA0A2K5NK97_CERAT\nCAMK2G\tA0A2K5NKA5_CERAT\nCAMK2G\tA0A2K5NKH4_CERAT\nCAMK2G\tA0A2K5NKK1_CERAT\nCAMK2G\tA0A2K5NKK3_CERAT\nCAMK2G\tA0A2K5NKK8_CERAT\nCAMK2G\tA0A2K5NKN7_CERAT\nCAMK2G\tA0A2K5NKQ9_CERAT\nCAMK2G\tA0A2K5NKV8_CERAT\nCAMK2G\tA0A2K5NKZ5_CERAT\nCAMK2G\tA0A2K5R9E9_CEBCA\nCAMK2G\tA0A2K5R9G7_CEBCA\nCAMK2G\tA0A2K5UV06_MACFA\nCAMK2G\tA0A2K5UV14_MACFA\nCAMK2G\tA0A2K5UV18_MACFA\nCAMK2G\tA0A2K5UV68_MACFA\nCAMK2G\tA0A2K5UV80_MACFA\nCAMK2G\tA0A2K5UV98_MACFA\nCAMK2G\tA0A2K5UVA0_MACFA\nCAMK2G\tA0A2K5UVA9_MACFA\nCAMK2G\tA0A2K5UVJ4_MACFA\nCAMK2G\tA0A2K5UVL3_MACFA\nCAMK2G\tA0A2K5XY24_MANLE\nCAMK2G\tA0A2K5XY96_MANLE\nCAMK2G\tA0A2K5XYA2_MANLE\nCAMK2G\tA0A2K5XYA8_MANLE\nCAMK2G\tA0A2K5XYD6_MANLE\nCAMK2G\tA0A2K6DKJ5_MACNE\nCAMK2G\tA0A2K6DKL9_MACNE\nCAMK2G\tA0A2K6DKN1_MACNE\nCAMK2G\tA0A2K6DKN8_MACNE\nCAMK2G\tA0A2K6DKP4_MACNE\nCAMK2G\tA0A2K6DKU2_MACNE\nCAMK2G\tA0A2K6DKU9_MACNE\nCAMK2G\tA0A2K6DKV4_MACNE\nCAMK2G\tA0A2K6DKV7_MACNE\nCAMK2G\tA0A2K6DKW7_MACNE\nCAMK2G\tA0A2K6G7W2_PROCO\nCAMK2G\tA0A2K6G7W5_PROCO\nCAMK2G\tA0A2K6G7X0_PROCO\nCAMK2G\tA0A2K6G7X4_PROCO\nCAMK2G\tA0A2K6MRT8_RHIBE\nCAMK2G\tA0A2K6MRT9_RHIBE\nCAMK2G\tA0A2K6MRV3_RHIBE\nCAMK2G\tA0A2K6MRV4_RHIBE\nCAMK2G\tA0A2K6MRW4_RHIBE\nCAMK2G\tA0A2K6RA92_RHIRO\nCAMK2G\tA0A2K6RAJ3_RHIRO\nCAMK2G\tA0A2K6RAJ6_RHIRO\nCAMK2G\tA0A2K6RAK3_RHIRO\nCAMK2G\tA0A2K6RAK5_RHIRO\nCAMK2G\tA0A2K6RAP2_RHIRO\nCAMK2G\tA0A2K6RAP8_RHIRO\nCAMK2G\tA0A2K6RAQ4_RHIRO\nCAMK2G\tA0A2K6TLD7_SAIBB\nCAMK2G\tA0A2K6TLF7_SAIBB\nCAMK2G\tA0A2K6TLG7_SAIBB\nCAMK2G\tA0A2K6TLK9_SAIBB\nCAMK2G\tA0A2Q3DQE3_HUMAN\nCAMK2G\tA0A2R8Z6L8_PANPA\nCAMK2G\tA0A2R8Z6M6_PANPA\nCAMK2G\tA0A2R8Z6M9_PANPA\nCAMK2G\tA0A2R8Z6P3_PANPA\nCAMK2G\tA0A2R8ZCT0_PANPA\nCAMK2G\tA0A2U3WEP4_ODORO\nCAMK2G\tA0A2U3WEP5_ODORO\nCAMK2G\tA0A2U3WEP6_ODORO\nCAMK2G\tA0A2U3WEP9_ODORO\nCAMK2G\tA0A2U3WEQ0_ODORO\nCAMK2G\tA0A2U3WEQ1_ODORO\nCAMK2G\tA0A2U3WEQ4_ODORO\nCAMK2G\tA0A2U3WEQ8_ODORO\nCAMK2G\tA0A2U3WER5_ODORO\nCAMK2G\tA0A2U3XQ41_LEPWE\nCAMK2G\tA0A2U3ZMF6_ODORO\nCAMK2G\tA0A2U4BEF3_TURTR\nCAMK2G\tA0A2U4BEF5_TURTR\nCAMK2G\tA0A2U4BEF7_TURTR\nCAMK2G\tA0A2U4BEF8_TURTR\nCAMK2G\tA0A2U4BEG0_TURTR\nCAMK2G\tA0A2U4BEG2_TURTR\nCAMK2G\tA0A2U4BEG3_TURTR\nCAMK2G\tA0A2U4BEG5_TURTR\nCAMK2G\tA0A2U4BEG7_TURTR\nCAMK2G\tA0A2U4BEH8_TURTR\nCAMK2G\tA0A2U4BEI6_TURTR\nCAMK2G\tA0A2U4BEJ0_TURTR\nCAMK2G\tA0A2U4BEJ5_TURTR\nCAMK2G\tA0A2U4BEK1_TURTR\nCAMK2G\tA0A2U4BES5_TURTR\nCAMK2G\tA0A2U4BET0_TURTR\nCAMK2G\tA0A2U4BET5_TURTR\nCAMK2G\tA0A2U4BEU3_TURTR\nCAMK2G\tA0A2Y9HUT7_NEOSC\nCAMK2G\tA0A2Y9HUU4_NEOSC\nCAMK2G\tA0A2Y9HUU9_NEOSC\nCAMK2G\tA0A2Y9HVD6_NEOSC\nCAMK2G\tA0A2Y9HVE0_NEOSC\nCAMK2G\tA0A2Y9HVE1_NEOSC\nCAMK2G\tA0A2Y9HVE6_NEOSC\nCAMK2G\tA0A2Y9I056_NEOSC\nCAMK2G\tA0A2Y9I061_NEOSC\nCAMK2G\tA0A2Y9I067_NEOSC\nCAMK2G\tA0A2Y9I1Q2_NEOSC\nCAMK2G\tA0A2Y9I1R0_NEOSC\nCAMK2G\tA0A2Y9I1R5_NEOSC\nCAMK2G\tA0A2Y9PI43_DELLE\nCAMK2G\tA0A2Y9PI48_DELLE\nCAMK2G\tA0A2Y9PI53_DELLE\nCAMK2G\tA0A2Y9PI58_DELLE\nCAMK2G\tA0A2Y9PI63_DELLE\nCAMK2G\tA0A2Y9PLF6_DELLE\nCAMK2G\tA0A2Y9PLG2_DELLE\nCAMK2G\tA0A2Y9PMY2_DELLE\nCAMK2G\tA0A2Y9PMY7_DELLE\nCAMK2G\tA0A2Y9PMZ3_DELLE\nCAMK2G\tA0A2Y9PMZ7_DELLE\nCAMK2G\tA0A2Y9PTF3_DELLE\nCAMK2G\tA0A2Y9PTF9_DELLE\nCAMK2G\tA0A2Y9PTG4_DELLE\nCAMK2G\tA0A2Y9PTG9_DELLE\nCAMK2G\tA0A2Y9PUF0_DELLE\nCAMK2G\tA0A2Y9PUF6_DELLE\nCAMK2G\tA0A2Y9PUG4_DELLE\nCAMK2G\tA0A2Y9TC37_PHYMC\nCAMK2G\tA0A2Y9TC49_PHYMC\nCAMK2G\tA0A2Y9TE06_PHYMC\nCAMK2G\tA0A2Y9TE12_PHYMC\nCAMK2G\tA0A2Y9TG39_PHYMC\nCAMK2G\tA0A2Y9TG44_PHYMC\nCAMK2G\tA0A2Y9THQ5_PHYMC\nCAMK2G\tA0A337SHR9_FELCA\nCAMK2G\tA0A337SLY7_FELCA\nCAMK2G\tA0A340Y5E7_LIPVE\nCAMK2G\tA0A341B0X0_NEOAA\nCAMK2G\tA0A341B0X8_NEOAA\nCAMK2G\tA0A341B0Y3_NEOAA\nCAMK2G\tA0A341B102_NEOAA\nCAMK2G\tA0A341B137_NEOAA\nCAMK2G\tA0A341B2U9_NEOAA\nCAMK2G\tA0A341B2V9_NEOAA\nCAMK2G\tA0A341B3X8_NEOAA\nCAMK2G\tA0A384CI94_URSMA\nCAMK2G\tA0A3Q0DBZ6_MESAU\nCAMK2G\tA0A3Q0EFL3_TARSY\nCAMK2G\tA0A3Q0H8R8_ALLSI\nCAMK2G\tA0A3Q0H8R9_ALLSI\nCAMK2G\tA0A3Q0H8T0_ALLSI\nCAMK2G\tA0A3Q0H8T1_ALLSI\nCAMK2G\tA0A3Q0H8U6_ALLSI\nCAMK2G\tA0A3Q0HA95_ALLSI\nCAMK2G\tA0A3Q0HA99_ALLSI\nCAMK2G\tA0A3Q0HDB3_ALLSI\nCAMK2G\tA0A3Q1LVD5_BOVIN\nCAMK2G\tA0A3Q1M066_BOVIN\nCAMK2G\tA0A3Q1M826_BOVIN\nCAMK2G\tA0A3Q1MCE4_BOVIN\nCAMK2G\tA0A3Q1MFS6_BOVIN\nCAMK2G\tA0A3Q1N1Z3_BOVIN\nCAMK2G\tA0A3Q1N6D2_BOVIN\nCAMK2G\tA0A3Q1NKC2_BOVIN\nCAMK2G\tA0A3Q2DGG4_CYPVA\nCAMK2G\tA0A3Q2DH13_CYPVA\nCAMK2G\tA0A3Q2EHF7_CYPVA\nCAMK2G\tA0A3Q2EHT2_CYPVA\nCAMK2G\tA0A3Q2EHX6_CYPVA\nCAMK2G\tA0A3Q2EHY3_CYPVA\nCAMK2G\tA0A3Q2ID69_HORSE\nCAMK2G\tA0A3Q2ILE3_HORSE\nCAMK2G\tA0A3Q2L898_HORSE\nCAMK2G\tA0A3Q2YJG5_HIPCM\nCAMK2G\tA0A3Q7SHL2_VULVU\nCAMK2G\tA0A3Q7SHN3_VULVU\nCAMK2G\tA0A3Q7SUU6_VULVU\nCAMK2G\tA0A3Q7SUV8_VULVU\nCAMK2G\tA0A3Q7SUW7_VULVU\nCAMK2G\tA0A3Q7SUY7_VULVU\nCAMK2G\tA0A3Q7SXG4_VULVU\nCAMK2G\tA0A3Q7SXG8_VULVU\nCAMK2G\tA0A3Q7SXI5_VULVU\nCAMK2G\tA0A3Q7SXJ7_VULVU\nCAMK2G\tA0A3Q7SXQ0_VULVU\nCAMK2G\tA0A3Q7TH25_VULVU\nCAMK2G\tA0A3Q7TH94_VULVU\nCAMK2G\tA0A3Q7TZ73_VULVU\nCAMK2G\tA0A3Q7TZ80_VULVU\nCAMK2G\tA0A3Q7W5R2_URSAR\nCAMK2G\tA0A3Q7WFB4_URSAR\nCAMK2G\tA0A3Q7X0Y3_URSAR\nCAMK2G\tA0A452CNC4_BALAS\nCAMK2G\tA0A452E2N0_CAPHI\nCAMK2G\tA0A452E2S0_CAPHI\nCAMK2G\tA0A452E2S8_CAPHI\nCAMK2G\tA0A452E2U3_CAPHI\nCAMK2G\tA0A452J684_9SAUR\nCAMK2G\tA0A452J685_9SAUR\nCAMK2G\tA0A452J689_9SAUR\nCAMK2G\tA0A452J6B2_9SAUR\nCAMK2G\tA0A452J6B3_9SAUR\nCAMK2G\tA0A452J6C7_9SAUR\nCAMK2G\tA0A452J6N5_9SAUR\nCAMK2G\tA0A452U8E4_URSMA\nCAMK2G\tA0A452U8L1_URSMA\nCAMK2G\tA0A452U8M2_URSMA\nCAMK2G\tA0A452U8N2_URSMA\nCAMK2G\tA0A455AJV6_PHYMC\nCAMK2G\tA0A455AJV9_PHYMC\nCAMK2G\tA0A455AJW2_PHYMC\nCAMK2G\tA0A455AK44_PHYMC\nCAMK2G\tA0A455AK48_PHYMC\nCAMK2G\tA0A455ALM3_PHYMC\nCAMK2G\tA0A455ALM9_PHYMC\nCAMK2G\tA0A455ALN4_PHYMC\nCAMK2G\tA0A455ALR0_PHYMC\nCAMK2G\tA0A455ALR5_PHYMC\nCAMK2G\tA0A455AYD4_PHYMC\nCAMK2G\tA0A455AYD9_PHYMC\nCAMK2G\tA0A4W2BVE3_BOBOX\nCAMK2G\tA0A4W2BVF3_BOBOX\nCAMK2G\tA0A4W2BY21_BOBOX\nCAMK2G\tA0A4W2DRD2_BOBOX\nCAMK2G\tA0A4W2DRD8_BOBOX\nCAMK2G\tA0A4W2FWR4_BOBOX\nCAMK2G\tA0A4W2FXK1_BOBOX\nCAMK2G\tA0A4W2FXW9_BOBOX\nCAMK2G\tA0A4W2HRJ3_BOBOX\nCAMK2G\tA0A4W4EJ95_ELEEL\nCAMK2G\tA0A4W4EJC4_ELEEL\nCAMK2G\tA0A4W4EJC9_ELEEL\nCAMK2G\tA0A4W4EK19_ELEEL\nCAMK2G\tA0A4W4EL24_ELEEL\nCAMK2G\tA0A4W4EL68_ELEEL\nCAMK2G\tA0A4W4EMJ7_ELEEL\nCAMK2G\tA0A4W4EMK7_ELEEL\nCAMK2G\tA0A4X1SXV6_PIG\nCAMK2G\tA0A4X1SYE8_PIG\nCAMK2G\tA0A4X1SYF4_PIG\nCAMK2G\tA0A4X1SZK4_PIG\nCAMK2G\tA0A4X1SZL4_PIG\nCAMK2G\tA0A4X1SZN8_PIG\nCAMK2G\tA0A4X1T007_PIG\nCAMK2G\tA0A4X1T018_PIG\nCAMK2G\tA0A4X1T056_PIG\nCAMK2G\tA0A4X1T088_PIG\nCAMK2G\tA0A4X2L189_VOMUR\nCAMK2G\tA0A4X2L1G7_VOMUR\nCAMK2G\tA0A4X2L6R0_VOMUR\nCAMK2G\tA0A4X2L6X1_VOMUR\nCAMK2G\tA0A4X2L7U8_VOMUR\nCAMK2G\tA0A4X2L9Q1_VOMUR\nCAMK2G\tA0A4X2L9X3_VOMUR\nCAMK2G\tA0A5F4BZ01_CANLF\nCAMK2G\tA0A5F4CV81_CANLF\nCAMK2G\tA0A5F4CVQ6_CANLF\nCAMK2G\tA0A5F4CWQ1_CANLF\nCAMK2G\tA0A5F4D0Z0_CANLF\nCAMK2G\tA0A5F4D153_CANLF\nCAMK2G\tA0A5F7ZUL4_MACMU\nCAMK2G\tA0A5F8G3G1_MONDO\nCAMK2G\tA0A5F8G3R2_MONDO\nCAMK2G\tA0A5F8GFW8_MONDO\nCAMK2G\tA0A5F8GKJ5_MONDO\nCAMK2G\tA0A5F8GUM4_MONDO\nCAMK2G\tA0A5F8H3M4_MONDO\nCAMK2G\tA0A5F8H6A9_MONDO\nCAMK2G\tA0A5F8HHE1_MONDO\nCAMK2G\tA0A5F9C492_RABIT\nCAMK2G\tA0A5F9CDC0_RABIT\nCAMK2G\tA0A5F9DJ64_RABIT\nCAMK2G\tA0A5F9DPL4_RABIT\nCAMK2G\tA0A5F9DQP0_RABIT\nCAMK2G\tA0A5F9DUG6_RABIT\nCAMK2G\tA0A5G2QSW9_PIG\nCAMK2G\tA0A5G2RLI5_PIG\nCAMK2G\tA0A5G3H669_XENTR\nCAMK2G\tA0A5G3H7W4_XENTR\nCAMK2G\tA0A5G3INX8_XENTR\nCAMK2G\tA0A5G3INY5_XENTR\nCAMK2G\tA0A5G3IP04_XENTR\nCAMK2G\tA0A5G3IP29_XENTR\nCAMK2G\tA0A5G3IP96_XENTR\nCAMK2G\tA0A5G3IPA4_XENTR\nCAMK2G\tA0A5G3IPI5_XENTR\nCAMK2G\tA0A5G3IPM9_XENTR\nCAMK2G\tA0A5G3IPU4_XENTR\nCAMK2G\tA0A5G3IPX7_XENTR\nCAMK2G\tA0A5G3IQ39_XENTR\nCAMK2G\tA0A5G3IQA9_XENTR\nCAMK2G\tA0A5G3IQI1_XENTR\nCAMK2G\tA0A5K1UJ76_PIG\nCAMK2G\tA4FV87_BOVIN\nCAMK2G\tB0BM46_XENTR\nCAMK2G\tB3KTH7_HUMAN\nCAMK2G\tE1BZ86_CHICK\nCAMK2G\tE2RG56_CANLF\nCAMK2G\tE2RG57_CANLF\nCAMK2G\tE6ZHR7_DICLA\nCAMK2G\tF1LUE2_RAT\nCAMK2G\tF1M3F8_RAT\nCAMK2G\tF1MG86_BOVIN\nCAMK2G\tF1MVF1_BOVIN\nCAMK2G\tF1P1J7_CHICK\nCAMK2G\tF6PKX6_MACMU\nCAMK2G\tF6QB46_PIG\nCAMK2G\tF6QB57_PIG\nCAMK2G\tF6TEC0_HORSE\nCAMK2G\tF6XMX4_MONDO\nCAMK2G\tF6Z5S9_XENTR\nCAMK2G\tF7AFY3_XENTR\nCAMK2G\tF7DVL1_XENTR\nCAMK2G\tF7DVW8_XENTR\nCAMK2G\tF7F2A9_MACMU\nCAMK2G\tF7FD75_MACMU\nCAMK2G\tF7FD81_MACMU\nCAMK2G\tF7FD86_MACMU\nCAMK2G\tF7FD93_MACMU\nCAMK2G\tF7IGF5_CALJA\nCAMK2G\tG1MEA7_AILME\nCAMK2G\tG1PLB8_MYOLU\nCAMK2G\tG1S4D0_NOMLE\nCAMK2G\tG1U0Y0_RABIT\nCAMK2G\tG3TJA3_LOXAF\nCAMK2G\tG3UHQ6_LOXAF\nCAMK2G\tG3W0K8_SARHA\nCAMK2G\tG3W0K9_SARHA\nCAMK2G\tG3X8K7_MELGA\nCAMK2G\tH0UV10_CAVPO\nCAMK2G\tH0XH29_OTOGA\nCAMK2G\tH0Y6G2_HUMAN\nCAMK2G\tH0Z8X5_TAEGU\nCAMK2G\tH2PXJ9_PONAB\nCAMK2G\tH2Q244_PANTR\nCAMK2G\tH2S5X9_TAKRU\nCAMK2G\tH3AV19_LATCH\nCAMK2G\tH3AV20_LATCH\nCAMK2G\tH9G0A0_MACMU\nCAMK2G\tH9G0A1_MACMU\nCAMK2G\tH9G0A2_MACMU\nCAMK2G\tH9G0A3_MACMU\nCAMK2G\tH9G0A4_MACMU\nCAMK2G\tH9GF19_ANOCA\nCAMK2G\tI2CY34_MACMU\nCAMK2G\tI2CYW5_MACMU\nCAMK2G\tI3MFI8_ICTTR\nCAMK2G\tK7APM5_PANTR\nCAMK2G\tK7ASQ6_PANTR\nCAMK2G\tK7AVH0_PANTR\nCAMK2G\tK7C9T1_PANTR\nCAMK2G\tK7CWX5_PANTR\nCAMK2G\tK7G6L2_PELSI\nCAMK2G\tK9J7U9_XENTR\nCAMK2G\tK9J7V0_XENTR\nCAMK2G\tKCC2G_HUMAN\nCAMK2G\tKCC2G_MOUSE\nCAMK2G\tKCC2G_RAT\nCAMK2G\tM3WP05_FELCA\nCAMK2G\tM3XW94_MUSPF\nCAMK2G\tQ5SWX3_HUMAN\nCAMK2G\tQ6ZWS7_MOUSE\nCAMK2G\tQ801N3_XENLA\nCAMK2G\tQ8BW40_MOUSE\nCAMK2G\tQ8WMW5_MUSPF\nCAMK2G\tQ8WMW6_MUSPF\nCAMK2G\tQ8WU40_HUMAN\nCAMK2G\tQ9DG00_XENLA\nCAMK2G\tQ9DG01_XENLA\nCAMK2G\tU3BKZ6_CALJA\nCAMK2G\tU3C5B6_CALJA\nCAMK2G\tU3CN12_CALJA\nCAMK2G\tU3CZS8_CALJA\nCAMK2G\tU3DKI3_CALJA\nCAMK2G\tU3EF89_CALJA\nCAMK2G\tU3EWU6_CALJA\nCAMK2G\tU3F232_CALJA\nCAMK2G\tU3F234_CALJA\nCAMK2G\tU3F4W3_CALJA\nCAMK2G\tU3FCM3_CALJA\nCAMK2G\tU3FRA3_CALJA\nCAMK2G\tU3FVH9_CALJA\nCAMK2G\tU3IMK8_ANAPP\nCAMK2G\tU3JX42_FICAL\nCAMK2G\tV8P304_OPHHA\nCAMK2G\tW5PET9_SHEEP\nCAMK2G\tW5PEU0_SHEEP\nCAMK2G\tW5ULY0_ICTPU\n','From\tTo\nCAMK2D\tA0A096N221_PAPAN\nCAMK2D\tA0A0G2JGS4_MOUSE\nCAMK2D\tA0A1D5NZG7_CHICK\nCAMK2D\tA0A1D5NZU2_CHICK\nCAMK2D\tA0A1D5PNB0_CHICK\nCAMK2D\tA0A1D5QEN3_MACMU\nCAMK2D\tA0A1D5QWP9_MACMU\nCAMK2D\tA0A1S3AJZ6_ERIEU\nCAMK2D\tA0A1S3AKJ7_ERIEU\nCAMK2D\tA0A1S3AKK6_ERIEU\nCAMK2D\tA0A1S3FRT5_DIPOR\nCAMK2D\tA0A1S3FRT6_DIPOR\nCAMK2D\tA0A1S3FSD2_DIPOR\nCAMK2D\tA0A1S3FTU6_DIPOR\nCAMK2D\tA0A1S3WSW6_ERIEU\nCAMK2D\tA0A1S3WSX3_ERIEU\nCAMK2D\tA0A1S3WSX9_ERIEU\nCAMK2D\tA0A1S3WSZ5_ERIEU\nCAMK2D\tA0A1S3WT51_ERIEU\nCAMK2D\tA0A1S3WTG6_ERIEU\nCAMK2D\tA0A1U7S5S9_ALLSI\nCAMK2D\tA0A1U7SJP5_ALLSI\nCAMK2D\tA0A1U7UE43_TARSY\nCAMK2D\tA0A1U7UK43_TARSY\nCAMK2D\tA0A1U7UP73_TARSY\nCAMK2D\tA0A1U7UXA4_TARSY\nCAMK2D\tA0A1U7UXA9_TARSY\nCAMK2D\tA0A1U7UYQ1_TARSY\nCAMK2D\tA0A1U7UYQ6_TARSY\nCAMK2D\tA0A1V4JU69_PATFA\nCAMK2D\tA0A1V4JUB0_PATFA\nCAMK2D\tA0A1V4JUG3_PATFA\nCAMK2D\tA0A1V4JUM4_PATFA\nCAMK2D\tA0A218UP00_9PASE\nCAMK2D\tA0A250Y069_CASCN\nCAMK2D\tA0A286XB96_CAVPO\nCAMK2D\tA0A286XIS4_CAVPO\nCAMK2D\tA0A286XPQ6_CAVPO\nCAMK2D\tA0A286XZ40_CAVPO\nCAMK2D\tA0A287AHX0_PIG\nCAMK2D\tA0A287CWZ3_ICTTR\nCAMK2D\tA0A287D274_ICTTR\nCAMK2D\tA0A287D6R2_ICTTR\nCAMK2D\tA0A287DB19_ICTTR\nCAMK2D\tA0A2B4RVM3_STYPI\nCAMK2D\tA0A2B4S1U9_STYPI\nCAMK2D\tA0A2B4SGR4_STYPI\nCAMK2D\tA0A2B4SUS7_STYPI\nCAMK2D\tA0A2C9F383_PIG\nCAMK2D\tA0A2I0M8Y2_COLLI\nCAMK2D\tA0A2I0M8Y4_COLLI\nCAMK2D\tA0A2I0M8Y5_COLLI\nCAMK2D\tA0A2I0M8Y6_COLLI\nCAMK2D\tA0A2I0M8Y7_COLLI\nCAMK2D\tA0A2I0M8Y8_COLLI\nCAMK2D\tA0A2I0M8Z0_COLLI\nCAMK2D\tA0A2I0M8Z1_COLLI\nCAMK2D\tA0A2I0M8Z3_COLLI\nCAMK2D\tA0A2I0M8Z4_COLLI\nCAMK2D\tA0A2I0M8Z8_COLLI\nCAMK2D\tA0A2I2UCF5_FELCA\nCAMK2D\tA0A2I3G811_NOMLE\nCAMK2D\tA0A2I3GBI1_NOMLE\nCAMK2D\tA0A2I3GBU5_NOMLE\nCAMK2D\tA0A2I3GJV2_NOMLE\nCAMK2D\tA0A2I3H8G5_NOMLE\nCAMK2D\tA0A2I3HR52_NOMLE\nCAMK2D\tA0A2I3LCR4_PAPAN\nCAMK2D\tA0A2I3LF88_PAPAN\nCAMK2D\tA0A2I3LTB9_PAPAN\nCAMK2D\tA0A2I3M3K4_PAPAN\nCAMK2D\tA0A2I3MA79_PAPAN\nCAMK2D\tA0A2I3N6S1_PAPAN\nCAMK2D\tA0A2I3NFL1_PAPAN\nCAMK2D\tA0A2I3RY65_PANTR\nCAMK2D\tA0A2I3S6C9_PANTR\nCAMK2D\tA0A2I3T870_PANTR\nCAMK2D\tA0A2I3TH19_PANTR\nCAMK2D\tA0A2K5ILY7_COLAP\nCAMK2D\tA0A2K5IMA5_COLAP\nCAMK2D\tA0A2K5IMG1_COLAP\nCAMK2D\tA0A2K5IMH1_COLAP\nCAMK2D\tA0A2K5IMP4_COLAP\nCAMK2D\tA0A2K5MK55_CERAT\nCAMK2D\tA0A2K5MK58_CERAT\nCAMK2D\tA0A2K5MK64_CERAT\nCAMK2D\tA0A2K5MK77_CERAT\nCAMK2D\tA0A2K5MKI0_CERAT\nCAMK2D\tA0A2K5RK61_CEBCA\nCAMK2D\tA0A2K5RK69_CEBCA\nCAMK2D\tA0A2K5RKC9_CEBCA\nCAMK2D\tA0A2K5RKE6_CEBCA\nCAMK2D\tA0A2K5RKJ3_CEBCA\nCAMK2D\tA0A2K5RKP1_CEBCA\nCAMK2D\tA0A2K5RKP2_CEBCA\nCAMK2D\tA0A2K5RKQ7_CEBCA\nCAMK2D\tA0A2K5XAA6_MANLE\nCAMK2D\tA0A2K5XAC7_MANLE\nCAMK2D\tA0A2K5XAC9_MANLE\nCAMK2D\tA0A2K5XAD8_MANLE\nCAMK2D\tA0A2K5XAF6_MANLE\nCAMK2D\tA0A2K6ED09_MACNE\nCAMK2D\tA0A2K6ED10_MACNE\nCAMK2D\tA0A2K6ED14_MACNE\nCAMK2D\tA0A2K6ED15_MACNE\nCAMK2D\tA0A2K6ED16_MACNE\nCAMK2D\tA0A2K6ED17_MACNE\nCAMK2D\tA0A2K6ED19_MACNE\nCAMK2D\tA0A2K6G6C7_PROCO\nCAMK2D\tA0A2K6G6D2_PROCO\nCAMK2D\tA0A2K6G6D6_PROCO\nCAMK2D\tA0A2K6G6F5_PROCO\nCAMK2D\tA0A2K6G6G1_PROCO\nCAMK2D\tA0A2K6N7J4_RHIRO\nCAMK2D\tA0A2K6N7J6_RHIRO\nCAMK2D\tA0A2K6N7K6_RHIRO\nCAMK2D\tA0A2K6N7K7_RHIRO\nCAMK2D\tA0A2K6N7K9_RHIRO\nCAMK2D\tA0A2K6N7L2_RHIRO\nCAMK2D\tA0A2R8M8I7_CALJA\nCAMK2D\tA0A2R8MP59_CALJA\nCAMK2D\tA0A2R8MX52_CALJA\nCAMK2D\tA0A2R8PG23_CALJA\nCAMK2D\tA0A2R9AEX7_PANPA\nCAMK2D\tA0A2R9AEY3_PANPA\nCAMK2D\tA0A2R9AFW6_PANPA\nCAMK2D\tA0A2R9AJS9_PANPA\nCAMK2D\tA0A2R9AJT5_PANPA\nCAMK2D\tA0A2S2NJP5_SCHGA\nCAMK2D\tA0A2U3WE87_ODORO\nCAMK2D\tA0A2U4AR17_TURTR\nCAMK2D\tA0A2U4AR19_TURTR\nCAMK2D\tA0A2U4AR20_TURTR\nCAMK2D\tA0A2U4AR21_TURTR\nCAMK2D\tA0A2U4AR24_TURTR\nCAMK2D\tA0A2U4AR25_TURTR\nCAMK2D\tA0A2U4AR27_TURTR\nCAMK2D\tA0A2U4AR30_TURTR\nCAMK2D\tA0A2U4AR32_TURTR\nCAMK2D\tA0A2U4ARH0_TURTR\nCAMK2D\tA0A2U4ARH5_TURTR\nCAMK2D\tA0A2U4ARI0_TURTR\nCAMK2D\tA0A2Y9GLU9_NEOSC\nCAMK2D\tA0A2Y9GLZ5_NEOSC\nCAMK2D\tA0A2Y9GM54_NEOSC\nCAMK2D\tA0A2Y9GSD3_NEOSC\nCAMK2D\tA0A2Y9PYV0_DELLE\nCAMK2D\tA0A2Y9PYV5_DELLE\nCAMK2D\tA0A2Y9PYV9_DELLE\nCAMK2D\tA0A2Y9PYW4_DELLE\nCAMK2D\tA0A2Y9Q2P3_DELLE\nCAMK2D\tA0A2Y9Q2P8_DELLE\nCAMK2D\tA0A2Y9Q2Q2_DELLE\nCAMK2D\tA0A2Y9Q2Q8_DELLE\nCAMK2D\tA0A2Y9Q397_DELLE\nCAMK2D\tA0A2Y9Q3A4_DELLE\nCAMK2D\tA0A2Y9Q3A6_DELLE\nCAMK2D\tA0A2Y9Q916_DELLE\nCAMK2D\tA0A2Y9Q949_DELLE\nCAMK2D\tA0A2Y9Q954_DELLE\nCAMK2D\tA0A2Y9S8C6_PHYMC\nCAMK2D\tA0A2Y9S8Y1_PHYMC\nCAMK2D\tA0A2Y9SC68_PHYMC\nCAMK2D\tA0A2Y9SF34_PHYMC\nCAMK2D\tA0A2Y9SGN6_PHYMC\nCAMK2D\tA0A337S4F5_FELCA\nCAMK2D\tA0A337SF67_FELCA\nCAMK2D\tA0A337SMM9_FELCA\nCAMK2D\tA0A340WZR3_LIPVE\nCAMK2D\tA0A341BF31_NEOAA\nCAMK2D\tA0A384AUP7_BALAS\nCAMK2D\tA0A384AUR0_BALAS\nCAMK2D\tA0A384AV07_BALAS\nCAMK2D\tA0A384AVG3_BALAS\nCAMK2D\tA0A384BYK3_URSMA\nCAMK2D\tA0A384BYK8_URSMA\nCAMK2D\tA0A384BYL0_URSMA\nCAMK2D\tA0A384BYL4_URSMA\nCAMK2D\tA0A384BZR3_URSMA\nCAMK2D\tA0A384BZT8_URSMA\nCAMK2D\tA0A3Q0DBL8_MESAU\nCAMK2D\tA0A3Q0DCK8_TARSY\nCAMK2D\tA0A3Q0DHA8_TARSY\nCAMK2D\tA0A3Q0DJ13_TARSY\nCAMK2D\tA0A3Q0HAN8_ALLSI\nCAMK2D\tA0A3Q0HAP0_ALLSI\nCAMK2D\tA0A3Q0HAP2_ALLSI\nCAMK2D\tA0A3Q0HAP6_ALLSI\nCAMK2D\tA0A3Q0HAP8_ALLSI\nCAMK2D\tA0A3Q0HC07_ALLSI\nCAMK2D\tA0A3Q0HC12_ALLSI\nCAMK2D\tA0A3Q0HF39_ALLSI\nCAMK2D\tA0A3Q0HF42_ALLSI\nCAMK2D\tA0A3Q0HF45_ALLSI\nCAMK2D\tA0A3Q0HF47_ALLSI\nCAMK2D\tA0A3Q1LT34_BOVIN\nCAMK2D\tA0A3Q1LX06_BOVIN\nCAMK2D\tA0A3Q1M647_BOVIN\nCAMK2D\tA0A3Q1M9V1_BOVIN\nCAMK2D\tA0A3Q2HH77_HORSE\nCAMK2D\tA0A3Q2HNC4_HORSE\nCAMK2D\tA0A3Q2HNM8_HORSE\nCAMK2D\tA0A3Q2HPA6_HORSE\nCAMK2D\tA0A3Q2HVL1_HORSE\nCAMK2D\tA0A3Q2I0K4_HORSE\nCAMK2D\tA0A3Q2U2C3_CHICK\nCAMK2D\tA0A3Q2UFJ3_CHICK\nCAMK2D\tA0A3Q2UMD3_CHICK\nCAMK2D\tA0A3Q2UPU8_CHICK\nCAMK2D\tA0A3Q7P7L2_CALUR\nCAMK2D\tA0A3Q7P7L7_CALUR\nCAMK2D\tA0A3Q7P7X4_CALUR\nCAMK2D\tA0A3Q7P7X7_CALUR\nCAMK2D\tA0A3Q7P8F8_CALUR\nCAMK2D\tA0A3Q7P8G3_CALUR\nCAMK2D\tA0A3Q7PIJ5_CALUR\nCAMK2D\tA0A3Q7PIK0_CALUR\nCAMK2D\tA0A3Q7QNI4_CALUR\nCAMK2D\tA0A3Q7QNI9_CALUR\nCAMK2D\tA0A3Q7SYG1_VULVU\nCAMK2D\tA0A3Q7SYG5_VULVU\nCAMK2D\tA0A3Q7SYG9_VULVU\nCAMK2D\tA0A3Q7THS9_VULVU\nCAMK2D\tA0A3Q7THT4_VULVU\nCAMK2D\tA0A3Q7THT9_VULVU\nCAMK2D\tA0A3Q7TVM3_VULVU\nCAMK2D\tA0A3Q7TVM6_VULVU\nCAMK2D\tA0A3Q7TVN1_VULVU\nCAMK2D\tA0A3Q7U7K2_VULVU\nCAMK2D\tA0A3Q7UPX9_VULVU\nCAMK2D\tA0A3Q7UPY4_VULVU\nCAMK2D\tA0A3Q7UPZ0_VULVU\nCAMK2D\tA0A3Q7V112_URSAR\nCAMK2D\tA0A3Q7V117_URSAR\nCAMK2D\tA0A3Q7V121_URSAR\nCAMK2D\tA0A3Q7VRX7_URSAR\nCAMK2D\tA0A3Q7VRY0_URSAR\nCAMK2D\tA0A3Q7W1A8_URSAR\nCAMK2D\tA0A3Q7W1B4_URSAR\nCAMK2D\tA0A3Q7WLG5_URSAR\nCAMK2D\tA0A3Q7WX66_URSAR\nCAMK2D\tA0A3Q7WX71_URSAR\nCAMK2D\tA0A3R7G1L8_CLOSI\nCAMK2D\tA0A3S5ZPT6_BOVIN\nCAMK2D\tA0A452CNB1_BALAS\nCAMK2D\tA0A452CNB3_BALAS\nCAMK2D\tA0A452CNB6_BALAS\nCAMK2D\tA0A452CNB7_BALAS\nCAMK2D\tA0A452CNB9_BALAS\nCAMK2D\tA0A452CNC6_BALAS\nCAMK2D\tA0A452CNH1_BALAS\nCAMK2D\tA0A452CNH8_BALAS\nCAMK2D\tA0A452E1P3_CAPHI\nCAMK2D\tA0A452E1R1_CAPHI\nCAMK2D\tA0A452E1R3_CAPHI\nCAMK2D\tA0A452E1T2_CAPHI\nCAMK2D\tA0A452E1Z6_CAPHI\nCAMK2D\tA0A452E231_CAPHI\nCAMK2D\tA0A452HMS4_9SAUR\nCAMK2D\tA0A452HMV8_9SAUR\nCAMK2D\tA0A452HPG6_9SAUR\nCAMK2D\tA0A452HPH7_9SAUR\nCAMK2D\tA0A452HPH9_9SAUR\nCAMK2D\tA0A452HPK4_9SAUR\nCAMK2D\tA0A452HPL3_9SAUR\nCAMK2D\tA0A452HPL8_9SAUR\nCAMK2D\tA0A452T7Y0_URSMA\nCAMK2D\tA0A455BGC3_PHYMC\nCAMK2D\tA0A455BGC8_PHYMC\nCAMK2D\tA0A455BGD3_PHYMC\nCAMK2D\tA0A455BGD7_PHYMC\nCAMK2D\tA0A455BGE4_PHYMC\nCAMK2D\tA0A455BGR7_PHYMC\nCAMK2D\tA0A455BGS2_PHYMC\nCAMK2D\tA0A455BGS7_PHYMC\nCAMK2D\tA0A455BGT2_PHYMC\nCAMK2D\tA0A455BGT7_PHYMC\nCAMK2D\tA0A455BGU2_PHYMC\nCAMK2D\tA0A455BGZ0_PHYMC\nCAMK2D\tA0A455BGZ5_PHYMC\nCAMK2D\tA0A455BH00_PHYMC\nCAMK2D\tA0A455BH04_PHYMC\nCAMK2D\tA0A455BH07_PHYMC\nCAMK2D\tA0A455BH12_PHYMC\nCAMK2D\tA0A455BJE4_PHYMC\nCAMK2D\tA0A455BJE9_PHYMC\nCAMK2D\tA0A455BJF4_PHYMC\nCAMK2D\tA0A455BJG0_PHYMC\nCAMK2D\tA0A455BJG5_PHYMC\nCAMK2D\tA0A455BW20_PHYMC\nCAMK2D\tA0A455BW25_PHYMC\nCAMK2D\tA0A455BW31_PHYMC\nCAMK2D\tA0A4W2DHU3_BOBOX\nCAMK2D\tA0A4W2DNR6_BOBOX\nCAMK2D\tA0A4W2F2M9_BOBOX\nCAMK2D\tA0A4W2F3R9_BOBOX\nCAMK2D\tA0A4W2F3Y8_BOBOX\nCAMK2D\tA0A4W2H4M4_BOBOX\nCAMK2D\tA0A4W2IJB8_BOBOX\nCAMK2D\tA0A4X2K6Z1_VOMUR\nCAMK2D\tA0A4X2KEC4_VOMUR\nCAMK2D\tA0A4X2KED7_VOMUR\nCAMK2D\tA0A4Z2GPQ5_9TELE\nCAMK2D\tA0A5B7ED49_PORTR\nCAMK2D\tA0A5F4CFS1_CANLF\nCAMK2D\tA0A5F4CJ79_CANLF\nCAMK2D\tA0A5F4CXE1_CANLF\nCAMK2D\tA0A5F4VZL2_CALJA\nCAMK2D\tA0A5F4W0R4_CALJA\nCAMK2D\tA0A5F4WAP1_CALJA\nCAMK2D\tA0A5F5PFI2_HORSE\nCAMK2D\tA0A5F5XU46_FELCA\nCAMK2D\tA0A5F5Y385_FELCA\nCAMK2D\tA0A5F9C3G4_RABIT\nCAMK2D\tA0A5F9C7N6_RABIT\nCAMK2D\tA0A5F9CFN3_RABIT\nCAMK2D\tA0A5F9D2W2_RABIT\nCAMK2D\tA0A5G2QQ20_PIG\nCAMK2D\tA0A5G2QS23_PIG\nCAMK2D\tA0A5G2QUE4_PIG\nCAMK2D\tA0A5G2R491_PIG\nCAMK2D\tA0A5G2R4J6_PIG\nCAMK2D\tA0A5G2RCS0_PIG\nCAMK2D\tA0A5G3I0T1_XENTR\nCAMK2D\tA5D9F0_BOVIN\nCAMK2D\tA9ULG8_XENTR\nCAMK2D\tB7ZPP0_XENLA\nCAMK2D\tD6R938_HUMAN\nCAMK2D\tD6RDQ8_MOUSE\nCAMK2D\tE2RR74_CANLF\nCAMK2D\tE2RR97_CANLF\nCAMK2D\tE9PBG7_HUMAN\nCAMK2D\tE9PF82_HUMAN\nCAMK2D\tE9PXV3_MOUSE\nCAMK2D\tE9Q1T1_MOUSE\nCAMK2D\tE9Q1V9_MOUSE\nCAMK2D\tE9Q1W0_MOUSE\nCAMK2D\tE9QAJ4_MOUSE\nCAMK2D\tF6PTM3_HORSE\nCAMK2D\tF6RWZ9_MOUSE\nCAMK2D\tF6UGY3_MACMU\nCAMK2D\tF6UHD3_MACMU\nCAMK2D\tF6UQZ6_XENTR\nCAMK2D\tF6X843_HORSE\nCAMK2D\tF6XS23_HORSE\nCAMK2D\tF6YPZ9_XENTR\nCAMK2D\tF7H159_CALJA\nCAMK2D\tF7H163_CALJA\nCAMK2D\tF7H4F4_MACMU\nCAMK2D\tF7H4G9_MACMU\nCAMK2D\tF7H4H1_MACMU\nCAMK2D\tF7H4H3_MACMU\nCAMK2D\tF7H6E2_CALJA\nCAMK2D\tF7H6F9_CALJA\nCAMK2D\tG1LRM2_AILME\nCAMK2D\tG1LRN5_AILME\nCAMK2D\tG1S2H9_NOMLE\nCAMK2D\tG1TVE5_RABIT\nCAMK2D\tG3SUT8_LOXAF\nCAMK2D\tG3UKK5_LOXAF\nCAMK2D\tH0V2G6_CAVPO\nCAMK2D\tH0X285_OTOGA\nCAMK2D\tH0Y9C2_HUMAN\nCAMK2D\tH0Y9J2_HUMAN\nCAMK2D\tH0YUC5_TAEGU\nCAMK2D\tH2PE58_PONAB\nCAMK2D\tH2QQ24_PANTR\nCAMK2D\tH2T2G1_TAKRU\nCAMK2D\tH2T2G3_TAKRU\nCAMK2D\tH9FC92_MACMU\nCAMK2D\tH9G098_MACMU\nCAMK2D\tH9ZE08_MACMU\nCAMK2D\tH9ZE10_MACMU\nCAMK2D\tI0FVE6_MACMU\nCAMK2D\tI0FVE7_MACMU\nCAMK2D\tI3LZ28_ICTTR\nCAMK2D\tI3NDL9_ICTTR\nCAMK2D\tK7BKT6_PANTR\nCAMK2D\tK7DC81_PANTR\nCAMK2D\tKC2D2_DANRE\nCAMK2D\tKCC2B_MOUSE\nCAMK2D\tKCC2D_BOVIN\nCAMK2D\tKCC2D_CHICK\nCAMK2D\tKCC2D_HUMAN\nCAMK2D\tKCC2D_MOUSE\nCAMK2D\tKCC2D_PIG\nCAMK2D\tKCC2D_RABIT\nCAMK2D\tKCC2D_RAT\nCAMK2D\tKCC2D_XENLA\nCAMK2D\tL7MZX9_ANOCA\nCAMK2D\tM3WNF0_FELCA\nCAMK2D\tO70459_MOUSE\nCAMK2D\tQ3V1X9_MOUSE\nCAMK2D\tQ4G1A8_HUMAN\nCAMK2D\tQ8C4I3_MOUSE\nCAMK2D\tQ8C8X9_MOUSE\nCAMK2D\tQ8CCM0_MOUSE\nCAMK2D\tU3D4V3_CALJA\nCAMK2D\tU3E7H3_CALJA\nCAMK2D\tU3EKS9_CALJA\nCAMK2D\tV8NMW0_OPHHA\nCAMK2D\tV8PA25_OPHHA\nCAMK2D\tW5QAM4_SHEEP\nCAMK2D\tW5QAM5_SHEEP\n','From\tTo\nCAMK2B\tA0A087Y8D1_POEFO\nCAMK2B\tA0A096M6G4_POEFO\nCAMK2B\tA0A096MY92_PAPAN\nCAMK2B\tA0A096NY28_PAPAN\nCAMK2B\tA0A0D9RSU7_CHLSB\nCAMK2B\tA0A0P6JUM5_HETGA\nCAMK2B\tA0A1A7XN39_9TELE\nCAMK2B\tA0A1A8A6I7_NOTFU\nCAMK2B\tA0A1A8CVL9_9TELE\nCAMK2B\tA0A1A8F2S2_9TELE\nCAMK2B\tA0A1A8HM65_9TELE\nCAMK2B\tA0A1A8HTC1_NOTKU\nCAMK2B\tA0A1A8NT94_9TELE\nCAMK2B\tA0A1A8Q6W4_9TELE\nCAMK2B\tA0A1A8SHA6_9TELE\nCAMK2B\tA0A1A8TZX3_NOTFU\nCAMK2B\tA0A1D5NYA6_CHICK\nCAMK2B\tA0A1D5PDX2_CHICK\nCAMK2B\tA0A1D5PXX2_CHICK\nCAMK2B\tA0A1D5R5T9_MACMU\nCAMK2B\tA0A1L8H1N2_XENLA\nCAMK2B\tA0A1L8H1R7_XENLA\nCAMK2B\tA0A1S2ZTT8_ERIEU\nCAMK2B\tA0A1S2ZTV0_ERIEU\nCAMK2B\tA0A1S2ZTV2_ERIEU\nCAMK2B\tA0A1S2ZTV6_ERIEU\nCAMK2B\tA0A1S2ZTV8_ERIEU\nCAMK2B\tA0A1S2ZTW4_ERIEU\nCAMK2B\tA0A1S3GJI3_DIPOR\nCAMK2B\tA0A1S3WCT8_ERIEU\nCAMK2B\tA0A1S3WD32_ERIEU\nCAMK2B\tA0A1U7R3L0_MESAU\nCAMK2B\tA0A1U7RE25_MESAU\nCAMK2B\tA0A1U7T5V7_TARSY\nCAMK2B\tA0A1U8CZB4_MESAU\nCAMK2B\tA0A218UCK4_9PASE\nCAMK2B\tA0A250Y2J5_CASCN\nCAMK2B\tA0A286ZL03_PIG\nCAMK2B\tA0A286ZTV0_PIG\nCAMK2B\tA0A287AGQ9_PIG\nCAMK2B\tA0A287BEM2_PIG\nCAMK2B\tA0A287CSC6_ICTTR\nCAMK2B\tA0A287CVR9_ICTTR\nCAMK2B\tA0A287D0T9_ICTTR\nCAMK2B\tA0A287D2A9_ICTTR\nCAMK2B\tA0A287D5Y9_ICTTR\nCAMK2B\tA0A287D9K7_ICTTR\nCAMK2B\tA0A2D0SQA9_ICTPU\nCAMK2B\tA0A2D0SQB6_ICTPU\nCAMK2B\tA0A2D0SQB7_ICTPU\nCAMK2B\tA0A2D0SQB8_ICTPU\nCAMK2B\tA0A2D0SQC0_ICTPU\nCAMK2B\tA0A2D0SQC1_ICTPU\nCAMK2B\tA0A2D0SQC5_ICTPU\nCAMK2B\tA0A2D0SQC6_ICTPU\nCAMK2B\tA0A2D0SQC9_ICTPU\nCAMK2B\tA0A2D0SQD1_ICTPU\nCAMK2B\tA0A2D0SQD4_ICTPU\nCAMK2B\tA0A2D0SQD6_ICTPU\nCAMK2B\tA0A2D0SQD9_ICTPU\nCAMK2B\tA0A2D0SQE1_ICTPU\nCAMK2B\tA0A2D0SQW6_ICTPU\nCAMK2B\tA0A2D0SQX1_ICTPU\nCAMK2B\tA0A2D0SQX7_ICTPU\nCAMK2B\tA0A2D0SQY2_ICTPU\nCAMK2B\tA0A2D0SQY6_ICTPU\nCAMK2B\tA0A2D0SRH6_ICTPU\nCAMK2B\tA0A2D0SRI3_ICTPU\nCAMK2B\tA0A2D0SRJ1_ICTPU\nCAMK2B\tA0A2I0LJN9_COLLI\nCAMK2B\tA0A2I2U876_FELCA\nCAMK2B\tA0A2I3FUP6_NOMLE\nCAMK2B\tA0A2I3G244_NOMLE\nCAMK2B\tA0A2I3GAK4_NOMLE\nCAMK2B\tA0A2I3H909_NOMLE\nCAMK2B\tA0A2I3HHI0_NOMLE\nCAMK2B\tA0A2I3HQB6_NOMLE\nCAMK2B\tA0A2I3HZZ1_NOMLE\nCAMK2B\tA0A2I3LCS5_PAPAN\nCAMK2B\tA0A2I3LUV8_PAPAN\nCAMK2B\tA0A2I3MQQ4_PAPAN\nCAMK2B\tA0A2I3MTV6_PAPAN\nCAMK2B\tA0A2I3MYF7_PAPAN\nCAMK2B\tA0A2I3NEF7_PAPAN\nCAMK2B\tA0A2I3NFR6_PAPAN\nCAMK2B\tA0A2I3NG95_PAPAN\nCAMK2B\tA0A2I3RCD9_PANTR\nCAMK2B\tA0A2I3RN94_PANTR\nCAMK2B\tA0A2I3RQS4_PANTR\nCAMK2B\tA0A2I3S680_PANTR\nCAMK2B\tA0A2I3S8G4_PANTR\nCAMK2B\tA0A2I3SHE1_PANTR\nCAMK2B\tA0A2I3TB62_PANTR\nCAMK2B\tA0A2I3TDQ6_PANTR\nCAMK2B\tA0A2J8L9G6_PANTR\nCAMK2B\tA0A2J8L9I0_PANTR\nCAMK2B\tA0A2K5DY70_AOTNA\nCAMK2B\tA0A2K5DYB7_AOTNA\nCAMK2B\tA0A2K5DYH9_AOTNA\nCAMK2B\tA0A2K5DYQ3_AOTNA\nCAMK2B\tA0A2K5DYR2_AOTNA\nCAMK2B\tA0A2K5DYT8_AOTNA\nCAMK2B\tA0A2K5DYU4_AOTNA\nCAMK2B\tA0A2K5DYV2_AOTNA\nCAMK2B\tA0A2K5DZ21_AOTNA\nCAMK2B\tA0A2K5DZ36_AOTNA\nCAMK2B\tA0A2K5ILW5_COLAP\nCAMK2B\tA0A2K5IMC0_COLAP\nCAMK2B\tA0A2K5IMD0_COLAP\nCAMK2B\tA0A2K5IMF9_COLAP\nCAMK2B\tA0A2K5IML0_COLAP\nCAMK2B\tA0A2K5IMN2_COLAP\nCAMK2B\tA0A2K5IMT5_COLAP\nCAMK2B\tA0A2K5IMV9_COLAP\nCAMK2B\tA0A2K5M5B7_CERAT\nCAMK2B\tA0A2K5M5C4_CERAT\nCAMK2B\tA0A2K5M5C7_CERAT\nCAMK2B\tA0A2K5M5D1_CERAT\nCAMK2B\tA0A2K5M5I4_CERAT\nCAMK2B\tA0A2K5M5K6_CERAT\nCAMK2B\tA0A2K5M5K7_CERAT\nCAMK2B\tA0A2K5M5L7_CERAT\nCAMK2B\tA0A2K5M5M1_CERAT\nCAMK2B\tA0A2K5R771_CEBCA\nCAMK2B\tA0A2K5R785_CEBCA\nCAMK2B\tA0A2K5R789_CEBCA\nCAMK2B\tA0A2K5R792_CEBCA\nCAMK2B\tA0A2K5R7I7_CEBCA\nCAMK2B\tA0A2K5R7J5_CEBCA\nCAMK2B\tA0A2K5R7K2_CEBCA\nCAMK2B\tA0A2K5R7K4_CEBCA\nCAMK2B\tA0A2K5R7K6_CEBCA\nCAMK2B\tA0A2K5R7L6_CEBCA\nCAMK2B\tA0A2K5WC51_MACFA\nCAMK2B\tA0A2K5WC79_MACFA\nCAMK2B\tA0A2K5WCA3_MACFA\nCAMK2B\tA0A2K5WCB6_MACFA\nCAMK2B\tA0A2K5WCC2_MACFA\nCAMK2B\tA0A2K5WCC8_MACFA\nCAMK2B\tA0A2K5WCD1_MACFA\nCAMK2B\tA0A2K5WCE7_MACFA\nCAMK2B\tA0A2K5WCJ1_MACFA\nCAMK2B\tA0A2K5WCJ6_MACFA\nCAMK2B\tA0A2K5XXN9_MANLE\nCAMK2B\tA0A2K5XXR4_MANLE\nCAMK2B\tA0A2K5XXS6_MANLE\nCAMK2B\tA0A2K5XXS9_MANLE\nCAMK2B\tA0A2K5XXT1_MANLE\nCAMK2B\tA0A2K5XXT5_MANLE\nCAMK2B\tA0A2K5XXU7_MANLE\nCAMK2B\tA0A2K5XXX5_MANLE\nCAMK2B\tA0A2K6D7W8_MACNE\nCAMK2B\tA0A2K6D7X5_MACNE\nCAMK2B\tA0A2K6D7Y6_MACNE\nCAMK2B\tA0A2K6D7Z3_MACNE\nCAMK2B\tA0A2K6D858_MACNE\nCAMK2B\tA0A2K6D881_MACNE\nCAMK2B\tA0A2K6D882_MACNE\nCAMK2B\tA0A2K6D895_MACNE\nCAMK2B\tA0A2K6D8A5_MACNE\nCAMK2B\tA0A2K6D8A8_MACNE\nCAMK2B\tA0A2K6F094_PROCO\nCAMK2B\tA0A2K6F0A4_PROCO\nCAMK2B\tA0A2K6F0A6_PROCO\nCAMK2B\tA0A2K6F0A8_PROCO\nCAMK2B\tA0A2K6F0B3_PROCO\nCAMK2B\tA0A2K6F0C6_PROCO\nCAMK2B\tA0A2K6F0C9_PROCO\nCAMK2B\tA0A2K6F0D3_PROCO\nCAMK2B\tA0A2K6F0D6_PROCO\nCAMK2B\tA0A2K6F0E4_PROCO\nCAMK2B\tA0A2K6R1V2_RHIRO\nCAMK2B\tA0A2K6R1X9_RHIRO\nCAMK2B\tA0A2K6R1Y4_RHIRO\nCAMK2B\tA0A2K6R1Y5_RHIRO\nCAMK2B\tA0A2K6R1Y8_RHIRO\nCAMK2B\tA0A2K6R1Z9_RHIRO\nCAMK2B\tA0A2K6R211_RHIRO\nCAMK2B\tA0A2K6R221_RHIRO\nCAMK2B\tA0A2K6R224_RHIRO\nCAMK2B\tA0A2K6UD84_SAIBB\nCAMK2B\tA0A2K6UD96_SAIBB\nCAMK2B\tA0A2K6UDC7_SAIBB\nCAMK2B\tA0A2K6UDE0_SAIBB\nCAMK2B\tA0A2K6UDE1_SAIBB\nCAMK2B\tA0A2K6UDH4_SAIBB\nCAMK2B\tA0A2K6UDH7_SAIBB\nCAMK2B\tA0A2K6UDI1_SAIBB\nCAMK2B\tA0A2K6UDI9_SAIBB\nCAMK2B\tA0A2R8MLB2_CALJA\nCAMK2B\tA0A2R8MZ87_CALJA\nCAMK2B\tA0A2R9ARJ0_PANPA\nCAMK2B\tA0A2R9ARN0_PANPA\nCAMK2B\tA0A2R9ARP7_PANPA\nCAMK2B\tA0A2R9AW05_PANPA\nCAMK2B\tA0A2R9AW15_PANPA\nCAMK2B\tA0A2R9AY32_PANPA\nCAMK2B\tA0A2R9AY62_PANPA\nCAMK2B\tA0A2R9AY83_PANPA\nCAMK2B\tA0A2U3WWZ9_ODORO\nCAMK2B\tA0A2U3WX03_ODORO\nCAMK2B\tA0A2U3WX04_ODORO\nCAMK2B\tA0A2U3WX23_ODORO\nCAMK2B\tA0A2U3WX25_ODORO\nCAMK2B\tA0A2U3WX29_ODORO\nCAMK2B\tA0A2U3WXA2_ODORO\nCAMK2B\tA0A2U3WXA7_ODORO\nCAMK2B\tA0A2Y9I6A8_NEOSC\nCAMK2B\tA0A2Y9PPJ9_DELLE\nCAMK2B\tA0A2Y9PTB7_DELLE\nCAMK2B\tA0A2Y9Q072_DELLE\nCAMK2B\tA0A337RWQ0_FELCA\nCAMK2B\tA0A337S374_FELCA\nCAMK2B\tA0A337S5U1_FELCA\nCAMK2B\tA0A337S6N5_FELCA\nCAMK2B\tA0A337S8U0_FELCA\nCAMK2B\tA0A337SA59_FELCA\nCAMK2B\tA0A337SW29_FELCA\nCAMK2B\tA0A340WBZ0_LIPVE\nCAMK2B\tA0A340WBZ2_LIPVE\nCAMK2B\tA0A340WEL0_LIPVE\nCAMK2B\tA0A340WEL5_LIPVE\nCAMK2B\tA0A340WHR4_LIPVE\nCAMK2B\tA0A340WIX2_LIPVE\nCAMK2B\tA0A340WJ77_LIPVE\nCAMK2B\tA0A340WJ85_LIPVE\nCAMK2B\tA0A341BMC7_NEOAA\nCAMK2B\tA0A384D4V4_URSMA\nCAMK2B\tA0A3B4XS66_SERLL\nCAMK2B\tA0A3B4XSF5_SERLL\nCAMK2B\tA0A3B4XTC9_SERLL\nCAMK2B\tA0A3B4XTJ9_SERLL\nCAMK2B\tA0A3B4XTK8_SERLL\nCAMK2B\tA0A3B4Y6C8_SERLL\nCAMK2B\tA0A3B4Y6L6_SERLL\nCAMK2B\tA0A3B4Y6N1_SERLL\nCAMK2B\tA0A3B5PW75_XIPMA\nCAMK2B\tA0A3B5PWA9_XIPMA\nCAMK2B\tA0A3B5Q363_XIPMA\nCAMK2B\tA0A3B5QAV5_XIPMA\nCAMK2B\tA0A3B5QKV6_XIPMA\nCAMK2B\tA0A3B5QQP8_XIPMA\nCAMK2B\tA0A3B5QXL9_XIPMA\nCAMK2B\tA0A3B5R7Q0_XIPMA\nCAMK2B\tA0A3B5REY3_XIPMA\nCAMK2B\tA0A3Q0CAA2_MESAU\nCAMK2B\tA0A3Q0FIP1_ALLSI\nCAMK2B\tA0A3Q0FIP5_ALLSI\nCAMK2B\tA0A3Q0FIP9_ALLSI\nCAMK2B\tA0A3Q0FIQ3_ALLSI\nCAMK2B\tA0A3Q0FJK8_ALLSI\nCAMK2B\tA0A3Q0FJL3_ALLSI\nCAMK2B\tA0A3Q0FJL8_ALLSI\nCAMK2B\tA0A3Q0FJR4_ALLSI\nCAMK2B\tA0A3Q0FJR9_ALLSI\nCAMK2B\tA0A3Q0FJS4_ALLSI\nCAMK2B\tA0A3Q0FJS9_ALLSI\nCAMK2B\tA0A3Q0FMP8_ALLSI\nCAMK2B\tA0A3Q0FMQ1_ALLSI\nCAMK2B\tA0A3Q0FMQ6_ALLSI\nCAMK2B\tA0A3Q0FMR7_ALLSI\nCAMK2B\tA0A3Q0FP75_ALLSI\nCAMK2B\tA0A3Q0FP80_ALLSI\nCAMK2B\tA0A3Q0FP86_ALLSI\nCAMK2B\tA0A3Q0FP92_ALLSI\nCAMK2B\tA0A3Q0FP96_ALLSI\nCAMK2B\tA0A3Q1HLD4_ANATE\nCAMK2B\tA0A3Q1HTI9_ANATE\nCAMK2B\tA0A3Q1M9U4_BOVIN\nCAMK2B\tA0A3Q1MNX4_BOVIN\nCAMK2B\tA0A3Q2GT91_HORSE\nCAMK2B\tA0A3Q2H701_HORSE\nCAMK2B\tA0A3Q2H7R0_HORSE\nCAMK2B\tA0A3Q2H9N5_HORSE\nCAMK2B\tA0A3Q2I7U5_HORSE\nCAMK2B\tA0A3Q2IF46_HORSE\nCAMK2B\tA0A3Q2KU94_HORSE\nCAMK2B\tA0A3Q2TYU0_CHICK\nCAMK2B\tA0A3Q2ZHC1_KRYMA\nCAMK2B\tA0A3Q2ZJK3_KRYMA\nCAMK2B\tA0A3Q2ZXA9_KRYMA\nCAMK2B\tA0A3Q3EMD9_KRYMA\nCAMK2B\tA0A3Q3KW95_9TELE\nCAMK2B\tA0A3Q3L826_9TELE\nCAMK2B\tA0A3Q3RNP4_9TELE\nCAMK2B\tA0A3Q7MP30_CALUR\nCAMK2B\tA0A3Q7RER4_VULVU\nCAMK2B\tA0A3Q7VZH5_URSAR\nCAMK2B\tA0A3Q7VZJ3_URSAR\nCAMK2B\tA0A3Q7WVC9_URSAR\nCAMK2B\tA0A3Q7X6E0_URSAR\nCAMK2B\tA0A3Q7X6F1_URSAR\nCAMK2B\tA0A3Q7X6G8_URSAR\nCAMK2B\tA0A452CKZ3_BALAS\nCAMK2B\tA0A452CL07_BALAS\nCAMK2B\tA0A452CL22_BALAS\nCAMK2B\tA0A452CL34_BALAS\nCAMK2B\tA0A452DIJ8_BOVIN\nCAMK2B\tA0A452FRZ7_CAPHI\nCAMK2B\tA0A452FS04_CAPHI\nCAMK2B\tA0A452FS38_CAPHI\nCAMK2B\tA0A452FS41_CAPHI\nCAMK2B\tA0A452FS53_CAPHI\nCAMK2B\tA0A452FS64_CAPHI\nCAMK2B\tA0A452FSC5_CAPHI\nCAMK2B\tA0A452FSG5_CAPHI\nCAMK2B\tA0A452FSP3_CAPHI\nCAMK2B\tA0A452HTH3_9SAUR\nCAMK2B\tA0A452HTH5_9SAUR\nCAMK2B\tA0A452HTH8_9SAUR\nCAMK2B\tA0A452HTI3_9SAUR\nCAMK2B\tA0A452RL21_URSAM\nCAMK2B\tA0A452RL50_URSAM\nCAMK2B\tA0A452RL92_URSAM\nCAMK2B\tA0A452RLA3_URSAM\nCAMK2B\tA0A452RLL0_URSAM\nCAMK2B\tA0A452RLU5_URSAM\nCAMK2B\tA0A452RLU7_URSAM\nCAMK2B\tA0A452RLW0_URSAM\nCAMK2B\tA0A452RLW3_URSAM\nCAMK2B\tA0A452RLW4_URSAM\nCAMK2B\tA0A455BDM7_PHYMC\nCAMK2B\tA0A4W2CQ94_BOBOX\nCAMK2B\tA0A4W2D9B0_BOBOX\nCAMK2B\tA0A4W2D9E1_BOBOX\nCAMK2B\tA0A4W2EGB8_BOBOX\nCAMK2B\tA0A4W2EI15_BOBOX\nCAMK2B\tA0A4W2GA24_BOBOX\nCAMK2B\tA0A4W2GDV7_BOBOX\nCAMK2B\tA0A4W2HVX9_BOBOX\nCAMK2B\tA0A4W2I3S2_BOBOX\nCAMK2B\tA0A4W2I4W4_BOBOX\nCAMK2B\tA0A4W6ETY9_LATCA\nCAMK2B\tA0A4W6EWK2_LATCA\nCAMK2B\tA0A4W6EXD0_LATCA\nCAMK2B\tA0A4W6EXD5_LATCA\nCAMK2B\tA0A4W6EXF4_LATCA\nCAMK2B\tA0A4W6EXG4_LATCA\nCAMK2B\tA0A4W6EXG9_LATCA\nCAMK2B\tA0A4W6EYB2_LATCA\nCAMK2B\tA0A4W6EZ85_LATCA\nCAMK2B\tA0A4W6EZC2_LATCA\nCAMK2B\tA0A4X1UCD2_PIG\nCAMK2B\tA0A4X1UCJ5_PIG\nCAMK2B\tA0A4X1UCM7_PIG\nCAMK2B\tA0A4X1UCX4_PIG\nCAMK2B\tA0A4X1UEX0_PIG\nCAMK2B\tA0A4X1UF88_PIG\nCAMK2B\tA0A4X1UFB4_PIG\nCAMK2B\tA0A4X1UFG0_PIG\nCAMK2B\tA0A4X1UH17_PIG\nCAMK2B\tA0A4X1UHW2_PIG\nCAMK2B\tA0A4X2JZM7_VOMUR\nCAMK2B\tA0A4X2K236_VOMUR\nCAMK2B\tA0A4X2K255_VOMUR\nCAMK2B\tA0A4Z2FWZ2_9TELE\nCAMK2B\tA0A5F5PV60_HORSE\nCAMK2B\tA0A5F8A8X6_MACMU\nCAMK2B\tA0A5F8GQP0_MONDO\nCAMK2B\tA0A5F8GS34_MONDO\nCAMK2B\tA0A5F8H0C2_MONDO\nCAMK2B\tA0A5F8HK67_MONDO\nCAMK2B\tA0A5G2QCQ9_PIG\nCAMK2B\tA0A5G2QK95_PIG\nCAMK2B\tA0A5G2QQK7_PIG\nCAMK2B\tA0A5G2QVX7_PIG\nCAMK2B\tA0A5G2R2W2_PIG\nCAMK2B\tA4D2J9_HUMAN\nCAMK2B\tA4IIZ8_XENTR\nCAMK2B\tD3DVK8_HUMAN\nCAMK2B\tE7EQE4_HUMAN\nCAMK2B\tE7ERS6_HUMAN\nCAMK2B\tE7ETC9_HUMAN\nCAMK2B\tE9PBE8_HUMAN\nCAMK2B\tF1LNI8_RAT\nCAMK2B\tF1LUE2_RAT\nCAMK2B\tF1SSH3_PIG\nCAMK2B\tF6PJT4_MACMU\nCAMK2B\tF6PJW4_MACMU\nCAMK2B\tF6PJY7_MACMU\nCAMK2B\tF6PKV0_MACMU\nCAMK2B\tF6PKZ8_MACMU\nCAMK2B\tF6QEM9_HORSE\nCAMK2B\tF6YPS0_XENTR\nCAMK2B\tF6ZKX5_CALJA\nCAMK2B\tF6ZRQ1_XENTR\nCAMK2B\tF7ACX9_XENTR\nCAMK2B\tF7CW72_CALJA\nCAMK2B\tF7DG01_CALJA\nCAMK2B\tF7EZ83_CALJA\nCAMK2B\tF7G8T0_MONDO\nCAMK2B\tF7H015_CALJA\nCAMK2B\tF7IAF3_CALJA\nCAMK2B\tF7IMR0_CALJA\nCAMK2B\tG1MH24_AILME\nCAMK2B\tG1QU55_NOMLE\nCAMK2B\tG1QU64_NOMLE\nCAMK2B\tG3TA76_LOXAF\nCAMK2B\tG3V9G3_RAT\nCAMK2B\tG3WQP9_SARHA\nCAMK2B\tG3WQQ0_SARHA\nCAMK2B\tH0W1F5_CAVPO\nCAMK2B\tH0XMT7_OTOGA\nCAMK2B\tH2PM82_PONAB\nCAMK2B\tH2UVZ0_TAKRU\nCAMK2B\tH2UVZ4_TAKRU\nCAMK2B\tH2VEF9_TAKRU\nCAMK2B\tH2VEG0_TAKRU\nCAMK2B\tH2VEG1_TAKRU\nCAMK2B\tH2VEG2_TAKRU\nCAMK2B\tH3AVR3_LATCH\nCAMK2B\tH7BXS4_HUMAN\nCAMK2B\tH7BZC6_HUMAN\nCAMK2B\tH7C394_HUMAN\nCAMK2B\tH9FC88_MACMU\nCAMK2B\tH9FC90_MACMU\nCAMK2B\tH9FM52_MACMU\nCAMK2B\tH9FM53_MACMU\nCAMK2B\tH9GIS3_ANOCA\nCAMK2B\tI2CT74_MACMU\nCAMK2B\tI2CYW4_MACMU\nCAMK2B\tI3K0Y5_ORENI\nCAMK2B\tI3K0Y6_ORENI\nCAMK2B\tI3LWN4_ICTTR\nCAMK2B\tKCC2B_BOVIN\nCAMK2B\tKCC2B_HUMAN\nCAMK2B\tKCC2B_MOUSE\nCAMK2B\tKCC2B_RAT\nCAMK2B\tM3WNR2_FELCA\nCAMK2B\tM3Z3C5_MUSPF\nCAMK2B\tM3ZKE9_XIPMA\nCAMK2B\tQ0V9X2_XENTR\nCAMK2B\tQ3TY93_MOUSE\nCAMK2B\tQ5SVI0_MOUSE\nCAMK2B\tQ5SVI1_MOUSE\nCAMK2B\tQ5SVI2_MOUSE\nCAMK2B\tQ5SVI3_MOUSE\nCAMK2B\tQ5SVI9_MOUSE\nCAMK2B\tQ5SVJ0_MOUSE\nCAMK2B\tQ5SVJ1_MOUSE\nCAMK2B\tQ63094_RAT\nCAMK2B\tQ68EG2_MOUSE\nCAMK2B\tQ6AX64_XENLA\nCAMK2B\tQ75KE9_HUMAN\nCAMK2B\tQ75LA8_HUMAN\nCAMK2B\tQ8BL41_MOUSE\nCAMK2B\tQ8K4W4_MOUSE\nCAMK2B\tQ8K4W5_MOUSE\nCAMK2B\tQ91549_XENLA\nCAMK2B\tQ9WVJ1_MOUSE\nCAMK2B\tU3CC53_CALJA\nCAMK2B\tU3CUG5_CALJA\nCAMK2B\tU3CYR4_CALJA\nCAMK2B\tU3JBA6_FICAL\nCAMK2B\tV8PCK0_OPHHA\nCAMK2B\tV8PCW5_OPHHA\nCAMK2B\tW5Q2E7_SHEEP\nCAMK2B\tW5UQH7_ICTPU\n','From\tTo\nCAMK2A\tA0A096N2J8_PAPAN\nCAMK2A\tA0A0A9YGW4_LYGHE\nCAMK2A\tA0A0D9RGP3_CHLSB\nCAMK2A\tA0A0G2JUV8_RAT\nCAMK2A\tA0A1A7WXT6_9TELE\nCAMK2A\tA0A1A8AY40_NOTFU\nCAMK2A\tA0A1A8DTV7_9TELE\nCAMK2A\tA0A1A8G7E5_9TELE\nCAMK2A\tA0A1A8GQ04_9TELE\nCAMK2A\tA0A1A8IV64_NOTKU\nCAMK2A\tA0A1A8KLU4_NOTKU\nCAMK2A\tA0A1A8RH33_9TELE\nCAMK2A\tA0A1Q9DEP5_SYMMI\nCAMK2A\tA0A1S2ZAN5_ERIEU\nCAMK2A\tA0A1S2ZAP0_ERIEU\nCAMK2A\tA0A1S3FWM4_DIPOR\nCAMK2A\tA0A1S3FY50_DIPOR\nCAMK2A\tA0A1U7RYB9_ALLSI\nCAMK2A\tA0A1U7TS41_TARSY\nCAMK2A\tA0A1U7U1Q7_TARSY\nCAMK2A\tA0A1V4KUI1_PATFA\nCAMK2A\tA0A1V4KUU3_PATFA\nCAMK2A\tA0A218V0K2_9PASE\nCAMK2A\tA0A250XW24_CASCN\nCAMK2A\tA0A286XFT7_CAVPO\nCAMK2A\tA0A287BAW1_PIG\nCAMK2A\tA0A287DDG1_ICTTR\nCAMK2A\tA0A2I0M9X4_COLLI\nCAMK2A\tA0A2I0M9X6_COLLI\nCAMK2A\tA0A2I0M9Y3_COLLI\nCAMK2A\tA0A2I2UEW6_FELCA\nCAMK2A\tA0A2I2V4I2_FELCA\nCAMK2A\tA0A2I2Y6I6_GORGO\nCAMK2A\tA0A2I2YC97_GORGO\nCAMK2A\tA0A2I3HH49_NOMLE\nCAMK2A\tA0A2I3LMM9_PAPAN\nCAMK2A\tA0A2I3MCB0_PAPAN\nCAMK2A\tA0A2I3MV14_PAPAN\nCAMK2A\tA0A2I3RUN1_PANTR\nCAMK2A\tA0A2I3SNX6_PANTR\nCAMK2A\tA0A2I3STB7_PANTR\nCAMK2A\tA0A2K5CH86_AOTNA\nCAMK2A\tA0A2K5CH97_AOTNA\nCAMK2A\tA0A2K5CHD7_AOTNA\nCAMK2A\tA0A2K5CHL8_AOTNA\nCAMK2A\tA0A2K5IBR7_COLAP\nCAMK2A\tA0A2K5IBS8_COLAP\nCAMK2A\tA0A2K5IBY8_COLAP\nCAMK2A\tA0A2K5ICC5_COLAP\nCAMK2A\tA0A2K5MDN6_CERAT\nCAMK2A\tA0A2K5MDP1_CERAT\nCAMK2A\tA0A2K5MDR3_CERAT\nCAMK2A\tA0A2K5MDT1_CERAT\nCAMK2A\tA0A2K5R6Y7_CEBCA\nCAMK2A\tA0A2K5R6Z2_CEBCA\nCAMK2A\tA0A2K5VDG6_MACFA\nCAMK2A\tA0A2K5VDI3_MACFA\nCAMK2A\tA0A2K5VDJ3_MACFA\nCAMK2A\tA0A2K5VDJ9_MACFA\nCAMK2A\tA0A2K5VDN1_MACFA\nCAMK2A\tA0A2K5ZWB6_MANLE\nCAMK2A\tA0A2K5ZWD5_MANLE\nCAMK2A\tA0A2K5ZWE4_MANLE\nCAMK2A\tA0A2K5ZWE5_MANLE\nCAMK2A\tA0A2K6E5G5_MACNE\nCAMK2A\tA0A2K6E5H1_MACNE\nCAMK2A\tA0A2K6E5H9_MACNE\nCAMK2A\tA0A2K6E5I1_MACNE\nCAMK2A\tA0A2K6MEX2_RHIBE\nCAMK2A\tA0A2K6MEY7_RHIBE\nCAMK2A\tA0A2K6MEZ9_RHIBE\nCAMK2A\tA0A2K6MF02_RHIBE\nCAMK2A\tA0A2K6PNM1_RHIRO\nCAMK2A\tA0A2K6PNT9_RHIRO\nCAMK2A\tA0A2K6PNU4_RHIRO\nCAMK2A\tA0A2K6PNU5_RHIRO\nCAMK2A\tA0A2K6PNV3_RHIRO\nCAMK2A\tA0A2K6TEC6_SAIBB\nCAMK2A\tA0A2K6TEF2_SAIBB\nCAMK2A\tA0A2K6TEH0_SAIBB\nCAMK2A\tA0A2K6TEH2_SAIBB\nCAMK2A\tA0A2R8MGS7_CALJA\nCAMK2A\tA0A2R8MMQ6_CALJA\nCAMK2A\tA0A2R9CAT1_PANPA\nCAMK2A\tA0A2R9CAY6_PANPA\nCAMK2A\tA0A2R9CLM7_PANPA\nCAMK2A\tA0A2U3W497_ODORO\nCAMK2A\tA0A2U3W4A5_ODORO\nCAMK2A\tA0A2U3Z4L3_LEPWE\nCAMK2A\tA0A2U4AI09_TURTR\nCAMK2A\tA0A2Y9EVL3_PHYMC\nCAMK2A\tA0A2Y9HLY1_NEOSC\nCAMK2A\tA0A2Y9PFY4_DELLE\nCAMK2A\tA0A2Y9PR48_DELLE\nCAMK2A\tA0A340WHE3_LIPVE\nCAMK2A\tA0A340WM67_LIPVE\nCAMK2A\tA0A341ADM0_NEOAA\nCAMK2A\tA0A384AMY1_BALAS\nCAMK2A\tA0A384AND8_BALAS\nCAMK2A\tA0A384C7A3_URSMA\nCAMK2A\tA0A384C8D3_URSMA\nCAMK2A\tA0A3B3IEC7_ORYLA\nCAMK2A\tA0A3B3IFJ4_ORYLA\nCAMK2A\tA0A3B3ZCL6_9GOBI\nCAMK2A\tA0A3B5A2E4_9TELE\nCAMK2A\tA0A3Q0CXR3_MESAU\nCAMK2A\tA0A3Q0E287_TARSY\nCAMK2A\tA0A3Q1LJZ2_BOVIN\nCAMK2A\tA0A3Q1LUJ3_BOVIN\nCAMK2A\tA0A3Q1MB42_BOVIN\nCAMK2A\tA0A3Q1MKC7_BOVIN\nCAMK2A\tA0A3Q2H2A7_HORSE\nCAMK2A\tA0A3Q7NYQ4_CALUR\nCAMK2A\tA0A3Q7SPL4_VULVU\nCAMK2A\tA0A3Q7T543_VULVU\nCAMK2A\tA0A3Q7WXG0_URSAR\nCAMK2A\tA0A3Q7XTV1_URSAR\nCAMK2A\tA0A452G0S5_CAPHI\nCAMK2A\tA0A452G1N3_CAPHI\nCAMK2A\tA0A452G1X0_CAPHI\nCAMK2A\tA0A4W2DF77_BOBOX\nCAMK2A\tA0A4W2GFU9_BOBOX\nCAMK2A\tA0A4W3HBJ7_CALMI\nCAMK2A\tA0A4W3HBL5_CALMI\nCAMK2A\tA0A4W3HG70_CALMI\nCAMK2A\tA0A4W3HGM0_CALMI\nCAMK2A\tA0A4W3HGN1_CALMI\nCAMK2A\tA0A4W3HZ85_CALMI\nCAMK2A\tA0A4W3HZ91_CALMI\nCAMK2A\tA0A4W3HZ98_CALMI\nCAMK2A\tA0A4W3HZE2_CALMI\nCAMK2A\tA0A4X1T7G0_PIG\nCAMK2A\tA0A4X1TAV3_PIG\nCAMK2A\tA0A4X1TAZ9_PIG\nCAMK2A\tA0A4X2KKF4_VOMUR\nCAMK2A\tA0A4X2KKG1_VOMUR\nCAMK2A\tA0A4Z2E922_9TELE\nCAMK2A\tA0A4Z2F8Q9_9TELE\nCAMK2A\tA0A5F4CXR8_CANLF\nCAMK2A\tA0A5F5XGU0_FELCA\nCAMK2A\tA0A5F5XSX8_FELCA\nCAMK2A\tA0A5F7ZRA5_MACMU\nCAMK2A\tA0A5F7ZTL2_MACMU\nCAMK2A\tA0A5F8G5P4_MONDO\nCAMK2A\tA0A5F9CGM6_RABIT\nCAMK2A\tA0A5F9ZH21_HUMAN\nCAMK2A\tA0A5F9ZH50_HUMAN\nCAMK2A\tA0A5F9ZH90_HUMAN\nCAMK2A\tA0A5F9ZHM9_HUMAN\nCAMK2A\tA0A5F9ZHR5_HUMAN\nCAMK2A\tA0A5F9ZHY2_HUMAN\nCAMK2A\tA0A5G3H7C2_XENTR\nCAMK2A\tA0A5G3IJN7_XENTR\nCAMK2A\tA0A5G3IJY4_XENTR\nCAMK2A\tA0A5G3IK94_XENTR\nCAMK2A\tA0A5G3IKS7_XENTR\nCAMK2A\tA0A5G3ILN0_XENTR\nCAMK2A\tA0A5G3ILT8_XENTR\nCAMK2A\tA0A5K1UA08_CALJA\nCAMK2A\tA4IIE5_XENTR\nCAMK2A\tA8E4M2_BOVIN\nCAMK2A\tA8K161_HUMAN\nCAMK2A\tD3Z7K9_MOUSE\nCAMK2A\tD6RFJ0_HUMAN\nCAMK2A\tE7F2G0_DANRE\nCAMK2A\tF1LZG4_RAT\nCAMK2A\tF1NGS8_CHICK\nCAMK2A\tF1P011_CHICK\nCAMK2A\tF1PB67_CANLF\nCAMK2A\tF1RL74_PIG\nCAMK2A\tF6QSC1_MONDO\nCAMK2A\tF6RZZ5_MONDO\nCAMK2A\tF6SJ48_XENTR\nCAMK2A\tF6SJ67_XENTR\nCAMK2A\tF6TUX4_MACMU\nCAMK2A\tF6TVB2_MACMU\nCAMK2A\tF6WHR9_MOUSE\nCAMK2A\tF6YQ17_XENTR\nCAMK2A\tF6Z478_XENTR\nCAMK2A\tF7AK14_ORNAN\nCAMK2A\tF7BNV0_XENTR\nCAMK2A\tF7CER2_HORSE\nCAMK2A\tF7CJA9_HORSE\nCAMK2A\tF7DHR7_XENTR\nCAMK2A\tF7E869_XENTR\nCAMK2A\tF7H024_CALJA\nCAMK2A\tF8WHB5_MOUSE\nCAMK2A\tF8WIS9_MOUSE\nCAMK2A\tG1LDX1_AILME\nCAMK2A\tG1N4P5_MELGA\nCAMK2A\tG1PMD4_MYOLU\nCAMK2A\tG1RH50_NOMLE\nCAMK2A\tG1TEL6_RABIT\nCAMK2A\tG3QD05_GORGO\nCAMK2A\tG3URH2_MELGA\nCAMK2A\tG3URZ9_MELGA\nCAMK2A\tG3WYI8_SARHA\nCAMK2A\tH0V888_CAVPO\nCAMK2A\tH0WQA2_OTOGA\nCAMK2A\tH0YRM5_TAEGU\nCAMK2A\tH2MC30_ORYLA\nCAMK2A\tH2MC31_ORYLA\nCAMK2A\tH2PH29_PONAB\nCAMK2A\tH2QRT0_PANTR\nCAMK2A\tH3B5X6_LATCH\nCAMK2A\tH9EMI1_MACMU\nCAMK2A\tH9EMI2_MACMU\nCAMK2A\tI3IT67_DANRE\nCAMK2A\tI3LNG5_PIG\nCAMK2A\tI3MCL2_ICTTR\nCAMK2A\tK7GAH2_PELSI\nCAMK2A\tKCC2A_HUMAN\nCAMK2A\tKCC2A_MOUSE\nCAMK2A\tKCC2A_PONAB\nCAMK2A\tKCC2A_RAT\nCAMK2A\tL7MZY1_ANOCA\nCAMK2A\tL8ECG5_HUMAN\nCAMK2A\tM3W0P7_FELCA\nCAMK2A\tM3WEF9_FELCA\nCAMK2A\tM3YQY2_MUSPF\nCAMK2A\tQ08E45_BOVIN\nCAMK2A\tQ27I69_HUMAN\nCAMK2A\tQ32PV2_DANRE\nCAMK2A\tQ561S2_DANRE\nCAMK2A\tQ6GLR3_XENLA\nCAMK2A\tQ6S5C4_FELCA\nCAMK2A\tQ7LDD5_HUMAN\nCAMK2A\tQ80TN1_MOUSE\nCAMK2A\tQ8IWE0_HUMAN\nCAMK2A\tQ91549_XENLA\nCAMK2A\tQ924T0_RAT\nCAMK2A\tT2M388_HYDVU\nCAMK2A\tU3E1F8_CALJA\nCAMK2A\tU3F557_CALJA\nCAMK2A\tU3FCU4_CALJA\nCAMK2A\tU3FU39_CALJA\nCAMK2A\tU3JZX5_FICAL\nCAMK2A\tV8PCD9_OPHHA\nCAMK2A\tV9KL89_CALMI\nCAMK2A\tW5MZ72_LEPOC\nCAMK2A\tW5MZ82_LEPOC\nCAMK2A\tW5PAI9_SHEEP\nCAMK2A\tW5UIP1_ICTPU\n','From\tTo\nCASK\tA0A067XG53_MOUSE\nCASK\tA0A096N1G2_PAPAN\nCASK\tA0A0A9WSH7_LYGHE\nCASK\tA0A0B4KH39_DROME\nCASK\tA0A0B6XJY0_HUMAN\nCASK\tA0A0C9R1A7_9HYME\nCASK\tA0A0C9RIN5_9HYME\nCASK\tA0A0G2JVI6_RAT\nCASK\tA0A0G2K2L8_RAT\nCASK\tA0A0G2K8F4_RAT\nCASK\tA0A0P6J5K4_HETGA\nCASK\tA0A0V0U1T6_9BILA\nCASK\tA0A0V0XXV3_TRIPS\nCASK\tA0A0V0XY29_TRIPS\nCASK\tA0A0V0XY92_TRIPS\nCASK\tA0A0V1C2A8_TRISP\nCASK\tA0A0V1C2D7_TRISP\nCASK\tA0A0V1C2M5_TRISP\nCASK\tA0A0V1C2R3_TRISP\nCASK\tA0A0V1DG99_TRIBR\nCASK\tA0A0V1DGH3_TRIBR\nCASK\tA0A0V1ECC9_TRIPS\nCASK\tA0A0V1FP37_TRIPS\nCASK\tA0A0V1HFX3_9BILA\nCASK\tA0A0V1NZV5_9BILA\nCASK\tA0A151NQI7_ALLMI\nCASK\tA0A151NQQ3_ALLMI\nCASK\tA0A151NRK9_ALLMI\nCASK\tA0A1A7WI85_9TELE\nCASK\tA0A1A7X9E6_9TELE\nCASK\tA0A1A7Y6M5_9TELE\nCASK\tA0A1A7Y781_9TELE\nCASK\tA0A1A7Y8J9_9TELE\nCASK\tA0A1A7YTN4_9TELE\nCASK\tA0A1A7ZKP4_NOTFU\nCASK\tA0A1A7ZU22_NOTFU\nCASK\tA0A1A7ZU26_NOTFU\nCASK\tA0A1A8CFJ0_9TELE\nCASK\tA0A1A8DCX4_9TELE\nCASK\tA0A1A8DFA8_9TELE\nCASK\tA0A1A8DHT2_9TELE\nCASK\tA0A1A8EVU2_9TELE\nCASK\tA0A1A8EWZ5_9TELE\nCASK\tA0A1A8F108_9TELE\nCASK\tA0A1A8FRX7_9TELE\nCASK\tA0A1A8G9S7_9TELE\nCASK\tA0A1A8GNU1_9TELE\nCASK\tA0A1A8H979_9TELE\nCASK\tA0A1A8I6A5_NOTKU\nCASK\tA0A1A8J0C7_NOTKU\nCASK\tA0A1A8J0E1_NOTKU\nCASK\tA0A1A8J866_NOTKU\nCASK\tA0A1A8J8Z3_NOTKU\nCASK\tA0A1A8JRE0_NOTKU\nCASK\tA0A1A8K2U7_NOTKU\nCASK\tA0A1A8KHD8_NOTKU\nCASK\tA0A1A8LHE4_9TELE\nCASK\tA0A1A8LYV4_9TELE\nCASK\tA0A1A8MMF5_9TELE\nCASK\tA0A1A8N1K9_9TELE\nCASK\tA0A1A8P4F9_9TELE\nCASK\tA0A1A8QQ23_9TELE\nCASK\tA0A1A8R792_9TELE\nCASK\tA0A1A8RJ76_9TELE\nCASK\tA0A1A8SIB1_9TELE\nCASK\tA0A1A8SJY0_9TELE\nCASK\tA0A1A8U5Y7_NOTFU\nCASK\tA0A1A8VC26_NOTFU\nCASK\tA0A1D5NXM9_CHICK\nCASK\tA0A1D5P7L1_CHICK\nCASK\tA0A1D5PB73_CHICK\nCASK\tA0A1D5PHD7_CHICK\nCASK\tA0A1D5QVF6_MACMU\nCASK\tA0A1D5R5P2_MACMU\nCASK\tA0A1J4JF54_9EUKA\nCASK\tA0A1L8HF63_XENLA\nCASK\tA0A1Q9DCH3_SYMMI\nCASK\tA0A1S2ZLD0_ERIEU\nCASK\tA0A1S2ZLD5_ERIEU\nCASK\tA0A1S2ZLD7_ERIEU\nCASK\tA0A1S2ZLD9_ERIEU\nCASK\tA0A1S2ZLE4_ERIEU\nCASK\tA0A1S2ZLE8_ERIEU\nCASK\tA0A1S3GBU9_DIPOR\nCASK\tA0A1S3GBV8_DIPOR\nCASK\tA0A1S3GDJ7_DIPOR\nCASK\tA0A1S3GE79_DIPOR\nCASK\tA0A1S3NX24_SALSA\nCASK\tA0A1S3NX28_SALSA\nCASK\tA0A1S3NX30_SALSA\nCASK\tA0A1S3NX33_SALSA\nCASK\tA0A1S3NX35_SALSA\nCASK\tA0A1S3NX39_SALSA\nCASK\tA0A1S3NX44_SALSA\nCASK\tA0A1S3NX46_SALSA\nCASK\tA0A1S3NX49_SALSA\nCASK\tA0A1S3NX54_SALSA\nCASK\tA0A1S3NXF5_SALSA\nCASK\tA0A1S3NXG0_SALSA\nCASK\tA0A1S3NXG5_SALSA\nCASK\tA0A1S3NXH0_SALSA\nCASK\tA0A1S3NXH1_SALSA\nCASK\tA0A1S3NXH6_SALSA\nCASK\tA0A1S3NXI1_SALSA\nCASK\tA0A1S3NXI6_SALSA\nCASK\tA0A1U8DEB0_ALLSI\nCASK\tA0A1V4JBP2_PATFA\nCASK\tA0A1V4JBW2_PATFA\nCASK\tA0A1V4JCE0_PATFA\nCASK\tA0A250Y5Q8_CASCN\nCASK\tA0A286XPR4_CAVPO\nCASK\tA0A286XQH7_CAVPO\nCASK\tA0A286XQM5_CAVPO\nCASK\tA0A286Y563_CAVPO\nCASK\tA0A286ZNV9_PIG\nCASK\tA0A287CZG9_ICTTR\nCASK\tA0A2B4RFT3_STYPI\nCASK\tA0A2D0R342_ICTPU\nCASK\tA0A2D0R346_ICTPU\nCASK\tA0A2D0R347_ICTPU\nCASK\tA0A2D0R379_ICTPU\nCASK\tA0A2D0R383_ICTPU\nCASK\tA0A2D0R4P8_ICTPU\nCASK\tA0A2D0R5D0_ICTPU\nCASK\tA0A2D0R5D5_ICTPU\nCASK\tA0A2D0R5E0_ICTPU\nCASK\tA0A2I0M5L6_COLLI\nCASK\tA0A2I0M5L7_COLLI\nCASK\tA0A2I0M5L8_COLLI\nCASK\tA0A2I0M5L9_COLLI\nCASK\tA0A2I0M5M4_COLLI\nCASK\tA0A2I0M5M6_COLLI\nCASK\tA0A2I2U6Z5_FELCA\nCASK\tA0A2I2UB95_FELCA\nCASK\tA0A2I2UBK1_FELCA\nCASK\tA0A2I2US52_FELCA\nCASK\tA0A2I3GJI0_NOMLE\nCASK\tA0A2I3H3Z4_NOMLE\nCASK\tA0A2I3HNN9_NOMLE\nCASK\tA0A2I3LFE5_PAPAN\nCASK\tA0A2I3LLU6_PAPAN\nCASK\tA0A2I3LXC9_PAPAN\nCASK\tA0A2I3M1E2_PAPAN\nCASK\tA0A2I3MJ73_PAPAN\nCASK\tA0A2I3N7H0_PAPAN\nCASK\tA0A2I3TCK4_PANTR\nCASK\tA0A2I3TJD4_PANTR\nCASK\tA0A2I3TMJ6_PANTR\nCASK\tA0A2I3TNN5_PANTR\nCASK\tA0A2I4BNJ3_9TELE\nCASK\tA0A2K5CGC6_AOTNA\nCASK\tA0A2K5CGD1_AOTNA\nCASK\tA0A2K5CGD7_AOTNA\nCASK\tA0A2K5CGF4_AOTNA\nCASK\tA0A2K5CGJ7_AOTNA\nCASK\tA0A2K5CGK4_AOTNA\nCASK\tA0A2K5CGL7_AOTNA\nCASK\tA0A2K5CGM5_AOTNA\nCASK\tA0A2K5J111_COLAP\nCASK\tA0A2K5J117_COLAP\nCASK\tA0A2K5J1A1_COLAP\nCASK\tA0A2K5J1I0_COLAP\nCASK\tA0A2K5J1I2_COLAP\nCASK\tA0A2K5J1P4_COLAP\nCASK\tA0A2K5J1R5_COLAP\nCASK\tA0A2K5P0T0_CERAT\nCASK\tA0A2K5P0V7_CERAT\nCASK\tA0A2K5P0W5_CERAT\nCASK\tA0A2K5P0Y8_CERAT\nCASK\tA0A2K5P101_CERAT\nCASK\tA0A2K5P128_CERAT\nCASK\tA0A2K5P146_CERAT\nCASK\tA0A2K5RQ88_CEBCA\nCASK\tA0A2K5RQ91_CEBCA\nCASK\tA0A2K5RQA0_CEBCA\nCASK\tA0A2K5VCX7_MACFA\nCASK\tA0A2K5VD39_MACFA\nCASK\tA0A2K5VD88_MACFA\nCASK\tA0A2K5VD89_MACFA\nCASK\tA0A2K5VD99_MACFA\nCASK\tA0A2K5VDD3_MACFA\nCASK\tA0A2K5YGA4_MANLE\nCASK\tA0A2K5YGB8_MANLE\nCASK\tA0A2K5YGD2_MANLE\nCASK\tA0A2K5YGK2_MANLE\nCASK\tA0A2K5YGN5_MANLE\nCASK\tA0A2K5YGN7_MANLE\nCASK\tA0A2K5YGR9_MANLE\nCASK\tA0A2K6D560_MACNE\nCASK\tA0A2K6D592_MACNE\nCASK\tA0A2K6D5A9_MACNE\nCASK\tA0A2K6D5B7_MACNE\nCASK\tA0A2K6D5D2_MACNE\nCASK\tA0A2K6D5F0_MACNE\nCASK\tA0A2K6D5H0_MACNE\nCASK\tA0A2K6GIK5_PROCO\nCASK\tA0A2K6GIL6_PROCO\nCASK\tA0A2K6GIL8_PROCO\nCASK\tA0A2K6GIM4_PROCO\nCASK\tA0A2K6GIM7_PROCO\nCASK\tA0A2K6JM39_RHIBE\nCASK\tA0A2K6JM49_RHIBE\nCASK\tA0A2K6QAP8_RHIRO\nCASK\tA0A2K6QAQ4_RHIRO\nCASK\tA0A2K6QAQ9_RHIRO\nCASK\tA0A2K6QAR2_RHIRO\nCASK\tA0A2K6QAR4_RHIRO\nCASK\tA0A2K6QAR7_RHIRO\nCASK\tA0A2K6QAS6_RHIRO\nCASK\tA0A2K6TAK4_SAIBB\nCASK\tA0A2K6TAN4_SAIBB\nCASK\tA0A2K6TAN6_SAIBB\nCASK\tA0A2K6TAS7_SAIBB\nCASK\tA0A2K6TAT0_SAIBB\nCASK\tA0A2K6TAU7_SAIBB\nCASK\tA0A2K6TAX6_SAIBB\nCASK\tA0A2P8Y5H2_BLAGE\nCASK\tA0A2R8MLT5_CALJA\nCASK\tA0A2R8MUH7_CALJA\nCASK\tA0A2R8P5U8_CALJA\nCASK\tA0A2R8Y3B3_HUMAN\nCASK\tA0A2R8Y472_HUMAN\nCASK\tA0A2R8Y4K4_HUMAN\nCASK\tA0A2R8Y6D8_HUMAN\nCASK\tA0A2R8Y6F8_HUMAN\nCASK\tA0A2R8Y7X6_HUMAN\nCASK\tA0A2R8YE77_HUMAN\nCASK\tA0A2R8YEJ7_HUMAN\nCASK\tA0A2R8YEK3_HUMAN\nCASK\tA0A2R8YFN5_HUMAN\nCASK\tA0A2R8YGH2_HUMAN\nCASK\tA0A2R9BYQ4_PANPA\nCASK\tA0A2R9BYR1_PANPA\nCASK\tA0A2R9BZ46_PANPA\nCASK\tA0A2R9C1W5_PANPA\nCASK\tA0A2R9C8L1_PANPA\nCASK\tA0A2R9C8M8_PANPA\nCASK\tA0A2R9C8N3_PANPA\nCASK\tA0A2S2Q3F2_9HEMI\nCASK\tA0A2U3TZM1_HUMAN\nCASK\tA0A2U3TZM4_HUMAN\nCASK\tA0A2U3TZN6_HUMAN\nCASK\tA0A2U3ZF03_ODORO\nCASK\tA0A2U4C105_TURTR\nCASK\tA0A2U4C110_TURTR\nCASK\tA0A2U4C112_TURTR\nCASK\tA0A2U4C115_TURTR\nCASK\tA0A2U4C117_TURTR\nCASK\tA0A2U4C122_TURTR\nCASK\tA0A2U4C136_TURTR\nCASK\tA0A2U4C140_TURTR\nCASK\tA0A2U4C158_TURTR\nCASK\tA0A2U4C163_TURTR\nCASK\tA0A2U4C192_TURTR\nCASK\tA0A2U4C197_TURTR\nCASK\tA0A2U4C1A2_TURTR\nCASK\tA0A2Y9EHL5_PHYMC\nCASK\tA0A2Y9EID9_PHYMC\nCASK\tA0A2Y9EJ11_PHYMC\nCASK\tA0A2Y9EJ14_PHYMC\nCASK\tA0A2Y9EJP8_PHYMC\nCASK\tA0A2Y9EJQ0_PHYMC\nCASK\tA0A2Y9GF10_NEOSC\nCASK\tA0A2Y9LQV8_DELLE\nCASK\tA0A2Y9LQW3_DELLE\nCASK\tA0A2Y9LRZ7_DELLE\nCASK\tA0A2Y9LTT7_DELLE\nCASK\tA0A2Y9LTU0_DELLE\nCASK\tA0A2Y9LYL8_DELLE\nCASK\tA0A2Y9LYM3_DELLE\nCASK\tA0A2Y9M444_DELLE\nCASK\tA0A337SRG8_FELCA\nCASK\tA0A340X6R8_LIPVE\nCASK\tA0A340X7E1_LIPVE\nCASK\tA0A340XB07_LIPVE\nCASK\tA0A340XDH7_LIPVE\nCASK\tA0A340XDZ5_LIPVE\nCASK\tA0A341CLC4_NEOAA\nCASK\tA0A341CLR9_NEOAA\nCASK\tA0A341CLV9_NEOAA\nCASK\tA0A341CP20_NEOAA\nCASK\tA0A384DIX1_URSMA\nCASK\tA0A3B3BNS2_ORYME\nCASK\tA0A3B3BNV7_ORYME\nCASK\tA0A3B3BP26_ORYME\nCASK\tA0A3B3BP97_ORYME\nCASK\tA0A3B3BQN2_ORYME\nCASK\tA0A3B3BQR2_ORYME\nCASK\tA0A3B3D885_ORYME\nCASK\tA0A3B3DAM6_ORYME\nCASK\tA0A3B3HCN8_ORYLA\nCASK\tA0A3B3HDN3_ORYLA\nCASK\tA0A3B3HIW5_ORYLA\nCASK\tA0A3B3HJA1_ORYLA\nCASK\tA0A3B3IAP5_ORYLA\nCASK\tA0A3B3IBI1_ORYLA\nCASK\tA0A3B3IEA6_ORYLA\nCASK\tA0A3B3ILC1_ORYLA\nCASK\tA0A3B3RKX1_9TELE\nCASK\tA0A3B3RLW5_9TELE\nCASK\tA0A3B3RMV5_9TELE\nCASK\tA0A3B3RN03_9TELE\nCASK\tA0A3B4XFX3_SERLL\nCASK\tA0A3B4XK94_SERLL\nCASK\tA0A3B4XKE5_SERLL\nCASK\tA0A3B4XZF8_SERLL\nCASK\tA0A3B5BDT3_9TELE\nCASK\tA0A3P8TU50_AMPPE\nCASK\tA0A3P8TU94_AMPPE\nCASK\tA0A3P8U013_AMPPE\nCASK\tA0A3P8U053_AMPPE\nCASK\tA0A3P9HNY1_ORYLA\nCASK\tA0A3P9HP66_ORYLA\nCASK\tA0A3P9HPB8_ORYLA\nCASK\tA0A3P9HQ75_ORYLA\nCASK\tA0A3P9J6X3_ORYLA\nCASK\tA0A3P9J772_ORYLA\nCASK\tA0A3P9J7J2_ORYLA\nCASK\tA0A3Q0D3D6_MESAU\nCASK\tA0A3Q0D6Z8_MESAU\nCASK\tA0A3Q0D7G5_MESAU\nCASK\tA0A3Q0D7Z8_MESAU\nCASK\tA0A3Q0D8G6_MESAU\nCASK\tA0A3Q0E8H8_TARSY\nCASK\tA0A3Q0GLV8_ALLSI\nCASK\tA0A3Q0GLW5_ALLSI\nCASK\tA0A3Q0GLW7_ALLSI\nCASK\tA0A3Q0GLZ3_ALLSI\nCASK\tA0A3Q0GMP9_ALLSI\nCASK\tA0A3Q0GMR5_ALLSI\nCASK\tA0A3Q0GRE7_ALLSI\nCASK\tA0A3Q0GRI8_ALLSI\nCASK\tA0A3Q0GRL2_ALLSI\nCASK\tA0A3Q1EH26_9TELE\nCASK\tA0A3Q1F4M4_9TELE\nCASK\tA0A3Q1FW10_9TELE\nCASK\tA0A3Q1G136_9TELE\nCASK\tA0A3Q1G172_9TELE\nCASK\tA0A3Q1G3Z4_9TELE\nCASK\tA0A3Q1IYY9_ANATE\nCASK\tA0A3Q1JAU2_ANATE\nCASK\tA0A3Q1JB30_ANATE\nCASK\tA0A3Q1K652_ANATE\nCASK\tA0A3Q1K663_ANATE\nCASK\tA0A3Q1LMD0_BOVIN\nCASK\tA0A3Q1LPV4_BOVIN\nCASK\tA0A3Q1N8J3_BOVIN\nCASK\tA0A3Q2H018_HORSE\nCASK\tA0A3Q2H6I3_HORSE\nCASK\tA0A3Q2HQM5_HORSE\nCASK\tA0A3Q2HV90_HORSE\nCASK\tA0A3Q2I1F3_HORSE\nCASK\tA0A3Q2I1W8_HORSE\nCASK\tA0A3Q2I3D6_HORSE\nCASK\tA0A3Q2I8Y0_HORSE\nCASK\tA0A3Q2ULY1_CHICK\nCASK\tA0A3Q2UPZ2_CHICK\nCASK\tA0A3Q3ACZ6_KRYMA\nCASK\tA0A3Q3ADF1_KRYMA\nCASK\tA0A3Q3AE13_KRYMA\nCASK\tA0A3Q3AST1_KRYMA\nCASK\tA0A3Q3F9V0_9LABR\nCASK\tA0A3Q3K6P1_MONAL\nCASK\tA0A3Q3K6S7_MONAL\nCASK\tA0A3Q3KGF2_MONAL\nCASK\tA0A3Q3KGV3_MONAL\nCASK\tA0A3Q3KH40_MONAL\nCASK\tA0A3Q3R7S5_MONAL\nCASK\tA0A3Q3R7X2_MONAL\nCASK\tA0A3Q3W2X2_MOLML\nCASK\tA0A3Q7MYE3_CALUR\nCASK\tA0A3Q7N0X5_CALUR\nCASK\tA0A3Q7N3Q4_CALUR\nCASK\tA0A3Q7PC45_CALUR\nCASK\tA0A3Q7SW18_VULVU\nCASK\tA0A3Q7SW24_VULVU\nCASK\tA0A3Q7SW28_VULVU\nCASK\tA0A3Q7TE07_VULVU\nCASK\tA0A3Q7TE13_VULVU\nCASK\tA0A3Q7TS35_VULVU\nCASK\tA0A3Q7TS40_VULVU\nCASK\tA0A3Q7U3Q2_VULVU\nCASK\tA0A3Q7U3Q7_VULVU\nCASK\tA0A3Q7UL29_VULVU\nCASK\tA0A3Q7UL40_VULVU\nCASK\tA0A3Q7WVP5_URSAR\nCASK\tA0A3Q7X486_URSAR\nCASK\tA0A3Q7X491_URSAR\nCASK\tA0A3Q7XQM6_URSAR\nCASK\tA0A3Q7XQN1_URSAR\nCASK\tA0A3Q7XZJ8_URSAR\nCASK\tA0A3Q7XZK0_URSAR\nCASK\tA0A419PZV3_CLOSI\nCASK\tA0A452CJT0_BALAS\nCASK\tA0A452EQX2_CAPHI\nCASK\tA0A452EQZ5_CAPHI\nCASK\tA0A452ERD6_CAPHI\nCASK\tA0A452ERF0_CAPHI\nCASK\tA0A452ESC6_CAPHI\nCASK\tA0A452I1J3_9SAUR\nCASK\tA0A452I1J5_9SAUR\nCASK\tA0A452I1K1_9SAUR\nCASK\tA0A452I1K6_9SAUR\nCASK\tA0A452I1L1_9SAUR\nCASK\tA0A452I1L5_9SAUR\nCASK\tA0A452I1M3_9SAUR\nCASK\tA0A452UPD2_URSMA\nCASK\tA0A452UPE8_URSMA\nCASK\tA0A452UPF1_URSMA\nCASK\tA0A452UPF9_URSMA\nCASK\tA0A452UPH3_URSMA\nCASK\tA0A452UPH9_URSMA\nCASK\tA0A452UPK2_URSMA\nCASK\tA0A452UPW7_URSMA\nCASK\tA0A452UPX3_URSMA\nCASK\tA0A455AQ37_PHYMC\nCASK\tA0A455AQC3_PHYMC\nCASK\tA0A455ARP0_PHYMC\nCASK\tA0A455ARP5_PHYMC\nCASK\tA0A455AS88_PHYMC\nCASK\tA0A455B421_PHYMC\nCASK\tA0A493STP6_ANAPP\nCASK\tA0A493STZ0_ANAPP\nCASK\tA0A493T1A4_ANAPP\nCASK\tA0A493TIU8_ANAPP\nCASK\tA0A493TWE5_ANAPP\nCASK\tA0A4W2G2Q1_BOBOX\nCASK\tA0A4W2HR23_BOBOX\nCASK\tA0A4W2HUX5_BOBOX\nCASK\tA0A4W3I6C4_CALMI\nCASK\tA0A4W3I6D0_CALMI\nCASK\tA0A4W3I9X9_CALMI\nCASK\tA0A4W3IFA6_CALMI\nCASK\tA0A4W3IFB1_CALMI\nCASK\tA0A4W3IFC4_CALMI\nCASK\tA0A4W3IY34_CALMI\nCASK\tA0A4W3IY45_CALMI\nCASK\tA0A4W4ETF9_ELEEL\nCASK\tA0A4W4ETG4_ELEEL\nCASK\tA0A4W4ETW8_ELEEL\nCASK\tA0A4W4ETX4_ELEEL\nCASK\tA0A4W4EUJ5_ELEEL\nCASK\tA0A4W4EUK0_ELEEL\nCASK\tA0A4W4EW34_ELEEL\nCASK\tA0A4W4EW41_ELEEL\nCASK\tA0A4W4EW47_ELEEL\nCASK\tA0A4W6EFH8_LATCA\nCASK\tA0A4W6EFI3_LATCA\nCASK\tA0A4W6EFJ8_LATCA\nCASK\tA0A4W6EHS7_LATCA\nCASK\tA0A4W6EHT2_LATCA\nCASK\tA0A4W6EHU8_LATCA\nCASK\tA0A4W6EJ03_LATCA\nCASK\tA0A4W6EK05_LATCA\nCASK\tA0A4W6EKN8_LATCA\nCASK\tA0A4X1VHR9_PIG\nCASK\tA0A4X1VHS4_PIG\nCASK\tA0A4X1VHY6_PIG\nCASK\tA0A4X1VIU9_PIG\nCASK\tA0A4X1VKH9_PIG\nCASK\tA0A4X1VKI3_PIG\nCASK\tA0A4X2JV61_VOMUR\nCASK\tA0A4X2K2R0_VOMUR\nCASK\tA0A4X2K2X6_VOMUR\nCASK\tA0A4X2K320_VOMUR\nCASK\tA0A4X2K348_VOMUR\nCASK\tA0A4Z2HX27_9TELE\nCASK\tA0A5F4BSN7_CANLF\nCASK\tA0A5F4BTJ2_CANLF\nCASK\tA0A5F4BWV1_CANLF\nCASK\tA0A5F4CK42_CANLF\nCASK\tA0A5F4D0G0_CANLF\nCASK\tA0A5F4DDU9_CANLF\nCASK\tA0A5F8GA00_MONDO\nCASK\tA0A5F8GHE9_MONDO\nCASK\tA0A5F8GLR6_MONDO\nCASK\tA0A5F8GN34_MONDO\nCASK\tA0A5F8H278_MONDO\nCASK\tA0A5F9CDQ6_RABIT\nCASK\tA0A5F9CL83_RABIT\nCASK\tA0A5F9CXB8_RABIT\nCASK\tA0A5F9DLG1_RABIT\nCASK\tA0A5F9DPU0_RABIT\nCASK\tA0A5F9DQ03_RABIT\nCASK\tA0A5G2Q877_PIG\nCASK\tA0A5G2QEM6_PIG\nCASK\tA0A5G2RAD8_PIG\nCASK\tA0A5G2RKB8_PIG\nCASK\tA0A5K1UUM3_CALJA\nCASK\tA5D7B9_BOVIN\nCASK\tA8JR71_DROME\nCASK\tA8JR72_DROME\nCASK\tB1AUH5_MOUSE\nCASK\tB1AUH6_MOUSE\nCASK\tB3DH78_DANRE\nCASK\tB7ZKY2_HUMAN\nCASK\tB7ZP42_MOUSE\nCASK\tB9EJ23_MOUSE\nCASK\tC1LRH3_SCHJA\nCASK\tC1LRH4_SCHJA\nCASK\tC1LRH5_SCHJA\nCASK\tCASK_HUMAN\nCASK\tCSK2A_DICDI\nCASK\tCSKP_DROME\nCASK\tCSKP_HUMAN\nCASK\tCSKP_MOUSE\nCASK\tCSKP_RAT\nCASK\tD3B471_POLPP\nCASK\tE1JIS7_DROME\nCASK\tE2RLG1_CANLF\nCASK\tF1QWL2_DANRE\nCASK\tF4PVJ1_CAVFA\nCASK\tF6UGQ3_XENTR\nCASK\tF6UIF1_MONDO\nCASK\tF6W7Q7_CALJA\nCASK\tF6Y9I5_MOUSE\nCASK\tF6ZED9_MACMU\nCASK\tF6ZEF8_MACMU\nCASK\tF6ZQR2_MOUSE\nCASK\tF7BKQ5_MACMU\nCASK\tF7BPU5_CALJA\nCASK\tF7CD87_MOUSE\nCASK\tF7DK34_HORSE\nCASK\tF7DNN7_HORSE\nCASK\tG1KUI6_ANOCA\nCASK\tG1LJM6_AILME\nCASK\tG1LJQ0_AILME\nCASK\tG1NP41_MELGA\nCASK\tG1NZ58_MYOLU\nCASK\tG1QP12_NOMLE\nCASK\tG1SMP3_RABIT\nCASK\tG3NZW3_GASAC\nCASK\tG3SZ61_LOXAF\nCASK\tG3TSB0_LOXAF\nCASK\tG3UBH3_LOXAF\nCASK\tG3VH62_SARHA\nCASK\tH0VHV2_CAVPO\nCASK\tH0WQT1_OTOGA\nCASK\tH2MEX3_ORYLA\nCASK\tH2MEX6_ORYLA\nCASK\tH2PVC6_PONAB\nCASK\tH2R4R8_PANTR\nCASK\tH2TXP5_TAKRU\nCASK\tH2TXP6_TAKRU\nCASK\tH2TXP8_TAKRU\nCASK\tH3AP66_LATCH\nCASK\tH9EPC3_MACMU\nCASK\tH9FCF1_MACMU\nCASK\tH9G157_MACMU\nCASK\tI3M703_ICTTR\nCASK\tI3N909_ICTTR\nCASK\tJ9NZ27_CANLF\nCASK\tK7AHC2_PANTR\nCASK\tK7BZH5_PANTR\nCASK\tK7GD09_PELSI\nCASK\tK7GD15_PELSI\nCASK\tK7GSB2_PIG\nCASK\tM3WEK7_FELCA\nCASK\tM3YMI8_MUSPF\nCASK\tQ08D54_XENTR\nCASK\tQ28CJ8_XENTR\nCASK\tQ3TNZ0_MOUSE\nCASK\tQ5JS72_HUMAN\nCASK\tQ5JS79_HUMAN\nCASK\tQ6DCN2_XENLA\nCASK\tQ6XCE7_RAT\nCASK\tQ910A4_DANRE\nCASK\tQ91VJ0_MOUSE\nCASK\tQ9XSD6_TRIVU\nCASK\tR4GB75_ANOCA\nCASK\tU3BRM7_CALJA\nCASK\tU3CM89_CALJA\nCASK\tU3DET2_CALJA\nCASK\tU3DLD1_CALJA\nCASK\tU3EY01_CALJA\nCASK\tU3J2V9_ANAPP\nCASK\tU3JFN2_FICAL\nCASK\tV8NH05_OPHHA\nCASK\tV9KCJ3_CALMI\nCASK\tW5NXK9_SHEEP\nCASK\tW5NXL1_SHEEP\n','From\tTo\nCAMK1G\tA0A096NAP5_PAPAN\nCAMK1G\tA0A0D9RRT3_CHLSB\nCAMK1G\tA0A151MJP0_ALLMI\nCAMK1G\tA0A151MJP9_ALLMI\nCAMK1G\tA0A1D1XVT9_9ARAE\nCAMK1G\tA0A1S3F3A2_DIPOR\nCAMK1G\tA0A1S3WLP4_ERIEU\nCAMK1G\tA0A1U7R038_MESAU\nCAMK1G\tA0A1U7SJ08_ALLSI\nCAMK1G\tA0A1U7TP77_TARSY\nCAMK1G\tA0A1U7TWX5_TARSY\nCAMK1G\tA0A1U8CFA5_MESAU\nCAMK1G\tA0A1V4KXG9_PATFA\nCAMK1G\tA0A218UGM1_9PASE\nCAMK1G\tA0A286ZZN1_PIG\nCAMK1G\tA0A287B345_PIG\nCAMK1G\tA0A287D6H4_ICTTR\nCAMK1G\tA0A2D0SKI3_ICTPU\nCAMK1G\tA0A2D0SKI6_ICTPU\nCAMK1G\tA0A2I0M063_COLLI\nCAMK1G\tA0A2J8QVK8_PANTR\nCAMK1G\tA0A2J8V6J6_PONAB\nCAMK1G\tA0A2K5DA64_AOTNA\nCAMK1G\tA0A2K5HYZ1_COLAP\nCAMK1G\tA0A2K5N4K8_CERAT\nCAMK1G\tA0A2K5RSI9_CEBCA\nCAMK1G\tA0A2K6AA90_MANLE\nCAMK1G\tA0A2K6BLD4_MACNE\nCAMK1G\tA0A2K6GKT7_PROCO\nCAMK1G\tA0A2K6LUU6_RHIBE\nCAMK1G\tA0A2K6R3C7_RHIRO\nCAMK1G\tA0A2K6TMG5_SAIBB\nCAMK1G\tA0A2K6TMI4_SAIBB\nCAMK1G\tA0A2R9CCR5_PANPA\nCAMK1G\tA0A2U3V5L7_TURTR\nCAMK1G\tA0A2U3WJ26_ODORO\nCAMK1G\tA0A2U3XQG2_LEPWE\nCAMK1G\tA0A2Y9GGG6_NEOSC\nCAMK1G\tA0A2Y9Q1D0_DELLE\nCAMK1G\tA0A340WQ33_LIPVE\nCAMK1G\tA0A340WX23_LIPVE\nCAMK1G\tA0A341BVW0_NEOAA\nCAMK1G\tA0A383YMH6_BALAS\nCAMK1G\tA0A384CY96_URSMA\nCAMK1G\tA0A3Q1MQD2_BOVIN\nCAMK1G\tA0A3Q1MV02_BOVIN\nCAMK1G\tA0A3Q3FZW7_9LABR\nCAMK1G\tA0A3Q3G1F2_9LABR\nCAMK1G\tA0A3Q3G1I6_9LABR\nCAMK1G\tA0A3Q3G3W8_9LABR\nCAMK1G\tA0A3Q7PUP8_CALUR\nCAMK1G\tA0A3Q7R7Q1_VULVU\nCAMK1G\tA0A3Q7VUP7_URSAR\nCAMK1G\tA0A3Q7XEK6_URSAR\nCAMK1G\tA0A452EN30_CAPHI\nCAMK1G\tA0A452EN66_CAPHI\nCAMK1G\tA0A452GTV3_9SAUR\nCAMK1G\tA0A452RP62_URSAM\nCAMK1G\tA0A452V7Z3_URSMA\nCAMK1G\tA0A452V813_URSMA\nCAMK1G\tA0A455B4S7_PHYMC\nCAMK1G\tA0A494C109_HUMAN\nCAMK1G\tA0A4W2C6U9_BOBOX\nCAMK1G\tA0A4W2GQB1_BOBOX\nCAMK1G\tA0A4W2GQC2_BOBOX\nCAMK1G\tA0A4W2I801_BOBOX\nCAMK1G\tA0A4W2I8B0_BOBOX\nCAMK1G\tA0A4X1TG11_PIG\nCAMK1G\tA0A4X1TGI4_PIG\nCAMK1G\tA0A4X1THA1_PIG\nCAMK1G\tA0A4X1THA9_PIG\nCAMK1G\tA0A4X1THD2_PIG\nCAMK1G\tA0A4X2L1X7_VOMUR\nCAMK1G\tA0A5F4CZY4_CANLF\nCAMK1G\tA0A5F5PKW3_HORSE\nCAMK1G\tA0A5F9C9R3_RABIT\nCAMK1G\tA0A5F9DBN0_RABIT\nCAMK1G\tA0A5G2QLB1_PIG\nCAMK1G\tA0A5G3HM68_XENTR\nCAMK1G\tA0A5K1UNP7_FELCA\nCAMK1G\tC9IYV2_HUMAN\nCAMK1G\tD2HIK0_AILME\nCAMK1G\tE1BQ82_CHICK\nCAMK1G\tE2QUC1_CANLF\nCAMK1G\tE6ZFK0_DICLA\nCAMK1G\tE9Q679_MOUSE\nCAMK1G\tF1N2U4_BOVIN\nCAMK1G\tF1SF44_PIG\nCAMK1G\tF6XVV1_XENTR\nCAMK1G\tF6YRI7_MOUSE\nCAMK1G\tF6Z9D3_CIOIN\nCAMK1G\tF7DEH2_MONDO\nCAMK1G\tF7DNG7_HORSE\nCAMK1G\tF7E4N8_ORNAN\nCAMK1G\tF7E4P4_ORNAN\nCAMK1G\tF7G5K6_CALJA\nCAMK1G\tG1KEK3_ANOCA\nCAMK1G\tG1MYT8_MELGA\nCAMK1G\tG1P2J2_MYOLU\nCAMK1G\tG1QM17_NOMLE\nCAMK1G\tG1SKW2_RABIT\nCAMK1G\tG3SVS5_LOXAF\nCAMK1G\tG3W703_SARHA\nCAMK1G\tG7MEP2_MACMU\nCAMK1G\tG7NUP1_MACFA\nCAMK1G\tH0V2L7_CAVPO\nCAMK1G\tH0X264_OTOGA\nCAMK1G\tH0YTE1_TAEGU\nCAMK1G\tH3BH41_LATCH\nCAMK1G\tH9F0S8_MACMU\nCAMK1G\tH9F6A9_MACMU\nCAMK1G\tI3MRR4_ICTTR\nCAMK1G\tK7F6N9_PELSI\nCAMK1G\tKCC1G_HUMAN\nCAMK1G\tKCC1G_MOUSE\nCAMK1G\tKCC1G_RAT\nCAMK1G\tM3XV47_MUSPF\nCAMK1G\tQ08DI2_BOVIN\nCAMK1G\tQ6PC33_DANRE\nCAMK1G\tQ86SC3_CIOIN\nCAMK1G\tU3ITI2_ANAPP\nCAMK1G\tU3JCM7_FICAL\nCAMK1G\tW5PUX8_SHEEP\n','From\tTo\nCAMK4\tA0A096N9Y8_PAPAN\nCAMK4\tA0A0D9RR33_CHLSB\nCAMK4\tA0A151PIC0_ALLMI\nCAMK4\tA0A162TUT4_PHYB8\nCAMK4\tA0A1L1RWI9_CHICK\nCAMK4\tA0A1Q9D2C1_SYMMI\nCAMK4\tA0A1Q9DJV7_SYMMI\nCAMK4\tA0A1Q9DNJ8_SYMMI\nCAMK4\tA0A1S3AAM3_ERIEU\nCAMK4\tA0A1S3FHI4_DIPOR\nCAMK4\tA0A1S3KTT0_SALSA\nCAMK4\tA0A1U7Q625_MESAU\nCAMK4\tA0A1V4KPV1_PATFA\nCAMK4\tA0A286ZKQ7_PIG\nCAMK4\tA0A287A4F5_PIG\nCAMK4\tA0A287AY42_PIG\nCAMK4\tA0A287BHI0_PIG\nCAMK4\tA0A2B4ST94_STYPI\nCAMK4\tA0A2I0M8Y1_COLLI\nCAMK4\tA0A2I3H4N3_NOMLE\nCAMK4\tA0A2I3MM10_PAPAN\nCAMK4\tA0A2I4CQV8_9TELE\nCAMK4\tA0A2I4CQW1_9TELE\nCAMK4\tA0A2J7R296_9NEOP\nCAMK4\tA0A2J8KIU8_PANTR\nCAMK4\tA0A2K5CIB1_AOTNA\nCAMK4\tA0A2K5CIB7_AOTNA\nCAMK4\tA0A2K5H8H2_COLAP\nCAMK4\tA0A2K5H8H6_COLAP\nCAMK4\tA0A2K5M5V2_CERAT\nCAMK4\tA0A2K5M5W2_CERAT\nCAMK4\tA0A2K5M5X4_CERAT\nCAMK4\tA0A2K5R5Q4_CEBCA\nCAMK4\tA0A2K5W3H1_MACFA\nCAMK4\tA0A2K5W3H6_MACFA\nCAMK4\tA0A2K5W3M8_MACFA\nCAMK4\tA0A2K5Z2N1_MANLE\nCAMK4\tA0A2K5Z2Q8_MANLE\nCAMK4\tA0A2K6BPX3_MACNE\nCAMK4\tA0A2K6BPY7_MACNE\nCAMK4\tA0A2K6GZ48_PROCO\nCAMK4\tA0A2K6LFI5_RHIBE\nCAMK4\tA0A2K6NFS4_RHIRO\nCAMK4\tA0A2K6SQ98_SAIBB\nCAMK4\tA0A2P8YEJ6_BLAGE\nCAMK4\tA0A2R9C7Q5_PANPA\nCAMK4\tA0A2U3W0F9_ODORO\nCAMK4\tA0A2U3Z2E8_LEPWE\nCAMK4\tA0A2U4BT38_TURTR\nCAMK4\tA0A2Y9HZA8_NEOSC\nCAMK4\tA0A2Y9SRK2_PHYMC\nCAMK4\tA0A340WUI5_LIPVE\nCAMK4\tA0A341BA93_NEOAA\nCAMK4\tA0A341BBR7_NEOAA\nCAMK4\tA0A384AXJ7_BALAS\nCAMK4\tA0A384AXL9_BALAS\nCAMK4\tA0A384AYK1_BALAS\nCAMK4\tA0A384D8Q2_URSMA\nCAMK4\tA0A3Q0DBQ6_MESAU\nCAMK4\tA0A3Q0E7U5_TARSY\nCAMK4\tA0A3Q0GI22_ALLSI\nCAMK4\tA0A3Q1M5T5_BOVIN\nCAMK4\tA0A3Q2GXT0_HORSE\nCAMK4\tA0A3Q2I361_HORSE\nCAMK4\tA0A3Q4EG96_MOUSE\nCAMK4\tA0A3Q7NM17_CALUR\nCAMK4\tA0A3Q7NWR6_CALUR\nCAMK4\tA0A3Q7Q308_CALUR\nCAMK4\tA0A3Q7S1V5_VULVU\nCAMK4\tA0A3Q7UZ42_URSAR\nCAMK4\tA0A419Q6M6_CLOSI\nCAMK4\tA0A452EA99_CAPHI\nCAMK4\tA0A452I5U5_9SAUR\nCAMK4\tA0A452I5V7_9SAUR\nCAMK4\tA0A452I5W9_9SAUR\nCAMK4\tA0A452I613_9SAUR\nCAMK4\tA0A452UMY0_URSMA\nCAMK4\tA0A452UN17_URSMA\nCAMK4\tA0A452UNB0_URSMA\nCAMK4\tA0A480RUZ5_PIG\nCAMK4\tA0A493T2Y6_ANAPP\nCAMK4\tA0A493T6S7_ANAPP\nCAMK4\tA0A493TCQ6_ANAPP\nCAMK4\tA0A493TZ09_ANAPP\nCAMK4\tA0A4W2D4I1_BOBOX\nCAMK4\tA0A4W2EPT1_BOBOX\nCAMK4\tA0A4W2F7E1_BOBOX\nCAMK4\tA0A4W4G3X2_ELEEL\nCAMK4\tA0A4W6F7M2_LATCA\nCAMK4\tA0A4X1TW31_PIG\nCAMK4\tA0A4X1TWC7_PIG\nCAMK4\tA0A4X1TWE0_PIG\nCAMK4\tA0A4X1U185_PIG\nCAMK4\tA0A4X1U195_PIG\nCAMK4\tA0A4X2KGD6_VOMUR\nCAMK4\tA0A5F4VSP9_CALJA\nCAMK4\tA0A5F5XMK9_FELCA\nCAMK4\tA0A5F5XYB7_FELCA\nCAMK4\tA0A5F7ZQ49_MACMU\nCAMK4\tA0A5F8A305_MACMU\nCAMK4\tA0A5G3HRS0_XENTR\nCAMK4\tA0A5K1VBI2_MACMU\nCAMK4\tCAMK3_ARATH\nCAMK4\tD6RCD6_HUMAN\nCAMK4\tD6RE65_HUMAN\nCAMK4\tD6REY7_HUMAN\nCAMK4\tE2QWL2_CANLF\nCAMK4\tF6W090_ORNAN\nCAMK4\tF6WCB6_CALJA\nCAMK4\tF6WUG4_HORSE\nCAMK4\tF6WYU7_XENTR\nCAMK4\tF6WZB6_XENTR\nCAMK4\tF6XL87_XENTR\nCAMK4\tF7CAE2_XENTR\nCAMK4\tF7I122_CALJA\nCAMK4\tG1M258_AILME\nCAMK4\tG1M260_AILME\nCAMK4\tG1N4Q2_MELGA\nCAMK4\tG1P903_MYOLU\nCAMK4\tG1RL45_NOMLE\nCAMK4\tG3SYI8_LOXAF\nCAMK4\tG3UBW4_LOXAF\nCAMK4\tG3X816_BOVIN\nCAMK4\tH0V9B3_CAVPO\nCAMK4\tH0WWM2_OTOGA\nCAMK4\tH0YRW5_TAEGU\nCAMK4\tH2TJC9_TAKRU\nCAMK4\tH2ZYK1_LATCH\nCAMK4\tH9ER84_MACMU\nCAMK4\tH9GCJ3_ANOCA\nCAMK4\tI3M4D4_ICTTR\nCAMK4\tK7G0Z5_PELSI\nCAMK4\tKCC4_HUMAN\nCAMK4\tKCC4_MOUSE\nCAMK4\tKCC4_RAT\nCAMK4\tM3VWV3_FELCA\nCAMK4\tQ568I9_DANRE\nCAMK4\tQ8BGR3_MOUSE\nCAMK4\tQ98TZ2_XENLA\nCAMK4\tT2MCL6_HYDVU\nCAMK4\tU3J1V2_ANAPP\nCAMK4\tU3K7Z2_FICAL\nCAMK4\tV8NHP9_OPHHA\nCAMK4\tV8NNG5_OPHHA\nCAMK4\tW5NPS2_SHEEP\nCAMK4\tW5U8H1_ICTPU\nCAMK4\tW8SMR0_9BASI\n','From\tTo\nDAPK2\tA0A024R603_HUMAN\nDAPK2\tA0A096NMB1_PAPAN\nDAPK2\tA0A0G2JX07_RAT\nDAPK2\tA0A0S7I3B6_9TELE\nDAPK2\tA0A0S7I6T2_9TELE\nDAPK2\tA0A0S7ID43_9TELE\nDAPK2\tA0A0S7ID58_9TELE\nDAPK2\tA0A0S7KVW4_9TELE\nDAPK2\tA0A151MA36_ALLMI\nDAPK2\tA0A151MA96_ALLMI\nDAPK2\tA0A1A7WC41_9TELE\nDAPK2\tA0A1A7XI84_9TELE\nDAPK2\tA0A1A7XZ98_9TELE\nDAPK2\tA0A1A7YY15_9TELE\nDAPK2\tA0A1A8AEZ3_NOTFU\nDAPK2\tA0A1A8CP36_9TELE\nDAPK2\tA0A1A8DJS3_9TELE\nDAPK2\tA0A1A8G6Y0_9TELE\nDAPK2\tA0A1A8HRC8_NOTKU\nDAPK2\tA0A1A8L2Z2_9TELE\nDAPK2\tA0A1A8MU14_9TELE\nDAPK2\tA0A1D5R7Y5_MACMU\nDAPK2\tA0A1Q9F4K3_SYMMI\nDAPK2\tA0A1S3AG22_ERIEU\nDAPK2\tA0A1U7QVH7_MESAU\nDAPK2\tA0A1U7S871_ALLSI\nDAPK2\tA0A1U7TZ34_TARSY\nDAPK2\tA0A1U7U0T1_TARSY\nDAPK2\tA0A1V4JCV0_PATFA\nDAPK2\tA0A218UNF4_9PASE\nDAPK2\tA0A250XZI7_CASCN\nDAPK2\tA0A286X7D6_CAVPO\nDAPK2\tA0A286ZJ67_PIG\nDAPK2\tA0A286ZQY3_PIG\nDAPK2\tA0A287B2C0_PIG\nDAPK2\tA0A287D8V6_ICTTR\nDAPK2\tA0A2B4RSJ6_STYPI\nDAPK2\tA0A2B4T058_STYPI\nDAPK2\tA0A2D0SFR6_ICTPU\nDAPK2\tA0A2D0SH17_ICTPU\nDAPK2\tA0A2I0M6J0_COLLI\nDAPK2\tA0A2I0M6J7_COLLI\nDAPK2\tA0A2I0M6K3_COLLI\nDAPK2\tA0A2I2UQ18_FELCA\nDAPK2\tA0A2I3FWL6_NOMLE\nDAPK2\tA0A2I3HB00_NOMLE\nDAPK2\tA0A2I3LV46_PAPAN\nDAPK2\tA0A2I3SLN8_PANTR\nDAPK2\tA0A2I3SS06_PANTR\nDAPK2\tA0A2J8NRQ6_PANTR\nDAPK2\tA0A2K5D4H1_AOTNA\nDAPK2\tA0A2K5D4L1_AOTNA\nDAPK2\tA0A2K5JUW0_COLAP\nDAPK2\tA0A2K5JV28_COLAP\nDAPK2\tA0A2K5JV34_COLAP\nDAPK2\tA0A2K5NY79_CERAT\nDAPK2\tA0A2K5NYB9_CERAT\nDAPK2\tA0A2K5NYE0_CERAT\nDAPK2\tA0A2K5QPX9_CEBCA\nDAPK2\tA0A2K5QPY7_CEBCA\nDAPK2\tA0A2K5UL87_MACFA\nDAPK2\tA0A2K5YTB1_MANLE\nDAPK2\tA0A2K5YTB3_MANLE\nDAPK2\tA0A2K5YTE0_MANLE\nDAPK2\tA0A2K6CYH2_MACNE\nDAPK2\tA0A2K6CYH4_MACNE\nDAPK2\tA0A2K6F0W8_PROCO\nDAPK2\tA0A2K6F0X4_PROCO\nDAPK2\tA0A2K6F0Y2_PROCO\nDAPK2\tA0A2K6F0Y7_PROCO\nDAPK2\tA0A2K6KKK2_RHIBE\nDAPK2\tA0A2K6KKK8_RHIBE\nDAPK2\tA0A2K6P1G1_RHIRO\nDAPK2\tA0A2K6P1G2_RHIRO\nDAPK2\tA0A2K6SAK7_SAIBB\nDAPK2\tA0A2K6SAN0_SAIBB\nDAPK2\tA0A2U3WKR3_ODORO\nDAPK2\tA0A2U4CLE0_TURTR\nDAPK2\tA0A2Y9HDC3_NEOSC\nDAPK2\tA0A2Y9HDQ2_NEOSC\nDAPK2\tA0A2Y9NN50_DELLE\nDAPK2\tA0A2Y9NW99_DELLE\nDAPK2\tA0A340WPR1_LIPVE\nDAPK2\tA0A341AMZ6_NEOAA\nDAPK2\tA0A341AP43_NEOAA\nDAPK2\tA0A383YT35_BALAS\nDAPK2\tA0A383YT46_BALAS\nDAPK2\tA0A383YTD9_BALAS\nDAPK2\tA0A384DIG3_URSMA\nDAPK2\tA0A384DIK1_URSMA\nDAPK2\tA0A3B4U1W3_SERDU\nDAPK2\tA0A3B4U226_SERDU\nDAPK2\tA0A3B4U2C6_SERDU\nDAPK2\tA0A3B4U3D3_SERDU\nDAPK2\tA0A3B4YQ71_SERLL\nDAPK2\tA0A3B4YQ77_SERLL\nDAPK2\tA0A3B4YUE4_SERLL\nDAPK2\tA0A3B4YUJ2_SERLL\nDAPK2\tA0A3P9HNZ0_ORYLA\nDAPK2\tA0A3P9J6L6_ORYLA\nDAPK2\tA0A3P9KCT2_ORYLA\nDAPK2\tA0A3P9LXN5_ORYLA\nDAPK2\tA0A3Q0CZL9_MESAU\nDAPK2\tA0A3Q0GWM4_ALLSI\nDAPK2\tA0A3Q0H162_ALLSI\nDAPK2\tA0A3Q1MRM4_BOVIN\nDAPK2\tA0A3Q1N4B5_BOVIN\nDAPK2\tA0A3Q2HPG2_HORSE\nDAPK2\tA0A3Q2L5L8_HORSE\nDAPK2\tA0A3Q3AJ65_CHICK\nDAPK2\tA0A3Q7Q280_CALUR\nDAPK2\tA0A3Q7QD32_CALUR\nDAPK2\tA0A3Q7RHY4_CALUR\nDAPK2\tA0A3Q7TXR9_VULVU\nDAPK2\tA0A3Q7U1F8_URSAR\nDAPK2\tA0A3Q7URG8_URSAR\nDAPK2\tA0A452DNH7_CAPHI\nDAPK2\tA0A452H6S4_9SAUR\nDAPK2\tA0A452U978_URSMA\nDAPK2\tA0A455AUW2_PHYMC\nDAPK2\tA0A455AUW7_PHYMC\nDAPK2\tA0A455AV66_PHYMC\nDAPK2\tA0A455AW71_PHYMC\nDAPK2\tA0A455AW76_PHYMC\nDAPK2\tA0A455AX28_PHYMC\nDAPK2\tA0A455AX33_PHYMC\nDAPK2\tA0A455B8X5_PHYMC\nDAPK2\tA0A455B8X9_PHYMC\nDAPK2\tA0A493TX91_ANAPP\nDAPK2\tA0A4W2CSR2_BOBOX\nDAPK2\tA0A4W2CWT6_BOBOX\nDAPK2\tA0A4W2ECW3_BOBOX\nDAPK2\tA0A4W2ELP2_BOBOX\nDAPK2\tA0A4W2GKZ3_BOBOX\nDAPK2\tA0A4W2GNY1_BOBOX\nDAPK2\tA0A4W2I758_BOBOX\nDAPK2\tA0A4W2I796_BOBOX\nDAPK2\tA0A4W3HAW9_CALMI\nDAPK2\tA0A4X1TWE9_PIG\nDAPK2\tA0A4X1TZ06_PIG\nDAPK2\tA0A4X1U1W3_PIG\nDAPK2\tA0A4X1U1Z6_PIG\nDAPK2\tA0A4X2LGL5_VOMUR\nDAPK2\tA0A5F4BQ81_CANLF\nDAPK2\tA0A5F4CYR0_CANLF\nDAPK2\tA0A5F4D9T8_CANLF\nDAPK2\tA0A5F4DM06_CANLF\nDAPK2\tA0A5F5XFA4_FELCA\nDAPK2\tA0A5F5XTT4_FELCA\nDAPK2\tA0A5F5XV22_FELCA\nDAPK2\tA0A5F7ZJ67_MACMU\nDAPK2\tA0A5F8A5R6_MACMU\nDAPK2\tA0A5F8HHC4_MONDO\nDAPK2\tA0A5F9C8L0_RABIT\nDAPK2\tA0A5F9D8P8_RABIT\nDAPK2\tA0A5F9DL09_RABIT\nDAPK2\tA0A5G2QUK0_PIG\nDAPK2\tA0A5G2R2H7_PIG\nDAPK2\tA0A5G3HJZ6_XENTR\nDAPK2\tA3KNX9_DANRE\nDAPK2\tB0JZ00_XENTR\nDAPK2\tB1H1H5_DANRE\nDAPK2\tDAPK2_HUMAN\nDAPK2\tDAPK2_MOUSE\nDAPK2\tE9JGN0_MOUSE\nDAPK2\tF1MHX3_BOVIN\nDAPK2\tF1NUY1_CHICK\nDAPK2\tF1Q4B7_CANLF\nDAPK2\tF6QTW8_HORSE\nDAPK2\tF6SJH2_XENTR\nDAPK2\tF6YX98_MOUSE\nDAPK2\tF7CSM8_MONDO\nDAPK2\tF7EDV0_MACMU\nDAPK2\tF7IJT0_CALJA\nDAPK2\tG1L6P1_AILME\nDAPK2\tG1MUJ5_MELGA\nDAPK2\tG1MXX9_MELGA\nDAPK2\tG1NTX9_MYOLU\nDAPK2\tG1RP78_NOMLE\nDAPK2\tG1SDA7_RABIT\nDAPK2\tG3SLI9_LOXAF\nDAPK2\tG3X1H9_SARHA\nDAPK2\tG7P8U1_MACFA\nDAPK2\tH0UUZ1_CAVPO\nDAPK2\tH0XHZ1_OTOGA\nDAPK2\tH0YKW8_HUMAN\nDAPK2\tH0YL86_HUMAN\nDAPK2\tH0YLP5_HUMAN\nDAPK2\tH0YMK1_HUMAN\nDAPK2\tH0YYF0_TAEGU\nDAPK2\tH2LT55_ORYLA\nDAPK2\tH2Q9K2_PANTR\nDAPK2\tH2U7P7_TAKRU\nDAPK2\tH3A429_LATCH\nDAPK2\tH9EQY6_MACMU\nDAPK2\tI3KDW3_ORENI\nDAPK2\tI3LIX3_PIG\nDAPK2\tI3MQW0_ICTTR\nDAPK2\tK7BJM8_PANTR\nDAPK2\tK7GBV5_PELSI\nDAPK2\tM3W1Q1_FELCA\nDAPK2\tM3XNF2_MUSPF\nDAPK2\tQ1RMF4_HUMAN\nDAPK2\tQ5BJN2_RAT\nDAPK2\tQ5I021_XENLA\nDAPK2\tT2M3L1_HYDVU\nDAPK2\tU3E7Z0_CALJA\nDAPK2\tU3I4S4_ANAPP\nDAPK2\tU3K7Q3_FICAL\nDAPK2\tU6D2L3_NEOVI\nDAPK2\tV5GC01_ANOGL\nDAPK2\tV8PES4_OPHHA\nDAPK2\tW5QHW2_SHEEP\n','From\tTo\nMAP3K9\tA0A087WW79_HUMAN\nMAP3K9\tA0A096NVW0_PAPAN\nMAP3K9\tA0A0D9RKJ5_CHLSB\nMAP3K9\tA0A0G2JUN9_RAT\nMAP3K9\tA0A0V0SE35_9BILA\nMAP3K9\tA0A0V0SE73_9BILA\nMAP3K9\tA0A0V0WAA7_9BILA\nMAP3K9\tA0A0V0WB71_9BILA\nMAP3K9\tA0A0V0WB89_9BILA\nMAP3K9\tA0A0V0Y3E6_TRIPS\nMAP3K9\tA0A0V0ZE18_9BILA\nMAP3K9\tA0A0V0ZEC8_9BILA\nMAP3K9\tA0A0V1CFY9_TRIBR\nMAP3K9\tA0A0V1CG06_TRIBR\nMAP3K9\tA0A0V1CG75_TRIBR\nMAP3K9\tA0A0V1CG78_TRIBR\nMAP3K9\tA0A0V1CGA8_TRIBR\nMAP3K9\tA0A0V1CGI0_TRIBR\nMAP3K9\tA0A0V1CHP6_TRIBR\nMAP3K9\tA0A0V1EAJ0_TRIPS\nMAP3K9\tA0A0V1FHW9_TRIPS\nMAP3K9\tA0A0V1H068_9BILA\nMAP3K9\tA0A0V1H0N0_9BILA\nMAP3K9\tA0A0V1IDR8_TRIPS\nMAP3K9\tA0A0V1KWC5_9BILA\nMAP3K9\tA0A0V1KWD5_9BILA\nMAP3K9\tA0A0V1KXJ8_9BILA\nMAP3K9\tA0A0V1KY03_9BILA\nMAP3K9\tA0A0V1MWS7_9BILA\nMAP3K9\tA0A0V1NRG3_9BILA\nMAP3K9\tA0A0V1NRN0_9BILA\nMAP3K9\tA0A0V1NS14_9BILA\nMAP3K9\tA0A151MB77_ALLMI\nMAP3K9\tA0A1A7WFA9_9TELE\nMAP3K9\tA0A1A7X6H3_9TELE\nMAP3K9\tA0A1A7ZPN8_NOTFU\nMAP3K9\tA0A1A8A6Y9_NOTFU\nMAP3K9\tA0A1A8B6F5_NOTFU\nMAP3K9\tA0A1A8BIJ4_9TELE\nMAP3K9\tA0A1A8BZT0_9TELE\nMAP3K9\tA0A1A8E1S9_9TELE\nMAP3K9\tA0A1A8EMT9_9TELE\nMAP3K9\tA0A1A8FSX6_9TELE\nMAP3K9\tA0A1A8G779_9TELE\nMAP3K9\tA0A1A8GSU0_9TELE\nMAP3K9\tA0A1A8HPR8_NOTKU\nMAP3K9\tA0A1A8I560_NOTKU\nMAP3K9\tA0A1A8IQ96_NOTKU\nMAP3K9\tA0A1A8JT88_NOTKU\nMAP3K9\tA0A1A8LM87_9TELE\nMAP3K9\tA0A1A8RAB1_9TELE\nMAP3K9\tA0A1A8RMJ2_9TELE\nMAP3K9\tA0A1A8RZI2_9TELE\nMAP3K9\tA0A1A8V4M3_NOTFU\nMAP3K9\tA0A1A8VLI1_NOTFU\nMAP3K9\tA0A1D5P4R5_CHICK\nMAP3K9\tA0A1D5R3L0_MACMU\nMAP3K9\tA0A1L8EZS5_XENLA\nMAP3K9\tA0A1L8FAK0_XENLA\nMAP3K9\tA0A1S3A2X5_ERIEU\nMAP3K9\tA0A1S3F9D6_DIPOR\nMAP3K9\tA0A1S3R7H6_SALSA\nMAP3K9\tA0A1S3R7X9_SALSA\nMAP3K9\tA0A1U7S449_ALLSI\nMAP3K9\tA0A1U8BXN4_MESAU\nMAP3K9\tA0A1U8C5L6_MESAU\nMAP3K9\tA0A1V4KF40_PATFA\nMAP3K9\tA0A1V4KF48_PATFA\nMAP3K9\tA0A1Y7VKX4_MOUSE\nMAP3K9\tA0A218U885_9PASE\nMAP3K9\tA0A218UFE9_9PASE\nMAP3K9\tA0A286XRQ3_CAVPO\nMAP3K9\tA0A287A0S9_PIG\nMAP3K9\tA0A2I0MPT1_COLLI\nMAP3K9\tA0A2I2U1R9_FELCA\nMAP3K9\tA0A2I3H3X6_NOMLE\nMAP3K9\tA0A2I3HFP7_NOMLE\nMAP3K9\tA0A2I3LH51_PAPAN\nMAP3K9\tA0A2I3LNV8_PAPAN\nMAP3K9\tA0A2I3N826_PAPAN\nMAP3K9\tA0A2I3RIG6_PANTR\nMAP3K9\tA0A2I3RQ30_PANTR\nMAP3K9\tA0A2I3SGS3_PANTR\nMAP3K9\tA0A2I4D433_9TELE\nMAP3K9\tA0A2K5BV90_AOTNA\nMAP3K9\tA0A2K5BV99_AOTNA\nMAP3K9\tA0A2K5BVD3_AOTNA\nMAP3K9\tA0A2K5HRU1_COLAP\nMAP3K9\tA0A2K5HRU2_COLAP\nMAP3K9\tA0A2K5HRU5_COLAP\nMAP3K9\tA0A2K5HRW2_COLAP\nMAP3K9\tA0A2K5LGE6_CERAT\nMAP3K9\tA0A2K5LGG2_CERAT\nMAP3K9\tA0A2K5LGI1_CERAT\nMAP3K9\tA0A2K5LGL5_CERAT\nMAP3K9\tA0A2K5S5A7_CEBCA\nMAP3K9\tA0A2K5S5B4_CEBCA\nMAP3K9\tA0A2K5S5D4_CEBCA\nMAP3K9\tA0A2K5U0R9_MACFA\nMAP3K9\tA0A2K5U0S9_MACFA\nMAP3K9\tA0A2K5U0T1_MACFA\nMAP3K9\tA0A2K5U0T5_MACFA\nMAP3K9\tA0A2K5XHG1_MANLE\nMAP3K9\tA0A2K5XHG7_MANLE\nMAP3K9\tA0A2K5XHG9_MANLE\nMAP3K9\tA0A2K5XHH3_MANLE\nMAP3K9\tA0A2K6CZA7_MACNE\nMAP3K9\tA0A2K6CZC7_MACNE\nMAP3K9\tA0A2K6CZF3_MACNE\nMAP3K9\tA0A2K6CZG6_MACNE\nMAP3K9\tA0A2K6FZR0_PROCO\nMAP3K9\tA0A2K6FZS7_PROCO\nMAP3K9\tA0A2K6FZU3_PROCO\nMAP3K9\tA0A2K6FZU5_PROCO\nMAP3K9\tA0A2K6KS99_RHIBE\nMAP3K9\tA0A2K6KSA1_RHIBE\nMAP3K9\tA0A2K6KSB3_RHIBE\nMAP3K9\tA0A2K6KSB6_RHIBE\nMAP3K9\tA0A2K6NLI8_RHIRO\nMAP3K9\tA0A2K6NLJ3_RHIRO\nMAP3K9\tA0A2K6NLJ4_RHIRO\nMAP3K9\tA0A2K6NLJ6_RHIRO\nMAP3K9\tA0A2K6NLJ8_RHIRO\nMAP3K9\tA0A2K6TM22_SAIBB\nMAP3K9\tA0A2K6TM56_SAIBB\nMAP3K9\tA0A2K6TM74_SAIBB\nMAP3K9\tA0A2P8Y8M9_BLAGE\nMAP3K9\tA0A2R8M2C9_CALJA\nMAP3K9\tA0A2R8ZX22_PANPA\nMAP3K9\tA0A2R8ZX37_PANPA\nMAP3K9\tA0A2R8ZX43_PANPA\nMAP3K9\tA0A2R8ZZF8_PANPA\nMAP3K9\tA0A2R9A1I1_PANPA\nMAP3K9\tA0A2U3X2N8_ODORO\nMAP3K9\tA0A2U3XSU7_LEPWE\nMAP3K9\tA0A2U4A929_TURTR\nMAP3K9\tA0A2U4A942_TURTR\nMAP3K9\tA0A2U4A9G8_TURTR\nMAP3K9\tA0A2Y9G8T6_NEOSC\nMAP3K9\tA0A2Y9G8W9_NEOSC\nMAP3K9\tA0A2Y9GBF6_NEOSC\nMAP3K9\tA0A2Y9N3G2_DELLE\nMAP3K9\tA0A2Y9SBY8_PHYMC\nMAP3K9\tA0A2Y9SBZ3_PHYMC\nMAP3K9\tA0A337S0E9_FELCA\nMAP3K9\tA0A340Y6P6_LIPVE\nMAP3K9\tA0A340YD55_LIPVE\nMAP3K9\tA0A341B0S6_NEOAA\nMAP3K9\tA0A341B1E7_NEOAA\nMAP3K9\tA0A367KPT2_RHIST\nMAP3K9\tA0A384AAU1_BALAS\nMAP3K9\tA0A384CUC4_URSMA\nMAP3K9\tA0A3B3HNV9_ORYLA\nMAP3K9\tA0A3B4WRN4_SERLL\nMAP3K9\tA0A3Q0E0S9_TARSY\nMAP3K9\tA0A3Q1LKT7_BOVIN\nMAP3K9\tA0A3Q1MHH5_BOVIN\nMAP3K9\tA0A3Q2LNS7_HORSE\nMAP3K9\tA0A3Q7NVH1_CALUR\nMAP3K9\tA0A3Q7NVR0_CALUR\nMAP3K9\tA0A3Q7SPA1_VULVU\nMAP3K9\tA0A3Q7T4L7_VULVU\nMAP3K9\tA0A3Q7TFP0_VULVU\nMAP3K9\tA0A3Q7U9Z4_VULVU\nMAP3K9\tA0A3Q7XLC2_URSAR\nMAP3K9\tA0A3Q7Y8W6_URSAR\nMAP3K9\tA0A419QA56_CLOSI\nMAP3K9\tA0A452EWR0_CAPHI\nMAP3K9\tA0A452EWR2_CAPHI\nMAP3K9\tA0A452EWR9_CAPHI\nMAP3K9\tA0A452I5S8_9SAUR\nMAP3K9\tA0A452I5X4_9SAUR\nMAP3K9\tA0A452SNF2_URSAM\nMAP3K9\tA0A452SNK3_URSAM\nMAP3K9\tA0A452TX65_URSMA\nMAP3K9\tA0A452TXG6_URSMA\nMAP3K9\tA0A452TXM4_URSMA\nMAP3K9\tA0A4W2D810_BOBOX\nMAP3K9\tA0A4W2ES21_BOBOX\nMAP3K9\tA0A4X1U0F8_PIG\nMAP3K9\tA0A4X1U376_PIG\nMAP3K9\tA0A4X2KWB8_VOMUR\nMAP3K9\tA0A4X2KWC3_VOMUR\nMAP3K9\tA0A4X2LXD6_VOMUR\nMAP3K9\tA0A5B7J7H8_PORTR\nMAP3K9\tA0A5F8G873_MONDO\nMAP3K9\tB8JHT2_DANRE\nMAP3K9\tF1LRA7_RAT\nMAP3K9\tF1N3K4_BOVIN\nMAP3K9\tF1P626_CANLF\nMAP3K9\tF1QKS7_DANRE\nMAP3K9\tF1S494_PIG\nMAP3K9\tF6RYB4_MONDO\nMAP3K9\tF6VUC2_MACMU\nMAP3K9\tF6YAA2_HORSE\nMAP3K9\tF6ZUF3_XENTR\nMAP3K9\tF7DAN9_MACMU\nMAP3K9\tF7GHN8_CALJA\nMAP3K9\tF7GPM0_CALJA\nMAP3K9\tF7H042_CALJA\nMAP3K9\tG1KDB6_ANOCA\nMAP3K9\tG1LN06_AILME\nMAP3K9\tG1LN14_AILME\nMAP3K9\tG1NGV0_MELGA\nMAP3K9\tG1PWP4_MYOLU\nMAP3K9\tG1QMU3_NOMLE\nMAP3K9\tG1SJ83_RABIT\nMAP3K9\tG1U5P3_RABIT\nMAP3K9\tG3TL22_LOXAF\nMAP3K9\tG3U5L5_LOXAF\nMAP3K9\tG3V347_HUMAN\nMAP3K9\tG3V4P9_HUMAN\nMAP3K9\tH0UX31_CAVPO\nMAP3K9\tH0X470_OTOGA\nMAP3K9\tH0ZM86_TAEGU\nMAP3K9\tH2M0D0_ORYLA\nMAP3K9\tH2NLP2_PONAB\nMAP3K9\tH2Q8J6_PANTR\nMAP3K9\tH2TXS4_TAKRU\nMAP3K9\tH3AIR1_LATCH\nMAP3K9\tH9F919_MACMU\nMAP3K9\tH9FJK0_MACMU\nMAP3K9\tI3KU71_ORENI\nMAP3K9\tI3MM95_ICTTR\nMAP3K9\tJ3KPI6_HUMAN\nMAP3K9\tK7GB78_PELSI\nMAP3K9\tK7GB91_PELSI\nMAP3K9\tM3K9_HUMAN\nMAP3K9\tM3K9_MOUSE\nMAP3K9\tM3W676_FELCA\nMAP3K9\tM3XWY7_MUSPF\nMAP3K9\tQ8K2L2_MOUSE\nMAP3K9\tT2MEX1_HYDVU\nMAP3K9\tU3J967_ANAPP\nMAP3K9\tU3JQQ8_FICAL\nMAP3K9\tV8PG52_OPHHA\nMAP3K9\tW5QJ82_SHEEP\n','From\tTo\nDAPK3\tA0A0A0MVV4_PAPAN\nDAPK3\tA0A0A9WV49_LYGHE\nDAPK3\tA0A0A9YBG6_LYGHE\nDAPK3\tA0A0D9REQ5_CHLSB\nDAPK3\tA0A0S7I3F3_9TELE\nDAPK3\tA0A0S7I3F6_9TELE\nDAPK3\tA0A0S7I5L3_9TELE\nDAPK3\tA0A0S7I7K2_9TELE\nDAPK3\tA0A0S7I941_9TELE\nDAPK3\tA0A0S7LWS4_9TELE\nDAPK3\tA0A0S7MD11_9TELE\nDAPK3\tA0A0S7MDQ9_9TELE\nDAPK3\tA0A0X3PPT8_SCHSO\nDAPK3\tA0A151NX73_ALLMI\nDAPK3\tA0A1A7X0G3_9TELE\nDAPK3\tA0A1A8CTE4_9TELE\nDAPK3\tA0A1A8GJH4_9TELE\nDAPK3\tA0A1A8IGT5_NOTKU\nDAPK3\tA0A1A8PZJ6_9TELE\nDAPK3\tA0A1A8QYS5_9TELE\nDAPK3\tA0A1A8TYZ8_NOTFU\nDAPK3\tA0A1D5PRN8_CHICK\nDAPK3\tA0A1D5QUH7_MACMU\nDAPK3\tA0A1S3FP12_DIPOR\nDAPK3\tA0A1S3WDP2_ERIEU\nDAPK3\tA0A1U7QPF5_MESAU\nDAPK3\tA0A1U7RRC0_ALLSI\nDAPK3\tA0A1U7SZD8_TARSY\nDAPK3\tA0A1V4KQ22_PATFA\nDAPK3\tA0A1W2P6J6_MOUSE\nDAPK3\tA0A1W2P6K3_MOUSE\nDAPK3\tA0A1W2P730_MOUSE\nDAPK3\tA0A250YII4_CASCN\nDAPK3\tA0A286XZK0_CAVPO\nDAPK3\tA0A287AN27_PIG\nDAPK3\tA0A287DG04_ICTTR\nDAPK3\tA0A2I2UGG3_FELCA\nDAPK3\tA0A2I3LSS4_PAPAN\nDAPK3\tA0A2I3TG13_PANTR\nDAPK3\tA0A2I4B546_9TELE\nDAPK3\tA0A2K5DTE6_AOTNA\nDAPK3\tA0A2K5H920_COLAP\nDAPK3\tA0A2K5H986_COLAP\nDAPK3\tA0A2K5NLU3_CERAT\nDAPK3\tA0A2K5SBU9_CEBCA\nDAPK3\tA0A2K5SBV1_CEBCA\nDAPK3\tA0A2K5UM82_MACFA\nDAPK3\tA0A2K5UMA3_MACFA\nDAPK3\tA0A2K5Y0G7_MANLE\nDAPK3\tA0A2K6CCR2_MACNE\nDAPK3\tA0A2K6G821_PROCO\nDAPK3\tA0A2K6G825_PROCO\nDAPK3\tA0A2K6MN97_RHIBE\nDAPK3\tA0A2K6RX21_RHIRO\nDAPK3\tA0A2K6RX22_RHIRO\nDAPK3\tA0A2K6UMB9_SAIBB\nDAPK3\tA0A2R8MC23_CALJA\nDAPK3\tA0A2R8PM73_CALJA\nDAPK3\tA0A2R9A7I4_PANPA\nDAPK3\tA0A2R9A7I9_PANPA\nDAPK3\tA0A2U3VKF3_ODORO\nDAPK3\tA0A2U3Y4Y3_LEPWE\nDAPK3\tA0A2Y9EV55_PHYMC\nDAPK3\tA0A2Y9IHP7_NEOSC\nDAPK3\tA0A2Y9LTT8_DELLE\nDAPK3\tA0A340XGL8_LIPVE\nDAPK3\tA0A341BJ12_NEOAA\nDAPK3\tA0A341BJ71_NEOAA\nDAPK3\tA0A383Z2C1_BALAS\nDAPK3\tA0A384DRE1_URSMA\nDAPK3\tA0A3B3HH00_ORYLA\nDAPK3\tA0A3Q0DC95_MESAU\nDAPK3\tA0A3Q1LUP5_BOVIN\nDAPK3\tA0A3Q2H206_HORSE\nDAPK3\tA0A3Q7NB64_CALUR\nDAPK3\tA0A3Q7TCX9_VULVU\nDAPK3\tA0A3Q7TKY8_URSAR\nDAPK3\tA0A3Q7VG35_URSAR\nDAPK3\tA0A452F2U1_CAPHI\nDAPK3\tA0A452F3E9_CAPHI\nDAPK3\tA0A452GH28_9SAUR\nDAPK3\tA0A452SFE1_URSAM\nDAPK3\tA0A452TNM9_URSMA\nDAPK3\tA0A455C3X5_PHYMC\nDAPK3\tA0A455C3Y0_PHYMC\nDAPK3\tA0A493SXF7_ANAPP\nDAPK3\tA0A493T705_ANAPP\nDAPK3\tA0A4R8TT55_9PEZI\nDAPK3\tA0A4W2D5W3_BOBOX\nDAPK3\tA0A4W2D5W8_BOBOX\nDAPK3\tA0A4W2FUU4_BOBOX\nDAPK3\tA0A4W2HU93_BOBOX\nDAPK3\tA0A4W3JR04_CALMI\nDAPK3\tA0A4W4G5V2_ELEEL\nDAPK3\tA0A4W4G912_ELEEL\nDAPK3\tA0A4W6E505_LATCA\nDAPK3\tA0A4W6E7T3_LATCA\nDAPK3\tA0A4X2ME59_VOMUR\nDAPK3\tA0A5B7KFQ2_PORTR\nDAPK3\tA0A5F8HHR2_MONDO\nDAPK3\tA0A5G2RIF1_PIG\nDAPK3\tA0A5G3H7E5_XENTR\nDAPK3\tA0A5G3L7T8_XENTR\nDAPK3\tA7MB69_BOVIN\nDAPK3\tB7ZST5_XENTR\nDAPK3\tC0H9A6_SALSA\nDAPK3\tDAPK3_HUMAN\nDAPK3\tDAPK3_MOUSE\nDAPK3\tDAPK3_RAT\nDAPK3\tE2QVH6_CANLF\nDAPK3\tE9QHI3_DANRE\nDAPK3\tF6SGZ8_HORSE\nDAPK3\tF7AED4_ORNAN\nDAPK3\tF7AKL8_XENTR\nDAPK3\tF7AKU7_XENTR\nDAPK3\tF7EKI7_CALJA\nDAPK3\tF7GC28_MACMU\nDAPK3\tG1LHK7_AILME\nDAPK3\tG1P6K0_MYOLU\nDAPK3\tG1QRN6_NOMLE\nDAPK3\tG2HGB3_PANTR\nDAPK3\tG3TYC3_LOXAF\nDAPK3\tG5BZB9_HETGA\nDAPK3\tH0X2D2_OTOGA\nDAPK3\tH2MUM3_ORYLA\nDAPK3\tH2NX15_PONAB\nDAPK3\tH2STS3_TAKRU\nDAPK3\tI0FPJ9_MACMU\nDAPK3\tI3KQN7_ORENI\nDAPK3\tI3KQN8_ORENI\nDAPK3\tI3M5C2_ICTTR\nDAPK3\tK7ANU7_PANTR\nDAPK3\tK7FVU7_PELSI\nDAPK3\tM0QYW5_HUMAN\nDAPK3\tM0QYY8_HUMAN\nDAPK3\tM0R0D0_HUMAN\nDAPK3\tM3V830_PIG\nDAPK3\tM3WIZ2_FELCA\nDAPK3\tM3XZD2_MUSPF\nDAPK3\tQ05A21_MOUSE\nDAPK3\tQ28F04_XENTR\nDAPK3\tQ4V808_XENLA\nDAPK3\tR7VPW6_COLLI\nDAPK3\tU3CW76_CALJA\nDAPK3\tU3EB78_CALJA\nDAPK3\tU3EE09_CALJA\nDAPK3\tU3JDM8_FICAL\nDAPK3\tU6DVC6_NEOVI\nDAPK3\tV8N6U8_OPHHA\nDAPK3\tV8N7K6_OPHHA\nDAPK3\tW5PNP6_SHEEP\nDAPK3\tW5U6K9_ICTPU\n','From\tTo\nDAPK1\tA0A096P1X2_PAPAN\nDAPK1\tA0A0B8RZM4_PIG\nDAPK1\tA0A0C9RWW0_9HYME\nDAPK1\tA0A0D9R4X3_CHLSB\nDAPK1\tA0A0G2K5V8_RAT\nDAPK1\tA0A0P6J4K7_HETGA\nDAPK1\tA0A0S7EGX8_9TELE\nDAPK1\tA0A0S7ELH5_9TELE\nDAPK1\tA0A0S7HJT6_9TELE\nDAPK1\tA0A0S7HP36_9TELE\nDAPK1\tA0A0S7I6K5_9TELE\nDAPK1\tA0A0S7IAF2_9TELE\nDAPK1\tA0A0S7IGZ5_9TELE\nDAPK1\tA0A0S7IJW8_9TELE\nDAPK1\tA0A0S7JSE9_9TELE\nDAPK1\tA0A0V0SND3_9BILA\nDAPK1\tA0A0V1AFK4_9BILA\nDAPK1\tA0A0V1AGJ0_9BILA\nDAPK1\tA0A0V1LT80_9BILA\nDAPK1\tA0A0X3NMB4_SCHSO\nDAPK1\tA0A0X3NY06_SCHSO\nDAPK1\tA0A0X3P0J1_SCHSO\nDAPK1\tA0A0X3P4F7_SCHSO\nDAPK1\tA0A0X3PEL8_SCHSO\nDAPK1\tA0A0X3PR14_SCHSO\nDAPK1\tA0A0X3Q5D1_SCHSO\nDAPK1\tA0A146L3Q0_LYGHE\nDAPK1\tA0A151M869_ALLMI\nDAPK1\tA0A1A7Z2X3_9TELE\nDAPK1\tA0A1A8ACG9_NOTFU\nDAPK1\tA0A1A8APP8_NOTFU\nDAPK1\tA0A1A8BM54_9TELE\nDAPK1\tA0A1A8BWK6_9TELE\nDAPK1\tA0A1A8CF52_9TELE\nDAPK1\tA0A1A8CI28_9TELE\nDAPK1\tA0A1A8DNM3_9TELE\nDAPK1\tA0A1A8F037_9TELE\nDAPK1\tA0A1A8FGJ4_9TELE\nDAPK1\tA0A1A8FXW6_9TELE\nDAPK1\tA0A1A8G6B0_9TELE\nDAPK1\tA0A1A8HSS5_NOTKU\nDAPK1\tA0A1A8HZX3_NOTKU\nDAPK1\tA0A1A8MMD5_9TELE\nDAPK1\tA0A1A8N2G5_9TELE\nDAPK1\tA0A1A8ND07_9TELE\nDAPK1\tA0A1A8PLV3_9TELE\nDAPK1\tA0A1A8PRE5_9TELE\nDAPK1\tA0A1A8RSA5_9TELE\nDAPK1\tA0A1D5NT66_CHICK\nDAPK1\tA0A1D5PI81_CHICK\nDAPK1\tA0A1D5RJR8_MACMU\nDAPK1\tA0A1L8HPE3_XENLA\nDAPK1\tA0A1Q9CMW3_SYMMI\nDAPK1\tA0A1Q9CN23_SYMMI\nDAPK1\tA0A1Q9DPL8_SYMMI\nDAPK1\tA0A1Q9F3I8_SYMMI\nDAPK1\tA0A1S3A1L3_ERIEU\nDAPK1\tA0A1S3GGX0_DIPOR\nDAPK1\tA0A1S3M7U9_SALSA\nDAPK1\tA0A1U7QGH0_MESAU\nDAPK1\tA0A1U7RI93_ALLSI\nDAPK1\tA0A1U7TJS8_TARSY\nDAPK1\tA0A1V4KET3_PATFA\nDAPK1\tA0A218UNT5_9PASE\nDAPK1\tA0A286YDD8_MOUSE\nDAPK1\tA0A287B8E5_PIG\nDAPK1\tA0A287BKV4_PIG\nDAPK1\tA0A287D217_ICTTR\nDAPK1\tA0A2B4RB09_STYPI\nDAPK1\tA0A2D0PPC4_ICTPU\nDAPK1\tA0A2D0PQB6_ICTPU\nDAPK1\tA0A2I0MFR8_COLLI\nDAPK1\tA0A2I2UZ82_FELCA\nDAPK1\tA0A2I3G6V5_NOMLE\nDAPK1\tA0A2I3GY78_NOMLE\nDAPK1\tA0A2I3M4S5_PAPAN\nDAPK1\tA0A2I3N4T5_PAPAN\nDAPK1\tA0A2I3RT26_PANTR\nDAPK1\tA0A2I3S054_PANTR\nDAPK1\tA0A2J8KTK7_PANTR\nDAPK1\tA0A2K5E6T2_AOTNA\nDAPK1\tA0A2K5E726_AOTNA\nDAPK1\tA0A2K5E738_AOTNA\nDAPK1\tA0A2K5E746_AOTNA\nDAPK1\tA0A2K5E749_AOTNA\nDAPK1\tA0A2K5K1V3_COLAP\nDAPK1\tA0A2K5K1V8_COLAP\nDAPK1\tA0A2K5K1W9_COLAP\nDAPK1\tA0A2K5M5T8_CERAT\nDAPK1\tA0A2K5M5V4_CERAT\nDAPK1\tA0A2K5M5Y7_CERAT\nDAPK1\tA0A2K5PZ54_CEBCA\nDAPK1\tA0A2K5PZ73_CEBCA\nDAPK1\tA0A2K5PZD7_CEBCA\nDAPK1\tA0A2K5PZE3_CEBCA\nDAPK1\tA0A2K5W5Y9_MACFA\nDAPK1\tA0A2K5W610_MACFA\nDAPK1\tA0A2K5W619_MACFA\nDAPK1\tA0A2K5YSC4_MANLE\nDAPK1\tA0A2K5YSD1_MANLE\nDAPK1\tA0A2K5YSE2_MANLE\nDAPK1\tA0A2K6E5N0_MACNE\nDAPK1\tA0A2K6E5N6_MACNE\nDAPK1\tA0A2K6E5P4_MACNE\nDAPK1\tA0A2K6F4K5_PROCO\nDAPK1\tA0A2K6F4L3_PROCO\nDAPK1\tA0A2K6F4L8_PROCO\nDAPK1\tA0A2K6M595_RHIBE\nDAPK1\tA0A2K6M596_RHIBE\nDAPK1\tA0A2K6M599_RHIBE\nDAPK1\tA0A2K6M5A7_RHIBE\nDAPK1\tA0A2K6RFR5_RHIRO\nDAPK1\tA0A2K6RFS9_RHIRO\nDAPK1\tA0A2K6TPE5_SAIBB\nDAPK1\tA0A2K6TPF7_SAIBB\nDAPK1\tA0A2K6TPG8_SAIBB\nDAPK1\tA0A2K6TPH3_SAIBB\nDAPK1\tA0A2K6TPH4_SAIBB\nDAPK1\tA0A2N6N961_BEABA\nDAPK1\tA0A2P8YA61_BLAGE\nDAPK1\tA0A2R8MWT6_CALJA\nDAPK1\tA0A2R8QS56_DANRE\nDAPK1\tA0A2R9BC17_PANPA\nDAPK1\tA0A2R9BFU1_PANPA\nDAPK1\tA0A2R9BFU5_PANPA\nDAPK1\tA0A2U3ZRE7_ODORO\nDAPK1\tA0A2U4BRY4_TURTR\nDAPK1\tA0A2U4BS84_TURTR\nDAPK1\tA0A2Y9G3K5_NEOSC\nDAPK1\tA0A2Y9LZ28_DELLE\nDAPK1\tA0A2Y9M6B0_DELLE\nDAPK1\tA0A2Y9TIG9_PHYMC\nDAPK1\tA0A340WFH3_LIPVE\nDAPK1\tA0A340WI00_LIPVE\nDAPK1\tA0A341C555_NEOAA\nDAPK1\tA0A341C5J2_NEOAA\nDAPK1\tA0A371H9E6_MUCPR\nDAPK1\tA0A383ZPI8_BALAS\nDAPK1\tA0A383ZPJ5_BALAS\nDAPK1\tA0A383ZPJ9_BALAS\nDAPK1\tA0A383ZQ33_BALAS\nDAPK1\tA0A384DSI7_URSMA\nDAPK1\tA0A384DTB0_URSMA\nDAPK1\tA0A385LUT7_HELAM\nDAPK1\tA0A3B3HED4_ORYLA\nDAPK1\tA0A3B3IPN6_ORYLA\nDAPK1\tA0A3B5K0G1_TAKRU\nDAPK1\tA0A3P9DTG1_9CICH\nDAPK1\tA0A3Q0D323_MESAU\nDAPK1\tA0A3Q0E3S7_TARSY\nDAPK1\tA0A3Q0E5E2_TARSY\nDAPK1\tA0A3Q0E7A2_TARSY\nDAPK1\tA0A3Q1NM57_BOVIN\nDAPK1\tA0A3Q2HFW5_HORSE\nDAPK1\tA0A3Q2I4Q9_HORSE\nDAPK1\tA0A3Q2IA50_HORSE\nDAPK1\tA0A3Q2L7T3_HORSE\nDAPK1\tA0A3Q2LTA0_HORSE\nDAPK1\tA0A3Q3AMT9_CHICK\nDAPK1\tA0A3Q7MKB5_CALUR\nDAPK1\tA0A3Q7NVL6_CALUR\nDAPK1\tA0A3Q7R3U4_VULVU\nDAPK1\tA0A3Q7Y2C1_URSAR\nDAPK1\tA0A3R7GTS0_CLOSI\nDAPK1\tA0A3R7JIK1_CLOSI\nDAPK1\tA0A438J273_VITVI\nDAPK1\tA0A452EGR4_CAPHI\nDAPK1\tA0A452EGS9_CAPHI\nDAPK1\tA0A452EGT5_CAPHI\nDAPK1\tA0A452EH42_CAPHI\nDAPK1\tA0A452EHC1_CAPHI\nDAPK1\tA0A452GRD9_9SAUR\nDAPK1\tA0A452GRZ8_9SAUR\nDAPK1\tA0A452GS36_9SAUR\nDAPK1\tA0A452GS37_9SAUR\nDAPK1\tA0A452GSG5_9SAUR\nDAPK1\tA0A452TRF4_URSMA\nDAPK1\tA0A452TRG9_URSMA\nDAPK1\tA0A452TRH3_URSMA\nDAPK1\tA0A452TRJ9_URSMA\nDAPK1\tA0A493U044_ANAPP\nDAPK1\tA0A493U3E6_ANAPP\nDAPK1\tA0A4R8TQ67_9PEZI\nDAPK1\tA0A4W2DQ20_BOBOX\nDAPK1\tA0A4W2G805_BOBOX\nDAPK1\tA0A4W2GBQ9_BOBOX\nDAPK1\tA0A4W6FIR6_LATCA\nDAPK1\tA0A4W6FK20_LATCA\nDAPK1\tA0A4W6FK24_LATCA\nDAPK1\tA0A4X1V291_PIG\nDAPK1\tA0A4X1V2B3_PIG\nDAPK1\tA0A4X1V3X4_PIG\nDAPK1\tA0A4X1V3Y4_PIG\nDAPK1\tA0A4X2KIF8_VOMUR\nDAPK1\tA0A4X2KIU1_VOMUR\nDAPK1\tA0A4Y2BMQ8_ARAVE\nDAPK1\tA0A5F4CIW2_CANLF\nDAPK1\tA0A5F4CL24_CANLF\nDAPK1\tA0A5F4D189_CANLF\nDAPK1\tA0A5F5PGZ0_HORSE\nDAPK1\tA0A5F5PQ24_HORSE\nDAPK1\tA0A5F5PTJ0_HORSE\nDAPK1\tA0A5F5PYC6_HORSE\nDAPK1\tA0A5F8AII9_MACMU\nDAPK1\tA0A5F8GAY0_MONDO\nDAPK1\tA0A5F8GEV1_MONDO\nDAPK1\tA0A5F9C196_RABIT\nDAPK1\tA0A5F9CHH7_RABIT\nDAPK1\tA0A5F9CSZ1_RABIT\nDAPK1\tA0A5F9DFH0_RABIT\nDAPK1\tA0A5F9DN44_RABIT\nDAPK1\tA0A5G3K8S3_XENTR\nDAPK1\tA0A5G3KXV9_XENTR\nDAPK1\tA0A5G3KZ04_XENTR\nDAPK1\tA0A5G3L012_XENTR\nDAPK1\tA0A5G3L031_XENTR\nDAPK1\tA0T398_HUMAN\nDAPK1\tA5WVL9_DANRE\nDAPK1\tA9ULF1_XENTR\nDAPK1\tB0JZD4_XENTR\nDAPK1\tB2L455_XENBO\nDAPK1\tB2L513_XENBO\nDAPK1\tB5DFH6_RAT\nDAPK1\tDAPK1_HUMAN\nDAPK1\tDAPK1_MOUSE\nDAPK1\tE1C3M5_CHICK\nDAPK1\tE2JE41_CANLF\nDAPK1\tF1LNN8_RAT\nDAPK1\tF1MRL0_BOVIN\nDAPK1\tF1S4J4_PIG\nDAPK1\tF6Y048_XENTR\nDAPK1\tF7A0A5_MONDO\nDAPK1\tF7BUF8_CALJA\nDAPK1\tF7DZT2_HORSE\nDAPK1\tF7HLL1_MACMU\nDAPK1\tF7HNL1_MACMU\nDAPK1\tF8WCQ3_HUMAN\nDAPK1\tG1KNU5_ANOCA\nDAPK1\tG1PKL0_MYOLU\nDAPK1\tG1QZR2_NOMLE\nDAPK1\tG1SJW2_RABIT\nDAPK1\tG3T4R5_LOXAF\nDAPK1\tG3TQ37_LOXAF\nDAPK1\tG7NGF7_MACMU\nDAPK1\tH0UV62_CAVPO\nDAPK1\tH0WIE4_OTOGA\nDAPK1\tH2N2S0_ORYLA\nDAPK1\tH2PSK8_PONAB\nDAPK1\tH2R0V4_PANTR\nDAPK1\tH2UU72_TAKRU\nDAPK1\tH3AVX6_LATCH\nDAPK1\tI3K9T9_ORENI\nDAPK1\tI3K9U0_ORENI\nDAPK1\tI3LU37_PIG\nDAPK1\tI3MPU2_ICTTR\nDAPK1\tK1C18_DANRE\nDAPK1\tK7D1C4_PANTR\nDAPK1\tK7F754_PELSI\nDAPK1\tK7F757_PELSI\nDAPK1\tM3W3G8_FELCA\nDAPK1\tM3XSX4_MUSPF\nDAPK1\tQ3UKP8_MOUSE\nDAPK1\tQ6DDZ1_XENLA\nDAPK1\tQ6NRK5_XENLA\nDAPK1\tU3EA50_CALJA\nDAPK1\tU3J4W9_ANAPP\nDAPK1\tU3JLN1_FICAL\nDAPK1\tU6DNI2_NEOVI\nDAPK1\tV8NL18_OPHHA\nDAPK1\tW5PFX9_SHEEP\n','From\tTo\nCHEK2\tA0A087X102_HUMAN\nCHEK2\tA0A096NRH9_PAPAN\nCHEK2\tA0A096NRI0_PAPAN\nCHEK2\tA0A0C9RB38_9HYME\nCHEK2\tA0A0G2JDS0_MOUSE\nCHEK2\tA0A0G2JDU5_MOUSE\nCHEK2\tA0A0G2JGG6_MOUSE\nCHEK2\tA0A0P6J943_HETGA\nCHEK2\tA0A151NKN3_ALLMI\nCHEK2\tA0A151NKW7_ALLMI\nCHEK2\tA0A1A7XUX3_9TELE\nCHEK2\tA0A1A7Y2X0_9TELE\nCHEK2\tA0A1A8A471_NOTFU\nCHEK2\tA0A1A8H7W3_9TELE\nCHEK2\tA0A1A8JDE5_NOTKU\nCHEK2\tA0A1A8K730_NOTKU\nCHEK2\tA0A1A8LFC1_9TELE\nCHEK2\tA0A1A8NJI7_9TELE\nCHEK2\tA0A1A8UT82_NOTFU\nCHEK2\tA0A1D5PAM1_CHICK\nCHEK2\tA0A1Q9DU52_SYMMI\nCHEK2\tA0A1Q9EHR6_SYMMI\nCHEK2\tA0A1S3EZ79_DIPOR\nCHEK2\tA0A1S3WJK1_ERIEU\nCHEK2\tA0A1U7RJ66_ALLSI\nCHEK2\tA0A1U7TGB1_TARSY\nCHEK2\tA0A1U7TKF0_TARSY\nCHEK2\tA0A1U7TT62_TARSY\nCHEK2\tA0A1U7TUY0_TARSY\nCHEK2\tA0A1U8CXH2_MESAU\nCHEK2\tA0A1V4JRT3_PATFA\nCHEK2\tA0A218V5P2_9PASE\nCHEK2\tA0A286ZMG9_PIG\nCHEK2\tA0A287A7U1_PIG\nCHEK2\tA0A287B5S2_PIG\nCHEK2\tA0A2B4RWE0_STYPI\nCHEK2\tA0A2D0PLT9_ICTPU\nCHEK2\tA0A2I0M8C8_COLLI\nCHEK2\tA0A2I0M8D5_COLLI\nCHEK2\tA0A2I2USC4_FELCA\nCHEK2\tA0A2I3GB49_NOMLE\nCHEK2\tA0A2I3GNG2_NOMLE\nCHEK2\tA0A2I3H2E8_NOMLE\nCHEK2\tA0A2I3HYH9_NOMLE\nCHEK2\tA0A2I3LF29_PAPAN\nCHEK2\tA0A2I3MWZ6_PAPAN\nCHEK2\tA0A2I3N6I7_PAPAN\nCHEK2\tA0A2I3RSK6_PANTR\nCHEK2\tA0A2I3TPQ6_PANTR\nCHEK2\tA0A2I4D9N2_9TELE\nCHEK2\tA0A2J8QN60_PANTR\nCHEK2\tA0A2J8QN95_PANTR\nCHEK2\tA0A2K5CE68_AOTNA\nCHEK2\tA0A2K5CEP9_AOTNA\nCHEK2\tA0A2K5CEQ7_AOTNA\nCHEK2\tA0A2K5CER1_AOTNA\nCHEK2\tA0A2K5HBP7_COLAP\nCHEK2\tA0A2K5HBT3_COLAP\nCHEK2\tA0A2K5HBU3_COLAP\nCHEK2\tA0A2K5HBU7_COLAP\nCHEK2\tA0A2K5HBV2_COLAP\nCHEK2\tA0A2K5MD71_CERAT\nCHEK2\tA0A2K5MD75_CERAT\nCHEK2\tA0A2K5MD84_CERAT\nCHEK2\tA0A2K5MDB2_CERAT\nCHEK2\tA0A2K5MDC1_CERAT\nCHEK2\tA0A2K5MDD0_CERAT\nCHEK2\tA0A2K5Q631_CEBCA\nCHEK2\tA0A2K5Q653_CEBCA\nCHEK2\tA0A2K5Q656_CEBCA\nCHEK2\tA0A2K5Q670_CEBCA\nCHEK2\tA0A2K5Q6A8_CEBCA\nCHEK2\tA0A2K5VBP2_MACFA\nCHEK2\tA0A2K5VC10_MACFA\nCHEK2\tA0A2K5VC13_MACFA\nCHEK2\tA0A2K5VC20_MACFA\nCHEK2\tA0A2K5VC29_MACFA\nCHEK2\tA0A2K5VC41_MACFA\nCHEK2\tA0A2K5Y3A2_MANLE\nCHEK2\tA0A2K5Y3B2_MANLE\nCHEK2\tA0A2K5Y3D7_MANLE\nCHEK2\tA0A2K6DW25_MACNE\nCHEK2\tA0A2K6DW40_MACNE\nCHEK2\tA0A2K6DW45_MACNE\nCHEK2\tA0A2K6DW50_MACNE\nCHEK2\tA0A2K6DW58_MACNE\nCHEK2\tA0A2K6DW63_MACNE\nCHEK2\tA0A2K6GKI8_PROCO\nCHEK2\tA0A2K6GKK2_PROCO\nCHEK2\tA0A2K6GKN4_PROCO\nCHEK2\tA0A2K6GKP1_PROCO\nCHEK2\tA0A2K6KRN0_RHIBE\nCHEK2\tA0A2K6KST4_RHIBE\nCHEK2\tA0A2K6KSU1_RHIBE\nCHEK2\tA0A2K6KSU3_RHIBE\nCHEK2\tA0A2K6KSV4_RHIBE\nCHEK2\tA0A2K6KSV7_RHIBE\nCHEK2\tA0A2K6Q1X9_RHIRO\nCHEK2\tA0A2K6Q1Z1_RHIRO\nCHEK2\tA0A2K6Q1Z9_RHIRO\nCHEK2\tA0A2K6Q202_RHIRO\nCHEK2\tA0A2K6Q208_RHIRO\nCHEK2\tA0A2K6UF92_SAIBB\nCHEK2\tA0A2S2QV50_9HEMI\nCHEK2\tA0A2U3VXA1_ODORO\nCHEK2\tA0A2U3VXI3_ODORO\nCHEK2\tA0A2U3YD03_LEPWE\nCHEK2\tA0A2U3ZFE3_ODORO\nCHEK2\tA0A2U4CKY9_TURTR\nCHEK2\tA0A2U4CKZ4_TURTR\nCHEK2\tA0A2Y9EK13_PHYMC\nCHEK2\tA0A2Y9I5M3_NEOSC\nCHEK2\tA0A2Y9LCS9_DELLE\nCHEK2\tA0A337SAT4_FELCA\nCHEK2\tA0A340XIG4_LIPVE\nCHEK2\tA0A340XM52_LIPVE\nCHEK2\tA0A340XPH4_LIPVE\nCHEK2\tA0A341DCJ2_NEOAA\nCHEK2\tA0A367JK48_RHIAZ\nCHEK2\tA0A367K2W6_RHIST\nCHEK2\tA0A383Z7H9_BALAS\nCHEK2\tA0A384BQQ5_URSMA\nCHEK2\tA0A384BQQ6_URSMA\nCHEK2\tA0A384BRP6_URSMA\nCHEK2\tA0A3B3HNS7_ORYLA\nCHEK2\tA0A3B3ITA7_HUMAN\nCHEK2\tA0A3Q2LKM3_HORSE\nCHEK2\tA0A3Q7N2M1_CALUR\nCHEK2\tA0A3Q7N5C1_CALUR\nCHEK2\tA0A3Q7QUX2_VULVU\nCHEK2\tA0A3Q7QXF6_VULVU\nCHEK2\tA0A3Q7R749_VULVU\nCHEK2\tA0A3Q7TWU9_URSAR\nCHEK2\tA0A3Q7UJX9_URSAR\nCHEK2\tA0A3S5ZPJ3_CHICK\nCHEK2\tA0A419PKE5_CLOSI\nCHEK2\tA0A452FUC8_CAPHI\nCHEK2\tA0A452R6H0_URSAM\nCHEK2\tA0A452R6K0_URSAM\nCHEK2\tA0A493TF29_ANAPP\nCHEK2\tA0A4P8PR71_HUMAN\nCHEK2\tA0A4V1ES41_HUMAN\nCHEK2\tA0A4V1L6R8_COCIT\nCHEK2\tA0A4W2E5N2_BOBOX\nCHEK2\tA0A4X1VI56_PIG\nCHEK2\tA0A4X1VI61_PIG\nCHEK2\tA0A4X1VI74_PIG\nCHEK2\tA0A4X2KUQ5_VOMUR\nCHEK2\tA0A4Z0XGT2_ARTOL\nCHEK2\tA0A4Z2HI11_9TELE\nCHEK2\tA0A5B7CSI6_PORTR\nCHEK2\tA0A5F5PJ40_HORSE\nCHEK2\tA0A5F5PS80_HORSE\nCHEK2\tA0A5F5Q4P4_HORSE\nCHEK2\tA0A5F5XMM2_FELCA\nCHEK2\tA0A5F9CET7_RABIT\nCHEK2\tA0A5F9D4A0_RABIT\nCHEK2\tA1EAT1_CHICK\nCHEK2\tB0BM27_XENTR\nCHEK2\tB0V1J5_DANRE\nCHEK2\tB7ZBF2_HUMAN\nCHEK2\tB7ZBF6_HUMAN\nCHEK2\tB7ZBF7_HUMAN\nCHEK2\tB7ZBF8_HUMAN\nCHEK2\tC1L4S6_SCHJA\nCHEK2\tC1L4S7_SCHJA\nCHEK2\tC9JFD7_HUMAN\nCHEK2\tCHK2_HUMAN\nCHEK2\tCHK2_MOUSE\nCHEK2\tF6R5E4_BOVIN\nCHEK2\tF6TMJ6_XENTR\nCHEK2\tF6VJA7_MACMU\nCHEK2\tF7B9A4_HORSE\nCHEK2\tF7CDL0_MONDO\nCHEK2\tF7FWX1_ORNAN\nCHEK2\tF7GL05_CALJA\nCHEK2\tF7HZ73_CALJA\nCHEK2\tF7IDD5_CALJA\nCHEK2\tF8WCV2_HUMAN\nCHEK2\tG1LPH2_AILME\nCHEK2\tG1N5B4_MELGA\nCHEK2\tG1PUY6_MYOLU\nCHEK2\tG1QPF8_NOMLE\nCHEK2\tG1TF11_RABIT\nCHEK2\tG3STB3_LOXAF\nCHEK2\tG3W8L3_SARHA\nCHEK2\tH0V769_CAVPO\nCHEK2\tH0WNV6_OTOGA\nCHEK2\tH0Y4V6_HUMAN\nCHEK2\tH0Y820_HUMAN\nCHEK2\tH0ZC01_TAEGU\nCHEK2\tH2P3X6_PONAB\nCHEK2\tH2UFT7_TAKRU\nCHEK2\tH2UFT8_TAKRU\nCHEK2\tH2ZRX2_LATCH\nCHEK2\tH7BZ30_HUMAN\nCHEK2\tH7C0V7_HUMAN\nCHEK2\tI3MHL4_ICTTR\nCHEK2\tK7CUS1_PANTR\nCHEK2\tK7D8Y1_PANTR\nCHEK2\tK7GD99_PELSI\nCHEK2\tM3WKK7_FELCA\nCHEK2\tM3YQ55_MUSPF\nCHEK2\tQ543W6_MOUSE\nCHEK2\tQ58DJ9_BOVIN\nCHEK2\tQ803E2_DANRE\nCHEK2\tQ90ZY5_DANRE\nCHEK2\tQ9I8V3_XENLA\nCHEK2\tQ9R019_RAT\nCHEK2\tT2MH34_HYDVU\nCHEK2\tU3IQ29_ANAPP\nCHEK2\tU3K124_FICAL\nCHEK2\tV8NHB2_OPHHA\nCHEK2\tW5NWR8_SHEEP\n','From\tTo\nCAMK1D\tA0A087WUR6_HUMAN\nCAMK1D\tA0A096P3F8_PAPAN\nCAMK1D\tA0A0D9RLA9_CHLSB\nCAMK1D\tA0A0P6JAR9_HETGA\nCAMK1D\tA0A151NPI1_ALLMI\nCAMK1D\tA0A1C7N3K6_9FUNG\nCAMK1D\tA0A1D5P814_CHICK\nCAMK1D\tA0A1D5R2J9_MACMU\nCAMK1D\tA0A1Q9DES4_SYMMI\nCAMK1D\tA0A1S2ZUD6_ERIEU\nCAMK1D\tA0A1S3F1M5_DIPOR\nCAMK1D\tA0A1V4K061_PATFA\nCAMK1D\tA0A218VB79_9PASE\nCAMK1D\tA0A286XG34_CAVPO\nCAMK1D\tA0A287DAC0_ICTTR\nCAMK1D\tA0A2B4RGG7_STYPI\nCAMK1D\tA0A2I0MH53_COLLI\nCAMK1D\tA0A2I3GP64_NOMLE\nCAMK1D\tA0A2I3RCY1_PANTR\nCAMK1D\tA0A2I3SMX4_PANTR\nCAMK1D\tA0A2I3T4J2_PANTR\nCAMK1D\tA0A2I4BXW1_9TELE\nCAMK1D\tA0A2K5EPA7_AOTNA\nCAMK1D\tA0A2K5EPB7_AOTNA\nCAMK1D\tA0A2K5HSP9_COLAP\nCAMK1D\tA0A2K5HSS0_COLAP\nCAMK1D\tA0A2K5LSL0_CERAT\nCAMK1D\tA0A2K5LSL3_CERAT\nCAMK1D\tA0A2K5QC83_CEBCA\nCAMK1D\tA0A2K5QC85_CEBCA\nCAMK1D\tA0A2K5QCA3_CEBCA\nCAMK1D\tA0A2K5QCF6_CEBCA\nCAMK1D\tA0A2K5WZ26_MACFA\nCAMK1D\tA0A2K5WZK2_MACFA\nCAMK1D\tA0A2K5XCD3_MANLE\nCAMK1D\tA0A2K5XCD9_MANLE\nCAMK1D\tA0A2K5XCF3_MANLE\nCAMK1D\tA0A2K6DCE4_MACNE\nCAMK1D\tA0A2K6DCG1_MACNE\nCAMK1D\tA0A2K6FNU4_PROCO\nCAMK1D\tA0A2K6FNU5_PROCO\nCAMK1D\tA0A2K6FNW8_PROCO\nCAMK1D\tA0A2K6MHJ6_RHIBE\nCAMK1D\tA0A2K6MHK5_RHIBE\nCAMK1D\tA0A2K6QRY2_RHIRO\nCAMK1D\tA0A2K6SFH1_SAIBB\nCAMK1D\tA0A2K6SFI3_SAIBB\nCAMK1D\tA0A2K6SFI4_SAIBB\nCAMK1D\tA0A2R9CEE6_PANPA\nCAMK1D\tA0A2R9CEF4_PANPA\nCAMK1D\tA0A2U3WJZ8_ODORO\nCAMK1D\tA0A2U4B6J6_TURTR\nCAMK1D\tA0A2Y9HL77_NEOSC\nCAMK1D\tA0A2Y9HLE6_NEOSC\nCAMK1D\tA0A2Y9NLN2_DELLE\nCAMK1D\tA0A2Y9NLN7_DELLE\nCAMK1D\tA0A2Y9NPW6_DELLE\nCAMK1D\tA0A2Y9NSU4_DELLE\nCAMK1D\tA0A2Y9NY85_DELLE\nCAMK1D\tA0A2Y9P1P7_DELLE\nCAMK1D\tA0A2Y9T4S3_PHYMC\nCAMK1D\tA0A2Y9T929_PHYMC\nCAMK1D\tA0A2Y9TBC4_PHYMC\nCAMK1D\tA0A337S5J2_FELCA\nCAMK1D\tA0A340X000_LIPVE\nCAMK1D\tA0A341AK28_NEOAA\nCAMK1D\tA0A383YX65_BALAS\nCAMK1D\tA0A383YX67_BALAS\nCAMK1D\tA0A383YXD1_BALAS\nCAMK1D\tA0A383YXR9_BALAS\nCAMK1D\tA0A384DA61_URSMA\nCAMK1D\tA0A3B3HK06_ORYLA\nCAMK1D\tA0A3Q0DF68_TARSY\nCAMK1D\tA0A3Q0DJE8_TARSY\nCAMK1D\tA0A3Q0DMP3_TARSY\nCAMK1D\tA0A3Q0GEP9_ALLSI\nCAMK1D\tA0A3Q0GI98_ALLSI\nCAMK1D\tA0A3Q2KUR9_HORSE\nCAMK1D\tA0A3Q2TYQ0_CHICK\nCAMK1D\tA0A3Q7MQ04_CALUR\nCAMK1D\tA0A3Q7MQA9_CALUR\nCAMK1D\tA0A3Q7MQB4_CALUR\nCAMK1D\tA0A3Q7TR86_VULVU\nCAMK1D\tA0A3Q7UE40_URSAR\nCAMK1D\tA0A3Q7UGG1_VULVU\nCAMK1D\tA0A3R7D3K4_CLOSI\nCAMK1D\tA0A452DU38_CAPHI\nCAMK1D\tA0A452DU49_CAPHI\nCAMK1D\tA0A452IDK2_9SAUR\nCAMK1D\tA0A452UAY6_URSMA\nCAMK1D\tA0A452UB88_URSMA\nCAMK1D\tA0A452UBF0_URSMA\nCAMK1D\tA0A455BTK4_PHYMC\nCAMK1D\tA0A493T843_ANAPP\nCAMK1D\tA0A493TK12_ANAPP\nCAMK1D\tA0A4W2E2L8_BOBOX\nCAMK1D\tA0A4W2HIX5_BOBOX\nCAMK1D\tA0A4W6F412_LATCA\nCAMK1D\tA0A4W6F6K7_LATCA\nCAMK1D\tA0A4W6F7T8_LATCA\nCAMK1D\tA0A4X1TEF6_PIG\nCAMK1D\tA0A4X1TGA1_PIG\nCAMK1D\tA0A4X1TGB0_PIG\nCAMK1D\tA0A4X2L2V2_VOMUR\nCAMK1D\tA0A5F4CKK3_CANLF\nCAMK1D\tA0A5F4CPH3_CANLF\nCAMK1D\tA0A5F5XEM5_FELCA\nCAMK1D\tA0A5F5XXT7_FELCA\nCAMK1D\tA0A5F9CWE6_RABIT\nCAMK1D\tA0A5G2QEK8_PIG\nCAMK1D\tA0A5G3J9A6_XENTR\nCAMK1D\tA1L290_DANRE\nCAMK1D\tB1AW58_MOUSE\nCAMK1D\tB5MF71_DANRE\nCAMK1D\tB5MF72_DANRE\nCAMK1D\tF1LVR4_RAT\nCAMK1D\tF1N063_BOVIN\nCAMK1D\tF1PD00_CANLF\nCAMK1D\tF1RUP9_PIG\nCAMK1D\tF6R272_MONDO\nCAMK1D\tF6UEC8_ORNAN\nCAMK1D\tF6Z6H1_XENTR\nCAMK1D\tF7BKP2_HORSE\nCAMK1D\tF7HCQ0_MACMU\nCAMK1D\tF7HID5_MACMU\nCAMK1D\tG1MH77_AILME\nCAMK1D\tG1MZS4_MELGA\nCAMK1D\tG1P8F5_MYOLU\nCAMK1D\tG1RNV3_NOMLE\nCAMK1D\tG1T1N2_RABIT\nCAMK1D\tG3SL66_LOXAF\nCAMK1D\tG3WJ71_SARHA\nCAMK1D\tH0UZW6_CAVPO\nCAMK1D\tH0WYR0_OTOGA\nCAMK1D\tH2N9S4_PONAB\nCAMK1D\tH2Q1M3_PANTR\nCAMK1D\tH3A3S7_LATCH\nCAMK1D\tH9FXH1_MACMU\nCAMK1D\tH9GEM9_ANOCA\nCAMK1D\tH9Z1J0_MACMU\nCAMK1D\tI3LNH4_PIG\nCAMK1D\tI3MRM4_ICTTR\nCAMK1D\tK7BDT2_PANTR\nCAMK1D\tK7CLC7_PANTR\nCAMK1D\tK7EW04_PONAB\nCAMK1D\tK7GBM4_PELSI\nCAMK1D\tKCC1D_HUMAN\nCAMK1D\tKCC1D_MOUSE\nCAMK1D\tM3WLG6_FELCA\nCAMK1D\tM3XPL3_MUSPF\nCAMK1D\tQ3TDM2_MOUSE\nCAMK1D\tQ3UH04_MOUSE\nCAMK1D\tQ5SQQ7_HUMAN\nCAMK1D\tQ8AYR3_XENLA\nCAMK1D\tQ8R193_MOUSE\nCAMK1D\tT2MA43_HYDVU\nCAMK1D\tU3C199_CALJA\nCAMK1D\tU3DAZ0_CALJA\nCAMK1D\tU3ICA5_ANAPP\nCAMK1D\tU3JGS4_FICAL\nCAMK1D\tV8NJ59_OPHHA\nCAMK1D\tW5Q1E0_SHEEP\nCAMK1D\tW5Q1E2_SHEEP\nCAMK1D\tW5UL44_ICTPU\n','From\tTo\nSGK1\tA0A096N3Y2_PAPAN\nSGK1\tA0A0A9YGB3_LYGHE\nSGK1\tA0A0D9RVZ9_CHLSB\nSGK1\tA0A0G2JVM7_RAT\nSGK1\tA0A0N8EUM4_HETGA\nSGK1\tA0A0S7FRZ6_9TELE\nSGK1\tA0A0S7FV58_9TELE\nSGK1\tA0A0S7LB25_9TELE\nSGK1\tA0A0S7LB29_9TELE\nSGK1\tA0A0S7LB33_9TELE\nSGK1\tA0A0S7LB58_9TELE\nSGK1\tA0A0S7LB91_9TELE\nSGK1\tA0A0S7M454_9TELE\nSGK1\tA0A0S7M4H2_9TELE\nSGK1\tA0A0S7M4Z0_9TELE\nSGK1\tA0A146L589_LYGHE\nSGK1\tA0A146L8X3_LYGHE\nSGK1\tA0A146WFF5_FUNHE\nSGK1\tA0A151M3D2_ALLMI\nSGK1\tA0A151M3G4_ALLMI\nSGK1\tA0A161I6U4_CONMO\nSGK1\tA0A1A7XFV1_9TELE\nSGK1\tA0A1A7Y262_9TELE\nSGK1\tA0A1A7ZL67_NOTFU\nSGK1\tA0A1A8D2F9_9TELE\nSGK1\tA0A1A8DAD1_9TELE\nSGK1\tA0A1A8ETM9_9TELE\nSGK1\tA0A1A8J6W5_NOTKU\nSGK1\tA0A1A8JUX9_NOTKU\nSGK1\tA0A1A8NQK1_9TELE\nSGK1\tA0A1A8NUY0_9TELE\nSGK1\tA0A1A8SMF1_9TELE\nSGK1\tA0A1J4JQ66_9EUKA\nSGK1\tA0A1J4K6M5_9EUKA\nSGK1\tA0A1J4KDB9_9EUKA\nSGK1\tA0A1J4KJ33_9EUKA\nSGK1\tA0A1J4KKB0_9EUKA\nSGK1\tA0A1J4KV33_9EUKA\nSGK1\tA0A1L8G2L0_XENLA\nSGK1\tA0A1S2ZIZ5_ERIEU\nSGK1\tA0A1S2ZIZ6_ERIEU\nSGK1\tA0A1S2ZIZ9_ERIEU\nSGK1\tA0A1S2ZJ04_ERIEU\nSGK1\tA0A1S3G0H6_DIPOR\nSGK1\tA0A1S3G2M0_DIPOR\nSGK1\tA0A1S3W734_ERIEU\nSGK1\tA0A1U7QED6_MESAU\nSGK1\tA0A1U7QGZ1_MESAU\nSGK1\tA0A1U7SW18_ALLSI\nSGK1\tA0A1U7V276_TARSY\nSGK1\tA0A1U7V3P2_TARSY\nSGK1\tA0A1V4JLQ6_PATFA\nSGK1\tA0A1V4JLW5_PATFA\nSGK1\tA0A1Y0F4D9_CONFG\nSGK1\tA0A218V717_9PASE\nSGK1\tA0A250YHG4_CASCN\nSGK1\tA0A286XVE3_CAVPO\nSGK1\tA0A286XX95_CAVPO\nSGK1\tA0A286ZMR5_PIG\nSGK1\tA0A286ZRM8_PIG\nSGK1\tA0A287A719_PIG\nSGK1\tA0A287D6T6_ICTTR\nSGK1\tA0A287D6X5_ICTTR\nSGK1\tA0A2B4S0K6_STYPI\nSGK1\tA0A2D0SL28_ICTPU\nSGK1\tA0A2D0SLP0_ICTPU\nSGK1\tA0A2I0MUK6_COLLI\nSGK1\tA0A2I0MUK7_COLLI\nSGK1\tA0A2I0MUL3_COLLI\nSGK1\tA0A2I2UTB1_FELCA\nSGK1\tA0A2I2V2C0_FELCA\nSGK1\tA0A2I3G826_NOMLE\nSGK1\tA0A2I3GN77_NOMLE\nSGK1\tA0A2I3GXX5_NOMLE\nSGK1\tA0A2I3H0Z4_NOMLE\nSGK1\tA0A2I3HBM3_NOMLE\nSGK1\tA0A2I3HFR8_NOMLE\nSGK1\tA0A2I3M866_PAPAN\nSGK1\tA0A2I3M8F8_PAPAN\nSGK1\tA0A2I3MAB4_PAPAN\nSGK1\tA0A2I3MDF8_PAPAN\nSGK1\tA0A2I3MLK8_PAPAN\nSGK1\tA0A2I3RLD7_PANTR\nSGK1\tA0A2I3SQ91_PANTR\nSGK1\tA0A2I3T8I3_PANTR\nSGK1\tA0A2I4C308_9TELE\nSGK1\tA0A2K5F181_AOTNA\nSGK1\tA0A2K5F1B3_AOTNA\nSGK1\tA0A2K5F1B5_AOTNA\nSGK1\tA0A2K5F1C1_AOTNA\nSGK1\tA0A2K5F1C9_AOTNA\nSGK1\tA0A2K5F1E9_AOTNA\nSGK1\tA0A2K5JG49_COLAP\nSGK1\tA0A2K5JG95_COLAP\nSGK1\tA0A2K5JGD4_COLAP\nSGK1\tA0A2K5JGF0_COLAP\nSGK1\tA0A2K5JGF7_COLAP\nSGK1\tA0A2K5JGH5_COLAP\nSGK1\tA0A2K5P1E8_CERAT\nSGK1\tA0A2K5P1F8_CERAT\nSGK1\tA0A2K5P1K9_CERAT\nSGK1\tA0A2K5P1N6_CERAT\nSGK1\tA0A2K5P1S4_CERAT\nSGK1\tA0A2K5RV22_CEBCA\nSGK1\tA0A2K5RV26_CEBCA\nSGK1\tA0A2K5RV29_CEBCA\nSGK1\tA0A2K5RV34_CEBCA\nSGK1\tA0A2K5RV37_CEBCA\nSGK1\tA0A2K5RV41_CEBCA\nSGK1\tA0A2K5RV42_CEBCA\nSGK1\tA0A2K5RV61_CEBCA\nSGK1\tA0A2K5U6V5_MACFA\nSGK1\tA0A2K5U6V8_MACFA\nSGK1\tA0A2K5U6W3_MACFA\nSGK1\tA0A2K5U6W9_MACFA\nSGK1\tA0A2K5U6Y3_MACFA\nSGK1\tA0A2K5U6Y4_MACFA\nSGK1\tA0A2K5U712_MACFA\nSGK1\tA0A2K5YMT6_MANLE\nSGK1\tA0A2K5YMU2_MANLE\nSGK1\tA0A2K5YMU8_MANLE\nSGK1\tA0A2K5YMV5_MANLE\nSGK1\tA0A2K5YMX4_MANLE\nSGK1\tA0A2K5YMY9_MANLE\nSGK1\tA0A2K6BV29_MACNE\nSGK1\tA0A2K6BV62_MACNE\nSGK1\tA0A2K6BV80_MACNE\nSGK1\tA0A2K6BV88_MACNE\nSGK1\tA0A2K6BVB2_MACNE\nSGK1\tA0A2K6BVB9_MACNE\nSGK1\tA0A2K6FZ52_PROCO\nSGK1\tA0A2K6FZ54_PROCO\nSGK1\tA0A2K6FZ58_PROCO\nSGK1\tA0A2K6FZ61_PROCO\nSGK1\tA0A2K6FZ76_PROCO\nSGK1\tA0A2K6FZ79_PROCO\nSGK1\tA0A2K6MC41_RHIBE\nSGK1\tA0A2K6MC57_RHIBE\nSGK1\tA0A2K6MC70_RHIBE\nSGK1\tA0A2K6MC92_RHIBE\nSGK1\tA0A2K6MCA3_RHIBE\nSGK1\tA0A2K6MCB3_RHIBE\nSGK1\tA0A2K6QRV5_RHIRO\nSGK1\tA0A2K6QRW0_RHIRO\nSGK1\tA0A2K6QRW5_RHIRO\nSGK1\tA0A2K6QRW7_RHIRO\nSGK1\tA0A2K6QRX0_RHIRO\nSGK1\tA0A2K6QRX5_RHIRO\nSGK1\tA0A2K6US82_SAIBB\nSGK1\tA0A2K6USA3_SAIBB\nSGK1\tA0A2K6USA6_SAIBB\nSGK1\tA0A2K6USB4_SAIBB\nSGK1\tA0A2K6USC6_SAIBB\nSGK1\tA0A2K6USC9_SAIBB\nSGK1\tA0A2N6NS80_BEABA\nSGK1\tA0A2R9CBV4_PANPA\nSGK1\tA0A2R9CBV5_PANPA\nSGK1\tA0A2R9CBX2_PANPA\nSGK1\tA0A2R9CBY5_PANPA\nSGK1\tA0A2R9CC20_PANPA\nSGK1\tA0A2R9CMS1_PANPA\nSGK1\tA0A2R9CMT6_PANPA\nSGK1\tA0A2U3VZY4_ODORO\nSGK1\tA0A2U3W036_ODORO\nSGK1\tA0A2U3W066_ODORO\nSGK1\tA0A2U3W077_ODORO\nSGK1\tA0A2U3YT65_LEPWE\nSGK1\tA0A2U3YT82_LEPWE\nSGK1\tA0A2U3YT83_LEPWE\nSGK1\tA0A2U3YTM0_LEPWE\nSGK1\tA0A2U3ZGI1_ODORO\nSGK1\tA0A2U3ZGI6_ODORO\nSGK1\tA0A2U4CNM0_TURTR\nSGK1\tA0A2Y9SPT4_PHYMC\nSGK1\tA0A2Y9SRL0_PHYMC\nSGK1\tA0A337S1I1_FELCA\nSGK1\tA0A337SGE8_FELCA\nSGK1\tA0A340WU93_LIPVE\nSGK1\tA0A340WVJ6_LIPVE\nSGK1\tA0A340WZD8_LIPVE\nSGK1\tA0A340X148_LIPVE\nSGK1\tA0A369KG96_HYPMA\nSGK1\tA0A384AU91_BALAS\nSGK1\tA0A384AU94_BALAS\nSGK1\tA0A384AU95_BALAS\nSGK1\tA0A384AUZ9_BALAS\nSGK1\tA0A384C368_URSMA\nSGK1\tA0A384C4D8_URSMA\nSGK1\tA0A3B3CBT2_ORYME\nSGK1\tA0A3B3CCL5_ORYME\nSGK1\tA0A3B3DNI7_ORYME\nSGK1\tA0A3B3INP7_ORYLA\nSGK1\tA0A3B3QMN2_9TELE\nSGK1\tA0A3B3QND4_9TELE\nSGK1\tA0A3B3QNG1_9TELE\nSGK1\tA0A3B3QNJ5_9TELE\nSGK1\tA0A3B3QNM3_9TELE\nSGK1\tA0A3B3QP29_9TELE\nSGK1\tA0A3B3QPQ4_9TELE\nSGK1\tA0A3B3TW26_9TELE\nSGK1\tA0A3B3TW35_9TELE\nSGK1\tA0A3B3XE79_9TELE\nSGK1\tA0A3B3XE89_9TELE\nSGK1\tA0A3B3YQP3_9TELE\nSGK1\tA0A3B4B027_9GOBI\nSGK1\tA0A3B4D621_PYGNA\nSGK1\tA0A3B4G3T4_9CICH\nSGK1\tA0A3B4G3U4_9CICH\nSGK1\tA0A3B4UGV0_SERDU\nSGK1\tA0A3B4UGZ4_SERDU\nSGK1\tA0A3B4UJP7_SERDU\nSGK1\tA0A3B4WN32_SERLL\nSGK1\tA0A3B4WNV5_SERLL\nSGK1\tA0A3B4WTD9_SERLL\nSGK1\tA0A3B4ZYG0_9TELE\nSGK1\tA0A3B5AH59_9TELE\nSGK1\tA0A3B5K3H2_TAKRU\nSGK1\tA0A3B5L3I3_9TELE\nSGK1\tA0A3B5PV02_XIPMA\nSGK1\tA0A3B5QG12_XIPMA\nSGK1\tA0A3B5QLT1_XIPMA\nSGK1\tA0A3P8PEV0_ASTCA\nSGK1\tA0A3P8PEW2_ASTCA\nSGK1\tA0A3P8PF10_ASTCA\nSGK1\tA0A3P8PF16_ASTCA\nSGK1\tA0A3P8SIP0_AMPPE\nSGK1\tA0A3P8W481_CYNSE\nSGK1\tA0A3P8W738_CYNSE\nSGK1\tA0A3P8W9K6_CYNSE\nSGK1\tA0A3P8XLB2_ESOLU\nSGK1\tA0A3P8ZL27_ESOLU\nSGK1\tA0A3P8ZM47_ESOLU\nSGK1\tA0A3P9CEX0_9CICH\nSGK1\tA0A3P9CEZ6_9CICH\nSGK1\tA0A3P9CFA9_9CICH\nSGK1\tA0A3P9P432_POERE\nSGK1\tA0A3P9P457_POERE\nSGK1\tA0A3Q0CC04_MESAU\nSGK1\tA0A3Q0DK19_TARSY\nSGK1\tA0A3Q0DLS8_TARSY\nSGK1\tA0A3Q0SFY6_AMPCI\nSGK1\tA0A3Q1BKE9_AMPOC\nSGK1\tA0A3Q1BKG0_AMPOC\nSGK1\tA0A3Q1CTY0_AMPOC\nSGK1\tA0A3Q1DDF0_AMPOC\nSGK1\tA0A3Q1HDF7_ANATE\nSGK1\tA0A3Q1HGV9_ANATE\nSGK1\tA0A3Q1HGZ5_ANATE\nSGK1\tA0A3Q1LSX1_BOVIN\nSGK1\tA0A3Q1M1M6_BOVIN\nSGK1\tA0A3Q1MGM4_BOVIN\nSGK1\tA0A3Q2C798_CYPVA\nSGK1\tA0A3Q2C7C5_CYPVA\nSGK1\tA0A3Q2HY27_HORSE\nSGK1\tA0A3Q2IEM2_HORSE\nSGK1\tA0A3Q2P2L1_FUNHE\nSGK1\tA0A3Q2U2M8_CHICK\nSGK1\tA0A3Q2UHJ7_CHICK\nSGK1\tA0A3Q2VIT4_HAPBU\nSGK1\tA0A3Q2VKG1_HAPBU\nSGK1\tA0A3Q2WSG8_HAPBU\nSGK1\tA0A3Q2WTG6_HAPBU\nSGK1\tA0A3Q2XZN0_HIPCM\nSGK1\tA0A3Q3AJC6_KRYMA\nSGK1\tA0A3Q3ARM6_CHICK\nSGK1\tA0A3Q3D8Q5_HIPCM\nSGK1\tA0A3Q3D8R7_HIPCM\nSGK1\tA0A3Q3E7S0_9LABR\nSGK1\tA0A3Q3E7T6_9LABR\nSGK1\tA0A3Q3E7X2_9LABR\nSGK1\tA0A3Q3EBM7_9LABR\nSGK1\tA0A3Q3JFU4_MONAL\nSGK1\tA0A3Q3JMW0_MONAL\nSGK1\tA0A3Q3JMX5_MONAL\nSGK1\tA0A3Q3MHQ8_9TELE\nSGK1\tA0A3Q3QM64_MONAL\nSGK1\tA0A3Q3SHH4_9TELE\nSGK1\tA0A3Q3WY18_MOLML\nSGK1\tA0A3Q4HR97_NEOBR\nSGK1\tA0A3Q4HVI5_NEOBR\nSGK1\tA0A3Q4N3T6_NEOBR\nSGK1\tA0A3Q7NIU2_CALUR\nSGK1\tA0A3Q7NIU9_CALUR\nSGK1\tA0A3Q7NIV5_CALUR\nSGK1\tA0A3Q7S4C6_VULVU\nSGK1\tA0A3Q7SC23_VULVU\nSGK1\tA0A3Q7SEL6_VULVU\nSGK1\tA0A3Q7T165_VULVU\nSGK1\tA0A3Q7W6V2_URSAR\nSGK1\tA0A3Q7WGB4_URSAR\nSGK1\tA0A3Q7X202_URSAR\nSGK1\tA0A3R7K115_CLOSI\nSGK1\tA0A452E385_CAPHI\nSGK1\tA0A452E3B6_CAPHI\nSGK1\tA0A452IQW4_9SAUR\nSGK1\tA0A452IR84_9SAUR\nSGK1\tA0A452IRG5_9SAUR\nSGK1\tA0A452QRA0_URSAM\nSGK1\tA0A452QRA3_URSAM\nSGK1\tA0A452QRC1_URSAM\nSGK1\tA0A452QRD2_URSAM\nSGK1\tA0A452QRE0_URSAM\nSGK1\tA0A452QRE4_URSAM\nSGK1\tA0A493STP2_ANAPP\nSGK1\tA0A493SUZ6_ANAPP\nSGK1\tA0A493T1T2_ANAPP\nSGK1\tA0A493U3L4_ANAPP\nSGK1\tA0A4R8Q8C4_9PEZI\nSGK1\tA0A4R8RC27_COLTR\nSGK1\tA0A4W2CRZ4_BOBOX\nSGK1\tA0A4W2CVY8_BOBOX\nSGK1\tA0A4W2DB67_BOBOX\nSGK1\tA0A4W2H6U7_BOBOX\nSGK1\tA0A4W2IJR4_BOBOX\nSGK1\tA0A4W2IJX4_BOBOX\nSGK1\tA0A4W4ERQ0_ELEEL\nSGK1\tA0A4W4ERT2_ELEEL\nSGK1\tA0A4W4ES20_ELEEL\nSGK1\tA0A4W4ES32_ELEEL\nSGK1\tA0A4W4ESK5_ELEEL\nSGK1\tA0A4W4ETF0_ELEEL\nSGK1\tA0A4W4ETF4_ELEEL\nSGK1\tA0A4W4EUA9_ELEEL\nSGK1\tA0A4W4EUC1_ELEEL\nSGK1\tA0A4W6EAW3_LATCA\nSGK1\tA0A4W6EAW8_LATCA\nSGK1\tA0A4X1V9R0_PIG\nSGK1\tA0A4X1V9R3_PIG\nSGK1\tA0A4X1VDA7_PIG\nSGK1\tA0A4X1VFV7_PIG\nSGK1\tA0A4X1VG12_PIG\nSGK1\tA0A4X2K0E5_VOMUR\nSGK1\tA0A4X2K0H2_VOMUR\nSGK1\tA0A4X2K823_VOMUR\nSGK1\tA0A4X2K868_VOMUR\nSGK1\tA0A4Z2FTE7_9TELE\nSGK1\tA0A4Z2IZC0_9TELE\nSGK1\tA0A5B7DXZ0_PORTR\nSGK1\tA0A5F4BS33_CANLF\nSGK1\tA0A5F4CGK0_CANLF\nSGK1\tA0A5F4CNK4_CANLF\nSGK1\tA0A5F4D1F5_CANLF\nSGK1\tA0A5F4DBG3_CANLF\nSGK1\tA0A5F5PMZ2_HORSE\nSGK1\tA0A5F5PNL0_HORSE\nSGK1\tA0A5F5XE60_FELCA\nSGK1\tA0A5F5XKK3_FELCA\nSGK1\tA0A5F8GZ56_MONDO\nSGK1\tA0A5F8HAH1_MONDO\nSGK1\tA0A5F8HJ36_MONDO\nSGK1\tA0A5F9CAX8_RABIT\nSGK1\tA0A5F9CN92_RABIT\nSGK1\tA0A5F9CNA9_RABIT\nSGK1\tA0A5F9CXY9_RABIT\nSGK1\tA0A5F9DHT6_RABIT\nSGK1\tA0A5G3H2B5_XENTR\nSGK1\tA0A5G3I8Q6_XENTR\nSGK1\tA0A5G3I8S4_XENTR\nSGK1\tA0A5G3I8T1_XENTR\nSGK1\tA0A5G3I8U7_XENTR\nSGK1\tA0A5G3I940_XENTR\nSGK1\tA0A5G3IA53_XENTR\nSGK1\tA0A5G3IC07_XENTR\nSGK1\tA0A5K1VLQ6_MACMU\nSGK1\tC0H8Y8_SALSA\nSGK1\tC0L096_CAVPO\nSGK1\tC0PUD5_SALSA\nSGK1\tD2HJ43_AILME\nSGK1\tD3YV51_MOUSE\nSGK1\tD3YXA3_MOUSE\nSGK1\tD3Z0E8_MOUSE\nSGK1\tD4A128_RAT\nSGK1\tE9PJN2_HUMAN\nSGK1\tE9PP33_HUMAN\nSGK1\tE9PR89_HUMAN\nSGK1\tE9Q1B0_MOUSE\nSGK1\tE9Q8C1_MOUSE\nSGK1\tE9QEW2_DANRE\nSGK1\tE9QI98_DANRE\nSGK1\tE9QIF7_DANRE\nSGK1\tF1LXA3_RAT\nSGK1\tF1PA42_CANLF\nSGK1\tF6R5T0_XENTR\nSGK1\tF6RKM6_MOUSE\nSGK1\tF6USQ9_XENTR\nSGK1\tF6XCT5_XENTR\nSGK1\tF6YI88_ORNAN\nSGK1\tF7BY67_HORSE\nSGK1\tF7C4I0_XENTR\nSGK1\tF7F0V9_MONDO\nSGK1\tF7GPM7_MACMU\nSGK1\tF7GPM9_MACMU\nSGK1\tF7HWA9_CALJA\nSGK1\tF7HYN1_CALJA\nSGK1\tF7I0Y2_CALJA\nSGK1\tF7I0Y6_CALJA\nSGK1\tG1MC21_AILME\nSGK1\tG1NKB1_MELGA\nSGK1\tG1PVE1_MYOLU\nSGK1\tG1RXA9_NOMLE\nSGK1\tG3PAC8_GASAC\nSGK1\tG3PAD0_GASAC\nSGK1\tG3PAD3_GASAC\nSGK1\tG3T9X1_LOXAF\nSGK1\tG3U4Z7_LOXAF\nSGK1\tG3W1X6_SARHA\nSGK1\tH0UWC0_CAVPO\nSGK1\tH0WZ40_OTOGA\nSGK1\tH0YCJ3_HUMAN\nSGK1\tH0ZMR5_TAEGU\nSGK1\tH2MPM1_ORYLA\nSGK1\tH2MPM3_ORYLA\nSGK1\tH2PKD8_PONAB\nSGK1\tH2R6X6_PANTR\nSGK1\tH2S244_TAKRU\nSGK1\tH2S245_TAKRU\nSGK1\tH2ZUF7_LATCH\nSGK1\tH3DFX2_TETNG\nSGK1\tH9GG89_ANOCA\nSGK1\tI0FU80_MACMU\nSGK1\tI3IVD3_ORENI\nSGK1\tI3IVD4_ORENI\nSGK1\tI3LS78_PIG\nSGK1\tI3MS62_ICTTR\nSGK1\tJ9NWR6_CANLF\nSGK1\tK4HA68_9SAUR\nSGK1\tK4HA71_9SAUR\nSGK1\tK4HAL3_9SAUR\nSGK1\tK4HAL7_9SAUR\nSGK1\tK4HAM1_9SAUR\nSGK1\tK4HBP8_APAFE\nSGK1\tK4HBQ2_9SAUR\nSGK1\tK4HBQ5_9SAUR\nSGK1\tK4HDS8_9PICI\nSGK1\tK4HDT0_MALTE\nSGK1\tK4HDT2_9SAUR\nSGK1\tK4HFP0_9SAUR\nSGK1\tK4HFP1_9SAUR\nSGK1\tK4HFP2_9SAUR\nSGK1\tK7BJD5_PANTR\nSGK1\tK7C1X8_PANTR\nSGK1\tK7CFP7_PANTR\nSGK1\tK7ED10_ORNAN\nSGK1\tK7F7D8_PELSI\nSGK1\tM3WJM4_FELCA\nSGK1\tM3YTN6_MUSPF\nSGK1\tM3ZQ62_XIPMA\nSGK1\tN4V6L0_COLOR\nSGK1\tQ68G05_RAT\nSGK1\tQ7Z3I4_HUMAN\nSGK1\tQ95N98_CANLF\nSGK1\tQ99LU4_MOUSE\nSGK1\tR4I3I2_HUMAN\nSGK1\tSGK1_BOVIN\nSGK1\tSGK1_CHICK\nSGK1\tSGK1_DANRE\nSGK1\tSGK1_FUNHE\nSGK1\tSGK1_HUMAN\nSGK1\tSGK1_MACFA\nSGK1\tSGK1_MOUSE\nSGK1\tSGK1_RABIT\nSGK1\tSGK1_RAT\nSGK1\tSGK1_XENTR\nSGK1\tU3DA47_CALJA\nSGK1\tU3DK67_CALJA\nSGK1\tU3IK42_ANAPP\nSGK1\tU3KBK9_FICAL\nSGK1\tV8NXC2_OPHHA\nSGK1\tW5NH97_LEPOC\nSGK1\tW5NH98_LEPOC\nSGK1\tW5PZ92_SHEEP\n','From\tTo\nACVR2A\tA0A096NSM4_PAPAN\nACVR2A\tA0A096NSM5_PAPAN\nACVR2A\tA0A0B2VYD1_TOXCA\nACVR2A\tA0A0D9RUH1_CHLSB\nACVR2A\tA0A0P6JI96_HETGA\nACVR2A\tA0A0V0S629_9BILA\nACVR2A\tA0A0V0S645_9BILA\nACVR2A\tA0A0V0UH63_9BILA\nACVR2A\tA0A0V0WV52_9BILA\nACVR2A\tA0A0V0WVB0_9BILA\nACVR2A\tA0A0V0WVV4_9BILA\nACVR2A\tA0A0V0YIY5_TRIPS\nACVR2A\tA0A0V0ZUA8_9BILA\nACVR2A\tA0A0V0ZV51_9BILA\nACVR2A\tA0A0V1BSA0_TRISP\nACVR2A\tA0A0V1D8Q1_TRIBR\nACVR2A\tA0A0V1G580_TRIPS\nACVR2A\tA0A0V1HXJ9_9BILA\nACVR2A\tA0A0V1J8N5_TRIPS\nACVR2A\tA0A0V1K8B9_TRIPS\nACVR2A\tA0A0V1LQ10_9BILA\nACVR2A\tA0A0V1MK07_9BILA\nACVR2A\tA0A0V1MLH7_9BILA\nACVR2A\tA0A0V1PLB9_9BILA\nACVR2A\tA0A0V1PLG5_9BILA\nACVR2A\tA0A151M0J5_ALLMI\nACVR2A\tA0A191VKI7_SHEEP\nACVR2A\tA0A1L8EVT5_XENLA\nACVR2A\tA0A1S2ZNW0_ERIEU\nACVR2A\tA0A1S2ZNX0_ERIEU\nACVR2A\tA0A1S3GB61_DIPOR\nACVR2A\tA0A1S3GB71_DIPOR\nACVR2A\tA0A1U7Q9T3_MESAU\nACVR2A\tA0A1U7SB41_ALLSI\nACVR2A\tA0A1U7TWM1_TARSY\nACVR2A\tA0A1U8BKP3_MESAU\nACVR2A\tA0A1V4K5E6_PATFA\nACVR2A\tA0A1V4K5F7_PATFA\nACVR2A\tA0A218UES0_9PASE\nACVR2A\tA0A250Y2A8_CASCN\nACVR2A\tA0A287A331_PIG\nACVR2A\tA0A287CV02_ICTTR\nACVR2A\tA0A2D0S232_ICTPU\nACVR2A\tA0A2I0MG51_COLLI\nACVR2A\tA0A2I2URP7_FELCA\nACVR2A\tA0A2I4D5N2_9TELE\nACVR2A\tA0A2J8V7M3_PONAB\nACVR2A\tA0A2K5DVV2_AOTNA\nACVR2A\tA0A2K5DVX1_AOTNA\nACVR2A\tA0A2K5DVZ9_AOTNA\nACVR2A\tA0A2K5HMQ2_COLAP\nACVR2A\tA0A2K5HMS7_COLAP\nACVR2A\tA0A2K5M3P0_CERAT\nACVR2A\tA0A2K5M3R5_CERAT\nACVR2A\tA0A2K5S090_CEBCA\nACVR2A\tA0A2K5S0E7_CEBCA\nACVR2A\tA0A2K5Y373_MANLE\nACVR2A\tA0A2K5Y375_MANLE\nACVR2A\tA0A2K6DZY5_MACNE\nACVR2A\tA0A2K6F959_PROCO\nACVR2A\tA0A2K6MRS5_RHIBE\nACVR2A\tA0A2K6NIR7_RHIRO\nACVR2A\tA0A2K6NIS6_RHIRO\nACVR2A\tA0A2K6TBC2_SAIBB\nACVR2A\tA0A2P8XBR7_BLAGE\nACVR2A\tA0A2R9CLL3_PANPA\nACVR2A\tA0A2S2N6G2_SCHGA\nACVR2A\tA0A2S2Q7T7_9HEMI\nACVR2A\tA0A2U3W0V4_ODORO\nACVR2A\tA0A2U3X8H0_LEPWE\nACVR2A\tA0A2U3ZGV4_ODORO\nACVR2A\tA0A2U4AJC4_TURTR\nACVR2A\tA0A2U4AJE0_TURTR\nACVR2A\tA0A2U4AJX7_TURTR\nACVR2A\tA0A2Y9ETQ5_PHYMC\nACVR2A\tA0A2Y9I869_NEOSC\nACVR2A\tA0A2Y9MSN2_DELLE\nACVR2A\tA0A2Y9N832_DELLE\nACVR2A\tA0A340WZX3_LIPVE\nACVR2A\tA0A340X6B4_LIPVE\nACVR2A\tA0A341BAF0_NEOAA\nACVR2A\tA0A341BAS0_NEOAA\nACVR2A\tA0A341BCF2_NEOAA\nACVR2A\tA0A384A711_BALAS\nACVR2A\tA0A384CI68_URSMA\nACVR2A\tA0A3B3H8W0_ORYLA\nACVR2A\tA0A3B4H1S2_9CICH\nACVR2A\tA0A3B4H7W2_9CICH\nACVR2A\tA0A3P8PXB0_ASTCA\nACVR2A\tA0A3P8PXC7_ASTCA\nACVR2A\tA0A3P9CFS5_9CICH\nACVR2A\tA0A3P9CFT3_9CICH\nACVR2A\tA0A3Q0E247_TARSY\nACVR2A\tA0A3Q0RTG5_AMPCI\nACVR2A\tA0A3Q1LSH5_BOVIN\nACVR2A\tA0A3Q2HWZ1_HORSE\nACVR2A\tA0A3Q2I125_HORSE\nACVR2A\tA0A3Q2V8Z3_HAPBU\nACVR2A\tA0A3Q2WLY5_HAPBU\nACVR2A\tA0A3Q2WVU5_HAPBU\nACVR2A\tA0A3Q4HC86_NEOBR\nACVR2A\tA0A3Q4HQ10_NEOBR\nACVR2A\tA0A3Q7MXI7_CALUR\nACVR2A\tA0A3Q7SNA6_VULVU\nACVR2A\tA0A3Q7TDY5_VULVU\nACVR2A\tA0A3Q7V911_URSAR\nACVR2A\tA0A452EGE7_CAPHI\nACVR2A\tA0A452EGE9_CAPHI\nACVR2A\tA0A452EGK7_CAPHI\nACVR2A\tA0A452IVZ5_9SAUR\nACVR2A\tA0A452IW09_9SAUR\nACVR2A\tA0A452R971_URSAM\nACVR2A\tA0A493SZE2_ANAPP\nACVR2A\tA0A4C1TJU0_9NEOP\nACVR2A\tA0A4C1XRM6_9NEOP\nACVR2A\tA0A4W2G5G8_BOBOX\nACVR2A\tA0A4W2G835_BOBOX\nACVR2A\tA0A4W3I0K7_CALMI\nACVR2A\tA0A4W3I798_CALMI\nACVR2A\tA0A4W3I7A8_CALMI\nACVR2A\tA0A4W3I7C3_CALMI\nACVR2A\tA0A4W3I7D2_CALMI\nACVR2A\tA0A4W3I8Q3_CALMI\nACVR2A\tA0A4W3IRT8_CALMI\nACVR2A\tA0A4W3IRU8_CALMI\nACVR2A\tA0A4W4F998_ELEEL\nACVR2A\tA0A4W4FCF9_ELEEL\nACVR2A\tA0A4X1TD26_PIG\nACVR2A\tA0A4X1TEV4_PIG\nACVR2A\tA0A4X2K4M2_VOMUR\nACVR2A\tA0A5F4CQY0_CANLF\nACVR2A\tA0A5F4CVK0_CANLF\nACVR2A\tA0A5F4W6S4_CALJA\nACVR2A\tA0A5F8A1T8_MACMU\nACVR2A\tA0A5F8H1Q5_MONDO\nACVR2A\tA0A5F9CMM0_RABIT\nACVR2A\tA0A5G3KKY9_XENTR\nACVR2A\tA0A5G3KKZ7_XENTR\nACVR2A\tA0A5G3KLD7_XENTR\nACVR2A\tA0A5G3KLE4_XENTR\nACVR2A\tA0A5G3KNH8_XENTR\nACVR2A\tA2AI38_MOUSE\nACVR2A\tA9CAY0_DANRE\nACVR2A\tAVR2A_BOVIN\nACVR2A\tAVR2A_CHICK\nACVR2A\tAVR2A_HUMAN\nACVR2A\tAVR2A_MOUSE\nACVR2A\tAVR2A_RAT\nACVR2A\tAVR2A_SHEEP\nACVR2A\tAVR2A_XENLA\nACVR2A\tB5DDW8_XENTR\nACVR2A\tC1LE74_SCHJA\nACVR2A\tD9IC47_SHEEP\nACVR2A\tD9IVI6_SHEEP\nACVR2A\tE9M2M2_PIG\nACVR2A\tF1MA24_RAT\nACVR2A\tF1PKQ5_CANLF\nACVR2A\tF1QG01_DANRE\nACVR2A\tF1R4D0_DANRE\nACVR2A\tF1S0H6_PIG\nACVR2A\tF6R871_XENTR\nACVR2A\tF6XMR2_ORNAN\nACVR2A\tF7AZW8_CALJA\nACVR2A\tF7CD19_HORSE\nACVR2A\tF7CST5_XENTR\nACVR2A\tF7D5M3_XENTR\nACVR2A\tF7DBX1_XENTR\nACVR2A\tF7E7Y1_XENTR\nACVR2A\tF7EG01_MONDO\nACVR2A\tF7HPH2_MACMU\nACVR2A\tF7I897_CALJA\nACVR2A\tG1M4A6_AILME\nACVR2A\tG1NJ60_MELGA\nACVR2A\tG1P750_MYOLU\nACVR2A\tG1RCV0_NOMLE\nACVR2A\tG1TBJ7_RABIT\nACVR2A\tG3SQU9_LOXAF\nACVR2A\tG3UQ49_MELGA\nACVR2A\tG3W8S1_SARHA\nACVR2A\tG3W8S2_SARHA\nACVR2A\tG7N815_MACMU\nACVR2A\tG7PKJ4_MACFA\nACVR2A\tG8XXY0_OREMO\nACVR2A\tH0VCN4_CAVPO\nACVR2A\tH0WNZ5_OTOGA\nACVR2A\tH2L316_ORYLA\nACVR2A\tH2SET9_TAKRU\nACVR2A\tH2ZYU4_LATCH\nACVR2A\tH9G7M4_ANOCA\nACVR2A\tI3K2W5_ORENI\nACVR2A\tI3K2W6_ORENI\nACVR2A\tI3MXE2_ICTTR\nACVR2A\tK7ALD8_PANTR\nACVR2A\tK7GH55_PELSI\nACVR2A\tM3X9E4_FELCA\nACVR2A\tM3XWY2_MUSPF\nACVR2A\tQ56E96_DANRE\nACVR2A\tQ8BRV4_MOUSE\nACVR2A\tQ9PSM0_XENLA\nACVR2A\tT2MGF9_HYDVU\nACVR2A\tU3IUS5_ANAPP\nACVR2A\tU3KHF5_FICAL\nACVR2A\tU3KHF7_FICAL\nACVR2A\tV8PC84_OPHHA\nACVR2A\tW0U1H3_DICLA\nACVR2A\tW5PJD6_SHEEP\nACVR2A\tW8S9Q5_CHICK\nACVR2A\tW8SB82_CHICK\nACVR2A\tW8SJD6_CHICK\nACVR2A\tW8SNB1_CHICK\nACVR2A\tW8T7K3_CHICK\n','From\tTo\nKSR2\tA0A034WFW8_BACDO\nKSR2\tA0A075EIS2_MOUSE\nKSR2\tA0A096N4C3_PAPAN\nKSR2\tA0A0A1XGG7_ZEUCU\nKSR2\tA0A0A9WAF3_LYGHE\nKSR2\tA0A0B2VVD1_TOXCA\nKSR2\tA0A0D9S4D1_CHLSB\nKSR2\tA0A0R4IHD1_DANRE\nKSR2\tA0A0S7EK42_9TELE\nKSR2\tA0A0S7IKI4_9TELE\nKSR2\tA0A0S7IMT1_9TELE\nKSR2\tA0A0S7IV28_9TELE\nKSR2\tA0A0V0S4F5_9BILA\nKSR2\tA0A0V0SLK2_9BILA\nKSR2\tA0A0V0U6V3_9BILA\nKSR2\tA0A0V0U7M3_9BILA\nKSR2\tA0A0V0U8V0_9BILA\nKSR2\tA0A0V0WDR0_9BILA\nKSR2\tA0A0V0WE34_9BILA\nKSR2\tA0A0V0WZI3_9BILA\nKSR2\tA0A0V0WZW4_9BILA\nKSR2\tA0A0V0YD16_TRIPS\nKSR2\tA0A0V1A131_9BILA\nKSR2\tA0A0V1A290_9BILA\nKSR2\tA0A0V1A558_9BILA\nKSR2\tA0A0V1BAR5_TRISP\nKSR2\tA0A0V1BXS1_TRISP\nKSR2\tA0A0V1BYK2_TRISP\nKSR2\tA0A0V1D9L6_TRIBR\nKSR2\tA0A0V1DE35_TRIBR\nKSR2\tA0A0V1DE74_TRIBR\nKSR2\tA0A0V1E8T5_TRIPS\nKSR2\tA0A0V1FI69_TRIPS\nKSR2\tA0A0V1G6G2_TRIPS\nKSR2\tA0A0V1HF34_9BILA\nKSR2\tA0A0V1I366_9BILA\nKSR2\tA0A0V1J9K5_TRIPS\nKSR2\tA0A0V1JZR9_TRIPS\nKSR2\tA0A0V1LKD0_9BILA\nKSR2\tA0A0V1LKD4_9BILA\nKSR2\tA0A0V1LT89_9BILA\nKSR2\tA0A0V1MC22_9BILA\nKSR2\tA0A0V1MJL0_9BILA\nKSR2\tA0A0V1MK78_9BILA\nKSR2\tA0A0V1P0B9_9BILA\nKSR2\tA0A0V1PG08_9BILA\nKSR2\tA0A151N566_ALLMI\nKSR2\tA0A1A7WYH6_9TELE\nKSR2\tA0A1A7X3S1_9TELE\nKSR2\tA0A1A7Z418_9TELE\nKSR2\tA0A1A7ZWI0_NOTFU\nKSR2\tA0A1A8A5L5_NOTFU\nKSR2\tA0A1A8B134_NOTFU\nKSR2\tA0A1A8BD30_9TELE\nKSR2\tA0A1A8C6W9_9TELE\nKSR2\tA0A1A8CLF9_9TELE\nKSR2\tA0A1A8EE25_9TELE\nKSR2\tA0A1A8F6V2_9TELE\nKSR2\tA0A1A8G432_9TELE\nKSR2\tA0A1A8HU86_NOTKU\nKSR2\tA0A1A8HYX8_NOTKU\nKSR2\tA0A1A8KMQ2_NOTKU\nKSR2\tA0A1A8MF75_9TELE\nKSR2\tA0A1A8PHN1_9TELE\nKSR2\tA0A1A8PRQ7_9TELE\nKSR2\tA0A1A8Q2Y4_9TELE\nKSR2\tA0A1A8QAH8_9TELE\nKSR2\tA0A1A8QE42_9TELE\nKSR2\tA0A1D5PX90_CHICK\nKSR2\tA0A1D5R2U3_MACMU\nKSR2\tA0A1L8I175_XENLA\nKSR2\tA0A1S3A3G3_ERIEU\nKSR2\tA0A1S3F0R4_DIPOR\nKSR2\tA0A1S3NJ97_SALSA\nKSR2\tA0A1S3NJA9_SALSA\nKSR2\tA0A1S3NJQ9_SALSA\nKSR2\tA0A1S3NJW3_SALSA\nKSR2\tA0A1U7UI07_TARSY\nKSR2\tA0A1U8C6Z5_MESAU\nKSR2\tA0A1U8C9Y2_MESAU\nKSR2\tA0A1V4JR61_PATFA\nKSR2\tA0A1V4JR95_PATFA\nKSR2\tA0A218UI42_9PASE\nKSR2\tA0A250Y249_CASCN\nKSR2\tA0A286YAS8_DANRE\nKSR2\tA0A287CZB0_ICTTR\nKSR2\tA0A2B4SFU9_STYPI\nKSR2\tA0A2B4SZG7_STYPI\nKSR2\tA0A2D0PRL6_ICTPU\nKSR2\tA0A2I0LZ08_COLLI\nKSR2\tA0A2I3MDP2_PAPAN\nKSR2\tA0A2I3MVL1_PAPAN\nKSR2\tA0A2I3TK97_PANTR\nKSR2\tA0A2I4BFS5_9TELE\nKSR2\tA0A2I4BFT0_9TELE\nKSR2\tA0A2K5D2J5_AOTNA\nKSR2\tA0A2K5D2K0_AOTNA\nKSR2\tA0A2K5JRS5_COLAP\nKSR2\tA0A2K5P2M0_CERAT\nKSR2\tA0A2K5P2U5_CERAT\nKSR2\tA0A2K5QG65_CEBCA\nKSR2\tA0A2K5QG71_CEBCA\nKSR2\tA0A2K5VCJ1_MACFA\nKSR2\tA0A2K5VCK2_MACFA\nKSR2\tA0A2K6AET8_MANLE\nKSR2\tA0A2K6AEW5_MANLE\nKSR2\tA0A2K6DZ55_MACNE\nKSR2\tA0A2K6DZ73_MACNE\nKSR2\tA0A2K6FZZ7_PROCO\nKSR2\tA0A2K6G019_PROCO\nKSR2\tA0A2K6M335_RHIBE\nKSR2\tA0A2K6M336_RHIBE\nKSR2\tA0A2K6M347_RHIBE\nKSR2\tA0A2K6QW50_RHIRO\nKSR2\tA0A2K6QW85_RHIRO\nKSR2\tA0A2K6USV9_SAIBB\nKSR2\tA0A2K6USX1_SAIBB\nKSR2\tA0A2K6UT29_SAIBB\nKSR2\tA0A2R8P9Q6_CALJA\nKSR2\tA0A2R9AVZ4_PANPA\nKSR2\tA0A2R9AW42_PANPA\nKSR2\tA0A2R9AW60_PANPA\nKSR2\tA0A2S2QAP5_9HEMI\nKSR2\tA0A2U3VNR8_ODORO\nKSR2\tA0A2U4CJP2_TURTR\nKSR2\tA0A2Y9FQ18_PHYMC\nKSR2\tA0A2Y9HRU6_NEOSC\nKSR2\tA0A2Y9Q429_DELLE\nKSR2\tA0A2Y9Q8A9_DELLE\nKSR2\tA0A2Y9QEC5_DELLE\nKSR2\tA0A2Y9QEN0_DELLE\nKSR2\tA0A340X2Y4_LIPVE\nKSR2\tA0A341CHS1_NEOAA\nKSR2\tA0A384AQU8_BALAS\nKSR2\tA0A384BX37_URSMA\nKSR2\tA0A3B3H8G0_ORYLA\nKSR2\tA0A3B3HW36_ORYLA\nKSR2\tA0A3B3IHW8_ORYLA\nKSR2\tA0A3Q0GY26_ALLSI\nKSR2\tA0A3Q7T3Z2_VULVU\nKSR2\tA0A3Q7UEQ3_VULVU\nKSR2\tA0A3Q7WCH4_URSAR\nKSR2\tA0A3R7C2U1_CLOSI\nKSR2\tA0A452FV96_CAPHI\nKSR2\tA0A452H566_9SAUR\nKSR2\tA0A452H575_9SAUR\nKSR2\tA0A452H580_9SAUR\nKSR2\tA0A452H5I6_9SAUR\nKSR2\tA0A452U4W7_URSMA\nKSR2\tA0A452U4X0_URSMA\nKSR2\tA0A452U4X6_URSMA\nKSR2\tA0A4C1XB82_9NEOP\nKSR2\tA0A4W2C8D1_BOBOX\nKSR2\tA0A4W6E7I8_LATCA\nKSR2\tA0A4W6EBD7_LATCA\nKSR2\tA0A4X1TN97_PIG\nKSR2\tA0A4X2K730_VOMUR\nKSR2\tA0A4X2K762_VOMUR\nKSR2\tA0A4X2KC64_VOMUR\nKSR2\tA0A4X2KES4_VOMUR\nKSR2\tA0A5F8GJC0_MONDO\nKSR2\tA0A5F8GYK6_MONDO\nKSR2\tA5X3G4_MOUSE\nKSR2\tC4IXV3_MOUSE\nKSR2\tCNKR2_HUMAN\nKSR2\tE9PB13_HUMAN\nKSR2\tF1LY04_RAT\nKSR2\tF1MF36_BOVIN\nKSR2\tF1NCU3_CHICK\nKSR2\tF1P721_CANLF\nKSR2\tF1RKG1_PIG\nKSR2\tF5H8P1_DANRE\nKSR2\tF6SHX4_HORSE\nKSR2\tF6WHP0_MONDO\nKSR2\tF7GRN9_MACMU\nKSR2\tF7GRP4_MACMU\nKSR2\tG1L2K9_AILME\nKSR2\tG1NC56_MELGA\nKSR2\tG1PDC8_MYOLU\nKSR2\tG1QHI1_NOMLE\nKSR2\tG1SG01_RABIT\nKSR2\tG3T635_LOXAF\nKSR2\tH0UUS8_CAVPO\nKSR2\tH0WSX5_OTOGA\nKSR2\tH0ZJH2_TAEGU\nKSR2\tH2L662_ORYLA\nKSR2\tH2NIT5_PONAB\nKSR2\tH2Q6Z2_PANTR\nKSR2\tH2UXV4_TAKRU\nKSR2\tH3BC71_LATCH\nKSR2\tH9FKM4_MACMU\nKSR2\tH9GNE1_ANOCA\nKSR2\tI3JWR4_ORENI\nKSR2\tI3JWR5_ORENI\nKSR2\tI3NGH9_ICTTR\nKSR2\tK7FC59_PELSI\nKSR2\tKSR2_HUMAN\nKSR2\tKSR2_MOUSE\nKSR2\tM0QW59_MOUSE\nKSR2\tM0RBD3_RAT\nKSR2\tM3VY73_FELCA\nKSR2\tM3XUQ3_MUSPF\nKSR2\tT2MB09_HYDVU\nKSR2\tU3IAH5_ANAPP\nKSR2\tU3JXB9_FICAL\nKSR2\tU6D9G8_NEOVI\nKSR2\tV5H1C8_ANOGL\nKSR2\tV8N9N6_OPHHA\nKSR2\tV8NHD2_OPHHA\nKSR2\tW5P071_SHEEP\nKSR2\tW8B6D9_CERCA\n','From\tTo\nIRAK4\tA0A068AVD9_9TELE\nIRAK4\tA0A096N6Q4_PAPAN\nIRAK4\tA0A0B4PLN5_RHIMP\nIRAK4\tA0A0D9R382_CHLSB\nIRAK4\tA0A0K2SWB5_LEPSM\nIRAK4\tA0A0P6J4P2_HETGA\nIRAK4\tA0A0R4IP49_DANRE\nIRAK4\tA0A0R4IQ44_DANRE\nIRAK4\tA0A0R4J2F4_HUMAN\nIRAK4\tA0A0S7K4B5_9TELE\nIRAK4\tA0A0S7K4G7_9TELE\nIRAK4\tA0A0S7K4T3_9TELE\nIRAK4\tA0A0S7K5B1_9TELE\nIRAK4\tA0A0S7K5P9_9TELE\nIRAK4\tA0A0S7K5T7_9TELE\nIRAK4\tA0A151PFA4_ALLMI\nIRAK4\tA0A191S1V4_CRAGI\nIRAK4\tA0A1A7WV19_9TELE\nIRAK4\tA0A1A7Z6Y8_9TELE\nIRAK4\tA0A1A7Z869_9TELE\nIRAK4\tA0A1A8ALR2_NOTFU\nIRAK4\tA0A1A8C7T8_9TELE\nIRAK4\tA0A1A8DU97_9TELE\nIRAK4\tA0A1A8GYM4_9TELE\nIRAK4\tA0A1A8I0U7_NOTKU\nIRAK4\tA0A1A8IIJ5_NOTKU\nIRAK4\tA0A1A8KHM0_NOTKU\nIRAK4\tA0A1A8MVS0_9TELE\nIRAK4\tA0A1A8PNI4_9TELE\nIRAK4\tA0A1A8QGK9_9TELE\nIRAK4\tA0A1A8RIV7_9TELE\nIRAK4\tA0A1A8S434_9TELE\nIRAK4\tA0A1A8SIG9_9TELE\nIRAK4\tA0A1A8UG45_NOTFU\nIRAK4\tA0A1C7A892_TRAOV\nIRAK4\tA0A1L5YS25_LARCR\nIRAK4\tA0A1L8GYU8_XENLA\nIRAK4\tA0A1S3FDF2_DIPOR\nIRAK4\tA0A1S3FE13_DIPOR\nIRAK4\tA0A1S3WID2_ERIEU\nIRAK4\tA0A1U7UT77_TARSY\nIRAK4\tA0A1U8BJZ2_MESAU\nIRAK4\tA0A1U8D7Q5_ALLSI\nIRAK4\tA0A1U8D7Q8_ALLSI\nIRAK4\tA0A1V4J9A2_PATFA\nIRAK4\tA0A218V577_9PASE\nIRAK4\tA0A250YIG4_CASCN\nIRAK4\tA0A2I0MJ95_COLLI\nIRAK4\tA0A2I3G6P7_NOMLE\nIRAK4\tA0A2I3HYP4_NOMLE\nIRAK4\tA0A2I3LXU7_PAPAN\nIRAK4\tA0A2I3TDT1_PANTR\nIRAK4\tA0A2I4C8E8_9TELE\nIRAK4\tA0A2J8LQW3_PANTR\nIRAK4\tA0A2K5C9A4_AOTNA\nIRAK4\tA0A2K5C9D1_AOTNA\nIRAK4\tA0A2K5JMP0_COLAP\nIRAK4\tA0A2K5JMQ1_COLAP\nIRAK4\tA0A2K5JMU3_COLAP\nIRAK4\tA0A2K5MIH3_CERAT\nIRAK4\tA0A2K5MIJ2_CERAT\nIRAK4\tA0A2K5QLF3_CEBCA\nIRAK4\tA0A2K5QLK6_CEBCA\nIRAK4\tA0A2K5V3Y9_MACFA\nIRAK4\tA0A2K5V422_MACFA\nIRAK4\tA0A2K5V433_MACFA\nIRAK4\tA0A2K5YF13_MANLE\nIRAK4\tA0A2K5YF18_MANLE\nIRAK4\tA0A2K6DC27_MACNE\nIRAK4\tA0A2K6DC69_MACNE\nIRAK4\tA0A2K6GQL3_PROCO\nIRAK4\tA0A2K6GQL7_PROCO\nIRAK4\tA0A2K6MSR5_RHIBE\nIRAK4\tA0A2K6MSV3_RHIBE\nIRAK4\tA0A2K6QAA7_RHIRO\nIRAK4\tA0A2K6UWM4_SAIBB\nIRAK4\tA0A2K6UWR0_SAIBB\nIRAK4\tA0A2K6UWR7_SAIBB\nIRAK4\tA0A2K6UWU0_SAIBB\nIRAK4\tA0A2R8ZWD7_PANPA\nIRAK4\tA0A2R8ZXJ4_PANPA\nIRAK4\tA0A2R9A1Z9_PANPA\nIRAK4\tA0A2U3VM01_ODORO\nIRAK4\tA0A2U3VM08_ODORO\nIRAK4\tA0A2U3XFR3_LEPWE\nIRAK4\tA0A2U3XFU3_LEPWE\nIRAK4\tA0A2U4BAB1_TURTR\nIRAK4\tA0A2U4BAB5_TURTR\nIRAK4\tA0A2U4BAE7_TURTR\nIRAK4\tA0A2U4BAP6_TURTR\nIRAK4\tA0A2U9BWA7_SCOMX\nIRAK4\tA0A2Y9EQ34_PHYMC\nIRAK4\tA0A2Y9EQJ3_PHYMC\nIRAK4\tA0A2Y9ERL0_PHYMC\nIRAK4\tA0A2Y9I997_NEOSC\nIRAK4\tA0A2Y9N0D4_DELLE\nIRAK4\tA0A2Y9N1Q6_DELLE\nIRAK4\tA0A2Y9N4T5_DELLE\nIRAK4\tA0A337S8S7_FELCA\nIRAK4\tA0A340XM89_LIPVE\nIRAK4\tA0A340XQM9_LIPVE\nIRAK4\tA0A341B093_NEOAA\nIRAK4\tA0A341B0B5_NEOAA\nIRAK4\tA0A341B2Z3_NEOAA\nIRAK4\tA0A346SC80_BISBO\nIRAK4\tA0A383YZT9_BALAS\nIRAK4\tA0A383YZU7_BALAS\nIRAK4\tA0A383Z0H2_BALAS\nIRAK4\tA0A384D6K1_URSMA\nIRAK4\tA0A385EI95_STIJA\nIRAK4\tA0A3B3QNA2_9TELE\nIRAK4\tA0A3B5K2L4_TAKRU\nIRAK4\tA0A3Q0GXQ2_ALLSI\nIRAK4\tA0A3Q7PYN0_CALUR\nIRAK4\tA0A3Q7REG0_CALUR\nIRAK4\tA0A3Q7T6J3_VULVU\nIRAK4\tA0A3Q7W8Z7_URSAR\nIRAK4\tA0A3Q7WUK7_URSAR\nIRAK4\tA0A3Q7X5H8_URSAR\nIRAK4\tA0A3Q8TK32_9TELE\nIRAK4\tA0A411P9V2_CTEID\nIRAK4\tA0A438HQ46_VITVI\nIRAK4\tA0A452F398_CAPHI\nIRAK4\tA0A452I2K1_9SAUR\nIRAK4\tA0A452SQP8_URSAM\nIRAK4\tA0A452SQT1_URSAM\nIRAK4\tA0A452SQW7_URSAM\nIRAK4\tA0A452SR13_URSAM\nIRAK4\tA0A493TB44_ANAPP\nIRAK4\tA0A493TSS1_ANAPP\nIRAK4\tA0A4C1WBZ2_9NEOP\nIRAK4\tA0A4W2CZ86_BOBOX\nIRAK4\tA0A4W2CZA8_BOBOX\nIRAK4\tA0A4W2IBW0_BOBOX\nIRAK4\tA0A4W3I6H4_CALMI\nIRAK4\tA0A4W3IYA9_CALMI\nIRAK4\tA0A4W6FJQ8_LATCA\nIRAK4\tA0A4X1TXU3_PIG\nIRAK4\tA0A4X1TXY3_PIG\nIRAK4\tA0A4X2K4W3_VOMUR\nIRAK4\tA0A4Y2RZJ0_ARAVE\nIRAK4\tA0A4Z2F5U4_9TELE\nIRAK4\tA0A5B8ZTJ9_ORENI\nIRAK4\tA0A5B9BNX3_TAKOB\nIRAK4\tA0A5F4CLL7_CANLF\nIRAK4\tA0A5F9CDQ3_RABIT\nIRAK4\tA0A5F9DDH8_RABIT\nIRAK4\tA0A5G2QQD7_PIG\nIRAK4\tA0A5G3HAA0_XENTR\nIRAK4\tA0A5G3HRZ9_XENTR\nIRAK4\tA0A5G3HSJ3_XENTR\nIRAK4\tA0A5G3JHI5_XENTR\nIRAK4\tA0A5G3JHM5_XENTR\nIRAK4\tA0A5G3JHP2_XENTR\nIRAK4\tA0A5G3JI06_XENTR\nIRAK4\tA0A5G3JIW3_XENTR\nIRAK4\tA9QT42_PIG\nIRAK4\tB1H1F5_XENTR\nIRAK4\tB3FXL6_BOVIN\nIRAK4\tB5X366_SALSA\nIRAK4\tB7U176_SHEEP\nIRAK4\tC4B6S8_PLEAT\nIRAK4\tD1J6Y3_ONCMY\nIRAK4\tD1J6Y4_9TELE\nIRAK4\tD1J6Y7_SALSA\nIRAK4\tD1J6Y8_SALSA\nIRAK4\tD4A7K4_RAT\nIRAK4\tE2RDL0_CANLF\nIRAK4\tE3TE16_ICTPU\nIRAK4\tF1P2L8_CHICK\nIRAK4\tF6Q7E6_PIG\nIRAK4\tF6UDP1_XENTR\nIRAK4\tF6XRK8_XENTR\nIRAK4\tF6YIJ1_XENTR\nIRAK4\tF6YPU9_XENTR\nIRAK4\tF6ZUT2_CALJA\nIRAK4\tF7BTN5_XENTR\nIRAK4\tF7BZX8_HORSE\nIRAK4\tF7GD74_MONDO\nIRAK4\tF7HHC6_MACMU\nIRAK4\tF8VR40_HUMAN\nIRAK4\tF8VVZ1_HUMAN\nIRAK4\tF8VW24_HUMAN\nIRAK4\tG1KBF6_ANOCA\nIRAK4\tG1MG21_AILME\nIRAK4\tG1NB62_MELGA\nIRAK4\tG1P7F0_MYOLU\nIRAK4\tG1S8E5_NOMLE\nIRAK4\tG1SL29_RABIT\nIRAK4\tG3SNZ8_LOXAF\nIRAK4\tG3WNL8_SARHA\nIRAK4\tG9K636_MUSPF\nIRAK4\tH0UX02_CAVPO\nIRAK4\tH0WWG9_OTOGA\nIRAK4\tH0Z6A6_TAEGU\nIRAK4\tH2LDA6_ORYLA\nIRAK4\tH2NH07_PONAB\nIRAK4\tH2Q5R2_PANTR\nIRAK4\tH3B977_LATCH\nIRAK4\tH9FV97_MACMU\nIRAK4\tI3MLT1_ICTTR\nIRAK4\tIRAK4_BOVIN\nIRAK4\tIRAK4_HUMAN\nIRAK4\tIRAK4_MOUSE\nIRAK4\tK7AYU9_PANTR\nIRAK4\tK7G2W6_PELSI\nIRAK4\tM3WWH5_FELCA\nIRAK4\tQ28G53_XENTR\nIRAK4\tQ32S47_EUPSC\nIRAK4\tQ3USX5_MOUSE\nIRAK4\tQ69FE3_HUMAN\nIRAK4\tQ6IWL3_DANRE\nIRAK4\tQ7Z6A7_HUMAN\nIRAK4\tQ7Z6A8_HUMAN\nIRAK4\tQ7ZVW7_DANRE\nIRAK4\tQ9D250_MOUSE\nIRAK4\tT2L2I0_SALSA\nIRAK4\tU3IXR5_ANAPP\nIRAK4\tU3K6I3_FICAL\nIRAK4\tU6CSQ5_NEOVI\nIRAK4\tV8NMG3_OPHHA\nIRAK4\tW5M0F9_LEPOC\nIRAK4\tW5QEI7_SHEEP\n','From\tTo\nCHEK1\tA0A087WT52_HUMAN\nCHEK1\tA0A096MSF0_PAPAN\nCHEK1\tA0A0A9Z0U3_LYGHE\nCHEK1\tA0A0D9S4F3_CHLSB\nCHEK1\tA0A0G2JWH5_RAT\nCHEK1\tA0A0P6IZ75_HETGA\nCHEK1\tA0A0V0SDT5_9BILA\nCHEK1\tA0A0V0SE41_9BILA\nCHEK1\tA0A0V0T9L9_9BILA\nCHEK1\tA0A0V0T9T5_9BILA\nCHEK1\tA0A0V0WNI1_9BILA\nCHEK1\tA0A0V0WP75_9BILA\nCHEK1\tA0A0V0YDS8_TRIPS\nCHEK1\tA0A0V0Z6U7_9BILA\nCHEK1\tA0A0V0Z747_9BILA\nCHEK1\tA0A0V0Z7H1_9BILA\nCHEK1\tA0A0V1BHQ4_TRISP\nCHEK1\tA0A0V1BHW4_TRISP\nCHEK1\tA0A0V1C777_TRIBR\nCHEK1\tA0A0V1C7C0_TRIBR\nCHEK1\tA0A0V1C7G3_TRIBR\nCHEK1\tA0A0V1G5C7_TRIPS\nCHEK1\tA0A0V1I300_9BILA\nCHEK1\tA0A0V1I401_9BILA\nCHEK1\tA0A0V1JHR2_TRIPS\nCHEK1\tA0A0V1KDI2_TRIPS\nCHEK1\tA0A0V1KDK3_TRIPS\nCHEK1\tA0A0V1KDN0_TRIPS\nCHEK1\tA0A0V1KN54_9BILA\nCHEK1\tA0A0V1KNR0_9BILA\nCHEK1\tA0A0V1N2I8_9BILA\nCHEK1\tA0A0V1N2L9_9BILA\nCHEK1\tA0A0V1PNR5_9BILA\nCHEK1\tA0A0V1PP97_9BILA\nCHEK1\tA0A146LR46_LYGHE\nCHEK1\tA0A151NG72_ALLMI\nCHEK1\tA0A1A7WSS4_9TELE\nCHEK1\tA0A1A7YJI8_9TELE\nCHEK1\tA0A1A8A4M7_NOTFU\nCHEK1\tA0A1A8AW86_NOTFU\nCHEK1\tA0A1A8D4C2_9TELE\nCHEK1\tA0A1A8D651_9TELE\nCHEK1\tA0A1A8FMT9_9TELE\nCHEK1\tA0A1A8GBB0_9TELE\nCHEK1\tA0A1A8I5M8_NOTKU\nCHEK1\tA0A1A8KQX9_NOTKU\nCHEK1\tA0A1A8LCY8_9TELE\nCHEK1\tA0A1A8MD82_9TELE\nCHEK1\tA0A1A8QCY5_9TELE\nCHEK1\tA0A1A8QDG7_9TELE\nCHEK1\tA0A1A8RI64_9TELE\nCHEK1\tA0A1A8SHI5_9TELE\nCHEK1\tA0A1A8U4I6_NOTFU\nCHEK1\tA0A1B8XX80_XENTR\nCHEK1\tA0A1L8FFS1_XENLA\nCHEK1\tA0A1L8FL95_XENLA\nCHEK1\tA0A1S2ZKC6_ERIEU\nCHEK1\tA0A1S2ZKD6_ERIEU\nCHEK1\tA0A1S3ELS6_DIPOR\nCHEK1\tA0A1S3L363_SALSA\nCHEK1\tA0A1U7ULN8_TARSY\nCHEK1\tA0A1U7UUR0_TARSY\nCHEK1\tA0A1U7UW70_TARSY\nCHEK1\tA0A1U8CLY7_MESAU\nCHEK1\tA0A1V4J5X2_PATFA\nCHEK1\tA0A218UTC9_9PASE\nCHEK1\tA0A286X8I4_CAVPO\nCHEK1\tA0A286XVL9_CAVPO\nCHEK1\tA0A287BJH4_PIG\nCHEK1\tA0A2B4SDK4_STYPI\nCHEK1\tA0A2D0SXH9_ICTPU\nCHEK1\tA0A2H8TMA4_9HEMI\nCHEK1\tA0A2I0LSB0_COLLI\nCHEK1\tA0A2I2U2T7_FELCA\nCHEK1\tA0A2I3GYB3_NOMLE\nCHEK1\tA0A2I3H2X0_NOMLE\nCHEK1\tA0A2I3HA72_NOMLE\nCHEK1\tA0A2I3RNL3_PANTR\nCHEK1\tA0A2I3RRH9_PANTR\nCHEK1\tA0A2I3T978_PANTR\nCHEK1\tA0A2I4AT21_9TELE\nCHEK1\tA0A2K5L0X8_CERAT\nCHEK1\tA0A2K5L110_CERAT\nCHEK1\tA0A2K5L117_CERAT\nCHEK1\tA0A2K5L120_CERAT\nCHEK1\tA0A2K5R8Q2_CEBCA\nCHEK1\tA0A2K5R8T3_CEBCA\nCHEK1\tA0A2K5R8V7_CEBCA\nCHEK1\tA0A2K5R8X1_CEBCA\nCHEK1\tA0A2K5R8X7_CEBCA\nCHEK1\tA0A2K5UAB7_MACFA\nCHEK1\tA0A2K5UAC4_MACFA\nCHEK1\tA0A2K5UAD2_MACFA\nCHEK1\tA0A2K5UAE4_MACFA\nCHEK1\tA0A2K5UAE9_MACFA\nCHEK1\tA0A2K5UAG3_MACFA\nCHEK1\tA0A2K5UAL2_MACFA\nCHEK1\tA0A2K5YPC9_MANLE\nCHEK1\tA0A2K5YPE4_MANLE\nCHEK1\tA0A2K5YPF6_MANLE\nCHEK1\tA0A2K5YPG5_MANLE\nCHEK1\tA0A2K6E9I2_MACNE\nCHEK1\tA0A2K6E9I3_MACNE\nCHEK1\tA0A2K6E9I6_MACNE\nCHEK1\tA0A2K6E9J8_MACNE\nCHEK1\tA0A2K6E9J9_MACNE\nCHEK1\tA0A2K6FSV0_PROCO\nCHEK1\tA0A2K6FSV8_PROCO\nCHEK1\tA0A2K6FSY3_PROCO\nCHEK1\tA0A2K6UQM7_SAIBB\nCHEK1\tA0A2K6UQN3_SAIBB\nCHEK1\tA0A2K6UQN8_SAIBB\nCHEK1\tA0A2K6UQR1_SAIBB\nCHEK1\tA0A2K6UQU6_SAIBB\nCHEK1\tA0A2U3X4N3_ODORO\nCHEK1\tA0A2U3X517_ODORO\nCHEK1\tA0A2U3ZY01_ODORO\nCHEK1\tA0A2U4BVK9_TURTR\nCHEK1\tA0A2U4BVS1_TURTR\nCHEK1\tA0A2Y9EPM3_PHYMC\nCHEK1\tA0A2Y9ER27_PHYMC\nCHEK1\tA0A2Y9ER94_PHYMC\nCHEK1\tA0A2Y9HQX7_NEOSC\nCHEK1\tA0A2Y9HRR2_NEOSC\nCHEK1\tA0A2Y9M200_DELLE\nCHEK1\tA0A2Y9MFJ9_DELLE\nCHEK1\tA0A337S630_FELCA\nCHEK1\tA0A337S750_FELCA\nCHEK1\tA0A340XQP2_LIPVE\nCHEK1\tA0A340XQQ8_LIPVE\nCHEK1\tA0A340XU34_LIPVE\nCHEK1\tA0A341B146_NEOAA\nCHEK1\tA0A384A831_BALAS\nCHEK1\tA0A384BV98_URSMA\nCHEK1\tA0A384BW62_URSMA\nCHEK1\tA0A384BWG2_URSMA\nCHEK1\tA0A3Q0GBR5_ALLSI\nCHEK1\tA0A3Q1MLF0_BOVIN\nCHEK1\tA0A3Q2H892_HORSE\nCHEK1\tA0A3Q2H9I6_HORSE\nCHEK1\tA0A3Q7QT60_CALUR\nCHEK1\tA0A3Q7S7R5_CALUR\nCHEK1\tA0A3Q7THU7_VULVU\nCHEK1\tA0A3Q7VJU0_URSAR\nCHEK1\tA0A3Q7X3X3_URSAR\nCHEK1\tA0A3R7H2M8_CLOSI\nCHEK1\tA0A452G2C0_CAPHI\nCHEK1\tA0A452J1I4_9SAUR\nCHEK1\tA0A452QP46_URSAM\nCHEK1\tA0A452QP71_URSAM\nCHEK1\tA0A452QP82_URSAM\nCHEK1\tA0A452QP86_URSAM\nCHEK1\tA0A4W2BMU2_BOBOX\nCHEK1\tA0A4W2BQ72_BOBOX\nCHEK1\tA0A4W2BQW6_BOBOX\nCHEK1\tA0A4W2DL16_BOBOX\nCHEK1\tA0A4W2DRK0_BOBOX\nCHEK1\tA0A4W2H024_BOBOX\nCHEK1\tA0A4W2H236_BOBOX\nCHEK1\tA0A4W2H2N5_BOBOX\nCHEK1\tA0A4W2ICI2_BOBOX\nCHEK1\tA0A4W2ICM7_BOBOX\nCHEK1\tA0A4W3H2U7_CALMI\nCHEK1\tA0A4W3H5U3_CALMI\nCHEK1\tA0A4W3H6W3_CALMI\nCHEK1\tA0A4W3H7E0_CALMI\nCHEK1\tA0A4W3H7E7_CALMI\nCHEK1\tA0A4W6D592_LATCA\nCHEK1\tA0A4W6D5P7_LATCA\nCHEK1\tA0A4W6D5T9_LATCA\nCHEK1\tA0A4X1VBZ6_PIG\nCHEK1\tA0A4X1VFV8_PIG\nCHEK1\tA0A4X1VFW9_PIG\nCHEK1\tA0A4X1VGX4_PIG\nCHEK1\tA0A4X2LU65_VOMUR\nCHEK1\tA0A4X2LY74_VOMUR\nCHEK1\tA0A4X2M4T6_VOMUR\nCHEK1\tA0A4X2M641_VOMUR\nCHEK1\tA0A5B7CI05_PORTR\nCHEK1\tA0A5F4C6R3_CANLF\nCHEK1\tA0A5F4CMH7_CANLF\nCHEK1\tA0A5F5PHP7_HORSE\nCHEK1\tA0A5F5PJU4_HORSE\nCHEK1\tA0A5F5PZ36_HORSE\nCHEK1\tA0A5F5XK57_FELCA\nCHEK1\tA0A5F8GAU9_MONDO\nCHEK1\tA0A5F8GXS9_MONDO\nCHEK1\tA0A5F8H5X5_MONDO\nCHEK1\tA0A5F8HJI8_MONDO\nCHEK1\tA0A5F8HKX5_MONDO\nCHEK1\tA0A5G2R0V2_PIG\nCHEK1\tA0A5G2R991_PIG\nCHEK1\tA0A5K1TVD8_MACMU\nCHEK1\tA5D7V1_BOVIN\nCHEK1\tB4F7C9_RAT\nCHEK1\tCHK1_CHICK\nCHEK1\tCHK1_HUMAN\nCHEK1\tCHK1_MOUSE\nCHEK1\tCHK1_RAT\nCHEK1\tCHK1_XENLA\nCHEK1\tE7EPP6_HUMAN\nCHEK1\tE9PJI4_HUMAN\nCHEK1\tE9PKQ3_HUMAN\nCHEK1\tE9PPA5_HUMAN\nCHEK1\tE9PQW7_HUMAN\nCHEK1\tE9PRU7_HUMAN\nCHEK1\tF1PUR2_CANLF\nCHEK1\tF1S774_PIG\nCHEK1\tF6Q6K1_BOVIN\nCHEK1\tF6UDX2_MONDO\nCHEK1\tF6UR87_ORNAN\nCHEK1\tF6UX38_HORSE\nCHEK1\tF6WF09_HORSE\nCHEK1\tF6YCB3_MACMU\nCHEK1\tF7CZU5_XENTR\nCHEK1\tF7DQ46_MACMU\nCHEK1\tG1L2J8_AILME\nCHEK1\tG1P5M5_MYOLU\nCHEK1\tG1R7A0_NOMLE\nCHEK1\tG3TP32_LOXAF\nCHEK1\tG3UX84_MOUSE\nCHEK1\tG3UYC5_MOUSE\nCHEK1\tG3WYR5_SARHA\nCHEK1\tG3WYR6_SARHA\nCHEK1\tH0VVF5_CAVPO\nCHEK1\tH0WJG5_OTOGA\nCHEK1\tH0YRA3_TAEGU\nCHEK1\tH2LMC8_ORYLA\nCHEK1\tH2Q525_PANTR\nCHEK1\tH2V3U1_TAKRU\nCHEK1\tH2V3U2_TAKRU\nCHEK1\tH3BAR5_LATCH\nCHEK1\tH9G3B5_ANOCA\nCHEK1\tH9H052_MELGA\nCHEK1\tI3JBE8_ORENI\nCHEK1\tI3JBE9_ORENI\nCHEK1\tI3M0G9_ICTTR\nCHEK1\tK7GCA8_PELSI\nCHEK1\tM3WLY2_FELCA\nCHEK1\tM3Y538_MUSPF\nCHEK1\tQ28EK6_XENTR\nCHEK1\tQ497T8_MOUSE\nCHEK1\tQ6W8P8_HUMAN\nCHEK1\tQ7ZVD7_DANRE\nCHEK1\tR9WA93_DAPPU\nCHEK1\tS4RSG8_PETMA\nCHEK1\tT2MJ26_HYDVU\nCHEK1\tU3J5R3_ANAPP\nCHEK1\tU3JRB8_FICAL\nCHEK1\tW5PPW3_SHEEP\n','From\tTo\nRPS6KA5\tA0A096NRH4_PAPAN\nRPS6KA5\tA0A0A9Y4H7_LYGHE\nRPS6KA5\tA0A0D9RG74_CHLSB\nRPS6KA5\tA0A0G2K366_RAT\nRPS6KA5\tA0A0P6JEZ8_HETGA\nRPS6KA5\tA0A1A7XK19_9TELE\nRPS6KA5\tA0A1A7YMQ2_9TELE\nRPS6KA5\tA0A1A8AXK9_NOTFU\nRPS6KA5\tA0A1A8BH39_9TELE\nRPS6KA5\tA0A1A8E0N0_9TELE\nRPS6KA5\tA0A1A8FH08_9TELE\nRPS6KA5\tA0A1A8IZQ4_NOTKU\nRPS6KA5\tA0A1A8K8Y5_NOTKU\nRPS6KA5\tA0A1A8M529_9TELE\nRPS6KA5\tA0A1A8S4B0_9TELE\nRPS6KA5\tA0A1A8V0Y2_NOTFU\nRPS6KA5\tA0A1D5PXU3_CHICK\nRPS6KA5\tA0A1L8F9G5_XENLA\nRPS6KA5\tA0A1Q9C8E3_SYMMI\nRPS6KA5\tA0A1S2ZZ75_ERIEU\nRPS6KA5\tA0A1S3EY13_DIPOR\nRPS6KA5\tA0A1U7QL93_MESAU\nRPS6KA5\tA0A1U7U1R1_TARSY\nRPS6KA5\tA0A1V4KAR8_PATFA\nRPS6KA5\tA0A1Y7VL39_MOUSE\nRPS6KA5\tA0A218UNF9_9PASE\nRPS6KA5\tA0A250Y2Z4_CASCN\nRPS6KA5\tA0A286XJX3_CAVPO\nRPS6KA5\tA0A287A1C4_PIG\nRPS6KA5\tA0A287BP56_PIG\nRPS6KA5\tA0A287D7D6_ICTTR\nRPS6KA5\tA0A2B4SQF0_STYPI\nRPS6KA5\tA0A2I0MQ34_COLLI\nRPS6KA5\tA0A2I3G979_NOMLE\nRPS6KA5\tA0A2I3H8S9_NOMLE\nRPS6KA5\tA0A2I3MGX6_PAPAN\nRPS6KA5\tA0A2I3N4F9_PAPAN\nRPS6KA5\tA0A2I3NEJ5_PAPAN\nRPS6KA5\tA0A2I3RSK2_PANTR\nRPS6KA5\tA0A2I3TUB5_PANTR\nRPS6KA5\tA0A2I4CL92_9TELE\nRPS6KA5\tA0A2I4CLA7_9TELE\nRPS6KA5\tA0A2K5CVM5_AOTNA\nRPS6KA5\tA0A2K5CVN0_AOTNA\nRPS6KA5\tA0A2K5CVN3_AOTNA\nRPS6KA5\tA0A2K5KEG0_COLAP\nRPS6KA5\tA0A2K5MZ74_CERAT\nRPS6KA5\tA0A2K5MZ75_CERAT\nRPS6KA5\tA0A2K5MZ79_CERAT\nRPS6KA5\tA0A2K5MZ94_CERAT\nRPS6KA5\tA0A2K5MZ99_CERAT\nRPS6KA5\tA0A2K5MZA9_CERAT\nRPS6KA5\tA0A2K5MZP5_CERAT\nRPS6KA5\tA0A2K5QXL7_CEBCA\nRPS6KA5\tA0A2K5QXN5_CEBCA\nRPS6KA5\tA0A2K5QXP5_CEBCA\nRPS6KA5\tA0A2K5VW15_MACFA\nRPS6KA5\tA0A2K5VW22_MACFA\nRPS6KA5\tA0A2K5VW70_MACFA\nRPS6KA5\tA0A2K6AJA6_MANLE\nRPS6KA5\tA0A2K6AJF8_MANLE\nRPS6KA5\tA0A2K6CZQ3_MACNE\nRPS6KA5\tA0A2K6CZS1_MACNE\nRPS6KA5\tA0A2K6ERA8_PROCO\nRPS6KA5\tA0A2K6MLT1_RHIBE\nRPS6KA5\tA0A2K6MLT4_RHIBE\nRPS6KA5\tA0A2K6PKG4_RHIRO\nRPS6KA5\tA0A2K6SR89_SAIBB\nRPS6KA5\tA0A2K6SRB3_SAIBB\nRPS6KA5\tA0A2R9BLI4_PANPA\nRPS6KA5\tA0A2R9BP98_PANPA\nRPS6KA5\tA0A2R9BUL8_PANPA\nRPS6KA5\tA0A2U3VI61_ODORO\nRPS6KA5\tA0A2U4A0U0_TURTR\nRPS6KA5\tA0A2U4A0U1_TURTR\nRPS6KA5\tA0A2U4A0U2_TURTR\nRPS6KA5\tA0A2U4A0V4_TURTR\nRPS6KA5\tA0A2U4A0W0_TURTR\nRPS6KA5\tA0A2U4A0X5_TURTR\nRPS6KA5\tA0A2Y9G8F3_NEOSC\nRPS6KA5\tA0A2Y9G8J7_NEOSC\nRPS6KA5\tA0A2Y9G8P9_NEOSC\nRPS6KA5\tA0A2Y9GC15_NEOSC\nRPS6KA5\tA0A2Y9NCX8_DELLE\nRPS6KA5\tA0A2Y9NGB2_DELLE\nRPS6KA5\tA0A2Y9NKX2_DELLE\nRPS6KA5\tA0A2Y9NRL4_DELLE\nRPS6KA5\tA0A2Y9TFC0_PHYMC\nRPS6KA5\tA0A340YE36_LIPVE\nRPS6KA5\tA0A341AN34_NEOAA\nRPS6KA5\tA0A341AQ30_NEOAA\nRPS6KA5\tA0A341AQY6_NEOAA\nRPS6KA5\tA0A369JLU3_HYPMA\nRPS6KA5\tA0A384A0R9_BALAS\nRPS6KA5\tA0A384D1M6_URSMA\nRPS6KA5\tA0A3B3ISL8_HUMAN\nRPS6KA5\tA0A3Q0CH63_MESAU\nRPS6KA5\tA0A3Q0CME2_MESAU\nRPS6KA5\tA0A3Q0CMU5_MESAU\nRPS6KA5\tA0A3Q0FIE7_ALLSI\nRPS6KA5\tA0A3Q0FIG9_ALLSI\nRPS6KA5\tA0A3Q0FLI8_ALLSI\nRPS6KA5\tA0A3Q0FN09_ALLSI\nRPS6KA5\tA0A3Q1LRM7_BOVIN\nRPS6KA5\tA0A3Q1MUT9_BOVIN\nRPS6KA5\tA0A3Q7NV18_CALUR\nRPS6KA5\tA0A3Q7NVD4_CALUR\nRPS6KA5\tA0A3Q7QYS2_VULVU\nRPS6KA5\tA0A3Q7R8C2_VULVU\nRPS6KA5\tA0A3Q7SAB5_VULVU\nRPS6KA5\tA0A3Q7X1Y3_URSAR\nRPS6KA5\tA0A3Q7XW04_URSAR\nRPS6KA5\tA0A452CG40_BALAS\nRPS6KA5\tA0A452EZT2_CAPHI\nRPS6KA5\tA0A452I9R7_9SAUR\nRPS6KA5\tA0A452I9S1_9SAUR\nRPS6KA5\tA0A452I9S7_9SAUR\nRPS6KA5\tA0A452I9S8_9SAUR\nRPS6KA5\tA0A452I9U1_9SAUR\nRPS6KA5\tA0A452I9U2_9SAUR\nRPS6KA5\tA0A452I9U9_9SAUR\nRPS6KA5\tA0A452I9V2_9SAUR\nRPS6KA5\tA0A452RFI4_URSAM\nRPS6KA5\tA0A452TJ60_URSMA\nRPS6KA5\tA0A452TJ85_URSMA\nRPS6KA5\tA0A452TJ93_URSMA\nRPS6KA5\tA0A452TJB4_URSMA\nRPS6KA5\tA0A452TJC9_URSMA\nRPS6KA5\tA0A452TJD0_URSMA\nRPS6KA5\tA0A452TJE5_URSMA\nRPS6KA5\tA0A452TJK2_URSMA\nRPS6KA5\tA0A452TJX1_URSMA\nRPS6KA5\tA0A452TK07_URSMA\nRPS6KA5\tA0A455BTR4_PHYMC\nRPS6KA5\tA0A455C847_PHYMC\nRPS6KA5\tA0A4C1SCX0_9NEOP\nRPS6KA5\tA0A4C1UIN0_9NEOP\nRPS6KA5\tA0A4C1ZSD6_9NEOP\nRPS6KA5\tA0A4C1ZUI3_9NEOP\nRPS6KA5\tA0A4W2D374_BOBOX\nRPS6KA5\tA0A4W2DIF7_BOBOX\nRPS6KA5\tA0A4W2G6H2_BOBOX\nRPS6KA5\tA0A4W2G6I4_BOBOX\nRPS6KA5\tA0A4X1SS38_PIG\nRPS6KA5\tA0A4X1STI8_PIG\nRPS6KA5\tA0A4X1STL4_PIG\nRPS6KA5\tA0A4X1STT8_PIG\nRPS6KA5\tA0A4X2L8Z7_VOMUR\nRPS6KA5\tA0A4Y2FHM4_ARAVE\nRPS6KA5\tA0A5B7CFU3_PORTR\nRPS6KA5\tA0A5F8GA19_MONDO\nRPS6KA5\tA0A5F9CES9_RABIT\nRPS6KA5\tA0A5F9D8L0_RABIT\nRPS6KA5\tA0A5F9DES2_RABIT\nRPS6KA5\tA0A5F9DLD2_RABIT\nRPS6KA5\tA0A5G2QU14_PIG\nRPS6KA5\tD3ZY17_RAT\nRPS6KA5\tE1BN61_BOVIN\nRPS6KA5\tE7FE05_DANRE\nRPS6KA5\tF1PMJ3_CANLF\nRPS6KA5\tF1SDA3_PIG\nRPS6KA5\tF6XUU5_HORSE\nRPS6KA5\tF7B9C0_ORNAN\nRPS6KA5\tF7FUC1_MONDO\nRPS6KA5\tF7G4I3_MACMU\nRPS6KA5\tF7HGG0_CALJA\nRPS6KA5\tG1KJL7_ANOCA\nRPS6KA5\tG1L4Q5_AILME\nRPS6KA5\tG1NK68_MELGA\nRPS6KA5\tG1P546_MYOLU\nRPS6KA5\tG1S5E3_NOMLE\nRPS6KA5\tG1SW53_RABIT\nRPS6KA5\tG3SL50_LOXAF\nRPS6KA5\tG3UN48_LOXAF\nRPS6KA5\tG3UR38_MELGA\nRPS6KA5\tG3V2D1_HUMAN\nRPS6KA5\tG3V4D7_HUMAN\nRPS6KA5\tG3WLS5_SARHA\nRPS6KA5\tH0VSH3_CAVPO\nRPS6KA5\tH0X9Z1_OTOGA\nRPS6KA5\tH0YJE1_HUMAN\nRPS6KA5\tH2NM09_PONAB\nRPS6KA5\tH2Q8S3_PANTR\nRPS6KA5\tH2TW15_TAKRU\nRPS6KA5\tH3ATP4_LATCH\nRPS6KA5\tH3ATP5_LATCH\nRPS6KA5\tH9EPT1_MACMU\nRPS6KA5\tI3LQZ5_PIG\nRPS6KA5\tI3MI29_ICTTR\nRPS6KA5\tK7DH81_PANTR\nRPS6KA5\tK7FNE5_PELSI\nRPS6KA5\tKS6A5_CHICK\nRPS6KA5\tKS6A5_HUMAN\nRPS6KA5\tKS6A5_MOUSE\nRPS6KA5\tKS6A5_PONAB\nRPS6KA5\tM3VWX0_FELCA\nRPS6KA5\tM3Y9I1_MUSPF\nRPS6KA5\tQ3TUH8_MOUSE\nRPS6KA5\tU3KCI1_FICAL\nRPS6KA5\tV8NUS2_OPHHA\nRPS6KA5\tW5P069_SHEEP\nRPS6KA5\tW5P073_SHEEP\n','From\tTo\nBMPR2\tA0A034W5E4_BACDO\nBMPR2\tA0A034W836_BACDO\nBMPR2\tA0A096MZD4_PAPAN\nBMPR2\tA0A0A1X7U5_ZEUCU\nBMPR2\tA0A0D9RDM2_CHLSB\nBMPR2\tA0A0K8VTF1_BACLA\nBMPR2\tA0A0N9NXS0_9CICH\nBMPR2\tA0A0S7FYV7_9TELE\nBMPR2\tA0A0S7FYW6_9TELE\nBMPR2\tA0A0S7G2H4_9TELE\nBMPR2\tA0A0S7HAL7_9TELE\nBMPR2\tA0A0S7IF39_9TELE\nBMPR2\tA0A0S7IJH3_9TELE\nBMPR2\tA0A0S7LRM3_9TELE\nBMPR2\tA0A0S7LRP8_9TELE\nBMPR2\tA0A0S7LRP9_9TELE\nBMPR2\tA0A0S7M5C9_9TELE\nBMPR2\tA0A0S7M5S5_9TELE\nBMPR2\tA0A0S7MBC8_9TELE\nBMPR2\tA0A0V0VC36_9BILA\nBMPR2\tA0A0V0VF33_9BILA\nBMPR2\tA0A0V0W6T3_9BILA\nBMPR2\tA0A0V1FD01_TRIPS\nBMPR2\tA0A151N9A5_ALLMI\nBMPR2\tA0A1A7XTF7_9TELE\nBMPR2\tA0A1A8A8N6_NOTFU\nBMPR2\tA0A1A8BXH3_9TELE\nBMPR2\tA0A1A8E029_9TELE\nBMPR2\tA0A1A8H7H8_9TELE\nBMPR2\tA0A1A8JCA8_NOTKU\nBMPR2\tA0A1A8LC68_9TELE\nBMPR2\tA0A1A8REK8_9TELE\nBMPR2\tA0A1D1XM31_9ARAE\nBMPR2\tA0A1L8EWX2_XENLA\nBMPR2\tA0A1S3F4A4_DIPOR\nBMPR2\tA0A1S3QF47_SALSA\nBMPR2\tA0A1S3W828_ERIEU\nBMPR2\tA0A1U7TZX1_TARSY\nBMPR2\tA0A1U8D3Y9_ALLSI\nBMPR2\tA0A1V4JVU3_PATFA\nBMPR2\tA0A1W2PPB5_HUMAN\nBMPR2\tA0A218UXA6_9PASE\nBMPR2\tA0A250Y6K0_CASCN\nBMPR2\tA0A286XKP8_CAVPO\nBMPR2\tA0A286XUH9_CAVPO\nBMPR2\tA0A2B4SU78_STYPI\nBMPR2\tA0A2I0LQA6_COLLI\nBMPR2\tA0A2I2U0V3_FELCA\nBMPR2\tA0A2I3FSJ4_NOMLE\nBMPR2\tA0A2I3G466_NOMLE\nBMPR2\tA0A2I3GXL8_NOMLE\nBMPR2\tA0A2I3H0C5_NOMLE\nBMPR2\tA0A2I3HEB5_NOMLE\nBMPR2\tA0A2I3HUB1_NOMLE\nBMPR2\tA0A2I3LN87_PAPAN\nBMPR2\tA0A2I3LWH9_PAPAN\nBMPR2\tA0A2I3MXI3_PAPAN\nBMPR2\tA0A2I3N3P5_PAPAN\nBMPR2\tA0A2I3NIH5_PAPAN\nBMPR2\tA0A2K5DRS0_AOTNA\nBMPR2\tA0A2K5DRT1_AOTNA\nBMPR2\tA0A2K5DS24_AOTNA\nBMPR2\tA0A2K5JTS6_COLAP\nBMPR2\tA0A2K5JTT2_COLAP\nBMPR2\tA0A2K5JTW2_COLAP\nBMPR2\tA0A2K5M0N9_CERAT\nBMPR2\tA0A2K5M0R3_CERAT\nBMPR2\tA0A2K5QF26_CEBCA\nBMPR2\tA0A2K5QF59_CEBCA\nBMPR2\tA0A2K5V0S3_MACFA\nBMPR2\tA0A2K5V0S9_MACFA\nBMPR2\tA0A2K5V0T2_MACFA\nBMPR2\tA0A2K5V0T5_MACFA\nBMPR2\tA0A2K5V0V9_MACFA\nBMPR2\tA0A2K5V0X2_MACFA\nBMPR2\tA0A2K5XE85_MANLE\nBMPR2\tA0A2K5XE93_MANLE\nBMPR2\tA0A2K5XE94_MANLE\nBMPR2\tA0A2K6C0I0_MACNE\nBMPR2\tA0A2K6C0J9_MACNE\nBMPR2\tA0A2K6C0K0_MACNE\nBMPR2\tA0A2K6C0K9_MACNE\nBMPR2\tA0A2K6C0L0_MACNE\nBMPR2\tA0A2K6C0L7_MACNE\nBMPR2\tA0A2K6FMM9_PROCO\nBMPR2\tA0A2K6FMN4_PROCO\nBMPR2\tA0A2K6FMN7_PROCO\nBMPR2\tA0A2K6FMP3_PROCO\nBMPR2\tA0A2K6KNJ8_RHIBE\nBMPR2\tA0A2K6KNL8_RHIBE\nBMPR2\tA0A2K6KNM1_RHIBE\nBMPR2\tA0A2K6KNM2_RHIBE\nBMPR2\tA0A2K6KNS0_RHIBE\nBMPR2\tA0A2K6KNS4_RHIBE\nBMPR2\tA0A2K6KNT4_RHIBE\nBMPR2\tA0A2K6KNY1_RHIBE\nBMPR2\tA0A2K6P1L9_RHIRO\nBMPR2\tA0A2K6P1M1_RHIRO\nBMPR2\tA0A2K6P1M7_RHIRO\nBMPR2\tA0A2K6TCR7_SAIBB\nBMPR2\tA0A2K6TCV1_SAIBB\nBMPR2\tA0A2K6TCV4_SAIBB\nBMPR2\tA0A2R8MXD7_CALJA\nBMPR2\tA0A2R8NAH4_CALJA\nBMPR2\tA0A2R8ZGK3_PANPA\nBMPR2\tA0A2R8ZH57_PANPA\nBMPR2\tA0A2R8ZJS7_PANPA\nBMPR2\tA0A2R8ZMW0_PANPA\nBMPR2\tA0A2U3VFL6_ODORO\nBMPR2\tA0A2U3XT63_LEPWE\nBMPR2\tA0A2U4BN67_TURTR\nBMPR2\tA0A2U4BNC6_TURTR\nBMPR2\tA0A2Y9I466_NEOSC\nBMPR2\tA0A2Y9MMB7_DELLE\nBMPR2\tA0A2Y9SPM8_PHYMC\nBMPR2\tA0A2Y9SWW4_PHYMC\nBMPR2\tA0A340Y6D0_LIPVE\nBMPR2\tA0A341AUA3_NEOAA\nBMPR2\tA0A384ATI5_BALAS\nBMPR2\tA0A384BSN6_URSMA\nBMPR2\tA0A3Q0CSR8_MESAU\nBMPR2\tA0A3Q0FYM0_ALLSI\nBMPR2\tA0A3Q1LP90_BOVIN\nBMPR2\tA0A3Q2L5V6_HORSE\nBMPR2\tA0A3Q7QQX6_CALUR\nBMPR2\tA0A3Q7QT48_CALUR\nBMPR2\tA0A3Q7TST8_VULVU\nBMPR2\tA0A3Q7WC88_URSAR\nBMPR2\tA0A452FD53_CAPHI\nBMPR2\tA0A452FDH9_CAPHI\nBMPR2\tA0A452J0C0_9SAUR\nBMPR2\tA0A452J0H4_9SAUR\nBMPR2\tA0A452SVE8_URSAM\nBMPR2\tA0A4C1YA56_9NEOP\nBMPR2\tA0A4W2CJQ4_BOBOX\nBMPR2\tA0A4W2EED7_BOBOX\nBMPR2\tA0A4W2EK72_BOBOX\nBMPR2\tA0A4W2IRF7_BOBOX\nBMPR2\tA0A4W2ITY8_BOBOX\nBMPR2\tA0A4W6EGA1_LATCA\nBMPR2\tA0A4X1V4U3_PIG\nBMPR2\tA0A4X1V8C6_PIG\nBMPR2\tA0A4X2K6T2_VOMUR\nBMPR2\tA0A5B7EHY4_PORTR\nBMPR2\tA0A5F4BPG7_CANLF\nBMPR2\tA0A5F5Q257_HORSE\nBMPR2\tA0A5F9CTC2_RABIT\nBMPR2\tA0A5F9D9H4_RABIT\nBMPR2\tA0A5G2QG49_PIG\nBMPR2\tA0A5G2QH51_PIG\nBMPR2\tA0A5G2R0W5_PIG\nBMPR2\tA0A5G2R9P4_PIG\nBMPR2\tA0A5G3L718_XENTR\nBMPR2\tA0A5G3L7Z4_XENTR\nBMPR2\tA0A5G3LA36_XENTR\nBMPR2\tB3GEL9_PIG\nBMPR2\tBMPR2_HUMAN\nBMPR2\tBMPR2_MOUSE\nBMPR2\tF1LQC5_RAT\nBMPR2\tF1MH00_BOVIN\nBMPR2\tF1PCC1_CANLF\nBMPR2\tF1SI16_PIG\nBMPR2\tF6Q787_XENTR\nBMPR2\tF6RIN4_CHICK\nBMPR2\tF6SUZ9_HORSE\nBMPR2\tF7AU53_MONDO\nBMPR2\tF7CLH1_XENTR\nBMPR2\tF7CLI5_XENTR\nBMPR2\tF7CRX8_XENTR\nBMPR2\tF7DKT4_ORNAN\nBMPR2\tF7EGF0_MACMU\nBMPR2\tF7H249_MACMU\nBMPR2\tF7HI91_CALJA\nBMPR2\tG1M3F0_AILME\nBMPR2\tG1N7Y4_MELGA\nBMPR2\tG1N888_MELGA\nBMPR2\tG1PQS5_MYOLU\nBMPR2\tG1R5S5_NOMLE\nBMPR2\tG1SGJ0_RABIT\nBMPR2\tG3TEP1_LOXAF\nBMPR2\tG3W7I5_SARHA\nBMPR2\tG3W7I6_SARHA\nBMPR2\tH0VEY4_CAVPO\nBMPR2\tH0WX01_OTOGA\nBMPR2\tH0ZIM1_TAEGU\nBMPR2\tH2LCZ6_ORYLA\nBMPR2\tH2LCZ8_ORYLA\nBMPR2\tH2P8B8_PONAB\nBMPR2\tH2R686_PANTR\nBMPR2\tH2SS86_TAKRU\nBMPR2\tH2SS87_TAKRU\nBMPR2\tH3B5H3_LATCH\nBMPR2\tH3B5H4_LATCH\nBMPR2\tH8YSF8_DICLA\nBMPR2\tH9G9P4_ANOCA\nBMPR2\tI3M095_ICTTR\nBMPR2\tK7DD64_PANTR\nBMPR2\tK7G4Z5_PELSI\nBMPR2\tK7G501_PELSI\nBMPR2\tM3WK25_FELCA\nBMPR2\tM3XMQ1_MUSPF\nBMPR2\tO42124_CHICK\nBMPR2\tP79954_XENLA\nBMPR2\tQ2Q1N5_CEREL\nBMPR2\tQ3UER5_MOUSE\nBMPR2\tQ3UG09_MOUSE\nBMPR2\tQ3UU71_MOUSE\nBMPR2\tR4GD98_ANOCA\nBMPR2\tU3DPK3_CALJA\nBMPR2\tU3IKP1_ANAPP\nBMPR2\tU3JN66_FICAL\nBMPR2\tV5JFE3_ANGAU\nBMPR2\tV8ND28_OPHHA\nBMPR2\tV8NZW0_OPHHA\nBMPR2\tV9HXG2_SHEEP\nBMPR2\tW5Q885_SHEEP\nBMPR2\tW5UFT6_ICTPU\nBMPR2\tW5UHT0_ICTPU\nBMPR2\tW8AW93_CERCA\nBMPR2\tX2CRM8_SPAAU\n','From\tTo\nTGFBR1\tA0A096P118_PAPAN\nTGFBR1\tA0A0B2V048_TOXCA\nTGFBR1\tA0A0D9RF96_CHLSB\nTGFBR1\tA0A0V0RVN2_9BILA\nTGFBR1\tA0A0V0S0N7_9BILA\nTGFBR1\tA0A0V0TV85_9BILA\nTGFBR1\tA0A0V0UBI1_9BILA\nTGFBR1\tA0A0V0UD44_9BILA\nTGFBR1\tA0A0V0WQX8_9BILA\nTGFBR1\tA0A0V0WR47_9BILA\nTGFBR1\tA0A0V0WR54_9BILA\nTGFBR1\tA0A0V0WRA0_9BILA\nTGFBR1\tA0A0V0WRA6_9BILA\nTGFBR1\tA0A0V0WYM4_9BILA\nTGFBR1\tA0A0V0XVA2_TRIPS\nTGFBR1\tA0A0V0Y9E0_TRIPS\nTGFBR1\tA0A0V0YA19_TRIPS\nTGFBR1\tA0A0V0YAC5_TRIPS\nTGFBR1\tA0A0V0ZBJ1_9BILA\nTGFBR1\tA0A0V0ZBV9_9BILA\nTGFBR1\tA0A0V1A0R2_9BILA\nTGFBR1\tA0A0V1B6W3_TRISP\nTGFBR1\tA0A0V1CME1_TRIBR\nTGFBR1\tA0A0V1CN83_TRIBR\nTGFBR1\tA0A0V1CNA9_TRIBR\nTGFBR1\tA0A0V1CNH4_TRIBR\nTGFBR1\tA0A0V1CNZ1_TRIBR\nTGFBR1\tA0A0V1EBL7_TRIPS\nTGFBR1\tA0A0V1EBR1_TRIPS\nTGFBR1\tA0A0V1EBS5_TRIPS\nTGFBR1\tA0A0V1FAD3_TRIPS\nTGFBR1\tA0A0V1FJ93_TRIPS\nTGFBR1\tA0A0V1H2S2_9BILA\nTGFBR1\tA0A0V1H3C3_9BILA\nTGFBR1\tA0A0V1H3L0_9BILA\nTGFBR1\tA0A0V1H400_9BILA\nTGFBR1\tA0A0V1IJL9_TRIPS\nTGFBR1\tA0A0V1IPJ4_TRIPS\nTGFBR1\tA0A0V1JXQ4_TRIPS\nTGFBR1\tA0A0V1JXR4_TRIPS\nTGFBR1\tA0A0V1L2Y5_9BILA\nTGFBR1\tA0A0V1L356_9BILA\nTGFBR1\tA0A0V1L499_9BILA\nTGFBR1\tA0A0V1M9F0_9BILA\nTGFBR1\tA0A0V1MWZ6_9BILA\nTGFBR1\tA0A0V1NRZ1_9BILA\nTGFBR1\tA0A0V1PC28_9BILA\nTGFBR1\tA0A146KLL6_LYGHE\nTGFBR1\tA0A173FE22_MARMO\nTGFBR1\tA0A1D5QUK7_MACMU\nTGFBR1\tA0A1D9CFA2_CASFI\nTGFBR1\tA0A1J0N264_CASFI\nTGFBR1\tA0A1J0N2E5_CASFI\nTGFBR1\tA0A1S3FQQ6_DIPOR\nTGFBR1\tA0A1S3WEU2_ERIEU\nTGFBR1\tA0A1U7QQK5_MESAU\nTGFBR1\tA0A1U7R104_MESAU\nTGFBR1\tA0A1U8CHK7_MESAU\nTGFBR1\tA0A1V4JV94_PATFA\nTGFBR1\tA0A218UZK9_9PASE\nTGFBR1\tA0A250YJ25_CASCN\nTGFBR1\tA0A286Y2I7_CAVPO\nTGFBR1\tA0A287BLR9_PIG\nTGFBR1\tA0A287CYZ5_ICTTR\nTGFBR1\tA0A287D4D4_ICTTR\nTGFBR1\tA0A2B4RNM4_STYPI\nTGFBR1\tA0A2B4SBV2_STYPI\nTGFBR1\tA0A2I0M4X2_COLLI\nTGFBR1\tA0A2I3G8Z0_NOMLE\nTGFBR1\tA0A2I3GNU8_NOMLE\nTGFBR1\tA0A2I3H152_NOMLE\nTGFBR1\tA0A2I3HC40_NOMLE\nTGFBR1\tA0A2I3HMM5_NOMLE\nTGFBR1\tA0A2I3HQ20_NOMLE\nTGFBR1\tA0A2I3HUG6_NOMLE\nTGFBR1\tA0A2I3LE16_PAPAN\nTGFBR1\tA0A2I3LUA8_PAPAN\nTGFBR1\tA0A2I3M6Q3_PAPAN\nTGFBR1\tA0A2I3M9J6_PAPAN\nTGFBR1\tA0A2I3MSS0_PAPAN\nTGFBR1\tA0A2I3N288_PAPAN\nTGFBR1\tA0A2I3NF04_PAPAN\nTGFBR1\tA0A2I3NGA1_PAPAN\nTGFBR1\tA0A2I3RCK1_PANTR\nTGFBR1\tA0A2I3RLL8_PANTR\nTGFBR1\tA0A2I3RZI1_PANTR\nTGFBR1\tA0A2I3S7X1_PANTR\nTGFBR1\tA0A2I3SFQ2_PANTR\nTGFBR1\tA0A2I3SKY8_PANTR\nTGFBR1\tA0A2I3SP42_PANTR\nTGFBR1\tA0A2I3T772_PANTR\nTGFBR1\tA0A2I3TJH3_PANTR\nTGFBR1\tA0A2J7RDI5_9NEOP\nTGFBR1\tA0A2K5CS88_AOTNA\nTGFBR1\tA0A2K5CS91_AOTNA\nTGFBR1\tA0A2K5CS96_AOTNA\nTGFBR1\tA0A2K5CSA5_AOTNA\nTGFBR1\tA0A2K5CSC9_AOTNA\nTGFBR1\tA0A2K5CSE5_AOTNA\nTGFBR1\tA0A2K5JLQ4_COLAP\nTGFBR1\tA0A2K5JLQ7_COLAP\nTGFBR1\tA0A2K5JLU3_COLAP\nTGFBR1\tA0A2K5JLU6_COLAP\nTGFBR1\tA0A2K5JLV0_COLAP\nTGFBR1\tA0A2K5JLV3_COLAP\nTGFBR1\tA0A2K5JLW0_COLAP\nTGFBR1\tA0A2K5LD97_CERAT\nTGFBR1\tA0A2K5LDR2_CERAT\nTGFBR1\tA0A2K5LE04_CERAT\nTGFBR1\tA0A2K5LE16_CERAT\nTGFBR1\tA0A2K5LE19_CERAT\nTGFBR1\tA0A2K5LE32_CERAT\nTGFBR1\tA0A2K5LE36_CERAT\nTGFBR1\tA0A2K5RQI5_CEBCA\nTGFBR1\tA0A2K5RQV9_CEBCA\nTGFBR1\tA0A2K5RQW5_CEBCA\nTGFBR1\tA0A2K5RQW9_CEBCA\nTGFBR1\tA0A2K5RQX0_CEBCA\nTGFBR1\tA0A2K5RQY0_CEBCA\nTGFBR1\tA0A2K5RQZ2_CEBCA\nTGFBR1\tA0A2K5RQZ9_CEBCA\nTGFBR1\tA0A2K5VRX0_MACFA\nTGFBR1\tA0A2K5VS44_MACFA\nTGFBR1\tA0A2K5VS75_MACFA\nTGFBR1\tA0A2K5VSA7_MACFA\nTGFBR1\tA0A2K5VSC7_MACFA\nTGFBR1\tA0A2K5VSD7_MACFA\nTGFBR1\tA0A2K5VSE5_MACFA\nTGFBR1\tA0A2K5VSI0_MACFA\nTGFBR1\tA0A2K5Z135_MANLE\nTGFBR1\tA0A2K5Z1A5_MANLE\nTGFBR1\tA0A2K5Z1A6_MANLE\nTGFBR1\tA0A2K5Z1D1_MANLE\nTGFBR1\tA0A2K5Z1D2_MANLE\nTGFBR1\tA0A2K5Z1D7_MANLE\nTGFBR1\tA0A2K5Z1E0_MANLE\nTGFBR1\tA0A2K5Z1E3_MANLE\nTGFBR1\tA0A2K6BUL9_MACNE\nTGFBR1\tA0A2K6BUU5_MACNE\nTGFBR1\tA0A2K6BV73_MACNE\nTGFBR1\tA0A2K6BV76_MACNE\nTGFBR1\tA0A2K6BV84_MACNE\nTGFBR1\tA0A2K6BV96_MACNE\nTGFBR1\tA0A2K6BV97_MACNE\nTGFBR1\tA0A2K6BVB3_MACNE\nTGFBR1\tA0A2K6BVB7_MACNE\nTGFBR1\tA0A2K6BVB8_MACNE\nTGFBR1\tA0A2K6FXH8_PROCO\nTGFBR1\tA0A2K6MED3_RHIBE\nTGFBR1\tA0A2K6MED7_RHIBE\nTGFBR1\tA0A2K6MED8_RHIBE\nTGFBR1\tA0A2K6MEE3_RHIBE\nTGFBR1\tA0A2K6MEF6_RHIBE\nTGFBR1\tA0A2K6MEG2_RHIBE\nTGFBR1\tA0A2K6MEG8_RHIBE\nTGFBR1\tA0A2K6MEJ7_RHIBE\nTGFBR1\tA0A2K6Q723_RHIRO\nTGFBR1\tA0A2K6Q727_RHIRO\nTGFBR1\tA0A2K6Q730_RHIRO\nTGFBR1\tA0A2K6Q731_RHIRO\nTGFBR1\tA0A2K6Q732_RHIRO\nTGFBR1\tA0A2K6Q737_RHIRO\nTGFBR1\tA0A2K6Q739_RHIRO\nTGFBR1\tA0A2K6Q744_RHIRO\nTGFBR1\tA0A2K6Q747_RHIRO\nTGFBR1\tA0A2K6SB79_SAIBB\nTGFBR1\tA0A2K6SBA3_SAIBB\nTGFBR1\tA0A2K6SBC1_SAIBB\nTGFBR1\tA0A2K6SBC4_SAIBB\nTGFBR1\tA0A2K6SBC7_SAIBB\nTGFBR1\tA0A2K6SBK1_SAIBB\nTGFBR1\tA0A2K6SBP2_SAIBB\nTGFBR1\tA0A2K6SBQ2_SAIBB\nTGFBR1\tA0A2K9Y5T5_BOSMU\nTGFBR1\tA0A2P8ZN60_BLAGE\nTGFBR1\tA0A2R8MQ84_CALJA\nTGFBR1\tA0A2R9ACY9_PANPA\nTGFBR1\tA0A2R9ACZ7_PANPA\nTGFBR1\tA0A2R9AD01_PANPA\nTGFBR1\tA0A2R9AD06_PANPA\nTGFBR1\tA0A2R9AD23_PANPA\nTGFBR1\tA0A2R9AD86_PANPA\nTGFBR1\tA0A2R9AE23_PANPA\nTGFBR1\tA0A2R9AE37_PANPA\nTGFBR1\tA0A2R9AE57_PANPA\nTGFBR1\tA0A2R9AJ89_PANPA\nTGFBR1\tA0A2S1UF15_CAPHI\nTGFBR1\tA0A2S1YEP4_CAPHI\nTGFBR1\tA0A2U3VCU1_ODORO\nTGFBR1\tA0A2U3ZET6_ODORO\nTGFBR1\tA0A2U4CGK0_TURTR\nTGFBR1\tA0A2U4CGL0_TURTR\nTGFBR1\tA0A2U4CGL3_TURTR\nTGFBR1\tA0A2Y9HDE3_NEOSC\nTGFBR1\tA0A2Y9QEC8_DELLE\nTGFBR1\tA0A2Y9QES0_DELLE\nTGFBR1\tA0A2Y9SUF7_PHYMC\nTGFBR1\tA0A2Y9SVU1_PHYMC\nTGFBR1\tA0A2Y9SYA1_PHYMC\nTGFBR1\tA0A2Y9T0D7_PHYMC\nTGFBR1\tA0A337SBT0_FELCA\nTGFBR1\tA0A337SQY1_FELCA\nTGFBR1\tA0A337SV88_FELCA\nTGFBR1\tA0A340XJZ0_LIPVE\nTGFBR1\tA0A341AJY0_NEOAA\nTGFBR1\tA0A341AKW2_NEOAA\nTGFBR1\tA0A384BBR1_BALAS\nTGFBR1\tA0A384DGZ1_URSMA\nTGFBR1\tA0A3G5E9S7_HAECO\nTGFBR1\tA0A3G5E9U6_HAECO\nTGFBR1\tA0A3Q0DSX6_TARSY\nTGFBR1\tA0A3Q0G355_ALLSI\nTGFBR1\tA0A3Q1LQ16_BOVIN\nTGFBR1\tA0A3Q1MQ56_BOVIN\nTGFBR1\tA0A3Q2HN32_HORSE\nTGFBR1\tA0A3Q2IKY4_HORSE\nTGFBR1\tA0A3Q7QBC7_CALUR\nTGFBR1\tA0A3Q7QE05_CALUR\nTGFBR1\tA0A3Q7TUV3_VULVU\nTGFBR1\tA0A3Q7WK46_URSAR\nTGFBR1\tA0A3Q7XHE1_URSAR\nTGFBR1\tA0A411EWG5_PENVA\nTGFBR1\tA0A419PNJ3_CLOSI\nTGFBR1\tA0A452FNE6_CAPHI\nTGFBR1\tA0A452FNE7_CAPHI\nTGFBR1\tA0A452FNF4_CAPHI\nTGFBR1\tA0A452FNF9_CAPHI\nTGFBR1\tA0A452HIT2_9SAUR\nTGFBR1\tA0A452RDR1_URSAM\nTGFBR1\tA0A452RDU3_URSAM\nTGFBR1\tA0A452U8R0_URSMA\nTGFBR1\tA0A452U8Y8_URSMA\nTGFBR1\tA0A4W2CMT7_BOBOX\nTGFBR1\tA0A4W2H6X7_BOBOX\nTGFBR1\tA0A4W4GIM7_ELEEL\nTGFBR1\tA0A4W4GKD1_ELEEL\nTGFBR1\tA0A4W4GKD4_ELEEL\nTGFBR1\tA0A4W4GNB0_ELEEL\nTGFBR1\tA0A4W4GNJ9_ELEEL\nTGFBR1\tA0A4X1SN23_PIG\nTGFBR1\tA0A4X1SNS3_PIG\nTGFBR1\tA0A4X1SQN6_PIG\nTGFBR1\tA0A4X1SR87_PIG\nTGFBR1\tA0A5F4CRQ8_CANLF\nTGFBR1\tA0A5G3K5P7_XENTR\nTGFBR1\tA0A5G3K5T5_XENTR\nTGFBR1\tA0A5G3K6A7_XENTR\nTGFBR1\tA0A5G3K6L5_XENTR\nTGFBR1\tA0A5G3K762_XENTR\nTGFBR1\tA0A5G3K7H9_XENTR\nTGFBR1\tA6MIV6_HUMAN\nTGFBR1\tA6MIV7_HUMAN\nTGFBR1\tB0EXP6_DANRE\nTGFBR1\tB1WAU7_XENTR\nTGFBR1\tB4DY26_HUMAN\nTGFBR1\tE9Q418_MOUSE\nTGFBR1\tF1PS63_CANLF\nTGFBR1\tF6Q736_PIG\nTGFBR1\tF6V2S7_XENTR\nTGFBR1\tF6VGC3_MONDO\nTGFBR1\tF6X0S8_ORNAN\nTGFBR1\tF6XTG5_XENTR\nTGFBR1\tF6XUH4_XENTR\nTGFBR1\tF7BNR8_MACMU\nTGFBR1\tF7H0K3_MACMU\nTGFBR1\tF7H171_CALJA\nTGFBR1\tF7H653_CALJA\nTGFBR1\tF8J3B8_OREMO\nTGFBR1\tF8VRH6_HUMAN\nTGFBR1\tF8VVC4_HUMAN\nTGFBR1\tF8VXZ5_HUMAN\nTGFBR1\tF8W0K6_HUMAN\nTGFBR1\tF8W1R9_HUMAN\nTGFBR1\tG1KIM7_ANOCA\nTGFBR1\tG1LCI9_AILME\nTGFBR1\tG1MT75_MELGA\nTGFBR1\tG1PRA8_MYOLU\nTGFBR1\tG1S5B6_NOMLE\nTGFBR1\tG1TBB8_RABIT\nTGFBR1\tG3T232_LOXAF\nTGFBR1\tG3U5X5_LOXAF\nTGFBR1\tG3WW65_SARHA\nTGFBR1\tH0VS30_CAVPO\nTGFBR1\tH0WPB6_OTOGA\nTGFBR1\tH0YHS4_HUMAN\nTGFBR1\tH2PSV9_PONAB\nTGFBR1\tH2QXK2_PANTR\nTGFBR1\tH2SFY4_TAKRU\nTGFBR1\tH3AEX5_LATCH\nTGFBR1\tH3AEX6_LATCH\nTGFBR1\tH9FBZ3_MACMU\nTGFBR1\tI3ML93_ICTTR\nTGFBR1\tJ9NST9_CANLF\nTGFBR1\tK7A2F9_PANTR\nTGFBR1\tK7B517_PANTR\nTGFBR1\tK7CDZ3_PANTR\nTGFBR1\tK7DBV5_PANTR\nTGFBR1\tK7GJF5_PELSI\nTGFBR1\tK7GNG9_PIG\nTGFBR1\tK7GSM3_PIG\nTGFBR1\tK9M283_9BIVA\nTGFBR1\tM0R9M6_RAT\nTGFBR1\tM3Y5M1_MUSPF\nTGFBR1\tQ06900_CHICK\nTGFBR1\tQ19ML3_PIG\nTGFBR1\tQ19ML6_PIG\nTGFBR1\tQ19ML7_PIG\nTGFBR1\tQ19ML9_PIG\nTGFBR1\tQ28E60_XENTR\nTGFBR1\tQ32LW6_DANRE\nTGFBR1\tQ3KS05_HORSE\nTGFBR1\tQ3U485_MOUSE\nTGFBR1\tQ4FJL1_MOUSE\nTGFBR1\tQ5CD20_PIG\nTGFBR1\tQ5CD21_PIG\nTGFBR1\tQ5M9H3_RAT\nTGFBR1\tQ5MPP9_SHEEP\nTGFBR1\tQ5T7S2_HUMAN\nTGFBR1\tQ702V6_PIG\nTGFBR1\tQ702V7_PIG\nTGFBR1\tQ91595_XENLA\nTGFBR1\tQ9CVP4_MOUSE\nTGFBR1\tQ9D5H8_MOUSE\nTGFBR1\tT2MFK9_HYDVU\nTGFBR1\tTGFR1_BOVIN\nTGFBR1\tTGFR1_HUMAN\nTGFBR1\tTGFR1_MOUSE\nTGFBR1\tTGFR1_PIG\nTGFBR1\tTGFR1_RAT\nTGFBR1\tU3IR17_ANAPP\nTGFBR1\tU3KFR0_FICAL\nTGFBR1\tV8NXV1_OPHHA\nTGFBR1\tW5PMT5_SHEEP\nTGFBR1\tW5UIN1_ICTPU\n','From\tTo\nBMPR1B\tA0A096N2I0_PAPAN\nBMPR1B\tA0A0A1WGT5_ZEUCU\nBMPR1B\tA0A0A9Y2K4_LYGHE\nBMPR1B\tA0A0B5GET5_BUBBU\nBMPR1B\tA0A0C9RSR1_9HYME\nBMPR1B\tA0A0D9QVV7_CHLSB\nBMPR1B\tA0A0E3N966_SHEEP\nBMPR1B\tA0A0V0S576_9BILA\nBMPR1B\tA0A0V0UAC6_9BILA\nBMPR1B\tA0A0V0X3A5_9BILA\nBMPR1B\tA0A0V0ZM91_9BILA\nBMPR1B\tA0A0V1CP12_TRIBR\nBMPR1B\tA0A0V1H679_9BILA\nBMPR1B\tA0A0V1IP86_TRIPS\nBMPR1B\tA0A0V1JPI1_TRIPS\nBMPR1B\tA0A0V1LMM5_9BILA\nBMPR1B\tA0A0V1MSP0_9BILA\nBMPR1B\tA0A0V1MSX5_9BILA\nBMPR1B\tA0A0V1MT86_9BILA\nBMPR1B\tA0A0V1P2F7_9BILA\nBMPR1B\tA0A142FHG7_9CICH\nBMPR1B\tA0A1D5QK37_MACMU\nBMPR1B\tA0A1L5BX17_CASFI\nBMPR1B\tA0A1S2ZU53_ERIEU\nBMPR1B\tA0A1S3G8M5_DIPOR\nBMPR1B\tA0A1S3GAY2_DIPOR\nBMPR1B\tA0A1S3PJZ0_SALSA\nBMPR1B\tA0A1S3PK33_SALSA\nBMPR1B\tA0A1U7SIG6_ALLSI\nBMPR1B\tA0A1U7SQX7_ALLSI\nBMPR1B\tA0A1U7TES7_TARSY\nBMPR1B\tA0A1U8CD62_MESAU\nBMPR1B\tA0A1V4JTZ4_PATFA\nBMPR1B\tA0A218V2B4_9PASE\nBMPR1B\tA0A286XCP6_CAVPO\nBMPR1B\tA0A2B4SW39_STYPI\nBMPR1B\tA0A2D0SPZ4_ICTPU\nBMPR1B\tA0A2D0SQ02_ICTPU\nBMPR1B\tA0A2D0SR58_ICTPU\nBMPR1B\tA0A2D0SR63_ICTPU\nBMPR1B\tA0A2I0M921_COLLI\nBMPR1B\tA0A2I2UAH1_FELCA\nBMPR1B\tA0A2I2ULJ9_FELCA\nBMPR1B\tA0A2I2UWR4_FELCA\nBMPR1B\tA0A2I2V488_FELCA\nBMPR1B\tA0A2I3HUQ3_NOMLE\nBMPR1B\tA0A2I3LDV3_PAPAN\nBMPR1B\tA0A2I3SZW7_PANTR\nBMPR1B\tA0A2I4D0F2_9TELE\nBMPR1B\tA0A2K5CSR4_AOTNA\nBMPR1B\tA0A2K5CSS8_AOTNA\nBMPR1B\tA0A2K5HUU0_COLAP\nBMPR1B\tA0A2K5HV09_COLAP\nBMPR1B\tA0A2K5HV35_COLAP\nBMPR1B\tA0A2K5LBZ9_CERAT\nBMPR1B\tA0A2K5LC35_CERAT\nBMPR1B\tA0A2K5LC51_CERAT\nBMPR1B\tA0A2K5QLX7_CEBCA\nBMPR1B\tA0A2K5QLY9_CEBCA\nBMPR1B\tA0A2K5QM21_CEBCA\nBMPR1B\tA0A2K5W4N9_MACFA\nBMPR1B\tA0A2K5W4V9_MACFA\nBMPR1B\tA0A2K5YT18_MANLE\nBMPR1B\tA0A2K5YT56_MANLE\nBMPR1B\tA0A2K5YT67_MANLE\nBMPR1B\tA0A2K6DFG4_MACNE\nBMPR1B\tA0A2K6DFH4_MACNE\nBMPR1B\tA0A2K6FLX1_PROCO\nBMPR1B\tA0A2K6FLX2_PROCO\nBMPR1B\tA0A2K6MHF0_RHIBE\nBMPR1B\tA0A2K6MHI4_RHIBE\nBMPR1B\tA0A2K6PW17_RHIRO\nBMPR1B\tA0A2K6PW22_RHIRO\nBMPR1B\tA0A2K6T526_SAIBB\nBMPR1B\tA0A2K6T548_SAIBB\nBMPR1B\tA0A2P8ZEI5_BLAGE\nBMPR1B\tA0A2R8ZLH8_PANPA\nBMPR1B\tA0A2R8ZP92_PANPA\nBMPR1B\tA0A2S2P2D3_SCHGA\nBMPR1B\tA0A2S2R5I1_9HEMI\nBMPR1B\tA0A2U3VAR8_ODORO\nBMPR1B\tA0A2U3VAS7_ODORO\nBMPR1B\tA0A2U3XQ65_LEPWE\nBMPR1B\tA0A2U3XQB2_LEPWE\nBMPR1B\tA0A2U4ALZ8_TURTR\nBMPR1B\tA0A2U4AM02_TURTR\nBMPR1B\tA0A2Y9F7N1_PHYMC\nBMPR1B\tA0A2Y9GDC7_NEOSC\nBMPR1B\tA0A2Y9NSQ4_DELLE\nBMPR1B\tA0A2Y9NVL6_DELLE\nBMPR1B\tA0A2Y9NYN1_DELLE\nBMPR1B\tA0A2Y9P774_DELLE\nBMPR1B\tA0A2Y9P783_DELLE\nBMPR1B\tA0A340XSX9_LIPVE\nBMPR1B\tA0A340XT35_LIPVE\nBMPR1B\tA0A340XW73_LIPVE\nBMPR1B\tA0A341AMD9_NEOAA\nBMPR1B\tA0A341AMP9_NEOAA\nBMPR1B\tA0A341AMQ4_NEOAA\nBMPR1B\tA0A384A9X1_BALAS\nBMPR1B\tA0A384DEE9_URSMA\nBMPR1B\tA0A384DEG1_URSMA\nBMPR1B\tA0A384DF28_URSMA\nBMPR1B\tA0A3B3U2Z5_9TELE\nBMPR1B\tA0A3B3VH12_9TELE\nBMPR1B\tA0A3B4B5Z7_9GOBI\nBMPR1B\tA0A3B4VH66_SERDU\nBMPR1B\tA0A3B4ZWA2_9TELE\nBMPR1B\tA0A3P8VY48_CYNSE\nBMPR1B\tA0A3Q1D304_AMPOC\nBMPR1B\tA0A3Q1LSA1_BOVIN\nBMPR1B\tA0A3Q1MKC5_BOVIN\nBMPR1B\tA0A3Q1MUE8_BOVIN\nBMPR1B\tA0A3Q2DUP2_CYPVA\nBMPR1B\tA0A3Q2I2T7_HORSE\nBMPR1B\tA0A3Q2IDY5_HORSE\nBMPR1B\tA0A3Q2KTF4_HORSE\nBMPR1B\tA0A3Q2XPG3_HIPCM\nBMPR1B\tA0A3Q7P6Y4_CALUR\nBMPR1B\tA0A3Q7PGW5_CALUR\nBMPR1B\tA0A3Q7U194_VULVU\nBMPR1B\tA0A3Q7UDX3_VULVU\nBMPR1B\tA0A3Q7UVL1_VULVU\nBMPR1B\tA0A3Q7WK59_URSAR\nBMPR1B\tA0A3Q7XR92_URSAR\nBMPR1B\tA0A3R7GTX1_CLOSI\nBMPR1B\tA0A452F1S6_CAPHI\nBMPR1B\tA0A452F241_CAPHI\nBMPR1B\tA0A452F280_CAPHI\nBMPR1B\tA0A452F2E9_CAPHI\nBMPR1B\tA0A452HGP1_9SAUR\nBMPR1B\tA0A452SJU6_URSAM\nBMPR1B\tA0A455BER5_PHYMC\nBMPR1B\tA0A455BFA8_PHYMC\nBMPR1B\tA0A455BFD5_PHYMC\nBMPR1B\tA0A455BFE6_PHYMC\nBMPR1B\tA0A455BFE9_PHYMC\nBMPR1B\tA0A455BUF3_PHYMC\nBMPR1B\tA0A4W2DK56_BOBOX\nBMPR1B\tA0A4W2GBZ1_BOBOX\nBMPR1B\tA0A4W2GCR8_BOBOX\nBMPR1B\tA0A4W2HX00_BOBOX\nBMPR1B\tA0A4W2HX12_BOBOX\nBMPR1B\tA0A4W2I4S4_BOBOX\nBMPR1B\tA0A4W4F596_ELEEL\nBMPR1B\tA0A4W4F6W1_ELEEL\nBMPR1B\tA0A4W4F7G6_ELEEL\nBMPR1B\tA0A4W4F841_ELEEL\nBMPR1B\tA0A4W6GCQ2_LATCA\nBMPR1B\tA0A4X1UEW4_PIG\nBMPR1B\tA0A4X1UG64_PIG\nBMPR1B\tA0A4X1UG71_PIG\nBMPR1B\tA0A4X2K0N9_VOMUR\nBMPR1B\tA0A4Y2FDP4_ARAVE\nBMPR1B\tA0A5B7DDQ1_PORTR\nBMPR1B\tA0A5F4BTE9_CANLF\nBMPR1B\tA0A5F8HAK1_MONDO\nBMPR1B\tA0A5F9DCV6_RABIT\nBMPR1B\tA0A5G3K811_XENTR\nBMPR1B\tA0A5G3K824_XENTR\nBMPR1B\tA0A5G3KA15_XENTR\nBMPR1B\tA0A5G3KAJ0_XENTR\nBMPR1B\tA0A5G3KAL7_XENTR\nBMPR1B\tA0A5G3L8X5_XENTR\nBMPR1B\tA0A5K1UKI8_PIG\nBMPR1B\tA7YWG6_BOVIN\nBMPR1B\tB2CSP0_SHEEP\nBMPR1B\tB2RRZ4_MOUSE\nBMPR1B\tB3V7E3_BOVIN\nBMPR1B\tB5AFX6_CAPHI\nBMPR1B\tBMR1B_CHICK\nBMPR1B\tBMR1B_HUMAN\nBMPR1B\tBMR1B_MOUSE\nBMPR1B\tC9DF84_CAPHI\nBMPR1B\tC9VZX1_BOVIN\nBMPR1B\tC9VZX2_BOVIN\nBMPR1B\tD0ENQ3_SHEEP\nBMPR1B\tD0ENQ5_SHEEP\nBMPR1B\tD1M6Z9_SHEEP\nBMPR1B\tD1M700_SHEEP\nBMPR1B\tD6Q1N2_BUBBU\nBMPR1B\tD6RET5_MOUSE\nBMPR1B\tD6RGW8_HUMAN\nBMPR1B\tE2RA74_CANLF\nBMPR1B\tE6YCX6_OREMO\nBMPR1B\tF1NTD3_CHICK\nBMPR1B\tF6Q8L1_MONDO\nBMPR1B\tF6REM9_CALJA\nBMPR1B\tF6SYZ2_HORSE\nBMPR1B\tF6W0R7_XENTR\nBMPR1B\tF6ZJS8_XENTR\nBMPR1B\tF7A5W4_ORNAN\nBMPR1B\tF7HJS7_MACMU\nBMPR1B\tF7IS54_CALJA\nBMPR1B\tG1KP73_ANOCA\nBMPR1B\tG1LH84_AILME\nBMPR1B\tG1NF29_MELGA\nBMPR1B\tG1PCW7_MYOLU\nBMPR1B\tG1RVB7_NOMLE\nBMPR1B\tG1RVC0_NOMLE\nBMPR1B\tG1T8L9_RABIT\nBMPR1B\tG3R8S9_GORGO\nBMPR1B\tG3T672_LOXAF\nBMPR1B\tG3USG9_MELGA\nBMPR1B\tG3WBC1_SARHA\nBMPR1B\tG3WBC2_SARHA\nBMPR1B\tG7P5Y0_MACFA\nBMPR1B\tG9F9A1_TRISP\nBMPR1B\tH0UWM6_CAVPO\nBMPR1B\tH0WM21_OTOGA\nBMPR1B\tH2PDX5_PONAB\nBMPR1B\tH2QPW8_PANTR\nBMPR1B\tH2ZVV5_LATCH\nBMPR1B\tI0FFR8_MACMU\nBMPR1B\tI3MPB5_ICTTR\nBMPR1B\tJ7SBL8_CAPHI\nBMPR1B\tK7FD35_PELSI\nBMPR1B\tK7FD41_PELSI\nBMPR1B\tK7TGV0_CAPHI\nBMPR1B\tM0R3J4_RAT\nBMPR1B\tM3XFJ2_FELCA\nBMPR1B\tM3XSM6_MUSPF\nBMPR1B\tQ0IHM5_XENTR\nBMPR1B\tQ3USS1_MOUSE\nBMPR1B\tQ569A5_RAT\nBMPR1B\tQ95L23_PIG\nBMPR1B\tQ9W629_DANRE\nBMPR1B\tU3I6E9_ANAPP\nBMPR1B\tU3JUC2_FICAL\nBMPR1B\tU3PT42_PINFU\nBMPR1B\tV8PDT6_OPHHA\nBMPR1B\tW0NTQ8_CAPHI\nBMPR1B\tW5Q6S5_SHEEP\nBMPR1B\tW5UBH6_ICTPU\n','From\tTo\nMERTK\tA0A087WZQ5_HUMAN\nMERTK\tA0A096N5V8_PAPAN\nMERTK\tA0A096NFF2_PAPAN\nMERTK\tA0A0B8RVQ2_PIG\nMERTK\tA0A0D9RW84_CHLSB\nMERTK\tA0A0P6J3I5_HETGA\nMERTK\tA0A0S7LBQ0_9TELE\nMERTK\tA0A0S7LBW2_9TELE\nMERTK\tA0A0S7LCW3_9TELE\nMERTK\tA0A0S7M702_9TELE\nMERTK\tA0A151MV45_ALLMI\nMERTK\tA0A1A7XKK1_9TELE\nMERTK\tA0A1A7XZF7_9TELE\nMERTK\tA0A1A7ZXT6_NOTFU\nMERTK\tA0A1A8C6V9_9TELE\nMERTK\tA0A1A8DZ92_9TELE\nMERTK\tA0A1A8EQT1_9TELE\nMERTK\tA0A1A8J2Z4_NOTKU\nMERTK\tA0A1A8K275_NOTKU\nMERTK\tA0A1A8L3G1_9TELE\nMERTK\tA0A1A8NJQ6_9TELE\nMERTK\tA0A1A8NJY9_9TELE\nMERTK\tA0A1A8QJ28_9TELE\nMERTK\tA0A1A8V1U6_NOTFU\nMERTK\tA0A1D5PMV1_CHICK\nMERTK\tA0A1D5PUU0_CHICK\nMERTK\tA0A1D5QYP2_MACMU\nMERTK\tA0A1L5L8Q5_RAT\nMERTK\tA0A1L8G7B7_XENLA\nMERTK\tA0A1S3FGX0_DIPOR\nMERTK\tA0A1S3WMX7_ERIEU\nMERTK\tA0A1U7S3K8_ALLSI\nMERTK\tA0A1U7S623_ALLSI\nMERTK\tA0A1U7TQD8_TARSY\nMERTK\tA0A1U8BNU6_MESAU\nMERTK\tA0A1V4IG40_PATFA\nMERTK\tA0A1V4IG41_PATFA\nMERTK\tA0A1V4IG81_PATFA\nMERTK\tA0A218UIJ1_9PASE\nMERTK\tA0A287A2E8_PIG\nMERTK\tA0A287CUK3_ICTTR\nMERTK\tA0A2D0QNU4_ICTPU\nMERTK\tA0A2D0QPB5_ICTPU\nMERTK\tA0A2I0M9H2_COLLI\nMERTK\tA0A2I3SHV9_PANTR\nMERTK\tA0A2I4AP31_9TELE\nMERTK\tA0A2K5EPN5_AOTNA\nMERTK\tA0A2K5EPP6_AOTNA\nMERTK\tA0A2K5HKS2_COLAP\nMERTK\tA0A2K5HKU3_COLAP\nMERTK\tA0A2K5MDM8_CERAT\nMERTK\tA0A2K5MDN3_CERAT\nMERTK\tA0A2K5MDR5_CERAT\nMERTK\tA0A2K5QPQ0_CEBCA\nMERTK\tA0A2K5V1H9_MACFA\nMERTK\tA0A2K5V1K4_MACFA\nMERTK\tA0A2K5V1L4_MACFA\nMERTK\tA0A2K5ZXC3_MANLE\nMERTK\tA0A2K5ZXE5_MANLE\nMERTK\tA0A2K6CCZ1_MACNE\nMERTK\tA0A2K6CD17_MACNE\nMERTK\tA0A2K6EP21_PROCO\nMERTK\tA0A2K6L763_RHIBE\nMERTK\tA0A2K6L7A3_RHIBE\nMERTK\tA0A2K6L7B0_RHIBE\nMERTK\tA0A2K6RNA4_RHIRO\nMERTK\tA0A2K6RNA7_RHIRO\nMERTK\tA0A2K6RNC1_RHIRO\nMERTK\tA0A2K6RNC4_RHIRO\nMERTK\tA0A2K6TW62_SAIBB\nMERTK\tA0A2K6TW83_SAIBB\nMERTK\tA0A2R9AB01_PANPA\nMERTK\tA0A2U3VWW5_ODORO\nMERTK\tA0A2U3Y127_LEPWE\nMERTK\tA0A2U4B024_TURTR\nMERTK\tA0A2Y9HUM5_NEOSC\nMERTK\tA0A2Y9QH76_DELLE\nMERTK\tA0A2Y9T9U6_PHYMC\nMERTK\tA0A340YFQ2_LIPVE\nMERTK\tA0A341BA76_NEOAA\nMERTK\tA0A384BCJ6_BALAS\nMERTK\tA0A384BCL6_BALAS\nMERTK\tA0A384BDA8_BALAS\nMERTK\tA0A384CW94_URSMA\nMERTK\tA0A3B3IG08_ORYLA\nMERTK\tA0A3Q0CMR7_MESAU\nMERTK\tA0A3Q2I0E2_HORSE\nMERTK\tA0A3Q7NQP2_CALUR\nMERTK\tA0A3Q7S5V6_VULVU\nMERTK\tA0A3Q7U7H4_URSAR\nMERTK\tA0A452FV68_CAPHI\nMERTK\tA0A452HL88_9SAUR\nMERTK\tA0A452HL99_9SAUR\nMERTK\tA0A452HLA1_9SAUR\nMERTK\tA0A452HLA6_9SAUR\nMERTK\tA0A452HLB3_9SAUR\nMERTK\tA0A452HLD9_9SAUR\nMERTK\tA0A452HLH9_9SAUR\nMERTK\tA0A452QEL5_URSAM\nMERTK\tA0A452TTT2_URSMA\nMERTK\tA0A452TTT7_URSMA\nMERTK\tA0A452TTV8_URSMA\nMERTK\tA0A452TTW5_URSMA\nMERTK\tA0A452TTX6_URSMA\nMERTK\tA0A452TU00_URSMA\nMERTK\tA0A4W2CY96_BOBOX\nMERTK\tA0A4W2CYA7_BOBOX\nMERTK\tA0A4W2D2Q1_BOBOX\nMERTK\tA0A4W2HW46_BOBOX\nMERTK\tA0A4W3IYP9_CALMI\nMERTK\tA0A4W3J9C0_CALMI\nMERTK\tA0A4W3J9D1_CALMI\nMERTK\tA0A4W3JRT0_CALMI\nMERTK\tA0A4X1VJC4_PIG\nMERTK\tA0A4X1VJD9_PIG\nMERTK\tA0A4X1VL26_PIG\nMERTK\tA0A4X2L7H2_VOMUR\nMERTK\tA0A4X2LIP0_VOMUR\nMERTK\tA0A4Z2HPH7_9TELE\nMERTK\tA0A5F5Y369_FELCA\nMERTK\tA0A5F8G245_MONDO\nMERTK\tA0A5F8HEC7_MONDO\nMERTK\tA0A5F8HGC6_MONDO\nMERTK\tA0A5G2QN46_PIG\nMERTK\tA0A5G3I3X9_XENTR\nMERTK\tA0A5G3I5G1_XENTR\nMERTK\tA0A5G3JYS3_XENTR\nMERTK\tD2HPH1_AILME\nMERTK\tE9PD22_HUMAN\nMERTK\tE9PHX8_HUMAN\nMERTK\tF1N381_BOVIN\nMERTK\tF1P3V0_CHICK\nMERTK\tF1SU88_PIG\nMERTK\tF6T7N0_MACMU\nMERTK\tF6Y5H9_HORSE\nMERTK\tF6ZJI2_MONDO\nMERTK\tF7AQA7_CALJA\nMERTK\tF7DRB6_XENTR\nMERTK\tF7DVZ6_XENTR\nMERTK\tF7DWH1_XENTR\nMERTK\tG1MVH9_MELGA\nMERTK\tG1PWB6_MYOLU\nMERTK\tG1QLP6_NOMLE\nMERTK\tG1U620_RABIT\nMERTK\tG3THB0_LOXAF\nMERTK\tH0UVA1_CAVPO\nMERTK\tH0WKZ3_OTOGA\nMERTK\tH0ZAR8_TAEGU\nMERTK\tH2P5E7_PONAB\nMERTK\tH2QIJ5_PANTR\nMERTK\tH2U232_TAKRU\nMERTK\tH3AMI0_LATCH\nMERTK\tH7C3L9_HUMAN\nMERTK\tH9F4R6_MACMU\nMERTK\tH9FDD5_MACMU\nMERTK\tH9GAY1_ANOCA\nMERTK\tI3M7K8_ICTTR\nMERTK\tJ9P347_CANLF\nMERTK\tK7FP08_PELSI\nMERTK\tL7N3K7_XENTR\nMERTK\tM3W8Y3_FELCA\nMERTK\tM3YDN7_MUSPF\nMERTK\tMERTK_HUMAN\nMERTK\tMERTK_MOUSE\nMERTK\tMERTK_RAT\nMERTK\tQ1RMG3_HUMAN\nMERTK\tQ28G68_XENTR\nMERTK\tQ53QW0_HUMAN\nMERTK\tQ53R53_HUMAN\nMERTK\tQ8C584_MOUSE\nMERTK\tQ8CE52_MOUSE\nMERTK\tQ9JL63_RAT\nMERTK\tU3KAK9_FICAL\nMERTK\tU3KAL4_FICAL\nMERTK\tU6CUT8_NEOVI\nMERTK\tV8P5E3_OPHHA\nMERTK\tW5PYE8_SHEEP\nMERTK\tW5PYF1_SHEEP\n','From\tTo\nRAF1\tA0A024A3Q9_GOBRA\nRAF1\tA0A024R2G5_HUMAN\nRAF1\tA0A088ACF7_APIME\nRAF1\tA0A0B4J1W9_HUMAN\nRAF1\tA0A0D9RB89_CHLSB\nRAF1\tA0A0F7PZM3_9SACH\nRAF1\tA0A0N4SV36_MOUSE\nRAF1\tA0A0N4SWH5_MOUSE\nRAF1\tA0A0P6J7B2_HETGA\nRAF1\tA0A0S2Z4L5_HUMAN\nRAF1\tA0A0S2Z559_HUMAN\nRAF1\tA0A0S7HNI8_9TELE\nRAF1\tA0A0S7HNL6_9TELE\nRAF1\tA0A0S7HNR5_9TELE\nRAF1\tA0A0S7HNU4_9TELE\nRAF1\tA0A0S7HNU6_9TELE\nRAF1\tA0A0S7HQY5_9TELE\nRAF1\tA0A0S7HRL2_9TELE\nRAF1\tA0A0S7HS52_9TELE\nRAF1\tA0A0S7HSC3_9TELE\nRAF1\tA0A0S7HTC4_9TELE\nRAF1\tA0A0S7HTQ3_9TELE\nRAF1\tA0A0X3PN88_SCHSO\nRAF1\tA0A1A7WW20_9TELE\nRAF1\tA0A1A7XA70_9TELE\nRAF1\tA0A1A7XRH2_9TELE\nRAF1\tA0A1A7ZNZ9_NOTFU\nRAF1\tA0A1A8A4X0_NOTFU\nRAF1\tA0A1A8AA18_NOTFU\nRAF1\tA0A1A8CIG7_9TELE\nRAF1\tA0A1A8CK35_9TELE\nRAF1\tA0A1A8CY57_9TELE\nRAF1\tA0A1A8D682_9TELE\nRAF1\tA0A1A8FES5_9TELE\nRAF1\tA0A1A8FFR6_9TELE\nRAF1\tA0A1A8H201_9TELE\nRAF1\tA0A1A8HB99_9TELE\nRAF1\tA0A1A8INN9_NOTKU\nRAF1\tA0A1A8J6I5_NOTKU\nRAF1\tA0A1A8JSH6_NOTKU\nRAF1\tA0A1A8M8V9_9TELE\nRAF1\tA0A1A8MQG4_9TELE\nRAF1\tA0A1A8N6U0_9TELE\nRAF1\tA0A1A8P0K3_9TELE\nRAF1\tA0A1A8P368_9TELE\nRAF1\tA0A1A8Q8D3_9TELE\nRAF1\tA0A1A8QKN4_9TELE\nRAF1\tA0A1A8QN95_9TELE\nRAF1\tA0A1A8R468_9TELE\nRAF1\tA0A1A8RU20_9TELE\nRAF1\tA0A1A8U4Z8_NOTFU\nRAF1\tA0A1A8UG55_NOTFU\nRAF1\tA0A1C7NE35_9FUNG\nRAF1\tA0A1D5QK38_MACMU\nRAF1\tA0A1D5R9U8_MACMU\nRAF1\tA0A1S2ZC90_ERIEU\nRAF1\tA0A1S2ZC98_ERIEU\nRAF1\tA0A1S3EN51_DIPOR\nRAF1\tA0A1S3W3X7_ERIEU\nRAF1\tA0A1U7Q6W2_MESAU\nRAF1\tA0A1U7RQ35_ALLSI\nRAF1\tA0A1U7S1H1_ALLSI\nRAF1\tA0A1U7TP74_TARSY\nRAF1\tA0A1V4K3A4_PATFA\nRAF1\tA0A1V4K3C8_PATFA\nRAF1\tA0A1V4K3E4_PATFA\nRAF1\tA0A1W5BFV0_CIOIN\nRAF1\tA0A1W5BFV1_CIOIN\nRAF1\tA0A1W5BJV6_CIOIN\nRAF1\tA0A1W5BL61_CIOIN\nRAF1\tA0A218UPA9_9PASE\nRAF1\tA0A250YKR0_CASCN\nRAF1\tA0A286XGN9_CAVPO\nRAF1\tA0A287AZ70_PIG\nRAF1\tA0A2C5X1M6_9PEZI\nRAF1\tA0A2D0PJA7_ICTPU\nRAF1\tA0A2D0PJB2_ICTPU\nRAF1\tA0A2D0PKX2_ICTPU\nRAF1\tA0A2I0LTX6_COLLI\nRAF1\tA0A2I2UIN3_FELCA\nRAF1\tA0A2I3SK30_PANTR\nRAF1\tA0A2I3THL4_PANTR\nRAF1\tA0A2I4BE50_9TELE\nRAF1\tA0A2I4BE53_9TELE\nRAF1\tA0A2I4BE59_9TELE\nRAF1\tA0A2K5E4P6_AOTNA\nRAF1\tA0A2K5E4T9_AOTNA\nRAF1\tA0A2K5IP18_COLAP\nRAF1\tA0A2K5IPD8_COLAP\nRAF1\tA0A2K5Q435_CEBCA\nRAF1\tA0A2K5Q436_CEBCA\nRAF1\tA0A2K5WKH0_MACFA\nRAF1\tA0A2K5WKV3_MACFA\nRAF1\tA0A2K5Y6Z9_MANLE\nRAF1\tA0A2K5Y723_MANLE\nRAF1\tA0A2K6DFM6_MACNE\nRAF1\tA0A2K6DFP8_MACNE\nRAF1\tA0A2K6DFQ1_MACNE\nRAF1\tA0A2K6ETW5_PROCO\nRAF1\tA0A2K6EU02_PROCO\nRAF1\tA0A2K6S6V4_SAIBB\nRAF1\tA0A2K6S6V6_SAIBB\nRAF1\tA0A2N6P2R4_BEABA\nRAF1\tA0A2R9B7H5_PANPA\nRAF1\tA0A2U3V629_TURTR\nRAF1\tA0A2U3VL55_ODORO\nRAF1\tA0A2U3VL57_ODORO\nRAF1\tA0A2U3Y9A9_LEPWE\nRAF1\tA0A2U3ZB31_ODORO\nRAF1\tA0A2U4C3Y3_TURTR\nRAF1\tA0A2Y9FMZ7_PHYMC\nRAF1\tA0A2Y9HGE7_NEOSC\nRAF1\tA0A2Y9M8J2_DELLE\nRAF1\tA0A2Y9MMZ0_DELLE\nRAF1\tA0A340XQI5_LIPVE\nRAF1\tA0A340XRE8_LIPVE\nRAF1\tA0A341CBQ7_NEOAA\nRAF1\tA0A341CBZ0_NEOAA\nRAF1\tA0A384AYZ0_BALAS\nRAF1\tA0A384AZ17_BALAS\nRAF1\tA0A384CML7_URSMA\nRAF1\tA0A3B3I426_ORYLA\nRAF1\tA0A3B3ICW7_ORYLA\nRAF1\tA0A3Q0E2S2_TARSY\nRAF1\tA0A3Q0E4G1_TARSY\nRAF1\tA0A3Q0E721_TARSY\nRAF1\tA0A3Q0K110_CIOIN\nRAF1\tA0A3Q7Q420_CALUR\nRAF1\tA0A3Q7Q547_CALUR\nRAF1\tA0A3Q7QFP0_CALUR\nRAF1\tA0A3Q7T6P7_VULVU\nRAF1\tA0A3Q7TS98_VULVU\nRAF1\tA0A3Q7U485_VULVU\nRAF1\tA0A3Q7UYK3_VULVU\nRAF1\tA0A3Q7V7N6_URSAR\nRAF1\tA0A3Q7VX19_URSAR\nRAF1\tA0A452CPH0_BALAS\nRAF1\tA0A452CPH2_BALAS\nRAF1\tA0A452DTI5_CAPHI\nRAF1\tA0A452ITN4_9SAUR\nRAF1\tA0A452ITV7_9SAUR\nRAF1\tA0A452J7W6_CHICK\nRAF1\tA0A452SE86_URSAM\nRAF1\tA0A452SEC3_URSAM\nRAF1\tA0A452SEG2_URSAM\nRAF1\tA0A452SEG7_URSAM\nRAF1\tA0A455AGW1_PHYMC\nRAF1\tA0A484G6H9_COLOR\nRAF1\tA0A493THG9_ANAPP\nRAF1\tA0A493TMR9_ANAPP\nRAF1\tA0A493TYC1_ANAPP\nRAF1\tA0A493TZ34_ANAPP\nRAF1\tA0A4R8Q8H6_9PEZI\nRAF1\tA0A4R8TF87_9PEZI\nRAF1\tA0A4U6XAV2_9PEZI\nRAF1\tA0A4W2BS84_BOBOX\nRAF1\tA0A4W2IB76_BOBOX\nRAF1\tA0A4W5MVV2_9TELE\nRAF1\tA0A4W5MVZ6_9TELE\nRAF1\tA0A4W5MW14_9TELE\nRAF1\tA0A4W5MXJ9_9TELE\nRAF1\tA0A4W5N0G1_9TELE\nRAF1\tA0A4W5N0I7_9TELE\nRAF1\tA0A4W5N0J3_9TELE\nRAF1\tA0A4W6EQM8_LATCA\nRAF1\tA0A4W6ET56_LATCA\nRAF1\tA0A4W6ET66_LATCA\nRAF1\tA0A4W6EU31_LATCA\nRAF1\tA0A4W6EUY6_LATCA\nRAF1\tA0A4W6EV13_LATCA\nRAF1\tA0A4X1UCF1_PIG\nRAF1\tA0A4X1UER2_PIG\nRAF1\tA0A4X1UES1_PIG\nRAF1\tA0A4X1UHC1_PIG\nRAF1\tA0A4X2L511_VOMUR\nRAF1\tA0A4X2L8A6_VOMUR\nRAF1\tA0A4Z2GB15_9TELE\nRAF1\tA0A559L9L8_FUSOX\nRAF1\tA0A559MLC3_9HELO\nRAF1\tA0A5F4CX59_CANLF\nRAF1\tA0A5F5PZU2_HORSE\nRAF1\tA0A5F7ZR25_MACMU\nRAF1\tA0A5F8HEF7_MONDO\nRAF1\tA0A5F8HFB8_MONDO\nRAF1\tA0A5F9CTE6_RABIT\nRAF1\tA0A5F9D2K3_RABIT\nRAF1\tA0A5F9DAY5_RABIT\nRAF1\tA0A5F9DW46_RABIT\nRAF1\tA0A5G3KWH1_XENTR\nRAF1\tA0A5G3KXN5_XENTR\nRAF1\tA0A5G3KYK8_XENTR\nRAF1\tA0A5G3KZX9_XENTR\nRAF1\tA0A5K1VRL0_PIG\nRAF1\tC7GUF4_YEAS2\nRAF1\tE2REI6_CANLF\nRAF1\tF6PU04_ORNAN\nRAF1\tF6QHE2_HORSE\nRAF1\tF6RZI1_XENTR\nRAF1\tF6SWC0_MACMU\nRAF1\tF6TUC4_MOUSE\nRAF1\tF7BRK5_MONDO\nRAF1\tF7CJC0_CALJA\nRAF1\tF7CJL2_CALJA\nRAF1\tF7HVT5_CALJA\nRAF1\tG1KNI7_ANOCA\nRAF1\tG1LDI1_AILME\nRAF1\tG1LDI9_AILME\nRAF1\tG1N0N1_MELGA\nRAF1\tG1PXC8_MYOLU\nRAF1\tG1QXT4_NOMLE\nRAF1\tG1RUM5_NOMLE\nRAF1\tG1TKQ9_RABIT\nRAF1\tG3T7U2_LOXAF\nRAF1\tG3WC19_SARHA\nRAF1\tH0UVP8_CAVPO\nRAF1\tH0WME6_OTOGA\nRAF1\tH0XN54_OTOGA\nRAF1\tH0Z0N6_TAEGU\nRAF1\tH0Z0P1_TAEGU\nRAF1\tH2M7B1_ORYLA\nRAF1\tH2NKI8_PONAB\nRAF1\tH2SV11_TAKRU\nRAF1\tH2SV12_TAKRU\nRAF1\tH2SV14_TAKRU\nRAF1\tH3AED0_LATCH\nRAF1\tH3AED1_LATCH\nRAF1\tH7C155_HUMAN\nRAF1\tI0J1A8_ORYNI\nRAF1\tI0J1A9_ORYRU\nRAF1\tI0J1B0_9ORYZ\nRAF1\tI0J1B1_ORYPU\nRAF1\tI0J1B2_9ORYZ\nRAF1\tI0J1B3_9ORYZ\nRAF1\tI3LAS8_PIG\nRAF1\tI3N585_ICTTR\nRAF1\tJ9P4L9_CANLF\nRAF1\tK7DGA2_PANTR\nRAF1\tK7FRZ8_PELSI\nRAF1\tK7FS06_PELSI\nRAF1\tL7N3R3_XENTR\nRAF1\tL7RRS6_HUMAN\nRAF1\tM3V846_PIG\nRAF1\tM3XB73_FELCA\nRAF1\tM3Z0G4_MUSPF\nRAF1\tM3ZCA6_NOMLE\nRAF1\tQ28GW9_XENTR\nRAF1\tQ4H2W3_CIOIN\nRAF1\tQ53ZF0_MOUSE\nRAF1\tQ5NSW0_TAKRU\nRAF1\tQ5WN11_RAT\nRAF1\tQ6BCX4_ONCKE\nRAF1\tQ6BCX7_ONCKE\nRAF1\tQ6BE22_DANRE\nRAF1\tQ6F5H4_ONCKE\nRAF1\tQ6GQJ4_XENLA\nRAF1\tQ6L767_ONCMY\nRAF1\tQ76C97_SERQU\nRAF1\tQ76FQ3_DANRE\nRAF1\tQ76H93_ONCKE\nRAF1\tQ76I87_DANRE\nRAF1\tQ7ZTR8_DANRE\nRAF1\tQ7ZZW3_DANRE\nRAF1\tQ8C3L4_MOUSE\nRAF1\tQ8C6X7_MOUSE\nRAF1\tQ8VI85_MOUSE\nRAF1\tQ90458_DANRE\nRAF1\tQ99N73_MOUSE\nRAF1\tQ9CU36_MOUSE\nRAF1\tR4G9F6_ANOCA\nRAF1\tRAF1_BOVIN\nRAF1\tRAF1_CHICK\nRAF1\tRAF1_HUMAN\nRAF1\tRAF1_MAIZE\nRAF1\tRAF1_MOUSE\nRAF1\tRAF1_ORYSJ\nRAF1\tRAF1_PONAB\nRAF1\tRAF1_RAT\nRAF1\tRAF1_SCHPO\nRAF1\tRAF1_XENLA\nRAF1\tRAF_YEAST\nRAF1\tU3E4N6_CALJA\nRAF1\tU3J465_ANAPP\nRAF1\tU3JB65_FICAL\nRAF1\tU3JB66_FICAL\nRAF1\tU6D4U3_NEOVI\nRAF1\tV8NV15_OPHHA\nRAF1\tW0UTE6_TARSY\nRAF1\tW0UTG5_ICTTR\nRAF1\tW0UUR6_HUMAN\nRAF1\tW0UUV6_MACMU\nRAF1\tW0UUX2_OTOGA\nRAF1\tW0UUZ7_RABIT\nRAF1\tW0UV32_PANTR\nRAF1\tW0UV57_TUPBE\nRAF1\tW0UV87_VICPA\nRAF1\tW0UVB1_PAPHA\nRAF1\tW0UVC4_9PRIM\nRAF1\tW0UVG9_MYOLU\nRAF1\tW0UVH9_OCHPR\nRAF1\tW0UVI7_HORSE\nRAF1\tW5Q234_SHEEP\nRAF1\tW5UKA2_ICTPU\n','From\tTo\nITK\tA0A096MS14_PAPAN\nITK\tA0A0D9RE75_CHLSB\nITK\tA0A0S7J412_9TELE\nITK\tA0A0S7MBB2_9TELE\nITK\tA0A140LG31_DANRE\nITK\tA0A151MEE0_ALLMI\nITK\tA0A1A7WJ31_9TELE\nITK\tA0A1A8AHD7_NOTFU\nITK\tA0A1A8BL32_9TELE\nITK\tA0A1A8BN76_9TELE\nITK\tA0A1A8I022_NOTKU\nITK\tA0A1S3A3C0_ERIEU\nITK\tA0A1S3GCB7_DIPOR\nITK\tA0A1U7Q7L8_MESAU\nITK\tA0A1U7RJK1_ALLSI\nITK\tA0A1U7T0M5_TARSY\nITK\tA0A218UNY2_9PASE\nITK\tA0A250YKA1_CASCN\nITK\tA0A286Y0H2_CAVPO\nITK\tA0A286Y1Z4_CAVPO\nITK\tA0A286ZPK2_PIG\nITK\tA0A2D0RIR7_ICTPU\nITK\tA0A2I0LWK9_COLLI\nITK\tA0A2I3N767_PAPAN\nITK\tA0A2I3SW87_PANTR\nITK\tA0A2I4BEA2_9TELE\nITK\tA0A2K5DDK8_AOTNA\nITK\tA0A2K5DDM3_AOTNA\nITK\tA0A2K5DDM8_AOTNA\nITK\tA0A2K5JT53_COLAP\nITK\tA0A2K5JT58_COLAP\nITK\tA0A2K5LNC1_CERAT\nITK\tA0A2K5SAK2_CEBCA\nITK\tA0A2K5SAK4_CEBCA\nITK\tA0A2K5SAL3_CEBCA\nITK\tA0A2K5X992_MACFA\nITK\tA0A2K6A5M1_MANLE\nITK\tA0A2K6A5N1_MANLE\nITK\tA0A2K6AVK0_MACNE\nITK\tA0A2K6AVL7_MACNE\nITK\tA0A2K6H0D9_PROCO\nITK\tA0A2K6MCD4_RHIBE\nITK\tA0A2K6MCJ0_RHIBE\nITK\tA0A2K6QBG7_RHIRO\nITK\tA0A2K6QBG9_RHIRO\nITK\tA0A2K6TZT1_SAIBB\nITK\tA0A2K6TZU4_SAIBB\nITK\tA0A2R8QJ75_DANRE\nITK\tA0A2R8RK79_DANRE\nITK\tA0A2R9AHL5_PANPA\nITK\tA0A2U3W9A0_ODORO\nITK\tA0A2U3XGU2_LEPWE\nITK\tA0A2U4AHD8_TURTR\nITK\tA0A2U4AHJ9_TURTR\nITK\tA0A2Y9EJ79_PHYMC\nITK\tA0A2Y9HNE6_NEOSC\nITK\tA0A2Y9PG90_DELLE\nITK\tA0A2Y9PJK6_DELLE\nITK\tA0A340WJ94_LIPVE\nITK\tA0A341CM86_NEOAA\nITK\tA0A341CNG1_NEOAA\nITK\tA0A383YS19_BALAS\nITK\tA0A384C6Z9_URSMA\nITK\tA0A3B5K594_TAKRU\nITK\tA0A3Q1LNN2_BOVIN\nITK\tA0A3Q2HIP7_HORSE\nITK\tA0A3Q2IBM0_HORSE\nITK\tA0A3Q7NMP3_CALUR\nITK\tA0A3Q7Q3U3_CALUR\nITK\tA0A3Q7TYM9_VULVU\nITK\tA0A3Q7XJM6_URSAR\nITK\tA0A452GAH1_CAPHI\nITK\tA0A452SNA4_URSAM\nITK\tA0A4W2F2U3_BOBOX\nITK\tA0A4W3H718_CALMI\nITK\tA0A4W3H777_CALMI\nITK\tA0A4W3HA64_CALMI\nITK\tA0A4W3HA68_CALMI\nITK\tA0A4W3HBH6_CALMI\nITK\tA0A4W3HBI3_CALMI\nITK\tA0A4W3HBJ6_CALMI\nITK\tA0A4W3HC28_CALMI\nITK\tA0A4W3HC46_CALMI\nITK\tA0A4W3HUB1_CALMI\nITK\tA0A4W5LQF6_9TELE\nITK\tA0A4W5LRA6_9TELE\nITK\tA0A4W6CDM6_LATCA\nITK\tA0A4W6CEX4_LATCA\nITK\tA0A4W6CJ04_LATCA\nITK\tA0A4X1TY42_PIG\nITK\tA0A4X2K2D6_VOMUR\nITK\tA0A5G2QY89_PIG\nITK\tA1A560_MOUSE\nITK\tA7Z039_BOVIN\nITK\tD4A7W7_RAT\nITK\tE2RP25_CANLF\nITK\tE5RFR5_HUMAN\nITK\tE5RJY4_HUMAN\nITK\tF1R3M1_DANRE\nITK\tF6PFT9_ORNAN\nITK\tF7FTR6_MONDO\nITK\tF7GS57_MACMU\nITK\tF7IG16_CALJA\nITK\tG1LHK5_AILME\nITK\tG1N100_MELGA\nITK\tG1PT68_MYOLU\nITK\tG1QV28_NOMLE\nITK\tG1SHE5_RABIT\nITK\tG3SSP1_LOXAF\nITK\tG3UBJ2_LOXAF\nITK\tG3WSK6_SARHA\nITK\tG7P6R0_MACFA\nITK\tH0VGK5_CAVPO\nITK\tH0XFK8_OTOGA\nITK\tH0XWD5_OTOGA\nITK\tH0YQX2_TAEGU\nITK\tH2LUG2_ORYLA\nITK\tH2PH70_PONAB\nITK\tH2QRW3_PANTR\nITK\tH2SDI7_TAKRU\nITK\tH3B303_LATCH\nITK\tH9G851_ANOCA\nITK\tI3MD63_ICTTR\nITK\tITK_HUMAN\nITK\tITK_MOUSE\nITK\tK7F8D8_PELSI\nITK\tM3WCF4_FELCA\nITK\tM3YPX8_MUSPF\nITK\tO42200_DANRE\nITK\tQ3U5G1_MOUSE\nITK\tQ5STT7_MOUSE\nITK\tQ5STT8_MOUSE\nITK\tQ6IZB0_PHOVI\nITK\tQ76N02_MOUSE\nITK\tQ8CAW3_MOUSE\nITK\tU3I6V9_ANAPP\nITK\tU3K153_FICAL\nITK\tU6DUH4_NEOVI\nITK\tW5PNG3_SHEEP\n','From\tTo\nTEK\tA0A096P1E5_PAPAN\nTEK\tA0A096P1E6_PAPAN\nTEK\tA0A0D9RBN2_CHLSB\nTEK\tA0A0S4TXQ9_RALSL\nTEK\tA0A0S4VNY4_RALSL\nTEK\tA0A0S4WDQ9_RALSL\nTEK\tA0A151NEE9_ALLMI\nTEK\tA0A1D5P7I7_CHICK\nTEK\tA0A1L8HP14_XENLA\nTEK\tA0A1S3FIZ1_DIPOR\nTEK\tA0A1S3FJS6_DIPOR\nTEK\tA0A1S3LUD1_SALSA\nTEK\tA0A1S3WST0_ERIEU\nTEK\tA0A1U7T6J8_TARSY\nTEK\tA0A1U7TI54_TARSY\nTEK\tA0A1U7TKB1_TARSY\nTEK\tA0A1U8DFQ1_ALLSI\nTEK\tA0A1V0CKQ5_9NEOP\nTEK\tA0A1V4KY21_PATFA\nTEK\tA0A218UKB3_9PASE\nTEK\tA0A286XAP8_CAVPO\nTEK\tA0A287BC64_PIG\nTEK\tA0A287DAL9_ICTTR\nTEK\tA0A2B4R7S3_STYPI\nTEK\tA0A2B4RAZ1_STYPI\nTEK\tA0A2B4RTE0_STYPI\nTEK\tA0A2B4SFK2_STYPI\nTEK\tA0A2B4SNC8_STYPI\nTEK\tA0A2D0QC83_ICTPU\nTEK\tA0A2I0M5I8_COLLI\nTEK\tA0A2I3GP71_NOMLE\nTEK\tA0A2I3H465_NOMLE\nTEK\tA0A2I3LQP2_PAPAN\nTEK\tA0A2I3RZH3_PANTR\nTEK\tA0A2I3SG17_PANTR\nTEK\tA0A2I3SN32_PANTR\nTEK\tA0A2I3TNW7_PANTR\nTEK\tA0A2I3TRZ6_PANTR\nTEK\tA0A2K5E132_AOTNA\nTEK\tA0A2K5E137_AOTNA\nTEK\tA0A2K5E194_AOTNA\nTEK\tA0A2K5M468_CERAT\nTEK\tA0A2K5M487_CERAT\nTEK\tA0A2K5M492_CERAT\nTEK\tA0A2K5M496_CERAT\nTEK\tA0A2K5RQP4_CEBCA\nTEK\tA0A2K5RQP9_CEBCA\nTEK\tA0A2K5RQS0_CEBCA\nTEK\tA0A2K5VRE1_MACFA\nTEK\tA0A2K5VRI3_MACFA\nTEK\tA0A2K5VRJ5_MACFA\nTEK\tA0A2K5VRK8_MACFA\nTEK\tA0A2K5ZN87_MANLE\nTEK\tA0A2K5ZNA1_MANLE\nTEK\tA0A2K5ZNC4_MANLE\nTEK\tA0A2K5ZNC5_MANLE\nTEK\tA0A2K6CQ87_MACNE\nTEK\tA0A2K6CQG2_MACNE\nTEK\tA0A2K6CQH6_MACNE\nTEK\tA0A2K6CQJ0_MACNE\nTEK\tA0A2K6F3M6_PROCO\nTEK\tA0A2K6F3N1_PROCO\nTEK\tA0A2K6F3P6_PROCO\nTEK\tA0A2K6LY28_RHIBE\nTEK\tA0A2K6LY32_RHIBE\nTEK\tA0A2K6LY40_RHIBE\nTEK\tA0A2K6LY54_RHIBE\nTEK\tA0A2K6P0F7_RHIRO\nTEK\tA0A2K6P0G6_RHIRO\nTEK\tA0A2K6P0G7_RHIRO\nTEK\tA0A2K6P0G8_RHIRO\nTEK\tA0A2K6P0G9_RHIRO\nTEK\tA0A2K6P0H1_RHIRO\nTEK\tA0A2K6TDX8_SAIBB\nTEK\tA0A2K6TDZ7_SAIBB\nTEK\tA0A2K6TE00_SAIBB\nTEK\tA0A2K6TE01_SAIBB\nTEK\tA0A2R8M860_CALJA\nTEK\tA0A2R8PN89_CALJA\nTEK\tA0A2R9AYN0_PANPA\nTEK\tA0A2R9AYN5_PANPA\nTEK\tA0A2R9AYP9_PANPA\nTEK\tA0A2R9B2H8_PANPA\nTEK\tA0A2R9B4I9_PANPA\nTEK\tA0A2R9B4K4_PANPA\nTEK\tA0A2S0RR57_CANLF\nTEK\tA0A2S1ZPZ5_9NEOP\nTEK\tA0A2S1ZPZ9_9NEOP\nTEK\tA0A2S1ZQ04_9NEOP\nTEK\tA0A2S1ZQ05_9NEOP\nTEK\tA0A2S1ZQ06_TITHA\nTEK\tA0A2S1ZQ07_9NEOP\nTEK\tA0A2S1ZQ08_9NEOP\nTEK\tA0A2S1ZQ10_9NEOP\nTEK\tA0A2S1ZQ11_AEREU\nTEK\tA0A2S1ZQ12_9NEOP\nTEK\tA0A2S1ZQ13_9NEOP\nTEK\tA0A2S1ZQ14_9NEOP\nTEK\tA0A2S1ZQ15_9NEOP\nTEK\tA0A2S1ZQ16_9NEOP\nTEK\tA0A2S1ZQ17_9NEOP\nTEK\tA0A2S1ZQ18_9NEOP\nTEK\tA0A2S1ZQ19_9NEOP\nTEK\tA0A2S1ZQ20_9NEOP\nTEK\tA0A2S1ZQ21_9NEOP\nTEK\tA0A2S1ZQ22_9NEOP\nTEK\tA0A2S1ZQ23_9NEOP\nTEK\tA0A2S1ZQ24_9NEOP\nTEK\tA0A2S1ZQ25_9NEOP\nTEK\tA0A2S1ZQ26_9NEOP\nTEK\tA0A2S1ZQ27_9NEOP\nTEK\tA0A2S1ZQ30_9NEOP\nTEK\tA0A2S1ZQ31_9NEOP\nTEK\tA0A2S1ZQ32_9NEOP\nTEK\tA0A2S1ZQ33_9NEOP\nTEK\tA0A2S1ZQ34_9NEOP\nTEK\tA0A2S1ZQ35_9NEOP\nTEK\tA0A2S1ZQ36_9NEOP\nTEK\tA0A2S1ZQ37_9NEOP\nTEK\tA0A2S1ZQ38_9NEOP\nTEK\tA0A2S1ZQ39_9NEOP\nTEK\tA0A2S1ZQ40_9NEOP\nTEK\tA0A2S1ZQ41_9NEOP\nTEK\tA0A2S1ZQ42_9NEOP\nTEK\tA0A2S1ZQ43_9NEOP\nTEK\tA0A2S1ZQ44_9NEOP\nTEK\tA0A2S1ZQ45_9NEOP\nTEK\tA0A2S1ZQ46_9NEOP\nTEK\tA0A2S1ZQ47_9NEOP\nTEK\tA0A2S1ZQ48_9NEOP\nTEK\tA0A2S1ZQ49_9NEOP\nTEK\tA0A2S1ZQ50_9NEOP\nTEK\tA0A2S1ZQ51_9NEOP\nTEK\tA0A2S1ZQ53_9NEOP\nTEK\tA0A2S1ZQ54_9NEOP\nTEK\tA0A2S1ZQ55_9NEOP\nTEK\tA0A2S1ZQ56_9NEOP\nTEK\tA0A2S1ZQ57_9NEOP\nTEK\tA0A2S1ZQ58_9NEOP\nTEK\tA0A2S1ZQ59_9NEOP\nTEK\tA0A2S1ZQ60_9NEOP\nTEK\tA0A2S1ZQ61_9NEOP\nTEK\tA0A2S1ZQ62_9NEOP\nTEK\tA0A2S1ZQ63_9NEOP\nTEK\tA0A2S1ZQ64_9NEOP\nTEK\tA0A2S1ZQ65_9NEOP\nTEK\tA0A2S1ZQ66_9NEOP\nTEK\tA0A2S1ZQ68_9NEOP\nTEK\tA0A2S1ZQ69_9NEOP\nTEK\tA0A2S1ZQ70_9NEOP\nTEK\tA0A2S1ZQ71_9NEOP\nTEK\tA0A2S1ZQ72_9NEOP\nTEK\tA0A2S1ZQ74_9NEOP\nTEK\tA0A2S1ZQ80_9NEOP\nTEK\tA0A2S2QAU1_9HEMI\nTEK\tA0A2U3X3V2_ODORO\nTEK\tA0A2U3X475_ODORO\nTEK\tA0A2U3Y080_LEPWE\nTEK\tA0A2Y9ETZ2_PHYMC\nTEK\tA0A2Y9EV88_PHYMC\nTEK\tA0A2Y9GGU5_NEOSC\nTEK\tA0A2Y9LPU5_DELLE\nTEK\tA0A337RXB5_FELCA\nTEK\tA0A337SQE0_FELCA\nTEK\tA0A340X351_LIPVE\nTEK\tA0A340X7H4_LIPVE\nTEK\tA0A340X9W4_LIPVE\nTEK\tA0A341C6K7_NEOAA\nTEK\tA0A341C7B2_NEOAA\nTEK\tA0A375BID1_9BURK\nTEK\tA0A375BKI3_9BURK\nTEK\tA0A375CUJ7_9BURK\nTEK\tA0A375DI89_9BURK\nTEK\tA0A375DYF4_9BURK\nTEK\tA0A375EF43_9BURK\nTEK\tA0A375F7P2_9BURK\nTEK\tA0A375G813_9BURK\nTEK\tA0A375GGG3_9BURK\nTEK\tA0A375H922_9BURK\nTEK\tA0A375IBP8_9BURK\nTEK\tA0A375IWZ2_9BURK\nTEK\tA0A383ZP53_BALAS\nTEK\tA0A383ZP66_BALAS\nTEK\tA0A383ZPQ6_BALAS\nTEK\tA0A384CER1_URSMA\nTEK\tA0A3Q0CXB4_MESAU\nTEK\tA0A3Q0GSI0_ALLSI\nTEK\tA0A3Q0GX51_ALLSI\nTEK\tA0A3Q1NG24_BOVIN\nTEK\tA0A3Q2H831_HORSE\nTEK\tA0A3Q2HJC0_HORSE\nTEK\tA0A3Q2I6T2_HORSE\nTEK\tA0A3Q2UKD3_CHICK\nTEK\tA0A3Q7MP50_CALUR\nTEK\tA0A3Q7T5W3_VULVU\nTEK\tA0A3Q7TU44_VULVU\nTEK\tA0A3Q7WL03_URSAR\nTEK\tA0A3Q7X6T1_URSAR\nTEK\tA0A452EJB7_CAPHI\nTEK\tA0A452EJM8_CAPHI\nTEK\tA0A452EJQ1_CAPHI\nTEK\tA0A452HA47_9SAUR\nTEK\tA0A452HA48_9SAUR\nTEK\tA0A452HA54_9SAUR\nTEK\tA0A452RF89_URSAM\nTEK\tA0A452RFF4_URSAM\nTEK\tA0A452TH49_URSMA\nTEK\tA0A455C1J5_PHYMC\nTEK\tA0A4W2CXK1_BOBOX\nTEK\tA0A4W2ENE3_BOBOX\nTEK\tA0A4W2ENF5_BOBOX\nTEK\tA0A4W2H037_BOBOX\nTEK\tA0A4W2ICJ9_BOBOX\nTEK\tA0A4W2ILN8_BOBOX\nTEK\tA0A4W3HH87_CALMI\nTEK\tA0A4W3HM33_CALMI\nTEK\tA0A4W3HMJ9_CALMI\nTEK\tA0A4W4H7B1_ELEEL\nTEK\tA0A4W4HBH0_ELEEL\nTEK\tA0A4W4HED8_ELEEL\nTEK\tA0A4W4HF91_ELEEL\nTEK\tA0A4X1W6Y3_PIG\nTEK\tA0A4X1W6Y9_PIG\nTEK\tA0A4X1WCC8_PIG\nTEK\tA0A4X1WCD2_PIG\nTEK\tA0A4X1WCD5_PIG\nTEK\tA0A4X1WCL1_PIG\nTEK\tA0A4X1WCL5_PIG\nTEK\tA0A4X1WDM4_PIG\nTEK\tA0A4X2KE91_VOMUR\nTEK\tA0A4X2KEA3_VOMUR\nTEK\tA0A4X2KGK5_VOMUR\nTEK\tA0A5F4BUW1_CANLF\nTEK\tA0A5F4BZ37_CANLF\nTEK\tA0A5F4C429_CANLF\nTEK\tA0A5F4CS11_CANLF\nTEK\tA0A5F4DGV2_CANLF\nTEK\tA0A5F8G4U6_MONDO\nTEK\tA0A5G3HJN8_XENTR\nTEK\tA0A5G3KBC4_XENTR\nTEK\tA0A5G3KCI8_XENTR\nTEK\tA0A5G3KCZ8_XENTR\nTEK\tA0A5G3KDD3_XENTR\nTEK\tA0A5G3KE53_XENTR\nTEK\tA3RXH0_RALSL\nTEK\tAHL16_ARATH\nTEK\tB1AWS8_MOUSE\nTEK\tB3DJR6_DANRE\nTEK\tB3R2A0_CUPTR\nTEK\tB5A953_HUMAN\nTEK\tB5A954_HUMAN\nTEK\tD3ZCD0_RAT\nTEK\tD8NDM4_RALSL\nTEK\tD8P4N6_RALSL\nTEK\tD9MND0_HUMAN\nTEK\tD9MND2_HUMAN\nTEK\tE5RIV9_HUMAN\nTEK\tF1NFQ9_CHICK\nTEK\tF1P8U6_CANLF\nTEK\tF1SP85_PIG\nTEK\tF6G9I3_RALS8\nTEK\tF6TEV1_MONDO\nTEK\tF6UTL2_ORNAN\nTEK\tF6VW12_MACMU\nTEK\tF6W2M5_XENTR\nTEK\tF6WQ81_HORSE\nTEK\tF6XN02_CALJA\nTEK\tF7B019_XENTR\nTEK\tF7BM77_XENTR\nTEK\tF7C661_XENTR\nTEK\tF7EC20_XENTR\nTEK\tF8W2R6_DANRE\nTEK\tG1KU09_ANOCA\nTEK\tG1L4T4_AILME\nTEK\tG1N963_MELGA\nTEK\tG1PDN9_MYOLU\nTEK\tG1S2P7_NOMLE\nTEK\tG1TE63_RABIT\nTEK\tG2ZST4_9RALS\nTEK\tG3AA08_9RALS\nTEK\tG3T455_LOXAF\nTEK\tG3TXJ0_LOXAF\nTEK\tG3UEJ7_LOXAF\nTEK\tH0VFN6_CAVPO\nTEK\tH0XG18_OTOGA\nTEK\tH0YT27_TAEGU\nTEK\tH2QX37_PANTR\nTEK\tH3AN63_LATCH\nTEK\tH9EZ95_MACMU\nTEK\tH9FBY8_MACMU\nTEK\tH9FJG2_MACMU\nTEK\tH9FLX0_MACMU\nTEK\tI3MFX6_ICTTR\nTEK\tK7FZR6_PELSI\nTEK\tK7FZT0_PELSI\nTEK\tK7GLN0_PIG\nTEK\tK7GQ34_PIG\nTEK\tM3VZ78_FELCA\nTEK\tM3X412_FELCA\nTEK\tM3YWW8_MUSPF\nTEK\tQ0IIL9_BOVIN\nTEK\tQ45469_RALSL\nTEK\tQ80YS4_MOUSE\nTEK\tQ8XR59_RALSO\nTEK\tTIE2_BOVIN\nTEK\tTIE2_HUMAN\nTEK\tTIE2_MOUSE\nTEK\tU3IZL3_ANAPP\nTEK\tU3JY02_FICAL\nTEK\tW5PYW6_SHEEP\n','From\tTo\nMET\tA0A024R728_HUMAN\nMET\tA0A024R759_HUMAN\nMET\tA0A060UTW6_9PROT\nMET\tA0A068J5U3_NILLU\nMET\tA0A077SK47_BLAGE\nMET\tA0A088FQG6_DIPPU\nMET\tA0A0A0MXS4_PONAB\nMET\tA0A0B2VFA5_TOXCA\nMET\tA0A0B2W404_TOXCA\nMET\tA0A0B4LF49_DROME\nMET\tA0A0B5JQ16_DANRE\nMET\tA0A0D9REU3_CHLSB\nMET\tA0A0E3KKF5_CORUL\nMET\tA0A0E3MRK0_ENTFL\nMET\tA0A0G2JY19_RAT\nMET\tA0A0H5AX60_ZOONE\nMET\tA0A0M4L9Q5_CORUL\nMET\tA0A0P6JLW7_HETGA\nMET\tA0A0Q9WCT5_DROVI\nMET\tA0A0S2SQ19_9NEOP\nMET\tA0A0S7HNS5_9TELE\nMET\tA0A0S7KP72_9TELE\nMET\tA0A0S7KPB0_9TELE\nMET\tA0A0S7KPB7_9TELE\nMET\tA0A0S7KQ73_9TELE\nMET\tA0A0S7KQE7_9TELE\nMET\tA0A0U3BUN0_NILLU\nMET\tA0A0U5HZ63_STEMA\nMET\tA0A0V0RLU8_9BILA\nMET\tA0A0V0RME5_9BILA\nMET\tA0A0V0TVR3_9BILA\nMET\tA0A0V0TVU9_9BILA\nMET\tA0A0V0TW23_9BILA\nMET\tA0A0V0X8L4_9BILA\nMET\tA0A0V0X8S9_9BILA\nMET\tA0A0V1BVD0_TRISP\nMET\tA0A0V1BW00_TRISP\nMET\tA0A0V1BW09_TRISP\nMET\tA0A0V1BW13_TRISP\nMET\tA0A0V1BWK1_TRISP\nMET\tA0A0V1DHM5_TRIBR\nMET\tA0A0V1DHS6_TRIBR\nMET\tA0A0V1DIG9_TRIBR\nMET\tA0A0V1E9Y9_TRIPS\nMET\tA0A0V1FI48_TRIPS\nMET\tA0A0V1FVM5_TRIPS\nMET\tA0A0V1FVU7_TRIPS\nMET\tA0A0V1GZF2_TRIPS\nMET\tA0A0V1IPJ9_TRIPS\nMET\tA0A0V1ISJ9_TRIPS\nMET\tA0A0V1LSM8_9BILA\nMET\tA0A0V1LSR5_9BILA\nMET\tA0A140GDI6_ENTFL\nMET\tA0A146J401_9HEMI\nMET\tA0A151P7F6_ALLMI\nMET\tA0A192ZIL9_9EUKA\nMET\tA0A192ZJJ1_9EUKA\nMET\tA0A1A7WZH1_9TELE\nMET\tA0A1A7X010_9TELE\nMET\tA0A1A8A135_NOTFU\nMET\tA0A1A8C649_9TELE\nMET\tA0A1A8C8M6_9TELE\nMET\tA0A1A8FQQ7_9TELE\nMET\tA0A1A8ID47_NOTKU\nMET\tA0A1A8LDH3_9TELE\nMET\tA0A1A8UF14_NOTFU\nMET\tA0A1B2INL9_PLOIN\nMET\tA0A1B2INM5_APHGO\nMET\tA0A1B2INN0_MONAT\nMET\tA0A1D0C1X1_STAAU\nMET\tA0A1D5QZ12_MACMU\nMET\tA0A1D5RB36_MACMU\nMET\tA0A1S2ZW47_ERIEU\nMET\tA0A1S3FHH0_DIPOR\nMET\tA0A1U7U371_TARSY\nMET\tA0A1U8CAY0_MESAU\nMET\tA0A1W1H8Y4_9DELT\nMET\tA0A1Z1V368_ECOLX\nMET\tA0A218UDI2_9PASE\nMET\tA0A250Y7H3_CASCN\nMET\tA0A286XHP3_CAVPO\nMET\tA0A290DSG2_9NEOP\nMET\tA0A2B4S2H5_STYPI\nMET\tA0A2C9PIR6_9STRE\nMET\tA0A2D0RH02_ICTPU\nMET\tA0A2D0RH06_ICTPU\nMET\tA0A2I0MJ90_COLLI\nMET\tA0A2I2U567_FELCA\nMET\tA0A2I3FQG6_NOMLE\nMET\tA0A2I3HWP8_NOMLE\nMET\tA0A2I3RTG1_PANTR\nMET\tA0A2I3SU60_PANTR\nMET\tA0A2I3T1S4_PANTR\nMET\tA0A2I4BDA3_9TELE\nMET\tA0A2J8QXG8_PANTR\nMET\tA0A2K5CM73_AOTNA\nMET\tA0A2K5CM90_AOTNA\nMET\tA0A2K5CMA7_AOTNA\nMET\tA0A2K5J2I8_COLAP\nMET\tA0A2K5J2J6_COLAP\nMET\tA0A2K5NUS4_CERAT\nMET\tA0A2K5NV40_CERAT\nMET\tA0A2K5NVA6_CERAT\nMET\tA0A2K5NVB8_CERAT\nMET\tA0A2K5RFG6_CEBCA\nMET\tA0A2K5RFH8_CEBCA\nMET\tA0A2K5RFJ2_CEBCA\nMET\tA0A2K5UM05_MACFA\nMET\tA0A2K5UM36_MACFA\nMET\tA0A2K5UM43_MACFA\nMET\tA0A2K5UM47_MACFA\nMET\tA0A2K5XT53_MANLE\nMET\tA0A2K5XT72_MANLE\nMET\tA0A2K5XT98_MANLE\nMET\tA0A2K6D3L1_MACNE\nMET\tA0A2K6D3N6_MACNE\nMET\tA0A2K6D3N9_MACNE\nMET\tA0A2K6D3P1_MACNE\nMET\tA0A2K6G6M6_PROCO\nMET\tA0A2K6G6Q1_PROCO\nMET\tA0A2K6MDJ7_RHIBE\nMET\tA0A2K6MDK6_RHIBE\nMET\tA0A2K6MDN5_RHIBE\nMET\tA0A2K6MDR6_RHIBE\nMET\tA0A2K6RB83_RHIRO\nMET\tA0A2K6RB97_RHIRO\nMET\tA0A2K6RB98_RHIRO\nMET\tA0A2K6RBA7_RHIRO\nMET\tA0A2K6SAK2_SAIBB\nMET\tA0A2K6SAN2_SAIBB\nMET\tA0A2K6SAN8_SAIBB\nMET\tA0A2K6SAP7_SAIBB\nMET\tA0A2K9P5C3_9DIPT\nMET\tA0A2R2YUM5_SPAAU\nMET\tA0A2R7W3U3_ONCFA\nMET\tA0A2R7X1N8_ONCFA\nMET\tA0A2R8A227_CARDV\nMET\tA0A2R9BKS2_PANPA\nMET\tA0A2R9BTV0_PANPA\nMET\tA0A2S1PL36_ENTFL\nMET\tA0A2U3WIC3_ODORO\nMET\tA0A2U3XC96_LEPWE\nMET\tA0A2U4AHV4_TURTR\nMET\tA0A2U4AHV8_TURTR\nMET\tA0A2U4AID1_TURTR\nMET\tA0A2Y9F7T1_PHYMC\nMET\tA0A2Y9HTP5_NEOSC\nMET\tA0A2Y9NWP7_DELLE\nMET\tA0A2Y9NZN9_DELLE\nMET\tA0A2Z6C4I2_9BACU\nMET\tA0A2Z6C5L0_9ABAC\nMET\tA0A2Z6C6L4_NPVAP\nMET\tA0A2Z6C7K6_9BACU\nMET\tA0A340WMU0_LIPVE\nMET\tA0A340WUV6_LIPVE\nMET\tA0A341CFM6_NEOAA\nMET\tA0A341CG14_NEOAA\nMET\tA0A341CGB1_NEOAA\nMET\tA0A341CI06_NEOAA\nMET\tA0A384B629_BALAS\nMET\tA0A384B696_BALAS\nMET\tA0A384B698_BALAS\nMET\tA0A384DAD6_URSMA\nMET\tA0A3Q0EBU9_TARSY\nMET\tA0A3Q0GJS3_ALLSI\nMET\tA0A3Q0GJS9_ALLSI\nMET\tA0A3Q0GP31_ALLSI\nMET\tA0A3Q0GP50_ALLSI\nMET\tA0A3Q1NAI5_BOVIN\nMET\tA0A3Q2HUW1_HORSE\nMET\tA0A3Q2HWY0_HORSE\nMET\tA0A3Q2I975_HORSE\nMET\tA0A3Q7QL67_CALUR\nMET\tA0A3Q7SVW1_VULVU\nMET\tA0A3Q7UKU6_VULVU\nMET\tA0A3Q7VX94_URSAR\nMET\tA0A452G0H2_CAPHI\nMET\tA0A452G0I2_CAPHI\nMET\tA0A452GEQ5_HORSE\nMET\tA0A452I1B6_9SAUR\nMET\tA0A452I1D9_9SAUR\nMET\tA0A452QT04_URSAM\nMET\tA0A452QUQ0_URSAM\nMET\tA0A452QUR5_URSAM\nMET\tA0A452QUS1_URSAM\nMET\tA0A452QUV0_URSAM\nMET\tA0A452QUV5_URSAM\nMET\tA0A452QUW4_URSAM\nMET\tA0A452T5W9_URSMA\nMET\tA0A452T5X9_URSMA\nMET\tA0A452T5Y6_URSMA\nMET\tA0A452T5Z4_URSMA\nMET\tA0A452T624_URSMA\nMET\tA0A452T6N1_URSMA\nMET\tA0A481WYX9_KLEPN\nMET\tA0A4W2EDJ9_BOBOX\nMET\tA0A4W2HG15_BOBOX\nMET\tA0A4W2HJB0_BOBOX\nMET\tA0A4W3HX17_CALMI\nMET\tA0A4W3HZZ7_CALMI\nMET\tA0A4W3I007_CALMI\nMET\tA0A4W3I3H2_CALMI\nMET\tA0A4W3I3M3_CALMI\nMET\tA0A4W3I4P4_CALMI\nMET\tA0A4W3IMS6_CALMI\nMET\tA0A4W3IMU6_CALMI\nMET\tA0A4X1W3C9_PIG\nMET\tA0A4X1W479_PIG\nMET\tA0A4X1W4M3_PIG\nMET\tA0A4X2LQ56_VOMUR\nMET\tA0A4X2LX78_VOMUR\nMET\tA0A4X2LX94_VOMUR\nMET\tA0A4X2LYS5_VOMUR\nMET\tA0A4Y2QDX2_ARAVE\nMET\tA0A4Y2U090_ARAVE\nMET\tA0A4Z2EU33_9TELE\nMET\tA0A515EWL2_9ABAC\nMET\tA0A5F4BUR8_CANLF\nMET\tA0A5F4D3R5_CANLF\nMET\tA0A5F4D4T5_CANLF\nMET\tA0A5F4DJ79_CANLF\nMET\tA0A5F5PLQ3_HORSE\nMET\tA0A5F5PNT8_HORSE\nMET\tA0A5F8A7A5_MACMU\nMET\tA0A5F8GV38_MONDO\nMET\tA0A5F9C507_RABIT\nMET\tA0A5F9CEP1_RABIT\nMET\tA0A5F9CL76_RABIT\nMET\tA0A5F9DAK3_RABIT\nMET\tA0A5F9DAP1_RABIT\nMET\tA0A5F9DCS8_RABIT\nMET\tA0A5F9DE67_RABIT\nMET\tA0A5G2QT81_PIG\nMET\tA0A5J6D2S8_CULPA\nMET\tA0A5J6KIQ8_SCHGR\nMET\tA0M8T9_CHICK\nMET\tA0M8V8_DANRE\nMET\tA0PJF7_HUMAN\nMET\tA1A597_MOUSE\nMET\tA1YJ78_NPVSF\nMET\tA1Z8G0_DROME\nMET\tA2VCS0_MOUSE\nMET\tA4D7S7_MACEU\nMET\tA4D7S8_MACEU\nMET\tA4FUV6_MOUSE\nMET\tA4FUV7_MOUSE\nMET\tA5X9I4_9CAUD\nMET\tA6H7D2_BOVIN\nMET\tA6MUT7_TRICA\nMET\tA8C664_NPVAP\nMET\tB4M2H9_DROVI\nMET\tB5A929_HUMAN\nMET\tB5A930_HUMAN\nMET\tB5A931_HUMAN\nMET\tB5A933_HUMAN\nMET\tB5A934_HUMAN\nMET\tB5A935_HUMAN\nMET\tB5A936_HUMAN\nMET\tB5A937_HUMAN\nMET\tB5A938_HUMAN\nMET\tB5A939_HUMAN\nMET\tB5A940_HUMAN\nMET\tB5A941_HUMAN\nMET\tB5A942_HUMAN\nMET\tB5A943_HUMAN\nMET\tB6VB33_BOMMO\nMET\tB7YZF0_DROME\nMET\tB8BQ32_THAPS\nMET\tC3W5J2_ELAOL\nMET\tC9JKM5_HUMAN\nMET\tD3YVY2_MOUSE\nMET\tD4Q8Y3_9ROSI\nMET\tD4Q8Y4_9ROSI\nMET\tD4Q8Y6_9ROSI\nMET\tD4Q8Z7_9ROSI\nMET\tD4Q8Z9_9ROSI\nMET\tD4Q902_9ROSI\nMET\tD4Q903_9ROSI\nMET\tD4Q907_9ROSI\nMET\tD4Q909_9ROSI\nMET\tD4Q911_9ROSI\nMET\tD4Q912_9ROSI\nMET\tD4Q915_9ROSI\nMET\tD4Q922_9ROSI\nMET\tD4Q925_9ROSI\nMET\tD4Q927_9ROSI\nMET\tD4Q931_9ROSI\nMET\tD4Q936_9ROSI\nMET\tD4Q937_9ROSI\nMET\tD5DC48_BACMD\nMET\tD5DC50_BACMD\nMET\tE5S4W9_TRISP\nMET\tE7BDD0_9BROM\nMET\tE7BDD1_9BROM\nMET\tE7BDD4_9BROM\nMET\tE7BDD5_9BROM\nMET\tE7BDD6_9BROM\nMET\tF1P040_CHICK\nMET\tF1QLR7_DANRE\nMET\tF1SJC6_PIG\nMET\tF2Y6H4_NEOPH\nMET\tF2Y6H5_LIPVE\nMET\tF2Y6H6_DELCA\nMET\tF2Y6H7_GRAGR\nMET\tF2Y6H8_STEAT\nMET\tF2Y6H9_TURAD\nMET\tF2Y6I0_STECO\nMET\tF2Y6I1_SOUCH\nMET\tF2Y6I2_BALOM\nMET\tF2Y6I3_KOGSI\nMET\tF2Y6I4_PLAGN\nMET\tF2Y6I5_ELADA\nMET\tF2Y6I6_MOSMO\nMET\tF2Y6I7_DELLE\nMET\tF4ID86_ARATH\nMET\tF4YEF7_TURTR\nMET\tF4YEF8_BALAC\nMET\tF4YEF9_HIPAM\nMET\tF4YEG0_PIG\nMET\tF4YEG1_CAMBA\nMET\tF4YEG2_CERSI\nMET\tF4YEG3_MANPE\nMET\tF6RCC5_MOUSE\nMET\tF6VAI0_CALJA\nMET\tF6VHT9_XENTR\nMET\tF6W2E4_ORNAN\nMET\tF6X333_MOUSE\nMET\tF7FDY1_MONDO\nMET\tF7G070_CALJA\nMET\tF7GC98_CALJA\nMET\tF7H1K4_MACMU\nMET\tF8VQL0_MOUSE\nMET\tG0UDV1_9BROM\nMET\tG1FF51_SESPO\nMET\tG1KI46_ANOCA\nMET\tG1L3D7_AILME\nMET\tG1L3E0_AILME\nMET\tG1N9U2_MELGA\nMET\tG1PAA9_MYOLU\nMET\tG1RLI8_NOMLE\nMET\tG3T807_LOXAF\nMET\tG3UNS1_LOXAF\nMET\tG3WTF3_SARHA\nMET\tG7MM61_MACMU\nMET\tG8S8Z1_ACTS5\nMET\tG8S8Z4_ACTS5\nMET\tG8S9Q3_ACTS5\nMET\tG8SEU4_ACTS5\nMET\tG9HZS8_PYRAP\nMET\tG9HZS9_RHOPR\nMET\tG9HZT0_9INSE\nMET\tH0WNV1_OTOGA\nMET\tH0Z494_TAEGU\nMET\tH2MZM4_ORYLA\nMET\tH2V3S1_TAKRU\nMET\tH3B042_LATCH\nMET\tH6U8T8_RALSL\nMET\tH7C130_HUMAN\nMET\tH7C174_HUMAN\nMET\tI3N352_ICTTR\nMET\tJ7G1V8_9CRYP\nMET\tJ9P743_CANLF\nMET\tK4K0S1_SALDE\nMET\tK7C095_PANTR\nMET\tK7DGW4_PANTR\nMET\tK7GB62_PELSI\nMET\tK7GPY4_PIG\nMET\tK8ERY3_CARML\nMET\tM1V8C3_9CRUS\nMET\tM1VN89_DAPPU\nMET\tM3XI74_LATCH\nMET\tM3Y737_MUSPF\nMET\tMETE_PLESU\nMET\tMET_AOTNA\nMET\tMET_ATEAB\nMET\tMET_ATEGE\nMET\tMET_BOVIN\nMET\tMET_CALJA\nMET\tMET_CANLF\nMET\tMET_CHLAE\nMET\tMET_COLGU\nMET\tMET_DASNO\nMET\tMET_ECHTE\nMET\tMET_EULMM\nMET\tMET_FELCA\nMET\tMET_GORGO\nMET\tMET_HORSE\nMET\tMET_HUMAN\nMET\tMET_LOXAF\nMET\tMET_MICMU\nMET\tMET_MOUSE\nMET\tMET_MUNMU\nMET\tMET_MUSPF\nMET\tMET_NEONE\nMET\tMET_NOMLE\nMET\tMET_ORNAN\nMET\tMET_OTOGA\nMET\tMET_PANTR\nMET\tMET_PAPAN\nMET\tMET_PIG\nMET\tMET_PLEMO\nMET\tMET_PONAB\nMET\tMET_RABIT\nMET\tMET_RAT\nMET\tMET_RHIFE\nMET\tMET_SAIBB\nMET\tMET_SHEEP\nMET\tO44118_DROME\nMET\tO49125_FRIAG\nMET\tO49126_FRIAG\nMET\tO49127_FRIAG\nMET\tQ07E11_CAVPO\nMET\tQ14825_HUMAN\nMET\tQ2IBC6_RAT\nMET\tQ2IBC7_RAT\nMET\tQ2IBC9_RAT\nMET\tQ2IBD1_RAT\nMET\tQ2NNZ1_NPVHC\nMET\tQ2PGH4_HAELO\nMET\tQ2PGH5_HAELO\nMET\tQ2PGH7_HAELO\nMET\tQ2QLD2_CARPS\nMET\tQ3HR41_METSA\nMET\tQ3TCS4_MOUSE\nMET\tQ549V8_VICFA\nMET\tQ58FS0_AEDAE\nMET\tQ64GK4_DANRE\nMET\tQ6SZ39_STRPY\nMET\tQ75NH4_LABPU\nMET\tQ75NH5_LABPU\nMET\tQ75NH6_PEA\nMET\tQ75NH8_VICFA\nMET\tQ75NH9_PHAAN\nMET\tQ75NI0_PHAAN\nMET\tQ75NI1_SOYBN\nMET\tQ75NI2_SOYBN\nMET\tQ75NI3_VIGRR\nMET\tQ75NI4_VIGRR\nMET\tQ8L4H9_9GENT\nMET\tQ8L4I0_9GENT\nMET\tQ8L575_9GENT\nMET\tQ8L6S9_9GENT\nMET\tQ8L6T0_9GENT\nMET\tQ8L6T1_9GENT\nMET\tQ8L6T2_9GENT\nMET\tQ8L6T3_9GENT\nMET\tQ8L6T4_9GENT\nMET\tQ8L6T5_9GENT\nMET\tQ8L6T6_9GENT\nMET\tQ8LTD1_9CAUD\nMET\tQ8T5S9_DROME\nMET\tQ8VHF9_RAT\nMET\tQ8W5X3_9CAUD\nMET\tQ8WZH7_AGABI\nMET\tQ90ZG4_DANRE\nMET\tQ961H2_DROME\nMET\tQ98RN8_GUITH\nMET\tQ9UEJ3_HUMAN\nMET\tQ9VYW2_DROME\nMET\tQ9W650_XENLA\nMET\tR4IJP6_GLOMM\nMET\tS3FZY0_9BACL\nMET\tS5MR49_9ABAC\nMET\tS5N9Q1_NPVCF\nMET\tS6EQZ5_LACLL\nMET\tSLTM_HUMAN\nMET\tSLTM_MOUSE\nMET\tU3IU33_ANAPP\nMET\tU3K693_FICAL\nMET\tU6DF55_NEOVI\nMET\tV5KYX7_LOCMI\nMET\tV5L1M9_LOCMI\nMET\tV7ZBD3_9XANT\nMET\tV8NSN5_OPHHA\nMET\tW5NRM5_SHEEP\nMET\tW5NRN1_SHEEP\nMET\tZ4YHD9_BOVIN\nMET\tZ4YHE9_CANLF\n','From\tTo\nFGFR1\tA0A034VE72_BACDO\nFGFR1\tA0A061S910_9CHLO\nFGFR1\tA0A077JIL4_CYNPY\nFGFR1\tA0A087YIN0_POEFO\nFGFR1\tA0A096MD12_POEFO\nFGFR1\tA0A096MT31_PAPAN\nFGFR1\tA0A0D9RSA9_CHLSB\nFGFR1\tA0A0K1P4M5_GOBRA\nFGFR1\tA0A0S2Z3P4_HUMAN\nFGFR1\tA0A0S2Z3P8_HUMAN\nFGFR1\tA0A0S2Z3Q6_HUMAN\nFGFR1\tA0A0S2Z3T4_HUMAN\nFGFR1\tA0A0S2Z3T9_HUMAN\nFGFR1\tA0A0S7M127_9TELE\nFGFR1\tA0A0X3PTC7_SCHSO\nFGFR1\tA0A1A7XU13_9TELE\nFGFR1\tA0A1A7Y3Y7_9TELE\nFGFR1\tA0A1A8B519_NOTFU\nFGFR1\tA0A1A8B774_NOTFU\nFGFR1\tA0A1A8CUK9_9TELE\nFGFR1\tA0A1A8DA50_9TELE\nFGFR1\tA0A1A8DCX2_9TELE\nFGFR1\tA0A1A8FFX3_9TELE\nFGFR1\tA0A1A8GEU0_9TELE\nFGFR1\tA0A1A8GGW5_9TELE\nFGFR1\tA0A1A8IF41_NOTKU\nFGFR1\tA0A1A8IF52_NOTKU\nFGFR1\tA0A1A8KVP8_NOTKU\nFGFR1\tA0A1A8KY84_NOTKU\nFGFR1\tA0A1A8LBZ2_9TELE\nFGFR1\tA0A1A8MJT3_9TELE\nFGFR1\tA0A1A8PX81_9TELE\nFGFR1\tA0A1A8QZ83_9TELE\nFGFR1\tA0A1A8R0N4_9TELE\nFGFR1\tA0A1A8SLN6_9TELE\nFGFR1\tA0A1A8TZF5_NOTFU\nFGFR1\tA0A1B0GSD9_MOUSE\nFGFR1\tA0A1B0GSQ7_MOUSE\nFGFR1\tA0A1B0GSZ6_MOUSE\nFGFR1\tA0A1B0RPQ4_HUMAN\nFGFR1\tA0A1D5Q279_MACMU\nFGFR1\tA0A1D5R2V9_MACMU\nFGFR1\tA0A1L8GRK9_XENLA\nFGFR1\tA0A1L8H273_XENLA\nFGFR1\tA0A1S2ZBD6_ERIEU\nFGFR1\tA0A1S2ZBE0_ERIEU\nFGFR1\tA0A1S2ZBE1_ERIEU\nFGFR1\tA0A1S2ZBE2_ERIEU\nFGFR1\tA0A1S2ZBE5_ERIEU\nFGFR1\tA0A1S2ZBE6_ERIEU\nFGFR1\tA0A1S2ZBF6_ERIEU\nFGFR1\tA0A1S3FXN9_DIPOR\nFGFR1\tA0A1S3FXS5_DIPOR\nFGFR1\tA0A1S3FY83_DIPOR\nFGFR1\tA0A1S3FZR2_DIPOR\nFGFR1\tA0A1S3FZW2_DIPOR\nFGFR1\tA0A1S3W311_ERIEU\nFGFR1\tA0A1S3W3C6_ERIEU\nFGFR1\tA0A1S3W3G6_ERIEU\nFGFR1\tA0A1S3W4K2_ERIEU\nFGFR1\tA0A1U7RGJ6_ALLSI\nFGFR1\tA0A1U7UK65_TARSY\nFGFR1\tA0A1U8BFM4_MESAU\nFGFR1\tA0A1U8BJ90_MESAU\nFGFR1\tA0A1U8BSQ0_MESAU\nFGFR1\tA0A1V1G4T2_HUMAN\nFGFR1\tA0A1V1G971_HUMAN\nFGFR1\tA0A218UJD4_9PASE\nFGFR1\tA0A248SLA9_POLSP\nFGFR1\tA0A250Y5C7_CASCN\nFGFR1\tA0A286XMS2_CAVPO\nFGFR1\tA0A287D073_ICTTR\nFGFR1\tA0A287D0Q7_ICTTR\nFGFR1\tA0A287D284_ICTTR\nFGFR1\tA0A2B4R462_STYPI\nFGFR1\tA0A2B4R5X1_STYPI\nFGFR1\tA0A2B4R7W2_STYPI\nFGFR1\tA0A2B4RBN4_STYPI\nFGFR1\tA0A2B4RGR4_STYPI\nFGFR1\tA0A2B4RJB9_STYPI\nFGFR1\tA0A2B4RMK8_STYPI\nFGFR1\tA0A2B4RPF4_STYPI\nFGFR1\tA0A2B4RSX6_STYPI\nFGFR1\tA0A2B4RVN9_STYPI\nFGFR1\tA0A2B4RYN2_STYPI\nFGFR1\tA0A2B4S5D0_STYPI\nFGFR1\tA0A2B4SPJ5_STYPI\nFGFR1\tA0A2B4SRQ2_STYPI\nFGFR1\tA0A2B4SV80_STYPI\nFGFR1\tA0A2I2UK29_FELCA\nFGFR1\tA0A2I3G8L3_NOMLE\nFGFR1\tA0A2I3GJP7_NOMLE\nFGFR1\tA0A2I3GUX4_NOMLE\nFGFR1\tA0A2I3GV98_NOMLE\nFGFR1\tA0A2I3LLV8_PAPAN\nFGFR1\tA0A2I3LVF3_PAPAN\nFGFR1\tA0A2I3LXJ6_PAPAN\nFGFR1\tA0A2I3MJI7_PAPAN\nFGFR1\tA0A2I3MWJ3_PAPAN\nFGFR1\tA0A2I3MWW0_PAPAN\nFGFR1\tA0A2I3NHZ6_PAPAN\nFGFR1\tA0A2I3RMA9_PANTR\nFGFR1\tA0A2I3S0G3_PANTR\nFGFR1\tA0A2I3S1J5_PANTR\nFGFR1\tA0A2I3S9P6_PANTR\nFGFR1\tA0A2I3SHB9_PANTR\nFGFR1\tA0A2I3TVJ3_PANTR\nFGFR1\tA0A2K5CCY5_AOTNA\nFGFR1\tA0A2K5CCZ1_AOTNA\nFGFR1\tA0A2K5CDB8_AOTNA\nFGFR1\tA0A2K5CDC6_AOTNA\nFGFR1\tA0A2K5CDE8_AOTNA\nFGFR1\tA0A2K5CDF7_AOTNA\nFGFR1\tA0A2K5CDM9_AOTNA\nFGFR1\tA0A2K5CDN1_AOTNA\nFGFR1\tA0A2K5K6C5_COLAP\nFGFR1\tA0A2K5K6H4_COLAP\nFGFR1\tA0A2K5K6I5_COLAP\nFGFR1\tA0A2K5K6I7_COLAP\nFGFR1\tA0A2K5K6J0_COLAP\nFGFR1\tA0A2K5K6J3_COLAP\nFGFR1\tA0A2K5K6K0_COLAP\nFGFR1\tA0A2K5K6K7_COLAP\nFGFR1\tA0A2K5K6K9_COLAP\nFGFR1\tA0A2K5NUX9_CERAT\nFGFR1\tA0A2K5NV10_CERAT\nFGFR1\tA0A2K5NV80_CERAT\nFGFR1\tA0A2K5NV99_CERAT\nFGFR1\tA0A2K5NVA1_CERAT\nFGFR1\tA0A2K5NVH2_CERAT\nFGFR1\tA0A2K5QLV3_CEBCA\nFGFR1\tA0A2K5QLY2_CEBCA\nFGFR1\tA0A2K5QM22_CEBCA\nFGFR1\tA0A2K5QM26_CEBCA\nFGFR1\tA0A2K5QM34_CEBCA\nFGFR1\tA0A2K5QM45_CEBCA\nFGFR1\tA0A2K5QM87_CEBCA\nFGFR1\tA0A2K5WHH4_MACFA\nFGFR1\tA0A2K5WIM5_MACFA\nFGFR1\tA0A2K5WIN3_MACFA\nFGFR1\tA0A2K5WIP3_MACFA\nFGFR1\tA0A2K5WIQ3_MACFA\nFGFR1\tA0A2K5WIQ8_MACFA\nFGFR1\tA0A2K5WIR1_MACFA\nFGFR1\tA0A2K5Y6L9_MANLE\nFGFR1\tA0A2K5Y6P7_MANLE\nFGFR1\tA0A2K5Y6T6_MANLE\nFGFR1\tA0A2K5Y6W6_MANLE\nFGFR1\tA0A2K5Y6Z5_MANLE\nFGFR1\tA0A2K5Y704_MANLE\nFGFR1\tA0A2K5Y707_MANLE\nFGFR1\tA0A2K5Y712_MANLE\nFGFR1\tA0A2K6C8V4_MACNE\nFGFR1\tA0A2K6C8W0_MACNE\nFGFR1\tA0A2K6C8X2_MACNE\nFGFR1\tA0A2K6C8Z7_MACNE\nFGFR1\tA0A2K6C918_MACNE\nFGFR1\tA0A2K6C923_MACNE\nFGFR1\tA0A2K6C928_MACNE\nFGFR1\tA0A2K6C933_MACNE\nFGFR1\tA0A2K6C941_MACNE\nFGFR1\tA0A2K6EKX0_PROCO\nFGFR1\tA0A2K6EKX1_PROCO\nFGFR1\tA0A2K6EKX7_PROCO\nFGFR1\tA0A2K6EKX8_PROCO\nFGFR1\tA0A2K6EKY1_PROCO\nFGFR1\tA0A2K6EKY2_PROCO\nFGFR1\tA0A2K6EKY4_PROCO\nFGFR1\tA0A2K6EKY5_PROCO\nFGFR1\tA0A2K6L1U8_RHIBE\nFGFR1\tA0A2K6L1X1_RHIBE\nFGFR1\tA0A2K6L1Z4_RHIBE\nFGFR1\tA0A2K6L201_RHIBE\nFGFR1\tA0A2K6L206_RHIBE\nFGFR1\tA0A2K6L209_RHIBE\nFGFR1\tA0A2K6L213_RHIBE\nFGFR1\tA0A2K6L214_RHIBE\nFGFR1\tA0A2K6Q900_RHIRO\nFGFR1\tA0A2K6Q903_RHIRO\nFGFR1\tA0A2K6Q904_RHIRO\nFGFR1\tA0A2K6Q919_RHIRO\nFGFR1\tA0A2K6Q923_RHIRO\nFGFR1\tA0A2K6Q924_RHIRO\nFGFR1\tA0A2K6Q929_RHIRO\nFGFR1\tA0A2K6Q932_RHIRO\nFGFR1\tA0A2K6U470_SAIBB\nFGFR1\tA0A2K6U4D7_SAIBB\nFGFR1\tA0A2K6U4E2_SAIBB\nFGFR1\tA0A2K6U4I4_SAIBB\nFGFR1\tA0A2K6U4K3_SAIBB\nFGFR1\tA0A2K6U4L2_SAIBB\nFGFR1\tA0A2K6U4M4_SAIBB\nFGFR1\tA0A2K6U4Q3_SAIBB\nFGFR1\tA0A2R8NE20_CALJA\nFGFR1\tA0A2R9AGY9_PANPA\nFGFR1\tA0A2R9AH20_PANPA\nFGFR1\tA0A2R9AHX6_PANPA\nFGFR1\tA0A2R9AHZ1_PANPA\nFGFR1\tA0A2R9ANV9_PANPA\nFGFR1\tA0A2S2PME2_SCHGA\nFGFR1\tA0A2U3VI28_ODORO\nFGFR1\tA0A2U4A8T0_TURTR\nFGFR1\tA0A2U8UYB3_ANAPL\nFGFR1\tA0A2Y9FNE6_PHYMC\nFGFR1\tA0A2Y9FNL5_PHYMC\nFGFR1\tA0A2Y9FNX5_PHYMC\nFGFR1\tA0A2Y9FQA4_PHYMC\nFGFR1\tA0A2Y9FR04_PHYMC\nFGFR1\tA0A2Y9GDU6_NEOSC\nFGFR1\tA0A2Y9GDY1_NEOSC\nFGFR1\tA0A2Y9GE03_NEOSC\nFGFR1\tA0A2Y9GE07_NEOSC\nFGFR1\tA0A2Y9GGZ3_NEOSC\nFGFR1\tA0A2Y9GIF9_NEOSC\nFGFR1\tA0A2Y9MX06_DELLE\nFGFR1\tA0A2Y9MX11_DELLE\nFGFR1\tA0A2Y9MYI8_DELLE\nFGFR1\tA0A2Y9MYJ4_DELLE\nFGFR1\tA0A2Y9N1B4_DELLE\nFGFR1\tA0A2Y9N1C0_DELLE\nFGFR1\tA0A2Y9N6A7_DELLE\nFGFR1\tA0A2Y9N6B4_DELLE\nFGFR1\tA0A2Y9N6B9_DELLE\nFGFR1\tA0A2Y9NCI2_DELLE\nFGFR1\tA0A2Y9NCJ5_DELLE\nFGFR1\tA0A2Y9T8F9_PHYMC\nFGFR1\tA0A2Y9TG76_PHYMC\nFGFR1\tA0A337S6K3_FELCA\nFGFR1\tA0A337SKI7_FELCA\nFGFR1\tA0A340Y865_LIPVE\nFGFR1\tA0A340Y871_LIPVE\nFGFR1\tA0A340Y9S9_LIPVE\nFGFR1\tA0A340Y9T5_LIPVE\nFGFR1\tA0A340YBM5_LIPVE\nFGFR1\tA0A340YBN0_LIPVE\nFGFR1\tA0A340YD19_LIPVE\nFGFR1\tA0A340YD23_LIPVE\nFGFR1\tA0A340YEI9_LIPVE\nFGFR1\tA0A340YEJ2_LIPVE\nFGFR1\tA0A341D1B5_NEOAA\nFGFR1\tA0A384BQI7_URSMA\nFGFR1\tA0A3B3CC90_ORYME\nFGFR1\tA0A3B3CD03_ORYME\nFGFR1\tA0A3B3ISD1_HUMAN\nFGFR1\tA0A3B3QWM7_9TELE\nFGFR1\tA0A3B3UA53_9TELE\nFGFR1\tA0A3B3YW73_9TELE\nFGFR1\tA0A3B4BMX6_9GOBI\nFGFR1\tA0A3B4ESF0_9CICH\nFGFR1\tA0A3B4ET77_9CICH\nFGFR1\tA0A3B4VQ16_SERDU\nFGFR1\tA0A3B4XMI6_SERLL\nFGFR1\tA0A3B4XRQ1_SERLL\nFGFR1\tA0A3B4XUP4_SERLL\nFGFR1\tA0A3B5AIW7_9TELE\nFGFR1\tA0A3B5AJ17_9TELE\nFGFR1\tA0A3B5PV53_XIPMA\nFGFR1\tA0A3P8Q348_ASTCA\nFGFR1\tA0A3P8Q3V9_ASTCA\nFGFR1\tA0A3P8SFV6_AMPPE\nFGFR1\tA0A3P8SFZ0_AMPPE\nFGFR1\tA0A3P8YR04_ESOLU\nFGFR1\tA0A3P9AAJ6_ESOLU\nFGFR1\tA0A3P9AC63_ESOLU\nFGFR1\tA0A3P9BJK6_9CICH\nFGFR1\tA0A3P9JXC6_ORYLA\nFGFR1\tA0A3P9LLS2_ORYLA\nFGFR1\tA0A3P9MUI0_POERE\nFGFR1\tA0A3Q0CEE4_MESAU\nFGFR1\tA0A3Q0CJ89_MESAU\nFGFR1\tA0A3Q0CJE3_MESAU\nFGFR1\tA0A3Q0CJS8_MESAU\nFGFR1\tA0A3Q0DCL5_TARSY\nFGFR1\tA0A3Q0DJ22_TARSY\nFGFR1\tA0A3Q0GHW4_ALLSI\nFGFR1\tA0A3Q0GLX8_ALLSI\nFGFR1\tA0A3Q0GM31_ALLSI\nFGFR1\tA0A3Q0GM38_ALLSI\nFGFR1\tA0A3Q0T1W2_AMPCI\nFGFR1\tA0A3Q1BE29_AMPOC\nFGFR1\tA0A3Q1CKV0_AMPOC\nFGFR1\tA0A3Q1FZB6_9TELE\nFGFR1\tA0A3Q1GY06_9TELE\nFGFR1\tA0A3Q1K5U9_ANATE\nFGFR1\tA0A3Q1K5V9_ANATE\nFGFR1\tA0A3Q1LUE0_BOVIN\nFGFR1\tA0A3Q1M016_BOVIN\nFGFR1\tA0A3Q1M2N2_BOVIN\nFGFR1\tA0A3Q1MDS3_BOVIN\nFGFR1\tA0A3Q2E728_CYPVA\nFGFR1\tA0A3Q2E8U1_CYPVA\nFGFR1\tA0A3Q2L3B3_HORSE\nFGFR1\tA0A3Q2NNK6_FUNHE\nFGFR1\tA0A3Q2VTJ2_HAPBU\nFGFR1\tA0A3Q2WR75_HAPBU\nFGFR1\tA0A3Q2Y001_HIPCM\nFGFR1\tA0A3Q3BMB2_KRYMA\nFGFR1\tA0A3Q3EPQ6_9LABR\nFGFR1\tA0A3Q3EQ36_9LABR\nFGFR1\tA0A3Q3K186_MONAL\nFGFR1\tA0A3Q3K1D5_MONAL\nFGFR1\tA0A3Q3KZU9_9TELE\nFGFR1\tA0A3Q3L030_9TELE\nFGFR1\tA0A3Q3LBD2_9TELE\nFGFR1\tA0A3Q3LKU1_9LABR\nFGFR1\tA0A3Q3R5N4_MONAL\nFGFR1\tA0A3Q3WG85_MOLML\nFGFR1\tA0A3Q3WGP8_MOLML\nFGFR1\tA0A3Q3WMA4_MOLML\nFGFR1\tA0A3Q3WMJ1_MOLML\nFGFR1\tA0A3Q3WMK1_MOLML\nFGFR1\tA0A3Q3WMN7_MOLML\nFGFR1\tA0A3Q3WMY2_MOLML\nFGFR1\tA0A3Q3WV01_MOLML\nFGFR1\tA0A3Q3X399_MOLML\nFGFR1\tA0A3Q3X3F2_MOLML\nFGFR1\tA0A3Q4H7F6_NEOBR\nFGFR1\tA0A3Q7Q5Z5_CALUR\nFGFR1\tA0A3Q7Q849_CALUR\nFGFR1\tA0A3Q7Q852_CALUR\nFGFR1\tA0A3Q7Q854_CALUR\nFGFR1\tA0A3Q7Q857_CALUR\nFGFR1\tA0A3Q7QI61_CALUR\nFGFR1\tA0A3Q7QI66_CALUR\nFGFR1\tA0A3Q7RMX9_CALUR\nFGFR1\tA0A3Q7RMY2_CALUR\nFGFR1\tA0A3Q7RTG9_VULVU\nFGFR1\tA0A3Q7RVF1_VULVU\nFGFR1\tA0A3Q7S128_VULVU\nFGFR1\tA0A3Q7S3Z2_VULVU\nFGFR1\tA0A3Q7S3Z6_VULVU\nFGFR1\tA0A3Q7T3H2_VULVU\nFGFR1\tA0A3Q7T3H7_VULVU\nFGFR1\tA0A3Q7VSI9_URSAR\nFGFR1\tA0A3Q7VSJ4_URSAR\nFGFR1\tA0A3Q7WGG2_URSAR\nFGFR1\tA0A3Q7WGG7_URSAR\nFGFR1\tA0A3Q7XCG4_URSAR\nFGFR1\tA0A3Q7XMT9_URSAR\nFGFR1\tA0A3Q7XMU6_URSAR\nFGFR1\tA0A419PEM8_CLOSI\nFGFR1\tA0A419PNR1_CLOSI\nFGFR1\tA0A419PU82_CLOSI\nFGFR1\tA0A452CE71_BALAS\nFGFR1\tA0A452CE73_BALAS\nFGFR1\tA0A452CE75_BALAS\nFGFR1\tA0A452CE79_BALAS\nFGFR1\tA0A452CE83_BALAS\nFGFR1\tA0A452CE86_BALAS\nFGFR1\tA0A452E6E7_CAPHI\nFGFR1\tA0A452E6I3_CAPHI\nFGFR1\tA0A452E6I8_CAPHI\nFGFR1\tA0A452E6P2_CAPHI\nFGFR1\tA0A452IF51_9SAUR\nFGFR1\tA0A452IFN7_9SAUR\nFGFR1\tA0A452IFP2_9SAUR\nFGFR1\tA0A452S0M0_URSAM\nFGFR1\tA0A452S0P6_URSAM\nFGFR1\tA0A452S0U3_URSAM\nFGFR1\tA0A452S0U5_URSAM\nFGFR1\tA0A452S0V4_URSAM\nFGFR1\tA0A452S0V6_URSAM\nFGFR1\tA0A452S104_URSAM\nFGFR1\tA0A452S185_URSAM\nFGFR1\tA0A452S1A2_URSAM\nFGFR1\tA0A452S1A9_URSAM\nFGFR1\tA0A452TVY0_URSMA\nFGFR1\tA0A452TVY2_URSMA\nFGFR1\tA0A4W2BV75_BOBOX\nFGFR1\tA0A4W2BXS0_BOBOX\nFGFR1\tA0A4W2BXT6_BOBOX\nFGFR1\tA0A4W2CCM9_BOBOX\nFGFR1\tA0A4W2DR41_BOBOX\nFGFR1\tA0A4W2DRQ9_BOBOX\nFGFR1\tA0A4W2DWZ2_BOBOX\nFGFR1\tA0A4W2FAT4_BOBOX\nFGFR1\tA0A4W2FAZ1_BOBOX\nFGFR1\tA0A4W2FB25_BOBOX\nFGFR1\tA0A4W2HDL5_BOBOX\nFGFR1\tA0A4W2HDM5_BOBOX\nFGFR1\tA0A4W2HFN9_BOBOX\nFGFR1\tA0A4W5RCH8_9TELE\nFGFR1\tA0A4W5RI50_9TELE\nFGFR1\tA0A4W5RQI6_9TELE\nFGFR1\tA0A4W5RQY0_9TELE\nFGFR1\tA0A4W5RR25_9TELE\nFGFR1\tA0A4W5RR55_9TELE\nFGFR1\tA0A4W5RUC1_9TELE\nFGFR1\tA0A4W5RUN7_9TELE\nFGFR1\tA0A4W5RUP2_9TELE\nFGFR1\tA0A4W6FGS3_LATCA\nFGFR1\tA0A4W6FHM3_LATCA\nFGFR1\tA0A4W6FHW1_LATCA\nFGFR1\tA0A4W6FHX5_LATCA\nFGFR1\tA0A4W6FHY4_LATCA\nFGFR1\tA0A4W6FHZ6_LATCA\nFGFR1\tA0A4W6FIX2_LATCA\nFGFR1\tA0A4W6FIY9_LATCA\nFGFR1\tA0A4W6FJY2_LATCA\nFGFR1\tA0A4X1VV81_PIG\nFGFR1\tA0A4X1VWC3_PIG\nFGFR1\tA0A4X1VXC3_PIG\nFGFR1\tA0A4X2KMN9_VOMUR\nFGFR1\tA0A4X2KSW2_VOMUR\nFGFR1\tA0A4X2KV96_VOMUR\nFGFR1\tA0A4X2KVD2_VOMUR\nFGFR1\tA0A5F4C361_CANLF\nFGFR1\tA0A5F4CQJ1_CANLF\nFGFR1\tA0A5F4CSN0_CANLF\nFGFR1\tA0A5F4DN57_CANLF\nFGFR1\tA0A5F4W6N8_CALJA\nFGFR1\tA0A5F4W851_CALJA\nFGFR1\tA0A5F5PH67_HORSE\nFGFR1\tA0A5F5PIJ2_HORSE\nFGFR1\tA0A5F5PNY6_HORSE\nFGFR1\tA0A5F9C0P9_RABIT\nFGFR1\tA0A5F9C676_RABIT\nFGFR1\tA0A5F9C7Y0_RABIT\nFGFR1\tA0A5F9CFG0_RABIT\nFGFR1\tA0A5F9D998_RABIT\nFGFR1\tA0A5F9DJS2_RABIT\nFGFR1\tA0A5G2QI33_PIG\nFGFR1\tA0A5G2QRT9_PIG\nFGFR1\tA0A5G3JNJ4_XENTR\nFGFR1\tA4IFL5_BOVIN\nFGFR1\tB2L479_XENBO\nFGFR1\tB5A958_HUMAN\nFGFR1\tB5A959_HUMAN\nFGFR1\tB7ZRE2_XENLA\nFGFR1\tC1KBH7_HUMAN\nFGFR1\tC9J1L5_HUMAN\nFGFR1\tC9J205_HUMAN\nFGFR1\tD3DSX2_HUMAN\nFGFR1\tD3DSX4_HUMAN\nFGFR1\tD3YW77_MOUSE\nFGFR1\tD3Z4V6_MOUSE\nFGFR1\tD5FGF2_SQUAC\nFGFR1\tD5FGF3_LEUER\nFGFR1\tD5FGJ8_SQUAC\nFGFR1\tE2R186_CANLF\nFGFR1\tE7EU09_HUMAN\nFGFR1\tE9PKF2_HUMAN\nFGFR1\tE9PKV7_HUMAN\nFGFR1\tE9PKX3_HUMAN\nFGFR1\tE9PN14_HUMAN\nFGFR1\tE9PNM3_HUMAN\nFGFR1\tE9PQ40_HUMAN\nFGFR1\tF1LM54_RAT\nFGFR1\tF1RZJ8_PIG\nFGFR1\tF6RX30_XENTR\nFGFR1\tF6V0C7_HORSE\nFGFR1\tF6V0E6_HORSE\nFGFR1\tF6W904_CALJA\nFGFR1\tF6YNK3_CALJA\nFGFR1\tF7A4R9_MACMU\nFGFR1\tF7A4T4_MACMU\nFGFR1\tF7A4U3_MACMU\nFGFR1\tF7D4I5_MONDO\nFGFR1\tF7EMT5_CALJA\nFGFR1\tF7EMY1_CALJA\nFGFR1\tF7I8W1_CALJA\nFGFR1\tF7IIP4_CALJA\nFGFR1\tFGFR1_CHICK\nFGFR1\tFGFR1_DUGJA\nFGFR1\tFGFR1_HUMAN\nFGFR1\tFGFR1_MOUSE\nFGFR1\tFGFR1_PLEWA\nFGFR1\tFGFR1_RAT\nFGFR1\tFGFR1_XENLA\nFGFR1\tFGR1A_DANRE\nFGFR1\tG1MAQ3_AILME\nFGFR1\tG1MT22_MELGA\nFGFR1\tG1NWC0_MYOLU\nFGFR1\tG1RNZ6_NOMLE\nFGFR1\tG1SL66_RABIT\nFGFR1\tG3PFN6_GASAC\nFGFR1\tG3TKE5_LOXAF\nFGFR1\tG3WZH4_SARHA\nFGFR1\tG3WZH5_SARHA\nFGFR1\tH0V817_CAVPO\nFGFR1\tH0XBR9_OTOGA\nFGFR1\tH0YE20_HUMAN\nFGFR1\tH0Z3I3_TAEGU\nFGFR1\tH2MGT2_ORYLA\nFGFR1\tH2PQ36_PONAB\nFGFR1\tH2R210_PANTR\nFGFR1\tH2UZ64_TAKRU\nFGFR1\tH2UZ69_TAKRU\nFGFR1\tH2UZ73_TAKRU\nFGFR1\tH3B0H0_LATCH\nFGFR1\tH3B0H1_LATCH\nFGFR1\tH3DND6_TETNG\nFGFR1\tH9FRD4_MACMU\nFGFR1\tH9GPZ1_ANOCA\nFGFR1\tH9U390_FELCA\nFGFR1\tH9U391_FELCA\nFGFR1\tI3JYB1_ORENI\nFGFR1\tI3JYB2_ORENI\nFGFR1\tI3LY43_ICTTR\nFGFR1\tJ3QN85_MOUSE\nFGFR1\tK7CCP1_PANTR\nFGFR1\tK7FGA2_PELSI\nFGFR1\tK7GQJ1_PIG\nFGFR1\tM3YE16_MUSPF\nFGFR1\tQ28CQ6_XENTR\nFGFR1\tQ28J96_XENTR\nFGFR1\tQ3TJ05_MOUSE\nFGFR1\tQ60818_MOUSE\nFGFR1\tQ61565_MOUSE\nFGFR1\tQ63827_RAT\nFGFR1\tQ8CBY7_MOUSE\nFGFR1\tQ8CIM9_MOUSE\nFGFR1\tQ91146_NOTVI\nFGFR1\tQ91897_XENLA\nFGFR1\tT2MHI4_HYDVU\nFGFR1\tU3J361_ANAPP\nFGFR1\tU3JVZ0_FICAL\nFGFR1\tU3KNN6_RABIT\nFGFR1\tU6D362_NEOVI\nFGFR1\tV5GVL3_ANOGL\nFGFR1\tV8NKM6_OPHHA\nFGFR1\tW5NU11_SHEEP\nFGFR1\tW8BKV1_CERCA\n','From\tTo\nTNK2\tA0A096MWZ1_PAPAN\nTNK2\tA0A0A1XEP8_ZEUCU\nTNK2\tA0A0D9R851_CHLSB\nTNK2\tA0A0G2JYY3_RAT\nTNK2\tA0A0R4J1L0_MOUSE\nTNK2\tA0A146L5A1_LYGHE\nTNK2\tA0A146M0Y9_LYGHE\nTNK2\tA0A151MZG6_ALLMI\nTNK2\tA0A1A7XUC8_9TELE\nTNK2\tA0A1A8A3M4_NOTFU\nTNK2\tA0A1A8BBT9_9TELE\nTNK2\tA0A1A8CMV2_9TELE\nTNK2\tA0A1A8EC41_9TELE\nTNK2\tA0A1A8F5M6_9TELE\nTNK2\tA0A1A8F953_9TELE\nTNK2\tA0A1A8FZX7_9TELE\nTNK2\tA0A1A8GKW0_9TELE\nTNK2\tA0A1A8ISP3_NOTKU\nTNK2\tA0A1A8KL09_NOTKU\nTNK2\tA0A1A8L6E6_9TELE\nTNK2\tA0A1A8NS57_9TELE\nTNK2\tA0A1A8V3A4_NOTFU\nTNK2\tA0A1L8G3X6_XENLA\nTNK2\tA0A1L8G9W3_XENLA\nTNK2\tA0A1S3FFE2_DIPOR\nTNK2\tA0A1S3LUQ7_SALSA\nTNK2\tA0A1S3LUV7_SALSA\nTNK2\tA0A1S3LV56_SALSA\nTNK2\tA0A1S3LVW8_SALSA\nTNK2\tA0A1S3WWA8_ERIEU\nTNK2\tA0A1U7QDG8_MESAU\nTNK2\tA0A1U7QP12_MESAU\nTNK2\tA0A1U7SQD9_TARSY\nTNK2\tA0A1U7SX34_TARSY\nTNK2\tA0A1U7T894_TARSY\nTNK2\tA0A1U8BV42_MESAU\nTNK2\tA0A1U8C2H8_MESAU\nTNK2\tA0A1U8C2I1_MESAU\nTNK2\tA0A1U8C316_MESAU\nTNK2\tA0A1Y7VJA6_MOUSE\nTNK2\tA0A1Y7VLM9_MOUSE\nTNK2\tA0A218V922_9PASE\nTNK2\tA0A250Y6R4_CASCN\nTNK2\tA0A286XL20_CAVPO\nTNK2\tA0A286XR66_CAVPO\nTNK2\tA0A286Y420_CAVPO\nTNK2\tA0A286ZT97_PIG\nTNK2\tA0A287AYT9_PIG\nTNK2\tA0A287B071_PIG\nTNK2\tA0A287B0J6_PIG\nTNK2\tA0A287BC95_PIG\nTNK2\tA0A287BKC7_PIG\nTNK2\tA0A287CXA6_ICTTR\nTNK2\tA0A287CZE4_ICTTR\nTNK2\tA0A287DF43_ICTTR\nTNK2\tA0A2B4RIC6_STYPI\nTNK2\tA0A2D0T7X8_ICTPU\nTNK2\tA0A2D0T7Y0_ICTPU\nTNK2\tA0A2D0T7Y7_ICTPU\nTNK2\tA0A2D0T813_ICTPU\nTNK2\tA0A2D0T8L0_ICTPU\nTNK2\tA0A2H8TLD7_9HEMI\nTNK2\tA0A2I2UU89_FELCA\nTNK2\tA0A2I2V3Q8_FELCA\nTNK2\tA0A2I3FWR1_NOMLE\nTNK2\tA0A2I3HX48_NOMLE\nTNK2\tA0A2I3LD60_PAPAN\nTNK2\tA0A2I3MHV5_PAPAN\nTNK2\tA0A2I3SE81_PANTR\nTNK2\tA0A2I3SQD9_PANTR\nTNK2\tA0A2I3T8A3_PANTR\nTNK2\tA0A2I3T9N6_PANTR\nTNK2\tA0A2I3TMU8_PANTR\nTNK2\tA0A2I4BP66_9TELE\nTNK2\tA0A2I4BP67_9TELE\nTNK2\tA0A2I4BP72_9TELE\nTNK2\tA0A2I4BP74_9TELE\nTNK2\tA0A2I4BP76_9TELE\nTNK2\tA0A2I4BPA1_9TELE\nTNK2\tA0A2I4BPA4_9TELE\nTNK2\tA0A2I4BPA7_9TELE\nTNK2\tA0A2I4BPA8_9TELE\nTNK2\tA0A2K5EKR6_AOTNA\nTNK2\tA0A2K5EKS9_AOTNA\nTNK2\tA0A2K5EKU8_AOTNA\nTNK2\tA0A2K5EKV0_AOTNA\nTNK2\tA0A2K5EKZ4_AOTNA\nTNK2\tA0A2K5EKZ8_AOTNA\nTNK2\tA0A2K5HVV0_COLAP\nTNK2\tA0A2K5HVV5_COLAP\nTNK2\tA0A2K5HVZ6_COLAP\nTNK2\tA0A2K5HW48_COLAP\nTNK2\tA0A2K5P5Z7_CERAT\nTNK2\tA0A2K5P604_CERAT\nTNK2\tA0A2K5P607_CERAT\nTNK2\tA0A2K5P609_CERAT\nTNK2\tA0A2K5P611_CERAT\nTNK2\tA0A2K5RFS9_CEBCA\nTNK2\tA0A2K5RFT5_CEBCA\nTNK2\tA0A2K5RFU3_CEBCA\nTNK2\tA0A2K5RFU8_CEBCA\nTNK2\tA0A2K5WSY4_MACFA\nTNK2\tA0A2K5WSZ4_MACFA\nTNK2\tA0A2K5WSZ9_MACFA\nTNK2\tA0A2K5WT19_MACFA\nTNK2\tA0A2K5WT37_MACFA\nTNK2\tA0A2K5WT42_MACFA\nTNK2\tA0A2K6DLI5_MACNE\nTNK2\tA0A2K6DLM2_MACNE\nTNK2\tA0A2K6DLN3_MACNE\nTNK2\tA0A2K6DLP3_MACNE\nTNK2\tA0A2K6DLQ1_MACNE\nTNK2\tA0A2K6GB87_PROCO\nTNK2\tA0A2K6GB88_PROCO\nTNK2\tA0A2K6GBC0_PROCO\nTNK2\tA0A2K6MW79_RHIBE\nTNK2\tA0A2K6MW83_RHIBE\nTNK2\tA0A2K6MW92_RHIBE\nTNK2\tA0A2K6MW96_RHIBE\nTNK2\tA0A2K6Q9Q3_RHIRO\nTNK2\tA0A2K6Q9Q4_RHIRO\nTNK2\tA0A2K6Q9Q7_RHIRO\nTNK2\tA0A2K6Q9R3_RHIRO\nTNK2\tA0A2K6Q9R6_RHIRO\nTNK2\tA0A2K6Q9S9_RHIRO\nTNK2\tA0A2K6Q9T0_RHIRO\nTNK2\tA0A2K6UEM4_SAIBB\nTNK2\tA0A2K6UEM5_SAIBB\nTNK2\tA0A2K6UEP4_SAIBB\nTNK2\tA0A2K6UEQ7_SAIBB\nTNK2\tA0A2R8M6P3_CALJA\nTNK2\tA0A2R8MEY7_CALJA\nTNK2\tA0A2R8MSW9_CALJA\nTNK2\tA0A2R8PEX6_CALJA\nTNK2\tA0A2S2NIV5_SCHGA\nTNK2\tA0A2U3VBQ3_ODORO\nTNK2\tA0A2U3VBQ4_ODORO\nTNK2\tA0A2U3VBQ6_ODORO\nTNK2\tA0A2U3VC67_ODORO\nTNK2\tA0A2U3YDE6_LEPWE\nTNK2\tA0A2U3ZBN0_ODORO\nTNK2\tA0A2U3ZBP8_ODORO\nTNK2\tA0A2U3ZBQ8_ODORO\nTNK2\tA0A2Y9H9G5_NEOSC\nTNK2\tA0A2Y9H9G8_NEOSC\nTNK2\tA0A2Y9H9M2_NEOSC\nTNK2\tA0A2Y9H9M6_NEOSC\nTNK2\tA0A2Y9H9N8_NEOSC\nTNK2\tA0A2Y9H9P3_NEOSC\nTNK2\tA0A2Y9HFD8_NEOSC\nTNK2\tA0A2Y9HFE3_NEOSC\nTNK2\tA0A2Y9HG82_NEOSC\nTNK2\tA0A2Y9HG87_NEOSC\nTNK2\tA0A2Y9HG91_NEOSC\nTNK2\tA0A2Y9MXZ0_DELLE\nTNK2\tA0A2Y9MY15_DELLE\nTNK2\tA0A337RZ85_FELCA\nTNK2\tA0A340XX53_LIPVE\nTNK2\tA0A340XXR5_LIPVE\nTNK2\tA0A340Y0C5_LIPVE\nTNK2\tA0A340Y2R9_LIPVE\nTNK2\tA0A340Y385_LIPVE\nTNK2\tA0A341C6R9_NEOAA\nTNK2\tA0A341C6S4_NEOAA\nTNK2\tA0A341C6S7_NEOAA\nTNK2\tA0A341C6S8_NEOAA\nTNK2\tA0A341C6T2_NEOAA\nTNK2\tA0A341C6T7_NEOAA\nTNK2\tA0A341C7I6_NEOAA\nTNK2\tA0A341C7J2_NEOAA\nTNK2\tA0A341C7J6_NEOAA\nTNK2\tA0A341C7X3_NEOAA\nTNK2\tA0A341C7X8_NEOAA\nTNK2\tA0A341C994_NEOAA\nTNK2\tA0A341C999_NEOAA\nTNK2\tA0A341C9A5_NEOAA\nTNK2\tA0A383Z9E1_BALAS\nTNK2\tA0A384DIK2_URSMA\nTNK2\tA0A3Q0CM89_MESAU\nTNK2\tA0A3Q0CSL0_MESAU\nTNK2\tA0A3Q0CSN4_MESAU\nTNK2\tA0A3Q0CSN8_MESAU\nTNK2\tA0A3Q0CSY4_MESAU\nTNK2\tA0A3Q0CSY6_MESAU\nTNK2\tA0A3Q0DQ28_TARSY\nTNK2\tA0A3Q0DT52_TARSY\nTNK2\tA0A3Q0GG74_ALLSI\nTNK2\tA0A3Q1LXE5_BOVIN\nTNK2\tA0A3Q1MJR1_BOVIN\nTNK2\tA0A3Q2HHV8_HORSE\nTNK2\tA0A3Q2HPM4_HORSE\nTNK2\tA0A3Q2I8N3_HORSE\nTNK2\tA0A3Q2KRS7_HORSE\nTNK2\tA0A3Q2TTQ3_CHICK\nTNK2\tA0A3Q2UET3_CHICK\nTNK2\tA0A3Q2UHM8_CHICK\nTNK2\tA0A3Q7PCV5_CALUR\nTNK2\tA0A3Q7PCY6_CALUR\nTNK2\tA0A3Q7PNT0_CALUR\nTNK2\tA0A3Q7PNT8_CALUR\nTNK2\tA0A3Q7QTY6_CALUR\nTNK2\tA0A3Q7RJS8_VULVU\nTNK2\tA0A3Q7RJT4_VULVU\nTNK2\tA0A3Q7RJT7_VULVU\nTNK2\tA0A3Q7RK62_VULVU\nTNK2\tA0A3Q7RK67_VULVU\nTNK2\tA0A3Q7RLH0_VULVU\nTNK2\tA0A3Q7RLH6_VULVU\nTNK2\tA0A3Q7RLI1_VULVU\nTNK2\tA0A3Q7RVH3_VULVU\nTNK2\tA0A3Q7SUN9_VULVU\nTNK2\tA0A3Q7SUP4_VULVU\nTNK2\tA0A3Q7US42_URSAR\nTNK2\tA0A3Q7US65_URSAR\nTNK2\tA0A3Q7VQV4_URSAR\nTNK2\tA0A3Q7VQV9_URSAR\nTNK2\tA0A3Q7WA10_URSAR\nTNK2\tA0A3Q7WA36_URSAR\nTNK2\tA0A3Q7WA47_URSAR\nTNK2\tA0A3Q7WA58_URSAR\nTNK2\tA0A3Q7WLI8_URSAR\nTNK2\tA0A3Q7WLK3_URSAR\nTNK2\tA0A3Q7WLK8_URSAR\nTNK2\tA0A419PFB0_CLOSI\nTNK2\tA0A452EC43_CAPHI\nTNK2\tA0A452EC95_CAPHI\nTNK2\tA0A452ECG2_CAPHI\nTNK2\tA0A452ECK0_CAPHI\nTNK2\tA0A452ED19_CAPHI\nTNK2\tA0A452SWZ5_URSAM\nTNK2\tA0A452SX53_URSAM\nTNK2\tA0A452T7I6_URSMA\nTNK2\tA0A455BHZ7_PHYMC\nTNK2\tA0A455BID2_PHYMC\nTNK2\tA0A455BID7_PHYMC\nTNK2\tA0A455BL38_PHYMC\nTNK2\tA0A494BAM7_MOUSE\nTNK2\tA0A499FJ16_HUMAN\nTNK2\tA0A4C1TKI0_9NEOP\nTNK2\tA0A4C1VVS5_9NEOP\nTNK2\tA0A4W2EPY5_BOBOX\nTNK2\tA0A4W2EPZ0_BOBOX\nTNK2\tA0A4W2IN21_BOBOX\nTNK2\tA0A4W2IN59_BOBOX\nTNK2\tA0A4W2IPD7_BOBOX\nTNK2\tA0A4W3HS80_CALMI\nTNK2\tA0A4W3HV24_CALMI\nTNK2\tA0A4W3HXZ3_CALMI\nTNK2\tA0A4W3HY09_CALMI\nTNK2\tA0A4W3HY24_CALMI\nTNK2\tA0A4W3HY31_CALMI\nTNK2\tA0A4W3HZ51_CALMI\nTNK2\tA0A4W3IH01_CALMI\nTNK2\tA0A4W3IH05_CALMI\nTNK2\tA0A4W6BV56_LATCA\nTNK2\tA0A4W6BVM9_LATCA\nTNK2\tA0A4W6C0C0_LATCA\nTNK2\tA0A4W6C326_LATCA\nTNK2\tA0A4W6C342_LATCA\nTNK2\tA0A4X1TT80_PIG\nTNK2\tA0A4X1TTF5_PIG\nTNK2\tA0A4X2KDL5_VOMUR\nTNK2\tA0A4X2KDP2_VOMUR\nTNK2\tA0A4X2KJ71_VOMUR\nTNK2\tA0A4Y2ES56_ARAVE\nTNK2\tA0A5F4BYD9_CANLF\nTNK2\tA0A5F4BZQ0_CANLF\nTNK2\tA0A5F4CBQ6_CANLF\nTNK2\tA0A5F4CTG9_CANLF\nTNK2\tA0A5F4CWK3_CANLF\nTNK2\tA0A5F4W345_CALJA\nTNK2\tA0A5F5Q064_HORSE\nTNK2\tA0A5F5Q2U2_HORSE\nTNK2\tA0A5F7ZPT5_MACMU\nTNK2\tA0A5F7ZT94_MACMU\nTNK2\tA0A5F8GR19_MONDO\nTNK2\tA0A5F8GR85_MONDO\nTNK2\tA0A5F8GWM6_MONDO\nTNK2\tA0A5F8HB58_MONDO\nTNK2\tA0A5F8MPC4_MOUSE\nTNK2\tA0A5F8MPI0_MOUSE\nTNK2\tA0A5F8MPJ6_MOUSE\nTNK2\tA0A5F8MPJ9_MOUSE\nTNK2\tA0A5F8MPL4_MOUSE\nTNK2\tA0A5F8MPM0_MOUSE\nTNK2\tA0A5F8MQ87_MOUSE\nTNK2\tA0A5F9CVA0_RABIT\nTNK2\tA0A5F9DP33_RABIT\nTNK2\tA0A5F9DV87_RABIT\nTNK2\tA0A5F9ZGX5_HUMAN\nTNK2\tA0A5F9ZGY5_HUMAN\nTNK2\tA0A5F9ZH01_HUMAN\nTNK2\tA0A5F9ZH66_HUMAN\nTNK2\tA0A5F9ZHB0_HUMAN\nTNK2\tA0A5F9ZHC7_HUMAN\nTNK2\tA0A5F9ZHD0_HUMAN\nTNK2\tA0A5F9ZHI7_HUMAN\nTNK2\tA0A5F9ZHJ2_HUMAN\nTNK2\tA0A5F9ZHK8_HUMAN\nTNK2\tA0A5F9ZHL4_HUMAN\nTNK2\tA0A5F9ZHP4_HUMAN\nTNK2\tA0A5F9ZHP9_HUMAN\nTNK2\tA0A5F9ZHQ1_HUMAN\nTNK2\tA0A5F9ZHX6_HUMAN\nTNK2\tA0A5F9ZI29_HUMAN\nTNK2\tA0A5F9ZI44_HUMAN\nTNK2\tA0A5G3H6J3_XENTR\nTNK2\tA0A5G3IBL9_XENTR\nTNK2\tA0A5G3IC72_XENTR\nTNK2\tA0A5G3ICL8_XENTR\nTNK2\tA0JMM7_DANRE\nTNK2\tA4IGR9_XENTR\nTNK2\tACK1_BOVIN\nTNK2\tACK1_HUMAN\nTNK2\tACK1_MOUSE\nTNK2\tACK1_RAT\nTNK2\tC4R9I2_PIG\nTNK2\tC4R9I3_PIG\nTNK2\tC4R9I4_PIG\nTNK2\tC9J1X3_HUMAN\nTNK2\tD3Z3U5_MOUSE\nTNK2\tD3Z3U6_MOUSE\nTNK2\tD3Z3U7_MOUSE\nTNK2\tE2RFP7_CANLF\nTNK2\tE6ZJD6_DICLA\nTNK2\tE6ZJD7_DICLA\nTNK2\tE9PWA3_MOUSE\nTNK2\tE9Q5M1_MOUSE\nTNK2\tE9QN60_MOUSE\nTNK2\tE9QN61_MOUSE\nTNK2\tF1M7W9_RAT\nTNK2\tF1MSL0_BOVIN\nTNK2\tF6PN06_BOVIN\nTNK2\tF6RB26_XENTR\nTNK2\tF6RYG7_XENTR\nTNK2\tF6SXS7_MACMU\nTNK2\tF6VDS6_HORSE\nTNK2\tF6XU33_MONDO\nTNK2\tF6ZGF2_XENTR\nTNK2\tF7BA54_CALJA\nTNK2\tF7BAX3_CALJA\nTNK2\tF7DB92_XENTR\nTNK2\tF7DBC2_XENTR\nTNK2\tF7FZH0_MACMU\nTNK2\tF7FZJ1_MACMU\nTNK2\tG1LBE7_AILME\nTNK2\tG1LBF6_AILME\nTNK2\tG1PX27_MYOLU\nTNK2\tG1PYK5_MYOLU\nTNK2\tG1QKV5_NOMLE\nTNK2\tG1SVN8_RABIT\nTNK2\tG3SWX7_LOXAF\nTNK2\tG3UK03_LOXAF\nTNK2\tG3ULY6_LOXAF\nTNK2\tG3VNA6_SARHA\nTNK2\tG3X9X7_MOUSE\nTNK2\tH0VF76_CAVPO\nTNK2\tH0X1B9_OTOGA\nTNK2\tH2PCF4_PONAB\nTNK2\tH2RU63_TAKRU\nTNK2\tH9FNI3_MACMU\nTNK2\tH9FNI4_MACMU\nTNK2\tH9FNI5_MACMU\nTNK2\tI3LH58_PIG\nTNK2\tI3M4U6_ICTTR\nTNK2\tK7B5Z4_PANTR\nTNK2\tK7CPB4_PANTR\nTNK2\tK7D2N2_PANTR\nTNK2\tK7DR16_PANTR\nTNK2\tM3VYX2_FELCA\nTNK2\tM3XY32_MUSPF\nTNK2\tR4GK19_CHICK\nTNK2\tT2MED2_HYDVU\nTNK2\tU3FC53_CALJA\nTNK2\tU3FQ60_CALJA\nTNK2\tU3JYF1_FICAL\nTNK2\tV8N5L6_OPHHA\n','From\tTo\nFYN\tA0A096MKC1_RAT\nFYN\tA0A096MKC5_RAT\nFYN\tA0A096NW74_PAPAN\nFYN\tA0A0B2UP19_TOXCA\nFYN\tA0A0B2VU22_TOXCA\nFYN\tA0A0D9RST7_CHLSB\nFYN\tA0A0G2K2E3_RAT\nFYN\tA0A0S7EQR3_9TELE\nFYN\tA0A0S7ERG1_9TELE\nFYN\tA0A0S7F128_9TELE\nFYN\tA0A0S7HDM3_9TELE\nFYN\tA0A0S7HDU3_9TELE\nFYN\tA0A0S7HG13_9TELE\nFYN\tA0A0S7HG17_9TELE\nFYN\tA0A0S7HH45_9TELE\nFYN\tA0A0S7HHZ7_9TELE\nFYN\tA0A0S7HJ89_9TELE\nFYN\tA0A0S7HXL2_9TELE\nFYN\tA0A0S7I297_9TELE\nFYN\tA0A0S7I6X6_9TELE\nFYN\tA0A0S7I770_9TELE\nFYN\tA0A0V0U0Z1_9BILA\nFYN\tA0A0V0UYY7_9BILA\nFYN\tA0A0V0UZ47_9BILA\nFYN\tA0A0V0V0F1_9BILA\nFYN\tA0A0V0WJ14_9BILA\nFYN\tA0A0V0ZTZ0_9BILA\nFYN\tA0A0V1BRX8_TRISP\nFYN\tA0A0V1BS50_TRISP\nFYN\tA0A0V1CSF8_TRIBR\nFYN\tA0A0V1EPX3_TRIPS\nFYN\tA0A0V1H8T1_9BILA\nFYN\tA0A0V1IR95_TRIPS\nFYN\tA0A0V1JPQ0_TRIPS\nFYN\tA0A0V1LKP0_9BILA\nFYN\tA0A0V1N4A6_9BILA\nFYN\tA0A0V1N4S1_9BILA\nFYN\tA0A0V1P4C2_9BILA\nFYN\tA0A1L8G8Y4_XENLA\nFYN\tA0A1S3A277_ERIEU\nFYN\tA0A1S3A279_ERIEU\nFYN\tA0A1S3A2A8_ERIEU\nFYN\tA0A1S3GHG1_DIPOR\nFYN\tA0A1S3S793_SALSA\nFYN\tA0A1S3S798_SALSA\nFYN\tA0A1U7Q8Y9_MESAU\nFYN\tA0A1U7RSH7_ALLSI\nFYN\tA0A1U8CZC5_ALLSI\nFYN\tA0A1V4JLW1_PATFA\nFYN\tA0A1V4JM46_PATFA\nFYN\tA0A1V4JMB1_PATFA\nFYN\tA0A1Y1S4V9_9MICR\nFYN\tA0A218UXN4_9PASE\nFYN\tA0A250Y2G0_CASCN\nFYN\tA0A286XCC7_CAVPO\nFYN\tA0A286XJ68_CAVPO\nFYN\tA0A2B4S030_STYPI\nFYN\tA0A2I0MUC4_COLLI\nFYN\tA0A2I2UWP9_FELCA\nFYN\tA0A2I2V3Q2_FELCA\nFYN\tA0A2I3GI58_NOMLE\nFYN\tA0A2I3HC72_NOMLE\nFYN\tA0A2I3HVZ0_NOMLE\nFYN\tA0A2I3LE84_PAPAN\nFYN\tA0A2I3MR50_PAPAN\nFYN\tA0A2I3MTI7_PAPAN\nFYN\tA0A2I3RUG1_PANTR\nFYN\tA0A2I3SKP4_PANTR\nFYN\tA0A2I3TUY5_PANTR\nFYN\tA0A2K5F9P6_AOTNA\nFYN\tA0A2K5F9Q1_AOTNA\nFYN\tA0A2K5F9Q4_AOTNA\nFYN\tA0A2K5F9Q6_AOTNA\nFYN\tA0A2K5IMW1_COLAP\nFYN\tA0A2K5IN18_COLAP\nFYN\tA0A2K5IN97_COLAP\nFYN\tA0A2K5INB9_COLAP\nFYN\tA0A2K5MDU1_CERAT\nFYN\tA0A2K5MDU7_CERAT\nFYN\tA0A2K5MDW5_CERAT\nFYN\tA0A2K5MDY2_CERAT\nFYN\tA0A2K5QZC9_CEBCA\nFYN\tA0A2K5QZE4_CEBCA\nFYN\tA0A2K5QZE7_CEBCA\nFYN\tA0A2K5QZK0_CEBCA\nFYN\tA0A2K5TKJ7_MACFA\nFYN\tA0A2K5TKN0_MACFA\nFYN\tA0A2K5TKN9_MACFA\nFYN\tA0A2K5YJ25_MANLE\nFYN\tA0A2K5YJ28_MANLE\nFYN\tA0A2K5YJ47_MANLE\nFYN\tA0A2K5YJ85_MANLE\nFYN\tA0A2K6B9D1_MACNE\nFYN\tA0A2K6B9G7_MACNE\nFYN\tA0A2K6B9H5_MACNE\nFYN\tA0A2K6B9K9_MACNE\nFYN\tA0A2K6F3F9_PROCO\nFYN\tA0A2K6F3H1_PROCO\nFYN\tA0A2K6F3H3_PROCO\nFYN\tA0A2K6F3I9_PROCO\nFYN\tA0A2K6K4V6_RHIBE\nFYN\tA0A2K6K4W3_RHIBE\nFYN\tA0A2K6K4X7_RHIBE\nFYN\tA0A2K6K575_RHIBE\nFYN\tA0A2K6PQK4_RHIRO\nFYN\tA0A2K6PQM8_RHIRO\nFYN\tA0A2K6PQN0_RHIRO\nFYN\tA0A2K6PQN6_RHIRO\nFYN\tA0A2K6UHR8_SAIBB\nFYN\tA0A2K6UHS9_SAIBB\nFYN\tA0A2K6UHW1_SAIBB\nFYN\tA0A2K6UI26_SAIBB\nFYN\tA0A2R9B759_PANPA\nFYN\tA0A2R9B8P4_PANPA\nFYN\tA0A2R9BAX8_PANPA\nFYN\tA0A2R9BEL8_PANPA\nFYN\tA0A2U3WDX9_ODORO\nFYN\tA0A2U3ZM24_ODORO\nFYN\tA0A2U3ZM32_ODORO\nFYN\tA0A2U3ZN58_ODORO\nFYN\tA0A2U4AW09_TURTR\nFYN\tA0A2U4AW15_TURTR\nFYN\tA0A2U4AW31_TURTR\nFYN\tA0A2Y9HC59_NEOSC\nFYN\tA0A2Y9HIB0_NEOSC\nFYN\tA0A2Y9HIT8_NEOSC\nFYN\tA0A2Y9LLH3_DELLE\nFYN\tA0A2Y9LTA9_DELLE\nFYN\tA0A2Y9LYQ1_DELLE\nFYN\tA0A2Y9S551_PHYMC\nFYN\tA0A2Y9S5U4_PHYMC\nFYN\tA0A340WT33_LIPVE\nFYN\tA0A340X015_LIPVE\nFYN\tA0A340X0M5_LIPVE\nFYN\tA0A341B1L3_NEOAA\nFYN\tA0A341B1P7_NEOAA\nFYN\tA0A341B3I1_NEOAA\nFYN\tA0A384ATR0_BALAS\nFYN\tA0A384ATR3_BALAS\nFYN\tA0A384DBM2_URSMA\nFYN\tA0A384DBN0_URSMA\nFYN\tA0A384DCA1_URSMA\nFYN\tA0A3B3BAT7_ORYME\nFYN\tA0A3Q0CVN3_MESAU\nFYN\tA0A3Q0E3C6_TARSY\nFYN\tA0A3Q0G4K1_ALLSI\nFYN\tA0A3Q0G5A3_ALLSI\nFYN\tA0A3Q0RSH4_AMPCI\nFYN\tA0A3Q2I8H4_HORSE\nFYN\tA0A3Q2IC74_HORSE\nFYN\tA0A3Q2U639_CHICK\nFYN\tA0A3Q2UIT4_CHICK\nFYN\tA0A3Q2XKR5_HIPCM\nFYN\tA0A3Q3D7W2_HIPCM\nFYN\tA0A3Q7NGQ7_CALUR\nFYN\tA0A3Q7NHJ0_CALUR\nFYN\tA0A3Q7PVR2_CALUR\nFYN\tA0A3Q7TJW7_VULVU\nFYN\tA0A3Q7WQR0_URSAR\nFYN\tA0A3Q7WZP6_URSAR\nFYN\tA0A3Q7XLE0_URSAR\nFYN\tA0A3R7FZ78_CLOSI\nFYN\tA0A452DZ83_CAPHI\nFYN\tA0A452HNV0_9SAUR\nFYN\tA0A452HNW7_9SAUR\nFYN\tA0A452S607_URSAM\nFYN\tA0A452S628_URSAM\nFYN\tA0A452S641_URSAM\nFYN\tA0A452VNL4_URSMA\nFYN\tA0A452VNQ8_URSMA\nFYN\tA0A452VNS6_URSMA\nFYN\tA0A4W2CD24_BOBOX\nFYN\tA0A4W2EJI9_BOBOX\nFYN\tA0A4W2FE09_BOBOX\nFYN\tA0A4W3J1Z8_CALMI\nFYN\tA0A4W3J213_CALMI\nFYN\tA0A4W3J446_CALMI\nFYN\tA0A4W3J4P2_CALMI\nFYN\tA0A4W3J4P7_CALMI\nFYN\tA0A4W3JBR5_CALMI\nFYN\tA0A4W3JBX2_CALMI\nFYN\tA0A4W3JFZ5_CALMI\nFYN\tA0A4W3JG80_CALMI\nFYN\tA0A4X1V5F4_PIG\nFYN\tA0A4X1V8I1_PIG\nFYN\tA0A4X1V8J3_PIG\nFYN\tA0A4X1VB47_PIG\nFYN\tA0A4X2M0H8_VOMUR\nFYN\tA0A4X2M0I3_VOMUR\nFYN\tA0A4X2M6U9_VOMUR\nFYN\tA0A5F4CG29_CANLF\nFYN\tA0A5F4CLL3_CANLF\nFYN\tA0A5F4D8G4_CANLF\nFYN\tA0A5F5PHH1_HORSE\nFYN\tA0A5F5XIN2_FELCA\nFYN\tA0A5F7Z853_MACMU\nFYN\tA0A5F8A1I4_MACMU\nFYN\tA0A5F8ASS9_MACMU\nFYN\tA0A5F8GPV1_MONDO\nFYN\tA0A5F8HAY8_MONDO\nFYN\tA0A5F9CRM1_RABIT\nFYN\tA0A5F9DMQ8_RABIT\nFYN\tA0A5G3KPS3_XENTR\nFYN\tA0A5G3KQR2_XENTR\nFYN\tA0A5K1VSM7_FELCA\nFYN\tA1Y2K2_PIG\nFYN\tA2TNA6_PIG\nFYN\tB6E1W3_PIG\nFYN\tB7ZRP5_XENLA\nFYN\tC0H950_SALSA\nFYN\tD2H640_AILME\nFYN\tD3YVZ0_MOUSE\nFYN\tD3YZ57_MOUSE\nFYN\tD3YZA2_MOUSE\nFYN\tE2RC06_CANLF\nFYN\tE2RC09_CANLF\nFYN\tE5RFM0_HUMAN\nFYN\tE5RFM4_HUMAN\nFYN\tE5RFM6_HUMAN\nFYN\tE5RFS5_HUMAN\nFYN\tE5RGM6_HUMAN\nFYN\tE5RGT0_HUMAN\nFYN\tE5RH71_HUMAN\nFYN\tE5RHF7_HUMAN\nFYN\tE5RHX7_HUMAN\nFYN\tE5RI25_HUMAN\nFYN\tE5RIX5_HUMAN\nFYN\tE5RJX7_HUMAN\nFYN\tE5RK23_HUMAN\nFYN\tF1NQU9_CHICK\nFYN\tF1RZM8_PIG\nFYN\tF6PKK0_MACMU\nFYN\tF6PKK7_MACMU\nFYN\tF6S8L5_XENTR\nFYN\tF6T9G2_HORSE\nFYN\tF7DLX2_ORNAN\nFYN\tF7FG65_MONDO\nFYN\tF7FVL2_MACMU\nFYN\tF7IB43_CALJA\nFYN\tF7IDE6_CALJA\nFYN\tFYNA_DANRE\nFYN\tFYN_BOVIN\nFYN\tFYN_CHICK\nFYN\tFYN_HUMAN\nFYN\tFYN_MOUSE\nFYN\tFYN_PIG\nFYN\tFYN_RAT\nFYN\tFYN_XENLA\nFYN\tFYN_XIPHE\nFYN\tG1KLG4_ANOCA\nFYN\tG1NL23_MELGA\nFYN\tG1P913_MYOLU\nFYN\tG1RR59_NOMLE\nFYN\tG1TA73_RABIT\nFYN\tG3SQT6_LOXAF\nFYN\tG3UPE4_MELGA\nFYN\tG3WAB1_SARHA\nFYN\tG5AUN8_HETGA\nFYN\tG7P3M8_MACFA\nFYN\tH0VMB4_CAVPO\nFYN\tH0WNS4_OTOGA\nFYN\tH0ZP43_TAEGU\nFYN\tH2PK28_PONAB\nFYN\tH2QTK7_PANTR\nFYN\tH3AY24_LATCH\nFYN\tH3CJK4_TETNG\nFYN\tH9FSC3_MACMU\nFYN\tI3M9M0_ICTTR\nFYN\tK7F7Q6_PELSI\nFYN\tK7F7Q8_PELSI\nFYN\tM3X148_FELCA\nFYN\tM3YFP0_MUSPF\nFYN\tP78484_HUMAN\nFYN\tQ14342_HUMAN\nFYN\tQ32N74_XENLA\nFYN\tQ6IZB1_PHOVI\nFYN\tQ90YD8_XENLA\nFYN\tQ90YE9_TORMA\nFYN\tQ95M31_9PRIM\nFYN\tU3BSY6_CALJA\nFYN\tU3J5M4_ANAPP\nFYN\tU3K8R9_FICAL\nFYN\tU6CZS5_NEOVI\nFYN\tV8PFS0_OPHHA\nFYN\tV9KLV0_CALMI\nFYN\tW5PJ18_SHEEP\n','From\tTo\nINSR\tA0A034UZV1_BACDO\nINSR\tA0A034V1Y7_BACDO\nINSR\tA0A034V444_BACDO\nINSR\tA0A034V4N5_BACDO\nINSR\tA0A034VA78_BACDO\nINSR\tA0A034WAX2_BACDO\nINSR\tA0A034WCE7_BACDO\nINSR\tA0A0A0MW58_PAPAN\nINSR\tA0A0A0MW86_PAPAN\nINSR\tA0A0A1XG13_ZEUCU\nINSR\tA0A0A1XM25_ZEUCU\nINSR\tA0A0A9YR44_LYGHE\nINSR\tA0A0D9R902_CHLSB\nINSR\tA0A0K2V906_LEPSM\nINSR\tA0A0S7F1F9_9TELE\nINSR\tA0A0S7F1H6_9TELE\nINSR\tA0A0S7F1H8_9TELE\nINSR\tA0A0S7F1I1_9TELE\nINSR\tA0A0S7F1X0_9TELE\nINSR\tA0A0S7F4P3_9TELE\nINSR\tA0A0S7F6Y4_9TELE\nINSR\tA0A0S7F7U3_9TELE\nINSR\tA0A0S7F7Y6_9TELE\nINSR\tA0A0S7F9A7_9TELE\nINSR\tA0A0S7FA74_9TELE\nINSR\tA0A0S7FA91_9TELE\nINSR\tA0A0S7KG73_9TELE\nINSR\tA0A0S7KGA1_9TELE\nINSR\tA0A0S7KI82_9TELE\nINSR\tA0A0S7KPD5_9TELE\nINSR\tA0A0S7KPE5_9TELE\nINSR\tA0A0S7KPH0_9TELE\nINSR\tA0A0S7KPH3_9TELE\nINSR\tA0A0S7KPH8_9TELE\nINSR\tA0A0S7KPI6_9TELE\nINSR\tA0A0S7KPK6_9TELE\nINSR\tA0A0S7KQD5_9TELE\nINSR\tA0A0S7KQL4_9TELE\nINSR\tA0A0S7KQM3_9TELE\nINSR\tA0A0S7KRG9_9TELE\nINSR\tA0A0V0SDK8_9BILA\nINSR\tA0A0V0U4G9_9BILA\nINSR\tA0A0V0V5M8_9BILA\nINSR\tA0A0V0X2E0_9BILA\nINSR\tA0A0V0XD92_TRIPS\nINSR\tA0A0V1AG08_9BILA\nINSR\tA0A0V1BWA2_TRISP\nINSR\tA0A0V1CZ97_TRIBR\nINSR\tA0A0V1FN02_TRIPS\nINSR\tA0A0V1I0Z9_9BILA\nINSR\tA0A0V1K5E4_TRIPS\nINSR\tA0A0V1MLD9_9BILA\nINSR\tA0A0V1NWL6_9BILA\nINSR\tA0A140LI30_MOUSE\nINSR\tA0A151PE99_ALLMI\nINSR\tA0A1D5P6T3_CHICK\nINSR\tA0A1D5QCC6_MACMU\nINSR\tA0A1S3GLS9_DIPOR\nINSR\tA0A1S3WNI0_ERIEU\nINSR\tA0A1S3WNJ7_ERIEU\nINSR\tA0A1T4JH41_NEMVE\nINSR\tA0A1U7US90_TARSY\nINSR\tA0A1U7UTP0_TARSY\nINSR\tA0A1U8DJE2_ALLSI\nINSR\tA0A1V4KQM4_PATFA\nINSR\tA0A218UAG7_9PASE\nINSR\tA0A286XXU1_CAVPO\nINSR\tA0A287D4G1_ICTTR\nINSR\tA0A2B4SPX4_STYPI\nINSR\tA0A2I0LPD9_COLLI\nINSR\tA0A2I2UE37_FELCA\nINSR\tA0A2I3NG96_PAPAN\nINSR\tA0A2I3RFV3_PANTR\nINSR\tA0A2I3SGW4_PANTR\nINSR\tA0A2I4B558_9TELE\nINSR\tA0A2K5CFX9_AOTNA\nINSR\tA0A2K5CFZ2_AOTNA\nINSR\tA0A2K5K0T8_COLAP\nINSR\tA0A2K5K0V8_COLAP\nINSR\tA0A2K5K0Z5_COLAP\nINSR\tA0A2K5L4E5_CERAT\nINSR\tA0A2K5Q557_CEBCA\nINSR\tA0A2K5Q5C5_CEBCA\nINSR\tA0A2K5WLC0_MACFA\nINSR\tA0A2K5WLC5_MACFA\nINSR\tA0A2K5YPC1_MANLE\nINSR\tA0A2K5YPE3_MANLE\nINSR\tA0A2K6B0Q0_MACNE\nINSR\tA0A2K6GHI0_PROCO\nINSR\tA0A2K6GHI2_PROCO\nINSR\tA0A2K6LLA2_RHIBE\nINSR\tA0A2K6LLA6_RHIBE\nINSR\tA0A2K6LLE0_RHIBE\nINSR\tA0A2K6QQB2_RHIRO\nINSR\tA0A2K6QQB8_RHIRO\nINSR\tA0A2K6QQC1_RHIRO\nINSR\tA0A2K6T1C8_SAIBB\nINSR\tA0A2K6T1F3_SAIBB\nINSR\tA0A2K6T1L4_SAIBB\nINSR\tA0A2R2YUK7_SPAAU\nINSR\tA0A2R8ZZ37_PANPA\nINSR\tA0A2R9A069_PANPA\nINSR\tA0A2R9A3A9_PANPA\nINSR\tA0A2U3WSD9_ODORO\nINSR\tA0A2U3WSM4_ODORO\nINSR\tA0A2Y9IA86_NEOSC\nINSR\tA0A2Y9IGT8_NEOSC\nINSR\tA0A2Y9LSN6_DELLE\nINSR\tA0A340XGU4_LIPVE\nINSR\tA0A340XMY7_LIPVE\nINSR\tA0A341BFV4_NEOAA\nINSR\tA0A341BHG7_NEOAA\nINSR\tA0A341BIP5_NEOAA\nINSR\tA0A341BIV3_NEOAA\nINSR\tA0A383Z1V8_BALAS\nINSR\tA0A384D9G9_URSMA\nINSR\tA0A3B4ZGN4_9TELE\nINSR\tA0A3Q0H234_ALLSI\nINSR\tA0A3Q0H314_ALLSI\nINSR\tA0A3Q1LFL2_BOVIN\nINSR\tA0A3Q1LMR3_BOVIN\nINSR\tA0A3Q1LT37_BOVIN\nINSR\tA0A3Q1LTR4_BOVIN\nINSR\tA0A3Q1LXB5_BOVIN\nINSR\tA0A3Q1MJC8_BOVIN\nINSR\tA0A3Q1MJG0_BOVIN\nINSR\tA0A3Q1MMU0_BOVIN\nINSR\tA0A3Q1MPC4_BOVIN\nINSR\tA0A3Q7NBS9_CALUR\nINSR\tA0A3Q7NLC0_CALUR\nINSR\tA0A3Q7R5Q5_VULVU\nINSR\tA0A3Q7TLH5_URSAR\nINSR\tA0A3Q7UA15_URSAR\nINSR\tA0A3Q7ULR7_URSAR\nINSR\tA0A3Q7V2J0_URSAR\nINSR\tA0A3Q7VGR2_URSAR\nINSR\tA0A452EK73_CAPHI\nINSR\tA0A452EKQ6_CAPHI\nINSR\tA0A452IPX3_9SAUR\nINSR\tA0A452RT72_URSAM\nINSR\tA0A452RT74_URSAM\nINSR\tA0A452RT86_URSAM\nINSR\tA0A452RTB6_URSAM\nINSR\tA0A452UD86_URSMA\nINSR\tA0A452UE32_URSMA\nINSR\tA0A452UE39_URSMA\nINSR\tA0A452UEE0_URSMA\nINSR\tA0A452UEF8_URSMA\nINSR\tA0A452UEM0_URSMA\nINSR\tA0A452UEP0_URSMA\nINSR\tA0A452UER5_URSMA\nINSR\tA0A452UET4_URSMA\nINSR\tA0A452UEV8_URSMA\nINSR\tA0A455C2V3_PHYMC\nINSR\tA0A455C341_PHYMC\nINSR\tA0A455C3Y8_PHYMC\nINSR\tA0A493TF07_ANAPP\nINSR\tA0A4W2HBI4_BOBOX\nINSR\tA0A4W2IP97_BOBOX\nINSR\tA0A4W3GX65_CALMI\nINSR\tA0A4W3GXA7_CALMI\nINSR\tA0A4W3GZ00_CALMI\nINSR\tA0A4W3GZ16_CALMI\nINSR\tA0A4W3GZH8_CALMI\nINSR\tA0A4W3H0T6_CALMI\nINSR\tA0A4W3H0W1_CALMI\nINSR\tA0A4W3H153_CALMI\nINSR\tA0A4W3HGI8_CALMI\nINSR\tA0A4W3HGM8_CALMI\nINSR\tA0A4X1VQM1_PIG\nINSR\tA0A4X1VRT4_PIG\nINSR\tA0A4X2JP28_VOMUR\nINSR\tA0A4X2JTG6_VOMUR\nINSR\tA0A4X2JTI9_VOMUR\nINSR\tA0A4Y2NCJ2_ARAVE\nINSR\tA0A5F4D5H9_CANLF\nINSR\tA0A5F5Y2G1_FELCA\nINSR\tA0A5F8GJ01_MONDO\nINSR\tA0A5F9DAH8_RABIT\nINSR\tA0A5G3I4J7_XENTR\nINSR\tA0A5G3I4K3_XENTR\nINSR\tA0A5G3I4L4_XENTR\nINSR\tA0A5G3JZQ6_XENTR\nINSR\tA0A5G3KX78_XENTR\nINSR\tA0A5G3KY87_XENTR\nINSR\tA0A5G3KYE2_XENTR\nINSR\tA0A5G3KYZ9_XENTR\nINSR\tA0A5G3KZ68_XENTR\nINSR\tA0A5G3KZ87_XENTR\nINSR\tA0A5G3KZB2_XENTR\nINSR\tA0A5G3L0F7_XENTR\nINSR\tB4F451_PIG\nINSR\tB8Q3N4_MOUSE\nINSR\tB8YBH0_SOLSE\nINSR\tE2RPM8_CANLF\nINSR\tF1LPL6_RAT\nINSR\tF1MV64_BOVIN\nINSR\tF1SCK8_PIG\nINSR\tF1SCL0_PIG\nINSR\tF6R9D2_HORSE\nINSR\tF6RLJ3_HORSE\nINSR\tF6S8L3_MACMU\nINSR\tF6TEA3_XENTR\nINSR\tF6U5U9_XENTR\nINSR\tF6UY01_XENTR\nINSR\tF7F370_MONDO\nINSR\tF7HRK0_CALJA\nINSR\tG1LNQ5_AILME\nINSR\tG1N117_MELGA\nINSR\tG1PNP3_MYOLU\nINSR\tG1QHT7_NOMLE\nINSR\tG1QHU4_NOMLE\nINSR\tG1U0V9_RABIT\nINSR\tG3T361_LOXAF\nINSR\tG3UEM7_LOXAF\nINSR\tH0VMQ0_CAVPO\nINSR\tH0WLU7_OTOGA\nINSR\tH0YQ96_TAEGU\nINSR\tH2NX99_PONAB\nINSR\tH2QF61_PANTR\nINSR\tH2T2E6_TAKRU\nINSR\tH3ANU2_LATCH\nINSR\tH9F9P3_MACMU\nINSR\tI3LXY7_ICTTR\nINSR\tINSR_HUMAN\nINSR\tINSR_MACMU\nINSR\tINSR_MOUSE\nINSR\tINSR_RAT\nINSR\tINSR_XENLA\nINSR\tJ9NY09_CANLF\nINSR\tK7CDN0_PANTR\nINSR\tK7CJP0_PANTR\nINSR\tK7F278_PELSI\nINSR\tK7F279_PELSI\nINSR\tM0R3E6_HUMAN\nINSR\tM3VZL0_FELCA\nINSR\tM3YAM5_MUSPF\nINSR\tM4TKR7_MOUSE\nINSR\tO35466_MOUSE\nINSR\tQ2F2X9_PAPAN\nINSR\tQ3TPM5_MOUSE\nINSR\tQ3V2X8_MOUSE\nINSR\tQ3V313_MOUSE\nINSR\tQ86WY9_HUMAN\nINSR\tQ9YI42_CHICK\nINSR\tS4RRI3_PETMA\nINSR\tT2CB11_RAT\nINSR\tT2MGU8_HYDVU\nINSR\tU3CV09_CALJA\nINSR\tU3EYD1_CALJA\nINSR\tU3FCC1_CALJA\nINSR\tU3FWA3_CALJA\nINSR\tU3J3T4_ANAPP\nINSR\tU3KHF2_FICAL\nINSR\tU6D221_NEOVI\nINSR\tV5GGW7_ANOGL\nINSR\tV5GZ99_ANOGL\nINSR\tV5GZA3_ANOGL\nINSR\tV5IA51_ANOGL\nINSR\tV8PFA9_OPHHA\nINSR\tW5NQ38_SHEEP\nINSR\tW5NQ40_SHEEP\nINSR\tW8APQ1_CERCA\nINSR\tW8BST3_CERCA\nINSR\tW8C4F7_CERCA\n','From\tTo\nIGF1R\tA0A034VUF4_BACDO\nIGF1R\tA0A076FVI5_AMBHO\nIGF1R\tA0A076FVJ0_DESRO\nIGF1R\tA0A076FVK3_TACSP\nIGF1R\tA0A076FVY1_BATSU\nIGF1R\tA0A076FWU2_9HYST\nIGF1R\tA0A076FWU7_LOPEV\nIGF1R\tA0A076G1H5_FUKDA\nIGF1R\tA0A090XCE8_9PETR\nIGF1R\tA0A096NNC9_PAPAN\nIGF1R\tA0A0B2V157_TOXCA\nIGF1R\tA0A0D9RWL0_CHLSB\nIGF1R\tA0A0S7GCX5_9TELE\nIGF1R\tA0A0S7GCY0_9TELE\nIGF1R\tA0A0S7GCY1_9TELE\nIGF1R\tA0A0S7GCY6_9TELE\nIGF1R\tA0A0S7GCY9_9TELE\nIGF1R\tA0A0S7GCZ5_9TELE\nIGF1R\tA0A0S7GE45_9TELE\nIGF1R\tA0A0S7GFH0_9TELE\nIGF1R\tA0A0S7GFT1_9TELE\nIGF1R\tA0A0S7GGQ2_9TELE\nIGF1R\tA0A0S7GHH8_9TELE\nIGF1R\tA0A0S7GI23_9TELE\nIGF1R\tA0A0S7GIU8_9TELE\nIGF1R\tA0A0S7GJC7_9TELE\nIGF1R\tA0A0S7GKQ0_9TELE\nIGF1R\tA0A0S7HJA7_9TELE\nIGF1R\tA0A0S7HLL1_9TELE\nIGF1R\tA0A0S7JXB2_9TELE\nIGF1R\tA0A0S7JXD2_9TELE\nIGF1R\tA0A0S7JXF9_9TELE\nIGF1R\tA0A0S7JXH8_9TELE\nIGF1R\tA0A0S7JXH9_9TELE\nIGF1R\tA0A0S7JYE4_9TELE\nIGF1R\tA0A0S7JYN4_9TELE\nIGF1R\tA0A0S7JYU5_9TELE\nIGF1R\tA0A0S7KKK6_9TELE\nIGF1R\tA0A0S7KKM1_9TELE\nIGF1R\tA0A0S7KKN4_9TELE\nIGF1R\tA0A0S7KKP2_9TELE\nIGF1R\tA0A0S7KLK2_9TELE\nIGF1R\tA0A0S7MAN7_9TELE\nIGF1R\tA0A0U2XIF1_TEGGR\nIGF1R\tA0A0V0RGB9_9BILA\nIGF1R\tA0A0V0TE73_9BILA\nIGF1R\tA0A0V0TE79_9BILA\nIGF1R\tA0A0V0W514_9BILA\nIGF1R\tA0A0V0W566_9BILA\nIGF1R\tA0A0V0W5C0_9BILA\nIGF1R\tA0A0V0YAD2_TRIPS\nIGF1R\tA0A0V0ZTA8_9BILA\nIGF1R\tA0A0V0ZTN0_9BILA\nIGF1R\tA0A0V1AYY0_TRISP\nIGF1R\tA0A0V1AZ68_TRISP\nIGF1R\tA0A0V1CB08_TRIBR\nIGF1R\tA0A0V1CB90_TRIBR\nIGF1R\tA0A0V1DU27_TRIPS\nIGF1R\tA0A0V1DUD4_TRIPS\nIGF1R\tA0A0V1DUU6_TRIPS\nIGF1R\tA0A0V1F6H6_TRIPS\nIGF1R\tA0A0V1F7E4_TRIPS\nIGF1R\tA0A0V1F7F1_TRIPS\nIGF1R\tA0A0V1F8S7_TRIPS\nIGF1R\tA0A0V1GVM4_9BILA\nIGF1R\tA0A0V1GWB3_9BILA\nIGF1R\tA0A0V1HDU8_TRIPS\nIGF1R\tA0A0V1JJS0_TRIPS\nIGF1R\tA0A0V1JJT6_TRIPS\nIGF1R\tA0A0V1JJU5_TRIPS\nIGF1R\tA0A0V1JJW2_TRIPS\nIGF1R\tA0A0V1JJX5_TRIPS\nIGF1R\tA0A0V1JJZ0_TRIPS\nIGF1R\tA0A0V1JK72_TRIPS\nIGF1R\tA0A0V1KQ38_9BILA\nIGF1R\tA0A0V1KR95_9BILA\nIGF1R\tA0A0V1MKZ4_9BILA\nIGF1R\tA0A0V1NGY9_9BILA\nIGF1R\tA0A0V1NHI5_9BILA\nIGF1R\tA0A0X3NF87_SCHSO\nIGF1R\tA0A0X3NPX8_SCHSO\nIGF1R\tA0A0X3NU32_SCHSO\nIGF1R\tA0A0X3PFM2_SCHSO\nIGF1R\tA0A140TAB8_RAT\nIGF1R\tA0A1D5PVH7_CHICK\nIGF1R\tA0A1L8H104_XENLA\nIGF1R\tA0A1S2ZG47_ERIEU\nIGF1R\tA0A1S3EU99_DIPOR\nIGF1R\tA0A1S3KLV9_SALSA\nIGF1R\tA0A1S3KMN0_SALSA\nIGF1R\tA0A1U7RT92_ALLSI\nIGF1R\tA0A1U7TIH1_TARSY\nIGF1R\tA0A1U8CU03_MESAU\nIGF1R\tA0A1V4JDK1_PATFA\nIGF1R\tA0A1V4JDQ0_PATFA\nIGF1R\tA0A218V095_9PASE\nIGF1R\tA0A250Y7G3_CASCN\nIGF1R\tA0A288MWC0_TACFU\nIGF1R\tA0A2B4S233_STYPI\nIGF1R\tA0A2B4T087_STYPI\nIGF1R\tA0A2D0QU59_ICTPU\nIGF1R\tA0A2I0MD56_COLLI\nIGF1R\tA0A2I3GE04_NOMLE\nIGF1R\tA0A2I3HLU9_NOMLE\nIGF1R\tA0A2I3MLM3_PAPAN\nIGF1R\tA0A2I3MVA9_PAPAN\nIGF1R\tA0A2I3RAZ9_PANTR\nIGF1R\tA0A2I3SK97_PANTR\nIGF1R\tA0A2I4CQR9_9TELE\nIGF1R\tA0A2K5F1M0_AOTNA\nIGF1R\tA0A2K5F1R3_AOTNA\nIGF1R\tA0A2K5I8S9_COLAP\nIGF1R\tA0A2K5I8T0_COLAP\nIGF1R\tA0A2K5I8Z6_COLAP\nIGF1R\tA0A2K5L870_CERAT\nIGF1R\tA0A2K5L878_CERAT\nIGF1R\tA0A2K5L879_CERAT\nIGF1R\tA0A2K5RBA0_CEBCA\nIGF1R\tA0A2K5RBB9_CEBCA\nIGF1R\tA0A2K5W7A4_MACFA\nIGF1R\tA0A2K5W7B1_MACFA\nIGF1R\tA0A2K5Z8L3_MANLE\nIGF1R\tA0A2K5Z8V6_MANLE\nIGF1R\tA0A2K6D2L3_MACNE\nIGF1R\tA0A2K6D2M7_MACNE\nIGF1R\tA0A2K6D2N5_MACNE\nIGF1R\tA0A2K6G6K2_PROCO\nIGF1R\tA0A2K6G6M0_PROCO\nIGF1R\tA0A2K6N161_RHIBE\nIGF1R\tA0A2K6N168_RHIBE\nIGF1R\tA0A2K6PRD9_RHIRO\nIGF1R\tA0A2K6PRE4_RHIRO\nIGF1R\tA0A2K6PRE6_RHIRO\nIGF1R\tA0A2K6PRE8_RHIRO\nIGF1R\tA0A2K6SSM8_SAIBB\nIGF1R\tA0A2K6SSN0_SAIBB\nIGF1R\tA0A2R2YUM1_SPAAU\nIGF1R\tA0A2R8ZWC9_PANPA\nIGF1R\tA0A2R8ZWD4_PANPA\nIGF1R\tA0A2R8ZYJ5_PANPA\nIGF1R\tA0A2U3Y3Q9_LEPWE\nIGF1R\tA0A2U3ZLZ8_ODORO\nIGF1R\tA0A2U4C6V9_TURTR\nIGF1R\tA0A2U4C6W9_TURTR\nIGF1R\tA0A2U4C6X6_TURTR\nIGF1R\tA0A2U4C6X7_TURTR\nIGF1R\tA0A2U4C762_TURTR\nIGF1R\tA0A2Y9GBI4_NEOSC\nIGF1R\tA0A2Y9Q6B1_DELLE\nIGF1R\tA0A2Y9QAK7_DELLE\nIGF1R\tA0A2Y9QGQ9_DELLE\nIGF1R\tA0A2Y9QH28_DELLE\nIGF1R\tA0A2Y9TH39_PHYMC\nIGF1R\tA0A2Y9TK79_PHYMC\nIGF1R\tA0A340Y906_LIPVE\nIGF1R\tA0A341AAW7_NEOAA\nIGF1R\tA0A341AD38_NEOAA\nIGF1R\tA0A341AE09_NEOAA\nIGF1R\tA0A3Q0GQJ9_ALLSI\nIGF1R\tA0A3Q0GV29_ALLSI\nIGF1R\tA0A3Q1NB49_BOVIN\nIGF1R\tA0A3Q7QSI8_CALUR\nIGF1R\tA0A3Q7R2K1_CALUR\nIGF1R\tA0A3Q7SLN3_VULVU\nIGF1R\tA0A3Q7XIH4_URSAR\nIGF1R\tA0A3Q8BQ57_TURTR\nIGF1R\tA0A3Q8BR49_ESCRO\nIGF1R\tA0A3Q8BUQ5_PHYMC\nIGF1R\tA0A3Q8BX23_GLOMA\nIGF1R\tA0A3Q8C2J8_SOUCH\nIGF1R\tA0A3Q8CJ88_STEBR\nIGF1R\tA0A3Q8CKP6_FERAT\nIGF1R\tA0A3R7C2Z2_CLOSI\nIGF1R\tA0A3S5WL90_STEAT\nIGF1R\tA0A3S6P8G8_BALAC\nIGF1R\tA0A3S6P9Z6_MESDE\nIGF1R\tA0A3S6PAG7_TURAD\nIGF1R\tA0A3S6PAW2_NEOPH\nIGF1R\tA0A410SF50_NIBAL\nIGF1R\tA0A419PT21_CLOSI\nIGF1R\tA0A419Q5L8_CLOSI\nIGF1R\tA0A452CNG4_BALAS\nIGF1R\tA0A452FWJ9_CAPHI\nIGF1R\tA0A452J755_9SAUR\nIGF1R\tA0A452J765_9SAUR\nIGF1R\tA0A452J770_9SAUR\nIGF1R\tA0A452J774_9SAUR\nIGF1R\tA0A452J778_9SAUR\nIGF1R\tA0A452J789_9SAUR\nIGF1R\tA0A452R8K3_URSAM\nIGF1R\tA0A452R8P7_URSAM\nIGF1R\tA0A452R952_URSAM\nIGF1R\tA0A452R959_URSAM\nIGF1R\tA0A452R966_URSAM\nIGF1R\tA0A452R9B5_URSAM\nIGF1R\tA0A452R9G8_URSAM\nIGF1R\tA0A452R9J5_URSAM\nIGF1R\tA0A452VN05_URSMA\nIGF1R\tA0A452VN41_URSMA\nIGF1R\tA0A452VN68_URSMA\nIGF1R\tA0A452VN73_URSMA\nIGF1R\tA0A452VN78_URSMA\nIGF1R\tA0A452VNC0_URSMA\nIGF1R\tA0A452VND3_URSMA\nIGF1R\tA0A452VNE7_URSMA\nIGF1R\tA0A452VNF3_URSMA\nIGF1R\tA0A452VNF6_URSMA\nIGF1R\tA0A455BPW4_PHYMC\nIGF1R\tA0A455BPY1_PHYMC\nIGF1R\tA0A455BQL4_PHYMC\nIGF1R\tA0A455BT75_PHYMC\nIGF1R\tA0A4C1U2V6_9NEOP\nIGF1R\tA0A4W2IC47_BOBOX\nIGF1R\tA0A4W3HKM3_CALMI\nIGF1R\tA0A4W3HKQ2_CALMI\nIGF1R\tA0A4W3HNJ7_CALMI\nIGF1R\tA0A4W3HNK7_CALMI\nIGF1R\tA0A4W3HQS1_CALMI\nIGF1R\tA0A4W3HRH8_CALMI\nIGF1R\tA0A4W3HRL3_CALMI\nIGF1R\tA0A4W3HRL8_CALMI\nIGF1R\tA0A4W3I9H6_CALMI\nIGF1R\tA0A4X1UKG1_PIG\nIGF1R\tA0A4X1UL29_PIG\nIGF1R\tA0A4X2KQ38_VOMUR\nIGF1R\tA0A4Y6ETQ0_SHEEP\nIGF1R\tA0A4Y6JMB0_PAROL\nIGF1R\tA0A5F4BS76_CANLF\nIGF1R\tA0A5F4CEM9_CANLF\nIGF1R\tA0A5F8GU87_MONDO\nIGF1R\tA0A5F8H0Z6_MONDO\nIGF1R\tA0A5F8H861_MONDO\nIGF1R\tA0A5F8HGW3_MONDO\nIGF1R\tA0A5G3H9N0_XENTR\nIGF1R\tA0A5G3HAC2_XENTR\nIGF1R\tA0A5G3IUX8_XENTR\nIGF1R\tA0A5G3IX32_XENTR\nIGF1R\tA0A5G3IXF9_XENTR\nIGF1R\tA0A5G3IXI0_XENTR\nIGF1R\tA0A5G3IY13_XENTR\nIGF1R\tA0JGJ5_PIG\nIGF1R\tA6P6W2_COTJA\nIGF1R\tA6P6W3_COTJA\nIGF1R\tA7L5D1_SHEEP\nIGF1R\tB0Z830_GOBNI\nIGF1R\tB6UM06_BUBBU\nIGF1R\tB8YBH1_SOLSE\nIGF1R\tC9EEZ1_BOVIN\nIGF1R\tC9J5X1_HUMAN\nIGF1R\tD7PCU2_TRIVU\nIGF1R\tE0X640_AILME\nIGF1R\tE0X734_FELCA\nIGF1R\tE2J5T6_BOVIN\nIGF1R\tE9QNX9_MOUSE\nIGF1R\tF1CK17_PIG\nIGF1R\tF1PXU6_CANLF\nIGF1R\tF1SRY1_PIG\nIGF1R\tF6QWD3_XENTR\nIGF1R\tF6SJA3_MONDO\nIGF1R\tF6SN93_XENTR\nIGF1R\tF6YQU0_HORSE\nIGF1R\tG1K218_BOVIN\nIGF1R\tG1LGG8_AILME\nIGF1R\tG1NAI9_MELGA\nIGF1R\tG1NS76_MELGA\nIGF1R\tG1PWB2_MYOLU\nIGF1R\tG1QJ59_NOMLE\nIGF1R\tG1T855_RABIT\nIGF1R\tG1TLR2_RABIT\nIGF1R\tG3T7X6_LOXAF\nIGF1R\tG3UCL7_LOXAF\nIGF1R\tG3UT20_MELGA\nIGF1R\tG5DBJ3_BUBBU\nIGF1R\tG7P9I7_MACFA\nIGF1R\tG8GZ67_BOVIN\nIGF1R\tH0V5D0_CAVPO\nIGF1R\tH0X6D6_OTOGA\nIGF1R\tH0YMJ5_HUMAN\nIGF1R\tH0YNR0_HUMAN\nIGF1R\tH0ZEE4_TAEGU\nIGF1R\tH2NP98_PONAB\nIGF1R\tH2QA49_PANTR\nIGF1R\tH3AEG1_LATCH\nIGF1R\tH3BSZ8_HUMAN\nIGF1R\tH9F026_MACMU\nIGF1R\tH9GDM3_ANOCA\nIGF1R\tI1SWI7_BOVIN\nIGF1R\tI2CWY3_MACMU\nIGF1R\tI3MHA3_ICTTR\nIGF1R\tI3QHK5_BOVIN\nIGF1R\tI3WTW9_BOVIN\nIGF1R\tI6UW03_BOVIN\nIGF1R\tI6ZLE8_BOVIN\nIGF1R\tIGF1R_BOVIN\nIGF1R\tIGF1R_HUMAN\nIGF1R\tIGF1R_MOUSE\nIGF1R\tIGF1R_PIG\nIGF1R\tIGF1R_RAT\nIGF1R\tIGF1R_XENLA\nIGF1R\tJ7G3Y2_CERNI\nIGF1R\tK7G9J6_PELSI\nIGF1R\tM1Q751_BUBBU\nIGF1R\tM1RB53_BUBBU\nIGF1R\tM1RJ33_BUBBU\nIGF1R\tM3WPQ6_FELCA\nIGF1R\tM3YH59_MUSPF\nIGF1R\tM9WIE3_CHESE\nIGF1R\tO18855_SHEEP\nIGF1R\tQ1H5I9_NOTKU\nIGF1R\tQ1H5J0_NOTFU\nIGF1R\tQ3B833_HUMAN\nIGF1R\tQ3U1L4_MOUSE\nIGF1R\tQ3UVJ3_MOUSE\nIGF1R\tQ9XS87_HORSE\nIGF1R\tR4G9M3_ANOCA\nIGF1R\tU3DD61_CALJA\nIGF1R\tU3IM06_ANAPP\nIGF1R\tU3K6Y5_FICAL\nIGF1R\tU6D754_NEOVI\nIGF1R\tV5H1R5_ANOGL\nIGF1R\tV8NCE8_OPHHA\nIGF1R\tV8NS93_OPHHA\nIGF1R\tW0ZDW0_ORYME\nIGF1R\tW5PJW7_SHEEP\nIGF1R\tW5UAB0_ICTPU\nIGF1R\tW5UHH9_ICTPU\nIGF1R\tW5XTU7_CANLF\nIGF1R\tW8ASB6_CERCA\nIGF1R\tW8B6W5_CERCA\nIGF1R\tX4Y8Z9_SHEEP\n','From\tTo\nALK\tA0A034VMN7_BACDO\nALK\tA0A087WZL3_HUMAN\nALK\tA0A096N155_PAPAN\nALK\tA0A0B2W3D6_TOXCA\nALK\tA0A0C9QSM7_9HYME\nALK\tA0A0D9REH0_CHLSB\nALK\tA0A0K2YUJ3_HUMAN\nALK\tA0A0R4IRF5_DANRE\nALK\tA0A0S7ESM0_9TELE\nALK\tA0A0S7HE78_9TELE\nALK\tA0A0S7HEF4_9TELE\nALK\tA0A0S7HEM2_9TELE\nALK\tA0A0S7IJZ6_9TELE\nALK\tA0A0S7IKI8_9TELE\nALK\tA0A0S7IWM0_9TELE\nALK\tA0A0S7M167_9TELE\nALK\tA0A0S7MID0_9TELE\nALK\tA0A0V0U2C7_9BILA\nALK\tA0A0V0WJD9_9BILA\nALK\tA0A0V1FM89_TRIPS\nALK\tA0A0V1J0R3_TRIPS\nALK\tA0A0V1L8K5_9BILA\nALK\tA0A0V1MZ34_9BILA\nALK\tA0A0V1PM82_9BILA\nALK\tA0A0V1PMA2_9BILA\nALK\tA0A1A7WCN7_9TELE\nALK\tA0A1A7Z945_NOTFU\nALK\tA0A1A8A2P4_NOTFU\nALK\tA0A1A8BFZ6_9TELE\nALK\tA0A1A8C4K2_9TELE\nALK\tA0A1A8DK92_9TELE\nALK\tA0A1A8FRU6_9TELE\nALK\tA0A1A8G153_9TELE\nALK\tA0A1A8GC13_9TELE\nALK\tA0A1A8GVF1_9TELE\nALK\tA0A1A8HP24_NOTKU\nALK\tA0A1A8IDN6_NOTKU\nALK\tA0A1A8IDX2_NOTKU\nALK\tA0A1A8KMZ5_NOTKU\nALK\tA0A1A8LE84_9TELE\nALK\tA0A1A8LK67_9TELE\nALK\tA0A1A8NVM3_9TELE\nALK\tA0A1A8PC45_9TELE\nALK\tA0A1A8R2K9_9TELE\nALK\tA0A1A8RQB1_9TELE\nALK\tA0A1A8SID1_9TELE\nALK\tA0A1A8U9F5_NOTFU\nALK\tA0A1S2ZSP7_ERIEU\nALK\tA0A1S3EYJ7_DIPOR\nALK\tA0A1U7TKB8_TARSY\nALK\tA0A218UKW0_9PASE\nALK\tA0A218VAP7_9PASE\nALK\tA0A287DC37_ICTTR\nALK\tA0A2B4R9B1_STYPI\nALK\tA0A2B4RNY3_STYPI\nALK\tA0A2H8TFV9_9HEMI\nALK\tA0A2I0M4Y2_COLLI\nALK\tA0A2I3LJF7_PAPAN\nALK\tA0A2I3T219_PANTR\nALK\tA0A2I4BC03_9TELE\nALK\tA0A2K5EPL9_AOTNA\nALK\tA0A2K5HQS9_COLAP\nALK\tA0A2K5NFW7_CERAT\nALK\tA0A2K5NGG3_CERAT\nALK\tA0A2K5RSM5_CEBCA\nALK\tA0A2K5RSP7_CEBCA\nALK\tA0A2K5V4R3_MACFA\nALK\tA0A2K5V4W8_MACFA\nALK\tA0A2K6AA88_MANLE\nALK\tA0A2K6AAE1_MANLE\nALK\tA0A2K6DPF9_MACNE\nALK\tA0A2K6DPI0_MACNE\nALK\tA0A2K6EVY1_PROCO\nALK\tA0A2K6LKN5_RHIBE\nALK\tA0A2K6LKQ2_RHIBE\nALK\tA0A2K6NQW5_RHIRO\nALK\tA0A2K6SJW7_SAIBB\nALK\tA0A2K6SJY3_SAIBB\nALK\tA0A2R9CJK4_PANPA\nALK\tA0A2R9CM87_PANPA\nALK\tA0A2S2NLR1_SCHGA\nALK\tA0A2S2NY95_SCHGA\nALK\tA0A2U3VJD5_ODORO\nALK\tA0A2U4BXV0_TURTR\nALK\tA0A2Y9HNX1_NEOSC\nALK\tA0A337SD59_FELCA\nALK\tA0A340YD63_LIPVE\nALK\tA0A341A9G7_NEOAA\nALK\tA0A384AW41_BALAS\nALK\tA0A384CVH5_URSMA\nALK\tA0A3Q0CX95_MESAU\nALK\tA0A3Q0HII9_ALLSI\nALK\tA0A3Q0HMQ6_ALLSI\nALK\tA0A3Q1LTX0_BOVIN\nALK\tA0A3Q1LXZ1_BOVIN\nALK\tA0A3Q1M1I2_BOVIN\nALK\tA0A3Q1M2C1_BOVIN\nALK\tA0A3Q1M7V9_BOVIN\nALK\tA0A3Q1M943_BOVIN\nALK\tA0A3Q1MKW9_BOVIN\nALK\tA0A3Q1NGY3_BOVIN\nALK\tA0A3Q2IAL4_HORSE\nALK\tA0A3Q2U7B1_CHICK\nALK\tA0A3Q7NQ62_CALUR\nALK\tA0A3Q7UF19_VULVU\nALK\tA0A3Q7XCL6_URSAR\nALK\tA0A452DWX1_CAPHI\nALK\tA0A452DX63_CAPHI\nALK\tA0A452DX64_CAPHI\nALK\tA0A452IU41_9SAUR\nALK\tA0A452RKJ9_URSAM\nALK\tA0A452RKP7_URSAM\nALK\tA0A455BTT6_PHYMC\nALK\tA0A499UTR9_CANLF\nALK\tA0A4C1XH11_9NEOP\nALK\tA0A4W2ENF6_BOBOX\nALK\tA0A4W2H7S6_BOBOX\nALK\tA0A4X1TGY5_PIG\nALK\tA0A4X1TIJ8_PIG\nALK\tA0A4X1TIQ4_PIG\nALK\tA0A4Y2AMM7_ARAVE\nALK\tA0A5F9CKC3_RABIT\nALK\tA0A5F9D7R8_RABIT\nALK\tA0A5F9DI02_RABIT\nALK\tA0A5G3HS25_XENTR\nALK\tA0A5G3HSC0_XENTR\nALK\tA0A5G3JGW8_XENTR\nALK\tA0A5G3JJD7_XENTR\nALK\tA0A5K1UQR0_XENTR\nALK\tA1BVT1_DROAI\nALK\tA1BVT2_DROPS\nALK\tA1BVT3_DROMI\nALK\tA1BVV1_DROMI\nALK\tA7XY59_9BACT\nALK\tALK_HUMAN\nALK\tALK_MOUSE\nALK\tB2MXD7_HUMAN\nALK\tB2MXD8_HUMAN\nALK\tB2MXD9_HUMAN\nALK\tB2MXE0_HUMAN\nALK\tB2MXE1_HUMAN\nALK\tB2MXE2_HUMAN\nALK\tB2MXE3_HUMAN\nALK\tB2MXE4_HUMAN\nALK\tB2MXE5_HUMAN\nALK\tB2MXE6_HUMAN\nALK\tB2MXE7_HUMAN\nALK\tB6D4Y1_HUMAN\nALK\tB6D4Y2_HUMAN\nALK\tB6D4Y3_HUMAN\nALK\tB6D4Y5_HUMAN\nALK\tB6D4Y6_HUMAN\nALK\tB6D4Y7_HUMAN\nALK\tB6D4Y8_HUMAN\nALK\tB6ID84_9ZZZZ\nALK\tB6ID85_9ZZZZ\nALK\tB6ID86_9ZZZZ\nALK\tB6ID87_9ZZZZ\nALK\tB7U6M0_PSEAI\nALK\tB7U6M1_PSEAI\nALK\tD1MAM0_HUMAN\nALK\tD1MAM1_HUMAN\nALK\tD1MAM2_HUMAN\nALK\tD1MAM3_HUMAN\nALK\tD1MAM4_HUMAN\nALK\tD1MAM5_HUMAN\nALK\tE7EPW7_HUMAN\nALK\tF0UY65_HUMAN\nALK\tF0UY66_HUMAN\nALK\tF1LRZ0_RAT\nALK\tF1MQH1_BOVIN\nALK\tF1MUQ3_BOVIN\nALK\tF1NHS5_CHICK\nALK\tF1PZ19_CANLF\nALK\tF6PVR5_MACMU\nALK\tF6RKQ0_XENTR\nALK\tF6WYS9_XENTR\nALK\tF6X759_ORNAN\nALK\tF7B8H3_CALJA\nALK\tF7BR79_XENTR\nALK\tF7DDR7_HORSE\nALK\tF8W3R9_DANRE\nALK\tG1KIG5_ANOCA\nALK\tG1LFC2_AILME\nALK\tG1N1A3_MELGA\nALK\tG1PGV5_MYOLU\nALK\tG1RZS6_NOMLE\nALK\tG1SEQ3_RABIT\nALK\tG3QP17_GORGO\nALK\tG3TDJ2_LOXAF\nALK\tG3U8H0_LOXAF\nALK\tH0VBV9_CAVPO\nALK\tH0XB22_OTOGA\nALK\tH0Z4U7_TAEGU\nALK\tH2P6K9_PONAB\nALK\tH2QHP7_PANTR\nALK\tH3B463_LATCH\nALK\tH7BZ33_HUMAN\nALK\tH9FGC2_MACMU\nALK\tH9FHS5_MACMU\nALK\tH9FKG3_MACMU\nALK\tH9NJ48_MYCNE\nALK\tI3M160_ICTTR\nALK\tI7AVB4_PSEAI\nALK\tK7FUL4_PELSI\nALK\tK7FUN0_PELSI\nALK\tM3Z2Q8_MUSPF\nALK\tQ580I3_HUMAN\nALK\tQ7KJ08_DROME\nALK\tQ8CJH1_RAT\nALK\tQ8SWV4_DROME\nALK\tS6CWR2_COTCN\nALK\tSSY23_ORYSI\nALK\tSSY23_ORYSJ\nALK\tU3J006_ANAPP\nALK\tU3K9V0_FICAL\nALK\tU6D1R7_NEOVI\nALK\tV5I6P3_ANOGL\nALK\tV5I7H3_ANOGL\nALK\tV8NHP0_OPHHA\nALK\tV8NLF6_OPHHA\nALK\tV8NQW6_OPHHA\nALK\tV8P5B3_OPHHA\nALK\tV8P5C3_OPHHA\nALK\tW5QEC2_SHEEP\nALK\tW5UKX8_ICTPU\nALK\tW8AZW3_CERCA\nALK\tW8BQH5_CERCA\n','From\tTo\nCSK\tA0A034WIY7_BACDO\nCSK\tA0A034WKI0_BACDO\nCSK\tA0A061QM73_9CHLO\nCSK\tA0A061QZT9_9CHLO\nCSK\tA0A061R3F0_9CHLO\nCSK\tA0A061RFI0_9CHLO\nCSK\tA0A061RS86_9CHLO\nCSK\tA0A061RTK7_9CHLO\nCSK\tA0A061SM65_9CHLO\nCSK\tA0A088AI81_APIME\nCSK\tA0A096NMR6_PAPAN\nCSK\tA0A0A1XSB3_ZEUCU\nCSK\tA0A0B2V031_TOXCA\nCSK\tA0A0B2W2H4_TOXCA\nCSK\tA0A0D9RP44_CHLSB\nCSK\tA0A0R4IB02_DANRE\nCSK\tA0A0S7FJE6_9TELE\nCSK\tA0A0S7FMB5_9TELE\nCSK\tA0A0S7FNY7_9TELE\nCSK\tA0A0S7FQE8_9TELE\nCSK\tA0A0S7GPR3_9TELE\nCSK\tA0A0S7L042_9TELE\nCSK\tA0A0V0RNN4_9BILA\nCSK\tA0A0V0RNP8_9BILA\nCSK\tA0A0V0RNQ0_9BILA\nCSK\tA0A0V0RP60_9BILA\nCSK\tA0A0V0UGQ5_9BILA\nCSK\tA0A0V0UH33_9BILA\nCSK\tA0A0V0UHN1_9BILA\nCSK\tA0A0V0UIH6_9BILA\nCSK\tA0A0V0WFP7_9BILA\nCSK\tA0A0V0WGX5_9BILA\nCSK\tA0A0V0YBP9_TRIPS\nCSK\tA0A0V0ZNF5_9BILA\nCSK\tA0A0V0ZNJ6_9BILA\nCSK\tA0A0V0ZNN2_9BILA\nCSK\tA0A0V0ZNU6_9BILA\nCSK\tA0A0V0ZPV3_9BILA\nCSK\tA0A0V0ZPZ6_9BILA\nCSK\tA0A0V1AYQ8_TRISP\nCSK\tA0A0V1AYQ9_TRISP\nCSK\tA0A0V1AYS4_TRISP\nCSK\tA0A0V1AYV8_TRISP\nCSK\tA0A0V1CGQ5_TRIBR\nCSK\tA0A0V1CGT9_TRIBR\nCSK\tA0A0V1CGX4_TRIBR\nCSK\tA0A0V1CH03_TRIBR\nCSK\tA0A0V1G1Y1_TRIPS\nCSK\tA0A0V1H215_9BILA\nCSK\tA0A0V1H2M9_9BILA\nCSK\tA0A0V1IG62_TRIPS\nCSK\tA0A0V1JJS3_TRIPS\nCSK\tA0A0V1JJX1_TRIPS\nCSK\tA0A0V1L4S9_9BILA\nCSK\tA0A0V1L4T2_9BILA\nCSK\tA0A0V1M777_9BILA\nCSK\tA0A0V1PCU6_9BILA\nCSK\tA0A0X3P3L8_SCHSO\nCSK\tA0A0X3PPB2_SCHSO\nCSK\tA0A0X3PTW2_SCHSO\nCSK\tA0A151MAL0_ALLMI\nCSK\tA0A1A7WG89_9TELE\nCSK\tA0A1A7WJ01_9TELE\nCSK\tA0A1A7Z1Q9_9TELE\nCSK\tA0A1A7ZIF4_NOTFU\nCSK\tA0A1A7ZMM6_NOTFU\nCSK\tA0A1A8BMD0_9TELE\nCSK\tA0A1A8D8G9_9TELE\nCSK\tA0A1A8G6U7_9TELE\nCSK\tA0A1A8GA35_9TELE\nCSK\tA0A1A8I657_NOTKU\nCSK\tA0A1A8KAS9_NOTKU\nCSK\tA0A1A8KCY8_NOTKU\nCSK\tA0A1A8LT30_9TELE\nCSK\tA0A1A8LZV1_9TELE\nCSK\tA0A1A8S5A2_9TELE\nCSK\tA0A1A8SAC9_9TELE\nCSK\tA0A1D5PXS9_CHICK\nCSK\tA0A1L1SQQ5_MOUSE\nCSK\tA0A1L1SR46_MOUSE\nCSK\tA0A1L1STA1_MOUSE\nCSK\tA0A1L8GT96_XENLA\nCSK\tA0A1S3ALG0_ERIEU\nCSK\tA0A1S3ESN0_DIPOR\nCSK\tA0A1U7QKS8_MESAU\nCSK\tA0A1U7RLM8_ALLSI\nCSK\tA0A1U7UAV5_TARSY\nCSK\tA0A1V4JCZ7_PATFA\nCSK\tA0A218UB11_9PASE\nCSK\tA0A250YI82_CASCN\nCSK\tA0A251RUI8_HELAN\nCSK\tA0A286ZMV7_PIG\nCSK\tA0A287AEI0_PIG\nCSK\tA0A2B4RQM4_STYPI\nCSK\tA0A2B4S6P7_STYPI\nCSK\tA0A2B4SRA8_STYPI\nCSK\tA0A2D0RHC9_ICTPU\nCSK\tA0A2I0M6K5_COLLI\nCSK\tA0A2J7RIA3_9NEOP\nCSK\tA0A2K5CNT5_AOTNA\nCSK\tA0A2K5CNV9_AOTNA\nCSK\tA0A2K5I1Q3_COLAP\nCSK\tA0A2K5LNH2_CERAT\nCSK\tA0A2K5QC41_CEBCA\nCSK\tA0A2K5QC49_CEBCA\nCSK\tA0A2K5WPF1_MACFA\nCSK\tA0A2K5Z104_MANLE\nCSK\tA0A2K6CHS9_MACNE\nCSK\tA0A2K6GVY3_PROCO\nCSK\tA0A2K6K9Z4_RHIBE\nCSK\tA0A2K6RA36_RHIRO\nCSK\tA0A2K6UGP2_SAIBB\nCSK\tA0A2K6UGP5_SAIBB\nCSK\tA0A2R9A3Q5_PANPA\nCSK\tA0A2S2NTB5_SCHGA\nCSK\tA0A2S2QGB1_9HEMI\nCSK\tA0A2U3UZJ3_TURTR\nCSK\tA0A2U3WGL3_ODORO\nCSK\tA0A2U3Y9U7_LEPWE\nCSK\tA0A2U4CIB9_TURTR\nCSK\tA0A2U4CIC9_TURTR\nCSK\tA0A2U4CII9_TURTR\nCSK\tA0A2Y9HIQ2_NEOSC\nCSK\tA0A2Y9NNZ0_DELLE\nCSK\tA0A2Y9NS43_DELLE\nCSK\tA0A2Y9NX86_DELLE\nCSK\tA0A314KZC5_NICAT\nCSK\tA0A337SW10_FELCA\nCSK\tA0A340WB71_LIPVE\nCSK\tA0A340WGT5_LIPVE\nCSK\tA0A341CRS3_NEOAA\nCSK\tA0A341CUL8_NEOAA\nCSK\tA0A371G6W7_MUCPR\nCSK\tA0A384BER9_BALAS\nCSK\tA0A384DJA1_URSMA\nCSK\tA0A3B3CF11_ORYME\nCSK\tA0A3B3DQ71_ORYME\nCSK\tA0A3B3HZC3_ORYLA\nCSK\tA0A3B3U7M4_9TELE\nCSK\tA0A3B3VIC1_9TELE\nCSK\tA0A3B3WMT2_9TELE\nCSK\tA0A3B3Y8Y9_9TELE\nCSK\tA0A3B3ZVC3_9GOBI\nCSK\tA0A3B4FQB0_9CICH\nCSK\tA0A3B4FQC1_9CICH\nCSK\tA0A3B4TVZ3_SERDU\nCSK\tA0A3B4TWL4_SERDU\nCSK\tA0A3B5B2L6_9TELE\nCSK\tA0A3B5B2P1_9TELE\nCSK\tA0A3B5M5Q9_9TELE\nCSK\tA0A3P8VPW8_CYNSE\nCSK\tA0A3P8VSH5_CYNSE\nCSK\tA0A3P9I9H1_ORYLA\nCSK\tA0A3P9I9R3_ORYLA\nCSK\tA0A3P9NRH4_POERE\nCSK\tA0A3Q0R889_AMPCI\nCSK\tA0A3Q1EHN4_9TELE\nCSK\tA0A3Q1FX07_9TELE\nCSK\tA0A3Q1H198_9TELE\nCSK\tA0A3Q1HF26_ANATE\nCSK\tA0A3Q1IPD2_ANATE\nCSK\tA0A3Q1MM24_BOVIN\nCSK\tA0A3Q2HGY5_HORSE\nCSK\tA0A3Q2PXA6_FUNHE\nCSK\tA0A3Q2WB41_HAPBU\nCSK\tA0A3Q2X4T5_HAPBU\nCSK\tA0A3Q3DPS4_HIPCM\nCSK\tA0A3Q3EKS8_KRYMA\nCSK\tA0A3Q3EKV3_KRYMA\nCSK\tA0A3Q3L216_9TELE\nCSK\tA0A3Q3X0I8_MOLML\nCSK\tA0A3Q4GMH4_NEOBR\nCSK\tA0A3Q4GSV3_NEOBR\nCSK\tA0A3Q7RDQ4_VULVU\nCSK\tA0A3Q7RFS7_CALUR\nCSK\tA0A3Q7TEX9_URSAR\nCSK\tA0A3Q7U243_URSAR\nCSK\tA0A452FQV5_CAPHI\nCSK\tA0A452FQZ5_CAPHI\nCSK\tA0A452IXT4_9SAUR\nCSK\tA0A452IXV7_9SAUR\nCSK\tA0A452SQW6_URSAM\nCSK\tA0A452TEE0_URSMA\nCSK\tA0A452TEH9_URSMA\nCSK\tA0A455BSF5_PHYMC\nCSK\tA0A4W3I7C5_CALMI\nCSK\tA0A4W3IB46_CALMI\nCSK\tA0A4W3IER1_CALMI\nCSK\tA0A4W3IGH7_CALMI\nCSK\tA0A4W3IGN0_CALMI\nCSK\tA0A4W3IGN6_CALMI\nCSK\tA0A4W4EB15_ELEEL\nCSK\tA0A4W4EBB9_ELEEL\nCSK\tA0A4W4EC36_ELEEL\nCSK\tA0A4W4EC39_ELEEL\nCSK\tA0A4W4EC54_ELEEL\nCSK\tA0A4W4EC83_ELEEL\nCSK\tA0A4W4ED36_ELEEL\nCSK\tA0A4W4ED97_ELEEL\nCSK\tA0A4W4EDA5_ELEEL\nCSK\tA0A4W4EDW4_ELEEL\nCSK\tA0A4W6G579_LATCA\nCSK\tA0A4W6G584_LATCA\nCSK\tA0A4W6G5U1_LATCA\nCSK\tA0A4W6G797_LATCA\nCSK\tA0A4W6G895_LATCA\nCSK\tA0A4W6G8A0_LATCA\nCSK\tA0A4X2L318_VOMUR\nCSK\tA0A4Y2HVQ9_ARAVE\nCSK\tA0A4Z2FAP7_9TELE\nCSK\tA0A4Z2IWQ7_9TELE\nCSK\tA0A5G3J8A9_XENTR\nCSK\tA0A5G3J8G6_XENTR\nCSK\tA0A5G3J933_XENTR\nCSK\tA0A5G3KRC6_XENTR\nCSK\tA0JMN4_DANRE\nCSK\tB2R6Q4_HUMAN\nCSK\tB5BU52_HUMAN\nCSK\tB5DDX0_XENTR\nCSK\tCSK_ARATH\nCSK\tCSK_BOVIN\nCSK\tCSK_CHICK\nCSK\tCSK_HUMAN\nCSK\tCSK_MOUSE\nCSK\tCSK_RAT\nCSK\tD2HKY4_AILME\nCSK\tE2RFF7_CANLF\nCSK\tF1SJ27_PIG\nCSK\tF6QS08_CALJA\nCSK\tF6S8K4_XENTR\nCSK\tF6S9X9_XENTR\nCSK\tF6UTP2_XENTR\nCSK\tF6UX26_XENTR\nCSK\tF6ZZA7_HORSE\nCSK\tF7GJZ7_MONDO\nCSK\tF7GRP2_MACMU\nCSK\tG1MU24_MELGA\nCSK\tG1PN95_MYOLU\nCSK\tG1RWZ7_NOMLE\nCSK\tG3PP34_GASAC\nCSK\tG3T005_LOXAF\nCSK\tG3UP76_MELGA\nCSK\tG3WSI5_SARHA\nCSK\tG5AL62_HETGA\nCSK\tH0V0W0_CAVPO\nCSK\tH0WSD9_OTOGA\nCSK\tH2MHC1_ORYLA\nCSK\tH2NNS6_PONAB\nCSK\tH2V3G0_TAKRU\nCSK\tH3AGH9_LATCH\nCSK\tH3AGI0_LATCH\nCSK\tH3BN15_HUMAN\nCSK\tH3BU69_HUMAN\nCSK\tH3BUM9_HUMAN\nCSK\tH3C5K0_TETNG\nCSK\tH3D302_TETNG\nCSK\tH9GLA2_ANOCA\nCSK\tI3N6S6_ICTTR\nCSK\tK7AWV1_PANTR\nCSK\tK7GIT7_PELSI\nCSK\tM1R8K8_RANCL\nCSK\tM3VUK8_FELCA\nCSK\tM3Z0S9_MUSPF\nCSK\tM4AQP4_XIPMA\nCSK\tO73786_XENLA\nCSK\tO77132_HYDVU\nCSK\tQ3UVH2_MOUSE\nCSK\tQ5MAS9_XENTR\nCSK\tQ6R1Y6_PATMI\nCSK\tQ7KSQ2_DROME\nCSK\tQ8BIS9_MOUSE\nCSK\tQ8INJ8_DROME\nCSK\tQ98TP5_PLAFE\nCSK\tQ9VGK8_DROME\nCSK\tT2MFU5_HYDVU\nCSK\tU3K8U1_FICAL\nCSK\tU6DLW6_NEOVI\nCSK\tV5FVP7_ANOGL\nCSK\tV8NJS7_OPHHA\nCSK\tV9KIW4_CALMI\nCSK\tW5NZ63_SHEEP\nCSK\tW5UB96_ICTPU\nCSK\tW8AX52_CERCA\n','From\tTo\nKIT\tA0A059TX05_BOSMU\nKIT\tA0A059TZU0_BOSMU\nKIT\tA0A059V9P6_RABIT\nKIT\tA0A059VEH2_RABIT\nKIT\tA0A059VFB1_RABIT\nKIT\tA0A059VJ35_RABIT\nKIT\tA0A075CJU3_CAPHI\nKIT\tA0A075CK89_CAPHI\nKIT\tA0A075MG40_CANLF\nKIT\tA0A089NL01_NYCPP\nKIT\tA0A096P031_PAPAN\nKIT\tA0A0A9VPP4_LYGHE\nKIT\tA0A0C5G4P5_SHEEP\nKIT\tA0A0C5G4Q0_SHEEP\nKIT\tA0A0C5G4Q6_SHEEP\nKIT\tA0A0C5G4Q9_SHEEP\nKIT\tA0A0C5G4R4_SHEEP\nKIT\tA0A0C5G4R7_SHEEP\nKIT\tA0A0C5GBV1_SHEEP\nKIT\tA0A0C5GBW5_SHEEP\nKIT\tA0A0C5GBX5_SHEEP\nKIT\tA0A0C5GE17_SHEEP\nKIT\tA0A0C5GE22_SHEEP\nKIT\tA0A0C5GE36_SHEEP\nKIT\tA0A0C5GFN2_SHEEP\nKIT\tA0A0C5GFP0_SHEEP\nKIT\tA0A0C5GFP5_SHEEP\nKIT\tA0A0C5GFQ4_SHEEP\nKIT\tA0A0C5GRF2_SHEEP\nKIT\tA0A0C5GRF7_SHEEP\nKIT\tA0A0C5GRG2_SHEEP\nKIT\tA0A0C5GRG6_SHEEP\nKIT\tA0A0D9QY74_CHLSB\nKIT\tA0A0G2JTL4_RAT\nKIT\tA0A0N7ISQ0_COLLI\nKIT\tA0A0S7H958_9TELE\nKIT\tA0A0S7H9J2_9TELE\nKIT\tA0A0S7H9L8_9TELE\nKIT\tA0A0S7H9P3_9TELE\nKIT\tA0A0S7HA32_9TELE\nKIT\tA0A0S7HBC2_9TELE\nKIT\tA0A0S7HBS4_9TELE\nKIT\tA0A0S7HCZ2_9TELE\nKIT\tA0A0S7HD72_9TELE\nKIT\tA0A0S7M833_9TELE\nKIT\tA0A0U2N547_HUMAN\nKIT\tA0A151M7B1_ALLMI\nKIT\tA0A151M7H6_ALLMI\nKIT\tA0A1C8Y7T5_TYTAL\nKIT\tA0A1S3EKW5_DIPOR\nKIT\tA0A1S3EKX3_DIPOR\nKIT\tA0A1S3PA16_SALSA\nKIT\tA0A1S3PA35_SALSA\nKIT\tA0A1S3PA81_SALSA\nKIT\tA0A1S3WVX5_ERIEU\nKIT\tA0A1S3WW39_ERIEU\nKIT\tA0A1S3WW47_ERIEU\nKIT\tA0A1S3WWG3_ERIEU\nKIT\tA0A1U7R8C7_ALLSI\nKIT\tA0A1U7RFI5_ALLSI\nKIT\tA0A1U8CDI0_MESAU\nKIT\tA0A1U8CLV6_MESAU\nKIT\tA0A1U8D652_ALLSI\nKIT\tA0A1U9XAP2_CAMDR\nKIT\tA0A1V4JGB1_PATFA\nKIT\tA0A218VFF2_9PASE\nKIT\tA0A2B4RVA4_STYPI\nKIT\tA0A2D0Q0E5_ICTPU\nKIT\tA0A2D0Q235_ICTPU\nKIT\tA0A2D1CSI3_RABIT\nKIT\tA0A2H5ABP7_NEOVI\nKIT\tA0A2I0MMR1_COLLI\nKIT\tA0A2I4BZN7_9TELE\nKIT\tA0A2I7M852_9CHAR\nKIT\tA0A2K5DWT5_AOTNA\nKIT\tA0A2K5K5A0_COLAP\nKIT\tA0A2K5KJW0_CERAT\nKIT\tA0A2K5RSK3_CEBCA\nKIT\tA0A2K5U4C5_MACFA\nKIT\tA0A2K5ZVF6_MANLE\nKIT\tA0A2K6C474_MACNE\nKIT\tA0A2K6GC32_PROCO\nKIT\tA0A2K6LXX1_RHIBE\nKIT\tA0A2K6QZX5_RHIRO\nKIT\tA0A2K6QZY1_RHIRO\nKIT\tA0A2K6UGI5_SAIBB\nKIT\tA0A2K6UGI6_SAIBB\nKIT\tA0A2R8M5Y1_CALJA\nKIT\tA0A2R9AQA1_PANPA\nKIT\tA0A2R9AUT4_PANPA\nKIT\tA0A2S1UF08_NEOVI\nKIT\tA0A2U3VMP4_ODORO\nKIT\tA0A2U3ZBP0_ODORO\nKIT\tA0A2U3ZBP2_ODORO\nKIT\tA0A2U3ZBP5_ODORO\nKIT\tA0A2U3ZBQ0_ODORO\nKIT\tA0A2U3ZBR8_ODORO\nKIT\tA0A2U3ZBS3_ODORO\nKIT\tA0A2U3ZCP5_ODORO\nKIT\tA0A2U3ZCQ1_ODORO\nKIT\tA0A2U4ARY1_TURTR\nKIT\tA0A2U4ARY2_TURTR\nKIT\tA0A2U4ARZ4_TURTR\nKIT\tA0A2U4ASE4_TURTR\nKIT\tA0A2Y9FEQ2_PHYMC\nKIT\tA0A2Y9FET5_PHYMC\nKIT\tA0A2Y9FEW7_PHYMC\nKIT\tA0A2Y9FGN9_PHYMC\nKIT\tA0A2Y9I009_NEOSC\nKIT\tA0A2Y9I0C6_NEOSC\nKIT\tA0A2Y9I0D1_NEOSC\nKIT\tA0A2Y9I0I3_NEOSC\nKIT\tA0A2Y9I507_NEOSC\nKIT\tA0A2Y9I6X6_NEOSC\nKIT\tA0A2Y9MC24_DELLE\nKIT\tA0A2Y9MDA2_DELLE\nKIT\tA0A2Y9ML54_DELLE\nKIT\tA0A2Y9MRJ1_DELLE\nKIT\tA0A2Z5XXW3_MYORF\nKIT\tA0A2Z5XXW7_MYORF\nKIT\tA0A2Z5XXW9_MYORF\nKIT\tA0A340X496_LIPVE\nKIT\tA0A340X7Y4_LIPVE\nKIT\tA0A340XAV1_LIPVE\nKIT\tA0A341BL52_NEOAA\nKIT\tA0A341BL53_NEOAA\nKIT\tA0A383Z016_BALAS\nKIT\tA0A383Z037_BALAS\nKIT\tA0A383Z0B1_BALAS\nKIT\tA0A383Z0S2_BALAS\nKIT\tA0A384CY88_URSMA\nKIT\tA0A384DV37_FELCA\nKIT\tA0A3B3HC84_ORYLA\nKIT\tA0A3G1ZL07_NEOVI\nKIT\tA0A3Q0DL37_TARSY\nKIT\tA0A3Q7QC94_CALUR\nKIT\tA0A3Q7QCB1_CALUR\nKIT\tA0A3Q7U394_VULVU\nKIT\tA0A3Q7VU84_URSAR\nKIT\tA0A3Q7VU90_URSAR\nKIT\tA0A3Q7WI28_URSAR\nKIT\tA0A3Q7WI32_URSAR\nKIT\tA0A3Q7XE64_URSAR\nKIT\tA0A3Q7XE67_URSAR\nKIT\tA0A3Q7XPB3_URSAR\nKIT\tA0A452DHX3_BOVIN\nKIT\tA0A452FN97_CAPHI\nKIT\tA0A452FNE4_CAPHI\nKIT\tA0A452ITL2_9SAUR\nKIT\tA0A452RI37_URSAM\nKIT\tA0A452RI44_URSAM\nKIT\tA0A452RI48_URSAM\nKIT\tA0A452RIB7_URSAM\nKIT\tA0A452RIC3_URSAM\nKIT\tA0A452RIN5_URSAM\nKIT\tA0A452RIN8_URSAM\nKIT\tA0A452RIP0_URSAM\nKIT\tA0A452TFC7_URSMA\nKIT\tA0A452TFU4_URSMA\nKIT\tA0A452TFV7_URSMA\nKIT\tA0A452TG87_URSMA\nKIT\tA0A452TGB5_URSMA\nKIT\tA0A452TGC0_URSMA\nKIT\tA0A452TGF2_URSMA\nKIT\tA0A452TGG1_URSMA\nKIT\tA0A452TGX9_URSMA\nKIT\tA0A482JKW4_PIG\nKIT\tA0A482JPB3_PIG\nKIT\tA0A482JQ76_PIG\nKIT\tA0A493STX0_ANAPP\nKIT\tA0A493T1V0_ANAPP\nKIT\tA0A4W2DE79_BOBOX\nKIT\tA0A4W2HH25_BOBOX\nKIT\tA0A4W6CCD5_LATCA\nKIT\tA0A4W6CI82_LATCA\nKIT\tA0A4W6CJK7_LATCA\nKIT\tA0A4W6CJL0_LATCA\nKIT\tA0A4W6CJM3_LATCA\nKIT\tA0A4X2KQ01_VOMUR\nKIT\tA0A4X2KXM9_VOMUR\nKIT\tA0A5F4W8K4_CALJA\nKIT\tA0A5F4WMD5_CALJA\nKIT\tA0A5F7Z806_MACMU\nKIT\tA0A5F9CN05_RABIT\nKIT\tA0A5F9CQS3_RABIT\nKIT\tA0A5F9DUQ7_RABIT\nKIT\tA0A5G2QNL8_PIG\nKIT\tA0A5G3HRL7_XENTR\nKIT\tA0A5G3JGE4_XENTR\nKIT\tA0A5G3JGK4_XENTR\nKIT\tA0A5G3JGV0_XENTR\nKIT\tA0A5G3JGW1_XENTR\nKIT\tA0A5G3JI21_XENTR\nKIT\tA0A5G3JIG4_XENTR\nKIT\tA0A5G3JJ34_XENTR\nKIT\tA0A5G3KPV2_XENTR\nKIT\tA0A5G3KQ23_XENTR\nKIT\tA0A5G3KQ40_XENTR\nKIT\tA0A5G3KQA0_XENTR\nKIT\tA0A5G3KQC6_XENTR\nKIT\tA4UXZ0_PIG\nKIT\tA4ZVL1_MOUSE\nKIT\tA8E0G5_HORSE\nKIT\tB4F6V0_XENTR\nKIT\tB6DXN3_HUMAN\nKIT\tB6E0M6_HUMAN\nKIT\tB7SCJ0_RAT\nKIT\tB7SCJ1_RAT\nKIT\tB7SCJ2_RAT\nKIT\tC4PFJ6_PIG\nKIT\tC7ENW2_MOUSE\nKIT\tD5MAV8_HUMAN\nKIT\tD5MAV9_HUMAN\nKIT\tE2I6G0_HUMAN\nKIT\tE7BYE6_PIG\nKIT\tF1PUW9_CANLF\nKIT\tF1SE53_PIG\nKIT\tF5HS93_HORSE\nKIT\tF6RMQ0_XENTR\nKIT\tF6U370_ORNAN\nKIT\tF6V858_MACMU\nKIT\tF6WEC0_XENTR\nKIT\tF6YAJ6_CALJA\nKIT\tF6ZM42_HORSE\nKIT\tF6ZYT3_XENTR\nKIT\tF6ZZW6_HORSE\nKIT\tF7A906_MONDO\nKIT\tF7BCJ8_HORSE\nKIT\tF7CD30_XENTR\nKIT\tG1KJX6_ANOCA\nKIT\tG1MJB7_AILME\nKIT\tG1MJB8_AILME\nKIT\tG1NH11_MELGA\nKIT\tG1NXV5_MYOLU\nKIT\tG1RAC7_NOMLE\nKIT\tG1SSC3_RABIT\nKIT\tG3TS12_LOXAF\nKIT\tG3TWK9_LOXAF\nKIT\tG3TYJ4_LOXAF\nKIT\tG3UE86_LOXAF\nKIT\tG3UPP3_MELGA\nKIT\tG5BFM4_HETGA\nKIT\tG5E7C2_LOXAF\nKIT\tG9CTQ7_CERSI\nKIT\tG9CTQ8_DICSU\nKIT\tG9CTQ9_DICBI\nKIT\tG9CTR0_RHIUN\nKIT\tG9CTR1_TAPPI\nKIT\tG9CTR2_TAPTE\nKIT\tG9CTR3_TAPBA\nKIT\tG9CTR4_TAPIN\nKIT\tG9CTR5_HORSE\nKIT\tG9CTR6_EQUPR\nKIT\tG9CTR7_EQUAS\nKIT\tG9CTR8_9EUTH\nKIT\tG9CTR9_EQUHE\nKIT\tG9CTS0_EQUHM\nKIT\tG9CTS1_EQUBU\nKIT\tG9CTS2_EQUZE\nKIT\tG9CTS3_EQUGR\nKIT\tH0VJ44_CAVPO\nKIT\tH0X0T0_OTOGA\nKIT\tH0ZBL7_TAEGU\nKIT\tH2L4L2_ORYLA\nKIT\tH2QPH5_PANTR\nKIT\tH2RHI1_PANTR\nKIT\tH3A412_LATCH\nKIT\tH9FU74_MACMU\nKIT\tI3J4E4_ORENI\nKIT\tI3LXA5_ICTTR\nKIT\tI3W8V5_PIG\nKIT\tI3W8V6_PIG\nKIT\tI6UZ27_VULVU\nKIT\tI7C835_9CICH\nKIT\tI7C839_HAPBU\nKIT\tI7C842_9CICH\nKIT\tI7CHV1_TROMR\nKIT\tI7CHV7_9CICH\nKIT\tI7D5F6_9CICH\nKIT\tI7D5G0_9CICH\nKIT\tI7D5G2_9CICH\nKIT\tI7DEX5_9CICH\nKIT\tI7DEX8_9CICH\nKIT\tI7DEY2_9CICH\nKIT\tI7DHZ3_9CICH\nKIT\tI7DHZ7_9CICH\nKIT\tI7DI01_TYLPO\nKIT\tJ7GNJ8_CAPHI\nKIT\tK7CAF7_PANTR\nKIT\tK7FMK0_PELSI\nKIT\tK7WYD4_MESAU\nKIT\tK7XYE0_FELCA\nKIT\tKITA_DANRE\nKIT\tKIT_BOVIN\nKIT\tKIT_CALJA\nKIT\tKIT_CANLF\nKIT\tKIT_CAPHI\nKIT\tKIT_CHICK\nKIT\tKIT_FELCA\nKIT\tKIT_HUMAN\nKIT\tKIT_MOUSE\nKIT\tKIT_PIG\nKIT\tKIT_TAKRU\nKIT\tKIT_XENLA\nKIT\tL8E780_HUMAN\nKIT\tM3Y412_MUSPF\nKIT\tO77589_HORSE\nKIT\tQ0VJ99_SHEEP\nKIT\tQ14708_HUMAN\nKIT\tQ14709_HUMAN\nKIT\tQ15999_HUMAN\nKIT\tQ29440_PIG\nKIT\tQ29VW5_CAVPO\nKIT\tQ29VW6_CAVPO\nKIT\tQ2LJ92_PIG\nKIT\tQ2LJ95_PIG\nKIT\tQ2LJ97_PIG\nKIT\tQ2LJA0_PIG\nKIT\tQ2PZA7_PIG\nKIT\tQ2PZA8_PIG\nKIT\tQ32Z52_CANLF\nKIT\tQ3ULJ6_MOUSE\nKIT\tQ5D4S1_HORSE\nKIT\tQ5FYR8_HORSE\nKIT\tQ5FYR9_HORSE\nKIT\tQ63116_RAT\nKIT\tQ6JZS4_HORSE\nKIT\tQ6P6Y9_XENLA\nKIT\tQ865R6_CAPCR\nKIT\tQ8QGS1_COTJA\nKIT\tQ8TCG9_HUMAN\nKIT\tQ95KZ9_HORSE\nKIT\tQ9TQS0_BOVIN\nKIT\tQ9UMA0_HUMAN\nKIT\tU3CN62_CALJA\nKIT\tU3EUB3_CALJA\nKIT\tU3I8W5_ANAPP\nKIT\tU3JLK5_FICAL\nKIT\tU3JLL0_FICAL\nKIT\tU5LL56_VULLA\nKIT\tU6DG94_NEOVI\nKIT\tV8NQ86_OPHHA\nKIT\tW5NPJ1_SHEEP\nKIT\tW5UKQ9_ICTPU\n','From\tTo\nFGFR2\tA0A034VJE0_BACDO\nFGFR2\tA0A087WY21_HUMAN\nFGFR2\tA0A087X2D1_HUMAN\nFGFR2\tA0A096NU07_PAPAN\nFGFR2\tA0A0A0MR25_HUMAN\nFGFR2\tA0A0D9QXC2_CHLSB\nFGFR2\tA0A0K1P505_GOBRA\nFGFR2\tA0A0S7G5D9_9TELE\nFGFR2\tA0A0S7G5E2_9TELE\nFGFR2\tA0A0S7G5H8_9TELE\nFGFR2\tA0A0S7G6Z3_9TELE\nFGFR2\tA0A0S7GAA0_9TELE\nFGFR2\tA0A0S7GBU6_9TELE\nFGFR2\tA0A0S7GZB4_9TELE\nFGFR2\tA0A0S7H359_9TELE\nFGFR2\tA0A0S7H5E9_9TELE\nFGFR2\tA0A0V0J6Y3_SCHSO\nFGFR2\tA0A0V0J852_SCHSO\nFGFR2\tA0A0X3NME0_SCHSO\nFGFR2\tA0A0X3PAB5_SCHSO\nFGFR2\tA0A0X3PIE9_SCHSO\nFGFR2\tA0A141AXF0_HUMAN\nFGFR2\tA0A141AXF1_HUMAN\nFGFR2\tA0A141AXF2_HUMAN\nFGFR2\tA0A141AXF3_HUMAN\nFGFR2\tA0A141AXF4_HUMAN\nFGFR2\tA0A141AXF5_HUMAN\nFGFR2\tA0A141AXF6_HUMAN\nFGFR2\tA0A141AXF7_HUMAN\nFGFR2\tA0A141AXF8_HUMAN\nFGFR2\tA0A142FHI6_9CICH\nFGFR2\tA0A151NBT6_ALLMI\nFGFR2\tA0A1A7XB46_9TELE\nFGFR2\tA0A1A7XWX3_9TELE\nFGFR2\tA0A1A7ZLH9_NOTFU\nFGFR2\tA0A1A8BAP2_9TELE\nFGFR2\tA0A1A8C3U3_9TELE\nFGFR2\tA0A1A8DSP8_9TELE\nFGFR2\tA0A1A8EZL1_9TELE\nFGFR2\tA0A1A8GYK5_9TELE\nFGFR2\tA0A1A8IVL3_NOTKU\nFGFR2\tA0A1A8IZR2_NOTKU\nFGFR2\tA0A1A8K0D9_NOTKU\nFGFR2\tA0A1A8L873_9TELE\nFGFR2\tA0A1A8NP91_9TELE\nFGFR2\tA0A1A8P355_9TELE\nFGFR2\tA0A1A8UYJ2_NOTFU\nFGFR2\tA0A1B0GWF4_HUMAN\nFGFR2\tA0A1B0GYB8_HUMAN\nFGFR2\tA0A1B0GYP7_HUMAN\nFGFR2\tA0A1D5NXU2_CHICK\nFGFR2\tA0A1D5QFH7_MACMU\nFGFR2\tA0A1D5QYN7_MACMU\nFGFR2\tA0A1L8FE89_XENLA\nFGFR2\tA0A1L8FJ12_XENLA\nFGFR2\tA0A1S3FLI2_DIPOR\nFGFR2\tA0A1S3FM56_DIPOR\nFGFR2\tA0A1S3FN18_DIPOR\nFGFR2\tA0A1S3N338_SALSA\nFGFR2\tA0A1S3N342_SALSA\nFGFR2\tA0A1S3N355_SALSA\nFGFR2\tA0A1S3N3J6_SALSA\nFGFR2\tA0A1S3N4D1_SALSA\nFGFR2\tA0A1S3W6P6_ERIEU\nFGFR2\tA0A1U7RKR6_ALLSI\nFGFR2\tA0A1U7RSJ1_ALLSI\nFGFR2\tA0A1U7RWK2_ALLSI\nFGFR2\tA0A1U7S3Y8_ALLSI\nFGFR2\tA0A1V4KI52_PATFA\nFGFR2\tA0A1V4KI61_PATFA\nFGFR2\tA0A1V4KI94_PATFA\nFGFR2\tA0A1V4KID2_PATFA\nFGFR2\tA0A1W2PQT9_HUMAN\nFGFR2\tA0A218URJ2_9PASE\nFGFR2\tA0A286XDD5_CAVPO\nFGFR2\tA0A286XNN2_CAVPO\nFGFR2\tA0A286XXN2_CAVPO\nFGFR2\tA0A286Y3W6_CAVPO\nFGFR2\tA0A287CXN7_ICTTR\nFGFR2\tA0A287CZ45_ICTTR\nFGFR2\tA0A287D6C4_ICTTR\nFGFR2\tA0A287DDX1_ICTTR\nFGFR2\tA0A2B4R5B4_STYPI\nFGFR2\tA0A2B4RCA1_STYPI\nFGFR2\tA0A2B4RF21_STYPI\nFGFR2\tA0A2B4RWQ3_STYPI\nFGFR2\tA0A2B4SPP4_STYPI\nFGFR2\tA0A2B4SU28_STYPI\nFGFR2\tA0A2D0QM04_ICTPU\nFGFR2\tA0A2D0QMB0_ICTPU\nFGFR2\tA0A2D0QML6_ICTPU\nFGFR2\tA0A2D0QP72_ICTPU\nFGFR2\tA0A2D0QQ00_ICTPU\nFGFR2\tA0A2D0QQ05_ICTPU\nFGFR2\tA0A2I0MEG3_COLLI\nFGFR2\tA0A2I0MEG4_COLLI\nFGFR2\tA0A2I0MEG9_COLLI\nFGFR2\tA0A2I0MEH3_COLLI\nFGFR2\tA0A2I0MEH5_COLLI\nFGFR2\tA0A2I0MEH7_COLLI\nFGFR2\tA0A2I0MEI0_COLLI\nFGFR2\tA0A2I0MEI2_COLLI\nFGFR2\tA0A2I0MEJ0_COLLI\nFGFR2\tA0A2I3G2V9_NOMLE\nFGFR2\tA0A2I3GMT9_NOMLE\nFGFR2\tA0A2I3GXY6_NOMLE\nFGFR2\tA0A2I3GY41_NOMLE\nFGFR2\tA0A2I3H5G4_NOMLE\nFGFR2\tA0A2I3H724_NOMLE\nFGFR2\tA0A2I3HEC6_NOMLE\nFGFR2\tA0A2I3LIL7_PAPAN\nFGFR2\tA0A2I3LSQ2_PAPAN\nFGFR2\tA0A2I3LY86_PAPAN\nFGFR2\tA0A2I3M113_PAPAN\nFGFR2\tA0A2I3M4R3_PAPAN\nFGFR2\tA0A2I3MKQ8_PAPAN\nFGFR2\tA0A2I3NC14_PAPAN\nFGFR2\tA0A2I3NGK1_PAPAN\nFGFR2\tA0A2I3RBC2_PANTR\nFGFR2\tA0A2I3S3S0_PANTR\nFGFR2\tA0A2I3SJ27_PANTR\nFGFR2\tA0A2I3SNW1_PANTR\nFGFR2\tA0A2I3SPX8_PANTR\nFGFR2\tA0A2I3TFU4_PANTR\nFGFR2\tA0A2I3TJB1_PANTR\nFGFR2\tA0A2I3TLQ2_PANTR\nFGFR2\tA0A2I4C9D5_9TELE\nFGFR2\tA0A2K5E4D3_AOTNA\nFGFR2\tA0A2K5E4E2_AOTNA\nFGFR2\tA0A2K5E4J3_AOTNA\nFGFR2\tA0A2K5E4J9_AOTNA\nFGFR2\tA0A2K5E4N7_AOTNA\nFGFR2\tA0A2K5E4T0_AOTNA\nFGFR2\tA0A2K5E4Z1_AOTNA\nFGFR2\tA0A2K5E518_AOTNA\nFGFR2\tA0A2K5E522_AOTNA\nFGFR2\tA0A2K5IQJ1_COLAP\nFGFR2\tA0A2K5IQN9_COLAP\nFGFR2\tA0A2K5IQV7_COLAP\nFGFR2\tA0A2K5IQX5_COLAP\nFGFR2\tA0A2K5IRN4_COLAP\nFGFR2\tA0A2K5IRU3_COLAP\nFGFR2\tA0A2K5IS06_COLAP\nFGFR2\tA0A2K5IS63_COLAP\nFGFR2\tA0A2K5IS69_COLAP\nFGFR2\tA0A2K5MTR7_CERAT\nFGFR2\tA0A2K5MTV8_CERAT\nFGFR2\tA0A2K5MTX7_CERAT\nFGFR2\tA0A2K5MTZ2_CERAT\nFGFR2\tA0A2K5MTZ6_CERAT\nFGFR2\tA0A2K5MTZ9_CERAT\nFGFR2\tA0A2K5MU70_CERAT\nFGFR2\tA0A2K5MU75_CERAT\nFGFR2\tA0A2K5MU85_CERAT\nFGFR2\tA0A2K5MUF0_CERAT\nFGFR2\tA0A2K5R1R3_CEBCA\nFGFR2\tA0A2K5R1U8_CEBCA\nFGFR2\tA0A2K5R1V5_CEBCA\nFGFR2\tA0A2K5R1W3_CEBCA\nFGFR2\tA0A2K5R1X0_CEBCA\nFGFR2\tA0A2K5R1X4_CEBCA\nFGFR2\tA0A2K5R1Y2_CEBCA\nFGFR2\tA0A2K5R1Z2_CEBCA\nFGFR2\tA0A2K5TL65_MACFA\nFGFR2\tA0A2K5TL82_MACFA\nFGFR2\tA0A2K5TL84_MACFA\nFGFR2\tA0A2K5TL97_MACFA\nFGFR2\tA0A2K5TLA1_MACFA\nFGFR2\tA0A2K5TLA3_MACFA\nFGFR2\tA0A2K5TLA5_MACFA\nFGFR2\tA0A2K5TLA6_MACFA\nFGFR2\tA0A2K5TLC3_MACFA\nFGFR2\tA0A2K5XUS6_MANLE\nFGFR2\tA0A2K5XUT8_MANLE\nFGFR2\tA0A2K5XUY7_MANLE\nFGFR2\tA0A2K5XVB4_MANLE\nFGFR2\tA0A2K5XVG5_MANLE\nFGFR2\tA0A2K5XVI5_MANLE\nFGFR2\tA0A2K5XVS4_MANLE\nFGFR2\tA0A2K5XVV5_MANLE\nFGFR2\tA0A2K6DI84_MACNE\nFGFR2\tA0A2K6DIX2_MACNE\nFGFR2\tA0A2K6DJ09_MACNE\nFGFR2\tA0A2K6DJB4_MACNE\nFGFR2\tA0A2K6DJH6_MACNE\nFGFR2\tA0A2K6DJJ0_MACNE\nFGFR2\tA0A2K6DJM3_MACNE\nFGFR2\tA0A2K6DJM9_MACNE\nFGFR2\tA0A2K6DJN5_MACNE\nFGFR2\tA0A2K6GVP5_PROCO\nFGFR2\tA0A2K6GVQ0_PROCO\nFGFR2\tA0A2K6GVR5_PROCO\nFGFR2\tA0A2K6GVS4_PROCO\nFGFR2\tA0A2K6GVT5_PROCO\nFGFR2\tA0A2K6GVT9_PROCO\nFGFR2\tA0A2K6GVU0_PROCO\nFGFR2\tA0A2K6GVZ3_PROCO\nFGFR2\tA0A2K6KU51_RHIBE\nFGFR2\tA0A2K6KU97_RHIBE\nFGFR2\tA0A2K6KUA0_RHIBE\nFGFR2\tA0A2K6KUB2_RHIBE\nFGFR2\tA0A2K6KUU3_RHIBE\nFGFR2\tA0A2K6KVF7_RHIBE\nFGFR2\tA0A2K6KVL0_RHIBE\nFGFR2\tA0A2K6KVQ6_RHIBE\nFGFR2\tA0A2K6KVU1_RHIBE\nFGFR2\tA0A2K6KVU4_RHIBE\nFGFR2\tA0A2K6PLW4_RHIRO\nFGFR2\tA0A2K6PLY9_RHIRO\nFGFR2\tA0A2K6PM07_RHIRO\nFGFR2\tA0A2K6PM16_RHIRO\nFGFR2\tA0A2K6PM23_RHIRO\nFGFR2\tA0A2K6PM49_RHIRO\nFGFR2\tA0A2K6PM64_RHIRO\nFGFR2\tA0A2K6PM84_RHIRO\nFGFR2\tA0A2K6SD43_SAIBB\nFGFR2\tA0A2K6SD90_SAIBB\nFGFR2\tA0A2K6SD92_SAIBB\nFGFR2\tA0A2K6SD95_SAIBB\nFGFR2\tA0A2K6SDG6_SAIBB\nFGFR2\tA0A2K6SDJ4_SAIBB\nFGFR2\tA0A2K6SDK6_SAIBB\nFGFR2\tA0A2K6SDL1_SAIBB\nFGFR2\tA0A2K6SDN9_SAIBB\nFGFR2\tA0A2R9B172_PANPA\nFGFR2\tA0A2R9B1F3_PANPA\nFGFR2\tA0A2R9B1F5_PANPA\nFGFR2\tA0A2R9B1J4_PANPA\nFGFR2\tA0A2R9B1L0_PANPA\nFGFR2\tA0A2R9B1N1_PANPA\nFGFR2\tA0A2R9B2V2_PANPA\nFGFR2\tA0A2R9B3D6_PANPA\nFGFR2\tA0A2R9B588_PANPA\nFGFR2\tA0A2S2PCE8_SCHGA\nFGFR2\tA0A2S2QXK6_9HEMI\nFGFR2\tA0A2U3W792_ODORO\nFGFR2\tA0A2U3W799_ODORO\nFGFR2\tA0A2U3W7A7_ODORO\nFGFR2\tA0A2U3Y5B6_LEPWE\nFGFR2\tA0A2U3ZJB4_ODORO\nFGFR2\tA0A2U3ZJB7_ODORO\nFGFR2\tA0A2U3ZKD6_ODORO\nFGFR2\tA0A2U4BR53_TURTR\nFGFR2\tA0A2U4BR55_TURTR\nFGFR2\tA0A2U4BRB3_TURTR\nFGFR2\tA0A2U4BRG1_TURTR\nFGFR2\tA0A2Y9EWX8_PHYMC\nFGFR2\tA0A2Y9I675_NEOSC\nFGFR2\tA0A2Y9I6K5_NEOSC\nFGFR2\tA0A2Y9I720_NEOSC\nFGFR2\tA0A2Y9I725_NEOSC\nFGFR2\tA0A2Y9IB08_NEOSC\nFGFR2\tA0A2Y9IB12_NEOSC\nFGFR2\tA0A2Y9IDN3_NEOSC\nFGFR2\tA0A2Y9MPS2_DELLE\nFGFR2\tA0A2Y9MQL8_DELLE\nFGFR2\tA0A2Y9MQM4_DELLE\nFGFR2\tA0A2Y9MQM8_DELLE\nFGFR2\tA0A2Y9MQN3_DELLE\nFGFR2\tA0A2Y9MTL9_DELLE\nFGFR2\tA0A2Y9MTM5_DELLE\nFGFR2\tA0A2Y9MTN0_DELLE\nFGFR2\tA0A2Y9MTN9_DELLE\nFGFR2\tA0A2Y9MYZ6_DELLE\nFGFR2\tA0A2Y9MZ00_DELLE\nFGFR2\tA0A2Y9MZ05_DELLE\nFGFR2\tA0A2Y9N4Y4_DELLE\nFGFR2\tA0A2Y9N4Z0_DELLE\nFGFR2\tA0A2Y9N4Z5_DELLE\nFGFR2\tA0A337SV41_FELCA\nFGFR2\tA0A340Y920_LIPVE\nFGFR2\tA0A340YAX6_LIPVE\nFGFR2\tA0A340YCJ0_LIPVE\nFGFR2\tA0A341CAE9_NEOAA\nFGFR2\tA0A341CAF5_NEOAA\nFGFR2\tA0A341CAF9_NEOAA\nFGFR2\tA0A341CAG4_NEOAA\nFGFR2\tA0A341CAG9_NEOAA\nFGFR2\tA0A341CAH9_NEOAA\nFGFR2\tA0A341CAI4_NEOAA\nFGFR2\tA0A341CAJ2_NEOAA\nFGFR2\tA0A341CAJ7_NEOAA\nFGFR2\tA0A341CAK0_NEOAA\nFGFR2\tA0A341CB13_NEOAA\nFGFR2\tA0A341CB18_NEOAA\nFGFR2\tA0A341CB21_NEOAA\nFGFR2\tA0A341CBG3_NEOAA\nFGFR2\tA0A341CBH1_NEOAA\nFGFR2\tA0A341CBH5_NEOAA\nFGFR2\tA0A341CCW3_NEOAA\nFGFR2\tA0A341CCW8_NEOAA\nFGFR2\tA0A341CCX4_NEOAA\nFGFR2\tA0A383ZDN0_BALAS\nFGFR2\tA0A383ZDW8_BALAS\nFGFR2\tA0A383ZDX3_BALAS\nFGFR2\tA0A383ZDX8_BALAS\nFGFR2\tA0A383ZDY3_BALAS\nFGFR2\tA0A383ZEG2_BALAS\nFGFR2\tA0A383ZEH3_BALAS\nFGFR2\tA0A384BMS7_URSMA\nFGFR2\tA0A3B3I0E5_ORYLA\nFGFR2\tA0A3B3ICB8_ORYLA\nFGFR2\tA0A3B3IMU2_ORYLA\nFGFR2\tA0A3Q0CTT9_MESAU\nFGFR2\tA0A3Q0CTU9_MESAU\nFGFR2\tA0A3Q0CTZ3_MESAU\nFGFR2\tA0A3Q0DLJ3_TARSY\nFGFR2\tA0A3Q0DP04_TARSY\nFGFR2\tA0A3Q0GQI8_ALLSI\nFGFR2\tA0A3Q1LRU7_BOVIN\nFGFR2\tA0A3Q1M5P9_BOVIN\nFGFR2\tA0A3Q1MEB2_BOVIN\nFGFR2\tA0A3Q1MU70_BOVIN\nFGFR2\tA0A3Q2GS33_HORSE\nFGFR2\tA0A3Q2GYT7_HORSE\nFGFR2\tA0A3Q2L7S5_HORSE\nFGFR2\tA0A3Q2LHY7_HORSE\nFGFR2\tA0A3Q7RVA9_VULVU\nFGFR2\tA0A3Q7RXS2_VULVU\nFGFR2\tA0A3Q7RXZ5_VULVU\nFGFR2\tA0A3Q7S6B5_VULVU\nFGFR2\tA0A3Q7S6F0_VULVU\nFGFR2\tA0A3Q7S6F9_VULVU\nFGFR2\tA0A3Q7T634_VULVU\nFGFR2\tA0A3Q7T657_VULVU\nFGFR2\tA0A3Q7T666_VULVU\nFGFR2\tA0A3Q7T677_VULVU\nFGFR2\tA0A3Q7T6A6_VULVU\nFGFR2\tA0A3Q7UNC7_URSAR\nFGFR2\tA0A3Q7UND1_URSAR\nFGFR2\tA0A3Q7UND5_URSAR\nFGFR2\tA0A3Q7VD29_URSAR\nFGFR2\tA0A3Q7VD35_URSAR\nFGFR2\tA0A3Q7VD41_URSAR\nFGFR2\tA0A3Q7VD46_URSAR\nFGFR2\tA0A3Q7VM24_URSAR\nFGFR2\tA0A3Q7VM30_URSAR\nFGFR2\tA0A3Q7VM34_URSAR\nFGFR2\tA0A3Q7W5Y2_URSAR\nFGFR2\tA0A3Q7W5Y7_URSAR\nFGFR2\tA0A3Q7W5Z2_URSAR\nFGFR2\tA0A3Q7W5Z5_URSAR\nFGFR2\tA0A3Q7WI24_URSAR\nFGFR2\tA0A3Q7WI29_URSAR\nFGFR2\tA0A3Q7WI33_URSAR\nFGFR2\tA0A452CA99_BALAS\nFGFR2\tA0A452CAD4_BALAS\nFGFR2\tA0A452CAD7_BALAS\nFGFR2\tA0A452CAP8_BALAS\nFGFR2\tA0A452CAQ4_BALAS\nFGFR2\tA0A452CAU3_BALAS\nFGFR2\tA0A452CAU4_BALAS\nFGFR2\tA0A452CAU8_BALAS\nFGFR2\tA0A452F2R9_CAPHI\nFGFR2\tA0A452F2V8_CAPHI\nFGFR2\tA0A452F370_CAPHI\nFGFR2\tA0A452F397_CAPHI\nFGFR2\tA0A452F3C1_CAPHI\nFGFR2\tA0A452HGX7_9SAUR\nFGFR2\tA0A452HGY2_9SAUR\nFGFR2\tA0A452HGY6_9SAUR\nFGFR2\tA0A452HH06_9SAUR\nFGFR2\tA0A452RT42_URSAM\nFGFR2\tA0A452RT62_URSAM\nFGFR2\tA0A452TC15_URSMA\nFGFR2\tA0A455AJQ5_PHYMC\nFGFR2\tA0A455AY67_PHYMC\nFGFR2\tA0A493STH7_ANAPP\nFGFR2\tA0A493SXJ6_ANAPP\nFGFR2\tA0A493TY45_ANAPP\nFGFR2\tA0A4W2C5C2_BOBOX\nFGFR2\tA0A4W2C5Z9_BOBOX\nFGFR2\tA0A4W2C602_BOBOX\nFGFR2\tA0A4W2CL39_BOBOX\nFGFR2\tA0A4W2E2B2_BOBOX\nFGFR2\tA0A4W2GEQ6_BOBOX\nFGFR2\tA0A4W2GGT8_BOBOX\nFGFR2\tA0A4W2GGZ6_BOBOX\nFGFR2\tA0A4W2I4K1_BOBOX\nFGFR2\tA0A4W2I4Q7_BOBOX\nFGFR2\tA0A4W5MN56_9TELE\nFGFR2\tA0A4W5MN94_9TELE\nFGFR2\tA0A4W5MQL5_9TELE\nFGFR2\tA0A4W5MQM0_9TELE\nFGFR2\tA0A4W5MSF9_9TELE\nFGFR2\tA0A4W5MSL7_9TELE\nFGFR2\tA0A4W5MT53_9TELE\nFGFR2\tA0A4W5MT64_9TELE\nFGFR2\tA0A4W5MUI2_9TELE\nFGFR2\tA0A4W5MUS0_9TELE\nFGFR2\tA0A4X1SNM4_PIG\nFGFR2\tA0A4X1SNN3_PIG\nFGFR2\tA0A4X1SNQ2_PIG\nFGFR2\tA0A4X1SNU4_PIG\nFGFR2\tA0A4X1SP01_PIG\nFGFR2\tA0A4X1SP68_PIG\nFGFR2\tA0A4X1SPF6_PIG\nFGFR2\tA0A4X1SPZ0_PIG\nFGFR2\tA0A4X1SQA5_PIG\nFGFR2\tA0A4X1SQA6_PIG\nFGFR2\tA0A4X2LGI9_VOMUR\nFGFR2\tA0A4X2LGP3_VOMUR\nFGFR2\tA0A4X2LM57_VOMUR\nFGFR2\tA0A4X2LM64_VOMUR\nFGFR2\tA0A4X2LQB2_VOMUR\nFGFR2\tA0A4X2LQB6_VOMUR\nFGFR2\tA0A4X2LQD5_VOMUR\nFGFR2\tA0A4X2LQI5_VOMUR\nFGFR2\tA0A4Y2QDM3_ARAVE\nFGFR2\tA0A4Z2FH13_9TELE\nFGFR2\tA0A5B7K2A4_PORTR\nFGFR2\tA0A5F4BX38_CANLF\nFGFR2\tA0A5F4CIQ5_CANLF\nFGFR2\tA0A5F4CKV6_CANLF\nFGFR2\tA0A5F4CPR8_CANLF\nFGFR2\tA0A5F4CRK0_CANLF\nFGFR2\tA0A5F4D3V4_CANLF\nFGFR2\tA0A5F4D9G4_CANLF\nFGFR2\tA0A5F5PN21_HORSE\nFGFR2\tA0A5F5XNQ0_FELCA\nFGFR2\tA0A5F5XRU1_FELCA\nFGFR2\tA0A5F5Y2A8_FELCA\nFGFR2\tA0A5F5Y2W9_FELCA\nFGFR2\tA0A5F5Y405_FELCA\nFGFR2\tA0A5F5Y433_FELCA\nFGFR2\tA0A5F5Y691_FELCA\nFGFR2\tA0A5F5Y709_FELCA\nFGFR2\tA0A5F8G3A8_MONDO\nFGFR2\tA0A5F8GRK2_MONDO\nFGFR2\tA0A5F8GRZ6_MONDO\nFGFR2\tA0A5F8GYD3_MONDO\nFGFR2\tA0A5F8H0H0_MONDO\nFGFR2\tA0A5F8H4T4_MONDO\nFGFR2\tA0A5F9C2V1_RABIT\nFGFR2\tA0A5F9CKQ8_RABIT\nFGFR2\tA0A5F9CMH9_RABIT\nFGFR2\tA0A5F9CR83_RABIT\nFGFR2\tA0A5F9D0P5_RABIT\nFGFR2\tA0A5F9D372_RABIT\nFGFR2\tA0A5F9D4M5_RABIT\nFGFR2\tA0A5F9D5T0_RABIT\nFGFR2\tA0A5F9DN51_RABIT\nFGFR2\tA0A5G2R7G6_PIG\nFGFR2\tA0A5G2RKE2_PIG\nFGFR2\tA0A5K1V5S3_PIG\nFGFR2\tA1YYM3_MOUSE\nFGFR2\tA1YYM4_MOUSE\nFGFR2\tA1YYM5_MOUSE\nFGFR2\tA1YYM6_MOUSE\nFGFR2\tA1YYM7_MOUSE\nFGFR2\tA1YYM8_MOUSE\nFGFR2\tA1YYM9_MOUSE\nFGFR2\tA1YYN0_MOUSE\nFGFR2\tA1YYN1_MOUSE\nFGFR2\tA1YYN2_MOUSE\nFGFR2\tA1YYN3_MOUSE\nFGFR2\tA1YYN4_MOUSE\nFGFR2\tA1YYN5_MOUSE\nFGFR2\tA1YYN6_MOUSE\nFGFR2\tA1YYN7_MOUSE\nFGFR2\tA1YYN8_MOUSE\nFGFR2\tA1YYN9_MOUSE\nFGFR2\tA1YYP0_MOUSE\nFGFR2\tA1YYP1_MOUSE\nFGFR2\tA4IHW8_XENTR\nFGFR2\tA4JYI8_DANRE\nFGFR2\tA4QN31_DANRE\nFGFR2\tA5A754_PIG\nFGFR2\tA7E310_BOVIN\nFGFR2\tB0EXJ6_PARLI\nFGFR2\tB5A960_HUMAN\nFGFR2\tB5A961_HUMAN\nFGFR2\tB5A962_HUMAN\nFGFR2\tB5A963_HUMAN\nFGFR2\tB7ZWP2_MOUSE\nFGFR2\tB8XEQ1_POERE\nFGFR2\tB9EKY4_MOUSE\nFGFR2\tD2CGD1_HUMAN\nFGFR2\tD3DRD3_HUMAN\nFGFR2\tD3DRD4_HUMAN\nFGFR2\tD3DRD5_HUMAN\nFGFR2\tD3DRD6_HUMAN\nFGFR2\tD3DRD9_HUMAN\nFGFR2\tD3DRE0_HUMAN\nFGFR2\tD3Z5M8_MOUSE\nFGFR2\tD5FGF4_LEUER\nFGFR2\tD5FGF5_LEUER\nFGFR2\tD5FGJ3_SQUAC\nFGFR2\tD5FGJ4_SQUAC\nFGFR2\tD6RJK5_MOUSE\nFGFR2\tE2I6F9_HUMAN\nFGFR2\tE7EVR7_HUMAN\nFGFR2\tE9PVU6_MOUSE\nFGFR2\tE9PX60_MOUSE\nFGFR2\tE9PX67_MOUSE\nFGFR2\tE9PX88_MOUSE\nFGFR2\tE9PX90_MOUSE\nFGFR2\tE9PXV8_MOUSE\nFGFR2\tE9Q5C2_MOUSE\nFGFR2\tE9Q5C3_MOUSE\nFGFR2\tE9Q700_MOUSE\nFGFR2\tE9Q708_MOUSE\nFGFR2\tE9Q7C7_MOUSE\nFGFR2\tE9Q7E6_MOUSE\nFGFR2\tE9Q7E8_MOUSE\nFGFR2\tE9Q7T0_MOUSE\nFGFR2\tE9QK53_MOUSE\nFGFR2\tF1LN06_RAT\nFGFR2\tF1LNW0_RAT\nFGFR2\tF1LRU8_RAT\nFGFR2\tF1LSG7_RAT\nFGFR2\tF1MNW2_BOVIN\nFGFR2\tF1PPD8_CANLF\nFGFR2\tF1PPH0_CANLF\nFGFR2\tF1Q2E3_CANLF\nFGFR2\tF1RBS5_DANRE\nFGFR2\tF1RBT3_DANRE\nFGFR2\tF1SEH9_PIG\nFGFR2\tF2Z480_MOUSE\nFGFR2\tF6S6I6_HORSE\nFGFR2\tF6TVE0_MONDO\nFGFR2\tF6WGT9_HORSE\nFGFR2\tF6YFR4_XENTR\nFGFR2\tF7A5X3_MACMU\nFGFR2\tF7BDU9_MOUSE\nFGFR2\tF7C472_HORSE\nFGFR2\tF7C9J5_HORSE\nFGFR2\tF7CXP1_HORSE\nFGFR2\tF7DHF8_MONDO\nFGFR2\tF7EXS2_ORNAN\nFGFR2\tF7I9Y1_CALJA\nFGFR2\tF8W2E2_DANRE\nFGFR2\tF8W376_DANRE\nFGFR2\tFGFR2_CHICK\nFGFR2\tFGFR2_DANRE\nFGFR2\tFGFR2_DUGJA\nFGFR2\tFGFR2_HUMAN\nFGFR2\tFGFR2_MOUSE\nFGFR2\tFGFR2_NOTVI\nFGFR2\tFGFR2_PLEWA\nFGFR2\tFGFR2_XENLA\nFGFR2\tG1KTN6_ANOCA\nFGFR2\tG1LX56_AILME\nFGFR2\tG1LX76_AILME\nFGFR2\tG1NG34_MELGA\nFGFR2\tG1S4U5_NOMLE\nFGFR2\tG1SSB0_RABIT\nFGFR2\tG3T3F1_LOXAF\nFGFR2\tG3UN36_LOXAF\nFGFR2\tG3UTQ6_MELGA\nFGFR2\tG3VVF0_SARHA\nFGFR2\tG3VVF1_SARHA\nFGFR2\tG3VVF2_SARHA\nFGFR2\tG7N1A1_MACMU\nFGFR2\tH0VD85_CAVPO\nFGFR2\tH0W663_CAVPO\nFGFR2\tH0X6E4_OTOGA\nFGFR2\tH0ZLZ4_TAEGU\nFGFR2\tH2MDJ2_ORYLA\nFGFR2\tH2MDJ4_ORYLA\nFGFR2\tH2Q2P3_PANTR\nFGFR2\tH2V7F6_TAKRU\nFGFR2\tH2V7F7_TAKRU\nFGFR2\tH2V7F8_TAKRU\nFGFR2\tH2V7F9_TAKRU\nFGFR2\tH3ACS7_LATCH\nFGFR2\tH7BXU9_HUMAN\nFGFR2\tH7C265_HUMAN\nFGFR2\tH9EZJ1_MACMU\nFGFR2\tH9F0C9_MACMU\nFGFR2\tH9FRD6_MACMU\nFGFR2\tH9FRD7_MACMU\nFGFR2\tH9U392_FELCA\nFGFR2\tH9U393_FELCA\nFGFR2\tH9U394_FELCA\nFGFR2\tI1Z225_SEBMA\nFGFR2\tI3M3D7_ICTTR\nFGFR2\tJ9QD95_BOVIN\nFGFR2\tK7FL51_PELSI\nFGFR2\tK7FL60_PELSI\nFGFR2\tK7GK89_PIG\nFGFR2\tK7GKI2_PIG\nFGFR2\tK7GQY0_PIG\nFGFR2\tL0N326_MINFU\nFGFR2\tL8E779_HUMAN\nFGFR2\tL8E8R5_HUMAN\nFGFR2\tM3W0T1_FELCA\nFGFR2\tM3XV59_MUSPF\nFGFR2\tQ19N31_MICLV\nFGFR2\tQ5DU72_PIG\nFGFR2\tQ63236_RAT\nFGFR2\tQ63237_RAT\nFGFR2\tQ63238_RAT\nFGFR2\tQ63239_RAT\nFGFR2\tQ63240_RAT\nFGFR2\tQ63241_RAT\nFGFR2\tQ63242_RAT\nFGFR2\tQ6GNP8_XENLA\nFGFR2\tQ805B9_DANRE\nFGFR2\tQ80YG0_CAVPO\nFGFR2\tQ8AYP3_DANRE\nFGFR2\tQ8SPP5_MACMU\nFGFR2\tQ95JB7_SHEEP\nFGFR2\tQ95KU8_BOVIN\nFGFR2\tQ95KV2_BOVIN\nFGFR2\tQ9BDL9_CANLF\nFGFR2\tQ9R293_RAT\nFGFR2\tQ9R294_RAT\nFGFR2\tQ9R299_RAT\nFGFR2\tQ9R2A0_RAT\nFGFR2\tQ9TT07_CANLF\nFGFR2\tQ9TTZ3_RABIT\nFGFR2\tR4GC14_ANOCA\nFGFR2\tS4R381_HUMAN\nFGFR2\tS4R3B2_HUMAN\nFGFR2\tU3BMC6_CALJA\nFGFR2\tU3BNX8_CALJA\nFGFR2\tU3CL89_CALJA\nFGFR2\tU3D3K2_CALJA\nFGFR2\tU3D863_CALJA\nFGFR2\tU3DCN2_CALJA\nFGFR2\tU3EWT9_CALJA\nFGFR2\tU3IX50_ANAPP\nFGFR2\tU3JHI1_FICAL\nFGFR2\tV5GL95_ANOGL\nFGFR2\tV8NI55_OPHHA\nFGFR2\tV8NZR8_OPHHA\nFGFR2\tW5P5S2_SHEEP\nFGFR2\tW8C432_CERCA\nFGFR2\tW8CD61_CERCA\n','From\tTo\nFES\tA0A021X1C7_9RHIZ\nFES\tA0A023YT13_ECOLX\nFES\tA0A024RC92_HUMAN\nFES\tA0A029IBE2_ECOLX\nFES\tA0A029J179_ECOLX\nFES\tA0A037YKA9_ECOLX\nFES\tA0A061YAA8_ECOLX\nFES\tA0A068QR94_9GAMM\nFES\tA0A068Z0H4_9GAMM\nFES\tA0A068Z2S1_9GAMM\nFES\tA0A069XI15_ECOLX\nFES\tA0A070SQP8_ECOLX\nFES\tA0A070US17_ECOLX\nFES\tA0A073GCB1_ECOLX\nFES\tA0A073HRS0_ECOLX\nFES\tA0A073UM61_ECOLX\nFES\tA0A074ISF0_ECOLX\nFES\tA0A077N447_XENBV\nFES\tA0A077NNT1_XENBV\nFES\tA0A077NXV1_XENBV\nFES\tA0A077P213_XENBV\nFES\tA0A077PZX8_XENBV\nFES\tA0A077QGB6_XENBV\nFES\tA0A078LJG9_CITKO\nFES\tA0A078MSQ7_9MICC\nFES\tA0A080G2T7_ECOLX\nFES\tA0A080J761_ECOLX\nFES\tA0A085A0S7_9ENTR\nFES\tA0A085GIE7_9ENTR\nFES\tA0A085JQ94_9GAMM\nFES\tA0A085P4Y6_ECOLX\nFES\tA0A086IJB0_KLEPN\nFES\tA0A086U031_ECOLX\nFES\tA0A090UQW0_9ENTR\nFES\tA0A090V1N4_PSEVU\nFES\tA0A096NK78_PAPAN\nFES\tA0A0A0UDN3_9CARN\nFES\tA0A0A0WLC0_BACMY\nFES\tA0A0A1A7D5_ECOLX\nFES\tA0A0A8VMG5_YERRU\nFES\tA0A0B1G9T3_ECOLX\nFES\tA0A0B1K1R1_ECOLX\nFES\tA0A0B2V4K9_TOXCA\nFES\tA0A0B2V4X1_TOXCA\nFES\tA0A0B2VHG5_TOXCA\nFES\tA0A0B2VS98_TOXCA\nFES\tA0A0B2W5E0_TOXCA\nFES\tA0A0B6FPC2_YERFR\nFES\tA0A0B6X998_XENBV\nFES\tA0A0B7G9J4_KLEVA\nFES\tA0A0C7KKS5_KLEPN\nFES\tA0A0D6H6X2_SALTM\nFES\tA0A0D6HVE9_ALCXX\nFES\tA0A0D9RWV5_CHLSB\nFES\tA0A0E0V2F1_ECOLX\nFES\tA0A0E1LUH7_ECOLX\nFES\tA0A0E1NCK1_YEREN\nFES\tA0A0E1T1R6_ECOLX\nFES\tA0A0F3S7Y3_ECOLX\nFES\tA0A0F3UAL7_ECOLX\nFES\tA0A0F6AY74_SALT1\nFES\tA0A0F6QQN9_PHODC\nFES\tA0A0F6UDR3_PSEAI\nFES\tA0A0F6X1C9_ECOLX\nFES\tA0A0F7RGP1_BACAN\nFES\tA0A0G3CKV8_METBA\nFES\tA0A0G3K4K0_ECOLX\nFES\tA0A0G4DIB2_SARHA\nFES\tA0A0G4DIB4_MYRFA\nFES\tA0A0G4DIB5_SMICR\nFES\tA0A0G4DIM4_ANTFL\nFES\tA0A0G4DIV5_PARAP\nFES\tA0A0G4DIZ5_DASGE\nFES\tA0A0G4DIZ8_NOTTY\nFES\tA0A0G4JKU5_9BURK\nFES\tA0A0G4Q6X9_PROVU\nFES\tA0A0G4QDL6_PROVU\nFES\tA0A0H2V760_ECOL6\nFES\tA0A0H2VTI6_SHIFL\nFES\tA0A0H2WQZ4_SALPA\nFES\tA0A0H2YWH0_ECOK1\nFES\tA0A0H3D8W0_AMYMU\nFES\tA0A0H3JBY4_ECO57\nFES\tA0A0H3MEZ8_ECO7I\nFES\tA0A0H3NAE0_SALTS\nFES\tA0A0H3PRH5_ECO5C\nFES\tA0A0I1NAU3_SHISO\nFES\tA0A0J2B5U8_ECOLX\nFES\tA0A0K0H8B7_SALBC\nFES\tA0A0K3KYG4_ECOLX\nFES\tA0A0K3PL33_ECOLX\nFES\tA0A0K4A0N6_ECOLX\nFES\tA0A0K4PD78_ECOLX\nFES\tA0A0K4SVW0_ECOLX\nFES\tA0A0K5AYJ8_ECOLX\nFES\tA0A0K5FKP6_ECOLX\nFES\tA0A0L6Y3M3_ECOLX\nFES\tA0A0M7DGC0_ALCXX\nFES\tA0A0M7F201_9BURK\nFES\tA0A0M7NYG9_9BURK\nFES\tA0A0M9AHC5_THEAQ\nFES\tA0A0P6JUX7_HETGA\nFES\tA0A0P7QX01_ECOLX\nFES\tA0A0P8BPB9_9BACT\nFES\tA0A0P9I3P0_9GAMM\nFES\tA0A0S4Q4P6_BLAVI\nFES\tA0A0T7NTQ9_YEREN\nFES\tA0A0T9L0A8_9GAMM\nFES\tA0A0T9L4V8_YERKR\nFES\tA0A0T9MGG0_YERIN\nFES\tA0A0T9NEM3_YERKR\nFES\tA0A0T9PKC3_YERKR\nFES\tA0A0T9QA00_YERFR\nFES\tA0A0T9SF08_YEREN\nFES\tA0A0T9T7D1_YEREN\nFES\tA0A0T9TKS4_YERAL\nFES\tA0A0T9U5N8_YERKR\nFES\tA0A0T9UBP5_YERFR\nFES\tA0A0T9UC26_YERIN\nFES\tA0A0U1HBG5_YEREN\nFES\tA0A0U1HP56_YERRO\nFES\tA0A0U1IAH1_YERMO\nFES\tA0A0U1KBR2_YERMO\nFES\tA0A0U1RNL3_MOUSE\nFES\tA0A0U1RPB9_MOUSE\nFES\tA0A0U1RPD6_MOUSE\nFES\tA0A0U1RPM2_MOUSE\nFES\tA0A0U1RPN6_MOUSE\nFES\tA0A0U1RPQ5_MOUSE\nFES\tA0A0U1RQ80_MOUSE\nFES\tA0A0U5KXA7_9GAMM\nFES\tA0A0V1IKN8_TRIPS\nFES\tA0A0V1J541_TRIPS\nFES\tA0A0V9LAS3_ECOLX\nFES\tA0A0W0S115_FLUBO\nFES\tA0A0W0VZG4_9GAMM\nFES\tA0A0W0YJI0_9GAMM\nFES\tA0A0X8X721_9SPHI\nFES\tA0A100J1B7_9ACTN\nFES\tA0A100V769_SALER\nFES\tA0A128SGL3_LEGPN\nFES\tA0A142C5F7_9PRIM\nFES\tA0A142C5F8_CEBAL\nFES\tA0A142C5G1_9PRIM\nFES\tA0A142C5G3_9PRIM\nFES\tA0A142C5G5_9PRIM\nFES\tA0A142C5G6_9PRIM\nFES\tA0A142C5H4_PLEBR\nFES\tA0A142C5H7_9PRIM\nFES\tA0A142C5I1_PLEMO\nFES\tA0A142C5I4_9PRIM\nFES\tA0A142C5I8_PLECU\nFES\tA0A142C5J2_PLEHF\nFES\tA0A142C5J5_9PRIM\nFES\tA0A142C5K0_PLEMO\nFES\tA0A151NUC8_ALLMI\nFES\tA0A154WNV5_ENTCL\nFES\tA0A155XL45_ENTCL\nFES\tA0A155ZYP6_ENTCL\nFES\tA0A156RK45_ENTCL\nFES\tA0A157RAB7_9BORD\nFES\tA0A165RM07_9RHOB\nFES\tA0A165U9E6_9RHOB\nFES\tA0A165X183_9RHOB\nFES\tA0A171D9Y4_KLEOX\nFES\tA0A1A7W6D1_9TELE\nFES\tA0A1A7WZU7_9TELE\nFES\tA0A1A7ZCX5_NOTFU\nFES\tA0A1A8A373_NOTFU\nFES\tA0A1A8C501_9TELE\nFES\tA0A1A8DFK3_9TELE\nFES\tA0A1A8HVS6_NOTKU\nFES\tA0A1A8L6Y9_9TELE\nFES\tA0A1A8NRB4_9TELE\nFES\tA0A1A8PIF5_9TELE\nFES\tA0A1A8PV86_9TELE\nFES\tA0A1B1KUE0_SERPL\nFES\tA0A1B8YJD9_9GAMM\nFES\tA0A1B9SYC6_RHIRD\nFES\tA0A1C0TYW6_9GAMM\nFES\tA0A1C3JVR2_9GAMM\nFES\tA0A1C3PXB0_KLEPN\nFES\tA0A1D5PZX7_CHICK\nFES\tA0A1D5QQF2_MACMU\nFES\tA0A1F2PTF8_RHOER\nFES\tA0A1G4FH61_9FIRM\nFES\tA0A1L8H0V7_XENLA\nFES\tA0A1M0D5H7_ECOLX\nFES\tA0A1M2U121_ECOLX\nFES\tA0A1M6AL15_9VIBR\nFES\tA0A1M7YXD4_9VIBR\nFES\tA0A1N6M7U3_9VIBR\nFES\tA0A1Q9NKG8_9ARCH\nFES\tA0A1R2KMG5_SALEN\nFES\tA0A1R4B8E0_9VIBR\nFES\tA0A1R4LSH4_9VIBR\nFES\tA0A1S3AHM6_ERIEU\nFES\tA0A1S3AJ25_ERIEU\nFES\tA0A1S3AKH6_ERIEU\nFES\tA0A1S3ALX8_ERIEU\nFES\tA0A1S3ET42_DIPOR\nFES\tA0A1S3ET47_DIPOR\nFES\tA0A1S3EUI0_DIPOR\nFES\tA0A1S3EVX8_DIPOR\nFES\tA0A1S3WBX3_ERIEU\nFES\tA0A1S3WC08_ERIEU\nFES\tA0A1U7QRS8_MESAU\nFES\tA0A1U7S460_ALLSI\nFES\tA0A1U7SKP3_TARSY\nFES\tA0A1U7SSB8_TARSY\nFES\tA0A1U7SW95_TARSY\nFES\tA0A1U7T3I7_TARSY\nFES\tA0A1U7T684_TARSY\nFES\tA0A1V4JDJ0_PATFA\nFES\tA0A1V6B9T2_9BACT\nFES\tA0A1V6HDZ5_9BACT\nFES\tA0A1W5DN78_9GAMM\nFES\tA0A1X0YX72_ECOLX\nFES\tA0A1Y5S5U8_9RHOB\nFES\tA0A1Y6AJG8_BACCE\nFES\tA0A1Y6AVL7_BACCE\nFES\tA0A1Y6B0Y4_BACCE\nFES\tA0A1Y6B1W8_BACCE\nFES\tA0A1Y6IUE7_9VIBR\nFES\tA0A1Z1WPZ4_9ACTN\nFES\tA0A1Z1WQE0_9ACTN\nFES\tA0A1Z1WRS9_9ACTN\nFES\tA0A1Z1WRX6_9ACTN\nFES\tA0A1Z3UTQ2_ECOLX\nFES\tA0A210GJW0_ECOLX\nFES\tA0A210VWW9_9BURK\nFES\tA0A212I5P4_9ENTR\nFES\tA0A237P3Q3_ECOLX\nFES\tA0A239SHI3_9BURK\nFES\tA0A239V3V2_9MICO\nFES\tA0A265B6Y6_SALET\nFES\tA0A285B0L2_9ENTR\nFES\tA0A290TAB7_9GAMM\nFES\tA0A2A3W3F3_ECOLX\nFES\tA0A2A3WU93_ECOLX\nFES\tA0A2B7MIC6_9ESCH\nFES\tA0A2D0NPJ7_ECOLX\nFES\tA0A2D0RTQ1_ICTPU\nFES\tA0A2G8ZZB5_ECOLX\nFES\tA0A2H4TPE2_ECOLX\nFES\tA0A2H5C9R7_ECOLX\nFES\tA0A2I1X0A3_CORAY\nFES\tA0A2I2U194_FELCA\nFES\tA0A2I2V228_FELCA\nFES\tA0A2I3H4F1_NOMLE\nFES\tA0A2I3HNA4_NOMLE\nFES\tA0A2I3MLY7_PAPAN\nFES\tA0A2I3RDX1_PANTR\nFES\tA0A2I3SF41_PANTR\nFES\tA0A2I3STZ1_PANTR\nFES\tA0A2I4CMP1_9TELE\nFES\tA0A2I9DME2_9NOCA\nFES\tA0A2J8VXG9_PONAB\nFES\tA0A2K5F199_AOTNA\nFES\tA0A2K5F1D2_AOTNA\nFES\tA0A2K5F1H3_AOTNA\nFES\tA0A2K5K561_COLAP\nFES\tA0A2K5K597_COLAP\nFES\tA0A2K5K5B3_COLAP\nFES\tA0A2K5K5B8_COLAP\nFES\tA0A2K5LPP9_CERAT\nFES\tA0A2K5LPT4_CERAT\nFES\tA0A2K5LPT5_CERAT\nFES\tA0A2K5LPW0_CERAT\nFES\tA0A2K5LPW2_CERAT\nFES\tA0A2K5LPW9_CERAT\nFES\tA0A2K5LPX7_CERAT\nFES\tA0A2K5S734_CEBCA\nFES\tA0A2K5S739_CEBCA\nFES\tA0A2K5S746_CEBCA\nFES\tA0A2K5VV58_MACFA\nFES\tA0A2K5VVA5_MACFA\nFES\tA0A2K5VVC8_MACFA\nFES\tA0A2K5VVF8_MACFA\nFES\tA0A2K5VVG5_MACFA\nFES\tA0A2K5Y0F5_MANLE\nFES\tA0A2K5Y0H5_MANLE\nFES\tA0A2K5Y0I5_MANLE\nFES\tA0A2K5Y0J7_MANLE\nFES\tA0A2K6DRL1_MACNE\nFES\tA0A2K6DRP0_MACNE\nFES\tA0A2K6DRP3_MACNE\nFES\tA0A2K6DRV0_MACNE\nFES\tA0A2K6DRW1_MACNE\nFES\tA0A2K6GYN7_PROCO\nFES\tA0A2K6GYS0_PROCO\nFES\tA0A2K6GYV3_PROCO\nFES\tA0A2K6LDB2_RHIBE\nFES\tA0A2K6LDC6_RHIBE\nFES\tA0A2K6LDF0_RHIBE\nFES\tA0A2K6LDF2_RHIBE\nFES\tA0A2K6LDF8_RHIBE\nFES\tA0A2K6R316_RHIRO\nFES\tA0A2K6R325_RHIRO\nFES\tA0A2K6R331_RHIRO\nFES\tA0A2K6R332_RHIRO\nFES\tA0A2K6R344_RHIRO\nFES\tA0A2K6U797_SAIBB\nFES\tA0A2K6U7A0_SAIBB\nFES\tA0A2K6U7A9_SAIBB\nFES\tA0A2K6U7E0_SAIBB\nFES\tA0A2K8QXW5_9ACTN\nFES\tA0A2L1C1S6_KLEPN\nFES\tA0A2L2LCY7_RHIRD\nFES\tA0A2N9B2C7_STRCX\nFES\tA0A2P4UFC1_9ACTN\nFES\tA0A2P8AGU2_9ACTN\nFES\tA0A2R8Q7T0_DANRE\nFES\tA0A2R9B472_PANPA\nFES\tA0A2R9B5Q6_PANPA\nFES\tA0A2R9B814_PANPA\nFES\tA0A2R9BB03_PANPA\nFES\tA0A2R9BB16_PANPA\nFES\tA0A2S1CTA1_ENTAG\nFES\tA0A2S1CTA2_ENTAG\nFES\tA0A2S1CTC9_9GAMM\nFES\tA0A2S1CTG8_9GAMM\nFES\tA0A2S1CTH0_9GAMM\nFES\tA0A2S1CTI4_9GAMM\nFES\tA0A2S1CTL3_9GAMM\nFES\tA0A2S1CTL4_9GAMM\nFES\tA0A2S2PFD6_SCHGA\nFES\tA0A2S6XGP2_9ACTN\nFES\tA0A2S8DDQ7_SHIDY\nFES\tA0A2S8DT68_SHIBO\nFES\tA0A2S8JNR8_ECOLX\nFES\tA0A2S9XP20_9BACI\nFES\tA0A2S9YCX7_9DELT\nFES\tA0A2S9YHL1_9DELT\nFES\tA0A2U3VSP0_ODORO\nFES\tA0A2U3VSQ7_ODORO\nFES\tA0A2U3VSV6_ODORO\nFES\tA0A2U3VSY8_ODORO\nFES\tA0A2U4C7H5_TURTR\nFES\tA0A2V2LR55_ECOLX\nFES\tA0A2X0U331_9ACTO\nFES\tA0A2X1NE93_ECOLX\nFES\tA0A2X1QIX0_KLEPN\nFES\tA0A2X1TU69_MORMO\nFES\tA0A2X2C8K0_PROMI\nFES\tA0A2X2DL27_PRORE\nFES\tA0A2X2GBX3_9GAMM\nFES\tA0A2X2K095_SHIDY\nFES\tA0A2X3DTN6_KLUCR\nFES\tA0A2X3E111_9MICC\nFES\tA0A2X3EYS5_KLEPN\nFES\tA0A2X3JAF8_9ENTR\nFES\tA0A2X4TAG7_CORJE\nFES\tA0A2X4UWN1_SERPL\nFES\tA0A2X5NF38_9GAMM\nFES\tA0A2X6CDN0_ECOLX\nFES\tA0A2X7IQ42_ECOLX\nFES\tA0A2X7RPT9_ECOLX\nFES\tA0A2Y0XL29_ECOLX\nFES\tA0A2Y1M0X8_STAAU\nFES\tA0A2Y2V290_SHIFL\nFES\tA0A2Y9ESW1_PHYMC\nFES\tA0A2Y9EUU2_PHYMC\nFES\tA0A2Y9GFQ6_NEOSC\nFES\tA0A2Y9MFF9_DELLE\nFES\tA0A2Z2NIB5_9GAMM\nFES\tA0A315GSI7_SALET\nFES\tA0A330QDP8_KLEPN\nFES\tA0A331MVJ4_KLEPN\nFES\tA0A334NI72_ACIBA\nFES\tA0A340YA79_LIPVE\nFES\tA0A340YC35_LIPVE\nFES\tA0A340YDV5_LIPVE\nFES\tA0A340YF16_LIPVE\nFES\tA0A340YF21_LIPVE\nFES\tA0A340YGN5_LIPVE\nFES\tA0A340YGP0_LIPVE\nFES\tA0A341D5I6_NEOAA\nFES\tA0A341D5J6_NEOAA\nFES\tA0A341D6D2_NEOAA\nFES\tA0A341D8H7_NEOAA\nFES\tA0A376E548_9ENTR\nFES\tA0A376ESB4_9MICO\nFES\tA0A376FQR8_ECOLX\nFES\tA0A376HTU5_ECOLX\nFES\tA0A376I454_ECOLX\nFES\tA0A376JAT3_ECOLX\nFES\tA0A376K1R2_ECOLX\nFES\tA0A376KA96_ECOLX\nFES\tA0A376LJN5_ECOLX\nFES\tA0A376MT57_ECOLX\nFES\tA0A376P1W5_ECOLX\nFES\tA0A376PYZ0_ECOLX\nFES\tA0A376QYT7_ECOLX\nFES\tA0A376UEN8_ECOLX\nFES\tA0A376V6N0_ECOLX\nFES\tA0A376ZPZ8_ECOLX\nFES\tA0A377C1B2_ECOLX\nFES\tA0A377DIW6_ECOLX\nFES\tA0A377F850_ECOLX\nFES\tA0A377GHJ6_9GAMM\nFES\tA0A377M723_ENTCL\nFES\tA0A377N1E6_ECOLX\nFES\tA0A377NVE4_ECOLX\nFES\tA0A377R9C4_KLEAE\nFES\tA0A377SJG0_KLEOX\nFES\tA0A377STP7_9NEIS\nFES\tA0A377TN04_KLEPN\nFES\tA0A377WVY0_KLEPN\nFES\tA0A377XY60_KLEPN\nFES\tA0A377Z8H9_KLEPO\nFES\tA0A377Z9K1_KLEPN\nFES\tA0A377ZVV7_KLEPN\nFES\tA0A378A5V8_KLEPN\nFES\tA0A378CD22_KLEPN\nFES\tA0A378DBW8_KLEOX\nFES\tA0A378E008_KLEOX\nFES\tA0A378GNG2_9ENTR\nFES\tA0A378I2I7_9GAMM\nFES\tA0A378MP19_VIBAN\nFES\tA0A378N803_MANHA\nFES\tA0A378PLZ5_9GAMM\nFES\tA0A378QUV7_9GAMM\nFES\tA0A378TQA8_MORLA\nFES\tA0A378Y5L2_PAEPO\nFES\tA0A379ALN4_ENTAG\nFES\tA0A379EQG4_9GAMM\nFES\tA0A379FAG6_PROVU\nFES\tA0A379FM40_PRORE\nFES\tA0A379GC00_PROMI\nFES\tA0A379GWF7_PROST\nFES\tA0A379HH49_PROST\nFES\tA0A379YIE9_9GAMM\nFES\tA0A380ADU5_9GAMM\nFES\tA0A380B935_SHIFL\nFES\tA0A380PU64_YERFR\nFES\tA0A380Q1N1_VIBME\nFES\tA0A380SX05_9PSED\nFES\tA0A381CD79_9ENTR\nFES\tA0A381EVR0_CHRVL\nFES\tA0A381FRG6_CHRVL\nFES\tA0A381GQR4_CITFR\nFES\tA0A381KEN9_CORJE\nFES\tA0A383S869_9ACTN\nFES\tA0A384B3R6_BALAS\nFES\tA0A384B415_BALAS\nFES\tA0A384B481_BALAS\nFES\tA0A384BU14_URSMA\nFES\tA0A384ISG3_KLEOX\nFES\tA0A385Y0I9_9GAMM\nFES\tA0A399H3X0_9ACTN\nFES\tA0A3E2YUG0_9ACTN\nFES\tA0A3G2CQR0_9RHIZ\nFES\tA0A3G6IW73_9CORY\nFES\tA0A3G6J5B7_9CORY\nFES\tA0A3G6JBI5_9CORY\nFES\tA0A3N6FEK9_9ACTN\nFES\tA0A3N6FIT8_9ACTN\nFES\tA0A3N6FJY0_9ACTN\nFES\tA0A3N6IJC3_9ACTN\nFES\tA0A3P4ANX3_THETH\nFES\tA0A3P4I1K8_KLEPN\nFES\tA0A3P6JN33_9CLOT\nFES\tA0A3P8LGY0_TSUPA\nFES\tA0A3Q0D447_MESAU\nFES\tA0A3Q0H864_ALLSI\nFES\tA0A3Q7QQH6_CALUR\nFES\tA0A3Q7TN51_VULVU\nFES\tA0A3Q7XIU0_URSAR\nFES\tA0A3S4G5L8_SERPL\nFES\tA0A3S4H279_MORMO\nFES\tA0A3S4IG37_CHRVL\nFES\tA0A3S4IVM9_ECOLX\nFES\tA0A3S4JMV6_KLEAE\nFES\tA0A3S4L774_9GAMM\nFES\tA0A3S4MZC5_ECOLX\nFES\tA0A3S4X8R1_SERFO\nFES\tA0A3S4Y7Y5_9ACTN\nFES\tA0A3S4YHI7_9MICC\nFES\tA0A3S4YXS2_9CORY\nFES\tA0A3S4YZ12_9ACTN\nFES\tA0A3S4Z603_9MICC\nFES\tA0A3S5BC80_MANHA\nFES\tA0A3T1B645_9ACTN\nFES\tA0A3T2YG21_SALET\nFES\tA0A3T3IL59_SALDU\nFES\tA0A3V2G1K0_SALET\nFES\tA0A3V2T3M3_SALET\nFES\tA0A3V4SP38_SALET\nFES\tA0A3V5Z9W4_SALET\nFES\tA0A3V7IP03_SALRU\nFES\tA0A3V7KM26_SALET\nFES\tA0A3V8D2P4_SALET\nFES\tA0A3V8VKX4_SALET\nFES\tA0A3V9PY22_SALET\nFES\tA0A3W0FF17_SALET\nFES\tA0A3W0KBY5_SALON\nFES\tA0A3Y8PFE7_SALVI\nFES\tA0A3Z6VV93_SALET\nFES\tA0A401N0W2_RHOER\nFES\tA0A401YE62_9ACTN\nFES\tA0A401ZEE7_9CHLR\nFES\tA0A401ZWF7_9CHLR\nFES\tA0A402B6D3_9CHLR\nFES\tA0A410RKF7_CORCK\nFES\tA0A410WAZ2_9CORY\nFES\tA0A445NHA7_STRNE\nFES\tA0A446CQE9_9BURK\nFES\tA0A446CQL6_9BURK\nFES\tA0A446ZJN4_ACICA\nFES\tA0A447KU41_SEROD\nFES\tA0A447LRT6_ATLHE\nFES\tA0A447MJV4_KLUIN\nFES\tA0A448BKQ8_PSEFL\nFES\tA0A448JPL1_9BURK\nFES\tA0A448PIT2_ACTVI\nFES\tA0A449C7Z8_ECOLX\nFES\tA0A452EZ52_CAPHI\nFES\tA0A452S2Z4_URSAM\nFES\tA0A452S303_URSAM\nFES\tA0A452S311_URSAM\nFES\tA0A452S316_URSAM\nFES\tA0A452S317_URSAM\nFES\tA0A452S324_URSAM\nFES\tA0A452TUN7_URSMA\nFES\tA0A452TUP8_URSMA\nFES\tA0A452TUW1_URSMA\nFES\tA0A452TUX3_URSMA\nFES\tA0A452TV07_URSMA\nFES\tA0A455RVS9_SALET\nFES\tA0A455T6H1_9CHLR\nFES\tA0A483J537_KLEPN\nFES\tA0A483L6Y0_KLEPN\nFES\tA0A483M6Y9_KLEPN\nFES\tA0A483RMJ5_KLEPN\nFES\tA0A484YYN0_SERLI\nFES\tA0A485BH74_RAOTE\nFES\tA0A485CJC3_KLUCR\nFES\tA0A485EAJ6_PSEAI\nFES\tA0A485EHP1_PSEAI\nFES\tA0A485FLW4_PSEAI\nFES\tA0A485IP93_PSEAI\nFES\tA0A485IVZ2_PSEAI\nFES\tA0A485JLT6_ECOLX\nFES\tA0A485JSJ3_KLEAE\nFES\tA0A485SYE1_KLEPN\nFES\tA0A485TZ23_KLEPN\nFES\tA0A485VX55_KLEPN\nFES\tA0A485WBV5_KLEPN\nFES\tA0A485XQN4_KLEPN\nFES\tA0A485XUW7_KLEPN\nFES\tA0A485YI38_KLEPN\nFES\tA0A485Z466_KLEPN\nFES\tA0A486CJP4_KLEPN\nFES\tA0A486DRU0_KLEPN\nFES\tA0A486EE56_KLEPN\nFES\tA0A486F9C5_KLEPN\nFES\tA0A486FG32_KLEPN\nFES\tA0A486INW1_KLEPN\nFES\tA0A486J0I9_KLEPN\nFES\tA0A486QSY8_KLEPN\nFES\tA0A486RMI1_KLEPN\nFES\tA0A486S9V2_KLEPN\nFES\tA0A486T5V0_KLEPN\nFES\tA0A486T887_KLEPN\nFES\tA0A493T0D7_ANAPP\nFES\tA0A493T3T1_ANAPP\nFES\tA0A493TNI6_ANAPP\nFES\tA0A4C1SKB4_9NEOP\nFES\tA0A4C4JY99_ECOLX\nFES\tA0A4C6WV89_ECOLX\nFES\tA0A4C9BIT4_ECOLX\nFES\tA0A4D0XLK4_ECOLX\nFES\tA0A4P1KJN3_KLEPN\nFES\tA0A4S3IJL3_SALET\nFES\tA0A4S3N4A2_SHIFL\nFES\tA0A4U8J1U6_SALET\nFES\tA0A4U9ATF6_KLEPN\nFES\tA0A4U9DYX2_KLEPN\nFES\tA0A4U9J3C0_YEREN\nFES\tA0A4U9M5K8_STREE\nFES\tA0A4U9N5F2_ECOLX\nFES\tA0A4U9P178_CAMJU\nFES\tA0A4U9S3J5_PSEAI\nFES\tA0A4U9TDF8_SERFO\nFES\tA0A4U9U1M6_ECOLX\nFES\tA0A4U9WHL5_ECOLX\nFES\tA0A4V0B0X5_STREQ\nFES\tA0A4V0HB41_ECOLX\nFES\tA0A4V6ING5_KLEPN\nFES\tA0A4V6JHP3_9ENTR\nFES\tA0A4W2CPP0_BOBOX\nFES\tA0A4W2DZ58_BOBOX\nFES\tA0A4W2GAB0_BOBOX\nFES\tA0A4W3HVW9_CALMI\nFES\tA0A4W3HYY1_CALMI\nFES\tA0A4W3I3D6_CALMI\nFES\tA0A4W3ILD9_CALMI\nFES\tA0A4X1UBC9_PIG\nFES\tA0A4X1UGB8_PIG\nFES\tA0A4X1UGS1_PIG\nFES\tA0A4X2K9J8_VOMUR\nFES\tA0A4X2K9M3_VOMUR\nFES\tA0A4X2K9P1_VOMUR\nFES\tA0A4Y1W3X9_BACAN\nFES\tA0A4Y6MGJ2_SALET\nFES\tA0A508UCN7_9GAMM\nFES\tA0A509G4D0_PAEPO\nFES\tA0A510ICE9_9VIBR\nFES\tA0A511M5H6_9NOCA\nFES\tA0A511T6A0_MYXFU\nFES\tA0A511Z032_9CELL\nFES\tA0A514EV23_9ENTR\nFES\tA0A515BVD9_PSEAI\nFES\tA0A517M4M1_9BACT\nFES\tA0A517NTY3_9BACT\nFES\tA0A517PIF6_9BACT\nFES\tA0A517Q230_9BACT\nFES\tA0A517S921_9BACT\nFES\tA0A517VYZ4_9BACT\nFES\tA0A517WVX8_9BACT\nFES\tA0A518A1N0_9BACT\nFES\tA0A518B5U6_9BACT\nFES\tA0A518FPI0_9BACT\nFES\tA0A518IY10_9BACT\nFES\tA0A518J483_MARAM\nFES\tA0A518J4B2_MARAM\nFES\tA0A518J4C3_MARAM\nFES\tA0A518J4C4_MARAM\nFES\tA0A518JWT9_9BACT\nFES\tA0A564SL01_ESCFE\nFES\tA0A5A5T5A2_9CHLR\nFES\tA0A5A5YR39_ECOLX\nFES\tA0A5A6F8B7_ECOLX\nFES\tA0A5A6TRR7_ECOLX\nFES\tA0A5A6XUK2_ECOLX\nFES\tA0A5A7E507_ECOLX\nFES\tA0A5A7MCZ8_COMTE\nFES\tA0A5A8XYV2_9ENTR\nFES\tA0A5A9A1B5_ENTAS\nFES\tA0A5A9ASZ7_9ENTR\nFES\tA0A5A9BAS7_9ENTR\nFES\tA0A5A9BN70_ENTAS\nFES\tA0A5A9BVG2_9ENTR\nFES\tA0A5A9C664_9ENTR\nFES\tA0A5A9CLG4_9ENTR\nFES\tA0A5A9DNX2_CITBR\nFES\tA0A5A9EHJ7_9ENTR\nFES\tA0A5A9JJ73_ECOLX\nFES\tA0A5A9M505_ECOLX\nFES\tA0A5A9MDS6_ECOLX\nFES\tA0A5B0K3X7_9ENTR\nFES\tA0A5B0T3D1_9ENTR\nFES\tA0A5B0T3M2_9ENTR\nFES\tA0A5B1GH68_ECOLX\nFES\tA0A5B2KIG2_KLEVA\nFES\tA0A5B2Q5X8_KLEPN\nFES\tA0A5B7V6F5_9ACTN\nFES\tA0A5B9SWX5_ECOLX\nFES\tA0A5C0FT70_ECOLX\nFES\tA0A5C0H879_ECOLX\nFES\tA0A5C0HNH4_ECOLX\nFES\tA0A5C1CWD7_9ENTR\nFES\tA0A5C1XLH9_9ENTR\nFES\tA0A5C2EB83_SHIFL\nFES\tA0A5C2M3I8_SALER\nFES\tA0A5C2PWJ8_9ENTR\nFES\tA0A5C5UW79_9BACT\nFES\tA0A5C5W7V8_9BACT\nFES\tA0A5C6DHT1_9BACT\nFES\tA0A5C9AER5_ECOLX\nFES\tA0A5C9ANM4_ECOLX\nFES\tA0A5D0NNF2_9ACTN\nFES\tA0A5D3R174_SALTM\nFES\tA0A5D4G4S5_ENTCL\nFES\tA0A5D4JS51_9ACTN\nFES\tA0A5D4JXT3_9ENTR\nFES\tA0A5D4L580_KLEPN\nFES\tA0A5D4LLN9_CITFR\nFES\tA0A5D4NHG8_9ENTR\nFES\tA0A5D4PPH1_ECOLX\nFES\tA0A5D5AYN0_ECOLX\nFES\tA0A5D6A2F2_KLEPN\nFES\tA0A5D6ADD6_KLEPN\nFES\tA0A5D6C6Z7_ECOLX\nFES\tA0A5D6D2J3_KLEPN\nFES\tA0A5D6LZH7_KLEPN\nFES\tA0A5D6MM69_ECOLX\nFES\tA0A5D6Q9K8_KLEPN\nFES\tA0A5D9JFE7_ENTCL\nFES\tA0A5E0FMQ3_ENTCL\nFES\tA0A5E0PDI1_ENTCL\nFES\tA0A5E0RSH4_ENTCL\nFES\tA0A5E1A3W3_9ENTR\nFES\tA0A5E1CUV8_ENTCL\nFES\tA0A5E1DW20_KLEPN\nFES\tA0A5E1J1H1_KLEPN\nFES\tA0A5E1M9R4_KLEPN\nFES\tA0A5E1R2R4_KLEPN\nFES\tA0A5E1RVH6_KLEPN\nFES\tA0A5E1SNG9_KLEPN\nFES\tA0A5E1T7J1_KLEPN\nFES\tA0A5E1YGZ2_KLEPN\nFES\tA0A5E2B7H2_KLEPN\nFES\tA0A5E2D0F1_KLEPN\nFES\tA0A5E2DP28_KLEPN\nFES\tA0A5E2FW18_KLEPN\nFES\tA0A5E2GNT9_KLEPN\nFES\tA0A5E2UX35_KLEPN\nFES\tA0A5E2XJY3_KLEPN\nFES\tA0A5E3AJ51_KLEPN\nFES\tA0A5E3DIF5_KLEPN\nFES\tA0A5E3RAV3_KLEPN\nFES\tA0A5E3VEP4_ENTCL\nFES\tA0A5E3W2F3_ENTCL\nFES\tA0A5E4S0W6_9BURK\nFES\tA0A5E4VK92_9BURK\nFES\tA0A5E4X1T5_9BURK\nFES\tA0A5E5QUT1_PSEAI\nFES\tA0A5E5TDX5_9ENTR\nFES\tA0A5E5TKG6_KLEPN\nFES\tA0A5E5X493_KLEPN\nFES\tA0A5E6ALC2_KLEPN\nFES\tA0A5E6CCP9_KLEPN\nFES\tA0A5E6HKS0_KLEPN\nFES\tA0A5E8G6F0_ECOLX\nFES\tA0A5E8IIU5_ECOLX\nFES\tA0A5E8IX73_ECOLX\nFES\tA0A5F4BRI6_CANLF\nFES\tA0A5F4BU25_CANLF\nFES\tA0A5F5PNK4_HORSE\nFES\tA0A5F5PW05_HORSE\nFES\tA0A5F5XDH2_FELCA\nFES\tA0A5F5XGS8_FELCA\nFES\tA0A5F5Y5K8_FELCA\nFES\tA0A5F8GAV8_MONDO\nFES\tA0A5F8GV24_MONDO\nFES\tA0A5F9CIX2_RABIT\nFES\tA0A5F9D844_RABIT\nFES\tA0A5F9DNM8_RABIT\nFES\tA0A5G2R173_PIG\nFES\tA0A5G2R861_PIG\nFES\tA0A5G3I7Z6_XENTR\nFES\tA0A5G3I834_XENTR\nFES\tA0A5G3I8C3_XENTR\nFES\tA0A5G3I8F6_XENTR\nFES\tA0A5G3I9F9_XENTR\nFES\tA0A5G3JTU5_XENTR\nFES\tA0A5H2ZYG3_9ALTE\nFES\tA0A5H8SU42_SALET\nFES\tA0A5H9D252_SALET\nFES\tA0A5H9DRE1_SALMS\nFES\tA0A5I3GEZ5_SALET\nFES\tA0A5I6FKG5_SALON\nFES\tA0A5I7PED0_SALET\nFES\tA0A5I8H6C2_SALRU\nFES\tA0A5I9F9M4_SALET\nFES\tA0A5I9QPY2_SALPT\nFES\tA0A5I9SA69_SALET\nFES\tA0A5I9VEX8_SALET\nFES\tA0A5I9VUZ4_SALET\nFES\tA0A5I9X877_SALET\nFES\tA0A5J0A2M9_SALET\nFES\tA0A5J0AFX2_SALET\nFES\tA0A5J0BZJ0_SALET\nFES\tA0A5J0C4V1_SALET\nFES\tA0A5J0CC78_SALET\nFES\tA0A5J0DUR0_SALAB\nFES\tA0A5J0EG83_SALET\nFES\tA0A5J0F5J2_SALET\nFES\tA0A5J0GAR8_SALET\nFES\tA0A5J0GEE3_SALET\nFES\tA0A5J0HK89_SALET\nFES\tA0A5J0HT91_SALET\nFES\tA0A5J0IG07_SALET\nFES\tA0A5J0ILZ5_SALET\nFES\tA0A5J0JQR3_SALET\nFES\tA0A5J0L0T3_SALET\nFES\tA0A5J0LVC2_SALET\nFES\tA0A5J0NKZ2_SALET\nFES\tA0A5J0NQT3_SALET\nFES\tA0A5J0NX96_SALET\nFES\tA0A5J0Q443_SALET\nFES\tA0A5J0QHY7_SALET\nFES\tA0A5J0RHQ7_SALET\nFES\tA0A5J0RL73_SALET\nFES\tA0A5J0SBA3_SALET\nFES\tA0A5J0TJE0_SALET\nFES\tA0A5J0UAI0_SALVI\nFES\tA0A5J0VAR9_SALET\nFES\tA0A5J0VVS9_SALET\nFES\tA0A5J0WMA7_SALET\nFES\tA0A5J0YMK5_SALET\nFES\tA0A5J0ZVN1_SALET\nFES\tA0A5J1ATB8_SALMS\nFES\tA0A5J1B6N5_SALET\nFES\tA0A5J1BWV6_SALET\nFES\tA0A5J1CIU8_SALET\nFES\tA0A5J1INJ3_SALET\nFES\tA0A5J1IQW6_SALET\nFES\tA0A5J1J5Z9_SALET\nFES\tA0A5J1MBK4_SALET\nFES\tA0A5J1RMR2_SALET\nFES\tA0A5J1SQ86_SALET\nFES\tA0A5J1TW80_SALET\nFES\tA0A5J1WUN3_SALET\nFES\tA0A5J4K6J1_9CHLR\nFES\tA0A5J4KPD6_9CHLR\nFES\tA1JQM9_YERE8\nFES\tA6T632_KLEP7\nFES\tA7ZIW8_ECO24\nFES\tA9J3Z3_URSMA\nFES\tA9J3Z5_URSAR\nFES\tA9J3Z9_HELMA\nFES\tA9J400_MELUS\nFES\tA9J402_URSTH\nFES\tA9J404_TREOR\nFES\tA9J406_AILME\nFES\tA9XJT6_9CARN\nFES\tA9XJT7_9CARN\nFES\tA9XJT8_9CARN\nFES\tB1EKE8_ESCAT\nFES\tB1LKH7_ECOSM\nFES\tB2GV16_RAT\nFES\tB2TTE3_SHIB3\nFES\tB3FG11_9CARN\nFES\tB3FG12_MUSEV\nFES\tB3FG13_MUSLU\nFES\tB3FG14_MUSNG\nFES\tB3FG15_MUSNI\nFES\tB3FG16_MUSPU\nFES\tB3FG17_MUSSB\nFES\tB3FG18_9CARN\nFES\tB3FG19_MARFA\nFES\tB3FG20_MARFO\nFES\tB3FG21_MARMT\nFES\tB3FG22_MARME\nFES\tB3FG23_MARZI\nFES\tB3FG24_9CARN\nFES\tB3FG25_MELCA\nFES\tB5R708_SALG2\nFES\tB5XZY8_KLEP3\nFES\tB7L8G7_ECO55\nFES\tB7LLL6_ESCF3\nFES\tB7MEZ6_ECO45\nFES\tB7MRM1_ECO81\nFES\tB7N9J1_ECOLU\nFES\tB7UKM8_ECO27\nFES\tB8X3C6_GALVR\nFES\tB8X3C9_GALVR\nFES\tB8X3D2_CYNVO\nFES\tC0PVN3_SALPC\nFES\tC0SQJ7_ZALCA\nFES\tC7BJQ6_PHOAA\nFES\tC8UJQ3_ECO1A\nFES\tC9J3W6_HUMAN\nFES\tC9XYM8_CROTZ\nFES\tD2A984_SHIF2\nFES\tD2EB75_THESC\nFES\tD2JZK7_PHOVI\nFES\tD2JZK9_PUSCA\nFES\tD2JZL0_PHOFA\nFES\tD2TMM5_CITRI\nFES\tD3GWC4_ECO44\nFES\tD3K2Y8_ECOLX\nFES\tD3QLM7_ECOCB\nFES\tD3V0J8_XENBS\nFES\tD4A584_RAT\nFES\tD4E3C4_SEROD\nFES\tD5AR04_RHOCB\nFES\tD6MZY4_PUSSI\nFES\tD6MZZ2_HYDLE\nFES\tE0J1Z0_ECOLW\nFES\tE1SC73_PANVC\nFES\tE7EMJ7_HUMAN\nFES\tE7ENM8_HUMAN\nFES\tE8X957_SALT4\nFES\tE9PGC7_HUMAN\nFES\tE9PIJ7_HUMAN\nFES\tF0JM71_ESCFE\nFES\tF1NXT2_CHICK\nFES\tF1PUK7_CANLF\nFES\tF1RMJ0_PIG\nFES\tF4N322_YEREN\nFES\tF4NMX6_ECOLX\nFES\tF5RUE4_9ENTR\nFES\tF6RPL0_XENTR\nFES\tF6RWR2_MONDO\nFES\tF6XQG4_HORSE\nFES\tF6Y8A0_HORSE\nFES\tF7B8W2_XENTR\nFES\tF7BVM9_XENTR\nFES\tF7IBY8_CALJA\nFES\tF7IC12_CALJA\nFES\tF7IEC5_CALJA\nFES\tF8W425_DANRE\nFES\tF8W5D1_DANRE\nFES\tFES_DICD3\nFES\tFES_ECOLI\nFES\tFES_FELCA\nFES\tFES_HUMAN\nFES\tFES_MOUSE\nFES\tFES_OCIBA\nFES\tFES_YEREN\nFES\tG0HGL0_CORVD\nFES\tG1M5C6_AILME\nFES\tG1NDE5_MELGA\nFES\tG1NDE7_MELGA\nFES\tG1RNK7_NOMLE\nFES\tG1SX69_RABIT\nFES\tG3SX96_LOXAF\nFES\tG3W6K1_SARHA\nFES\tG7MVY9_MACMU\nFES\tG8LDL2_9ENTR\nFES\tG8PG30_PSEUV\nFES\tH0W9I7_CAVPO\nFES\tH0WSP6_OTOGA\nFES\tH0YNN8_HUMAN\nFES\tH0YNT6_HUMAN\nFES\tH0ZIM7_TAEGU\nFES\tH1XUH4_9BACT\nFES\tH2LZI2_ORYLA\nFES\tH2QA35_PANTR\nFES\tH2VAN4_TAKRU\nFES\tH3B7G3_LATCH\nFES\tH4I7M9_ECOLX\nFES\tH4IN56_ECOLX\nFES\tH4J4U3_ECOLX\nFES\tH4KD71_ECOLX\nFES\tH4L7R0_ECOLX\nFES\tH4UFQ3_ECOLX\nFES\tH5V2Q9_ATLHE\nFES\tH8MIP1_CORCM\nFES\tH9FAE2_MACMU\nFES\tH9FHS3_MACMU\nFES\tH9FKB2_MACMU\nFES\tH9G971_ANOCA\nFES\tH9H312_MACMU\nFES\tI2B7I0_SHIBC\nFES\tI2FIC0_9CARN\nFES\tI2FIC9_9CARN\nFES\tI2FIE2_MUSAL\nFES\tI2FIE6_MUSIT\nFES\tI2FIF0_MUSKA\nFES\tI2FIF3_9CARN\nFES\tI2FIF5_MYDJA\nFES\tI2FIG8_VORPE\nFES\tI2RAJ3_ECOLX\nFES\tI2RQ45_ECOLX\nFES\tI2T032_ECOLX\nFES\tI2ULT1_ECOLX\nFES\tI2W578_ECOLX\nFES\tI2WT30_ECOLX\nFES\tI2XG95_ECOLX\nFES\tI3JE74_ORENI\nFES\tI3MNM8_ICTTR\nFES\tI6D478_SHIFL\nFES\tI6ESF1_SHIBO\nFES\tJ5VAP1_9ENTR\nFES\tJ7QJJ4_ECOLX\nFES\tJ9Q1Y8_TURTR\nFES\tK0XAF8_SHIFL\nFES\tK6Y1T5_9ALTE\nFES\tK7CQE2_PANTR\nFES\tM3W596_FELCA\nFES\tM3YW42_MUSPF\nFES\tM9GCM4_ECOLX\nFES\tM9PBZ2_DROME\nFES\tQ09XK1_LYNCA\nFES\tQ09XK2_VULLA\nFES\tQ09XK3_CANLU\nFES\tQ09XK4_CANLF\nFES\tQ09XK5_PHOLR\nFES\tQ09XK6_LEPWE\nFES\tQ09XK7_HALGR\nFES\tQ09XK8_PAGGO\nFES\tQ09XK9_OMMRO\nFES\tQ09XL0_PUSHI\nFES\tQ09XL1_MONMN\nFES\tQ09XL2_NEOSC\nFES\tQ09XL3_MIRAN\nFES\tQ09XL4_LOBCR\nFES\tQ09XL5_ERIBA\nFES\tQ09XL6_CYSCR\nFES\tQ09XL7_9CARN\nFES\tQ09XL8_ARCGZ\nFES\tQ09XL9_EUMJU\nFES\tQ09XM0_OTABY\nFES\tQ09XM1_ARCFO\nFES\tQ09XM2_ARCAU\nFES\tQ09XM3_MUSNI\nFES\tQ09XM4_MUSPF\nFES\tQ09XM5_9CARN\nFES\tQ09XM6_SPIGR\nFES\tQ09XM7_AILFU\nFES\tQ09XM8_POTFL\nFES\tQ09XM9_9CARN\nFES\tQ09XN0_MELUS\nFES\tQ09XN1_HELMA\nFES\tQ09XN2_URSAM\nFES\tQ09XN3_URSMA\nFES\tQ0IKM8_VIBA7\nFES\tQ0MYM3_VIBAN\nFES\tQ0T737_SHIF8\nFES\tQ1REX8_ECOUT\nFES\tQ24474_DROME\nFES\tQ2VDU6_9CARN\nFES\tQ2VDU7_VULCH\nFES\tQ2VDU8_9CARN\nFES\tQ2VDU9_UROLI\nFES\tQ2VDV0_LYCVE\nFES\tQ2VDV1_9CARN\nFES\tQ2VDV2_LYCCU\nFES\tQ2VDV3_CANSI\nFES\tQ2Y0A3_9CARN\nFES\tQ2Y0A4_MIRAN\nFES\tQ2Y0A5_AILME\nFES\tQ2YF37_URSAM\nFES\tQ324V9_SHIBS\nFES\tQ32IY7_SHIDS\nFES\tQ3YBM7_YERIN\nFES\tQ3YBM8_YERFR\nFES\tQ3YBM9_YEREN\nFES\tQ3YBN0_YEREN\nFES\tQ3Z4K9_SHISS\nFES\tQ3ZV02_CROSK\nFES\tQ4JFK7_HUMAN\nFES\tQ4JFK8_HUMAN\nFES\tQ4JTA2_CORJK\nFES\tQ57RY8_SALCH\nFES\tQ5E9H3_BOVIN\nFES\tQ6GNQ8_XENLA\nFES\tQ7NVW1_CHRVO\nFES\tQ7UJV7_RHOBA\nFES\tQ83WI4_ACIBA\nFES\tQ863Y0_LUTLU\nFES\tQ863Y1_LONLO\nFES\tQ863Y3_LONCN\nFES\tQ8CGG9_MOUSE\nFES\tQ8XBW6_ECO57\nFES\tQ8Z8L7_SALTI\nFES\tQ8ZR39_SALTY\nFES\tQ9VQJ0_DROME\nFES\tS4T660_MUSFR\nFES\tS4T6D1_MUSER\nFES\tU3ID89_ANAPP\nFES\tU3IDA5_ANAPP\nFES\tU3K9G9_FICAL\nFES\tU5QIR7_9CYAN\nFES\tU6DU97_NEOVI\nFES\tV5B4S2_ENTCL\nFES\tV5NTN1_YEREN\nFES\tV6FRF2_ECOLX\nFES\tW0AN69_9ESCH\nFES\tW1J5F8_9GAMM\nFES\tW5PSX7_SHEEP\nFES\tW5UHW8_ICTPU\nFES\tW7PEQ6_9ENTR\nFES\tW8T5X7_ECOLX\nFES\tW9BNM6_KLEPN\nFES\tX5E9Y3_9CORY\n','From\tTo\nLYN\tA0A096MX07_PAPAN\nLYN\tA0A0A1WZM1_ZEUCU\nLYN\tA0A0B8RVQ8_PIG\nLYN\tA0A0D9RQ35_CHLSB\nLYN\tA0A0P6JY89_HETGA\nLYN\tA0A0S7HMW2_9TELE\nLYN\tA0A0S7HMW6_9TELE\nLYN\tA0A0S7HQ69_9TELE\nLYN\tA0A0S7MIH3_9TELE\nLYN\tA0A1A7W8Q5_9TELE\nLYN\tA0A1A7Z7E5_9TELE\nLYN\tA0A1A7ZCG1_NOTFU\nLYN\tA0A1A8BB21_9TELE\nLYN\tA0A1A8DHB3_9TELE\nLYN\tA0A1A8FZ46_9TELE\nLYN\tA0A1A8HVS7_NOTKU\nLYN\tA0A1A8LFE8_9TELE\nLYN\tA0A1A8PL67_9TELE\nLYN\tA0A1A8PLN1_9TELE\nLYN\tA0A1A8PTR5_9TELE\nLYN\tA0A1D5P360_CHICK\nLYN\tA0A1D5QVT2_MACMU\nLYN\tA0A1L8FT48_XENLA\nLYN\tA0A1L8FZ11_XENLA\nLYN\tA0A1S3A9J6_ERIEU\nLYN\tA0A1S3A9N9_ERIEU\nLYN\tA0A1S3EKB5_DIPOR\nLYN\tA0A1S3EMU2_DIPOR\nLYN\tA0A1U7QWZ8_MESAU\nLYN\tA0A1U7SB62_ALLSI\nLYN\tA0A1U7TAU6_TARSY\nLYN\tA0A1U7TEW3_TARSY\nLYN\tA0A1U7TPE7_TARSY\nLYN\tA0A1V4KGA9_PATFA\nLYN\tA0A1V4KI11_PATFA\nLYN\tA0A218VBI1_9PASE\nLYN\tA0A250YI12_CASCN\nLYN\tA0A286YB74_DANRE\nLYN\tA0A286ZMM8_PIG\nLYN\tA0A286ZS09_PIG\nLYN\tA0A287AZU5_PIG\nLYN\tA0A287D7G8_ICTTR\nLYN\tA0A2I0MVY8_COLLI\nLYN\tA0A2I0MVY9_COLLI\nLYN\tA0A2K5CBF8_AOTNA\nLYN\tA0A2K5K1L2_COLAP\nLYN\tA0A2K5LGW3_CERAT\nLYN\tA0A2K5QN97_CEBCA\nLYN\tA0A2K5XDX8_MANLE\nLYN\tA0A2K6DNJ8_MACNE\nLYN\tA0A2K6GF82_PROCO\nLYN\tA0A2K6GF94_PROCO\nLYN\tA0A2K6MLC5_RHIBE\nLYN\tA0A2K6NRF7_RHIRO\nLYN\tA0A2K6TRJ9_SAIBB\nLYN\tA0A2K6TRL3_SAIBB\nLYN\tA0A2R9AHH9_PANPA\nLYN\tA0A2U3WXY8_ODORO\nLYN\tA0A2U3WXZ8_ODORO\nLYN\tA0A2U3ZW16_ODORO\nLYN\tA0A2U4AIS2_TURTR\nLYN\tA0A2U4AIS8_TURTR\nLYN\tA0A2U4AJ04_TURTR\nLYN\tA0A2Y9EH44_PHYMC\nLYN\tA0A2Y9GY53_NEOSC\nLYN\tA0A2Y9GYH1_NEOSC\nLYN\tA0A2Y9LTB7_DELLE\nLYN\tA0A2Y9LTC2_DELLE\nLYN\tA0A2Y9LV36_DELLE\nLYN\tA0A2Y9LZZ2_DELLE\nLYN\tA0A2Y9M5F9_DELLE\nLYN\tA0A340XS03_LIPVE\nLYN\tA0A340XV93_LIPVE\nLYN\tA0A340XXR3_LIPVE\nLYN\tA0A341B6K5_NEOAA\nLYN\tA0A341B6L0_NEOAA\nLYN\tA0A341B8C4_NEOAA\nLYN\tA0A341B9T5_NEOAA\nLYN\tA0A341B9W4_NEOAA\nLYN\tA0A383Z0H9_BALAS\nLYN\tA0A383Z0K5_BALAS\nLYN\tA0A383Z193_BALAS\nLYN\tA0A384DDN0_URSMA\nLYN\tA0A384DDS7_URSMA\nLYN\tA0A3B3SGX8_9TELE\nLYN\tA0A3Q0HHK2_ALLSI\nLYN\tA0A3Q2HZU0_HORSE\nLYN\tA0A3Q2I2V8_HORSE\nLYN\tA0A3Q7NXI9_CALUR\nLYN\tA0A3Q7QE41_CALUR\nLYN\tA0A3Q7TQ84_VULVU\nLYN\tA0A3Q7UJC8_VULVU\nLYN\tA0A3Q7XCU3_URSAR\nLYN\tA0A3Q7Y569_URSAR\nLYN\tA0A452FNL9_CAPHI\nLYN\tA0A452I590_9SAUR\nLYN\tA0A455CB94_PHYMC\nLYN\tA0A4W2EEP1_BOBOX\nLYN\tA0A4X1SVW9_PIG\nLYN\tA0A4X1SVX9_PIG\nLYN\tA0A4Y2JB04_ARAVE\nLYN\tA0A4Z2GRK0_9TELE\nLYN\tA0A5F4C1A0_CANLF\nLYN\tA0A5F4CAR7_CANLF\nLYN\tA0A5F4CN96_CANLF\nLYN\tA0A5F4CP75_CANLF\nLYN\tA0A5F4CV35_CANLF\nLYN\tA0A5F4D4H5_CANLF\nLYN\tA0A5F8G5Y6_MONDO\nLYN\tA0A5F8GGQ0_MONDO\nLYN\tA0A5F8GPX2_MONDO\nLYN\tA0A5F8HDB2_MONDO\nLYN\tA0A5F8HKW5_MONDO\nLYN\tA0A5F9C6N2_RABIT\nLYN\tA0A5F9CT28_RABIT\nLYN\tA0A5G3HA61_XENTR\nLYN\tB5X488_SALSA\nLYN\tC5ISA0_SHEEP\nLYN\tD4QGC1_BOVIN\nLYN\tD6RVQ8_BOVIN\nLYN\tE2RBB8_CANLF\nLYN\tE5RJ37_HUMAN\nLYN\tF1N261_BOVIN\nLYN\tF1R4S6_DANRE\nLYN\tF1RT53_PIG\nLYN\tF6X6I5_ORNAN\nLYN\tF7AQZ6_HORSE\nLYN\tF7B0V9_MONDO\nLYN\tF7B7G2_CALJA\nLYN\tF7BV42_MACMU\nLYN\tG1L4Q2_AILME\nLYN\tG1PQZ4_MYOLU\nLYN\tG1QKJ0_NOMLE\nLYN\tG1SIY6_RABIT\nLYN\tG3T0P1_LOXAF\nLYN\tG3WDZ1_SARHA\nLYN\tG7PBV3_MACFA\nLYN\tG9K914_MUSPF\nLYN\tH0VJS5_CAVPO\nLYN\tH0WM37_OTOGA\nLYN\tH0ZLG0_TAEGU\nLYN\tH2PQC0_PONAB\nLYN\tH2R453_PANTR\nLYN\tH3A9Z1_LATCH\nLYN\tH9F8Q0_MACMU\nLYN\tH9GPY4_ANOCA\nLYN\tI3M8K4_ICTTR\nLYN\tK7AHF5_PANTR\nLYN\tK7GAD2_PELSI\nLYN\tLYN_HUMAN\nLYN\tLYN_MOUSE\nLYN\tLYN_RAT\nLYN\tM3WF65_FELCA\nLYN\tM3X677_FELCA\nLYN\tO13064_XENLA\nLYN\tQ05DV0_MOUSE\nLYN\tQ0P4J0_XENTR\nLYN\tQ28IV0_XENTR\nLYN\tQ3TCS3_MOUSE\nLYN\tQ3U6Q5_MOUSE\nLYN\tQ66I04_DANRE\nLYN\tQ6IZB2_PHOVI\nLYN\tQ6IZB3_PHOVI\nLYN\tQ6NUK7_HUMAN\nLYN\tQ8CEI0_MOUSE\nLYN\tTBCD1_BOVIN\nLYN\tU3D0I6_CALJA\nLYN\tU3DUJ3_CALJA\nLYN\tU3IEV3_ANAPP\nLYN\tU3K6N5_FICAL\nLYN\tU6DCD0_NEOVI\nLYN\tV8NSL8_OPHHA\nLYN\tV9KPC8_CALMI\nLYN\tW5PZG6_SHEEP\nLYN\tW5UC78_ICTPU\n','From\tTo\nEPHA3\tA0A096N0A8_PAPAN\nEPHA3\tA0A0B2V143_TOXCA\nEPHA3\tA0A0D9R0V3_CHLSB\nEPHA3\tA0A0G2JT71_RAT\nEPHA3\tA0A0K0K9A7_SCYCA\nEPHA3\tA0A0P6J723_HETGA\nEPHA3\tA0A0S7FM75_9TELE\nEPHA3\tA0A0S7FM88_9TELE\nEPHA3\tA0A0S7FM94_9TELE\nEPHA3\tA0A0S7FM96_9TELE\nEPHA3\tA0A0S7FM97_9TELE\nEPHA3\tA0A0S7FNF1_9TELE\nEPHA3\tA0A0S7FP88_9TELE\nEPHA3\tA0A0S7FQK0_9TELE\nEPHA3\tA0A0S7FS92_9TELE\nEPHA3\tA0A0S7FSR9_9TELE\nEPHA3\tA0A0S7FTT0_9TELE\nEPHA3\tA0A0S7FUP9_9TELE\nEPHA3\tA0A0S7FV13_9TELE\nEPHA3\tA0A0S7HV15_9TELE\nEPHA3\tA0A0S7HV37_9TELE\nEPHA3\tA0A0S7HV40_9TELE\nEPHA3\tA0A0S7HVH4_9TELE\nEPHA3\tA0A0S7HYE3_9TELE\nEPHA3\tA0A0S7HZ49_9TELE\nEPHA3\tA0A0S7I057_9TELE\nEPHA3\tA0A0S7IMK4_9TELE\nEPHA3\tA0A0S7IPF3_9TELE\nEPHA3\tA0A0S7IQH6_9TELE\nEPHA3\tA0A0X3PFJ2_SCHSO\nEPHA3\tA0A140VJJ0_HUMAN\nEPHA3\tA0A1S3AJG9_ERIEU\nEPHA3\tA0A1S3FSL8_DIPOR\nEPHA3\tA0A1S3FT61_DIPOR\nEPHA3\tA0A1S3FUN4_DIPOR\nEPHA3\tA0A1S3WSR6_ERIEU\nEPHA3\tA0A1U7QDJ5_MESAU\nEPHA3\tA0A1U7QQ03_MESAU\nEPHA3\tA0A1U7SLE0_ALLSI\nEPHA3\tA0A1V4JAR1_PATFA\nEPHA3\tA0A1V4JAY2_PATFA\nEPHA3\tA0A218V280_9PASE\nEPHA3\tA0A286ZNB6_PIG\nEPHA3\tA0A287ATL8_PIG\nEPHA3\tA0A2B4RJ52_STYPI\nEPHA3\tA0A2B4S0R2_STYPI\nEPHA3\tA0A2B4SH02_STYPI\nEPHA3\tA0A2B4SJ87_STYPI\nEPHA3\tA0A2D0R5Q1_ICTPU\nEPHA3\tA0A2D0R7Y0_ICTPU\nEPHA3\tA0A2I0MKZ5_COLLI\nEPHA3\tA0A2I2U9J4_FELCA\nEPHA3\tA0A2I4BGU4_9TELE\nEPHA3\tA0A2K5I3J4_COLAP\nEPHA3\tA0A2K5MKF6_CERAT\nEPHA3\tA0A2K5MKG7_CERAT\nEPHA3\tA0A2K5UU29_MACFA\nEPHA3\tA0A2K5ZZ33_MANLE\nEPHA3\tA0A2K6CN86_MACNE\nEPHA3\tA0A2K6GA11_PROCO\nEPHA3\tA0A2K6RAC4_RHIRO\nEPHA3\tA0A2K6V269_SAIBB\nEPHA3\tA0A2R9AMJ1_PANPA\nEPHA3\tA0A2U3VB91_ODORO\nEPHA3\tA0A2U3XAJ9_LEPWE\nEPHA3\tA0A2U3ZQV9_ODORO\nEPHA3\tA0A2U4A9V0_TURTR\nEPHA3\tA0A2U4A9V2_TURTR\nEPHA3\tA0A2U4AA76_TURTR\nEPHA3\tA0A2Y9H0M4_NEOSC\nEPHA3\tA0A2Y9H5A8_NEOSC\nEPHA3\tA0A2Y9NP36_DELLE\nEPHA3\tA0A2Y9SIN9_PHYMC\nEPHA3\tA0A2Y9SKA7_PHYMC\nEPHA3\tA0A338P6A7_MOUSE\nEPHA3\tA0A338P7J9_MOUSE\nEPHA3\tA0A340Y4R2_LIPVE\nEPHA3\tA0A340Y8E6_LIPVE\nEPHA3\tA0A341CQQ8_NEOAA\nEPHA3\tA0A341CR54_NEOAA\nEPHA3\tA0A383YMU3_BALAS\nEPHA3\tA0A384D7D3_URSMA\nEPHA3\tA0A3B3HD96_ORYLA\nEPHA3\tA0A3B3HRU1_ORYLA\nEPHA3\tA0A3Q2HI71_HORSE\nEPHA3\tA0A3Q7NRB9_CALUR\nEPHA3\tA0A3Q7RW79_VULVU\nEPHA3\tA0A3Q7VCA8_URSAR\nEPHA3\tA0A452ER07_CAPHI\nEPHA3\tA0A452GKD2_9SAUR\nEPHA3\tA0A452RVY6_URSAM\nEPHA3\tA0A452T2D8_URSMA\nEPHA3\tA0A452T2N9_URSMA\nEPHA3\tA0A493T2H7_ANAPP\nEPHA3\tA0A493TK26_ANAPP\nEPHA3\tA0A493TQV5_ANAPP\nEPHA3\tA0A493U0I0_ANAPP\nEPHA3\tA0A4W2DMT6_BOBOX\nEPHA3\tA0A4W2GGS5_BOBOX\nEPHA3\tA0A4W2GJR5_BOBOX\nEPHA3\tA0A4W2GJS1_BOBOX\nEPHA3\tA0A4X1UND4_PIG\nEPHA3\tA0A4X1UNY9_PIG\nEPHA3\tA0A4X1UPW4_PIG\nEPHA3\tA0A4X2LDU0_VOMUR\nEPHA3\tA0A5B7JJP6_PORTR\nEPHA3\tA0A5F4DC92_CANLF\nEPHA3\tA0A5F4W4A4_CALJA\nEPHA3\tA0A5F4WDG9_CALJA\nEPHA3\tA0A5F5Y6L9_FELCA\nEPHA3\tA0A5F7ZKD5_MACMU\nEPHA3\tA0A5F8AEN9_MACMU\nEPHA3\tA0A5F9C9C1_RABIT\nEPHA3\tA0A5F9CBR5_RABIT\nEPHA3\tA0A5F9CUK7_RABIT\nEPHA3\tA0A5F9DRH5_RABIT\nEPHA3\tA0A5G2RNA3_PIG\nEPHA3\tB7ZNN5_MOUSE\nEPHA3\tC9JXA2_HUMAN\nEPHA3\tD3K5K5_PIG\nEPHA3\tE1BJS9_BOVIN\nEPHA3\tE2I6F6_HUMAN\nEPHA3\tEPHA3_CHICK\nEPHA3\tEPHA3_DANRE\nEPHA3\tEPHA3_HUMAN\nEPHA3\tEPHA3_MOUSE\nEPHA3\tEPHA3_RAT\nEPHA3\tF1NJX6_CHICK\nEPHA3\tF1P925_CANLF\nEPHA3\tF1Q9D9_DANRE\nEPHA3\tF6UM70_MONDO\nEPHA3\tF6VIU7_XENTR\nEPHA3\tF7A3W1_XENTR\nEPHA3\tF7C3U9_HORSE\nEPHA3\tF7CM77_CALJA\nEPHA3\tF7EX10_ORNAN\nEPHA3\tF7HDY6_MACMU\nEPHA3\tG1K9V3_ANOCA\nEPHA3\tG1LPI0_AILME\nEPHA3\tG1LPI5_AILME\nEPHA3\tG1NNL7_MELGA\nEPHA3\tG1NXM4_MYOLU\nEPHA3\tG1RM08_NOMLE\nEPHA3\tG1SZC2_RABIT\nEPHA3\tG3SL04_LOXAF\nEPHA3\tG3V9D5_RAT\nEPHA3\tG3VHF3_SARHA\nEPHA3\tH0VYB1_CAVPO\nEPHA3\tH0X8S9_OTOGA\nEPHA3\tH0ZTE5_TAEGU\nEPHA3\tH2MMA8_ORYLA\nEPHA3\tH2MMA9_ORYLA\nEPHA3\tH2PA12_PONAB\nEPHA3\tH2QMY7_PANTR\nEPHA3\tH2RVG5_TAKRU\nEPHA3\tH9FH03_MACMU\nEPHA3\tH9FK89_MACMU\nEPHA3\tI3MNR9_ICTTR\nEPHA3\tK7EVF2_PONAB\nEPHA3\tK7GIR0_PELSI\nEPHA3\tK7GIS4_PELSI\nEPHA3\tM3YA44_MUSPF\nEPHA3\tQ4H3M4_CIOIN\nEPHA3\tQ6P4R6_HUMAN\nEPHA3\tQ8BRB1_MOUSE\nEPHA3\tQ8C3U1_MOUSE\nEPHA3\tQ8C9K6_MOUSE\nEPHA3\tU3DED6_CALJA\nEPHA3\tU3IYY7_ANAPP\nEPHA3\tU3JG76_FICAL\nEPHA3\tU6DAW2_NEOVI\nEPHA3\tV8NEI7_OPHHA\nEPHA3\tV8NMB6_OPHHA\nEPHA3\tV8NMK5_OPHHA\nEPHA3\tW5Q6R5_SHEEP\nEPHA3\tW5UFX2_ICTPU\n','From\tTo\nEPHA7\tA0A096NGE6_PAPAN\nEPHA7\tA0A0A7RBW9_XENLA\nEPHA7\tA0A0A7RFN6_XENLA\nEPHA7\tA0A0B5JQ25_DANRE\nEPHA7\tA0A0D9RXG4_CHLSB\nEPHA7\tA0A0R4IEC7_DANRE\nEPHA7\tA0A0S7EWG3_9TELE\nEPHA7\tA0A0S7HFH8_9TELE\nEPHA7\tA0A0S7HFI2_9TELE\nEPHA7\tA0A0S7HG74_9TELE\nEPHA7\tA0A0S7HG83_9TELE\nEPHA7\tA0A0S7HGH4_9TELE\nEPHA7\tA0A0S7HJS4_9TELE\nEPHA7\tA0A0S7J1C7_9TELE\nEPHA7\tA0A0S7MBD8_9TELE\nEPHA7\tA0A1A7X1X7_9TELE\nEPHA7\tA0A1A7YD39_9TELE\nEPHA7\tA0A1A7ZBW6_NOTFU\nEPHA7\tA0A1A8BKA0_9TELE\nEPHA7\tA0A1A8BKL3_9TELE\nEPHA7\tA0A1A8IQ72_NOTKU\nEPHA7\tA0A1A8LF46_9TELE\nEPHA7\tA0A1A8PWE6_9TELE\nEPHA7\tA0A1A8PX43_9TELE\nEPHA7\tA0A1A8VG09_NOTFU\nEPHA7\tA0A1A8VGP9_NOTFU\nEPHA7\tA0A1L8G369_XENLA\nEPHA7\tA0A1L8G975_XENLA\nEPHA7\tA0A1S3F692_DIPOR\nEPHA7\tA0A1S3F7C8_DIPOR\nEPHA7\tA0A1S3F7Y1_DIPOR\nEPHA7\tA0A1S3S7Y7_SALSA\nEPHA7\tA0A1S3S7Y8_SALSA\nEPHA7\tA0A1S3S7Y9_SALSA\nEPHA7\tA0A1S3S7Z2_SALSA\nEPHA7\tA0A1S3S7Z5_SALSA\nEPHA7\tA0A1S3S814_SALSA\nEPHA7\tA0A1S3WC27_ERIEU\nEPHA7\tA0A1U7RVZ9_ALLSI\nEPHA7\tA0A1V4JM51_PATFA\nEPHA7\tA0A1V4JM55_PATFA\nEPHA7\tA0A1V4JNH7_PATFA\nEPHA7\tA0A218VBN7_9PASE\nEPHA7\tA0A286XGX9_CAVPO\nEPHA7\tA0A286XY87_CAVPO\nEPHA7\tA0A287BAJ5_PIG\nEPHA7\tA0A287CVX4_ICTTR\nEPHA7\tA0A287D397_ICTTR\nEPHA7\tA0A2D0PS44_ICTPU\nEPHA7\tA0A2D0PS60_ICTPU\nEPHA7\tA0A2D0PTM9_ICTPU\nEPHA7\tA0A2D0PVC8_ICTPU\nEPHA7\tA0A2D0PW84_ICTPU\nEPHA7\tA0A2I3H8M4_NOMLE\nEPHA7\tA0A2I3HWP7_NOMLE\nEPHA7\tA0A2I3MZF7_PAPAN\nEPHA7\tA0A2I3RH81_PANTR\nEPHA7\tA0A2I3TVE2_PANTR\nEPHA7\tA0A2I4C3L0_9TELE\nEPHA7\tA0A2K5D4Z6_AOTNA\nEPHA7\tA0A2K5D505_AOTNA\nEPHA7\tA0A2K5D508_AOTNA\nEPHA7\tA0A2K5K3J1_COLAP\nEPHA7\tA0A2K5K3J7_COLAP\nEPHA7\tA0A2K5K3N3_COLAP\nEPHA7\tA0A2K5KXK5_CERAT\nEPHA7\tA0A2K5KXN8_CERAT\nEPHA7\tA0A2K5Q0S1_CEBCA\nEPHA7\tA0A2K5Q0S9_CEBCA\nEPHA7\tA0A2K5TN18_MACFA\nEPHA7\tA0A2K6ACU7_MANLE\nEPHA7\tA0A2K6ACW3_MANLE\nEPHA7\tA0A2K6CFV5_MACNE\nEPHA7\tA0A2K6CFW2_MACNE\nEPHA7\tA0A2K6M9U4_RHIBE\nEPHA7\tA0A2K6M9V1_RHIBE\nEPHA7\tA0A2K6PFG5_RHIRO\nEPHA7\tA0A2K6PFH6_RHIRO\nEPHA7\tA0A2K6UHT1_SAIBB\nEPHA7\tA0A2K6UHV5_SAIBB\nEPHA7\tA0A2K6UHV8_SAIBB\nEPHA7\tA0A2R8P5U2_CALJA\nEPHA7\tA0A2R8RS29_DANRE\nEPHA7\tA0A2R9A7M8_PANPA\nEPHA7\tA0A2R9A7N6_PANPA\nEPHA7\tA0A2R9ACJ6_PANPA\nEPHA7\tA0A2S2PM87_SCHGA\nEPHA7\tA0A2U3WZC6_ODORO\nEPHA7\tA0A2U3WZD3_ODORO\nEPHA7\tA0A2U3WZP4_ODORO\nEPHA7\tA0A2U3X826_LEPWE\nEPHA7\tA0A2U3X839_LEPWE\nEPHA7\tA0A2U3X851_LEPWE\nEPHA7\tA0A2U3ZWN6_ODORO\nEPHA7\tA0A2U4AVK5_TURTR\nEPHA7\tA0A2U4AVL5_TURTR\nEPHA7\tA0A2U4AVN6_TURTR\nEPHA7\tA0A2Y9HAL2_NEOSC\nEPHA7\tA0A2Y9HAW4_NEOSC\nEPHA7\tA0A2Y9HGN5_NEOSC\nEPHA7\tA0A2Y9HHA7_NEOSC\nEPHA7\tA0A2Y9LFZ6_DELLE\nEPHA7\tA0A2Y9LT25_DELLE\nEPHA7\tA0A2Y9SCQ9_PHYMC\nEPHA7\tA0A2Y9SH95_PHYMC\nEPHA7\tA0A340XT98_LIPVE\nEPHA7\tA0A340XYU9_LIPVE\nEPHA7\tA0A340XZ94_LIPVE\nEPHA7\tA0A341AA74_NEOAA\nEPHA7\tA0A384B8K6_BALAS\nEPHA7\tA0A384B8M7_BALAS\nEPHA7\tA0A384B8P0_BALAS\nEPHA7\tA0A384B924_BALAS\nEPHA7\tA0A384D862_URSMA\nEPHA7\tA0A384D880_URSMA\nEPHA7\tA0A384D8H1_URSMA\nEPHA7\tA0A384D8Z5_URSMA\nEPHA7\tA0A3B3H476_ORYLA\nEPHA7\tA0A3B3WHZ3_9TELE\nEPHA7\tA0A3B3WI78_9TELE\nEPHA7\tA0A3Q0DXW1_TARSY\nEPHA7\tA0A3Q0DZL4_TARSY\nEPHA7\tA0A3Q0E217_TARSY\nEPHA7\tA0A3Q0G6V6_ALLSI\nEPHA7\tA0A3Q1LF73_BOVIN\nEPHA7\tA0A3Q2H8R4_HORSE\nEPHA7\tA0A3Q7N3Z9_CALUR\nEPHA7\tA0A3Q7N7E3_CALUR\nEPHA7\tA0A3Q7N7G4_CALUR\nEPHA7\tA0A3Q7N7W3_CALUR\nEPHA7\tA0A3Q7SDX3_VULVU\nEPHA7\tA0A3Q7SPT8_VULVU\nEPHA7\tA0A3Q7SPU2_VULVU\nEPHA7\tA0A3Q7SSG5_VULVU\nEPHA7\tA0A3Q7TBL4_VULVU\nEPHA7\tA0A3Q7TBL9_VULVU\nEPHA7\tA0A3Q7TTR0_VULVU\nEPHA7\tA0A3Q7TTR5_VULVU\nEPHA7\tA0A3Q7W369_URSAR\nEPHA7\tA0A3Q7WR80_URSAR\nEPHA7\tA0A3Q7X042_URSAR\nEPHA7\tA0A3Q7XLS5_URSAR\nEPHA7\tA0A3Q7XWE9_URSAR\nEPHA7\tA0A452FFD4_CAPHI\nEPHA7\tA0A452FFL7_CAPHI\nEPHA7\tA0A452I0Q5_9SAUR\nEPHA7\tA0A452I0R8_9SAUR\nEPHA7\tA0A452QTN2_URSAM\nEPHA7\tA0A452QTN9_URSAM\nEPHA7\tA0A452QTP0_URSAM\nEPHA7\tA0A452QTQ1_URSAM\nEPHA7\tA0A452QTQ3_URSAM\nEPHA7\tA0A452V8P1_URSMA\nEPHA7\tA0A452V8Y6_URSMA\nEPHA7\tA0A493SYB0_ANAPP\nEPHA7\tA0A493TRW2_ANAPP\nEPHA7\tA0A493TVD4_ANAPP\nEPHA7\tA0A493U1E4_ANAPP\nEPHA7\tA0A4W2BLI5_BOBOX\nEPHA7\tA0A4W2C350_BOBOX\nEPHA7\tA0A4W2H8Q7_BOBOX\nEPHA7\tA0A4W2HAJ5_BOBOX\nEPHA7\tA0A4W2HBS5_BOBOX\nEPHA7\tA0A4W4EA33_ELEEL\nEPHA7\tA0A4W4EAB8_ELEEL\nEPHA7\tA0A4W5PU90_9TELE\nEPHA7\tA0A4X1VK53_PIG\nEPHA7\tA0A4X2KHL6_VOMUR\nEPHA7\tA0A5F5XNL0_FELCA\nEPHA7\tA0A5F9CH48_RABIT\nEPHA7\tA0A5F9DU25_RABIT\nEPHA7\tA0A5G3KUZ9_XENTR\nEPHA7\tA0A5G3KVS5_XENTR\nEPHA7\tA0A5G3KWP1_XENTR\nEPHA7\tA0A5G3KWR9_XENTR\nEPHA7\tA2BDQ4_MOUSE\nEPHA7\tD2HE60_AILME\nEPHA7\tE2R7J5_CANLF\nEPHA7\tEPHA7_CHICK\nEPHA7\tEPHA7_HUMAN\nEPHA7\tEPHA7_MOUSE\nEPHA7\tEPHA7_RAT\nEPHA7\tF1N780_BOVIN\nEPHA7\tF6RSL5_XENTR\nEPHA7\tF6XPG2_CALJA\nEPHA7\tF6Z105_HORSE\nEPHA7\tF7CWR4_MONDO\nEPHA7\tF7D2V3_ORNAN\nEPHA7\tG1KC20_ANOCA\nEPHA7\tG1NLJ3_MELGA\nEPHA7\tG1P4L7_MYOLU\nEPHA7\tG1RSQ5_NOMLE\nEPHA7\tG1SDT6_RABIT\nEPHA7\tG3TK41_LOXAF\nEPHA7\tG3TZE6_LOXAF\nEPHA7\tG3WUH1_SARHA\nEPHA7\tG7MPY1_MACMU\nEPHA7\tG7P422_MACFA\nEPHA7\tH0V191_CAVPO\nEPHA7\tH0WJP9_OTOGA\nEPHA7\tH0ZQ33_TAEGU\nEPHA7\tH2MUM5_ORYLA\nEPHA7\tH2PJU4_PONAB\nEPHA7\tH2QTF1_PANTR\nEPHA7\tH3A1I8_LATCH\nEPHA7\tH9F0W4_MACMU\nEPHA7\tH9FAC2_MACMU\nEPHA7\tI3JZE8_ORENI\nEPHA7\tI3M3Q7_ICTTR\nEPHA7\tL8EAA4_HUMAN\nEPHA7\tM3WPK6_FELCA\nEPHA7\tM3YCD3_MUSPF\nEPHA7\tQ1MT83_DANRE\nEPHA7\tQ3V3T7_MOUSE\nEPHA7\tQ5VSJ4_DANRE\nEPHA7\tQ8CC52_MOUSE\nEPHA7\tQ8CCF9_MOUSE\nEPHA7\tQ8R381_MOUSE\nEPHA7\tT2MDF6_HYDVU\nEPHA7\tU3DW98_CALJA\nEPHA7\tU3I4U8_ANAPP\nEPHA7\tU3JKV4_FICAL\nEPHA7\tU6DEA0_NEOVI\nEPHA7\tV8PEN2_OPHHA\nEPHA7\tV8PF82_OPHHA\nEPHA7\tW5NK05_LEPOC\nEPHA7\tW5PRL0_SHEEP\n','From\tTo\nEPHB4\tA0A096N812_PAPAN\nEPHB4\tA0A0D9RZG7_CHLSB\nEPHB4\tA0A0S7GC67_9TELE\nEPHB4\tA0A0S7GFU8_9TELE\nEPHB4\tA0A0S7GHE0_9TELE\nEPHB4\tA0A0S7H3Z0_9TELE\nEPHB4\tA0A0S7HV95_9TELE\nEPHB4\tA0A0S7HVF9_9TELE\nEPHB4\tA0A0S7HYT8_9TELE\nEPHB4\tA0A0X3P9F8_SCHSO\nEPHB4\tA0A1D5QUS8_MACMU\nEPHB4\tA0A1D5R737_MACMU\nEPHB4\tA0A1S3AGZ7_ERIEU\nEPHB4\tA0A1S3GGN7_DIPOR\nEPHB4\tA0A1U7T158_TARSY\nEPHB4\tA0A1U7TEX7_TARSY\nEPHB4\tA0A1U8CKP6_MESAU\nEPHB4\tA0A218U7Q6_9PASE\nEPHB4\tA0A250Y686_CASCN\nEPHB4\tA0A286XZH1_CAVPO\nEPHB4\tA0A287AYF2_PIG\nEPHB4\tA0A287CZ82_ICTTR\nEPHB4\tA0A2D0QAC9_ICTPU\nEPHB4\tA0A2D0QBH4_ICTPU\nEPHB4\tA0A2D0QBH6_ICTPU\nEPHB4\tA0A2D0QD49_ICTPU\nEPHB4\tA0A2D0QE47_ICTPU\nEPHB4\tA0A2D0QE52_ICTPU\nEPHB4\tA0A2I2V293_FELCA\nEPHB4\tA0A2I3FWW9_NOMLE\nEPHB4\tA0A2I3LDL4_PAPAN\nEPHB4\tA0A2I3MY19_PAPAN\nEPHB4\tA0A2I3RMZ6_PANTR\nEPHB4\tA0A2I3TPV3_PANTR\nEPHB4\tA0A2I4C7X9_9TELE\nEPHB4\tA0A2K5DHJ7_AOTNA\nEPHB4\tA0A2K5DHP3_AOTNA\nEPHB4\tA0A2K5DHQ6_AOTNA\nEPHB4\tA0A2K5IW78_COLAP\nEPHB4\tA0A2K5IWC7_COLAP\nEPHB4\tA0A2K5IWT9_COLAP\nEPHB4\tA0A2K5P8K8_CERAT\nEPHB4\tA0A2K5P8M1_CERAT\nEPHB4\tA0A2K5P8M3_CERAT\nEPHB4\tA0A2K5QVP9_CEBCA\nEPHB4\tA0A2K5QVT9_CEBCA\nEPHB4\tA0A2K5V953_MACFA\nEPHB4\tA0A2K5V978_MACFA\nEPHB4\tA0A2K5V983_MACFA\nEPHB4\tA0A2K6A7A6_MANLE\nEPHB4\tA0A2K6A7C8_MANLE\nEPHB4\tA0A2K6A7D5_MANLE\nEPHB4\tA0A2K6BID8_MACNE\nEPHB4\tA0A2K6BIG9_MACNE\nEPHB4\tA0A2K6BIN7_MACNE\nEPHB4\tA0A2K6G212_PROCO\nEPHB4\tA0A2K6G220_PROCO\nEPHB4\tA0A2K6G238_PROCO\nEPHB4\tA0A2K6JT79_RHIBE\nEPHB4\tA0A2K6JT84_RHIBE\nEPHB4\tA0A2K6JT92_RHIBE\nEPHB4\tA0A2K6RMZ1_RHIRO\nEPHB4\tA0A2K6RMZ5_RHIRO\nEPHB4\tA0A2K6RN08_RHIRO\nEPHB4\tA0A2K6UPI6_SAIBB\nEPHB4\tA0A2K6UPK0_SAIBB\nEPHB4\tA0A2K6UPQ8_SAIBB\nEPHB4\tA0A2R9B1N3_PANPA\nEPHB4\tA0A2R9B3J8_PANPA\nEPHB4\tA0A2R9B3L9_PANPA\nEPHB4\tA0A2U3VUX2_ODORO\nEPHB4\tA0A2U3XQS1_LEPWE\nEPHB4\tA0A2U4C6V5_TURTR\nEPHB4\tA0A2Y9GA60_NEOSC\nEPHB4\tA0A2Y9PEE2_DELLE\nEPHB4\tA0A340Y8Y5_LIPVE\nEPHB4\tA0A341BEK1_NEOAA\nEPHB4\tA0A384AH47_BALAS\nEPHB4\tA0A384CQI1_URSMA\nEPHB4\tA0A3B3H5V1_ORYLA\nEPHB4\tA0A3B3IH53_ORYLA\nEPHB4\tA0A3Q0H8U2_ALLSI\nEPHB4\tA0A3Q7NH54_CALUR\nEPHB4\tA0A3Q7UBW7_VULVU\nEPHB4\tA0A3Q7XBV0_URSAR\nEPHB4\tA0A419Q4K4_CLOSI\nEPHB4\tA0A452FJ70_CAPHI\nEPHB4\tA0A452FK54_CAPHI\nEPHB4\tA0A452SVG0_URSAM\nEPHB4\tA0A452SVG5_URSAM\nEPHB4\tA0A452SVX3_URSAM\nEPHB4\tA0A452SVY1_URSAM\nEPHB4\tA0A452SVY6_URSAM\nEPHB4\tA0A452SW24_URSAM\nEPHB4\tA0A452U378_URSMA\nEPHB4\tA0A452U380_URSMA\nEPHB4\tA0A452U3D9_URSMA\nEPHB4\tA0A455AWT3_PHYMC\nEPHB4\tA0A4W2BNK2_BOBOX\nEPHB4\tA0A4W2DK64_BOBOX\nEPHB4\tA0A4W2FLH8_BOBOX\nEPHB4\tA0A4W2HJW2_BOBOX\nEPHB4\tA0A4W4G581_ELEEL\nEPHB4\tA0A4W4G5C6_ELEEL\nEPHB4\tA0A4W4G5V3_ELEEL\nEPHB4\tA0A4W4G6H5_ELEEL\nEPHB4\tA0A4W4G9J2_ELEEL\nEPHB4\tA0A4W4G9P0_ELEEL\nEPHB4\tA0A4W4G9T2_ELEEL\nEPHB4\tA0A4W4G9X2_ELEEL\nEPHB4\tA0A4W4G9Y8_ELEEL\nEPHB4\tA0A4W4GAH6_ELEEL\nEPHB4\tA0A4W6CMZ4_LATCA\nEPHB4\tA0A4W6CMZ8_LATCA\nEPHB4\tA0A4W6CN04_LATCA\nEPHB4\tA0A4W6CN33_LATCA\nEPHB4\tA0A4W6CN50_LATCA\nEPHB4\tA0A4X1UDQ7_PIG\nEPHB4\tA0A4X1UDR8_PIG\nEPHB4\tA0A4X2JW32_VOMUR\nEPHB4\tA0A4Z2E794_9TELE\nEPHB4\tA0A5F4CQV5_CANLF\nEPHB4\tA0A5F4D945_CANLF\nEPHB4\tA0A5F4D9M8_CANLF\nEPHB4\tA0A5F5PIH8_HORSE\nEPHB4\tA0A5F7ZIQ6_MACMU\nEPHB4\tA0A5F8GR61_MONDO\nEPHB4\tA0A5F9CMM7_RABIT\nEPHB4\tA0A5F9DDV6_RABIT\nEPHB4\tA0A5G3HQF2_XENTR\nEPHB4\tA0A5G3HWJ5_XENTR\nEPHB4\tE1BIE4_BOVIN\nEPHB4\tE9PWK7_MOUSE\nEPHB4\tEPHB4_HUMAN\nEPHB4\tEPHB4_MOUSE\nEPHB4\tF1PRQ8_CANLF\nEPHB4\tF1RMX5_PIG\nEPHB4\tF6XJC5_MONDO\nEPHB4\tF6YTU9_XENTR\nEPHB4\tF7BG55_HORSE\nEPHB4\tF7C3F7_ORNAN\nEPHB4\tF7E595_MACMU\nEPHB4\tF7GU59_CALJA\nEPHB4\tF7GUF2_CALJA\nEPHB4\tF7GZF0_CALJA\nEPHB4\tG1M7Y7_AILME\nEPHB4\tG1PFB6_MYOLU\nEPHB4\tG1RMR2_NOMLE\nEPHB4\tG1TCL2_RABIT\nEPHB4\tG1TTY5_RABIT\nEPHB4\tG3TNU6_LOXAF\nEPHB4\tG3W0Z3_SARHA\nEPHB4\tH0VR59_CAVPO\nEPHB4\tH0X0E5_OTOGA\nEPHB4\tH2LRI3_ORYLA\nEPHB4\tH2PLQ6_PONAB\nEPHB4\tH2QV43_PANTR\nEPHB4\tH3A109_LATCH\nEPHB4\tH9GHX4_ANOCA\nEPHB4\tI3MLL6_ICTTR\nEPHB4\tM0RDA4_RAT\nEPHB4\tM3W6L6_FELCA\nEPHB4\tM3YWL4_MUSPF\nEPHB4\tQ32NP2_XENLA\nEPHB4\tQ3V1K8_MOUSE\nEPHB4\tQ541P7_HUMAN\nEPHB4\tQ62128_MOUSE\nEPHB4\tQ8C7S3_MOUSE\nEPHB4\tQ8C8K1_MOUSE\nEPHB4\tQ91YM0_MOUSE\nEPHB4\tQ96L35_HUMAN\nEPHB4\tU6DHH6_NEOVI\nEPHB4\tV8N5I6_OPHHA\nEPHB4\tV8N652_OPHHA\nEPHB4\tV8NCD6_OPHHA\nEPHB4\tW5Q2P9_SHEEP\nEPHB4\tW5UAG3_ICTPU\n','From\tTo\nBTK\tA0A096NC59_PAPAN\nBTK\tA0A0D9RBX3_CHLSB\nBTK\tA0A0G2K134_RAT\nBTK\tA0A0G2K6M9_RAT\nBTK\tA0A0S7M827_9TELE\nBTK\tA0A1A7XJ36_9TELE\nBTK\tA0A1A7XV43_9TELE\nBTK\tA0A1A7XX29_9TELE\nBTK\tA0A1A8B0U3_NOTFU\nBTK\tA0A1A8B3J1_NOTFU\nBTK\tA0A1A8D4L8_9TELE\nBTK\tA0A1A8E606_9TELE\nBTK\tA0A1A8EW77_9TELE\nBTK\tA0A1A8HG82_9TELE\nBTK\tA0A1A8HKP3_9TELE\nBTK\tA0A1A8J844_NOTKU\nBTK\tA0A1A8JB97_NOTKU\nBTK\tA0A1A8M9B5_9TELE\nBTK\tA0A1A8PBE1_9TELE\nBTK\tA0A1A8QPU1_9TELE\nBTK\tA0A1A8QZR0_9TELE\nBTK\tA0A1A8R7L6_9TELE\nBTK\tA0A1A8V5F6_NOTFU\nBTK\tA0A1D5NXK5_CHICK\nBTK\tA0A1L8F6P2_XENLA\nBTK\tA0A1S3ANW1_ERIEU\nBTK\tA0A1S3ANW5_ERIEU\nBTK\tA0A1S3GNU6_DIPOR\nBTK\tA0A1S3RQG7_SALSA\nBTK\tA0A1U7QQI2_MESAU\nBTK\tA0A1U7S9B1_ALLSI\nBTK\tA0A1U7T043_TARSY\nBTK\tA0A1U7T452_TARSY\nBTK\tA0A1V4KZN5_PATFA\nBTK\tA0A218UKK7_9PASE\nBTK\tA0A250YKQ2_CASCN\nBTK\tA0A286Y0E9_CAVPO\nBTK\tA0A287CW11_ICTTR\nBTK\tA0A2D0RJ90_ICTPU\nBTK\tA0A2I0LS31_COLLI\nBTK\tA0A2I3H7M2_NOMLE\nBTK\tA0A2I3NCH0_PAPAN\nBTK\tA0A2I3RVY9_PANTR\nBTK\tA0A2I3SPK7_PANTR\nBTK\tA0A2I4AUW1_9TELE\nBTK\tA0A2I4AUX4_9TELE\nBTK\tA0A2K5ETH5_AOTNA\nBTK\tA0A2K5ETL2_AOTNA\nBTK\tA0A2K5ETL3_AOTNA\nBTK\tA0A2K5JKA9_COLAP\nBTK\tA0A2K5JKB3_COLAP\nBTK\tA0A2K5JKB9_COLAP\nBTK\tA0A2K5M5C5_CERAT\nBTK\tA0A2K5M5F9_CERAT\nBTK\tA0A2K5PE35_CEBCA\nBTK\tA0A2K5PE53_CEBCA\nBTK\tA0A2K5VDX8_MACFA\nBTK\tA0A2K6A443_MANLE\nBTK\tA0A2K6A460_MANLE\nBTK\tA0A2K6A468_MANLE\nBTK\tA0A2K6CUV5_MACNE\nBTK\tA0A2K6CV18_MACNE\nBTK\tA0A2K6GTE4_PROCO\nBTK\tA0A2K6GTI4_PROCO\nBTK\tA0A2K6GTI5_PROCO\nBTK\tA0A2K6N2U5_RHIBE\nBTK\tA0A2K6N2U9_RHIBE\nBTK\tA0A2K6N2V6_RHIBE\nBTK\tA0A2K6N2X9_RHIBE\nBTK\tA0A2K6PTH4_RHIRO\nBTK\tA0A2K6PTI6_RHIRO\nBTK\tA0A2K6PTJ2_RHIRO\nBTK\tA0A2K6PTJ8_RHIRO\nBTK\tA0A2K6TV37_SAIBB\nBTK\tA0A2K6TV40_SAIBB\nBTK\tA0A2K6TV44_SAIBB\nBTK\tA0A2R8RU01_DANRE\nBTK\tA0A2R9BPC2_PANPA\nBTK\tA0A2R9BQ26_PANPA\nBTK\tA0A2R9BSW7_PANPA\nBTK\tA0A2U3Y201_LEPWE\nBTK\tA0A2U3ZYD6_ODORO\nBTK\tA0A2U3ZYF3_ODORO\nBTK\tA0A2U3ZYF8_ODORO\nBTK\tA0A2U3ZYF9_ODORO\nBTK\tA0A2U4B3L5_TURTR\nBTK\tA0A2U4B3N0_TURTR\nBTK\tA0A2Y9FBB9_PHYMC\nBTK\tA0A2Y9HPA0_NEOSC\nBTK\tA0A2Y9NR66_DELLE\nBTK\tA0A2Y9NU27_DELLE\nBTK\tA0A337RZH9_FELCA\nBTK\tA0A340Y2U3_LIPVE\nBTK\tA0A340Y974_LIPVE\nBTK\tA0A341BHR2_NEOAA\nBTK\tA0A341BJB2_NEOAA\nBTK\tA0A384A2D9_BALAS\nBTK\tA0A384CYK1_URSMA\nBTK\tA0A384CZK1_URSMA\nBTK\tA0A3Q0GU06_ALLSI\nBTK\tA0A3Q0GYK9_ALLSI\nBTK\tA0A3Q1N4X6_BOVIN\nBTK\tA0A3Q7P7Y2_CALUR\nBTK\tA0A3Q7P8B4_CALUR\nBTK\tA0A3Q7TM74_VULVU\nBTK\tA0A3Q7TTF1_URSAR\nBTK\tA0A3Q7UGP0_URSAR\nBTK\tA0A452FHA4_CAPHI\nBTK\tA0A452IAM5_9SAUR\nBTK\tA0A452S5Y2_URSAM\nBTK\tA0A452S617_URSAM\nBTK\tA0A4C1U5X0_9NEOP\nBTK\tA0A4W2G0N8_BOBOX\nBTK\tA0A4W2HUX3_BOBOX\nBTK\tA0A4W4G1P8_ELEEL\nBTK\tA0A4W4G1Q6_ELEEL\nBTK\tA0A4W4G1R1_ELEEL\nBTK\tA0A4W4G3Z8_ELEEL\nBTK\tA0A4W4G403_ELEEL\nBTK\tA0A4W4G4Z1_ELEEL\nBTK\tA0A4W4G5F0_ELEEL\nBTK\tA0A4W6BTH1_LATCA\nBTK\tA0A4W6BTI1_LATCA\nBTK\tA0A4W6BWA1_LATCA\nBTK\tA0A4W6BWA6_LATCA\nBTK\tA0A4X1VZ31_PIG\nBTK\tA0A4X1W3S0_PIG\nBTK\tA0A4X2JYQ0_VOMUR\nBTK\tA0A4X2K126_VOMUR\nBTK\tA0A4Z2I0E5_9TELE\nBTK\tA0A5F4BPN2_CANLF\nBTK\tA0A5F4BTR5_CANLF\nBTK\tA0A5F4C1L3_CANLF\nBTK\tA0A5F4C9U8_CANLF\nBTK\tA0A5F4DAQ5_CANLF\nBTK\tA0A5F5PIX5_HORSE\nBTK\tA0A5F8GP57_MONDO\nBTK\tA0A5F9CKZ3_RABIT\nBTK\tA0A5G3L8B4_XENTR\nBTK\tA0A5G3LAH7_XENTR\nBTK\tA2BDW0_MOUSE\nBTK\tB3DLB1_XENTR\nBTK\tB5X1K0_SALSA\nBTK\tB8X3E0_GALVR\nBTK\tB8X3E3_GALVR\nBTK\tB8X3E6_CYNVO\nBTK\tBTK_CHICK\nBTK\tBTK_HUMAN\nBTK\tBTK_MOUSE\nBTK\tE2RMF7_CANLF\nBTK\tE9FTY5_DAPPU\nBTK\tF1S1L0_PIG\nBTK\tF6S5Q6_HORSE\nBTK\tF6V0I6_MACMU\nBTK\tF6V9R3_XENTR\nBTK\tF6YES1_MONDO\nBTK\tF7BGN5_MACMU\nBTK\tF7BKT0_ORNAN\nBTK\tF7BXJ7_XENTR\nBTK\tF7BY20_XENTR\nBTK\tF7IHG0_CALJA\nBTK\tF7IHH4_CALJA\nBTK\tF7IHI3_CALJA\nBTK\tF8W3B7_DANRE\nBTK\tG1LS18_AILME\nBTK\tG1NB26_MELGA\nBTK\tG1P273_MYOLU\nBTK\tG1R7I9_NOMLE\nBTK\tG1SZN9_RABIT\nBTK\tG3TIS4_LOXAF\nBTK\tG3WK98_SARHA\nBTK\tG7Q392_MACFA\nBTK\tH0UU58_CAVPO\nBTK\tH0WQ47_OTOGA\nBTK\tH0XW86_OTOGA\nBTK\tH0ZY02_TAEGU\nBTK\tH2L5G0_ORYLA\nBTK\tH2PW99_PONAB\nBTK\tH2QYW8_PANTR\nBTK\tH2U1C1_TAKRU\nBTK\tH3BCB6_LATCH\nBTK\tH9FBY6_MACMU\nBTK\tH9FLW7_MACMU\nBTK\tH9GDX7_ANOCA\nBTK\tH9T842_LETCA\nBTK\tI3J2L1_ORENI\nBTK\tI3J2L2_ORENI\nBTK\tI3LN58_PIG\nBTK\tI3M4T4_ICTTR\nBTK\tK7F777_PELSI\nBTK\tM3WMI5_FELCA\nBTK\tM3XHH0_LATCH\nBTK\tM3XSU9_MUSPF\nBTK\tM9PCD8_DROME\nBTK\tM9PCX8_DROME\nBTK\tQ25C03_COTJA\nBTK\tQ3ZC95_BOVIN\nBTK\tQ572P0_HUMAN\nBTK\tQ572P1_HUMAN\nBTK\tQ572P2_HUMAN\nBTK\tQ572P3_HUMAN\nBTK\tQ572P4_HUMAN\nBTK\tQ572P5_HUMAN\nBTK\tQ5JY90_HUMAN\nBTK\tQ5S255_RAT\nBTK\tQ5ZLF2_CHICK\nBTK\tQ7TMU1_MOUSE\nBTK\tQ90YE8_TETNG\nBTK\tQ90YI6_TAKRU\nBTK\tR4GB22_ANOCA\nBTK\tU3KA25_FICAL\nBTK\tU3NG26_HUMAN\nBTK\tU6CY89_NEOVI\nBTK\tV5KUQ4_LEOWI\nBTK\tV5KUW5_LEOGE\nBTK\tV5KUY0_LEOTI\nBTK\tV5KUY7_LEOCO\nBTK\tV8NGR6_OPHHA\nBTK\tV8NKA4_OPHHA\nBTK\tW5NU60_SHEEP\n','From\tTo\nKDR\tA0A096NNZ6_PAPAN\nKDR\tA0A0B2W507_TOXCA\nKDR\tA0A0B5CU28_RHIMP\nKDR\tA0A0D9QY88_CHLSB\nKDR\tA0A0F6VY88_ANOST\nKDR\tA0A0F6YG18_CULQU\nKDR\tA0A0G2JW10_RAT\nKDR\tA0A0H4A777_HUMAN\nKDR\tA0A0H4A7S0_HUMAN\nKDR\tA0A0H4A7S4_HUMAN\nKDR\tA0A0H4ADZ1_HUMAN\nKDR\tA0A0H4AF76_HUMAN\nKDR\tA0A0H4AF85_HUMAN\nKDR\tA0A0K1YUU7_RHIMP\nKDR\tA0A0K1YV48_RHIMP\nKDR\tA0A0K1YW32_9ACAR\nKDR\tA0A0U2VU55_HUMAN\nKDR\tA0A0U2WZU8_HUMAN\nKDR\tA0A0U2X4C8_HUMAN\nKDR\tA0A0V0U6D7_9BILA\nKDR\tA0A0V0WZC0_9BILA\nKDR\tA0A0V1BKX9_TRISP\nKDR\tA0A0V1PAK4_9BILA\nKDR\tA0A140C6U9_AEDAE\nKDR\tA0A140C6V0_AEDAE\nKDR\tA0A142ECX5_RHIMP\nKDR\tA0A1A7YG38_9TELE\nKDR\tA0A1A7ZJE4_NOTFU\nKDR\tA0A1A8G7L9_9TELE\nKDR\tA0A1A8M1M1_9TELE\nKDR\tA0A1A8Q8R1_9TELE\nKDR\tA0A1A8S2T1_9TELE\nKDR\tA0A1A8V1S1_NOTFU\nKDR\tA0A1L8HLA0_XENLA\nKDR\tA0A1S3AGB0_ERIEU\nKDR\tA0A1S3EKU0_DIPOR\nKDR\tA0A1S3PA47_SALSA\nKDR\tA0A1S3PA84_SALSA\nKDR\tA0A1S3PAA8_SALSA\nKDR\tA0A1U7QTW7_MESAU\nKDR\tA0A1U7SWT9_TARSY\nKDR\tA0A1V4JGG1_PATFA\nKDR\tA0A218VFX7_9PASE\nKDR\tA0A250Y7G7_CASCN\nKDR\tA0A2B4R8S4_STYPI\nKDR\tA0A2B4R9J0_STYPI\nKDR\tA0A2C9PGE3_AEDAE\nKDR\tA0A2C9PGF4_AEDAE\nKDR\tA0A2C9PGF5_AEDAE\nKDR\tA0A2C9PGI2_AEDAE\nKDR\tA0A2C9PGI3_AEDAE\nKDR\tA0A2C9PGI5_AEDAE\nKDR\tA0A2C9PH24_AEDAE\nKDR\tA0A2D0Q0U8_ICTPU\nKDR\tA0A2D0Q2J2_ICTPU\nKDR\tA0A2D0Q3G5_ICTPU\nKDR\tA0A2I0MMR6_COLLI\nKDR\tA0A2I2UIN2_FELCA\nKDR\tA0A2I3HKR2_NOMLE\nKDR\tA0A2I4D494_9TELE\nKDR\tA0A2K5E9A8_AOTNA\nKDR\tA0A2K5JW57_COLAP\nKDR\tA0A2K5L771_CERAT\nKDR\tA0A2K5QSW9_CEBCA\nKDR\tA0A2K5VE63_MACFA\nKDR\tA0A2K5ZP61_MANLE\nKDR\tA0A2K6DVH4_MACNE\nKDR\tA0A2K6ETL3_PROCO\nKDR\tA0A2K6JUD3_RHIBE\nKDR\tA0A2K6Q7I7_RHIRO\nKDR\tA0A2K6TAN1_SAIBB\nKDR\tA0A2R9CED2_PANPA\nKDR\tA0A2S2Q083_9HEMI\nKDR\tA0A2S2QSH2_9HEMI\nKDR\tA0A2U3VMN3_ODORO\nKDR\tA0A2U4ARX6_TURTR\nKDR\tA0A2U4ARY0_TURTR\nKDR\tA0A2U4ARY9_TURTR\nKDR\tA0A2Y9FEK6_PHYMC\nKDR\tA0A2Y9HZU6_NEOSC\nKDR\tA0A2Y9ML28_DELLE\nKDR\tA0A2Y9MRB1_DELLE\nKDR\tA0A2Y9MRD4_DELLE\nKDR\tA0A337S7I2_FELCA\nKDR\tA0A340WTC3_LIPVE\nKDR\tA0A341BI95_NEOAA\nKDR\tA0A341BI97_NEOAA\nKDR\tA0A341BL62_NEOAA\nKDR\tA0A341BL63_NEOAA\nKDR\tA0A383Z051_BALAS\nKDR\tA0A384CY40_URSMA\nKDR\tA0A386TK93_AEDAE\nKDR\tA0A386TKB2_AEDAE\nKDR\tA0A386TKT6_AEDAE\nKDR\tA0A386TKZ9_AEDAE\nKDR\tA0A386TL59_AEDAE\nKDR\tA0A386TLK0_AEDAE\nKDR\tA0A3B3H9J1_ORYLA\nKDR\tA0A3B3I765_ORYLA\nKDR\tA0A3Q0G398_ALLSI\nKDR\tA0A3Q0G6U5_ALLSI\nKDR\tA0A3Q2HTL0_HORSE\nKDR\tA0A3Q2I213_HORSE\nKDR\tA0A3Q7QC90_CALUR\nKDR\tA0A3Q7SUC0_VULVU\nKDR\tA0A3Q7WSK3_URSAR\nKDR\tA0A3Q7XE71_URSAR\nKDR\tA0A3Q7XPB8_URSAR\nKDR\tA0A452DTF0_CAPHI\nKDR\tA0A452DTJ3_CAPHI\nKDR\tA0A452DTK5_CAPHI\nKDR\tA0A452IT68_9SAUR\nKDR\tA0A452RUL0_URSAM\nKDR\tA0A452RUP6_URSAM\nKDR\tA0A452TI79_URSMA\nKDR\tA0A452TIB7_URSMA\nKDR\tA0A452TIC7_URSMA\nKDR\tA0A452TII1_URSMA\nKDR\tA0A452TIP4_URSMA\nKDR\tA0A452TIW7_URSMA\nKDR\tA0A455BF76_PHYMC\nKDR\tA0A455BFW0_PHYMC\nKDR\tA0A455BIA2_PHYMC\nKDR\tA0A455BUW6_PHYMC\nKDR\tA0A4P8DGT6_AEDAE\nKDR\tA0A4P8DGU7_AEDAE\nKDR\tA0A4P8DH00_AEDAE\nKDR\tA0A4P8DH21_AEDAE\nKDR\tA0A4P8DH45_AEDAE\nKDR\tA0A4W2D952_BOBOX\nKDR\tA0A4W2FF35_BOBOX\nKDR\tA0A4W3III1_CALMI\nKDR\tA0A4W3IIQ2_CALMI\nKDR\tA0A4W3IM00_CALMI\nKDR\tA0A4W3IM12_CALMI\nKDR\tA0A4W3IM20_CALMI\nKDR\tA0A4W3IRK4_CALMI\nKDR\tA0A4W3IRM4_CALMI\nKDR\tA0A4W3IRM9_CALMI\nKDR\tA0A4W3IV93_CALMI\nKDR\tA0A4W3JCV7_CALMI\nKDR\tA0A4W6CE05_LATCA\nKDR\tA0A4W6CF56_LATCA\nKDR\tA0A4W6CIS6_LATCA\nKDR\tA0A4W6CJ20_LATCA\nKDR\tA0A4W6CJ54_LATCA\nKDR\tA0A4W6CK86_LATCA\nKDR\tA0A4W6CKA9_LATCA\nKDR\tA0A4W6CKB6_LATCA\nKDR\tA0A4W6CKC0_LATCA\nKDR\tA0A4X1U5F1_PIG\nKDR\tA0A4X1UAE3_PIG\nKDR\tA0A4X2KQ52_VOMUR\nKDR\tA0A4Y2I0J9_ARAVE\nKDR\tA0A5B7CVK3_PORTR\nKDR\tA0A5B7IFK4_PORTR\nKDR\tA0A5G2QBK4_PIG\nKDR\tA0A5G3KP42_XENTR\nKDR\tA0A5G3KR28_XENTR\nKDR\tA0A5G3KR64_XENTR\nKDR\tE7CW71_GRAMO\nKDR\tF1MWF1_BOVIN\nKDR\tF1NW21_CHICK\nKDR\tF1RWF1_PIG\nKDR\tF6RPC6_XENTR\nKDR\tF6U3B1_ORNAN\nKDR\tF6ZTV3_CALJA\nKDR\tF6ZYN4_XENTR\nKDR\tF7BUI5_XENTR\nKDR\tF7CET7_HORSE\nKDR\tF7DLA5_MONDO\nKDR\tF7E313_MACMU\nKDR\tG1KJF5_ANOCA\nKDR\tG1MJC8_AILME\nKDR\tG1NGX1_MELGA\nKDR\tG1PNH9_MYOLU\nKDR\tG1RAQ1_NOMLE\nKDR\tG1SN50_RABIT\nKDR\tG3TDA1_LOXAF\nKDR\tG3UPD9_MELGA\nKDR\tG3W302_SARHA\nKDR\tH0VFW3_CAVPO\nKDR\tH0X9F8_OTOGA\nKDR\tH0ZBE3_TAEGU\nKDR\tH2L4N1_ORYLA\nKDR\tH2PDE2_PONAB\nKDR\tH2QPH6_PANTR\nKDR\tH2UQU3_TAKRU\nKDR\tH2UQU4_TAKRU\nKDR\tH3ANA5_LATCH\nKDR\tI3J4B8_ORENI\nKDR\tI3MR17_ICTTR\nKDR\tK7FRT4_PELSI\nKDR\tK7GNF0_PIG\nKDR\tM1V1Y6_9SPHN\nKDR\tM3WBW2_FELCA\nKDR\tM3Y1D3_MUSPF\nKDR\tPRE6_ARATH\nKDR\tQ0N3S3_ANOFN\nKDR\tQ2XPT7_CANLF\nKDR\tQ3UQZ6_MOUSE\nKDR\tQ5DUF3_CHICK\nKDR\tQ5PQU0_RAT\nKDR\tQ5XL25_CYDPO\nKDR\tQ6XA00_HORSE\nKDR\tQ8CD05_MOUSE\nKDR\tQ8I9S4_MUSDO\nKDR\tQ8SPP2_MACMU\nKDR\tQ8VCD0_MOUSE\nKDR\tU3IDH1_ANAPP\nKDR\tU3JLX0_FICAL\nKDR\tU3JLX1_FICAL\nKDR\tV5MYD1_AEDAE\nKDR\tV5MZ29_AEDAE\nKDR\tV8NC22_OPHHA\nKDR\tV8NER0_OPHHA\nKDR\tV8NF40_OPHHA\nKDR\tVGFR2_COTJA\nKDR\tVGFR2_DANRE\nKDR\tVGFR2_HUMAN\nKDR\tVGFR2_MOUSE\nKDR\tVGFR2_RAT\nKDR\tVGFR4_DANRE\nKDR\tW5NSG6_SHEEP\n','From\tTo\nPAK1\tA0A024R5P0_HUMAN\nPAK1\tA0A034VAY1_BACDO\nPAK1\tA0A034W8W4_BACDO\nPAK1\tA0A034WAI7_BACDO\nPAK1\tA0A096MVT9_PAPAN\nPAK1\tA0A096N2S4_PAPAN\nPAK1\tA0A0A1XLK0_ZEUCU\nPAK1\tA0A0A9W6V6_LYGHE\nPAK1\tA0A0D9QV01_CHLSB\nPAK1\tA0A0R4IC19_DANRE\nPAK1\tA0A0R4IH73_DANRE\nPAK1\tA0A0S7HJC7_9TELE\nPAK1\tA0A0S7IKQ7_9TELE\nPAK1\tA0A0S7IN49_9TELE\nPAK1\tA0A0S7JLM0_9TELE\nPAK1\tA0A0S7JLN7_9TELE\nPAK1\tA0A0S7JLP5_9TELE\nPAK1\tA0A0S7JLQ1_9TELE\nPAK1\tA0A0S7JLR3_9TELE\nPAK1\tA0A0S7JLS8_9TELE\nPAK1\tA0A0S7JN32_9TELE\nPAK1\tA0A0S7JNL6_9TELE\nPAK1\tA0A0S7JNP7_9TELE\nPAK1\tA0A0S7JNW1_9TELE\nPAK1\tA0A0U1RQ87_MOUSE\nPAK1\tA0A0V0SDF7_9BILA\nPAK1\tA0A0V0UDV0_9BILA\nPAK1\tA0A0V0X784_9BILA\nPAK1\tA0A0V0YAY2_TRIPS\nPAK1\tA0A0V1ABN6_9BILA\nPAK1\tA0A0V1D764_TRIBR\nPAK1\tA0A0V1EGS6_TRIPS\nPAK1\tA0A0V1FC74_TRIPS\nPAK1\tA0A0V1H4D6_9BILA\nPAK1\tA0A0V1INA6_TRIPS\nPAK1\tA0A0V1JWR4_TRIPS\nPAK1\tA0A0V1LM05_9BILA\nPAK1\tA0A0V1MNU9_9BILA\nPAK1\tA0A0V1P6E2_9BILA\nPAK1\tA0A0V1P6H9_9BILA\nPAK1\tA0A177ADR3_9PEZI\nPAK1\tA0A1A7WIZ2_9TELE\nPAK1\tA0A1A8C4Q4_9TELE\nPAK1\tA0A1A8FN70_9TELE\nPAK1\tA0A1A8GSQ9_9TELE\nPAK1\tA0A1A8ID84_NOTKU\nPAK1\tA0A1A8V6B1_NOTFU\nPAK1\tA0A1D5RKP7_MACMU\nPAK1\tA0A1J4K913_9EUKA\nPAK1\tA0A1S3A5M2_ERIEU\nPAK1\tA0A1S3F098_DIPOR\nPAK1\tA0A1U7RL34_ALLSI\nPAK1\tA0A250YJV6_CASCN\nPAK1\tA0A286ZYT3_PIG\nPAK1\tA0A2D0QUT1_ICTPU\nPAK1\tA0A2H4R0M8_STIJA\nPAK1\tA0A2I0M2P1_COLLI\nPAK1\tA0A2I3GP44_NOMLE\nPAK1\tA0A2I3H6Y7_NOMLE\nPAK1\tA0A2I3SIE3_PANTR\nPAK1\tA0A2I3TG11_PANTR\nPAK1\tA0A2I4BUD8_9TELE\nPAK1\tA0A2K5C4L4_AOTNA\nPAK1\tA0A2K5C4P1_AOTNA\nPAK1\tA0A2K5C4Q3_AOTNA\nPAK1\tA0A2K5HQ00_COLAP\nPAK1\tA0A2K5HQ45_COLAP\nPAK1\tA0A2K5LUS9_CERAT\nPAK1\tA0A2K5QWC2_CEBCA\nPAK1\tA0A2K5QWE7_CEBCA\nPAK1\tA0A2K5QWJ5_CEBCA\nPAK1\tA0A2K5W9B3_MACFA\nPAK1\tA0A2K5W9C7_MACFA\nPAK1\tA0A2K5W9G9_MACFA\nPAK1\tA0A2K5YSN5_MANLE\nPAK1\tA0A2K5YT75_MANLE\nPAK1\tA0A2K5YT91_MANLE\nPAK1\tA0A2K6E8P1_MACNE\nPAK1\tA0A2K6E8T4_MACNE\nPAK1\tA0A2K6E8V3_MACNE\nPAK1\tA0A2K6F0N4_PROCO\nPAK1\tA0A2K6LB27_RHIBE\nPAK1\tA0A2K6LB33_RHIBE\nPAK1\tA0A2K6RTM5_RHIRO\nPAK1\tA0A2K6RTN1_RHIRO\nPAK1\tA0A2K6RTN2_RHIRO\nPAK1\tA0A2K6V2F4_SAIBB\nPAK1\tA0A2K6V2F5_SAIBB\nPAK1\tA0A2K6V2G6_SAIBB\nPAK1\tA0A2R9CEA2_PANPA\nPAK1\tA0A2R9CED6_PANPA\nPAK1\tA0A2R9CPC4_PANPA\nPAK1\tA0A2S2PID1_SCHGA\nPAK1\tA0A2S2QGG9_9HEMI\nPAK1\tA0A2U3VBI6_ODORO\nPAK1\tA0A2U3Y773_LEPWE\nPAK1\tA0A2U4CFJ0_TURTR\nPAK1\tA0A2Y9I8W1_NEOSC\nPAK1\tA0A2Y9PVS7_DELLE\nPAK1\tA0A2Y9TCT9_PHYMC\nPAK1\tA0A340XNR4_LIPVE\nPAK1\tA0A341BTC1_NEOAA\nPAK1\tA0A341BUV1_NEOAA\nPAK1\tA0A384DEZ9_URSMA\nPAK1\tA0A3B3H728_ORYLA\nPAK1\tA0A3B3HET5_ORYLA\nPAK1\tA0A3Q0CWT5_MESAU\nPAK1\tA0A3Q0E5E7_TARSY\nPAK1\tA0A3Q0EBI9_TARSY\nPAK1\tA0A3Q0GDT6_ALLSI\nPAK1\tA0A3Q7QW86_CALUR\nPAK1\tA0A3Q7QWA0_CALUR\nPAK1\tA0A3Q7UFL3_VULVU\nPAK1\tA0A3Q7Y1B2_URSAR\nPAK1\tA0A3R7FAZ8_CLOSI\nPAK1\tA0A3R7GKK4_CLOSI\nPAK1\tA0A419PPJ4_CLOSI\nPAK1\tA0A419Q1S4_CLOSI\nPAK1\tA0A419QEM0_CLOSI\nPAK1\tA0A452CKW1_BALAS\nPAK1\tA0A452DP45_CAPHI\nPAK1\tA0A452DPB5_CAPHI\nPAK1\tA0A452DPI1_CAPHI\nPAK1\tA0A452IYU6_9SAUR\nPAK1\tA0A452QNN5_URSAM\nPAK1\tA0A4W2I0B8_BOBOX\nPAK1\tA0A4W4HFH2_ELEEL\nPAK1\tA0A4W4HFJ1_ELEEL\nPAK1\tA0A4W4HIX6_ELEEL\nPAK1\tA0A4W4HLG3_ELEEL\nPAK1\tA0A4W4HN43_ELEEL\nPAK1\tA0A4W4HNL0_ELEEL\nPAK1\tA0A4X1UCQ5_PIG\nPAK1\tA0A4X1UCR1_PIG\nPAK1\tA0A4X1UI77_PIG\nPAK1\tA0A4X2LJ78_VOMUR\nPAK1\tA0A4Y2WQB7_ARAVE\nPAK1\tA0A4Z0XCS2_ARTOL\nPAK1\tA0A5B7EER7_PORTR\nPAK1\tA0A5C6G8J7_9HYPO\nPAK1\tA0A5F4BS20_CANLF\nPAK1\tA0A5F4BVK1_CANLF\nPAK1\tA0A5F4C732_CANLF\nPAK1\tA0A5F7ZUG6_MACMU\nPAK1\tA0A5F9CTI6_RABIT\nPAK1\tA0A5F9DF67_RABIT\nPAK1\tA0A5F9DGV0_RABIT\nPAK1\tA0A5G2QV02_PIG\nPAK1\tA0A5G2QY88_PIG\nPAK1\tA0A5G2R5X6_PIG\nPAK1\tA0A5G3HXD8_XENTR\nPAK1\tA0A5G3JNT0_XENTR\nPAK1\tA0A5K1VQY4_CALJA\nPAK1\tA0A5K1VSS6_PIG\nPAK1\tB3KNX7_HUMAN\nPAK1\tB7Z0W0_DROME\nPAK1\tD9IQE8_IXOSC\nPAK1\tE1C3P6_CHICK\nPAK1\tE2REH3_CANLF\nPAK1\tE5SJA2_TRISP\nPAK1\tE9PJF8_HUMAN\nPAK1\tE9PKH9_HUMAN\nPAK1\tE9PM17_HUMAN\nPAK1\tE9PMP2_HUMAN\nPAK1\tE9PQW5_HUMAN\nPAK1\tE9PRP6_HUMAN\nPAK1\tF1SUK2_PIG\nPAK1\tF6T2D3_XENTR\nPAK1\tF6XUP1_MACMU\nPAK1\tF6ZSD4_ORNAN\nPAK1\tF7A4M5_MONDO\nPAK1\tF7AM77_HORSE\nPAK1\tF7D0B4_MACMU\nPAK1\tF7D0C3_MACMU\nPAK1\tF7GZK8_CALJA\nPAK1\tG1LGB3_AILME\nPAK1\tG1NQT1_MELGA\nPAK1\tG1PMI5_MYOLU\nPAK1\tG1S463_NOMLE\nPAK1\tG1TLL9_RABIT\nPAK1\tG3BKE1_MUSSP\nPAK1\tG3BKE5_MOUSE\nPAK1\tG3T828_LOXAF\nPAK1\tG3TJL0_LOXAF\nPAK1\tG3UTB8_MELGA\nPAK1\tG3VVJ3_SARHA\nPAK1\tG5AW75_HETGA\nPAK1\tG5E884_MOUSE\nPAK1\tH0UTB5_CAVPO\nPAK1\tH0WY55_OTOGA\nPAK1\tH0YCG5_HUMAN\nPAK1\tH0YCM0_HUMAN\nPAK1\tH0YF55_HUMAN\nPAK1\tH0ZS31_TAEGU\nPAK1\tH2L8Y8_ORYLA\nPAK1\tH2NER3_PONAB\nPAK1\tH2Q4G3_PANTR\nPAK1\tH2SHM9_TAKRU\nPAK1\tH2SHN0_TAKRU\nPAK1\tH3AGW5_LATCH\nPAK1\tH9EM79_MACMU\nPAK1\tH9G5W4_ANOCA\nPAK1\tI3IYW0_ORENI\nPAK1\tI3IYW1_ORENI\nPAK1\tI3MJT6_ICTTR\nPAK1\tK7DSU6_PANTR\nPAK1\tK7FAU6_PELSI\nPAK1\tK7GRH8_PIG\nPAK1\tM3W1P1_FELCA\nPAK1\tM3YWY4_MUSPF\nPAK1\tO57318_XENLA\nPAK1\tO60018_PICAN\nPAK1\tPAK1_BOVIN\nPAK1\tPAK1_HUMAN\nPAK1\tPAK1_MOUSE\nPAK1\tPAK1_RAT\nPAK1\tPKN1_HUMAN\nPAK1\tPRK1_YEAST\nPAK1\tQ27169_PARTE\nPAK1\tQ30H42_CRYGA\nPAK1\tQ5Q022_SETTU\nPAK1\tQ66J11_XENLA\nPAK1\tQ6P0I7_DANRE\nPAK1\tQ6XPS1_CRYNV\nPAK1\tQ803Z0_DANRE\nPAK1\tQ9PW62_XENLA\nPAK1\tQ9U002_PARTE\nPAK1\tQ9VI13_DROME\nPAK1\tS4R2K7_MOUSE\nPAK1\tSAK1_YEAST\nPAK1\tSTE20_SCHPO\nPAK1\tU3DML4_CALJA\nPAK1\tU3EH84_CALJA\nPAK1\tU3JYS0_FICAL\nPAK1\tU6DTB2_NEOVI\nPAK1\tV8N1E8_OPHHA\nPAK1\tV8N8C8_OPHHA\nPAK1\tW5PBF0_SHEEP\nPAK1\tW8B049_CERCA\nPAK1\tW8BN03_CERCA\n','From\tTo\nHCK\tA0A096NUZ0_PAPAN\nHCK\tA0A0D9RJR2_CHLSB\nHCK\tA0A0R4J755_DANRE\nHCK\tA0A0S7IIB6_9TELE\nHCK\tA0A0S7IIE0_9TELE\nHCK\tA0A0S7IKH7_9TELE\nHCK\tA0A0S7IL58_9TELE\nHCK\tA0A0S7LYS9_9TELE\nHCK\tA0A0S7LYV9_9TELE\nHCK\tA0A0S7LZ53_9TELE\nHCK\tA0A0S7LZ55_9TELE\nHCK\tA0A0S7MHB7_9TELE\nHCK\tA0A0X3Q242_SCHSO\nHCK\tA0A1A7WQQ8_9TELE\nHCK\tA0A1A7YVG5_9TELE\nHCK\tA0A1A8BVP8_9TELE\nHCK\tA0A1A8DQ67_9TELE\nHCK\tA0A1A8EWH4_9TELE\nHCK\tA0A1A8GNG9_9TELE\nHCK\tA0A1A8I9W0_NOTKU\nHCK\tA0A1A8QVE1_9TELE\nHCK\tA0A1A8SG80_9TELE\nHCK\tA0A1A8UPS1_NOTFU\nHCK\tA0A1A8UUP5_NOTFU\nHCK\tA0A1L8ERL5_XENLA\nHCK\tA0A1S3EQC8_DIPOR\nHCK\tA0A1S3WB47_ERIEU\nHCK\tA0A1S3WBB5_ERIEU\nHCK\tA0A1U7SEU6_ALLSI\nHCK\tA0A1U7T4L7_TARSY\nHCK\tA0A1U7TBZ9_TARSY\nHCK\tA0A1U7TEC4_TARSY\nHCK\tA0A1U8DE75_ALLSI\nHCK\tA0A1V4KKM3_PATFA\nHCK\tA0A218US61_9PASE\nHCK\tA0A250YJ34_CASCN\nHCK\tA0A287BSA9_PIG\nHCK\tA0A2B4QZQ1_STYPI\nHCK\tA0A2D0SJ06_ICTPU\nHCK\tA0A2I0LN09_COLLI\nHCK\tA0A2I0LN26_COLLI\nHCK\tA0A2I2U9B2_FELCA\nHCK\tA0A2I3HUM8_NOMLE\nHCK\tA0A2I3MKW3_PAPAN\nHCK\tA0A2I3TD86_PANTR\nHCK\tA0A2I4BVJ6_9TELE\nHCK\tA0A2J8PYX8_PANTR\nHCK\tA0A2K5F287_AOTNA\nHCK\tA0A2K5JJX3_COLAP\nHCK\tA0A2K5M7C8_CERAT\nHCK\tA0A2K5VBG4_MACFA\nHCK\tA0A2K5ZFJ9_MANLE\nHCK\tA0A2K6DQ69_MACNE\nHCK\tA0A2K6FCW8_PROCO\nHCK\tA0A2K6JTG1_RHIBE\nHCK\tA0A2K6JTG7_RHIBE\nHCK\tA0A2K6RJP6_RHIRO\nHCK\tA0A2K6RJP8_RHIRO\nHCK\tA0A2K6SB12_SAIBB\nHCK\tA0A2R8MY76_CALJA\nHCK\tA0A2R9BJ64_PANPA\nHCK\tA0A2R9BT02_PANPA\nHCK\tA0A2U3VEK8_ODORO\nHCK\tA0A2U3YTH3_LEPWE\nHCK\tA0A2U3YTH7_LEPWE\nHCK\tA0A2U3ZNH9_ODORO\nHCK\tA0A2U4CL37_TURTR\nHCK\tA0A2Y9GZD3_NEOSC\nHCK\tA0A2Y9H5R2_NEOSC\nHCK\tA0A2Y9PT83_DELLE\nHCK\tA0A2Y9PYK7_DELLE\nHCK\tA0A340WBM8_LIPVE\nHCK\tA0A340WHD7_LIPVE\nHCK\tA0A341B7S0_NEOAA\nHCK\tA0A341B7U5_NEOAA\nHCK\tA0A341B9M5_NEOAA\nHCK\tA0A341BB34_NEOAA\nHCK\tA0A341BB43_NEOAA\nHCK\tA0A384B1B9_BALAS\nHCK\tA0A384B246_BALAS\nHCK\tA0A384D3Y0_URSMA\nHCK\tA0A3B3IGN4_ORYLA\nHCK\tA0A3B5KEM5_TAKRU\nHCK\tA0A3Q0CDU7_MESAU\nHCK\tA0A3Q1MYA3_BOVIN\nHCK\tA0A3Q3AH40_CHICK\nHCK\tA0A3Q7PDK1_CALUR\nHCK\tA0A3Q7PNS7_CALUR\nHCK\tA0A3Q7RI48_VULVU\nHCK\tA0A3Q7W217_URSAR\nHCK\tA0A3Q7WXQ1_URSAR\nHCK\tA0A452CQ43_BALAS\nHCK\tA0A452CQ98_BALAS\nHCK\tA0A452FJZ3_CAPHI\nHCK\tA0A452FK03_CAPHI\nHCK\tA0A452FK13_CAPHI\nHCK\tA0A452FK69_CAPHI\nHCK\tA0A452ILD9_9SAUR\nHCK\tA0A452SDS3_URSAM\nHCK\tA0A452SDW5_URSAM\nHCK\tA0A452SDX7_URSAM\nHCK\tA0A452SYD9_URSMA\nHCK\tA0A452SYJ1_URSMA\nHCK\tA0A452SYM2_URSMA\nHCK\tA0A455CAH1_PHYMC\nHCK\tA0A4W2H220_BOBOX\nHCK\tA0A4W2IHX9_BOBOX\nHCK\tA0A4W3JCH3_CALMI\nHCK\tA0A4W3JCJ6_CALMI\nHCK\tA0A4W3JCK6_CALMI\nHCK\tA0A4W3JNG5_CALMI\nHCK\tA0A4W3JPC0_CALMI\nHCK\tA0A4W3JPC5_CALMI\nHCK\tA0A4W3JSE3_CALMI\nHCK\tA0A4W3JSI9_CALMI\nHCK\tA0A4W3JSJ4_CALMI\nHCK\tA0A4W4FAZ3_ELEEL\nHCK\tA0A4W4FC38_ELEEL\nHCK\tA0A4W4FC52_ELEEL\nHCK\tA0A4W4FCG6_ELEEL\nHCK\tA0A4W4FCM5_ELEEL\nHCK\tA0A4W4FDM0_ELEEL\nHCK\tA0A4W4FDU0_ELEEL\nHCK\tA0A4W4FDX1_ELEEL\nHCK\tA0A4W4FE13_ELEEL\nHCK\tA0A4W4FE40_ELEEL\nHCK\tA0A4W6BNZ5_LATCA\nHCK\tA0A4W6BPN4_LATCA\nHCK\tA0A4W6BTP0_LATCA\nHCK\tA0A4W6BW07_LATCA\nHCK\tA0A4W6BW83_LATCA\nHCK\tA0A4W6BWG6_LATCA\nHCK\tA0A4W6BWL3_LATCA\nHCK\tA0A4W6BWL8_LATCA\nHCK\tA0A4W6BWR1_LATCA\nHCK\tA0A4X1SUD3_PIG\nHCK\tA0A4X2LB35_VOMUR\nHCK\tA0A4Z2FGE2_9TELE\nHCK\tA0A5B7E4N7_PORTR\nHCK\tA0A5F4WLR2_CALJA\nHCK\tA0A5F4WME7_CALJA\nHCK\tA0A5F9C6P5_RABIT\nHCK\tA0A5G3IMD6_XENTR\nHCK\tC1BXS5_ESOLU\nHCK\tD2H906_AILME\nHCK\tE1BXA3_CHICK\nHCK\tE7F0I7_DANRE\nHCK\tF1MTD8_BOVIN\nHCK\tF1MWN2_BOVIN\nHCK\tF1PSI2_CANLF\nHCK\tF6RBD8_MONDO\nHCK\tF6UND7_MOUSE\nHCK\tF7AT17_HORSE\nHCK\tF7H8A7_MACMU\nHCK\tF7HZQ6_CALJA\nHCK\tG1M4J6_AILME\nHCK\tG1N615_MELGA\nHCK\tG1PBA7_MYOLU\nHCK\tG1RF76_NOMLE\nHCK\tG1SXU8_RABIT\nHCK\tG3T700_LOXAF\nHCK\tG3ULK6_LOXAF\nHCK\tG3WE58_SARHA\nHCK\tG3WE59_SARHA\nHCK\tH0VB15_CAVPO\nHCK\tH0WS71_OTOGA\nHCK\tH0Y3C5_HUMAN\nHCK\tH0Z868_TAEGU\nHCK\tH2LAN1_ORYLA\nHCK\tH2P1L5_PONAB\nHCK\tH2RUE1_TAKRU\nHCK\tH7BXG4_HUMAN\nHCK\tH7C5P9_HUMAN\nHCK\tH9EZ11_MACMU\nHCK\tH9FAT6_MACMU\nHCK\tH9GEL5_ANOCA\nHCK\tH9YVQ2_MACMU\nHCK\tHCK_HUMAN\nHCK\tHCK_MACFA\nHCK\tHCK_MOUSE\nHCK\tHCK_RAT\nHCK\tI3M562_ICTTR\nHCK\tJ3KPD6_HUMAN\nHCK\tK7F786_PELSI\nHCK\tM3Z2G4_MUSPF\nHCK\tO93411_XENLA\nHCK\tQ5FW27_XENTR\nHCK\tQ9DDK6_SALSA\nHCK\tU3JSC9_FICAL\nHCK\tU6DGA1_NEOVI\nHCK\tV9KMS5_CALMI\nHCK\tW5NXU7_SHEEP\nHCK\tW5NXU9_SHEEP\nHCK\tW5U9K8_ICTPU\n','From\tTo\nOXSR1\tA0A034VXJ7_BACDO\nOXSR1\tA0A061S1D0_9CHLO\nOXSR1\tA0A096MTU2_PAPAN\nOXSR1\tA0A0A9Y005_LYGHE\nOXSR1\tA0A0B2VYC0_TOXCA\nOXSR1\tA0A0B8RTE4_PIG\nOXSR1\tA0A0D9RPJ4_CHLSB\nOXSR1\tA0A0S7EW04_9TELE\nOXSR1\tA0A0S7FFA6_9TELE\nOXSR1\tA0A0S7FFF7_9TELE\nOXSR1\tA0A0S7FG72_9TELE\nOXSR1\tA0A0S7FJ52_9TELE\nOXSR1\tA0A0S7FJV0_9TELE\nOXSR1\tA0A0S7FM34_9TELE\nOXSR1\tA0A0S7HI35_9TELE\nOXSR1\tA0A0S7HIL2_9TELE\nOXSR1\tA0A0S7HIM1_9TELE\nOXSR1\tA0A0S7HIU6_9TELE\nOXSR1\tA0A0S7HLX9_9TELE\nOXSR1\tA0A0S7IG46_9TELE\nOXSR1\tA0A1B8Y962_XENTR\nOXSR1\tA0A1D5R0F4_MACMU\nOXSR1\tA0A1J4JDH5_9EUKA\nOXSR1\tA0A1J4JR57_9EUKA\nOXSR1\tA0A1Q9EAT8_SYMMI\nOXSR1\tA0A1S2ZC81_ERIEU\nOXSR1\tA0A1S3GBT9_DIPOR\nOXSR1\tA0A1S3W3W9_ERIEU\nOXSR1\tA0A1V4KLA9_PATFA\nOXSR1\tA0A218UXG5_9PASE\nOXSR1\tA0A250Y2C5_CASCN\nOXSR1\tA0A2B4RNV7_STYPI\nOXSR1\tA0A2D0PPP8_ICTPU\nOXSR1\tA0A2D0PRC4_ICTPU\nOXSR1\tA0A2D0PT04_ICTPU\nOXSR1\tA0A2D0PTX3_ICTPU\nOXSR1\tA0A2I0MQS3_COLLI\nOXSR1\tA0A2I3G5V9_NOMLE\nOXSR1\tA0A2I3GWJ4_NOMLE\nOXSR1\tA0A2I3HTY9_NOMLE\nOXSR1\tA0A2I3LDF9_PAPAN\nOXSR1\tA0A2I3LZX4_PAPAN\nOXSR1\tA0A2I3MX73_PAPAN\nOXSR1\tA0A2I3RAY0_PANTR\nOXSR1\tA0A2K5CEQ2_AOTNA\nOXSR1\tA0A2K5CER0_AOTNA\nOXSR1\tA0A2K5HTC8_COLAP\nOXSR1\tA0A2K5HTE8_COLAP\nOXSR1\tA0A2K5HTF6_COLAP\nOXSR1\tA0A2K5KV82_CERAT\nOXSR1\tA0A2K5KV88_CERAT\nOXSR1\tA0A2K5KVB2_CERAT\nOXSR1\tA0A2K5RQ14_CEBCA\nOXSR1\tA0A2K5RQ20_CEBCA\nOXSR1\tA0A2K5RQ36_CEBCA\nOXSR1\tA0A2K5U420_MACFA\nOXSR1\tA0A2K5XQ24_MANLE\nOXSR1\tA0A2K5XQ27_MANLE\nOXSR1\tA0A2K6CX13_MACNE\nOXSR1\tA0A2K6CX15_MACNE\nOXSR1\tA0A2K6EKX2_PROCO\nOXSR1\tA0A2K6EKX4_PROCO\nOXSR1\tA0A2K6L5I7_RHIBE\nOXSR1\tA0A2K6L5J0_RHIBE\nOXSR1\tA0A2K6PQU6_RHIRO\nOXSR1\tA0A2K6PQW1_RHIRO\nOXSR1\tA0A2K6PQW7_RHIRO\nOXSR1\tA0A2K6SW36_SAIBB\nOXSR1\tA0A2K6SW49_SAIBB\nOXSR1\tA0A2K6SW54_SAIBB\nOXSR1\tA0A2P8Z613_BLAGE\nOXSR1\tA0A2R8MXM4_CALJA\nOXSR1\tA0A2R9A1K9_PANPA\nOXSR1\tA0A2R9A600_PANPA\nOXSR1\tA0A2U3WSP4_ODORO\nOXSR1\tA0A2U3ZSU3_ODORO\nOXSR1\tA0A2U4CAF7_TURTR\nOXSR1\tA0A2U4CAG0_TURTR\nOXSR1\tA0A2U4CAG3_TURTR\nOXSR1\tA0A2U4CAM9_TURTR\nOXSR1\tA0A2Y9G2C4_NEOSC\nOXSR1\tA0A2Y9LWK0_DELLE\nOXSR1\tA0A2Y9TI44_PHYMC\nOXSR1\tA0A340XTU0_LIPVE\nOXSR1\tA0A340XUG7_LIPVE\nOXSR1\tA0A341CBP3_NEOAA\nOXSR1\tA0A341CCA2_NEOAA\nOXSR1\tA0A384AVV9_BALAS\nOXSR1\tA0A384DN23_URSMA\nOXSR1\tA0A3Q0DAJ4_MESAU\nOXSR1\tA0A3Q0EB90_TARSY\nOXSR1\tA0A3Q0EEP5_TARSY\nOXSR1\tA0A3Q0EH42_TARSY\nOXSR1\tA0A3Q0GHN0_ALLSI\nOXSR1\tA0A3Q2U9U9_CHICK\nOXSR1\tA0A3Q7Q251_CALUR\nOXSR1\tA0A3Q7RHV4_CALUR\nOXSR1\tA0A3Q7SAD4_VULVU\nOXSR1\tA0A3Q7TNQ4_VULVU\nOXSR1\tA0A3Q7VU87_URSAR\nOXSR1\tA0A3Q7WDR8_URSAR\nOXSR1\tA0A3R7FBD9_CLOSI\nOXSR1\tA0A419PGF0_CLOSI\nOXSR1\tA0A452EZ97_CAPHI\nOXSR1\tA0A452IDS9_9SAUR\nOXSR1\tA0A452QMR7_URSAM\nOXSR1\tA0A452QMV6_URSAM\nOXSR1\tA0A452QMW5_URSAM\nOXSR1\tA0A452QN26_URSAM\nOXSR1\tA0A452QN38_URSAM\nOXSR1\tA0A452QN47_URSAM\nOXSR1\tA0A480I155_PIG\nOXSR1\tA0A4D8YLU5_SALSN\nOXSR1\tA0A4D8YYB9_SALSN\nOXSR1\tA0A4D8YYV7_SALSN\nOXSR1\tA0A4D8Z4K5_SALSN\nOXSR1\tA0A4D8ZER6_SALSN\nOXSR1\tA0A4D8ZFI8_SALSN\nOXSR1\tA0A4D8ZJ57_SALSN\nOXSR1\tA0A4D9A4A5_SALSN\nOXSR1\tA0A4D9AJI3_SALSN\nOXSR1\tA0A4D9AKZ3_SALSN\nOXSR1\tA0A4D9B537_SALSN\nOXSR1\tA0A4D9BBM8_SALSN\nOXSR1\tA0A4D9BUV8_SALSN\nOXSR1\tA0A4W2GPA2_BOBOX\nOXSR1\tA0A4W2GRI6_BOBOX\nOXSR1\tA0A4W2IBG9_BOBOX\nOXSR1\tA0A4W2IBI7_BOBOX\nOXSR1\tA0A4W4FEL0_ELEEL\nOXSR1\tA0A4W4FEP9_ELEEL\nOXSR1\tA0A4W4FGQ7_ELEEL\nOXSR1\tA0A4W4FGY8_ELEEL\nOXSR1\tA0A4W4FH03_ELEEL\nOXSR1\tA0A4W4FHA8_ELEEL\nOXSR1\tA0A4W4FHD7_ELEEL\nOXSR1\tA0A4X1TT38_PIG\nOXSR1\tA0A4X2KUP6_VOMUR\nOXSR1\tA0A4Y2ADS2_ARAVE\nOXSR1\tA0A5B7DMM6_PORTR\nOXSR1\tA0A5F4C8S8_CANLF\nOXSR1\tA0A5F4DCV5_CANLF\nOXSR1\tA0A5G2Q8B4_PIG\nOXSR1\tA0A5G2Q8F9_PIG\nOXSR1\tA0A5G3I2D8_XENTR\nOXSR1\tA0JPG0_XENLA\nOXSR1\tC9JIG9_HUMAN\nOXSR1\tD3ZUC9_RAT\nOXSR1\tD6RFV0_MOUSE\nOXSR1\tE1C6W3_CHICK\nOXSR1\tE2RAI8_CANLF\nOXSR1\tE6ZIW2_DICLA\nOXSR1\tF1MYV9_BOVIN\nOXSR1\tF6VHT2_XENTR\nOXSR1\tF6ZSB5_XENTR\nOXSR1\tF7AVI9_MOUSE\nOXSR1\tF7BZ31_XENTR\nOXSR1\tF7CMY7_HORSE\nOXSR1\tF7EBS8_CALJA\nOXSR1\tF7GAL3_ORNAN\nOXSR1\tF8WBK9_HUMAN\nOXSR1\tG1KJI2_ANOCA\nOXSR1\tG1MHD2_AILME\nOXSR1\tG1PJ04_MYOLU\nOXSR1\tG1QZY4_NOMLE\nOXSR1\tG7NYK6_MACFA\nOXSR1\tH0VDD6_CAVPO\nOXSR1\tH0WZU6_OTOGA\nOXSR1\tH0YQ83_TAEGU\nOXSR1\tH2QMA9_PANTR\nOXSR1\tH2V6C9_TAKRU\nOXSR1\tI3N8N6_ICTTR\nOXSR1\tL8ECC0_HUMAN\nOXSR1\tM3WGU7_FELCA\nOXSR1\tM3XU90_MUSPF\nOXSR1\tOXSR1_HUMAN\nOXSR1\tOXSR1_MOUSE\nOXSR1\tOXSR1_PIG\nOXSR1\tOXSR1_PONAB\nOXSR1\tQ1JQD5_BOVIN\nOXSR1\tQ6DD36_XENLA\nOXSR1\tQ6NVS1_XENTR\nOXSR1\tQ8VDZ9_MOUSE\nOXSR1\tT2MI78_HYDVU\nOXSR1\tU3DRL9_CALJA\nOXSR1\tU3JVV6_FICAL\nOXSR1\tU6DFH3_NEOVI\nOXSR1\tV8NP28_OPHHA\nOXSR1\tW5NSJ5_SHEEP\nOXSR1\tW8AZ88_CERCA\nOXSR1\tW8BXZ3_CERCA\n','From\tTo\nSRC\tA0A087Y5B9_POEFO\nSRC\tA0A096LTJ9_POEFO\nSRC\tA0A096NXH1_PAPAN\nSRC\tA0A0D9RML6_CHLSB\nSRC\tA0A0S7I621_9TELE\nSRC\tA0A0S7I7B0_9TELE\nSRC\tA0A0S7I7U5_9TELE\nSRC\tA0A0S7IDZ9_9TELE\nSRC\tA0A0S7IE36_9TELE\nSRC\tA0A0S7IL46_9TELE\nSRC\tA0A0S7IUK0_9TELE\nSRC\tA0A0S7IX11_9TELE\nSRC\tA0A0V0SIA0_9BILA\nSRC\tA0A147AT11_FUNHE\nSRC\tA0A151M5E9_ALLMI\nSRC\tA0A151M5F1_ALLMI\nSRC\tA0A151M5F6_ALLMI\nSRC\tA0A1A7Z0J2_9TELE\nSRC\tA0A1A8BLY8_9TELE\nSRC\tA0A1A8D7F8_9TELE\nSRC\tA0A1A8G2W6_9TELE\nSRC\tA0A1A8I1Q1_NOTKU\nSRC\tA0A1A8KEK2_NOTKU\nSRC\tA0A1A8QEF3_9TELE\nSRC\tA0A1A8QER3_9TELE\nSRC\tA0A1A8SB18_9TELE\nSRC\tA0A1A8U1I9_NOTFU\nSRC\tA0A1B2INM9_APHGO\nSRC\tA0A1B2INN3_PLOIN\nSRC\tA0A1B2INN6_MONAT\nSRC\tA0A1I7Q3Z0_CHICK\nSRC\tA0A1S2ZFA5_ERIEU\nSRC\tA0A1S3ERI1_DIPOR\nSRC\tA0A1S3W5M3_ERIEU\nSRC\tA0A1U7QRN8_MESAU\nSRC\tA0A1U7R2X7_MESAU\nSRC\tA0A1V4KKX2_PATFA\nSRC\tA0A1V4KL44_PATFA\nSRC\tA0A218USR6_9PASE\nSRC\tA0A222NUJ6_PENVA\nSRC\tA0A286X7I1_CAVPO\nSRC\tA0A290DSG3_9NEOP\nSRC\tA0A2I0M2A8_COLLI\nSRC\tA0A2I3HYA5_NOMLE\nSRC\tA0A2I3M0W2_PAPAN\nSRC\tA0A2I3MCC2_PAPAN\nSRC\tA0A2I3N2L0_PAPAN\nSRC\tA0A2I3SIH9_PANTR\nSRC\tA0A2I4CZY8_9TELE\nSRC\tA0A2I4D006_9TELE\nSRC\tA0A2K5EEL4_AOTNA\nSRC\tA0A2K5EEL8_AOTNA\nSRC\tA0A2K5EEP2_AOTNA\nSRC\tA0A2K5KGI9_COLAP\nSRC\tA0A2K5KGK1_COLAP\nSRC\tA0A2K5KTS1_CERAT\nSRC\tA0A2K5KTU0_CERAT\nSRC\tA0A2K5KTV3_CERAT\nSRC\tA0A2K5KTY0_CERAT\nSRC\tA0A2K5PYF7_CEBCA\nSRC\tA0A2K5PYF8_CEBCA\nSRC\tA0A2K5PYM6_CEBCA\nSRC\tA0A2K5V063_MACFA\nSRC\tA0A2K5V077_MACFA\nSRC\tA0A2K5V0C0_MACFA\nSRC\tA0A2K6A5Q6_MANLE\nSRC\tA0A2K6A5S8_MANLE\nSRC\tA0A2K6A5T9_MANLE\nSRC\tA0A2K6B4I3_MACNE\nSRC\tA0A2K6B4J2_MACNE\nSRC\tA0A2K6B4J6_MACNE\nSRC\tA0A2K6B4P5_MACNE\nSRC\tA0A2K6FX73_PROCO\nSRC\tA0A2K6FX78_PROCO\nSRC\tA0A2K6JST8_RHIBE\nSRC\tA0A2K6JSU6_RHIBE\nSRC\tA0A2K6JSW0_RHIBE\nSRC\tA0A2K6PTY5_RHIRO\nSRC\tA0A2K6PTY8_RHIRO\nSRC\tA0A2K6PU05_RHIRO\nSRC\tA0A2K6U3T7_SAIBB\nSRC\tA0A2K6U3X0_SAIBB\nSRC\tA0A2K6U400_SAIBB\nSRC\tA0A2R8QE56_DANRE\nSRC\tA0A2R9CBS4_PANPA\nSRC\tA0A2R9CBS9_PANPA\nSRC\tA0A2U3XQI2_LEPWE\nSRC\tA0A2U3ZNJ0_ODORO\nSRC\tA0A2U4CM12_TURTR\nSRC\tA0A2Y9EJT6_PHYMC\nSRC\tA0A2Y9H031_NEOSC\nSRC\tA0A2Y9PMN9_DELLE\nSRC\tA0A2Y9PRE7_DELLE\nSRC\tA0A2Y9PSD7_DELLE\nSRC\tA0A337RYE7_FELCA\nSRC\tA0A337SLC3_FELCA\nSRC\tA0A340WDY6_LIPVE\nSRC\tA0A341B9I2_NEOAA\nSRC\tA0A384B0N3_BALAS\nSRC\tA0A384B0N7_BALAS\nSRC\tA0A384CND9_URSMA\nSRC\tA0A3B1IPI6_ASTMX\nSRC\tA0A3B1JU47_ASTMX\nSRC\tA0A3B1JYF9_ASTMX\nSRC\tA0A3B3IAW1_ORYLA\nSRC\tA0A3B3UPP1_9TELE\nSRC\tA0A3B3WIJ6_9TELE\nSRC\tA0A3B3Y5N5_9TELE\nSRC\tA0A3B4BQY6_PYGNA\nSRC\tA0A3B4DIM1_PYGNA\nSRC\tA0A3B4FDV2_9CICH\nSRC\tA0A3B5MCL2_9TELE\nSRC\tA0A3B5MCN0_9TELE\nSRC\tA0A3B5Q3N4_XIPMA\nSRC\tA0A3P8QRS9_ASTCA\nSRC\tA0A3P8QS25_ASTCA\nSRC\tA0A3P9B5P4_9CICH\nSRC\tA0A3P9B5R2_9CICH\nSRC\tA0A3P9P3K2_POERE\nSRC\tA0A3Q0EIL5_TARSY\nSRC\tA0A3Q0FHP5_ALLSI\nSRC\tA0A3Q0FLP8_ALLSI\nSRC\tA0A3Q0FLR3_ALLSI\nSRC\tA0A3Q0FLS3_ALLSI\nSRC\tA0A3Q0FN65_ALLSI\nSRC\tA0A3Q2CV20_CYPVA\nSRC\tA0A3Q2E3U8_CYPVA\nSRC\tA0A3Q2FSJ8_CYPVA\nSRC\tA0A3Q2GHK1_CYPVA\nSRC\tA0A3Q2GV71_HORSE\nSRC\tA0A3Q2H6Q8_HORSE\nSRC\tA0A3Q2T8V2_FUNHE\nSRC\tA0A3Q2UWS9_HAPBU\nSRC\tA0A3Q2WLL0_HAPBU\nSRC\tA0A3Q3ATK1_KRYMA\nSRC\tA0A3Q3B6Q0_KRYMA\nSRC\tA0A3Q3E9H9_9LABR\nSRC\tA0A3Q3EAE4_9LABR\nSRC\tA0A3Q3ED96_9LABR\nSRC\tA0A3Q3JJK9_MONAL\nSRC\tA0A3Q3L3G1_9TELE\nSRC\tA0A3Q3L3H6_9TELE\nSRC\tA0A3Q4GJW3_NEOBR\nSRC\tA0A3Q7T7R4_VULVU\nSRC\tA0A3Q7TW93_VULVU\nSRC\tA0A3Q7UDJ7_VULVU\nSRC\tA0A3Q7VDT6_URSAR\nSRC\tA0A452CQ32_BALAS\nSRC\tA0A452DS57_CAPHI\nSRC\tA0A452DS86_CAPHI\nSRC\tA0A452HZ29_9SAUR\nSRC\tA0A452HZ72_9SAUR\nSRC\tA0A452HZ84_9SAUR\nSRC\tA0A452HZB4_9SAUR\nSRC\tA0A452SCQ0_URSAM\nSRC\tA0A452SCT6_URSAM\nSRC\tA0A452T8R1_URSMA\nSRC\tA0A455BYT7_PHYMC\nSRC\tA0A455BYU2_PHYMC\nSRC\tA0A455BYU3_PHYMC\nSRC\tA0A455BZU3_PHYMC\nSRC\tA0A455BZU9_PHYMC\nSRC\tA0A455BZV4_PHYMC\nSRC\tA0A455BZV7_PHYMC\nSRC\tA0A455C2R7_PHYMC\nSRC\tA0A455C2S3_PHYMC\nSRC\tA0A455CA81_PHYMC\nSRC\tA0A455CA82_PHYMC\nSRC\tA0A4W2DGI4_BOBOX\nSRC\tA0A4W2FMS2_BOBOX\nSRC\tA0A4W2FMT6_BOBOX\nSRC\tA0A4X1THR5_PIG\nSRC\tA0A4X1TM07_PIG\nSRC\tA0A4X2L7B4_VOMUR\nSRC\tA0A4X2L7C3_VOMUR\nSRC\tA0A5B7DI37_PORTR\nSRC\tA0A5F4BT17_CANLF\nSRC\tA0A5F4CWW8_CANLF\nSRC\tA0A5F4D0H5_CANLF\nSRC\tA0A5F4W7Z4_CALJA\nSRC\tA0A5F8GEV3_MONDO\nSRC\tA0A5F8GUX0_MONDO\nSRC\tA0A5F9CMU1_RABIT\nSRC\tA0A5F9D729_RABIT\nSRC\tA0A5F9DDQ5_RABIT\nSRC\tA0A5G2QDX5_PIG\nSRC\tA0A5G3KV86_XENTR\nSRC\tA0A5G3KW72_XENTR\nSRC\tA0A5G3KWH8_XENTR\nSRC\tA0A5G3KWW8_XENTR\nSRC\tA1Z6I9_DROME\nSRC\tB2GS52_DANRE\nSRC\tE1BIM8_BOVIN\nSRC\tE2RSH8_CANLF\nSRC\tE6ZF21_DICLA\nSRC\tF1SEK8_PIG\nSRC\tF6PKK5_XENTR\nSRC\tF6PPD4_XENTR\nSRC\tF6PR77_XENTR\nSRC\tF6Q890_XENTR\nSRC\tF6QH40_HORSE\nSRC\tF6W7M1_CALJA\nSRC\tF6WR53_MONDO\nSRC\tF7DXH6_XENTR\nSRC\tF7FWN4_MACMU\nSRC\tF7IJA5_CALJA\nSRC\tF7IJA6_CALJA\nSRC\tF8WI90_MOUSE\nSRC\tG0T5Q0_MESNU\nSRC\tG1M0W5_AILME\nSRC\tG1MWN3_MELGA\nSRC\tG1P2J0_MYOLU\nSRC\tG1Q0I2_MYOLU\nSRC\tG1R359_NOMLE\nSRC\tG1SXH7_RABIT\nSRC\tG3NPS8_GASAC\nSRC\tG3TDK8_LOXAF\nSRC\tG3UBT7_LOXAF\nSRC\tG3ULY5_LOXAF\nSRC\tG3UPV9_MELGA\nSRC\tG3USG0_MELGA\nSRC\tG3UTK7_MELGA\nSRC\tG3WZ14_SARHA\nSRC\tG8FGL9_ELAGV\nSRC\tH0VI86_CAVPO\nSRC\tH0XDE5_OTOGA\nSRC\tH0Z3F7_TAEGU\nSRC\tH2LQT5_ORYLA\nSRC\tH2LQT6_ORYLA\nSRC\tH2P1V4_PONAB\nSRC\tH2QKB4_PANTR\nSRC\tH2VCQ8_TAKRU\nSRC\tH2VCQ9_TAKRU\nSRC\tH3ACF3_LATCH\nSRC\tH6WWN8_BOMMO\nSRC\tH9GID5_ANOCA\nSRC\tI0FJ06_MACMU\nSRC\tI3JIJ0_ORENI\nSRC\tI3JIJ1_ORENI\nSRC\tI3MRX6_ICTTR\nSRC\tI3MZR7_ICTTR\nSRC\tI4DHB4_BOMMO\nSRC\tK7B763_PANTR\nSRC\tK7G355_PELSI\nSRC\tK7GM45_PIG\nSRC\tK7GPR7_PIG\nSRC\tK7RW18_CANLF\nSRC\tM1UYR5_9CRUS\nSRC\tM1VDR2_DAPPU\nSRC\tM3W6V8_FELCA\nSRC\tM3Z238_MUSPF\nSRC\tM4A733_XIPMA\nSRC\tM9MSK3_DROME\nSRC\tO92806_RSVP\nSRC\tO92957_RSVSB\nSRC\tO93080_9RETR\nSRC\tQ07461_9RETR\nSRC\tQ2TAR1_XENLA\nSRC\tQ3UKD6_MOUSE\nSRC\tQ5MAS9_XENTR\nSRC\tQ60F75_ELACL\nSRC\tQ60F76_IGUIG\nSRC\tQ60F77_CAICR\nSRC\tQ60F78_GEOEL\nSRC\tQ60F79_GEOEL\nSRC\tQ60F80_CYNPY\nSRC\tQ60F81_CARCR\nSRC\tQ60F82_CARCR\nSRC\tQ64817_AVIS2\nSRC\tQ64993_RSVSR\nSRC\tQ6GYP0_9TELE\nSRC\tQ6GYP1_9TELE\nSRC\tQ6GYP2_9TELE\nSRC\tQ70W05_CIOIN\nSRC\tQ71UK5_HUMAN\nSRC\tQ7KSQ2_DROME\nSRC\tQ7ZX73_XENLA\nSRC\tQ85650_MSVMO\nSRC\tQ8INJ8_DROME\nSRC\tQ8W267_CARPA\nSRC\tQ9VGK8_DROME\nSRC\tSRC_CHICK\nSRC\tSRC_DANRE\nSRC\tSRC_HUMAN\nSRC\tSRC_MOUSE\nSRC\tSRC_RAT\nSRC\tT2MHF8_HYDVU\nSRC\tU3BV42_CALJA\nSRC\tU3FMT0_CALJA\nSRC\tU3JSL6_FICAL\nSRC\tU3JSM0_FICAL\nSRC\tU6DPX3_NEOVI\nSRC\tV5KXX2_LOCMI\nSRC\tV8PCK4_OPHHA\nSRC\tW5L535_ASTMX\nSRC\tW5Q964_SHEEP\nSRC\tW5Q965_SHEEP\nSRC\tW5UA54_ICTPU\nSRC\tX2JCI2_DROME\n','From\tTo\nTNIK\tA0A034VMB2_BACDO\nTNIK\tA0A034VPS4_BACDO\nTNIK\tA0A034VSC7_BACDO\nTNIK\tA0A096MXB6_PAPAN\nTNIK\tA0A0D9RJZ0_CHLSB\nTNIK\tA0A0G2K490_RAT\nTNIK\tA0A0P6J5Y1_HETGA\nTNIK\tA0A0S7G0P4_9TELE\nTNIK\tA0A0S7G0Q0_9TELE\nTNIK\tA0A0S7G7K0_9TELE\nTNIK\tA0A0S7H2S0_9TELE\nTNIK\tA0A0S7H2Y8_9TELE\nTNIK\tA0A0S7H2Z6_9TELE\nTNIK\tA0A0S7H402_9TELE\nTNIK\tA0A0S7H501_9TELE\nTNIK\tA0A0S7H581_9TELE\nTNIK\tA0A0S7H5S3_9TELE\nTNIK\tA0A0S7H702_9TELE\nTNIK\tA0A0S7H723_9TELE\nTNIK\tA0A0S7H7I0_9TELE\nTNIK\tA0A0S7H9H6_9TELE\nTNIK\tA0A0S7H9J6_9TELE\nTNIK\tA0A0S7HBT2_9TELE\nTNIK\tA0A0S7HHE2_9TELE\nTNIK\tA0A0S7ILM4_9TELE\nTNIK\tA0A0S7ILP4_9TELE\nTNIK\tA0A0S7IWQ5_9TELE\nTNIK\tA0A0S7KRG0_9TELE\nTNIK\tA0A0S7KRG8_9TELE\nTNIK\tA0A0S7KRJ3_9TELE\nTNIK\tA0A0S7LFB6_9TELE\nTNIK\tA0A0S7LFF4_9TELE\nTNIK\tA0A0S7LFF6_9TELE\nTNIK\tA0A0S7M1I7_9TELE\nTNIK\tA0A0S7M1Z3_9TELE\nTNIK\tA0A0S7M1Z4_9TELE\nTNIK\tA0A1A7W930_9TELE\nTNIK\tA0A1A7WBL3_9TELE\nTNIK\tA0A1A7WG98_9TELE\nTNIK\tA0A1A7WRD3_9TELE\nTNIK\tA0A1A7X7Z2_9TELE\nTNIK\tA0A1A7YLS2_9TELE\nTNIK\tA0A1A7YYK1_9TELE\nTNIK\tA0A1A7ZAD6_NOTFU\nTNIK\tA0A1A8A552_NOTFU\nTNIK\tA0A1A8AE45_NOTFU\nTNIK\tA0A1A8APR1_NOTFU\nTNIK\tA0A1A8AS10_NOTFU\nTNIK\tA0A1A8BTY9_9TELE\nTNIK\tA0A1A8BXN4_9TELE\nTNIK\tA0A1A8C441_9TELE\nTNIK\tA0A1A8D8E8_9TELE\nTNIK\tA0A1A8DTB3_9TELE\nTNIK\tA0A1A8DZC1_9TELE\nTNIK\tA0A1A8E5E6_9TELE\nTNIK\tA0A1A8EKY9_9TELE\nTNIK\tA0A1A8FHR4_9TELE\nTNIK\tA0A1A8FU71_9TELE\nTNIK\tA0A1A8GWD8_9TELE\nTNIK\tA0A1A8HL68_9TELE\nTNIK\tA0A1A8HTL9_NOTKU\nTNIK\tA0A1A8I7B5_NOTKU\nTNIK\tA0A1A8IHG2_NOTKU\nTNIK\tA0A1A8JF40_NOTKU\nTNIK\tA0A1A8JN88_NOTKU\nTNIK\tA0A1A8JP93_NOTKU\nTNIK\tA0A1A8K9U7_NOTKU\nTNIK\tA0A1A8KE62_NOTKU\nTNIK\tA0A1A8KHJ3_NOTKU\nTNIK\tA0A1A8KZN6_9TELE\nTNIK\tA0A1A8LHJ8_9TELE\nTNIK\tA0A1A8LJ25_9TELE\nTNIK\tA0A1A8LQ51_9TELE\nTNIK\tA0A1A8N178_9TELE\nTNIK\tA0A1A8NBQ6_9TELE\nTNIK\tA0A1A8Q560_9TELE\nTNIK\tA0A1A8Q833_9TELE\nTNIK\tA0A1A8QQ54_9TELE\nTNIK\tA0A1A8R1N8_9TELE\nTNIK\tA0A1A8R4F3_9TELE\nTNIK\tA0A1A8R798_9TELE\nTNIK\tA0A1A8RFV4_9TELE\nTNIK\tA0A1A8S2W3_9TELE\nTNIK\tA0A1A8SH64_9TELE\nTNIK\tA0A1A8UKA9_NOTFU\nTNIK\tA0A1A8UL27_NOTFU\nTNIK\tA0A1A8UNA3_NOTFU\nTNIK\tA0A1A8UXD6_NOTFU\nTNIK\tA0A1A8UY98_NOTFU\nTNIK\tA0A1A8VF67_NOTFU\nTNIK\tA0A1D5RCW4_MACMU\nTNIK\tA0A1L8G422_XENLA\nTNIK\tA0A1S2ZF60_ERIEU\nTNIK\tA0A1S3F4N3_DIPOR\nTNIK\tA0A1U7RX15_ALLSI\nTNIK\tA0A1U7TJB1_TARSY\nTNIK\tA0A1V4K3S4_PATFA\nTNIK\tA0A1V4K3U9_PATFA\nTNIK\tA0A218V3M5_9PASE\nTNIK\tA0A218V4C6_9PASE\nTNIK\tA0A250Y7I2_CASCN\nTNIK\tA0A286XD46_CAVPO\nTNIK\tA0A286XIV7_CAVPO\nTNIK\tA0A286XZM8_CAVPO\nTNIK\tA0A287A5W8_PIG\nTNIK\tA0A287BA39_PIG\nTNIK\tA0A287CSD7_ICTTR\nTNIK\tA0A287D754_ICTTR\nTNIK\tA0A287D8F3_ICTTR\nTNIK\tA0A287D9I9_ICTTR\nTNIK\tA0A2B4SE57_STYPI\nTNIK\tA0A2D0PQ28_ICTPU\nTNIK\tA0A2D0PQ33_ICTPU\nTNIK\tA0A2D0PRN4_ICTPU\nTNIK\tA0A2D0PRP8_ICTPU\nTNIK\tA0A2D0PRQ3_ICTPU\nTNIK\tA0A2D0PTD3_ICTPU\nTNIK\tA0A2D0PTD8_ICTPU\nTNIK\tA0A2D0PU97_ICTPU\nTNIK\tA0A2I0MBG6_COLLI\nTNIK\tA0A2I2UIX0_FELCA\nTNIK\tA0A2I2UJY6_FELCA\nTNIK\tA0A2I3FRS1_NOMLE\nTNIK\tA0A2I3H9D1_NOMLE\nTNIK\tA0A2I3HG20_NOMLE\nTNIK\tA0A2I3HZE8_NOMLE\nTNIK\tA0A2I3HZF9_NOMLE\nTNIK\tA0A2I3LVB9_PAPAN\nTNIK\tA0A2I3MB92_PAPAN\nTNIK\tA0A2I3MGE0_PAPAN\nTNIK\tA0A2I3MJ82_PAPAN\nTNIK\tA0A2I3MNN6_PAPAN\nTNIK\tA0A2I3RBR0_PANTR\nTNIK\tA0A2I3RIW2_PANTR\nTNIK\tA0A2I3RVD5_PANTR\nTNIK\tA0A2I3RYJ5_PANTR\nTNIK\tA0A2I3TX78_PANTR\nTNIK\tA0A2K5DWI7_AOTNA\nTNIK\tA0A2K5DWJ4_AOTNA\nTNIK\tA0A2K5DWL1_AOTNA\nTNIK\tA0A2K5DWL7_AOTNA\nTNIK\tA0A2K5DWM1_AOTNA\nTNIK\tA0A2K5DWU3_AOTNA\nTNIK\tA0A2K5JNM3_COLAP\nTNIK\tA0A2K5JNM4_COLAP\nTNIK\tA0A2K5JNM6_COLAP\nTNIK\tA0A2K5JNN3_COLAP\nTNIK\tA0A2K5JNQ5_COLAP\nTNIK\tA0A2K5JNR5_COLAP\nTNIK\tA0A2K5KL36_CERAT\nTNIK\tA0A2K5KL42_CERAT\nTNIK\tA0A2K5KL45_CERAT\nTNIK\tA0A2K5KL53_CERAT\nTNIK\tA0A2K5KL69_CERAT\nTNIK\tA0A2K5KL72_CERAT\nTNIK\tA0A2K5S5H6_CEBCA\nTNIK\tA0A2K5S5I0_CEBCA\nTNIK\tA0A2K5S5I6_CEBCA\nTNIK\tA0A2K5S5L0_CEBCA\nTNIK\tA0A2K5S5M7_CEBCA\nTNIK\tA0A2K5S5N8_CEBCA\nTNIK\tA0A2K5S5P6_CEBCA\nTNIK\tA0A2K5U9V1_MACFA\nTNIK\tA0A2K5U9Z8_MACFA\nTNIK\tA0A2K5UA16_MACFA\nTNIK\tA0A2K5UAA5_MACFA\nTNIK\tA0A2K5UAB2_MACFA\nTNIK\tA0A2K5UAB8_MACFA\nTNIK\tA0A2K5YHZ2_MANLE\nTNIK\tA0A2K5YI00_MANLE\nTNIK\tA0A2K5YI19_MANLE\nTNIK\tA0A2K5YI50_MANLE\nTNIK\tA0A2K5YI61_MANLE\nTNIK\tA0A2K5YI73_MANLE\nTNIK\tA0A2K6E699_MACNE\nTNIK\tA0A2K6E6A2_MACNE\nTNIK\tA0A2K6E6B5_MACNE\nTNIK\tA0A2K6E6B8_MACNE\nTNIK\tA0A2K6E6C5_MACNE\nTNIK\tA0A2K6E6F7_MACNE\nTNIK\tA0A2K6E6G9_MACNE\nTNIK\tA0A2K6FYH7_PROCO\nTNIK\tA0A2K6FYI0_PROCO\nTNIK\tA0A2K6FYI3_PROCO\nTNIK\tA0A2K6FYK5_PROCO\nTNIK\tA0A2K6MVA3_RHIBE\nTNIK\tA0A2K6MVC4_RHIBE\nTNIK\tA0A2K6MVC5_RHIBE\nTNIK\tA0A2K6MVC9_RHIBE\nTNIK\tA0A2K6MVD2_RHIBE\nTNIK\tA0A2K6MVD4_RHIBE\nTNIK\tA0A2K6MVE1_RHIBE\nTNIK\tA0A2K6MVE5_RHIBE\nTNIK\tA0A2K6RSD3_RHIRO\nTNIK\tA0A2K6RSE2_RHIRO\nTNIK\tA0A2K6RSE3_RHIRO\nTNIK\tA0A2K6RSE5_RHIRO\nTNIK\tA0A2K6RSF1_RHIRO\nTNIK\tA0A2K6RSF6_RHIRO\nTNIK\tA0A2K6RSF9_RHIRO\nTNIK\tA0A2K6TW36_SAIBB\nTNIK\tA0A2K6TW40_SAIBB\nTNIK\tA0A2K6TW48_SAIBB\nTNIK\tA0A2K6TW77_SAIBB\nTNIK\tA0A2K6TW92_SAIBB\nTNIK\tA0A2K6TWA3_SAIBB\nTNIK\tA0A2K6TWC6_SAIBB\nTNIK\tA0A2P8ZAJ3_BLAGE\nTNIK\tA0A2R8MM42_CALJA\nTNIK\tA0A2R8N493_CALJA\nTNIK\tA0A2R8PLS6_CALJA\nTNIK\tA0A2R9BCA9_PANPA\nTNIK\tA0A2R9BCC3_PANPA\nTNIK\tA0A2R9BG34_PANPA\nTNIK\tA0A2R9BG41_PANPA\nTNIK\tA0A2R9BG46_PANPA\nTNIK\tA0A2R9BG67_PANPA\nTNIK\tA0A2S2RA95_9HEMI\nTNIK\tA0A2U3VF08_ODORO\nTNIK\tA0A2U3VF21_ODORO\nTNIK\tA0A2U3VF26_ODORO\nTNIK\tA0A2U3VF27_ODORO\nTNIK\tA0A2U3VF35_ODORO\nTNIK\tA0A2U3VF41_ODORO\nTNIK\tA0A2U3VFH9_ODORO\nTNIK\tA0A2U3VFI4_ODORO\nTNIK\tA0A2U4C8U9_TURTR\nTNIK\tA0A2Y9I397_NEOSC\nTNIK\tA0A2Y9MZZ8_DELLE\nTNIK\tA0A2Y9N1J1_DELLE\nTNIK\tA0A2Y9N4I1_DELLE\nTNIK\tA0A2Y9N970_DELLE\nTNIK\tA0A2Y9N980_DELLE\nTNIK\tA0A2Y9N987_DELLE\nTNIK\tA0A2Y9N9B0_DELLE\nTNIK\tA0A2Y9N9C1_DELLE\nTNIK\tA0A2Y9N9D4_DELLE\nTNIK\tA0A2Y9NFD2_DELLE\nTNIK\tA0A2Y9NFE0_DELLE\nTNIK\tA0A2Y9SMV4_PHYMC\nTNIK\tA0A2Y9SNT7_PHYMC\nTNIK\tA0A2Y9SRB0_PHYMC\nTNIK\tA0A2Y9SRB7_PHYMC\nTNIK\tA0A2Y9SU11_PHYMC\nTNIK\tA0A337S967_FELCA\nTNIK\tA0A337SB99_FELCA\nTNIK\tA0A337SDC4_FELCA\nTNIK\tA0A340XTT5_LIPVE\nTNIK\tA0A340XU41_LIPVE\nTNIK\tA0A340XU48_LIPVE\nTNIK\tA0A340XX33_LIPVE\nTNIK\tA0A340XZQ7_LIPVE\nTNIK\tA0A340XZR2_LIPVE\nTNIK\tA0A340XZY8_LIPVE\nTNIK\tA0A340XZZ2_LIPVE\nTNIK\tA0A341AFU9_NEOAA\nTNIK\tA0A341AG41_NEOAA\nTNIK\tA0A341AJ01_NEOAA\nTNIK\tA0A341AKA3_NEOAA\nTNIK\tA0A384BCW6_BALAS\nTNIK\tA0A384BD00_BALAS\nTNIK\tA0A384BD19_BALAS\nTNIK\tA0A384D973_URSMA\nTNIK\tA0A3B5K088_TAKRU\nTNIK\tA0A3Q0CPH5_MESAU\nTNIK\tA0A3Q0GCY9_ALLSI\nTNIK\tA0A3Q0GD52_ALLSI\nTNIK\tA0A3Q0GD58_ALLSI\nTNIK\tA0A3Q0GDK0_ALLSI\nTNIK\tA0A3Q0GGL1_ALLSI\nTNIK\tA0A3Q0GH89_ALLSI\nTNIK\tA0A3Q1M7Q2_BOVIN\nTNIK\tA0A3Q2GXK6_HORSE\nTNIK\tA0A3Q2HIA8_HORSE\nTNIK\tA0A3Q2HM08_HORSE\nTNIK\tA0A3Q2I182_HORSE\nTNIK\tA0A3Q2IA25_HORSE\nTNIK\tA0A3Q2P290_FUNHE\nTNIK\tA0A3Q2QCG7_FUNHE\nTNIK\tA0A3Q2QCK7_FUNHE\nTNIK\tA0A3Q3B3I7_KRYMA\nTNIK\tA0A3Q7PM38_CALUR\nTNIK\tA0A3Q7PXJ0_CALUR\nTNIK\tA0A3Q7SMX9_VULVU\nTNIK\tA0A3Q7SMY2_VULVU\nTNIK\tA0A3Q7T2D4_VULVU\nTNIK\tA0A3Q7TD69_VULVU\nTNIK\tA0A3Q7TQH6_VULVU\nTNIK\tA0A3Q7U814_VULVU\nTNIK\tA0A3Q7U821_VULVU\nTNIK\tA0A3Q7UZH8_URSAR\nTNIK\tA0A3Q7UZI7_URSAR\nTNIK\tA0A3Q7VQ97_URSAR\nTNIK\tA0A3Q7VQB9_URSAR\nTNIK\tA0A3Q7VQC4_URSAR\nTNIK\tA0A3Q7VZS1_URSAR\nTNIK\tA0A3Q7WVE8_URSAR\nTNIK\tA0A452F118_CAPHI\nTNIK\tA0A452F156_CAPHI\nTNIK\tA0A452F1D6_CAPHI\nTNIK\tA0A452F1T5_CAPHI\nTNIK\tA0A452IDS7_9SAUR\nTNIK\tA0A452IDV9_9SAUR\nTNIK\tA0A452IDW4_9SAUR\nTNIK\tA0A452IE86_9SAUR\nTNIK\tA0A452SL74_URSAM\nTNIK\tA0A452SL88_URSAM\nTNIK\tA0A452SLG4_URSAM\nTNIK\tA0A452SLJ1_URSAM\nTNIK\tA0A452SMC7_URSAM\nTNIK\tA0A452SMD6_URSAM\nTNIK\tA0A452SMG1_URSAM\nTNIK\tA0A452SMH4_URSAM\nTNIK\tA0A452SMI2_URSAM\nTNIK\tA0A452SMK7_URSAM\nTNIK\tA0A452U0C9_URSMA\nTNIK\tA0A452U0K7_URSMA\nTNIK\tA0A452U0M4_URSMA\nTNIK\tA0A455BE16_PHYMC\nTNIK\tA0A455BGF5_PHYMC\nTNIK\tA0A4W2DBE4_BOBOX\nTNIK\tA0A4W2DBK4_BOBOX\nTNIK\tA0A4W2DBN2_BOBOX\nTNIK\tA0A4W2DRF6_BOBOX\nTNIK\tA0A4W2EU73_BOBOX\nTNIK\tA0A4W2EUF4_BOBOX\nTNIK\tA0A4W2EUV2_BOBOX\nTNIK\tA0A4W2FXZ4_BOBOX\nTNIK\tA0A4W2FYC4_BOBOX\nTNIK\tA0A4W2FZH6_BOBOX\nTNIK\tA0A4W2FZW3_BOBOX\nTNIK\tA0A4W2G071_BOBOX\nTNIK\tA0A4W2G087_BOBOX\nTNIK\tA0A4W2G229_BOBOX\nTNIK\tA0A4W2HT31_BOBOX\nTNIK\tA0A4W2HX91_BOBOX\nTNIK\tA0A4W4H5C5_ELEEL\nTNIK\tA0A4W4H5M0_ELEEL\nTNIK\tA0A4W4H5R4_ELEEL\nTNIK\tA0A4W4H9X5_ELEEL\nTNIK\tA0A4W4HA27_ELEEL\nTNIK\tA0A4W4HD16_ELEEL\nTNIK\tA0A4W4HDC1_ELEEL\nTNIK\tA0A4W4HDM0_ELEEL\nTNIK\tA0A4W4HDR2_ELEEL\nTNIK\tA0A4X1V8J4_PIG\nTNIK\tA0A4X1V8L3_PIG\nTNIK\tA0A4X1VD28_PIG\nTNIK\tA0A4X1VD34_PIG\nTNIK\tA0A4X1VD55_PIG\nTNIK\tA0A4X1VES9_PIG\nTNIK\tA0A4X1VEU3_PIG\nTNIK\tA0A4X1VEU8_PIG\nTNIK\tA0A4X2KZ51_VOMUR\nTNIK\tA0A4X2KZD1_VOMUR\nTNIK\tA0A4X2KZF9_VOMUR\nTNIK\tA0A4X2L4H1_VOMUR\nTNIK\tA0A4X2L5N1_VOMUR\nTNIK\tA0A4X2L5R1_VOMUR\nTNIK\tA0A4X2L783_VOMUR\nTNIK\tA0A5F4C2Z6_CANLF\nTNIK\tA0A5F4C856_CANLF\nTNIK\tA0A5F4CF97_CANLF\nTNIK\tA0A5F4CUI3_CANLF\nTNIK\tA0A5F4CYZ6_CANLF\nTNIK\tA0A5F4VRG1_CALJA\nTNIK\tA0A5F4WAM2_CALJA\nTNIK\tA0A5F5PLZ1_HORSE\nTNIK\tA0A5F5XFB1_FELCA\nTNIK\tA0A5F5XLU9_FELCA\nTNIK\tA0A5F7ZS60_MACMU\nTNIK\tA0A5F8G7I4_MONDO\nTNIK\tA0A5F8GIM5_MONDO\nTNIK\tA0A5F8GLF7_MONDO\nTNIK\tA0A5F8H5J7_MONDO\nTNIK\tA0A5F8HFZ9_MONDO\nTNIK\tA0A5F9C983_RABIT\nTNIK\tA0A5F9CBY8_RABIT\nTNIK\tA0A5F9CCY4_RABIT\nTNIK\tA0A5F9D591_RABIT\nTNIK\tA0A5F9D5H9_RABIT\nTNIK\tA0A5F9DBE7_RABIT\nTNIK\tA0A5F9DCA9_RABIT\nTNIK\tA0A5F9DMN3_RABIT\nTNIK\tA0A5F9DQ79_RABIT\nTNIK\tA0A5G2R5I7_PIG\nTNIK\tA0A5G3K7E0_XENTR\nTNIK\tA0A5G3K853_XENTR\nTNIK\tA0A5G3K935_XENTR\nTNIK\tA0A5G3K969_XENTR\nTNIK\tA0A5G3K976_XENTR\nTNIK\tA0A5G3K9D6_XENTR\nTNIK\tA0A5G3KAA9_XENTR\nTNIK\tA0A5K1V623_CALJA\nTNIK\tA0A5K1VLN8_MACMU\nTNIK\tA7MBK9_BOVIN\nTNIK\tB2RQ80_MOUSE\nTNIK\tB2RXX2_MOUSE\nTNIK\tB2RY17_MOUSE\nTNIK\tB5XEG3_SALSA\nTNIK\tB7TYI7_PIG\nTNIK\tB9EKN8_MOUSE\nTNIK\tC9J338_HUMAN\nTNIK\tC9JVV1_HUMAN\nTNIK\tD3ZZQ0_RAT\nTNIK\tE0CXD6_MOUSE\nTNIK\tE0CY98_MOUSE\nTNIK\tE0CYV5_MOUSE\nTNIK\tE0CZD7_MOUSE\nTNIK\tE0CZF8_MOUSE\nTNIK\tE1BIS8_BOVIN\nTNIK\tE9PUL9_MOUSE\nTNIK\tF1PPZ5_CANLF\nTNIK\tF6U5M5_MACMU\nTNIK\tF6V7J2_XENTR\nTNIK\tF6V804_XENTR\nTNIK\tF6VAR3_XENTR\nTNIK\tF6XGR0_MACMU\nTNIK\tF6XGR9_MACMU\nTNIK\tF6XP83_MACMU\nTNIK\tF6XP91_MACMU\nTNIK\tF6Z7Z3_HORSE\nTNIK\tF6Z841_HORSE\nTNIK\tF7A879_ORNAN\nTNIK\tF7B7L0_CALJA\nTNIK\tF7BVY5_XENTR\nTNIK\tF7EGD9_MONDO\nTNIK\tG1KSB4_ANOCA\nTNIK\tG1LEA9_AILME\nTNIK\tG1LEC2_AILME\nTNIK\tG1PU47_MYOLU\nTNIK\tG1R076_NOMLE\nTNIK\tG1SM46_RABIT\nTNIK\tG3TIY0_LOXAF\nTNIK\tG3UGJ9_LOXAF\nTNIK\tG3URV2_MELGA\nTNIK\tG3WC40_SARHA\nTNIK\tG3WC41_SARHA\nTNIK\tG7MJA7_MACMU\nTNIK\tG7NZE8_MACFA\nTNIK\tH0VKL2_CAVPO\nTNIK\tH0X2H0_OTOGA\nTNIK\tH0ZKX8_TAEGU\nTNIK\tH2PBZ9_PONAB\nTNIK\tH2R2F6_PANTR\nTNIK\tH2U2C5_TAKRU\nTNIK\tH2U2C6_TAKRU\nTNIK\tH2U2C7_TAKRU\nTNIK\tH2U2D1_TAKRU\nTNIK\tH3AUW7_LATCH\nTNIK\tH9FRM1_MACMU\nTNIK\tI3LA58_PIG\nTNIK\tI3LSZ7_PIG\nTNIK\tI3M149_ICTTR\nTNIK\tI7GI18_MACFA\nTNIK\tK7CGE9_PANTR\nTNIK\tK7GK27_PELSI\nTNIK\tK7GK28_PELSI\nTNIK\tK9J4V3_PIG\nTNIK\tK9J805_XENTR\nTNIK\tM3WEP1_FELCA\nTNIK\tM3WYW1_FELCA\nTNIK\tM3Z0C5_MUSPF\nTNIK\tM3ZBI4_NOMLE\nTNIK\tQ32NV8_XENLA\nTNIK\tQ7Z4L4_HUMAN\nTNIK\tQ8C575_MOUSE\nTNIK\tQ8C9G9_MOUSE\nTNIK\tTNIK_HUMAN\nTNIK\tTNIK_MOUSE\nTNIK\tU3BI13_CALJA\nTNIK\tU3CMD0_CALJA\nTNIK\tU3EVE6_CALJA\nTNIK\tU3FS54_CALJA\nTNIK\tU3K1K0_FICAL\nTNIK\tV8NDL9_OPHHA\nTNIK\tV8P8Z9_OPHHA\nTNIK\tW5QHW4_SHEEP\nTNIK\tW5QHW5_SHEEP\nTNIK\tW8B6E2_CERCA\nTNIK\tW8BAW5_CERCA\nTNIK\tW8BLE8_CERCA\n','From\tTo\nMAP2K6\tA0A024R8K3_HUMAN\nMAP2K6\tA0A087XLJ4_POEFO\nMAP2K6\tA0A096NU44_PAPAN\nMAP2K6\tA0A0A0MRF7_HUMAN\nMAP2K6\tA0A0B2UPI8_TOXCA\nMAP2K6\tA0A0B2UWY4_TOXCA\nMAP2K6\tA0A0C9R2F8_9HYME\nMAP2K6\tA0A0D9QWI2_CHLSB\nMAP2K6\tA0A0F6QNT5_CLABA\nMAP2K6\tA0A0K8UXF9_BACLA\nMAP2K6\tA0A0P6K1X0_HETGA\nMAP2K6\tA0A0V0YNY7_TRIPS\nMAP2K6\tA0A0V1ACB1_9BILA\nMAP2K6\tA0A0V1I085_9BILA\nMAP2K6\tA0A0V1KEM2_TRIPS\nMAP2K6\tA0A151N2F2_ALLMI\nMAP2K6\tA0A1A7WDZ3_9TELE\nMAP2K6\tA0A1A7YDG2_9TELE\nMAP2K6\tA0A1A8BH86_9TELE\nMAP2K6\tA0A1A8GJ53_9TELE\nMAP2K6\tA0A1A8IKX2_NOTKU\nMAP2K6\tA0A1A8J4T1_NOTKU\nMAP2K6\tA0A1A8LRF8_9TELE\nMAP2K6\tA0A1A8P3W7_9TELE\nMAP2K6\tA0A1A8V7U9_NOTFU\nMAP2K6\tA0A1D5PHN6_CHICK\nMAP2K6\tA0A1D5RHD1_MACMU\nMAP2K6\tA0A1L1S0H7_CHICK\nMAP2K6\tA0A1L5BX33_CASFI\nMAP2K6\tA0A1S2ZTY0_ERIEU\nMAP2K6\tA0A1S3GYE9_DIPOR\nMAP2K6\tA0A1U7QB33_MESAU\nMAP2K6\tA0A1V4KVG6_PATFA\nMAP2K6\tA0A218UWQ6_9PASE\nMAP2K6\tA0A250XYQ1_CASCN\nMAP2K6\tA0A286XA62_CAVPO\nMAP2K6\tA0A286Y813_DANRE\nMAP2K6\tA0A2B4SLW1_STYPI\nMAP2K6\tA0A2I0M7D4_COLLI\nMAP2K6\tA0A2I3H1P4_NOMLE\nMAP2K6\tA0A2I3HWG2_NOMLE\nMAP2K6\tA0A2I3LHS5_PAPAN\nMAP2K6\tA0A2I3LN66_PAPAN\nMAP2K6\tA0A2I3RB56_PANTR\nMAP2K6\tA0A2I3T6V3_PANTR\nMAP2K6\tA0A2J7PXF6_9NEOP\nMAP2K6\tA0A2J8KKV2_PANTR\nMAP2K6\tA0A2K5C929_AOTNA\nMAP2K6\tA0A2K5C931_AOTNA\nMAP2K6\tA0A2K5HRG1_COLAP\nMAP2K6\tA0A2K5HRK6_COLAP\nMAP2K6\tA0A2K5HRL3_COLAP\nMAP2K6\tA0A2K5P0H0_CERAT\nMAP2K6\tA0A2K5P0S9_CERAT\nMAP2K6\tA0A2K5QT88_CEBCA\nMAP2K6\tA0A2K5QTG5_CEBCA\nMAP2K6\tA0A2K5YG13_MANLE\nMAP2K6\tA0A2K5YG49_MANLE\nMAP2K6\tA0A2K5YG66_MANLE\nMAP2K6\tA0A2K6CS09_MACNE\nMAP2K6\tA0A2K6CS82_MACNE\nMAP2K6\tA0A2K6CSB0_MACNE\nMAP2K6\tA0A2K6CSD9_MACNE\nMAP2K6\tA0A2K6GHB9_PROCO\nMAP2K6\tA0A2K6GHC3_PROCO\nMAP2K6\tA0A2K6LB21_RHIBE\nMAP2K6\tA0A2K6LB36_RHIBE\nMAP2K6\tA0A2K6LB44_RHIBE\nMAP2K6\tA0A2K6LB52_RHIBE\nMAP2K6\tA0A2K6P648_RHIRO\nMAP2K6\tA0A2K6P656_RHIRO\nMAP2K6\tA0A2K6P659_RHIRO\nMAP2K6\tA0A2K6P684_RHIRO\nMAP2K6\tA0A2K6S0K5_SAIBB\nMAP2K6\tA0A2K6S0L0_SAIBB\nMAP2K6\tA0A2K6S0L2_SAIBB\nMAP2K6\tA0A2K6S0Q5_SAIBB\nMAP2K6\tA0A2K6S0V2_SAIBB\nMAP2K6\tA0A2P8Y5R8_BLAGE\nMAP2K6\tA0A2R9BGH1_PANPA\nMAP2K6\tA0A2R9BHG8_PANPA\nMAP2K6\tA0A2R9BHH8_PANPA\nMAP2K6\tA0A2R9BQ61_PANPA\nMAP2K6\tA0A2S2NRT5_SCHGA\nMAP2K6\tA0A2S2QHA2_9HEMI\nMAP2K6\tA0A2U3WBJ3_ODORO\nMAP2K6\tA0A2U3XLA8_LEPWE\nMAP2K6\tA0A2U4BMB9_TURTR\nMAP2K6\tA0A2Y9G5I1_NEOSC\nMAP2K6\tA0A2Y9PI64_DELLE\nMAP2K6\tA0A2Y9PQ85_DELLE\nMAP2K6\tA0A2Y9PRH8_DELLE\nMAP2K6\tA0A341CQ37_NEOAA\nMAP2K6\tA0A384AEA6_BALAS\nMAP2K6\tA0A384CAS3_URSMA\nMAP2K6\tA0A3B3D870_ORYME\nMAP2K6\tA0A3B3DAC6_ORYME\nMAP2K6\tA0A3B3UEX5_9TELE\nMAP2K6\tA0A3B3UFC6_9TELE\nMAP2K6\tA0A3B3XEX9_9TELE\nMAP2K6\tA0A3B3YRN3_9TELE\nMAP2K6\tA0A3B3ZVQ4_9GOBI\nMAP2K6\tA0A3B4GTW4_9CICH\nMAP2K6\tA0A3B4UFY1_SERDU\nMAP2K6\tA0A3B4UG08_SERDU\nMAP2K6\tA0A3B4YKZ7_SERLL\nMAP2K6\tA0A3B4YL11_SERLL\nMAP2K6\tA0A3B4ZBG8_9TELE\nMAP2K6\tA0A3B4ZEG6_9TELE\nMAP2K6\tA0A3B5LQ06_9TELE\nMAP2K6\tA0A3P8R573_ASTCA\nMAP2K6\tA0A3P8S2E3_AMPPE\nMAP2K6\tA0A3P9C6Y1_9CICH\nMAP2K6\tA0A3P9C6Z1_9CICH\nMAP2K6\tA0A3P9N464_POERE\nMAP2K6\tA0A3Q0CQB9_MESAU\nMAP2K6\tA0A3Q0FXN2_ALLSI\nMAP2K6\tA0A3Q0G1S6_ALLSI\nMAP2K6\tA0A3Q0QPE2_AMPCI\nMAP2K6\tA0A3Q1BRW2_AMPOC\nMAP2K6\tA0A3Q1EGX8_9TELE\nMAP2K6\tA0A3Q1FYJ3_9TELE\nMAP2K6\tA0A3Q1JJ13_ANATE\nMAP2K6\tA0A3Q1KGE6_ANATE\nMAP2K6\tA0A3Q1LLK9_BOVIN\nMAP2K6\tA0A3Q1M703_BOVIN\nMAP2K6\tA0A3Q2FTF6_CYPVA\nMAP2K6\tA0A3Q2FTF8_CYPVA\nMAP2K6\tA0A3Q2HTP6_HORSE\nMAP2K6\tA0A3Q2IET4_HORSE\nMAP2K6\tA0A3Q2LN28_HORSE\nMAP2K6\tA0A3Q2V4C9_HAPBU\nMAP2K6\tA0A3Q2V4E2_HAPBU\nMAP2K6\tA0A3Q3IC83_MONAL\nMAP2K6\tA0A3Q3MVI4_9TELE\nMAP2K6\tA0A3Q3NGP7_9TELE\nMAP2K6\tA0A3Q3SWG8_9TELE\nMAP2K6\tA0A3Q3WPW0_MOLML\nMAP2K6\tA0A3Q4IDV2_NEOBR\nMAP2K6\tA0A3Q4N8G3_NEOBR\nMAP2K6\tA0A3Q7QGM7_CALUR\nMAP2K6\tA0A3Q7QUB4_CALUR\nMAP2K6\tA0A3Q7SJ79_VULVU\nMAP2K6\tA0A3Q7TJX3_VULVU\nMAP2K6\tA0A3Q7X1M6_URSAR\nMAP2K6\tA0A452FY02_CAPHI\nMAP2K6\tA0A452FY29_CAPHI\nMAP2K6\tA0A452J571_9SAUR\nMAP2K6\tA0A452J573_9SAUR\nMAP2K6\tA0A452S4E2_URSAM\nMAP2K6\tA0A455C325_PHYMC\nMAP2K6\tA0A455C6B6_PHYMC\nMAP2K6\tA0A493TSY7_ANAPP\nMAP2K6\tA0A4W2CKD7_BOBOX\nMAP2K6\tA0A4W2E5A9_BOBOX\nMAP2K6\tA0A4W5QQB7_9TELE\nMAP2K6\tA0A4W5QQD1_9TELE\nMAP2K6\tA0A4W6FZY8_LATCA\nMAP2K6\tA0A4W6G1S0_LATCA\nMAP2K6\tA0A4X1SM67_PIG\nMAP2K6\tA0A4X1SME1_PIG\nMAP2K6\tA0A4X1SNE3_PIG\nMAP2K6\tA0A4X2L4Z1_VOMUR\nMAP2K6\tA0A5B7FLY6_PORTR\nMAP2K6\tA0A5F4CXH4_CANLF\nMAP2K6\tA0A5G3HGT1_XENTR\nMAP2K6\tA0A5G3HGU1_XENTR\nMAP2K6\tA0A5G3J5B7_XENTR\nMAP2K6\tA0A5G3J5I3_XENTR\nMAP2K6\tA0A5G3J5K2_XENTR\nMAP2K6\tA0A5G3J610_XENTR\nMAP2K6\tA0A5K1UKA4_PIG\nMAP2K6\tA2AGS2_MOUSE\nMAP2K6\tA8K3Y2_HUMAN\nMAP2K6\tB2GTB0_DANRE\nMAP2K6\tB5BU16_HUMAN\nMAP2K6\tE2R7W7_CANLF\nMAP2K6\tF1NVF2_CHICK\nMAP2K6\tF1RV28_PIG\nMAP2K6\tF6QDN0_HORSE\nMAP2K6\tF6RI32_XENTR\nMAP2K6\tF7AGV5_MONDO\nMAP2K6\tF7CFF1_XENTR\nMAP2K6\tF7CJ25_XENTR\nMAP2K6\tF7G9N9_ORNAN\nMAP2K6\tF7GTD8_CALJA\nMAP2K6\tG1LLJ8_AILME\nMAP2K6\tG1N4E6_MELGA\nMAP2K6\tG1P7K5_MYOLU\nMAP2K6\tG1QQD5_NOMLE\nMAP2K6\tG1STE8_RABIT\nMAP2K6\tG3PA38_GASAC\nMAP2K6\tG3PA47_GASAC\nMAP2K6\tG3SUS9_LOXAF\nMAP2K6\tG3UEC7_LOXAF\nMAP2K6\tG7YF87_CLOSI\nMAP2K6\tH0WA39_CAVPO\nMAP2K6\tH0X2L9_OTOGA\nMAP2K6\tH2NUJ7_PONAB\nMAP2K6\tH2QZW5_PANTR\nMAP2K6\tH2VAA3_TAKRU\nMAP2K6\tH3B5K1_LATCH\nMAP2K6\tH3CWP0_TETNG\nMAP2K6\tH9EP89_MACMU\nMAP2K6\tI3JD00_ORENI\nMAP2K6\tK7EIW3_HUMAN\nMAP2K6\tK7ELM6_HUMAN\nMAP2K6\tK7GH75_PELSI\nMAP2K6\tM3WN25_FELCA\nMAP2K6\tM3YSA1_MUSPF\nMAP2K6\tMP2K6_BOVIN\nMAP2K6\tMP2K6_DANRE\nMAP2K6\tMP2K6_HUMAN\nMAP2K6\tMP2K6_MOUSE\nMAP2K6\tP70273_MOUSE\nMAP2K6\tQ3UU57_MOUSE\nMAP2K6\tQ543Z5_MOUSE\nMAP2K6\tQ5FWR2_XENTR\nMAP2K6\tQ60521_MOUSE\nMAP2K6\tQ6GL46_XENTR\nMAP2K6\tQ6PF61_XENLA\nMAP2K6\tQ8BP44_MOUSE\nMAP2K6\tQ925D6_RAT\nMAP2K6\tR4GDI3_ANOCA\nMAP2K6\tT2MF77_HYDVU\nMAP2K6\tU3IDW3_ANAPP\nMAP2K6\tU3JIS3_FICAL\nMAP2K6\tV8NKC9_OPHHA\nMAP2K6\tV8P1I1_OPHHA\nMAP2K6\tV8P2J7_OPHHA\nMAP2K6\tW5PWP0_SHEEP\nMAP2K6\tW5UAE0_ICTPU\n','From\tTo\nSTRADA\tA0A096P6E5_PAPAN\nSTRADA\tA0A0A9XVC4_LYGHE\nSTRADA\tA0A0B2UUJ6_TOXCA\nSTRADA\tA0A0D9QV60_CHLSB\nSTRADA\tA0A0G2JLH2_HUMAN\nSTRADA\tA0A0V0SL26_9BILA\nSTRADA\tA0A0V0SL68_9BILA\nSTRADA\tA0A0V0TZ75_9BILA\nSTRADA\tA0A0V0TZ90_9BILA\nSTRADA\tA0A0V0WI54_9BILA\nSTRADA\tA0A0V0WJX8_9BILA\nSTRADA\tA0A0V1A4Z7_9BILA\nSTRADA\tA0A0V1A567_9BILA\nSTRADA\tA0A0V1BUQ8_TRISP\nSTRADA\tA0A0V1BWV7_TRISP\nSTRADA\tA0A0V1DJ85_TRIBR\nSTRADA\tA0A0V1DJQ5_TRIBR\nSTRADA\tA0A0V1DJX1_TRIBR\nSTRADA\tA0A0V1EE87_TRIPS\nSTRADA\tA0A0V1GTI2_9BILA\nSTRADA\tA0A0V1GTP1_9BILA\nSTRADA\tA0A0V1GTV7_9BILA\nSTRADA\tA0A0V1I015_9BILA\nSTRADA\tA0A0V1I0B8_9BILA\nSTRADA\tA0A0V1I0P9_9BILA\nSTRADA\tA0A0V1JDE3_TRIPS\nSTRADA\tA0A0V1KAN8_TRIPS\nSTRADA\tA0A0V1MUU7_9BILA\nSTRADA\tA0A0V1PPH6_9BILA\nSTRADA\tA0A0V1PPR0_9BILA\nSTRADA\tA0A151PC92_ALLMI\nSTRADA\tA0A1A7Y3D0_9TELE\nSTRADA\tA0A1A8BHW9_9TELE\nSTRADA\tA0A1A8BX39_9TELE\nSTRADA\tA0A1A8BZW0_9TELE\nSTRADA\tA0A1A8FZ21_9TELE\nSTRADA\tA0A1A8GWP1_9TELE\nSTRADA\tA0A1A8HPN0_NOTKU\nSTRADA\tA0A1A8KVP9_NOTKU\nSTRADA\tA0A1A8L8T0_9TELE\nSTRADA\tA0A1A8QL10_9TELE\nSTRADA\tA0A1A8S2E4_9TELE\nSTRADA\tA0A1A8UU05_NOTFU\nSTRADA\tA0A1D5PFV5_CHICK\nSTRADA\tA0A1D5PQK3_CHICK\nSTRADA\tA0A1D5QVR6_MACMU\nSTRADA\tA0A1D5QYD1_MACMU\nSTRADA\tA0A1L8ESD1_XENLA\nSTRADA\tA0A1S3A143_ERIEU\nSTRADA\tA0A1S3A152_ERIEU\nSTRADA\tA0A1S3A164_ERIEU\nSTRADA\tA0A1S3GIZ4_DIPOR\nSTRADA\tA0A1S3GKJ7_DIPOR\nSTRADA\tA0A1S3GKK3_DIPOR\nSTRADA\tA0A1S3GL81_DIPOR\nSTRADA\tA0A1S3WH65_ERIEU\nSTRADA\tA0A1S3WHC0_ERIEU\nSTRADA\tA0A1S3WHF3_ERIEU\nSTRADA\tA0A1S3WHW9_ERIEU\nSTRADA\tA0A1U7RP95_ALLSI\nSTRADA\tA0A1U8BZC6_MESAU\nSTRADA\tA0A1W2PNV1_HUMAN\nSTRADA\tA0A1W2PNV7_HUMAN\nSTRADA\tA0A1W2PP17_HUMAN\nSTRADA\tA0A1W2PP78_HUMAN\nSTRADA\tA0A1W2PPG2_HUMAN\nSTRADA\tA0A1W2PPI1_HUMAN\nSTRADA\tA0A1W2PPJ9_HUMAN\nSTRADA\tA0A1W2PPM8_HUMAN\nSTRADA\tA0A1W2PQ00_HUMAN\nSTRADA\tA0A1W2PQ20_HUMAN\nSTRADA\tA0A1W2PQE8_HUMAN\nSTRADA\tA0A1W2PQF1_HUMAN\nSTRADA\tA0A1W2PR00_HUMAN\nSTRADA\tA0A1W2PR44_HUMAN\nSTRADA\tA0A1W2PR65_HUMAN\nSTRADA\tA0A1W2PRI8_HUMAN\nSTRADA\tA0A1W2PRQ6_HUMAN\nSTRADA\tA0A1W2PRW9_HUMAN\nSTRADA\tA0A1W2PS04_HUMAN\nSTRADA\tA0A218UI90_9PASE\nSTRADA\tA0A250XZL0_CASCN\nSTRADA\tA0A286XYF9_CAVPO\nSTRADA\tA0A286ZP26_PIG\nSTRADA\tA0A286ZUY1_PIG\nSTRADA\tA0A286ZXB3_PIG\nSTRADA\tA0A287A131_PIG\nSTRADA\tA0A287AN83_PIG\nSTRADA\tA0A287B232_PIG\nSTRADA\tA0A287B8U4_PIG\nSTRADA\tA0A287BQ59_PIG\nSTRADA\tA0A287D132_ICTTR\nSTRADA\tA0A2B4RI02_STYPI\nSTRADA\tA0A2D0Q792_ICTPU\nSTRADA\tA0A2H8TD03_9HEMI\nSTRADA\tA0A2I0LN89_COLLI\nSTRADA\tA0A2I0LN96_COLLI\nSTRADA\tA0A2I0LNB1_COLLI\nSTRADA\tA0A2I2U0Z7_FELCA\nSTRADA\tA0A2I3G4N6_NOMLE\nSTRADA\tA0A2I3GCF7_NOMLE\nSTRADA\tA0A2I3GK49_NOMLE\nSTRADA\tA0A2I3H311_NOMLE\nSTRADA\tA0A2I3H8Q3_NOMLE\nSTRADA\tA0A2I3HMS3_NOMLE\nSTRADA\tA0A2I3HS96_NOMLE\nSTRADA\tA0A2I3HVF4_NOMLE\nSTRADA\tA0A2I3HZP5_NOMLE\nSTRADA\tA0A2I3LD26_PAPAN\nSTRADA\tA0A2I3LJX4_PAPAN\nSTRADA\tA0A2I3LN32_PAPAN\nSTRADA\tA0A2I3LS24_PAPAN\nSTRADA\tA0A2I3M0Q1_PAPAN\nSTRADA\tA0A2I3N2Z3_PAPAN\nSTRADA\tA0A2I3N5Y4_PAPAN\nSTRADA\tA0A2I3ND31_PAPAN\nSTRADA\tA0A2I3NFV1_PAPAN\nSTRADA\tA0A2I3RJJ8_PANTR\nSTRADA\tA0A2I3RXH6_PANTR\nSTRADA\tA0A2I3S334_PANTR\nSTRADA\tA0A2I3SF10_PANTR\nSTRADA\tA0A2I3T2F2_PANTR\nSTRADA\tA0A2I3TAB9_PANTR\nSTRADA\tA0A2I3TLI1_PANTR\nSTRADA\tA0A2I3TSZ0_PANTR\nSTRADA\tA0A2I4D3M6_9TELE\nSTRADA\tA0A2I4D3M9_9TELE\nSTRADA\tA0A2I4D3R1_9TELE\nSTRADA\tA0A2K5EZW3_AOTNA\nSTRADA\tA0A2K5EZW6_AOTNA\nSTRADA\tA0A2K5EZX2_AOTNA\nSTRADA\tA0A2K5EZX4_AOTNA\nSTRADA\tA0A2K5EZZ9_AOTNA\nSTRADA\tA0A2K5F003_AOTNA\nSTRADA\tA0A2K5F013_AOTNA\nSTRADA\tA0A2K5F037_AOTNA\nSTRADA\tA0A2K5F038_AOTNA\nSTRADA\tA0A2K5J9K0_COLAP\nSTRADA\tA0A2K5J9W4_COLAP\nSTRADA\tA0A2K5J9W7_COLAP\nSTRADA\tA0A2K5J9W9_COLAP\nSTRADA\tA0A2K5J9X5_COLAP\nSTRADA\tA0A2K5JA16_COLAP\nSTRADA\tA0A2K5JA18_COLAP\nSTRADA\tA0A2K5JA61_COLAP\nSTRADA\tA0A2K5JAI8_COLAP\nSTRADA\tA0A2K5NWR0_CERAT\nSTRADA\tA0A2K5NWV3_CERAT\nSTRADA\tA0A2K5NWV8_CERAT\nSTRADA\tA0A2K5NWX5_CERAT\nSTRADA\tA0A2K5NWY9_CERAT\nSTRADA\tA0A2K5NWZ0_CERAT\nSTRADA\tA0A2K5NWZ5_CERAT\nSTRADA\tA0A2K5NX65_CERAT\nSTRADA\tA0A2K5NX70_CERAT\nSTRADA\tA0A2K5NXA0_CERAT\nSTRADA\tA0A2K5U803_MACFA\nSTRADA\tA0A2K5U819_MACFA\nSTRADA\tA0A2K5U831_MACFA\nSTRADA\tA0A2K5U836_MACFA\nSTRADA\tA0A2K5U851_MACFA\nSTRADA\tA0A2K5U872_MACFA\nSTRADA\tA0A2K5U873_MACFA\nSTRADA\tA0A2K5U8E7_MACFA\nSTRADA\tA0A2K5U8F9_MACFA\nSTRADA\tA0A2K5YY64_MANLE\nSTRADA\tA0A2K5YYB3_MANLE\nSTRADA\tA0A2K5YYC7_MANLE\nSTRADA\tA0A2K5YYH6_MANLE\nSTRADA\tA0A2K5YYJ9_MANLE\nSTRADA\tA0A2K5YYL9_MANLE\nSTRADA\tA0A2K5YYQ0_MANLE\nSTRADA\tA0A2K5YYR3_MANLE\nSTRADA\tA0A2K5YYT9_MANLE\nSTRADA\tA0A2K6DFZ4_MACNE\nSTRADA\tA0A2K6DG44_MACNE\nSTRADA\tA0A2K6DG75_MACNE\nSTRADA\tA0A2K6DGB8_MACNE\nSTRADA\tA0A2K6DGC0_MACNE\nSTRADA\tA0A2K6DGE0_MACNE\nSTRADA\tA0A2K6DGE1_MACNE\nSTRADA\tA0A2K6DGF5_MACNE\nSTRADA\tA0A2K6DGJ7_MACNE\nSTRADA\tA0A2K6DGM1_MACNE\nSTRADA\tA0A2K6F7A1_PROCO\nSTRADA\tA0A2K6F7A6_PROCO\nSTRADA\tA0A2K6F7A9_PROCO\nSTRADA\tA0A2K6F7B1_PROCO\nSTRADA\tA0A2K6F7C1_PROCO\nSTRADA\tA0A2K6F7C5_PROCO\nSTRADA\tA0A2K6F7D0_PROCO\nSTRADA\tA0A2K6F7E2_PROCO\nSTRADA\tA0A2K6LAC9_RHIBE\nSTRADA\tA0A2K6LAE2_RHIBE\nSTRADA\tA0A2K6LAE3_RHIBE\nSTRADA\tA0A2K6LAE6_RHIBE\nSTRADA\tA0A2K6LAF4_RHIBE\nSTRADA\tA0A2K6LAF7_RHIBE\nSTRADA\tA0A2K6LAG9_RHIBE\nSTRADA\tA0A2K6LAH9_RHIBE\nSTRADA\tA0A2K6QLU9_RHIRO\nSTRADA\tA0A2K6QLY8_RHIRO\nSTRADA\tA0A2K6QLZ7_RHIRO\nSTRADA\tA0A2K6QM11_RHIRO\nSTRADA\tA0A2K6QM12_RHIRO\nSTRADA\tA0A2K6QM19_RHIRO\nSTRADA\tA0A2K6QM23_RHIRO\nSTRADA\tA0A2K6QM29_RHIRO\nSTRADA\tA0A2K6QM35_RHIRO\nSTRADA\tA0A2K6T2L8_SAIBB\nSTRADA\tA0A2K6T2R1_SAIBB\nSTRADA\tA0A2K6T2R4_SAIBB\nSTRADA\tA0A2K6T2T1_SAIBB\nSTRADA\tA0A2K6T2T4_SAIBB\nSTRADA\tA0A2K6T2Y9_SAIBB\nSTRADA\tA0A2K6T310_SAIBB\nSTRADA\tA0A2K6T317_SAIBB\nSTRADA\tA0A2K6T321_SAIBB\nSTRADA\tA0A2P8XKZ0_BLAGE\nSTRADA\tA0A2R8MKC9_CALJA\nSTRADA\tA0A2R8MQP5_CALJA\nSTRADA\tA0A2R8ND98_CALJA\nSTRADA\tA0A2R8P282_CALJA\nSTRADA\tA0A2R8PI98_CALJA\nSTRADA\tA0A2R9A6A2_PANPA\nSTRADA\tA0A2R9A7H1_PANPA\nSTRADA\tA0A2R9A7L9_PANPA\nSTRADA\tA0A2R9A7Q1_PANPA\nSTRADA\tA0A2R9AAV9_PANPA\nSTRADA\tA0A2R9AB44_PANPA\nSTRADA\tA0A2R9AB57_PANPA\nSTRADA\tA0A2R9AB63_PANPA\nSTRADA\tA0A2R9AB84_PANPA\nSTRADA\tA0A2R9ABS5_PANPA\nSTRADA\tA0A2S2PDE3_SCHGA\nSTRADA\tA0A2S2R4W6_9HEMI\nSTRADA\tA0A2U3WBU3_ODORO\nSTRADA\tA0A2U3WBU8_ODORO\nSTRADA\tA0A2U3WBU9_ODORO\nSTRADA\tA0A2U3WBV0_ODORO\nSTRADA\tA0A2U3WBV1_ODORO\nSTRADA\tA0A2U3WBV2_ODORO\nSTRADA\tA0A2U3Y200_LEPWE\nSTRADA\tA0A2U3Y249_LEPWE\nSTRADA\tA0A2U4C330_TURTR\nSTRADA\tA0A2U4C335_TURTR\nSTRADA\tA0A2U4C338_TURTR\nSTRADA\tA0A2U4C340_TURTR\nSTRADA\tA0A2U4C349_TURTR\nSTRADA\tA0A2Y9G4Y0_NEOSC\nSTRADA\tA0A2Y9G8I9_NEOSC\nSTRADA\tA0A2Y9PD46_DELLE\nSTRADA\tA0A2Y9PG49_DELLE\nSTRADA\tA0A2Y9PHT1_DELLE\nSTRADA\tA0A2Y9PNG0_DELLE\nSTRADA\tA0A2Y9PPU2_DELLE\nSTRADA\tA0A2Y9T512_PHYMC\nSTRADA\tA0A2Y9T6Q9_PHYMC\nSTRADA\tA0A2Y9TAY3_PHYMC\nSTRADA\tA0A2Y9TD77_PHYMC\nSTRADA\tA0A337S536_FELCA\nSTRADA\tA0A337S872_FELCA\nSTRADA\tA0A337SAT7_FELCA\nSTRADA\tA0A337SAZ9_FELCA\nSTRADA\tA0A337SQP1_FELCA\nSTRADA\tA0A337SR70_FELCA\nSTRADA\tA0A340X5J3_LIPVE\nSTRADA\tA0A340X9T1_LIPVE\nSTRADA\tA0A340XC92_LIPVE\nSTRADA\tA0A340XCS0_LIPVE\nSTRADA\tA0A341CNN9_NEOAA\nSTRADA\tA0A341CNP3_NEOAA\nSTRADA\tA0A341CNS5_NEOAA\nSTRADA\tA0A341CNT0_NEOAA\nSTRADA\tA0A341CP06_NEOAA\nSTRADA\tA0A341CPY6_NEOAA\nSTRADA\tA0A341CRF2_NEOAA\nSTRADA\tA0A383ZL50_BALAS\nSTRADA\tA0A383ZL59_BALAS\nSTRADA\tA0A383ZLS1_BALAS\nSTRADA\tA0A383ZLS7_BALAS\nSTRADA\tA0A384CAC9_URSMA\nSTRADA\tA0A384CAE7_URSMA\nSTRADA\tA0A384CAI3_URSMA\nSTRADA\tA0A384CBP1_URSMA\nSTRADA\tA0A3Q0DN12_TARSY\nSTRADA\tA0A3Q1LPE0_BOVIN\nSTRADA\tA0A3Q1MX20_BOVIN\nSTRADA\tA0A3Q2H0B3_HORSE\nSTRADA\tA0A3Q2H719_HORSE\nSTRADA\tA0A3Q2HE02_HORSE\nSTRADA\tA0A3Q2HR02_HORSE\nSTRADA\tA0A3Q2HZR5_HORSE\nSTRADA\tA0A3Q7SGC2_VULVU\nSTRADA\tA0A3Q7SIZ3_VULVU\nSTRADA\tA0A3Q7TJL3_VULVU\nSTRADA\tA0A3Q7WTB1_URSAR\nSTRADA\tA0A3Q7WTB6_URSAR\nSTRADA\tA0A3Q7XNK5_URSAR\nSTRADA\tA0A3Q7XNK9_URSAR\nSTRADA\tA0A452CCJ0_BALAS\nSTRADA\tA0A452DS93_CAPHI\nSTRADA\tA0A452DS99_CAPHI\nSTRADA\tA0A452IJY7_9SAUR\nSTRADA\tA0A452IK51_9SAUR\nSTRADA\tA0A452QT09_URSAM\nSTRADA\tA0A455C426_PHYMC\nSTRADA\tA0A493TX39_ANAPP\nSTRADA\tA0A4C1YPX6_9NEOP\nSTRADA\tA0A4W2C9Q4_BOBOX\nSTRADA\tA0A4W2E760_BOBOX\nSTRADA\tA0A4W2E765_BOBOX\nSTRADA\tA0A4W2E9C7_BOBOX\nSTRADA\tA0A4W2GC17_BOBOX\nSTRADA\tA0A4W2GDP3_BOBOX\nSTRADA\tA0A4W2GDS3_BOBOX\nSTRADA\tA0A4W2GE20_BOBOX\nSTRADA\tA0A4W3GZA8_CALMI\nSTRADA\tA0A4W3H1Z7_CALMI\nSTRADA\tA0A4W6CKS7_LATCA\nSTRADA\tA0A4W6CKU8_LATCA\nSTRADA\tA0A4W6CKV8_LATCA\nSTRADA\tA0A4W6CKW2_LATCA\nSTRADA\tA0A4X1T1G0_PIG\nSTRADA\tA0A4X1T2L5_PIG\nSTRADA\tA0A4X1T392_PIG\nSTRADA\tA0A4X1T3D2_PIG\nSTRADA\tA0A4X1T434_PIG\nSTRADA\tA0A4X1T455_PIG\nSTRADA\tA0A4X2KLW1_VOMUR\nSTRADA\tA0A4X2KSW7_VOMUR\nSTRADA\tA0A4X2KUY8_VOMUR\nSTRADA\tA0A4X2KVK1_VOMUR\nSTRADA\tA0A4Y2DRP8_ARAVE\nSTRADA\tA0A4Z2F391_9TELE\nSTRADA\tA0A5F4BUR9_CANLF\nSTRADA\tA0A5F4C4X0_CANLF\nSTRADA\tA0A5F4CS87_CANLF\nSTRADA\tA0A5F4D786_CANLF\nSTRADA\tA0A5F4DC30_CANLF\nSTRADA\tA0A5F5PH80_HORSE\nSTRADA\tA0A5F5PR38_HORSE\nSTRADA\tA0A5F8GB38_MONDO\nSTRADA\tA0A5F8GCD1_MONDO\nSTRADA\tA0A5F8GRI5_MONDO\nSTRADA\tA0A5F8GVS1_MONDO\nSTRADA\tA0A5F8HEW6_MONDO\nSTRADA\tA0A5F9C0N7_RABIT\nSTRADA\tA0A5F9CCJ5_RABIT\nSTRADA\tA0A5F9CPT3_RABIT\nSTRADA\tA0A5F9CVR2_RABIT\nSTRADA\tA0A5F9CZ22_RABIT\nSTRADA\tA2VE62_BOVIN\nSTRADA\tB1AR89_MOUSE\nSTRADA\tB1AR92_MOUSE\nSTRADA\tE2RGK3_CANLF\nSTRADA\tF1M341_RAT\nSTRADA\tF1NL91_CHICK\nSTRADA\tF1PFF0_CANLF\nSTRADA\tF1Q5T1_DANRE\nSTRADA\tF1RSI6_PIG\nSTRADA\tF6PP88_CALJA\nSTRADA\tF6PYZ3_CALJA\nSTRADA\tF6QWI1_BOVIN\nSTRADA\tF6SFF9_MONDO\nSTRADA\tF6TYM5_CALJA\nSTRADA\tF6X620_XENTR\nSTRADA\tF6X629_XENTR\nSTRADA\tF6Z533_HORSE\nSTRADA\tF7AHH3_MACMU\nSTRADA\tF7AI65_MACMU\nSTRADA\tF7AI73_MACMU\nSTRADA\tF7CIH6_MACMU\nSTRADA\tF7CPQ4_XENTR\nSTRADA\tG1KMC3_ANOCA\nSTRADA\tG1L980_AILME\nSTRADA\tG1MSJ4_MELGA\nSTRADA\tG1PIN5_MYOLU\nSTRADA\tG1QWK7_NOMLE\nSTRADA\tG1SLH4_RABIT\nSTRADA\tG3T567_LOXAF\nSTRADA\tG3UT21_MELGA\nSTRADA\tG3V9L2_RAT\nSTRADA\tG3WGJ1_SARHA\nSTRADA\tG5BDJ4_HETGA\nSTRADA\tG7PV48_MACFA\nSTRADA\tH0V5W2_CAVPO\nSTRADA\tH0WV31_OTOGA\nSTRADA\tH0YUN3_TAEGU\nSTRADA\tH2LAU8_ORYLA\nSTRADA\tH2LAU9_ORYLA\nSTRADA\tH2NUE3_PONAB\nSTRADA\tH2NUE4_PONAB\nSTRADA\tH2QDN2_PANTR\nSTRADA\tH2TT06_TAKRU\nSTRADA\tH2TT07_TAKRU\nSTRADA\tH3AZL4_LATCH\nSTRADA\tH3AZL5_LATCH\nSTRADA\tH9G1U4_MACMU\nSTRADA\tI3MCH9_ICTTR\nSTRADA\tJ3KSA2_HUMAN\nSTRADA\tJ3KSK5_HUMAN\nSTRADA\tJ3QKR7_HUMAN\nSTRADA\tJ3QKU4_HUMAN\nSTRADA\tJ3QQS3_HUMAN\nSTRADA\tJ3QR78_HUMAN\nSTRADA\tJ3QRC1_HUMAN\nSTRADA\tJ3QRH3_HUMAN\nSTRADA\tJ3QS66_HUMAN\nSTRADA\tK7AMF3_PANTR\nSTRADA\tK7B944_PANTR\nSTRADA\tK7DAR4_PANTR\nSTRADA\tK7FXF4_PELSI\nSTRADA\tK7FXI4_PELSI\nSTRADA\tM3W0G1_FELCA\nSTRADA\tM3YU11_MUSPF\nSTRADA\tQ66IG0_XENTR\nSTRADA\tQ7ZUU2_DANRE\nSTRADA\tQ80VZ9_MOUSE\nSTRADA\tQ86YC8_HUMAN\nSTRADA\tSTRAA_BOVIN\nSTRADA\tSTRAA_CHICK\nSTRADA\tSTRAA_HUMAN\nSTRADA\tSTRAA_MACFA\nSTRADA\tSTRAA_MOUSE\nSTRADA\tSTRAA_PONAB\nSTRADA\tSTRAA_RAT\nSTRADA\tU3CSP5_CALJA\nSTRADA\tU3IX29_ANAPP\nSTRADA\tU3JPY2_FICAL\nSTRADA\tV8NDD3_OPHHA\nSTRADA\tV9KMC3_CALMI\nSTRADA\tW5PVB8_SHEEP\nSTRADA\tW5UJB6_ICTPU\n','From\tTo\nMAP2K1\tA0A096NMF0_PAPAN\nMAP2K1\tA0A0D9RL74_CHLSB\nMAP2K1\tA0A0H2UHI2_RAT\nMAP2K1\tA0A0V0S1D5_9BILA\nMAP2K1\tA0A0V0S1J3_9BILA\nMAP2K1\tA0A0V0TSP9_9BILA\nMAP2K1\tA0A0V0WFD8_9BILA\nMAP2K1\tA0A0V0XR49_TRIPS\nMAP2K1\tA0A0V0ZJR7_9BILA\nMAP2K1\tA0A0V1D8C8_TRIBR\nMAP2K1\tA0A0V1FWZ8_TRIPS\nMAP2K1\tA0A0V1HHM0_9BILA\nMAP2K1\tA0A0V1IIV7_TRIPS\nMAP2K1\tA0A0V1L227_9BILA\nMAP2K1\tA0A0V1L2L9_9BILA\nMAP2K1\tA0A0V1MID8_9BILA\nMAP2K1\tA0A0V1P2Z7_9BILA\nMAP2K1\tA0A146LUR3_LYGHE\nMAP2K1\tA0A151NXN5_ALLMI\nMAP2K1\tA0A1A7YGK9_9TELE\nMAP2K1\tA0A1A8BFA1_9TELE\nMAP2K1\tA0A1A8BPN0_9TELE\nMAP2K1\tA0A1A8CZI3_9TELE\nMAP2K1\tA0A1A8FKQ0_9TELE\nMAP2K1\tA0A1A8J3W7_NOTKU\nMAP2K1\tA0A1A8M8S3_9TELE\nMAP2K1\tA0A1A8PPC5_9TELE\nMAP2K1\tA0A1A8RG63_9TELE\nMAP2K1\tA0A1A8V806_NOTFU\nMAP2K1\tA0A1C7N308_9FUNG\nMAP2K1\tA0A1D5R0Y1_MACMU\nMAP2K1\tA0A1L1RPJ2_CHICK\nMAP2K1\tA0A1L8GSC6_XENLA\nMAP2K1\tA0A1L8H157_XENLA\nMAP2K1\tA0A1S3EMP1_DIPOR\nMAP2K1\tA0A1S3PE66_SALSA\nMAP2K1\tA0A1S3WUS5_ERIEU\nMAP2K1\tA0A1U7QVL0_MESAU\nMAP2K1\tA0A1U7RAB7_ALLSI\nMAP2K1\tA0A1V4JDE3_PATFA\nMAP2K1\tA0A218V0K5_9PASE\nMAP2K1\tA0A250YDH8_CASCN\nMAP2K1\tA0A287BK75_PIG\nMAP2K1\tA0A287D7T7_ICTTR\nMAP2K1\tA0A2B4RGK8_STYPI\nMAP2K1\tA0A2D0QUI9_ICTPU\nMAP2K1\tA0A2I0MD87_COLLI\nMAP2K1\tA0A2I3G300_NOMLE\nMAP2K1\tA0A2I3GQR6_NOMLE\nMAP2K1\tA0A2I3GXI0_NOMLE\nMAP2K1\tA0A2I3HU38_NOMLE\nMAP2K1\tA0A2I3LF26_PAPAN\nMAP2K1\tA0A2I3MHA0_PAPAN\nMAP2K1\tA0A2I3N0U2_PAPAN\nMAP2K1\tA0A2I3RF04_PANTR\nMAP2K1\tA0A2I3RPN9_PANTR\nMAP2K1\tA0A2I3RYK1_PANTR\nMAP2K1\tA0A2I3SKS8_PANTR\nMAP2K1\tA0A2I3T202_PANTR\nMAP2K1\tA0A2I4BN15_9TELE\nMAP2K1\tA0A2K5CPJ5_AOTNA\nMAP2K1\tA0A2K5CPL3_AOTNA\nMAP2K1\tA0A2K5CPL9_AOTNA\nMAP2K1\tA0A2K5CPN3_AOTNA\nMAP2K1\tA0A2K5K1U3_COLAP\nMAP2K1\tA0A2K5K1W0_COLAP\nMAP2K1\tA0A2K5K1Y2_COLAP\nMAP2K1\tA0A2K5K1Z3_COLAP\nMAP2K1\tA0A2K5MX47_CERAT\nMAP2K1\tA0A2K5MX93_CERAT\nMAP2K1\tA0A2K5TK68_MACFA\nMAP2K1\tA0A2K5TK74_MACFA\nMAP2K1\tA0A2K5TK82_MACFA\nMAP2K1\tA0A2K5TK93_MACFA\nMAP2K1\tA0A2K5TK95_MACFA\nMAP2K1\tA0A2K5TKA9_MACFA\nMAP2K1\tA0A2K5YWG9_MANLE\nMAP2K1\tA0A2K5YWI6_MANLE\nMAP2K1\tA0A2K5YWK0_MANLE\nMAP2K1\tA0A2K5YWM1_MANLE\nMAP2K1\tA0A2K6B4J8_MACNE\nMAP2K1\tA0A2K6B4K4_MACNE\nMAP2K1\tA0A2K6FPP4_PROCO\nMAP2K1\tA0A2K6FPQ7_PROCO\nMAP2K1\tA0A2K6KU74_RHIBE\nMAP2K1\tA0A2K6KU99_RHIBE\nMAP2K1\tA0A2K6KUD8_RHIBE\nMAP2K1\tA0A2K6KUE0_RHIBE\nMAP2K1\tA0A2K6PWS8_RHIRO\nMAP2K1\tA0A2K6PWT6_RHIRO\nMAP2K1\tA0A2K6PWU9_RHIRO\nMAP2K1\tA0A2K6PWV1_RHIRO\nMAP2K1\tA0A2K6SZ10_SAIBB\nMAP2K1\tA0A2K6SZ51_SAIBB\nMAP2K1\tA0A2K6SZ56_SAIBB\nMAP2K1\tA0A2K6SZ66_SAIBB\nMAP2K1\tA0A2R8MZZ1_CALJA\nMAP2K1\tA0A2R9A0I9_PANPA\nMAP2K1\tA0A2R9A0J4_PANPA\nMAP2K1\tA0A2R9A0L1_PANPA\nMAP2K1\tA0A2R9A4U3_PANPA\nMAP2K1\tA0A2R9A4V5_PANPA\nMAP2K1\tA0A2U3WY84_ODORO\nMAP2K1\tA0A2U3WYJ0_ODORO\nMAP2K1\tA0A2U3XED8_LEPWE\nMAP2K1\tA0A2U4CFK4_TURTR\nMAP2K1\tA0A2Y9FRV8_PHYMC\nMAP2K1\tA0A2Y9HDF9_NEOSC\nMAP2K1\tA0A2Y9P0A4_DELLE\nMAP2K1\tA0A340WM60_LIPVE\nMAP2K1\tA0A340WME3_LIPVE\nMAP2K1\tA0A341AJX6_NEOAA\nMAP2K1\tA0A383YTE9_BALAS\nMAP2K1\tA0A384DIG4_URSMA\nMAP2K1\tA0A386H9M9_CYPCA\nMAP2K1\tA0A3Q0DLF7_TARSY\nMAP2K1\tA0A3Q7Q000_CALUR\nMAP2K1\tA0A3Q7TEL4_URSAR\nMAP2K1\tA0A3Q7U1P7_URSAR\nMAP2K1\tA0A452FI55_CAPHI\nMAP2K1\tA0A452FIW7_CAPHI\nMAP2K1\tA0A452GU44_9SAUR\nMAP2K1\tA0A452GU48_9SAUR\nMAP2K1\tA0A452GUE7_9SAUR\nMAP2K1\tA0A452GUF3_9SAUR\nMAP2K1\tA0A4D8YBX4_SALSN\nMAP2K1\tA0A4D8YJ89_SALSN\nMAP2K1\tA0A4D8YRS9_SALSN\nMAP2K1\tA0A4D8YTM7_SALSN\nMAP2K1\tA0A4D8YYB7_SALSN\nMAP2K1\tA0A4D8Z029_SALSN\nMAP2K1\tA0A4D9AI04_SALSN\nMAP2K1\tA0A4D9AL83_SALSN\nMAP2K1\tA0A4D9ARP5_SALSN\nMAP2K1\tA0A4D9B9Y2_SALSN\nMAP2K1\tA0A4D9BEM6_SALSN\nMAP2K1\tA0A4D9BIC8_SALSN\nMAP2K1\tA0A4D9BNA2_SALSN\nMAP2K1\tA0A4D9C3Q3_SALSN\nMAP2K1\tA0A4D9C4X6_SALSN\nMAP2K1\tA0A4W2D4M5_BOBOX\nMAP2K1\tA0A4W2F693_BOBOX\nMAP2K1\tA0A4W3JQ90_CALMI\nMAP2K1\tA0A4W3K0S4_CALMI\nMAP2K1\tA0A4W3KEW1_CALMI\nMAP2K1\tA0A4W4HDE6_ELEEL\nMAP2K1\tA0A4W4HDF7_ELEEL\nMAP2K1\tA0A4W4HDH7_ELEEL\nMAP2K1\tA0A4W4HGA6_ELEEL\nMAP2K1\tA0A4W4HGC7_ELEEL\nMAP2K1\tA0A4W4HH83_ELEEL\nMAP2K1\tA0A4W4HH89_ELEEL\nMAP2K1\tA0A4W4HHA9_ELEEL\nMAP2K1\tA0A4W4HHB4_ELEEL\nMAP2K1\tA0A4W6DL43_LATCA\nMAP2K1\tA0A4W6DL46_LATCA\nMAP2K1\tA0A4W6DMV2_LATCA\nMAP2K1\tA0A4W6DMZ5_LATCA\nMAP2K1\tA0A4W6DN17_LATCA\nMAP2K1\tA0A4X1TBL5_PIG\nMAP2K1\tA0A4X1TBP6_PIG\nMAP2K1\tA0A4X1TC54_PIG\nMAP2K1\tA0A4X1TDM5_PIG\nMAP2K1\tA0A4X2LLJ7_VOMUR\nMAP2K1\tA0A5F4BTT9_CANLF\nMAP2K1\tA0A5F4CA32_CANLF\nMAP2K1\tA0A5F5XCZ7_FELCA\nMAP2K1\tA0A5F9C5E9_RABIT\nMAP2K1\tA0A5G2R297_PIG\nMAP2K1\tA0A5G3JSB5_XENTR\nMAP2K1\tA0A5G3JSI0_XENTR\nMAP2K1\tA0A5G3JSK6_XENTR\nMAP2K1\tA4QPA9_HUMAN\nMAP2K1\tA5WWB4_DANRE\nMAP2K1\tB4DFY5_HUMAN\nMAP2K1\tB8XSJ8_PIG\nMAP2K1\tD2GYT9_AILME\nMAP2K1\tE5SMI4_TRISP\nMAP2K1\tF1PJI7_CANLF\nMAP2K1\tF6RQW8_XENTR\nMAP2K1\tF6XT65_MONDO\nMAP2K1\tF6YYZ1_XENTR\nMAP2K1\tF6Z152_ORNAN\nMAP2K1\tF7BWL2_HORSE\nMAP2K1\tF7C389_MACMU\nMAP2K1\tG1NC05_MELGA\nMAP2K1\tG1PJ27_MYOLU\nMAP2K1\tG1RR24_NOMLE\nMAP2K1\tG1SVV6_RABIT\nMAP2K1\tG3SYA0_LOXAF\nMAP2K1\tG3ULR7_LOXAF\nMAP2K1\tG5E7U6_MELGA\nMAP2K1\tG7YFH4_CLOSI\nMAP2K1\tH0UZY7_CAVPO\nMAP2K1\tH0XE68_OTOGA\nMAP2K1\tH0ZGH0_TAEGU\nMAP2K1\tH2LRK2_ORYLA\nMAP2K1\tH2NNJ6_PONAB\nMAP2K1\tH2Q9N2_PANTR\nMAP2K1\tH2UJM6_TAKRU\nMAP2K1\tH3ACB8_LATCH\nMAP2K1\tH3BRW9_HUMAN\nMAP2K1\tI2CR71_NANGC\nMAP2K1\tI3KE74_ORENI\nMAP2K1\tI3MD33_ICTTR\nMAP2K1\tJ9P5W6_CANLF\nMAP2K1\tK7FXY9_PELSI\nMAP2K1\tK8YSV9_NANGC\nMAP2K1\tM3X621_FELCA\nMAP2K1\tM3XMR0_MUSPF\nMAP2K1\tMP2K1_CRIGR\nMAP2K1\tMP2K1_HUMAN\nMAP2K1\tMP2K1_MOUSE\nMAP2K1\tMP2K1_PANTR\nMAP2K1\tMP2K1_RABIT\nMAP2K1\tMP2K1_RAT\nMAP2K1\tMP2K1_SERCA\nMAP2K1\tMP2K1_XENLA\nMAP2K1\tQ0VD16_BOVIN\nMAP2K1\tQ1LUY7_DANRE\nMAP2K1\tQ3TMJ8_MOUSE\nMAP2K1\tQ3TMW4_MOUSE\nMAP2K1\tQ3U3F4_MOUSE\nMAP2K1\tQ3UK05_MOUSE\nMAP2K1\tQ4V947_DANRE\nMAP2K1\tQ5ZIF0_CHICK\nMAP2K1\tQ66JE6_XENTR\nMAP2K1\tQ7ZTX6_DANRE\nMAP2K1\tQ9JJE1_MOUSE\nMAP2K1\tR9SA15_CAPHI\nMAP2K1\tT2ME67_HYDVU\nMAP2K1\tU3CZR8_CALJA\nMAP2K1\tU3J7A4_ANAPP\nMAP2K1\tU3K5Z4_FICAL\nMAP2K1\tV8N837_OPHHA\nMAP2K1\tV9KDJ0_CALMI\nMAP2K1\tW5Q9X5_SHEEP\n','From\tTo\nPLK4\tA0A034W3X3_BACDO\nPLK4\tA0A034W4X6_BACDO\nPLK4\tA0A090BML5_HUMAN\nPLK4\tA0A096MT32_PAPAN\nPLK4\tA0A0A9Z7A8_LYGHE\nPLK4\tA0A0D9S0Q9_CHLSB\nPLK4\tA0A0G2JZ86_RAT\nPLK4\tA0A0N4SWB8_MOUSE\nPLK4\tA0A0S7EW45_9TELE\nPLK4\tA0A0S7MDC9_9TELE\nPLK4\tA0A0X3Q083_SCHSO\nPLK4\tA0A151PEB0_ALLMI\nPLK4\tA0A1A7XQ12_9TELE\nPLK4\tA0A1A8CFR9_9TELE\nPLK4\tA0A1A8DT80_9TELE\nPLK4\tA0A1A8FCW3_9TELE\nPLK4\tA0A1A8H2I4_9TELE\nPLK4\tA0A1A8ING0_NOTKU\nPLK4\tA0A1A8K390_NOTKU\nPLK4\tA0A1A8K3B6_NOTKU\nPLK4\tA0A1A8M660_9TELE\nPLK4\tA0A1A8N9K1_9TELE\nPLK4\tA0A1A8S5H3_9TELE\nPLK4\tA0A1A8U1R0_NOTFU\nPLK4\tA0A1L8HLY0_XENLA\nPLK4\tA0A1L8HUQ2_XENLA\nPLK4\tA0A1S2ZR20_ERIEU\nPLK4\tA0A1S3GN79_DIPOR\nPLK4\tA0A1U7QAN3_MESAU\nPLK4\tA0A1U8BR27_MESAU\nPLK4\tA0A1U8D3E9_ALLSI\nPLK4\tA0A1V4JUF1_PATFA\nPLK4\tA0A218UEI3_9PASE\nPLK4\tA0A286XIR9_CAVPO\nPLK4\tA0A286Y1S1_CAVPO\nPLK4\tA0A286ZIK8_PIG\nPLK4\tA0A287AGN3_PIG\nPLK4\tA0A287BE28_PIG\nPLK4\tA0A287D5F1_ICTTR\nPLK4\tA0A291S6W7_9HYME\nPLK4\tA0A2D0RL52_ICTPU\nPLK4\tA0A2D0RLP9_ICTPU\nPLK4\tA0A2I0MH42_COLLI\nPLK4\tA0A2I3FWI6_NOMLE\nPLK4\tA0A2I3H8G3_NOMLE\nPLK4\tA0A2I3LCT7_PAPAN\nPLK4\tA0A2I3LDF1_PAPAN\nPLK4\tA0A2I3M938_PAPAN\nPLK4\tA0A2I3SKY1_PANTR\nPLK4\tA0A2I3T2C5_PANTR\nPLK4\tA0A2I4BMX8_9TELE\nPLK4\tA0A2J8XPI8_PONAB\nPLK4\tA0A2K5EC51_AOTNA\nPLK4\tA0A2K5EC56_AOTNA\nPLK4\tA0A2K5ECA9_AOTNA\nPLK4\tA0A2K5HW41_COLAP\nPLK4\tA0A2K5HW46_COLAP\nPLK4\tA0A2K5HW98_COLAP\nPLK4\tA0A2K5ML92_CERAT\nPLK4\tA0A2K5MLB2_CERAT\nPLK4\tA0A2K5MLK0_CERAT\nPLK4\tA0A2K5W9E4_MACFA\nPLK4\tA0A2K5W9H1_MACFA\nPLK4\tA0A2K5W9L2_MACFA\nPLK4\tA0A2K5YMT9_MANLE\nPLK4\tA0A2K5YMX8_MANLE\nPLK4\tA0A2K5YMY3_MANLE\nPLK4\tA0A2K6BS58_MACNE\nPLK4\tA0A2K6BS75_MACNE\nPLK4\tA0A2K6BS89_MACNE\nPLK4\tA0A2K6EX17_PROCO\nPLK4\tA0A2K6EX25_PROCO\nPLK4\tA0A2K6EX61_PROCO\nPLK4\tA0A2K6K3M9_RHIBE\nPLK4\tA0A2K6K3N5_RHIBE\nPLK4\tA0A2K6K3P5_RHIBE\nPLK4\tA0A2K6RQ83_RHIRO\nPLK4\tA0A2K6RQ87_RHIRO\nPLK4\tA0A2K6RQB6_RHIRO\nPLK4\tA0A2K6V7Q2_SAIBB\nPLK4\tA0A2K6V7S2_SAIBB\nPLK4\tA0A2K6V7S5_SAIBB\nPLK4\tA0A2R9A702_PANPA\nPLK4\tA0A2R9A842_PANPA\nPLK4\tA0A2R9AC53_PANPA\nPLK4\tA0A2U3V7K8_TURTR\nPLK4\tA0A2U3V7L8_TURTR\nPLK4\tA0A2U3VFY2_ODORO\nPLK4\tA0A2U3VFZ8_ODORO\nPLK4\tA0A2U3VGE0_ODORO\nPLK4\tA0A2U3X7P2_LEPWE\nPLK4\tA0A2U4AR77_TURTR\nPLK4\tA0A2U4AR78_TURTR\nPLK4\tA0A2U4AR79_TURTR\nPLK4\tA0A2U4AR86_TURTR\nPLK4\tA0A2Y9FEY9_PHYMC\nPLK4\tA0A2Y9FGU0_PHYMC\nPLK4\tA0A2Y9GDI8_NEOSC\nPLK4\tA0A2Y9LTG5_DELLE\nPLK4\tA0A2Y9LV76_DELLE\nPLK4\tA0A2Y9M043_DELLE\nPLK4\tA0A337SPE5_FELCA\nPLK4\tA0A340WA44_LIPVE\nPLK4\tA0A340WCN5_LIPVE\nPLK4\tA0A340WFG7_LIPVE\nPLK4\tA0A340WGM5_LIPVE\nPLK4\tA0A341BGF6_NEOAA\nPLK4\tA0A341BGJ0_NEOAA\nPLK4\tA0A383ZBB8_BALAS\nPLK4\tA0A383ZC28_BALAS\nPLK4\tA0A384CTX8_URSMA\nPLK4\tA0A3Q0DRX5_TARSY\nPLK4\tA0A3Q2GX71_HORSE\nPLK4\tA0A3Q2L9B7_HORSE\nPLK4\tA0A3Q7P844_CALUR\nPLK4\tA0A3Q7TK87_VULVU\nPLK4\tA0A3Q7VS50_URSAR\nPLK4\tA0A452FSQ0_CAPHI\nPLK4\tA0A452FT92_CAPHI\nPLK4\tA0A452FTA6_CAPHI\nPLK4\tA0A452I741_9SAUR\nPLK4\tA0A452I773_9SAUR\nPLK4\tA0A452QVE0_URSAM\nPLK4\tA0A452QVE4_URSAM\nPLK4\tA0A4W2BX42_BOBOX\nPLK4\tA0A4W2DSH0_BOBOX\nPLK4\tA0A4W2DYC2_BOBOX\nPLK4\tA0A4W2FJ48_BOBOX\nPLK4\tA0A4W2FJ63_BOBOX\nPLK4\tA0A4W2HLC9_BOBOX\nPLK4\tA0A4W4EKM0_ELEEL\nPLK4\tA0A4W4EL86_ELEEL\nPLK4\tA0A4W4EMC6_ELEEL\nPLK4\tA0A4W4ENP5_ELEEL\nPLK4\tA0A4W6FA03_LATCA\nPLK4\tA0A4W6FC39_LATCA\nPLK4\tA0A4W6FC44_LATCA\nPLK4\tA0A4W6FD10_LATCA\nPLK4\tA0A4X2KFX5_VOMUR\nPLK4\tA0A4X2KG58_VOMUR\nPLK4\tA0A4Z2J3Z4_9TELE\nPLK4\tA0A5B7FGW0_PORTR\nPLK4\tA0A5F4BX84_CANLF\nPLK4\tA0A5F4VYF7_CALJA\nPLK4\tA0A5F9DG32_RABIT\nPLK4\tA0A5G2QN48_PIG\nPLK4\tA0A5G2QSN4_PIG\nPLK4\tC3KH52_ANOFI\nPLK4\tE1C1M7_CHICK\nPLK4\tE2RST5_CANLF\nPLK4\tE9PVH4_MOUSE\nPLK4\tE9Q4W7_MOUSE\nPLK4\tF1RB74_DANRE\nPLK4\tF1RRA7_PIG\nPLK4\tF1RRA8_PIG\nPLK4\tF6QIM3_MOUSE\nPLK4\tF6TBK5_HORSE\nPLK4\tF6U615_XENTR\nPLK4\tF6YGH3_MONDO\nPLK4\tF7E1H1_CALJA\nPLK4\tF7EHF3_MACMU\nPLK4\tF7GW78_CALJA\nPLK4\tG1KLA0_ANOCA\nPLK4\tG1L7I3_AILME\nPLK4\tG1MZR4_MELGA\nPLK4\tG1P4M4_MYOLU\nPLK4\tG1RTU0_NOMLE\nPLK4\tG1SN07_RABIT\nPLK4\tG3SLN8_LOXAF\nPLK4\tG3UAN2_LOXAF\nPLK4\tG3WKI0_SARHA\nPLK4\tG3WKI1_SARHA\nPLK4\tG3WKI2_SARHA\nPLK4\tG7MRY8_MACMU\nPLK4\tH0VTN5_CAVPO\nPLK4\tH0WSS6_OTOGA\nPLK4\tH0YAL6_HUMAN\nPLK4\tH0YV27_TAEGU\nPLK4\tH2MMP1_ORYLA\nPLK4\tH2T1P5_TAKRU\nPLK4\tH2T1P6_TAKRU\nPLK4\tH3B8Z3_LATCH\nPLK4\tH9Z4J3_MACMU\nPLK4\tI3KW71_ORENI\nPLK4\tI3N028_ICTTR\nPLK4\tJ3KR82_HUMAN\nPLK4\tJ3KR84_HUMAN\nPLK4\tJ9P681_CANLF\nPLK4\tK7BZR6_PANTR\nPLK4\tK7FLW5_PELSI\nPLK4\tK7FLX1_PELSI\nPLK4\tK7J0A2_NASVI\nPLK4\tM3VU55_FELCA\nPLK4\tM3Z0Z9_MUSPF\nPLK4\tM4SMK4_9BILA\nPLK4\tM4SZK9_9BILA\nPLK4\tPLK4_BOVIN\nPLK4\tPLK4_DANRE\nPLK4\tPLK4_HUMAN\nPLK4\tPLK4_MOUSE\nPLK4\tPLK4_PONAB\nPLK4\tPLK4_RAT\nPLK4\tPLK4_XENLA\nPLK4\tPLK4_XENTR\nPLK4\tQ3V382_MOUSE\nPLK4\tQ8C3W9_MOUSE\nPLK4\tR0KNG3_NOSB1\nPLK4\tT2MJ85_HYDVU\nPLK4\tU3IPH8_ANAPP\nPLK4\tU3K0B6_FICAL\nPLK4\tU6CPY8_NEOVI\nPLK4\tV8P501_OPHHA\nPLK4\tW5Q2A1_SHEEP\nPLK4\tW5Q2A3_SHEEP\nPLK4\tW5UBF0_ICTPU\nPLK4\tW8AFI5_CERCA\nPLK4\tW8AR33_CERCA\n','From\tTo\nPLK1\tA0A061QJ96_9CHLO\nPLK1\tA0A061SMU7_9CHLO\nPLK1\tA0A096NHF8_PAPAN\nPLK1\tA0A0A9W8D2_LYGHE\nPLK1\tA0A0B2UZP3_TOXCA\nPLK1\tA0A0B2V9T6_TOXCA\nPLK1\tA0A0B2VW69_TOXCA\nPLK1\tA0A0D9R331_CHLSB\nPLK1\tA0A0K2T7L5_LEPSM\nPLK1\tA0A0K2UDM3_LEPSM\nPLK1\tA0A0P6JUX9_HETGA\nPLK1\tA0A0S7KGW7_9TELE\nPLK1\tA0A0S7KGW8_9TELE\nPLK1\tA0A0S7KI08_9TELE\nPLK1\tA0A0U1RNM9_MOUSE\nPLK1\tA0A0U1RQ54_MOUSE\nPLK1\tA0A0V0SEK9_9BILA\nPLK1\tA0A0V0TN46_9BILA\nPLK1\tA0A0V0WKC1_9BILA\nPLK1\tA0A0V0XR31_TRIPS\nPLK1\tA0A0V0ZPZ4_9BILA\nPLK1\tA0A0V1B3B1_TRISP\nPLK1\tA0A0V1CYQ3_TRIBR\nPLK1\tA0A0V1E3W1_TRIPS\nPLK1\tA0A0V1FR85_TRIPS\nPLK1\tA0A0V1HZM5_9BILA\nPLK1\tA0A0V1LP83_9BILA\nPLK1\tA0A0V1MTD3_9BILA\nPLK1\tA0A0V1MTJ0_9BILA\nPLK1\tA0A0V1PM88_9BILA\nPLK1\tA0A140TAD2_RAT\nPLK1\tA0A151NWB1_ALLMI\nPLK1\tA0A1A7WC43_9TELE\nPLK1\tA0A1A8AAM9_NOTFU\nPLK1\tA0A1A8BVS1_9TELE\nPLK1\tA0A1A8DW05_9TELE\nPLK1\tA0A1A8DXX4_9TELE\nPLK1\tA0A1A8FIC4_9TELE\nPLK1\tA0A1A8IHW3_NOTKU\nPLK1\tA0A1A8IJY0_NOTKU\nPLK1\tA0A1A8M2G1_9TELE\nPLK1\tA0A1A8QL57_9TELE\nPLK1\tA0A1A8QZL9_9TELE\nPLK1\tA0A1A8QZM5_9TELE\nPLK1\tA0A1A8R0K5_9TELE\nPLK1\tA0A1B1FFQ3_XENTR\nPLK1\tA0A1J4JAE1_9EUKA\nPLK1\tA0A1J4K7Y1_9EUKA\nPLK1\tA0A1L1RPS3_CHICK\nPLK1\tA0A1L8EYC7_XENLA\nPLK1\tA0A1S2ZB20_ERIEU\nPLK1\tA0A1S3EQG1_DIPOR\nPLK1\tA0A1S3QHM5_SALSA\nPLK1\tA0A1U7QC35_MESAU\nPLK1\tA0A1U7RWN2_ALLSI\nPLK1\tA0A1U7TBJ7_TARSY\nPLK1\tA0A1V4KT09_PATFA\nPLK1\tA0A218UAG4_9PASE\nPLK1\tA0A286XZU5_CAVPO\nPLK1\tA0A291S6W5_9HYME\nPLK1\tA0A2B4R581_STYPI\nPLK1\tA0A2B4S7G0_STYPI\nPLK1\tA0A2B4SHS1_STYPI\nPLK1\tA0A2D0Q3S0_ICTPU\nPLK1\tA0A2I0LX11_COLLI\nPLK1\tA0A2I3NF45_PAPAN\nPLK1\tA0A2I3RMF0_PANTR\nPLK1\tA0A2I3TX53_PANTR\nPLK1\tA0A2I4BU86_9TELE\nPLK1\tA0A2K5D5J2_AOTNA\nPLK1\tA0A2K5D5K5_AOTNA\nPLK1\tA0A2K5HUR1_COLAP\nPLK1\tA0A2K5HUY6_COLAP\nPLK1\tA0A2K5HV13_COLAP\nPLK1\tA0A2K5NUI3_CERAT\nPLK1\tA0A2K5NUI4_CERAT\nPLK1\tA0A2K5QTL9_CEBCA\nPLK1\tA0A2K5QTM3_CEBCA\nPLK1\tA0A2K5Z4L7_MANLE\nPLK1\tA0A2K5Z4S2_MANLE\nPLK1\tA0A2K6E282_MACNE\nPLK1\tA0A2K6E288_MACNE\nPLK1\tA0A2K6F253_PROCO\nPLK1\tA0A2K6F259_PROCO\nPLK1\tA0A2K6N4W3_RHIBE\nPLK1\tA0A2K6N4W8_RHIBE\nPLK1\tA0A2K6N4X6_RHIBE\nPLK1\tA0A2K6N4Y9_RHIBE\nPLK1\tA0A2K6N532_RHIBE\nPLK1\tA0A2K6QHX7_RHIRO\nPLK1\tA0A2K6QHY2_RHIRO\nPLK1\tA0A2K6QHY8_RHIRO\nPLK1\tA0A2K6QHZ0_RHIRO\nPLK1\tA0A2K6QHZ1_RHIRO\nPLK1\tA0A2K6SMW1_SAIBB\nPLK1\tA0A2K6SMZ8_SAIBB\nPLK1\tA0A2K6SN19_SAIBB\nPLK1\tA0A2P8XV43_BLAGE\nPLK1\tA0A2R8ZMQ3_PANPA\nPLK1\tA0A2R8ZMS0_PANPA\nPLK1\tA0A2R8ZMS5_PANPA\nPLK1\tA0A2R8ZNY1_PANPA\nPLK1\tA0A2U3VU23_ODORO\nPLK1\tA0A2U3XHB0_LEPWE\nPLK1\tA0A2U4A9N9_TURTR\nPLK1\tA0A2Y9EPA8_PHYMC\nPLK1\tA0A2Y9GTD1_NEOSC\nPLK1\tA0A2Y9PEA9_DELLE\nPLK1\tA0A340WSL1_LIPVE\nPLK1\tA0A341BD30_NEOAA\nPLK1\tA0A384AG87_BALAS\nPLK1\tA0A384CLW1_URSMA\nPLK1\tA0A3B3CK63_ORYME\nPLK1\tA0A3B3CLN3_ORYME\nPLK1\tA0A3B3IBE9_ORYLA\nPLK1\tA0A3B3SRT7_9TELE\nPLK1\tA0A3B3SSZ3_9TELE\nPLK1\tA0A3B3YYC4_9TELE\nPLK1\tA0A3B3YZ39_9TELE\nPLK1\tA0A3B4APJ9_9GOBI\nPLK1\tA0A3B4FDS4_9CICH\nPLK1\tA0A3B4UQC0_SERDU\nPLK1\tA0A3B4YCI7_SERLL\nPLK1\tA0A3B5AG94_9TELE\nPLK1\tA0A3B5LHT0_9TELE\nPLK1\tA0A3B5PST5_XIPMA\nPLK1\tA0A3P8NNH3_ASTCA\nPLK1\tA0A3P8STL8_AMPPE\nPLK1\tA0A3P8ULK4_CYNSE\nPLK1\tA0A3P8YJU0_ESOLU\nPLK1\tA0A3P8YJW0_ESOLU\nPLK1\tA0A3P8YK22_ESOLU\nPLK1\tA0A3P9C789_9CICH\nPLK1\tA0A3P9HNG6_ORYLA\nPLK1\tA0A3P9HNY3_ORYLA\nPLK1\tA0A3P9KN58_ORYLA\nPLK1\tA0A3P9M3V1_ORYLA\nPLK1\tA0A3P9MRW2_POERE\nPLK1\tA0A3Q0T0F5_AMPCI\nPLK1\tA0A3Q1BTM6_AMPOC\nPLK1\tA0A3Q1DGG0_AMPOC\nPLK1\tA0A3Q1FH85_9TELE\nPLK1\tA0A3Q1IVS9_ANATE\nPLK1\tA0A3Q1LFD8_BOVIN\nPLK1\tA0A3Q2EDD9_CYPVA\nPLK1\tA0A3Q2NWH2_FUNHE\nPLK1\tA0A3Q2SUI3_FUNHE\nPLK1\tA0A3Q2VQ40_HAPBU\nPLK1\tA0A3Q2WWN9_HAPBU\nPLK1\tA0A3Q2XVG1_HIPCM\nPLK1\tA0A3Q2ZAF2_HIPCM\nPLK1\tA0A3Q3EV54_9LABR\nPLK1\tA0A3Q3EVA0_9LABR\nPLK1\tA0A3Q3EVD5_9LABR\nPLK1\tA0A3Q3FGL0_KRYMA\nPLK1\tA0A3Q3K3G4_MONAL\nPLK1\tA0A3Q3L9M7_9TELE\nPLK1\tA0A3Q3LL79_9TELE\nPLK1\tA0A3Q4GGS2_NEOBR\nPLK1\tA0A3Q7MVJ8_CALUR\nPLK1\tA0A3Q7RBS4_VULVU\nPLK1\tA0A3Q7XWE2_URSAR\nPLK1\tA0A3R7DNG4_CLOSI\nPLK1\tA0A452FQM0_CAPHI\nPLK1\tA0A452GMP8_9SAUR\nPLK1\tA0A452TUC2_URSMA\nPLK1\tA0A452TUI5_URSMA\nPLK1\tA0A4W2BRV5_BOBOX\nPLK1\tA0A4W2E8U8_BOBOX\nPLK1\tA0A4W2I357_BOBOX\nPLK1\tA0A4W3IPN3_CALMI\nPLK1\tA0A4W3IT11_CALMI\nPLK1\tA0A4W3IT24_CALMI\nPLK1\tA0A4W4GA92_ELEEL\nPLK1\tA0A4W4GEW4_ELEEL\nPLK1\tA0A4W6FDG4_LATCA\nPLK1\tA0A4W6FED8_LATCA\nPLK1\tA0A4X2L7Q2_VOMUR\nPLK1\tA0A4X2L8R3_VOMUR\nPLK1\tA0A4X2L8S7_VOMUR\nPLK1\tA0A4X2LAP1_VOMUR\nPLK1\tA0A4Y2CFM5_ARAVE\nPLK1\tA0A4Z2FSG3_9TELE\nPLK1\tA0A5F9CIJ3_RABIT\nPLK1\tA0A5G3HCG4_XENTR\nPLK1\tA0A5G3IWP9_XENTR\nPLK1\tC1LE39_SCHJA\nPLK1\tD3DWF3_HUMAN\nPLK1\tD9I8G6_HALRO\nPLK1\tF1NCX7_CHICK\nPLK1\tF6RRI6_HORSE\nPLK1\tF6W916_ORNAN\nPLK1\tF7F258_MONDO\nPLK1\tF7HSC1_MACMU\nPLK1\tF7I711_CALJA\nPLK1\tF7I713_CALJA\nPLK1\tG1K3E4_XENTR\nPLK1\tG1M6R4_AILME\nPLK1\tG1N426_MELGA\nPLK1\tG1PJY4_MYOLU\nPLK1\tG1RMX3_NOMLE\nPLK1\tG1SV90_RABIT\nPLK1\tG2HFG8_PANTR\nPLK1\tG3PW36_GASAC\nPLK1\tG3SW75_LOXAF\nPLK1\tG3VS91_SARHA\nPLK1\tH0WSC6_OTOGA\nPLK1\tH0Z6U1_TAEGU\nPLK1\tH2M0W2_ORYLA\nPLK1\tH2NQF2_PONAB\nPLK1\tH2ULF7_TAKRU\nPLK1\tH3A4R9_LATCH\nPLK1\tH9FBA4_MACMU\nPLK1\tH9FLA7_MACMU\nPLK1\tH9GIK2_ANOCA\nPLK1\tH9ZA67_MACMU\nPLK1\tI3IUE2_ORENI\nPLK1\tI3L2H5_HUMAN\nPLK1\tI3L309_HUMAN\nPLK1\tI3L387_HUMAN\nPLK1\tI3LII4_PIG\nPLK1\tI3MJ33_ICTTR\nPLK1\tJ9P0Q4_CANLF\nPLK1\tK7FEE7_PELSI\nPLK1\tM3WHX6_FELCA\nPLK1\tM3YL84_MUSPF\nPLK1\tM4AEH2_XIPMA\nPLK1\tM4SJ30_9BILA\nPLK1\tM4SL44_9BILA\nPLK1\tPLK1_BOVIN\nPLK1\tPLK1_HUMAN\nPLK1\tPLK1_MOUSE\nPLK1\tPLK1_RAT\nPLK1\tPLK1_XENLA\nPLK1\tPLK1_XENTR\nPLK1\tQ3TPZ2_MOUSE\nPLK1\tQ3UG81_MOUSE\nPLK1\tQ4KMI8_DANRE\nPLK1\tQ6DRK7_DANRE\nPLK1\tS5MCV3_BOMMO\nPLK1\tT2MFR1_HYDVU\nPLK1\tU3JMD3_FICAL\nPLK1\tU6D0E2_NEOVI\nPLK1\tU6HQM1_ECHMU\nPLK1\tW5MEL5_LEPOC\nPLK1\tW5Q7C4_SHEEP\n','From\tTo\nNEK7\tA0A096NWS1_PAPAN\nNEK7\tA0A0B2W117_TOXCA\nNEK7\tA0A0D9RMG7_CHLSB\nNEK7\tA0A0G2JUP0_RAT\nNEK7\tA0A0S7LY83_9TELE\nNEK7\tA0A151NCK8_ALLMI\nNEK7\tA0A151NCL7_ALLMI\nNEK7\tA0A1A7WMK5_9TELE\nNEK7\tA0A1A8ECI9_9TELE\nNEK7\tA0A1A8FYJ7_9TELE\nNEK7\tA0A1A8I0D6_NOTKU\nNEK7\tA0A1A8NLM5_9TELE\nNEK7\tA0A1A8QEI2_9TELE\nNEK7\tA0A1A8V0G1_NOTFU\nNEK7\tA0A1D5QLT8_MACMU\nNEK7\tA0A1D5R0M3_MACMU\nNEK7\tA0A1I9LS93_ARATH\nNEK7\tA0A1I9LS94_ARATH\nNEK7\tA0A1J4K3R6_9EUKA\nNEK7\tA0A1Q9EFM1_SYMMI\nNEK7\tA0A1S3ASM1_ERIEU\nNEK7\tA0A1S3F8M6_DIPOR\nNEK7\tA0A1S3P9A2_SALSA\nNEK7\tA0A1S3P9B8_SALSA\nNEK7\tA0A1S3P9G8_SALSA\nNEK7\tA0A1U7QS17_MESAU\nNEK7\tA0A1U7TKP1_TARSY\nNEK7\tA0A1U8D4N5_ALLSI\nNEK7\tA0A1U8DGA6_ALLSI\nNEK7\tA0A1V4JYB4_PATFA\nNEK7\tA0A218UVY5_9PASE\nNEK7\tA0A250XYC4_CASCN\nNEK7\tA0A286XQA0_CAVPO\nNEK7\tA0A287BE42_PIG\nNEK7\tA0A2B4SS79_STYPI\nNEK7\tA0A2D0RUF8_ICTPU\nNEK7\tA0A2D0RUG7_ICTPU\nNEK7\tA0A2I0ME18_COLLI\nNEK7\tA0A2I3FRQ1_NOMLE\nNEK7\tA0A2I3HDZ1_NOMLE\nNEK7\tA0A2I3HPJ3_NOMLE\nNEK7\tA0A2I3LDI7_PAPAN\nNEK7\tA0A2I3N4R0_PAPAN\nNEK7\tA0A2I3RIA1_PANTR\nNEK7\tA0A2I3S2T5_PANTR\nNEK7\tA0A2I4B4U9_9TELE\nNEK7\tA0A2J8UXU6_PONAB\nNEK7\tA0A2K5D2V7_AOTNA\nNEK7\tA0A2K5D2W1_AOTNA\nNEK7\tA0A2K5D2W5_AOTNA\nNEK7\tA0A2K5D2X3_AOTNA\nNEK7\tA0A2K5D306_AOTNA\nNEK7\tA0A2K5I1L3_COLAP\nNEK7\tA0A2K5I1Q4_COLAP\nNEK7\tA0A2K5I1R4_COLAP\nNEK7\tA0A2K5L532_CERAT\nNEK7\tA0A2K5L554_CERAT\nNEK7\tA0A2K5L564_CERAT\nNEK7\tA0A2K5L596_CERAT\nNEK7\tA0A2K5VYZ2_MACFA\nNEK7\tA0A2K5Y4T6_MANLE\nNEK7\tA0A2K5Y4U2_MANLE\nNEK7\tA0A2K5Y4X7_MANLE\nNEK7\tA0A2K6FB42_PROCO\nNEK7\tA0A2K6FB45_PROCO\nNEK7\tA0A2K6LSM4_RHIBE\nNEK7\tA0A2K6LSN3_RHIBE\nNEK7\tA0A2K6LSP2_RHIBE\nNEK7\tA0A2K6LSQ4_RHIBE\nNEK7\tA0A2K6RTX0_RHIRO\nNEK7\tA0A2K6RTX1_RHIRO\nNEK7\tA0A2K6RTX5_RHIRO\nNEK7\tA0A2K6RTX6_RHIRO\nNEK7\tA0A2K6RTX8_RHIRO\nNEK7\tA0A2K6RTZ1_RHIRO\nNEK7\tA0A2K6UIJ0_SAIBB\nNEK7\tA0A2K6UIM0_SAIBB\nNEK7\tA0A2P8YSN0_BLAGE\nNEK7\tA0A2R9A9G9_PANPA\nNEK7\tA0A2R9A9J7_PANPA\nNEK7\tA0A2R9AAL7_PANPA\nNEK7\tA0A2S2PL41_SCHGA\nNEK7\tA0A2S2PWB2_9HEMI\nNEK7\tA0A2S2QG99_9HEMI\nNEK7\tA0A2U3X3Y4_ODORO\nNEK7\tA0A2U3XDS3_LEPWE\nNEK7\tA0A2U4A4T5_TURTR\nNEK7\tA0A2U4A4U0_TURTR\nNEK7\tA0A2U4A4U2_TURTR\nNEK7\tA0A2U4A4U8_TURTR\nNEK7\tA0A2U4A4V0_TURTR\nNEK7\tA0A2U4A4V4_TURTR\nNEK7\tA0A2U4A4X1_TURTR\nNEK7\tA0A2Y9GH21_NEOSC\nNEK7\tA0A2Y9NLI3_DELLE\nNEK7\tA0A2Y9NPR4_DELLE\nNEK7\tA0A337S1S6_FELCA\nNEK7\tA0A337SAV8_FELCA\nNEK7\tA0A340WSG1_LIPVE\nNEK7\tA0A341ABH8_NEOAA\nNEK7\tA0A341AFU7_NEOAA\nNEK7\tA0A383YWR5_BALAS\nNEK7\tA0A383YWR8_BALAS\nNEK7\tA0A384BL59_URSMA\nNEK7\tA0A384BM08_URSMA\nNEK7\tA0A384BMF7_URSMA\nNEK7\tA0A3B3HEY4_ORYLA\nNEK7\tA0A3Q0CW06_MESAU\nNEK7\tA0A3Q0GQP9_ALLSI\nNEK7\tA0A3Q0GQS3_ALLSI\nNEK7\tA0A3Q0GVA7_ALLSI\nNEK7\tA0A3Q0GVB4_ALLSI\nNEK7\tA0A3Q1M4K7_BOVIN\nNEK7\tA0A3Q1MRG8_BOVIN\nNEK7\tA0A3Q2HLD5_HORSE\nNEK7\tA0A3Q2I054_HORSE\nNEK7\tA0A3Q7MUK2_CALUR\nNEK7\tA0A3Q7MUN3_CALUR\nNEK7\tA0A3Q7TK21_URSAR\nNEK7\tA0A3Q7UKP1_VULVU\nNEK7\tA0A3Q7VF28_URSAR\nNEK7\tA0A419Q1T8_CLOSI\nNEK7\tA0A452ENX3_CAPHI\nNEK7\tA0A452IJ34_9SAUR\nNEK7\tA0A452IJ97_9SAUR\nNEK7\tA0A452S2A0_URSAM\nNEK7\tA0A455B566_PHYMC\nNEK7\tA0A455B5K0_PHYMC\nNEK7\tA0A455B5K5_PHYMC\nNEK7\tA0A455B5L0_PHYMC\nNEK7\tA0A455B644_PHYMC\nNEK7\tA0A455B7W2_PHYMC\nNEK7\tA0A455B7W7_PHYMC\nNEK7\tA0A493TRM7_ANAPP\nNEK7\tA0A4W2C3D3_BOBOX\nNEK7\tA0A4W3I2R6_CALMI\nNEK7\tA0A4W3I5V9_CALMI\nNEK7\tA0A4W3I5W5_CALMI\nNEK7\tA0A4W3I5X9_CALMI\nNEK7\tA0A4W3I9M5_CALMI\nNEK7\tA0A4W3I9N6_CALMI\nNEK7\tA0A4W3IB00_CALMI\nNEK7\tA0A4W3IB06_CALMI\nNEK7\tA0A4W3IB26_CALMI\nNEK7\tA0A4W5R5Q3_9TELE\nNEK7\tA0A4W5RAD4_9TELE\nNEK7\tA0A4X1SPG3_PIG\nNEK7\tA0A4X1SR21_PIG\nNEK7\tA0A4X2K7Y3_VOMUR\nNEK7\tA0A4Y2IS11_ARAVE\nNEK7\tA0A4Y2V2X3_ARAVE\nNEK7\tA0A5B7JKY3_PORTR\nNEK7\tA0A5F4CY21_CANLF\nNEK7\tA0A5F5XCB7_FELCA\nNEK7\tA0A5F8G201_MONDO\nNEK7\tA0A5F8H4V7_MONDO\nNEK7\tA0A5F9CWG7_RABIT\nNEK7\tA0A5G3HJK1_XENTR\nNEK7\tA0A5G3HJS7_XENTR\nNEK7\tA0A5G3HJX8_XENTR\nNEK7\tA0A5G3HKY3_XENTR\nNEK7\tA0A5G3J6J3_XENTR\nNEK7\tB5XEJ8_SALSA\nNEK7\tB5XEN9_SALSA\nNEK7\tC9J1H8_HUMAN\nNEK7\tE1BDZ2_BOVIN\nNEK7\tE2R2U0_CANLF\nNEK7\tF4J8P0_ARATH\nNEK7\tF5H3U7_HUMAN\nNEK7\tF6TZY1_XENTR\nNEK7\tF6WY54_HORSE\nNEK7\tF7BN89_XENTR\nNEK7\tF7BNM2_XENTR\nNEK7\tF7EFY6_MONDO\nNEK7\tF7FA18_MACMU\nNEK7\tF8WAG2_HUMAN\nNEK7\tG1MGW0_AILME\nNEK7\tG1N2D2_MELGA\nNEK7\tG1PB68_MYOLU\nNEK7\tG1S8V7_NOMLE\nNEK7\tG1T198_RABIT\nNEK7\tG3SVQ4_LOXAF\nNEK7\tG3WKT1_SARHA\nNEK7\tG5BM78_HETGA\nNEK7\tG7NWH7_MACFA\nNEK7\tH0W7L5_CAVPO\nNEK7\tH0WP79_OTOGA\nNEK7\tH0Z1R3_TAEGU\nNEK7\tH2Q0U7_PANTR\nNEK7\tH2RPN9_TAKRU\nNEK7\tH2ZV56_LATCH\nNEK7\tH2ZV57_LATCH\nNEK7\tH9GL80_ANOCA\nNEK7\tI3ML78_ICTTR\nNEK7\tK4FSV2_CALMI\nNEK7\tK7FY74_PELSI\nNEK7\tM3YG99_MUSPF\nNEK7\tNEK7_ARATH\nNEK7\tNEK7_HUMAN\nNEK7\tNEK7_MOUSE\nNEK7\tNEK7_RAT\nNEK7\tQ0IHX5_XENTR\nNEK7\tQ3TN15_MOUSE\nNEK7\tQ5F3P1_CHICK\nNEK7\tQ6DEH7_DANRE\nNEK7\tQ6IU13_XENLA\nNEK7\tT2M984_HYDVU\nNEK7\tU3FJS7_CALJA\nNEK7\tU3JJB6_FICAL\nNEK7\tU6DSD2_NEOVI\nNEK7\tV8P2E9_OPHHA\nNEK7\tW5Q1L7_SHEEP\n','From\tTo\nNEK2\tA0A034VMB0_BACDO\nNEK2\tA0A060RUL7_PLARE\nNEK2\tA0A096NPN1_PAPAN\nNEK2\tA0A0A1WUJ4_ZEUCU\nNEK2\tA0A0D9RS67_CHLSB\nNEK2\tA0A0K8VEK1_BACLA\nNEK2\tA0A0S7ERG9_9TELE\nNEK2\tA0A0S7HZ14_9TELE\nNEK2\tA0A0Y9YUG4_PLABE\nNEK2\tA0A151P3L4_ALLMI\nNEK2\tA0A1A7ZBH6_NOTFU\nNEK2\tA0A1A8F375_9TELE\nNEK2\tA0A1A8JEI4_NOTKU\nNEK2\tA0A1A8MEW9_9TELE\nNEK2\tA0A1A8NCY3_9TELE\nNEK2\tA0A1A8UNC8_NOTFU\nNEK2\tA0A1C3KTD9_9APIC\nNEK2\tA0A1C6YJ52_PLACH\nNEK2\tA0A1C6YPX8_PLACH\nNEK2\tA0A1D3S1P6_PLACH\nNEK2\tA0A1D3U8D9_9APIC\nNEK2\tA0A1D5QTK8_MACMU\nNEK2\tA0A1J1GL27_PLAGA\nNEK2\tA0A1J1H6K3_PLARL\nNEK2\tA0A1Q9CW17_SYMMI\nNEK2\tA0A1Q9E2T3_SYMMI\nNEK2\tA0A1Q9F4U9_SYMMI\nNEK2\tA0A1S3A8S3_ERIEU\nNEK2\tA0A1S3A8T0_ERIEU\nNEK2\tA0A1S3F3R7_DIPOR\nNEK2\tA0A1U7R7Y8_MESAU\nNEK2\tA0A1U7SBE0_ALLSI\nNEK2\tA0A1U7T326_TARSY\nNEK2\tA0A1V4JJE2_PATFA\nNEK2\tA0A1X1BP59_9APIC\nNEK2\tA0A1Y3DVA7_PLAKN\nNEK2\tA0A218V4L3_9PASE\nNEK2\tA0A2B4SDX3_STYPI\nNEK2\tA0A2D0PZG6_ICTPU\nNEK2\tA0A2I0BAY5_9ASPA\nNEK2\tA0A2I0MAP8_COLLI\nNEK2\tA0A2I0V6M8_9ASPA\nNEK2\tA0A2I3MFZ5_PAPAN\nNEK2\tA0A2I3MVK0_PAPAN\nNEK2\tA0A2I3S721_PANTR\nNEK2\tA0A2I4CZ59_9TELE\nNEK2\tA0A2I4CZ74_9TELE\nNEK2\tA0A2K5EMS6_AOTNA\nNEK2\tA0A2K5IEU2_COLAP\nNEK2\tA0A2K5KM94_CERAT\nNEK2\tA0A2K5KMA5_CERAT\nNEK2\tA0A2K5RB66_CEBCA\nNEK2\tA0A2K5RB67_CEBCA\nNEK2\tA0A2K5RB70_CEBCA\nNEK2\tA0A2K5VX62_MACFA\nNEK2\tA0A2K5VX77_MACFA\nNEK2\tA0A2K6AAM9_MANLE\nNEK2\tA0A2K6AAR3_MANLE\nNEK2\tA0A2K6BA37_MACNE\nNEK2\tA0A2K6FCL7_PROCO\nNEK2\tA0A2K6KFZ2_RHIBE\nNEK2\tA0A2K6KG08_RHIBE\nNEK2\tA0A2K6KG43_RHIBE\nNEK2\tA0A2K6KG65_RHIBE\nNEK2\tA0A2K6P7G7_RHIRO\nNEK2\tA0A2K6P7I0_RHIRO\nNEK2\tA0A2K6P7L7_RHIRO\nNEK2\tA0A2K6UW98_SAIBB\nNEK2\tA0A2R9AXA8_PANPA\nNEK2\tA0A2U3V0E5_TURTR\nNEK2\tA0A2U3X554_ODORO\nNEK2\tA0A2U3YZ32_LEPWE\nNEK2\tA0A2Y9EH49_PHYMC\nNEK2\tA0A2Y9GGI0_NEOSC\nNEK2\tA0A2Y9Q096_DELLE\nNEK2\tA0A340WRD4_LIPVE\nNEK2\tA0A340WX02_LIPVE\nNEK2\tA0A341BVW1_NEOAA\nNEK2\tA0A371GR50_MUCPR\nNEK2\tA0A371GW57_MUCPR\nNEK2\tA0A371HZ54_MUCPR\nNEK2\tA0A383YMK6_BALAS\nNEK2\tA0A383YML2_BALAS\nNEK2\tA0A384CXE3_URSMA\nNEK2\tA0A3G2S115_9BASI\nNEK2\tA0A3M9Y034_9PEZI\nNEK2\tA0A3Q1MS69_BOVIN\nNEK2\tA0A3Q2HBN8_HORSE\nNEK2\tA0A3Q7QY41_VULVU\nNEK2\tA0A3Q7R0U7_CALUR\nNEK2\tA0A3Q7XPS7_URSAR\nNEK2\tA0A452E0B7_CAPHI\nNEK2\tA0A452HS05_9SAUR\nNEK2\tA0A452QTM3_URSAM\nNEK2\tA0A452QTP2_URSAM\nNEK2\tA0A452V6A7_URSMA\nNEK2\tA0A4W2GT77_BOBOX\nNEK2\tA0A4W2IFV5_BOBOX\nNEK2\tA0A4W3IF37_CALMI\nNEK2\tA0A4W3IH48_CALMI\nNEK2\tA0A4W3IZS2_CALMI\nNEK2\tA0A4W4EVX0_ELEEL\nNEK2\tA0A4W4EWD5_ELEEL\nNEK2\tA0A4W4EXW4_ELEEL\nNEK2\tA0A4W6F327_LATCA\nNEK2\tA0A4X1TGQ4_PIG\nNEK2\tA0A4X2KYQ6_VOMUR\nNEK2\tA0A4Z2IDW6_9TELE\nNEK2\tA0A5F8H6Z9_MONDO\nNEK2\tA0A5G2Q9G8_PIG\nNEK2\tA0A5G2RC59_PIG\nNEK2\tA0A5G3HND6_XENTR\nNEK2\tA0A5G3HNM1_XENTR\nNEK2\tD3BUT4_POLPP\nNEK2\tE2RSC1_CANLF\nNEK2\tF1NI69_CHICK\nNEK2\tF1S2U5_PIG\nNEK2\tF4PS00_CAVFA\nNEK2\tF6QUK8_XENTR\nNEK2\tF6U4U2_HUMAN\nNEK2\tF6VX92_HORSE\nNEK2\tF6YG37_MONDO\nNEK2\tF7CCX6_ORNAN\nNEK2\tF7HLJ2_CALJA\nNEK2\tF7IBD3_CALJA\nNEK2\tG1KSG7_ANOCA\nNEK2\tG1L2Z6_AILME\nNEK2\tG1NBW4_MELGA\nNEK2\tG1PHW7_MYOLU\nNEK2\tG1QQG6_NOMLE\nNEK2\tG1SXR8_RABIT\nNEK2\tG3TEI4_LOXAF\nNEK2\tG3V6L2_RAT\nNEK2\tG3WS04_SARHA\nNEK2\tH0VIS5_CAVPO\nNEK2\tH0WFP7_OTOGA\nNEK2\tH0YY79_TAEGU\nNEK2\tH2M9X0_ORYLA\nNEK2\tH2N3S6_PONAB\nNEK2\tH2Q134_PANTR\nNEK2\tH2VA53_TAKRU\nNEK2\tH3ASR1_LATCH\nNEK2\tI3JFH0_ORENI\nNEK2\tI3MMB3_ICTTR\nNEK2\tK7FK82_PELSI\nNEK2\tM3WQM5_FELCA\nNEK2\tM3YWN6_MUSPF\nNEK2\tM5BPH4_THACB\nNEK2\tNEK2_ARATH\nNEK2\tNEK2_DICDI\nNEK2\tNEK2_HUMAN\nNEK2\tNEK2_MOUSE\nNEK2\tNEK2_ORYSI\nNEK2\tNEK2_ORYSJ\nNEK2\tQ28EE1_XENTR\nNEK2\tQ28HV4_XENTR\nNEK2\tQ2KIQ0_BOVIN\nNEK2\tQ3TF15_MOUSE\nNEK2\tQ6DII1_XENTR\nNEK2\tQ6NY12_XENTR\nNEK2\tQ6PFG1_MOUSE\nNEK2\tQ7ZUN2_DANRE\nNEK2\tQ7ZYE3_XENLA\nNEK2\tQ8BP64_MOUSE\nNEK2\tQ91XQ1_RAT\nNEK2\tQ91Z18_MOUSE\nNEK2\tQ921N9_MOUSE\nNEK2\tQ9W3N8_DROME\nNEK2\tQ9W622_XENLA\nNEK2\tQ9W623_XENLA\nNEK2\tR4GEP7_DANRE\nNEK2\tT2MES7_HYDVU\nNEK2\tU3ITS2_ANAPP\nNEK2\tU3KDC8_FICAL\nNEK2\tU6DUZ8_NEOVI\nNEK2\tV8P266_OPHHA\nNEK2\tV8P725_OPHHA\nNEK2\tV9KT24_CALMI\nNEK2\tV9L1L0_CALMI\nNEK2\tW5PPB7_SHEEP\nNEK2\tW5UJ87_ICTPU\n','From\tTo\nWEE1\tA0A034VEX8_BACDO\nWEE1\tA0A096MRA8_PAPAN\nWEE1\tA0A096NGC9_PAPAN\nWEE1\tA0A0C5CIB7_9MUSC\nWEE1\tA0A0C9R5U9_9HYME\nWEE1\tA0A0D9QWX0_CHLSB\nWEE1\tA0A0F8DIX1_CERFI\nWEE1\tA0A0K6GA26_9AGAM\nWEE1\tA0A0S7I996_9TELE\nWEE1\tA0A0S7J320_9TELE\nWEE1\tA0A141DX32_9CARN\nWEE1\tA0A141DX33_AILFU\nWEE1\tA0A141DX34_MEPME\nWEE1\tA0A141DX35_ARCPU\nWEE1\tA0A151MCB5_ALLMI\nWEE1\tA0A1A8DQI1_9TELE\nWEE1\tA0A1A8DRT7_9TELE\nWEE1\tA0A1A8FC96_9TELE\nWEE1\tA0A1A8IS80_NOTKU\nWEE1\tA0A1A8K4B8_NOTKU\nWEE1\tA0A1A8M7G6_9TELE\nWEE1\tA0A1A8Q5D8_9TELE\nWEE1\tA0A1A8S3A5_9TELE\nWEE1\tA0A1A8U3I7_NOTFU\nWEE1\tA0A1C7LTP4_GRIFR\nWEE1\tA0A1C7MZQ0_9FUNG\nWEE1\tA0A1J6ITA5_NICAT\nWEE1\tA0A1S3A8V4_ERIEU\nWEE1\tA0A1S3G781_DIPOR\nWEE1\tA0A1S3KNE1_SALSA\nWEE1\tA0A1S3KVG2_SALSA\nWEE1\tA0A1U7QYH7_MESAU\nWEE1\tA0A1U7RGQ9_ALLSI\nWEE1\tA0A1V4JNW9_PATFA\nWEE1\tA0A1Z1RF25_ORYSA\nWEE1\tA0A218UQA5_9PASE\nWEE1\tA0A251UG37_HELAN\nWEE1\tA0A2B4SG00_STYPI\nWEE1\tA0A2C5WUR9_9PEZI\nWEE1\tA0A2H8TGV6_9HEMI\nWEE1\tA0A2H8TI83_9HEMI\nWEE1\tA0A2I0B6R0_9ASPA\nWEE1\tA0A2I0MI67_COLLI\nWEE1\tA0A2I0VLK2_9ASPA\nWEE1\tA0A2I3HLW5_NOMLE\nWEE1\tA0A2I3RFD4_PANTR\nWEE1\tA0A2I4CLF9_9TELE\nWEE1\tA0A2J8ABC0_9CHLO\nWEE1\tA0A2K5EYC9_AOTNA\nWEE1\tA0A2K5JAH6_COLAP\nWEE1\tA0A2K5M4P6_CERAT\nWEE1\tA0A2K5M4R4_CERAT\nWEE1\tA0A2K5R4H6_CEBCA\nWEE1\tA0A2K5WGS2_MACFA\nWEE1\tA0A2K5XL40_MANLE\nWEE1\tA0A2K6DHG8_MACNE\nWEE1\tA0A2K6DHI2_MACNE\nWEE1\tA0A2K6FTJ3_PROCO\nWEE1\tA0A2K6MP78_RHIBE\nWEE1\tA0A2K6MP81_RHIBE\nWEE1\tA0A2K6PJF6_RHIRO\nWEE1\tA0A2K6UP92_SAIBB\nWEE1\tA0A2N6NWJ4_BEABA\nWEE1\tA0A2R8QB38_DANRE\nWEE1\tA0A2S2N8C5_SCHGA\nWEE1\tA0A2S2PNQ1_SCHGA\nWEE1\tA0A2S2PXX5_9HEMI\nWEE1\tA0A2S2QK51_9HEMI\nWEE1\tA0A2U3VID9_ODORO\nWEE1\tA0A2U3XAV5_LEPWE\nWEE1\tA0A2U3Z9Y7_ODORO\nWEE1\tA0A2U4CF00_TURTR\nWEE1\tA0A2U4CF06_TURTR\nWEE1\tA0A2Y9F794_PHYMC\nWEE1\tA0A2Y9F8V1_PHYMC\nWEE1\tA0A2Y9GW01_NEOSC\nWEE1\tA0A2Y9LRX4_DELLE\nWEE1\tA0A2Y9LWP1_DELLE\nWEE1\tA0A340YEW6_LIPVE\nWEE1\tA0A341CE60_NEOAA\nWEE1\tA0A341CEE8_NEOAA\nWEE1\tA0A369K0I7_HYPMA\nWEE1\tA0A371ESJ8_MUCPR\nWEE1\tA0A371I8R3_MUCPR\nWEE1\tA0A384BFN1_BALAS\nWEE1\tA0A384BHE1_URSMA\nWEE1\tA0A384BIM8_URSMA\nWEE1\tA0A3B4WCU5_SERLL\nWEE1\tA0A3B4WE49_SERLL\nWEE1\tA0A3G2S2R8_9BASI\nWEE1\tA0A3Q0CZW9_MESAU\nWEE1\tA0A3Q0DTV2_TARSY\nWEE1\tA0A3Q0FXY0_ALLSI\nWEE1\tA0A3Q7R6P1_CALUR\nWEE1\tA0A3Q7T3Z5_VULVU\nWEE1\tA0A3Q7TEX2_VULVU\nWEE1\tA0A3Q7TS10_VULVU\nWEE1\tA0A3Q7U0W7_URSAR\nWEE1\tA0A3Q7UNZ8_URSAR\nWEE1\tA0A452EYW3_CAPHI\nWEE1\tA0A452HPA6_9SAUR\nWEE1\tA0A452QSE1_URSAM\nWEE1\tA0A452U9G7_URSMA\nWEE1\tA0A452U9H2_URSMA\nWEE1\tA0A452U9K5_URSMA\nWEE1\tA0A452U9N3_URSMA\nWEE1\tA0A452U9W8_URSMA\nWEE1\tA0A484G6X9_COLOR\nWEE1\tA0A493TPU4_ANAPP\nWEE1\tA0A493TSA5_ANAPP\nWEE1\tA0A4C1SM47_9NEOP\nWEE1\tA0A4C1YIV5_9NEOP\nWEE1\tA0A4R8PP31_9PEZI\nWEE1\tA0A4R8RF41_COLTR\nWEE1\tA0A4R8TT50_9PEZI\nWEE1\tA0A4U6XCW4_9PEZI\nWEE1\tA0A4W2G425_BOBOX\nWEE1\tA0A4W2HZR0_BOBOX\nWEE1\tA0A4W6BWD1_LATCA\nWEE1\tA0A4W6C4I2_LATCA\nWEE1\tA0A4W6C4I7_LATCA\nWEE1\tA0A4X1TU94_PIG\nWEE1\tA0A4X2K727_VOMUR\nWEE1\tA0A4Z2ED24_9TELE\nWEE1\tA0A4Z2IHS7_9TELE\nWEE1\tA0A559MEL6_9HELO\nWEE1\tA0A5B7G8W7_PORTR\nWEE1\tA0A5B9BLA4_9MUSC\nWEE1\tA0A5B9BLC7_9MUSC\nWEE1\tA0A5B9BLN6_9MUSC\nWEE1\tA0A5B9BMB3_9MUSC\nWEE1\tA0A5B9BMM5_9MUSC\nWEE1\tA0A5B9BP71_9MUSC\nWEE1\tA0A5F4C4M3_CANLF\nWEE1\tA0A5F4CJT7_CANLF\nWEE1\tA0A5F4CRP6_CANLF\nWEE1\tA0A5F5PJ71_HORSE\nWEE1\tA0A5F5XT58_FELCA\nWEE1\tA0A5F5XVQ7_FELCA\nWEE1\tA0A5F5XVZ1_FELCA\nWEE1\tA0A5F5XYF5_FELCA\nWEE1\tA0A5F8GXZ4_MONDO\nWEE1\tA7E332_BOVIN\nWEE1\tA7MBC3_BOVIN\nWEE1\tA8JFV0_CHLRE\nWEE1\tA9VAS1_MONBE\nWEE1\tB1H303_XENTR\nWEE1\tB6GV50_TOBAC\nWEE1\tC0HAG4_SALSA\nWEE1\tC1N9T4_MICPC\nWEE1\tD2H3H7_AILME\nWEE1\tD8UE72_VOLCA\nWEE1\tE2RIU0_CANLF\nWEE1\tE6ZZ68_SPORE\nWEE1\tE9PQ51_HUMAN\nWEE1\tE9PRI3_HUMAN\nWEE1\tE9PRU3_HUMAN\nWEE1\tF1S6U8_PIG\nWEE1\tF6TY09_HORSE\nWEE1\tF6XVD1_ORNAN\nWEE1\tF7DQS9_MACMU\nWEE1\tF7EQW9_CALJA\nWEE1\tF7G957_MONDO\nWEE1\tG1KIQ9_ANOCA\nWEE1\tG1N3A8_MELGA\nWEE1\tG1QFG1_MYOLU\nWEE1\tG1S740_NOMLE\nWEE1\tG1T4N7_RABIT\nWEE1\tG3T1M8_LOXAF\nWEE1\tG3TRQ4_LOXAF\nWEE1\tG3UQU2_MELGA\nWEE1\tG3X2R7_SARHA\nWEE1\tH0V042_CAVPO\nWEE1\tH0X500_OTOGA\nWEE1\tH0YEJ2_HUMAN\nWEE1\tH0ZCT5_TAEGU\nWEE1\tH2KVT2_CLOSI\nWEE1\tH2L6C5_ORYLA\nWEE1\tH2NE46_PONAB\nWEE1\tH2Q362_PANTR\nWEE1\tH2THT0_TAKRU\nWEE1\tH2ZSE0_LATCH\nWEE1\tH9FC18_MACMU\nWEE1\tI3KEI4_ORENI\nWEE1\tI3ME01_ICTTR\nWEE1\tK7FQS4_PELSI\nWEE1\tK8YTV1_NANGC\nWEE1\tM3X956_FELCA\nWEE1\tM3YN03_MUSPF\nWEE1\tM4SL49_9BILA\nWEE1\tM9PB57_DROME\nWEE1\tO46149_PLADU\nWEE1\tQ1JPW8_DANRE\nWEE1\tQ1LYE1_DANRE\nWEE1\tQ2P9R8_SOLLC\nWEE1\tQ2WEC0_TOBAC\nWEE1\tQ2WEC1_TOBAC\nWEE1\tQ2WEC2_TOBAC\nWEE1\tQ2WEC3_TOBAC\nWEE1\tQ52NW9_USTMD\nWEE1\tQ5NJ45_TOBAC\nWEE1\tQ5SCB3_OSTTA\nWEE1\tQ5TJC0_SOLLC\nWEE1\tQ6DRI0_DANRE\nWEE1\tQ6NSL2_HUMAN\nWEE1\tQ6PEG5_DANRE\nWEE1\tQ78DI9_CARLG\nWEE1\tQ86V29_HUMAN\nWEE1\tQ8LKQ0_ORYSA\nWEE1\tQ90YC6_CARAU\nWEE1\tQ95JK5_MACFA\nWEE1\tQ9P860_YARLL\nWEE1\tQ9SPZ8_MAIZE\nWEE1\tR0MB31_NOSB1\nWEE1\tR0MPZ5_NOSB1\nWEE1\tR4GJW5_CHICK\nWEE1\tR4X5L6_PYRCO\nWEE1\tT2MFV7_HYDVU\nWEE1\tU3E3H1_CALJA\nWEE1\tU3EXU2_CALJA\nWEE1\tU3F9E3_CALJA\nWEE1\tU3JE23_FICAL\nWEE1\tU6DRV8_NEOVI\nWEE1\tW5PS12_SHEEP\nWEE1\tW5UL74_ICTPU\nWEE1\tW8BEX8_CERCA\nWEE1\tWEE1_ARATH\nWEE1\tWEE1_DROME\nWEE1\tWEE1_HUMAN\nWEE1\tWEE1_MOUSE\nWEE1\tWEE1_ORYSJ\nWEE1\tWEE1_RAT\nWEE1\tWEE1_SCHPO\nWEE1\tWEE2B_XENLA\n','From\tTo\nERN1\tA0A061QX37_9CHLO\nERN1\tA0A061S2H4_9CHLO\nERN1\tA0A096NTU6_PAPAN\nERN1\tA0A0A1X756_ZEUCU\nERN1\tA0A0D9QVG4_CHLSB\nERN1\tA0A0F8B7A3_CERFI\nERN1\tA0A0G2K2H4_RAT\nERN1\tA0A0P6JWE7_HETGA\nERN1\tA0A0S7F836_9TELE\nERN1\tA0A0S7F862_9TELE\nERN1\tA0A0S7F883_9TELE\nERN1\tA0A0S7F884_9TELE\nERN1\tA0A0S7F897_9TELE\nERN1\tA0A0S7F965_9TELE\nERN1\tA0A0S7F9J1_9TELE\nERN1\tA0A0S7FAW8_9TELE\nERN1\tA0A0S7FBY5_9TELE\nERN1\tA0A0S7FD23_9TELE\nERN1\tA0A0S7FE15_9TELE\nERN1\tA0A0S7FE17_9TELE\nERN1\tA0A0S7FFI5_9TELE\nERN1\tA0A0S7FFL0_9TELE\nERN1\tA0A0S7FGC5_9TELE\nERN1\tA0A0S7FHU3_9TELE\nERN1\tA0A0S7H0S5_9TELE\nERN1\tA0A0S7H0U8_9TELE\nERN1\tA0A0S7H1Z0_9TELE\nERN1\tA0A0S7H3P2_9TELE\nERN1\tA0A0S7H516_9TELE\nERN1\tA0A0S7LH01_9TELE\nERN1\tA0A0S7LH09_9TELE\nERN1\tA0A0S7LH16_9TELE\nERN1\tA0A0V0RH37_9BILA\nERN1\tA0A0V0RH95_9BILA\nERN1\tA0A0V0RHU8_9BILA\nERN1\tA0A0V1JTD6_TRIPS\nERN1\tA0A0V1JTE5_TRIPS\nERN1\tA0A151N2H5_ALLMI\nERN1\tA0A1A7WDJ8_9TELE\nERN1\tA0A1A8BZM7_9TELE\nERN1\tA0A1A8DXV3_9TELE\nERN1\tA0A1A8FGK7_9TELE\nERN1\tA0A1A8IHQ8_NOTKU\nERN1\tA0A1A8KE03_NOTKU\nERN1\tA0A1A8M0X3_9TELE\nERN1\tA0A1A8QPE1_9TELE\nERN1\tA0A1A8QTG5_9TELE\nERN1\tA0A1A8QWF3_9TELE\nERN1\tA0A1A8QY75_9TELE\nERN1\tA0A1A8VHV2_NOTFU\nERN1\tA0A1L8END6_XENLA\nERN1\tA0A1L8EV21_XENLA\nERN1\tA0A1S3A169_ERIEU\nERN1\tA0A1S3GXP0_DIPOR\nERN1\tA0A1U7RR33_ALLSI\nERN1\tA0A218UB56_9PASE\nERN1\tA0A218UWX0_9PASE\nERN1\tA0A250Y680_CASCN\nERN1\tA0A286XQB0_CAVPO\nERN1\tA0A287CZI9_ICTTR\nERN1\tA0A287CZT3_ICTTR\nERN1\tA0A2B4R6G7_STYPI\nERN1\tA0A2B4RSR3_STYPI\nERN1\tA0A2B4RV69_STYPI\nERN1\tA0A2B4SX49_STYPI\nERN1\tA0A2D0SXE7_ICTPU\nERN1\tA0A2I0M745_COLLI\nERN1\tA0A2I0M753_COLLI\nERN1\tA0A2I3HVQ1_NOMLE\nERN1\tA0A2I3N376_PAPAN\nERN1\tA0A2I4BCX8_9TELE\nERN1\tA0A2K5C3X4_AOTNA\nERN1\tA0A2K5C3Y1_AOTNA\nERN1\tA0A2K5C400_AOTNA\nERN1\tA0A2K5IMW3_COLAP\nERN1\tA0A2K5NET2_CERAT\nERN1\tA0A2K5NF57_CERAT\nERN1\tA0A2K5RZW4_CEBCA\nERN1\tA0A2K5S003_CEBCA\nERN1\tA0A2K5WZZ8_MACFA\nERN1\tA0A2K5X0L5_MACFA\nERN1\tA0A2K5YU22_MANLE\nERN1\tA0A2K6E565_MACNE\nERN1\tA0A2K6E566_MACNE\nERN1\tA0A2K6E572_MACNE\nERN1\tA0A2K6G9K2_PROCO\nERN1\tA0A2K6L3W6_RHIBE\nERN1\tA0A2K6L3W7_RHIBE\nERN1\tA0A2K6NJ52_RHIRO\nERN1\tA0A2K6NJ59_RHIRO\nERN1\tA0A2K6V400_SAIBB\nERN1\tA0A2K6V442_SAIBB\nERN1\tA0A2P8YZR7_BLAGE\nERN1\tA0A2R8M4S3_CALJA\nERN1\tA0A2R9AFY5_PANPA\nERN1\tA0A2R9AFZ7_PANPA\nERN1\tA0A2U3WBU0_ODORO\nERN1\tA0A2U4C310_TURTR\nERN1\tA0A2Y9G8F7_NEOSC\nERN1\tA0A2Y9PD79_DELLE\nERN1\tA0A340X9X4_LIPVE\nERN1\tA0A341CP62_NEOAA\nERN1\tA0A383ZLB2_BALAS\nERN1\tA0A384CCH7_URSMA\nERN1\tA0A3Q0CQE4_MESAU\nERN1\tA0A3Q0E301_TARSY\nERN1\tA0A3Q0FWX9_ALLSI\nERN1\tA0A3Q0FXL5_ALLSI\nERN1\tA0A3Q0FXT7_ALLSI\nERN1\tA0A3Q0G1X9_ALLSI\nERN1\tA0A3Q7SG31_VULVU\nERN1\tA0A3Q7XZC4_URSAR\nERN1\tA0A452G4R0_CAPHI\nERN1\tA0A452QTG3_URSAM\nERN1\tA0A452QTG8_URSAM\nERN1\tA0A452USW1_URSMA\nERN1\tA0A452USX7_URSMA\nERN1\tA0A452UT09_URSMA\nERN1\tA0A452UT31_URSMA\nERN1\tA0A452UT37_URSMA\nERN1\tA0A452UT51_URSMA\nERN1\tA0A452UT60_URSMA\nERN1\tA0A452UTI9_URSMA\nERN1\tA0A493TNE9_ANAPP\nERN1\tA0A4D8Z7W3_SALSN\nERN1\tA0A4D8Z8Q3_SALSN\nERN1\tA0A4D8ZF83_SALSN\nERN1\tA0A4D9A7A7_SALSN\nERN1\tA0A4U6XIR7_9PEZI\nERN1\tA0A4W2C9A0_BOBOX\nERN1\tA0A4W2CBZ9_BOBOX\nERN1\tA0A4W2HVP6_BOBOX\nERN1\tA0A4W2I0F1_BOBOX\nERN1\tA0A4W4DSS0_ELEEL\nERN1\tA0A4W4DST6_ELEEL\nERN1\tA0A4W4DSY6_ELEEL\nERN1\tA0A4W4DT05_ELEEL\nERN1\tA0A4W4DT92_ELEEL\nERN1\tA0A4W4DTT3_ELEEL\nERN1\tA0A4W4DU46_ELEEL\nERN1\tA0A4W4DU76_ELEEL\nERN1\tA0A4W4DU83_ELEEL\nERN1\tA0A4X1SZY9_PIG\nERN1\tA0A4X2KND2_VOMUR\nERN1\tA0A4X2KUE8_VOMUR\nERN1\tA0A5F8A6W0_MACMU\nERN1\tA8JR46_DROME\nERN1\tB1ARJ5_MOUSE\nERN1\tB1ATQ7_MOUSE\nERN1\tERN1_HUMAN\nERN1\tERN1_LOTJA\nERN1\tERN1_MEDTR\nERN1\tERN1_MOUSE\nERN1\tF1LSY7_RAT\nERN1\tF1MM00_BOVIN\nERN1\tF1NPU5_CHICK\nERN1\tF1P7G9_CANLF\nERN1\tF1QYL2_DANRE\nERN1\tF6WBI3_MACMU\nERN1\tF7BA63_HORSE\nERN1\tF7ENI2_ORNAN\nERN1\tF7FQL2_CALJA\nERN1\tG1L816_AILME\nERN1\tG1N759_MELGA\nERN1\tG1PVB0_MYOLU\nERN1\tG1RYK6_NOMLE\nERN1\tG3T1Y5_LOXAF\nERN1\tG3UF31_LOXAF\nERN1\tG3WME2_SARHA\nERN1\tG5ELX3_ORYLA\nERN1\tH0UU76_CAVPO\nERN1\tH0WNM6_OTOGA\nERN1\tH2LYJ6_ORYLA\nERN1\tH2NUF8_PONAB\nERN1\tH2QDP0_PANTR\nERN1\tH2RQL9_TAKRU\nERN1\tH3B8N1_LATCH\nERN1\tH9GNJ9_ANOCA\nERN1\tI3MCW6_ICTTR\nERN1\tIRE1_YEAST\nERN1\tK7BU85_PANTR\nERN1\tK7ET37_PONAB\nERN1\tK7GH11_PELSI\nERN1\tM3W0G8_FELCA\nERN1\tM3YTK1_MUSPF\nERN1\tQ38MV1_SOLLC\nERN1\tQ3TBZ5_MOUSE\nERN1\tQ504E3_DANRE\nERN1\tQ5V6B6_HALMA\nERN1\tR0MMH2_NOSB1\nERN1\tU3IJT6_ANAPP\nERN1\tU3JHZ6_FICAL\nERN1\tU6D4D5_NEOVI\nERN1\tV5GHN5_ANOGL\nERN1\tV8P3R1_OPHHA\nERN1\tW5N8F3_LEPOC\nERN1\tW5N8F6_LEPOC\nERN1\tW5PYC6_SHEEP\nERN1\tW5PYD0_SHEEP\nERN1\tW5U6S4_ICTPU\nERN1\tX1WEF9_DANRE\n','From\tTo\nCAMKK2\tA0A024RBP6_HUMAN\nCAMKK2\tA0A024RBP8_HUMAN\nCAMKK2\tA0A024RBQ0_HUMAN\nCAMKK2\tA0A024RBQ3_HUMAN\nCAMKK2\tA0A096N7S7_PAPAN\nCAMKK2\tA0A0A1WLP9_ZEUCU\nCAMKK2\tA0A0B2USY5_TOXCA\nCAMKK2\tA0A0D9S4N6_CHLSB\nCAMKK2\tA0A0G2JEC5_MOUSE\nCAMKK2\tA0A0G2JEF4_MOUSE\nCAMKK2\tA0A0G2JET2_MOUSE\nCAMKK2\tA0A0G2JFG5_MOUSE\nCAMKK2\tA0A0G2JGX0_MOUSE\nCAMKK2\tA0A0P6J7J9_HETGA\nCAMKK2\tA0A0V1EYA7_TRIPS\nCAMKK2\tA0A0V1KGL0_TRIPS\nCAMKK2\tA0A0V1KGL2_TRIPS\nCAMKK2\tA0A0V1KGP1_TRIPS\nCAMKK2\tA0A151NSI4_ALLMI\nCAMKK2\tA0A151NT19_ALLMI\nCAMKK2\tA0A1A7YPC9_9TELE\nCAMKK2\tA0A1A7ZUK0_NOTFU\nCAMKK2\tA0A1A8CXP0_9TELE\nCAMKK2\tA0A1A8FDH8_9TELE\nCAMKK2\tA0A1A8I218_NOTKU\nCAMKK2\tA0A1A8MDJ7_9TELE\nCAMKK2\tA0A1A8PDM4_9TELE\nCAMKK2\tA0A1A8SLT9_9TELE\nCAMKK2\tA0A1C7N6Z6_9FUNG\nCAMKK2\tA0A1D5PQU7_CHICK\nCAMKK2\tA0A1D5QDC1_MACMU\nCAMKK2\tA0A1D5RIA0_MACMU\nCAMKK2\tA0A1Q9CAI6_SYMMI\nCAMKK2\tA0A1Q9EYP8_SYMMI\nCAMKK2\tA0A1S2YNQ6_CICAR\nCAMKK2\tA0A1S3A4G7_ERIEU\nCAMKK2\tA0A1S3A4K0_ERIEU\nCAMKK2\tA0A1S3FPU0_DIPOR\nCAMKK2\tA0A1S3FPV3_DIPOR\nCAMKK2\tA0A1S3FQE2_DIPOR\nCAMKK2\tA0A1S3FQE8_DIPOR\nCAMKK2\tA0A1S3FRD0_DIPOR\nCAMKK2\tA0A1S3FRV3_DIPOR\nCAMKK2\tA0A1S3LF69_SALSA\nCAMKK2\tA0A1S3LF83_SALSA\nCAMKK2\tA0A1S3WJ62_ERIEU\nCAMKK2\tA0A1U7R8G2_ALLSI\nCAMKK2\tA0A1U7TDN9_TARSY\nCAMKK2\tA0A1U7TP53_TARSY\nCAMKK2\tA0A1U7TWV6_TARSY\nCAMKK2\tA0A1U7TYL4_TARSY\nCAMKK2\tA0A1U8D8H6_ALLSI\nCAMKK2\tA0A1V4JS75_PATFA\nCAMKK2\tA0A1V4JS76_PATFA\nCAMKK2\tA0A218UMC1_9PASE\nCAMKK2\tA0A250Y2Q2_CASCN\nCAMKK2\tA0A286XKN5_CAVPO\nCAMKK2\tA0A286Y209_CAVPO\nCAMKK2\tA0A286Y558_CAVPO\nCAMKK2\tA0A287A4U9_PIG\nCAMKK2\tA0A287AT01_PIG\nCAMKK2\tA0A287CTR0_ICTTR\nCAMKK2\tA0A287CVE9_ICTTR\nCAMKK2\tA0A287D6Y3_ICTTR\nCAMKK2\tA0A2B4SCX5_STYPI\nCAMKK2\tA0A2D0SYE8_ICTPU\nCAMKK2\tA0A2D0SYF2_ICTPU\nCAMKK2\tA0A2D0SYF9_ICTPU\nCAMKK2\tA0A2I0M8K2_COLLI\nCAMKK2\tA0A2I0M8K7_COLLI\nCAMKK2\tA0A2I2U0V7_FELCA\nCAMKK2\tA0A2I2UP44_FELCA\nCAMKK2\tA0A2I3GE15_NOMLE\nCAMKK2\tA0A2I3GZA0_NOMLE\nCAMKK2\tA0A2I3HEJ7_NOMLE\nCAMKK2\tA0A2I3HGX3_NOMLE\nCAMKK2\tA0A2I3HNU9_NOMLE\nCAMKK2\tA0A2I3HT71_NOMLE\nCAMKK2\tA0A2I3LPV4_PAPAN\nCAMKK2\tA0A2I3M567_PAPAN\nCAMKK2\tA0A2I3M8X0_PAPAN\nCAMKK2\tA0A2I3MA88_PAPAN\nCAMKK2\tA0A2I3RD10_PANTR\nCAMKK2\tA0A2I3SKA1_PANTR\nCAMKK2\tA0A2I3SSH6_PANTR\nCAMKK2\tA0A2J8XJS8_PONAB\nCAMKK2\tA0A2K5E4B6_AOTNA\nCAMKK2\tA0A2K5E4K3_AOTNA\nCAMKK2\tA0A2K5E4Q4_AOTNA\nCAMKK2\tA0A2K5E4R5_AOTNA\nCAMKK2\tA0A2K5E4U4_AOTNA\nCAMKK2\tA0A2K5E4V5_AOTNA\nCAMKK2\tA0A2K5HJ20_COLAP\nCAMKK2\tA0A2K5HJ54_COLAP\nCAMKK2\tA0A2K5HJ59_COLAP\nCAMKK2\tA0A2K5HJF0_COLAP\nCAMKK2\tA0A2K5HJH1_COLAP\nCAMKK2\tA0A2K5M0D8_CERAT\nCAMKK2\tA0A2K5M0E2_CERAT\nCAMKK2\tA0A2K5M0E5_CERAT\nCAMKK2\tA0A2K5M0F1_CERAT\nCAMKK2\tA0A2K5M0G5_CERAT\nCAMKK2\tA0A2K5PPI6_CEBCA\nCAMKK2\tA0A2K5PPK8_CEBCA\nCAMKK2\tA0A2K5PPM6_CEBCA\nCAMKK2\tA0A2K5PPN3_CEBCA\nCAMKK2\tA0A2K5PPP5_CEBCA\nCAMKK2\tA0A2K5PPQ2_CEBCA\nCAMKK2\tA0A2K6AKP2_MANLE\nCAMKK2\tA0A2K6AKP6_MANLE\nCAMKK2\tA0A2K6AKQ6_MANLE\nCAMKK2\tA0A2K6AKR1_MANLE\nCAMKK2\tA0A2K6AKR2_MANLE\nCAMKK2\tA0A2K6AKX5_MANLE\nCAMKK2\tA0A2K6BST2_MACNE\nCAMKK2\tA0A2K6BST9_MACNE\nCAMKK2\tA0A2K6BSU6_MACNE\nCAMKK2\tA0A2K6BSU9_MACNE\nCAMKK2\tA0A2K6BSX7_MACNE\nCAMKK2\tA0A2K6EKU0_PROCO\nCAMKK2\tA0A2K6EKU5_PROCO\nCAMKK2\tA0A2K6EKV0_PROCO\nCAMKK2\tA0A2K6EKV1_PROCO\nCAMKK2\tA0A2K6EKV4_PROCO\nCAMKK2\tA0A2K6N2Y1_RHIBE\nCAMKK2\tA0A2K6N2Z6_RHIBE\nCAMKK2\tA0A2K6N310_RHIBE\nCAMKK2\tA0A2K6N314_RHIBE\nCAMKK2\tA0A2K6N316_RHIBE\nCAMKK2\tA0A2K6N368_RHIBE\nCAMKK2\tA0A2K6RS75_RHIRO\nCAMKK2\tA0A2K6RS90_RHIRO\nCAMKK2\tA0A2K6RSA0_RHIRO\nCAMKK2\tA0A2K6RSA4_RHIRO\nCAMKK2\tA0A2K6RSC0_RHIRO\nCAMKK2\tA0A2K6RSC5_RHIRO\nCAMKK2\tA0A2K6RSD0_RHIRO\nCAMKK2\tA0A2K6V8G0_SAIBB\nCAMKK2\tA0A2K6V8G5_SAIBB\nCAMKK2\tA0A2K6V8H2_SAIBB\nCAMKK2\tA0A2K6V8H7_SAIBB\nCAMKK2\tA0A2K6V8I1_SAIBB\nCAMKK2\tA0A2K6V8I2_SAIBB\nCAMKK2\tA0A2N6NRU9_BEABA\nCAMKK2\tA0A2R9B8E6_PANPA\nCAMKK2\tA0A2R9B9U5_PANPA\nCAMKK2\tA0A2R9B9V7_PANPA\nCAMKK2\tA0A2R9BFS2_PANPA\nCAMKK2\tA0A2S2N887_SCHGA\nCAMKK2\tA0A2S2R0R7_9HEMI\nCAMKK2\tA0A2U3VLP4_ODORO\nCAMKK2\tA0A2U3VLP8_ODORO\nCAMKK2\tA0A2U3VLQ1_ODORO\nCAMKK2\tA0A2U3XZX4_LEPWE\nCAMKK2\tA0A2U3XZY7_LEPWE\nCAMKK2\tA0A2U3Y062_LEPWE\nCAMKK2\tA0A2U4CJF7_TURTR\nCAMKK2\tA0A2Y9FHY0_PHYMC\nCAMKK2\tA0A2Y9HR11_NEOSC\nCAMKK2\tA0A2Y9Q341_DELLE\nCAMKK2\tA0A2Y9Q7F7_DELLE\nCAMKK2\tA0A2Y9Q7G3_DELLE\nCAMKK2\tA0A2Y9Q7J2_DELLE\nCAMKK2\tA0A2Y9Q7J7_DELLE\nCAMKK2\tA0A2Y9SHA7_PHYMC\nCAMKK2\tA0A2Y9SM03_PHYMC\nCAMKK2\tA0A337SI08_FELCA\nCAMKK2\tA0A340WGQ4_LIPVE\nCAMKK2\tA0A340WJ78_LIPVE\nCAMKK2\tA0A340WMA9_LIPVE\nCAMKK2\tA0A340WNY6_LIPVE\nCAMKK2\tA0A340WP60_LIPVE\nCAMKK2\tA0A341CHK7_NEOAA\nCAMKK2\tA0A341CI21_NEOAA\nCAMKK2\tA0A341CI89_NEOAA\nCAMKK2\tA0A341CID3_NEOAA\nCAMKK2\tA0A346FR90_TEGGR\nCAMKK2\tA0A384AQG0_BALAS\nCAMKK2\tA0A384AR19_BALAS\nCAMKK2\tA0A384BX86_URSMA\nCAMKK2\tA0A3M9Y1R4_9PEZI\nCAMKK2\tA0A3Q0D3H1_MESAU\nCAMKK2\tA0A3Q0D730_MESAU\nCAMKK2\tA0A3Q0D831_MESAU\nCAMKK2\tA0A3Q0D8J8_MESAU\nCAMKK2\tA0A3Q0DXZ4_TARSY\nCAMKK2\tA0A3Q0E4R8_TARSY\nCAMKK2\tA0A3Q1LXX7_BOVIN\nCAMKK2\tA0A3Q1M1W0_BOVIN\nCAMKK2\tA0A3Q1MHV2_BOVIN\nCAMKK2\tA0A3Q2GV42_HORSE\nCAMKK2\tA0A3Q2HI52_HORSE\nCAMKK2\tA0A3Q2ICK3_HORSE\nCAMKK2\tA0A3Q7QG42_CALUR\nCAMKK2\tA0A3Q7QG48_CALUR\nCAMKK2\tA0A3Q7QID9_CALUR\nCAMKK2\tA0A3Q7QTU3_CALUR\nCAMKK2\tA0A3Q7QTU8_CALUR\nCAMKK2\tA0A3Q7RXQ5_CALUR\nCAMKK2\tA0A3Q7TBX0_VULVU\nCAMKK2\tA0A3Q7TY34_VULVU\nCAMKK2\tA0A3Q7U9T2_VULVU\nCAMKK2\tA0A3Q7V4W4_VULVU\nCAMKK2\tA0A3Q7WC57_URSAR\nCAMKK2\tA0A3Q7WC60_URSAR\nCAMKK2\tA0A3Q7X084_URSAR\nCAMKK2\tA0A3Q7X087_URSAR\nCAMKK2\tA0A3Q7X8H0_URSAR\nCAMKK2\tA0A3Q7XUH5_URSAR\nCAMKK2\tA0A3Q7XUH9_URSAR\nCAMKK2\tA0A3Q7Y2E2_URSAR\nCAMKK2\tA0A3T0TUP0_TACFU\nCAMKK2\tA0A452CMC4_BALAS\nCAMKK2\tA0A452FL54_CAPHI\nCAMKK2\tA0A452TJS3_URSMA\nCAMKK2\tA0A452TJT1_URSMA\nCAMKK2\tA0A452TK48_URSMA\nCAMKK2\tA0A452TKD6_URSMA\nCAMKK2\tA0A452TKF7_URSMA\nCAMKK2\tA0A455AJN2_PHYMC\nCAMKK2\tA0A455AJU0_PHYMC\nCAMKK2\tA0A4U6XP46_9PEZI\nCAMKK2\tA0A4W2C9B6_BOBOX\nCAMKK2\tA0A4W2C9C1_BOBOX\nCAMKK2\tA0A4W2CCV4_BOBOX\nCAMKK2\tA0A4W2CT60_BOBOX\nCAMKK2\tA0A4W2GG26_BOBOX\nCAMKK2\tA0A4W2GK33_BOBOX\nCAMKK2\tA0A4W2I452_BOBOX\nCAMKK2\tA0A4W3HBN2_CALMI\nCAMKK2\tA0A4W3HD20_CALMI\nCAMKK2\tA0A4X1U2G6_PIG\nCAMKK2\tA0A4X1U2J0_PIG\nCAMKK2\tA0A4X1U7P3_PIG\nCAMKK2\tA0A4X2K195_VOMUR\nCAMKK2\tA0A4X2K1B8_VOMUR\nCAMKK2\tA0A4X2K1D6_VOMUR\nCAMKK2\tA0A4X2K6J5_VOMUR\nCAMKK2\tA0A4X2K6L1_VOMUR\nCAMKK2\tA0A4Y2AUJ8_ARAVE\nCAMKK2\tA0A4Z2HY14_9TELE\nCAMKK2\tA0A4Z2HYF8_9TELE\nCAMKK2\tA0A5B0PMU4_PUCGR\nCAMKK2\tA0A5F4CR55_CANLF\nCAMKK2\tA0A5F4DCT7_CANLF\nCAMKK2\tA0A5F5PI62_HORSE\nCAMKK2\tA0A5F5XRI8_FELCA\nCAMKK2\tA0A5F7Z825_MACMU\nCAMKK2\tA0A5F7ZX95_MACMU\nCAMKK2\tA0A5F9C7G4_RABIT\nCAMKK2\tA0A5F9CBC6_RABIT\nCAMKK2\tA0A5F9DBX7_RABIT\nCAMKK2\tA0A5G3I6L9_XENTR\nCAMKK2\tA0A5G3I6P1_XENTR\nCAMKK2\tA0A5G3I6T2_XENTR\nCAMKK2\tA0A5G3I7A1_XENTR\nCAMKK2\tA0A5G3I7E8_XENTR\nCAMKK2\tA0A5G3I8B5_XENTR\nCAMKK2\tA0A5G3LCF1_XENTR\nCAMKK2\tA4GGG0_RHIFE\nCAMKK2\tA4GGG1_RHIFE\nCAMKK2\tA4GGG2_RHIFE\nCAMKK2\tA4GGG3_RHIFE\nCAMKK2\tE2R3S0_CANLF\nCAMKK2\tF1LPT4_RAT\nCAMKK2\tF1RNP6_PIG\nCAMKK2\tF5GZ00_HUMAN\nCAMKK2\tF5H360_HUMAN\nCAMKK2\tF5H4I7_HUMAN\nCAMKK2\tF6R1V5_CALJA\nCAMKK2\tF6RN62_HORSE\nCAMKK2\tF6TEG6_ORNAN\nCAMKK2\tF6X5L5_XENTR\nCAMKK2\tF7GFG0_MACMU\nCAMKK2\tF7GFG2_MACMU\nCAMKK2\tF7GFG6_MACMU\nCAMKK2\tF7I4F8_CALJA\nCAMKK2\tF7I4G2_CALJA\nCAMKK2\tF7I4J2_CALJA\nCAMKK2\tG1M6L7_AILME\nCAMKK2\tG1MYG6_MELGA\nCAMKK2\tG1PV77_MYOLU\nCAMKK2\tG1U863_RABIT\nCAMKK2\tG3TIW9_LOXAF\nCAMKK2\tG3U4K9_LOXAF\nCAMKK2\tG3WUT8_SARHA\nCAMKK2\tH0VA11_CAVPO\nCAMKK2\tH0XBY7_OTOGA\nCAMKK2\tH3BGR3_LATCH\nCAMKK2\tH9FNV8_MACMU\nCAMKK2\tH9FNV9_MACMU\nCAMKK2\tI3MEC6_ICTTR\nCAMKK2\tK7CIA2_PANTR\nCAMKK2\tK7DNN3_PANTR\nCAMKK2\tKKCC2_HUMAN\nCAMKK2\tKKCC2_MOUSE\nCAMKK2\tKKCC2_RAT\nCAMKK2\tM3WGZ4_FELCA\nCAMKK2\tQ148H3_BOVIN\nCAMKK2\tQ3TV45_MOUSE\nCAMKK2\tQ8IZW1_HUMAN\nCAMKK2\tT2M740_HYDVU\nCAMKK2\tU3C9F1_CALJA\nCAMKK2\tU3K3G4_FICAL\nCAMKK2\tW5Q048_SHEEP\nCAMKK2\tW5Q050_SHEEP\n','From\tTo\nSRPK2\tA0A024R704_HUMAN\nSRPK2\tA0A034V1Y9_BACDO\nSRPK2\tA0A034V443_BACDO\nSRPK2\tA0A034V567_BACDO\nSRPK2\tA0A034V571_BACDO\nSRPK2\tA0A034V6B6_BACDO\nSRPK2\tA0A034W0B2_BACDO\nSRPK2\tA0A060RZH6_PLARE\nSRPK2\tA0A096MJB4_RAT\nSRPK2\tA0A096NGT2_PAPAN\nSRPK2\tA0A096NGT3_PAPAN\nSRPK2\tA0A0A1XAT3_ZEUCU\nSRPK2\tA0A0D9RGK3_CHLSB\nSRPK2\tA0A0G2JDT6_MOUSE\nSRPK2\tA0A0G2JDY2_MOUSE\nSRPK2\tA0A0G2JEI1_MOUSE\nSRPK2\tA0A0G2JG74_MOUSE\nSRPK2\tA0A0G2JX62_RAT\nSRPK2\tA0A0K8UIY9_BACLA\nSRPK2\tA0A0R4J124_MOUSE\nSRPK2\tA0A0S7EFX9_9TELE\nSRPK2\tA0A0S7EI79_9TELE\nSRPK2\tA0A0S7GHX5_9TELE\nSRPK2\tA0A0S7GI45_9TELE\nSRPK2\tA0A0S7GIF8_9TELE\nSRPK2\tA0A0S7GKE2_9TELE\nSRPK2\tA0A0S7GLN6_9TELE\nSRPK2\tA0A0S7GP48_9TELE\nSRPK2\tA0A0S7K114_9TELE\nSRPK2\tA0A0S7MJV6_9TELE\nSRPK2\tA0A113SHK8_PLABE\nSRPK2\tA0A151N8E6_ALLMI\nSRPK2\tA0A151N9B9_ALLMI\nSRPK2\tA0A1A8DLU8_9TELE\nSRPK2\tA0A1A8GBQ6_9TELE\nSRPK2\tA0A1A8GTT9_9TELE\nSRPK2\tA0A1A8HT84_NOTKU\nSRPK2\tA0A1A8K9Z4_NOTKU\nSRPK2\tA0A1A8MR80_9TELE\nSRPK2\tA0A1A8R3J6_9TELE\nSRPK2\tA0A1A8RZL0_9TELE\nSRPK2\tA0A1A8SK50_9TELE\nSRPK2\tA0A1A8UQX5_NOTFU\nSRPK2\tA0A1C3KWC0_9APIC\nSRPK2\tA0A1C3L1R3_PLAMA\nSRPK2\tA0A1C6XIQ6_PLACH\nSRPK2\tA0A1C6XTR3_PLACH\nSRPK2\tA0A1D3LGM2_PLACH\nSRPK2\tA0A1D3LL56_PLACH\nSRPK2\tA0A1D3SQG9_PLAMA\nSRPK2\tA0A1D3TLM2_9APIC\nSRPK2\tA0A1D5PSU1_CHICK\nSRPK2\tA0A1D5Q1L9_MACMU\nSRPK2\tA0A1D5Q4Z0_MACMU\nSRPK2\tA0A1D5QEA9_MACMU\nSRPK2\tA0A1D5QN30_MACMU\nSRPK2\tA0A1D5QSB4_MACMU\nSRPK2\tA0A1D5RAP6_MACMU\nSRPK2\tA0A1J1GUA7_PLAGA\nSRPK2\tA0A1J1HAF8_PLARL\nSRPK2\tA0A1S2ZZK6_ERIEU\nSRPK2\tA0A1S3GN98_DIPOR\nSRPK2\tA0A1U7QWZ9_MESAU\nSRPK2\tA0A1U7UVQ0_TARSY\nSRPK2\tA0A1U8CLF2_MESAU\nSRPK2\tA0A1V4JZX9_PATFA\nSRPK2\tA0A1V4JZY4_PATFA\nSRPK2\tA0A1V4K056_PATFA\nSRPK2\tA0A1Y3DMZ3_PLAKN\nSRPK2\tA0A250Y463_CASCN\nSRPK2\tA0A287A5X3_PIG\nSRPK2\tA0A287AKZ4_PIG\nSRPK2\tA0A287AP93_PIG\nSRPK2\tA0A287B6X0_PIG\nSRPK2\tA0A287B7R0_PIG\nSRPK2\tA0A2C5WWF5_9PEZI\nSRPK2\tA0A2D0SCH6_ICTPU\nSRPK2\tA0A2D0SCH7_ICTPU\nSRPK2\tA0A2D0SCI0_ICTPU\nSRPK2\tA0A2D0SD85_ICTPU\nSRPK2\tA0A2D0SDV8_ICTPU\nSRPK2\tA0A2I0MHF1_COLLI\nSRPK2\tA0A2I2UJJ4_FELCA\nSRPK2\tA0A2I3GHN1_NOMLE\nSRPK2\tA0A2I3H945_NOMLE\nSRPK2\tA0A2I3MDQ8_PAPAN\nSRPK2\tA0A2I3MNA0_PAPAN\nSRPK2\tA0A2I3N9S8_PAPAN\nSRPK2\tA0A2I3RI25_PANTR\nSRPK2\tA0A2I3RYB1_PANTR\nSRPK2\tA0A2I3RZH1_PANTR\nSRPK2\tA0A2I3S440_PANTR\nSRPK2\tA0A2I3SE34_PANTR\nSRPK2\tA0A2I3SEK1_PANTR\nSRPK2\tA0A2K5D7W7_AOTNA\nSRPK2\tA0A2K5D7X2_AOTNA\nSRPK2\tA0A2K5D7Y1_AOTNA\nSRPK2\tA0A2K5H8Z0_COLAP\nSRPK2\tA0A2K5H8Z3_COLAP\nSRPK2\tA0A2K5H927_COLAP\nSRPK2\tA0A2K5H931_COLAP\nSRPK2\tA0A2K5H935_COLAP\nSRPK2\tA0A2K5N8M9_CERAT\nSRPK2\tA0A2K5N8N0_CERAT\nSRPK2\tA0A2K5N8Q7_CERAT\nSRPK2\tA0A2K5N8W7_CERAT\nSRPK2\tA0A2K5N938_CERAT\nSRPK2\tA0A2K5QKT9_CEBCA\nSRPK2\tA0A2K5QL13_CEBCA\nSRPK2\tA0A2K5QL25_CEBCA\nSRPK2\tA0A2K5QL41_CEBCA\nSRPK2\tA0A2K5QL50_CEBCA\nSRPK2\tA0A2K5Z4K5_MANLE\nSRPK2\tA0A2K5Z4L0_MANLE\nSRPK2\tA0A2K5Z4M4_MANLE\nSRPK2\tA0A2K5Z4N8_MANLE\nSRPK2\tA0A2K6CCI4_MACNE\nSRPK2\tA0A2K6CCJ4_MACNE\nSRPK2\tA0A2K6CCL1_MACNE\nSRPK2\tA0A2K6CCL4_MACNE\nSRPK2\tA0A2K6CCM5_MACNE\nSRPK2\tA0A2K6GGW7_PROCO\nSRPK2\tA0A2K6GGX1_PROCO\nSRPK2\tA0A2K6GGX4_PROCO\nSRPK2\tA0A2K6GGY5_PROCO\nSRPK2\tA0A2K6KGN8_RHIBE\nSRPK2\tA0A2K6KGQ6_RHIBE\nSRPK2\tA0A2K6KGQ7_RHIBE\nSRPK2\tA0A2K6KGQ9_RHIBE\nSRPK2\tA0A2K6KGR1_RHIBE\nSRPK2\tA0A2K6KGS8_RHIBE\nSRPK2\tA0A2K6RIF3_RHIRO\nSRPK2\tA0A2K6RIF7_RHIRO\nSRPK2\tA0A2K6RIF9_RHIRO\nSRPK2\tA0A2K6RIG4_RHIRO\nSRPK2\tA0A2K6RIG8_RHIRO\nSRPK2\tA0A2K6RIH1_RHIRO\nSRPK2\tA0A2K6S2Q4_SAIBB\nSRPK2\tA0A2K6S2R7_SAIBB\nSRPK2\tA0A2K6S2S8_SAIBB\nSRPK2\tA0A2K6S2T8_SAIBB\nSRPK2\tA0A2K6S2U0_SAIBB\nSRPK2\tA0A2N6NP86_BEABA\nSRPK2\tA0A2R8M8C9_CALJA\nSRPK2\tA0A2R8Z8U8_PANPA\nSRPK2\tA0A2R8Z916_PANPA\nSRPK2\tA0A2R8ZFF8_PANPA\nSRPK2\tA0A2U3W3D7_ODORO\nSRPK2\tA0A2U3W3E4_ODORO\nSRPK2\tA0A2U4BSC6_TURTR\nSRPK2\tA0A2U4BSD3_TURTR\nSRPK2\tA0A2U4BSD8_TURTR\nSRPK2\tA0A2U4BSE4_TURTR\nSRPK2\tA0A2U4BSE5_TURTR\nSRPK2\tA0A2U4BSE9_TURTR\nSRPK2\tA0A2U4BSJ5_TURTR\nSRPK2\tA0A2U4BSK1_TURTR\nSRPK2\tA0A2U4BSP5_TURTR\nSRPK2\tA0A2Y9GHI8_NEOSC\nSRPK2\tA0A2Y9GHJ4_NEOSC\nSRPK2\tA0A2Y9GHJ9_NEOSC\nSRPK2\tA0A2Y9GHN7_NEOSC\nSRPK2\tA0A2Y9GKV7_NEOSC\nSRPK2\tA0A2Y9GMM5_NEOSC\nSRPK2\tA0A2Y9LK85_DELLE\nSRPK2\tA0A2Y9LK92_DELLE\nSRPK2\tA0A2Y9LKS4_DELLE\nSRPK2\tA0A2Y9LKS9_DELLE\nSRPK2\tA0A2Y9LMX2_DELLE\nSRPK2\tA0A2Y9LMX6_DELLE\nSRPK2\tA0A2Y9LSK4_DELLE\nSRPK2\tA0A2Y9LSL3_DELLE\nSRPK2\tA0A2Y9LSL6_DELLE\nSRPK2\tA0A2Y9LXZ1_DELLE\nSRPK2\tA0A2Y9LXZ6_DELLE\nSRPK2\tA0A2Y9RZ66_PHYMC\nSRPK2\tA0A2Y9RZE9_PHYMC\nSRPK2\tA0A2Y9S2Q0_PHYMC\nSRPK2\tA0A2Y9S6B8_PHYMC\nSRPK2\tA0A337RVJ0_FELCA\nSRPK2\tA0A337S2L4_FELCA\nSRPK2\tA0A337SHG6_FELCA\nSRPK2\tA0A340YC76_LIPVE\nSRPK2\tA0A341CCH4_NEOAA\nSRPK2\tA0A341CCH9_NEOAA\nSRPK2\tA0A341CCQ0_NEOAA\nSRPK2\tA0A341CD19_NEOAA\nSRPK2\tA0A341CDC2_NEOAA\nSRPK2\tA0A383ZQH4_BALAS\nSRPK2\tA0A383ZQH5_BALAS\nSRPK2\tA0A383ZQI0_BALAS\nSRPK2\tA0A384D3Z0_URSMA\nSRPK2\tA0A384D403_URSMA\nSRPK2\tA0A384D482_URSMA\nSRPK2\tA0A384D4S1_URSMA\nSRPK2\tA0A384D4X0_URSMA\nSRPK2\tA0A3B3BNL1_ORYME\nSRPK2\tA0A3B3BPD7_ORYME\nSRPK2\tA0A3B3BPF0_ORYME\nSRPK2\tA0A3B3BPI2_ORYME\nSRPK2\tA0A3B3BQ32_ORYME\nSRPK2\tA0A3B3D851_ORYME\nSRPK2\tA0A3B3D900_ORYME\nSRPK2\tA0A3B3HD36_ORYLA\nSRPK2\tA0A3B3HG42_ORYLA\nSRPK2\tA0A3B3HM64_ORYLA\nSRPK2\tA0A3B4YRC2_SERLL\nSRPK2\tA0A3B4Z6V1_SERLL\nSRPK2\tA0A3B4Z6X9_SERLL\nSRPK2\tA0A3B5K7N5_TAKRU\nSRPK2\tA0A3P9LCZ8_ORYLA\nSRPK2\tA0A3P9MKK2_ORYLA\nSRPK2\tA0A3P9MKL1_ORYLA\nSRPK2\tA0A3P9MKN8_ORYLA\nSRPK2\tA0A3P9MKP4_ORYLA\nSRPK2\tA0A3P9MKX3_ORYLA\nSRPK2\tA0A3Q0D438_MESAU\nSRPK2\tA0A3Q0EGD6_TARSY\nSRPK2\tA0A3Q0EGE3_TARSY\nSRPK2\tA0A3Q0EHJ6_TARSY\nSRPK2\tA0A3Q0EJ93_TARSY\nSRPK2\tA0A3Q0EJ98_TARSY\nSRPK2\tA0A3Q0ELC5_TARSY\nSRPK2\tA0A3Q0ELT0_TARSY\nSRPK2\tA0A3Q0ELT6_TARSY\nSRPK2\tA0A3Q0HIK3_ALLSI\nSRPK2\tA0A3Q1LKL2_BOVIN\nSRPK2\tA0A3Q1LWI0_BOVIN\nSRPK2\tA0A3Q1MH41_BOVIN\nSRPK2\tA0A3Q1NBC4_BOVIN\nSRPK2\tA0A3Q2H7N0_HORSE\nSRPK2\tA0A3Q2HBK9_HORSE\nSRPK2\tA0A3Q2HEF6_HORSE\nSRPK2\tA0A3Q2HRQ3_HORSE\nSRPK2\tA0A3Q2HW90_HORSE\nSRPK2\tA0A3Q2I4J5_HORSE\nSRPK2\tA0A3Q2ILP7_HORSE\nSRPK2\tA0A3Q2LNC7_HORSE\nSRPK2\tA0A3Q2LPA1_HORSE\nSRPK2\tA0A3Q7P3I1_CALUR\nSRPK2\tA0A3Q7P4D1_CALUR\nSRPK2\tA0A3Q7P4D7_CALUR\nSRPK2\tA0A3Q7P4H9_CALUR\nSRPK2\tA0A3Q7P4I5_CALUR\nSRPK2\tA0A3Q7P4J0_CALUR\nSRPK2\tA0A3Q7PEF9_CALUR\nSRPK2\tA0A3Q7PEG3_CALUR\nSRPK2\tA0A3Q7QJT8_CALUR\nSRPK2\tA0A3Q7QJU3_CALUR\nSRPK2\tA0A3Q7SJS3_VULVU\nSRPK2\tA0A3Q7T0K4_VULVU\nSRPK2\tA0A3Q7U3L2_VULVU\nSRPK2\tA0A3Q7VF79_URSAR\nSRPK2\tA0A3Q7VF91_URSAR\nSRPK2\tA0A3Q7W450_URSAR\nSRPK2\tA0A3Q7WDH0_URSAR\nSRPK2\tA0A3Q7WDJ2_URSAR\nSRPK2\tA0A3Q7WZF9_URSAR\nSRPK2\tA0A3Q7XAZ7_URSAR\nSRPK2\tA0A452FT07_CAPHI\nSRPK2\tA0A452FT36_CAPHI\nSRPK2\tA0A452FTD6_CAPHI\nSRPK2\tA0A452IX29_9SAUR\nSRPK2\tA0A452IX33_9SAUR\nSRPK2\tA0A452IX50_9SAUR\nSRPK2\tA0A452IX52_9SAUR\nSRPK2\tA0A452IX81_9SAUR\nSRPK2\tA0A452IX82_9SAUR\nSRPK2\tA0A452IX86_9SAUR\nSRPK2\tA0A452IX88_9SAUR\nSRPK2\tA0A452S4P3_URSAM\nSRPK2\tA0A493TGV1_ANAPP\nSRPK2\tA0A493TLG7_ANAPP\nSRPK2\tA0A493TT84_ANAPP\nSRPK2\tA0A493TVN2_ANAPP\nSRPK2\tA0A493TWC9_ANAPP\nSRPK2\tA0A4D8YW38_SALSN\nSRPK2\tA0A4R8PX61_9PEZI\nSRPK2\tA0A4U6X108_9PEZI\nSRPK2\tA0A4W2DI50_BOBOX\nSRPK2\tA0A4W2DNJ5_BOBOX\nSRPK2\tA0A4W2DNK9_BOBOX\nSRPK2\tA0A4W2DPD0_BOBOX\nSRPK2\tA0A4W2DPD5_BOBOX\nSRPK2\tA0A4W2F480_BOBOX\nSRPK2\tA0A4W2IHY1_BOBOX\nSRPK2\tA0A4W2IIG4_BOBOX\nSRPK2\tA0A4W2IK98_BOBOX\nSRPK2\tA0A4W3IS72_CALMI\nSRPK2\tA0A4W3IS75_CALMI\nSRPK2\tA0A4W3IXY2_CALMI\nSRPK2\tA0A4W3J1T8_CALMI\nSRPK2\tA0A4W3JJ59_CALMI\nSRPK2\tA0A4W3JJ80_CALMI\nSRPK2\tA0A4W3JJA9_CALMI\nSRPK2\tA0A4X1V528_PIG\nSRPK2\tA0A4X1V898_PIG\nSRPK2\tA0A4X1V8H6_PIG\nSRPK2\tA0A4X1V8K2_PIG\nSRPK2\tA0A4X1V8L2_PIG\nSRPK2\tA0A4X1V9A7_PIG\nSRPK2\tA0A4X1V9D1_PIG\nSRPK2\tA0A4X1VAJ8_PIG\nSRPK2\tA0A4X1VAL9_PIG\nSRPK2\tA0A4X2KRJ4_VOMUR\nSRPK2\tA0A4X2KWB3_VOMUR\nSRPK2\tA0A4X2KWE2_VOMUR\nSRPK2\tA0A4X2KY96_VOMUR\nSRPK2\tA0A4Y2NU61_ARAVE\nSRPK2\tA0A4Z2F577_9TELE\nSRPK2\tA0A5F4BZ55_CANLF\nSRPK2\tA0A5F5Y3R3_FELCA\nSRPK2\tA0A5F5Y3V4_FELCA\nSRPK2\tA0A5F5Y6A6_FELCA\nSRPK2\tA0A5F7ZN78_MACMU\nSRPK2\tA0A5F9C4B9_RABIT\nSRPK2\tA0A5F9C9G4_RABIT\nSRPK2\tA0A5F9CDM2_RABIT\nSRPK2\tA0A5F9CJB4_RABIT\nSRPK2\tA0A5F9CLC6_RABIT\nSRPK2\tA0A5F9CNT0_RABIT\nSRPK2\tA0A5F9D9J6_RABIT\nSRPK2\tA0A5F9DE20_RABIT\nSRPK2\tA0A5F9DJD2_RABIT\nSRPK2\tA0A5G3I2T7_XENTR\nSRPK2\tA0A5G3JWE1_XENTR\nSRPK2\tA0A5G3JWF6_XENTR\nSRPK2\tA0A5G3JWZ9_XENTR\nSRPK2\tA4IHA0_XENTR\nSRPK2\tB1WBT4_RAT\nSRPK2\tC9J2M4_HUMAN\nSRPK2\tC9JWF7_HUMAN\nSRPK2\tF1MNS2_BOVIN\nSRPK2\tF1ND58_CHICK\nSRPK2\tF1PR36_CANLF\nSRPK2\tF6RZE0_XENTR\nSRPK2\tF6UQU0_CANLF\nSRPK2\tF6V7S1_ORNAN\nSRPK2\tF7D504_XENTR\nSRPK2\tF7DQW6_HORSE\nSRPK2\tF7GFY2_MACMU\nSRPK2\tF7I592_CALJA\nSRPK2\tF7I7K2_CALJA\nSRPK2\tF8WAV6_HUMAN\nSRPK2\tF8WBW4_HUMAN\nSRPK2\tG1KG81_ANOCA\nSRPK2\tG1M3K0_AILME\nSRPK2\tG1N4H7_MELGA\nSRPK2\tG1NYH5_MYOLU\nSRPK2\tG1RCP0_NOMLE\nSRPK2\tG1SRM2_RABIT\nSRPK2\tG3SU62_LOXAF\nSRPK2\tG3VRM8_SARHA\nSRPK2\tG3VRM9_SARHA\nSRPK2\tH0UVZ9_CAVPO\nSRPK2\tH2LBK2_ORYLA\nSRPK2\tH2PN54_PONAB\nSRPK2\tH2UZ31_TAKRU\nSRPK2\tH3B9K5_LATCH\nSRPK2\tH3CJU9_TETNG\nSRPK2\tH3CJV0_TETNG\nSRPK2\tH3DEW0_TETNG\nSRPK2\tH7C521_HUMAN\nSRPK2\tH7C5L6_HUMAN\nSRPK2\tH9FYW3_MACMU\nSRPK2\tH9FYW4_MACMU\nSRPK2\tH9FYW5_MACMU\nSRPK2\tI3L729_PIG\nSRPK2\tI3MDX4_ICTTR\nSRPK2\tK7A8Q2_PANTR\nSRPK2\tK7AQP5_PANTR\nSRPK2\tK7F7X4_PELSI\nSRPK2\tK7F7X9_PELSI\nSRPK2\tM3W0B4_FELCA\nSRPK2\tM3Z3I4_MUSPF\nSRPK2\tO08551_MOUSE\nSRPK2\tQ17QG7_BOVIN\nSRPK2\tQ3TEK9_MOUSE\nSRPK2\tQ3TTG2_MOUSE\nSRPK2\tQ3UF35_MOUSE\nSRPK2\tQ6DJ09_XENTR\nSRPK2\tQ6P5E1_MOUSE\nSRPK2\tQ75MW9_HUMAN\nSRPK2\tQ8C9R8_MOUSE\nSRPK2\tQ8CBI1_MOUSE\nSRPK2\tQ8CEC7_MOUSE\nSRPK2\tQ9SHL5_ARATH\nSRPK2\tS4RMY6_PETMA\nSRPK2\tSRPK2_HUMAN\nSRPK2\tSRPK2_MOUSE\nSRPK2\tU3AQE7_CALJA\nSRPK2\tU3B9Q7_CALJA\nSRPK2\tU3I8J4_ANAPP\nSRPK2\tU3K6W7_FICAL\nSRPK2\tV8NIQ2_OPHHA\nSRPK2\tW5NEK1_LEPOC\nSRPK2\tW5NY28_SHEEP\nSRPK2\tW5NY30_SHEEP\nSRPK2\tW8BCF7_CERCA\nSRPK2\tW8BMU6_CERCA\nSRPK2\tW8BZK4_CERCA\n','From\tTo\nDYRK2\tA0A034V6Q7_BACDO\nDYRK2\tA0A034VX00_BACDO\nDYRK2\tA0A096MMM8_PAPAN\nDYRK2\tA0A0A0MR56_HUMAN\nDYRK2\tA0A0D9QWF0_CHLSB\nDYRK2\tA0A0P6JXM8_HETGA\nDYRK2\tA0A0R4ITS5_DANRE\nDYRK2\tA0A0S7H005_9TELE\nDYRK2\tA0A0S7HPT6_9TELE\nDYRK2\tA0A0S7LGD3_9TELE\nDYRK2\tA0A0S7LGE8_9TELE\nDYRK2\tA0A0S7LHQ9_9TELE\nDYRK2\tA0A151PFD7_ALLMI\nDYRK2\tA0A1A7WLF5_9TELE\nDYRK2\tA0A1A7Z2W2_9TELE\nDYRK2\tA0A1A8AR25_NOTFU\nDYRK2\tA0A1A8CF19_9TELE\nDYRK2\tA0A1A8FXU7_9TELE\nDYRK2\tA0A1A8GV89_9TELE\nDYRK2\tA0A1A8I057_NOTKU\nDYRK2\tA0A1A8N1U1_9TELE\nDYRK2\tA0A1A8RTA9_9TELE\nDYRK2\tA0A1A8SBS8_9TELE\nDYRK2\tA0A1A8UET4_NOTFU\nDYRK2\tA0A1J4JEC9_9EUKA\nDYRK2\tA0A1S3A2U9_ERIEU\nDYRK2\tA0A1S3A2Y1_ERIEU\nDYRK2\tA0A1S3FKV2_DIPOR\nDYRK2\tA0A1S3SJ03_SALSA\nDYRK2\tA0A1U7QXZ3_MESAU\nDYRK2\tA0A1U7RM61_ALLSI\nDYRK2\tA0A1V4J9G1_PATFA\nDYRK2\tA0A218V480_9PASE\nDYRK2\tA0A250Y2E0_CASCN\nDYRK2\tA0A286ZM94_PIG\nDYRK2\tA0A286ZZG5_PIG\nDYRK2\tA0A2B4RNR8_STYPI\nDYRK2\tA0A2H8TTT7_9HEMI\nDYRK2\tA0A2I0MJ20_COLLI\nDYRK2\tA0A2I3G8H5_NOMLE\nDYRK2\tA0A2I3HPT8_NOMLE\nDYRK2\tA0A2I3HY93_NOMLE\nDYRK2\tA0A2K5D6R3_AOTNA\nDYRK2\tA0A2K5D6V0_AOTNA\nDYRK2\tA0A2K5IUG4_COLAP\nDYRK2\tA0A2K5P3W4_CERAT\nDYRK2\tA0A2K5SCX2_CEBCA\nDYRK2\tA0A2K5V441_MACFA\nDYRK2\tA0A2K5V496_MACFA\nDYRK2\tA0A2K5V4A3_MACFA\nDYRK2\tA0A2K5Z961_MANLE\nDYRK2\tA0A2K5Z965_MANLE\nDYRK2\tA0A2K6AWN5_MACNE\nDYRK2\tA0A2K6G6Z1_PROCO\nDYRK2\tA0A2K6KTP4_RHIBE\nDYRK2\tA0A2K6KTP6_RHIBE\nDYRK2\tA0A2K6P807_RHIRO\nDYRK2\tA0A2K6SL42_SAIBB\nDYRK2\tA0A2K6SL48_SAIBB\nDYRK2\tA0A2P8XTB8_BLAGE\nDYRK2\tA0A2P8YEX4_BLAGE\nDYRK2\tA0A2R8QCS7_DANRE\nDYRK2\tA0A2R9CSN8_PANPA\nDYRK2\tA0A2S2NY50_SCHGA\nDYRK2\tA0A2S2PWE3_9HEMI\nDYRK2\tA0A2U3V819_TURTR\nDYRK2\tA0A2U3X3H9_ODORO\nDYRK2\tA0A2U3X3I5_ODORO\nDYRK2\tA0A2U3YSL8_LEPWE\nDYRK2\tA0A2Y9FTH5_PHYMC\nDYRK2\tA0A2Y9G5P6_NEOSC\nDYRK2\tA0A2Y9G643_NEOSC\nDYRK2\tA0A2Y9MZI6_DELLE\nDYRK2\tA0A2Y9SQG6_PHYMC\nDYRK2\tA0A340XTZ9_LIPVE\nDYRK2\tA0A341AP26_NEOAA\nDYRK2\tA0A341AR34_NEOAA\nDYRK2\tA0A371FCJ9_MUCPR\nDYRK2\tA0A384B632_BALAS\nDYRK2\tA0A384BJ26_URSMA\nDYRK2\tA0A3B5B7V7_9TELE\nDYRK2\tA0A3B5KC00_TAKRU\nDYRK2\tA0A3B5KDC4_TAKRU\nDYRK2\tA0A3Q0EEM3_TARSY\nDYRK2\tA0A3Q2U9V8_CHICK\nDYRK2\tA0A3Q2UD47_CHICK\nDYRK2\tA0A3Q7RE91_CALUR\nDYRK2\tA0A3Q7SPL6_VULVU\nDYRK2\tA0A3Q7WZR3_URSAR\nDYRK2\tA0A3R7CAG7_CLOSI\nDYRK2\tA0A3R7JJH4_CLOSI\nDYRK2\tA0A419PV68_CLOSI\nDYRK2\tA0A419PWU5_CLOSI\nDYRK2\tA0A452FX07_CAPHI\nDYRK2\tA0A452J0Q3_9SAUR\nDYRK2\tA0A452SE69_URSAM\nDYRK2\tA0A452V4R3_URSMA\nDYRK2\tA0A455BB11_PHYMC\nDYRK2\tA0A4C1US15_9NEOP\nDYRK2\tA0A4C1W8L9_9NEOP\nDYRK2\tA0A4W2CW94_BOBOX\nDYRK2\tA0A4W4GHJ9_ELEEL\nDYRK2\tA0A4W6CXM1_LATCA\nDYRK2\tA0A4W6CXY8_LATCA\nDYRK2\tA0A4X1THJ4_PIG\nDYRK2\tA0A4X1THK3_PIG\nDYRK2\tA0A4X2MD98_VOMUR\nDYRK2\tA0A4Z2HCT5_9TELE\nDYRK2\tA0A4Z2IUR5_9TELE\nDYRK2\tA0A5F4CHE2_CANLF\nDYRK2\tA0A5F4D6I5_CANLF\nDYRK2\tA0A5G3JFT9_XENTR\nDYRK2\tA0A5G3JGK3_XENTR\nDYRK2\tA0A5G3JHB7_XENTR\nDYRK2\tA0A5G3JHD2_XENTR\nDYRK2\tA8JCG2_CHLRE\nDYRK2\tD0VY19_9SAUR\nDYRK2\tD3B6H6_POLPP\nDYRK2\tDYRK2_CHICK\nDYRK2\tDYRK2_DICDI\nDYRK2\tDYRK2_DROME\nDYRK2\tDYRK2_HUMAN\nDYRK2\tDYRK2_MOUSE\nDYRK2\tE1BG90_BOVIN\nDYRK2\tE2R3D8_CANLF\nDYRK2\tF1LYY7_RAT\nDYRK2\tF1RYI2_PIG\nDYRK2\tF4PRC6_CAVFA\nDYRK2\tF5GXG1_HUMAN\nDYRK2\tF5H5L5_HUMAN\nDYRK2\tF6PQX9_ORNAN\nDYRK2\tF6PT02_MACMU\nDYRK2\tF6UT97_HORSE\nDYRK2\tF6WIM3_XENTR\nDYRK2\tF6WMH0_XENTR\nDYRK2\tF8UV41_9TRYP\nDYRK2\tG1LQ54_AILME\nDYRK2\tG1ND88_MELGA\nDYRK2\tG1PMI8_MYOLU\nDYRK2\tG1U832_RABIT\nDYRK2\tG3U961_LOXAF\nDYRK2\tG3WHK7_SARHA\nDYRK2\tH0UUA9_CAVPO\nDYRK2\tH0XS18_OTOGA\nDYRK2\tH0Z8J7_TAEGU\nDYRK2\tH2MSF8_ORYLA\nDYRK2\tH2NHZ2_PONAB\nDYRK2\tH3A3Z0_LATCH\nDYRK2\tH9F9L4_MACMU\nDYRK2\tH9G877_ANOCA\nDYRK2\tI0B8L2_DROME\nDYRK2\tI3KNL6_ORENI\nDYRK2\tI3MAP7_ICTTR\nDYRK2\tK7BQM1_PANTR\nDYRK2\tK7GF33_PELSI\nDYRK2\tL8ECA4_HUMAN\nDYRK2\tM3WEG8_FELCA\nDYRK2\tM3Z5N1_MUSPF\nDYRK2\tM9PDF9_DROME\nDYRK2\tQ5RHV3_DANRE\nDYRK2\tT2M7R8_HYDVU\nDYRK2\tU3DTS3_CALJA\nDYRK2\tU3I1C5_ANAPP\nDYRK2\tU3KJU8_FICAL\nDYRK2\tU6DY67_NEOVI\nDYRK2\tV5FSL7_ANOGL\nDYRK2\tV5FYI0_ANOGL\nDYRK2\tV5I7X1_ANOGL\nDYRK2\tV8NV09_OPHHA\nDYRK2\tW5Q8S2_SHEEP\nDYRK2\tW5U6X4_ICTPU\nDYRK2\tW5UFV8_ICTPU\nDYRK2\tW8B0J9_CERCA\nDYRK2\tW8B3P1_CERCA\nDYRK2\tW8B815_CERCA\nDYRK2\tW8BIV0_CERCA\nDYRK2\tX5I0Y5_9SAUR\n','From\tTo\nPCTK1\tA0A024R183_HUMAN\nPCTK1\tA6QR30_BOVIN\nPCTK1\tCDK16_HUMAN\nPCTK1\tCDK16_MOUSE\nPCTK1\tCDK16_RAT\nPCTK1\tQ3TM24_MOUSE\nPCTK1\tQ3UPI3_MOUSE\nPCTK1\tQ543G3_MOUSE\nPCTK1\tQ68G39_RAT\nPCTK1\tQ91WP4_MOUSE\nPCTK1\tQ96GA5_HUMAN\nPCTK1\tQ9BRL4_HUMAN\n','From\tTo\nMAPK1\tA0A060RZH9_PLARE\nMAPK1\tA0A068J3H9_GOBRA\nMAPK1\tA0A076G731_ANAPL\nMAPK1\tA0A096NSA3_PAPAN\nMAPK1\tA0A0A0R5M8_MUSAC\nMAPK1\tA0A0A0Z0N3_MAIZE\nMAPK1\tA0A0A7CJ31_VIGMU\nMAPK1\tA0A0D9RMI9_CHLSB\nMAPK1\tA0A0M3SGL2_POLSE\nMAPK1\tA0A0M4JJ87_TRASE\nMAPK1\tA0A0M4JSH6_LEUOC\nMAPK1\tA0A0M4K603_ANOSA\nMAPK1\tA0A0M5KA53_SQUAC\nMAPK1\tA0A0M5KC71_CRONI\nMAPK1\tA0A0U2I723_HEVBR\nMAPK1\tA0A0V0RKA9_9BILA\nMAPK1\tA0A0V0RKD6_9BILA\nMAPK1\tA0A0V0TT56_9BILA\nMAPK1\tA0A0V0TTB6_9BILA\nMAPK1\tA0A0V0WMA4_9BILA\nMAPK1\tA0A0V0YD49_TRIPS\nMAPK1\tA0A0V0ZP97_9BILA\nMAPK1\tA0A0V0ZQ21_9BILA\nMAPK1\tA0A0V1BLD9_TRISP\nMAPK1\tA0A0V1BML7_TRISP\nMAPK1\tA0A0V1BN81_TRISP\nMAPK1\tA0A0V1D293_TRIBR\nMAPK1\tA0A0V1EXR1_TRIPS\nMAPK1\tA0A0V1I965_9BILA\nMAPK1\tA0A0V1JIM3_TRIPS\nMAPK1\tA0A0V1KFD7_TRIPS\nMAPK1\tA0A0V1LQ42_9BILA\nMAPK1\tA0A0V1MQS8_9BILA\nMAPK1\tA0A0V1MR09_9BILA\nMAPK1\tA0A0V1P3A5_9BILA\nMAPK1\tA0A122IB84_PLABE\nMAPK1\tA0A151NRL4_ALLMI\nMAPK1\tA0A1A7YGE2_9TELE\nMAPK1\tA0A1A8EFV6_9TELE\nMAPK1\tA0A1A8EYV9_9TELE\nMAPK1\tA0A1A8IU32_NOTKU\nMAPK1\tA0A1A8JWG3_NOTKU\nMAPK1\tA0A1A8LXV5_9TELE\nMAPK1\tA0A1A8PU06_9TELE\nMAPK1\tA0A1A8V640_NOTFU\nMAPK1\tA0A1A8WCK0_PLAMA\nMAPK1\tA0A1C3KEW1_PLAMA\nMAPK1\tA0A1C3KX22_9APIC\nMAPK1\tA0A1C6XE60_PLACH\nMAPK1\tA0A1C6XQ96_PLACH\nMAPK1\tA0A1D3LER1_PLACH\nMAPK1\tA0A1D3LKE7_PLACH\nMAPK1\tA0A1D3TME4_9APIC\nMAPK1\tA0A1J1GUA1_PLAGA\nMAPK1\tA0A1J1HE67_PLARL\nMAPK1\tA0A1L8HQD8_XENLA\nMAPK1\tA0A1S3AK70_ERIEU\nMAPK1\tA0A1S3GE91_DIPOR\nMAPK1\tA0A1S3M6Y0_SALSA\nMAPK1\tA0A1S3WT01_ERIEU\nMAPK1\tA0A1S3WTK1_ERIEU\nMAPK1\tA0A1S6Q401_CASFI\nMAPK1\tA0A1U7QEZ9_MESAU\nMAPK1\tA0A1U7U7P5_TARSY\nMAPK1\tA0A1U8CZE0_ALLSI\nMAPK1\tA0A1V4JRY5_PATFA\nMAPK1\tA0A1Y3DT37_PLAKN\nMAPK1\tA0A218V2J1_9PASE\nMAPK1\tA0A286XEA7_CAVPO\nMAPK1\tA0A286ZPF5_PIG\nMAPK1\tA0A287D7G2_ICTTR\nMAPK1\tA0A289ZMV7_SINCH\nMAPK1\tA0A2B4RIB9_STYPI\nMAPK1\tA0A2D0PN84_ICTPU\nMAPK1\tA0A2H8TXZ7_9HEMI\nMAPK1\tA0A2I0M867_COLLI\nMAPK1\tA0A2I0M868_COLLI\nMAPK1\tA0A2I3G715_NOMLE\nMAPK1\tA0A2I3GWB6_NOMLE\nMAPK1\tA0A2I3LQV7_PAPAN\nMAPK1\tA0A2I3MJP4_PAPAN\nMAPK1\tA0A2I3MU63_PAPAN\nMAPK1\tA0A2I3RP31_PANTR\nMAPK1\tA0A2I3TP29_PANTR\nMAPK1\tA0A2J7QM98_9NEOP\nMAPK1\tA0A2K4Y315_9TELE\nMAPK1\tA0A2K5D1P7_AOTNA\nMAPK1\tA0A2K5D1Q4_AOTNA\nMAPK1\tA0A2K5D1R1_AOTNA\nMAPK1\tA0A2K5D1R3_AOTNA\nMAPK1\tA0A2K5D1S7_AOTNA\nMAPK1\tA0A2K5JCS7_COLAP\nMAPK1\tA0A2K5JCU9_COLAP\nMAPK1\tA0A2K5JD19_COLAP\nMAPK1\tA0A2K5JD23_COLAP\nMAPK1\tA0A2K5LSN3_CERAT\nMAPK1\tA0A2K5LSR7_CERAT\nMAPK1\tA0A2K5LSR8_CERAT\nMAPK1\tA0A2K5QFC4_CEBCA\nMAPK1\tA0A2K5QFF8_CEBCA\nMAPK1\tA0A2K5QFG7_CEBCA\nMAPK1\tA0A2K5QFI1_CEBCA\nMAPK1\tA0A2K5TXB4_MACFA\nMAPK1\tA0A2K5TXF6_MACFA\nMAPK1\tA0A2K5TXG6_MACFA\nMAPK1\tA0A2K5Z779_MANLE\nMAPK1\tA0A2K5Z799_MANLE\nMAPK1\tA0A2K6BK71_MACNE\nMAPK1\tA0A2K6BK85_MACNE\nMAPK1\tA0A2K6BKH0_MACNE\nMAPK1\tA0A2K6BKH9_MACNE\nMAPK1\tA0A2K6BKJ4_MACNE\nMAPK1\tA0A2K6GUD5_PROCO\nMAPK1\tA0A2K6GUF3_PROCO\nMAPK1\tA0A2K6GUF7_PROCO\nMAPK1\tA0A2K6GUJ9_PROCO\nMAPK1\tA0A2K6LSC3_RHIBE\nMAPK1\tA0A2K6LSC4_RHIBE\nMAPK1\tA0A2K6LSG3_RHIBE\nMAPK1\tA0A2K6Q7K0_RHIRO\nMAPK1\tA0A2K6Q7K2_RHIRO\nMAPK1\tA0A2K6Q7L7_RHIRO\nMAPK1\tA0A2K6TW10_SAIBB\nMAPK1\tA0A2K6TW25_SAIBB\nMAPK1\tA0A2K6TW35_SAIBB\nMAPK1\tA0A2K6TW57_SAIBB\nMAPK1\tA0A2K6TWC7_SAIBB\nMAPK1\tA0A2K6TWD4_SAIBB\nMAPK1\tA0A2P8XXP6_BLAGE\nMAPK1\tA0A2R8ZMR5_PANPA\nMAPK1\tA0A2R8ZMR6_PANPA\nMAPK1\tA0A2R8ZTR1_PANPA\nMAPK1\tA0A2S2PE68_SCHGA\nMAPK1\tA0A2U3VYB0_ODORO\nMAPK1\tA0A2U3VYF5_ODORO\nMAPK1\tA0A2U3YIW0_LEPWE\nMAPK1\tA0A2U4AL99_TURTR\nMAPK1\tA0A2Y9H0I4_NEOSC\nMAPK1\tA0A2Y9LBR5_DELLE\nMAPK1\tA0A2Y9LEH4_DELLE\nMAPK1\tA0A2Y9LJ56_DELLE\nMAPK1\tA0A2Y9LP94_DELLE\nMAPK1\tA0A2Y9LP99_DELLE\nMAPK1\tA0A2Y9SG06_PHYMC\nMAPK1\tA0A337SUJ5_FELCA\nMAPK1\tA0A338P736_MOUSE\nMAPK1\tA0A338P781_MOUSE\nMAPK1\tA0A340WNH5_LIPVE\nMAPK1\tA0A341CXW3_NEOAA\nMAPK1\tA0A345DKQ6_9TELE\nMAPK1\tA0A384B9K1_BALAS\nMAPK1\tA0A384CLZ9_URSMA\nMAPK1\tA0A3B5K7P3_TAKRU\nMAPK1\tA0A3P8VHM2_CYNSE\nMAPK1\tA0A3Q0E9L9_TARSY\nMAPK1\tA0A3Q1GAH0_9TELE\nMAPK1\tA0A3Q4HZL2_NEOBR\nMAPK1\tA0A3Q7PBC8_CALUR\nMAPK1\tA0A3Q7QZE3_VULVU\nMAPK1\tA0A3Q7UU47_URSAR\nMAPK1\tA0A452DK54_CAPHI\nMAPK1\tA0A452DK70_CAPHI\nMAPK1\tA0A452IKT2_9SAUR\nMAPK1\tA0A452R544_URSAM\nMAPK1\tA0A452R547_URSAM\nMAPK1\tA0A452R5K9_URSAM\nMAPK1\tA0A452T8N6_URSMA\nMAPK1\tA0A452T8Y0_URSMA\nMAPK1\tA0A4C1TIW1_9NEOP\nMAPK1\tA0A4C1TIY7_9NEOP\nMAPK1\tA0A4C1TK16_9NEOP\nMAPK1\tA0A4W2CKY1_BOBOX\nMAPK1\tA0A4W2CPM8_BOBOX\nMAPK1\tA0A4W2FYH7_BOBOX\nMAPK1\tA0A4W2G022_BOBOX\nMAPK1\tA0A4W2G2L3_BOBOX\nMAPK1\tA0A4W3IYY8_CALMI\nMAPK1\tA0A4W3J331_CALMI\nMAPK1\tA0A4W3J339_CALMI\nMAPK1\tA0A4W4F0W8_ELEEL\nMAPK1\tA0A4W4F1B9_ELEEL\nMAPK1\tA0A4W4F1D5_ELEEL\nMAPK1\tA0A4W4F236_ELEEL\nMAPK1\tA0A4W4F2W3_ELEEL\nMAPK1\tA0A4W4F2Y0_ELEEL\nMAPK1\tA0A4W4F3V0_ELEEL\nMAPK1\tA0A4W4F453_ELEEL\nMAPK1\tA0A4W6FKA8_LATCA\nMAPK1\tA0A4W6FKB3_LATCA\nMAPK1\tA0A4W6FMF1_LATCA\nMAPK1\tA0A4X2M571_VOMUR\nMAPK1\tA0A4Y2H9G2_ARAVE\nMAPK1\tA0A5B7DVR5_PORTR\nMAPK1\tA0A5B7EHU8_PORTR\nMAPK1\tA0A5B7KF30_PORTR\nMAPK1\tA0A5F4CNM8_CANLF\nMAPK1\tA0A5F5PIW7_HORSE\nMAPK1\tA0A5F5Y7K8_FELCA\nMAPK1\tA0A5G2RG05_PIG\nMAPK1\tA0A5G3KNQ2_XENTR\nMAPK1\tA0A5K1UVT7_PIG\nMAPK1\tA1BLL8_9BASI\nMAPK1\tB0LXE2_DATME\nMAPK1\tB6VDZ7_LITCT\nMAPK1\tB7ZEB8_EUPRA\nMAPK1\tD2HUM0_AILME\nMAPK1\tD8TR37_VOLCA\nMAPK1\tE2R2N2_CANLF\nMAPK1\tF1RL02_PIG\nMAPK1\tF6PMR8_HORSE\nMAPK1\tF6U3T6_XENTR\nMAPK1\tF7AKA7_CALJA\nMAPK1\tF7D5W5_XENTR\nMAPK1\tF7ECR3_MACMU\nMAPK1\tF7FHD1_MONDO\nMAPK1\tF7G0S9_CALJA\nMAPK1\tF8W3M8_DANRE\nMAPK1\tG1MQT8_MELGA\nMAPK1\tG1QHA1_NOMLE\nMAPK1\tG1T5Y1_RABIT\nMAPK1\tG3T4W5_LOXAF\nMAPK1\tG3W0G0_SARHA\nMAPK1\tG9C080_HUMLU\nMAPK1\tH0VMU3_CAVPO\nMAPK1\tH0WZV9_OTOGA\nMAPK1\tH0ZE69_TAEGU\nMAPK1\tH2QLB4_PANTR\nMAPK1\tH2UQ68_TAKRU\nMAPK1\tH3B1U6_LATCH\nMAPK1\tH9D0Z6_9LILI\nMAPK1\tH9FBJ3_MACMU\nMAPK1\tH9G541_ANOCA\nMAPK1\tH9TUY1_CITSI\nMAPK1\tI2FLA0_9BASI\nMAPK1\tI3K9F0_ORENI\nMAPK1\tI3MNK8_ICTTR\nMAPK1\tK7F5Q1_PELSI\nMAPK1\tK7F5Q2_PELSI\nMAPK1\tK7RCQ1_CAPHI\nMAPK1\tL7P8C9_CENAM\nMAPK1\tM3WWM5_FELCA\nMAPK1\tM3XFY0_FELCA\nMAPK1\tM3XVL2_MUSPF\nMAPK1\tM5BQF7_THACB\nMAPK1\tM9SYG8_EUPNO\nMAPK1\tMK01_BOVIN\nMAPK1\tMK01_HUMAN\nMAPK1\tMK01_MOUSE\nMAPK1\tMK01_RAT\nMAPK1\tMK01_XENLA\nMAPK1\tMPK12_ORYSJ\nMAPK1\tQ1HBJ4_HUMAN\nMAPK1\tQ28H11_XENTR\nMAPK1\tQ3UF82_MOUSE\nMAPK1\tQ499G7_HUMAN\nMAPK1\tQ4VH14_DANRE\nMAPK1\tQ5K6Q4_SOYBN\nMAPK1\tQ6DFK6_XENLA\nMAPK1\tQ6P023_DANRE\nMAPK1\tQ70AJ4_WHEAT\nMAPK1\tQ7Z8F1_METAN\nMAPK1\tQ7ZA49_9HYPO\nMAPK1\tQ7ZW72_DANRE\nMAPK1\tQ8VXC2_ORYSA\nMAPK1\tQ922X7_MOUSE\nMAPK1\tQ9DGR5_DANRE\nMAPK1\tR1E169_EMIHU\nMAPK1\tT2MFY0_HYDVU\nMAPK1\tU2FP23_9GAMM\nMAPK1\tU3DQA9_CALJA\nMAPK1\tU3JZS8_FICAL\nMAPK1\tV5W4Q0_GANLU\nMAPK1\tV8NJR1_OPHHA\nMAPK1\tV9KVY3_CALMI\nMAPK1\tW5Q1R8_SHEEP\nMAPK1\tW9SAM8_9ROSA\n','From\tTo\nMAPK3\tA0A096NHR1_PAPAN\nMAPK3\tA0A0A0R603_MUSAC\nMAPK3\tA0A0D9R1K1_CHLSB\nMAPK3\tA0A0E3X917_CLABA\nMAPK3\tA0A0M3SGK7_TRASE\nMAPK3\tA0A0M4JA61_CRONI\nMAPK3\tA0A0M4JAW9_POLSE\nMAPK3\tA0A0M4JN57_ANOSA\nMAPK3\tA0A0S7HRA7_9TELE\nMAPK3\tA0A0S7LC67_9TELE\nMAPK3\tA0A0S7LC85_9TELE\nMAPK3\tA0A0S7LCM7_9TELE\nMAPK3\tA0A0U1RPX4_MOUSE\nMAPK3\tA0A0U1RPZ0_MOUSE\nMAPK3\tA0A1A7WUV8_9TELE\nMAPK3\tA0A1A8B6U7_NOTFU\nMAPK3\tA0A1A8CLX9_9TELE\nMAPK3\tA0A1A8DDU9_9TELE\nMAPK3\tA0A1A8FLP8_9TELE\nMAPK3\tA0A1A8IIC8_NOTKU\nMAPK3\tA0A1A8PLF6_9TELE\nMAPK3\tA0A1A8SIT7_9TELE\nMAPK3\tA0A1A8U5S1_NOTFU\nMAPK3\tA0A1D5QZ91_MACMU\nMAPK3\tA0A1L2D3T2_9ROSI\nMAPK3\tA0A1S2XLY6_CICAR\nMAPK3\tA0A1S2XNB2_CICAR\nMAPK3\tA0A1S3AGJ4_ERIEU\nMAPK3\tA0A1S3DYS5_CICAR\nMAPK3\tA0A1S3ET79_DIPOR\nMAPK3\tA0A1S3P212_SALSA\nMAPK3\tA0A1U7UM96_TARSY\nMAPK3\tA0A250YFI8_CASCN\nMAPK3\tA0A286XFG6_CAVPO\nMAPK3\tA0A286XN62_CAVPO\nMAPK3\tA0A287AJK9_PIG\nMAPK3\tA0A287BLM6_PIG\nMAPK3\tA0A287D3P7_ICTTR\nMAPK3\tA0A289Z111_SINCH\nMAPK3\tA0A2I2UPF0_FELCA\nMAPK3\tA0A2I3MRY2_PAPAN\nMAPK3\tA0A2I3NHG2_PAPAN\nMAPK3\tA0A2I3RF13_PANTR\nMAPK3\tA0A2I3S3J0_PANTR\nMAPK3\tA0A2K4Y349_9TELE\nMAPK3\tA0A2K5F5S8_AOTNA\nMAPK3\tA0A2K5F5T4_AOTNA\nMAPK3\tA0A2K5F5U3_AOTNA\nMAPK3\tA0A2K5F5Y3_AOTNA\nMAPK3\tA0A2K5IFT8_COLAP\nMAPK3\tA0A2K5IFV4_COLAP\nMAPK3\tA0A2K5IG43_COLAP\nMAPK3\tA0A2K5IG64_COLAP\nMAPK3\tA0A2K5LEU5_CERAT\nMAPK3\tA0A2K5LEY9_CERAT\nMAPK3\tA0A2K5LF29_CERAT\nMAPK3\tA0A2K5LF39_CERAT\nMAPK3\tA0A2K5SAA4_CEBCA\nMAPK3\tA0A2K5SAA7_CEBCA\nMAPK3\tA0A2K5SAF7_CEBCA\nMAPK3\tA0A2K5VXW8_MACFA\nMAPK3\tA0A2K5VXX8_MACFA\nMAPK3\tA0A2K5VXY8_MACFA\nMAPK3\tA0A2K5VY25_MACFA\nMAPK3\tA0A2K5Z098_MANLE\nMAPK3\tA0A2K5Z0A4_MANLE\nMAPK3\tA0A2K5Z0A7_MANLE\nMAPK3\tA0A2K5Z0D6_MANLE\nMAPK3\tA0A2K6BWG8_MACNE\nMAPK3\tA0A2K6BWJ3_MACNE\nMAPK3\tA0A2K6BWK9_MACNE\nMAPK3\tA0A2K6F2Q7_PROCO\nMAPK3\tA0A2K6F2R4_PROCO\nMAPK3\tA0A2K6F2W7_PROCO\nMAPK3\tA0A2K6JND3_RHIBE\nMAPK3\tA0A2K6JNE3_RHIBE\nMAPK3\tA0A2K6JNE5_RHIBE\nMAPK3\tA0A2K6QTI1_RHIRO\nMAPK3\tA0A2K6QTI3_RHIRO\nMAPK3\tA0A2K6QTJ2_RHIRO\nMAPK3\tA0A2K6QTM1_RHIRO\nMAPK3\tA0A2K6U8Z7_SAIBB\nMAPK3\tA0A2K6U924_SAIBB\nMAPK3\tA0A2K6U929_SAIBB\nMAPK3\tA0A2K6U968_SAIBB\nMAPK3\tA0A2R8MEW2_CALJA\nMAPK3\tA0A2R9BKB0_PANPA\nMAPK3\tA0A2R9BKG3_PANPA\nMAPK3\tA0A2S2P210_SCHGA\nMAPK3\tA0A2S2Q4T9_9HEMI\nMAPK3\tA0A2U3VPX4_ODORO\nMAPK3\tA0A2U3VPZ0_ODORO\nMAPK3\tA0A2U4A994_TURTR\nMAPK3\tA0A2Y9HX44_NEOSC\nMAPK3\tA0A2Y9PDR5_DELLE\nMAPK3\tA0A2Y9SSQ5_PHYMC\nMAPK3\tA0A337S673_FELCA\nMAPK3\tA0A340WYU9_LIPVE\nMAPK3\tA0A341BDH3_NEOAA\nMAPK3\tA0A345DKQ7_9TELE\nMAPK3\tA0A384AGR0_BALAS\nMAPK3\tA0A384BLI2_URSMA\nMAPK3\tA0A384BLM5_URSMA\nMAPK3\tA0A3Q0CVF3_MESAU\nMAPK3\tA0A3Q1M7B5_BOVIN\nMAPK3\tA0A3Q1MNN7_BOVIN\nMAPK3\tA0A3Q3ALB3_CHICK\nMAPK3\tA0A3Q7U1D8_VULVU\nMAPK3\tA0A3Q7X2M0_URSAR\nMAPK3\tA0A3Q7XWN6_URSAR\nMAPK3\tA0A3S5ZPA9_CHICK\nMAPK3\tA0A419Q8I2_CLOSI\nMAPK3\tA0A451G5T3_BOSMU\nMAPK3\tA0A452ETI1_CAPHI\nMAPK3\tA0A452ETJ2_CAPHI\nMAPK3\tA0A452EU65_CAPHI\nMAPK3\tA0A452GKE1_9SAUR\nMAPK3\tA0A452SGC3_URSAM\nMAPK3\tA0A452SGD4_URSAM\nMAPK3\tA0A452SGF2_URSAM\nMAPK3\tA0A499QUT9_9TELE\nMAPK3\tA0A4W2DMZ4_BOBOX\nMAPK3\tA0A4W2GPB1_BOBOX\nMAPK3\tA0A4W2GQ76_BOBOX\nMAPK3\tA0A4W2IA23_BOBOX\nMAPK3\tA0A4X1U507_PIG\nMAPK3\tA0A4X1U522_PIG\nMAPK3\tA0A4X1U710_PIG\nMAPK3\tA0A4X1U727_PIG\nMAPK3\tA0A4X1U7I1_PIG\nMAPK3\tA0A4X2KGZ8_VOMUR\nMAPK3\tA0A4X2KHA3_VOMUR\nMAPK3\tA0A4X2KQ11_VOMUR\nMAPK3\tA0A4Y6GP12_9ROSI\nMAPK3\tA0A5B0N2D7_PUCGR\nMAPK3\tA0A5F4C5E5_CANLF\nMAPK3\tA0A5F4WMH8_CALJA\nMAPK3\tA0A5F5PK37_HORSE\nMAPK3\tA0A5F5Q3S5_HORSE\nMAPK3\tA0A5F8GWX8_MONDO\nMAPK3\tA0A5G2QCY4_PIG\nMAPK3\tA0A5G2QKL2_PIG\nMAPK3\tA8IKE3_CHLRE\nMAPK3\tB3KR49_HUMAN\nMAPK3\tB5X0U4_SALSA\nMAPK3\tB5XF28_SALSA\nMAPK3\tD3Z3G6_MOUSE\nMAPK3\tD3Z6D8_MOUSE\nMAPK3\tD8TP73_VOLCA\nMAPK3\tE1B8P9_BOVIN\nMAPK3\tE6ZFU9_DICLA\nMAPK3\tE9PBK7_HUMAN\nMAPK3\tE9PJF0_HUMAN\nMAPK3\tE9PQW4_HUMAN\nMAPK3\tE9PRH7_HUMAN\nMAPK3\tF6PID7_MONDO\nMAPK3\tF7C0M6_HORSE\nMAPK3\tF7EI48_MONDO\nMAPK3\tF7EZ96_MACMU\nMAPK3\tF7GDI6_CALJA\nMAPK3\tF7GMI8_CALJA\nMAPK3\tG1M9W1_AILME\nMAPK3\tG1PHW8_MYOLU\nMAPK3\tG1QN56_NOMLE\nMAPK3\tG1T5Z7_RABIT\nMAPK3\tG3TJS2_LOXAF\nMAPK3\tG3UNI7_LOXAF\nMAPK3\tG3WBE0_SARHA\nMAPK3\tH0V0F7_CAVPO\nMAPK3\tH0XRG7_OTOGA\nMAPK3\tH0YDH9_HUMAN\nMAPK3\tH0YEX6_HUMAN\nMAPK3\tH2NQJ9_PONAB\nMAPK3\tH2RA86_PANTR\nMAPK3\tH9F7V0_MACMU\nMAPK3\tH9F856_MACMU\nMAPK3\tH9GLX1_ANOCA\nMAPK3\tI3LXI1_ICTTR\nMAPK3\tK7CJC8_PANTR\nMAPK3\tL7RXH5_HUMAN\nMAPK3\tM3V839_PIG\nMAPK3\tM3WK78_FELCA\nMAPK3\tM3YMI6_MUSPF\nMAPK3\tM5BQA4_THACB\nMAPK3\tMK03_HUMAN\nMAPK3\tMK03_MOUSE\nMAPK3\tMK03_RAT\nMAPK3\tQ3U2H3_MOUSE\nMAPK3\tQ7ZVK8_DANRE\nMAPK3\tQ84MI4_SOLLC\nMAPK3\tQ8VXC1_ORYSA\nMAPK3\tQ9BWJ1_HUMAN\nMAPK3\tQ9DGR6_DANRE\nMAPK3\tQ9LKZ2_CAPAN\nMAPK3\tQ9M6S1_PEA\nMAPK3\tR1D881_EMIHU\nMAPK3\tT2DJ89_BRAJU\nMAPK3\tT2DJU7_BRAJU\nMAPK3\tT2HU73_BRAJU\nMAPK3\tU6DJJ4_NEOVI\nMAPK3\tV5W5U9_GANLU\nMAPK3\tV8NI48_OPHHA\nMAPK3\tW5P0L9_SHEEP\nMAPK3\tW5P0M0_SHEEP\nMAPK3\tW9SCE8_9ROSA\n','From\tTo\nAURKA\tA0A034WT24_BACDO\nAURKA\tA0A096NNK5_PAPAN\nAURKA\tA0A0A9XG60_LYGHE\nAURKA\tA0A0D9RS62_CHLSB\nAURKA\tA0A0S7H184_9TELE\nAURKA\tA0A0S7H1G4_9TELE\nAURKA\tA0A0V0RQT5_9BILA\nAURKA\tA0A0V0RVY0_9BILA\nAURKA\tA0A0V0TRI9_9BILA\nAURKA\tA0A0V0UHD7_9BILA\nAURKA\tA0A0V0W1G1_9BILA\nAURKA\tA0A0V0WQ14_9BILA\nAURKA\tA0A0V0Y429_TRIPS\nAURKA\tA0A0V0YGZ4_TRIPS\nAURKA\tA0A0V0YH10_TRIPS\nAURKA\tA0A0V0ZD19_9BILA\nAURKA\tA0A0V0ZED4_9BILA\nAURKA\tA0A0V0ZT17_9BILA\nAURKA\tA0A0V1B461_TRISP\nAURKA\tA0A0V1CFX0_TRIBR\nAURKA\tA0A0V1CNX0_TRIBR\nAURKA\tA0A0V1ELM4_TRIPS\nAURKA\tA0A0V1ELP7_TRIPS\nAURKA\tA0A0V1EPC2_TRIPS\nAURKA\tA0A0V1GZP0_9BILA\nAURKA\tA0A0V1HGH7_9BILA\nAURKA\tA0A0V1IN39_TRIPS\nAURKA\tA0A0V1IU35_TRIPS\nAURKA\tA0A0V1JME2_TRIPS\nAURKA\tA0A0V1KVT6_9BILA\nAURKA\tA0A0V1LD29_9BILA\nAURKA\tA0A0V1MCU0_9BILA\nAURKA\tA0A0V1MK65_9BILA\nAURKA\tA0A0V1NL56_9BILA\nAURKA\tA0A0V1PHH6_9BILA\nAURKA\tA0A0X3P0Y9_SCHSO\nAURKA\tA0A0X3PGN5_SCHSO\nAURKA\tA0A0X3PVM4_SCHSO\nAURKA\tA0A146LZD1_LYGHE\nAURKA\tA0A151NYV8_ALLMI\nAURKA\tA0A1A7XP04_9TELE\nAURKA\tA0A1A8A3F3_NOTFU\nAURKA\tA0A1A8AZ97_NOTFU\nAURKA\tA0A1A8BU11_9TELE\nAURKA\tA0A1A8D3P2_9TELE\nAURKA\tA0A1A8E7V6_9TELE\nAURKA\tA0A1A8FQ15_9TELE\nAURKA\tA0A1A8GEE6_9TELE\nAURKA\tA0A1A8JET5_NOTKU\nAURKA\tA0A1A8JH51_NOTKU\nAURKA\tA0A1A8K5F3_NOTKU\nAURKA\tA0A1A8K5S1_NOTKU\nAURKA\tA0A1A8L6L8_9TELE\nAURKA\tA0A1A8MAQ1_9TELE\nAURKA\tA0A1A8N316_9TELE\nAURKA\tA0A1A8N339_9TELE\nAURKA\tA0A1A8N790_9TELE\nAURKA\tA0A1A8QY93_9TELE\nAURKA\tA0A1A8RFF2_9TELE\nAURKA\tA0A1A8U660_NOTFU\nAURKA\tA0A1A8UYU4_NOTFU\nAURKA\tA0A1C7NKY9_9FUNG\nAURKA\tA0A1J4K3C2_9EUKA\nAURKA\tA0A1J4K6V0_9EUKA\nAURKA\tA0A1L8ESU8_XENLA\nAURKA\tA0A1Q9DAZ3_SYMMI\nAURKA\tA0A1S3ADG6_ERIEU\nAURKA\tA0A1S3FT78_DIPOR\nAURKA\tA0A1S3WNU8_ERIEU\nAURKA\tA0A1S3WNV3_ERIEU\nAURKA\tA0A1S3WPA0_ERIEU\nAURKA\tA0A1U7QWD0_MESAU\nAURKA\tA0A1U7R6R7_ALLSI\nAURKA\tA0A1U7TEF7_TARSY\nAURKA\tA0A1U7TPV5_TARSY\nAURKA\tA0A1V4KE16_PATFA\nAURKA\tA0A1Y1S5B4_9MICR\nAURKA\tA0A1Y1S7A6_9MICR\nAURKA\tA0A218UM55_9PASE\nAURKA\tA0A286ZWN5_PIG\nAURKA\tA0A2I0M5W6_COLLI\nAURKA\tA0A2I3TQI8_PANTR\nAURKA\tA0A2I4AXM0_9TELE\nAURKA\tA0A2J8TXS8_PONAB\nAURKA\tA0A2K5HS25_COLAP\nAURKA\tA0A2K5PYP0_CEBCA\nAURKA\tA0A2K5TW98_MACFA\nAURKA\tA0A2K5TWC7_MACFA\nAURKA\tA0A2K5XCD6_MANLE\nAURKA\tA0A2K6AZG6_MACNE\nAURKA\tA0A2K6F1Q3_PROCO\nAURKA\tA0A2K6S2P5_SAIBB\nAURKA\tA0A2K6S2R0_SAIBB\nAURKA\tA0A2N6NVZ3_BEABA\nAURKA\tA0A2R9ANX1_PANPA\nAURKA\tA0A2R9ANX7_PANPA\nAURKA\tA0A2U3V4Z5_TURTR\nAURKA\tA0A2U3V5C7_TURTR\nAURKA\tA0A2U3X450_ODORO\nAURKA\tA0A2U4C1P7_TURTR\nAURKA\tA0A2U4C1Y2_TURTR\nAURKA\tA0A2Y9ELJ6_PHYMC\nAURKA\tA0A2Y9G376_NEOSC\nAURKA\tA0A2Y9Q6F5_DELLE\nAURKA\tA0A2Y9QCE6_DELLE\nAURKA\tA0A340XSQ0_LIPVE\nAURKA\tA0A340XVV9_LIPVE\nAURKA\tA0A340XYD2_LIPVE\nAURKA\tA0A341BTD3_NEOAA\nAURKA\tA0A341BVN3_NEOAA\nAURKA\tA0A341BWB9_NEOAA\nAURKA\tA0A344X2G9_LATJA\nAURKA\tA0A384ADF4_BALAS\nAURKA\tA0A384CSB7_URSMA\nAURKA\tA0A3B3HNX1_ORYLA\nAURKA\tA0A3Q0E547_TARSY\nAURKA\tA0A3Q2IIQ8_HORSE\nAURKA\tA0A3Q2LCY5_HORSE\nAURKA\tA0A3Q7MPR9_CALUR\nAURKA\tA0A3Q7P0E0_CALUR\nAURKA\tA0A3Q7UUA3_VULVU\nAURKA\tA0A3Q7VEP9_URSAR\nAURKA\tA0A452FXV9_CAPHI\nAURKA\tA0A452FY65_CAPHI\nAURKA\tA0A452FYJ8_CAPHI\nAURKA\tA0A452I551_9SAUR\nAURKA\tA0A452RW58_URSAM\nAURKA\tA0A493T9N6_ANAPP\nAURKA\tA0A493TE53_ANAPP\nAURKA\tA0A4W2DCW2_BOBOX\nAURKA\tA0A4W2IAI2_BOBOX\nAURKA\tA0A4W3IYX2_CALMI\nAURKA\tA0A4X1UKM8_PIG\nAURKA\tA0A4X2LRA6_VOMUR\nAURKA\tA0A5B7F7L9_PORTR\nAURKA\tA0A5F5XZQ6_FELCA\nAURKA\tA3KFJ0_HUMAN\nAURKA\tA3KFJ1_HUMAN\nAURKA\tA3KFJ2_HUMAN\nAURKA\tA4UTN8_PIG\nAURKA\tAURAA_XENLA\nAURKA\tAURAB_XENLA\nAURKA\tAURKA_BOVIN\nAURKA\tAURKA_HUMAN\nAURKA\tAURKA_MOUSE\nAURKA\tAURKA_PIG\nAURKA\tAURKA_RAT\nAURKA\tB5DFP5_XENTR\nAURKA\tB7U174_SHEEP\nAURKA\tD7ELW1_TRICA\nAURKA\tE5STA0_TRISP\nAURKA\tF1QEM3_DANRE\nAURKA\tF6S2E1_ORNAN\nAURKA\tF6U2S3_XENTR\nAURKA\tF6YZI7_HORSE\nAURKA\tF7ASA8_MONDO\nAURKA\tF7GNU0_MACMU\nAURKA\tF7GVV3_CALJA\nAURKA\tF8W4T4_DANRE\nAURKA\tG1LKV1_AILME\nAURKA\tG1NAG1_MELGA\nAURKA\tG1PAC6_MYOLU\nAURKA\tG1R5V0_NOMLE\nAURKA\tG1TEP5_RABIT\nAURKA\tG2HFJ0_PANTR\nAURKA\tG3T1Q4_LOXAF\nAURKA\tG3WJ52_SARHA\nAURKA\tG5AW12_HETGA\nAURKA\tH0V884_CAVPO\nAURKA\tH0ZDQ2_TAEGU\nAURKA\tH2LM03_ORYLA\nAURKA\tH2LM09_ORYLA\nAURKA\tH2UAA9_TAKRU\nAURKA\tH2UAB0_TAKRU\nAURKA\tH3B947_LATCH\nAURKA\tH9F795_MACMU\nAURKA\tH9GMT8_ANOCA\nAURKA\tI3MIA4_ICTTR\nAURKA\tJ9NYY2_CANLF\nAURKA\tK7G824_PELSI\nAURKA\tM3WUX0_FELCA\nAURKA\tM3XKI8_LATCH\nAURKA\tM3Y4C9_MUSPF\nAURKA\tQ28HZ5_XENTR\nAURKA\tQ2I6Y0_PIG\nAURKA\tQ3MHU2_RAT\nAURKA\tQ3TEY6_MOUSE\nAURKA\tQ5QPD1_HUMAN\nAURKA\tQ5QPD2_HUMAN\nAURKA\tQ5QPD4_HUMAN\nAURKA\tQ6DBZ4_DANRE\nAURKA\tQ7TNK2_MOUSE\nAURKA\tQ8BP87_MOUSE\nAURKA\tQ9VGF9_DROME\nAURKA\tT2MJJ8_HYDVU\nAURKA\tU3IAE4_ANAPP\nAURKA\tU3K8Q3_FICAL\nAURKA\tW5QAH3_SHEEP\nAURKA\tW8CEK8_CERCA\n','From\tTo\nDYRK1A\tA0A096NJ33_PAPAN\nDYRK1A\tA0A096NJ35_PAPAN\nDYRK1A\tA0A0D9R801_CHLSB\nDYRK1A\tA0A151NDQ0_ALLMI\nDYRK1A\tA0A1L8FPR3_XENLA\nDYRK1A\tA0A1S3AJH4_ERIEU\nDYRK1A\tA0A1S3AJJ0_ERIEU\nDYRK1A\tA0A1S3G2G8_DIPOR\nDYRK1A\tA0A1S3G4N1_DIPOR\nDYRK1A\tA0A1S3LP30_SALSA\nDYRK1A\tA0A1S3LP70_SALSA\nDYRK1A\tA0A1S3LPE8_SALSA\nDYRK1A\tA0A1U7QVD2_MESAU\nDYRK1A\tA0A1U7S7B2_ALLSI\nDYRK1A\tA0A1U7UMK0_TARSY\nDYRK1A\tA0A1U7UZQ6_TARSY\nDYRK1A\tA0A1U8C7W5_MESAU\nDYRK1A\tA0A1V4J309_PATFA\nDYRK1A\tA0A218US16_9PASE\nDYRK1A\tA0A250Y4X5_CASCN\nDYRK1A\tA0A286X9J3_CAVPO\nDYRK1A\tA0A287A073_PIG\nDYRK1A\tA0A287AIV4_PIG\nDYRK1A\tA0A287DDK1_ICTTR\nDYRK1A\tA0A2B4S3I6_STYPI\nDYRK1A\tA0A2D0SS82_ICTPU\nDYRK1A\tA0A2I0MSK1_COLLI\nDYRK1A\tA0A2I2UUE1_FELCA\nDYRK1A\tA0A2I3FS59_NOMLE\nDYRK1A\tA0A2I3MUD0_PAPAN\nDYRK1A\tA0A2I3T8W3_PANTR\nDYRK1A\tA0A2I3TH21_PANTR\nDYRK1A\tA0A2I3TNJ2_PANTR\nDYRK1A\tA0A2I4D8D7_9TELE\nDYRK1A\tA0A2K5D150_AOTNA\nDYRK1A\tA0A2K5D156_AOTNA\nDYRK1A\tA0A2K5D160_AOTNA\nDYRK1A\tA0A2K5D171_AOTNA\nDYRK1A\tA0A2K5KCG7_COLAP\nDYRK1A\tA0A2K5KCH2_COLAP\nDYRK1A\tA0A2K5KCH6_COLAP\nDYRK1A\tA0A2K5P1R4_CERAT\nDYRK1A\tA0A2K5P1R8_CERAT\nDYRK1A\tA0A2K5P1S9_CERAT\nDYRK1A\tA0A2K5P1W4_CERAT\nDYRK1A\tA0A2K5P1Z0_CERAT\nDYRK1A\tA0A2K5QAG7_CEBCA\nDYRK1A\tA0A2K5QAJ0_CEBCA\nDYRK1A\tA0A2K5QAM3_CEBCA\nDYRK1A\tA0A2K5VA44_MACFA\nDYRK1A\tA0A2K5VA66_MACFA\nDYRK1A\tA0A2K5VAB0_MACFA\nDYRK1A\tA0A2K5ZRQ1_MANLE\nDYRK1A\tA0A2K5ZRR7_MANLE\nDYRK1A\tA0A2K5ZRS4_MANLE\nDYRK1A\tA0A2K6B4V6_MACNE\nDYRK1A\tA0A2K6B4W0_MACNE\nDYRK1A\tA0A2K6B4X8_MACNE\nDYRK1A\tA0A2K6FLN2_PROCO\nDYRK1A\tA0A2K6FLN8_PROCO\nDYRK1A\tA0A2K6FLP5_PROCO\nDYRK1A\tA0A2K6N3H9_RHIBE\nDYRK1A\tA0A2K6N3I8_RHIBE\nDYRK1A\tA0A2K6N3R2_RHIBE\nDYRK1A\tA0A2K6NGQ6_RHIRO\nDYRK1A\tA0A2K6NGS1_RHIRO\nDYRK1A\tA0A2K6NGT7_RHIRO\nDYRK1A\tA0A2K6UGP0_SAIBB\nDYRK1A\tA0A2K6UGP9_SAIBB\nDYRK1A\tA0A2K6UGR6_SAIBB\nDYRK1A\tA0A2R8P0U5_CALJA\nDYRK1A\tA0A2R8Y443_HUMAN\nDYRK1A\tA0A2R8Y6I6_HUMAN\nDYRK1A\tA0A2R8Y6L5_HUMAN\nDYRK1A\tA0A2R8YDF3_HUMAN\nDYRK1A\tA0A2R8YEY4_HUMAN\nDYRK1A\tA0A2R9AN42_PANPA\nDYRK1A\tA0A2R9AN66_PANPA\nDYRK1A\tA0A2R9ANM7_PANPA\nDYRK1A\tA0A2R9AV40_PANPA\nDYRK1A\tA0A2U3WXB3_ODORO\nDYRK1A\tA0A2U3YTL6_LEPWE\nDYRK1A\tA0A2U3YTM4_LEPWE\nDYRK1A\tA0A2U3ZUR1_ODORO\nDYRK1A\tA0A2U4AAX8_TURTR\nDYRK1A\tA0A2U4AB21_TURTR\nDYRK1A\tA0A2U4AB35_TURTR\nDYRK1A\tA0A2U4AB40_TURTR\nDYRK1A\tA0A2U4AB97_TURTR\nDYRK1A\tA0A2U4ABA4_TURTR\nDYRK1A\tA0A2U4ABL4_TURTR\nDYRK1A\tA0A2U4ABL8_TURTR\nDYRK1A\tA0A2Y9EWY3_PHYMC\nDYRK1A\tA0A2Y9G6F4_NEOSC\nDYRK1A\tA0A2Y9LG22_DELLE\nDYRK1A\tA0A2Y9LG27_DELLE\nDYRK1A\tA0A2Y9LIM7_DELLE\nDYRK1A\tA0A2Y9LIM9_DELLE\nDYRK1A\tA0A2Y9LNG7_DELLE\nDYRK1A\tA0A2Y9LNH2_DELLE\nDYRK1A\tA0A2Y9LTG8_DELLE\nDYRK1A\tA0A2Y9SBL3_PHYMC\nDYRK1A\tA0A337RZ15_FELCA\nDYRK1A\tA0A340YEW8_LIPVE\nDYRK1A\tA0A341CGL3_NEOAA\nDYRK1A\tA0A341CH34_NEOAA\nDYRK1A\tA0A341CHA7_NEOAA\nDYRK1A\tA0A341CHB1_NEOAA\nDYRK1A\tA0A384A8Z7_BALAS\nDYRK1A\tA0A384A927_BALAS\nDYRK1A\tA0A384A932_BALAS\nDYRK1A\tA0A384CZ62_URSMA\nDYRK1A\tA0A384CZ96_URSMA\nDYRK1A\tA0A384CZA5_URSMA\nDYRK1A\tA0A3B3I4M3_ORYLA\nDYRK1A\tA0A3B3SXR1_9TELE\nDYRK1A\tA0A3B3SY66_9TELE\nDYRK1A\tA0A3Q0DHH9_TARSY\nDYRK1A\tA0A3Q0H8E5_ALLSI\nDYRK1A\tA0A3Q0HBM3_ALLSI\nDYRK1A\tA0A3Q1LNU2_BOVIN\nDYRK1A\tA0A3Q2I3Y0_HORSE\nDYRK1A\tA0A3Q2IAQ6_HORSE\nDYRK1A\tA0A3Q7PAK0_CALUR\nDYRK1A\tA0A3Q7QGC9_CALUR\nDYRK1A\tA0A3Q7RCN3_VULVU\nDYRK1A\tA0A3Q7SNL3_VULVU\nDYRK1A\tA0A3Q7W461_URSAR\nDYRK1A\tA0A3Q7W493_URSAR\nDYRK1A\tA0A3Q7WZ47_URSAR\nDYRK1A\tA0A452G609_CAPHI\nDYRK1A\tA0A452IDG4_9SAUR\nDYRK1A\tA0A452QD38_URSAM\nDYRK1A\tA0A452SXJ2_URSMA\nDYRK1A\tA0A455BXE2_PHYMC\nDYRK1A\tA0A4W2F2W2_BOBOX\nDYRK1A\tA0A4W2F2W7_BOBOX\nDYRK1A\tA0A4W2F2Z1_BOBOX\nDYRK1A\tA0A4W2G5A9_BOBOX\nDYRK1A\tA0A4W4FNM7_ELEEL\nDYRK1A\tA0A4W4FPR4_ELEEL\nDYRK1A\tA0A4W4FPS0_ELEEL\nDYRK1A\tA0A4W6FG04_LATCA\nDYRK1A\tA0A4W6FHV4_LATCA\nDYRK1A\tA0A4W6FIW5_LATCA\nDYRK1A\tA0A4X1TNT0_PIG\nDYRK1A\tA0A4X1TR90_PIG\nDYRK1A\tA0A4X1TRK8_PIG\nDYRK1A\tA0A4X2KBJ7_VOMUR\nDYRK1A\tA0A4Y2TK16_ARAVE\nDYRK1A\tA0A4Z2H1V0_9TELE\nDYRK1A\tA0A5B7CQ33_PORTR\nDYRK1A\tA0A5F4BQ19_CANLF\nDYRK1A\tA0A5F4C2U0_CANLF\nDYRK1A\tA0A5F4VRT2_CALJA\nDYRK1A\tA0A5F4VYG1_CALJA\nDYRK1A\tA0A5F5PVB4_HORSE\nDYRK1A\tA0A5F8G6H1_MONDO\nDYRK1A\tA0A5F8GQ47_MONDO\nDYRK1A\tA0A5F8H9S8_MONDO\nDYRK1A\tA0A5F9C6I3_RABIT\nDYRK1A\tA0A5F9CH10_RABIT\nDYRK1A\tA0A5G2QQP5_PIG\nDYRK1A\tA0A5G2R6S7_PIG\nDYRK1A\tA0A5G2R7M7_PIG\nDYRK1A\tA0A5G3K213_XENTR\nDYRK1A\tA1L1U2_DANRE\nDYRK1A\tA1L341_MOUSE\nDYRK1A\tA9C475_MOUSE\nDYRK1A\tA9C477_MOUSE\nDYRK1A\tA9C478_MOUSE\nDYRK1A\tB1H2N4_XENTR\nDYRK1A\tDYR1A_HUMAN\nDYRK1A\tDYR1A_MOUSE\nDYRK1A\tDYR1A_RABIT\nDYRK1A\tDYR1A_RAT\nDYRK1A\tDYR1A_XENLA\nDYRK1A\tDYR1A_XENTR\nDYRK1A\tE7EMI5_HUMAN\nDYRK1A\tF1N2J2_BOVIN\nDYRK1A\tF1P0S9_CHICK\nDYRK1A\tF1P9L4_CANLF\nDYRK1A\tF1SGV5_PIG\nDYRK1A\tF6QKT7_XENTR\nDYRK1A\tF6S1R4_HORSE\nDYRK1A\tF6S561_MONDO\nDYRK1A\tF6U6X3_MOUSE\nDYRK1A\tF6WIF6_ORNAN\nDYRK1A\tF6Y848_CANLF\nDYRK1A\tF7BM65_CALJA\nDYRK1A\tF7FBT2_MACMU\nDYRK1A\tF7FBT7_MACMU\nDYRK1A\tG1KII2_ANOCA\nDYRK1A\tG1MIS8_AILME\nDYRK1A\tG1NNY8_MELGA\nDYRK1A\tG1NVL6_MYOLU\nDYRK1A\tG1QXR7_NOMLE\nDYRK1A\tG1SPQ6_RABIT\nDYRK1A\tG3SME5_LOXAF\nDYRK1A\tG3TY11_LOXAF\nDYRK1A\tG3WFE4_SARHA\nDYRK1A\tH0X4E2_OTOGA\nDYRK1A\tH0Z3I7_TAEGU\nDYRK1A\tH2LLB3_ORYLA\nDYRK1A\tH2LLB6_ORYLA\nDYRK1A\tH2P346_PONAB\nDYRK1A\tH2QL08_PANTR\nDYRK1A\tH2SWR8_TAKRU\nDYRK1A\tH2SWR9_TAKRU\nDYRK1A\tH3AJD9_LATCH\nDYRK1A\tH3AJE0_LATCH\nDYRK1A\tH9Z608_MACMU\nDYRK1A\tI0FH67_MACMU\nDYRK1A\tI3L9V1_PIG\nDYRK1A\tI3MR02_ICTTR\nDYRK1A\tK7DQB0_PANTR\nDYRK1A\tK7FXM2_PELSI\nDYRK1A\tM3YAN5_MUSPF\nDYRK1A\tN0GVR9_HUMAN\nDYRK1A\tQ3USJ7_MOUSE\nDYRK1A\tQ61158_MOUSE\nDYRK1A\tQ7ZXV4_XENLA\nDYRK1A\tQ80ZK5_MOUSE\nDYRK1A\tQ8C774_MOUSE\nDYRK1A\tQ8K006_MOUSE\nDYRK1A\tQ9VX06_DROME\nDYRK1A\tT2ME21_HYDVU\nDYRK1A\tU3D765_CALJA\nDYRK1A\tU3E817_CALJA\nDYRK1A\tU3IW45_ANAPP\nDYRK1A\tU3JRF7_FICAL\nDYRK1A\tU3JRF8_FICAL\nDYRK1A\tV8PFW2_OPHHA\nDYRK1A\tW5PY89_SHEEP\nDYRK1A\tX2JFM3_DROME\n','From\tTo\nCDK2\tA0A024RB10_HUMAN\nCDK2\tA0A024RB77_HUMAN\nCDK2\tA0A061R8C0_9CHLO\nCDK2\tA0A061SMA7_9CHLO\nCDK2\tA0A096MYH8_PAPAN\nCDK2\tA0A0A1X1I0_ZEUCU\nCDK2\tA0A0B2W0G1_TOXCA\nCDK2\tA0A0C9R376_9HYME\nCDK2\tA0A0D9QZ04_CHLSB\nCDK2\tA0A0K2TL95_LEPSM\nCDK2\tA0A0S7ITG3_9TELE\nCDK2\tA0A0S7IVP4_9TELE\nCDK2\tA0A0V0S3P1_9BILA\nCDK2\tA0A0V0TII8_9BILA\nCDK2\tA0A0V0TVX0_9BILA\nCDK2\tA0A0V0VFD0_9BILA\nCDK2\tA0A0V0XVH7_TRIPS\nCDK2\tA0A0V0ZFR1_9BILA\nCDK2\tA0A0V0ZFR5_9BILA\nCDK2\tA0A0V1B9X8_TRISP\nCDK2\tA0A0V1BA46_TRISP\nCDK2\tA0A0V1CXU2_TRIBR\nCDK2\tA0A0V1EJA0_TRIPS\nCDK2\tA0A0V1EJT8_TRIPS\nCDK2\tA0A0V1FG35_TRIPS\nCDK2\tA0A0V1GVW6_9BILA\nCDK2\tA0A0V1ICQ6_TRIPS\nCDK2\tA0A0V1ICV4_TRIPS\nCDK2\tA0A0V1IDY9_TRIPS\nCDK2\tA0A0V1J500_TRIPS\nCDK2\tA0A0V1LND8_9BILA\nCDK2\tA0A0V1M3U6_9BILA\nCDK2\tA0A0V1NJH2_9BILA\nCDK2\tA0A151MIN2_ALLMI\nCDK2\tA0A1A7XBQ9_9TELE\nCDK2\tA0A1A7Y5U8_9TELE\nCDK2\tA0A1A7ZQH0_NOTFU\nCDK2\tA0A1A8CXW1_9TELE\nCDK2\tA0A1A8DG11_9TELE\nCDK2\tA0A1A8EXF6_9TELE\nCDK2\tA0A1A8HA13_9TELE\nCDK2\tA0A1A8J5H4_NOTKU\nCDK2\tA0A1A8MPG1_9TELE\nCDK2\tA0A1A8P5E1_9TELE\nCDK2\tA0A1J4J764_9EUKA\nCDK2\tA0A1J4JXW0_9EUKA\nCDK2\tA0A1J4KRU2_9EUKA\nCDK2\tA0A1S3FMH0_DIPOR\nCDK2\tA0A1S3FPI0_DIPOR\nCDK2\tA0A1S3WPE2_ERIEU\nCDK2\tA0A1U7SP26_ALLSI\nCDK2\tA0A1U7TQE9_TARSY\nCDK2\tA0A1U7U0H3_TARSY\nCDK2\tA0A1U7U8H5_TARSY\nCDK2\tA0A1U7UAC1_TARSY\nCDK2\tA0A1U8C8E6_MESAU\nCDK2\tA0A250YG64_CASCN\nCDK2\tA0A286XH18_CAVPO\nCDK2\tA0A286ZS27_PIG\nCDK2\tA0A287ARC7_PIG\nCDK2\tA0A287AXP1_PIG\nCDK2\tA0A287BB95_PIG\nCDK2\tA0A287BNU5_PIG\nCDK2\tA0A287D5T7_ICTTR\nCDK2\tA0A291S6U6_9HYME\nCDK2\tA0A2B4SSW6_STYPI\nCDK2\tA0A2B4SXK1_STYPI\nCDK2\tA0A2D0PKQ8_ICTPU\nCDK2\tA0A2I2UCH2_FELCA\nCDK2\tA0A2I2UFY5_FELCA\nCDK2\tA0A2I2V4K7_FELCA\nCDK2\tA0A2I3GPW6_NOMLE\nCDK2\tA0A2I3H0F0_NOMLE\nCDK2\tA0A2I3H535_NOMLE\nCDK2\tA0A2I3M965_PAPAN\nCDK2\tA0A2I3MV08_PAPAN\nCDK2\tA0A2I3N0J9_PAPAN\nCDK2\tA0A2I3RR15_PANTR\nCDK2\tA0A2I4AHH7_9TELE\nCDK2\tA0A2K5IH92_COLAP\nCDK2\tA0A2K5IHA2_COLAP\nCDK2\tA0A2K5IHD1_COLAP\nCDK2\tA0A2K5IHJ1_COLAP\nCDK2\tA0A2K5NT27_CERAT\nCDK2\tA0A2K5NT32_CERAT\nCDK2\tA0A2K5NTB1_CERAT\nCDK2\tA0A2K5NTH0_CERAT\nCDK2\tA0A2K5QXK5_CEBCA\nCDK2\tA0A2K5QXK8_CEBCA\nCDK2\tA0A2K5QXN6_CEBCA\nCDK2\tA0A2K5QXP1_CEBCA\nCDK2\tA0A2K5VPW1_MACFA\nCDK2\tA0A2K5VQ23_MACFA\nCDK2\tA0A2K5VQ72_MACFA\nCDK2\tA0A2K5XCP3_MANLE\nCDK2\tA0A2K5XCQ2_MANLE\nCDK2\tA0A2K5XCR3_MANLE\nCDK2\tA0A2K5XCR8_MANLE\nCDK2\tA0A2K6E4V8_MACNE\nCDK2\tA0A2K6E4W9_MACNE\nCDK2\tA0A2K6E501_MACNE\nCDK2\tA0A2K6E508_MACNE\nCDK2\tA0A2K6GY41_PROCO\nCDK2\tA0A2K6GY84_PROCO\nCDK2\tA0A2K6GYA0_PROCO\nCDK2\tA0A2K6GYA6_PROCO\nCDK2\tA0A2K6MM85_RHIBE\nCDK2\tA0A2K6MMA3_RHIBE\nCDK2\tA0A2K6MMD4_RHIBE\nCDK2\tA0A2K6MME8_RHIBE\nCDK2\tA0A2K6P9I2_RHIRO\nCDK2\tA0A2K6P9L8_RHIRO\nCDK2\tA0A2K6P9M3_RHIRO\nCDK2\tA0A2K6P9R7_RHIRO\nCDK2\tA0A2K6V5J0_SAIBB\nCDK2\tA0A2K6V5L6_SAIBB\nCDK2\tA0A2P8ZP43_BLAGE\nCDK2\tA0A2R8NAU3_CALJA\nCDK2\tA0A2R9B0D1_PANPA\nCDK2\tA0A2R9B4E4_PANPA\nCDK2\tA0A2R9B6W1_PANPA\nCDK2\tA0A2R9B6W4_PANPA\nCDK2\tA0A2U3W192_ODORO\nCDK2\tA0A2U3W1B4_ODORO\nCDK2\tA0A2U3W1B7_ODORO\nCDK2\tA0A2U3XBF9_LEPWE\nCDK2\tA0A2U3XBV5_LEPWE\nCDK2\tA0A2U3ZH13_ODORO\nCDK2\tA0A2U3ZH15_ODORO\nCDK2\tA0A2U4CQF2_TURTR\nCDK2\tA0A2Y9FRS2_PHYMC\nCDK2\tA0A2Y9FRU8_PHYMC\nCDK2\tA0A2Y9GUJ7_NEOSC\nCDK2\tA0A2Y9MYS3_DELLE\nCDK2\tA0A2Y9N0A8_DELLE\nCDK2\tA0A2Y9N3B1_DELLE\nCDK2\tA0A2Y9NEE7_DELLE\nCDK2\tA0A340Y2B2_LIPVE\nCDK2\tA0A340Y3G3_LIPVE\nCDK2\tA0A340Y630_LIPVE\nCDK2\tA0A340Y8M0_LIPVE\nCDK2\tA0A341AR07_NEOAA\nCDK2\tA0A341ARZ7_NEOAA\nCDK2\tA0A383ZWW0_BALAS\nCDK2\tA0A383ZXG1_BALAS\nCDK2\tA0A384DTF0_URSMA\nCDK2\tA0A384DTG0_URSMA\nCDK2\tA0A3B3H975_ORYLA\nCDK2\tA0A3B3HBM3_ORYLA\nCDK2\tA0A3B5KDF8_TAKRU\nCDK2\tA0A3B5MM41_9TELE\nCDK2\tA0A3Q0E4M5_TARSY\nCDK2\tA0A3Q2H2L0_HORSE\nCDK2\tA0A3Q7PZW1_CALUR\nCDK2\tA0A3Q7RF90_CALUR\nCDK2\tA0A3Q7SQ67_VULVU\nCDK2\tA0A3Q7TRE0_VULVU\nCDK2\tA0A3Q7V4A9_URSAR\nCDK2\tA0A3Q7X0F6_URSAR\nCDK2\tA0A419PMX5_CLOSI\nCDK2\tA0A452CFA3_BALAS\nCDK2\tA0A452DTW9_CAPHI\nCDK2\tA0A452GT32_9SAUR\nCDK2\tA0A4E0RJD8_9HYME\nCDK2\tA0A4W2F3T5_BOBOX\nCDK2\tA0A4W2FYG1_BOBOX\nCDK2\tA0A4W2HT73_BOBOX\nCDK2\tA0A4W2HT93_BOBOX\nCDK2\tA0A4W3IX19_CALMI\nCDK2\tA0A4W3JAN5_CALMI\nCDK2\tA0A4W3JT71_CALMI\nCDK2\tA0A4X1W3W2_PIG\nCDK2\tA0A4X1W9B6_PIG\nCDK2\tA0A4X1W9I0_PIG\nCDK2\tA0A4X1W9S8_PIG\nCDK2\tA0A4X2M9L3_VOMUR\nCDK2\tA0A4X2MB74_VOMUR\nCDK2\tA0A4X2MBS7_VOMUR\nCDK2\tA0A4X2MBT1_VOMUR\nCDK2\tA0A4Y2FWQ1_ARAVE\nCDK2\tA0A4Z2ITZ4_9TELE\nCDK2\tA0A559MK53_9HELO\nCDK2\tA0A5B6UR42_9ROSI\nCDK2\tA0A5B6VDS5_9ROSI\nCDK2\tA0A5B7DBH3_PORTR\nCDK2\tA0A5F4BUT0_CANLF\nCDK2\tA0A5F4BX87_CANLF\nCDK2\tA0A5F4D4H3_CANLF\nCDK2\tA0A5F5PKM3_HORSE\nCDK2\tA0A5F5Q1D4_HORSE\nCDK2\tA0A5F9C2I2_RABIT\nCDK2\tA0A5F9CC98_RABIT\nCDK2\tA0A5F9CMA3_RABIT\nCDK2\tA0A5G2QRI3_PIG\nCDK2\tA0A5G3HIN1_XENTR\nCDK2\tA0A5G3HIT7_XENTR\nCDK2\tA0A5G3HIV4_XENTR\nCDK2\tA0A5G3HIZ5_XENTR\nCDK2\tA0A5G3HJZ4_XENTR\nCDK2\tA0A5G3J655_XENTR\nCDK2\tA0A5G3J656_XENTR\nCDK2\tA0A5G3J758_XENTR\nCDK2\tA0A5G3LA66_XENTR\nCDK2\tA0A5G3LAS6_XENTR\nCDK2\tA0A5G3LC46_XENTR\nCDK2\tA0MSV8_CAPHI\nCDK2\tA2IAR9_CHICK\nCDK2\tA8UKE6_RABIT\nCDK2\tA9UYT0_MONBE\nCDK2\tB5X1T4_SALSA\nCDK2\tB7U171_SHEEP\nCDK2\tC1C4M4_LITCT\nCDK2\tC7G1W0_PATPE\nCDK2\tCDK2_BOVIN\nCDK2\tCDK2_CARAU\nCDK2\tCDK2_CRIGR\nCDK2\tCDK2_DROME\nCDK2\tCDK2_HUMAN\nCDK2\tCDK2_MESAU\nCDK2\tCDK2_MOUSE\nCDK2\tCDK2_RAT\nCDK2\tCDK2_XENLA\nCDK2\tD2H336_AILME\nCDK2\tD3PJA7_LEPSM\nCDK2\tE1JIR3_DROME\nCDK2\tE2GHU0_BISBE\nCDK2\tE2QW70_CANLF\nCDK2\tE4XXK6_OIKDI\nCDK2\tE7ESI2_HUMAN\nCDK2\tE9H705_DAPPU\nCDK2\tF0YED1_AURAN\nCDK2\tF1SPH6_PIG\nCDK2\tF6US96_HORSE\nCDK2\tF7AJ05_XENTR\nCDK2\tF7B5H6_CALJA\nCDK2\tF7BWH8_CALJA\nCDK2\tF7CB13_CALJA\nCDK2\tF7EQG8_XENTR\nCDK2\tG1P317_MYOLU\nCDK2\tG1S737_NOMLE\nCDK2\tG1U4R9_RABIT\nCDK2\tG3TH75_LOXAF\nCDK2\tG3TWQ8_LOXAF\nCDK2\tG3V317_HUMAN\nCDK2\tG3V5T9_HUMAN\nCDK2\tG3W161_SARHA\nCDK2\tG5BB95_HETGA\nCDK2\tG7N7A7_MACMU\nCDK2\tG7PIG0_MACFA\nCDK2\tG8GMT6_CAVPO\nCDK2\tH0VFS0_CAVPO\nCDK2\tH0WVH2_OTOGA\nCDK2\tH2MJC4_ORYLA\nCDK2\tH2NHM8_PONAB\nCDK2\tH2Q670_PANTR\nCDK2\tH2Q671_PANTR\nCDK2\tH2RH70_PANTR\nCDK2\tH2VDI9_TAKRU\nCDK2\tH3BI57_LATCH\nCDK2\tH6VMF4_BOMMO\nCDK2\tH9GA76_ANOCA\nCDK2\tI3KST3_ORENI\nCDK2\tI3LWC0_ICTTR\nCDK2\tI3UJK1_MACRS\nCDK2\tK0KDG7_WICCF\nCDK2\tK7F554_PELSI\nCDK2\tM3WJU6_FELCA\nCDK2\tM3XPD8_MUSPF\nCDK2\tM4SL30_9BILA\nCDK2\tM4SML8_9BILA\nCDK2\tO46161_SPHGR\nCDK2\tO55077_CRIGR\nCDK2\tQ3U307_MOUSE\nCDK2\tQ3U6X7_MOUSE\nCDK2\tQ3UGB9_MOUSE\nCDK2\tQ60545_MESAU\nCDK2\tQ66IH7_XENTR\nCDK2\tQ6IRQ7_XENLA\nCDK2\tQ6P751_RAT\nCDK2\tQ78ZD4_CRIGR\nCDK2\tQ7ZWB1_DANRE\nCDK2\tQ8MXA6_AXICO\nCDK2\tQ8SPZ2_MACMU\nCDK2\tQ9PSU0_XENLA\nCDK2\tQ9XYV1_PARTE\nCDK2\tT2HWV6_ANGJA\nCDK2\tT2MG16_HYDVU\nCDK2\tU6DE15_NEOVI\nCDK2\tV8PHU8_OPHHA\nCDK2\tW5M961_LEPOC\nCDK2\tW5PLX3_SHEEP\nCDK2\tW6QY44_PSEP5\n','From\tTo\nCDK9\tA0A024R880_HUMAN\nCDK9\tA0A034VVI4_BACDO\nCDK9\tA0A088ANS9_APIME\nCDK9\tA0A096P1Z7_PAPAN\nCDK9\tA0A0A1XJV8_ZEUCU\nCDK9\tA0A0A9XX78_LYGHE\nCDK9\tA0A0B7FRT5_THACB\nCDK9\tA0A0B8RZP0_PIG\nCDK9\tA0A0C9RBK7_9HYME\nCDK9\tA0A0D9RMW9_CHLSB\nCDK9\tA0A0K2U5H4_LEPSM\nCDK9\tA0A0S7LLS4_9TELE\nCDK9\tA0A0S7LLT9_9TELE\nCDK9\tA0A0S7LLU8_9TELE\nCDK9\tA0A0V0J614_SCHSO\nCDK9\tA0A0V0RIJ8_9BILA\nCDK9\tA0A0V0RU52_9BILA\nCDK9\tA0A0V0RU64_9BILA\nCDK9\tA0A0V0RU82_9BILA\nCDK9\tA0A0V0RU84_9BILA\nCDK9\tA0A0V0RUA8_9BILA\nCDK9\tA0A0V0RUX3_9BILA\nCDK9\tA0A0V0T9Q9_9BILA\nCDK9\tA0A0V0U4H8_9BILA\nCDK9\tA0A0V0U752_9BILA\nCDK9\tA0A0V0U769_9BILA\nCDK9\tA0A0V0U7C5_9BILA\nCDK9\tA0A0V0U7E9_9BILA\nCDK9\tA0A0V0U7S0_9BILA\nCDK9\tA0A0V0UVN9_9BILA\nCDK9\tA0A0V0UVP3_9BILA\nCDK9\tA0A0V0UVR6_9BILA\nCDK9\tA0A0V0UVR9_9BILA\nCDK9\tA0A0V0UWL0_9BILA\nCDK9\tA0A0V0X027_9BILA\nCDK9\tA0A0V0X8C2_9BILA\nCDK9\tA0A0V0X8P5_9BILA\nCDK9\tA0A0V0X8R8_9BILA\nCDK9\tA0A0V0X8U7_9BILA\nCDK9\tA0A0V0X910_9BILA\nCDK9\tA0A0V0X9I5_9BILA\nCDK9\tA0A0V0YHU7_TRIPS\nCDK9\tA0A0V0ZJ47_9BILA\nCDK9\tA0A0V0ZW74_9BILA\nCDK9\tA0A0V0ZWH1_9BILA\nCDK9\tA0A0V0ZXK4_9BILA\nCDK9\tA0A0V0ZXY3_9BILA\nCDK9\tA0A0V1BI08_TRISP\nCDK9\tA0A0V1BI14_TRISP\nCDK9\tA0A0V1BI71_TRISP\nCDK9\tA0A0V1BI90_TRISP\nCDK9\tA0A0V1BIC1_TRISP\nCDK9\tA0A0V1D173_TRIBR\nCDK9\tA0A0V1D1C2_TRIBR\nCDK9\tA0A0V1D1F1_TRIBR\nCDK9\tA0A0V1D1J3_TRIBR\nCDK9\tA0A0V1D1L3_TRIBR\nCDK9\tA0A0V1EBV6_TRIPS\nCDK9\tA0A0V1EC99_TRIPS\nCDK9\tA0A0V1ECB0_TRIPS\nCDK9\tA0A0V1FQF8_TRIPS\nCDK9\tA0A0V1HC60_9BILA\nCDK9\tA0A0V1I4Q4_9BILA\nCDK9\tA0A0V1ICH8_TRIPS\nCDK9\tA0A0V1ILD2_TRIPS\nCDK9\tA0A0V1ILE0_TRIPS\nCDK9\tA0A0V1ILN9_TRIPS\nCDK9\tA0A0V1ILW8_TRIPS\nCDK9\tA0A0V1IY78_TRIPS\nCDK9\tA0A0V1L2R3_9BILA\nCDK9\tA0A0V1LBA0_9BILA\nCDK9\tA0A0V1MUH1_9BILA\nCDK9\tA0A0V1MUJ4_9BILA\nCDK9\tA0A0V1MUX4_9BILA\nCDK9\tA0A0V1MV53_9BILA\nCDK9\tA0A0V1MVC8_9BILA\nCDK9\tA0A0V1P2G4_9BILA\nCDK9\tA0A0V1P2M7_9BILA\nCDK9\tA0A0V1P2Q6_9BILA\nCDK9\tA0A0V1P2S1_9BILA\nCDK9\tA0A0V1P2S5_9BILA\nCDK9\tA0A0V1P753_9BILA\nCDK9\tA0A0X3NHY4_SCHSO\nCDK9\tA0A0X3PNL4_SCHSO\nCDK9\tA0A0X3Q2V5_SCHSO\nCDK9\tA0A146LQA7_LYGHE\nCDK9\tA0A151MFB8_ALLMI\nCDK9\tA0A1A7W7B2_9TELE\nCDK9\tA0A1A8BS66_9TELE\nCDK9\tA0A1A8GTU2_9TELE\nCDK9\tA0A1A8IGU2_NOTKU\nCDK9\tA0A1A8K8P7_NOTKU\nCDK9\tA0A1A8QBK0_9TELE\nCDK9\tA0A1A8RAD0_9TELE\nCDK9\tA0A1A8VFS4_NOTFU\nCDK9\tA0A1C7MNF1_GRIFR\nCDK9\tA0A1C7ND36_9FUNG\nCDK9\tA0A1S3A3W0_ERIEU\nCDK9\tA0A1S3FLT2_DIPOR\nCDK9\tA0A1S3M6H3_SALSA\nCDK9\tA0A1U7R4Z0_MESAU\nCDK9\tA0A1U7T204_TARSY\nCDK9\tA0A1U8DD56_ALLSI\nCDK9\tA0A1V4J726_PATFA\nCDK9\tA0A218V1L0_9PASE\nCDK9\tA0A250YD91_CASCN\nCDK9\tA0A2B4S7Q5_STYPI\nCDK9\tA0A2I0LHG2_COLLI\nCDK9\tA0A2I2U8H4_FELCA\nCDK9\tA0A2I4CST4_9TELE\nCDK9\tA0A2K5IP49_COLAP\nCDK9\tA0A2K5NJJ6_CERAT\nCDK9\tA0A2K5WDZ1_MACFA\nCDK9\tA0A2K5XIA2_MANLE\nCDK9\tA0A2K6D689_MACNE\nCDK9\tA0A2K6GC13_PROCO\nCDK9\tA0A2K6M3Z3_RHIBE\nCDK9\tA0A2K6R0R0_RHIRO\nCDK9\tA0A2P8Y570_BLAGE\nCDK9\tA0A2R9B5G9_PANPA\nCDK9\tA0A2S2PR41_SCHGA\nCDK9\tA0A2U3V0Q2_TURTR\nCDK9\tA0A2U3VDT7_ODORO\nCDK9\tA0A2U3YKG2_LEPWE\nCDK9\tA0A2U4CHC6_TURTR\nCDK9\tA0A2Y9EFU9_PHYMC\nCDK9\tA0A2Y9HB82_NEOSC\nCDK9\tA0A2Y9NHG1_DELLE\nCDK9\tA0A2Y9NK07_DELLE\nCDK9\tA0A340YDP8_LIPVE\nCDK9\tA0A341CUT8_NEOAA\nCDK9\tA0A369KI12_HYPMA\nCDK9\tA0A384B4X9_BALAS\nCDK9\tA0A384CQP0_URSMA\nCDK9\tA0A3Q2HNR9_HORSE\nCDK9\tA0A3Q7QBV8_CALUR\nCDK9\tA0A3Q7T6T9_VULVU\nCDK9\tA0A3Q7W9S9_URSAR\nCDK9\tA0A3R7GNE5_CLOSI\nCDK9\tA0A3R7HC27_CLOSI\nCDK9\tA0A452FZI9_CAPHI\nCDK9\tA0A452HPQ5_9SAUR\nCDK9\tA0A452J859_CHICK\nCDK9\tA0A452SBL8_URSAM\nCDK9\tA0A452SYT1_URSMA\nCDK9\tA0A452SYU3_URSMA\nCDK9\tA0A493TJ88_ANAPP\nCDK9\tA0A493TRT1_ANAPP\nCDK9\tA0A4C1T7G6_9NEOP\nCDK9\tA0A4W2G140_BOBOX\nCDK9\tA0A4X1U7M2_PIG\nCDK9\tA0A4X2LH49_VOMUR\nCDK9\tA0A4Y2J1G5_ARAVE\nCDK9\tA0A507QPZ4_MONPU\nCDK9\tA0A5K1U9Z8_CALJA\nCDK9\tA0A5K1VI64_PIG\nCDK9\tB0R021_MOUSE\nCDK9\tB5BU53_HUMAN\nCDK9\tB5X0Q0_SALSA\nCDK9\tC9E1C9_PIG\nCDK9\tCDK9_BOVIN\nCDK9\tCDK9_CHICK\nCDK9\tCDK9_HUMAN\nCDK9\tCDK9_MOUSE\nCDK9\tCDK9_RAT\nCDK9\tCDK9_SCHPO\nCDK9\tCDK9_XENTR\nCDK9\tD6RGN3_MOUSE\nCDK9\tD6RHP4_MOUSE\nCDK9\tF6V4K1_HORSE\nCDK9\tF6XLQ3_ORNAN\nCDK9\tG1M948_AILME\nCDK9\tG1MXB5_MELGA\nCDK9\tG1S6U6_NOMLE\nCDK9\tG3THQ7_LOXAF\nCDK9\tG3UQR4_MELGA\nCDK9\tG5BLG4_HETGA\nCDK9\tG9MEC3_HYPVG\nCDK9\tG9NF57_HYPAI\nCDK9\tH0VGQ5_CAVPO\nCDK9\tH0WXQ5_OTOGA\nCDK9\tH0YZG4_TAEGU\nCDK9\tH2MVN9_ORYLA\nCDK9\tH2QXX8_PANTR\nCDK9\tH2TD26_TAKRU\nCDK9\tH3BBX6_LATCH\nCDK9\tH9FNK6_MACMU\nCDK9\tH9GBZ2_ANOCA\nCDK9\tI3K416_ORENI\nCDK9\tI3MBB8_ICTTR\nCDK9\tJ9NWT0_CANLF\nCDK9\tK7E4Z9_MONDO\nCDK9\tK7G056_PELSI\nCDK9\tM3YQG9_MUSPF\nCDK9\tM4SL35_9BILA\nCDK9\tM4SL60_9BILA\nCDK9\tM5CA92_THACB\nCDK9\tO17432_DROME\nCDK9\tQ2PQN8_GLOMM\nCDK9\tQ7SXE8_DANRE\nCDK9\tQ7T3L5_DANRE\nCDK9\tT2MFX2_HYDVU\nCDK9\tU3DPZ7_CALJA\nCDK9\tU3JF41_FICAL\nCDK9\tU6DM45_NEOVI\nCDK9\tV5GP23_ANOGL\nCDK9\tW5PPL8_SHEEP\nCDK9\tW5UIF0_ICTPU\nCDK9\tW8BDC5_CERCA\nCDK9\tX6RE90_HUMAN\n','From\tTo\nCLK1\tA0A060S047_PLARE\nCLK1\tA0A087WVK0_HUMAN\nCLK1\tA0A096MV69_PAPAN\nCLK1\tA0A0D9REN9_CHLSB\nCLK1\tA0A113SI58_PLABE\nCLK1\tA0A1A8WAW2_PLAMA\nCLK1\tA0A1C3KWA4_9APIC\nCLK1\tA0A1C3L1Q3_PLAMA\nCLK1\tA0A1D3TLL3_9APIC\nCLK1\tA0A1J1GQT2_PLAGA\nCLK1\tA0A1J1H9I9_PLARL\nCLK1\tA0A1J4JAP8_9EUKA\nCLK1\tA0A1Q9D7L4_SYMMI\nCLK1\tA0A1Q9DST5_SYMMI\nCLK1\tA0A1Q9EI45_SYMMI\nCLK1\tA0A1S3EL76_DIPOR\nCLK1\tA0A1S3ENR3_DIPOR\nCLK1\tA0A1S3W7Q7_ERIEU\nCLK1\tA0A1S3W877_ERIEU\nCLK1\tA0A1U7QEL6_MESAU\nCLK1\tA0A1U7REC4_ALLSI\nCLK1\tA0A1U7UJT8_TARSY\nCLK1\tA0A1V4JVQ3_PATFA\nCLK1\tA0A250XX58_CASCN\nCLK1\tA0A2I0LQB9_COLLI\nCLK1\tA0A2I0LQC3_COLLI\nCLK1\tA0A2I0LQC5_COLLI\nCLK1\tA0A2I3GPP1_NOMLE\nCLK1\tA0A2I3H9B3_NOMLE\nCLK1\tA0A2I3MAL6_PAPAN\nCLK1\tA0A2I3MPR2_PAPAN\nCLK1\tA0A2I3NG66_PAPAN\nCLK1\tA0A2I3T987_PANTR\nCLK1\tA0A2K5DCC7_AOTNA\nCLK1\tA0A2K5DCD1_AOTNA\nCLK1\tA0A2K5DCD4_AOTNA\nCLK1\tA0A2K5I975_COLAP\nCLK1\tA0A2K5I9L5_COLAP\nCLK1\tA0A2K5LKS8_CERAT\nCLK1\tA0A2K5LKX9_CERAT\nCLK1\tA0A2K5LKZ1_CERAT\nCLK1\tA0A2K5LL15_CERAT\nCLK1\tA0A2K5PV00_CEBCA\nCLK1\tA0A2K5PV06_CEBCA\nCLK1\tA0A2K5PV16_CEBCA\nCLK1\tA0A2K5Y6K4_MANLE\nCLK1\tA0A2K5Y6L2_MANLE\nCLK1\tA0A2K5Y6M5_MANLE\nCLK1\tA0A2K5Y6Q3_MANLE\nCLK1\tA0A2K6BSQ4_MACNE\nCLK1\tA0A2K6BSR5_MACNE\nCLK1\tA0A2K6BSS8_MACNE\nCLK1\tA0A2K6BSS9_MACNE\nCLK1\tA0A2K6BSX4_MACNE\nCLK1\tA0A2K6FGP6_PROCO\nCLK1\tA0A2K6FGQ1_PROCO\nCLK1\tA0A2K6FGR7_PROCO\nCLK1\tA0A2K6M242_RHIBE\nCLK1\tA0A2K6M245_RHIBE\nCLK1\tA0A2K6M257_RHIBE\nCLK1\tA0A2K6Q239_RHIRO\nCLK1\tA0A2K6Q241_RHIRO\nCLK1\tA0A2K6Q242_RHIRO\nCLK1\tA0A2K6UFZ8_SAIBB\nCLK1\tA0A2K6UG02_SAIBB\nCLK1\tA0A2K6UG27_SAIBB\nCLK1\tA0A2R9BDX4_PANPA\nCLK1\tA0A2R9BF59_PANPA\nCLK1\tA0A2U3V164_TURTR\nCLK1\tA0A2U3VFP1_ODORO\nCLK1\tA0A2U3YQH1_LEPWE\nCLK1\tA0A2U4BNE3_TURTR\nCLK1\tA0A2Y9ESI5_PHYMC\nCLK1\tA0A2Y9I4Q6_NEOSC\nCLK1\tA0A2Y9IBZ9_NEOSC\nCLK1\tA0A2Y9MLD5_DELLE\nCLK1\tA0A2Y9N1B1_DELLE\nCLK1\tA0A2Y9SNM5_PHYMC\nCLK1\tA0A337SR13_FELCA\nCLK1\tA0A340YA02_LIPVE\nCLK1\tA0A341CEN7_NEOAA\nCLK1\tA0A341CEW2_NEOAA\nCLK1\tA0A384AT51_BALAS\nCLK1\tA0A384BTI9_URSMA\nCLK1\tA0A384BTS7_URSMA\nCLK1\tA0A3Q0CRA6_MESAU\nCLK1\tA0A3Q0GIM6_ALLSI\nCLK1\tA0A3Q1LX76_BOVIN\nCLK1\tA0A3Q7QSP6_CALUR\nCLK1\tA0A3Q7R509_CALUR\nCLK1\tA0A3Q7SRP6_VULVU\nCLK1\tA0A3Q7V7B4_URSAR\nCLK1\tA0A452CN33_BALAS\nCLK1\tA0A452F4V5_CAPHI\nCLK1\tA0A452HFY8_9SAUR\nCLK1\tA0A452R6E9_URSAM\nCLK1\tA0A4W2IRB5_BOBOX\nCLK1\tA0A4W2ITW8_BOBOX\nCLK1\tA0A4X1VBB5_PIG\nCLK1\tA0A4X2KQS4_VOMUR\nCLK1\tA0A5F4CL45_CANLF\nCLK1\tA0A5F4DKQ3_CANLF\nCLK1\tA0A5G2RKY9_PIG\nCLK1\tA0A5G3HFT0_XENTR\nCLK1\tA4FVF2_XENLA\nCLK1\tA6QPK8_BOVIN\nCLK1\tATG1_COLLN\nCLK1\tB1A9K1_PIG\nCLK1\tB1A9K3_PIG\nCLK1\tB8ZZR0_HUMAN\nCLK1\tCLK1_HUMAN\nCLK1\tCLK1_MOUSE\nCLK1\tD2H2P8_AILME\nCLK1\tD4ADG3_RAT\nCLK1\tF1N596_BOVIN\nCLK1\tF4YD06_BUBBU\nCLK1\tF6Q9Y8_PIG\nCLK1\tF6UY30_CANLF\nCLK1\tF6V522_HORSE\nCLK1\tF6YF59_ORNAN\nCLK1\tF6ZMR6_XENTR\nCLK1\tF7ATK6_CALJA\nCLK1\tF7B4G9_MACMU\nCLK1\tF7B4I0_MACMU\nCLK1\tF7BXF2_MONDO\nCLK1\tF7DPC2_XENTR\nCLK1\tF7H430_MACMU\nCLK1\tF8WBF5_HUMAN\nCLK1\tG1DGG2_CAPHI\nCLK1\tG1KTF0_ANOCA\nCLK1\tG1LH16_AILME\nCLK1\tG1N6N9_MELGA\nCLK1\tG1NT33_MYOLU\nCLK1\tG1SZR1_RABIT\nCLK1\tG3T4J6_LOXAF\nCLK1\tG3WGP6_SARHA\nCLK1\tH0VWA7_CAVPO\nCLK1\tH0X6M8_OTOGA\nCLK1\tH2P889_PONAB\nCLK1\tH2QJ82_PANTR\nCLK1\tH2ZZJ0_LATCH\nCLK1\tH9FPQ6_MACMU\nCLK1\tI2AWF7_COCLU\nCLK1\tI2CWF1_MACMU\nCLK1\tI3MDT4_ICTTR\nCLK1\tJ9NZK4_CANLF\nCLK1\tK7F7I1_PELSI\nCLK1\tL7MRL1_HORSE\nCLK1\tL7MSD7_HORSE\nCLK1\tM3XM98_MUSPF\nCLK1\tQ28EQ6_XENTR\nCLK1\tQ3UKA7_MOUSE\nCLK1\tR4GGT7_CHICK\nCLK1\tRCK2_YEAST\nCLK1\tU3J8U3_ANAPP\nCLK1\tV8NWN1_OPHHA\nCLK1\tW5Q4X3_SHEEP\n','From\tTo\nGSK3B\tA0A024AH73_9NEOP\nGSK3B\tA0A024AH91_9NEOP\nGSK3B\tA0A024AHA0_9NEOP\nGSK3B\tA0A024AHA6_9NEOP\nGSK3B\tA0A024AHB3_9NEOP\nGSK3B\tA0A024AHB6_9NEOP\nGSK3B\tA0A024AHK5_9NEOP\nGSK3B\tA0A024AHM5_9NEOP\nGSK3B\tA0A024AHN6_9NEOP\nGSK3B\tA0A024AHP1_9NEOP\nGSK3B\tA0A024AHP6_9NEOP\nGSK3B\tA0A024AHQ1_9NEOP\nGSK3B\tA0A024AHR8_9NEOP\nGSK3B\tA0A024AHT0_9NEOP\nGSK3B\tA0A024AHT8_9NEOP\nGSK3B\tA0A024AHU4_9NEOP\nGSK3B\tA0A024AHU6_9NEOP\nGSK3B\tA0A024AHV1_9NEOP\nGSK3B\tA0A024AI59_9NEOP\nGSK3B\tA0A024AI69_9NEOP\nGSK3B\tA0A024AI73_9NEOP\nGSK3B\tA0A024AI78_9NEOP\nGSK3B\tA0A024AI82_9NEOP\nGSK3B\tA0A024AI86_9NEOP\nGSK3B\tA0A024AIR7_9NEOP\nGSK3B\tA0A024AIS7_9NEOP\nGSK3B\tA0A024AIT0_9NEOP\nGSK3B\tA0A024AIT5_9NEOP\nGSK3B\tA0A024AIT9_9NEOP\nGSK3B\tA0A061R5S0_9CHLO\nGSK3B\tA0A061SD45_9CHLO\nGSK3B\tA0A096MZ43_PAPAN\nGSK3B\tA0A097P8R4_CAPHI\nGSK3B\tA0A0B2UT01_TOXCA\nGSK3B\tA0A0B2VQ82_TOXCA\nGSK3B\tA0A0B2VR93_TOXCA\nGSK3B\tA0A0B8S076_PIG\nGSK3B\tA0A0C9QU91_9HYME\nGSK3B\tA0A0C9R1V4_9HYME\nGSK3B\tA0A0D9R447_CHLSB\nGSK3B\tA0A0G2JSH4_RAT\nGSK3B\tA0A0G2KB98_RAT\nGSK3B\tA0A0P6JAL8_HETGA\nGSK3B\tA0A0S1LIA1_LARCR\nGSK3B\tA0A0S2GMD7_SCOMX\nGSK3B\tA0A0S7HBF6_9TELE\nGSK3B\tA0A0S7HBH1_9TELE\nGSK3B\tA0A0S7HGU9_9TELE\nGSK3B\tA0A0S7IG10_9TELE\nGSK3B\tA0A0V0SFM6_9BILA\nGSK3B\tA0A0V0SII0_9BILA\nGSK3B\tA0A0V0TG70_9BILA\nGSK3B\tA0A0V0TG77_9BILA\nGSK3B\tA0A0V0UIB0_9BILA\nGSK3B\tA0A0V0VUB4_9BILA\nGSK3B\tA0A0V0VUM5_9BILA\nGSK3B\tA0A0V0VVL3_9BILA\nGSK3B\tA0A0V0WI19_9BILA\nGSK3B\tA0A0V0WIB4_9BILA\nGSK3B\tA0A0V0X3T8_9BILA\nGSK3B\tA0A0V0YI65_TRIPS\nGSK3B\tA0A0V0ZX03_9BILA\nGSK3B\tA0A0V1BGP2_TRISP\nGSK3B\tA0A0V1BP46_TRISP\nGSK3B\tA0A0V1DI40_TRIBR\nGSK3B\tA0A0V1G2N2_TRIPS\nGSK3B\tA0A0V1H5A8_9BILA\nGSK3B\tA0A0V1I9N9_9BILA\nGSK3B\tA0A0V1JA92_TRIPS\nGSK3B\tA0A0V1K9U4_TRIPS\nGSK3B\tA0A0V1LFX8_9BILA\nGSK3B\tA0A0V1LL56_9BILA\nGSK3B\tA0A0V1MJ95_9BILA\nGSK3B\tA0A0V1PI96_9BILA\nGSK3B\tA0A0V1PJ64_9BILA\nGSK3B\tA0A0X3NVV7_SCHSO\nGSK3B\tA0A151N132_ALLMI\nGSK3B\tA0A1A7XQY8_9TELE\nGSK3B\tA0A1A7Y620_9TELE\nGSK3B\tA0A1A7ZZ07_NOTFU\nGSK3B\tA0A1A8BWD4_9TELE\nGSK3B\tA0A1A8E3H6_9TELE\nGSK3B\tA0A1A8EQA4_9TELE\nGSK3B\tA0A1A8HAL4_9TELE\nGSK3B\tA0A1A8JHP1_NOTKU\nGSK3B\tA0A1A8K3Z2_NOTKU\nGSK3B\tA0A1A8L7U5_9TELE\nGSK3B\tA0A1A8N521_9TELE\nGSK3B\tA0A1A8NC31_9TELE\nGSK3B\tA0A1A8R6F6_9TELE\nGSK3B\tA0A1A8UW18_NOTFU\nGSK3B\tA0A1C7NEK1_9FUNG\nGSK3B\tA0A1J4JR56_9EUKA\nGSK3B\tA0A1S3AAY2_ERIEU\nGSK3B\tA0A1S3AB56_ERIEU\nGSK3B\tA0A1S3FF26_DIPOR\nGSK3B\tA0A1S3FFR6_DIPOR\nGSK3B\tA0A1S3NY87_SALSA\nGSK3B\tA0A1S3NY90_SALSA\nGSK3B\tA0A1S3WML7_ERIEU\nGSK3B\tA0A1S3WMU6_ERIEU\nGSK3B\tA0A1U7S8A3_ALLSI\nGSK3B\tA0A1U7SIZ8_ALLSI\nGSK3B\tA0A1U7SU00_ALLSI\nGSK3B\tA0A1U8BRN3_MESAU\nGSK3B\tA0A1V4JB40_PATFA\nGSK3B\tA0A1V4JBC6_PATFA\nGSK3B\tA0A218VDR1_9PASE\nGSK3B\tA0A250Y0J1_CASCN\nGSK3B\tA0A286RXX4_PROCL\nGSK3B\tA0A287AQ89_PIG\nGSK3B\tA0A289ZBT8_SINCH\nGSK3B\tA0A2B4SZ91_STYPI\nGSK3B\tA0A2D0R3I2_ICTPU\nGSK3B\tA0A2D0R506_ICTPU\nGSK3B\tA0A2I0MLE2_COLLI\nGSK3B\tA0A2I0MLE4_COLLI\nGSK3B\tA0A2I2U7S7_FELCA\nGSK3B\tA0A2I3G294_NOMLE\nGSK3B\tA0A2I3GEZ5_NOMLE\nGSK3B\tA0A2I3HG71_NOMLE\nGSK3B\tA0A2I3HP19_NOMLE\nGSK3B\tA0A2I3LKM7_PAPAN\nGSK3B\tA0A2I3LTA9_PAPAN\nGSK3B\tA0A2I3LTM8_PAPAN\nGSK3B\tA0A2I3LYB2_PAPAN\nGSK3B\tA0A2I3M128_PAPAN\nGSK3B\tA0A2I3N172_PAPAN\nGSK3B\tA0A2I3S1U3_PANTR\nGSK3B\tA0A2I3SXI2_PANTR\nGSK3B\tA0A2I4CRH6_9TELE\nGSK3B\tA0A2J7R578_9NEOP\nGSK3B\tA0A2K5D775_AOTNA\nGSK3B\tA0A2K5D779_AOTNA\nGSK3B\tA0A2K5D780_AOTNA\nGSK3B\tA0A2K5D784_AOTNA\nGSK3B\tA0A2K5D7C6_AOTNA\nGSK3B\tA0A2K5D7D6_AOTNA\nGSK3B\tA0A2K5I346_COLAP\nGSK3B\tA0A2K5I3H2_COLAP\nGSK3B\tA0A2K5I3J2_COLAP\nGSK3B\tA0A2K5M9Z3_CERAT\nGSK3B\tA0A2K5MA20_CERAT\nGSK3B\tA0A2K5MA25_CERAT\nGSK3B\tA0A2K5MA39_CERAT\nGSK3B\tA0A2K5QF67_CEBCA\nGSK3B\tA0A2K5QF81_CEBCA\nGSK3B\tA0A2K5QFB9_CEBCA\nGSK3B\tA0A2K5QFD4_CEBCA\nGSK3B\tA0A2K5QFD7_CEBCA\nGSK3B\tA0A2K5QFE9_CEBCA\nGSK3B\tA0A2K5U1P9_MACFA\nGSK3B\tA0A2K5U1T8_MACFA\nGSK3B\tA0A2K5U1V1_MACFA\nGSK3B\tA0A2K5U1W2_MACFA\nGSK3B\tA0A2K5U1X8_MACFA\nGSK3B\tA0A2K5Z9V4_MANLE\nGSK3B\tA0A2K5Z9W2_MANLE\nGSK3B\tA0A2K5Z9X6_MANLE\nGSK3B\tA0A2K5Z9Z3_MANLE\nGSK3B\tA0A2K5ZA13_MANLE\nGSK3B\tA0A2K6B412_MACNE\nGSK3B\tA0A2K6B419_MACNE\nGSK3B\tA0A2K6G9B3_PROCO\nGSK3B\tA0A2K6G9D9_PROCO\nGSK3B\tA0A2K6LLD3_RHIBE\nGSK3B\tA0A2K6LLE8_RHIBE\nGSK3B\tA0A2K6LLG4_RHIBE\nGSK3B\tA0A2K6LLG9_RHIBE\nGSK3B\tA0A2K6N6X1_RHIRO\nGSK3B\tA0A2K6SYL2_SAIBB\nGSK3B\tA0A2K6SYP5_SAIBB\nGSK3B\tA0A2K6SYU3_SAIBB\nGSK3B\tA0A2K6SYX3_SAIBB\nGSK3B\tA0A2K6SYY1_SAIBB\nGSK3B\tA0A2K6SYZ3_SAIBB\nGSK3B\tA0A2R8M2M3_CALJA\nGSK3B\tA0A2R8M2U0_CALJA\nGSK3B\tA0A2R9C3U2_PANPA\nGSK3B\tA0A2R9C6C8_PANPA\nGSK3B\tA0A2R9CDM4_PANPA\nGSK3B\tA0A2R9CDS9_PANPA\nGSK3B\tA0A2S2PQD5_SCHGA\nGSK3B\tA0A2U3W5Y9_ODORO\nGSK3B\tA0A2U3W5Z0_ODORO\nGSK3B\tA0A2U3W616_ODORO\nGSK3B\tA0A2U3YEV1_LEPWE\nGSK3B\tA0A2U3YF60_LEPWE\nGSK3B\tA0A2U3ZIU4_ODORO\nGSK3B\tA0A2U3ZIU8_ODORO\nGSK3B\tA0A2U3ZIV6_ODORO\nGSK3B\tA0A2U4AAG1_TURTR\nGSK3B\tA0A2U4AAG3_TURTR\nGSK3B\tA0A2U4AAG5_TURTR\nGSK3B\tA0A2U4AAG6_TURTR\nGSK3B\tA0A2U4AAI9_TURTR\nGSK3B\tA0A2U4AAR6_TURTR\nGSK3B\tA0A2U4AAS0_TURTR\nGSK3B\tA0A2U4AB47_TURTR\nGSK3B\tA0A2Y9H9J4_NEOSC\nGSK3B\tA0A2Y9H9Q5_NEOSC\nGSK3B\tA0A2Y9M5K9_DELLE\nGSK3B\tA0A2Y9M920_DELLE\nGSK3B\tA0A2Y9ME39_DELLE\nGSK3B\tA0A2Y9MJV1_DELLE\nGSK3B\tA0A2Y9MK29_DELLE\nGSK3B\tA0A2Y9SC11_PHYMC\nGSK3B\tA0A2Y9SFX4_PHYMC\nGSK3B\tA0A2Y9SKD3_PHYMC\nGSK3B\tA0A337S9S1_FELCA\nGSK3B\tA0A338P6P8_MOUSE\nGSK3B\tA0A340WM12_LIPVE\nGSK3B\tA0A340WP80_LIPVE\nGSK3B\tA0A340WSK2_LIPVE\nGSK3B\tA0A340WU37_LIPVE\nGSK3B\tA0A340WUL7_LIPVE\nGSK3B\tA0A341DD22_NEOAA\nGSK3B\tA0A341DDJ5_NEOAA\nGSK3B\tA0A341DDM6_NEOAA\nGSK3B\tA0A341DDU8_NEOAA\nGSK3B\tA0A341DEG4_NEOAA\nGSK3B\tA0A384A3J7_BALAS\nGSK3B\tA0A384A3K0_BALAS\nGSK3B\tA0A384A3N2_BALAS\nGSK3B\tA0A384A3X3_BALAS\nGSK3B\tA0A384DHG1_URSMA\nGSK3B\tA0A384DI01_URSMA\nGSK3B\tA0A3B3ITW1_HUMAN\nGSK3B\tA0A3P8XJV7_ESOLU\nGSK3B\tA0A3P8ZHQ8_ESOLU\nGSK3B\tA0A3Q0DWQ2_TARSY\nGSK3B\tA0A3Q0FLT7_ALLSI\nGSK3B\tA0A3Q0FMQ7_ALLSI\nGSK3B\tA0A3Q1LMN7_BOVIN\nGSK3B\tA0A3Q1MT31_BOVIN\nGSK3B\tA0A3Q1NDU7_BOVIN\nGSK3B\tA0A3Q2H039_HORSE\nGSK3B\tA0A3Q7P9Y0_CALUR\nGSK3B\tA0A3Q7QQP5_CALUR\nGSK3B\tA0A3Q7RJF7_VULVU\nGSK3B\tA0A3Q7RJR8_VULVU\nGSK3B\tA0A3Q7RL27_VULVU\nGSK3B\tA0A3Q7VCW8_URSAR\nGSK3B\tA0A3Q7VCY8_URSAR\nGSK3B\tA0A3Q7WHX4_URSAR\nGSK3B\tA0A3Q7WHX8_URSAR\nGSK3B\tA0A3R7FH60_CLOSI\nGSK3B\tA0A419PC65_CLOSI\nGSK3B\tA0A452EM17_CAPHI\nGSK3B\tA0A452R1J0_URSAM\nGSK3B\tA0A452R1J6_URSAM\nGSK3B\tA0A455BML6_PHYMC\nGSK3B\tA0A455BMM1_PHYMC\nGSK3B\tA0A493SWT3_ANAPP\nGSK3B\tA0A493T2C1_ANAPP\nGSK3B\tA0A493TEB9_ANAPP\nGSK3B\tA0A493TJD5_ANAPP\nGSK3B\tA0A493U3E4_ANAPP\nGSK3B\tA0A4W2CXQ6_BOBOX\nGSK3B\tA0A4W6G4E7_LATCA\nGSK3B\tA0A4W6G4S1_LATCA\nGSK3B\tA0A4W6G5U6_LATCA\nGSK3B\tA0A4W6G6Q0_LATCA\nGSK3B\tA0A4W6G6Q5_LATCA\nGSK3B\tA0A4X1T4K1_PIG\nGSK3B\tA0A4X1T4Q4_PIG\nGSK3B\tA0A4X1T5F0_PIG\nGSK3B\tA0A4X1T5J0_PIG\nGSK3B\tA0A4X1T5J5_PIG\nGSK3B\tA0A4X1T638_PIG\nGSK3B\tA0A4X2LD33_VOMUR\nGSK3B\tA0A5B6UG68_9ROSI\nGSK3B\tA0A5B6UH57_9ROSI\nGSK3B\tA0A5B6V854_9ROSI\nGSK3B\tA0A5B6V890_9ROSI\nGSK3B\tA0A5B6VJT5_9ROSI\nGSK3B\tA0A5B6VRK3_9ROSI\nGSK3B\tA0A5B6VUH7_9ROSI\nGSK3B\tA0A5B6WAY0_9ROSI\nGSK3B\tA0A5F8A1A5_MACMU\nGSK3B\tA0A5F8GBH0_MONDO\nGSK3B\tA0A5F8HIG1_MONDO\nGSK3B\tA0A5F9CM08_RABIT\nGSK3B\tA0A5F9DFZ1_RABIT\nGSK3B\tA0A5F9DUX8_RABIT\nGSK3B\tA0A5G3HE38_XENTR\nGSK3B\tA0A5G3HEQ8_XENTR\nGSK3B\tA0A5G3HFI4_XENTR\nGSK3B\tA0A5G3J261_XENTR\nGSK3B\tA0A5G3KXF4_XENTR\nGSK3B\tA0A5G3KZJ1_XENTR\nGSK3B\tA0A5K1UHY0_PIG\nGSK3B\tA4V3W1_DROME\nGSK3B\tA4V3W2_DROME\nGSK3B\tA8JUV9_DROME\nGSK3B\tB5BUC0_HUMAN\nGSK3B\tC0PUX5_DROME\nGSK3B\tC3Y0G2_BRAFL\nGSK3B\tD3DN88_HUMAN\nGSK3B\tE2R4Y4_CANLF\nGSK3B\tE2RB53_CANLF\nGSK3B\tE9QAQ5_MOUSE\nGSK3B\tF1MIC3_BOVIN\nGSK3B\tF6PGQ1_MONDO\nGSK3B\tF6SA86_HORSE\nGSK3B\tF6V0J0_CALJA\nGSK3B\tF6VAP6_MACMU\nGSK3B\tF6XE59_XENTR\nGSK3B\tF6YPA3_CALJA\nGSK3B\tF7ACT4_XENTR\nGSK3B\tF7APF7_XENTR\nGSK3B\tF7BEB2_XENTR\nGSK3B\tF7GCK3_ORNAN\nGSK3B\tG1FKY2_GEKJA\nGSK3B\tG1LKA6_AILME\nGSK3B\tG1LKA8_AILME\nGSK3B\tG1NNA1_MELGA\nGSK3B\tG1NYW2_MYOLU\nGSK3B\tG1QYY9_NOMLE\nGSK3B\tG1T8E2_RABIT\nGSK3B\tG3K0T2_HYDVU\nGSK3B\tG3N1T2_BOVIN\nGSK3B\tG3T0Q7_LOXAF\nGSK3B\tG3TTQ7_LOXAF\nGSK3B\tG3UFV6_LOXAF\nGSK3B\tG3VTE8_SARHA\nGSK3B\tGSK3B_HUMAN\nGSK3B\tGSK3B_MOUSE\nGSK3B\tGSK3B_RAT\nGSK3B\tGSK3B_SPECI\nGSK3B\tGSK3B_XENLA\nGSK3B\tH0VQQ9_CAVPO\nGSK3B\tH0WTC1_OTOGA\nGSK3B\tH0ZT29_TAEGU\nGSK3B\tH2MUK4_ORYLA\nGSK3B\tH2P9M0_PONAB\nGSK3B\tH2RYS5_TAKRU\nGSK3B\tH3AT64_LATCH\nGSK3B\tH3AT65_LATCH\nGSK3B\tH9EPM1_MACMU\nGSK3B\tH9EPM2_MACMU\nGSK3B\tH9G7Y1_ANOCA\nGSK3B\tI3KAF5_ORENI\nGSK3B\tI3MNZ2_ICTTR\nGSK3B\tI3NFP6_ICTTR\nGSK3B\tI6YP72_PIG\nGSK3B\tI6YZK2_PIG\nGSK3B\tI6ZJZ7_PIG\nGSK3B\tI6ZT54_PIG\nGSK3B\tI6ZX61_PIG\nGSK3B\tK7CFE4_PANTR\nGSK3B\tK7CFX8_PANTR\nGSK3B\tK7FPQ8_PELSI\nGSK3B\tK7FPR5_PELSI\nGSK3B\tM3XL05_LATCH\nGSK3B\tM3YAD9_MUSPF\nGSK3B\tM9PGC4_DROME\nGSK3B\tM9PGV7_DROME\nGSK3B\tQ0KHW6_DROME\nGSK3B\tQ0PKV3_LEIME\nGSK3B\tQ28JA1_XENTR\nGSK3B\tQ3TB31_MOUSE\nGSK3B\tQ5KU03_MOUSE\nGSK3B\tQ6FI27_HUMAN\nGSK3B\tQ7KVY5_DROME\nGSK3B\tQ86TM2_HUMAN\nGSK3B\tQ9IBD2_DANRE\nGSK3B\tQ9NL43_CIOIN\nGSK3B\tQ9YH60_DANRE\nGSK3B\tR0KPV0_NOSB1\nGSK3B\tR4GAM0_ANOCA\nGSK3B\tU3IK92_ANAPP\nGSK3B\tU3JS27_FICAL\nGSK3B\tV8NXP7_OPHHA\nGSK3B\tW5QEZ9_SHEEP\nGSK3B\tW5QF00_SHEEP\nGSK3B\tW5UGM6_ICTPU\n','From\tTo\nCDK5\tA0A034W7H6_BACDO\nCDK5\tA0A087ZW58_APIME\nCDK5\tA0A090N7W4_HUMAN\nCDK5\tA0A096MSP7_PAPAN\nCDK5\tA0A0A1WUF8_ZEUCU\nCDK5\tA0A0A9YMH8_LYGHE\nCDK5\tA0A0B2VWG0_TOXCA\nCDK5\tA0A0B7FBQ2_THACB\nCDK5\tA0A0G2JDL3_MOUSE\nCDK5\tA0A0K6GIC5_9AGAM\nCDK5\tA0A0K8UAM9_BACLA\nCDK5\tA0A0P6J1W7_HETGA\nCDK5\tA0A0S2Z355_HUMAN\nCDK5\tA0A0S7G806_9TELE\nCDK5\tA0A0S7G815_9TELE\nCDK5\tA0A0S7GBD4_9TELE\nCDK5\tA0A0S7GC63_9TELE\nCDK5\tA0A0S7GCT3_9TELE\nCDK5\tA0A0S7GDI4_9TELE\nCDK5\tA0A0S7GEG7_9TELE\nCDK5\tA0A0V0SKP3_9BILA\nCDK5\tA0A0V0SL89_9BILA\nCDK5\tA0A0V0U1M2_9BILA\nCDK5\tA0A0V0UY83_9BILA\nCDK5\tA0A0V0X7Y3_9BILA\nCDK5\tA0A0V0X8R4_9BILA\nCDK5\tA0A0V0YBN3_TRIPS\nCDK5\tA0A0V0YCT3_TRIPS\nCDK5\tA0A0V1A885_9BILA\nCDK5\tA0A0V1A8E7_9BILA\nCDK5\tA0A0V1DDW7_TRIBR\nCDK5\tA0A0V1DEF9_TRIBR\nCDK5\tA0A0V1E9Q5_TRIPS\nCDK5\tA0A0V1E9V2_TRIPS\nCDK5\tA0A0V1G6V9_TRIPS\nCDK5\tA0A0V1HT78_9BILA\nCDK5\tA0A0V1HVT0_9BILA\nCDK5\tA0A0V1JIE3_TRIPS\nCDK5\tA0A0V1JIF9_TRIPS\nCDK5\tA0A0V1MVG6_9BILA\nCDK5\tA0A0V1MW58_9BILA\nCDK5\tA0A0V1NV07_9BILA\nCDK5\tA0A0X3NWZ9_SCHSO\nCDK5\tA0A0X3PLI3_SCHSO\nCDK5\tA0A1A7WAC1_9TELE\nCDK5\tA0A1A8A9Q1_NOTFU\nCDK5\tA0A1A8DXC0_9TELE\nCDK5\tA0A1A8FIK8_9TELE\nCDK5\tA0A1A8GVW5_9TELE\nCDK5\tA0A1A8IK16_NOTKU\nCDK5\tA0A1A8KEU6_NOTKU\nCDK5\tA0A1A8QFW7_9TELE\nCDK5\tA0A1A8R4Y8_9TELE\nCDK5\tA0A1C7NB83_9FUNG\nCDK5\tA0A1J4J3R0_9EUKA\nCDK5\tA0A1J4JWW7_9EUKA\nCDK5\tA0A1S3AKJ9_ERIEU\nCDK5\tA0A1S3AKW1_ERIEU\nCDK5\tA0A1S3G1J6_DIPOR\nCDK5\tA0A1S3G1Z7_DIPOR\nCDK5\tA0A1U7R6W2_MESAU\nCDK5\tA0A1U7UY79_TARSY\nCDK5\tA0A1V4JF68_PATFA\nCDK5\tA0A218UGL2_9PASE\nCDK5\tA0A250XXV0_CASCN\nCDK5\tA0A286Y447_CAVPO\nCDK5\tA0A287DE97_ICTTR\nCDK5\tA0A2B4SM62_STYPI\nCDK5\tA0A2H8TII6_9HEMI\nCDK5\tA0A2I0LI52_COLLI\nCDK5\tA0A2I3HDN8_NOMLE\nCDK5\tA0A2I3NEP8_PAPAN\nCDK5\tA0A2I3SUJ4_PANTR\nCDK5\tA0A2I4CZ65_9TELE\nCDK5\tA0A2J7PH71_9NEOP\nCDK5\tA0A2K5DNI7_AOTNA\nCDK5\tA0A2K5DNK2_AOTNA\nCDK5\tA0A2K5DNW1_AOTNA\nCDK5\tA0A2K5IU84_COLAP\nCDK5\tA0A2K5IUR4_COLAP\nCDK5\tA0A2K5M9K0_CERAT\nCDK5\tA0A2K5M9S7_CERAT\nCDK5\tA0A2K5PRA2_CEBCA\nCDK5\tA0A2K5PRA8_CEBCA\nCDK5\tA0A2K5PRE6_CEBCA\nCDK5\tA0A2K5TYL9_MACFA\nCDK5\tA0A2K5TYN5_MACFA\nCDK5\tA0A2K5YV25_MANLE\nCDK5\tA0A2K5YV42_MANLE\nCDK5\tA0A2K6E678_MACNE\nCDK5\tA0A2K6E683_MACNE\nCDK5\tA0A2K6GJB6_PROCO\nCDK5\tA0A2K6GJI5_PROCO\nCDK5\tA0A2K6LLP8_RHIBE\nCDK5\tA0A2K6LLU2_RHIBE\nCDK5\tA0A2K6PSN5_RHIRO\nCDK5\tA0A2K6PSQ1_RHIRO\nCDK5\tA0A2K6S806_SAIBB\nCDK5\tA0A2K6S818_SAIBB\nCDK5\tA0A2P8Z818_BLAGE\nCDK5\tA0A2R9B1T9_PANPA\nCDK5\tA0A2R9B3E0_PANPA\nCDK5\tA0A2R9B886_PANPA\nCDK5\tA0A2S2Q8T2_9HEMI\nCDK5\tA0A2U3WJ68_ODORO\nCDK5\tA0A2U3WJA9_ODORO\nCDK5\tA0A2U3Z7K7_LEPWE\nCDK5\tA0A2Y9FM68_PHYMC\nCDK5\tA0A2Y9HGG3_NEOSC\nCDK5\tA0A2Y9NQ07_DELLE\nCDK5\tA0A337SJH7_FELCA\nCDK5\tA0A340Y9Y7_LIPVE\nCDK5\tA0A340YD64_LIPVE\nCDK5\tA0A341C5Y2_NEOAA\nCDK5\tA0A383YRG4_BALAS\nCDK5\tA0A383YRM8_BALAS\nCDK5\tA0A384BW98_URSMA\nCDK5\tA0A3B3H884_ORYLA\nCDK5\tA0A3Q0DHR6_TARSY\nCDK5\tA0A3Q2KMU3_HORSE\nCDK5\tA0A3Q2UBD9_CHICK\nCDK5\tA0A3Q7PPU0_CALUR\nCDK5\tA0A3Q7RTT5_VULVU\nCDK5\tA0A3Q7S1J8_VULVU\nCDK5\tA0A3Q7T426_VULVU\nCDK5\tA0A3Q7UFK6_URSAR\nCDK5\tA0A3R7C7D3_CLOSI\nCDK5\tA0A452G5B1_CAPHI\nCDK5\tA0A452SB52_URSAM\nCDK5\tA0A452UP33_URSMA\nCDK5\tA0A4C1TH11_9NEOP\nCDK5\tA0A4Q0TFS7_COCIT\nCDK5\tA0A4W2D998_BOBOX\nCDK5\tA0A4W2D9L4_BOBOX\nCDK5\tA0A4W2HKQ3_BOBOX\nCDK5\tA0A4W6C899_LATCA\nCDK5\tA0A4W6CFK2_LATCA\nCDK5\tA0A4X2KDE4_VOMUR\nCDK5\tA0A4X2KDF8_VOMUR\nCDK5\tA0A4Y2JDJ9_ARAVE\nCDK5\tA0A4Y2JV85_ARAVE\nCDK5\tA0A4Z2GPB7_9TELE\nCDK5\tA0A5B7CZN4_PORTR\nCDK5\tA0A5F4BS56_CANLF\nCDK5\tA0A5F4CCB1_CANLF\nCDK5\tA0A5F4CTH3_CANLF\nCDK5\tA0A5F8H0L1_MONDO\nCDK5\tA0A5F8HD62_MONDO\nCDK5\tA0A5F9CNY1_RABIT\nCDK5\tA0A5G2R8T5_PIG\nCDK5\tA2IBW2_USTMD\nCDK5\tA2TNA7_PIG\nCDK5\tA9UHN7_CARAU\nCDK5\tA9UYZ3_MONBE\nCDK5\tB6E1W1_CHICK\nCDK5\tB6E1W2_SHEEP\nCDK5\tB7UCA8_DIOKA\nCDK5\tC1BT30_LEPSM\nCDK5\tC1L6Q8_SCHJA\nCDK5\tC1LK39_SCHJA\nCDK5\tCDK5_BOVIN\nCDK5\tCDK5_DICDI\nCDK5\tCDK5_DROME\nCDK5\tCDK5_GLOMM\nCDK5\tCDK5_HUMAN\nCDK5\tCDK5_MOUSE\nCDK5\tCDK5_RAT\nCDK5\tCDK5_XENLA\nCDK5\tD3BMU7_POLPP\nCDK5\tD4FUN3_SPHGR\nCDK5\tD4FUN5_SPHGR\nCDK5\tE2RGN2_CANLF\nCDK5\tE3TFD4_ICTPU\nCDK5\tF4PKB5_CAVFA\nCDK5\tF6VW45_MONDO\nCDK5\tF6VZU3_MACMU\nCDK5\tF6W6J7_MACMU\nCDK5\tF6YF11_HORSE\nCDK5\tF6YNV4_XENTR\nCDK5\tF6ZE37_CALJA\nCDK5\tF6ZEN3_CALJA\nCDK5\tG1M5B9_AILME\nCDK5\tG1PQL9_MYOLU\nCDK5\tG1QVP2_NOMLE\nCDK5\tG1U1H1_RABIT\nCDK5\tG3T2I2_LOXAF\nCDK5\tG3W778_SARHA\nCDK5\tG4VXK7_VICPA\nCDK5\tG9MX84_HYPVG\nCDK5\tG9NEE0_HYPAI\nCDK5\tH0V324_CAVPO\nCDK5\tH0WJB4_OTOGA\nCDK5\tH2MT54_ORYLA\nCDK5\tH2PP22_PONAB\nCDK5\tH2QVM3_PANTR\nCDK5\tH2SAY8_TAKRU\nCDK5\tH3AIY6_LATCH\nCDK5\tH9EQ04_MACMU\nCDK5\tI3JSU3_ORENI\nCDK5\tI3L7B6_PIG\nCDK5\tI3MQV2_ICTTR\nCDK5\tJ9JF51_OIKDI\nCDK5\tJ9NWG1_CANLF\nCDK5\tK7C0V1_PANTR\nCDK5\tM3X3E1_FELCA\nCDK5\tM3YAB5_MUSPF\nCDK5\tM4SI13_9BILA\nCDK5\tM4SJ13_9BILA\nCDK5\tQ00LP8_VICPA\nCDK5\tQ0VFL9_XENTR\nCDK5\tQ197W4_PIG\nCDK5\tQ543F6_MOUSE\nCDK5\tQ5U3V4_DANRE\nCDK5\tQ6IAW3_HUMAN\nCDK5\tQ6LD52_9MURI\nCDK5\tQ9DE44_DANRE\nCDK5\tT2MER8_HYDVU\nCDK5\tU3D7M9_CALJA\nCDK5\tU3KF38_FICAL\nCDK5\tU6CN56_NEOVI\nCDK5\tV8N9V9_OPHHA\nCDK5\tW8C1L1_CERCA\n','From\tTo\nCSNK1G3\tA0A096MST0_PAPAN\nCSNK1G3\tA0A0A0MXY3_RAT\nCSNK1G3\tA0A0A1XBG9_ZEUCU\nCSNK1G3\tA0A0D9RPY8_CHLSB\nCSNK1G3\tA0A0K8V692_BACLA\nCSNK1G3\tA0A0P6J453_HETGA\nCSNK1G3\tA0A0V0RMF7_9BILA\nCSNK1G3\tA0A0V0U1R2_9BILA\nCSNK1G3\tA0A0V0Y909_TRIPS\nCSNK1G3\tA0A0V0YAJ9_TRIPS\nCSNK1G3\tA0A0V0ZS31_9BILA\nCSNK1G3\tA0A0V1EFC4_TRIPS\nCSNK1G3\tA0A0V1HYK6_9BILA\nCSNK1G3\tA0A0V1L828_9BILA\nCSNK1G3\tA0A0V1N245_9BILA\nCSNK1G3\tA0A0V1N2H2_9BILA\nCSNK1G3\tA0A146MBN4_LYGHE\nCSNK1G3\tA0A1S3A139_ERIEU\nCSNK1G3\tA0A1S3A157_ERIEU\nCSNK1G3\tA0A1S3A159_ERIEU\nCSNK1G3\tA0A1S3A172_ERIEU\nCSNK1G3\tA0A1S3A177_ERIEU\nCSNK1G3\tA0A1S3F3H0_DIPOR\nCSNK1G3\tA0A1S3F3H1_DIPOR\nCSNK1G3\tA0A1S3F3H6_DIPOR\nCSNK1G3\tA0A1S3F4M4_DIPOR\nCSNK1G3\tA0A1S3F538_DIPOR\nCSNK1G3\tA0A1S3F5T8_DIPOR\nCSNK1G3\tA0A1S3F5U3_DIPOR\nCSNK1G3\tA0A1S3WH21_ERIEU\nCSNK1G3\tA0A1S3WH66_ERIEU\nCSNK1G3\tA0A1U7Q4Z0_MESAU\nCSNK1G3\tA0A1U7Q4Z2_MESAU\nCSNK1G3\tA0A1U7Q9Y4_MESAU\nCSNK1G3\tA0A1U7QC00_MESAU\nCSNK1G3\tA0A1U7QC05_MESAU\nCSNK1G3\tA0A1U7QJW3_MESAU\nCSNK1G3\tA0A1U7QMC3_MESAU\nCSNK1G3\tA0A1U7RBJ3_ALLSI\nCSNK1G3\tA0A1U7RIQ6_ALLSI\nCSNK1G3\tA0A1U7ST87_TARSY\nCSNK1G3\tA0A1U7TBB2_TARSY\nCSNK1G3\tA0A1U7TDN5_TARSY\nCSNK1G3\tA0A1V4KCL9_PATFA\nCSNK1G3\tA0A1V4KCX5_PATFA\nCSNK1G3\tA0A218V539_9PASE\nCSNK1G3\tA0A250Y0Y9_CASCN\nCSNK1G3\tA0A286XNP0_CAVPO\nCSNK1G3\tA0A286ZTG8_PIG\nCSNK1G3\tA0A287AVA1_PIG\nCSNK1G3\tA0A287B261_PIG\nCSNK1G3\tA0A287D0G1_ICTTR\nCSNK1G3\tA0A287DDC5_ICTTR\nCSNK1G3\tA0A2B4RJK2_STYPI\nCSNK1G3\tA0A2H8TQ48_9HEMI\nCSNK1G3\tA0A2I0LS67_COLLI\nCSNK1G3\tA0A2I0LS73_COLLI\nCSNK1G3\tA0A2I2UAC8_FELCA\nCSNK1G3\tA0A2I3HFC2_NOMLE\nCSNK1G3\tA0A2I3HVN2_NOMLE\nCSNK1G3\tA0A2I3M5Y1_PAPAN\nCSNK1G3\tA0A2I3ML71_PAPAN\nCSNK1G3\tA0A2I3MZA9_PAPAN\nCSNK1G3\tA0A2I3RG71_PANTR\nCSNK1G3\tA0A2I3RZN6_PANTR\nCSNK1G3\tA0A2I3SFG5_PANTR\nCSNK1G3\tA0A2I3TNT3_PANTR\nCSNK1G3\tA0A2J7QV54_9NEOP\nCSNK1G3\tA0A2K5CUM0_AOTNA\nCSNK1G3\tA0A2K5CUM5_AOTNA\nCSNK1G3\tA0A2K5CUN8_AOTNA\nCSNK1G3\tA0A2K5K280_COLAP\nCSNK1G3\tA0A2K5K287_COLAP\nCSNK1G3\tA0A2K5K299_COLAP\nCSNK1G3\tA0A2K5K2A9_COLAP\nCSNK1G3\tA0A2K5MPD5_CERAT\nCSNK1G3\tA0A2K5MPE1_CERAT\nCSNK1G3\tA0A2K5MPG6_CERAT\nCSNK1G3\tA0A2K5MPR9_CERAT\nCSNK1G3\tA0A2K5S441_CEBCA\nCSNK1G3\tA0A2K5S478_CEBCA\nCSNK1G3\tA0A2K5S489_CEBCA\nCSNK1G3\tA0A2K5UYA5_MACFA\nCSNK1G3\tA0A2K5UYD7_MACFA\nCSNK1G3\tA0A2K5UYF3_MACFA\nCSNK1G3\tA0A2K5UYF8_MACFA\nCSNK1G3\tA0A2K6ACJ7_MANLE\nCSNK1G3\tA0A2K6ACK9_MANLE\nCSNK1G3\tA0A2K6ACM9_MANLE\nCSNK1G3\tA0A2K6ACR3_MANLE\nCSNK1G3\tA0A2K6C181_MACNE\nCSNK1G3\tA0A2K6C1A0_MACNE\nCSNK1G3\tA0A2K6C1B1_MACNE\nCSNK1G3\tA0A2K6C1B8_MACNE\nCSNK1G3\tA0A2K6C1C8_MACNE\nCSNK1G3\tA0A2K6G1V1_PROCO\nCSNK1G3\tA0A2K6G1W1_PROCO\nCSNK1G3\tA0A2K6G1X2_PROCO\nCSNK1G3\tA0A2K6N3F7_RHIBE\nCSNK1G3\tA0A2K6N3G2_RHIBE\nCSNK1G3\tA0A2K6N3I7_RHIBE\nCSNK1G3\tA0A2K6N3J9_RHIBE\nCSNK1G3\tA0A2K6N3K6_RHIBE\nCSNK1G3\tA0A2K6Q6Y3_RHIRO\nCSNK1G3\tA0A2K6Q6Y6_RHIRO\nCSNK1G3\tA0A2K6Q6Z0_RHIRO\nCSNK1G3\tA0A2K6Q6Z4_RHIRO\nCSNK1G3\tA0A2K6US65_SAIBB\nCSNK1G3\tA0A2K6US80_SAIBB\nCSNK1G3\tA0A2K6US83_SAIBB\nCSNK1G3\tA0A2K6US93_SAIBB\nCSNK1G3\tA0A2K6USG2_SAIBB\nCSNK1G3\tA0A2P8XH81_BLAGE\nCSNK1G3\tA0A2R8ZYA3_PANPA\nCSNK1G3\tA0A2R8ZZC9_PANPA\nCSNK1G3\tA0A2R9A278_PANPA\nCSNK1G3\tA0A2S2PCD8_SCHGA\nCSNK1G3\tA0A2S2R1C3_9HEMI\nCSNK1G3\tA0A2U3W3H4_ODORO\nCSNK1G3\tA0A2U3W3H8_ODORO\nCSNK1G3\tA0A2U3W3I1_ODORO\nCSNK1G3\tA0A2U3W3I4_ODORO\nCSNK1G3\tA0A2U3W3L6_ODORO\nCSNK1G3\tA0A2U3W3M3_ODORO\nCSNK1G3\tA0A2U3W3N0_ODORO\nCSNK1G3\tA0A2U3YI57_LEPWE\nCSNK1G3\tA0A2U3YI62_LEPWE\nCSNK1G3\tA0A2U3YI63_LEPWE\nCSNK1G3\tA0A2U3YI64_LEPWE\nCSNK1G3\tA0A2U3YI65_LEPWE\nCSNK1G3\tA0A2U3YIJ2_LEPWE\nCSNK1G3\tA0A2U3ZHZ4_ODORO\nCSNK1G3\tA0A2U3ZI02_ODORO\nCSNK1G3\tA0A2U4BTB6_TURTR\nCSNK1G3\tA0A2U4BTC0_TURTR\nCSNK1G3\tA0A2U4BTC8_TURTR\nCSNK1G3\tA0A2U4BTD3_TURTR\nCSNK1G3\tA0A2U4BTD5_TURTR\nCSNK1G3\tA0A2U4BTJ2_TURTR\nCSNK1G3\tA0A2U4BTP0_TURTR\nCSNK1G3\tA0A2Y9I1Y2_NEOSC\nCSNK1G3\tA0A2Y9I267_NEOSC\nCSNK1G3\tA0A2Y9I6K7_NEOSC\nCSNK1G3\tA0A2Y9I8P5_NEOSC\nCSNK1G3\tA0A2Y9PM79_DELLE\nCSNK1G3\tA0A2Y9PNM0_DELLE\nCSNK1G3\tA0A2Y9S647_PHYMC\nCSNK1G3\tA0A2Y9S6B5_PHYMC\nCSNK1G3\tA0A2Y9S6C1_PHYMC\nCSNK1G3\tA0A2Y9SCQ5_PHYMC\nCSNK1G3\tA0A2Y9SE84_PHYMC\nCSNK1G3\tA0A337S9W2_FELCA\nCSNK1G3\tA0A340X114_LIPVE\nCSNK1G3\tA0A340X1W9_LIPVE\nCSNK1G3\tA0A340X1X4_LIPVE\nCSNK1G3\tA0A340X5Q1_LIPVE\nCSNK1G3\tA0A340X7V6_LIPVE\nCSNK1G3\tA0A340X8E9_LIPVE\nCSNK1G3\tA0A341CG86_NEOAA\nCSNK1G3\tA0A384AME6_BALAS\nCSNK1G3\tA0A384AMF6_BALAS\nCSNK1G3\tA0A384AMH6_BALAS\nCSNK1G3\tA0A384AMJ8_BALAS\nCSNK1G3\tA0A384AN00_BALAS\nCSNK1G3\tA0A384CQX5_URSMA\nCSNK1G3\tA0A384CQY6_URSMA\nCSNK1G3\tA0A384CRW0_URSMA\nCSNK1G3\tA0A384CRY1_URSMA\nCSNK1G3\tA0A3Q0DMD8_TARSY\nCSNK1G3\tA0A3Q0DUA5_TARSY\nCSNK1G3\tA0A3Q0GCL9_ALLSI\nCSNK1G3\tA0A3Q0GCM4_ALLSI\nCSNK1G3\tA0A3Q0GDG0_ALLSI\nCSNK1G3\tA0A3Q1LXL6_BOVIN\nCSNK1G3\tA0A3Q2HLK2_HORSE\nCSNK1G3\tA0A3Q2HVZ2_HORSE\nCSNK1G3\tA0A3Q2HZS4_HORSE\nCSNK1G3\tA0A3Q7N5J8_CALUR\nCSNK1G3\tA0A3Q7N933_CALUR\nCSNK1G3\tA0A3Q7N9J1_CALUR\nCSNK1G3\tA0A3Q7ND66_CALUR\nCSNK1G3\tA0A3Q7PKW7_CALUR\nCSNK1G3\tA0A3Q7RW93_VULVU\nCSNK1G3\tA0A3Q7RZ08_VULVU\nCSNK1G3\tA0A3Q7S4N0_VULVU\nCSNK1G3\tA0A3Q7S7J1_VULVU\nCSNK1G3\tA0A3Q7T7A8_VULVU\nCSNK1G3\tA0A3Q7VR96_URSAR\nCSNK1G3\tA0A3Q7VRA0_URSAR\nCSNK1G3\tA0A3Q7WF79_URSAR\nCSNK1G3\tA0A3Q7WF82_URSAR\nCSNK1G3\tA0A3Q7WPU3_URSAR\nCSNK1G3\tA0A3Q7XB19_URSAR\nCSNK1G3\tA0A3Q7XB24_URSAR\nCSNK1G3\tA0A3Q7XLN5_URSAR\nCSNK1G3\tA0A452G1M8_CAPHI\nCSNK1G3\tA0A452G202_CAPHI\nCSNK1G3\tA0A452G207_CAPHI\nCSNK1G3\tA0A452G263_CAPHI\nCSNK1G3\tA0A452G266_CAPHI\nCSNK1G3\tA0A452H383_9SAUR\nCSNK1G3\tA0A452H390_9SAUR\nCSNK1G3\tA0A452SP93_URSAM\nCSNK1G3\tA0A452SPF5_URSAM\nCSNK1G3\tA0A452SPG3_URSAM\nCSNK1G3\tA0A452SPI0_URSAM\nCSNK1G3\tA0A452SPK9_URSAM\nCSNK1G3\tA0A455BJF5_PHYMC\nCSNK1G3\tA0A455BM74_PHYMC\nCSNK1G3\tA0A493TP39_ANAPP\nCSNK1G3\tA0A493TXX4_ANAPP\nCSNK1G3\tA0A493U2C8_ANAPP\nCSNK1G3\tA0A494BAN5_MOUSE\nCSNK1G3\tA0A494BAP2_MOUSE\nCSNK1G3\tA0A4C1V6H7_9NEOP\nCSNK1G3\tA0A4W2D4G0_BOBOX\nCSNK1G3\tA0A4W2D8X8_BOBOX\nCSNK1G3\tA0A4W2GG74_BOBOX\nCSNK1G3\tA0A4W3IE72_CALMI\nCSNK1G3\tA0A4W3II09_CALMI\nCSNK1G3\tA0A4W3IM06_CALMI\nCSNK1G3\tA0A4W3IQ35_CALMI\nCSNK1G3\tA0A4W3IQ52_CALMI\nCSNK1G3\tA0A4W3J7Z8_CALMI\nCSNK1G3\tA0A4W3J813_CALMI\nCSNK1G3\tA0A4X1UBX6_PIG\nCSNK1G3\tA0A4X1UBY6_PIG\nCSNK1G3\tA0A4X1UEC6_PIG\nCSNK1G3\tA0A4X1UEF5_PIG\nCSNK1G3\tA0A4Y2JBS1_ARAVE\nCSNK1G3\tA0A4Y2JEN8_ARAVE\nCSNK1G3\tA0A5B7E183_PORTR\nCSNK1G3\tA0A5F4BWX9_CANLF\nCSNK1G3\tA0A5F4CVA9_CANLF\nCSNK1G3\tA0A5F4CY55_CANLF\nCSNK1G3\tA0A5F4W9J2_CALJA\nCSNK1G3\tA0A5F9C8V8_RABIT\nCSNK1G3\tA0A5F9CG88_RABIT\nCSNK1G3\tA0A5F9D336_RABIT\nCSNK1G3\tA0A5F9D6R2_RABIT\nCSNK1G3\tA0A5F9DPZ3_RABIT\nCSNK1G3\tA0A5G2QF71_PIG\nCSNK1G3\tA1L1H1_XENLA\nCSNK1G3\tB6ID06_XENTR\nCSNK1G3\tF1RKN8_PIG\nCSNK1G3\tF6V3I5_CANLF\nCSNK1G3\tF7A8M8_XENTR\nCSNK1G3\tF7AW18_ORNAN\nCSNK1G3\tF7CH57_HORSE\nCSNK1G3\tF7D479_CALJA\nCSNK1G3\tF7DPL9_CALJA\nCSNK1G3\tF7H415_MACMU\nCSNK1G3\tF7HHU2_CALJA\nCSNK1G3\tG1L0F1_AILME\nCSNK1G3\tG1PNG8_MYOLU\nCSNK1G3\tG1RN92_NOMLE\nCSNK1G3\tG1T865_RABIT\nCSNK1G3\tG3T0L7_LOXAF\nCSNK1G3\tG3U3J2_LOXAF\nCSNK1G3\tG3U9D1_LOXAF\nCSNK1G3\tG3X7B2_BOVIN\nCSNK1G3\tH0VAK3_CAVPO\nCSNK1G3\tH0YBN6_HUMAN\nCSNK1G3\tH2PGD7_PONAB\nCSNK1G3\tH2RAT8_PANTR\nCSNK1G3\tH9EUN4_MACMU\nCSNK1G3\tH9FA48_MACMU\nCSNK1G3\tH9G782_ANOCA\nCSNK1G3\tH9YX23_MACMU\nCSNK1G3\tI0FHH4_MACMU\nCSNK1G3\tI3LZV3_ICTTR\nCSNK1G3\tJ9P9W3_CANLF\nCSNK1G3\tK7B839_PANTR\nCSNK1G3\tK7D891_PANTR\nCSNK1G3\tK7G855_PELSI\nCSNK1G3\tK7G871_PELSI\nCSNK1G3\tKC1G3_BOVIN\nCSNK1G3\tKC1G3_HUMAN\nCSNK1G3\tKC1G3_MOUSE\nCSNK1G3\tKC1G3_PONAB\nCSNK1G3\tKC1G3_RAT\nCSNK1G3\tM3X6V4_FELCA\nCSNK1G3\tM3YNE5_MUSPF\nCSNK1G3\tQ28IZ6_XENTR\nCSNK1G3\tQ3MHQ2_BOVIN\nCSNK1G3\tT2M1Y4_HYDVU\nCSNK1G3\tU3AVV3_CALJA\nCSNK1G3\tU3E6K8_CALJA\nCSNK1G3\tU3F5Z4_CALJA\nCSNK1G3\tU3I5I9_ANAPP\nCSNK1G3\tU3JYG5_FICAL\nCSNK1G3\tV8P0E6_OPHHA\nCSNK1G3\tV9KQG3_CALMI\nCSNK1G3\tW5MQ14_LEPOC\nCSNK1G3\tW5MQ25_LEPOC\nCSNK1G3\tW5QBQ3_SHEEP\nCSNK1G3\tW5QBQ4_SHEEP\n','From\tTo\nCSNK1G2\tA0A087WW97_HUMAN\nCSNK1G2\tA0A087WWF5_HUMAN\nCSNK1G2\tA0A087WXD7_HUMAN\nCSNK1G2\tA0A087WZK4_HUMAN\nCSNK1G2\tA0A087YC80_POEFO\nCSNK1G2\tA0A0A0MV40_PAPAN\nCSNK1G2\tA0A0A9XQ24_LYGHE\nCSNK1G2\tA0A0C9R0A2_9HYME\nCSNK1G2\tA0A0D9RHJ4_CHLSB\nCSNK1G2\tA0A0H2UHQ2_RAT\nCSNK1G2\tA0A0K8VT84_BACLA\nCSNK1G2\tA0A0P6JMY3_HETGA\nCSNK1G2\tA0A1A7WJZ9_9TELE\nCSNK1G2\tA0A1A7WPG7_9TELE\nCSNK1G2\tA0A1A7YBX4_9TELE\nCSNK1G2\tA0A1A7ZA23_NOTFU\nCSNK1G2\tA0A1A7ZWM6_NOTFU\nCSNK1G2\tA0A1A8CAN1_9TELE\nCSNK1G2\tA0A1A8D5G3_9TELE\nCSNK1G2\tA0A1A8F9X6_9TELE\nCSNK1G2\tA0A1A8HLC4_9TELE\nCSNK1G2\tA0A1A8I1B5_NOTKU\nCSNK1G2\tA0A1A8I8E7_NOTKU\nCSNK1G2\tA0A1A8IGP6_NOTKU\nCSNK1G2\tA0A1A8LU70_9TELE\nCSNK1G2\tA0A1A8NCG5_9TELE\nCSNK1G2\tA0A1A8Q0Y9_9TELE\nCSNK1G2\tA0A1A8Q1D8_9TELE\nCSNK1G2\tA0A1A8Q1L3_9TELE\nCSNK1G2\tA0A1A8QNG8_9TELE\nCSNK1G2\tA0A1A8UGX0_NOTFU\nCSNK1G2\tA0A1D1Z9Z5_9ARAE\nCSNK1G2\tA0A1L8HWS0_XENLA\nCSNK1G2\tA0A1S3FPJ1_DIPOR\nCSNK1G2\tA0A1S3FQI3_DIPOR\nCSNK1G2\tA0A1S3T3L3_SALSA\nCSNK1G2\tA0A1S3WE40_ERIEU\nCSNK1G2\tA0A1U7R0T4_MESAU\nCSNK1G2\tA0A1U7RPX4_ALLSI\nCSNK1G2\tA0A1U8CRW0_MESAU\nCSNK1G2\tA0A1V4KQR7_PATFA\nCSNK1G2\tA0A1W2P817_MOUSE\nCSNK1G2\tA0A218US70_9PASE\nCSNK1G2\tA0A250Y182_CASCN\nCSNK1G2\tA0A286XC41_CAVPO\nCSNK1G2\tA0A287DB85_ICTTR\nCSNK1G2\tA0A2D0QY87_ICTPU\nCSNK1G2\tA0A2I0LPC4_COLLI\nCSNK1G2\tA0A2I3MWE7_PAPAN\nCSNK1G2\tA0A2I4B4X7_9TELE\nCSNK1G2\tA0A2I4B4X9_9TELE\nCSNK1G2\tA0A2I4B4Z0_9TELE\nCSNK1G2\tA0A2I4B4Z2_9TELE\nCSNK1G2\tA0A2J8R3W5_PONAB\nCSNK1G2\tA0A2K5ELZ4_AOTNA\nCSNK1G2\tA0A2K5EM08_AOTNA\nCSNK1G2\tA0A2K5IUD5_COLAP\nCSNK1G2\tA0A2K5M1W7_CERAT\nCSNK1G2\tA0A2K5M1Y5_CERAT\nCSNK1G2\tA0A2K5V7P7_MACFA\nCSNK1G2\tA0A2K5V7R2_MACFA\nCSNK1G2\tA0A2K5V7S9_MACFA\nCSNK1G2\tA0A2K5XZY5_MANLE\nCSNK1G2\tA0A2K6CUL1_MACNE\nCSNK1G2\tA0A2K6CUQ7_MACNE\nCSNK1G2\tA0A2K6F4R0_PROCO\nCSNK1G2\tA0A2K6ML72_RHIBE\nCSNK1G2\tA0A2K6ML76_RHIBE\nCSNK1G2\tA0A2K6MLA8_RHIBE\nCSNK1G2\tA0A2K6U818_SAIBB\nCSNK1G2\tA0A2S2P0A3_SCHGA\nCSNK1G2\tA0A2S2PZ27_9HEMI\nCSNK1G2\tA0A2U3VKX2_ODORO\nCSNK1G2\tA0A2U3YJ54_LEPWE\nCSNK1G2\tA0A2Y9IBB3_NEOSC\nCSNK1G2\tA0A2Y9IBF9_NEOSC\nCSNK1G2\tA0A2Y9IFS3_NEOSC\nCSNK1G2\tA0A2Y9LT62_DELLE\nCSNK1G2\tA0A2Y9LU23_DELLE\nCSNK1G2\tA0A2Y9LVV5_DELLE\nCSNK1G2\tA0A2Y9M0T2_DELLE\nCSNK1G2\tA0A2Y9M682_DELLE\nCSNK1G2\tA0A2Y9TKT7_PHYMC\nCSNK1G2\tA0A340XG06_LIPVE\nCSNK1G2\tA0A340XG67_LIPVE\nCSNK1G2\tA0A340XJW4_LIPVE\nCSNK1G2\tA0A341B0X5_NEOAA\nCSNK1G2\tA0A341B1Y8_NEOAA\nCSNK1G2\tA0A341B2P5_NEOAA\nCSNK1G2\tA0A3B1IZD3_ASTMX\nCSNK1G2\tA0A3B3HJ60_ORYLA\nCSNK1G2\tA0A3B3T953_9TELE\nCSNK1G2\tA0A3B3TW63_9TELE\nCSNK1G2\tA0A3B3TWB2_9TELE\nCSNK1G2\tA0A3B3XN89_9TELE\nCSNK1G2\tA0A3B3YWE0_9TELE\nCSNK1G2\tA0A3B4B2W9_9GOBI\nCSNK1G2\tA0A3B4G6B8_9CICH\nCSNK1G2\tA0A3B4G6D9_9CICH\nCSNK1G2\tA0A3B4U851_SERDU\nCSNK1G2\tA0A3B4U865_SERDU\nCSNK1G2\tA0A3B4WH11_SERLL\nCSNK1G2\tA0A3B4WR71_SERLL\nCSNK1G2\tA0A3B4WRH3_SERLL\nCSNK1G2\tA0A3B4ZFU2_9TELE\nCSNK1G2\tA0A3B4ZU93_9TELE\nCSNK1G2\tA0A3B4ZUC5_9TELE\nCSNK1G2\tA0A3B5JZU2_TAKRU\nCSNK1G2\tA0A3B5LI07_9TELE\nCSNK1G2\tA0A3B5PUS1_XIPMA\nCSNK1G2\tA0A3B5RAI8_XIPMA\nCSNK1G2\tA0A3P8PJN1_ASTCA\nCSNK1G2\tA0A3P8PJV0_ASTCA\nCSNK1G2\tA0A3P8SEU2_AMPPE\nCSNK1G2\tA0A3P8SEV3_AMPPE\nCSNK1G2\tA0A3P8SFZ1_AMPPE\nCSNK1G2\tA0A3P8WF39_CYNSE\nCSNK1G2\tA0A3P8WI10_CYNSE\nCSNK1G2\tA0A3P8WKR2_CYNSE\nCSNK1G2\tA0A3P9CSJ0_9CICH\nCSNK1G2\tA0A3P9CSJ5_9CICH\nCSNK1G2\tA0A3P9J784_ORYLA\nCSNK1G2\tA0A3P9J7I3_ORYLA\nCSNK1G2\tA0A3P9NRC6_POERE\nCSNK1G2\tA0A3P9NRN0_POERE\nCSNK1G2\tA0A3P9NRR0_POERE\nCSNK1G2\tA0A3P9NRT0_POERE\nCSNK1G2\tA0A3Q0GWE4_ALLSI\nCSNK1G2\tA0A3Q0GWF0_ALLSI\nCSNK1G2\tA0A3Q0H0Y0_ALLSI\nCSNK1G2\tA0A3Q0RU42_AMPCI\nCSNK1G2\tA0A3Q1AGR6_AMPOC\nCSNK1G2\tA0A3Q1AGU9_AMPOC\nCSNK1G2\tA0A3Q1F0L4_9TELE\nCSNK1G2\tA0A3Q1FTH1_9TELE\nCSNK1G2\tA0A3Q1IXS6_ANATE\nCSNK1G2\tA0A3Q1J512_ANATE\nCSNK1G2\tA0A3Q2DH07_CYPVA\nCSNK1G2\tA0A3Q2GQV3_CYPVA\nCSNK1G2\tA0A3Q2H6M7_HORSE\nCSNK1G2\tA0A3Q2HIR6_HORSE\nCSNK1G2\tA0A3Q2UL43_CHICK\nCSNK1G2\tA0A3Q2UZA7_HAPBU\nCSNK1G2\tA0A3Q2UZV3_HAPBU\nCSNK1G2\tA0A3Q2WFE9_HAPBU\nCSNK1G2\tA0A3Q2WNY1_HAPBU\nCSNK1G2\tA0A3Q2XZ36_HIPCM\nCSNK1G2\tA0A3Q2YSS3_HIPCM\nCSNK1G2\tA0A3Q3G4R5_KRYMA\nCSNK1G2\tA0A3Q3G4T0_KRYMA\nCSNK1G2\tA0A3Q3NLH5_9LABR\nCSNK1G2\tA0A3Q3NLI3_9LABR\nCSNK1G2\tA0A3Q3RA60_MONAL\nCSNK1G2\tA0A3Q3RA67_MONAL\nCSNK1G2\tA0A3Q3RUZ5_9TELE\nCSNK1G2\tA0A3Q3WJJ9_MOLML\nCSNK1G2\tA0A3Q4HW99_NEOBR\nCSNK1G2\tA0A3Q7N6D8_CALUR\nCSNK1G2\tA0A3Q7NAH3_CALUR\nCSNK1G2\tA0A3Q7NEB5_CALUR\nCSNK1G2\tA0A3Q7TKQ3_URSAR\nCSNK1G2\tA0A3Q7U913_URSAR\nCSNK1G2\tA0A3Q7UPC1_VULVU\nCSNK1G2\tA0A452CCV6_BALAS\nCSNK1G2\tA0A452EAB7_CAPHI\nCSNK1G2\tA0A452EAU2_CAPHI\nCSNK1G2\tA0A452EAX9_CAPHI\nCSNK1G2\tA0A452I7C1_9SAUR\nCSNK1G2\tA0A452SR99_URSAM\nCSNK1G2\tA0A455C5E1_PHYMC\nCSNK1G2\tA0A455C8R9_PHYMC\nCSNK1G2\tA0A455CBJ4_PHYMC\nCSNK1G2\tA0A4W2DAF9_BOBOX\nCSNK1G2\tA0A4W2FFW2_BOBOX\nCSNK1G2\tA0A4W3GE59_CALMI\nCSNK1G2\tA0A4W3GEA1_CALMI\nCSNK1G2\tA0A4W3GUH1_CALMI\nCSNK1G2\tA0A4W6CSJ0_LATCA\nCSNK1G2\tA0A4W6CST3_LATCA\nCSNK1G2\tA0A4W6CSU4_LATCA\nCSNK1G2\tA0A4W6CSX9_LATCA\nCSNK1G2\tA0A4W6CSY0_LATCA\nCSNK1G2\tA0A4W6CSY4_LATCA\nCSNK1G2\tA0A4X1VT54_PIG\nCSNK1G2\tA0A4X1VTM8_PIG\nCSNK1G2\tA0A5F4DH76_CANLF\nCSNK1G2\tA0A5F8GVC3_MONDO\nCSNK1G2\tA0A5G2Q6V7_PIG\nCSNK1G2\tA0A5G3IT00_XENTR\nCSNK1G2\tA7MB90_BOVIN\nCSNK1G2\tB5DFV5_SALSA\nCSNK1G2\tE2QX25_CANLF\nCSNK1G2\tF1P4M2_CHICK\nCSNK1G2\tF1SDH5_PIG\nCSNK1G2\tF6UQA5_MONDO\nCSNK1G2\tF6ZCR5_XENTR\nCSNK1G2\tF6ZCS3_XENTR\nCSNK1G2\tF7DPD1_HORSE\nCSNK1G2\tF7GC09_ORNAN\nCSNK1G2\tF7GC13_ORNAN\nCSNK1G2\tF7H481_MACMU\nCSNK1G2\tG1LF55_AILME\nCSNK1G2\tG1MTK4_MELGA\nCSNK1G2\tG1P0S4_MYOLU\nCSNK1G2\tG1RRZ1_NOMLE\nCSNK1G2\tG3SQY7_LOXAF\nCSNK1G2\tG3VX41_SARHA\nCSNK1G2\tH0WCZ4_CAVPO\nCSNK1G2\tH0ZVD2_TAEGU\nCSNK1G2\tH2LUU9_ORYLA\nCSNK1G2\tH2LUV0_ORYLA\nCSNK1G2\tH2QEV4_PANTR\nCSNK1G2\tH3AW74_LATCH\nCSNK1G2\tH3AW75_LATCH\nCSNK1G2\tH9EP27_MACMU\nCSNK1G2\tI3JUZ8_ORENI\nCSNK1G2\tI3JUZ9_ORENI\nCSNK1G2\tI3MVW2_ICTTR\nCSNK1G2\tK7D4Y0_PANTR\nCSNK1G2\tK7EJC0_HUMAN\nCSNK1G2\tK7EP38_HUMAN\nCSNK1G2\tK7ESB6_HUMAN\nCSNK1G2\tKC1G2_HUMAN\nCSNK1G2\tKC1G2_MOUSE\nCSNK1G2\tKC1G2_RAT\nCSNK1G2\tM3WZ99_FELCA\nCSNK1G2\tM3Y7B4_MUSPF\nCSNK1G2\tQ3LS18_XENTR\nCSNK1G2\tQ499U5_RAT\nCSNK1G2\tQ4SF43_TETNG\nCSNK1G2\tQ503C4_DANRE\nCSNK1G2\tQ5FW24_XENTR\nCSNK1G2\tQ6GN14_XENLA\nCSNK1G2\tQ99JQ5_MOUSE\nCSNK1G2\tQ99K78_MOUSE\nCSNK1G2\tU3BWL0_CALJA\nCSNK1G2\tU3DKH4_CALJA\nCSNK1G2\tU3KH39_FICAL\nCSNK1G2\tV9KSQ9_CALMI\nCSNK1G2\tW5LAT2_ASTMX\nCSNK1G2\tW5PU37_SHEEP\nCSNK1G2\tW5UEL4_ICTPU\nCSNK1G2\tW5UJQ3_ICTPU\n','From\tTo\nSTK17B\tA0A087WPR0_MOUSE\nSTK17B\tA0A096MYN3_PAPAN\nSTK17B\tA0A0B8RT41_PIG\nSTK17B\tA0A0C9QJE1_9HYME\nSTK17B\tA0A0D9RGZ3_CHLSB\nSTK17B\tA0A0P6JJ05_HETGA\nSTK17B\tA0A1A7WME2_9TELE\nSTK17B\tA0A1A8E6J5_9TELE\nSTK17B\tA0A1A8G0B0_9TELE\nSTK17B\tA0A1A8KYA5_NOTKU\nSTK17B\tA0A1A8QJ14_9TELE\nSTK17B\tA0A1A8QW58_9TELE\nSTK17B\tA0A1A8UZ03_NOTFU\nSTK17B\tA0A1S3AID0_ERIEU\nSTK17B\tA0A1S3EKK8_DIPOR\nSTK17B\tA0A1U7QU47_MESAU\nSTK17B\tA0A1U7RZJ1_ALLSI\nSTK17B\tA0A1U7UTE3_TARSY\nSTK17B\tA0A1V4L0Y4_PATFA\nSTK17B\tA0A218UXU6_9PASE\nSTK17B\tA0A250YDI1_CASCN\nSTK17B\tA0A286XP92_CAVPO\nSTK17B\tA0A2I0LXL3_COLLI\nSTK17B\tA0A2I4BZS8_9TELE\nSTK17B\tA0A2J7QBF4_9NEOP\nSTK17B\tA0A2J8WVM3_PONAB\nSTK17B\tA0A2K5F8Z6_AOTNA\nSTK17B\tA0A2K5HPY8_COLAP\nSTK17B\tA0A2K5NGQ9_CERAT\nSTK17B\tA0A2K5QV84_CEBCA\nSTK17B\tA0A2K5UTS2_MACFA\nSTK17B\tA0A2K5ZU64_MANLE\nSTK17B\tA0A2K6ALQ1_MACNE\nSTK17B\tA0A2K6F287_PROCO\nSTK17B\tA0A2K6MQ75_RHIBE\nSTK17B\tA0A2K6QX26_RHIRO\nSTK17B\tA0A2K6TYV7_SAIBB\nSTK17B\tA0A2R8ZLI1_PANPA\nSTK17B\tA0A2U3VFR7_ODORO\nSTK17B\tA0A2U3X9Q0_LEPWE\nSTK17B\tA0A2U4BNK5_TURTR\nSTK17B\tA0A2Y9F6S2_PHYMC\nSTK17B\tA0A2Y9I3B8_NEOSC\nSTK17B\tA0A2Y9MPX9_DELLE\nSTK17B\tA0A340WE70_LIPVE\nSTK17B\tA0A341AFC6_NEOAA\nSTK17B\tA0A384ATR2_BALAS\nSTK17B\tA0A384BTM3_URSMA\nSTK17B\tA0A3B3HS21_ORYLA\nSTK17B\tA0A3B3IFA5_ORYLA\nSTK17B\tA0A3B5K3N9_TAKRU\nSTK17B\tA0A3Q0H152_ALLSI\nSTK17B\tA0A3Q0H202_ALLSI\nSTK17B\tA0A3Q2GXD2_HORSE\nSTK17B\tA0A3Q2TVM3_CHICK\nSTK17B\tA0A3Q7QWH3_CALUR\nSTK17B\tA0A3Q7TT28_VULVU\nSTK17B\tA0A3Q7UI11_URSAR\nSTK17B\tA0A3R7CIS5_CLOSI\nSTK17B\tA0A452EIV5_CAPHI\nSTK17B\tA0A452HT59_9SAUR\nSTK17B\tA0A452R5I6_URSAM\nSTK17B\tA0A480DMB9_PIG\nSTK17B\tA0A4W2FF90_BOBOX\nSTK17B\tA0A4W4DVR2_ELEEL\nSTK17B\tA0A4W6FSF1_LATCA\nSTK17B\tA0A4X1VIM3_PIG\nSTK17B\tA0A4X2LL25_VOMUR\nSTK17B\tA0A4Z2G8T0_9TELE\nSTK17B\tA0A5B7IBC8_PORTR\nSTK17B\tA0A5F9CNV3_RABIT\nSTK17B\tC0H9H7_SALSA\nSTK17B\tC9JZJ1_HUMAN\nSTK17B\tD2H630_AILME\nSTK17B\tE2R573_CANLF\nSTK17B\tF1MSI1_BOVIN\nSTK17B\tF1SN17_PIG\nSTK17B\tF7D434_HORSE\nSTK17B\tF7F3V4_CALJA\nSTK17B\tF7F3X9_CALJA\nSTK17B\tF7FU71_MONDO\nSTK17B\tG1P9Q8_MYOLU\nSTK17B\tG1R525_NOMLE\nSTK17B\tG1T1S3_RABIT\nSTK17B\tG3T8I7_LOXAF\nSTK17B\tG3USA6_MELGA\nSTK17B\tG3WPU3_SARHA\nSTK17B\tH0WM41_OTOGA\nSTK17B\tH2MKQ8_ORYLA\nSTK17B\tH2QJ65_PANTR\nSTK17B\tH9EW56_MACMU\nSTK17B\tH9GJ71_ANOCA\nSTK17B\tI3NA19_ICTTR\nSTK17B\tK7G4Y0_PELSI\nSTK17B\tM3X7F7_FELCA\nSTK17B\tM3Y835_MUSPF\nSTK17B\tQ32KS1_BOVIN\nSTK17B\tQ53QE7_HUMAN\nSTK17B\tQ5ZM34_CHICK\nSTK17B\tQ6FG83_HUMAN\nSTK17B\tQ6FG98_HUMAN\nSTK17B\tQ6PHV6_DANRE\nSTK17B\tST17B_HUMAN\nSTK17B\tST17B_MOUSE\nSTK17B\tST17B_RAT\nSTK17B\tU3J622_ANAPP\nSTK17B\tU3JP00_FICAL\nSTK17B\tU3KQF8_HUMAN\nSTK17B\tV8P2U7_OPHHA\nSTK17B\tW5PZD3_SHEEP\nSTK17B\tW5UJ61_ICTPU\n','From\tTo\nPRKAA2\tA0A087WXX9_HUMAN\nPRKAA2\tA0A096MX27_PAPAN\nPRKAA2\tA0A0A9Y017_LYGHE\nPRKAA2\tA0A0C9RV04_9HYME\nPRKAA2\tA0A0D9S759_CHLSB\nPRKAA2\tA0A0K8UVJ3_BACLA\nPRKAA2\tA0A0V0W8C1_9BILA\nPRKAA2\tA0A0V0W8J8_9BILA\nPRKAA2\tA0A0V0W8K6_9BILA\nPRKAA2\tA0A0V1AVK0_TRISP\nPRKAA2\tA0A0V1KUC9_9BILA\nPRKAA2\tA0A1A7WI81_9TELE\nPRKAA2\tA0A1A7XU41_9TELE\nPRKAA2\tA0A1A7XU93_9TELE\nPRKAA2\tA0A1A7YK16_9TELE\nPRKAA2\tA0A1A7YL55_9TELE\nPRKAA2\tA0A1A7ZIS0_NOTFU\nPRKAA2\tA0A1A8C9K0_9TELE\nPRKAA2\tA0A1A8EN68_9TELE\nPRKAA2\tA0A1A8HDX7_9TELE\nPRKAA2\tA0A1A8ICZ7_NOTKU\nPRKAA2\tA0A1A8JYS2_NOTKU\nPRKAA2\tA0A1A8NVL1_9TELE\nPRKAA2\tA0A1A8NVQ0_9TELE\nPRKAA2\tA0A1A8QXU0_9TELE\nPRKAA2\tA0A1A8UJ16_NOTFU\nPRKAA2\tA0A1Q9E5C7_SYMMI\nPRKAA2\tA0A1S2ZEB5_ERIEU\nPRKAA2\tA0A1S2ZEB6_ERIEU\nPRKAA2\tA0A1S3FM67_DIPOR\nPRKAA2\tA0A1S3PAQ3_SALSA\nPRKAA2\tA0A1S3PAZ4_SALSA\nPRKAA2\tA0A218UWT3_9PASE\nPRKAA2\tA0A250Y2Q7_CASCN\nPRKAA2\tA0A2B4RW22_STYPI\nPRKAA2\tA0A2I0MCC0_COLLI\nPRKAA2\tA0A2I2UPC3_FELCA\nPRKAA2\tA0A2I3I0K0_NOMLE\nPRKAA2\tA0A2I3LUY4_PAPAN\nPRKAA2\tA0A2I3THE0_PANTR\nPRKAA2\tA0A2K5F9F0_AOTNA\nPRKAA2\tA0A2K5F9L2_AOTNA\nPRKAA2\tA0A2K5JE08_COLAP\nPRKAA2\tA0A2K5JE31_COLAP\nPRKAA2\tA0A2K5NF48_CERAT\nPRKAA2\tA0A2K5NF49_CERAT\nPRKAA2\tA0A2K5NFN6_CERAT\nPRKAA2\tA0A2K5S336_CEBCA\nPRKAA2\tA0A2K5S359_CEBCA\nPRKAA2\tA0A2K5UX90_MACFA\nPRKAA2\tA0A2K5UX98_MACFA\nPRKAA2\tA0A2K5XK42_MANLE\nPRKAA2\tA0A2K5XK45_MANLE\nPRKAA2\tA0A2K6B0E2_MACNE\nPRKAA2\tA0A2K6B0E8_MACNE\nPRKAA2\tA0A2K6GM52_PROCO\nPRKAA2\tA0A2K6MRP5_RHIBE\nPRKAA2\tA0A2K6MRR4_RHIBE\nPRKAA2\tA0A2K6QX69_RHIRO\nPRKAA2\tA0A2K6QX75_RHIRO\nPRKAA2\tA0A2K6V8F5_SAIBB\nPRKAA2\tA0A2K6V8G1_SAIBB\nPRKAA2\tA0A2K6V8H5_SAIBB\nPRKAA2\tA0A2K6V8H8_SAIBB\nPRKAA2\tA0A2P8YIB1_BLAGE\nPRKAA2\tA0A2R9CJR2_PANPA\nPRKAA2\tA0A2R9CJS1_PANPA\nPRKAA2\tA0A2S2R755_9HEMI\nPRKAA2\tA0A2U3V6N5_TURTR\nPRKAA2\tA0A2U3WYR5_ODORO\nPRKAA2\tA0A2U3ZV84_ODORO\nPRKAA2\tA0A2U4BXM6_TURTR\nPRKAA2\tA0A2U4BXN6_TURTR\nPRKAA2\tA0A2Y9EKU6_PHYMC\nPRKAA2\tA0A2Y9GKJ8_NEOSC\nPRKAA2\tA0A2Y9NJB6_DELLE\nPRKAA2\tA0A2Y9NQ82_DELLE\nPRKAA2\tA0A340XU30_LIPVE\nPRKAA2\tA0A341CC44_NEOAA\nPRKAA2\tA0A367IIW6_RHIST\nPRKAA2\tA0A383YNJ7_BALAS\nPRKAA2\tA0A383YNN3_BALAS\nPRKAA2\tA0A384CRE4_URSMA\nPRKAA2\tA0A3Q0CTZ5_MESAU\nPRKAA2\tA0A3Q0DT06_TARSY\nPRKAA2\tA0A3Q0GRF5_ALLSI\nPRKAA2\tA0A3Q7P154_CALUR\nPRKAA2\tA0A3Q7SLJ1_VULVU\nPRKAA2\tA0A3Q7U629_VULVU\nPRKAA2\tA0A3Q7VNL5_URSAR\nPRKAA2\tA0A3Q7WHR6_URSAR\nPRKAA2\tA0A3R7D1L5_CLOSI\nPRKAA2\tA0A452EL67_CAPHI\nPRKAA2\tA0A452GMZ9_9SAUR\nPRKAA2\tA0A452RAE5_URSAM\nPRKAA2\tA0A452RAF3_URSAM\nPRKAA2\tA0A452VAN0_URSMA\nPRKAA2\tA0A455B8N7_PHYMC\nPRKAA2\tA0A455B914_PHYMC\nPRKAA2\tA0A455BB20_PHYMC\nPRKAA2\tA0A4C1VTJ9_9NEOP\nPRKAA2\tA0A4W2DEJ0_BOBOX\nPRKAA2\tA0A4W3JXC8_CALMI\nPRKAA2\tA0A4W4H541_ELEEL\nPRKAA2\tA0A4W4HCN0_ELEEL\nPRKAA2\tA0A4W6CUD2_LATCA\nPRKAA2\tA0A4X2KDJ5_VOMUR\nPRKAA2\tA0A4X2KJ10_VOMUR\nPRKAA2\tA0A4X2KKV0_VOMUR\nPRKAA2\tA0A4Y2L4E7_ARAVE\nPRKAA2\tA0A5A4DW27_TACFU\nPRKAA2\tA0A5F5XED4_FELCA\nPRKAA2\tA0A5F8H8U1_MONDO\nPRKAA2\tA0A5F8HIL0_MONDO\nPRKAA2\tA0MZF5_HUMAN\nPRKAA2\tA1E4B7_MELGA\nPRKAA2\tAAPK2_HUMAN\nPRKAA2\tAAPK2_MOUSE\nPRKAA2\tAAPK2_PIG\nPRKAA2\tAAPK2_PONAB\nPRKAA2\tAAPK2_RAT\nPRKAA2\tB4F6W2_XENTR\nPRKAA2\tB7UEY6_CARAX\nPRKAA2\tC9W108_PIG\nPRKAA2\tD1MI48_BOVIN\nPRKAA2\tD1MI49_BOSMU\nPRKAA2\tD1MI52_BOSMU\nPRKAA2\tD1MI53_BOVIN\nPRKAA2\tD1MI54_BOVIN\nPRKAA2\tD1MI60_BOSMU\nPRKAA2\tD1MI61_BOVIN\nPRKAA2\tD2HX61_AILME\nPRKAA2\tD9PMC1_9ZZZZ\nPRKAA2\tE7F9C4_DANRE\nPRKAA2\tF1MQV7_BOVIN\nPRKAA2\tF1PIW7_CANLF\nPRKAA2\tF6R539_XENTR\nPRKAA2\tF6T684_ORNAN\nPRKAA2\tF6T6B0_ORNAN\nPRKAA2\tF7D386_MACMU\nPRKAA2\tF7ETH9_XENTR\nPRKAA2\tF7FPU2_MONDO\nPRKAA2\tF7IRM3_CALJA\nPRKAA2\tG1KFT1_ANOCA\nPRKAA2\tG1NG26_MELGA\nPRKAA2\tG1PPM6_MYOLU\nPRKAA2\tG1RX86_NOMLE\nPRKAA2\tG1SMV0_RABIT\nPRKAA2\tG3TA29_LOXAF\nPRKAA2\tG3V715_RAT\nPRKAA2\tG3W3Y0_SARHA\nPRKAA2\tG3W3Y1_SARHA\nPRKAA2\tG8XUU5_MOUSE\nPRKAA2\tH0VFT6_CAVPO\nPRKAA2\tH0WLB8_OTOGA\nPRKAA2\tH0ZH19_TAEGU\nPRKAA2\tH2L8J1_ORYLA\nPRKAA2\tH2N793_PONAB\nPRKAA2\tH2PZ45_PANTR\nPRKAA2\tH2UY55_TAKRU\nPRKAA2\tH3A9U2_LATCH\nPRKAA2\tH9FBH4_MACMU\nPRKAA2\tI3J3R7_ORENI\nPRKAA2\tI3J3R8_ORENI\nPRKAA2\tI3MFA4_ICTTR\nPRKAA2\tK7F734_PELSI\nPRKAA2\tK7F737_PELSI\nPRKAA2\tK9J960_HUMAN\nPRKAA2\tK9J973_HUMAN\nPRKAA2\tM3Y782_MUSPF\nPRKAA2\tQ2I3D6_HORSE\nPRKAA2\tQ2I3D7_HORSE\nPRKAA2\tQ2KP06_PIG\nPRKAA2\tQ2LAI0_CHICK\nPRKAA2\tQ2LGF9_HORSE\nPRKAA2\tQ70PH3_PIG\nPRKAA2\tQ8UVW8_XENLA\nPRKAA2\tT2MFI8_HYDVU\nPRKAA2\tU3CGM7_CALJA\nPRKAA2\tU3IIN8_ANAPP\nPRKAA2\tU3KCR2_FICAL\nPRKAA2\tV8P811_OPHHA\nPRKAA2\tW5PDB5_SHEEP\nPRKAA2\tW5PDB6_SHEEP\nPRKAA2\tW5UDD8_ICTPU\n','From\tTo\nMYLK4\tA0A096NKJ1_PAPAN\nMYLK4\tA0A0D9R4C5_CHLSB\nMYLK4\tA0A0R4J7G3_DANRE\nMYLK4\tA0A0S7I7Z9_9TELE\nMYLK4\tA0A0S7LTK1_9TELE\nMYLK4\tA0A165R1Z8_SINCH\nMYLK4\tA0A1A7ZW65_NOTFU\nMYLK4\tA0A1A7ZYI4_NOTFU\nMYLK4\tA0A1A8FX54_9TELE\nMYLK4\tA0A1L8FSB4_XENLA\nMYLK4\tA0A1L8FXQ0_XENLA\nMYLK4\tA0A1S2ZNV3_ERIEU\nMYLK4\tA0A1S3F7N6_DIPOR\nMYLK4\tA0A1U7TF71_TARSY\nMYLK4\tA0A1V4JWM7_PATFA\nMYLK4\tA0A1V4JWM9_PATFA\nMYLK4\tA0A218UP55_9PASE\nMYLK4\tA0A250XZW4_CASCN\nMYLK4\tA0A287AC29_PIG\nMYLK4\tA0A2I0MX82_COLLI\nMYLK4\tA0A2I2UAH3_FELCA\nMYLK4\tA0A2I4D0H1_9TELE\nMYLK4\tA0A2I4D0H4_9TELE\nMYLK4\tA0A2I4D0L0_9TELE\nMYLK4\tA0A2K5CMC5_AOTNA\nMYLK4\tA0A2K5CMD1_AOTNA\nMYLK4\tA0A2K5HYV4_COLAP\nMYLK4\tA0A2K5MMQ9_CERAT\nMYLK4\tA0A2K5Q552_CEBCA\nMYLK4\tA0A2K5TZF8_MACFA\nMYLK4\tA0A2K6ACU2_MANLE\nMYLK4\tA0A2K6C8G0_MACNE\nMYLK4\tA0A2K6GNG5_PROCO\nMYLK4\tA0A2K6LQ12_RHIBE\nMYLK4\tA0A2K6QIH8_RHIRO\nMYLK4\tA0A2K6TI29_SAIBB\nMYLK4\tA0A2K6TI59_SAIBB\nMYLK4\tA0A2R8VI35_MOUSE\nMYLK4\tA0A2R8Y4U5_HUMAN\nMYLK4\tA0A2R9B3R8_PANPA\nMYLK4\tA0A2R9B7K9_PANPA\nMYLK4\tA0A2U3ZNY1_ODORO\nMYLK4\tA0A2U4C537_TURTR\nMYLK4\tA0A2Y9HTS2_NEOSC\nMYLK4\tA0A337SCZ6_FELCA\nMYLK4\tA0A337SKL6_FELCA\nMYLK4\tA0A340WRM5_LIPVE\nMYLK4\tA0A341B7E7_NEOAA\nMYLK4\tA0A384C8Z5_URSMA\nMYLK4\tA0A3B3I6P7_ORYLA\nMYLK4\tA0A3Q0CE48_MESAU\nMYLK4\tA0A3Q0CIY5_MESAU\nMYLK4\tA0A3Q0CL44_MESAU\nMYLK4\tA0A3Q0GNZ0_ALLSI\nMYLK4\tA0A3Q0GTN9_ALLSI\nMYLK4\tA0A3Q1LXF9_BOVIN\nMYLK4\tA0A3Q1M5L3_BOVIN\nMYLK4\tA0A3Q1MRK3_BOVIN\nMYLK4\tA0A3Q2IC91_HORSE\nMYLK4\tA0A3Q2TYD3_CHICK\nMYLK4\tA0A3Q7QLQ3_CALUR\nMYLK4\tA0A3Q7SAJ1_VULVU\nMYLK4\tA0A3Q7WPC0_URSAR\nMYLK4\tA0A452CB19_BALAS\nMYLK4\tA0A452CBC7_BALAS\nMYLK4\tA0A452CBJ6_BALAS\nMYLK4\tA0A452DVS9_CAPHI\nMYLK4\tA0A452DVZ5_CAPHI\nMYLK4\tA0A452DW64_CAPHI\nMYLK4\tA0A452DW74_CAPHI\nMYLK4\tA0A452IS72_9SAUR\nMYLK4\tA0A452ISL5_9SAUR\nMYLK4\tA0A452UXT4_URSMA\nMYLK4\tA0A455ADN0_PHYMC\nMYLK4\tA0A455ADN3_PHYMC\nMYLK4\tA0A455AE25_PHYMC\nMYLK4\tA0A455AG30_PHYMC\nMYLK4\tA0A493TJC3_ANAPP\nMYLK4\tA0A4W2BQ64_BOBOX\nMYLK4\tA0A4W2BSG8_BOBOX\nMYLK4\tA0A4W2DTG3_BOBOX\nMYLK4\tA0A4W2DU69_BOBOX\nMYLK4\tA0A4W2I7U5_BOBOX\nMYLK4\tA0A4W5ML18_9TELE\nMYLK4\tA0A4W5MNC3_9TELE\nMYLK4\tA0A4X1SR14_PIG\nMYLK4\tA0A4X1SSK3_PIG\nMYLK4\tA0A4X2K3H2_VOMUR\nMYLK4\tA0A4X2KBG0_VOMUR\nMYLK4\tA0A5F4C9W3_CANLF\nMYLK4\tA0A5F4VY02_CALJA\nMYLK4\tA0A5F5Y6T6_FELCA\nMYLK4\tA0A5F7ZMX8_MACMU\nMYLK4\tA0A5F8A9U2_MACMU\nMYLK4\tA0A5F8AAQ0_MACMU\nMYLK4\tA0A5F8GHK1_MONDO\nMYLK4\tA0A5F8H9C4_MONDO\nMYLK4\tA0A5F8HBU3_MONDO\nMYLK4\tA0A5F8HHA2_MONDO\nMYLK4\tA0A5F8MPR5_MOUSE\nMYLK4\tA0A5F9DHJ8_RABIT\nMYLK4\tA0A5F9DIG1_RABIT\nMYLK4\tA0A5F9DJE9_RABIT\nMYLK4\tA0A5F9DLM5_RABIT\nMYLK4\tA0A5F9DN99_RABIT\nMYLK4\tA0A5G2QP30_PIG\nMYLK4\tA0A5G2QXB9_PIG\nMYLK4\tA0A5G3HZ47_XENTR\nMYLK4\tA0A5G3JQI4_XENTR\nMYLK4\tA0A5G3JQW0_XENTR\nMYLK4\tA0A5K1UH87_CALJA\nMYLK4\tE1BU17_CHICK\nMYLK4\tF1MG18_BOVIN\nMYLK4\tF1PKT1_CANLF\nMYLK4\tF6UCL2_XENTR\nMYLK4\tF7A949_HORSE\nMYLK4\tF7DMT4_MACMU\nMYLK4\tF7DUB0_XENTR\nMYLK4\tF7F8Z3_CALJA\nMYLK4\tF7G9U2_ORNAN\nMYLK4\tF8WIP5_MOUSE\nMYLK4\tG1LWP0_AILME\nMYLK4\tG1MZA1_MELGA\nMYLK4\tG1PHN5_MYOLU\nMYLK4\tG1RJ43_NOMLE\nMYLK4\tG1SN58_RABIT\nMYLK4\tG3TDI5_LOXAF\nMYLK4\tH0UTP8_CAVPO\nMYLK4\tH0X0I3_OTOGA\nMYLK4\tH0YV66_TAEGU\nMYLK4\tH2MB15_ORYLA\nMYLK4\tH2MB22_ORYLA\nMYLK4\tH2PHR2_PONAB\nMYLK4\tH2QS75_PANTR\nMYLK4\tH9GRW3_ANOCA\nMYLK4\tI3L6E0_PIG\nMYLK4\tI3N441_ICTTR\nMYLK4\tJ9P428_CANLF\nMYLK4\tK7E3G5_MONDO\nMYLK4\tK7G769_PELSI\nMYLK4\tK7G776_PELSI\nMYLK4\tM0R632_RAT\nMYLK4\tM3XMV5_MUSPF\nMYLK4\tMYLK4_HUMAN\nMYLK4\tMYLK4_MOUSE\nMYLK4\tQ1LVF9_DANRE\nMYLK4\tQ1LVG0_DANRE\nMYLK4\tQ5BLI8_DANRE\nMYLK4\tQ6DJ97_XENTR\nMYLK4\tU3J1Y7_ANAPP\nMYLK4\tV8P192_OPHHA\n','From\tTo\nRPS6KA1\tA0A096MS33_PAPAN\nRPS6KA1\tA0A0D9S842_CHLSB\nRPS6KA1\tA0A0P6J7Q5_HETGA\nRPS6KA1\tA0A0V1HU86_TRIPS\nRPS6KA1\tA0A0V1HUY3_TRIPS\nRPS6KA1\tA0A0V1JQ35_TRIPS\nRPS6KA1\tA0A151NJS9_ALLMI\nRPS6KA1\tA0A151NJT0_ALLMI\nRPS6KA1\tA0A1A7WA66_9TELE\nRPS6KA1\tA0A1A7ZC06_NOTFU\nRPS6KA1\tA0A1A8C1U2_9TELE\nRPS6KA1\tA0A1A8DJV4_9TELE\nRPS6KA1\tA0A1A8EDZ8_9TELE\nRPS6KA1\tA0A1A8GAV4_9TELE\nRPS6KA1\tA0A1A8H9K1_9TELE\nRPS6KA1\tA0A1A8J4H5_NOTKU\nRPS6KA1\tA0A1A8JWI6_NOTKU\nRPS6KA1\tA0A1A8LEG9_9TELE\nRPS6KA1\tA0A1A8P054_9TELE\nRPS6KA1\tA0A1A8QTQ3_9TELE\nRPS6KA1\tA0A1A8SF14_9TELE\nRPS6KA1\tA0A1A8SHA7_9TELE\nRPS6KA1\tA0A1A8V5Z7_NOTFU\nRPS6KA1\tA0A1D5PCA5_CHICK\nRPS6KA1\tA0A1D5PNT2_CHICK\nRPS6KA1\tA0A1D5QBJ3_MACMU\nRPS6KA1\tA0A1D5QHC8_MACMU\nRPS6KA1\tA0A1D5QSJ2_MACMU\nRPS6KA1\tA0A1D5R395_MACMU\nRPS6KA1\tA0A1D5R4I1_MACMU\nRPS6KA1\tA0A1S3APX6_ERIEU\nRPS6KA1\tA0A1S3AQA2_ERIEU\nRPS6KA1\tA0A1S3AQC5_ERIEU\nRPS6KA1\tA0A1S3GG22_DIPOR\nRPS6KA1\tA0A1U7QUU2_MESAU\nRPS6KA1\tA0A1U7R554_MESAU\nRPS6KA1\tA0A1U7RFY5_ALLSI\nRPS6KA1\tA0A1U7UM12_TARSY\nRPS6KA1\tA0A1U7UZ65_TARSY\nRPS6KA1\tA0A250Y4Q6_CASCN\nRPS6KA1\tA0A286XCG0_CAVPO\nRPS6KA1\tA0A286XR34_CAVPO\nRPS6KA1\tA0A286XTG7_CAVPO\nRPS6KA1\tA0A286YAW9_DANRE\nRPS6KA1\tA0A286ZNG1_PIG\nRPS6KA1\tA0A287AJ05_PIG\nRPS6KA1\tA0A287BHE2_PIG\nRPS6KA1\tA0A287D3E2_ICTTR\nRPS6KA1\tA0A287D5V7_ICTTR\nRPS6KA1\tA0A287DD79_ICTTR\nRPS6KA1\tA0A2D0RN30_ICTPU\nRPS6KA1\tA0A2D0RN31_ICTPU\nRPS6KA1\tA0A2D0RN33_ICTPU\nRPS6KA1\tA0A2D0RP27_ICTPU\nRPS6KA1\tA0A2D0RPN6_ICTPU\nRPS6KA1\tA0A2I0LLR1_COLLI\nRPS6KA1\tA0A2I2U2K0_FELCA\nRPS6KA1\tA0A2I2UPE2_FELCA\nRPS6KA1\tA0A2I3G2C2_NOMLE\nRPS6KA1\tA0A2I3G5U5_NOMLE\nRPS6KA1\tA0A2I3HZ94_NOMLE\nRPS6KA1\tA0A2I3M5W9_PAPAN\nRPS6KA1\tA0A2I3NAK5_PAPAN\nRPS6KA1\tA0A2I3RZK8_PANTR\nRPS6KA1\tA0A2I3TE75_PANTR\nRPS6KA1\tA0A2I3TIH7_PANTR\nRPS6KA1\tA0A2I3TPM9_PANTR\nRPS6KA1\tA0A2I3TXG7_PANTR\nRPS6KA1\tA0A2K5BVN2_AOTNA\nRPS6KA1\tA0A2K5BVP5_AOTNA\nRPS6KA1\tA0A2K5BVS1_AOTNA\nRPS6KA1\tA0A2K5IFS5_COLAP\nRPS6KA1\tA0A2K5IFZ9_COLAP\nRPS6KA1\tA0A2K5IG80_COLAP\nRPS6KA1\tA0A2K5LWB9_CERAT\nRPS6KA1\tA0A2K5LWC1_CERAT\nRPS6KA1\tA0A2K5LWD1_CERAT\nRPS6KA1\tA0A2K5SD52_CEBCA\nRPS6KA1\tA0A2K5SD62_CEBCA\nRPS6KA1\tA0A2K5SD68_CEBCA\nRPS6KA1\tA0A2K5SD70_CEBCA\nRPS6KA1\tA0A2K5SD78_CEBCA\nRPS6KA1\tA0A2K5VR69_MACFA\nRPS6KA1\tA0A2K5VRB4_MACFA\nRPS6KA1\tA0A2K5VRG4_MACFA\nRPS6KA1\tA0A2K6AEA7_MANLE\nRPS6KA1\tA0A2K6AEC4_MANLE\nRPS6KA1\tA0A2K6AED4_MANLE\nRPS6KA1\tA0A2K6AEE0_MANLE\nRPS6KA1\tA0A2K6AEF2_MANLE\nRPS6KA1\tA0A2K6AEF6_MANLE\nRPS6KA1\tA0A2K6DY42_MACNE\nRPS6KA1\tA0A2K6DY48_MACNE\nRPS6KA1\tA0A2K6DY54_MACNE\nRPS6KA1\tA0A2K6FHB3_PROCO\nRPS6KA1\tA0A2K6FHB8_PROCO\nRPS6KA1\tA0A2K6FHC3_PROCO\nRPS6KA1\tA0A2K6FHC9_PROCO\nRPS6KA1\tA0A2K6KCL5_RHIBE\nRPS6KA1\tA0A2K6KCM1_RHIBE\nRPS6KA1\tA0A2K6KCM3_RHIBE\nRPS6KA1\tA0A2K6KCQ7_RHIBE\nRPS6KA1\tA0A2K6QS01_RHIRO\nRPS6KA1\tA0A2K6QS13_RHIRO\nRPS6KA1\tA0A2K6QS22_RHIRO\nRPS6KA1\tA0A2K6QS29_RHIRO\nRPS6KA1\tA0A2K6QS34_RHIRO\nRPS6KA1\tA0A2K6UR82_SAIBB\nRPS6KA1\tA0A2K6URE6_SAIBB\nRPS6KA1\tA0A2K6URH6_SAIBB\nRPS6KA1\tA0A2K6URI6_SAIBB\nRPS6KA1\tA0A2K6URJ1_SAIBB\nRPS6KA1\tA0A2K6URJ3_SAIBB\nRPS6KA1\tA0A2K6URK4_SAIBB\nRPS6KA1\tA0A2R8QDF2_DANRE\nRPS6KA1\tA0A2R9C502_PANPA\nRPS6KA1\tA0A2R9C519_PANPA\nRPS6KA1\tA0A2R9C536_PANPA\nRPS6KA1\tA0A2R9C550_PANPA\nRPS6KA1\tA0A2R9C5C7_PANPA\nRPS6KA1\tA0A2R9C7W4_PANPA\nRPS6KA1\tA0A2U3VIR7_ODORO\nRPS6KA1\tA0A2U3VIR8_ODORO\nRPS6KA1\tA0A2U3ZA41_ODORO\nRPS6KA1\tA0A2Y9GKT2_NEOSC\nRPS6KA1\tA0A2Y9GKX3_NEOSC\nRPS6KA1\tA0A2Y9GL35_NEOSC\nRPS6KA1\tA0A2Y9LCL2_DELLE\nRPS6KA1\tA0A2Y9LCR6_DELLE\nRPS6KA1\tA0A2Y9LFC7_DELLE\nRPS6KA1\tA0A2Y9LFD2_DELLE\nRPS6KA1\tA0A2Y9LK46_DELLE\nRPS6KA1\tA0A2Y9LK51_DELLE\nRPS6KA1\tA0A2Y9LQ96_DELLE\nRPS6KA1\tA0A2Y9LQA1_DELLE\nRPS6KA1\tA0A340XCE6_LIPVE\nRPS6KA1\tA0A340XCL2_LIPVE\nRPS6KA1\tA0A340XGH3_LIPVE\nRPS6KA1\tA0A340XIR9_LIPVE\nRPS6KA1\tA0A341D5M9_NEOAA\nRPS6KA1\tA0A341D5N6_NEOAA\nRPS6KA1\tA0A341D5P1_NEOAA\nRPS6KA1\tA0A341D619_NEOAA\nRPS6KA1\tA0A341D8L2_NEOAA\nRPS6KA1\tA0A383ZUH1_BALAS\nRPS6KA1\tA0A384CD65_URSMA\nRPS6KA1\tA0A384CD84_URSMA\nRPS6KA1\tA0A3Q0D1P8_MESAU\nRPS6KA1\tA0A3Q0DJ29_TARSY\nRPS6KA1\tA0A3Q0G4C3_ALLSI\nRPS6KA1\tA0A3Q0G4E3_ALLSI\nRPS6KA1\tA0A3Q0G526_ALLSI\nRPS6KA1\tA0A3Q0G7X4_ALLSI\nRPS6KA1\tA0A3Q0G8S0_ALLSI\nRPS6KA1\tA0A3Q1LK61_BOVIN\nRPS6KA1\tA0A3Q1LNM0_BOVIN\nRPS6KA1\tA0A3Q1M0H8_BOVIN\nRPS6KA1\tA0A3Q1MLN6_BOVIN\nRPS6KA1\tA0A3Q2GTT1_HORSE\nRPS6KA1\tA0A3Q2GVU3_HORSE\nRPS6KA1\tA0A3Q2U5L9_CHICK\nRPS6KA1\tA0A3Q3ADZ8_CHICK\nRPS6KA1\tA0A3Q3AVG7_CHICK\nRPS6KA1\tA0A3Q7MW56_CALUR\nRPS6KA1\tA0A3Q7MW74_CALUR\nRPS6KA1\tA0A3Q7MXB7_CALUR\nRPS6KA1\tA0A3Q7P6Z5_CALUR\nRPS6KA1\tA0A3Q7T0C8_VULVU\nRPS6KA1\tA0A3Q7TYA2_VULVU\nRPS6KA1\tA0A3Q7USQ3_VULVU\nRPS6KA1\tA0A3Q7X5W8_URSAR\nRPS6KA1\tA0A3Q7XE38_URSAR\nRPS6KA1\tA0A3Q7XZ59_URSAR\nRPS6KA1\tA0A452EPI5_CAPHI\nRPS6KA1\tA0A452EPQ2_CAPHI\nRPS6KA1\tA0A452EQI4_CAPHI\nRPS6KA1\tA0A452EQW8_CAPHI\nRPS6KA1\tA0A452R6B3_URSAM\nRPS6KA1\tA0A452R6Q7_URSAM\nRPS6KA1\tA0A452R6Z7_URSAM\nRPS6KA1\tA0A452R7S0_URSAM\nRPS6KA1\tA0A452R7T5_URSAM\nRPS6KA1\tA0A452R7U7_URSAM\nRPS6KA1\tA0A452R7X0_URSAM\nRPS6KA1\tA0A452R7X8_URSAM\nRPS6KA1\tA0A452R7Y6_URSAM\nRPS6KA1\tA0A452R8D1_URSAM\nRPS6KA1\tA0A455B4R3_PHYMC\nRPS6KA1\tA0A493SZA4_ANAPP\nRPS6KA1\tA0A493T1D6_ANAPP\nRPS6KA1\tA0A493T773_ANAPP\nRPS6KA1\tA0A493TLY7_ANAPP\nRPS6KA1\tA0A493TV82_ANAPP\nRPS6KA1\tA0A493TX26_ANAPP\nRPS6KA1\tA0A493U2V5_ANAPP\nRPS6KA1\tA0A4W2CUX8_BOBOX\nRPS6KA1\tA0A4W2CZ47_BOBOX\nRPS6KA1\tA0A4W2CZ65_BOBOX\nRPS6KA1\tA0A4W2EI91_BOBOX\nRPS6KA1\tA0A4W2EJJ9_BOBOX\nRPS6KA1\tA0A4W2F9A9_BOBOX\nRPS6KA1\tA0A4W2F9D8_BOBOX\nRPS6KA1\tA0A4W2FAU9_BOBOX\nRPS6KA1\tA0A4W2FAW2_BOBOX\nRPS6KA1\tA0A4W2H9D3_BOBOX\nRPS6KA1\tA0A4W2H9E7_BOBOX\nRPS6KA1\tA0A4W3JM65_CALMI\nRPS6KA1\tA0A4W3JPH9_CALMI\nRPS6KA1\tA0A4W3JPK2_CALMI\nRPS6KA1\tA0A4W3JPK8_CALMI\nRPS6KA1\tA0A4W3K022_CALMI\nRPS6KA1\tA0A4W3K035_CALMI\nRPS6KA1\tA0A4W3K1U8_CALMI\nRPS6KA1\tA0A4W3KEJ0_CALMI\nRPS6KA1\tA0A4X1VFJ0_PIG\nRPS6KA1\tA0A4X1VFN0_PIG\nRPS6KA1\tA0A4X1VG05_PIG\nRPS6KA1\tA0A4X1VGD5_PIG\nRPS6KA1\tA0A4X1VJT1_PIG\nRPS6KA1\tA0A4X1VKK2_PIG\nRPS6KA1\tA0A4X1VKP0_PIG\nRPS6KA1\tA0A4X1VMW9_PIG\nRPS6KA1\tA0A4X2KAE4_VOMUR\nRPS6KA1\tA0A4X2KHP8_VOMUR\nRPS6KA1\tA0A4X2KHV5_VOMUR\nRPS6KA1\tA0A4X2KHY5_VOMUR\nRPS6KA1\tA0A4Y2WWT9_ARAVE\nRPS6KA1\tA0A559LKB4_FUSOX\nRPS6KA1\tA0A5F4C350_CANLF\nRPS6KA1\tA0A5F4CGV9_CANLF\nRPS6KA1\tA0A5F4CWJ0_CANLF\nRPS6KA1\tA0A5F4VUQ3_CALJA\nRPS6KA1\tA0A5F8GH41_MONDO\nRPS6KA1\tA0A5F8H8P5_MONDO\nRPS6KA1\tA0A5F9C7V1_RABIT\nRPS6KA1\tA0A5F9CU69_RABIT\nRPS6KA1\tA0A5F9D4X6_RABIT\nRPS6KA1\tA0A5G2QVK7_PIG\nRPS6KA1\tA0A5G3HSL0_XENTR\nRPS6KA1\tA0A5G3HSM0_XENTR\nRPS6KA1\tA0A5G3HTK4_XENTR\nRPS6KA1\tA0A5G3HTL1_XENTR\nRPS6KA1\tA0A5G3HTV4_XENTR\nRPS6KA1\tA0A5G3IPB8_XENTR\nRPS6KA1\tA0A5G3JIE4_XENTR\nRPS6KA1\tA0A5G3JIT2_XENTR\nRPS6KA1\tA4IFF4_BOVIN\nRPS6KA1\tA6NND1_HUMAN\nRPS6KA1\tA8WFX4_DANRE\nRPS6KA1\tB1WAT6_XENTR\nRPS6KA1\tE2RQU4_CANLF\nRPS6KA1\tE9PAN7_HUMAN\nRPS6KA1\tE9PGT3_HUMAN\nRPS6KA1\tE9PMM7_HUMAN\nRPS6KA1\tE9PPC1_HUMAN\nRPS6KA1\tE9PPN6_HUMAN\nRPS6KA1\tE9PWV3_MOUSE\nRPS6KA1\tF1LXV0_RAT\nRPS6KA1\tF1PIL4_CANLF\nRPS6KA1\tF1QYQ1_DANRE\nRPS6KA1\tF1RAV5_DANRE\nRPS6KA1\tF1STS3_PIG\nRPS6KA1\tF6PQK5_ORNAN\nRPS6KA1\tF6PVL3_XENTR\nRPS6KA1\tF6PXZ6_XENTR\nRPS6KA1\tF6Q8A4_MOUSE\nRPS6KA1\tF6RQA2_MOUSE\nRPS6KA1\tF6RWB8_HORSE\nRPS6KA1\tF6S354_HORSE\nRPS6KA1\tF6X0J4_MONDO\nRPS6KA1\tF6YKQ4_XENTR\nRPS6KA1\tF6YXI9_XENTR\nRPS6KA1\tF7CI72_XENTR\nRPS6KA1\tF7G9C5_CALJA\nRPS6KA1\tF7G9I4_CALJA\nRPS6KA1\tF7GM80_MACMU\nRPS6KA1\tF7H7V7_MACMU\nRPS6KA1\tF7H7W3_MACMU\nRPS6KA1\tG1LW37_AILME\nRPS6KA1\tG1NXY5_MYOLU\nRPS6KA1\tG1RDR9_NOMLE\nRPS6KA1\tG1U4N1_RABIT\nRPS6KA1\tG3SV84_LOXAF\nRPS6KA1\tG3UZI3_MOUSE\nRPS6KA1\tG3WGG0_SARHA\nRPS6KA1\tG7MHW8_MACMU\nRPS6KA1\tH0V219_CAVPO\nRPS6KA1\tH0WNW3_OTOGA\nRPS6KA1\tH0YRY6_TAEGU\nRPS6KA1\tH2N8F2_PONAB\nRPS6KA1\tH2TQW4_TAKRU\nRPS6KA1\tH2TQW6_TAKRU\nRPS6KA1\tH2TQW7_TAKRU\nRPS6KA1\tH3AFS7_LATCH\nRPS6KA1\tH9FY69_MACMU\nRPS6KA1\tH9GND5_ANOCA\nRPS6KA1\tH9H1L4_MELGA\nRPS6KA1\tI3M441_ICTTR\nRPS6KA1\tK7CK76_PANTR\nRPS6KA1\tK7DY81_DANRE\nRPS6KA1\tK7DYC0_DANRE\nRPS6KA1\tK7GI40_PELSI\nRPS6KA1\tKS6A1_HUMAN\nRPS6KA1\tKS6A1_MOUSE\nRPS6KA1\tKS6A1_RAT\nRPS6KA1\tM3WHN5_FELCA\nRPS6KA1\tM3YW83_MUSPF\nRPS6KA1\tQ08CV8_DANRE\nRPS6KA1\tQ28CC6_XENTR\nRPS6KA1\tQ3TIM6_MOUSE\nRPS6KA1\tQ3UFZ5_MOUSE\nRPS6KA1\tQ505N6_MOUSE\nRPS6KA1\tQ5SVM6_HUMAN\nRPS6KA1\tQ5SVM7_HUMAN\nRPS6KA1\tQ6GR32_XENLA\nRPS6KA1\tQ810V8_MOUSE\nRPS6KA1\tQ9Z278_MOUSE\nRPS6KA1\tU3EP01_CALJA\nRPS6KA1\tU3ITV1_ANAPP\nRPS6KA1\tU3JC64_FICAL\nRPS6KA1\tV9KE64_CALMI\nRPS6KA1\tV9KGW9_CALMI\nRPS6KA1\tW5P2Q3_SHEEP\n','From\tTo\nPIM1\tA0A010RFD1_9PEZI\nPIM1\tA0A015K2G0_RHIIW\nPIM1\tA0A017SN12_9EURO\nPIM1\tA0A022W5Y6_TRIRU\nPIM1\tA0A022W6R1_TRIRU\nPIM1\tA0A022XWH3_TRISD\nPIM1\tA0A022XXN9_TRISD\nPIM1\tA0A024RD25_HUMAN\nPIM1\tA0A024SL92_HYPJR\nPIM1\tA0A059JD78_TRIIM\nPIM1\tA0A060S682_PYCCI\nPIM1\tA0A061ALV9_RHOTO\nPIM1\tA0A061AYU3_CYBFA\nPIM1\tA0A063BXV3_USTVR\nPIM1\tA0A066WHZ3_TILAU\nPIM1\tA0A066WX09_COLSU\nPIM1\tA0A067MH98_9AGAM\nPIM1\tA0A067N8L4_PLEOS\nPIM1\tA0A067PEG7_9AGAM\nPIM1\tA0A067SRC3_GALM3\nPIM1\tA0A068RDM9_9FUNG\nPIM1\tA0A068SBF1_9FUNG\nPIM1\tA0A072PGQ0_9EURO\nPIM1\tA0A074S1E2_9AGAM\nPIM1\tA0A074VQG4_9PEZI\nPIM1\tA0A074WQ43_9PEZI\nPIM1\tA0A074XIE0_AURPU\nPIM1\tA0A074YQV9_AURSE\nPIM1\tA0A077QZ07_9BASI\nPIM1\tA0A077WGU2_9FUNG\nPIM1\tA0A077WTZ3_9FUNG\nPIM1\tA0A084B9L4_STACB\nPIM1\tA0A084G176_9PEZI\nPIM1\tA0A086T3W1_ACRC1\nPIM1\tA0A090MEZ3_9HYPO\nPIM1\tA0A093VB43_TALMA\nPIM1\tA0A093XMH3_9PEZI\nPIM1\tA0A093YNZ5_9PEZI\nPIM1\tA0A093ZE14_9PEZI\nPIM1\tA0A094A9B5_9PEZI\nPIM1\tA0A094AFS2_9PEZI\nPIM1\tA0A094BYN6_9PEZI\nPIM1\tA0A094C524_9PEZI\nPIM1\tA0A094EZB8_9PEZI\nPIM1\tA0A094FDY0_9PEZI\nPIM1\tA0A094FMW6_9PEZI\nPIM1\tA0A094GQE3_9PEZI\nPIM1\tA0A094H3W5_9PEZI\nPIM1\tA0A094HJJ3_9PEZI\nPIM1\tA0A095D0I4_CRYGR\nPIM1\tA0A096NHU6_PAPAN\nPIM1\tA0A096PE80_FUSPS\nPIM1\tA0A099P3N7_PICKU\nPIM1\tA0A0A1MZR4_RHIZD\nPIM1\tA0A0A1NIM9_RHIZD\nPIM1\tA0A0A1TFW6_9HYPO\nPIM1\tA0A0A1UVQ4_9HYPO\nPIM1\tA0A0A2JBX0_PENEN\nPIM1\tA0A0A2L202_PENIT\nPIM1\tA0A0A2VXW3_BEABA\nPIM1\tA0A0A8L906_9SACH\nPIM1\tA0A0B0E541_NEUCS\nPIM1\tA0A0B1PC67_UNCNE\nPIM1\tA0A0B2V2Y7_TOXCA\nPIM1\tA0A0B2VFC8_TOXCA\nPIM1\tA0A0B2W2F7_TOXCA\nPIM1\tA0A0B2WL56_METAS\nPIM1\tA0A0B4HPV7_METGA\nPIM1\tA0A0B7FCI0_THACB\nPIM1\tA0A0B7JHT5_BIOOC\nPIM1\tA0A0B7MY97_9FUNG\nPIM1\tA0A0B8RZA1_PIG\nPIM1\tA0A0C2IYZ5_9PEZI\nPIM1\tA0A0C2W8X6_AMAMU\nPIM1\tA0A0C2XST5_HEBCY\nPIM1\tA0A0C3A3U8_9AGAM\nPIM1\tA0A0C3BN82_9AGAM\nPIM1\tA0A0C3HZJ1_9PEZI\nPIM1\tA0A0C3JKH6_PISTI\nPIM1\tA0A0C3S5F0_PHLGI\nPIM1\tA0A0C4DM94_MAGP6\nPIM1\tA0A0C7AX12_RHIZD\nPIM1\tA0A0C7AX16_RHIZD\nPIM1\tA0A0C7N585_9SACH\nPIM1\tA0A0C9N8V1_9FUNG\nPIM1\tA0A0C9SLD2_PLICR\nPIM1\tA0A0C9T519_PAXIN\nPIM1\tA0A0C9WAR8_9AGAM\nPIM1\tA0A0C9WX67_9AGAR\nPIM1\tA0A0C9ZP40_9AGAM\nPIM1\tA0A0C9ZZR4_9AGAM\nPIM1\tA0A0D0D3S4_9AGAM\nPIM1\tA0A0D0V694_9TREE\nPIM1\tA0A0D0VWI7_CRYGA\nPIM1\tA0A0D0X113_9TREE\nPIM1\tA0A0D0YHE0_9TREE\nPIM1\tA0A0D1CDJ3_USTMA\nPIM1\tA0A0D1X903_9EURO\nPIM1\tA0A0D1XST7_9PEZI\nPIM1\tA0A0D1Z3R0_EXOME\nPIM1\tA0A0D2BDW7_9EURO\nPIM1\tA0A0D2CPE9_9EURO\nPIM1\tA0A0D2DZE0_9EURO\nPIM1\tA0A0D2E9S5_XYLBA\nPIM1\tA0A0D2F6I2_9EURO\nPIM1\tA0A0D2FIH9_9EURO\nPIM1\tA0A0D2GQM6_9EURO\nPIM1\tA0A0D2I8K3_9EURO\nPIM1\tA0A0D2KD65_9EURO\nPIM1\tA0A0D2NLL0_9AGAR\nPIM1\tA0A0D2XX10_FUSO4\nPIM1\tA0A0D7BR97_9AGAR\nPIM1\tA0A0D9P6G1_METAN\nPIM1\tA0A0D9RDG7_CHLSB\nPIM1\tA0A0E9NJS8_SAICN\nPIM1\tA0A0F0HZM6_ASPPU\nPIM1\tA0A0F2LZP8_SPOSC\nPIM1\tA0A0F4G873_9PEZI\nPIM1\tA0A0F4Z5X4_TALEM\nPIM1\tA0A0F4ZI54_9PEZI\nPIM1\tA0A0F7S9R7_9BASI\nPIM1\tA0A0F7TB83_9EURO\nPIM1\tA0A0F8D1T0_CERFI\nPIM1\tA0A0F8UUP0_9EURO\nPIM1\tA0A0F8WGN2_9EURO\nPIM1\tA0A0F9XFS0_TRIHA\nPIM1\tA0A0G2FR99_9EURO\nPIM1\tA0A0G2FVS9_9PEZI\nPIM1\tA0A0G2GYX9_9PEZI\nPIM1\tA0A0G2HS14_9EURO\nPIM1\tA0A0G4MGH5_9PEZI\nPIM1\tA0A0G4P5T1_PENCA\nPIM1\tA0A0H1BMX2_9EURO\nPIM1\tA0A0H2S394_9AGAM\nPIM1\tA0A0I9ZFB4_GIBFU\nPIM1\tA0A0J0XEH3_9TREE\nPIM1\tA0A0J5PYZ3_ASPFM\nPIM1\tA0A0J6IBL1_COCPO\nPIM1\tA0A0J6YCW7_COCIT\nPIM1\tA0A0J8R6X0_COCIT\nPIM1\tA0A0J8RGT0_COCIT\nPIM1\tA0A0J9X4P2_GEOCN\nPIM1\tA0A0J9XEH2_GEOCN\nPIM1\tA0A0K3C6Q9_RHOTO\nPIM1\tA0A0K6G6Y2_9AGAM\nPIM1\tA0A0K8LHK0_9EURO\nPIM1\tA0A0L0HFF9_SPIPD\nPIM1\tA0A0L0N475_9HYPO\nPIM1\tA0A0L0NTB0_CANAR\nPIM1\tA0A0L0T0K6_ALLM3\nPIM1\tA0A0L0T4E5_ALLM3\nPIM1\tA0A0L0VF30_9BASI\nPIM1\tA0A0L1J5H2_ASPNO\nPIM1\tA0A0L6VPB1_9BASI\nPIM1\tA0A0L6WQ18_9AGAR\nPIM1\tA0A0L8RMQ0_SACEU\nPIM1\tA0A0L8VVG2_9SACH\nPIM1\tA0A0M8MR90_9BASI\nPIM1\tA0A0M8MWP4_9HYPO\nPIM1\tA0A0M8PDG6_9EURO\nPIM1\tA0A0M9EU30_FUSLA\nPIM1\tA0A0N1HAR8_9EURO\nPIM1\tA0A0P1KN02_9SACH\nPIM1\tA0A0P7BSK3_9HYPO\nPIM1\tA0A0R4ISY2_DANRE\nPIM1\tA0A0S2LII6_CRYNJ\nPIM1\tA0A0S6XQG3_9FUNG\nPIM1\tA0A0S7DRN1_9EURO\nPIM1\tA0A0S7LYF7_9TELE\nPIM1\tA0A0S7LYS1_9TELE\nPIM1\tA0A0S7LYW0_9TELE\nPIM1\tA0A0U1LW93_TALIS\nPIM1\tA0A0U5FUN6_9EURO\nPIM1\tA0A0V1Q4J7_9ASCO\nPIM1\tA0A0W0D2G5_CANGB\nPIM1\tA0A0W0FE50_9AGAR\nPIM1\tA0A0W4ZT20_PNEC8\nPIM1\tA0A0W4ZWT6_PNEJ7\nPIM1\tA0A0X8HU44_9SACH\nPIM1\tA0A100I4Y7_ASPNG\nPIM1\tA0A101MDR4_9EURO\nPIM1\tA0A125PIE2_9BASI\nPIM1\tA0A127ZF81_9BASI\nPIM1\tA0A132BD19_9HELO\nPIM1\tA0A135LNG5_PENPA\nPIM1\tA0A135SPC6_9PEZI\nPIM1\tA0A135T5L7_9PEZI\nPIM1\tA0A135TTB0_9PEZI\nPIM1\tA0A136J557_9PEZI\nPIM1\tA0A137PB82_CONC2\nPIM1\tA0A139AXR3_GONPJ\nPIM1\tA0A139HRD4_9PEZI\nPIM1\tA0A146FNB2_9EURO\nPIM1\tA0A146HH61_9AGAR\nPIM1\tA0A150VGU4_9PEZI\nPIM1\tA0A151GQI1_9HYPO\nPIM1\tA0A151N8M6_ALLMI\nPIM1\tA0A161W0T9_9PEZI\nPIM1\tA0A162LRU5_CORDF\nPIM1\tA0A162LZD7_9HYPO\nPIM1\tA0A162PNH7_9PEZI\nPIM1\tA0A162YHH3_MUCCL\nPIM1\tA0A162ZHN3_DIDRA\nPIM1\tA0A163TFA8_ABSGL\nPIM1\tA0A164TQD3_9AGAM\nPIM1\tA0A165BPK3_9APHY\nPIM1\tA0A165DPP0_9BASI\nPIM1\tA0A165ITC2_XYLHT\nPIM1\tA0A165NVW6_9APHY\nPIM1\tA0A165QYP0_9AGAM\nPIM1\tA0A166ACD2_EXIGL\nPIM1\tA0A166B2A6_9AGAM\nPIM1\tA0A166H0T4_9AGAM\nPIM1\tA0A166JCA4_9AGAM\nPIM1\tA0A166UDX8_9HYPO\nPIM1\tA0A167CHR3_9ASCO\nPIM1\tA0A167EV24_9ASCO\nPIM1\tA0A167HM42_9HYPO\nPIM1\tA0A167PHJ7_9PEZI\nPIM1\tA0A167QCX9_PHYB8\nPIM1\tA0A167R4B5_CALVF\nPIM1\tA0A167RTG1_PENCH\nPIM1\tA0A167XQF5_9EURO\nPIM1\tA0A168ETI3_9HYPO\nPIM1\tA0A168T4N2_ABSGL\nPIM1\tA0A175W1N7_9PEZI\nPIM1\tA0A177AGU5_9PEZI\nPIM1\tA0A177CAI7_9PLEO\nPIM1\tA0A177DYV7_ALTAL\nPIM1\tA0A177F5H5_9EURO\nPIM1\tA0A177TEK9_9BASI\nPIM1\tA0A177V4A7_9BASI\nPIM1\tA0A177V854_9BASI\nPIM1\tA0A177VUZ8_9BASI\nPIM1\tA0A177W9G1_BATDL\nPIM1\tA0A178B0Y2_9PLEO\nPIM1\tA0A178BRB3_9EURO\nPIM1\tA0A178CT45_9EURO\nPIM1\tA0A178DQC6_9PLEO\nPIM1\tA0A178EVQ7_TRIRU\nPIM1\tA0A178F995_TRIVO\nPIM1\tA0A178ZK23_9EURO\nPIM1\tA0A179FUV1_METCM\nPIM1\tA0A179HE99_PURLI\nPIM1\tA0A179HJQ0_PURLI\nPIM1\tA0A179U9N7_BLAGS\nPIM1\tA0A180GYX8_PUCT1\nPIM1\tA0A180GZZ4_PUCT1\nPIM1\tA0A194SAX6_RHOGW\nPIM1\tA0A194UQE8_9PEZI\nPIM1\tA0A194VYN1_9PEZI\nPIM1\tA0A197JYS2_9FUNG\nPIM1\tA0A1A6A2V7_9TREE\nPIM1\tA0A1A7YI97_9TELE\nPIM1\tA0A1A8B620_NOTFU\nPIM1\tA0A1A8BL54_9TELE\nPIM1\tA0A1A8DRL1_9TELE\nPIM1\tA0A1A8EF17_9TELE\nPIM1\tA0A1A8FDS0_9TELE\nPIM1\tA0A1A8GBB8_9TELE\nPIM1\tA0A1A8GZX8_9TELE\nPIM1\tA0A1A8HNT0_NOTKU\nPIM1\tA0A1A8INX4_NOTKU\nPIM1\tA0A1A8K576_NOTKU\nPIM1\tA0A1A8KSD4_NOTKU\nPIM1\tA0A1A8KZN5_9TELE\nPIM1\tA0A1A8MA33_9TELE\nPIM1\tA0A1A8N9H6_9TELE\nPIM1\tA0A1A8PWQ2_9TELE\nPIM1\tA0A1A8Q9Y0_9TELE\nPIM1\tA0A1A8U5N2_NOTFU\nPIM1\tA0A1B2JHL3_PICPA\nPIM1\tA0A1B7MSQ1_9AGAM\nPIM1\tA0A1B7NYT3_9EURO\nPIM1\tA0A1B7SFE5_9ASCO\nPIM1\tA0A1B8AN78_FUSPO\nPIM1\tA0A1B8CLR2_9PEZI\nPIM1\tA0A1B8DAY2_9PEZI\nPIM1\tA0A1B8E090_9PEZI\nPIM1\tA0A1B8EPR4_9PEZI\nPIM1\tA0A1B8FUI6_9PEZI\nPIM1\tA0A1B8GK78_9PEZI\nPIM1\tA0A1B9G081_9TREE\nPIM1\tA0A1B9GHH6_9TREE\nPIM1\tA0A1B9H7J8_9TREE\nPIM1\tA0A1B9I817_9TREE\nPIM1\tA0A1B9INH9_9TREE\nPIM1\tA0A1C1CR16_9EURO\nPIM1\tA0A1C1CT84_9EURO\nPIM1\tA0A1C3YK08_GIBZE\nPIM1\tA0A1C7LQR6_GRIFR\nPIM1\tA0A1C7NHY3_9FUNG\nPIM1\tA0A1D2JC28_PARBR\nPIM1\tA0A1D5NZV7_CHICK\nPIM1\tA0A1D8NQ29_YARLL\nPIM1\tA0A1D9PS38_SCLS1\nPIM1\tA0A1E1K6K1_9HELO\nPIM1\tA0A1E1KUY9_9HELO\nPIM1\tA0A1E1M411_RHYSE\nPIM1\tA0A1E3B9V8_9EURO\nPIM1\tA0A1E3HZL0_9TREE\nPIM1\tA0A1E3HZP9_9TREE\nPIM1\tA0A1E3IZ64_9TREE\nPIM1\tA0A1E3JG01_9TREE\nPIM1\tA0A1E3JZ59_9TREE\nPIM1\tA0A1E3KCK4_9TREE\nPIM1\tA0A1E3NLW4_9ASCO\nPIM1\tA0A1E3Q1N3_LIPST\nPIM1\tA0A1E4RED6_9ASCO\nPIM1\tA0A1E4S0N3_CYBJN\nPIM1\tA0A1E4SKG1_9ASCO\nPIM1\tA0A1E4TA13_9ASCO\nPIM1\tA0A1E4TT78_PACTA\nPIM1\tA0A1E5RW45_9ASCO\nPIM1\tA0A1F5LK92_9EURO\nPIM1\tA0A1F7ZJW4_9EURO\nPIM1\tA0A1G4BBQ5_9PEZI\nPIM1\tA0A1G4IXH8_9SACH\nPIM1\tA0A1G4J1N5_9SACH\nPIM1\tA0A1G4JL65_9SACH\nPIM1\tA0A1G4K7I9_9SACH\nPIM1\tA0A1G4KHJ4_9SACH\nPIM1\tA0A1G4MKG2_LACFM\nPIM1\tA0A1J7IW95_9PEZI\nPIM1\tA0A1J8QG54_9AGAM\nPIM1\tA0A1J9PNJ1_9EURO\nPIM1\tA0A1J9RIA4_9EURO\nPIM1\tA0A1J9S8M8_9PEZI\nPIM1\tA0A1K0H2T2_9BASI\nPIM1\tA0A1L0DT19_9ASCO\nPIM1\tA0A1L0GH49_9ASCO\nPIM1\tA0A1L7TLP2_FUSMA\nPIM1\tA0A1L7VAH8_FUSPR\nPIM1\tA0A1L7XBJ0_9HELO\nPIM1\tA0A1L9NAK0_ASPTC\nPIM1\tA0A1L9PIY6_ASPVE\nPIM1\tA0A1L9RBQ9_ASPWE\nPIM1\tA0A1L9S6B9_9EURO\nPIM1\tA0A1L9THI3_9EURO\nPIM1\tA0A1L9UXC6_ASPBC\nPIM1\tA0A1L9VED1_ASPGL\nPIM1\tA0A1L9WXH2_ASPAC\nPIM1\tA0A1M2VGE7_TRAPU\nPIM1\tA0A1M3TYJ5_ASPLC\nPIM1\tA0A1M8A730_MALS4\nPIM1\tA0A1Q2YFX3_9ASCO\nPIM1\tA0A1Q3A5K2_ZYGRO\nPIM1\tA0A1Q3AJ34_ZYGRO\nPIM1\tA0A1Q3EJL3_LENED\nPIM1\tA0A1Q5T8K7_9EURO\nPIM1\tA0A1Q8S270_9PEZI\nPIM1\tA0A1Q9C423_SYMMI\nPIM1\tA0A1Q9D105_SYMMI\nPIM1\tA0A1Q9DX37_SYMMI\nPIM1\tA0A1Q9E362_SYMMI\nPIM1\tA0A1Q9E4C9_SYMMI\nPIM1\tA0A1Q9E8D2_SYMMI\nPIM1\tA0A1Q9EG29_SYMMI\nPIM1\tA0A1Q9ELM4_SYMMI\nPIM1\tA0A1Q9ESF8_SYMMI\nPIM1\tA0A1Q9F2C7_SYMMI\nPIM1\tA0A1R0GQQ2_9FUNG\nPIM1\tA0A1R1YA93_9FUNG\nPIM1\tA0A1R1YTJ5_9FUNG\nPIM1\tA0A1R3RYR7_ASPC5\nPIM1\tA0A1S2ZDV4_ERIEU\nPIM1\tA0A1S3G878_DIPOR\nPIM1\tA0A1S7HGW8_9SACH\nPIM1\tA0A1S7HL15_9SACH\nPIM1\tA0A1S8BI08_9PEZI\nPIM1\tA0A1S9E0V8_ASPOZ\nPIM1\tA0A1S9RL68_9EURO\nPIM1\tA0A1T3CAW6_9HYPO\nPIM1\tA0A1U7LRQ5_NEOID\nPIM1\tA0A1U7TNX6_TARSY\nPIM1\tA0A1U8D729_ALLSI\nPIM1\tA0A1V1SU92_9FUNG\nPIM1\tA0A1V2LAF4_CYBFA\nPIM1\tA0A1V2LN50_PICKU\nPIM1\tA0A1V4KYF5_PATFA\nPIM1\tA0A1V6NBC7_9EURO\nPIM1\tA0A1V6PEL1_PENDC\nPIM1\tA0A1V6PSK9_9EURO\nPIM1\tA0A1V6QW98_9EURO\nPIM1\tA0A1V6RSM8_9EURO\nPIM1\tA0A1V6SQP3_9EURO\nPIM1\tA0A1V6TZT6_9EURO\nPIM1\tA0A1V6U8P7_9EURO\nPIM1\tA0A1V6YZU8_PENNA\nPIM1\tA0A1V8SUC5_9PEZI\nPIM1\tA0A1V8T2Y6_9PEZI\nPIM1\tA0A1W0E3F7_9MICR\nPIM1\tA0A1W2TB46_ROSNE\nPIM1\tA0A1W5DCN8_9LECA\nPIM1\tA0A1X0RIN5_RHIZD\nPIM1\tA0A1X0S5F6_RHIZD\nPIM1\tA0A1X2GNL1_9FUNG\nPIM1\tA0A1X2H0S1_SYNRA\nPIM1\tA0A1X2ICU5_9FUNG\nPIM1\tA0A1X2IQZ8_9FUNG\nPIM1\tA0A1X6MND7_9APHY\nPIM1\tA0A1X7QZ62_9SACH\nPIM1\tA0A1X7RV30_ZYMTR\nPIM1\tA0A1Y1UIJ0_9TREE\nPIM1\tA0A1Y1VIK9_9FUNG\nPIM1\tA0A1Y1W092_9FUNG\nPIM1\tA0A1Y1X1V6_9FUNG\nPIM1\tA0A1Y1X746_9FUNG\nPIM1\tA0A1Y1XDE3_9FUNG\nPIM1\tA0A1Y2AGA8_9FUNG\nPIM1\tA0A1Y2AT41_9TREE\nPIM1\tA0A1Y2D3E3_9FUNG\nPIM1\tA0A1Y2DUZ6_9PEZI\nPIM1\tA0A1Y2FG65_PROLT\nPIM1\tA0A1Y2FZ20_9BASI\nPIM1\tA0A1Y2G6T0_9FUNG\nPIM1\tA0A1Y2HHR0_9FUNG\nPIM1\tA0A1Y2IZS8_PYCCO\nPIM1\tA0A1Y2LWZ7_EPING\nPIM1\tA0A1Y2TUC4_9PEZI\nPIM1\tA0A1Y2V0K1_9PEZI\nPIM1\tA0A1Y2W0A4_9PEZI\nPIM1\tA0A1Y2XD10_9PEZI\nPIM1\tA0A1Y3NE27_PIRSE\nPIM1\tA0A1Y6LKA1_ZYMTR\nPIM1\tA0A1Z5SYB0_HORWE\nPIM1\tA0A1Z5TN04_HORWE\nPIM1\tA0A202G9T1_CLALS\nPIM1\tA0A212MC54_ZYGBA\nPIM1\tA0A218Z5S3_9HELO\nPIM1\tA0A225AGA9_9EURO\nPIM1\tA0A225Z167_CRYNV\nPIM1\tA0A226BIX1_CRYNV\nPIM1\tA0A229W7H7_ASPFM\nPIM1\tA0A229X3C6_9EURO\nPIM1\tA0A232M232_9EURO\nPIM1\tA0A238F6L8_9BASI\nPIM1\tA0A250YF76_CASCN\nPIM1\tA0A254TTE7_ASPNG\nPIM1\tA0A261Y1N7_9FUNG\nPIM1\tA0A284RMK7_9AGAR\nPIM1\tA0A286U9T2_9AGAM\nPIM1\tA0A292Q7R6_9PEZI\nPIM1\tA0A2A9N8M8_9AGAR\nPIM1\tA0A2A9P581_9HYPO\nPIM1\tA0A2B4RIG4_STYPI\nPIM1\tA0A2B4SMP7_STYPI\nPIM1\tA0A2B7WLG5_9EURO\nPIM1\tA0A2B7X4N8_9EURO\nPIM1\tA0A2B7XJY8_9EURO\nPIM1\tA0A2B7Z6Z1_9EURO\nPIM1\tA0A2C5XAF9_9PEZI\nPIM1\tA0A2C5Y1R2_9HYPO\nPIM1\tA0A2C5Y8L3_9HYPO\nPIM1\tA0A2C5YZC9_9HYPO\nPIM1\tA0A2D1QA39_STATI\nPIM1\tA0A2D1QA63_9SAUR\nPIM1\tA0A2D1QAC0_9SAUR\nPIM1\tA0A2D1QAC4_APASP\nPIM1\tA0A2D3V9G3_9PEZI\nPIM1\tA0A2G4T9K5_RHIZD\nPIM1\tA0A2G5BDB7_COERN\nPIM1\tA0A2G5HKL0_CERBT\nPIM1\tA0A2G7G579_9EURO\nPIM1\tA0A2G8SP50_9APHY\nPIM1\tA0A2H0ZF20_CANAR\nPIM1\tA0A2H1GHV4_ZYMTR\nPIM1\tA0A2H2Z6M5_9HYPO\nPIM1\tA0A2H3C9X8_9AGAR\nPIM1\tA0A2H3E729_ARMGA\nPIM1\tA0A2H3F2C7_9HELO\nPIM1\tA0A2H3FWY4_GIBZA\nPIM1\tA0A2H3HVE5_FUSOX\nPIM1\tA0A2H3IKY4_9EURO\nPIM1\tA0A2H3K101_WOLCO\nPIM1\tA0A2H3TNM4_FUSOX\nPIM1\tA0A2H4PJI6_PENVA\nPIM1\tA0A2H5R4Y0_RHIID\nPIM1\tA0A2H9TJM1_9FUNG\nPIM1\tA0A2I0LZS5_COLLI\nPIM1\tA0A2I0S3U1_9PEZI\nPIM1\tA0A2I1CMX4_9EURO\nPIM1\tA0A2I1CUV5_9EURO\nPIM1\tA0A2I1EJ10_9GLOM\nPIM1\tA0A2I1GEI3_9GLOM\nPIM1\tA0A2I2FFZ5_9EURO\nPIM1\tA0A2I2G4U8_9EURO\nPIM1\tA0A2I4D5T0_9TELE\nPIM1\tA0A2J5HGA4_9EURO\nPIM1\tA0A2J6QA79_9HELO\nPIM1\tA0A2J6S9F1_9HELO\nPIM1\tA0A2J6SHQ7_9HELO\nPIM1\tA0A2J8ELA6_VERDA\nPIM1\tA0A2K0TJK4_9HYPO\nPIM1\tA0A2K0U301_TRIHA\nPIM1\tA0A2K0W210_GIBNY\nPIM1\tA0A2K1QMF9_9PEZI\nPIM1\tA0A2K3QQD8_9HYPO\nPIM1\tA0A2K5D0P8_AOTNA\nPIM1\tA0A2K5KE27_COLAP\nPIM1\tA0A2K5KE30_COLAP\nPIM1\tA0A2K5P1E2_CERAT\nPIM1\tA0A2K5P1I7_CERAT\nPIM1\tA0A2K5R4J9_CEBCA\nPIM1\tA0A2K5W2N0_MACFA\nPIM1\tA0A2K5YIW3_MANLE\nPIM1\tA0A2K5YIX5_MANLE\nPIM1\tA0A2K6BXK6_MACNE\nPIM1\tA0A2K6F2P9_PROCO\nPIM1\tA0A2K6L287_RHIBE\nPIM1\tA0A2K6SFP3_SAIBB\nPIM1\tA0A2L2TVA7_9HYPO\nPIM1\tA0A2N0RRK8_9GLOM\nPIM1\tA0A2N1JCN8_9BASI\nPIM1\tA0A2N1LQ42_TRIHA\nPIM1\tA0A2N1NHE0_9GLOM\nPIM1\tA0A2N3N942_9PEZI\nPIM1\tA0A2N5RXL2_9BASI\nPIM1\tA0A2N6NRF1_BEABA\nPIM1\tA0A2N8U8P0_9BASI\nPIM1\tA0A2P2EID2_9EURO\nPIM1\tA0A2P2H8Y5_ASPFA\nPIM1\tA0A2P4QYB4_RHIID\nPIM1\tA0A2P4ZDQ6_9HYPO\nPIM1\tA0A2P5IDK9_9PEZI\nPIM1\tA0A2P7YDA5_9PEZI\nPIM1\tA0A2P7YU18_9ASCO\nPIM1\tA0A2R6P059_9APHY\nPIM1\tA0A2S4L1A5_9HYPO\nPIM1\tA0A2S5B6R9_9BASI\nPIM1\tA0A2S6C7J1_9PEZI\nPIM1\tA0A2S7PM98_9HELO\nPIM1\tA0A2S7QR17_9HELO\nPIM1\tA0A2S7QYK5_9HELO\nPIM1\tA0A2S7Y979_BEABA\nPIM1\tA0A2T2NVI4_CORCC\nPIM1\tA0A2T3AKG8_9PEZI\nPIM1\tA0A2T3BBC9_AMORE\nPIM1\tA0A2T3ZLG9_9HYPO\nPIM1\tA0A2T4AV57_TRIHA\nPIM1\tA0A2T4BNC2_9HYPO\nPIM1\tA0A2T4CG54_TRILO\nPIM1\tA0A2T4GQY6_FUSCU\nPIM1\tA0A2T5LT24_9EURO\nPIM1\tA0A2T7A0M8_TUBBO\nPIM1\tA0A2T9YHI6_9FUNG\nPIM1\tA0A2T9ZCJ2_9FUNG\nPIM1\tA0A2U1J552_9FUNG\nPIM1\tA0A2U1JCJ5_9FUNG\nPIM1\tA0A2U3E1G7_PURLI\nPIM1\tA0A2U3WLQ2_ODORO\nPIM1\tA0A2U3XRH4_LEPWE\nPIM1\tA0A2U4CJ99_TURTR\nPIM1\tA0A2U9R0K0_PICKU\nPIM1\tA0A2V1AHI5_9ASCO\nPIM1\tA0A2V1AYC5_9ASCO\nPIM1\tA0A2V1CK14_9HELO\nPIM1\tA0A2V1D8Z9_9PLEO\nPIM1\tA0A2V5GYD3_9EURO\nPIM1\tA0A2V5HLA2_9EURO\nPIM1\tA0A2W1DSA8_9PLEO\nPIM1\tA0A2W1F188_9PLEO\nPIM1\tA0A2X0LVR8_9BASI\nPIM1\tA0A2X0MGW9_9BASI\nPIM1\tA0A2Y9F4G5_PHYMC\nPIM1\tA0A2Y9GM71_NEOSC\nPIM1\tA0A2Y9GMC8_NEOSC\nPIM1\tA0A2Y9NTX7_DELLE\nPIM1\tA0A2Y9NX53_DELLE\nPIM1\tA0A2Y9SKC6_PHYMC\nPIM1\tA0A2Z6RLL7_9GLOM\nPIM1\tA0A316U5L1_9BASI\nPIM1\tA0A316V084_9BASI\nPIM1\tA0A316VD54_9BASI\nPIM1\tA0A316VWM4_9BASI\nPIM1\tA0A316YNH9_9BASI\nPIM1\tA0A316ZHM4_9BASI\nPIM1\tA0A317SG10_9PEZI\nPIM1\tA0A317V1I3_9EURO\nPIM1\tA0A317VET4_9EURO\nPIM1\tA0A317VG26_ASPEC\nPIM1\tA0A318YPF4_ASPNB\nPIM1\tA0A319ABG7_9EURO\nPIM1\tA0A319AYC2_ASPLB\nPIM1\tA0A319BQF2_ASPVC\nPIM1\tA0A319CBT2_9EURO\nPIM1\tA0A319DK81_9EURO\nPIM1\tA0A319EDH4_ASPSB\nPIM1\tA0A340Y145_LIPVE\nPIM1\tA0A341D4N7_NEOAA\nPIM1\tA0A341D4X2_NEOAA\nPIM1\tA0A364L2K5_9EURO\nPIM1\tA0A364LP68_ASPFL\nPIM1\tA0A364N1D3_9PLEO\nPIM1\tA0A365N8P1_GIBIN\nPIM1\tA0A366P843_VERDA\nPIM1\tA0A366PLN3_GIBMO\nPIM1\tA0A366RI11_GIBMO\nPIM1\tA0A366RMC1_9HYPO\nPIM1\tA0A367KHN0_RHIAZ\nPIM1\tA0A367KYM1_RHIST\nPIM1\tA0A367LHC5_9HYPO\nPIM1\tA0A369GUN8_9HYPO\nPIM1\tA0A369H775_9HYPO\nPIM1\tA0A369K771_HYPMA\nPIM1\tA0A370CFP2_ASPNG\nPIM1\tA0A370PI47_ASPPH\nPIM1\tA0A370TTP5_9PEZI\nPIM1\tA0A371C7T6_YARLL\nPIM1\tA0A371DXA9_9APHY\nPIM1\tA0A372RGY8_9GLOM\nPIM1\tA0A376B473_9ASCO\nPIM1\tA0A381L859_BLUGR\nPIM1\tA0A383UP76_BLUGH\nPIM1\tA0A384BF64_BALAS\nPIM1\tA0A384CB72_URSMA\nPIM1\tA0A384JNN4_BOTFB\nPIM1\tA0A395GUJ6_9EURO\nPIM1\tA0A395IEU6_ASPHC\nPIM1\tA0A395IVD4_9HELO\nPIM1\tA0A395MRL6_9HYPO\nPIM1\tA0A395NMW3_TRIAR\nPIM1\tA0A395SKW5_FUSSP\nPIM1\tA0A395T7M0_9HYPO\nPIM1\tA0A397FZD0_9EURO\nPIM1\tA0A397GLG7_9GLOM\nPIM1\tA0A397T2B7_9GLOM\nPIM1\tA0A397VAN2_9GLOM\nPIM1\tA0A399GCG2_9PLEO\nPIM1\tA0A3A2ZTH9_9EURO\nPIM1\tA0A3D8S0B0_9HELO\nPIM1\tA0A3D8SXG4_9EURO\nPIM1\tA0A3D8T0M7_9HELO\nPIM1\tA0A3F2XWC3_DEKBR\nPIM1\tA0A3F3PXT1_9EURO\nPIM1\tA0A3G2S9U9_9BASI\nPIM1\tA0A3G2SD66_9BASI\nPIM1\tA0A3L6NL32_FUSOX\nPIM1\tA0A3M2RXS5_9HYPO\nPIM1\tA0A3M2SWX4_9EURO\nPIM1\tA0A3M6WX25_HORWE\nPIM1\tA0A3M6X980_HORWE\nPIM1\tA0A3M6ZUZ5_HORWE\nPIM1\tA0A3M7D2S6_HORWE\nPIM1\tA0A3M7ECW7_HORWE\nPIM1\tA0A3M7ETU8_HORWE\nPIM1\tA0A3M7F760_HORWE\nPIM1\tA0A3M7H3T4_HORWE\nPIM1\tA0A3M7HAS3_HORWE\nPIM1\tA0A3M7IF97_HORWE\nPIM1\tA0A3M7ITR2_HORWE\nPIM1\tA0A3M7MG13_9PLEO\nPIM1\tA0A3M9XZX7_9PEZI\nPIM1\tA0A3N2Q181_9PEZI\nPIM1\tA0A3N4IJ07_ASCIM\nPIM1\tA0A3N4J465_9PEZI\nPIM1\tA0A3N4KI15_9PEZI\nPIM1\tA0A3N4LGC6_9PEZI\nPIM1\tA0A3Q4EGB5_MOUSE\nPIM1\tA0A3Q4EGX3_MOUSE\nPIM1\tA0A3Q7NI56_CALUR\nPIM1\tA0A3Q7SWZ0_VULVU\nPIM1\tA0A3Q7TR70_URSAR\nPIM1\tA0A3R7FQ03_CLOSI\nPIM1\tA0A401H0E7_9APHY\nPIM1\tA0A401KH78_ASPAW\nPIM1\tA0A409XLI3_PSICY\nPIM1\tA0A409YE52_9AGAR\nPIM1\tA0A420HQF9_9PEZI\nPIM1\tA0A420HVM1_9PEZI\nPIM1\tA0A420IMM0_9PEZI\nPIM1\tA0A420JAP7_9PEZI\nPIM1\tA0A420QK08_FUSOX\nPIM1\tA0A420R807_FUSOX\nPIM1\tA0A420U6F4_GIBIN\nPIM1\tA0A420YCZ9_9PEZI\nPIM1\tA0A421J818_9ASCO\nPIM1\tA0A423VL06_9PEZI\nPIM1\tA0A423WG81_9PEZI\nPIM1\tA0A423XEV0_9PEZI\nPIM1\tA0A425BT40_9PEZI\nPIM1\tA0A427YB56_9TREE\nPIM1\tA0A427YIW5_9TREE\nPIM1\tA0A428NEV8_9HYPO\nPIM1\tA0A428PU78_9HYPO\nPIM1\tA0A428Q5Z2_9HYPO\nPIM1\tA0A428S2R7_9HYPO\nPIM1\tA0A428SJ86_9HYPO\nPIM1\tA0A430L2J7_9HYPO\nPIM1\tA0A433PRV8_9FUNG\nPIM1\tA0A437AAA6_9PEZI\nPIM1\tA0A438MWJ3_EXOME\nPIM1\tA0A439CU40_9PEZI\nPIM1\tA0A443HI46_BYSSP\nPIM1\tA0A444S5I5_VERDA\nPIM1\tA0A446BMI1_9PEZI\nPIM1\tA0A447C506_9PEZI\nPIM1\tA0A448YK26_BRENA\nPIM1\tA0A452FVM1_CAPHI\nPIM1\tA0A452QMJ2_URSAM\nPIM1\tA0A452QML8_URSAM\nPIM1\tA0A452TYM8_URSMA\nPIM1\tA0A4C2E6N2_9SACH\nPIM1\tA0A4D9B8Q5_SALSN\nPIM1\tA0A4D9BRM4_SALSN\nPIM1\tA0A4P7N928_MAGOR\nPIM1\tA0A4Q0A279_9FUNG\nPIM1\tA0A4Q0TW53_COCIT\nPIM1\tA0A4Q1BJA8_TREME\nPIM1\tA0A4Q2DKY1_9AGAR\nPIM1\tA0A4Q4M0J3_9PLEO\nPIM1\tA0A4Q4NXC7_ALTAL\nPIM1\tA0A4Q4QVU7_9PLEO\nPIM1\tA0A4Q4RH09_9PLEO\nPIM1\tA0A4Q4RKF4_9PLEO\nPIM1\tA0A4Q4TPZ3_9PEZI\nPIM1\tA0A4Q4UVP7_9PEZI\nPIM1\tA0A4Q4VV51_9PEZI\nPIM1\tA0A4Q4VWA7_9PEZI\nPIM1\tA0A4Q4W8U4_9PEZI\nPIM1\tA0A4Q4XYL0_9PEZI\nPIM1\tA0A4Q4YIL6_9PEZI\nPIM1\tA0A4Q4ZRV2_9PEZI\nPIM1\tA0A4Q7K740_METCM\nPIM1\tA0A4Q9MZ21_9APHY\nPIM1\tA0A4Q9Q6Z1_9APHY\nPIM1\tA0A4R0R1S2_9APHY\nPIM1\tA0A4R8QFV2_9PEZI\nPIM1\tA0A4R8TPN1_9PEZI\nPIM1\tA0A4S2MXD6_9PEZI\nPIM1\tA0A4S3JTI2_9EURO\nPIM1\tA0A4S4K7F3_9APHY\nPIM1\tA0A4S4LSY9_9AGAM\nPIM1\tA0A4S4MYF4_9APHY\nPIM1\tA0A4S8M213_9AGAR\nPIM1\tA0A4S8R4Z3_9HELO\nPIM1\tA0A4S8SJ39_AURPU\nPIM1\tA0A4S8SU95_AURPU\nPIM1\tA0A4S8TB47_AURPU\nPIM1\tA0A4S8W3R6_AURPU\nPIM1\tA0A4S8X9G3_AURPU\nPIM1\tA0A4S8XCL5_AURPU\nPIM1\tA0A4S9ANX6_AURPU\nPIM1\tA0A4S9ENJ8_AURPU\nPIM1\tA0A4S9HCR2_AURPU\nPIM1\tA0A4S9HIL5_AURPU\nPIM1\tA0A4S9K2F2_AURPU\nPIM1\tA0A4S9L5H4_AURPU\nPIM1\tA0A4S9LBR0_AURPU\nPIM1\tA0A4S9NA21_AURPU\nPIM1\tA0A4S9NJZ3_AURPU\nPIM1\tA0A4S9QHX7_AURPU\nPIM1\tA0A4S9RJA1_AURPU\nPIM1\tA0A4S9SKX6_AURPU\nPIM1\tA0A4S9SXH1_AURPU\nPIM1\tA0A4S9VQ55_AURPU\nPIM1\tA0A4S9WVN6_AURPU\nPIM1\tA0A4S9YCI9_AURPU\nPIM1\tA0A4S9YPV1_AURPU\nPIM1\tA0A4T0AG37_AURPU\nPIM1\tA0A4T0BAP1_AURPU\nPIM1\tA0A4T0BLF7_AURPU\nPIM1\tA0A4T0DC98_AURPU\nPIM1\tA0A4T0DH43_AURPU\nPIM1\tA0A4T0FRI1_9BASI\nPIM1\tA0A4T0HY91_WALIC\nPIM1\tA0A4T0K7F8_WALIC\nPIM1\tA0A4T0LY90_9BASI\nPIM1\tA0A4T0MGL3_9BASI\nPIM1\tA0A4T0NMD3_9BASI\nPIM1\tA0A4T0R5M1_9BASI\nPIM1\tA0A4T0W5R5_9PEZI\nPIM1\tA0A4T0WWD9_9ASCO\nPIM1\tA0A4U0U7U2_9PEZI\nPIM1\tA0A4U0VES5_9PEZI\nPIM1\tA0A4U0W886_9BASI\nPIM1\tA0A4U0XTZ5_9PEZI\nPIM1\tA0A4U6XE37_9PEZI\nPIM1\tA0A4U6XK33_9PEZI\nPIM1\tA0A4U7B7N3_9PEZI\nPIM1\tA0A4U7KLA1_9BASI\nPIM1\tA0A4V1S1T8_FUSOX\nPIM1\tA0A4V3HWS6_COLTR\nPIM1\tA0A4V4IUC9_AURPU\nPIM1\tA0A4V4J2E5_AURPU\nPIM1\tA0A4V4LQA7_AURPU\nPIM1\tA0A4V4MUG7_9BASI\nPIM1\tA0A4V5N8M7_9PEZI\nPIM1\tA0A4V6DGQ4_9PEZI\nPIM1\tA0A4V6S0Z0_9AGAM\nPIM1\tA0A4W2BZW7_BOBOX\nPIM1\tA0A4W4HG10_ELEEL\nPIM1\tA0A4W6DD07_LATCA\nPIM1\tA0A4X1SRD6_PIG\nPIM1\tA0A4X1SRD7_PIG\nPIM1\tA0A4X2LV66_VOMUR\nPIM1\tA0A4Y7Q0S5_9AGAM\nPIM1\tA0A4Y7THH2_9AGAR\nPIM1\tA0A4Y8D2M5_9HELO\nPIM1\tA0A4Y9YEM1_9APHY\nPIM1\tA0A4Y9Z832_9AGAM\nPIM1\tA0A4Z0A156_9AGAM\nPIM1\tA0A4Z0Y8G9_ARTOL\nPIM1\tA0A4Z0Z7Q2_9PEZI\nPIM1\tA0A4Z1EQF1_9HELO\nPIM1\tA0A4Z1FZ72_9HELO\nPIM1\tA0A4Z1GIQ6_9HELO\nPIM1\tA0A4Z1I9C6_9HELO\nPIM1\tA0A4Z1ID04_9HELO\nPIM1\tA0A4Z1JEI6_9HELO\nPIM1\tA0A4Z1KJ38_9HELO\nPIM1\tA0A4Z1PND7_9PEZI\nPIM1\tA0A507AW37_9PEZI\nPIM1\tA0A507BXA3_9FUNG\nPIM1\tA0A507CZ63_9FUNG\nPIM1\tA0A507DBU6_9FUNG\nPIM1\tA0A507EID2_9FUNG\nPIM1\tA0A507EMV0_9FUNG\nPIM1\tA0A507FAX5_9FUNG\nPIM1\tA0A507QNN7_MONPU\nPIM1\tA0A510PC09_CANAR\nPIM1\tA0A511K8W9_RHOTO\nPIM1\tA0A517L2F1_9PEZI\nPIM1\tA0A545A3L4_9PEZI\nPIM1\tA0A545VXU3_9HYPO\nPIM1\tA0A550CIF9_9AGAR\nPIM1\tA0A553IDQ3_9PEZI\nPIM1\tA0A559LCZ0_FUSOX\nPIM1\tA0A559M3Z1_9HELO\nPIM1\tA0A5B0NBH1_PUCGR\nPIM1\tA0A5B0NPY7_PUCGR\nPIM1\tA0A5B1R7E8_9AGAM\nPIM1\tA0A5B6VI55_9ROSI\nPIM1\tA0A5B6VUK9_9ROSI\nPIM1\tA0A5B7H483_PORTR\nPIM1\tA0A5C2S8P3_9APHY\nPIM1\tA0A5C2TT82_9APHY\nPIM1\tA0A5C3DX86_9BASI\nPIM1\tA0A5C3DY59_9BASI\nPIM1\tA0A5C3F4V3_9BASI\nPIM1\tA0A5C3FIH1_PSEA2\nPIM1\tA0A5C3L5N0_9AGAR\nPIM1\tA0A5C3MXU6_9AGAM\nPIM1\tA0A5C3Q5E5_9APHY\nPIM1\tA0A5C3QDX4_9AGAR\nPIM1\tA0A5C5G1P5_9BASI\nPIM1\tA0A5C6GKE1_9HYPO\nPIM1\tA0A5C6TN89_FUSOX\nPIM1\tA0A5D3B899_9TREE\nPIM1\tA0A5E3WV32_9AGAM\nPIM1\tA0A5E8B7X1_9ASCO\nPIM1\tA0A5F4D6M8_CANLF\nPIM1\tA0A5G3J910_XENTR\nPIM1\tA1C6L5_ASPCL\nPIM1\tA1DH85_NEOFI\nPIM1\tA1XD85_PICAN\nPIM1\tA5DAP1_PICGU\nPIM1\tA5E0P2_LODEL\nPIM1\tA6ZKS6_YEAS7\nPIM1\tA7EWM6_SCLS1\nPIM1\tA8QCF3_MALGO\nPIM1\tB0D1S2_LACBS\nPIM1\tB0XSK2_ASPFC\nPIM1\tB2AZ54_PODAN\nPIM1\tB2W075_PYRTR\nPIM1\tB3LNF9_YEAS1\nPIM1\tB5VDV9_YEAS6\nPIM1\tB5X0V3_SALSA\nPIM1\tB6HH20_PENRW\nPIM1\tB6JX56_SCHJY\nPIM1\tB6QPB8_TALMQ\nPIM1\tB6QPB9_TALMQ\nPIM1\tB8M3Z1_TALSN\nPIM1\tB8NGC1_ASPFN\nPIM1\tC0JAN8_SHEEP\nPIM1\tC0JAN9_SHEEP\nPIM1\tC0NLF6_AJECG\nPIM1\tC0RZC9_PARBP\nPIM1\tC1G7T4_PARBD\nPIM1\tC1GYZ2_PARBA\nPIM1\tC1LG25_SCHJA\nPIM1\tC1LG26_SCHJA\nPIM1\tC4JFS5_UNCRE\nPIM1\tC4R7W9_KOMPG\nPIM1\tC4YLQ1_CANAW\nPIM1\tC5E015_ZYGRC\nPIM1\tC5E2W2_LACTC\nPIM1\tC5FFL0_ARTOC\nPIM1\tC5GNJ7_AJEDR\nPIM1\tC5M2W4_CANTT\nPIM1\tC5PJH7_COCP7\nPIM1\tC6H3A4_AJECH\nPIM1\tC7GX88_YEAS2\nPIM1\tC7Z734_NECH7\nPIM1\tC8Z3X4_YEAS8\nPIM1\tD4AY35_ARTBC\nPIM1\tD4DAJ3_TRIVH\nPIM1\tD5G5P8_TUBMM\nPIM1\tD7FNJ6_ECTSI\nPIM1\tD8QIQ7_SCHCM\nPIM1\tE3KTI4_PUCGT\nPIM1\tE3Q9V4_COLGM\nPIM1\tE3RH85_PYRTT\nPIM1\tE4V3H8_ARTGP\nPIM1\tE4ZH66_LEPMJ\nPIM1\tE6R4G0_CRYGW\nPIM1\tE6ZTB0_SPORE\nPIM1\tE9DK44_COCPS\nPIM1\tE9DUP3_METAQ\nPIM1\tE9F2G9_METRA\nPIM1\tF0U780_AJEC8\nPIM1\tF0XI58_GROCL\nPIM1\tF1PHQ6_CANLF\nPIM1\tF1RVS6_PIG\nPIM1\tF2QZ75_KOMPC\nPIM1\tF2RYY6_TRIT1\nPIM1\tF2SS61_TRIRC\nPIM1\tF2TH67_AJEDA\nPIM1\tF4NRF8_BATDJ\nPIM1\tF4RM49_MELLP\nPIM1\tF6UWX4_HORSE\nPIM1\tF7EGG8_XENTR\nPIM1\tF7EGH6_XENTR\nPIM1\tF7G9A3_MONDO\nPIM1\tF7HC67_CALJA\nPIM1\tF7VRP6_SORMK\nPIM1\tF8MJ84_NEUT8\nPIM1\tF8PXN2_SERL3\nPIM1\tF9FFM9_FUSOF\nPIM1\tF9XCF0_ZYMTI\nPIM1\tG0R9R2_HYPJQ\nPIM1\tG0S8D7_CHATD\nPIM1\tG0SX31_RHOT2\nPIM1\tG0V9N0_NAUCC\nPIM1\tG0W6C0_NAUDC\nPIM1\tG1MAA2_AILME\nPIM1\tG1MQC2_MELGA\nPIM1\tG1PQ47_MYOLU\nPIM1\tG1RJZ4_NOMLE\nPIM1\tG1T2G8_RABIT\nPIM1\tG1XT16_ARTOA\nPIM1\tG2Q4W0_MYCTT\nPIM1\tG2R5A9_THETT\nPIM1\tG2W8Y7_YEASK\nPIM1\tG2XFZ8_VERDV\nPIM1\tG2YUP8_BOTF4\nPIM1\tG3BFF0_CANTC\nPIM1\tG3JAA7_CORMM\nPIM1\tG3SZW8_LOXAF\nPIM1\tG3WQH0_SARHA\nPIM1\tG3Y2J5_ASPNA\nPIM1\tG4MVR9_MAGO7\nPIM1\tG4TBS7_SERID\nPIM1\tG4UJ55_NEUT9\nPIM1\tG7DT16_MIXOS\nPIM1\tG7X5E9_ASPKW\nPIM1\tG8BG94_CANPC\nPIM1\tG8BWS5_TETPH\nPIM1\tG8JV94_ERECY\nPIM1\tG8Y0S8_PICSO\nPIM1\tG8Y3R4_PICSO\nPIM1\tG8ZPM8_TORDC\nPIM1\tG9MSF5_HYPVG\nPIM1\tG9P1D7_HYPAI\nPIM1\tH0EVT3_GLAL7\nPIM1\tH0GC64_SACCK\nPIM1\tH0VW44_CAVPO\nPIM1\tH0XYN3_OTOGA\nPIM1\tH1VHN5_COLHI\nPIM1\tH2ASC1_KAZAF\nPIM1\tH2QSX4_PANTR\nPIM1\tH2SLA2_TAKRU\nPIM1\tH6C7P0_EXODN\nPIM1\tH6QQW1_PUCGT\nPIM1\tH8X6C4_CANO9\nPIM1\tH9FB95_MACMU\nPIM1\tH9FL94_MACMU\nPIM1\tH9GNL0_ANOCA\nPIM1\tH9YX84_MACMU\nPIM1\tI1CFF4_RHIO9\nPIM1\tI1RQ47_GIBZE\nPIM1\tI2G635_USTH4\nPIM1\tI2GX13_TETBL\nPIM1\tI2JZ14_DEKBR\nPIM1\tI3N826_ICTTR\nPIM1\tI4YGT2_WALMC\nPIM1\tI8IHP7_ASPO3\nPIM1\tJ3K4X2_COCIM\nPIM1\tJ3NNS9_GAGT3\nPIM1\tJ4I9P3_9APHY\nPIM1\tJ4W9R5_BEAB2\nPIM1\tJ7S0X7_KAZNA\nPIM1\tJ8Q4N9_SACAR\nPIM1\tJ9VN78_CRYNH\nPIM1\tK0KXE5_WICCF\nPIM1\tK1X3I6_MARBU\nPIM1\tK2S3Z8_MACPH\nPIM1\tK3VQJ0_FUSPC\nPIM1\tK5WHH0_PHACS\nPIM1\tK5WU80_AGABU\nPIM1\tK7GDW6_PELSI\nPIM1\tK9FTA3_PEND2\nPIM1\tK9GEI2_PEND1\nPIM1\tL0PEK9_PNEJ8\nPIM1\tL2FFY8_COLFN\nPIM1\tL7I3A6_MAGOY\nPIM1\tL7JAD9_MAGOP\nPIM1\tL8G9X1_PSED2\nPIM1\tL8WS98_THACA\nPIM1\tLONM_ASHGO\nPIM1\tLONM_ASPFU\nPIM1\tLONM_ASPNC\nPIM1\tLONM_CANAL\nPIM1\tLONM_CANDC\nPIM1\tLONM_CANGA\nPIM1\tLONM_CRYNB\nPIM1\tLONM_CRYNJ\nPIM1\tLONM_DEBHA\nPIM1\tLONM_EMENI\nPIM1\tLONM_KLULA\nPIM1\tLONM_NEUCR\nPIM1\tLONM_PICST\nPIM1\tLONM_SCHPO\nPIM1\tLONM_YARLI\nPIM1\tLONM_YEAST\nPIM1\tM1VZG7_CLAP2\nPIM1\tM2N3C3_BAUPA\nPIM1\tM2PDK0_CERS8\nPIM1\tM2SQQ5_COCSN\nPIM1\tM2UBV1_COCH5\nPIM1\tM3AQP1_PSEFD\nPIM1\tM3D7B4_SPHMS\nPIM1\tM3YRQ5_MUSPF\nPIM1\tM5G859_DACPD\nPIM1\tM7NTF9_PNEMU\nPIM1\tM7SEZ1_EUTLA\nPIM1\tM7TU36_BOTF1\nPIM1\tM7X643_RHOT1\nPIM1\tM9LUB0_PSEA3\nPIM1\tN1J4Q1_BLUG1\nPIM1\tN1P905_YEASC\nPIM1\tN1PCR8_DOTSN\nPIM1\tN1R6Z2_FUSC4\nPIM1\tN4UGK1_FUSC1\nPIM1\tN4VHC6_COLOR\nPIM1\tN4X0V1_COCH4\nPIM1\tO77626_BOVIN\nPIM1\tPIM1_BOVIN\nPIM1\tPIM1_FELCA\nPIM1\tPIM1_HUMAN\nPIM1\tPIM1_MOUSE\nPIM1\tPIM1_RAT\nPIM1\tPIM2_DANRE\nPIM1\tPIM3_XENLA\nPIM1\tQ08BC0_DANRE\nPIM1\tQ0CYX2_ASPTN\nPIM1\tQ0U3Z3_PHANO\nPIM1\tQ2H096_CHAGB\nPIM1\tQ2UEU9_ASPOR\nPIM1\tQ3TYQ0_MOUSE\nPIM1\tQ3U4N9_MOUSE\nPIM1\tQ5U489_XENLA\nPIM1\tQ66II1_XENTR\nPIM1\tQ700R0_PIG\nPIM1\tQ8WPC8_PIMHY\nPIM1\tQ96WN2_PARBR\nPIM1\tQ9LXU1_ARATH\nPIM1\tQ9P8F9_PICPA\nPIM1\tR0IEH6_SETT2\nPIM1\tR1EMZ7_BOTPV\nPIM1\tR4X8A6_TAPDE\nPIM1\tR7YW37_CONA1\nPIM1\tR8BAL1_TOGMI\nPIM1\tR9APS2_WALI9\nPIM1\tR9P788_PSEHS\nPIM1\tR9XKP5_ASHAC\nPIM1\tRCC1_SCHPO\nPIM1\tS0DMN4_GIBF5\nPIM1\tS2J7E1_MUCC1\nPIM1\tS3C041_OPHP1\nPIM1\tS3CYJ2_GLAL2\nPIM1\tS6EHZ8_ZYGB2\nPIM1\tS7REE1_GLOTA\nPIM1\tS7ZHP4_PENO1\nPIM1\tS8C4N0_DACHA\nPIM1\tS8E9Q2_FOMPI\nPIM1\tS9RKC0_SCHOY\nPIM1\tS9XKF9_SCHCR\nPIM1\tT0K218_COLGC\nPIM1\tT5B616_AJEDE\nPIM1\tU1I0H0_ENDPU\nPIM1\tU4LNT7_PYROM\nPIM1\tU5HA14_USTV1\nPIM1\tU6DKB3_NEOVI\nPIM1\tU7PTF6_SPOS1\nPIM1\tU9UI38_RHIID\nPIM1\tV2XAL6_MONRO\nPIM1\tV5G1N3_BYSSN\nPIM1\tV5GK38_KALBG\nPIM1\tV8NSE6_OPHHA\nPIM1\tV9DL58_9EURO\nPIM1\tW0T609_KLUMD\nPIM1\tW1QGV9_OGAPD\nPIM1\tW2S1W0_9EURO\nPIM1\tW3VJU3_PSEA5\nPIM1\tW3XD35_PESFW\nPIM1\tW4KC98_9AGAM\nPIM1\tW5PY79_SHEEP\nPIM1\tW6MS05_9ASCO\nPIM1\tW6PS64_PENRF\nPIM1\tW6QMY0_PENRF\nPIM1\tW6YXK9_COCCA\nPIM1\tW6ZK67_COCMI\nPIM1\tW7E855_COCVI\nPIM1\tW7M2T0_GIBM7\nPIM1\tW9CLE4_SCLBF\nPIM1\tW9IW54_FUSOX\nPIM1\tW9KJG2_FUSOX\nPIM1\tW9Q1E5_FUSOX\nPIM1\tW9VKI5_9EURO\nPIM1\tW9VQZ9_9EURO\nPIM1\tW9XKA2_9EURO\nPIM1\tW9Z6S6_9EURO\nPIM1\tX0AG74_FUSOX\nPIM1\tX0CNC0_FUSOX\nPIM1\tX0HQZ7_FUSOX\nPIM1\tX0JUB1_FUSOX\nPIM1\tX0LUN6_FUSOX\n','From\tTo\nROCK1\tA0A096NF71_PAPAN\nROCK1\tA0A0C9R0Z7_9HYME\nROCK1\tA0A0D9RYH3_CHLSB\nROCK1\tA0A0K8VV64_BACLA\nROCK1\tA0A0R4IVY5_DANRE\nROCK1\tA0A0S7FMZ3_9TELE\nROCK1\tA0A0S7FMZ4_9TELE\nROCK1\tA0A0S7FN14_9TELE\nROCK1\tA0A0S7FN21_9TELE\nROCK1\tA0A0S7FN33_9TELE\nROCK1\tA0A0S7FN95_9TELE\nROCK1\tA0A0S7FNB4_9TELE\nROCK1\tA0A0S7FP64_9TELE\nROCK1\tA0A0S7FPV0_9TELE\nROCK1\tA0A0S7FPX0_9TELE\nROCK1\tA0A0S7FRC6_9TELE\nROCK1\tA0A0S7FS04_9TELE\nROCK1\tA0A0S7FT04_9TELE\nROCK1\tA0A0S7FTH4_9TELE\nROCK1\tA0A0S7FUH8_9TELE\nROCK1\tA0A0S7FVG8_9TELE\nROCK1\tA0A0S7FVS3_9TELE\nROCK1\tA0A0S7FVU5_9TELE\nROCK1\tA0A0S7FX03_9TELE\nROCK1\tA0A0S7FX22_9TELE\nROCK1\tA0A0S7LGB8_9TELE\nROCK1\tA0A0S7LGC5_9TELE\nROCK1\tA0A0S7LGC9_9TELE\nROCK1\tA0A0U1RQV4_HUMAN\nROCK1\tA0A0V0RV22_9BILA\nROCK1\tA0A0V0RV33_9BILA\nROCK1\tA0A0V0RV85_9BILA\nROCK1\tA0A0V0U539_9BILA\nROCK1\tA0A0V0U540_9BILA\nROCK1\tA0A0V0XBA0_9BILA\nROCK1\tA0A0V0XBC3_9BILA\nROCK1\tA0A0V0XBG6_9BILA\nROCK1\tA0A0V0XBY7_9BILA\nROCK1\tA0A0V0Y8F8_TRIPS\nROCK1\tA0A0V0Y8Q8_TRIPS\nROCK1\tA0A0V0Y9K2_TRIPS\nROCK1\tA0A0V1AA49_9BILA\nROCK1\tA0A0V1AAB6_9BILA\nROCK1\tA0A0V1AAL6_9BILA\nROCK1\tA0A0V1ABS4_9BILA\nROCK1\tA0A0V1CWT8_TRIBR\nROCK1\tA0A0V1CX02_TRIBR\nROCK1\tA0A0V1CXA9_TRIBR\nROCK1\tA0A0V1CXC0_TRIBR\nROCK1\tA0A0V1EAR2_TRIPS\nROCK1\tA0A0V1FNV1_TRIPS\nROCK1\tA0A0V1HUS0_9BILA\nROCK1\tA0A0V1HUY6_9BILA\nROCK1\tA0A0V1HV72_9BILA\nROCK1\tA0A0V1J5I8_TRIPS\nROCK1\tA0A1A7WKP0_9TELE\nROCK1\tA0A1A7Z3E8_9TELE\nROCK1\tA0A1A8APP7_NOTFU\nROCK1\tA0A1A8CDP2_9TELE\nROCK1\tA0A1A8DMH9_9TELE\nROCK1\tA0A1A8FVN1_9TELE\nROCK1\tA0A1A8GU87_9TELE\nROCK1\tA0A1A8HZJ1_NOTKU\nROCK1\tA0A1A8KJB4_NOTKU\nROCK1\tA0A1A8LBB2_9TELE\nROCK1\tA0A1A8N1Z8_9TELE\nROCK1\tA0A1A8N6M1_9TELE\nROCK1\tA0A1A8N792_9TELE\nROCK1\tA0A1A8NFP1_9TELE\nROCK1\tA0A1A8QSU8_9TELE\nROCK1\tA0A1A8RBD1_9TELE\nROCK1\tA0A1A8RVJ3_9TELE\nROCK1\tA0A1A8S9Z6_9TELE\nROCK1\tA0A1A8SGC6_9TELE\nROCK1\tA0A1A8UDN7_NOTFU\nROCK1\tA0A1D5PK41_CHICK\nROCK1\tA0A1S3F137_DIPOR\nROCK1\tA0A1S3WUE6_ERIEU\nROCK1\tA0A1U7R0D3_MESAU\nROCK1\tA0A1U7S2B7_ALLSI\nROCK1\tA0A1U7SCA9_ALLSI\nROCK1\tA0A1U7UIA3_TARSY\nROCK1\tA0A1U8CFL3_MESAU\nROCK1\tA0A1V4KFV2_PATFA\nROCK1\tA0A1V4KHQ9_PATFA\nROCK1\tA0A218V1C6_9PASE\nROCK1\tA0A250YMK1_CASCN\nROCK1\tA0A286XKW0_CAVPO\nROCK1\tA0A287A705_PIG\nROCK1\tA0A287ASJ3_PIG\nROCK1\tA0A287CYP5_ICTTR\nROCK1\tA0A2D0R7D9_ICTPU\nROCK1\tA0A2D0R7E3_ICTPU\nROCK1\tA0A2D0R9K0_ICTPU\nROCK1\tA0A2H8TVT7_9HEMI\nROCK1\tA0A2I0MW50_COLLI\nROCK1\tA0A2I2UXM8_FELCA\nROCK1\tA0A2I3HK74_NOMLE\nROCK1\tA0A2I3LC55_PAPAN\nROCK1\tA0A2I3LGE9_PAPAN\nROCK1\tA0A2I3MQ82_PAPAN\nROCK1\tA0A2I3T6I0_PANTR\nROCK1\tA0A2I3TWH9_PANTR\nROCK1\tA0A2I4CXV9_9TELE\nROCK1\tA0A2I4CXX9_9TELE\nROCK1\tA0A2K5EGW4_AOTNA\nROCK1\tA0A2K5EGW8_AOTNA\nROCK1\tA0A2K5EGY9_AOTNA\nROCK1\tA0A2K5EH21_AOTNA\nROCK1\tA0A2K5EH32_AOTNA\nROCK1\tA0A2K5IG11_COLAP\nROCK1\tA0A2K5IG79_COLAP\nROCK1\tA0A2K5LSE2_CERAT\nROCK1\tA0A2K5LSG8_CERAT\nROCK1\tA0A2K5Q381_CEBCA\nROCK1\tA0A2K5Q3G2_CEBCA\nROCK1\tA0A2K5WB67_MACFA\nROCK1\tA0A2K5WB74_MACFA\nROCK1\tA0A2K5WB78_MACFA\nROCK1\tA0A2K6AAY4_MANLE\nROCK1\tA0A2K6AB07_MANLE\nROCK1\tA0A2K6AB30_MANLE\nROCK1\tA0A2K6CYT9_MACNE\nROCK1\tA0A2K6CYX4_MACNE\nROCK1\tA0A2K6CYY0_MACNE\nROCK1\tA0A2K6F1E5_PROCO\nROCK1\tA0A2K6F1E7_PROCO\nROCK1\tA0A2K6KFZ5_RHIBE\nROCK1\tA0A2K6KFZ6_RHIBE\nROCK1\tA0A2K6RJ35_RHIRO\nROCK1\tA0A2K6RJ43_RHIRO\nROCK1\tA0A2P1MWP0_PIG\nROCK1\tA0A2P8ZJS0_BLAGE\nROCK1\tA0A2R9BZ36_PANPA\nROCK1\tA0A2R9BZ53_PANPA\nROCK1\tA0A2R9BZS0_PANPA\nROCK1\tA0A2S2PIQ9_SCHGA\nROCK1\tA0A2U3TAX1_CTEID\nROCK1\tA0A2U3WQ01_ODORO\nROCK1\tA0A2U3ZRM2_ODORO\nROCK1\tA0A2U4AYR8_TURTR\nROCK1\tA0A2U4AYS1_TURTR\nROCK1\tA0A2U4AZ74_TURTR\nROCK1\tA0A2Y9GRA2_NEOSC\nROCK1\tA0A2Y9PB74_DELLE\nROCK1\tA0A2Y9PIM2_DELLE\nROCK1\tA0A2Y9SKQ6_PHYMC\nROCK1\tA0A341CGA3_NEOAA\nROCK1\tA0A341CGC3_NEOAA\nROCK1\tA0A341CGJ9_NEOAA\nROCK1\tA0A384AW51_BALAS\nROCK1\tA0A384AW62_BALAS\nROCK1\tA0A384C1N6_URSMA\nROCK1\tA0A384C1R7_URSMA\nROCK1\tA0A384C2Z3_URSMA\nROCK1\tA0A3B3INV7_ORYLA\nROCK1\tA0A3Q1N562_BOVIN\nROCK1\tA0A3Q1ND05_BOVIN\nROCK1\tA0A3Q2L241_HORSE\nROCK1\tA0A3Q4EGR2_MOUSE\nROCK1\tA0A3Q4L2X0_MOUSE\nROCK1\tA0A3Q7QAB5_CALUR\nROCK1\tA0A3Q7QAC6_CALUR\nROCK1\tA0A3Q7QKA1_CALUR\nROCK1\tA0A3Q7RPX9_CALUR\nROCK1\tA0A3Q7SKN4_VULVU\nROCK1\tA0A3Q7T229_VULVU\nROCK1\tA0A3Q7U4V4_VULVU\nROCK1\tA0A3Q7VS94_URSAR\nROCK1\tA0A3Q7WBL2_URSAR\nROCK1\tA0A3Q7WMW5_URSAR\nROCK1\tA0A3R7C484_CLOSI\nROCK1\tA0A452EUH9_CAPHI\nROCK1\tA0A452RHQ2_URSAM\nROCK1\tA0A452RIP4_URSAM\nROCK1\tA0A452RIT7_URSAM\nROCK1\tA0A452RIV9_URSAM\nROCK1\tA0A452RIY1_URSAM\nROCK1\tA0A452RJ41_URSAM\nROCK1\tA0A452RJ68_URSAM\nROCK1\tA0A452RJK9_URSAM\nROCK1\tA0A452RJL2_URSAM\nROCK1\tA0A452RJP9_URSAM\nROCK1\tA0A452V132_URSMA\nROCK1\tA0A493U293_ANAPP\nROCK1\tA0A4W2BUZ1_BOBOX\nROCK1\tA0A4W2DWD9_BOBOX\nROCK1\tA0A4W2FUV4_BOBOX\nROCK1\tA0A4W3J5H8_CALMI\nROCK1\tA0A4W3J8E1_CALMI\nROCK1\tA0A4W3J8F1_CALMI\nROCK1\tA0A4W3JG56_CALMI\nROCK1\tA0A4W3JG66_CALMI\nROCK1\tA0A4W3JG77_CALMI\nROCK1\tA0A4W3JK40_CALMI\nROCK1\tA0A4W3JK55_CALMI\nROCK1\tA0A4W3K343_CALMI\nROCK1\tA0A4W6BZL1_LATCA\nROCK1\tA0A4W6BZS8_LATCA\nROCK1\tA0A4W6BZT8_LATCA\nROCK1\tA0A4W6C0E6_LATCA\nROCK1\tA0A4W6C5H1_LATCA\nROCK1\tA0A4W6C714_LATCA\nROCK1\tA0A4W6C731_LATCA\nROCK1\tA0A4W6C8N7_LATCA\nROCK1\tA0A4W6C8T4_LATCA\nROCK1\tA0A4X1VJA6_PIG\nROCK1\tA0A4X1VJB6_PIG\nROCK1\tA0A4X1VNF3_PIG\nROCK1\tA0A4X1VP18_PIG\nROCK1\tA0A4X1VPV6_PIG\nROCK1\tA0A4Z2HQK9_9TELE\nROCK1\tA0A5F4BRJ3_CANLF\nROCK1\tA0A5F4C813_CANLF\nROCK1\tA0A5F4CAD0_CANLF\nROCK1\tA0A5F4CZX6_CANLF\nROCK1\tA0A5F8G2A2_MONDO\nROCK1\tA0A5F8GA82_MONDO\nROCK1\tA0A5F8GTK1_MONDO\nROCK1\tA0A5F8H6U5_MONDO\nROCK1\tA0A5F9CLC5_RABIT\nROCK1\tA0A5G2QIS7_PIG\nROCK1\tA0A5G3H322_XENTR\nROCK1\tA0A5G3H339_XENTR\nROCK1\tA0A5G3H5C9_XENTR\nROCK1\tA0A5G3IBA5_XENTR\nROCK1\tA0A5G3IBB1_XENTR\nROCK1\tA0A5G3IBB5_XENTR\nROCK1\tA0A5G3IC77_XENTR\nROCK1\tA0A5G3L094_XENTR\nROCK1\tA0A5G3L1J1_XENTR\nROCK1\tA0A5G3L1N5_XENTR\nROCK1\tA0A5G3L2A6_XENTR\nROCK1\tA0A5G3L2C2_XENTR\nROCK1\tA0A5G3L2I0_XENTR\nROCK1\tA0A5G3L2N3_XENTR\nROCK1\tA0A5G3L3Y9_XENTR\nROCK1\tB5A7L3_MESAU\nROCK1\tD3ZN37_RAT\nROCK1\tD4A5S0_RAT\nROCK1\tD9ZGF8_HUMAN\nROCK1\tF1MF50_BOVIN\nROCK1\tF1NRH6_CHICK\nROCK1\tF1PU60_CANLF\nROCK1\tF6RGZ0_XENTR\nROCK1\tF6RHF0_XENTR\nROCK1\tF6VAP3_XENTR\nROCK1\tF6W1K2_XENTR\nROCK1\tF6XP01_XENTR\nROCK1\tF6XUQ9_HORSE\nROCK1\tF7G2Y1_MONDO\nROCK1\tG1KHH5_ANOCA\nROCK1\tG1LJU4_AILME\nROCK1\tG1LJU8_AILME\nROCK1\tG1NC22_MELGA\nROCK1\tG1PP69_MYOLU\nROCK1\tG1R545_NOMLE\nROCK1\tG1SMN4_RABIT\nROCK1\tG3TF10_LOXAF\nROCK1\tG3U5Y3_LOXAF\nROCK1\tH0VGZ3_CAVPO\nROCK1\tH0X6A1_OTOGA\nROCK1\tH2MTH9_ORYLA\nROCK1\tH2NW04_PONAB\nROCK1\tH2QEB3_PANTR\nROCK1\tH2UA39_TAKRU\nROCK1\tH2UA41_TAKRU\nROCK1\tH3B2B3_LATCH\nROCK1\tH9EPT5_MACMU\nROCK1\tI3LCJ1_PIG\nROCK1\tI3LFH3_PIG\nROCK1\tI3N5C1_ICTTR\nROCK1\tJ3KRH6_HUMAN\nROCK1\tK7FM57_PELSI\nROCK1\tL7MRQ2_HORSE\nROCK1\tM3X9E5_FELCA\nROCK1\tM3XIE5_LATCH\nROCK1\tM3Y296_MUSPF\nROCK1\tQ3TBE0_MOUSE\nROCK1\tQ3UE22_MOUSE\nROCK1\tQ49AA3_HUMAN\nROCK1\tQ9VXE3_DROME\nROCK1\tROCK1_ARATH\nROCK1\tROCK1_BOVIN\nROCK1\tROCK1_HUMAN\nROCK1\tROCK1_MOUSE\nROCK1\tROCK1_PANTR\nROCK1\tROCK1_RABIT\nROCK1\tROCK1_RAT\nROCK1\tU3DGP9_CALJA\nROCK1\tU3I4S8_ANAPP\nROCK1\tU6DY02_NEOVI\nROCK1\tV5G6D9_ANOGL\nROCK1\tV5I7V4_ANOGL\nROCK1\tV8NLF1_OPHHA\nROCK1\tV8NLP1_OPHHA\nROCK1\tV8NM03_OPHHA\nROCK1\tV8NP56_OPHHA\nROCK1\tV9K875_CALMI\nROCK1\tW5PGB5_SHEEP\nROCK1\tW5PGC0_SHEEP\nROCK1\tX2JC83_DROME\nROCK1\tX2JE40_DROME\nROCK1\tX2JKQ8_DROME\n','From\tTo\nPRKACA\tA0A024R7J0_HUMAN\nPRKACA\tA0A0A0MW38_PAPAN\nPRKACA\tA0A0A9VYG3_LYGHE\nPRKACA\tA0A0A9Y9R0_LYGHE\nPRKACA\tA0A0D9R215_CHLSB\nPRKACA\tA0A0P6JE74_HETGA\nPRKACA\tA0A0V0RPI9_9BILA\nPRKACA\tA0A0V0RPL6_9BILA\nPRKACA\tA0A0V0T3B5_9BILA\nPRKACA\tA0A0V0TPK9_9BILA\nPRKACA\tA0A0V0U4X9_9BILA\nPRKACA\tA0A0V0U512_9BILA\nPRKACA\tA0A0V0V4D0_9BILA\nPRKACA\tA0A0V0V6F9_9BILA\nPRKACA\tA0A0V0V6W7_9BILA\nPRKACA\tA0A0V0WLC0_9BILA\nPRKACA\tA0A0V0YM39_TRIPS\nPRKACA\tA0A0V0YM61_TRIPS\nPRKACA\tA0A0V0Z4A1_9BILA\nPRKACA\tA0A0V1BGF5_TRISP\nPRKACA\tA0A0V1BGY6_TRISP\nPRKACA\tA0A0V1C531_TRIBR\nPRKACA\tA0A0V1CV00_TRIBR\nPRKACA\tA0A0V1ELU6_TRIPS\nPRKACA\tA0A0V1EM22_TRIPS\nPRKACA\tA0A0V1G2I9_TRIPS\nPRKACA\tA0A0V1GW98_9BILA\nPRKACA\tA0A0V1GWP3_9BILA\nPRKACA\tA0A0V1GWT5_9BILA\nPRKACA\tA0A0V1GXE7_9BILA\nPRKACA\tA0A0V1H9T2_9BILA\nPRKACA\tA0A0V1HA13_9BILA\nPRKACA\tA0A0V1L4Q2_9BILA\nPRKACA\tA0A0V1L672_9BILA\nPRKACA\tA0A0V1LE74_9BILA\nPRKACA\tA0A0V1N976_9BILA\nPRKACA\tA0A0V1N9C2_9BILA\nPRKACA\tA0A0V1PPJ4_9BILA\nPRKACA\tA0A151N0U3_ALLMI\nPRKACA\tA0A1D5R1I4_MACMU\nPRKACA\tA0A1Q9CC87_SYMMI\nPRKACA\tA0A1S3GN80_DIPOR\nPRKACA\tA0A1S9RJK3_9EURO\nPRKACA\tA0A1U7S855_ALLSI\nPRKACA\tA0A1U7SIV0_ALLSI\nPRKACA\tA0A1U8DK74_ALLSI\nPRKACA\tA0A250YGG2_CASCN\nPRKACA\tA0A286Y5R9_CAVPO\nPRKACA\tA0A286ZUD6_PIG\nPRKACA\tA0A287ACC3_PIG\nPRKACA\tA0A2D0Q3W6_ICTPU\nPRKACA\tA0A2I3FWY5_NOMLE\nPRKACA\tA0A2I3HLP5_NOMLE\nPRKACA\tA0A2I3HUW2_NOMLE\nPRKACA\tA0A2I3LKB7_PAPAN\nPRKACA\tA0A2I3N1F5_PAPAN\nPRKACA\tA0A2I3NH23_PAPAN\nPRKACA\tA0A2K5CV01_AOTNA\nPRKACA\tA0A2K5ING1_COLAP\nPRKACA\tA0A2K5INT4_COLAP\nPRKACA\tA0A2K5INT9_COLAP\nPRKACA\tA0A2K5INU4_COLAP\nPRKACA\tA0A2K5P1Q6_CERAT\nPRKACA\tA0A2K5P1S2_CERAT\nPRKACA\tA0A2K5P1U0_CERAT\nPRKACA\tA0A2K5P1Y5_CERAT\nPRKACA\tA0A2K5RY70_CEBCA\nPRKACA\tA0A2K5RY88_CEBCA\nPRKACA\tA0A2K5RYB0_CEBCA\nPRKACA\tA0A2K5RYE0_CEBCA\nPRKACA\tA0A2K5UC31_MACFA\nPRKACA\tA0A2K5UC50_MACFA\nPRKACA\tA0A2K5YZ11_MANLE\nPRKACA\tA0A2K5YZ36_MANLE\nPRKACA\tA0A2K5YZ39_MANLE\nPRKACA\tA0A2K5YZ44_MANLE\nPRKACA\tA0A2K6DZQ7_MACNE\nPRKACA\tA0A2K6DZS8_MACNE\nPRKACA\tA0A2K6DZT3_MACNE\nPRKACA\tA0A2K6GRD5_PROCO\nPRKACA\tA0A2K6GRD9_PROCO\nPRKACA\tA0A2K6GRG8_PROCO\nPRKACA\tA0A2K6KPD0_RHIBE\nPRKACA\tA0A2K6KPD5_RHIBE\nPRKACA\tA0A2K6P4Q4_RHIRO\nPRKACA\tA0A2K6P4R4_RHIRO\nPRKACA\tA0A2K6P4S5_RHIRO\nPRKACA\tA0A2K6TUM0_SAIBB\nPRKACA\tA0A2K6TUM1_SAIBB\nPRKACA\tA0A2K6TUV3_SAIBB\nPRKACA\tA0A2P8YZZ2_BLAGE\nPRKACA\tA0A2R8ZE76_PANPA\nPRKACA\tA0A2R8ZE91_PANPA\nPRKACA\tA0A2R8ZHU4_PANPA\nPRKACA\tA0A2R8ZKS6_PANPA\nPRKACA\tA0A2U3VUI0_ODORO\nPRKACA\tA0A2U3VUM7_ODORO\nPRKACA\tA0A2U3VUS3_ODORO\nPRKACA\tA0A2U3Z6A9_LEPWE\nPRKACA\tA0A2U3Z737_LEPWE\nPRKACA\tA0A2Y9IBD9_NEOSC\nPRKACA\tA0A2Y9LZR7_DELLE\nPRKACA\tA0A2Y9TGS0_PHYMC\nPRKACA\tA0A337SVP4_FELCA\nPRKACA\tA0A340Y602_LIPVE\nPRKACA\tA0A340Y7C0_LIPVE\nPRKACA\tA0A340Y9M2_LIPVE\nPRKACA\tA0A341DAT0_NEOAA\nPRKACA\tA0A384A439_BALAS\nPRKACA\tA0A384DSH0_URSMA\nPRKACA\tA0A3Q0CG49_MESAU\nPRKACA\tA0A3Q0E622_TARSY\nPRKACA\tA0A3Q0FMQ3_ALLSI\nPRKACA\tA0A3Q0FS80_ALLSI\nPRKACA\tA0A3Q1N5A9_BOVIN\nPRKACA\tA0A3Q2IEW5_HORSE\nPRKACA\tA0A3Q2L0R7_HORSE\nPRKACA\tA0A3Q7NB76_CALUR\nPRKACA\tA0A3Q7NE67_CALUR\nPRKACA\tA0A3Q7NF15_CALUR\nPRKACA\tA0A3Q7SCK9_VULVU\nPRKACA\tA0A3Q7V4H7_URSAR\nPRKACA\tA0A452DIU2_BOVIN\nPRKACA\tA0A452EW03_CAPHI\nPRKACA\tA0A452EW28_CAPHI\nPRKACA\tA0A452EW89_CAPHI\nPRKACA\tA0A452TCI3_URSMA\nPRKACA\tA0A452TCI8_URSMA\nPRKACA\tA0A452TCJ6_URSMA\nPRKACA\tA0A452TCN2_URSMA\nPRKACA\tA0A452TCN4_URSMA\nPRKACA\tA0A455BC39_PHYMC\nPRKACA\tA0A4W2F1D4_BOBOX\nPRKACA\tA0A4W2GDJ9_BOBOX\nPRKACA\tA0A4W3JU56_CALMI\nPRKACA\tA0A4W3JW39_CALMI\nPRKACA\tA0A4W3JW43_CALMI\nPRKACA\tA0A4W3JW49_CALMI\nPRKACA\tA0A4W3K6P3_CALMI\nPRKACA\tA0A4W3KHU7_CALMI\nPRKACA\tA0A4X1V8A4_PIG\nPRKACA\tA0A4X1V926_PIG\nPRKACA\tA0A4X1VAB5_PIG\nPRKACA\tA0A4X1VAC1_PIG\nPRKACA\tA0A4X2M629_VOMUR\nPRKACA\tA0A4X2MCR9_VOMUR\nPRKACA\tA0A5F4CHW2_CANLF\nPRKACA\tA0A5F4WED9_CALJA\nPRKACA\tA0A5F8GM38_MONDO\nPRKACA\tA1L1M0_RAT\nPRKACA\tE6YE28_XENTR\nPRKACA\tF1PAT6_CANLF\nPRKACA\tF1SD45_PIG\nPRKACA\tF6UR98_ORNAN\nPRKACA\tF6URA9_ORNAN\nPRKACA\tF7AI06_HORSE\nPRKACA\tF7C1A7_MACMU\nPRKACA\tF7CFU5_XENTR\nPRKACA\tF7G4Q0_MONDO\nPRKACA\tF7I3D4_CALJA\nPRKACA\tG1M3I9_AILME\nPRKACA\tG1PB26_MYOLU\nPRKACA\tG1RSN2_NOMLE\nPRKACA\tG3SUV1_LOXAF\nPRKACA\tG3TYJ8_LOXAF\nPRKACA\tG3W5L7_SARHA\nPRKACA\tH0W1P5_CAVPO\nPRKACA\tH0X5M7_OTOGA\nPRKACA\tH2KTF1_CLOSI\nPRKACA\tH2KU48_CLOSI\nPRKACA\tH2NXS7_PONAB\nPRKACA\tH2QFK0_PANTR\nPRKACA\tH9EQQ4_MACMU\nPRKACA\tI3KHS2_ORENI\nPRKACA\tI3KHS3_ORENI\nPRKACA\tI3M074_ICTTR\nPRKACA\tJ9JDU9_MOUSE\nPRKACA\tJ9NXT4_CANLF\nPRKACA\tK7AVQ3_PANTR\nPRKACA\tK7EMV1_HUMAN\nPRKACA\tK7ENJ5_HUMAN\nPRKACA\tK7ERP6_HUMAN\nPRKACA\tKAPCA_BOVIN\nPRKACA\tKAPCA_CANLF\nPRKACA\tKAPCA_CRIGR\nPRKACA\tKAPCA_HUMAN\nPRKACA\tKAPCA_MOUSE\nPRKACA\tKAPCA_PIG\nPRKACA\tKAPCA_RAT\nPRKACA\tKAPCA_SHEEP\nPRKACA\tKPCA_HUMAN\nPRKACA\tM3WWL5_FELCA\nPRKACA\tM3XVE5_MUSPF\nPRKACA\tQ15136_HUMAN\nPRKACA\tQ28GZ8_XENTR\nPRKACA\tQ90WN3_XENLA\nPRKACA\tT2MGS3_HYDVU\nPRKACA\tU3D6C9_CALJA\nPRKACA\tV8PG77_OPHHA\nPRKACA\tV9KWZ2_CALMI\nPRKACA\tW5P925_SHEEP\n','From\tTo\nPRKCB\tA0A087XV30_POEFO\nPRKCB\tA0A096M811_POEFO\nPRKCB\tA0A096NHG6_PAPAN\nPRKCB\tA0A0D9R2T7_CHLSB\nPRKCB\tA0A0G2K5Q0_RAT\nPRKCB\tA0A1D5PCZ1_CHICK\nPRKCB\tA0A1D5PUY5_CHICK\nPRKCB\tA0A1S2ZYP8_ERIEU\nPRKCB\tA0A1S2ZYW3_ERIEU\nPRKCB\tA0A1S3EQF8_DIPOR\nPRKCB\tA0A1S3WGL9_ERIEU\nPRKCB\tA0A1U7T5L3_TARSY\nPRKCB\tA0A1U7T5L9_TARSY\nPRKCB\tA0A1V4KSW4_PATFA\nPRKCB\tA0A1V4KT03_PATFA\nPRKCB\tA0A218U8U5_9PASE\nPRKCB\tA0A218UAG9_9PASE\nPRKCB\tA0A250YJK3_CASCN\nPRKCB\tA0A286XTF5_CAVPO\nPRKCB\tA0A287D2K4_ICTTR\nPRKCB\tA0A2I0LWT2_COLLI\nPRKCB\tA0A2I2U7E2_FELCA\nPRKCB\tA0A2I3GPJ3_NOMLE\nPRKCB\tA0A2I3HJQ3_NOMLE\nPRKCB\tA0A2I3LEI3_PAPAN\nPRKCB\tA0A2I3N8C1_PAPAN\nPRKCB\tA0A2I3TR88_PANTR\nPRKCB\tA0A2I4D217_9TELE\nPRKCB\tA0A2I4D224_9TELE\nPRKCB\tA0A2K5DFZ0_AOTNA\nPRKCB\tA0A2K5DG25_AOTNA\nPRKCB\tA0A2K5DG42_AOTNA\nPRKCB\tA0A2K5DG53_AOTNA\nPRKCB\tA0A2K5J0A4_COLAP\nPRKCB\tA0A2K5J0B3_COLAP\nPRKCB\tA0A2K5J0T6_COLAP\nPRKCB\tA0A2K5L6J4_CERAT\nPRKCB\tA0A2K5L6M3_CERAT\nPRKCB\tA0A2K5L6M7_CERAT\nPRKCB\tA0A2K5S8J8_CEBCA\nPRKCB\tA0A2K5S8K8_CEBCA\nPRKCB\tA0A2K5S8K9_CEBCA\nPRKCB\tA0A2K5S8L4_CEBCA\nPRKCB\tA0A2K5S8L9_CEBCA\nPRKCB\tA0A2K5UT59_MACFA\nPRKCB\tA0A2K5UT70_MACFA\nPRKCB\tA0A2K5UT89_MACFA\nPRKCB\tA0A2K5UTE2_MACFA\nPRKCB\tA0A2K5Z9G2_MANLE\nPRKCB\tA0A2K5Z9H5_MANLE\nPRKCB\tA0A2K5Z9I6_MANLE\nPRKCB\tA0A2K6B2U4_MACNE\nPRKCB\tA0A2K6B2W5_MACNE\nPRKCB\tA0A2K6B2X5_MACNE\nPRKCB\tA0A2K6GTC6_PROCO\nPRKCB\tA0A2K6GTC7_PROCO\nPRKCB\tA0A2K6GTE2_PROCO\nPRKCB\tA0A2K6LEC2_RHIBE\nPRKCB\tA0A2K6LEF5_RHIBE\nPRKCB\tA0A2K6LEF6_RHIBE\nPRKCB\tA0A2K6PWJ6_RHIRO\nPRKCB\tA0A2K6TWG3_SAIBB\nPRKCB\tA0A2K6TWG6_SAIBB\nPRKCB\tA0A2K6TWG8_SAIBB\nPRKCB\tA0A2K6TWH1_SAIBB\nPRKCB\tA0A2R9AYU7_PANPA\nPRKCB\tA0A2R9B4X0_PANPA\nPRKCB\tA0A2U3VU21_ODORO\nPRKCB\tA0A2U3VU64_ODORO\nPRKCB\tA0A2U4A9B3_TURTR\nPRKCB\tA0A2U4A9B4_TURTR\nPRKCB\tA0A2Y9GZ49_NEOSC\nPRKCB\tA0A2Y9PG08_DELLE\nPRKCB\tA0A2Y9PLP9_DELLE\nPRKCB\tA0A2Y9PN85_DELLE\nPRKCB\tA0A2Y9SRW1_PHYMC\nPRKCB\tA0A2Y9STV5_PHYMC\nPRKCB\tA0A340WMW7_LIPVE\nPRKCB\tA0A340WQY5_LIPVE\nPRKCB\tA0A341BBW5_NEOAA\nPRKCB\tA0A341BC93_NEOAA\nPRKCB\tA0A341BDS4_NEOAA\nPRKCB\tA0A341BFC2_NEOAA\nPRKCB\tA0A384AGA0_BALAS\nPRKCB\tA0A384CKT0_URSMA\nPRKCB\tA0A3B3CLX1_ORYME\nPRKCB\tA0A3B3UBB3_9TELE\nPRKCB\tA0A3B3VJY8_9TELE\nPRKCB\tA0A3B3Y4F1_9TELE\nPRKCB\tA0A3B4AD67_9GOBI\nPRKCB\tA0A3B5MJS9_9TELE\nPRKCB\tA0A3B5PWJ6_XIPMA\nPRKCB\tA0A3B5PYW2_XIPMA\nPRKCB\tA0A3P8YBA3_ESOLU\nPRKCB\tA0A3P9Q318_POERE\nPRKCB\tA0A3P9Q3G8_POERE\nPRKCB\tA0A3Q0CN94_MESAU\nPRKCB\tA0A3Q0DLT1_TARSY\nPRKCB\tA0A3Q0GX99_ALLSI\nPRKCB\tA0A3Q1MGP0_BOVIN\nPRKCB\tA0A3Q2DI58_CYPVA\nPRKCB\tA0A3Q2G685_CYPVA\nPRKCB\tA0A3Q2GR43_CYPVA\nPRKCB\tA0A3Q2H8B6_HORSE\nPRKCB\tA0A3Q2I8A3_HORSE\nPRKCB\tA0A3Q2NVQ7_FUNHE\nPRKCB\tA0A3Q3K1E5_MONAL\nPRKCB\tA0A3Q3KC96_MONAL\nPRKCB\tA0A3Q3L8M0_9TELE\nPRKCB\tA0A3Q3R4H8_MONAL\nPRKCB\tA0A3Q7MTM5_CALUR\nPRKCB\tA0A3Q7MVL8_CALUR\nPRKCB\tA0A3Q7R2N3_VULVU\nPRKCB\tA0A3Q7RBT0_VULVU\nPRKCB\tA0A3Q7WEB4_URSAR\nPRKCB\tA0A3Q7X2C3_URSAR\nPRKCB\tA0A452DJ88_BOVIN\nPRKCB\tA0A452EYA6_CAPHI\nPRKCB\tA0A452EYD2_CAPHI\nPRKCB\tA0A452EYN7_CAPHI\nPRKCB\tA0A452IDS4_9SAUR\nPRKCB\tA0A452QPY0_URSAM\nPRKCB\tA0A452QQ87_URSAM\nPRKCB\tA0A452QQA1_URSAM\nPRKCB\tA0A452QQC6_URSAM\nPRKCB\tA0A452QQC9_URSAM\nPRKCB\tA0A452QQH0_URSAM\nPRKCB\tA0A452TU06_URSMA\nPRKCB\tA0A452TU12_URSMA\nPRKCB\tA0A452TU20_URSMA\nPRKCB\tA0A452TU27_URSMA\nPRKCB\tA0A493TNP5_ANAPP\nPRKCB\tA0A4W2BTX0_BOBOX\nPRKCB\tA0A4W2DNT1_BOBOX\nPRKCB\tA0A4W2I6A3_BOBOX\nPRKCB\tA0A4W5PJS1_9TELE\nPRKCB\tA0A4W6BX51_LATCA\nPRKCB\tA0A4W6BXP5_LATCA\nPRKCB\tA0A4X1UUL3_PIG\nPRKCB\tA0A4X1UWG6_PIG\nPRKCB\tA0A4X2LBZ0_VOMUR\nPRKCB\tA0A4Z2EIZ8_9TELE\nPRKCB\tA0A5F4C1S1_CANLF\nPRKCB\tA0A5F5Q0Y9_HORSE\nPRKCB\tA0A5F5XF74_FELCA\nPRKCB\tA0A5F5XMU0_FELCA\nPRKCB\tA0A5F8AEL4_MACMU\nPRKCB\tA0A5F8ANG9_MACMU\nPRKCB\tA0A5F8AS19_MACMU\nPRKCB\tA0A5F8AS63_MACMU\nPRKCB\tA0A5F8GDI8_MONDO\nPRKCB\tA0A5F8H1T3_MONDO\nPRKCB\tA0A5F8HD54_MONDO\nPRKCB\tA0A5F8HK24_MONDO\nPRKCB\tA0A5F9C2Q9_RABIT\nPRKCB\tA0A5F9DEA2_RABIT\nPRKCB\tA0A5F9DIQ6_RABIT\nPRKCB\tA0A5G3HY95_XENTR\nPRKCB\tA0A5G3HYK6_XENTR\nPRKCB\tA0A5G3HYP6_XENTR\nPRKCB\tA0A5G3HYU9_XENTR\nPRKCB\tA0A5G3HYX2_XENTR\nPRKCB\tA0A5G3JPE2_XENTR\nPRKCB\tA0A5G3JRB6_XENTR\nPRKCB\tA0A5G3JRE1_XENTR\nPRKCB\tA0A5G3KHB5_XENTR\nPRKCB\tF1LS36_RAT\nPRKCB\tF1LS42_RAT\nPRKCB\tF1PKU7_CANLF\nPRKCB\tF6VNZ8_XENTR\nPRKCB\tF7A632_MONDO\nPRKCB\tF7AMF7_HORSE\nPRKCB\tF7ASI5_HORSE\nPRKCB\tF7BYH7_MACMU\nPRKCB\tF7BYI4_MACMU\nPRKCB\tF7CL97_XENTR\nPRKCB\tF7DGK3_XENTR\nPRKCB\tF7I4K6_CALJA\nPRKCB\tF7I4K8_CALJA\nPRKCB\tG1M6H0_AILME\nPRKCB\tG1M6H5_AILME\nPRKCB\tG1N3S9_MELGA\nPRKCB\tG1N3X0_MELGA\nPRKCB\tG1P813_MYOLU\nPRKCB\tG1RMP5_NOMLE\nPRKCB\tG1THE4_RABIT\nPRKCB\tG3SMU4_LOXAF\nPRKCB\tG3UK81_LOXAF\nPRKCB\tG3W328_SARHA\nPRKCB\tH0V0W8_CAVPO\nPRKCB\tH0WI40_OTOGA\nPRKCB\tH0XI40_OTOGA\nPRKCB\tH0Z757_TAEGU\nPRKCB\tH2LY63_ORYLA\nPRKCB\tH2N315_ORYLA\nPRKCB\tH2NQF5_PONAB\nPRKCB\tH2QAS6_PANTR\nPRKCB\tH2QAS7_PANTR\nPRKCB\tH3A0J1_LATCH\nPRKCB\tH3A0J2_LATCH\nPRKCB\tH3BV73_HUMAN\nPRKCB\tH9FX30_MACMU\nPRKCB\tH9ZB65_MACMU\nPRKCB\tI3L148_HUMAN\nPRKCB\tI3L1Z0_HUMAN\nPRKCB\tI3MJ52_ICTTR\nPRKCB\tK7F9H3_PELSI\nPRKCB\tK9J4L8_PIG\nPRKCB\tKPCB_BOVIN\nPRKCB\tKPCB_HUMAN\nPRKCB\tKPCB_MOUSE\nPRKCB\tKPCB_RABIT\nPRKCB\tKPCB_RAT\nPRKCB\tKPCB_XENLA\nPRKCB\tKPCB_XENTR\nPRKCB\tM3WEI7_FELCA\nPRKCB\tM3YL93_MUSPF\nPRKCB\tM3ZDB8_XIPMA\nPRKCB\tQ80ZQ8_MOUSE\nPRKCB\tU3EZL6_CALJA\nPRKCB\tU3IN81_ANAPP\nPRKCB\tU3JM87_FICAL\nPRKCB\tW5Q878_SHEEP\n','From\tTo\nPRKCI\tA0A096N5E6_PAPAN\nPRKCI\tA0A0A1XPH0_ZEUCU\nPRKCI\tA0A0B8RVK6_PIG\nPRKCI\tA0A0D9RJT0_CHLSB\nPRKCI\tA0A0K2TS93_LEPSM\nPRKCI\tA0A0R4PUY9_9EUTH\nPRKCI\tA0A0R4PV00_SYLOL\nPRKCI\tA0A0R4PV75_BLABR\nPRKCI\tA0A0R4PV84_CROHI\nPRKCI\tA0A0R4PV94_9EUTH\nPRKCI\tA0A0R4PVA5_SUNMU\nPRKCI\tA0A0R4PZ46_9EUTH\nPRKCI\tA0A0R4PZ53_9EUTH\nPRKCI\tA0A0R4PZ65_9EUTH\nPRKCI\tA0A0R4PZ77_9EUTH\nPRKCI\tA0A0R4Q5W2_9EUTH\nPRKCI\tA0A0R4Q5X5_9EUTH\nPRKCI\tA0A0R4Q5Y5_9EUTH\nPRKCI\tA0A0R4QCG1_9EUTH\nPRKCI\tA0A0R4QCG3_9EUTH\nPRKCI\tA0A0R4QCG7_9EUTH\nPRKCI\tA0A0R4QD80_9EUTH\nPRKCI\tA0A0R4QD90_9EUTH\nPRKCI\tA0A0R4QD99_9EUTH\nPRKCI\tA0A0R4QEY4_9EUTH\nPRKCI\tA0A0R4QEZ6_SORMI\nPRKCI\tA0A0R4QF08_9EUTH\nPRKCI\tA0A0R4QF18_9EUTH\nPRKCI\tA0A0R4QF74_9EUTH\nPRKCI\tA0A0R4QF85_9EUTH\nPRKCI\tA0A0R4QF99_9EUTH\nPRKCI\tA0A0R4QFA8_9EUTH\nPRKCI\tA0A0R4QIF0_9EUTH\nPRKCI\tA0A0R4QIG2_CRORS\nPRKCI\tA0A0R4QIH6_9EUTH\nPRKCI\tA0A0R4QII6_9EUTH\nPRKCI\tA0A0R4QKM7_9EUTH\nPRKCI\tA0A0R4QKP0_9EUTH\nPRKCI\tA0A0R4QKQ5_9EUTH\nPRKCI\tA0A0R4QKR6_SUNET\nPRKCI\tA0A0R4QSR6_9EUTH\nPRKCI\tA0A0R4QSS5_9EUTH\nPRKCI\tA0A0R4QSU6_9EUTH\nPRKCI\tA0A0R4RBU7_9EUTH\nPRKCI\tA0A0R4RBW2_9EUTH\nPRKCI\tA0A0R4RBX3_9EUTH\nPRKCI\tA0A0R4RKT0_9EUTH\nPRKCI\tA0A0R4RKU4_9EUTH\nPRKCI\tA0A0R4RKV9_SYLJO\nPRKCI\tA0A0R4RR74_9EUTH\nPRKCI\tA0A0R4RR85_9EUTH\nPRKCI\tA0A0R4RR96_9EUTH\nPRKCI\tA0A0R4RSE1_9EUTH\nPRKCI\tA0A0R4RSF0_9EUTH\nPRKCI\tA0A0R4RSG1_9EUTH\nPRKCI\tA0A0R4RSH7_SUNMG\nPRKCI\tA0A0R4RT17_9EUTH\nPRKCI\tA0A0R4RT31_9EUTH\nPRKCI\tA0A0R4RT43_9EUTH\nPRKCI\tA0A0V0TY73_9BILA\nPRKCI\tA0A0V0TY93_9BILA\nPRKCI\tA0A0V0TYA5_9BILA\nPRKCI\tA0A0V0TYB1_9BILA\nPRKCI\tA0A0V0TYB6_9BILA\nPRKCI\tA0A0V0TYB8_9BILA\nPRKCI\tA0A0V0TYE4_9BILA\nPRKCI\tA0A0V0TYH8_9BILA\nPRKCI\tA0A0V0XGI2_TRIPS\nPRKCI\tA0A0V1FFM4_TRIPS\nPRKCI\tA0A0V1H8Q9_9BILA\nPRKCI\tA0A0V1H9W6_9BILA\nPRKCI\tA0A0V1HA87_9BILA\nPRKCI\tA0A0V1IZT3_TRIPS\nPRKCI\tA0A0V1JGP8_TRIPS\nPRKCI\tA0A0V1JGQ5_TRIPS\nPRKCI\tA0A0V1JGS7_TRIPS\nPRKCI\tA0A0V1NY07_9BILA\nPRKCI\tA0A0V1NY41_9BILA\nPRKCI\tA0A0V1NY54_9BILA\nPRKCI\tA0A0V1NY98_9BILA\nPRKCI\tA0A0V1NYB1_9BILA\nPRKCI\tA0A0V1NYI6_9BILA\nPRKCI\tA0A146MCV9_LYGHE\nPRKCI\tA0A1A7YHV0_9TELE\nPRKCI\tA0A1A7ZI05_NOTFU\nPRKCI\tA0A1A8EJ58_9TELE\nPRKCI\tA0A1A8F7E6_9TELE\nPRKCI\tA0A1A8JR27_NOTKU\nPRKCI\tA0A1A8NUG5_9TELE\nPRKCI\tA0A1A8PEQ8_9TELE\nPRKCI\tA0A1D5RJL1_MACMU\nPRKCI\tA0A1S3A5T4_ERIEU\nPRKCI\tA0A1S3F5T2_DIPOR\nPRKCI\tA0A1U7QNF6_MESAU\nPRKCI\tA0A1U7STS9_TARSY\nPRKCI\tA0A1U8DQ39_ALLSI\nPRKCI\tA0A1V4K3L7_PATFA\nPRKCI\tA0A218V3A1_9PASE\nPRKCI\tA0A220XJ50_GAZDO\nPRKCI\tA0A220XJ53_GAZLE\nPRKCI\tA0A220XJ61_GAZCU\nPRKCI\tA0A220XJ64_NANDA\nPRKCI\tA0A220XJ73_EUDRU\nPRKCI\tA0A286ZLS5_PIG\nPRKCI\tA0A287B5V4_PIG\nPRKCI\tA0A287DE40_ICTTR\nPRKCI\tA0A287DF27_ICTTR\nPRKCI\tA0A2B4SBT9_STYPI\nPRKCI\tA0A2D0R8U9_ICTPU\nPRKCI\tA0A2I0MBA3_COLLI\nPRKCI\tA0A2I3M796_PAPAN\nPRKCI\tA0A2I3N5M9_PAPAN\nPRKCI\tA0A2I3RQJ8_PANTR\nPRKCI\tA0A2I4B545_9TELE\nPRKCI\tA0A2K5D6N5_AOTNA\nPRKCI\tA0A2K5D6P4_AOTNA\nPRKCI\tA0A2K5IIP4_COLAP\nPRKCI\tA0A2K5IIW1_COLAP\nPRKCI\tA0A2K5LZ26_CERAT\nPRKCI\tA0A2K5LZ41_CERAT\nPRKCI\tA0A2K5LZ47_CERAT\nPRKCI\tA0A2K5Q4Q6_CEBCA\nPRKCI\tA0A2K5Q4U6_CEBCA\nPRKCI\tA0A2K5WLR1_MACFA\nPRKCI\tA0A2K5ZTE4_MANLE\nPRKCI\tA0A2K5ZTG3_MANLE\nPRKCI\tA0A2K6DWH7_MACNE\nPRKCI\tA0A2K6DWK5_MACNE\nPRKCI\tA0A2K6EV59_PROCO\nPRKCI\tA0A2K6MTA9_RHIBE\nPRKCI\tA0A2K6MTC9_RHIBE\nPRKCI\tA0A2K6QNJ3_RHIRO\nPRKCI\tA0A2K6TXQ0_SAIBB\nPRKCI\tA0A2P8Y6N6_BLAGE\nPRKCI\tA0A2P8YBS6_BLAGE\nPRKCI\tA0A2R9AXC3_PANPA\nPRKCI\tA0A2R9AXJ6_PANPA\nPRKCI\tA0A2R9B127_PANPA\nPRKCI\tA0A2U3VF24_ODORO\nPRKCI\tA0A2U3XY54_LEPWE\nPRKCI\tA0A2U3ZPS4_ODORO\nPRKCI\tA0A2U4C875_TURTR\nPRKCI\tA0A2U4C878_TURTR\nPRKCI\tA0A2U4C879_TURTR\nPRKCI\tA0A2U4C8G1_TURTR\nPRKCI\tA0A2Y9EFE9_PHYMC\nPRKCI\tA0A2Y9I7N5_NEOSC\nPRKCI\tA0A2Y9N5A7_DELLE\nPRKCI\tA0A2Y9NA44_DELLE\nPRKCI\tA0A2Y9NG85_DELLE\nPRKCI\tA0A340XZR9_LIPVE\nPRKCI\tA0A341AGT8_NEOAA\nPRKCI\tA0A341AJP6_NEOAA\nPRKCI\tA0A341AKZ7_NEOAA\nPRKCI\tA0A384BD15_BALAS\nPRKCI\tA0A384D9Y9_URSMA\nPRKCI\tA0A3Q2H1V5_HORSE\nPRKCI\tA0A3Q7PMB3_CALUR\nPRKCI\tA0A3Q7PMG1_CALUR\nPRKCI\tA0A3Q7U837_VULVU\nPRKCI\tA0A3Q7W095_URSAR\nPRKCI\tA0A419PLI3_CLOSI\nPRKCI\tA0A452C694_BALAS\nPRKCI\tA0A452G7X5_CAPHI\nPRKCI\tA0A452G824_CAPHI\nPRKCI\tA0A452G8A4_CAPHI\nPRKCI\tA0A452H0I6_9SAUR\nPRKCI\tA0A452RHX4_URSAM\nPRKCI\tA0A452RIV6_URSAM\nPRKCI\tA0A452RJ64_URSAM\nPRKCI\tA0A452RJ71_URSAM\nPRKCI\tA0A452RJ73_URSAM\nPRKCI\tA0A452RJA0_URSAM\nPRKCI\tA0A452RJB2_URSAM\nPRKCI\tA0A452RJC1_URSAM\nPRKCI\tA0A452RJC4_URSAM\nPRKCI\tA0A452RJI2_URSAM\nPRKCI\tA0A452U3Y1_URSMA\nPRKCI\tA0A452U469_URSMA\nPRKCI\tA0A452U4D6_URSMA\nPRKCI\tA0A452U4E6_URSMA\nPRKCI\tA0A452U4F5_URSMA\nPRKCI\tA0A452U4G5_URSMA\nPRKCI\tA0A452U4G9_URSMA\nPRKCI\tA0A452U4J0_URSMA\nPRKCI\tA0A452U4J5_URSMA\nPRKCI\tA0A452U4L5_URSMA\nPRKCI\tA0A455BD63_PHYMC\nPRKCI\tA0A455BG89_PHYMC\nPRKCI\tA0A4P2UHS4_9LAGO\nPRKCI\tA0A4W2G0S5_BOBOX\nPRKCI\tA0A4W2HNK4_BOBOX\nPRKCI\tA0A4W3HY80_CALMI\nPRKCI\tA0A4W3HY96_CALMI\nPRKCI\tA0A4W3I197_CALMI\nPRKCI\tA0A4W3I4T2_CALMI\nPRKCI\tA0A4W3I4U7_CALMI\nPRKCI\tA0A4W3I4V7_CALMI\nPRKCI\tA0A4W3I5Y3_CALMI\nPRKCI\tA0A4W3I610_CALMI\nPRKCI\tA0A4W3IP57_CALMI\nPRKCI\tA0A4X1V9A5_PIG\nPRKCI\tA0A4X1VCU7_PIG\nPRKCI\tA0A4X1VCV9_PIG\nPRKCI\tA0A4X1VD21_PIG\nPRKCI\tA0A4X2L879_VOMUR\nPRKCI\tA0A4Z2FJG1_9TELE\nPRKCI\tA0A4Z2GXR4_9TELE\nPRKCI\tA0A5F4BPH2_CANLF\nPRKCI\tA0A5F4C0Z0_CANLF\nPRKCI\tA0A5F4C4C4_CANLF\nPRKCI\tA0A5F4CCN8_CANLF\nPRKCI\tA0A5F4CGG1_CANLF\nPRKCI\tA0A5F4CGL1_CANLF\nPRKCI\tA0A5F7Z9M4_MACMU\nPRKCI\tA0A5F7ZX99_MACMU\nPRKCI\tA0A5F8AS54_MACMU\nPRKCI\tA0A5F9CT72_RABIT\nPRKCI\tA0A5G3HK13_XENTR\nPRKCI\tB5BUK7_HUMAN\nPRKCI\tD3YXY0_MOUSE\nPRKCI\tE1BWA3_CHICK\nPRKCI\tF1MQ96_BOVIN\nPRKCI\tF1P0L9_CHICK\nPRKCI\tF1PG28_CANLF\nPRKCI\tF1SH28_PIG\nPRKCI\tF6QWV4_MOUSE\nPRKCI\tF6WUE5_XENTR\nPRKCI\tF6WY19_MACMU\nPRKCI\tF7C4A7_CALJA\nPRKCI\tF7CNR8_XENTR\nPRKCI\tF7D3T6_HORSE\nPRKCI\tF7DVL1_XENTR\nPRKCI\tF7EGB8_ORNAN\nPRKCI\tF7G9R7_MONDO\nPRKCI\tF8SXE7_CALER\nPRKCI\tF8SXE8_DREPE\nPRKCI\tF8SXF5_9SCIU\nPRKCI\tF8SXG4_9SCIU\nPRKCI\tF8SXG8_9SCIU\nPRKCI\tF8SXG9_TAMSW\nPRKCI\tG1KRY6_ANOCA\nPRKCI\tG1LDD4_AILME\nPRKCI\tG1LDE2_AILME\nPRKCI\tG1NCU1_MELGA\nPRKCI\tG1P6M6_MYOLU\nPRKCI\tG1QZH0_NOMLE\nPRKCI\tG1T7N4_RABIT\nPRKCI\tG3SV72_LOXAF\nPRKCI\tG3UQ38_MELGA\nPRKCI\tG3WQL9_SARHA\nPRKCI\tG8FPL2_NANDA\nPRKCI\tH0VSU5_CAVPO\nPRKCI\tH0WSB9_OTOGA\nPRKCI\tH0ZL30_TAEGU\nPRKCI\tH2MUM2_ORYLA\nPRKCI\tH2PBZ2_PONAB\nPRKCI\tH2QNQ4_PANTR\nPRKCI\tH2SM61_TAKRU\nPRKCI\tH2SM62_TAKRU\nPRKCI\tH3B4M1_LATCH\nPRKCI\tH9F9M5_MACMU\nPRKCI\tI0FTC9_MACMU\nPRKCI\tI3KQP3_ORENI\nPRKCI\tI3KQP4_ORENI\nPRKCI\tI3M147_ICTTR\nPRKCI\tJ9QCZ2_TRASP\nPRKCI\tJ9QCZ4_SYLGR\nPRKCI\tJ9QCZ5_CEPSI\nPRKCI\tJ9QCZ6_CEPNG\nPRKCI\tJ9QCZ7_CEPNA\nPRKCI\tJ9QDV1_CEPNG\nPRKCI\tJ9QDV3_CEPNI\nPRKCI\tJ9QDV4_CEPRF\nPRKCI\tJ9QDZ9_CEPSI\nPRKCI\tJ9QE01_CEPOG\nPRKCI\tJ9QE02_CEPAD\nPRKCI\tJ9QE03_CEPSP\nPRKCI\tJ9QE48_CEPDO\nPRKCI\tJ9QE49_CEPLE\nPRKCI\tJ9QE50_PHIMA\nPRKCI\tJ9QE51_PHIMO\nPRKCI\tJ9QE52_CEPHR\nPRKCI\tJ9QE53_CEPHR\nPRKCI\tJ9QE97_TRASR\nPRKCI\tJ9QE98_CEPDO\nPRKCI\tJ9QEA0_CEPCA\nPRKCI\tJ9QEA1_SYLGR\nPRKCI\tJ9QEA2_CEPJE\nPRKCI\tK7G1M2_PELSI\nPRKCI\tKPCI_DANRE\nPRKCI\tKPCI_HUMAN\nPRKCI\tKPCI_MOUSE\nPRKCI\tKPCI_PONAB\nPRKCI\tKPCI_RAT\nPRKCI\tM3XF13_FELCA\nPRKCI\tM3Z088_MUSPF\nPRKCI\tM3Z9F6_NOMLE\nPRKCI\tQ28D02_XENTR\nPRKCI\tQ28EN9_XENTR\nPRKCI\tQ3TJJ5_MOUSE\nPRKCI\tQ3UGW9_MOUSE\nPRKCI\tQ3UMD5_MOUSE\nPRKCI\tQ50G96_TRASR\nPRKCI\tQ50G97_TRASP\nPRKCI\tQ50G99_TRAEU\nPRKCI\tQ50GA1_9CETA\nPRKCI\tQ50GA3_TRAAN\nPRKCI\tQ50GA4_TRAIM\nPRKCI\tQ50GA6_TRAST\nPRKCI\tQ50GA8_TRAOR\nPRKCI\tQ50GA9_TAUDE\nPRKCI\tQ5DTK3_MOUSE\nPRKCI\tQ5EGD6_XENTR\nPRKCI\tQ6DI53_DANRE\nPRKCI\tQ6GLE4_XENTR\nPRKCI\tQ6UIN1_MACMU\nPRKCI\tQ6UIN2_PANTR\nPRKCI\tQ91569_XENLA\nPRKCI\tQ95MV0_OUROU\nPRKCI\tQ95MV1_SYLGR\nPRKCI\tQ95MV2_PHIMO\nPRKCI\tQ95MV3_OREOR\nPRKCI\tQ95MV4_NEOMO\nPRKCI\tQ95MV5_RAPME\nPRKCI\tQ95MV6_RAPSH\nPRKCI\tQ95MV7_RAPCA\nPRKCI\tQ95MV8_LITWA\nPRKCI\tQ95MV9_ANTMR\nPRKCI\tQ95MW0_PELCP\nPRKCI\tQ95MW1_KOBLE\nPRKCI\tQ95MW2_REDRE\nPRKCI\tQ95MW3_REDFU\nPRKCI\tQ95MW4_CONGN\nPRKCI\tQ95MW5_ALCBU\nPRKCI\tQ95MW6_BEAHU\nPRKCI\tQ95MW7_HIPEQ\nPRKCI\tQ95MW8_ORYDA\nPRKCI\tQ95MW9_ORYGA\nPRKCI\tQ95MX0_TRAOR\nPRKCI\tQ95MX1_SYNCA\nPRKCI\tT2MGA0_HYDVU\nPRKCI\tU3FID0_CALJA\nPRKCI\tU3ILM2_ANAPP\nPRKCI\tU3K1R8_FICAL\nPRKCI\tV8P786_OPHHA\nPRKCI\tW0HDI0_KOBKO\nPRKCI\tW0HDL0_KOBLE\nPRKCI\tW0HGW8_REDAR\nPRKCI\tW0HGX3_KOBVA\nPRKCI\tW0HHY0_KOBEL\nPRKCI\tW0HI34_KOBME\nPRKCI\tW0HI39_PELCP\nPRKCI\tW5NQ81_SHEEP\n','From\tTo\nAKT2\tA0A0A0MRF1_HUMAN\nAKT2\tA0A0B8RZS0_PIG\nAKT2\tA0A0D9QUT3_CHLSB\nAKT2\tA0A0P6K3T3_HETGA\nAKT2\tA0A0S7HIT4_9TELE\nAKT2\tA0A0S7HKU5_9TELE\nAKT2\tA0A0S7HL38_9TELE\nAKT2\tA0A0S7HMK6_9TELE\nAKT2\tA0A0S7HN38_9TELE\nAKT2\tA0A0S7HNR1_9TELE\nAKT2\tA0A0U4CQG9_HUMAN\nAKT2\tA0A0V1AG58_9BILA\nAKT2\tA0A0V1LT75_9BILA\nAKT2\tA0A1B0GXA2_HUMAN\nAKT2\tA0A1D5RH80_MACMU\nAKT2\tA0A1L8F8Y4_XENLA\nAKT2\tA0A1P8B652_ARATH\nAKT2\tA0A1Q1NIF4_TACFU\nAKT2\tA0A1Q9DSV8_SYMMI\nAKT2\tA0A1Q9E8T4_SYMMI\nAKT2\tA0A1S3ALD0_ERIEU\nAKT2\tA0A1S3ALD3_ERIEU\nAKT2\tA0A1S3GIW6_DIPOR\nAKT2\tA0A1U7RDQ5_MESAU\nAKT2\tA0A1U7SGJ4_ALLSI\nAKT2\tA0A1U8CNN2_MESAU\nAKT2\tA0A250YJ14_CASCN\nAKT2\tA0A251TJS6_HELAN\nAKT2\tA0A286Y1G4_CAVPO\nAKT2\tA0A287AQP0_PIG\nAKT2\tA0A287D5Z6_ICTTR\nAKT2\tA0A287D7Q9_ICTTR\nAKT2\tA0A289ZN17_SINCH\nAKT2\tA0A2I0X5V6_9ASPA\nAKT2\tA0A2I2U7T4_FELCA\nAKT2\tA0A2I2YDQ0_GORGO\nAKT2\tA0A2I3FY31_NOMLE\nAKT2\tA0A2I3HF09_NOMLE\nAKT2\tA0A2I3RZ59_PANTR\nAKT2\tA0A2I3T1M4_PANTR\nAKT2\tA0A2J8QFS7_PANTR\nAKT2\tA0A2K5CD49_AOTNA\nAKT2\tA0A2K5CDJ6_AOTNA\nAKT2\tA0A2K5CDQ7_AOTNA\nAKT2\tA0A2K5CDS3_AOTNA\nAKT2\tA0A2K5IYW9_COLAP\nAKT2\tA0A2K5IYZ8_COLAP\nAKT2\tA0A2K5IZE2_COLAP\nAKT2\tA0A2K5IZI9_COLAP\nAKT2\tA0A2K5N3I1_CERAT\nAKT2\tA0A2K5N3I8_CERAT\nAKT2\tA0A2K5N3L8_CERAT\nAKT2\tA0A2K5N3Z5_CERAT\nAKT2\tA0A2K5QVH7_CEBCA\nAKT2\tA0A2K5QVI3_CEBCA\nAKT2\tA0A2K5WA10_MACFA\nAKT2\tA0A2K5WA33_MACFA\nAKT2\tA0A2K5WA53_MACFA\nAKT2\tA0A2K5WA57_MACFA\nAKT2\tA0A2K5Z269_MANLE\nAKT2\tA0A2K5Z2C1_MANLE\nAKT2\tA0A2K5Z2C9_MANLE\nAKT2\tA0A2K5Z2E7_MANLE\nAKT2\tA0A2K6CZ34_MACNE\nAKT2\tA0A2K6CZ71_MACNE\nAKT2\tA0A2K6CZ90_MACNE\nAKT2\tA0A2K6CZB9_MACNE\nAKT2\tA0A2K6FY47_PROCO\nAKT2\tA0A2K6FY52_PROCO\nAKT2\tA0A2K6FY63_PROCO\nAKT2\tA0A2K6ME01_RHIBE\nAKT2\tA0A2K6ME02_RHIBE\nAKT2\tA0A2K6ME06_RHIBE\nAKT2\tA0A2K6ME19_RHIBE\nAKT2\tA0A2K6RHX4_RHIRO\nAKT2\tA0A2K6RHX8_RHIRO\nAKT2\tA0A2K6RHY4_RHIRO\nAKT2\tA0A2K6RI07_RHIRO\nAKT2\tA0A2K6RI08_RHIRO\nAKT2\tA0A2K6V3L9_SAIBB\nAKT2\tA0A2K6V3M0_SAIBB\nAKT2\tA0A2K6V3N9_SAIBB\nAKT2\tA0A2K6V3R5_SAIBB\nAKT2\tA0A2P0XJ58_PAROL\nAKT2\tA0A2P0XJ59_CTEID\nAKT2\tA0A2R9A0Q4_PANPA\nAKT2\tA0A2R9A1U2_PANPA\nAKT2\tA0A2R9A659_PANPA\nAKT2\tA0A2U3VMW7_ODORO\nAKT2\tA0A2U3VMY0_ODORO\nAKT2\tA0A2U3VMY7_ODORO\nAKT2\tA0A2U4BTN9_TURTR\nAKT2\tA0A2Y9I541_NEOSC\nAKT2\tA0A2Y9PDM4_DELLE\nAKT2\tA0A340XBA5_LIPVE\nAKT2\tA0A340XF35_LIPVE\nAKT2\tA0A341DDW1_NEOAA\nAKT2\tA0A371E534_MUCPR\nAKT2\tA0A383ZY25_BALAS\nAKT2\tA0A384BWG0_URSMA\nAKT2\tA0A384BWG4_URSMA\nAKT2\tA0A384BWJ6_URSMA\nAKT2\tA0A384BXF3_URSMA\nAKT2\tA0A384BXP4_URSMA\nAKT2\tA0A3B3R5A8_9TELE\nAKT2\tA0A3P8WWB4_CYNSE\nAKT2\tA0A3P8ZBC9_ESOLU\nAKT2\tA0A3Q0E6C3_TARSY\nAKT2\tA0A3Q1MQ45_BOVIN\nAKT2\tA0A3Q1NH21_BOVIN\nAKT2\tA0A3Q3RJF1_9TELE\nAKT2\tA0A3Q7MSL1_CALUR\nAKT2\tA0A3Q7TKI8_VULVU\nAKT2\tA0A3Q7TPR2_URSAR\nAKT2\tA0A3Q7UCY3_URSAR\nAKT2\tA0A3Q7UPM4_URSAR\nAKT2\tA0A3Q7V5C6_URSAR\nAKT2\tA0A3Q7VJF8_URSAR\nAKT2\tA0A452EP20_CAPHI\nAKT2\tA0A452EPC7_CAPHI\nAKT2\tA0A452EQ89_CAPHI\nAKT2\tA0A452ISF6_9SAUR\nAKT2\tA0A452ISW3_9SAUR\nAKT2\tA0A452SX32_URSAM\nAKT2\tA0A452V0L7_URSMA\nAKT2\tA0A452V0Q6_URSMA\nAKT2\tA0A4W2CI15_BOBOX\nAKT2\tA0A4W4EQ44_ELEEL\nAKT2\tA0A4W4EQ51_ELEEL\nAKT2\tA0A4W4EQC7_ELEEL\nAKT2\tA0A4W4ER08_ELEEL\nAKT2\tA0A4W4ER13_ELEEL\nAKT2\tA0A4W4ET51_ELEEL\nAKT2\tA0A4X1SRE9_PIG\nAKT2\tA0A4X1SS21_PIG\nAKT2\tA0A4X2LZF9_VOMUR\nAKT2\tA0A4X2LZG3_VOMUR\nAKT2\tA0A5F4BNM0_CANLF\nAKT2\tA0A5F5PXD8_HORSE\nAKT2\tA0A5F5PYR8_HORSE\nAKT2\tA0A5F5Y0J8_FELCA\nAKT2\tA0A5F9ZHJ8_HUMAN\nAKT2\tA0A5G2R290_PIG\nAKT2\tA0A5G3HWP6_XENTR\nAKT2\tA0A5G3HWV2_XENTR\nAKT2\tA0A5G3JNX6_XENTR\nAKT2\tA5PH37_PHYPA\nAKT2\tA8MX96_HUMAN\nAKT2\tACT2_SCHCO\nAKT2\tAKT2_ALTAL\nAKT2\tAKT2_ARATH\nAKT2\tAKT2_HUMAN\nAKT2\tAKT2_MOUSE\nAKT2\tAKT2_RAT\nAKT2\tB1WB72_XENTR\nAKT2\tB5DGG0_SALSA\nAKT2\tC9J258_HUMAN\nAKT2\tC9JC83_HUMAN\nAKT2\tC9JHS6_HUMAN\nAKT2\tC9JIF6_HUMAN\nAKT2\tC9JIJ1_HUMAN\nAKT2\tD0QYP3_SALSA\nAKT2\tD3YXM7_MOUSE\nAKT2\tD3YZJ5_MOUSE\nAKT2\tD3Z0M3_MOUSE\nAKT2\tD3Z3N2_MOUSE\nAKT2\tD3Z490_MOUSE\nAKT2\tD3Z5X2_MOUSE\nAKT2\tE0AFZ5_FELCA\nAKT2\tE1B9D1_BOVIN\nAKT2\tE7EVP8_HUMAN\nAKT2\tF1PRG4_CANLF\nAKT2\tF1PSA7_CANLF\nAKT2\tF1RHD4_PIG\nAKT2\tF6QE96_MACMU\nAKT2\tF6RGP1_HORSE\nAKT2\tF6UIV6_XENTR\nAKT2\tF6XQD8_MACMU\nAKT2\tF6Z3Z2_XENTR\nAKT2\tF7BKI3_MONDO\nAKT2\tF7CIM4_XENTR\nAKT2\tF7CPB0_XENTR\nAKT2\tF7D9P4_XENTR\nAKT2\tF7DV69_XENTR\nAKT2\tF7EDW3_XENTR\nAKT2\tF7HW88_CALJA\nAKT2\tF7HZP9_CALJA\nAKT2\tF7I4A2_CALJA\nAKT2\tF8WHG5_MOUSE\nAKT2\tG1M1Q8_AILME\nAKT2\tG1NXG4_MYOLU\nAKT2\tG1SD70_RABIT\nAKT2\tG3QGC8_GORGO\nAKT2\tG3SRC6_LOXAF\nAKT2\tG9BWQ2_PIG\nAKT2\tH0VK35_CAVPO\nAKT2\tH0XA83_OTOGA\nAKT2\tH2MGP2_ORYLA\nAKT2\tH2NYU6_PONAB\nAKT2\tH2QGC2_PANTR\nAKT2\tH2TAG4_TAKRU\nAKT2\tH3BHT2_LATCH\nAKT2\tH9G6X3_ANOCA\nAKT2\tI3M1D1_ICTTR\nAKT2\tJ3KRI8_HUMAN\nAKT2\tJ3KSY8_HUMAN\nAKT2\tJ3KT31_HUMAN\nAKT2\tJ3KTC6_HUMAN\nAKT2\tJ3KTP4_HUMAN\nAKT2\tJ3QKW1_HUMAN\nAKT2\tJ3QL45_HUMAN\nAKT2\tJ3QLS6_HUMAN\nAKT2\tK0KGH2_WICCF\nAKT2\tK7CC20_PANTR\nAKT2\tK7CDC2_PANTR\nAKT2\tK7GLV2_PIG\nAKT2\tK7GMU6_PIG\nAKT2\tM0QZK3_HUMAN\nAKT2\tM0QZW8_HUMAN\nAKT2\tM0R0P9_HUMAN\nAKT2\tM0R275_HUMAN\nAKT2\tM0R283_HUMAN\nAKT2\tM3WGH1_FELCA\nAKT2\tM3Z1Y7_MUSPF\nAKT2\tQ06XL5_HORVU\nAKT2\tQ0P4T1_XENTR\nAKT2\tQ3HSE5_RAT\nAKT2\tQ3TY95_MOUSE\nAKT2\tQ6P4H3_HUMAN\nAKT2\tQ8CE74_MOUSE\nAKT2\tQ8UUX0_DANRE\nAKT2\tQ9PUJ3_CHICK\nAKT2\tU3F4Y8_CALJA\nAKT2\tU6D7I2_NEOVI\nAKT2\tW5P981_SHEEP\nAKT2\tW5UM01_ICTPU\nAKT2\tW5XLD4_9AVES\n','From\tTo\nGRK6\tA0A096NBB0_PAPAN\nGRK6\tA0A0D9R8V9_CHLSB\nGRK6\tA0A0G2K6E5_RAT\nGRK6\tA0A0S7KLS1_9TELE\nGRK6\tA0A0S7KMH1_9TELE\nGRK6\tA0A0S7KMR8_9TELE\nGRK6\tA0A0S7KR09_9TELE\nGRK6\tA0A0S7KR69_9TELE\nGRK6\tA0A1A7WSL9_9TELE\nGRK6\tA0A1A7YKD7_9TELE\nGRK6\tA0A1A8BZ45_9TELE\nGRK6\tA0A1A8DVU2_9TELE\nGRK6\tA0A1A8E9N4_9TELE\nGRK6\tA0A1A8FHT3_9TELE\nGRK6\tA0A1A8FXU2_9TELE\nGRK6\tA0A1A8IJA0_NOTKU\nGRK6\tA0A1A8KTW4_NOTKU\nGRK6\tA0A1A8M0R2_9TELE\nGRK6\tA0A1A8QMZ0_9TELE\nGRK6\tA0A1A8QWW9_9TELE\nGRK6\tA0A1A8QX66_9TELE\nGRK6\tA0A1A8RMD8_9TELE\nGRK6\tA0A1A8V207_NOTFU\nGRK6\tA0A1D5NUM4_CHICK\nGRK6\tA0A1D5PXA1_CHICK\nGRK6\tA0A1L8GXX0_XENLA\nGRK6\tA0A1S2ZTP5_ERIEU\nGRK6\tA0A1S2ZTR0_ERIEU\nGRK6\tA0A1S3F5D2_DIPOR\nGRK6\tA0A1S3WCV2_ERIEU\nGRK6\tA0A1S3WDF2_ERIEU\nGRK6\tA0A1U7QGC4_MESAU\nGRK6\tA0A1U7QPW8_MESAU\nGRK6\tA0A1U7TZ80_TARSY\nGRK6\tA0A1U7U964_TARSY\nGRK6\tA0A1U8DIR7_ALLSI\nGRK6\tA0A1U8DIV4_ALLSI\nGRK6\tA0A1V4KU87_PATFA\nGRK6\tA0A218UP77_9PASE\nGRK6\tA0A250Y2Y5_CASCN\nGRK6\tA0A286XFK9_CAVPO\nGRK6\tA0A286XKP6_CAVPO\nGRK6\tA0A286YDA5_MOUSE\nGRK6\tA0A286YDZ5_MOUSE\nGRK6\tA0A286YE10_MOUSE\nGRK6\tA0A287A215_PIG\nGRK6\tA0A287AM21_PIG\nGRK6\tA0A287BRD7_PIG\nGRK6\tA0A287D7V6_ICTTR\nGRK6\tA0A2D0SYU1_ICTPU\nGRK6\tA0A2I3M216_PAPAN\nGRK6\tA0A2I3MBD9_PAPAN\nGRK6\tA0A2I3NHW7_PAPAN\nGRK6\tA0A2I3RAH3_PANTR\nGRK6\tA0A2I3T5E1_PANTR\nGRK6\tA0A2I4AVK8_9TELE\nGRK6\tA0A2J8RDS4_PONAB\nGRK6\tA0A2K5CTF8_AOTNA\nGRK6\tA0A2K5CTG9_AOTNA\nGRK6\tA0A2K5ISL7_COLAP\nGRK6\tA0A2K5ISN2_COLAP\nGRK6\tA0A2K5MXF0_CERAT\nGRK6\tA0A2K5MXK6_CERAT\nGRK6\tA0A2K5RHX3_CEBCA\nGRK6\tA0A2K5RHY6_CEBCA\nGRK6\tA0A2K5RI18_CEBCA\nGRK6\tA0A2K5RI36_CEBCA\nGRK6\tA0A2K5WTN2_MACFA\nGRK6\tA0A2K5WTX8_MACFA\nGRK6\tA0A2K5YJE3_MANLE\nGRK6\tA0A2K5YJK0_MANLE\nGRK6\tA0A2K6DHF4_MACNE\nGRK6\tA0A2K6DHK6_MACNE\nGRK6\tA0A2K6DHL0_MACNE\nGRK6\tA0A2K6FPT6_PROCO\nGRK6\tA0A2K6FPU2_PROCO\nGRK6\tA0A2K6FPU3_PROCO\nGRK6\tA0A2K6KH99_RHIBE\nGRK6\tA0A2K6KHE5_RHIBE\nGRK6\tA0A2K6KHG0_RHIBE\nGRK6\tA0A2K6KHH0_RHIBE\nGRK6\tA0A2K6RXK6_RHIRO\nGRK6\tA0A2K6RXL0_RHIRO\nGRK6\tA0A2K6RXL2_RHIRO\nGRK6\tA0A2K6RXP7_RHIRO\nGRK6\tA0A2K6RXQ2_RHIRO\nGRK6\tA0A2K6TGL4_SAIBB\nGRK6\tA0A2K6TGQ0_SAIBB\nGRK6\tA0A2K6TGS9_SAIBB\nGRK6\tA0A2K6TGW4_SAIBB\nGRK6\tA0A2R9C3V6_PANPA\nGRK6\tA0A2R9C3W2_PANPA\nGRK6\tA0A2U3W0N2_ODORO\nGRK6\tA0A2U3W0Q9_ODORO\nGRK6\tA0A2U3W0R5_ODORO\nGRK6\tA0A2U3XP02_LEPWE\nGRK6\tA0A2U3XP03_LEPWE\nGRK6\tA0A2U4BZ35_TURTR\nGRK6\tA0A2U4BZ41_TURTR\nGRK6\tA0A2U4BZ45_TURTR\nGRK6\tA0A2U4BZ46_TURTR\nGRK6\tA0A2U4BZ82_TURTR\nGRK6\tA0A2U4BZ88_TURTR\nGRK6\tA0A2U4BZ89_TURTR\nGRK6\tA0A2U4BZ93_TURTR\nGRK6\tA0A2U4BZA0_TURTR\nGRK6\tA0A2Y9HRY0_NEOSC\nGRK6\tA0A2Y9HYT7_NEOSC\nGRK6\tA0A2Y9MD32_DELLE\nGRK6\tA0A2Y9MGK2_DELLE\nGRK6\tA0A2Y9MM68_DELLE\nGRK6\tA0A2Y9MSJ1_DELLE\nGRK6\tA0A2Y9S0F2_PHYMC\nGRK6\tA0A2Y9S7F1_PHYMC\nGRK6\tA0A337SEA7_FELCA\nGRK6\tA0A340XP23_LIPVE\nGRK6\tA0A340XP92_LIPVE\nGRK6\tA0A340XSI3_LIPVE\nGRK6\tA0A341BEU7_NEOAA\nGRK6\tA0A341BF50_NEOAA\nGRK6\tA0A341BI16_NEOAA\nGRK6\tA0A341BI20_NEOAA\nGRK6\tA0A383Z3U8_BALAS\nGRK6\tA0A383Z3W2_BALAS\nGRK6\tA0A383Z424_BALAS\nGRK6\tA0A383Z447_BALAS\nGRK6\tA0A384C7T5_URSMA\nGRK6\tA0A384C823_URSMA\nGRK6\tA0A3B3I332_ORYLA\nGRK6\tA0A3B3IL67_ORYLA\nGRK6\tA0A3Q2LD08_HORSE\nGRK6\tA0A3Q7NM74_CALUR\nGRK6\tA0A3Q7NNE5_CALUR\nGRK6\tA0A3Q7NWY4_CALUR\nGRK6\tA0A3Q7Q380_CALUR\nGRK6\tA0A3Q7SK55_VULVU\nGRK6\tA0A3Q7T122_VULVU\nGRK6\tA0A3Q7TL39_VULVU\nGRK6\tA0A3Q7TL70_VULVU\nGRK6\tA0A3Q7WP64_URSAR\nGRK6\tA0A3Q7WYA7_URSAR\nGRK6\tA0A3Q7XK61_URSAR\nGRK6\tA0A452C8H3_BALAS\nGRK6\tA0A452FBC0_CAPHI\nGRK6\tA0A452FBD0_CAPHI\nGRK6\tA0A452GJB3_9SAUR\nGRK6\tA0A452SE61_URSAM\nGRK6\tA0A452VJ06_URSMA\nGRK6\tA0A452VJ42_URSMA\nGRK6\tA0A4W2DAY3_BOBOX\nGRK6\tA0A4W2EUA8_BOBOX\nGRK6\tA0A4W2ID07_BOBOX\nGRK6\tA0A4W3GCZ5_CALMI\nGRK6\tA0A4W3GD20_CALMI\nGRK6\tA0A4W3GD59_CALMI\nGRK6\tA0A4W3GD68_CALMI\nGRK6\tA0A4W3GD71_CALMI\nGRK6\tA0A4W3GD83_CALMI\nGRK6\tA0A4W3GD93_CALMI\nGRK6\tA0A4W3GF79_CALMI\nGRK6\tA0A4W3GT99_CALMI\nGRK6\tA0A4W4GYJ0_ELEEL\nGRK6\tA0A4W4H0R8_ELEEL\nGRK6\tA0A4W4H162_ELEEL\nGRK6\tA0A4W4H1C6_ELEEL\nGRK6\tA0A4W4H4T3_ELEEL\nGRK6\tA0A4W4H5J8_ELEEL\nGRK6\tA0A4W4H5P8_ELEEL\nGRK6\tA0A4W4H5Z9_ELEEL\nGRK6\tA0A4W4H651_ELEEL\nGRK6\tA0A4X1SE20_PIG\nGRK6\tA0A4X1SE31_PIG\nGRK6\tA0A4X1SE32_PIG\nGRK6\tA0A4X1SF50_PIG\nGRK6\tA0A4X2JMJ1_VOMUR\nGRK6\tA0A4X2JR08_VOMUR\nGRK6\tA0A5F4BPS2_CANLF\nGRK6\tA0A5F4CGL6_CANLF\nGRK6\tA0A5F8A9L1_MACMU\nGRK6\tA0A5F9CFH9_RABIT\nGRK6\tA0A5F9CW95_RABIT\nGRK6\tA0A5F9D238_RABIT\nGRK6\tA0A5F9D4L0_RABIT\nGRK6\tA0A5F9DKS1_RABIT\nGRK6\tD6R9V4_HUMAN\nGRK6\tD6RC88_HUMAN\nGRK6\tD6RDA3_HUMAN\nGRK6\tD6RHC7_HUMAN\nGRK6\tD6RHX8_HUMAN\nGRK6\tE1BP29_BOVIN\nGRK6\tE2RB39_CANLF\nGRK6\tE7F3T7_DANRE\nGRK6\tF1LNP2_RAT\nGRK6\tF1P1Z9_CHICK\nGRK6\tF6VXX6_XENTR\nGRK6\tF6X9W7_CANLF\nGRK6\tF6Y4G2_MONDO\nGRK6\tF7CEP4_HORSE\nGRK6\tF7CTI8_XENTR\nGRK6\tF7FDB6_ORNAN\nGRK6\tF7FY59_MACMU\nGRK6\tF8W9W2_HUMAN\nGRK6\tG1KF64_ANOCA\nGRK6\tG1MIH0_AILME\nGRK6\tG1MYV0_MELGA\nGRK6\tG1P216_MYOLU\nGRK6\tG1RQQ7_NOMLE\nGRK6\tG1SPY5_RABIT\nGRK6\tG3T5W5_LOXAF\nGRK6\tG3TWM8_LOXAF\nGRK6\tG3WNW9_SARHA\nGRK6\tGRK6_HUMAN\nGRK6\tGRK6_MOUSE\nGRK6\tGRK6_RAT\nGRK6\tH0V297_CAVPO\nGRK6\tH0WUF8_OTOGA\nGRK6\tH0Y967_HUMAN\nGRK6\tH0YQ60_TAEGU\nGRK6\tH2MVW1_ORYLA\nGRK6\tH2RQK6_TAKRU\nGRK6\tH2RQK8_TAKRU\nGRK6\tH3B618_LATCH\nGRK6\tH9F8Z1_MACMU\nGRK6\tH9YV86_MACMU\nGRK6\tI3KHR7_ORENI\nGRK6\tI3MBK9_ICTTR\nGRK6\tJ9P2C2_CANLF\nGRK6\tK7BUA3_PANTR\nGRK6\tK7F674_PELSI\nGRK6\tM3VWU6_FELCA\nGRK6\tM3YKS1_MUSPF\nGRK6\tO70296_MOUSE\nGRK6\tP97548_RAT\nGRK6\tP97549_RAT\nGRK6\tQ3TTW2_MOUSE\nGRK6\tQ3UK88_MOUSE\nGRK6\tQ68F06_XENLA\nGRK6\tQ792R0_MOUSE\nGRK6\tQ792R1_RAT\nGRK6\tQ96AD6_HUMAN\nGRK6\tQ9EP84_MOUSE\nGRK6\tQ9EPB9_MOUSE\nGRK6\tU3DNH5_CALJA\nGRK6\tU3FAF6_CALJA\nGRK6\tU3FHI5_CALJA\nGRK6\tU3K273_FICAL\nGRK6\tU6DLF6_NEOVI\nGRK6\tV8PEN5_OPHHA\nGRK6\tW5P4U4_SHEEP\n','From\tTo\nPRKCQ\tA0A087X0I9_HUMAN\nPRKCQ\tA0A096P3D1_PAPAN\nPRKCQ\tA0A0B8RTC1_PIG\nPRKCQ\tA0A0D9RM39_CHLSB\nPRKCQ\tA0A0P6JK15_HETGA\nPRKCQ\tA0A1A7XCJ3_9TELE\nPRKCQ\tA0A1A7YGW3_9TELE\nPRKCQ\tA0A1A8EZ63_9TELE\nPRKCQ\tA0A1A8IXA0_NOTKU\nPRKCQ\tA0A1A8V841_NOTFU\nPRKCQ\tA0A1S3AG78_ERIEU\nPRKCQ\tA0A1S3AG83_ERIEU\nPRKCQ\tA0A1S3EKE5_DIPOR\nPRKCQ\tA0A1S3SG97_SALSA\nPRKCQ\tA0A1S3WQ86_ERIEU\nPRKCQ\tA0A1S3WQ96_ERIEU\nPRKCQ\tA0A1U7QPA6_MESAU\nPRKCQ\tA0A1U7RLA2_ALLSI\nPRKCQ\tA0A1U7TP62_TARSY\nPRKCQ\tA0A1U7TZ83_TARSY\nPRKCQ\tA0A1U7U962_TARSY\nPRKCQ\tA0A1U8BRT8_MESAU\nPRKCQ\tA0A1V4JZE7_PATFA\nPRKCQ\tA0A218V9U0_9PASE\nPRKCQ\tA0A250YL60_CASCN\nPRKCQ\tA0A287A0E4_PIG\nPRKCQ\tA0A287AGV0_PIG\nPRKCQ\tA0A287AH32_PIG\nPRKCQ\tA0A287AK84_PIG\nPRKCQ\tA0A287AS62_PIG\nPRKCQ\tA0A287AT11_PIG\nPRKCQ\tA0A287BIN7_PIG\nPRKCQ\tA0A2D0T4R5_ICTPU\nPRKCQ\tA0A2I0MH50_COLLI\nPRKCQ\tA0A2I2UC76_FELCA\nPRKCQ\tA0A2I2UL73_FELCA\nPRKCQ\tA0A2I3H238_NOMLE\nPRKCQ\tA0A2I3H5M6_NOMLE\nPRKCQ\tA0A2I3LI69_PAPAN\nPRKCQ\tA0A2I3M7Y4_PAPAN\nPRKCQ\tA0A2I3MUY1_PAPAN\nPRKCQ\tA0A2I3SNQ3_PANTR\nPRKCQ\tA0A2I3SR38_PANTR\nPRKCQ\tA0A2I3TX37_PANTR\nPRKCQ\tA0A2I4C126_9TELE\nPRKCQ\tA0A2I4C150_9TELE\nPRKCQ\tA0A2K5DWW3_AOTNA\nPRKCQ\tA0A2K5DWY4_AOTNA\nPRKCQ\tA0A2K5DWY9_AOTNA\nPRKCQ\tA0A2K5HWM0_COLAP\nPRKCQ\tA0A2K5HWM4_COLAP\nPRKCQ\tA0A2K5HWT1_COLAP\nPRKCQ\tA0A2K5MA47_CERAT\nPRKCQ\tA0A2K5MA50_CERAT\nPRKCQ\tA0A2K5MAA7_CERAT\nPRKCQ\tA0A2K5RVT4_CEBCA\nPRKCQ\tA0A2K5RVV1_CEBCA\nPRKCQ\tA0A2K5RVV6_CEBCA\nPRKCQ\tA0A2K5WLP6_MACFA\nPRKCQ\tA0A2K5WLW3_MACFA\nPRKCQ\tA0A2K5WLX2_MACFA\nPRKCQ\tA0A2K5YIJ5_MANLE\nPRKCQ\tA0A2K5YIK0_MANLE\nPRKCQ\tA0A2K5YIL7_MANLE\nPRKCQ\tA0A2K6B686_MACNE\nPRKCQ\tA0A2K6B692_MACNE\nPRKCQ\tA0A2K6B696_MACNE\nPRKCQ\tA0A2K6GLQ5_PROCO\nPRKCQ\tA0A2K6GLS1_PROCO\nPRKCQ\tA0A2K6GLT4_PROCO\nPRKCQ\tA0A2K6GLU8_PROCO\nPRKCQ\tA0A2K6M5K4_RHIBE\nPRKCQ\tA0A2K6M5K5_RHIBE\nPRKCQ\tA0A2K6M5K7_RHIBE\nPRKCQ\tA0A2K6Q449_RHIRO\nPRKCQ\tA0A2K6Q452_RHIRO\nPRKCQ\tA0A2K6Q470_RHIRO\nPRKCQ\tA0A2K6Q476_RHIRO\nPRKCQ\tA0A2K6SVS1_SAIBB\nPRKCQ\tA0A2K6SVT4_SAIBB\nPRKCQ\tA0A2K6SVT9_SAIBB\nPRKCQ\tA0A2R9A0W2_PANPA\nPRKCQ\tA0A2R9A0X9_PANPA\nPRKCQ\tA0A2R9A237_PANPA\nPRKCQ\tA0A2R9A580_PANPA\nPRKCQ\tA0A2R9A6E9_PANPA\nPRKCQ\tA0A2U3VSM6_ODORO\nPRKCQ\tA0A2U3VSR6_ODORO\nPRKCQ\tA0A2U3VSV1_ODORO\nPRKCQ\tA0A2U3XTI0_LEPWE\nPRKCQ\tA0A2U3ZDN6_ODORO\nPRKCQ\tA0A2U4B6D0_TURTR\nPRKCQ\tA0A2Y9HKX9_NEOSC\nPRKCQ\tA0A2Y9HL49_NEOSC\nPRKCQ\tA0A2Y9NPP6_DELLE\nPRKCQ\tA0A2Y9NSM1_DELLE\nPRKCQ\tA0A2Y9NSM5_DELLE\nPRKCQ\tA0A2Y9NXW9_DELLE\nPRKCQ\tA0A2Y9P1H2_DELLE\nPRKCQ\tA0A2Y9T381_PHYMC\nPRKCQ\tA0A337S2V7_FELCA\nPRKCQ\tA0A340XXI8_LIPVE\nPRKCQ\tA0A340Y2K8_LIPVE\nPRKCQ\tA0A340Y348_LIPVE\nPRKCQ\tA0A341ABI2_NEOAA\nPRKCQ\tA0A341ADR8_NEOAA\nPRKCQ\tA0A341AEP0_NEOAA\nPRKCQ\tA0A341AG18_NEOAA\nPRKCQ\tA0A369J7Q3_HYPMA\nPRKCQ\tA0A369JJH1_HYPMA\nPRKCQ\tA0A383YXG1_BALAS\nPRKCQ\tA0A384DA05_URSMA\nPRKCQ\tA0A384DA14_URSMA\nPRKCQ\tA0A3Q0HAA6_ALLSI\nPRKCQ\tA0A3Q2GTN9_HORSE\nPRKCQ\tA0A3Q2HMP5_HORSE\nPRKCQ\tA0A3Q7MPX7_CALUR\nPRKCQ\tA0A3Q7T6D8_VULVU\nPRKCQ\tA0A3Q7UEB1_URSAR\nPRKCQ\tA0A3Q7V9I8_URSAR\nPRKCQ\tA0A452EU22_CAPHI\nPRKCQ\tA0A452EU28_CAPHI\nPRKCQ\tA0A452H4G1_9SAUR\nPRKCQ\tA0A452H4G8_9SAUR\nPRKCQ\tA0A452QHX6_URSAM\nPRKCQ\tA0A452QI12_URSAM\nPRKCQ\tA0A452QI35_URSAM\nPRKCQ\tA0A452QI36_URSAM\nPRKCQ\tA0A452QI53_URSAM\nPRKCQ\tA0A455BTB8_PHYMC\nPRKCQ\tA0A455BTC5_PHYMC\nPRKCQ\tA0A455BU04_PHYMC\nPRKCQ\tA0A455BU10_PHYMC\nPRKCQ\tA0A455BWQ6_PHYMC\nPRKCQ\tA0A455BWR1_PHYMC\nPRKCQ\tA0A493TMB6_ANAPP\nPRKCQ\tA0A4W2CDT9_BOBOX\nPRKCQ\tA0A4W2E3Z0_BOBOX\nPRKCQ\tA0A4W2E9V6_BOBOX\nPRKCQ\tA0A4W2EJR5_BOBOX\nPRKCQ\tA0A4W2FVT8_BOBOX\nPRKCQ\tA0A4W3IF95_CALMI\nPRKCQ\tA0A4W3IFA8_CALMI\nPRKCQ\tA0A4W3IN58_CALMI\nPRKCQ\tA0A4W3IN72_CALMI\nPRKCQ\tA0A4W3IN83_CALMI\nPRKCQ\tA0A4W3IRA7_CALMI\nPRKCQ\tA0A4W3IRC4_CALMI\nPRKCQ\tA0A4W4G397_ELEEL\nPRKCQ\tA0A4W4G3U5_ELEEL\nPRKCQ\tA0A4W4G3V0_ELEEL\nPRKCQ\tA0A4W4G6B9_ELEEL\nPRKCQ\tA0A4W4G6C4_ELEEL\nPRKCQ\tA0A4W6D8A2_LATCA\nPRKCQ\tA0A4W6D8B0_LATCA\nPRKCQ\tA0A4W6D8C1_LATCA\nPRKCQ\tA0A4X1T700_PIG\nPRKCQ\tA0A4X1T8P0_PIG\nPRKCQ\tA0A4X1T9P8_PIG\nPRKCQ\tA0A4X1T9R8_PIG\nPRKCQ\tA0A4X1TAI7_PIG\nPRKCQ\tA0A4X1TAL6_PIG\nPRKCQ\tA0A4X2L6K7_VOMUR\nPRKCQ\tA0A4X2LFJ4_VOMUR\nPRKCQ\tA0A4Y2AG13_ARAVE\nPRKCQ\tA0A4Z2GAN2_9TELE\nPRKCQ\tA0A5B8WEK9_CTEID\nPRKCQ\tA0A5F4BVV7_CANLF\nPRKCQ\tA0A5F4C928_CANLF\nPRKCQ\tA0A5F4CBW9_CANLF\nPRKCQ\tA0A5F4DHB8_CANLF\nPRKCQ\tA0A5F5PWV5_HORSE\nPRKCQ\tA0A5F5Q0K3_HORSE\nPRKCQ\tA0A5F8GJ78_MONDO\nPRKCQ\tA0A5F8GR82_MONDO\nPRKCQ\tA0A5F9C0X2_RABIT\nPRKCQ\tA0A5F9CGE6_RABIT\nPRKCQ\tA0A5G3KA22_XENTR\nPRKCQ\tA3KP71_DANRE\nPRKCQ\tA4IGL9_XENTR\nPRKCQ\tA6H667_MOUSE\nPRKCQ\tE1BMG4_BOVIN\nPRKCQ\tE2RNP2_CANLF\nPRKCQ\tF1LM10_RAT\nPRKCQ\tF1QTL5_DANRE\nPRKCQ\tF1R506_DANRE\nPRKCQ\tF1RUL3_PIG\nPRKCQ\tF6PVR3_ORNAN\nPRKCQ\tF6WA17_HORSE\nPRKCQ\tF7AYM7_XENTR\nPRKCQ\tF7G6Y7_MONDO\nPRKCQ\tF7HC75_MACMU\nPRKCQ\tF7IL85_CALJA\nPRKCQ\tF8WGT6_MOUSE\nPRKCQ\tG1KDN6_ANOCA\nPRKCQ\tG1MBR9_AILME\nPRKCQ\tG1MY89_MELGA\nPRKCQ\tG1RM82_NOMLE\nPRKCQ\tG1SY23_RABIT\nPRKCQ\tG3SNX2_LOXAF\nPRKCQ\tG3WMI8_SARHA\nPRKCQ\tG3WMI9_SARHA\nPRKCQ\tH0VS11_CAVPO\nPRKCQ\tH0X8H6_OTOGA\nPRKCQ\tH0YUY9_TAEGU\nPRKCQ\tH2MQW5_ORYLA\nPRKCQ\tH2N9P8_PONAB\nPRKCQ\tH2R0T5_PANTR\nPRKCQ\tH2TZU4_TAKRU\nPRKCQ\tH3BA09_LATCH\nPRKCQ\tH9FX48_MACMU\nPRKCQ\tI3KN71_ORENI\nPRKCQ\tI3N2U5_ICTTR\nPRKCQ\tK7FRE0_PELSI\nPRKCQ\tKPCT_HUMAN\nPRKCQ\tKPCT_MOUSE\nPRKCQ\tKPCT_RAT\nPRKCQ\tM3WBI7_FELCA\nPRKCQ\tM3YY49_MUSPF\nPRKCQ\tQ3UK53_MOUSE\nPRKCQ\tQ8CAV6_MOUSE\nPRKCQ\tQ920N8_MOUSE\nPRKCQ\tQ9CU43_MOUSE\nPRKCQ\tU3FER6_CALJA\nPRKCQ\tU3J095_ANAPP\nPRKCQ\tU3JGG1_FICAL\nPRKCQ\tV8N5Y1_OPHHA\nPRKCQ\tV8N6E1_OPHHA\nPRKCQ\tW5PUR2_SHEEP\nPRKCQ\tW5URG8_ICTPU\n','From\tTo\nPRKCA\tA0A096NTX9_PAPAN\nPRKCA\tA0A0D9QVS7_CHLSB\nPRKCA\tA0A0V1FKW9_TRIPS\nPRKCA\tA0A0V1GXC6_9BILA\nPRKCA\tA0A0V1GYA1_9BILA\nPRKCA\tA0A0V1JKX3_TRIPS\nPRKCA\tA0A169TED2_HUMAN\nPRKCA\tA0A1A7X2G4_9TELE\nPRKCA\tA0A1A7XVH2_9TELE\nPRKCA\tA0A1A7Y171_9TELE\nPRKCA\tA0A1A7Y343_9TELE\nPRKCA\tA0A1A7ZSN2_NOTFU\nPRKCA\tA0A1A8A501_NOTFU\nPRKCA\tA0A1A8AQR1_NOTFU\nPRKCA\tA0A1A8C2N1_9TELE\nPRKCA\tA0A1A8CD13_9TELE\nPRKCA\tA0A1A8ELL6_9TELE\nPRKCA\tA0A1A8EW70_9TELE\nPRKCA\tA0A1A8FUH4_9TELE\nPRKCA\tA0A1A8FY69_9TELE\nPRKCA\tA0A1A8GU49_9TELE\nPRKCA\tA0A1A8H4N4_9TELE\nPRKCA\tA0A1A8HYS1_NOTKU\nPRKCA\tA0A1A8J745_NOTKU\nPRKCA\tA0A1A8JEG6_NOTKU\nPRKCA\tA0A1A8JYV9_NOTKU\nPRKCA\tA0A1A8KYE7_9TELE\nPRKCA\tA0A1A8LDC5_9TELE\nPRKCA\tA0A1A8N0J2_9TELE\nPRKCA\tA0A1A8N0P7_9TELE\nPRKCA\tA0A1A8NDH9_9TELE\nPRKCA\tA0A1A8NIA1_9TELE\nPRKCA\tA0A1A8NQ45_9TELE\nPRKCA\tA0A1A8Q707_9TELE\nPRKCA\tA0A1A8RRD6_9TELE\nPRKCA\tA0A1A8V3A3_NOTFU\nPRKCA\tA0A1A8VHT1_NOTFU\nPRKCA\tA0A1D5R985_MACMU\nPRKCA\tA0A1S2ZUZ9_ERIEU\nPRKCA\tA0A1S3GXB8_DIPOR\nPRKCA\tA0A1U7QLD2_MESAU\nPRKCA\tA0A1U7UAG6_TARSY\nPRKCA\tA0A1V4KX14_PATFA\nPRKCA\tA0A218UWZ0_9PASE\nPRKCA\tA0A250Y4C6_CASCN\nPRKCA\tA0A286X8V7_CAVPO\nPRKCA\tA0A2B4S2F0_STYPI\nPRKCA\tA0A2I0M7C5_COLLI\nPRKCA\tA0A2I2UL33_FELCA\nPRKCA\tA0A2I3LZB3_PAPAN\nPRKCA\tA0A2I3NI97_PAPAN\nPRKCA\tA0A2I3THG5_PANTR\nPRKCA\tA0A2I3THM8_PANTR\nPRKCA\tA0A2K5EGS9_AOTNA\nPRKCA\tA0A2K5HEA7_COLAP\nPRKCA\tA0A2K5HEC3_COLAP\nPRKCA\tA0A2K5LMT6_CERAT\nPRKCA\tA0A2K5LMU6_CERAT\nPRKCA\tA0A2K5LMX9_CERAT\nPRKCA\tA0A2K5Q714_CEBCA\nPRKCA\tA0A2K5UKX2_MACFA\nPRKCA\tA0A2K5UKY5_MACFA\nPRKCA\tA0A2K5UKZ1_MACFA\nPRKCA\tA0A2K5YZI3_MANLE\nPRKCA\tA0A2K5YZK3_MANLE\nPRKCA\tA0A2K6CYI9_MACNE\nPRKCA\tA0A2K6CYM9_MACNE\nPRKCA\tA0A2K6CYN4_MACNE\nPRKCA\tA0A2K6L9V9_RHIBE\nPRKCA\tA0A2K6RMH9_RHIRO\nPRKCA\tA0A2K6RMI1_RHIRO\nPRKCA\tA0A2K6V080_SAIBB\nPRKCA\tA0A2R8QIY5_DANRE\nPRKCA\tA0A2R9AIK8_PANPA\nPRKCA\tA0A2R9AQN8_PANPA\nPRKCA\tA0A2S2NAI7_SCHGA\nPRKCA\tA0A2U3X1C0_ODORO\nPRKCA\tA0A2U4BLW3_TURTR\nPRKCA\tA0A2U4BLW4_TURTR\nPRKCA\tA0A2U4BM05_TURTR\nPRKCA\tA0A2U4BM73_TURTR\nPRKCA\tA0A2Y9G7H4_NEOSC\nPRKCA\tA0A2Y9PID1_DELLE\nPRKCA\tA0A2Y9PJU4_DELLE\nPRKCA\tA0A2Y9PRN8_DELLE\nPRKCA\tA0A340X141_LIPVE\nPRKCA\tA0A340X7Y2_LIPVE\nPRKCA\tA0A341CPF7_NEOAA\nPRKCA\tA0A384AE52_BALAS\nPRKCA\tA0A384AE76_BALAS\nPRKCA\tA0A384CBF0_URSMA\nPRKCA\tA0A3Q0CQD1_MESAU\nPRKCA\tA0A3Q0FWU0_ALLSI\nPRKCA\tA0A3Q0FXQ2_ALLSI\nPRKCA\tA0A3Q1MB84_BOVIN\nPRKCA\tA0A3Q1MQ80_BOVIN\nPRKCA\tA0A3Q2HHM8_HORSE\nPRKCA\tA0A3Q2HQ65_HORSE\nPRKCA\tA0A3Q2I8D9_HORSE\nPRKCA\tA0A3Q7QGI9_CALUR\nPRKCA\tA0A3Q7QIS1_CALUR\nPRKCA\tA0A3Q7QIZ0_CALUR\nPRKCA\tA0A3Q7QU69_CALUR\nPRKCA\tA0A3Q7T5K7_VULVU\nPRKCA\tA0A3Q7W506_URSAR\nPRKCA\tA0A3R7CBU7_CLOSI\nPRKCA\tA0A452FXI9_CAPHI\nPRKCA\tA0A452FXR6_CAPHI\nPRKCA\tA0A452FXS0_CAPHI\nPRKCA\tA0A452FYG3_CAPHI\nPRKCA\tA0A452FYL0_CAPHI\nPRKCA\tA0A452H1Z7_9SAUR\nPRKCA\tA0A452H279_9SAUR\nPRKCA\tA0A452H384_9SAUR\nPRKCA\tA0A452H3F1_9SAUR\nPRKCA\tA0A452H4A3_9SAUR\nPRKCA\tA0A452H4P5_9SAUR\nPRKCA\tA0A452QXE0_URSAM\nPRKCA\tA0A452QXK2_URSAM\nPRKCA\tA0A452QXM7_URSAM\nPRKCA\tA0A452QXP5_URSAM\nPRKCA\tA0A452V5A2_URSMA\nPRKCA\tA0A452V5R1_URSMA\nPRKCA\tA0A452V621_URSMA\nPRKCA\tA0A452V661_URSMA\nPRKCA\tA0A452V664_URSMA\nPRKCA\tA0A452V669_URSMA\nPRKCA\tA0A452V694_URSMA\nPRKCA\tA0A452V6B2_URSMA\nPRKCA\tA0A452V6B4_URSMA\nPRKCA\tA0A452V6D1_URSMA\nPRKCA\tA0A493SZP2_ANAPP\nPRKCA\tA0A493T1Q3_ANAPP\nPRKCA\tA0A4W2GEP2_BOBOX\nPRKCA\tA0A4W2GFN9_BOBOX\nPRKCA\tA0A4X1ST13_PIG\nPRKCA\tA0A4X1STB5_PIG\nPRKCA\tA0A4X1SUN8_PIG\nPRKCA\tA0A4X2L349_VOMUR\nPRKCA\tA0A4X2L351_VOMUR\nPRKCA\tA0A4X2L5U5_VOMUR\nPRKCA\tA0A4X2LAC6_VOMUR\nPRKCA\tA0A4X2LB59_VOMUR\nPRKCA\tA0A4X2LBH0_VOMUR\nPRKCA\tA0A4X2LCA4_VOMUR\nPRKCA\tA0A5F5PFD4_HORSE\nPRKCA\tA0A5F5XGK3_FELCA\nPRKCA\tA0A5F5Y2P4_FELCA\nPRKCA\tA0A5F8A004_MACMU\nPRKCA\tA0A5F9DNQ2_RABIT\nPRKCA\tA0JM65_XENTR\nPRKCA\tB0JYP1_BOVIN\nPRKCA\tB0LPH5_HUMAN\nPRKCA\tB5DFC4_RAT\nPRKCA\tD9U9E1_DIDVI\nPRKCA\tD9U9E2_RHYRA\nPRKCA\tD9U9E3_CAEFU\nPRKCA\tD9U9E4_DROGL\nPRKCA\tD9U9E5_ISOOB\nPRKCA\tD9U9E6_TARRO\nPRKCA\tD9U9E7_PSEPE\nPRKCA\tD9U9E8_MACRO\nPRKCA\tD9U9E9_POTTR\nPRKCA\tD9U9F0_VOMUR\nPRKCA\tF1LS98_RAT\nPRKCA\tF1M2P8_RAT\nPRKCA\tF1MJX9_BOVIN\nPRKCA\tF1Q1D5_CANLF\nPRKCA\tF1R444_DANRE\nPRKCA\tF1RV14_PIG\nPRKCA\tF6RA79_MONDO\nPRKCA\tF6W6Q5_HORSE\nPRKCA\tF6YGF6_XENTR\nPRKCA\tF7CUS2_ORNAN\nPRKCA\tF7CUS9_ORNAN\nPRKCA\tF7CUT6_ORNAN\nPRKCA\tF7GSI8_CALJA\nPRKCA\tF7H1C6_MACMU\nPRKCA\tF8K567_PHATA\nPRKCA\tF8K568_DASGE\nPRKCA\tF8K569_SMICR\nPRKCA\tF8K570_MYRFA\nPRKCA\tF8K571_NOTTY\nPRKCA\tF8K572_MACLA\nPRKCA\tF8K573_PERGU\nPRKCA\tG1LZD5_AILME\nPRKCA\tG1N621_MELGA\nPRKCA\tG1NVY6_MYOLU\nPRKCA\tG1QHE4_NOMLE\nPRKCA\tG1U7J6_RABIT\nPRKCA\tG3SWW4_LOXAF\nPRKCA\tG3TUH2_LOXAF\nPRKCA\tG3W6T1_SARHA\nPRKCA\tH0VS40_CAVPO\nPRKCA\tH0X9H1_OTOGA\nPRKCA\tH0Z1Q4_TAEGU\nPRKCA\tH2MFH9_ORYLA\nPRKCA\tH2NUH6_PONAB\nPRKCA\tH2QDQ1_PANTR\nPRKCA\tH3A5S8_LATCH\nPRKCA\tH3A5S9_LATCH\nPRKCA\tH9EMT3_MACMU\nPRKCA\tH9G4Z4_ANOCA\nPRKCA\tI3M1P9_ICTTR\nPRKCA\tI3NGH3_ICTTR\nPRKCA\tJ3KN97_HUMAN\nPRKCA\tJ3KRN5_HUMAN\nPRKCA\tK7GDN9_PELSI\nPRKCA\tKPCA_BOVIN\nPRKCA\tKPCA_HUMAN\nPRKCA\tKPCA_MOUSE\nPRKCA\tKPCA_RABIT\nPRKCA\tKPCA_RAT\nPRKCA\tL7N367_XENTR\nPRKCA\tL7RSM7_HUMAN\nPRKCA\tM3WG61_FELCA\nPRKCA\tM3YT95_MUSPF\nPRKCA\tQ28HN6_XENTR\nPRKCA\tQ3TQ39_MOUSE\nPRKCA\tQ4VA93_MOUSE\nPRKCA\tQ5F3X1_CHICK\nPRKCA\tQ6AXM3_DANRE\nPRKCA\tQ6AZF7_XENLA\nPRKCA\tQ7Z727_HUMAN\nPRKCA\tQ8JFZ9_TAKRU\nPRKCA\tR4GHL2_CHICK\nPRKCA\tT2MGS7_HYDVU\nPRKCA\tU3BPE0_CALJA\nPRKCA\tU3IPH6_ANAPP\nPRKCA\tU3JIA8_FICAL\nPRKCA\tV8NG70_OPHHA\nPRKCA\tV8P3S0_OPHHA\nPRKCA\tW5Q297_SHEEP\nPRKCA\tW5UK87_ICTPU\n','From\tTo\nDMPK\tA0A077K913_PSEUF\nDMPK\tA0A0A0MXK6_PAPAN\nDMPK\tA0A0D9S218_CHLSB\nDMPK\tA0A0P0CMQ8_SALTM\nDMPK\tA0A0P6JDQ9_HETGA\nDMPK\tA0A151P781_ALLMI\nDMPK\tA0A1C9T803_SALET\nDMPK\tA0A1D5R5R7_MACMU\nDMPK\tA0A1L8F3G2_XENLA\nDMPK\tA0A1L8F863_XENLA\nDMPK\tA0A1S3AB90_ERIEU\nDMPK\tA0A1S3GHD7_DIPOR\nDMPK\tA0A1S3GHE2_DIPOR\nDMPK\tA0A1S3GHN7_DIPOR\nDMPK\tA0A1S3GHP2_DIPOR\nDMPK\tA0A1S3GJ20_DIPOR\nDMPK\tA0A1S3GJN1_DIPOR\nDMPK\tA0A1S3WMQ7_ERIEU\nDMPK\tA0A1S3WMS3_ERIEU\nDMPK\tA0A1S6KR68_SALEN\nDMPK\tA0A1U7QUJ9_MESAU\nDMPK\tA0A1U7R5F9_MESAU\nDMPK\tA0A1U8CNA5_MESAU\nDMPK\tA0A1W5YR18_9SPHN\nDMPK\tA0A218MP14_PROMI\nDMPK\tA0A250Y3Q7_CASCN\nDMPK\tA0A286XDC1_CAVPO\nDMPK\tA0A286XLT9_CAVPO\nDMPK\tA0A287A6K5_PIG\nDMPK\tA0A287BHF8_PIG\nDMPK\tA0A2D0SRV8_ICTPU\nDMPK\tA0A2D0SRV9_ICTPU\nDMPK\tA0A2D0SRW1_ICTPU\nDMPK\tA0A2D0SRW3_ICTPU\nDMPK\tA0A2D0SSF8_ICTPU\nDMPK\tA0A2D0SSZ9_ICTPU\nDMPK\tA0A2I2UG81_FELCA\nDMPK\tA0A2I2V1I9_FELCA\nDMPK\tA0A2I3H2R9_NOMLE\nDMPK\tA0A2I3LEV0_PAPAN\nDMPK\tA0A2I3N2L9_PAPAN\nDMPK\tA0A2I3S1L9_PANTR\nDMPK\tA0A2K5C9I8_AOTNA\nDMPK\tA0A2K5C9J1_AOTNA\nDMPK\tA0A2K5C9M6_AOTNA\nDMPK\tA0A2K5C9M9_AOTNA\nDMPK\tA0A2K5KAN8_COLAP\nDMPK\tA0A2K5KAP3_COLAP\nDMPK\tA0A2K5KAQ5_COLAP\nDMPK\tA0A2K5P066_CERAT\nDMPK\tA0A2K5P0A6_CERAT\nDMPK\tA0A2K5P0B3_CERAT\nDMPK\tA0A2K5P0D1_CERAT\nDMPK\tA0A2K5P9H2_CEBCA\nDMPK\tA0A2K5P9H8_CEBCA\nDMPK\tA0A2K5P9J6_CEBCA\nDMPK\tA0A2K5WJU6_MACFA\nDMPK\tA0A2K5WJX6_MACFA\nDMPK\tA0A2K5WK21_MACFA\nDMPK\tA0A2K5ZSI0_MANLE\nDMPK\tA0A2K5ZSK5_MANLE\nDMPK\tA0A2K5ZSP1_MANLE\nDMPK\tA0A2K5ZSQ5_MANLE\nDMPK\tA0A2K6CW92_MACNE\nDMPK\tA0A2K6CWA7_MACNE\nDMPK\tA0A2K6G0B9_PROCO\nDMPK\tA0A2K6G0E6_PROCO\nDMPK\tA0A2K6G0E7_PROCO\nDMPK\tA0A2K6L8B9_RHIBE\nDMPK\tA0A2K6L8C3_RHIBE\nDMPK\tA0A2K6L8H5_RHIBE\nDMPK\tA0A2K6QE74_RHIRO\nDMPK\tA0A2K6QEA0_RHIRO\nDMPK\tA0A2K6QEA6_RHIRO\nDMPK\tA0A2K6QEB3_RHIRO\nDMPK\tA0A2K6SW81_SAIBB\nDMPK\tA0A2K6SW83_SAIBB\nDMPK\tA0A2K6SWB5_SAIBB\nDMPK\tA0A2K6SWD5_SAIBB\nDMPK\tA0A2K6SWG6_SAIBB\nDMPK\tA0A2R9CH60_PANPA\nDMPK\tA0A2R9CH69_PANPA\nDMPK\tA0A2R9CJY6_PANPA\nDMPK\tA0A2R9CR13_PANPA\nDMPK\tA0A2U3VZM7_ODORO\nDMPK\tA0A2U3VZP3_ODORO\nDMPK\tA0A2U3Z3T6_LEPWE\nDMPK\tA0A2U3ZG98_ODORO\nDMPK\tA0A2U3ZGA1_ODORO\nDMPK\tA0A2U3ZH96_ODORO\nDMPK\tA0A2Y9GGY3_NEOSC\nDMPK\tA0A2Y9P1V4_DELLE\nDMPK\tA0A2Y9P729_DELLE\nDMPK\tA0A2Y9P733_DELLE\nDMPK\tA0A2Y9PCN1_DELLE\nDMPK\tA0A2Y9PCN6_DELLE\nDMPK\tA0A2Y9PEJ3_DELLE\nDMPK\tA0A340XAV9_LIPVE\nDMPK\tA0A340XEM9_LIPVE\nDMPK\tA0A341DBT6_NEOAA\nDMPK\tA0A341DCB9_NEOAA\nDMPK\tA0A341DE31_NEOAA\nDMPK\tA0A383YZD2_BALAS\nDMPK\tA0A383YZY2_BALAS\nDMPK\tA0A384BJE0_URSMA\nDMPK\tA0A384BJG2_URSMA\nDMPK\tA0A3Q0CE63_MESAU\nDMPK\tA0A3Q0EAM3_TARSY\nDMPK\tA0A3Q1LQM0_BOVIN\nDMPK\tA0A3Q2H2H3_HORSE\nDMPK\tA0A3Q2LCM0_HORSE\nDMPK\tA0A3Q7MF91_CALUR\nDMPK\tA0A3Q7MFA1_CALUR\nDMPK\tA0A3Q7MFF6_CALUR\nDMPK\tA0A3Q7SZ52_VULVU\nDMPK\tA0A3Q7TIU3_VULVU\nDMPK\tA0A3Q7TP01_URSAR\nDMPK\tA0A3Q7TP06_URSAR\nDMPK\tA0A3Q7TWJ5_VULVU\nDMPK\tA0A3Q7U8P8_VULVU\nDMPK\tA0A3Q7UC54_URSAR\nDMPK\tA0A3Q7UC59_URSAR\nDMPK\tA0A3Q7UNW2_URSAR\nDMPK\tA0A3Q7UNW7_URSAR\nDMPK\tA0A3Q7UQY1_VULVU\nDMPK\tA0A3Q7V4L6_URSAR\nDMPK\tA0A3Q7VIS1_URSAR\nDMPK\tA0A3Q8DK96_MORMO\nDMPK\tA0A3S4MIK1_PSEFL\nDMPK\tA0A452C6W1_BALAS\nDMPK\tA0A452C710_BALAS\nDMPK\tA0A452C7C6_BALAS\nDMPK\tA0A452C7F6_BALAS\nDMPK\tA0A452ESJ9_CAPHI\nDMPK\tA0A452ETW4_CAPHI\nDMPK\tA0A452S946_URSAM\nDMPK\tA0A452S994_URSAM\nDMPK\tA0A452S997_URSAM\nDMPK\tA0A452S9A4_URSAM\nDMPK\tA0A455AQE4_PHYMC\nDMPK\tA0A455AQF0_PHYMC\nDMPK\tA0A455AQM7_PHYMC\nDMPK\tA0A455AQN2_PHYMC\nDMPK\tA0A455ARY4_PHYMC\nDMPK\tA0A455ARY9_PHYMC\nDMPK\tA0A455ASJ5_PHYMC\nDMPK\tA0A455ASK1_PHYMC\nDMPK\tA0A455B4C9_PHYMC\nDMPK\tA0A4W2CH47_BOBOX\nDMPK\tA0A4W2CH98_BOBOX\nDMPK\tA0A4W2GPJ7_BOBOX\nDMPK\tA0A4W4GK25_ELEEL\nDMPK\tA0A4X1W110_PIG\nDMPK\tA0A4X1W1T4_PIG\nDMPK\tA0A4X1W3J6_PIG\nDMPK\tA0A5E7ZCJ7_9ALTE\nDMPK\tA0A5F4CBD1_CANLF\nDMPK\tA0A5F4CCM5_CANLF\nDMPK\tA0A5F4DAF0_CANLF\nDMPK\tA0A5F4DF63_CANLF\nDMPK\tA0A5F4VVS2_CALJA\nDMPK\tA0A5F5PLA9_HORSE\nDMPK\tA0A5F7Z8X2_MACMU\nDMPK\tA0A5F7ZIM5_MACMU\nDMPK\tA2SI52_METPP\nDMPK\tA2SL23_METPP\nDMPK\tC1DMU7_AZOVD\nDMPK\tD3YYG5_MOUSE\nDMPK\tD3ZYV4_RAT\nDMPK\tD6RI32_MOUSE\nDMPK\tDMPK_HUMAN\nDMPK\tDMPK_MOUSE\nDMPK\tDMPK_PSEUF\nDMPK\tE1BAC4_BOVIN\nDMPK\tE2RJF1_CANLF\nDMPK\tE5KR07_HUMAN\nDMPK\tE9Q6J9_MOUSE\nDMPK\tF6YS30_HORSE\nDMPK\tF6Z8J0_MACMU\nDMPK\tF7GBX0_MACMU\nDMPK\tF7GBX5_MACMU\nDMPK\tF7GBY5_MACMU\nDMPK\tF7GBZ8_MACMU\nDMPK\tF7HWP5_CALJA\nDMPK\tF7I625_CALJA\nDMPK\tG1L1Q5_AILME\nDMPK\tG1NVD1_MYOLU\nDMPK\tG1QPB1_NOMLE\nDMPK\tG3T682_LOXAF\nDMPK\tG3TR99_LOXAF\nDMPK\tG3TTA3_LOXAF\nDMPK\tH0VSI8_CAVPO\nDMPK\tH0X960_OTOGA\nDMPK\tH0XVQ7_OTOGA\nDMPK\tH1A0U5_TAEGU\nDMPK\tH2NZ96_PONAB\nDMPK\tH2QGL9_PANTR\nDMPK\tH3AX06_LATCH\nDMPK\tH3AX07_LATCH\nDMPK\tH9FQT3_MACMU\nDMPK\tH9FQT4_MACMU\nDMPK\tH9FQT6_MACMU\nDMPK\tI3L9J1_PIG\nDMPK\tI3LYQ5_ICTTR\nDMPK\tI6L989_HUMAN\nDMPK\tK6Z5E1_9ALTE\nDMPK\tK6ZAY9_9ALTE\nDMPK\tK6ZS07_PANTR\nDMPK\tK7AEP8_PANTR\nDMPK\tK7AIQ9_PANTR\nDMPK\tK7B806_PANTR\nDMPK\tK7BWP1_PANTR\nDMPK\tK7C5J8_PANTR\nDMPK\tK7CQC0_PANTR\nDMPK\tK7EQX1_HUMAN\nDMPK\tM0QXJ9_HUMAN\nDMPK\tM0QZ00_HUMAN\nDMPK\tM0R1F3_HUMAN\nDMPK\tM0R333_HUMAN\nDMPK\tM3W7J3_FELCA\nDMPK\tM3Y2N3_MUSPF\nDMPK\tM9YB29_AZOVI\nDMPK\tQ05CL1_MOUSE\nDMPK\tQ52161_PSEPU\nDMPK\tQ6DI71_MOUSE\nDMPK\tS6ALY0_PSERE\nDMPK\tU3C014_CALJA\nDMPK\tU3CS91_CALJA\nDMPK\tU3D963_CALJA\nDMPK\tU3DG78_CALJA\nDMPK\tU3DP20_CALJA\nDMPK\tU3E2C2_CALJA\nDMPK\tU3EI13_CALJA\nDMPK\tU3ET21_CALJA\nDMPK\tU3EZZ9_CALJA\nDMPK\tU3FSU5_CALJA\nDMPK\tU6DVI7_NEOVI\nDMPK\tW5PKP8_SHEEP\nDMPK\tX5D8Z6_HUMAN\nDMPK\tX5DNJ6_HUMAN\n','From\tTo\nMAP3K7\tA0A096N156_PAPAN\nMAP3K7\tA0A0A9ZE00_LYGHE\nMAP3K7\tA0A0B2UUA9_TOXCA\nMAP3K7\tA0A0B2VK16_TOXCA\nMAP3K7\tA0A0B5CUJ5_PAROL\nMAP3K7\tA0A0D9RXG7_CHLSB\nMAP3K7\tA0A0R4IVJ5_DANRE\nMAP3K7\tA0A0V0Y7Q8_TRIPS\nMAP3K7\tA0A0V1FMU8_TRIPS\nMAP3K7\tA0A0V1IUV4_TRIPS\nMAP3K7\tA0A151MTU0_ALLMI\nMAP3K7\tA0A151MTW3_ALLMI\nMAP3K7\tA0A1A7WR52_9TELE\nMAP3K7\tA0A1A7YG07_9TELE\nMAP3K7\tA0A1A8A7U0_NOTFU\nMAP3K7\tA0A1A8ASZ1_NOTFU\nMAP3K7\tA0A1A8ATD1_NOTFU\nMAP3K7\tA0A1A8B8Q8_NOTFU\nMAP3K7\tA0A1A8BD11_9TELE\nMAP3K7\tA0A1A8DYL6_9TELE\nMAP3K7\tA0A1A8F126_9TELE\nMAP3K7\tA0A1A8GVD1_9TELE\nMAP3K7\tA0A1A8KS67_NOTKU\nMAP3K7\tA0A1A8LAR0_9TELE\nMAP3K7\tA0A1A8LUE3_9TELE\nMAP3K7\tA0A1A8QT56_9TELE\nMAP3K7\tA0A1A8REC1_9TELE\nMAP3K7\tA0A1A8SGC9_9TELE\nMAP3K7\tA0A1D5NUC4_CHICK\nMAP3K7\tA0A1D5PAV9_CHICK\nMAP3K7\tA0A1S3A7U1_ERIEU\nMAP3K7\tA0A1S3F624_DIPOR\nMAP3K7\tA0A1S3F888_DIPOR\nMAP3K7\tA0A1S3S7Y1_SALSA\nMAP3K7\tA0A1S3S7Y3_SALSA\nMAP3K7\tA0A1S3S807_SALSA\nMAP3K7\tA0A1U7T692_TARSY\nMAP3K7\tA0A1U7TCL6_TARSY\nMAP3K7\tA0A1U7TGN7_TARSY\nMAP3K7\tA0A1U7TR65_TARSY\nMAP3K7\tA0A1U8CXA8_ALLSI\nMAP3K7\tA0A1U8D7L4_ALLSI\nMAP3K7\tA0A1V4JM61_PATFA\nMAP3K7\tA0A1V4JM62_PATFA\nMAP3K7\tA0A1V4JMG1_PATFA\nMAP3K7\tA0A1V4JMM1_PATFA\nMAP3K7\tA0A218VC23_9PASE\nMAP3K7\tA0A250YJV3_CASCN\nMAP3K7\tA0A286XTU4_CAVPO\nMAP3K7\tA0A286XXH2_CAVPO\nMAP3K7\tA0A287A5U5_PIG\nMAP3K7\tA0A287D2A0_ICTTR\nMAP3K7\tA0A287DFR1_ICTTR\nMAP3K7\tA0A2B4RIP8_STYPI\nMAP3K7\tA0A2B4RZ17_STYPI\nMAP3K7\tA0A2B4SZD5_STYPI\nMAP3K7\tA0A2D0PVV7_ICTPU\nMAP3K7\tA0A2I0MUQ3_COLLI\nMAP3K7\tA0A2I0MUQ6_COLLI\nMAP3K7\tA0A2I0MUQ8_COLLI\nMAP3K7\tA0A2I0MUQ9_COLLI\nMAP3K7\tA0A2I2V2F2_FELCA\nMAP3K7\tA0A2I3GLG2_NOMLE\nMAP3K7\tA0A2I3H0X8_NOMLE\nMAP3K7\tA0A2I3LXW7_PAPAN\nMAP3K7\tA0A2I3LYE7_PAPAN\nMAP3K7\tA0A2I3SPY9_PANTR\nMAP3K7\tA0A2I4D702_9TELE\nMAP3K7\tA0A2I4D704_9TELE\nMAP3K7\tA0A2I4D720_9TELE\nMAP3K7\tA0A2K5DBU5_AOTNA\nMAP3K7\tA0A2K5DBW4_AOTNA\nMAP3K7\tA0A2K5DBW5_AOTNA\nMAP3K7\tA0A2K5HLB2_COLAP\nMAP3K7\tA0A2K5HLB4_COLAP\nMAP3K7\tA0A2K5HLC8_COLAP\nMAP3K7\tA0A2K5NZJ0_CERAT\nMAP3K7\tA0A2K5NZJ9_CERAT\nMAP3K7\tA0A2K5NZW2_CERAT\nMAP3K7\tA0A2K5QS97_CEBCA\nMAP3K7\tA0A2K5QSB2_CEBCA\nMAP3K7\tA0A2K5QSD0_CEBCA\nMAP3K7\tA0A2K5QSF9_CEBCA\nMAP3K7\tA0A2K5QSJ7_CEBCA\nMAP3K7\tA0A2K5WPW4_MACFA\nMAP3K7\tA0A2K5WQ25_MACFA\nMAP3K7\tA0A2K5WQB4_MACFA\nMAP3K7\tA0A2K5ZSS7_MANLE\nMAP3K7\tA0A2K5ZSU5_MANLE\nMAP3K7\tA0A2K5ZSV0_MANLE\nMAP3K7\tA0A2K6D8Q7_MACNE\nMAP3K7\tA0A2K6D8R5_MACNE\nMAP3K7\tA0A2K6D8S3_MACNE\nMAP3K7\tA0A2K6EWF8_PROCO\nMAP3K7\tA0A2K6EWF9_PROCO\nMAP3K7\tA0A2K6EWI4_PROCO\nMAP3K7\tA0A2K6K1M1_RHIBE\nMAP3K7\tA0A2K6K1P8_RHIBE\nMAP3K7\tA0A2K6K1Q7_RHIBE\nMAP3K7\tA0A2K6PE55_RHIRO\nMAP3K7\tA0A2K6PE85_RHIRO\nMAP3K7\tA0A2K6PE98_RHIRO\nMAP3K7\tA0A2K6PEC2_RHIRO\nMAP3K7\tA0A2K6UPI5_SAIBB\nMAP3K7\tA0A2K6UPK9_SAIBB\nMAP3K7\tA0A2K6UPL8_SAIBB\nMAP3K7\tA0A2K6UPM4_SAIBB\nMAP3K7\tA0A2P8Z4I7_BLAGE\nMAP3K7\tA0A2R8ZW71_PANPA\nMAP3K7\tA0A2R8ZZW1_PANPA\nMAP3K7\tA0A2R8ZZW5_PANPA\nMAP3K7\tA0A2S2PAZ9_SCHGA\nMAP3K7\tA0A2U3W7Z9_ODORO\nMAP3K7\tA0A2U3W803_ODORO\nMAP3K7\tA0A2U3W812_ODORO\nMAP3K7\tA0A2U3W816_ODORO\nMAP3K7\tA0A2U4AVL4_TURTR\nMAP3K7\tA0A2U4AVN0_TURTR\nMAP3K7\tA0A2Y9FLL1_PHYMC\nMAP3K7\tA0A2Y9FM10_PHYMC\nMAP3K7\tA0A2Y9HC88_NEOSC\nMAP3K7\tA0A2Y9HCF3_NEOSC\nMAP3K7\tA0A2Y9LFX5_DELLE\nMAP3K7\tA0A2Y9LI75_DELLE\nMAP3K7\tA0A337SX42_FELCA\nMAP3K7\tA0A340WT62_LIPVE\nMAP3K7\tA0A340WWR6_LIPVE\nMAP3K7\tA0A340WYB6_LIPVE\nMAP3K7\tA0A340WZ20_LIPVE\nMAP3K7\tA0A341ACE3_NEOAA\nMAP3K7\tA0A341AEP2_NEOAA\nMAP3K7\tA0A384B8N5_BALAS\nMAP3K7\tA0A384B918_BALAS\nMAP3K7\tA0A384BTK6_URSMA\nMAP3K7\tA0A384BUJ5_URSMA\nMAP3K7\tA0A3B3H678_ORYLA\nMAP3K7\tA0A3B3I8L9_ORYLA\nMAP3K7\tA0A3B5K8D7_TAKRU\nMAP3K7\tA0A3Q0D412_MESAU\nMAP3K7\tA0A3Q0DYH6_TARSY\nMAP3K7\tA0A3Q0E0Z3_TARSY\nMAP3K7\tA0A3Q0G660_ALLSI\nMAP3K7\tA0A3Q0G665_ALLSI\nMAP3K7\tA0A3Q0G669_ALLSI\nMAP3K7\tA0A3Q0G675_ALLSI\nMAP3K7\tA0A3Q0G6V1_ALLSI\nMAP3K7\tA0A3Q0G9P7_ALLSI\nMAP3K7\tA0A3Q0GAI6_ALLSI\nMAP3K7\tA0A3Q0GAJ3_ALLSI\nMAP3K7\tA0A3Q1LPM7_BOVIN\nMAP3K7\tA0A3Q1NDP1_BOVIN\nMAP3K7\tA0A3Q2H834_HORSE\nMAP3K7\tA0A3Q2HB37_HORSE\nMAP3K7\tA0A3Q2I5D9_HORSE\nMAP3K7\tA0A3Q7NA84_CALUR\nMAP3K7\tA0A3Q7PRG4_CALUR\nMAP3K7\tA0A3Q7SDW9_VULVU\nMAP3K7\tA0A3Q7SPT4_VULVU\nMAP3K7\tA0A3Q7WR87_URSAR\nMAP3K7\tA0A3Q7XLS9_URSAR\nMAP3K7\tA0A419PXM1_CLOSI\nMAP3K7\tA0A452E9E0_CAPHI\nMAP3K7\tA0A452EA85_CAPHI\nMAP3K7\tA0A452INY8_9SAUR\nMAP3K7\tA0A452INZ8_9SAUR\nMAP3K7\tA0A452IP15_9SAUR\nMAP3K7\tA0A452SHC6_URSAM\nMAP3K7\tA0A452SHC8_URSAM\nMAP3K7\tA0A452U7V2_URSMA\nMAP3K7\tA0A452U7Z1_URSMA\nMAP3K7\tA0A452U844_URSMA\nMAP3K7\tA0A452U853_URSMA\nMAP3K7\tA0A452U882_URSMA\nMAP3K7\tA0A452U8A3_URSMA\nMAP3K7\tA0A493TEJ5_ANAPP\nMAP3K7\tA0A493U0K9_ANAPP\nMAP3K7\tA0A4C1VLX4_9NEOP\nMAP3K7\tA0A4C1VND8_9NEOP\nMAP3K7\tA0A4C1VNR2_9NEOP\nMAP3K7\tA0A4W2CF59_BOBOX\nMAP3K7\tA0A4W2E3Y7_BOBOX\nMAP3K7\tA0A4W2E4W0_BOBOX\nMAP3K7\tA0A4W2IL55_BOBOX\nMAP3K7\tA0A4W2IP30_BOBOX\nMAP3K7\tA0A4W2IP35_BOBOX\nMAP3K7\tA0A4W2IP95_BOBOX\nMAP3K7\tA0A4W2IPB2_BOBOX\nMAP3K7\tA0A4W3IZH1_CALMI\nMAP3K7\tA0A4W3J285_CALMI\nMAP3K7\tA0A4W3J946_CALMI\nMAP3K7\tA0A4W3J951_CALMI\nMAP3K7\tA0A4W3JDD8_CALMI\nMAP3K7\tA0A4W3JDE3_CALMI\nMAP3K7\tA0A4W3JW21_CALMI\nMAP3K7\tA0A4W3JW27_CALMI\nMAP3K7\tA0A4W3JW41_CALMI\nMAP3K7\tA0A4X1VDV1_PIG\nMAP3K7\tA0A4X1VIU2_PIG\nMAP3K7\tA0A4X1VIV6_PIG\nMAP3K7\tA0A4X1VKJ4_PIG\nMAP3K7\tA0A4X1VKK7_PIG\nMAP3K7\tA0A4X2KA61_VOMUR\nMAP3K7\tA0A4X2KA78_VOMUR\nMAP3K7\tA0A4X2KHP1_VOMUR\nMAP3K7\tA0A5B7D7G9_PORTR\nMAP3K7\tA0A5F4BV80_CANLF\nMAP3K7\tA0A5F5PSV6_HORSE\nMAP3K7\tA0A5F8G5G8_MONDO\nMAP3K7\tA0A5F8GJ40_MONDO\nMAP3K7\tA0A5F8HJG3_MONDO\nMAP3K7\tA0A5F9CAU4_RABIT\nMAP3K7\tA0A5F9CJ95_RABIT\nMAP3K7\tA0A5F9CWZ0_RABIT\nMAP3K7\tA0A5F9D3N0_RABIT\nMAP3K7\tA0A5G2QY77_PIG\nMAP3K7\tA0A5G3JYR7_XENTR\nMAP3K7\tA0A5G3JZ15_XENTR\nMAP3K7\tA0A5G3K0D8_XENTR\nMAP3K7\tA0A5G3K0S5_XENTR\nMAP3K7\tA0A5G3KKK4_XENTR\nMAP3K7\tA0A5G3KKW5_XENTR\nMAP3K7\tA0A5G3KLS0_XENTR\nMAP3K7\tA0A5G3KLT9_XENTR\nMAP3K7\tA0A5G3KLZ6_XENTR\nMAP3K7\tA0A5G3KM07_XENTR\nMAP3K7\tA0A5G3KM47_XENTR\nMAP3K7\tA0A5G3KMC9_XENTR\nMAP3K7\tA2AP92_MOUSE\nMAP3K7\tA2AP93_MOUSE\nMAP3K7\tA4IIZ7_XENTR\nMAP3K7\tB0LXP4_PIG\nMAP3K7\tB0LXP5_PIG\nMAP3K7\tC0H8V1_SALSA\nMAP3K7\tD2HRF5_AILME\nMAP3K7\tE1C063_CHICK\nMAP3K7\tE2R3R6_CANLF\nMAP3K7\tE2R3R9_CANLF\nMAP3K7\tF6U298_XENTR\nMAP3K7\tF6XAZ7_XENTR\nMAP3K7\tF7BAU3_HORSE\nMAP3K7\tF7BWT1_CALJA\nMAP3K7\tF7CCD5_MONDO\nMAP3K7\tF7ENG8_XENTR\nMAP3K7\tF7GDY4_MACMU\nMAP3K7\tF7H6S1_MACMU\nMAP3K7\tF7H6S6_MACMU\nMAP3K7\tG1KYM9_ANOCA\nMAP3K7\tG1NLK0_MELGA\nMAP3K7\tG1NZX8_MYOLU\nMAP3K7\tG1RSQ0_NOMLE\nMAP3K7\tG1SFY3_RABIT\nMAP3K7\tG3TL77_LOXAF\nMAP3K7\tG3TWR4_LOXAF\nMAP3K7\tG3WZI8_SARHA\nMAP3K7\tG5AL97_HETGA\nMAP3K7\tH0VBW5_CAVPO\nMAP3K7\tH0WTC8_OTOGA\nMAP3K7\tH0ZQ54_TAEGU\nMAP3K7\tH2MUJ3_ORYLA\nMAP3K7\tH2PJU2_PONAB\nMAP3K7\tH2PJU3_PONAB\nMAP3K7\tH2RCY2_PANTR\nMAP3K7\tH2SPG1_TAKRU\nMAP3K7\tH2SPG2_TAKRU\nMAP3K7\tH2SPG3_TAKRU\nMAP3K7\tH2SPG4_TAKRU\nMAP3K7\tH3ANE8_LATCH\nMAP3K7\tH3ANE9_LATCH\nMAP3K7\tI3IZG6_ORENI\nMAP3K7\tI3IZG7_ORENI\nMAP3K7\tI3MEL7_ICTTR\nMAP3K7\tK7C324_PANTR\nMAP3K7\tM3K7_BOVIN\nMAP3K7\tM3K7_HUMAN\nMAP3K7\tM3K7_MOUSE\nMAP3K7\tM3K7_PONAB\nMAP3K7\tM3K7_RAT\nMAP3K7\tM3WSG5_FELCA\nMAP3K7\tM3YCF7_MUSPF\nMAP3K7\tO73613_XENLA\nMAP3K7\tQ2KT22_DANRE\nMAP3K7\tQ3TXG1_MOUSE\nMAP3K7\tQ503G7_DANRE\nMAP3K7\tQ543B5_MOUSE\nMAP3K7\tQ5RIS4_DANRE\nMAP3K7\tQ5RIS5_DANRE\nMAP3K7\tQ801N1_XENLA\nMAP3K7\tQ923A8_MOUSE\nMAP3K7\tQ9UG54_HUMAN\nMAP3K7\tT2MIQ0_HYDVU\nMAP3K7\tU3CBH8_CALJA\nMAP3K7\tU3FA35_CALJA\nMAP3K7\tU3I4I3_ANAPP\nMAP3K7\tU3JAC3_DANRE\nMAP3K7\tU3JAM1_DANRE\nMAP3K7\tU3JKK0_FICAL\nMAP3K7\tV8P9L7_OPHHA\nMAP3K7\tW5PRX8_SHEEP\nMAP3K7\tW5PRX9_SHEEP\n','From\tTo\nAKT1\tA0A034VRF0_BACDO\nAKT1\tA0A059TXP8_GOSHI\nAKT1\tA0A060D9J3_9POAL\nAKT1\tA0A087WY56_HUMAN\nAKT1\tA0A087ZWA0_APIME\nAKT1\tA0A096NK32_PAPAN\nAKT1\tA0A0B4LIA3_DROME\nAKT1\tA0A0D9RBG0_CHLSB\nAKT1\tA0A0G2KG61_DANRE\nAKT1\tA0A0P6JFJ2_HETGA\nAKT1\tA0A0S2Z3D6_HUMAN\nAKT1\tA0A0S7KPL3_9TELE\nAKT1\tA0A0V0SNC6_9BILA\nAKT1\tA0A0V0SNJ8_9BILA\nAKT1\tA0A0V1C2D4_TRISP\nAKT1\tA0A0V1C2D6_TRISP\nAKT1\tA0A0V1C2F7_TRISP\nAKT1\tA0A0V1C341_TRISP\nAKT1\tA0A142IKA9_HUMAN\nAKT1\tA0A143RHU3_FELCA\nAKT1\tA0A1A7X0H5_9TELE\nAKT1\tA0A1A8B4K7_NOTFU\nAKT1\tA0A1A8BPM8_9TELE\nAKT1\tA0A1A8BRJ0_9TELE\nAKT1\tA0A1A8G406_9TELE\nAKT1\tA0A1A8G5V8_9TELE\nAKT1\tA0A1A8HU35_NOTKU\nAKT1\tA0A1A8L050_NOTKU\nAKT1\tA0A1A8L6V6_9TELE\nAKT1\tA0A1A8PN71_9TELE\nAKT1\tA0A1A8PZC4_9TELE\nAKT1\tA0A1A8RZV7_9TELE\nAKT1\tA0A1A8UUA5_NOTFU\nAKT1\tA0A1A8UV42_NOTFU\nAKT1\tA0A1D5RJS3_MACMU\nAKT1\tA0A1J4JE03_9EUKA\nAKT1\tA0A1L8F033_XENLA\nAKT1\tA0A1P8B0E9_ARATH\nAKT1\tA0A1Q1FEG4_LYCRU\nAKT1\tA0A1Q1NIF8_TACFU\nAKT1\tA0A1Q9C0V5_SYMMI\nAKT1\tA0A1Q9C7U7_SYMMI\nAKT1\tA0A1Q9CTF2_SYMMI\nAKT1\tA0A1Q9CXN8_SYMMI\nAKT1\tA0A1Q9D541_SYMMI\nAKT1\tA0A1Q9EDH5_SYMMI\nAKT1\tA0A1Q9ELC5_SYMMI\nAKT1\tA0A1S2ZF54_ERIEU\nAKT1\tA0A1S3EV04_DIPOR\nAKT1\tA0A1S3W5F0_ERIEU\nAKT1\tA0A1S3W5J0_ERIEU\nAKT1\tA0A1U7QIL2_MESAU\nAKT1\tA0A1U7S7Y2_ALLSI\nAKT1\tA0A1U7SR37_ALLSI\nAKT1\tA0A1V4KAM6_PATFA\nAKT1\tA0A1Y1S777_9MICR\nAKT1\tA0A249RNJ6_CAPHI\nAKT1\tA0A250Y1L6_CASCN\nAKT1\tA0A251UCH9_HELAN\nAKT1\tA0A286XKS8_CAVPO\nAKT1\tA0A287AL02_PIG\nAKT1\tA0A287AR39_PIG\nAKT1\tA0A287B2S5_PIG\nAKT1\tA0A287D4L0_ICTTR\nAKT1\tA0A289YPH8_SINCH\nAKT1\tA0A2D0RKM8_ICTPU\nAKT1\tA0A2I0A0I6_9ASPA\nAKT1\tA0A2I0ADZ7_9ASPA\nAKT1\tA0A2I0B3B9_9ASPA\nAKT1\tA0A2I0V7G9_9ASPA\nAKT1\tA0A2I0X8F8_9ASPA\nAKT1\tA0A2I0XB55_9ASPA\nAKT1\tA0A2I0XD07_9ASPA\nAKT1\tA0A2I3G9T3_NOMLE\nAKT1\tA0A2I3GD14_NOMLE\nAKT1\tA0A2I3HRL5_NOMLE\nAKT1\tA0A2I3LEM1_PAPAN\nAKT1\tA0A2I3N0A6_PAPAN\nAKT1\tA0A2I3SQJ8_PANTR\nAKT1\tA0A2I3TCP0_PANTR\nAKT1\tA0A2I3TVS4_PANTR\nAKT1\tA0A2I4BHV0_9TELE\nAKT1\tA0A2J7PG41_9NEOP\nAKT1\tA0A2K5DET4_AOTNA\nAKT1\tA0A2K5IDA1_COLAP\nAKT1\tA0A2K5IDD9_COLAP\nAKT1\tA0A2K5IDG0_COLAP\nAKT1\tA0A2K5L125_CERAT\nAKT1\tA0A2K5L168_CERAT\nAKT1\tA0A2K5L1H2_CERAT\nAKT1\tA0A2K5S2H1_CEBCA\nAKT1\tA0A2K5S2J8_CEBCA\nAKT1\tA0A2K5S2L0_CEBCA\nAKT1\tA0A2K5S2L6_CEBCA\nAKT1\tA0A2K5UY08_MACFA\nAKT1\tA0A2K5UY69_MACFA\nAKT1\tA0A2K5ZJ78_MANLE\nAKT1\tA0A2K5ZJA3_MANLE\nAKT1\tA0A2K6CQN8_MACNE\nAKT1\tA0A2K6CQV7_MACNE\nAKT1\tA0A2K6CQX5_MACNE\nAKT1\tA0A2K6CR17_MACNE\nAKT1\tA0A2K6GRZ0_PROCO\nAKT1\tA0A2K6KHG5_RHIBE\nAKT1\tA0A2K6KHH6_RHIBE\nAKT1\tA0A2K6NCS8_RHIRO\nAKT1\tA0A2K6NCT4_RHIRO\nAKT1\tA0A2K6NCV4_RHIRO\nAKT1\tA0A2K6NCX9_RHIRO\nAKT1\tA0A2K6SGN3_SAIBB\nAKT1\tA0A2K6SGS8_SAIBB\nAKT1\tA0A2K6SGW4_SAIBB\nAKT1\tA0A2K6SGX9_SAIBB\nAKT1\tA0A2P0XJ51_CTEID\nAKT1\tA0A2P0XJ55_PAROL\nAKT1\tA0A2R8P4S0_CALJA\nAKT1\tA0A2R9BZ66_PANPA\nAKT1\tA0A2R9BZT0_PANPA\nAKT1\tA0A2R9BZW8_PANPA\nAKT1\tA0A2R9C2A1_PANPA\nAKT1\tA0A2S2PB94_SCHGA\nAKT1\tA0A2U3VLF0_ODORO\nAKT1\tA0A2U3Y3V3_LEPWE\nAKT1\tA0A2U3ZB38_ODORO\nAKT1\tA0A2U4B7Z9_TURTR\nAKT1\tA0A2Y9GCY3_NEOSC\nAKT1\tA0A2Y9NBQ2_DELLE\nAKT1\tA0A2Y9NFC0_DELLE\nAKT1\tA0A2Y9TD83_PHYMC\nAKT1\tA0A2Y9TF62_PHYMC\nAKT1\tA0A340X250_LIPVE\nAKT1\tA0A340X6L4_LIPVE\nAKT1\tA0A341CGU0_NEOAA\nAKT1\tA0A371E619_MUCPR\nAKT1\tA0A371HN64_MUCPR\nAKT1\tA0A371ICD6_MUCPR\nAKT1\tA0A383ZMJ0_BALAS\nAKT1\tA0A384DC20_URSMA\nAKT1\tA0A3B3H2G6_ORYLA\nAKT1\tA0A3B3I3I2_ORYLA\nAKT1\tA0A3B4EF05_PYGNA\nAKT1\tA0A3B4EGX4_PYGNA\nAKT1\tA0A3P8XGA0_ESOLU\nAKT1\tA0A3Q0DUD4_TARSY\nAKT1\tA0A3Q1MJ77_BOVIN\nAKT1\tA0A3Q2HHJ1_HORSE\nAKT1\tA0A3Q2LC37_HORSE\nAKT1\tA0A3Q2U4X3_CHICK\nAKT1\tA0A3Q7NUI9_CALUR\nAKT1\tA0A3Q7NUR7_CALUR\nAKT1\tA0A3Q7SWZ3_VULVU\nAKT1\tA0A3Q7TFF3_VULVU\nAKT1\tA0A3Q7X157_URSAR\nAKT1\tA0A452FSJ6_CAPHI\nAKT1\tA0A452FSU6_CAPHI\nAKT1\tA0A452FT42_CAPHI\nAKT1\tA0A452QXV1_URSAM\nAKT1\tA0A452QY12_URSAM\nAKT1\tA0A452VKV6_URSMA\nAKT1\tA0A452VKY4_URSMA\nAKT1\tA0A452VL00_URSMA\nAKT1\tA0A452VL05_URSMA\nAKT1\tA0A452VL29_URSMA\nAKT1\tA0A452VL66_URSMA\nAKT1\tA0A452VL71_URSMA\nAKT1\tA0A455BTG1_PHYMC\nAKT1\tA0A4W2FNQ0_BOBOX\nAKT1\tA0A4W2HLB1_BOBOX\nAKT1\tA0A4W2HQL7_BOBOX\nAKT1\tA0A4W3IKQ0_CALMI\nAKT1\tA0A4W3IKS8_CALMI\nAKT1\tA0A4W3INX2_CALMI\nAKT1\tA0A4W3INX6_CALMI\nAKT1\tA0A4W3INY6_CALMI\nAKT1\tA0A4W3IP04_CALMI\nAKT1\tA0A4W3IU27_CALMI\nAKT1\tA0A4W3IU48_CALMI\nAKT1\tA0A4W3JF37_CALMI\nAKT1\tA0A4W4H370_ELEEL\nAKT1\tA0A4W4H3D0_ELEEL\nAKT1\tA0A4W4H3M0_ELEEL\nAKT1\tA0A4W4H7Q6_ELEEL\nAKT1\tA0A4W4HB07_ELEEL\nAKT1\tA0A4W4HBD5_ELEEL\nAKT1\tA0A4W4HBD7_ELEEL\nAKT1\tA0A4W4HBZ7_ELEEL\nAKT1\tA0A4W5QVH5_9TELE\nAKT1\tA0A4W5QZS5_9TELE\nAKT1\tA0A4W5R6L6_9TELE\nAKT1\tA0A4W5RBS1_9TELE\nAKT1\tA0A4W6DR17_LATCA\nAKT1\tA0A4W6DRR8_LATCA\nAKT1\tA0A4W6DT71_LATCA\nAKT1\tA0A4W6DTB2_LATCA\nAKT1\tA0A4W6DTD9_LATCA\nAKT1\tA0A4W6DTI0_LATCA\nAKT1\tA0A4W6DTM4_LATCA\nAKT1\tA0A4W6DTS1_LATCA\nAKT1\tA0A4W6DTX5_LATCA\nAKT1\tA0A4X1TTM8_PIG\nAKT1\tA0A4X1TYJ7_PIG\nAKT1\tA0A4X1TYL9_PIG\nAKT1\tA0A4X1TYM4_PIG\nAKT1\tA0A4X1TZ38_PIG\nAKT1\tA0A4X2KCT4_VOMUR\nAKT1\tA0A4Y2GEU1_ARAVE\nAKT1\tA0A5F4BYE3_CANLF\nAKT1\tA0A5F4VZT4_CALJA\nAKT1\tA0A5F4W0E7_CALJA\nAKT1\tA0A5F4WM82_CALJA\nAKT1\tA0A5F5PSV3_HORSE\nAKT1\tA0A5G3JB37_XENTR\nAKT1\tA4FUQ9_MOUSE\nAKT1\tA5PH36_PHYPA\nAKT1\tAKT1_ALTAL\nAKT1\tAKT1_ARATH\nAKT1\tAKT1_BOVIN\nAKT1\tAKT1_DROME\nAKT1\tAKT1_HUMAN\nAKT1\tAKT1_MOUSE\nAKT1\tAKT1_ORYSI\nAKT1\tAKT1_ORYSJ\nAKT1\tAKT1_RAT\nAKT1\tAKT1_XENLA\nAKT1\tB0LPE5_HUMAN\nAKT1\tC1PIG3_PIG\nAKT1\tD0EM91_9ROSI\nAKT1\tD2Y3E6_9POAL\nAKT1\tD3VYT7_PAPAN\nAKT1\tD3YXX3_MOUSE\nAKT1\tD3YYP9_MOUSE\nAKT1\tD3Z783_MOUSE\nAKT1\tD9IA52_CAPHI\nAKT1\tE0XKS5_9POAL\nAKT1\tE2RJX4_CANLF\nAKT1\tF1NMS0_CHICK\nAKT1\tF6QP11_MONDO\nAKT1\tF6RCU6_MACMU\nAKT1\tF6SAS5_MACMU\nAKT1\tF6WGH2_HORSE\nAKT1\tF6XP81_XENTR\nAKT1\tF7AMP8_HORSE\nAKT1\tF7DAX8_XENTR\nAKT1\tF7HW94_CALJA\nAKT1\tG1KLB6_ANOCA\nAKT1\tG1LMQ4_AILME\nAKT1\tG1NL48_MELGA\nAKT1\tG1NL49_MELGA\nAKT1\tG1S2V0_NOMLE\nAKT1\tG3U9P4_LOXAF\nAKT1\tG3V2I6_HUMAN\nAKT1\tG3V3X1_HUMAN\nAKT1\tG3V4I6_HUMAN\nAKT1\tG3W4L9_SARHA\nAKT1\tG3X7B5_BOVIN\nAKT1\tG7MWI7_MACMU\nAKT1\tG7PBQ9_MACFA\nAKT1\tG7YAG1_CLOSI\nAKT1\tG9BWQ1_PIG\nAKT1\tH0V394_CAVPO\nAKT1\tH0XPQ9_OTOGA\nAKT1\tH0ZRP6_TAEGU\nAKT1\tH1ZY63_DROME\nAKT1\tH1ZY64_DROME\nAKT1\tH2MRC2_ORYLA\nAKT1\tH2NME5_PONAB\nAKT1\tH2Q903_PANTR\nAKT1\tH2UBP8_TAKRU\nAKT1\tH2UBQ0_TAKRU\nAKT1\tH3A5T1_LATCH\nAKT1\tH3A5T2_LATCH\nAKT1\tI3ML41_ICTTR\nAKT1\tJ7M8E6_PROTE\nAKT1\tK7AGW5_PANTR\nAKT1\tK7BG24_PANTR\nAKT1\tK7C9N2_PANTR\nAKT1\tK7D8F6_PANTR\nAKT1\tK7FKB4_PELSI\nAKT1\tK7GNW4_PIG\nAKT1\tK7RTM0_9ROSI\nAKT1\tK7RY58_EUCCA\nAKT1\tK7RY62_PROJU\nAKT1\tK7S1W9_9ROSI\nAKT1\tK7SA28_PROJU\nAKT1\tK7SA33_9FABA\nAKT1\tK7SQ94_CASEQ\nAKT1\tK7SQA0_9ROSI\nAKT1\tM3XST6_MUSPF\nAKT1\tM4MD44_DANRE\nAKT1\tO57513_CHICK\nAKT1\tQ06XL6_HORVU\nAKT1\tQ64164_MOUSE\nAKT1\tQ8BS26_MOUSE\nAKT1\tT1WF18_9AVES\nAKT1\tU3C443_CALJA\nAKT1\tU3JJZ0_FICAL\nAKT1\tV5H036_ANOGL\nAKT1\tW5KPH4_ASTMX\nAKT1\tW5UQZ3_ICTPU\nAKT1\tW8BSF8_CERCA\nAKT1\tX2CUM2_MOUSE\nAKT1\tX2CV47_HUMAN\nAKT1\tX2CVF3_HUMAN\n','From\tTo\nILK\tA0A034VUS5_BACDO\nILK\tA0A087WW45_HUMAN\nILK\tA0A087WWY6_HUMAN\nILK\tA0A087XN61_POEFO\nILK\tA0A096NE53_PAPAN\nILK\tA0A0A0MTH3_HUMAN\nILK\tA0A0A1X3W0_ZEUCU\nILK\tA0A0A9Z7V8_LYGHE\nILK\tA0A0B2UUJ9_TOXCA\nILK\tA0A0D9QWF6_CHLSB\nILK\tA0A0E3DBV7_SPAAU\nILK\tA0A0K8WLI1_BACLA\nILK\tA0A0S7KHE0_9TELE\nILK\tA0A0S7KHE5_9TELE\nILK\tA0A0S7KHF1_9TELE\nILK\tA0A0S7KHF4_9TELE\nILK\tA0A0S7KHY5_9TELE\nILK\tA0A0S7KSC6_9TELE\nILK\tA0A0S7KSC9_9TELE\nILK\tA0A0S7KSD1_9TELE\nILK\tA0A0S7KSE1_9TELE\nILK\tA0A0V0RU22_9BILA\nILK\tA0A0V0TNY4_9BILA\nILK\tA0A0V0WKC6_9BILA\nILK\tA0A0V0YK45_TRIPS\nILK\tA0A0V0ZW05_9BILA\nILK\tA0A0V1AP26_TRISP\nILK\tA0A0V1B954_TRISP\nILK\tA0A0V1B987_TRISP\nILK\tA0A0V1CTG5_TRIBR\nILK\tA0A0V1I3E0_9BILA\nILK\tA0A0V1I3L2_9BILA\nILK\tA0A0V1I4V3_9BILA\nILK\tA0A0V1I4Z1_9BILA\nILK\tA0A0V1JB64_TRIPS\nILK\tA0A0V1LHP9_9BILA\nILK\tA0A0V1MTK0_9BILA\nILK\tA0A0V1PDR0_9BILA\nILK\tA0A151MYS5_ALLMI\nILK\tA0A1A7Z4M7_9TELE\nILK\tA0A1A8BTG6_9TELE\nILK\tA0A1A8DCF8_9TELE\nILK\tA0A1A8DRL3_9TELE\nILK\tA0A1A8FVU7_9TELE\nILK\tA0A1A8HWS6_NOTKU\nILK\tA0A1A8L1N0_9TELE\nILK\tA0A1A8LIK3_9TELE\nILK\tA0A1A8NIG1_9TELE\nILK\tA0A1A8PX80_9TELE\nILK\tA0A1A8RWF6_9TELE\nILK\tA0A1A8U7N3_NOTFU\nILK\tA0A1B0GR42_MOUSE\nILK\tA0A1B0GRF6_MOUSE\nILK\tA0A1B0GRW6_MOUSE\nILK\tA0A1D5R325_MACMU\nILK\tA0A1D5R4A2_MACMU\nILK\tA0A1D5R971_MACMU\nILK\tA0A1Q9EDP4_SYMMI\nILK\tA0A1S3A1B5_ERIEU\nILK\tA0A1S3A1C2_ERIEU\nILK\tA0A1S3A1D7_ERIEU\nILK\tA0A1S3GQC2_DIPOR\nILK\tA0A1S3GQI9_DIPOR\nILK\tA0A1U7RF83_MESAU\nILK\tA0A1U7SKB6_ALLSI\nILK\tA0A1U7T178_TARSY\nILK\tA0A1U7TBT2_TARSY\nILK\tA0A1U7TJC3_TARSY\nILK\tA0A1V4JK99_PATFA\nILK\tA0A218UHF3_9PASE\nILK\tA0A250YIA2_CASCN\nILK\tA0A287A8C3_PIG\nILK\tA0A287B6J1_PIG\nILK\tA0A2B4SVQ3_STYPI\nILK\tA0A2I0LHP8_COLLI\nILK\tA0A2I3FXC6_NOMLE\nILK\tA0A2I3HF70_NOMLE\nILK\tA0A2I3HM00_NOMLE\nILK\tA0A2I3MMG8_PAPAN\nILK\tA0A2I3N4G5_PAPAN\nILK\tA0A2I3RP73_PANTR\nILK\tA0A2I3SCZ1_PANTR\nILK\tA0A2I3T709_PANTR\nILK\tA0A2I4AT85_9TELE\nILK\tA0A2J8P6G0_PANTR\nILK\tA0A2K5BWB9_AOTNA\nILK\tA0A2K5BWC2_AOTNA\nILK\tA0A2K5BWE7_AOTNA\nILK\tA0A2K5IZE6_COLAP\nILK\tA0A2K5IZF2_COLAP\nILK\tA0A2K5IZW9_COLAP\nILK\tA0A2K5IZX3_COLAP\nILK\tA0A2K5NJA2_CERAT\nILK\tA0A2K5NJA8_CERAT\nILK\tA0A2K5NJI4_CERAT\nILK\tA0A2K5NJM8_CERAT\nILK\tA0A2K5NJV9_CERAT\nILK\tA0A2K5RE57_CEBCA\nILK\tA0A2K5RE73_CEBCA\nILK\tA0A2K5RE91_CEBCA\nILK\tA0A2K5REA0_CEBCA\nILK\tA0A2K5REA2_CEBCA\nILK\tA0A2K5WGG1_MACFA\nILK\tA0A2K5WGJ4_MACFA\nILK\tA0A2K5WGK5_MACFA\nILK\tA0A2K5WGL5_MACFA\nILK\tA0A2K5YLU9_MANLE\nILK\tA0A2K5YLV0_MANLE\nILK\tA0A2K5YLX2_MANLE\nILK\tA0A2K5YLX3_MANLE\nILK\tA0A2K6BHG9_MACNE\nILK\tA0A2K6BHH6_MACNE\nILK\tA0A2K6BHL0_MACNE\nILK\tA0A2K6BHL6_MACNE\nILK\tA0A2K6FLC2_PROCO\nILK\tA0A2K6FLC3_PROCO\nILK\tA0A2K6FLE0_PROCO\nILK\tA0A2K6FLE2_PROCO\nILK\tA0A2K6FLE7_PROCO\nILK\tA0A2K6LAY1_RHIBE\nILK\tA0A2K6LAY7_RHIBE\nILK\tA0A2K6LB10_RHIBE\nILK\tA0A2K6LB17_RHIBE\nILK\tA0A2K6NUG8_RHIRO\nILK\tA0A2K6NUG9_RHIRO\nILK\tA0A2K6NUH0_RHIRO\nILK\tA0A2K6NUH5_RHIRO\nILK\tA0A2K6NUH9_RHIRO\nILK\tA0A2K6TPF0_SAIBB\nILK\tA0A2K6TPG2_SAIBB\nILK\tA0A2K6TPH1_SAIBB\nILK\tA0A2K6TPH6_SAIBB\nILK\tA0A2P8YW21_BLAGE\nILK\tA0A2R9A7L1_PANPA\nILK\tA0A2R9A7L6_PANPA\nILK\tA0A2R9ACD3_PANPA\nILK\tA0A2R9ACR6_PANPA\nILK\tA0A2S2P792_SCHGA\nILK\tA0A2U3WV68_ODORO\nILK\tA0A2U3WV88_ODORO\nILK\tA0A2U3XMU7_LEPWE\nILK\tA0A2U3ZUW3_ODORO\nILK\tA0A2U4CDH8_TURTR\nILK\tA0A2Y9EJT9_PHYMC\nILK\tA0A2Y9GQ16_NEOSC\nILK\tA0A2Y9GQC5_NEOSC\nILK\tA0A2Y9MIP5_DELLE\nILK\tA0A340XEM4_LIPVE\nILK\tA0A340XIG2_LIPVE\nILK\tA0A341CDW5_NEOAA\nILK\tA0A341CE53_NEOAA\nILK\tA0A383ZCX5_BALAS\nILK\tA0A384DQY6_URSMA\nILK\tA0A3B3CZ99_ORYME\nILK\tA0A3B3QRV8_9TELE\nILK\tA0A3B3WZH2_9TELE\nILK\tA0A3B3YH05_9TELE\nILK\tA0A3B3ZX33_9GOBI\nILK\tA0A3B4ZMA7_9TELE\nILK\tA0A3B4ZMD8_9TELE\nILK\tA0A3B5LWH9_9TELE\nILK\tA0A3P8NGZ4_ASTCA\nILK\tA0A3P9BPR5_9CICH\nILK\tA0A3P9KQ95_ORYLA\nILK\tA0A3P9NS83_POERE\nILK\tA0A3Q0DUN6_TARSY\nILK\tA0A3Q0SW38_AMPCI\nILK\tA0A3Q0SW48_AMPCI\nILK\tA0A3Q1AV13_AMPOC\nILK\tA0A3Q1G635_9TELE\nILK\tA0A3Q1JX93_ANATE\nILK\tA0A3Q2FTC2_CYPVA\nILK\tA0A3Q2H733_HORSE\nILK\tA0A3Q2QVN5_FUNHE\nILK\tA0A3Q2W2G1_HAPBU\nILK\tA0A3Q3A8K5_KRYMA\nILK\tA0A3Q3EBY6_9LABR\nILK\tA0A3Q3K433_MONAL\nILK\tA0A3Q3LM37_9TELE\nILK\tA0A3Q3WYJ1_MOLML\nILK\tA0A3Q4BKH4_MOLML\nILK\tA0A3Q4I9A9_NEOBR\nILK\tA0A3Q7QWN8_CALUR\nILK\tA0A3Q7RAD1_VULVU\nILK\tA0A3Q7S8N0_CALUR\nILK\tA0A3Q7VGC7_URSAR\nILK\tA0A3R7CT15_CLOSI\nILK\tA0A452F0A3_CAPHI\nILK\tA0A452SMF7_URSAM\nILK\tA0A452SMI9_URSAM\nILK\tA0A480Q0Y5_PIG\nILK\tA0A493STL3_ANAPP\nILK\tA0A493TA26_ANAPP\nILK\tA0A4W2CIF7_BOBOX\nILK\tA0A4W5Q966_9TELE\nILK\tA0A4W5Q9W0_9TELE\nILK\tA0A4W6DQG2_LATCA\nILK\tA0A4W6DS73_LATCA\nILK\tA0A4W6DS78_LATCA\nILK\tA0A4X1SGZ5_PIG\nILK\tA0A4X1SH98_PIG\nILK\tA0A4X2K7M6_VOMUR\nILK\tA0A4X2KA90_VOMUR\nILK\tA0A5F5XK22_FELCA\nILK\tA0A5F5XNV9_FELCA\nILK\tA0A5F8AAX1_MACMU\nILK\tA0A5F8GJ56_MONDO\nILK\tA0A5F9DJG1_RABIT\nILK\tA0A5G3I3C7_XENTR\nILK\tA0A5G3I872_XENTR\nILK\tA0A5G3I8F3_XENTR\nILK\tA0A5G3I8I7_XENTR\nILK\tA0A5G3I8K5_XENTR\nILK\tA0A5G3I8P2_XENTR\nILK\tA0A5G3I9E9_XENTR\nILK\tA0A5G3IA28_XENTR\nILK\tB1H3J0_XENTR\nILK\tB5X1Y0_SALSA\nILK\tC0SSX9_CRIGR\nILK\tD2HR59_AILME\nILK\tD3PJS1_LEPSM\nILK\tD3YZA5_MOUSE\nILK\tE2R5H4_CANLF\nILK\tE3CTS5_DROME\nILK\tE9PQ52_HUMAN\nILK\tF6Q5Z1_MOUSE\nILK\tF6VX96_CALJA\nILK\tF6WGW3_CALJA\nILK\tF6XMH2_XENTR\nILK\tF7BVZ3_XENTR\nILK\tF7BZE6_HORSE\nILK\tF7C1M9_XENTR\nILK\tF7C2W0_MACMU\nILK\tF7DKL4_CALJA\nILK\tF7FRN8_MONDO\nILK\tG1PCN1_MYOLU\nILK\tG1S6P2_NOMLE\nILK\tG1SRA9_RABIT\nILK\tG3PBP9_GASAC\nILK\tG3TD22_LOXAF\nILK\tG3TSU4_LOXAF\nILK\tG3W679_SARHA\nILK\tG5BBQ5_HETGA\nILK\tG7PQV1_MACFA\nILK\tH0VLM2_CAVPO\nILK\tH0WVT1_OTOGA\nILK\tH0YQQ2_TAEGU\nILK\tH2LGW0_ORYLA\nILK\tH2NE91_PONAB\nILK\tH2Q325_PANTR\nILK\tH3AAX1_LATCH\nILK\tH9CH14_SHEEP\nILK\tH9GD59_ANOCA\nILK\tH9H1Z1_MELGA\nILK\tI3KJF6_ORENI\nILK\tI3L9C8_PIG\nILK\tI3MHP1_ICTTR\nILK\tILK_BOVIN\nILK\tILK_CAVPO\nILK\tILK_HUMAN\nILK\tILK_MOUSE\nILK\tILK_PONAB\nILK\tILK_RAT\nILK\tK7CTC8_PANTR\nILK\tK7FLA0_PELSI\nILK\tM3WMI0_FELCA\nILK\tM3Y2A3_MUSPF\nILK\tM3ZS50_XIPMA\nILK\tQ2PQN4_GLOMM\nILK\tQ3YAQ5_MACMU\nILK\tQ6AZQ2_XENLA\nILK\tQ6PHD6_DANRE\nILK\tQ9DF58_CHICK\nILK\tQ9NHC7_DROME\nILK\tQ9V400_DROME\nILK\tT2ME09_HYDVU\nILK\tU3IPV9_ANAPP\nILK\tU3K0I9_FICAL\nILK\tW5Q731_SHEEP\nILK\tW5UCU6_ICTPU\nILK\tW8BI94_CERCA\n','From\tTo\nACVRL1\tA0A096MQ05_PAPAN\nACVRL1\tA0A0A9YET5_LYGHE\nACVRL1\tA0A0D9R0K0_CHLSB\nACVRL1\tA0A0P6J6V4_HETGA\nACVRL1\tA0A0S2Z2Y4_HUMAN\nACVRL1\tA0A0S2Z310_HUMAN\nACVRL1\tA0A1A7YDS1_9TELE\nACVRL1\tA0A1A8ED61_9TELE\nACVRL1\tA0A1A8FGV4_9TELE\nACVRL1\tA0A1A8KGE2_NOTKU\nACVRL1\tA0A1A8LTC9_9TELE\nACVRL1\tA0A1A8NJX6_9TELE\nACVRL1\tA0A1A8P2R9_9TELE\nACVRL1\tA0A1A8V3F5_NOTFU\nACVRL1\tA0A1D5P3Z7_CHICK\nACVRL1\tA0A1L8HHN2_XENLA\nACVRL1\tA0A1S3FEB3_DIPOR\nACVRL1\tA0A1S3W596_ERIEU\nACVRL1\tA0A1U7U269_TARSY\nACVRL1\tA0A1U8BT72_MESAU\nACVRL1\tA0A1V4KZK7_PATFA\nACVRL1\tA0A218U9I8_9PASE\nACVRL1\tA0A250YJ31_CASCN\nACVRL1\tA0A287ACF0_PIG\nACVRL1\tA0A2H8TYP6_9HEMI\nACVRL1\tA0A2I0LPB0_COLLI\nACVRL1\tA0A2I3G8H9_NOMLE\nACVRL1\tA0A2I3MNJ2_PAPAN\nACVRL1\tA0A2I3MQC1_PAPAN\nACVRL1\tA0A2I3RJJ6_PANTR\nACVRL1\tA0A2K5DAB6_AOTNA\nACVRL1\tA0A2K5DAF7_AOTNA\nACVRL1\tA0A2K5I1I3_COLAP\nACVRL1\tA0A2K5I1N4_COLAP\nACVRL1\tA0A2K5N641_CERAT\nACVRL1\tA0A2K5N674_CERAT\nACVRL1\tA0A2K5N6A4_CERAT\nACVRL1\tA0A2K5RI74_CEBCA\nACVRL1\tA0A2K5RI87_CEBCA\nACVRL1\tA0A2K5WXU2_MACFA\nACVRL1\tA0A2K5WXV5_MACFA\nACVRL1\tA0A2K5ZZK2_MANLE\nACVRL1\tA0A2K5ZZK7_MANLE\nACVRL1\tA0A2K6D1X8_MACNE\nACVRL1\tA0A2K6D1Y7_MACNE\nACVRL1\tA0A2K6D1Z8_MACNE\nACVRL1\tA0A2K6G9A0_PROCO\nACVRL1\tA0A2K6G9F3_PROCO\nACVRL1\tA0A2K6KEW5_RHIBE\nACVRL1\tA0A2K6KEW9_RHIBE\nACVRL1\tA0A2K6PKT4_RHIRO\nACVRL1\tA0A2K6PKW1_RHIRO\nACVRL1\tA0A2K6UFQ0_SAIBB\nACVRL1\tA0A2K6UFS6_SAIBB\nACVRL1\tA0A2R9B9B4_PANPA\nACVRL1\tA0A2R9BCM1_PANPA\nACVRL1\tA0A2U3VZ86_ODORO\nACVRL1\tA0A2U3VZA0_ODORO\nACVRL1\tA0A2U3YIM9_LEPWE\nACVRL1\tA0A2U3ZG62_ODORO\nACVRL1\tA0A2Y9I940_NEOSC\nACVRL1\tA0A2Y9MZC5_DELLE\nACVRL1\tA0A2Y9NEZ5_DELLE\nACVRL1\tA0A2Y9SJY3_PHYMC\nACVRL1\tA0A2Y9SMQ1_PHYMC\nACVRL1\tA0A340WSA6_LIPVE\nACVRL1\tA0A340WTX0_LIPVE\nACVRL1\tA0A341AQI7_NEOAA\nACVRL1\tA0A341ASG8_NEOAA\nACVRL1\tA0A341ATD5_NEOAA\nACVRL1\tA0A383ZWE4_BALAS\nACVRL1\tA0A383ZWF1_BALAS\nACVRL1\tA0A384D6Y6_URSMA\nACVRL1\tA0A3Q0CKG9_MESAU\nACVRL1\tA0A3Q0E8N2_TARSY\nACVRL1\tA0A3Q0H7X9_ALLSI\nACVRL1\tA0A3Q1N0G5_BOVIN\nACVRL1\tA0A3Q1NCX6_BOVIN\nACVRL1\tA0A3Q2HH33_HORSE\nACVRL1\tA0A3Q2U1G8_CHICK\nACVRL1\tA0A3Q7PY60_CALUR\nACVRL1\tA0A3Q7PYT4_CALUR\nACVRL1\tA0A3Q7S942_VULVU\nACVRL1\tA0A3Q7V8X4_URSAR\nACVRL1\tA0A3Q7W8C7_URSAR\nACVRL1\tA0A452GAD6_CAPHI\nACVRL1\tA0A452RF79_URSAM\nACVRL1\tA0A452RFD9_URSAM\nACVRL1\tA0A455BDS8_PHYMC\nACVRL1\tA0A4W2CYR7_BOBOX\nACVRL1\tA0A4W2ENM8_BOBOX\nACVRL1\tA0A4W2G6L6_BOBOX\nACVRL1\tA0A4W2HZP7_BOBOX\nACVRL1\tA0A4X1W6F7_PIG\nACVRL1\tA0A4X1WBZ3_PIG\nACVRL1\tA0A4X1WDB9_PIG\nACVRL1\tA0A4X2KLG4_VOMUR\nACVRL1\tA0A4Z2IX54_9TELE\nACVRL1\tA0A5F4C0F0_CANLF\nACVRL1\tA0A5F5PUB3_HORSE\nACVRL1\tA0A5F5XQH6_FELCA\nACVRL1\tA0A5F9CLZ3_RABIT\nACVRL1\tA0A5G3I0Y3_XENTR\nACVRL1\tA0A5G3I0Y9_XENTR\nACVRL1\tA0A5G3I270_XENTR\nACVRL1\tA0A5G3JUD1_XENTR\nACVRL1\tA0A5G3JUT6_XENTR\nACVRL1\tA0A5K1VQB4_CALJA\nACVRL1\tA4FUX2_BOVIN\nACVRL1\tACVL1_HUMAN\nACVRL1\tACVL1_MOUSE\nACVRL1\tACVL1_PONAB\nACVRL1\tACVL1_RAT\nACVRL1\tB0I548_FELCA\nACVRL1\tD3YUY5_MOUSE\nACVRL1\tD3Z7H9_MOUSE\nACVRL1\tD9IPD9_HUMAN\nACVRL1\tE2R174_CANLF\nACVRL1\tE7EN07_HUMAN\nACVRL1\tF1MIW7_BOVIN\nACVRL1\tF1QH44_DANRE\nACVRL1\tF1SGK0_PIG\nACVRL1\tF6PLZ9_CALJA\nACVRL1\tF6PRD4_XENTR\nACVRL1\tF6QH24_XENTR\nACVRL1\tF6QHD9_XENTR\nACVRL1\tF6QI09_XENTR\nACVRL1\tF6S9C0_ORNAN\nACVRL1\tF6S9E3_ORNAN\nACVRL1\tF6YCL6_HORSE\nACVRL1\tF7HG73_MACMU\nACVRL1\tF8W0N2_HUMAN\nACVRL1\tG1KT96_ANOCA\nACVRL1\tG1L4X5_AILME\nACVRL1\tG1NVK2_MYOLU\nACVRL1\tG1S7T4_NOMLE\nACVRL1\tG1TG64_RABIT\nACVRL1\tG3T112_LOXAF\nACVRL1\tG3V1W8_HUMAN\nACVRL1\tG3WCX6_SARHA\nACVRL1\tH0UWQ3_CAVPO\nACVRL1\tH0WP57_OTOGA\nACVRL1\tH0YYJ6_TAEGU\nACVRL1\tH2NHC1_PONAB\nACVRL1\tH2Q5Y5_PANTR\nACVRL1\tH3BIN3_LATCH\nACVRL1\tH3BTZ2_HUMAN\nACVRL1\tH9F5D2_MACMU\nACVRL1\tH9H1J4_MELGA\nACVRL1\tI3LWQ1_ICTTR\nACVRL1\tK9IVI7_PIG\nACVRL1\tM3VUE2_FELCA\nACVRL1\tM3Y8Z4_MUSPF\nACVRL1\tQ498X0_DANRE\nACVRL1\tQ8JFB0_DANRE\nACVRL1\tQ91YV1_MOUSE\nACVRL1\tQ9CU19_MOUSE\nACVRL1\tR9ZUM5_SHEEP\nACVRL1\tU3FJW4_CALJA\nACVRL1\tU3JBE5_FICAL\nACVRL1\tW5Q713_SHEEP\n','From\tTo\nACVR1\tA0A034WVZ2_BACDO\nACVR1\tA0A096NSK5_PAPAN\nACVR1\tA0A096NXW5_PAPAN\nACVR1\tA0A0A1XQV2_ZEUCU\nACVR1\tA0A0D9RTR7_CHLSB\nACVR1\tA0A0G2K2P7_RAT\nACVR1\tA0A0S2Z2Y3_HUMAN\nACVR1\tA0A0S2Z345_HUMAN\nACVR1\tA0A0S7I345_9TELE\nACVR1\tA0A0S7JKA9_9TELE\nACVR1\tA0A0S7JKB4_9TELE\nACVR1\tA0A0S7JLP3_9TELE\nACVR1\tA0A0S7JMK1_9TELE\nACVR1\tA0A0S7KU78_9TELE\nACVR1\tA0A0S7KU83_9TELE\nACVR1\tA0A0S7KU87_9TELE\nACVR1\tA0A0S7KUA5_9TELE\nACVR1\tA0A0S7KUX5_9TELE\nACVR1\tA0A0S7KV44_9TELE\nACVR1\tA0A0S7KV47_9TELE\nACVR1\tA0A0S7KW09_9TELE\nACVR1\tA0A0X3NWF6_SCHSO\nACVR1\tA0A0X3P6S4_SCHSO\nACVR1\tA0A0X3PJC8_SCHSO\nACVR1\tA0A0X3PMW0_SCHSO\nACVR1\tA0A151MRU0_ALLMI\nACVR1\tA0A191VKG0_SHEEP\nACVR1\tA0A1A7WE97_9TELE\nACVR1\tA0A1A8AL72_NOTFU\nACVR1\tA0A1A8C9E2_9TELE\nACVR1\tA0A1A8FZQ3_9TELE\nACVR1\tA0A1A8KHL0_NOTKU\nACVR1\tA0A1A8QHK0_9TELE\nACVR1\tA0A1A8S1B2_9TELE\nACVR1\tA0A1B0GXA9_HUMAN\nACVR1\tA0A1D5P2D8_CHICK\nACVR1\tA0A1S2ZTK9_ERIEU\nACVR1\tA0A1S3GAS0_DIPOR\nACVR1\tA0A1S3GB35_DIPOR\nACVR1\tA0A1S3GCE6_DIPOR\nACVR1\tA0A1S3MX33_SALSA\nACVR1\tA0A1S3NWA6_SALSA\nACVR1\tA0A1S3NWN8_SALSA\nACVR1\tA0A1U7T2P7_TARSY\nACVR1\tA0A1V4K599_PATFA\nACVR1\tA0A218UDX4_9PASE\nACVR1\tA0A250Y237_CASCN\nACVR1\tA0A287B7B4_PIG\nACVR1\tA0A287BHU6_PIG\nACVR1\tA0A2D0R5W8_ICTPU\nACVR1\tA0A2D0R7D5_ICTPU\nACVR1\tA0A2I0MGC8_COLLI\nACVR1\tA0A2I3LHR3_PAPAN\nACVR1\tA0A2I3MM12_PAPAN\nACVR1\tA0A2I3RBB9_PANTR\nACVR1\tA0A2I3SFK9_PANTR\nACVR1\tA0A2I3T727_PANTR\nACVR1\tA0A2J8SUK4_PONAB\nACVR1\tA0A2K5F2H7_AOTNA\nACVR1\tA0A2K5F2K6_AOTNA\nACVR1\tA0A2K5I8W2_COLAP\nACVR1\tA0A2K5I8X3_COLAP\nACVR1\tA0A2K5NWC3_CERAT\nACVR1\tA0A2K5NWQ4_CERAT\nACVR1\tA0A2K5PVD6_CEBCA\nACVR1\tA0A2K5PVL1_CEBCA\nACVR1\tA0A2K5W014_MACFA\nACVR1\tA0A2K5ZEU5_MANLE\nACVR1\tA0A2K5ZEV2_MANLE\nACVR1\tA0A2K5ZF33_MANLE\nACVR1\tA0A2K6CVH7_MACNE\nACVR1\tA0A2K6CVP0_MACNE\nACVR1\tA0A2K6CVP4_MACNE\nACVR1\tA0A2K6CVU2_MACNE\nACVR1\tA0A2K6FS39_PROCO\nACVR1\tA0A2K6FS78_PROCO\nACVR1\tA0A2K6FS84_PROCO\nACVR1\tA0A2K6K749_RHIBE\nACVR1\tA0A2K6K7D4_RHIBE\nACVR1\tA0A2K6K7D9_RHIBE\nACVR1\tA0A2K6K7G0_RHIBE\nACVR1\tA0A2K6K7G5_RHIBE\nACVR1\tA0A2K6NQB5_RHIRO\nACVR1\tA0A2K6NQC5_RHIRO\nACVR1\tA0A2K6NQC7_RHIRO\nACVR1\tA0A2K6NQC8_RHIRO\nACVR1\tA0A2K6NQD8_RHIRO\nACVR1\tA0A2K6TC13_SAIBB\nACVR1\tA0A2K6TC40_SAIBB\nACVR1\tA0A2P8Z3I3_BLAGE\nACVR1\tA0A2R9BMX0_PANPA\nACVR1\tA0A2R9BMY4_PANPA\nACVR1\tA0A2S2QJX9_9HEMI\nACVR1\tA0A2U3V458_TURTR\nACVR1\tA0A2U3VJ27_ODORO\nACVR1\tA0A2U3XU21_LEPWE\nACVR1\tA0A2U4ACP0_TURTR\nACVR1\tA0A2Y9F110_PHYMC\nACVR1\tA0A2Y9I804_NEOSC\nACVR1\tA0A2Y9MSV9_DELLE\nACVR1\tA0A340WHJ8_LIPVE\nACVR1\tA0A341APG0_NEOAA\nACVR1\tA0A384A747_BALAS\nACVR1\tA0A384BST5_URSMA\nACVR1\tA0A3B1JW46_ASTMX\nACVR1\tA0A3B3REH5_9TELE\nACVR1\tA0A3B3RHB4_9TELE\nACVR1\tA0A3B4CDX4_PYGNA\nACVR1\tA0A3B4CH20_PYGNA\nACVR1\tA0A3B4CI27_PYGNA\nACVR1\tA0A3B4EYG0_9CICH\nACVR1\tA0A3P8YR47_ESOLU\nACVR1\tA0A3P9ABY9_ESOLU\nACVR1\tA0A3Q0CQC8_MESAU\nACVR1\tA0A3Q0DYH3_TARSY\nACVR1\tA0A3Q0G7S0_ALLSI\nACVR1\tA0A3Q0G7S5_ALLSI\nACVR1\tA0A3Q0G7V2_ALLSI\nACVR1\tA0A3Q0G8E1_ALLSI\nACVR1\tA0A3Q0GC30_ALLSI\nACVR1\tA0A3Q0GC34_ALLSI\nACVR1\tA0A3Q0QSD9_AMPCI\nACVR1\tA0A3Q2UPW5_FUNHE\nACVR1\tA0A3Q2YK25_HIPCM\nACVR1\tA0A3Q3DJJ0_HIPCM\nACVR1\tA0A3Q7MXF5_CALUR\nACVR1\tA0A3Q7RX17_VULVU\nACVR1\tA0A3Q7WDX2_URSAR\nACVR1\tA0A3R7G326_CLOSI\nACVR1\tA0A419PNJ1_CLOSI\nACVR1\tA0A452FPI0_CAPHI\nACVR1\tA0A452FPM7_CAPHI\nACVR1\tA0A452FPW1_CAPHI\nACVR1\tA0A452GKM7_9SAUR\nACVR1\tA0A452GL34_9SAUR\nACVR1\tA0A452J852_CHICK\nACVR1\tA0A452QG16_URSAM\nACVR1\tA0A455ACW0_PHYMC\nACVR1\tA0A493T534_ANAPP\nACVR1\tA0A493TCT9_ANAPP\nACVR1\tA0A4C1Y3W6_9NEOP\nACVR1\tA0A4C2ABW6_9NEOP\nACVR1\tA0A4W2DH41_BOBOX\nACVR1\tA0A4W2DWJ0_BOBOX\nACVR1\tA0A4W3HTP4_CALMI\nACVR1\tA0A4W3HWI4_CALMI\nACVR1\tA0A4W3HXM8_CALMI\nACVR1\tA0A4W3HXP4_CALMI\nACVR1\tA0A4W4H3Z0_ELEEL\nACVR1\tA0A4W6FPS0_LATCA\nACVR1\tA0A4X1VK14_PIG\nACVR1\tA0A4X1VP50_PIG\nACVR1\tA0A4Y2KGK5_ARAVE\nACVR1\tA0A5B7EIL5_PORTR\nACVR1\tA0A5B7EJ64_PORTR\nACVR1\tA0A5F4C195_CANLF\nACVR1\tA0A5F8GAB8_MONDO\nACVR1\tA0A5F8H7K7_MONDO\nACVR1\tA0A5G3JUL2_XENTR\nACVR1\tA0A5G3JUV3_XENTR\nACVR1\tA2VDM5_BOVIN\nACVR1\tACVR1_BOVIN\nACVR1\tACVR1_CHICK\nACVR1\tACVR1_HUMAN\nACVR1\tACVR1_MOUSE\nACVR1\tACVR1_RAT\nACVR1\tB1AW87_MOUSE\nACVR1\tB2L3W6_XENBO\nACVR1\tB2L5U2_9PIPI\nACVR1\tB3DM96_RAT\nACVR1\tB5X2Z1_SALSA\nACVR1\tC0HAE6_SALSA\nACVR1\tC7C1K1_PIG\nACVR1\tC9J1R3_HUMAN\nACVR1\tC9JHJ7_HUMAN\nACVR1\tC9JW28_HUMAN\nACVR1\tD3DPA4_HUMAN\nACVR1\tD3DPA5_HUMAN\nACVR1\tE2RKJ3_CANLF\nACVR1\tE9M2M1_PIG\nACVR1\tF5GY91_HUMAN\nACVR1\tF6QRK4_XENTR\nACVR1\tF6RIX5_BOVIN\nACVR1\tF6SCC5_MONDO\nACVR1\tF6SV95_XENTR\nACVR1\tF6VZX5_ORNAN\nACVR1\tF6YDA0_XENTR\nACVR1\tF7A9J8_MACMU\nACVR1\tF7B4G7_HORSE\nACVR1\tF7BQK6_CALJA\nACVR1\tF7EAM4_XENTR\nACVR1\tF7EEC7_XENTR\nACVR1\tG1M6Y2_AILME\nACVR1\tG1NJR2_MELGA\nACVR1\tG1PFA1_MYOLU\nACVR1\tG1QJA7_NOMLE\nACVR1\tG1SHK1_RABIT\nACVR1\tG3SZ04_LOXAF\nACVR1\tG3TUY1_LOXAF\nACVR1\tG3WYP9_SARHA\nACVR1\tG5B9D6_HETGA\nACVR1\tG7PKM2_MACFA\nACVR1\tH0VD50_CAVPO\nACVR1\tH0X5A0_OTOGA\nACVR1\tH0ZPM2_TAEGU\nACVR1\tH2MQA0_ORYLA\nACVR1\tH2MQA1_ORYLA\nACVR1\tH2QIU9_PANTR\nACVR1\tH2SXS3_TAKRU\nACVR1\tH2SXS5_TAKRU\nACVR1\tH3ANV2_LATCH\nACVR1\tH3ANV3_LATCH\nACVR1\tH9GCW2_ANOCA\nACVR1\tI3JQW7_ORENI\nACVR1\tI3JQW8_ORENI\nACVR1\tI3M4Q3_ICTTR\nACVR1\tK7FRC6_PELSI\nACVR1\tK7FRD7_PELSI\nACVR1\tK9IW76_PIG\nACVR1\tL8E7H6_HUMAN\nACVR1\tM3W8V8_FELCA\nACVR1\tM3XN09_MUSPF\nACVR1\tO42475_XENLA\nACVR1\tO73736_DANRE\nACVR1\tQ28E06_XENTR\nACVR1\tQ53SF4_HUMAN\nACVR1\tQ53SV1_HUMAN\nACVR1\tQ6DD64_XENLA\nACVR1\tQ8C7A7_MOUSE\nACVR1\tQ91432_XENLA\nACVR1\tQ9DGI6_DANRE\nACVR1\tT2MEL6_HYDVU\nACVR1\tU3IAS2_ANAPP\nACVR1\tU3JKB5_FICAL\nACVR1\tW5KU61_ASTMX\nACVR1\tW5MFC0_LEPOC\nACVR1\tW5PCY4_SHEEP\nACVR1\tW5PCY7_SHEEP\nACVR1\tW5U5S1_ICTPU\nACVR1\tW8C5V5_CERCA\nACVR1\tW8NXQ5_BUBBU\nACVR1\tW8VSE9_LACAG\n','From\tTo\nZAP70\tA0A087WQ05_MOUSE\nZAP70\tA0A096NGT8_PAPAN\nZAP70\tA0A0D9RXH6_CHLSB\nZAP70\tA0A0R4J8U1_RAT\nZAP70\tA0A0S7IRS3_9TELE\nZAP70\tA0A0S7MJV0_9TELE\nZAP70\tA0A1S3FGZ1_DIPOR\nZAP70\tA0A1V4KPZ2_PATFA\nZAP70\tA0A218UCB5_9PASE\nZAP70\tA0A250YFF5_CASCN\nZAP70\tA0A287CT83_ICTTR\nZAP70\tA0A2I3LEH3_PAPAN\nZAP70\tA0A2I4B505_9TELE\nZAP70\tA0A2I4B514_9TELE\nZAP70\tA0A2K5D0V8_AOTNA\nZAP70\tA0A2K5KA32_COLAP\nZAP70\tA0A2K5KA34_COLAP\nZAP70\tA0A2K5MYN1_CERAT\nZAP70\tA0A2K5MYN6_CERAT\nZAP70\tA0A2K5R1S1_CEBCA\nZAP70\tA0A2K5WKU2_MACFA\nZAP70\tA0A2K5WL69_MACFA\nZAP70\tA0A2K5X9R8_MANLE\nZAP70\tA0A2K5X9S6_MANLE\nZAP70\tA0A2K6CVN0_MACNE\nZAP70\tA0A2K6CVS8_MACNE\nZAP70\tA0A2K6GLH5_PROCO\nZAP70\tA0A2K6LTW9_RHIBE\nZAP70\tA0A2K6LTX0_RHIBE\nZAP70\tA0A2K6R1B7_RHIRO\nZAP70\tA0A2K6R1D7_RHIRO\nZAP70\tA0A2K6V1Q3_SAIBB\nZAP70\tA0A2K6V1R8_SAIBB\nZAP70\tA0A2R8ZAJ7_PANPA\nZAP70\tA0A2R8ZE55_PANPA\nZAP70\tA0A2U3VXL8_ODORO\nZAP70\tA0A2U3XR38_LEPWE\nZAP70\tA0A340YCX1_LIPVE\nZAP70\tA0A384CT52_URSMA\nZAP70\tA0A3B3X465_9TELE\nZAP70\tA0A3B3X487_9TELE\nZAP70\tA0A452EBG2_CAPHI\nZAP70\tA0A452EBV8_CAPHI\nZAP70\tA0A452U589_URSMA\nZAP70\tA0A4W2EMH8_BOBOX\nZAP70\tA0A4W3JEQ1_CALMI\nZAP70\tA0A4X2M3T2_VOMUR\nZAP70\tA0A4Z2EAR6_9TELE\nZAP70\tA0A5F4BYT5_CANLF\nZAP70\tA0A5F8GIN4_MONDO\nZAP70\tA0A5G2R169_PIG\nZAP70\tA0A5G3J838_XENTR\nZAP70\tA0A5K1V762_PIG\nZAP70\tE1BMP9_BOVIN\nZAP70\tE1BU42_CHICK\nZAP70\tE5E7T6_HIPHI\nZAP70\tE5E7T7_HIPHI\nZAP70\tF1PJS1_CANLF\nZAP70\tF1STD6_PIG\nZAP70\tF6SWY7_CALJA\nZAP70\tF7AR49_HORSE\nZAP70\tF7CSQ1_MONDO\nZAP70\tF7DLV9_XENTR\nZAP70\tF7FAI7_MACMU\nZAP70\tG1LF11_AILME\nZAP70\tG1MSR6_MELGA\nZAP70\tG1PNQ8_MYOLU\nZAP70\tG1QLX3_NOMLE\nZAP70\tG1SPQ8_RABIT\nZAP70\tG3U2F0_LOXAF\nZAP70\tG3UMI3_LOXAF\nZAP70\tG3VN82_SARHA\nZAP70\tH0V400_CAVPO\nZAP70\tH0XG88_OTOGA\nZAP70\tH0Y261_OTOGA\nZAP70\tH0YPY8_TAEGU\nZAP70\tH2P560_PONAB\nZAP70\tH2QIE3_PANTR\nZAP70\tH2SQL7_TAKRU\nZAP70\tH3AG62_LATCH\nZAP70\tI3MGH2_ICTTR\nZAP70\tK7FB55_PELSI\nZAP70\tM3WD79_FELCA\nZAP70\tM3XJU1_LATCH\nZAP70\tM3YIR3_MUSPF\nZAP70\tM9MM92_DANRE\nZAP70\tM9MMB1_DANRE\nZAP70\tQ503P3_DANRE\nZAP70\tQ6DCV6_XENLA\nZAP70\tQ6DF54_XENTR\nZAP70\tT2MFB2_HYDVU\nZAP70\tU3I3S8_ANAPP\nZAP70\tU3KHA8_FICAL\nZAP70\tU6DHR0_NEOVI\nZAP70\tV9KJG4_CALMI\nZAP70\tW5PW03_SHEEP\nZAP70\tW5ULB9_ICTPU\nZAP70\tZAP70_HUMAN\nZAP70\tZAP70_MOUSE\n','From\tTo\nTYK2\tA0A024R7E4_HUMAN\nTYK2\tA0A0A0MTV7_PAPAN\nTYK2\tA0A0A0MUA2_PAPAN\nTYK2\tA0A0B7FZA5_THACB\nTYK2\tA0A0D9R4G8_CHLSB\nTYK2\tA0A0S7F8H5_9TELE\nTYK2\tA0A0S7F8I9_9TELE\nTYK2\tA0A0S7F8K4_9TELE\nTYK2\tA0A0S7F9J4_9TELE\nTYK2\tA0A0S7F9Y9_9TELE\nTYK2\tA0A0S7FBA3_9TELE\nTYK2\tA0A0S7FDF1_9TELE\nTYK2\tA0A0S7FED9_9TELE\nTYK2\tA0A0S7FEE9_9TELE\nTYK2\tA0A0S7FFV6_9TELE\nTYK2\tA0A0S7FFZ7_9TELE\nTYK2\tA0A0S7FGT4_9TELE\nTYK2\tA0A0S7FI79_9TELE\nTYK2\tA0A0U2PQ07_TACFU\nTYK2\tA0A140DJC5_CTEID\nTYK2\tA0A1A7WT21_9TELE\nTYK2\tA0A1A7XAK9_9TELE\nTYK2\tA0A1A7ZPI2_NOTFU\nTYK2\tA0A1A8AX32_NOTFU\nTYK2\tA0A1A8BIF0_9TELE\nTYK2\tA0A1A8CZS2_9TELE\nTYK2\tA0A1A8EAK0_9TELE\nTYK2\tA0A1A8EXF5_9TELE\nTYK2\tA0A1A8FBR0_9TELE\nTYK2\tA0A1A8FXH6_9TELE\nTYK2\tA0A1A8I365_NOTKU\nTYK2\tA0A1A8J3N6_NOTKU\nTYK2\tA0A1A8LDC4_9TELE\nTYK2\tA0A1A8MPN4_9TELE\nTYK2\tA0A1A8P199_9TELE\nTYK2\tA0A1A8PDD1_9TELE\nTYK2\tA0A1A8RN50_9TELE\nTYK2\tA0A1A8VFR2_NOTFU\nTYK2\tA0A1D5PK82_CHICK\nTYK2\tA0A1L1SVI6_MOUSE\nTYK2\tA0A1L8H2U7_XENLA\nTYK2\tA0A1S3GKJ0_DIPOR\nTYK2\tA0A1S3WF45_ERIEU\nTYK2\tA0A1U7R3C7_MESAU\nTYK2\tA0A1U7S0K0_ALLSI\nTYK2\tA0A1U8D4W6_ALLSI\nTYK2\tA0A1U8DCB6_ALLSI\nTYK2\tA0A1V4KXU7_PATFA\nTYK2\tA0A1V4KY01_PATFA\nTYK2\tA0A1Z1B8L7_9GOBI\nTYK2\tA0A218U785_9PASE\nTYK2\tA0A218U7A5_9PASE\nTYK2\tA0A250YMD8_CASCN\nTYK2\tA0A286ZVG5_PIG\nTYK2\tA0A287ASU9_PIG\nTYK2\tA0A2D0PZ34_ICTPU\nTYK2\tA0A2I0LJ08_COLLI\nTYK2\tA0A2I3MPX7_PAPAN\nTYK2\tA0A2I3RSB7_PANTR\nTYK2\tA0A2I4CIS6_9TELE\nTYK2\tA0A2I4CIS9_9TELE\nTYK2\tA0A2K5F005_AOTNA\nTYK2\tA0A2K5F006_AOTNA\nTYK2\tA0A2K5F071_AOTNA\nTYK2\tA0A2K5F086_AOTNA\nTYK2\tA0A2K5ITF8_COLAP\nTYK2\tA0A2K5IU17_COLAP\nTYK2\tA0A2K5KJN0_CERAT\nTYK2\tA0A2K5KJN2_CERAT\nTYK2\tA0A2K5Q760_CEBCA\nTYK2\tA0A2K5Q799_CEBCA\nTYK2\tA0A2K5VPQ0_MACFA\nTYK2\tA0A2K5VPR0_MACFA\nTYK2\tA0A2K5XZL8_MANLE\nTYK2\tA0A2K5XZM0_MANLE\nTYK2\tA0A2K5XZN6_MANLE\nTYK2\tA0A2K6D4H6_MACNE\nTYK2\tA0A2K6EKT2_PROCO\nTYK2\tA0A2K6LIB7_RHIBE\nTYK2\tA0A2K6LIE0_RHIBE\nTYK2\tA0A2K6LIG0_RHIBE\nTYK2\tA0A2K6P919_RHIRO\nTYK2\tA0A2K6P929_RHIRO\nTYK2\tA0A2K6T829_SAIBB\nTYK2\tA0A2R8MWV0_CALJA\nTYK2\tA0A2R8QJ02_DANRE\nTYK2\tA0A2R9BS87_PANPA\nTYK2\tA0A2R9C1D6_PANPA\nTYK2\tA0A2U3VU74_ODORO\nTYK2\tA0A2U3VU82_ODORO\nTYK2\tA0A2U3YSI1_LEPWE\nTYK2\tA0A2U4B5Y4_TURTR\nTYK2\tA0A2Y9I9Q1_NEOSC\nTYK2\tA0A2Y9IAA7_NEOSC\nTYK2\tA0A2Y9IE55_NEOSC\nTYK2\tA0A2Y9IGV8_NEOSC\nTYK2\tA0A2Y9LU92_DELLE\nTYK2\tA0A2Y9M105_DELLE\nTYK2\tA0A2Y9TGK4_PHYMC\nTYK2\tA0A2Y9TJT6_PHYMC\nTYK2\tA0A340XN66_LIPVE\nTYK2\tA0A341BFK7_NEOAA\nTYK2\tA0A341BFL7_NEOAA\nTYK2\tA0A341BFN8_NEOAA\nTYK2\tA0A341BHA0_NEOAA\nTYK2\tA0A341BII3_NEOAA\nTYK2\tA0A341BIN6_NEOAA\nTYK2\tA0A383Z2F5_BALAS\nTYK2\tA0A384DH94_URSMA\nTYK2\tA0A3Q0DNG1_TARSY\nTYK2\tA0A3Q0GJJ7_ALLSI\nTYK2\tA0A3Q7NEC0_CALUR\nTYK2\tA0A3Q7R5E6_VULVU\nTYK2\tA0A3Q7R7F2_VULVU\nTYK2\tA0A3Q7R7F6_VULVU\nTYK2\tA0A3Q7SHZ4_VULVU\nTYK2\tA0A3Q7SHZ9_VULVU\nTYK2\tA0A3Q7UA82_URSAR\nTYK2\tA0A452F668_CAPHI\nTYK2\tA0A452HKY2_9SAUR\nTYK2\tA0A452HKY4_9SAUR\nTYK2\tA0A452S1S4_URSAM\nTYK2\tA0A452S216_URSAM\nTYK2\tA0A452S2E2_URSAM\nTYK2\tA0A452S2G3_URSAM\nTYK2\tA0A452S2R5_URSAM\nTYK2\tA0A452S2S7_URSAM\nTYK2\tA0A452S2S9_URSAM\nTYK2\tA0A452S2T0_URSAM\nTYK2\tA0A452VM22_URSMA\nTYK2\tA0A452VMA6_URSMA\nTYK2\tA0A452VMF2_URSMA\nTYK2\tA0A4W2DI55_BOBOX\nTYK2\tA0A4W4ECE2_ELEEL\nTYK2\tA0A4W4ED95_ELEEL\nTYK2\tA0A4W4EDS4_ELEEL\nTYK2\tA0A4W4EE62_ELEEL\nTYK2\tA0A4W4EEC1_ELEEL\nTYK2\tA0A4W4EFR3_ELEEL\nTYK2\tA0A4W4EG50_ELEEL\nTYK2\tA0A4W4EGP8_ELEEL\nTYK2\tA0A4W6BT82_LATCA\nTYK2\tA0A4W6BTH0_LATCA\nTYK2\tA0A4W6BTP8_LATCA\nTYK2\tA0A4W6BU27_LATCA\nTYK2\tA0A4W6BYG7_LATCA\nTYK2\tA0A4W6BYM9_LATCA\nTYK2\tA0A4W6BZX2_LATCA\nTYK2\tA0A4W6C0R2_LATCA\nTYK2\tA0A4W6C123_LATCA\nTYK2\tA0A4W6C1L4_LATCA\nTYK2\tA0A4X1V4Y7_PIG\nTYK2\tA0A4X1V868_PIG\nTYK2\tA0A4X1VAJ2_PIG\nTYK2\tA0A4Y5RAG1_ANAPL\nTYK2\tA0A4Z2HWR3_9TELE\nTYK2\tA0A5F4BSS7_CANLF\nTYK2\tA0A5F4BUS1_CANLF\nTYK2\tA0A5F4C2M3_CANLF\nTYK2\tA0A5F4C7C6_CANLF\nTYK2\tA0A5F4CCQ3_CANLF\nTYK2\tA0A5F4D515_CANLF\nTYK2\tA0A5F4DAD4_CANLF\nTYK2\tA0A5F4DBF0_CANLF\nTYK2\tA0A5F7ZDF4_MACMU\nTYK2\tA0A5F7ZPU9_MACMU\nTYK2\tA0A5F9CC30_RABIT\nTYK2\tA0A5F9CQX7_RABIT\nTYK2\tA0A5F9D5Z1_RABIT\nTYK2\tA0A5F9DNU8_RABIT\nTYK2\tA0A5G3HR20_XENTR\nTYK2\tA0A5G3HR45_XENTR\nTYK2\tA0A5G3HR96_XENTR\nTYK2\tA0A5G3HRZ7_XENTR\nTYK2\tA0A5G3HUM1_XENTR\nTYK2\tA0A5G3HUM6_XENTR\nTYK2\tA0A5G3HV32_XENTR\nTYK2\tA0A5G3HV92_XENTR\nTYK2\tA0A5G3JEZ2_XENTR\nTYK2\tA0A5G3JF65_XENTR\nTYK2\tA0A5G3JGR4_XENTR\nTYK2\tA0A5G3JHY5_XENTR\nTYK2\tA0A5G3JK70_XENTR\nTYK2\tA0A5G3JKD3_XENTR\nTYK2\tA0A5G3JM52_XENTR\nTYK2\tD3ZD03_RAT\nTYK2\tE9PM19_HUMAN\nTYK2\tE9PPF2_HUMAN\nTYK2\tE9PQE9_HUMAN\nTYK2\tE9PQL2_HUMAN\nTYK2\tE9QJS1_MOUSE\nTYK2\tF1MCX4_BOVIN\nTYK2\tF1PBD0_CANLF\nTYK2\tF6PPA5_XENTR\nTYK2\tF6PS10_XENTR\nTYK2\tF6QBA9_CALJA\nTYK2\tF6RW04_XENTR\nTYK2\tF6TJE9_HORSE\nTYK2\tF6V3I2_MONDO\nTYK2\tF7A948_XENTR\nTYK2\tF7CMB7_XENTR\nTYK2\tF7DRD4_MACMU\nTYK2\tF8W4H3_DANRE\nTYK2\tG1KHZ7_ANOCA\nTYK2\tG1LYZ3_AILME\nTYK2\tG1MVV3_MELGA\nTYK2\tG1PKC8_MYOLU\nTYK2\tG1RNJ0_NOMLE\nTYK2\tG1T864_RABIT\nTYK2\tG3STB4_LOXAF\nTYK2\tG3W042_SARHA\nTYK2\tH0V2Z6_CAVPO\nTYK2\tH0X425_OTOGA\nTYK2\tH0YCT7_HUMAN\nTYK2\tH0YE24_HUMAN\nTYK2\tH0YE41_HUMAN\nTYK2\tH2LLN7_ORYLA\nTYK2\tH2R6D4_PANTR\nTYK2\tH2TE39_TAKRU\nTYK2\tH2TE41_TAKRU\nTYK2\tH9Z2G6_MACMU\nTYK2\tI3M210_ICTTR\nTYK2\tK7AU67_PANTR\nTYK2\tK7B315_PANTR\nTYK2\tK7EJR6_HUMAN\nTYK2\tK7EM33_HUMAN\nTYK2\tM3WBF6_FELCA\nTYK2\tM3Y5N7_MUSPF\nTYK2\tQ2M4G5_MOUSE\nTYK2\tQ3TDN4_MOUSE\nTYK2\tQ3TIS9_MOUSE\nTYK2\tQ3U447_MOUSE\nTYK2\tQ684M7_PIG\nTYK2\tQ6GPK5_XENLA\nTYK2\tQ6JDV3_PIG\nTYK2\tQ90961_CHICK\nTYK2\tQ9N143_MACMU\nTYK2\tQ9PWD1_TETFL\nTYK2\tTYK2_HUMAN\nTYK2\tTYK2_MOUSE\nTYK2\tU3BEB2_CALJA\nTYK2\tU3BXC0_CALJA\nTYK2\tU3E506_CALJA\nTYK2\tU3F2R3_CALJA\nTYK2\tU3FFB2_CALJA\nTYK2\tV8NXT0_OPHHA\nTYK2\tW5Q3B2_SHEEP\n','From\tTo\nSYK\tA0A024R244_HUMAN\nSYK\tA0A024R273_HUMAN\nSYK\tA0A034W6E5_BACDO\nSYK\tA0A034W7K0_BACDO\nSYK\tA0A096P223_PAPAN\nSYK\tA0A0A0MQ53_CHICK\nSYK\tA0A0C5DLZ7_LETCA\nSYK\tA0A0D9R4K5_CHLSB\nSYK\tA0A0S7KA98_9TELE\nSYK\tA0A0S7KAA1_9TELE\nSYK\tA0A0S7KAA4_9TELE\nSYK\tA0A0S7KAN5_9TELE\nSYK\tA0A0S7KB33_9TELE\nSYK\tA0A0S7KBD2_9TELE\nSYK\tA0A0S7KBN6_9TELE\nSYK\tA0A0S7KC19_9TELE\nSYK\tA0A0S7KD85_9TELE\nSYK\tA0A151NWA6_ALLMI\nSYK\tA0A1A7X9N9_9TELE\nSYK\tA0A1A7XDQ8_9TELE\nSYK\tA0A1A7Y908_9TELE\nSYK\tA0A1A8A6M2_NOTFU\nSYK\tA0A1A8ALV2_NOTFU\nSYK\tA0A1A8CFL3_9TELE\nSYK\tA0A1A8DB04_9TELE\nSYK\tA0A1A8FA33_9TELE\nSYK\tA0A1A8IYZ0_NOTKU\nSYK\tA0A1A8JMM1_NOTKU\nSYK\tA0A1A8NLA5_9TELE\nSYK\tA0A1A8P2J4_9TELE\nSYK\tA0A1A8PYL7_9TELE\nSYK\tA0A1A8S4E3_9TELE\nSYK\tA0A1A8U6M3_NOTFU\nSYK\tA0A1L8HYI7_XENLA\nSYK\tA0A1S3AT02_ERIEU\nSYK\tA0A1S3FZQ3_DIPOR\nSYK\tA0A1S3G1C5_DIPOR\nSYK\tA0A1S3MBN4_SALSA\nSYK\tA0A1S3MBN9_SALSA\nSYK\tA0A1S3MBP8_SALSA\nSYK\tA0A1U7TBF8_TARSY\nSYK\tA0A1U7TLV7_TARSY\nSYK\tA0A1U8BEZ1_MESAU\nSYK\tA0A1U8BR23_MESAU\nSYK\tA0A1V4J5M0_PATFA\nSYK\tA0A218UQF1_9PASE\nSYK\tA0A250YKH5_CASCN\nSYK\tA0A251TTT3_HELAN\nSYK\tA0A286XZD6_CAVPO\nSYK\tA0A286ZZ61_PIG\nSYK\tA0A2B4SFM1_STYPI\nSYK\tA0A2D0SR71_ICTPU\nSYK\tA0A2D0SR82_ICTPU\nSYK\tA0A2D0SRT1_ICTPU\nSYK\tA0A2D0SSC0_ICTPU\nSYK\tA0A2I0M6D0_COLLI\nSYK\tA0A2I2UZ97_FELCA\nSYK\tA0A2I3GFV0_NOMLE\nSYK\tA0A2I3MM62_PAPAN\nSYK\tA0A2I3N2Y6_PAPAN\nSYK\tA0A2I4BS48_9TELE\nSYK\tA0A2J8KTF1_PANTR\nSYK\tA0A2J8KTF6_PANTR\nSYK\tA0A2K5CNH7_AOTNA\nSYK\tA0A2K5CNI6_AOTNA\nSYK\tA0A2K5IME6_COLAP\nSYK\tA0A2K5IMG7_COLAP\nSYK\tA0A2K5IMX3_COLAP\nSYK\tA0A2K5IN12_COLAP\nSYK\tA0A2K5KN52_CERAT\nSYK\tA0A2K5KN60_CERAT\nSYK\tA0A2K5KN86_CERAT\nSYK\tA0A2K5PKU4_CEBCA\nSYK\tA0A2K5PKV5_CEBCA\nSYK\tA0A2K5V790_MACFA\nSYK\tA0A2K5ZX02_MANLE\nSYK\tA0A2K5ZX15_MANLE\nSYK\tA0A2K5ZX19_MANLE\nSYK\tA0A2K6CYC6_MACNE\nSYK\tA0A2K6CYE5_MACNE\nSYK\tA0A2K6CYE9_MACNE\nSYK\tA0A2K6CYG0_MACNE\nSYK\tA0A2K6F641_PROCO\nSYK\tA0A2K6F642_PROCO\nSYK\tA0A2K6F665_PROCO\nSYK\tA0A2K6PUB7_RHIRO\nSYK\tA0A2K6PUC6_RHIRO\nSYK\tA0A2K6PUC9_RHIRO\nSYK\tA0A2K6PUD4_RHIRO\nSYK\tA0A2K6TQJ7_SAIBB\nSYK\tA0A2K6TQM1_SAIBB\nSYK\tA0A2R9BI29_PANPA\nSYK\tA0A2R9BI34_PANPA\nSYK\tA0A2R9BIZ9_PANPA\nSYK\tA0A2U3WLQ9_ODORO\nSYK\tA0A2U3WLR1_ODORO\nSYK\tA0A2U4BF57_TURTR\nSYK\tA0A2U4BF93_TURTR\nSYK\tA0A2U4BFJ9_TURTR\nSYK\tA0A2Y9FIZ2_PHYMC\nSYK\tA0A2Y9HKC1_NEOSC\nSYK\tA0A2Y9M3X0_DELLE\nSYK\tA0A2Y9MGM5_DELLE\nSYK\tA0A340W859_LIPVE\nSYK\tA0A340W865_LIPVE\nSYK\tA0A340WB15_LIPVE\nSYK\tA0A340WDP8_LIPVE\nSYK\tA0A340WDQ4_LIPVE\nSYK\tA0A340WEU5_LIPVE\nSYK\tA0A340WF74_LIPVE\nSYK\tA0A341AAL0_NEOAA\nSYK\tA0A341ACS0_NEOAA\nSYK\tA0A341ADP3_NEOAA\nSYK\tA0A384B1B1_BALAS\nSYK\tA0A384B1J5_BALAS\nSYK\tA0A3B3HPD6_ORYLA\nSYK\tA0A3B3HXM4_ORYLA\nSYK\tA0A3Q0H9L5_ALLSI\nSYK\tA0A3Q0H9L9_ALLSI\nSYK\tA0A3Q0HAZ8_ALLSI\nSYK\tA0A3Q0HE41_ALLSI\nSYK\tA0A3Q1MHG9_BOVIN\nSYK\tA0A3Q2H5Q2_HORSE\nSYK\tA0A3Q2H7A8_HORSE\nSYK\tA0A3Q2HPD6_HORSE\nSYK\tA0A3Q2HYK1_HORSE\nSYK\tA0A3Q7RCS6_VULVU\nSYK\tA0A3Q7SEB1_VULVU\nSYK\tA0A3Q7XBV3_URSAR\nSYK\tA0A3R7CDG8_CLOSI\nSYK\tA0A452EB02_CAPHI\nSYK\tA0A452EBF0_CAPHI\nSYK\tA0A452EBG1_CAPHI\nSYK\tA0A452EBK3_CAPHI\nSYK\tA0A452EBP4_CAPHI\nSYK\tA0A452HWZ8_9SAUR\nSYK\tA0A452HX03_9SAUR\nSYK\tA0A452HX19_9SAUR\nSYK\tA0A452HX98_9SAUR\nSYK\tA0A452SHK3_URSAM\nSYK\tA0A452SHL1_URSAM\nSYK\tA0A452UN14_URSMA\nSYK\tA0A452UN95_URSMA\nSYK\tA0A452UNC0_URSMA\nSYK\tA0A480EJV8_PIG\nSYK\tA0A493TXF0_ANAPP\nSYK\tA0A4W2DNE0_BOBOX\nSYK\tA0A4W2DNE9_BOBOX\nSYK\tA0A4W2FYX0_BOBOX\nSYK\tA0A4W2HWD7_BOBOX\nSYK\tA0A4W3IP48_CALMI\nSYK\tA0A4W3IP53_CALMI\nSYK\tA0A4W4GTH4_ELEEL\nSYK\tA0A4W4GWE3_ELEEL\nSYK\tA0A4W4H008_ELEEL\nSYK\tA0A4W4H013_ELEEL\nSYK\tA0A4W4H0B8_ELEEL\nSYK\tA0A4W4H0C3_ELEEL\nSYK\tA0A4W6DJJ4_LATCA\nSYK\tA0A4W6DKH2_LATCA\nSYK\tA0A4W6DKJ3_LATCA\nSYK\tA0A4W6DKT3_LATCA\nSYK\tA0A4X2KAJ5_VOMUR\nSYK\tA0A4X2KFY9_VOMUR\nSYK\tA0A4X2KG26_VOMUR\nSYK\tA0A4X2KHT5_VOMUR\nSYK\tA0A4Y2I1S4_ARAVE\nSYK\tA0A5B7HJE6_PORTR\nSYK\tA0A5F4CKS4_CANLF\nSYK\tA0A5F4CLC8_CANLF\nSYK\tA0A5F5PEA3_HORSE\nSYK\tA0A5F5XK64_FELCA\nSYK\tA0A5F5XR55_FELCA\nSYK\tA0A5F5XVZ3_FELCA\nSYK\tA0A5F7ZCV9_MACMU\nSYK\tA0A5F7ZUV3_MACMU\nSYK\tA0A5F8H766_MONDO\nSYK\tA0A5F9CRT8_RABIT\nSYK\tA0A5F9CUA6_RABIT\nSYK\tC0HA59_SALSA\nSYK\tC0PUL2_SALSA\nSYK\tC3W980_HUMAN\nSYK\tC3W981_HUMAN\nSYK\tD3ZH44_RAT\nSYK\tE9PWE9_MOUSE\nSYK\tE9QFU7_DANRE\nSYK\tF1PMP8_CANLF\nSYK\tF1PSC8_CANLF\nSYK\tF1QFN4_DANRE\nSYK\tF1RN11_PIG\nSYK\tF6VLR6_MONDO\nSYK\tF7BR88_HORSE\nSYK\tF7DXJ8_XENTR\nSYK\tF7GLV2_MACMU\nSYK\tF7GLV6_MACMU\nSYK\tF7HWV5_CALJA\nSYK\tG1K8U5_ANOCA\nSYK\tG1MGB0_AILME\nSYK\tG1NIC4_MELGA\nSYK\tG1PRD5_MYOLU\nSYK\tG1QHY9_NOMLE\nSYK\tG1TDW9_RABIT\nSYK\tG3T8M0_LOXAF\nSYK\tG3V7N4_RAT\nSYK\tG3VER0_SARHA\nSYK\tG7PSP1_MACFA\nSYK\tH0VGU1_CAVPO\nSYK\tH0X5F7_OTOGA\nSYK\tH0YQ82_TAEGU\nSYK\tH2MHT2_ORYLA\nSYK\tH2PSM9_PONAB\nSYK\tH2QXF8_PANTR\nSYK\tH2TIX5_TAKRU\nSYK\tH3ATL1_LATCH\nSYK\tH9FBY0_MACMU\nSYK\tH9FIZ6_MACMU\nSYK\tH9FLW1_MACMU\nSYK\tH9FLW2_MACMU\nSYK\tI3JZ29_ORENI\nSYK\tI3JZ30_ORENI\nSYK\tI3M6P2_ICTTR\nSYK\tI3NDE9_ICTTR\nSYK\tK7G0N5_PELSI\nSYK\tK7GP57_PIG\nSYK\tKSYK_CHICK\nSYK\tKSYK_HUMAN\nSYK\tKSYK_MOUSE\nSYK\tKSYK_PIG\nSYK\tKSYK_RAT\nSYK\tM3WKA9_FELCA\nSYK\tQ32PK0_BOVIN\nSYK\tQ3UPF7_MOUSE\nSYK\tQ3V3Q2_MOUSE\nSYK\tQ6DE64_XENLA\nSYK\tQ6IZA9_PHOVI\nSYK\tQ6JDG6_CANLF\nSYK\tQ6LEP4_MOUSE\nSYK\tQ6P1E0_MOUSE\nSYK\tQ6P6Z9_XENLA\nSYK\tQ910A2_DANRE\nSYK\tQ910A3_CYPCA\nSYK\tQ9EV02_DICCH\nSYK\tR0M6N5_NOSB1\nSYK\tU3BGC5_CALJA\nSYK\tU3I7C8_ANAPP\nSYK\tU3JQ94_FICAL\nSYK\tV8NCH3_OPHHA\nSYK\tW5PD78_SHEEP\nSYK\tW8BYC0_CERCA\nSYK\tW8CBC3_CERCA\n','From\tTo\nMST1R\tA0A087WZQ8_HUMAN\nMST1R\tA0A096MQM5_PAPAN\nMST1R\tA0A0A6YXY9_MOUSE\nMST1R\tA0A0D9RPP6_CHLSB\nMST1R\tA0A0N9LT37_FELCA\nMST1R\tA0A1A7ZDG5_NOTFU\nMST1R\tA0A1A8A2F2_NOTFU\nMST1R\tA0A1A8BTQ1_9TELE\nMST1R\tA0A1A8E4B3_9TELE\nMST1R\tA0A1A8E607_9TELE\nMST1R\tA0A1A8E655_9TELE\nMST1R\tA0A1A8JIR2_NOTKU\nMST1R\tA0A1A8K411_NOTKU\nMST1R\tA0A1A8K659_NOTKU\nMST1R\tA0A1A8L779_9TELE\nMST1R\tA0A1A8L9W9_9TELE\nMST1R\tA0A1A8N4G2_9TELE\nMST1R\tA0A1A8N4G4_9TELE\nMST1R\tA0A1A8N611_9TELE\nMST1R\tA0A1A8N6L1_9TELE\nMST1R\tA0A1S3A0D4_ERIEU\nMST1R\tA0A1S3G417_DIPOR\nMST1R\tA0A1S3G4G2_DIPOR\nMST1R\tA0A1S3G5J6_DIPOR\nMST1R\tA0A1U7TMW5_TARSY\nMST1R\tA0A1U7TY30_TARSY\nMST1R\tA0A1U7U7X3_TARSY\nMST1R\tA0A1U8D6R5_ALLSI\nMST1R\tA0A1V4K8U3_PATFA\nMST1R\tA0A1V4KAF7_PATFA\nMST1R\tA0A218URE4_9PASE\nMST1R\tA0A286XS04_CAVPO\nMST1R\tA0A286ZJ49_PIG\nMST1R\tA0A287A7A0_PIG\nMST1R\tA0A2I3G8S6_NOMLE\nMST1R\tA0A2I3GGS3_NOMLE\nMST1R\tA0A2I3GKF5_NOMLE\nMST1R\tA0A2I3HDY8_NOMLE\nMST1R\tA0A2I3HNI2_NOMLE\nMST1R\tA0A2I3LYE4_PAPAN\nMST1R\tA0A2I3M9C8_PAPAN\nMST1R\tA0A2I3MQ13_PAPAN\nMST1R\tA0A2I3NF03_PAPAN\nMST1R\tA0A2I3RUG3_PANTR\nMST1R\tA0A2I3SHB5_PANTR\nMST1R\tA0A2I3SPB0_PANTR\nMST1R\tA0A2I3T3X5_PANTR\nMST1R\tA0A2I4CDW0_9TELE\nMST1R\tA0A2K5DAA4_AOTNA\nMST1R\tA0A2K5DAE5_AOTNA\nMST1R\tA0A2K5DAE8_AOTNA\nMST1R\tA0A2K5KGX3_COLAP\nMST1R\tA0A2K5KGY2_COLAP\nMST1R\tA0A2K5KGY5_COLAP\nMST1R\tA0A2K5KGY6_COLAP\nMST1R\tA0A2K5KUK0_CERAT\nMST1R\tA0A2K5KUL6_CERAT\nMST1R\tA0A2K5KUN3_CERAT\nMST1R\tA0A2K5KUN4_CERAT\nMST1R\tA0A2K5KUP7_CERAT\nMST1R\tA0A2K5R2S4_CEBCA\nMST1R\tA0A2K5R2T4_CEBCA\nMST1R\tA0A2K5R2U1_CEBCA\nMST1R\tA0A2K5VT07_MACFA\nMST1R\tA0A2K5VT19_MACFA\nMST1R\tA0A2K5VT21_MACFA\nMST1R\tA0A2K5VT26_MACFA\nMST1R\tA0A2K5VT31_MACFA\nMST1R\tA0A2K5YGJ1_MANLE\nMST1R\tA0A2K5YGP7_MANLE\nMST1R\tA0A2K5YGQ4_MANLE\nMST1R\tA0A2K5YGQ6_MANLE\nMST1R\tA0A2K6CYS0_MACNE\nMST1R\tA0A2K6CYS5_MACNE\nMST1R\tA0A2K6CYT1_MACNE\nMST1R\tA0A2K6CYT6_MACNE\nMST1R\tA0A2K6CYT8_MACNE\nMST1R\tA0A2K6GAT3_PROCO\nMST1R\tA0A2K6GAV7_PROCO\nMST1R\tA0A2K6GAW0_PROCO\nMST1R\tA0A2K6MDM2_RHIBE\nMST1R\tA0A2K6MDM3_RHIBE\nMST1R\tA0A2K6MDR0_RHIBE\nMST1R\tA0A2K6MDS8_RHIBE\nMST1R\tA0A2K6MDT2_RHIBE\nMST1R\tA0A2K6QD56_RHIRO\nMST1R\tA0A2K6QD57_RHIRO\nMST1R\tA0A2K6QD58_RHIRO\nMST1R\tA0A2K6QD64_RHIRO\nMST1R\tA0A2K6QD66_RHIRO\nMST1R\tA0A2K6SQ73_SAIBB\nMST1R\tA0A2K6SQ85_SAIBB\nMST1R\tA0A2K6SQ97_SAIBB\nMST1R\tA0A2R9AHK2_PANPA\nMST1R\tA0A2R9AIE6_PANPA\nMST1R\tA0A2R9AM93_PANPA\nMST1R\tA0A2R9AM99_PANPA\nMST1R\tA0A2R9APN9_PANPA\nMST1R\tA0A2U3VVE2_ODORO\nMST1R\tA0A2U3VVG3_ODORO\nMST1R\tA0A2U4B206_TURTR\nMST1R\tA0A2U4B212_TURTR\nMST1R\tA0A2U4B217_TURTR\nMST1R\tA0A2U4B218_TURTR\nMST1R\tA0A2U4B221_TURTR\nMST1R\tA0A2U4B223_TURTR\nMST1R\tA0A2U4B228_TURTR\nMST1R\tA0A2U4B233_TURTR\nMST1R\tA0A2U4B251_TURTR\nMST1R\tA0A2U4B256_TURTR\nMST1R\tA0A2U4B261_TURTR\nMST1R\tA0A2U4B2F3_TURTR\nMST1R\tA0A2U4B2F6_TURTR\nMST1R\tA0A2U4B2G1_TURTR\nMST1R\tA0A2Y9GTT4_NEOSC\nMST1R\tA0A2Y9GY20_NEOSC\nMST1R\tA0A2Y9GZR0_NEOSC\nMST1R\tA0A2Y9N279_DELLE\nMST1R\tA0A2Y9N3K6_DELLE\nMST1R\tA0A2Y9N6W6_DELLE\nMST1R\tA0A2Y9NHM6_DELLE\nMST1R\tA0A2Y9SM63_PHYMC\nMST1R\tA0A340XYU1_LIPVE\nMST1R\tA0A341AR80_NEOAA\nMST1R\tA0A383Z1E6_BALAS\nMST1R\tA0A384D6G4_URSMA\nMST1R\tA0A3B3HCY1_ORYLA\nMST1R\tA0A3B3HT99_ORYLA\nMST1R\tA0A3B5K119_TAKRU\nMST1R\tA0A3Q0CTB1_MESAU\nMST1R\tA0A3Q0CXZ9_MESAU\nMST1R\tA0A3Q0CYI2_MESAU\nMST1R\tA0A3Q0GP42_ALLSI\nMST1R\tA0A3Q3AJC5_CHICK\nMST1R\tA0A3Q7Q609_CALUR\nMST1R\tA0A3Q7QGF6_CALUR\nMST1R\tA0A3Q7RCA0_VULVU\nMST1R\tA0A3Q7RCD0_VULVU\nMST1R\tA0A3Q7RD52_VULVU\nMST1R\tA0A3Q7RDV8_VULVU\nMST1R\tA0A3Q7RL43_CALUR\nMST1R\tA0A3Q7RMP5_VULVU\nMST1R\tA0A3Q7SNB9_VULVU\nMST1R\tA0A3Q7VVT5_URSAR\nMST1R\tA0A3Q7W5T9_URSAR\nMST1R\tA0A3Q7WR49_URSAR\nMST1R\tA0A452G432_CAPHI\nMST1R\tA0A452GQ54_9SAUR\nMST1R\tA0A452GQS8_9SAUR\nMST1R\tA0A452QSG5_URSAM\nMST1R\tA0A452VAM7_URSMA\nMST1R\tA0A452VAN3_URSMA\nMST1R\tA0A452VAN7_URSMA\nMST1R\tA0A493TGZ1_ANAPP\nMST1R\tA0A4W2BYF8_BOBOX\nMST1R\tA0A4W2FI41_BOBOX\nMST1R\tA0A4W6EPZ4_LATCA\nMST1R\tA0A4W6ES40_LATCA\nMST1R\tA0A4X1SNB8_PIG\nMST1R\tA0A4X1SNE7_PIG\nMST1R\tA0A4X1SP38_PIG\nMST1R\tA0A4X1SP51_PIG\nMST1R\tA0A4X1SPX9_PIG\nMST1R\tA0A4X2LHI0_VOMUR\nMST1R\tA0A4X2LNS3_VOMUR\nMST1R\tA0A4X2LNT8_VOMUR\nMST1R\tA0A4X2LNU4_VOMUR\nMST1R\tA0A4Z2HP54_9TELE\nMST1R\tA0A4Z2J4K9_9TELE\nMST1R\tA0A5F4CDG1_CANLF\nMST1R\tA0A5F4DBH5_CANLF\nMST1R\tA0A5F9CMW4_RABIT\nMST1R\tA0A5F9D4R4_RABIT\nMST1R\tA0A5G3JG87_XENTR\nMST1R\tB2RUR6_MOUSE\nMST1R\tD3ZYM4_RAT\nMST1R\tE1BCZ6_BOVIN\nMST1R\tE1C5T2_CHICK\nMST1R\tF1Q2W9_CANLF\nMST1R\tF6XEZ0_XENTR\nMST1R\tF6XRA9_CANLF\nMST1R\tF7DX11_XENTR\nMST1R\tF7E2F2_MONDO\nMST1R\tF7ED27_MACMU\nMST1R\tF7ED42_MACMU\nMST1R\tF7I635_CALJA\nMST1R\tF7I659_CALJA\nMST1R\tF7IAV5_CALJA\nMST1R\tG1KHT0_ANOCA\nMST1R\tG1M098_AILME\nMST1R\tG1PVE9_MYOLU\nMST1R\tG1R5H2_NOMLE\nMST1R\tG1SX27_RABIT\nMST1R\tG3SYP8_LOXAF\nMST1R\tG3X0S5_SARHA\nMST1R\tG3X0S6_SARHA\nMST1R\tH0UYB4_CAVPO\nMST1R\tH0Z3P7_TAEGU\nMST1R\tH2E272_HUMAN\nMST1R\tH2MH21_ORYLA\nMST1R\tH2PAQ3_PONAB\nMST1R\tH2QMM8_PANTR\nMST1R\tH2UFK9_TAKRU\nMST1R\tH7C074_HUMAN\nMST1R\tH7C3Z8_HUMAN\nMST1R\tH9FDJ8_MACMU\nMST1R\tH9FFI5_MACMU\nMST1R\tH9FL31_MACMU\nMST1R\tI3KWR0_ORENI\nMST1R\tI3KWR1_ORENI\nMST1R\tI3L5Q4_PIG\nMST1R\tI3LG23_PIG\nMST1R\tI3M4A6_ICTTR\nMST1R\tJ3KQS7_HUMAN\nMST1R\tM3YPD7_MUSPF\nMST1R\tP79950_XENLA\nMST1R\tQ66IM4_XENTR\nMST1R\tRON_HUMAN\nMST1R\tRON_MOUSE\nMST1R\tU3D7M8_CALJA\nMST1R\tU3K2W1_FICAL\nMST1R\tV8P1P1_OPHHA\nMST1R\tW5PNY9_SHEEP\nMST1R\tW5UA68_ICTPU\n','From\tTo\nRET\tA0A024R7S5_HUMAN\nRET\tA0A024R7T1_HUMAN\nRET\tA0A024R7T2_HUMAN\nRET\tA0A034WK70_BACDO\nRET\tA0A061R5T4_9CHLO\nRET\tA0A096P476_PAPAN\nRET\tA0A0A9XP98_LYGHE\nRET\tA0A0B2UPU7_TOXCA\nRET\tA0A0B2VQK2_TOXCA\nRET\tA0A0B5JW56_DANRE\nRET\tA0A0B6XFI8_XENBV\nRET\tA0A0C9R191_9HYME\nRET\tA0A0C9RNG2_9HYME\nRET\tA0A0D9REZ9_CHLSB\nRET\tA0A0K2TVB1_LEPSM\nRET\tA0A0K2TVS5_LEPSM\nRET\tA0A0K8U068_BACLA\nRET\tA0A0P6JVZ7_HETGA\nRET\tA0A0S7EX86_9TELE\nRET\tA0A0S7I9G6_9TELE\nRET\tA0A0S7I9I3_9TELE\nRET\tA0A0S7I9J9_9TELE\nRET\tA0A0S7IBU6_9TELE\nRET\tA0A0S7MDX4_9TELE\nRET\tA0A0V0S341_9BILA\nRET\tA0A0V0S362_9BILA\nRET\tA0A0V0S3E7_9BILA\nRET\tA0A0V0S444_9BILA\nRET\tA0A0V0TQZ0_9BILA\nRET\tA0A0V0TR16_9BILA\nRET\tA0A0V0WP70_9BILA\nRET\tA0A0V0WPD3_9BILA\nRET\tA0A0V0WPH0_9BILA\nRET\tA0A0V0WPK3_9BILA\nRET\tA0A0V0ZAX2_9BILA\nRET\tA0A0V0ZBL6_9BILA\nRET\tA0A0V0ZCK8_9BILA\nRET\tA0A0V1BA95_TRISP\nRET\tA0A0V1BAG0_TRISP\nRET\tA0A0V1BAR0_TRISP\nRET\tA0A0V1CJ82_TRIBR\nRET\tA0A0V1CJF8_TRIBR\nRET\tA0A0V1CJX8_TRIBR\nRET\tA0A0V1CK06_TRIBR\nRET\tA0A0V1LH40_9BILA\nRET\tA0A0V1LHB0_9BILA\nRET\tA0A0V1LHP6_9BILA\nRET\tA0A151MT35_ALLMI\nRET\tA0A1A7WKB6_9TELE\nRET\tA0A1A7X113_9TELE\nRET\tA0A1A7YWT1_9TELE\nRET\tA0A1A7ZKM3_NOTFU\nRET\tA0A1A8A2U0_NOTFU\nRET\tA0A1A8BQP8_9TELE\nRET\tA0A1A8C6U8_9TELE\nRET\tA0A1A8DII1_9TELE\nRET\tA0A1A8G7P0_9TELE\nRET\tA0A1A8GFH0_9TELE\nRET\tA0A1A8GW12_9TELE\nRET\tA0A1A8HZ30_NOTKU\nRET\tA0A1A8IE89_NOTKU\nRET\tA0A1A8KAF8_NOTKU\nRET\tA0A1A8KAP8_NOTKU\nRET\tA0A1A8LCY0_9TELE\nRET\tA0A1A8M2I9_9TELE\nRET\tA0A1A8NT73_9TELE\nRET\tA0A1A8Q1A1_9TELE\nRET\tA0A1A8Q804_9TELE\nRET\tA0A1A8Q8D8_9TELE\nRET\tA0A1A8SI87_9TELE\nRET\tA0A1A8UCS9_NOTFU\nRET\tA0A1A8UDM6_NOTFU\nRET\tA0A1L1RML1_CHICK\nRET\tA0A1S3GMF6_DIPOR\nRET\tA0A1S3WV58_ERIEU\nRET\tA0A1U7Q3L7_MESAU\nRET\tA0A1U7R7D4_ALLSI\nRET\tA0A1U7TWQ8_TARSY\nRET\tA0A1V4KJL4_PATFA\nRET\tA0A1V4KJU6_PATFA\nRET\tA0A1W2PPN7_HUMAN\nRET\tA0A1W2PSA1_HUMAN\nRET\tA0A218UTR5_9PASE\nRET\tA0A2B4R226_STYPI\nRET\tA0A2B4R7K3_STYPI\nRET\tA0A2B4R9U2_STYPI\nRET\tA0A2B4RBS2_STYPI\nRET\tA0A2B4RCU6_STYPI\nRET\tA0A2B4RD27_STYPI\nRET\tA0A2B4RFC2_STYPI\nRET\tA0A2B4RFD4_STYPI\nRET\tA0A2B4RFR7_STYPI\nRET\tA0A2B4RIC2_STYPI\nRET\tA0A2B4RJK9_STYPI\nRET\tA0A2B4RJW5_STYPI\nRET\tA0A2B4RMJ7_STYPI\nRET\tA0A2B4RMX4_STYPI\nRET\tA0A2B4RNQ2_STYPI\nRET\tA0A2B4RNX4_STYPI\nRET\tA0A2B4RP86_STYPI\nRET\tA0A2B4RPA2_STYPI\nRET\tA0A2B4RPH8_STYPI\nRET\tA0A2B4RPW1_STYPI\nRET\tA0A2B4RS99_STYPI\nRET\tA0A2B4RSK4_STYPI\nRET\tA0A2B4RV26_STYPI\nRET\tA0A2B4RZY8_STYPI\nRET\tA0A2B4S1F5_STYPI\nRET\tA0A2B4S289_STYPI\nRET\tA0A2B4S4R7_STYPI\nRET\tA0A2B4S5Z3_STYPI\nRET\tA0A2B4S6A0_STYPI\nRET\tA0A2B4SBA5_STYPI\nRET\tA0A2B4SQM7_STYPI\nRET\tA0A2B4SQY5_STYPI\nRET\tA0A2B4SR86_STYPI\nRET\tA0A2B4SS69_STYPI\nRET\tA0A2B4SW68_STYPI\nRET\tA0A2B4SWG7_STYPI\nRET\tA0A2B4SWI0_STYPI\nRET\tA0A2B4SY14_STYPI\nRET\tA0A2D0QL82_ICTPU\nRET\tA0A2H8TDK2_9HEMI\nRET\tA0A2I0MF50_COLLI\nRET\tA0A2I3H8C4_NOMLE\nRET\tA0A2I3LXM4_PAPAN\nRET\tA0A2I3MQI2_PAPAN\nRET\tA0A2I3TR47_PANTR\nRET\tA0A2I4C5X4_9TELE\nRET\tA0A2K5HW60_COLAP\nRET\tA0A2K5HW95_COLAP\nRET\tA0A2K5MKU6_CERAT\nRET\tA0A2K5PBV3_CEBCA\nRET\tA0A2K5V8L5_MACFA\nRET\tA0A2K5YH45_MANLE\nRET\tA0A2K5YH50_MANLE\nRET\tA0A2K6CCS4_MACNE\nRET\tA0A2K6GBZ9_PROCO\nRET\tA0A2K6GC20_PROCO\nRET\tA0A2K6LXG5_RHIBE\nRET\tA0A2K6LXG8_RHIBE\nRET\tA0A2K6R0I7_RHIRO\nRET\tA0A2K6R0L0_RHIRO\nRET\tA0A2K6SC33_SAIBB\nRET\tA0A2K6SC34_SAIBB\nRET\tA0A2N8ZEB6_9VIBR\nRET\tA0A2R8N4S3_CALJA\nRET\tA0A2R9AWY2_PANPA\nRET\tA0A2R9B0Q5_PANPA\nRET\tA0A2S2R3V9_9HEMI\nRET\tA0A2U3XAB2_LEPWE\nRET\tA0A2U3ZQJ6_ODORO\nRET\tA0A2U4BDU6_TURTR\nRET\tA0A2Y9HVS1_NEOSC\nRET\tA0A2Y9LJM0_DELLE\nRET\tA0A2Y9LPH3_DELLE\nRET\tA0A330LPU8_9GAMM\nRET\tA0A337S4G8_FELCA\nRET\tA0A337S9Z4_FELCA\nRET\tA0A340WNX7_LIPVE\nRET\tA0A341BXA0_NEOAA\nRET\tA0A341BXD1_NEOAA\nRET\tA0A383Z756_BALAS\nRET\tA0A3B3H9X3_ORYLA\nRET\tA0A3Q2I505_HORSE\nRET\tA0A3Q7R7R3_CALUR\nRET\tA0A3Q7RKR3_VULVU\nRET\tA0A3Q7UAM3_URSAR\nRET\tA0A452FIE1_CAPHI\nRET\tA0A452FIX0_CAPHI\nRET\tA0A452GQN0_9SAUR\nRET\tA0A452SFQ5_URSAM\nRET\tA0A452UNU5_URSMA\nRET\tA0A455AMB7_PHYMC\nRET\tA0A455RWQ2_SALET\nRET\tA0A4W2BYS0_BOBOX\nRET\tA0A4W2C1A7_BOBOX\nRET\tA0A4W2I9B2_BOBOX\nRET\tA0A4W2IB41_BOBOX\nRET\tA0A4W6CY63_LATCA\nRET\tA0A4W6CY82_LATCA\nRET\tA0A4W6CY97_LATCA\nRET\tA0A4W6CYA0_LATCA\nRET\tA0A4W6CYC5_LATCA\nRET\tA0A4X1U427_PIG\nRET\tA0A4X1U649_PIG\nRET\tA0A4X1U659_PIG\nRET\tA0A4X2LAV1_VOMUR\nRET\tA0A5F4BX99_CANLF\nRET\tA0A5F4D972_CANLF\nRET\tA0A5F4DJV4_CANLF\nRET\tA0A5F4VZ29_CALJA\nRET\tA0A5F8GAJ9_MONDO\nRET\tA0A5F9C4Y2_RABIT\nRET\tA0A5F9CQC7_RABIT\nRET\tA0A5F9ZHB7_HUMAN\nRET\tA0A5F9ZHR6_HUMAN\nRET\tA0A5G2R607_PIG\nRET\tA0A5G3H6H9_XENTR\nRET\tA0A5G3H6J5_XENTR\nRET\tA0A5G3HAD6_XENTR\nRET\tA0A5G3HCI4_XENTR\nRET\tA0A5G3ILS6_XENTR\nRET\tA0A5G3ILY6_XENTR\nRET\tA0A5G3IM06_XENTR\nRET\tA2SXS8_CANLF\nRET\tA6P3D4_HORSE\nRET\tA8CEP7_VIBCL\nRET\tA8E7C6_DANRE\nRET\tB4DGX8_HUMAN\nRET\tB5APJ4_MUSPF\nRET\tB5APJ5_MUSPF\nRET\tB5APJ6_MUSPF\nRET\tB5APJ7_MUSPF\nRET\tC9JYL6_HUMAN\nRET\tD6QWX7_DANRE\nRET\tF1MS00_BOVIN\nRET\tF1NL49_CHICK\nRET\tF1PLF6_CANLF\nRET\tF1R376_DANRE\nRET\tF1RG22_PIG\nRET\tF6S299_MACMU\nRET\tF6S7W7_MACMU\nRET\tF6WCZ4_XENTR\nRET\tF6WV32_HORSE\nRET\tF6X3K5_XENTR\nRET\tF7DJC8_XENTR\nRET\tF7DU26_XENTR\nRET\tF7G7S6_MONDO\nRET\tF7I536_CALJA\nRET\tF8TLS5_HUMAN\nRET\tF8TLW0_HUMAN\nRET\tG1MES8_AILME\nRET\tG1MR14_MELGA\nRET\tG1PWX2_MYOLU\nRET\tG1S142_NOMLE\nRET\tG1T0Q2_RABIT\nRET\tG3TIZ2_LOXAF\nRET\tG3UPL6_MELGA\nRET\tG3UR22_MELGA\nRET\tG3WA34_SARHA\nRET\tH0VQB6_CAVPO\nRET\tH0X4E9_OTOGA\nRET\tH0Z7B3_TAEGU\nRET\tH2MFY1_ORYLA\nRET\tH2NA70_PONAB\nRET\tH2Q1T9_PANTR\nRET\tH2V9H6_TAKRU\nRET\tH3A5G7_LATCH\nRET\tH9G9F1_ANOCA\nRET\tH9ZBZ9_MACMU\nRET\tH9ZC00_MACMU\nRET\tI3JVT0_ORENI\nRET\tI3LYN9_ICTTR\nRET\tJ9P5A3_CANLF\nRET\tK7FDV5_PELSI\nRET\tL8EA94_HUMAN\nRET\tM3WI08_FELCA\nRET\tM3Z0Q6_MUSPF\nRET\tM9Z0T2_HUMAN\nRET\tO42362_DANRE\nRET\tO43519_HUMAN\nRET\tP79726_DANRE\nRET\tQ47526_ECOLX\nRET\tQ63197_RAT\nRET\tQ6IZA6_PHOVI\nRET\tQ7JQD5_DROME\nRET\tQ7KA88_DROME\nRET\tQ7KA90_DROME\nRET\tQ7KAN0_DROME\nRET\tQ7KT06_DROME\nRET\tQ7YS62_HORSE\nRET\tQ80YG6_MOUSE\nRET\tQ8C8Q7_MOUSE\nRET\tQ8IZR8_HUMAN\nRET\tQ8L0W6_VIBPH\nRET\tQ8MJ54_PONPY\nRET\tQ8MJ55_9PRIM\nRET\tQ8MJ56_PANTR\nRET\tQ8MJ57_PONPY\nRET\tQ8MJ58_9PRIM\nRET\tQ8MJ59_PANTR\nRET\tQ8MJ60_PONPY\nRET\tQ8MJ61_9PRIM\nRET\tQ8MJ62_PANTR\nRET\tQ8NFE8_HUMAN\nRET\tQ8VRM1_9DELT\nRET\tQ95RP6_DROME\nRET\tQ99886_HUMAN\nRET\tQ9BHA9_DROME\nRET\tQ9BTX6_HUMAN\nRET\tQ9DGL0_XENLA\nRET\tQ9GP67_DROME\nRET\tQ9GP68_DROME\nRET\tQ9S1F2_VIBCL\nRET\tQ9UE13_HUMAN\nRET\tQ9UM90_HUMAN\nRET\tQ9UMQ4_HUMAN\nRET\tQ9UQV8_HUMAN\nRET\tQ9VIE4_DROME\nRET\tQ9YH35_TETFL\nRET\tRET_HUMAN\nRET\tRET_MOUSE\nRET\tRET_RAT\nRET\tRNMG_ASPRE\nRET\tU3IA91_ANAPP\nRET\tU3JWM3_FICAL\nRET\tU3MHC6_CAVPO\nRET\tU4E2A0_9VIBR\nRET\tU4KEQ6_9VIBR\nRET\tU6DV78_NEOVI\nRET\tV5GQX1_ANOGL\nRET\tV5GUT0_ANOGL\nRET\tV5IAC6_ANOGL\nRET\tV8P465_OPHHA\nRET\tW5P1A8_SHEEP\nRET\tW8BCC2_CERCA\nRET\tW8BST5_CERCA\nRET\tW8BUE5_CERCA\nRET\tW8C7A5_CERCA\n','From\tTo\nBRAF\tA0A021WZM4_9RHIZ\nBRAF\tA0A021X0F7_9RHIZ\nBRAF\tA0A021X4T0_9RHIZ\nBRAF\tA0A021X8Z1_9RHIZ\nBRAF\tA0A021XGX2_9RHIZ\nBRAF\tA0A023XIM9_BRAJP\nBRAF\tA0A066PJF0_9PROT\nBRAF\tA0A066PM51_9PROT\nBRAF\tA0A066PMB3_9PROT\nBRAF\tA0A066PPC4_9PROT\nBRAF\tA0A066PZX9_9PROT\nBRAF\tA0A068SWN9_RHIGA\nBRAF\tA0A068TAL1_RHIGA\nBRAF\tA0A071MDS9_9BURK\nBRAF\tA0A072BS01_RHILP\nBRAF\tA0A072BV21_RHILP\nBRAF\tA0A072C3B7_RHILP\nBRAF\tA0A075USX5_9PSEU\nBRAF\tA0A078BMW3_9BURK\nBRAF\tA0A081CQJ1_9RHIZ\nBRAF\tA0A096NXX5_PAPAN\nBRAF\tA0A096P039_PAPAN\nBRAF\tA0A097APR2_THEKI\nBRAF\tA0A098G0F8_9PSED\nBRAF\tA0A0A8X056_9DELT\nBRAF\tA0A0B6RQY6_9BURK\nBRAF\tA0A0C4LC38_HUMAN\nBRAF\tA0A0D6QF71_9DELT\nBRAF\tA0A0D6QLR2_9DELT\nBRAF\tA0A0D9R789_CHLSB\nBRAF\tA0A0H2Z7J0_PSEAB\nBRAF\tA0A0H2ZIA0_PSEAB\nBRAF\tA0A0H3LA92_PANAA\nBRAF\tA0A0K2YK15_9NOCA\nBRAF\tA0A0K8JEB0_9FIRM\nBRAF\tA0A0M3GJY1_9RHIZ\nBRAF\tA0A0R3CCR5_9PSED\nBRAF\tA0A0S2SH16_9GAMM\nBRAF\tA0A0S7FER8_9TELE\nBRAF\tA0A0S7H0X1_9TELE\nBRAF\tA0A0S7H0X5_9TELE\nBRAF\tA0A0S7H0X7_9TELE\nBRAF\tA0A0S7H0Y4_9TELE\nBRAF\tA0A0S7H0Y7_9TELE\nBRAF\tA0A0S7H0Y8_9TELE\nBRAF\tA0A0S7H101_9TELE\nBRAF\tA0A0S7H111_9TELE\nBRAF\tA0A0S7H135_9TELE\nBRAF\tA0A0S7H154_9TELE\nBRAF\tA0A0S7H185_9TELE\nBRAF\tA0A0S7H1Q6_9TELE\nBRAF\tA0A0S7H2C4_9TELE\nBRAF\tA0A0S7H367_9TELE\nBRAF\tA0A0S7H386_9TELE\nBRAF\tA0A0S7H412_9TELE\nBRAF\tA0A0S7H5D4_9TELE\nBRAF\tA0A0S7H5E3_9TELE\nBRAF\tA0A0S7H5F2_9TELE\nBRAF\tA0A0S7H5H4_9TELE\nBRAF\tA0A0S7H5K2_9TELE\nBRAF\tA0A0S7H611_9TELE\nBRAF\tA0A0S7H7T0_9TELE\nBRAF\tA0A0S7H7U0_9TELE\nBRAF\tA0A0T7FH77_RHIGA\nBRAF\tA0A0T7FI77_RHIGA\nBRAF\tA0A0T7FMZ7_RHIGA\nBRAF\tA0A0T7FPQ1_RHIGA\nBRAF\tA0A0T7GE97_RHIGA\nBRAF\tA0A0T7GM24_RHIGA\nBRAF\tA0A0T7HFH1_RHIGA\nBRAF\tA0A0U5MFN3_9PROT\nBRAF\tA0A0U5MKK2_9PROT\nBRAF\tA0A0V0SHY9_9BILA\nBRAF\tA0A0V0U287_9BILA\nBRAF\tA0A0V0XAF7_9BILA\nBRAF\tA0A0V0YCP9_TRIPS\nBRAF\tA0A0V0ZS61_9BILA\nBRAF\tA0A0V1DI89_TRIBR\nBRAF\tA0A0V1FXU5_TRIPS\nBRAF\tA0A0V1HF44_9BILA\nBRAF\tA0A0V1JV39_TRIPS\nBRAF\tA0A0V1L982_9BILA\nBRAF\tA0A0V1N6M5_9BILA\nBRAF\tA0A0V1N6R4_9BILA\nBRAF\tA0A0V1PE13_9BILA\nBRAF\tA0A0X3NY87_SCHSO\nBRAF\tA0A0X3P436_SCHSO\nBRAF\tA0A0X3P961_SCHSO\nBRAF\tA0A0X3PUS9_SCHSO\nBRAF\tA0A117KIL1_9GAMM\nBRAF\tA0A117KJ56_9GAMM\nBRAF\tA0A151PIV1_ALLMI\nBRAF\tA0A1A7X2V2_9TELE\nBRAF\tA0A1A7ZE31_NOTFU\nBRAF\tA0A1A8CNB3_9TELE\nBRAF\tA0A1A8HK08_9TELE\nBRAF\tA0A1A8JFV3_NOTKU\nBRAF\tA0A1A8JJF1_NOTKU\nBRAF\tA0A1A8MVJ6_9TELE\nBRAF\tA0A1A8PGG1_9TELE\nBRAF\tA0A1A8SBV0_9TELE\nBRAF\tA0A1A8UMF0_NOTFU\nBRAF\tA0A1A8XQE1_9RHOO\nBRAF\tA0A1D3JQV4_9PSED\nBRAF\tA0A1D3JT86_9PSED\nBRAF\tA0A1D3JVA0_9PSED\nBRAF\tA0A1D5PGI2_CHICK\nBRAF\tA0A1D5Q239_MACMU\nBRAF\tA0A1D5Q9W4_MACMU\nBRAF\tA0A1D5QBZ5_MACMU\nBRAF\tA0A1D5QI09_MACMU\nBRAF\tA0A1K0IAH3_CUPNE\nBRAF\tA0A1L8GQP3_XENLA\nBRAF\tA0A1N7RPJ1_9BURK\nBRAF\tA0A1N7S3I0_9BURK\nBRAF\tA0A1N7S457_9BURK\nBRAF\tA0A1R3V296_9RHIZ\nBRAF\tA0A1R3VDP2_9RHIZ\nBRAF\tA0A1S3AAZ0_ERIEU\nBRAF\tA0A1S3FMB4_DIPOR\nBRAF\tA0A1S3WMM5_ERIEU\nBRAF\tA0A1S7M438_9RHIZ\nBRAF\tA0A1S7MG64_9RHIZ\nBRAF\tA0A1S7MP66_9RHIZ\nBRAF\tA0A1S7NLB7_9RHIZ\nBRAF\tA0A1S7NYH7_RHIRD\nBRAF\tA0A1S7P2F6_RHIRD\nBRAF\tA0A1S7PFW5_9RHIZ\nBRAF\tA0A1S7Q2K2_9RHIZ\nBRAF\tA0A1S7QCT7_9RHIZ\nBRAF\tA0A1S7QCX1_9RHIZ\nBRAF\tA0A1S7QJD3_9RHIZ\nBRAF\tA0A1S7QRF8_9RHIZ\nBRAF\tA0A1S7SY43_RHIRD\nBRAF\tA0A1S7TRG5_9RHIZ\nBRAF\tA0A1S7U287_9RHIZ\nBRAF\tA0A1U7R9A5_ALLSI\nBRAF\tA0A1U7U9R8_TARSY\nBRAF\tA0A1V2XCJ4_9BURK\nBRAF\tA0A1V4K7S4_PATFA\nBRAF\tA0A1V4K999_PATFA\nBRAF\tA0A1W1HB18_9DELT\nBRAF\tA0A1W1HD64_9DELT\nBRAF\tA0A1X9T4I6_HUMAN\nBRAF\tA0A1Y6JGK7_PSEVI\nBRAF\tA0A1Y6JR63_PSEVI\nBRAF\tA0A1Y6KJ33_9BRAD\nBRAF\tA0A212IY99_9PROT\nBRAF\tA0A212LBQ3_9DELT\nBRAF\tA0A218V0S6_9PASE\nBRAF\tA0A286Y1E7_CAVPO\nBRAF\tA0A286ZJW4_PIG\nBRAF\tA0A286ZVY2_PIG\nBRAF\tA0A287AUJ1_PIG\nBRAF\tA0A287BFT4_PIG\nBRAF\tA0A287BSZ1_PIG\nBRAF\tA0A2B4S076_STYPI\nBRAF\tA0A2C8FAT3_9DELT\nBRAF\tA0A2D0T588_ICTPU\nBRAF\tA0A2I0MKF5_COLLI\nBRAF\tA0A2I0MKF7_COLLI\nBRAF\tA0A2I2ULH6_FELCA\nBRAF\tA0A2I3HCE0_NOMLE\nBRAF\tA0A2I3M231_PAPAN\nBRAF\tA0A2I3MXI2_PAPAN\nBRAF\tA0A2I3N6Y3_PAPAN\nBRAF\tA0A2I3NBG1_PAPAN\nBRAF\tA0A2I3REF3_PANTR\nBRAF\tA0A2I3RKV0_PANTR\nBRAF\tA0A2I3SIF2_PANTR\nBRAF\tA0A2I3SPY2_PANTR\nBRAF\tA0A2I3SZU6_PANTR\nBRAF\tA0A2I3TK00_PANTR\nBRAF\tA0A2I4CBR3_9TELE\nBRAF\tA0A2K5DY79_AOTNA\nBRAF\tA0A2K5DYC1_AOTNA\nBRAF\tA0A2K5DYC4_AOTNA\nBRAF\tA0A2K5DYE7_AOTNA\nBRAF\tA0A2K5DYH0_AOTNA\nBRAF\tA0A2K5DYL8_AOTNA\nBRAF\tA0A2K5K4L8_COLAP\nBRAF\tA0A2K5K4P5_COLAP\nBRAF\tA0A2K5K4S3_COLAP\nBRAF\tA0A2K5M7Q2_CERAT\nBRAF\tA0A2K5M7S9_CERAT\nBRAF\tA0A2K5M7T5_CERAT\nBRAF\tA0A2K5M7U5_CERAT\nBRAF\tA0A2K5RF62_CEBCA\nBRAF\tA0A2K5RF63_CEBCA\nBRAF\tA0A2K5RF78_CEBCA\nBRAF\tA0A2K5RF86_CEBCA\nBRAF\tA0A2K5U8J8_MACFA\nBRAF\tA0A2K5U8K6_MACFA\nBRAF\tA0A2K5U8L0_MACFA\nBRAF\tA0A2K5U8L2_MACFA\nBRAF\tA0A2K6A436_MANLE\nBRAF\tA0A2K6A487_MANLE\nBRAF\tA0A2K6A497_MANLE\nBRAF\tA0A2K6A498_MANLE\nBRAF\tA0A2K6B2L0_MACNE\nBRAF\tA0A2K6B2M6_MACNE\nBRAF\tA0A2K6B2P0_MACNE\nBRAF\tA0A2K6B2Q6_MACNE\nBRAF\tA0A2K6B2S2_MACNE\nBRAF\tA0A2K6GE74_PROCO\nBRAF\tA0A2K6LDW2_RHIBE\nBRAF\tA0A2K6LDZ1_RHIBE\nBRAF\tA0A2K6LDZ6_RHIBE\nBRAF\tA0A2K6NE27_RHIRO\nBRAF\tA0A2K6NE36_RHIRO\nBRAF\tA0A2K6NE40_RHIRO\nBRAF\tA0A2K6NE45_RHIRO\nBRAF\tA0A2K6NE48_RHIRO\nBRAF\tA0A2K6NE64_RHIRO\nBRAF\tA0A2K6TVE1_SAIBB\nBRAF\tA0A2K6TVE7_SAIBB\nBRAF\tA0A2K6TVH9_SAIBB\nBRAF\tA0A2L1D2G4_RHILV\nBRAF\tA0A2N9MLY1_9BURK\nBRAF\tA0A2P9AFD2_9RHIZ\nBRAF\tA0A2R8P5G8_CALJA\nBRAF\tA0A2R8PCU8_CALJA\nBRAF\tA0A2R8Y467_HUMAN\nBRAF\tA0A2R8Y492_HUMAN\nBRAF\tA0A2R8Y679_HUMAN\nBRAF\tA0A2R8Y8E0_HUMAN\nBRAF\tA0A2R8YDP5_HUMAN\nBRAF\tA0A2R8YES9_HUMAN\nBRAF\tA0A2R9B9I1_PANPA\nBRAF\tA0A2R9BBV0_PANPA\nBRAF\tA0A2R9BFH5_PANPA\nBRAF\tA0A2S2QXG3_9HEMI\nBRAF\tA0A2U3PYE7_9BRAD\nBRAF\tA0A2U3PYH4_9BRAD\nBRAF\tA0A2U3Q418_9BRAD\nBRAF\tA0A2U3Q8K3_9BRAD\nBRAF\tA0A2U3QC35_9BRAD\nBRAF\tA0A2U3QCV3_9BRAD\nBRAF\tA0A2U3TZI2_HUMAN\nBRAF\tA0A2U3ZUR6_ODORO\nBRAF\tA0A2U4C6J2_TURTR\nBRAF\tA0A2U4C6W2_TURTR\nBRAF\tA0A2U4C715_TURTR\nBRAF\tA0A2U4C7B5_TURTR\nBRAF\tA0A2X3L233_9ACTN\nBRAF\tA0A2X3L8L7_9ACTN\nBRAF\tA0A2Y9H9P5_NEOSC\nBRAF\tA0A2Y9NQW5_DELLE\nBRAF\tA0A2Y9NVV9_DELLE\nBRAF\tA0A2Y9SYF0_PHYMC\nBRAF\tA0A2Y9SYF4_PHYMC\nBRAF\tA0A2Y9SZY4_PHYMC\nBRAF\tA0A2Z6IT99_9BURK\nBRAF\tA0A2Z6ITU2_9BURK\nBRAF\tA0A2Z6IUX3_9BURK\nBRAF\tA0A2Z6IUY6_9RALS\nBRAF\tA0A340Y5C2_LIPVE\nBRAF\tA0A341B621_NEOAA\nBRAF\tA0A341B7T7_NEOAA\nBRAF\tA0A375CMT2_9BURK\nBRAF\tA0A375D8P5_9BURK\nBRAF\tA0A375DFE1_9BURK\nBRAF\tA0A375DQ20_9BURK\nBRAF\tA0A375EC07_9BURK\nBRAF\tA0A375ETJ0_9BURK\nBRAF\tA0A375FG37_9BURK\nBRAF\tA0A375FMX2_9BURK\nBRAF\tA0A375FYB8_9BURK\nBRAF\tA0A375GBB7_9BURK\nBRAF\tA0A375GDZ9_9BURK\nBRAF\tA0A375GH03_9BURK\nBRAF\tA0A375GXW6_9BURK\nBRAF\tA0A375HE46_9BURK\nBRAF\tA0A375HE48_9BURK\nBRAF\tA0A375HT70_9BURK\nBRAF\tA0A375IA02_9BURK\nBRAF\tA0A375IL85_9BURK\nBRAF\tA0A375JAA3_9BURK\nBRAF\tA0A380SVF8_9PSED\nBRAF\tA0A381JI19_9FIRM\nBRAF\tA0A383R502_PAEAL\nBRAF\tA0A383RQ62_9PSED\nBRAF\tA0A383RQB9_9PSED\nBRAF\tA0A383S275_9PSED\nBRAF\tA0A384CLE5_URSMA\nBRAF\tA0A3B3HTL3_ORYLA\nBRAF\tA0A3B5KZ01_9TELE\nBRAF\tA0A3B5L3D1_9TELE\nBRAF\tA0A3B5L4J7_9TELE\nBRAF\tA0A3B5L667_9TELE\nBRAF\tA0A3Q0C8X5_MESAU\nBRAF\tA0A3Q1LML5_BOVIN\nBRAF\tA0A3Q1M353_BOVIN\nBRAF\tA0A3Q2H5J3_HORSE\nBRAF\tA0A3Q2I804_HORSE\nBRAF\tA0A3Q2L1N8_HORSE\nBRAF\tA0A3Q2TXW8_CHICK\nBRAF\tA0A3Q3FJP5_9LABR\nBRAF\tA0A3Q3FL31_9LABR\nBRAF\tA0A3Q3MHJ7_9LABR\nBRAF\tA0A3Q7MS56_CALUR\nBRAF\tA0A3Q7MTR3_CALUR\nBRAF\tA0A3Q7MTR8_CALUR\nBRAF\tA0A3Q7MTU1_CALUR\nBRAF\tA0A3Q7MUE0_CALUR\nBRAF\tA0A3Q7P4K6_CALUR\nBRAF\tA0A3Q7QXV4_VULVU\nBRAF\tA0A3Q7R7I1_VULVU\nBRAF\tA0A3Q7X1G1_URSAR\nBRAF\tA0A3Q7XXL0_URSAR\nBRAF\tA0A3Q7XXL9_URSAR\nBRAF\tA0A3R7DBU0_CLOSI\nBRAF\tA0A445ZRM0_9PSED\nBRAF\tA0A452CD89_BALAS\nBRAF\tA0A452F4R9_CAPHI\nBRAF\tA0A452F5D0_CAPHI\nBRAF\tA0A452F5E2_CAPHI\nBRAF\tA0A452J488_9SAUR\nBRAF\tA0A452J496_9SAUR\nBRAF\tA0A452RYF7_URSAM\nBRAF\tA0A452UHU2_URSMA\nBRAF\tA0A455B935_PHYMC\nBRAF\tA0A455B9W2_PHYMC\nBRAF\tA0A455BBZ2_PHYMC\nBRAF\tA0A484HD38_9DELT\nBRAF\tA0A485LY20_9ZZZZ\nBRAF\tA0A485M4G1_9ZZZZ\nBRAF\tA0A493T0B7_ANAPP\nBRAF\tA0A4W2D401_BOBOX\nBRAF\tA0A4W2EPG5_BOBOX\nBRAF\tA0A4W2EPI4_BOBOX\nBRAF\tA0A4W2F4D3_BOBOX\nBRAF\tA0A4W2F4D4_BOBOX\nBRAF\tA0A4W2F4F6_BOBOX\nBRAF\tA0A4W2F4G4_BOBOX\nBRAF\tA0A4W2F5P0_BOBOX\nBRAF\tA0A4W2F5P1_BOBOX\nBRAF\tA0A4W2H609_BOBOX\nBRAF\tA0A4W6DQX0_LATCA\nBRAF\tA0A4W6DQX9_LATCA\nBRAF\tA0A4W6DRD8_LATCA\nBRAF\tA0A4W6DRE3_LATCA\nBRAF\tA0A4W6DRJ6_LATCA\nBRAF\tA0A4W6DRL7_LATCA\nBRAF\tA0A4W6DST8_LATCA\nBRAF\tA0A4W6DTA7_LATCA\nBRAF\tA0A4W6DTJ3_LATCA\nBRAF\tA0A4X1V2V1_PIG\nBRAF\tA0A4X1V5W4_PIG\nBRAF\tA0A4X1V6E8_PIG\nBRAF\tA0A4X1V741_PIG\nBRAF\tA0A4X1V816_PIG\nBRAF\tA0A4X1V851_PIG\nBRAF\tA0A4X2L4F9_VOMUR\nBRAF\tA0A4X2LCT2_VOMUR\nBRAF\tA0A4Y4GIT7_COBMA\nBRAF\tA0A5E4GY80_9RHIZ\nBRAF\tA0A5E4NWQ0_9RHIZ\nBRAF\tA0A5E7YKG0_9RHIZ\nBRAF\tA0A5E7ZAU9_9ALTE\nBRAF\tA0A5E8C7C6_9BRAD\nBRAF\tA0A5E8CBX5_9BRAD\nBRAF\tA0A5F4DDJ6_CANLF\nBRAF\tA0A5F7ZHN7_MACMU\nBRAF\tA0A5F8A7V3_MACMU\nBRAF\tA0A5F9CCZ3_RABIT\nBRAF\tA0A5F9CFV9_RABIT\nBRAF\tA0A5F9D5I8_RABIT\nBRAF\tA0A5F9DBE2_RABIT\nBRAF\tA0A5F9DEP2_RABIT\nBRAF\tA0A5G2QBS9_PIG\nBRAF\tA0A5G3HBM0_XENTR\nBRAF\tA0A5G3HC04_XENTR\nBRAF\tA0A5G3HDG4_XENTR\nBRAF\tA0A5G3HDS8_XENTR\nBRAF\tA0A5G3HUM9_XENTR\nBRAF\tA0A5G3HVD0_XENTR\nBRAF\tA0A5G3IYD9_XENTR\nBRAF\tA0A5G3IYE3_XENTR\nBRAF\tA0A5G3IZL1_XENTR\nBRAF\tA0A5G3JKW7_XENTR\nBRAF\tA0A5G3JL56_XENTR\nBRAF\tA0A5G3JM15_XENTR\nBRAF\tA0A5G3JMG2_XENTR\nBRAF\tA1K1N0_AZOSB\nBRAF\tA4Z042_BRASO\nBRAF\tA6V9C4_PSEA7\nBRAF\tB3DFX5_DANRE\nBRAF\tB3PXZ2_RHIE6\nBRAF\tB4EGH9_BURCJ\nBRAF\tB7ZRT9_XENLA\nBRAF\tB9J8N8_AGRRK\nBRAF\tB9JHQ6_AGRRK\nBRAF\tB9JHV3_AGRRK\nBRAF\tB9JI45_AGRRK\nBRAF\tB9JKH3_AGRRK\nBRAF\tBRAF_CHICK\nBRAF\tBRAF_COTJA\nBRAF\tBRAF_HUMAN\nBRAF\tBRAF_MOUSE\nBRAF\tBRAF_PSEAE\nBRAF\tC0QH70_DESAH\nBRAF\tC8SZI2_KLEPR\nBRAF\tC9Y9B9_9BURK\nBRAF\tC9YDU5_9BURK\nBRAF\tD4E0A5_SEROD\nBRAF\tD4GJJ8_PANAM\nBRAF\tD4X6W7_9BURK\nBRAF\tE2I6F4_HUMAN\nBRAF\tE5FF37_HUMAN\nBRAF\tE6Q428_9ZZZZ\nBRAF\tE6QRC8_9ZZZZ\nBRAF\tF1CC75_HUMAN\nBRAF\tF1M9C3_RAT\nBRAF\tF1MDL4_BOVIN\nBRAF\tF1P873_CANLF\nBRAF\tF1P874_CANLF\nBRAF\tF1SRQ1_PIG\nBRAF\tF5S319_9ENTR\nBRAF\tF6PN24_XENTR\nBRAF\tF6RRT4_HORSE\nBRAF\tF6SZ47_MOUSE\nBRAF\tF6TI51_XENTR\nBRAF\tF6V946_XENTR\nBRAF\tF6ZGS2_XENTR\nBRAF\tF6ZT87_XENTR\nBRAF\tF7BCJ7_XENTR\nBRAF\tF7DEX8_ORNAN\nBRAF\tF7FV05_MONDO\nBRAF\tF8HAC6_PSEUT\nBRAF\tG1K3G6_XENTR\nBRAF\tG1KL20_ANOCA\nBRAF\tG1M9X7_AILME\nBRAF\tG1NKK9_MELGA\nBRAF\tG1P9K1_MYOLU\nBRAF\tG1QP54_NOMLE\nBRAF\tG1SP13_RABIT\nBRAF\tG3THK2_LOXAF\nBRAF\tG3U327_LOXAF\nBRAF\tG3U9Y9_LOXAF\nBRAF\tG3USZ3_MELGA\nBRAF\tG3WTV7_SARHA\nBRAF\tG7ZA47_AZOL4\nBRAF\tG8ATU9_AZOBR\nBRAF\tG8LH72_9ENTR\nBRAF\tG8LPE8_9ENTR\nBRAF\tG8PL00_PSEUV\nBRAF\tH0SVM8_9BRAD\nBRAF\tH0VXS1_CAVPO\nBRAF\tH0X101_OTOGA\nBRAF\tH0ZN23_TAEGU\nBRAF\tH2M1R4_ORYLA\nBRAF\tH2M1R6_ORYLA\nBRAF\tH2PNQ1_PONAB\nBRAF\tH2QVI0_PANTR\nBRAF\tH2TXG3_TAKRU\nBRAF\tH2TXG7_TAKRU\nBRAF\tH2TXG8_TAKRU\nBRAF\tH3AT03_LATCH\nBRAF\tH6MQX6_GORPV\nBRAF\tH6RWU3_BLASD\nBRAF\tH7C4S5_HUMAN\nBRAF\tH7C560_HUMAN\nBRAF\tH7C5K3_HUMAN\nBRAF\tH9F989_MACMU\nBRAF\tI0L6C3_9ACTN\nBRAF\tI0L8I2_9ACTN\nBRAF\tI3DUI0_BACMM\nBRAF\tI3M0P3_ICTTR\nBRAF\tI3THU7_TISMK\nBRAF\tI3TI85_TISMK\nBRAF\tI3TJ26_TISMK\nBRAF\tI3TL45_TISMK\nBRAF\tI3TLC6_TISMK\nBRAF\tI3TM61_TISMK\nBRAF\tI3TMI2_TISMK\nBRAF\tI3TQ62_TISMK\nBRAF\tI3TQ94_TISMK\nBRAF\tI3TS33_TISMK\nBRAF\tI3TS95_TISMK\nBRAF\tI3TSG2_TISMK\nBRAF\tI3TST1_TISMK\nBRAF\tI3TT13_TISMK\nBRAF\tI3TUH6_TISMK\nBRAF\tI3TUL3_TISMK\nBRAF\tI3TUP5_TISMK\nBRAF\tI3TWA3_TISMK\nBRAF\tI3TX93_TISMK\nBRAF\tI4F4N6_9ACTN\nBRAF\tI4FRR9_MICAE\nBRAF\tI4GW51_MICAE\nBRAF\tI4HHU5_MICAE\nBRAF\tI4HTN2_MICAE\nBRAF\tI4KY40_9PSED\nBRAF\tI4KYH5_PSEFL\nBRAF\tJ2MUJ8_PSEFL\nBRAF\tJ2YL57_9PSED\nBRAF\tK7CLW3_PANTR\nBRAF\tK7FXR9_PELSI\nBRAF\tK7FXS3_PELSI\nBRAF\tK9NDN0_9PSED\nBRAF\tL8A2N2_9BACI\nBRAF\tM1FAS2_9ALTE\nBRAF\tM1FB04_9ALTE\nBRAF\tM1FDK3_9ALTE\nBRAF\tM1FG54_9ALTE\nBRAF\tM1FKR6_9ALTE\nBRAF\tM1MM61_9CLOT\nBRAF\tM1WSH6_PSEP2\nBRAF\tM3WHQ7_FELCA\nBRAF\tM3XWD1_MUSPF\nBRAF\tM3ZBG5_NOMLE\nBRAF\tM5EQL6_9RHIZ\nBRAF\tQ0D2E4_XENTR\nBRAF\tQ0RSQ9_FRAAA\nBRAF\tQ1LYG2_DANRE\nBRAF\tQ1MCU2_RHIL3\nBRAF\tQ2K4Z9_RHIEC\nBRAF\tQ2T469_BURTA\nBRAF\tQ4F9K6_XENLA\nBRAF\tQ4KH57_PSEF5\nBRAF\tQ4W6L8_TAKRU\nBRAF\tQ5H8C3_DANRE\nBRAF\tQ63MS0_BURPS\nBRAF\tQ698R0_CANLF\nBRAF\tQ698R1_CANLF\nBRAF\tQ6FAZ5_ACIAD\nBRAF\tQ6L8Q6_SERQU\nBRAF\tQ75MQ8_HUMAN\nBRAF\tQ75V91_DANRE\nBRAF\tQ75V92_DANRE\nBRAF\tQ761X4_DANRE\nBRAF\tQ767H4_DANRE\nBRAF\tQ767H5_XENLA\nBRAF\tQ88DG1_PSEPK\nBRAF\tQ9JJU4_MOUSE\nBRAF\tQ9JJU5_MOUSE\nBRAF\tQ9JJU6_MOUSE\nBRAF\tQ9L3M5_RHILV\nBRAF\tR4Z1Q7_9ACTN\nBRAF\tS0FSK4_9DELT\nBRAF\tS6ATD8_PSERE\nBRAF\tT2LEG1_9GAMM\nBRAF\tT2MGW2_HYDVU\nBRAF\tU3CMC6_CALJA\nBRAF\tU3J6J7_ANAPP\nBRAF\tU3JMB6_FICAL\nBRAF\tU6D8R7_NEOVI\nBRAF\tV5BES0_9GAMM\nBRAF\tV5ZBH2_9GAMM\nBRAF\tW5PVT2_SHEEP\nBRAF\tW5PVT5_SHEEP\nBRAF\tW6MA55_9GAMM\nBRAF\tW6RIZ4_9RHIZ\nBRAF\tW8YHU3_9BACL\n','From\tTo\nPTK2\tA0A024R9H6_HUMAN\nPTK2\tA0A096MSE0_PAPAN\nPTK2\tA0A0A1WY05_ZEUCU\nPTK2\tA0A0B2V5S5_TOXCA\nPTK2\tA0A0D9RB32_CHLSB\nPTK2\tA0A0G2KAT5_RAT\nPTK2\tA0A146KPZ3_LYGHE\nPTK2\tA0A151P831_ALLMI\nPTK2\tA0A1A7XAH4_9TELE\nPTK2\tA0A1A7XGS7_9TELE\nPTK2\tA0A1A7ZL93_NOTFU\nPTK2\tA0A1A8APA9_NOTFU\nPTK2\tA0A1A8CB38_9TELE\nPTK2\tA0A1A8CZ57_9TELE\nPTK2\tA0A1A8DAQ4_9TELE\nPTK2\tA0A1A8EWD1_9TELE\nPTK2\tA0A1A8EZ10_9TELE\nPTK2\tA0A1A8F225_9TELE\nPTK2\tA0A1A8FSP5_9TELE\nPTK2\tA0A1A8G7D7_9TELE\nPTK2\tA0A1A8HHG7_9TELE\nPTK2\tA0A1A8HLM4_NOTKU\nPTK2\tA0A1A8J0R4_NOTKU\nPTK2\tA0A1A8J4X9_NOTKU\nPTK2\tA0A1A8J9X6_NOTKU\nPTK2\tA0A1A8JXA3_NOTKU\nPTK2\tA0A1A8KAB8_NOTKU\nPTK2\tA0A1A8L1Z5_9TELE\nPTK2\tA0A1A8MTP4_9TELE\nPTK2\tA0A1A8NQK9_9TELE\nPTK2\tA0A1A8NVS5_9TELE\nPTK2\tA0A1A8NZD1_9TELE\nPTK2\tA0A1A8P9G6_9TELE\nPTK2\tA0A1A8PGU4_9TELE\nPTK2\tA0A1A8R493_9TELE\nPTK2\tA0A1A8R6A7_9TELE\nPTK2\tA0A1A8UDY7_NOTFU\nPTK2\tA0A1A8VGU3_NOTFU\nPTK2\tA0A1B2JAS1_PICPA\nPTK2\tA0A1D5NY34_CHICK\nPTK2\tA0A1D5Q106_MACMU\nPTK2\tA0A1D5Q4C1_MACMU\nPTK2\tA0A1D5QCX2_MACMU\nPTK2\tA0A1D5QNW1_MACMU\nPTK2\tA0A1D5QPJ2_MACMU\nPTK2\tA0A1D5QXZ7_MACMU\nPTK2\tA0A1D5RMT1_HUMAN\nPTK2\tA0A1L1RJ83_CHICK\nPTK2\tA0A1S3FW28_DIPOR\nPTK2\tA0A1S3WR70_ERIEU\nPTK2\tA0A1S3WR85_ERIEU\nPTK2\tA0A1S3WR97_ERIEU\nPTK2\tA0A1S3WRA8_ERIEU\nPTK2\tA0A1S3WRF6_ERIEU\nPTK2\tA0A1S3WRG1_ERIEU\nPTK2\tA0A1U7SF59_TARSY\nPTK2\tA0A1U7SLU4_TARSY\nPTK2\tA0A1U7SQQ9_TARSY\nPTK2\tA0A1U7T0R3_TARSY\nPTK2\tA0A1U8DEY6_ALLSI\nPTK2\tA0A1V4L1R8_PATFA\nPTK2\tA0A1V4L315_PATFA\nPTK2\tA0A218UMU6_9PASE\nPTK2\tA0A250Y6S0_CASCN\nPTK2\tA0A286Y189_CAVPO\nPTK2\tA0A286ZL12_PIG\nPTK2\tA0A286ZV94_PIG\nPTK2\tA0A287ABC9_PIG\nPTK2\tA0A287AJL8_PIG\nPTK2\tA0A287ARR4_PIG\nPTK2\tA0A287AYS3_PIG\nPTK2\tA0A287BMF3_PIG\nPTK2\tA0A287BRP4_PIG\nPTK2\tA0A287CYY1_ICTTR\nPTK2\tA0A287D1B4_ICTTR\nPTK2\tA0A287D281_ICTTR\nPTK2\tA0A287D7W7_ICTTR\nPTK2\tA0A287DCH5_ICTTR\nPTK2\tA0A2B4SCX7_STYPI\nPTK2\tA0A2B4SGH0_STYPI\nPTK2\tA0A2D0PSX8_ICTPU\nPTK2\tA0A2D0PSY2_ICTPU\nPTK2\tA0A2D0PUF6_ICTPU\nPTK2\tA0A2D0PUH5_ICTPU\nPTK2\tA0A2D0PW58_ICTPU\nPTK2\tA0A2D0PW63_ICTPU\nPTK2\tA0A2D0PX14_ICTPU\nPTK2\tA0A2D0PX19_ICTPU\nPTK2\tA0A2I0MXT6_COLLI\nPTK2\tA0A2I0MXU2_COLLI\nPTK2\tA0A2I0MXV0_COLLI\nPTK2\tA0A2I2UMM7_FELCA\nPTK2\tA0A2I3BPK4_MOUSE\nPTK2\tA0A2I3BQ85_MOUSE\nPTK2\tA0A2I3BR46_MOUSE\nPTK2\tA0A2I3BRG7_MOUSE\nPTK2\tA0A2I3FUW9_NOMLE\nPTK2\tA0A2I3GSZ4_NOMLE\nPTK2\tA0A2I3GZ87_NOMLE\nPTK2\tA0A2I3HC29_NOMLE\nPTK2\tA0A2I3HC46_NOMLE\nPTK2\tA0A2I3HKH3_NOMLE\nPTK2\tA0A2I3HQ39_NOMLE\nPTK2\tA0A2I3HTC6_NOMLE\nPTK2\tA0A2I3LJL3_PAPAN\nPTK2\tA0A2I3LSB6_PAPAN\nPTK2\tA0A2I3M2V1_PAPAN\nPTK2\tA0A2I3MAW9_PAPAN\nPTK2\tA0A2I3N5G5_PAPAN\nPTK2\tA0A2I3N7B8_PAPAN\nPTK2\tA0A2I3N8K5_PAPAN\nPTK2\tA0A2I3NA25_PAPAN\nPTK2\tA0A2I3NHS8_PAPAN\nPTK2\tA0A2I3RRG3_PANTR\nPTK2\tA0A2I3RTS1_PANTR\nPTK2\tA0A2I3RVX6_PANTR\nPTK2\tA0A2I3SEP3_PANTR\nPTK2\tA0A2I3SGF3_PANTR\nPTK2\tA0A2I3T3L8_PANTR\nPTK2\tA0A2I3TI11_PANTR\nPTK2\tA0A2J8LFB2_PANTR\nPTK2\tA0A2J8LFC5_PANTR\nPTK2\tA0A2K5ETM5_AOTNA\nPTK2\tA0A2K5ETT6_AOTNA\nPTK2\tA0A2K5ETV9_AOTNA\nPTK2\tA0A2K5ETY2_AOTNA\nPTK2\tA0A2K5ETZ8_AOTNA\nPTK2\tA0A2K5EU22_AOTNA\nPTK2\tA0A2K5EU28_AOTNA\nPTK2\tA0A2K5K7L5_COLAP\nPTK2\tA0A2K5K7N9_COLAP\nPTK2\tA0A2K5K7P7_COLAP\nPTK2\tA0A2K5K7Q8_COLAP\nPTK2\tA0A2K5K7R2_COLAP\nPTK2\tA0A2K5K7S1_COLAP\nPTK2\tA0A2K5K7S7_COLAP\nPTK2\tA0A2K5K7S8_COLAP\nPTK2\tA0A2K5K7T6_COLAP\nPTK2\tA0A2K5K7U7_COLAP\nPTK2\tA0A2K5KWQ4_CERAT\nPTK2\tA0A2K5KWT9_CERAT\nPTK2\tA0A2K5KWU6_CERAT\nPTK2\tA0A2K5KWU7_CERAT\nPTK2\tA0A2K5KWV2_CERAT\nPTK2\tA0A2K5KWZ5_CERAT\nPTK2\tA0A2K5QKM0_CEBCA\nPTK2\tA0A2K5QL11_CEBCA\nPTK2\tA0A2K5QL17_CEBCA\nPTK2\tA0A2K5QL70_CEBCA\nPTK2\tA0A2K5QL81_CEBCA\nPTK2\tA0A2K5QL95_CEBCA\nPTK2\tA0A2K5QL99_CEBCA\nPTK2\tA0A2K5QLB3_CEBCA\nPTK2\tA0A2K5QLD1_CEBCA\nPTK2\tA0A2K5TUX1_MACFA\nPTK2\tA0A2K5TUZ7_MACFA\nPTK2\tA0A2K5TV46_MACFA\nPTK2\tA0A2K5TV55_MACFA\nPTK2\tA0A2K5TV65_MACFA\nPTK2\tA0A2K5TV93_MACFA\nPTK2\tA0A2K5TV97_MACFA\nPTK2\tA0A2K5TVB8_MACFA\nPTK2\tA0A2K5TVC7_MACFA\nPTK2\tA0A2K5TVD4_MACFA\nPTK2\tA0A2K5YHF7_MANLE\nPTK2\tA0A2K5YHS4_MANLE\nPTK2\tA0A2K5YHV5_MANLE\nPTK2\tA0A2K5YHW0_MANLE\nPTK2\tA0A2K5YHX5_MANLE\nPTK2\tA0A2K5YHY1_MANLE\nPTK2\tA0A2K5YHY5_MANLE\nPTK2\tA0A2K5YHZ8_MANLE\nPTK2\tA0A2K5YHZ9_MANLE\nPTK2\tA0A2K6CLS5_MACNE\nPTK2\tA0A2K6CLS8_MACNE\nPTK2\tA0A2K6CLV4_MACNE\nPTK2\tA0A2K6CLW5_MACNE\nPTK2\tA0A2K6CLX3_MACNE\nPTK2\tA0A2K6CLY9_MACNE\nPTK2\tA0A2K6GHH6_PROCO\nPTK2\tA0A2K6GHI3_PROCO\nPTK2\tA0A2K6GHI6_PROCO\nPTK2\tA0A2K6KG32_RHIBE\nPTK2\tA0A2K6KG47_RHIBE\nPTK2\tA0A2K6KG60_RHIBE\nPTK2\tA0A2K6QVN3_RHIRO\nPTK2\tA0A2K6QVP3_RHIRO\nPTK2\tA0A2K6QVQ0_RHIRO\nPTK2\tA0A2K6QVQ1_RHIRO\nPTK2\tA0A2K6QVS2_RHIRO\nPTK2\tA0A2K6QVS8_RHIRO\nPTK2\tA0A2K6QVT5_RHIRO\nPTK2\tA0A2K6QVT7_RHIRO\nPTK2\tA0A2K6QVU4_RHIRO\nPTK2\tA0A2K6QVV4_RHIRO\nPTK2\tA0A2K6SPG9_SAIBB\nPTK2\tA0A2K6SPI2_SAIBB\nPTK2\tA0A2K6SPJ4_SAIBB\nPTK2\tA0A2K6SPL7_SAIBB\nPTK2\tA0A2R8MA85_CALJA\nPTK2\tA0A2R8MCW1_CALJA\nPTK2\tA0A2R8MME2_CALJA\nPTK2\tA0A2R8NCX4_CALJA\nPTK2\tA0A2R9CGB2_PANPA\nPTK2\tA0A2R9CGD9_PANPA\nPTK2\tA0A2R9CGE9_PANPA\nPTK2\tA0A2R9CGF4_PANPA\nPTK2\tA0A2R9CIZ6_PANPA\nPTK2\tA0A2R9CJ36_PANPA\nPTK2\tA0A2R9CJ54_PANPA\nPTK2\tA0A2R9CQL0_PANPA\nPTK2\tA0A2R9CQL8_PANPA\nPTK2\tA0A2S2Q285_9HEMI\nPTK2\tA0A2U3WB52_ODORO\nPTK2\tA0A2U3ZKY1_ODORO\nPTK2\tA0A2U3ZL01_ODORO\nPTK2\tA0A2U4B2R0_TURTR\nPTK2\tA0A2U4B2R3_TURTR\nPTK2\tA0A2U4B2T9_TURTR\nPTK2\tA0A2U4B2W0_TURTR\nPTK2\tA0A2U4B2W5_TURTR\nPTK2\tA0A2U4B367_TURTR\nPTK2\tA0A2Y9GYR9_NEOSC\nPTK2\tA0A2Y9GYS3_NEOSC\nPTK2\tA0A2Y9GZ26_NEOSC\nPTK2\tA0A2Y9GZ42_NEOSC\nPTK2\tA0A2Y9H3E3_NEOSC\nPTK2\tA0A2Y9H3E8_NEOSC\nPTK2\tA0A2Y9LMK4_DELLE\nPTK2\tA0A2Y9LML0_DELLE\nPTK2\tA0A2Y9LML5_DELLE\nPTK2\tA0A2Y9LMM6_DELLE\nPTK2\tA0A2Y9LNE3_DELLE\nPTK2\tA0A2Y9LNE8_DELLE\nPTK2\tA0A2Y9LNF3_DELLE\nPTK2\tA0A2Y9LQD3_DELLE\nPTK2\tA0A2Y9LQD9_DELLE\nPTK2\tA0A2Y9LQF5_DELLE\nPTK2\tA0A2Y9LV66_DELLE\nPTK2\tA0A2Y9LV71_DELLE\nPTK2\tA0A2Y9LV88_DELLE\nPTK2\tA0A2Y9M0M0_DELLE\nPTK2\tA0A2Y9M0N1_DELLE\nPTK2\tA0A2Y9M0P3_DELLE\nPTK2\tA0A337SM86_FELCA\nPTK2\tA0A340XZH0_LIPVE\nPTK2\tA0A341B1X0_NEOAA\nPTK2\tA0A341B1X4_NEOAA\nPTK2\tA0A341B204_NEOAA\nPTK2\tA0A341B4Y1_NEOAA\nPTK2\tA0A341B4Y6_NEOAA\nPTK2\tA0A341B5E7_NEOAA\nPTK2\tA0A383YVP0_BALAS\nPTK2\tA0A383YVP6_BALAS\nPTK2\tA0A383YVP9_BALAS\nPTK2\tA0A383YVQ4_BALAS\nPTK2\tA0A383YVQ6_BALAS\nPTK2\tA0A383YVR0_BALAS\nPTK2\tA0A383YVS2_BALAS\nPTK2\tA0A383YVT1_BALAS\nPTK2\tA0A383YVX3_BALAS\nPTK2\tA0A383YVX9_BALAS\nPTK2\tA0A383YVY4_BALAS\nPTK2\tA0A383YW87_BALAS\nPTK2\tA0A383YW96_BALAS\nPTK2\tA0A384CR20_URSMA\nPTK2\tA0A384CR38_URSMA\nPTK2\tA0A3B3HI75_ORYLA\nPTK2\tA0A3B3HJF9_ORYLA\nPTK2\tA0A3B3HWW7_ORYLA\nPTK2\tA0A3B3I4L7_ORYLA\nPTK2\tA0A3B3IHW7_ORYLA\nPTK2\tA0A3B3IP67_ORYLA\nPTK2\tA0A3B4Z483_9TELE\nPTK2\tA0A3P8PMM4_ASTCA\nPTK2\tA0A3P8PMS4_ASTCA\nPTK2\tA0A3P8PMX4_ASTCA\nPTK2\tA0A3P8PN80_ASTCA\nPTK2\tA0A3P8S4X0_AMPPE\nPTK2\tA0A3P8S594_AMPPE\nPTK2\tA0A3P8S5A9_AMPPE\nPTK2\tA0A3P8S5F2_AMPPE\nPTK2\tA0A3P9DLY2_9CICH\nPTK2\tA0A3P9DLY7_9CICH\nPTK2\tA0A3P9DNI2_9CICH\nPTK2\tA0A3P9DNR5_9CICH\nPTK2\tA0A3Q0DFW2_TARSY\nPTK2\tA0A3Q0DGC2_MESAU\nPTK2\tA0A3Q0DK65_TARSY\nPTK2\tA0A3Q0DK70_TARSY\nPTK2\tA0A3Q0DLR0_TARSY\nPTK2\tA0A3Q0DMJ9_TARSY\nPTK2\tA0A3Q0DMK4_TARSY\nPTK2\tA0A3Q0DNI0_TARSY\nPTK2\tA0A3Q0HEE0_ALLSI\nPTK2\tA0A3Q0HEG4_ALLSI\nPTK2\tA0A3Q0HEI6_ALLSI\nPTK2\tA0A3Q0HFB9_ALLSI\nPTK2\tA0A3Q0HFC3_ALLSI\nPTK2\tA0A3Q0HFD2_ALLSI\nPTK2\tA0A3Q0HFE2_ALLSI\nPTK2\tA0A3Q0HFF2_ALLSI\nPTK2\tA0A3Q0HII6_ALLSI\nPTK2\tA0A3Q0HIL7_ALLSI\nPTK2\tA0A3Q0HIQ6_ALLSI\nPTK2\tA0A3Q0HIS7_ALLSI\nPTK2\tA0A3Q0HIT1_ALLSI\nPTK2\tA0A3Q0HIT8_ALLSI\nPTK2\tA0A3Q0SMX2_AMPCI\nPTK2\tA0A3Q1LMR7_BOVIN\nPTK2\tA0A3Q1LX79_BOVIN\nPTK2\tA0A3Q1MAA1_BOVIN\nPTK2\tA0A3Q1N508_BOVIN\nPTK2\tA0A3Q2CAV5_CYPVA\nPTK2\tA0A3Q2DN61_CYPVA\nPTK2\tA0A3Q2DNP7_CYPVA\nPTK2\tA0A3Q2DP06_CYPVA\nPTK2\tA0A3Q2GA07_CYPVA\nPTK2\tA0A3Q2GWL9_HORSE\nPTK2\tA0A3Q2GZB7_HORSE\nPTK2\tA0A3Q2I072_HORSE\nPTK2\tA0A3Q2KJP8_HORSE\nPTK2\tA0A3Q2U9D2_CHICK\nPTK2\tA0A3Q2UBF5_CHICK\nPTK2\tA0A3Q3MIG1_9TELE\nPTK2\tA0A3Q3XK44_MOLML\nPTK2\tA0A3Q7P0B3_CALUR\nPTK2\tA0A3Q7P9R0_CALUR\nPTK2\tA0A3Q7P9R4_CALUR\nPTK2\tA0A3Q7QFL4_CALUR\nPTK2\tA0A3Q7QFL9_CALUR\nPTK2\tA0A3Q7R3M1_VULVU\nPTK2\tA0A3Q7R3M6_VULVU\nPTK2\tA0A3Q7R5R6_VULVU\nPTK2\tA0A3Q7R5S1_VULVU\nPTK2\tA0A3Q7R663_VULVU\nPTK2\tA0A3Q7R668_VULVU\nPTK2\tA0A3Q7RFB1_VULVU\nPTK2\tA0A3Q7RFB6_VULVU\nPTK2\tA0A3Q7RFC1_VULVU\nPTK2\tA0A3Q7SGL9_VULVU\nPTK2\tA0A3Q7SGM4_VULVU\nPTK2\tA0A3Q7UBU7_URSAR\nPTK2\tA0A3Q7UBV1_URSAR\nPTK2\tA0A3Q7V1I8_URSAR\nPTK2\tA0A3Q7V1J4_URSAR\nPTK2\tA0A3Q7V9S7_URSAR\nPTK2\tA0A3Q7VSP6_URSAR\nPTK2\tA0A3Q7VSQ3_URSAR\nPTK2\tA0A3Q7W5Y5_URSAR\nPTK2\tA0A452C378_BALAS\nPTK2\tA0A452C3C2_BALAS\nPTK2\tA0A452C3D3_BALAS\nPTK2\tA0A452C3D8_BALAS\nPTK2\tA0A452EIX2_CAPHI\nPTK2\tA0A452EIY2_CAPHI\nPTK2\tA0A452EIZ7_CAPHI\nPTK2\tA0A452EJE2_CAPHI\nPTK2\tA0A452EJF7_CAPHI\nPTK2\tA0A452EJG8_CAPHI\nPTK2\tA0A452EJI4_CAPHI\nPTK2\tA0A452EJL3_CAPHI\nPTK2\tA0A452EJY1_CAPHI\nPTK2\tA0A452GJT3_9SAUR\nPTK2\tA0A452GJW0_9SAUR\nPTK2\tA0A452GK20_9SAUR\nPTK2\tA0A452GK54_9SAUR\nPTK2\tA0A452GK61_9SAUR\nPTK2\tA0A452GK66_9SAUR\nPTK2\tA0A452V2W1_URSMA\nPTK2\tA0A452V381_URSMA\nPTK2\tA0A455C3Y9_PHYMC\nPTK2\tA0A455C3Z5_PHYMC\nPTK2\tA0A455C3Z9_PHYMC\nPTK2\tA0A455C474_PHYMC\nPTK2\tA0A455C479_PHYMC\nPTK2\tA0A455C484_PHYMC\nPTK2\tA0A455C503_PHYMC\nPTK2\tA0A455C508_PHYMC\nPTK2\tA0A455C513_PHYMC\nPTK2\tA0A455C518_PHYMC\nPTK2\tA0A455C876_PHYMC\nPTK2\tA0A455C882_PHYMC\nPTK2\tA0A455C885_PHYMC\nPTK2\tA0A455CBB3_PHYMC\nPTK2\tA0A455CBB4_PHYMC\nPTK2\tA0A455CBB5_PHYMC\nPTK2\tA0A455CBB6_PHYMC\nPTK2\tA0A455CBB7_PHYMC\nPTK2\tA0A4C1WA78_9NEOP\nPTK2\tA0A4C2EAV4_9SACH\nPTK2\tA0A4W2C3E4_BOBOX\nPTK2\tA0A4W2C3H1_BOBOX\nPTK2\tA0A4W2C609_BOBOX\nPTK2\tA0A4W2CLS7_BOBOX\nPTK2\tA0A4W2CLW0_BOBOX\nPTK2\tA0A4W2CLY5_BOBOX\nPTK2\tA0A4W2E2U0_BOBOX\nPTK2\tA0A4W2EG47_BOBOX\nPTK2\tA0A4W2IL06_BOBOX\nPTK2\tA0A4W2IL60_BOBOX\nPTK2\tA0A4W2IP59_BOBOX\nPTK2\tA0A4W2IP78_BOBOX\nPTK2\tA0A4W2IPV1_BOBOX\nPTK2\tA0A4W2IQ00_BOBOX\nPTK2\tA0A4W2ISV2_BOBOX\nPTK2\tA0A4W3J6Q7_CALMI\nPTK2\tA0A4W3J6T6_CALMI\nPTK2\tA0A4W3J9M2_CALMI\nPTK2\tA0A4W3JHG8_CALMI\nPTK2\tA0A4W3JHI4_CALMI\nPTK2\tA0A4W3JHJ3_CALMI\nPTK2\tA0A4W3JHJ8_CALMI\nPTK2\tA0A4W3JLF2_CALMI\nPTK2\tA0A4W3JLG8_CALMI\nPTK2\tA0A4W6DU76_LATCA\nPTK2\tA0A4W6DUK0_LATCA\nPTK2\tA0A4W6DUM2_LATCA\nPTK2\tA0A4W6DVB8_LATCA\nPTK2\tA0A4W6DVK5_LATCA\nPTK2\tA0A4W6DWN0_LATCA\nPTK2\tA0A4W6DWV9_LATCA\nPTK2\tA0A4W6DX24_LATCA\nPTK2\tA0A4W6DXP8_LATCA\nPTK2\tA0A4W6DXY4_LATCA\nPTK2\tA0A4X1SKF3_PIG\nPTK2\tA0A4X1SKI8_PIG\nPTK2\tA0A4X1SKN5_PIG\nPTK2\tA0A4X1SKR4_PIG\nPTK2\tA0A4X1SKR6_PIG\nPTK2\tA0A4X1SKT6_PIG\nPTK2\tA0A4X1SLM3_PIG\nPTK2\tA0A4X1SLV5_PIG\nPTK2\tA0A4X1SLX4_PIG\nPTK2\tA0A5F4BYH1_CANLF\nPTK2\tA0A5F4C7F6_CANLF\nPTK2\tA0A5F4CC25_CANLF\nPTK2\tA0A5F4CCJ5_CANLF\nPTK2\tA0A5F4CFI8_CANLF\nPTK2\tA0A5F4CKI5_CANLF\nPTK2\tA0A5F4CSN8_CANLF\nPTK2\tA0A5F4DI69_CANLF\nPTK2\tA0A5F4VZ48_CALJA\nPTK2\tA0A5F5PXV7_HORSE\nPTK2\tA0A5F5XMC4_FELCA\nPTK2\tA0A5F5XNV6_FELCA\nPTK2\tA0A5F5Y0K0_FELCA\nPTK2\tA0A5F5Y0W4_FELCA\nPTK2\tA0A5F8MPY3_MOUSE\nPTK2\tA0A5G3LCX9_XENTR\nPTK2\tA0A5G3LDI3_XENTR\nPTK2\tA0A5G3LGJ3_XENTR\nPTK2\tA0A5G3LGJ8_XENTR\nPTK2\tA6ZQ18_YEAS7\nPTK2\tB4DWJ1_HUMAN\nPTK2\tB7YZL9_DROME\nPTK2\tB7ZQN0_XENLA\nPTK2\tC7GME4_YEAS2\nPTK2\tE1JGM8_DROME\nPTK2\tE2RGP8_CANLF\nPTK2\tE2RGQ1_CANLF\nPTK2\tE5RFW9_HUMAN\nPTK2\tE5RG54_HUMAN\nPTK2\tE5RG66_HUMAN\nPTK2\tE5RG80_HUMAN\nPTK2\tE5RG86_HUMAN\nPTK2\tE5RGA6_HUMAN\nPTK2\tE5RGP1_HUMAN\nPTK2\tE5RH01_HUMAN\nPTK2\tE5RH08_HUMAN\nPTK2\tE5RH48_HUMAN\nPTK2\tE5RHD8_HUMAN\nPTK2\tE5RHK7_HUMAN\nPTK2\tE5RI03_HUMAN\nPTK2\tE5RI29_HUMAN\nPTK2\tE5RI72_HUMAN\nPTK2\tE5RII9_HUMAN\nPTK2\tE5RIK4_HUMAN\nPTK2\tE5RIR5_HUMAN\nPTK2\tE5RJI4_HUMAN\nPTK2\tE5RJN1_HUMAN\nPTK2\tE5RJP0_HUMAN\nPTK2\tE5RJQ2_HUMAN\nPTK2\tE7ESA6_HUMAN\nPTK2\tE9PEI4_HUMAN\nPTK2\tF1MS29_BOVIN\nPTK2\tF2QQQ6_KOMPC\nPTK2\tF6PRI2_CALJA\nPTK2\tF6PVK9_ORNAN\nPTK2\tF6TPE2_CALJA\nPTK2\tF6U0X2_CALJA\nPTK2\tF6U131_CALJA\nPTK2\tF6U9I3_HORSE\nPTK2\tF6W8H3_XENTR\nPTK2\tF7C660_MACMU\nPTK2\tF7CW18_XENTR\nPTK2\tF7CW93_XENTR\nPTK2\tFAK1_CHICK\nPTK2\tFAK1_HUMAN\nPTK2\tFAK1_MOUSE\nPTK2\tFAK1_RAT\nPTK2\tFAK1_XENLA\nPTK2\tG1K8J0_ANOCA\nPTK2\tG1LET6_AILME\nPTK2\tG1NJW1_MELGA\nPTK2\tG1NJW4_MELGA\nPTK2\tG1PH78_MYOLU\nPTK2\tG1QLM0_NOMLE\nPTK2\tG3N3B7_BOVIN\nPTK2\tG3SZP4_LOXAF\nPTK2\tG3TX03_LOXAF\nPTK2\tG3UAS6_LOXAF\nPTK2\tG3UCQ3_LOXAF\nPTK2\tG3UMP3_LOXAF\nPTK2\tG3WC11_SARHA\nPTK2\tH0UV17_CAVPO\nPTK2\tH0WZ74_OTOGA\nPTK2\tH0YAS0_HUMAN\nPTK2\tH0YB16_HUMAN\nPTK2\tH0YB33_HUMAN\nPTK2\tH0YB99_HUMAN\nPTK2\tH0YBP1_HUMAN\nPTK2\tH0YBZ1_HUMAN\nPTK2\tH0ZR18_TAEGU\nPTK2\tH2LIZ5_ORYLA\nPTK2\tH2PRA3_PONAB\nPTK2\tH2T0L4_TAKRU\nPTK2\tH2TU31_TAKRU\nPTK2\tH2TU32_TAKRU\nPTK2\tH2TU33_TAKRU\nPTK2\tH2TU34_TAKRU\nPTK2\tH3ASV0_LATCH\nPTK2\tI3MKF9_ICTTR\nPTK2\tI6L996_HUMAN\nPTK2\tK7F7V1_PELSI\nPTK2\tK7Q751_MOUSE\nPTK2\tM3XS70_MUSPF\nPTK2\tM3Z9U8_NOMLE\nPTK2\tPTK2_CANGA\nPTK2\tPTK2_YEAST\nPTK2\tQ0E917_DROME\nPTK2\tQ0VC86_BOVIN\nPTK2\tQ28CP6_XENTR\nPTK2\tQ2VQT9_MOUSE\nPTK2\tQ5A4M7_CANAL\nPTK2\tQ6IR54_XENLA\nPTK2\tQ7YRB1_BOVIN\nPTK2\tQ9FY04_POPPZ\nPTK2\tR4GCP5_ANOCA\nPTK2\tT2MDJ8_HYDVU\nPTK2\tU3JJM0_FICAL\nPTK2\tV8NDK2_OPHHA\nPTK2\tV9K9P6_CALMI\nPTK2\tW0TDK5_KLUMD\nPTK2\tW5NZQ9_SHEEP\n','From\tTo\nJAK1\tA0A096N179_PAPAN\nJAK1\tA0A0A0N0M2_HUMAN\nJAK1\tA0A0A9WJ71_LYGHE\nJAK1\tA0A0D6A7L4_CARAU\nJAK1\tA0A0D6A7N3_CARCW\nJAK1\tA0A0D6A7U5_CARAU\nJAK1\tA0A0D6A8P3_CYPCA\nJAK1\tA0A0D9S727_CHLSB\nJAK1\tA0A0K6FUT0_9AGAM\nJAK1\tA0A0P6J982_HETGA\nJAK1\tA0A0S7JBS4_9TELE\nJAK1\tA0A0S7JBT3_9TELE\nJAK1\tA0A0S7JBT8_9TELE\nJAK1\tA0A0S7JBU1_9TELE\nJAK1\tA0A0S7JBU3_9TELE\nJAK1\tA0A0S7JBU6_9TELE\nJAK1\tA0A0S7JBV1_9TELE\nJAK1\tA0A0S7JBV4_9TELE\nJAK1\tA0A0S7JBV6_9TELE\nJAK1\tA0A0S7JBV7_9TELE\nJAK1\tA0A0S7JBW0_9TELE\nJAK1\tA0A0S7JBW5_9TELE\nJAK1\tA0A0S7JBX6_9TELE\nJAK1\tA0A0S7JC64_9TELE\nJAK1\tA0A0S7JDM9_9TELE\nJAK1\tA0A0S7JDQ3_9TELE\nJAK1\tA0A0S7JDQ4_9TELE\nJAK1\tA0A0S7JDR1_9TELE\nJAK1\tA0A0S7JE36_9TELE\nJAK1\tA0A0S7JE51_9TELE\nJAK1\tA0A0S7JE78_9TELE\nJAK1\tA0A0S7JE95_9TELE\nJAK1\tA0A0S7JEH6_9TELE\nJAK1\tA0A0S7JEV7_9TELE\nJAK1\tA0A0S7JEX2_9TELE\nJAK1\tA0A0U2EU06_TACFU\nJAK1\tA0A140DJC4_CTEID\nJAK1\tA0A1A7WE33_9TELE\nJAK1\tA0A1A8AN86_NOTFU\nJAK1\tA0A1A8BYW1_9TELE\nJAK1\tA0A1A8FV37_9TELE\nJAK1\tA0A1A8GLZ1_9TELE\nJAK1\tA0A1A8I918_NOTKU\nJAK1\tA0A1A8KG15_NOTKU\nJAK1\tA0A1A8QVV4_9TELE\nJAK1\tA0A1A8QYA3_9TELE\nJAK1\tA0A1A8RB25_9TELE\nJAK1\tA0A1S3FJH6_DIPOR\nJAK1\tA0A1S3KKN4_SALSA\nJAK1\tA0A1S3WAF8_ERIEU\nJAK1\tA0A1U7R5X2_MESAU\nJAK1\tA0A1U7RXD8_ALLSI\nJAK1\tA0A1U7U068_TARSY\nJAK1\tA0A1V4JXL8_PATFA\nJAK1\tA0A1Z1BJL5_9GOBI\nJAK1\tA0A218UJX7_9PASE\nJAK1\tA0A250Y724_CASCN\nJAK1\tA0A286ZLI2_PIG\nJAK1\tA0A2D0RWX2_ICTPU\nJAK1\tA0A2I0MC95_COLLI\nJAK1\tA0A2I2U8B7_FELCA\nJAK1\tA0A2I3LGP0_PAPAN\nJAK1\tA0A2I3LH42_PAPAN\nJAK1\tA0A2I3RU79_PANTR\nJAK1\tA0A2I4BPV4_9TELE\nJAK1\tA0A2K5EZ81_AOTNA\nJAK1\tA0A2K5EZD2_AOTNA\nJAK1\tA0A2K5IGU5_COLAP\nJAK1\tA0A2K5IHF1_COLAP\nJAK1\tA0A2K5NH39_CERAT\nJAK1\tA0A2K5NHQ0_CERAT\nJAK1\tA0A2K5PWB1_CEBCA\nJAK1\tA0A2K5WAG8_MACFA\nJAK1\tA0A2K5WAH4_MACFA\nJAK1\tA0A2K5ZKJ6_MANLE\nJAK1\tA0A2K5ZKJ8_MANLE\nJAK1\tA0A2K6ARQ3_MACNE\nJAK1\tA0A2K6ARR6_MACNE\nJAK1\tA0A2K6FGU0_PROCO\nJAK1\tA0A2K6JTR0_RHIBE\nJAK1\tA0A2K6JTR6_RHIBE\nJAK1\tA0A2K6RHP9_RHIRO\nJAK1\tA0A2K6T879_SAIBB\nJAK1\tA0A2R8ZF74_PANPA\nJAK1\tA0A2U3ZSA9_ODORO\nJAK1\tA0A2U4A1Y4_TURTR\nJAK1\tA0A2Y9G9Q3_NEOSC\nJAK1\tA0A2Y9GDB3_NEOSC\nJAK1\tA0A2Y9P436_DELLE\nJAK1\tA0A2Y9SI10_PHYMC\nJAK1\tA0A340XM97_LIPVE\nJAK1\tA0A341D0I0_NEOAA\nJAK1\tA0A383YNH1_BALAS\nJAK1\tA0A384C3J0_URSMA\nJAK1\tA0A3Q2H1C4_HORSE\nJAK1\tA0A3Q2HL76_HORSE\nJAK1\tA0A3Q7P1J2_CALUR\nJAK1\tA0A3Q7RA95_VULVU\nJAK1\tA0A3Q7VNC1_URSAR\nJAK1\tA0A452C7C9_BALAS\nJAK1\tA0A452EDI9_CAPHI\nJAK1\tA0A452ID23_9SAUR\nJAK1\tA0A452R045_URSAM\nJAK1\tA0A452TSS6_URSMA\nJAK1\tA0A4W2DG60_BOBOX\nJAK1\tA0A4W2GNE8_BOBOX\nJAK1\tA0A4W6D9H2_LATCA\nJAK1\tA0A4W6D9I3_LATCA\nJAK1\tA0A4W6D9P7_LATCA\nJAK1\tA0A4W6D9S5_LATCA\nJAK1\tA0A4W6DA05_LATCA\nJAK1\tA0A4W6DA50_LATCA\nJAK1\tA0A4W6DA55_LATCA\nJAK1\tA0A4W6DA82_LATCA\nJAK1\tA0A4W6DA83_LATCA\nJAK1\tA0A4W6DA93_LATCA\nJAK1\tA0A4X1TFJ2_PIG\nJAK1\tA0A4X1TJT5_PIG\nJAK1\tA0A4Y5R7S0_ANAPL\nJAK1\tA0A5F7ZH35_MACMU\nJAK1\tA0A5F9ZH07_HUMAN\nJAK1\tA0A5F9ZH32_HUMAN\nJAK1\tA0A5F9ZH73_HUMAN\nJAK1\tA0A5F9ZHI1_HUMAN\nJAK1\tA0A5F9ZHK2_HUMAN\nJAK1\tA0A5F9ZHN8_HUMAN\nJAK1\tA0A5F9ZHW0_HUMAN\nJAK1\tA0A5F9ZI01_HUMAN\nJAK1\tA0A5F9ZI39_HUMAN\nJAK1\tA0A5G3KKZ2_XENTR\nJAK1\tA0A5G3KLM9_XENTR\nJAK1\tA0A5G3KM05_XENTR\nJAK1\tA0A5G3KN84_XENTR\nJAK1\tA0A5G3KS75_XENTR\nJAK1\tA0A5G3KS90_XENTR\nJAK1\tA0A5G3KSE1_XENTR\nJAK1\tA0A5G3KSS8_XENTR\nJAK1\tA0A5K1UN95_CALJA\nJAK1\tA3KMY9_BOVIN\nJAK1\tB1ASP2_MOUSE\nJAK1\tB1ASP3_MOUSE\nJAK1\tB9EQC7_SALSA\nJAK1\tC0SPC5_DANRE\nJAK1\tF1N0D6_BOVIN\nJAK1\tF1NMJ9_CHICK\nJAK1\tF1PIY7_CANLF\nJAK1\tF1R5C7_DANRE\nJAK1\tF4YD17_BUBBU\nJAK1\tF6REZ0_XENTR\nJAK1\tF6T0H5_XENTR\nJAK1\tF6TP44_XENTR\nJAK1\tF6YC06_XENTR\nJAK1\tF6YLX1_MACMU\nJAK1\tF7BLF9_HORSE\nJAK1\tF7BLR3_HORSE\nJAK1\tF7C806_XENTR\nJAK1\tF7DKS0_XENTR\nJAK1\tF7EBP4_XENTR\nJAK1\tG1KI54_ANOCA\nJAK1\tG1M9G8_AILME\nJAK1\tG1NGZ6_MELGA\nJAK1\tG1P9Z2_MYOLU\nJAK1\tG1RYT5_NOMLE\nJAK1\tG1SQL4_RABIT\nJAK1\tG3SSY6_LOXAF\nJAK1\tG3U018_LOXAF\nJAK1\tG3V9W2_RAT\nJAK1\tG3WN84_SARHA\nJAK1\tH0V6S7_CAVPO\nJAK1\tH0X486_OTOGA\nJAK1\tH0ZIA6_TAEGU\nJAK1\tH2M2I1_ORYLA\nJAK1\tH2N755_PONAB\nJAK1\tH2PZ66_PANTR\nJAK1\tH2VE39_TAKRU\nJAK1\tH3ADK6_LATCH\nJAK1\tI3MJ48_ICTTR\nJAK1\tI3N6E8_ICTTR\nJAK1\tJ9P349_CANLF\nJAK1\tJAK1_CYPCA\nJAK1\tJAK1_DANRE\nJAK1\tJAK1_HUMAN\nJAK1\tJAK1_MOUSE\nJAK1\tK7FRF6_PELSI\nJAK1\tK7FRH1_PELSI\nJAK1\tM3VXL7_FELCA\nJAK1\tM3XNR7_MUSPF\nJAK1\tM4XY31_9CARN\nJAK1\tM4XYK0_9CARN\nJAK1\tM4Y1N1_ICTST\nJAK1\tM4YHN8_GALVI\nJAK1\tO35803_RAT\nJAK1\tO57612_TETFL\nJAK1\tO62756_PIG\nJAK1\tQ0IIC9_BOVIN\nJAK1\tQ0VGA4_HUMAN\nJAK1\tQ3TI10_MOUSE\nJAK1\tQ3U8P8_MOUSE\nJAK1\tQ3URU8_MOUSE\nJAK1\tQ3UZ30_MOUSE\nJAK1\tQ4G3Y7_MACMU\nJAK1\tQ4LDX3_HUMAN\nJAK1\tQ5FVF5_RAT\nJAK1\tQ6DDJ0_XENLA\nJAK1\tQ6P669_HUMAN\nJAK1\tQ7ZU16_DANRE\nJAK1\tQ8CFX4_MOUSE\nJAK1\tQ8IV82_HUMAN\nJAK1\tQ8K0I7_MOUSE\nJAK1\tQ9PWM9_CHICK\nJAK1\tQ9TTJ1_PIG\nJAK1\tQ9YI47_CHICK\nJAK1\tU3BR01_CALJA\nJAK1\tU3IXD0_ANAPP\nJAK1\tU3KDU1_FICAL\nJAK1\tU6D683_NEOVI\nJAK1\tV8P909_OPHHA\nJAK1\tW5PK80_SHEEP\nJAK1\tW5UBW1_ICTPU\n','From\tTo\nPTK2B\tA0A096MP48_PAPAN\nPTK2B\tA0A096MXC3_PAPAN\nPTK2B\tA0A0D9RTY7_CHLSB\nPTK2B\tA0A1D5RKE2_MACMU\nPTK2B\tA0A1L8G5K3_XENLA\nPTK2B\tA0A1L8GC06_XENLA\nPTK2B\tA0A1S2ZCM2_ERIEU\nPTK2B\tA0A1S2ZCN0_ERIEU\nPTK2B\tA0A1S3FT05_DIPOR\nPTK2B\tA0A1S3FTK0_DIPOR\nPTK2B\tA0A1S3M7G7_SALSA\nPTK2B\tA0A1S3W449_ERIEU\nPTK2B\tA0A1U7QB92_MESAU\nPTK2B\tA0A1U7T5K8_TARSY\nPTK2B\tA0A1U8C2V3_MESAU\nPTK2B\tA0A1U8DNY3_ALLSI\nPTK2B\tA0A1V4J815_PATFA\nPTK2B\tA0A1V4J976_PATFA\nPTK2B\tA0A218UD32_9PASE\nPTK2B\tA0A286XZ23_CAVPO\nPTK2B\tA0A287AR80_PIG\nPTK2B\tA0A287BQ84_PIG\nPTK2B\tA0A287CWT1_ICTTR\nPTK2B\tA0A287D9Z9_ICTTR\nPTK2B\tA0A2I0LJD1_COLLI\nPTK2B\tA0A2I2ULP1_FELCA\nPTK2B\tA0A2I2UXW4_FELCA\nPTK2B\tA0A2I3LKF1_PAPAN\nPTK2B\tA0A2I3MDU3_PAPAN\nPTK2B\tA0A2I3MGM9_PAPAN\nPTK2B\tA0A2I3NES9_PAPAN\nPTK2B\tA0A2I3RNZ8_PANTR\nPTK2B\tA0A2I3RQJ7_PANTR\nPTK2B\tA0A2I3S7W0_PANTR\nPTK2B\tA0A2I3SUP1_PANTR\nPTK2B\tA0A2I4C4K1_9TELE\nPTK2B\tA0A2I4C4K2_9TELE\nPTK2B\tA0A2J8X527_PONAB\nPTK2B\tA0A2K5EIQ5_AOTNA\nPTK2B\tA0A2K5EIS3_AOTNA\nPTK2B\tA0A2K5EIU1_AOTNA\nPTK2B\tA0A2K5I3N2_COLAP\nPTK2B\tA0A2K5I3P7_COLAP\nPTK2B\tA0A2K5I3Q6_COLAP\nPTK2B\tA0A2K5LZR9_CERAT\nPTK2B\tA0A2K5LZU5_CERAT\nPTK2B\tA0A2K5LZW3_CERAT\nPTK2B\tA0A2K5LZX3_CERAT\nPTK2B\tA0A2K5M002_CERAT\nPTK2B\tA0A2K5QQZ8_CEBCA\nPTK2B\tA0A2K5QR89_CEBCA\nPTK2B\tA0A2K5QR98_CEBCA\nPTK2B\tA0A2K5QRC1_CEBCA\nPTK2B\tA0A2K5QRE5_CEBCA\nPTK2B\tA0A2K5QRH7_CEBCA\nPTK2B\tA0A2K5WCI3_MACFA\nPTK2B\tA0A2K5WCJ3_MACFA\nPTK2B\tA0A2K5WCP1_MACFA\nPTK2B\tA0A2K5WCP6_MACFA\nPTK2B\tA0A2K5WCQ3_MACFA\nPTK2B\tA0A2K5YE83_MANLE\nPTK2B\tA0A2K5YE87_MANLE\nPTK2B\tA0A2K5YE88_MANLE\nPTK2B\tA0A2K5YEA8_MANLE\nPTK2B\tA0A2K5YEB8_MANLE\nPTK2B\tA0A2K6C3W0_MACNE\nPTK2B\tA0A2K6C3Y3_MACNE\nPTK2B\tA0A2K6C417_MACNE\nPTK2B\tA0A2K6C467_MACNE\nPTK2B\tA0A2K6C470_MACNE\nPTK2B\tA0A2K6C491_MACNE\nPTK2B\tA0A2K6GNX7_PROCO\nPTK2B\tA0A2K6KQ05_RHIBE\nPTK2B\tA0A2K6KQ41_RHIBE\nPTK2B\tA0A2K6KQ64_RHIBE\nPTK2B\tA0A2K6RDK5_RHIRO\nPTK2B\tA0A2K6RDP1_RHIRO\nPTK2B\tA0A2K6RDQ1_RHIRO\nPTK2B\tA0A2K6RDT1_RHIRO\nPTK2B\tA0A2K6SLF5_SAIBB\nPTK2B\tA0A2K6SLI4_SAIBB\nPTK2B\tA0A2K6SLL5_SAIBB\nPTK2B\tA0A2K6SLL7_SAIBB\nPTK2B\tA0A2K6SLM1_SAIBB\nPTK2B\tA0A2R8M8V3_CALJA\nPTK2B\tA0A2R9BL74_PANPA\nPTK2B\tA0A2R9BP01_PANPA\nPTK2B\tA0A2R9BP22_PANPA\nPTK2B\tA0A2R9BP28_PANPA\nPTK2B\tA0A2U3WR41_ODORO\nPTK2B\tA0A2U3WR44_ODORO\nPTK2B\tA0A2Y9HS68_NEOSC\nPTK2B\tA0A2Y9HXJ4_NEOSC\nPTK2B\tA0A2Y9HZ33_NEOSC\nPTK2B\tA0A2Y9PZ03_DELLE\nPTK2B\tA0A2Y9PZX5_DELLE\nPTK2B\tA0A2Y9Q582_DELLE\nPTK2B\tA0A2Y9Q5J1_DELLE\nPTK2B\tA0A2Y9T8K9_PHYMC\nPTK2B\tA0A337ST84_FELCA\nPTK2B\tA0A340XYL7_LIPVE\nPTK2B\tA0A340Y3P2_LIPVE\nPTK2B\tA0A341D8V9_NEOAA\nPTK2B\tA0A341D9D3_NEOAA\nPTK2B\tA0A384AZD6_BALAS\nPTK2B\tA0A384AZH8_BALAS\nPTK2B\tA0A384AZI7_BALAS\nPTK2B\tA0A384B0I8_BALAS\nPTK2B\tA0A384D8E7_URSMA\nPTK2B\tA0A384D8Q9_URSMA\nPTK2B\tA0A384D983_URSMA\nPTK2B\tA0A3Q0DPF2_TARSY\nPTK2B\tA0A3Q0FP74_ALLSI\nPTK2B\tA0A3Q0FQ62_ALLSI\nPTK2B\tA0A3Q0FQ68_ALLSI\nPTK2B\tA0A3Q0FQ90_ALLSI\nPTK2B\tA0A3Q0FTH7_ALLSI\nPTK2B\tA0A3Q0FUR5_ALLSI\nPTK2B\tA0A3Q1LVP1_BOVIN\nPTK2B\tA0A3Q1MP24_BOVIN\nPTK2B\tA0A3Q2GT01_HORSE\nPTK2B\tA0A3Q2H8C0_HORSE\nPTK2B\tA0A3Q2HHZ1_HORSE\nPTK2B\tA0A3Q2HLC2_HORSE\nPTK2B\tA0A3Q2I0V2_HORSE\nPTK2B\tA0A3Q7P754_CALUR\nPTK2B\tA0A3Q7PHP1_CALUR\nPTK2B\tA0A3Q7QMR0_CALUR\nPTK2B\tA0A3Q7TQW0_VULVU\nPTK2B\tA0A3Q7WPA7_URSAR\nPTK2B\tA0A3Q7XJU3_URSAR\nPTK2B\tA0A3Q7Y851_URSAR\nPTK2B\tA0A419Q554_CLOSI\nPTK2B\tA0A452CPM2_BALAS\nPTK2B\tA0A452CPP7_BALAS\nPTK2B\tA0A452CPU9_BALAS\nPTK2B\tA0A452G6N3_CAPHI\nPTK2B\tA0A452G6Z3_CAPHI\nPTK2B\tA0A452IEH2_9SAUR\nPTK2B\tA0A452IEX4_9SAUR\nPTK2B\tA0A452IFK3_9SAUR\nPTK2B\tA0A452QIE2_URSAM\nPTK2B\tA0A452SYU1_URSMA\nPTK2B\tA0A455BMQ4_PHYMC\nPTK2B\tA0A455C238_PHYMC\nPTK2B\tA0A4W2D756_BOBOX\nPTK2B\tA0A4W2D794_BOBOX\nPTK2B\tA0A4W2EUD9_BOBOX\nPTK2B\tA0A4W2GP26_BOBOX\nPTK2B\tA0A4W2GR43_BOBOX\nPTK2B\tA0A4W2GSR6_BOBOX\nPTK2B\tA0A4W2ICH7_BOBOX\nPTK2B\tA0A4X1VBP3_PIG\nPTK2B\tA0A4X1VBQ9_PIG\nPTK2B\tA0A5F4CI52_CANLF\nPTK2B\tA0A5F4CK49_CANLF\nPTK2B\tA0A5F4DKM4_CANLF\nPTK2B\tA0A5F8GDL7_MONDO\nPTK2B\tA0A5F8GW94_MONDO\nPTK2B\tA0A5F8HKA6_MONDO\nPTK2B\tA0A5F9CER2_RABIT\nPTK2B\tA0A5G3HT23_XENTR\nPTK2B\tA0A5G3HTF5_XENTR\nPTK2B\tA0A5G3HTM1_XENTR\nPTK2B\tA0A5G3HTQ6_XENTR\nPTK2B\tA0A5G3JIP3_XENTR\nPTK2B\tA0A5G3JIP7_XENTR\nPTK2B\tA0A5G3L4P5_XENTR\nPTK2B\tA0A5G3L4Q1_XENTR\nPTK2B\tA0A5G3L6J6_XENTR\nPTK2B\tA0A5K1UCA5_PIG\nPTK2B\tA6QR59_BOVIN\nPTK2B\tC9JHV9_HUMAN\nPTK2B\tE1BTC3_CHICK\nPTK2B\tE2QTG4_CANLF\nPTK2B\tE5RHL2_HUMAN\nPTK2B\tE5RJ77_HUMAN\nPTK2B\tE5RK84_HUMAN\nPTK2B\tE9Q2A6_MOUSE\nPTK2B\tF1N0Z0_BOVIN\nPTK2B\tF1RJS6_PIG\nPTK2B\tF6SS26_MACMU\nPTK2B\tF6TJZ1_XENTR\nPTK2B\tF6UBB9_XENTR\nPTK2B\tF6Z395_MONDO\nPTK2B\tF7B9F2_XENTR\nPTK2B\tF7BZ44_ORNAN\nPTK2B\tF7CCX1_MOUSE\nPTK2B\tF7CGD4_HORSE\nPTK2B\tF7DEL0_XENTR\nPTK2B\tFAK2_HUMAN\nPTK2B\tFAK2_MOUSE\nPTK2B\tFAK2_RAT\nPTK2B\tG1L018_AILME\nPTK2B\tG1NMT5_MELGA\nPTK2B\tG1PGQ8_MYOLU\nPTK2B\tG1S2N5_NOMLE\nPTK2B\tG1S2N7_NOMLE\nPTK2B\tG1TNM8_RABIT\nPTK2B\tG3SVS7_LOXAF\nPTK2B\tG3VFW4_SARHA\nPTK2B\tG5BY94_HETGA\nPTK2B\tG7N0R8_MACMU\nPTK2B\tG7PCY5_MACFA\nPTK2B\tH0V3I6_CAVPO\nPTK2B\tH0WMA0_OTOGA\nPTK2B\tH0YB74_HUMAN\nPTK2B\tH2LYR2_ORYLA\nPTK2B\tH2QVX8_PANTR\nPTK2B\tH2T8Q7_TAKRU\nPTK2B\tH2ZS05_LATCH\nPTK2B\tH9G6F2_ANOCA\nPTK2B\tH9YW07_MACMU\nPTK2B\tI3MD41_ICTTR\nPTK2B\tK7CXI9_PANTR\nPTK2B\tK7G0E4_PELSI\nPTK2B\tK7QD41_MOUSE\nPTK2B\tM3WXA1_FELCA\nPTK2B\tM3XP71_MUSPF\nPTK2B\tQ3TY09_MOUSE\nPTK2B\tQ3UDE9_MOUSE\nPTK2B\tQ5XH98_XENTR\nPTK2B\tQ7T2P8_DANRE\nPTK2B\tQ7T2V4_DANRE\nPTK2B\tQ8C2G0_MOUSE\nPTK2B\tQ8C6R5_MOUSE\nPTK2B\tQ8C9L4_MOUSE\nPTK2B\tQ8QGM6_DANRE\nPTK2B\tU3E9K3_CALJA\nPTK2B\tU3EST3_CALJA\nPTK2B\tU3JM74_FICAL\nPTK2B\tV8P4C4_OPHHA\nPTK2B\tW5PIZ2_SHEEP\n','From\tTo\nJAK2\tA0A061SIX9_9CHLO\nJAK2\tA0A096P1L9_PAPAN\nJAK2\tA0A096P1M0_PAPAN\nJAK2\tA0A0A9ZBW4_LYGHE\nJAK2\tA0A0B7F8T0_THACB\nJAK2\tA0A0D9R8Z9_CHLSB\nJAK2\tA0A0S7FJS9_9TELE\nJAK2\tA0A0S7FM01_9TELE\nJAK2\tA0A0S7FN05_9TELE\nJAK2\tA0A0S7FPC3_9TELE\nJAK2\tA0A0S7FWM2_9TELE\nJAK2\tA0A0S7FWN8_9TELE\nJAK2\tA0A0S7G097_9TELE\nJAK2\tA0A0S7HFF6_9TELE\nJAK2\tA0A0S7HFM0_9TELE\nJAK2\tA0A0S7K451_9TELE\nJAK2\tA0A0S7K4W0_9TELE\nJAK2\tA0A0S7K581_9TELE\nJAK2\tA0A0S7K664_9TELE\nJAK2\tA0A0S7K6L9_9TELE\nJAK2\tA0A0S7K7L0_9TELE\nJAK2\tA0A0S7KJB9_9TELE\nJAK2\tA0A0S7KJC0_9TELE\nJAK2\tA0A0S7KJC3_9TELE\nJAK2\tA0A0S7KJF8_9TELE\nJAK2\tA0A0S7KJF9_9TELE\nJAK2\tA0A0S7KJG3_9TELE\nJAK2\tA0A0S7KKI8_9TELE\nJAK2\tA0A151M8K3_ALLMI\nJAK2\tA0A1B0GTR9_HUMAN\nJAK2\tA0A1B0GVR5_HUMAN\nJAK2\tA0A1L8HY58_XENLA\nJAK2\tA0A1S2ZMB9_ERIEU\nJAK2\tA0A1S3FBE8_DIPOR\nJAK2\tA0A1S3KQH9_SALSA\nJAK2\tA0A1S3KQI9_SALSA\nJAK2\tA0A1S3KQJ9_SALSA\nJAK2\tA0A1U7QD80_MESAU\nJAK2\tA0A1U7SPN9_TARSY\nJAK2\tA0A1U7STL7_TARSY\nJAK2\tA0A1V4J8Q4_PATFA\nJAK2\tA0A218VAM0_9PASE\nJAK2\tA0A250Y6V8_CASCN\nJAK2\tA0A2D0SYU4_ICTPU\nJAK2\tA0A2I0MG33_COLLI\nJAK2\tA0A2I3NI09_PAPAN\nJAK2\tA0A2I3SYV7_PANTR\nJAK2\tA0A2I4AUZ4_9TELE\nJAK2\tA0A2K5DBQ1_AOTNA\nJAK2\tA0A2K5JYU7_COLAP\nJAK2\tA0A2K5JYV7_COLAP\nJAK2\tA0A2K5L5K0_CERAT\nJAK2\tA0A2K5L5N3_CERAT\nJAK2\tA0A2K5Q1Z3_CEBCA\nJAK2\tA0A2K5Q1Z6_CEBCA\nJAK2\tA0A2K5WBJ6_MACFA\nJAK2\tA0A2K5WBL6_MACFA\nJAK2\tA0A2K5YD03_MANLE\nJAK2\tA0A2K5YD24_MANLE\nJAK2\tA0A2K6BRT5_MACNE\nJAK2\tA0A2K6BRZ3_MACNE\nJAK2\tA0A2K6N0P8_RHIBE\nJAK2\tA0A2K6N0Q5_RHIBE\nJAK2\tA0A2K6N0R1_RHIBE\nJAK2\tA0A2K6N0V3_RHIBE\nJAK2\tA0A2K6P3C9_RHIRO\nJAK2\tA0A2K6P3D0_RHIRO\nJAK2\tA0A2K6P3E3_RHIRO\nJAK2\tA0A2K6P3E5_RHIRO\nJAK2\tA0A2K8I1W1_ORYME\nJAK2\tA0A2R8ZHT5_PANPA\nJAK2\tA0A2R8ZP32_PANPA\nJAK2\tA0A2S2Q101_9HEMI\nJAK2\tA0A2U3WH50_ODORO\nJAK2\tA0A2U3Y110_LEPWE\nJAK2\tA0A2U3ZNI2_ODORO\nJAK2\tA0A2U4BAV6_TURTR\nJAK2\tA0A2U4BAY9_TURTR\nJAK2\tA0A2U4BB84_TURTR\nJAK2\tA0A2Y9HEC9_NEOSC\nJAK2\tA0A2Y9HEH0_NEOSC\nJAK2\tA0A2Y9HKP7_NEOSC\nJAK2\tA0A2Y9HKV9_NEOSC\nJAK2\tA0A2Y9N3T7_DELLE\nJAK2\tA0A2Y9SYZ0_PHYMC\nJAK2\tA0A337SI69_FELCA\nJAK2\tA0A337SUR0_FELCA\nJAK2\tA0A340WTZ3_LIPVE\nJAK2\tA0A341AKB6_NEOAA\nJAK2\tA0A384A4I8_BALAS\nJAK2\tA0A384A4L9_BALAS\nJAK2\tA0A384A4U9_BALAS\nJAK2\tA0A384C412_URSMA\nJAK2\tA0A3B3TRF3_9TELE\nJAK2\tA0A3Q0DLI5_TARSY\nJAK2\tA0A3Q0GQF4_ALLSI\nJAK2\tA0A3Q0GUV9_ALLSI\nJAK2\tA0A3Q0GV11_ALLSI\nJAK2\tA0A3Q2TXU4_CHICK\nJAK2\tA0A3Q3B2A8_CHICK\nJAK2\tA0A3Q7SBI6_VULVU\nJAK2\tA0A3Q7SP86_VULVU\nJAK2\tA0A3Q7TQE3_VULVU\nJAK2\tA0A3Q7WQ56_URSAR\nJAK2\tA0A3Q7Y8G7_URSAR\nJAK2\tA0A452CH73_BALAS\nJAK2\tA0A452DQ67_CAPHI\nJAK2\tA0A452DQ72_CAPHI\nJAK2\tA0A452I649_9SAUR\nJAK2\tA0A452QRE2_URSAM\nJAK2\tA0A452QRF7_URSAM\nJAK2\tA0A452QRG3_URSAM\nJAK2\tA0A452QRG6_URSAM\nJAK2\tA0A452QRI0_URSAM\nJAK2\tA0A452QRK9_URSAM\nJAK2\tA0A452QRM6_URSAM\nJAK2\tA0A452QRR1_URSAM\nJAK2\tA0A452QRS4_URSAM\nJAK2\tA0A452QRX6_URSAM\nJAK2\tA0A452USS5_URSMA\nJAK2\tA0A452USV8_URSMA\nJAK2\tA0A452USY7_URSMA\nJAK2\tA0A452UT20_URSMA\nJAK2\tA0A452UT34_URSMA\nJAK2\tA0A452UT85_URSMA\nJAK2\tA0A452UTH3_URSMA\nJAK2\tA0A452UTS5_URSMA\nJAK2\tA0A452UTW1_URSMA\nJAK2\tA0A452UUH2_URSMA\nJAK2\tA0A455BLA3_PHYMC\nJAK2\tA0A455BLI6_PHYMC\nJAK2\tA0A494BBB9_MOUSE\nJAK2\tA0A4P9D0S0_CTEID\nJAK2\tA0A4W2GAR3_BOBOX\nJAK2\tA0A4W2I1P0_BOBOX\nJAK2\tA0A4W2I4Q1_BOBOX\nJAK2\tA0A4X1W734_PIG\nJAK2\tA0A4X1W738_PIG\nJAK2\tA0A4X2K9Z9_VOMUR\nJAK2\tA0A4Y2HT44_ARAVE\nJAK2\tA0A5F4BRD0_CANLF\nJAK2\tA0A5F4BYM7_CANLF\nJAK2\tA0A5F4C8Z3_CANLF\nJAK2\tA0A5F4CEA7_CANLF\nJAK2\tA0A5F4D099_CANLF\nJAK2\tA0A5F8ATC5_MACMU\nJAK2\tA0A5G2QGG0_PIG\nJAK2\tA0A5G2R6U0_PIG\nJAK2\tA0A5G3IA32_XENTR\nJAK2\tA0JM01_XENTR\nJAK2\tA9QVL8_HUMAN\nJAK2\tC0SPC6_DANRE\nJAK2\tD2HMR9_AILME\nJAK2\tE1BCP6_BOVIN\nJAK2\tE2RAG1_CANLF\nJAK2\tE2RPW6_CANLF\nJAK2\tF1NQU4_CHICK\nJAK2\tF6RE38_HORSE\nJAK2\tF6RLI3_CALJA\nJAK2\tF6WH53_MONDO\nJAK2\tF7EM41_XENTR\nJAK2\tF7GT47_MACMU\nJAK2\tF8UM47_CTEID\nJAK2\tG1KCE0_ANOCA\nJAK2\tG1N138_MELGA\nJAK2\tG1PL96_MYOLU\nJAK2\tG1RCI3_NOMLE\nJAK2\tG1TCI4_RABIT\nJAK2\tG3T029_LOXAF\nJAK2\tG3U3K6_LOXAF\nJAK2\tG3WQ34_SARHA\nJAK2\tG5B4Y4_HETGA\nJAK2\tG5E852_MOUSE\nJAK2\tH0VRI8_CAVPO\nJAK2\tH0X6H8_OTOGA\nJAK2\tH0Z3Y5_TAEGU\nJAK2\tH2PS83_PONAB\nJAK2\tH2QWZ4_PANTR\nJAK2\tH2SBF2_TAKRU\nJAK2\tH2SBF3_TAKRU\nJAK2\tH3ABD2_LATCH\nJAK2\tH9F1D9_MACMU\nJAK2\tH9F904_MACMU\nJAK2\tI3M8D2_ICTTR\nJAK2\tJ9JIK9_PIG\nJAK2\tJAK2_CHICK\nJAK2\tJAK2_HUMAN\nJAK2\tJAK2_MOUSE\nJAK2\tJAK2_PIG\nJAK2\tJAK2_PONAB\nJAK2\tJAK2_RAT\nJAK2\tK7CX88_PANTR\nJAK2\tK7G2T9_PELSI\nJAK2\tM3XFX5_FELCA\nJAK2\tM3Y0T4_MUSPF\nJAK2\tO35804_RAT\nJAK2\tQ15I69_PIG\nJAK2\tQ2P9U8_CHICK\nJAK2\tQ4FZX4_XENLA\nJAK2\tQ506Q0_HUMAN\nJAK2\tQ52R86_HUMAN\nJAK2\tQ6GP63_XENLA\nJAK2\tQ6JDV5_PIG\nJAK2\tQ8CIN8_MOUSE\nJAK2\tQ8IXP2_HUMAN\nJAK2\tQ9PVI2_TETFL\nJAK2\tQ9TTI9_PIG\nJAK2\tQ9TTJ0_PIG\nJAK2\tU3IAY8_ANAPP\nJAK2\tU3K7L6_FICAL\nJAK2\tU6DBV5_NEOVI\nJAK2\tW5PVH1_SHEEP\nJAK2\tW5UJ19_ICTPU\n','From\tTo\nERBB4\tA0A096MT35_PAPAN\nERBB4\tA0A0A0MSE1_HUMAN\nERBB4\tA0A0P0I3A3_HUMAN\nERBB4\tA0A0P0IT07_HUMAN\nERBB4\tA0A0P6J9V6_HETGA\nERBB4\tA0A0S7EVB0_9TELE\nERBB4\tA0A0S7HVK4_9TELE\nERBB4\tA0A0S7HW25_9TELE\nERBB4\tA0A0S7HXQ2_9TELE\nERBB4\tA0A0S7HZD8_9TELE\nERBB4\tA0A0S7I005_9TELE\nERBB4\tA0A0S7I3Y9_9TELE\nERBB4\tA0A0S7I5E1_9TELE\nERBB4\tA0A0S7I5G2_9TELE\nERBB4\tA0A0S7I7E6_9TELE\nERBB4\tA0A0S7K1V1_9TELE\nERBB4\tA0A0S7K1V3_9TELE\nERBB4\tA0A0S7K522_9TELE\nERBB4\tA0A0S7KS59_9TELE\nERBB4\tA0A0S7MB54_9TELE\nERBB4\tA0A0V0J7D8_SCHSO\nERBB4\tA0A1A7WSM2_9TELE\nERBB4\tA0A1A7XBN8_9TELE\nERBB4\tA0A1A8AC39_NOTFU\nERBB4\tA0A1A8D4U8_9TELE\nERBB4\tA0A1A8D5T8_9TELE\nERBB4\tA0A1A8FRT3_9TELE\nERBB4\tA0A1A8G7J8_9TELE\nERBB4\tA0A1A8GCR9_9TELE\nERBB4\tA0A1A8HLK7_9TELE\nERBB4\tA0A1A8I7S2_NOTKU\nERBB4\tA0A1A8I7T2_NOTKU\nERBB4\tA0A1A8MBS5_9TELE\nERBB4\tA0A1A8MDF4_9TELE\nERBB4\tA0A1A8RHV4_9TELE\nERBB4\tA0A1A8UKZ6_NOTFU\nERBB4\tA0A1A8V7U5_NOTFU\nERBB4\tA0A1A8V8I6_NOTFU\nERBB4\tA0A1D5PHZ9_CHICK\nERBB4\tA0A1D5R123_MACMU\nERBB4\tA0A1L1S0Q6_CHICK\nERBB4\tA0A1L6BNZ8_BUBBU\nERBB4\tA0A1S2ZU32_ERIEU\nERBB4\tA0A1S2ZU38_ERIEU\nERBB4\tA0A1S2ZU45_ERIEU\nERBB4\tA0A1S2ZU54_ERIEU\nERBB4\tA0A1S3ERE6_DIPOR\nERBB4\tA0A1S3ERG7_DIPOR\nERBB4\tA0A1S3ESS2_DIPOR\nERBB4\tA0A1S3EST2_DIPOR\nERBB4\tA0A1S3NYR1_SALSA\nERBB4\tA0A1U7QBV4_MESAU\nERBB4\tA0A1U7QEH0_MESAU\nERBB4\tA0A1U7QMC2_MESAU\nERBB4\tA0A1U7QPU0_MESAU\nERBB4\tA0A1U7T888_TARSY\nERBB4\tA0A1V4K778_PATFA\nERBB4\tA0A218UCB9_9PASE\nERBB4\tA0A218UNF2_9PASE\nERBB4\tA0A286XGZ4_CAVPO\nERBB4\tA0A286XUS6_CAVPO\nERBB4\tA0A286ZJD5_PIG\nERBB4\tA0A2D0R3A6_ICTPU\nERBB4\tA0A2D0R4V7_ICTPU\nERBB4\tA0A2D0R5J2_ICTPU\nERBB4\tA0A2I0MIN8_COLLI\nERBB4\tA0A2I3HWU2_NOMLE\nERBB4\tA0A2I3MT62_PAPAN\nERBB4\tA0A2I3RF60_PANTR\nERBB4\tA0A2I3SQQ9_PANTR\nERBB4\tA0A2K5EHL0_AOTNA\nERBB4\tA0A2K5EHL1_AOTNA\nERBB4\tA0A2K5EHQ0_AOTNA\nERBB4\tA0A2K5IBL5_COLAP\nERBB4\tA0A2K5IBW3_COLAP\nERBB4\tA0A2K5NUZ3_CERAT\nERBB4\tA0A2K5NUZ8_CERAT\nERBB4\tA0A2K5NV86_CERAT\nERBB4\tA0A2K5NV91_CERAT\nERBB4\tA0A2K5SA08_CEBCA\nERBB4\tA0A2K5SA38_CEBCA\nERBB4\tA0A2K5W0D3_MACFA\nERBB4\tA0A2K5W0D7_MACFA\nERBB4\tA0A2K5W0J6_MACFA\nERBB4\tA0A2K5XR87_MANLE\nERBB4\tA0A2K5XRB5_MANLE\nERBB4\tA0A2K6DNJ0_MACNE\nERBB4\tA0A2K6DNJ5_MACNE\nERBB4\tA0A2K6L1R7_RHIBE\nERBB4\tA0A2K6L1T3_RHIBE\nERBB4\tA0A2K6QND4_RHIRO\nERBB4\tA0A2K6QNI4_RHIRO\nERBB4\tA0A2K6U634_SAIBB\nERBB4\tA0A2K6U638_SAIBB\nERBB4\tA0A2R8MQK7_CALJA\nERBB4\tA0A2R9C6L4_PANPA\nERBB4\tA0A2R9C6M0_PANPA\nERBB4\tA0A2R9C6N0_PANPA\nERBB4\tA0A2R9C6Q0_PANPA\nERBB4\tA0A2U4BMK4_TURTR\nERBB4\tA0A2U4BMK5_TURTR\nERBB4\tA0A2U4BML0_TURTR\nERBB4\tA0A2U4BMW9_TURTR\nERBB4\tA0A2Y9I403_NEOSC\nERBB4\tA0A2Y9MM13_DELLE\nERBB4\tA0A2Y9MQV3_DELLE\nERBB4\tA0A2Y9MW88_DELLE\nERBB4\tA0A2Y9N212_DELLE\nERBB4\tA0A2Y9SV06_PHYMC\nERBB4\tA0A337S8F4_FELCA\nERBB4\tA0A337S8Z7_FELCA\nERBB4\tA0A337SET3_FELCA\nERBB4\tA0A340Y8S3_LIPVE\nERBB4\tA0A340YAH7_LIPVE\nERBB4\tA0A340YCA6_LIPVE\nERBB4\tA0A340YDN4_LIPVE\nERBB4\tA0A341CW64_NEOAA\nERBB4\tA0A384AKM9_BALAS\nERBB4\tA0A384AKS3_BALAS\nERBB4\tA0A384BU71_URSMA\nERBB4\tA0A3B3I1Q5_ORYLA\nERBB4\tA0A3B3T1K5_9TELE\nERBB4\tA0A3B3T2I3_9TELE\nERBB4\tA0A3Q0DXT7_TARSY\nERBB4\tA0A3Q0DZI8_TARSY\nERBB4\tA0A3Q0E193_TARSY\nERBB4\tA0A3Q0GFM8_ALLSI\nERBB4\tA0A3Q1EZD2_9TELE\nERBB4\tA0A3Q1LU00_BOVIN\nERBB4\tA0A3Q1LXM1_BOVIN\nERBB4\tA0A3Q1LZT4_BOVIN\nERBB4\tA0A3Q1MCI9_BOVIN\nERBB4\tA0A3Q1MKB4_BOVIN\nERBB4\tA0A3Q2H823_HORSE\nERBB4\tA0A3Q2H8P4_HORSE\nERBB4\tA0A3Q2H998_HORSE\nERBB4\tA0A3Q2KZ26_HORSE\nERBB4\tA0A3Q2LCD3_HORSE\nERBB4\tA0A3Q2LE08_HORSE\nERBB4\tA0A3Q7MHA4_CALUR\nERBB4\tA0A3Q7TRI1_VULVU\nERBB4\tA0A3Q7V6T7_URSAR\nERBB4\tA0A3Q7VZ56_URSAR\nERBB4\tA0A452CL32_BALAS\nERBB4\tA0A455AL44_PHYMC\nERBB4\tA0A455ALG6_PHYMC\nERBB4\tA0A455AN62_PHYMC\nERBB4\tA0A455AZW7_PHYMC\nERBB4\tA0A493TSP8_ANAPP\nERBB4\tA0A4W2GYH3_BOBOX\nERBB4\tA0A4W2IFU5_BOBOX\nERBB4\tA0A4W3IWR2_CALMI\nERBB4\tA0A4W3IWR6_CALMI\nERBB4\tA0A4W3IWW5_CALMI\nERBB4\tA0A4W3IZM9_CALMI\nERBB4\tA0A4W3IZU1_CALMI\nERBB4\tA0A4W3J647_CALMI\nERBB4\tA0A4W3J667_CALMI\nERBB4\tA0A4W3J686_CALMI\nERBB4\tA0A4W3J6B7_CALMI\nERBB4\tA0A4W3JAC4_CALMI\nERBB4\tA0A4W4GJF9_ELEEL\nERBB4\tA0A4W4GJG7_ELEEL\nERBB4\tA0A4W4GLB0_ELEEL\nERBB4\tA0A4W4GP09_ELEEL\nERBB4\tA0A4W4GP91_ELEEL\nERBB4\tA0A4W4GP96_ELEEL\nERBB4\tA0A4W4GPB7_ELEEL\nERBB4\tA0A4W4GPX7_ELEEL\nERBB4\tA0A4W6FC06_LATCA\nERBB4\tA0A4W6FC20_LATCA\nERBB4\tA0A4W6FC30_LATCA\nERBB4\tA0A4W6FD39_LATCA\nERBB4\tA0A4W6FDX6_LATCA\nERBB4\tA0A4W6FE78_LATCA\nERBB4\tA0A4W6FEG9_LATCA\nERBB4\tA0A4W6FEM4_LATCA\nERBB4\tA0A4W6FEQ3_LATCA\nERBB4\tA0A4W6FEZ3_LATCA\nERBB4\tA0A4X1US52_PIG\nERBB4\tA0A4X1UUA1_PIG\nERBB4\tA0A4X1UUF0_PIG\nERBB4\tA0A4X1UUF2_PIG\nERBB4\tA0A4X1UVE3_PIG\nERBB4\tA0A4X1UVH6_PIG\nERBB4\tA0A4X2JYA4_VOMUR\nERBB4\tA0A4X2K636_VOMUR\nERBB4\tA0A5F4BQY4_CANLF\nERBB4\tA0A5F4BW02_CANLF\nERBB4\tA0A5F4BXE2_CANLF\nERBB4\tA0A5F4C1W1_CANLF\nERBB4\tA0A5F4CNL1_CANLF\nERBB4\tA0A5F4CWU5_CANLF\nERBB4\tA0A5F4D8Q5_CANLF\nERBB4\tA0A5F4DIS9_CANLF\nERBB4\tA0A5F4VS45_CALJA\nERBB4\tA0A5F4VV49_CALJA\nERBB4\tA0A5F4VVE6_CALJA\nERBB4\tA0A5F7ZRM5_MACMU\nERBB4\tA0A5F7ZUQ7_MACMU\nERBB4\tA0A5F8AA75_MACMU\nERBB4\tA0A5F8AC21_MACMU\nERBB4\tA0A5F8G6B1_MONDO\nERBB4\tA0A5F9CPT1_RABIT\nERBB4\tA0A5F9DS61_RABIT\nERBB4\tA0A5G2QMF7_PIG\nERBB4\tA0A5G2R0H8_PIG\nERBB4\tA0A5G3IAF3_XENTR\nERBB4\tA0A5G3KQK1_XENTR\nERBB4\tA0A5G3KR71_XENTR\nERBB4\tA0A5G3KS02_XENTR\nERBB4\tA0A5G3KT29_XENTR\nERBB4\tA0A5K1UHK1_MACMU\nERBB4\tB2KGF7_MOUSE\nERBB4\tE5KCJ1_FUKDA\nERBB4\tE5KCJ2_SPACA\nERBB4\tE5KCJ3_HETGA\nERBB4\tE9PDR1_HUMAN\nERBB4\tERBB4_HUMAN\nERBB4\tERBB4_MOUSE\nERBB4\tERBB4_RAT\nERBB4\tF1M7X4_RAT\nERBB4\tF1N758_BOVIN\nERBB4\tF1P258_CHICK\nERBB4\tF1PQ05_CANLF\nERBB4\tF1SSQ9_PIG\nERBB4\tF6RCM3_XENTR\nERBB4\tF6RFJ6_XENTR\nERBB4\tF6T421_HORSE\nERBB4\tF6T440_HORSE\nERBB4\tF6UHY4_CALJA\nERBB4\tF6XMH2_XENTR\nERBB4\tF6YEX8_XENTR\nERBB4\tF6Z2Q4_XENTR\nERBB4\tF6ZD16_XENTR\nERBB4\tF6ZSZ9_MONDO\nERBB4\tF7C7Z4_XENTR\nERBB4\tF7G3G3_MACMU\nERBB4\tF7G3H1_MACMU\nERBB4\tF7GAV2_ORNAN\nERBB4\tF7H0K6_CALJA\nERBB4\tF7HB76_CALJA\nERBB4\tG1KJS5_ANOCA\nERBB4\tG1LLD3_AILME\nERBB4\tG1LLD9_AILME\nERBB4\tG1R6F8_NOMLE\nERBB4\tG1SXY0_RABIT\nERBB4\tH0VIQ0_CAVPO\nERBB4\tH0ZMM7_TAEGU\nERBB4\tH2MMV9_ORYLA\nERBB4\tH2MMW0_ORYLA\nERBB4\tH2P8H3_PONAB\nERBB4\tH2QJC5_PANTR\nERBB4\tH2T9G1_TAKRU\nERBB4\tH2T9G3_TAKRU\nERBB4\tH3A3W5_LATCH\nERBB4\tH3BLT0_HUMAN\nERBB4\tI3LEB0_PIG\nERBB4\tI3LW46_ICTTR\nERBB4\tI3MUL3_ICTTR\nERBB4\tI3NAZ2_ICTTR\nERBB4\tK7G676_PELSI\nERBB4\tL8E872_HUMAN\nERBB4\tM3WTP2_FELCA\nERBB4\tM3WZ89_FELCA\nERBB4\tM3Y927_MUSPF\nERBB4\tO88459_MOUSE\nERBB4\tQ153D9_XENLA\nERBB4\tQ4PLA4_CHICK\nERBB4\tQ4PLA5_CHICK\nERBB4\tQ4ZG14_HUMAN\nERBB4\tQ53QS8_HUMAN\nERBB4\tQ53R25_HUMAN\nERBB4\tQ53R48_HUMAN\nERBB4\tQ53T57_HUMAN\nERBB4\tQ580Q7_HUMAN\nERBB4\tQ6UA27_RAT\nERBB4\tQ9W6F6_CHICK\nERBB4\tQ9YI48_CHICK\nERBB4\tU3C7Y1_CALJA\nERBB4\tU3I266_ANAPP\nERBB4\tU3KEW3_FICAL\nERBB4\tU6D6A2_NEOVI\nERBB4\tV8NJP2_OPHHA\nERBB4\tV8NY82_OPHHA\nERBB4\tV8PG67_OPHHA\nERBB4\tW5QD60_SHEEP\nERBB4\tW5QD61_SHEEP\n','From\tTo\nERBB3\tA0A024RB84_HUMAN\nERBB3\tA0A096N4L9_PAPAN\nERBB3\tA0A0D9QYY2_CHLSB\nERBB3\tA0A0P6JAG0_HETGA\nERBB3\tA0A0S7HNI6_9TELE\nERBB3\tA0A0S7HPN4_9TELE\nERBB3\tA0A0S7HSS5_9TELE\nERBB3\tA0A0S7IIQ6_9TELE\nERBB3\tA0A0S7IIR2_9TELE\nERBB3\tA0A0S7K298_9TELE\nERBB3\tA0A0S7K3T1_9TELE\nERBB3\tA0A0S7LG52_9TELE\nERBB3\tA0A0S7LG53_9TELE\nERBB3\tA0A0S7LG60_9TELE\nERBB3\tA0A0S7LG75_9TELE\nERBB3\tA0A0S7LG80_9TELE\nERBB3\tA0A0S7LG95_9TELE\nERBB3\tA0A0S7LGA5_9TELE\nERBB3\tA0A0S7LGM7_9TELE\nERBB3\tA0A0S7LHJ2_9TELE\nERBB3\tA0A0S7MDT8_9TELE\nERBB3\tA0A0S7ME22_9TELE\nERBB3\tA0A1D5PEH0_CHICK\nERBB3\tA0A1L8H9M6_XENLA\nERBB3\tA0A1L8HI31_XENLA\nERBB3\tA0A1S3ADP8_ERIEU\nERBB3\tA0A1S3FKE4_DIPOR\nERBB3\tA0A1V4JKE9_PATFA\nERBB3\tA0A218U7R8_9PASE\nERBB3\tA0A286ZMT1_PIG\nERBB3\tA0A287ADX1_PIG\nERBB3\tA0A2D0SJA7_ICTPU\nERBB3\tA0A2D0SJV7_ICTPU\nERBB3\tA0A2I3HRH7_NOMLE\nERBB3\tA0A2I3HUE2_NOMLE\nERBB3\tA0A2I3MMJ6_PAPAN\nERBB3\tA0A2I4BA09_9TELE\nERBB3\tA0A2K5D4J2_AOTNA\nERBB3\tA0A2K5D4M3_AOTNA\nERBB3\tA0A2K5HTE7_COLAP\nERBB3\tA0A2K5HTG1_COLAP\nERBB3\tA0A2K5HTG4_COLAP\nERBB3\tA0A2K5LCA3_CERAT\nERBB3\tA0A2K5LCB8_CERAT\nERBB3\tA0A2K5RXZ5_CEBCA\nERBB3\tA0A2K5RY68_CEBCA\nERBB3\tA0A2K5X0G5_MACFA\nERBB3\tA0A2K5YIV6_MANLE\nERBB3\tA0A2K5YIW8_MANLE\nERBB3\tA0A2K6BJG3_MACNE\nERBB3\tA0A2K6BJK1_MACNE\nERBB3\tA0A2K6GN04_PROCO\nERBB3\tA0A2K6GN46_PROCO\nERBB3\tA0A2K6LWQ3_RHIBE\nERBB3\tA0A2K6P363_RHIRO\nERBB3\tA0A2K6P368_RHIRO\nERBB3\tA0A2K6UTK8_SAIBB\nERBB3\tA0A2K6UTL3_SAIBB\nERBB3\tA0A2K6UTN2_SAIBB\nERBB3\tA0A2R8Y6T4_HUMAN\nERBB3\tA0A2R9B6A5_PANPA\nERBB3\tA0A2R9BBP6_PANPA\nERBB3\tA0A2U3W1C5_ODORO\nERBB3\tA0A2U3XBG8_LEPWE\nERBB3\tA0A2U4CQI1_TURTR\nERBB3\tA0A2U4CQI3_TURTR\nERBB3\tA0A2U4CQT5_TURTR\nERBB3\tA0A2Y9FTQ2_PHYMC\nERBB3\tA0A2Y9GYH7_NEOSC\nERBB3\tA0A2Y9N080_DELLE\nERBB3\tA0A340Y622_LIPVE\nERBB3\tA0A341AS90_NEOAA\nERBB3\tA0A341AT61_NEOAA\nERBB3\tA0A383ZWX2_BALAS\nERBB3\tA0A384BJ16_URSMA\nERBB3\tA0A3Q0D258_MESAU\nERBB3\tA0A3Q0FIQ4_ALLSI\nERBB3\tA0A3Q2HT42_HORSE\nERBB3\tA0A3Q7RE49_CALUR\nERBB3\tA0A3Q7TRC1_VULVU\nERBB3\tA0A3Q7W476_URSAR\nERBB3\tA0A3R7GZ79_CLOSI\nERBB3\tA0A452EJC8_CAPHI\nERBB3\tA0A452IK72_9SAUR\nERBB3\tA0A452IK90_9SAUR\nERBB3\tA0A452IKE0_9SAUR\nERBB3\tA0A452IKE8_9SAUR\nERBB3\tA0A452IKH5_9SAUR\nERBB3\tA0A452IKJ0_9SAUR\nERBB3\tA0A452IKJ9_9SAUR\nERBB3\tA0A452IKL0_9SAUR\nERBB3\tA0A452IKL6_9SAUR\nERBB3\tA0A452R5F0_URSAM\nERBB3\tA0A452UPG3_URSMA\nERBB3\tA0A452UPJ2_URSMA\nERBB3\tA0A452UPJ5_URSMA\nERBB3\tA0A452UPJ6_URSMA\nERBB3\tA0A452UPK7_URSMA\nERBB3\tA0A452UPQ1_URSMA\nERBB3\tA0A4W2F3Y2_BOBOX\nERBB3\tA0A4X1W9N6_PIG\nERBB3\tA0A4X1WAT0_PIG\nERBB3\tA0A4X2LZM1_VOMUR\nERBB3\tA0A4X2M9J5_VOMUR\nERBB3\tA0A4Z2IYA7_9TELE\nERBB3\tA0A5F4CX56_CANLF\nERBB3\tA0A5F5PXB5_HORSE\nERBB3\tA0A5F5XSC9_FELCA\nERBB3\tA0A5F8GCF8_MONDO\nERBB3\tA0A5F8GP87_MONDO\nERBB3\tA0A5G3HJI0_XENTR\nERBB3\tA0A5G3L862_XENTR\nERBB3\tA0A5G3L984_XENTR\nERBB3\tA0A5G3L9C4_XENTR\nERBB3\tA0A5G3L9Y9_XENTR\nERBB3\tA0A5G3LAA5_XENTR\nERBB3\tA0A5G3LB99_XENTR\nERBB3\tA0A5G3LDU4_XENTR\nERBB3\tA4UNU2_BOVIN\nERBB3\tA6QR62_BOVIN\nERBB3\tB3KWG5_HUMAN\nERBB3\tB4DGQ7_HUMAN\nERBB3\tD5KUG4_RAT\nERBB3\tERBB3_HUMAN\nERBB3\tERBB3_MOUSE\nERBB3\tERBB3_PONAB\nERBB3\tERBB3_RAT\nERBB3\tF1PVC5_CANLF\nERBB3\tF6PGX4_CALJA\nERBB3\tF6RXS0_HORSE\nERBB3\tF6ZQR1_XENTR\nERBB3\tF7AD42_XENTR\nERBB3\tF7ADY2_XENTR\nERBB3\tF7EBK6_XENTR\nERBB3\tF7HL92_MACMU\nERBB3\tF7HN95_MACMU\nERBB3\tF7HN99_MACMU\nERBB3\tF8T7F2_MESAU\nERBB3\tF8VRI5_HUMAN\nERBB3\tF8VRL0_HUMAN\nERBB3\tF8VW48_HUMAN\nERBB3\tF8VW56_HUMAN\nERBB3\tF8VX90_HUMAN\nERBB3\tF8VYK4_HUMAN\nERBB3\tG1L9H7_AILME\nERBB3\tG1NU59_MYOLU\nERBB3\tG1S724_NOMLE\nERBB3\tG1TDH0_RABIT\nERBB3\tG3TDW2_LOXAF\nERBB3\tG3V6N1_RAT\nERBB3\tG3VU99_SARHA\nERBB3\tG7N7B1_MACMU\nERBB3\tG7PIG5_MACFA\nERBB3\tH0VS00_CAVPO\nERBB3\tH0WIR3_OTOGA\nERBB3\tH2L7R6_ORYLA\nERBB3\tH2NHN2_PONAB\nERBB3\tH2Q675_PANTR\nERBB3\tH2TXM7_TAKRU\nERBB3\tH2ZVQ6_LATCH\nERBB3\tH9GA09_ANOCA\nERBB3\tH9H5Z1_MONDO\nERBB3\tI0FGA6_MACMU\nERBB3\tI3L6Z7_PIG\nERBB3\tI3NGW2_ICTTR\nERBB3\tK7A0H0_PANTR\nERBB3\tK7BG96_PANTR\nERBB3\tM3WLH4_FELCA\nERBB3\tM3XPI4_MUSPF\nERBB3\tO75812_HUMAN\nERBB3\tO88458_MOUSE\nERBB3\tQ2HZD7_CHICK\nERBB3\tQ5G254_DANRE\nERBB3\tU3FA11_CALJA\nERBB3\tV8P4L6_OPHHA\nERBB3\tW5PL95_SHEEP\n','From\tTo\nEPHA2\tA0A024QZA8_HUMAN\nEPHA2\tA0A096MVV0_PAPAN\nEPHA2\tA0A0A9YRN3_LYGHE\nEPHA2\tA0A0D9S8G9_CHLSB\nEPHA2\tA0A0P6J3A5_HETGA\nEPHA2\tA0A0S7HFD3_9TELE\nEPHA2\tA0A0S7K6T8_9TELE\nEPHA2\tA0A0S7K6T9_9TELE\nEPHA2\tA0A0S7K6U1_9TELE\nEPHA2\tA0A0S7K778_9TELE\nEPHA2\tA0A0S7K7E1_9TELE\nEPHA2\tA0A0S7K7G1_9TELE\nEPHA2\tA0A0S7K7V6_9TELE\nEPHA2\tA0A0S7K845_9TELE\nEPHA2\tA0A0S7K887_9TELE\nEPHA2\tA0A0S7K8Q5_9TELE\nEPHA2\tA0A0S7KA73_9TELE\nEPHA2\tA0A0S7KKK8_9TELE\nEPHA2\tA0A0S7KKQ7_9TELE\nEPHA2\tA0A0S7KKR1_9TELE\nEPHA2\tA0A0S7KMP8_9TELE\nEPHA2\tA0A0S7M6X2_9TELE\nEPHA2\tA0A0X3Q3E6_SCHSO\nEPHA2\tA0A1A7W9K6_9TELE\nEPHA2\tA0A1A7XU83_9TELE\nEPHA2\tA0A1A8A666_NOTFU\nEPHA2\tA0A1A8AEM1_NOTFU\nEPHA2\tA0A1A8AT39_NOTFU\nEPHA2\tA0A1A8CJQ5_9TELE\nEPHA2\tA0A1A8CLT2_9TELE\nEPHA2\tA0A1A8CMQ9_9TELE\nEPHA2\tA0A1A8CY03_9TELE\nEPHA2\tA0A1A8EJZ9_9TELE\nEPHA2\tA0A1A8EM29_9TELE\nEPHA2\tA0A1A8G5P2_9TELE\nEPHA2\tA0A1A8H839_9TELE\nEPHA2\tA0A1A8HLJ1_NOTKU\nEPHA2\tA0A1A8IBD7_NOTKU\nEPHA2\tA0A1A8JDU2_NOTKU\nEPHA2\tA0A1A8K6H4_NOTKU\nEPHA2\tA0A1A8LD18_9TELE\nEPHA2\tA0A1A8NA99_9TELE\nEPHA2\tA0A1A8NAY1_9TELE\nEPHA2\tA0A1A8R4K2_9TELE\nEPHA2\tA0A1A8R8N2_9TELE\nEPHA2\tA0A1A8S5V6_9TELE\nEPHA2\tA0A1A8UKN7_NOTFU\nEPHA2\tA0A1A8UM26_NOTFU\nEPHA2\tA0A1D5PXE5_CHICK\nEPHA2\tA0A1D5QYU6_MACMU\nEPHA2\tA0A1S2ZSB6_ERIEU\nEPHA2\tA0A1S3GPQ3_DIPOR\nEPHA2\tA0A1S3GRB2_DIPOR\nEPHA2\tA0A1S3P1X6_SALSA\nEPHA2\tA0A1S3P1Y3_SALSA\nEPHA2\tA0A1U8D1W0_ALLSI\nEPHA2\tA0A1V4KPD3_PATFA\nEPHA2\tA0A218UI11_9PASE\nEPHA2\tA0A250Y6A5_CASCN\nEPHA2\tA0A286R219_CANLF\nEPHA2\tA0A286XZ18_CAVPO\nEPHA2\tA0A2B4R626_STYPI\nEPHA2\tA0A2B4SS04_STYPI\nEPHA2\tA0A2B4SV19_STYPI\nEPHA2\tA0A2D0SI13_ICTPU\nEPHA2\tA0A2I0M2U4_COLLI\nEPHA2\tA0A2I3FY26_NOMLE\nEPHA2\tA0A2I3MAA0_PAPAN\nEPHA2\tA0A2I3RXP3_PANTR\nEPHA2\tA0A2I4B1P7_9TELE\nEPHA2\tA0A2K5F4F0_AOTNA\nEPHA2\tA0A2K5F4G4_AOTNA\nEPHA2\tA0A2K5JYU4_COLAP\nEPHA2\tA0A2K5JYU8_COLAP\nEPHA2\tA0A2K5LIB6_CERAT\nEPHA2\tA0A2K5LIF5_CERAT\nEPHA2\tA0A2K5PC42_CEBCA\nEPHA2\tA0A2K5PC45_CEBCA\nEPHA2\tA0A2K5VRZ8_MACFA\nEPHA2\tA0A2K5VS15_MACFA\nEPHA2\tA0A2K5YNX7_MANLE\nEPHA2\tA0A2K5YP16_MANLE\nEPHA2\tA0A2K6DYM1_MACNE\nEPHA2\tA0A2K6DYN4_MACNE\nEPHA2\tA0A2K6F8J8_PROCO\nEPHA2\tA0A2K6L4R9_RHIBE\nEPHA2\tA0A2K6P1S9_RHIRO\nEPHA2\tA0A2K6TZ20_SAIBB\nEPHA2\tA0A2K6TZ61_SAIBB\nEPHA2\tA0A2R8ZB35_PANPA\nEPHA2\tA0A2R8ZBI9_PANPA\nEPHA2\tA0A2U3VPS4_ODORO\nEPHA2\tA0A2Y9GQY5_NEOSC\nEPHA2\tA0A2Y9Q9I4_DELLE\nEPHA2\tA0A340XJY3_LIPVE\nEPHA2\tA0A341BVH9_NEOAA\nEPHA2\tA0A383ZJM2_BALAS\nEPHA2\tA0A384CF85_URSMA\nEPHA2\tA0A3Q0D9C4_MESAU\nEPHA2\tA0A3Q0EGA3_TARSY\nEPHA2\tA0A3Q1LPW3_BOVIN\nEPHA2\tA0A3Q2HMX6_HORSE\nEPHA2\tA0A3Q7MX44_CALUR\nEPHA2\tA0A3Q7N1V0_CALUR\nEPHA2\tA0A3Q7TD80_VULVU\nEPHA2\tA0A3Q7XDB8_URSAR\nEPHA2\tA0A452ER97_CAPHI\nEPHA2\tA0A452ERX4_CAPHI\nEPHA2\tA0A452ILD6_9SAUR\nEPHA2\tA0A452RI15_URSAM\nEPHA2\tA0A455ATF7_PHYMC\nEPHA2\tA0A455B763_PHYMC\nEPHA2\tA0A4W2EY98_BOBOX\nEPHA2\tA0A4W2EYZ8_BOBOX\nEPHA2\tA0A4W2GXT2_BOBOX\nEPHA2\tA0A4X1W1P7_PIG\nEPHA2\tA0A4X2KZN7_VOMUR\nEPHA2\tA0A4X2L669_VOMUR\nEPHA2\tA0A5F4D0D2_CANLF\nEPHA2\tA0A5F8AQ28_MACMU\nEPHA2\tA0A5F8GLH8_MONDO\nEPHA2\tA0A5F9CVW1_RABIT\nEPHA2\tD3ZBN3_RAT\nEPHA2\tE1BJ31_BOVIN\nEPHA2\tEPHA2_HUMAN\nEPHA2\tEPHA2_MACFA\nEPHA2\tEPHA2_MOUSE\nEPHA2\tF1PX14_CANLF\nEPHA2\tF1Q4C5_CANLF\nEPHA2\tF1QSD6_DANRE\nEPHA2\tF1SUT8_PIG\nEPHA2\tF6QAL5_HORSE\nEPHA2\tF6UVD5_XENTR\nEPHA2\tF6V9S3_MONDO\nEPHA2\tF6WYZ6_ORNAN\nEPHA2\tF7C2S3_XENTR\nEPHA2\tF7DZ20_MONDO\nEPHA2\tF7E018_MACMU\nEPHA2\tF7G5T3_CALJA\nEPHA2\tF7GH94_CALJA\nEPHA2\tG1LZ16_AILME\nEPHA2\tG1N053_MELGA\nEPHA2\tG1PR78_MYOLU\nEPHA2\tG1R858_NOMLE\nEPHA2\tG1THA5_RABIT\nEPHA2\tG3QY65_GORGO\nEPHA2\tG3T0H8_LOXAF\nEPHA2\tG3U212_LOXAF\nEPHA2\tG3WDU5_SARHA\nEPHA2\tH0UZZ7_CAVPO\nEPHA2\tH0WRA2_OTOGA\nEPHA2\tH0YV01_TAEGU\nEPHA2\tH2N8X8_PONAB\nEPHA2\tH2PY47_PANTR\nEPHA2\tH2RQZ0_TAKRU\nEPHA2\tH3AUH0_LATCH\nEPHA2\tH3AUH1_LATCH\nEPHA2\tH9GBA4_ANOCA\nEPHA2\tI3M1N8_ICTTR\nEPHA2\tK7BL04_PANTR\nEPHA2\tK7BTW8_PANTR\nEPHA2\tK7FBA4_PELSI\nEPHA2\tM3WCU0_FELCA\nEPHA2\tM3YXZ5_MUSPF\nEPHA2\tO73876_DANRE\nEPHA2\tQ1HKZ4_MACMU\nEPHA2\tQ6DII4_XENTR\nEPHA2\tQ6NZS1_DANRE\nEPHA2\tQ82GT6_STRAW\nEPHA2\tQ8IZL0_HUMAN\nEPHA2\tQ90ZJ1_CHICK\nEPHA2\tQ96HF4_HUMAN\nEPHA2\tQ9PWR5_XENLA\nEPHA2\tU3BB45_CALJA\nEPHA2\tU3JKW9_FICAL\nEPHA2\tU6D6X1_NEOVI\nEPHA2\tV8NVX2_OPHHA\nEPHA2\tV8NVZ2_OPHHA\nEPHA2\tW5PS60_SHEEP\nEPHA2\tW5UJT1_ICTPU\n','From\tTo\nLCK\tA0A034WKL4_BACDO\nLCK\tA0A096N492_PAPAN\nLCK\tA0A0C9QHU1_9HYME\nLCK\tA0A0D9S7X2_CHLSB\nLCK\tA0A0F8B4E7_CERFI\nLCK\tA0A0H2UHH7_RAT\nLCK\tA0A0S2Z3T3_HUMAN\nLCK\tA0A0S2Z3Y4_HUMAN\nLCK\tA0A0S2Z3Y8_HUMAN\nLCK\tA0A0S7HTT0_9TELE\nLCK\tA0A0S7HU79_9TELE\nLCK\tA0A0S7HUB9_9TELE\nLCK\tA0A0S7HXX8_9TELE\nLCK\tA0A0S7JSJ0_9TELE\nLCK\tA0A0S7JSS0_9TELE\nLCK\tA0A0S7JUB3_9TELE\nLCK\tA0A0S7JUD3_9TELE\nLCK\tA0A0S7JUK7_9TELE\nLCK\tA0A0S7MIT2_9TELE\nLCK\tA0A140WS07_ANGJA\nLCK\tA0A1A7WEG6_9TELE\nLCK\tA0A1A7ZBX2_NOTFU\nLCK\tA0A1A7ZCH6_NOTFU\nLCK\tA0A1D5Q9I1_MACMU\nLCK\tA0A1D5RGZ8_MACMU\nLCK\tA0A1L8H7S2_XENLA\nLCK\tA0A1S3FJ81_DIPOR\nLCK\tA0A1S3NCA5_SALSA\nLCK\tA0A1S3WBE5_ERIEU\nLCK\tA0A1S3WBK8_ERIEU\nLCK\tA0A1U7RJU8_ALLSI\nLCK\tA0A1U7SYE6_TARSY\nLCK\tA0A1V4JHQ6_PATFA\nLCK\tA0A218UGB0_9PASE\nLCK\tA0A250YJ65_CASCN\nLCK\tA0A287AH06_PIG\nLCK\tA0A2I0LNG7_COLLI\nLCK\tA0A2I2UG73_FELCA\nLCK\tA0A2I2V1Z0_FELCA\nLCK\tA0A2I3HXB4_NOMLE\nLCK\tA0A2I3MZ34_PAPAN\nLCK\tA0A2I3N3T1_PAPAN\nLCK\tA0A2I3NF98_PAPAN\nLCK\tA0A2I3SDX3_PANTR\nLCK\tA0A2I3T2K8_PANTR\nLCK\tA0A2I4BB45_9TELE\nLCK\tA0A2K5EMU3_AOTNA\nLCK\tA0A2K5EN11_AOTNA\nLCK\tA0A2K5EN25_AOTNA\nLCK\tA0A2K5KAG9_COLAP\nLCK\tA0A2K5KAI4_COLAP\nLCK\tA0A2K5KAI5_COLAP\nLCK\tA0A2K5P1D7_CERAT\nLCK\tA0A2K5P1F2_CERAT\nLCK\tA0A2K5P1N4_CERAT\nLCK\tA0A2K5P1N7_CERAT\nLCK\tA0A2K5P1R6_CERAT\nLCK\tA0A2K5RAZ6_CEBCA\nLCK\tA0A2K5RB52_CEBCA\nLCK\tA0A2K5RB55_CEBCA\nLCK\tA0A2K5UW36_MACFA\nLCK\tA0A2K5UWC1_MACFA\nLCK\tA0A2K5YHT3_MANLE\nLCK\tA0A2K5YHU8_MANLE\nLCK\tA0A2K5YHV9_MANLE\nLCK\tA0A2K5YHX0_MANLE\nLCK\tA0A2K6CSG7_MACNE\nLCK\tA0A2K6CSG8_MACNE\nLCK\tA0A2K6CSH0_MACNE\nLCK\tA0A2K6CSJ1_MACNE\nLCK\tA0A2K6F6Y4_PROCO\nLCK\tA0A2K6M742_RHIBE\nLCK\tA0A2K6M745_RHIBE\nLCK\tA0A2K6M753_RHIBE\nLCK\tA0A2K6M757_RHIBE\nLCK\tA0A2K6PA79_RHIRO\nLCK\tA0A2K6PA85_RHIRO\nLCK\tA0A2K6PAA1_RHIRO\nLCK\tA0A2K6PAA6_RHIRO\nLCK\tA0A2K6PAG2_RHIRO\nLCK\tA0A2K6TGI9_SAIBB\nLCK\tA0A2K6TGJ9_SAIBB\nLCK\tA0A2K6TGL2_SAIBB\nLCK\tA0A2K6TGN8_SAIBB\nLCK\tA0A2R9AFQ6_PANPA\nLCK\tA0A2R9AKH9_PANPA\nLCK\tA0A2R9AKJ0_PANPA\nLCK\tA0A2U3WD75_ODORO\nLCK\tA0A2U3XRY1_LEPWE\nLCK\tA0A2U4BJ53_TURTR\nLCK\tA0A2Y9FLW6_PHYMC\nLCK\tA0A2Y9GN32_NEOSC\nLCK\tA0A2Y9GPR2_NEOSC\nLCK\tA0A2Y9NNB7_DELLE\nLCK\tA0A340Y4W4_LIPVE\nLCK\tA0A340YBD8_LIPVE\nLCK\tA0A341D621_NEOAA\nLCK\tA0A383Z669_BALAS\nLCK\tA0A384CCZ4_URSMA\nLCK\tA0A3Q0DMM5_TARSY\nLCK\tA0A3Q1MDN9_BOVIN\nLCK\tA0A3Q7MV08_CALUR\nLCK\tA0A3Q7TY87_VULVU\nLCK\tA0A3Q7XDN4_URSAR\nLCK\tA0A452EIH9_CAPHI\nLCK\tA0A452IZ20_9SAUR\nLCK\tA0A452TSC6_URSMA\nLCK\tA0A452TSC9_URSMA\nLCK\tA0A4W2ECL3_BOBOX\nLCK\tA0A4W2HNJ5_BOBOX\nLCK\tA0A4W3J0E5_CALMI\nLCK\tA0A4W3J0F0_CALMI\nLCK\tA0A4W3J2L4_CALMI\nLCK\tA0A4W3J304_CALMI\nLCK\tA0A4W3J334_CALMI\nLCK\tA0A4W3J337_CALMI\nLCK\tA0A4W3J9Z6_CALMI\nLCK\tA0A4W3JX24_CALMI\nLCK\tA0A4W3JX80_CALMI\nLCK\tA0A4X1VTJ0_PIG\nLCK\tA0A4X1VY64_PIG\nLCK\tA0A4X1VYK5_PIG\nLCK\tA0A4X1W0F6_PIG\nLCK\tA0A4X2JME9_VOMUR\nLCK\tA0A4Z2IYQ7_9TELE\nLCK\tA0A5F7ZYA6_MACMU\nLCK\tA0A5F8A6G1_MACMU\nLCK\tA0A5F8ACH9_MACMU\nLCK\tA0A5F8AKW4_MACMU\nLCK\tA0A5F8ATM2_MACMU\nLCK\tA0A5G3IXT9_XENTR\nLCK\tA0A5G3IYT2_XENTR\nLCK\tA0A5G3LCH1_XENTR\nLCK\tA1L2U4_XENLA\nLCK\tB2KG65_MOUSE\nLCK\tB2ZZT0_ORYLA\nLCK\tB5X1E0_SALSA\nLCK\tB7U173_SHEEP\nLCK\tB8XSK4_PIG\nLCK\tD0QTE9_HIPHI\nLCK\tD0QTF0_HIPHI\nLCK\tE2R589_CANLF\nLCK\tE9PAP0_HUMAN\nLCK\tE9PI33_HUMAN\nLCK\tE9PJ92_HUMAN\nLCK\tE9PKQ8_HUMAN\nLCK\tE9PQ10_HUMAN\nLCK\tE9Q696_MOUSE\nLCK\tF1NP54_CHICK\nLCK\tF1SV94_PIG\nLCK\tF6USJ9_HORSE\nLCK\tF6Z574_MONDO\nLCK\tF6ZYN6_ORNAN\nLCK\tF7AYA4_XENTR\nLCK\tF7CSW9_CALJA\nLCK\tF7CSX5_CALJA\nLCK\tF8W6B9_HUMAN\nLCK\tG1LMC0_AILME\nLCK\tG1LMD8_AILME\nLCK\tG1MZR7_MELGA\nLCK\tG1P745_MYOLU\nLCK\tG1QPW2_NOMLE\nLCK\tG1SK64_RABIT\nLCK\tG3T7L2_LOXAF\nLCK\tG3W363_SARHA\nLCK\tG3X8M0_MELGA\nLCK\tH0VAT0_CAVPO\nLCK\tH0WSN1_OTOGA\nLCK\tH2MSY2_ORYLA\nLCK\tH2N854_PONAB\nLCK\tH2PYK3_PANTR\nLCK\tH2SFA4_TAKRU\nLCK\tH3AYU0_LATCH\nLCK\tH3K2C7_ANGJA\nLCK\tH9Z8V0_MACMU\nLCK\tI3JN72_ORENI\nLCK\tI3MIH9_ICTTR\nLCK\tK7G4Q7_PELSI\nLCK\tLCK_AOTNA\nLCK\tLCK_CHICK\nLCK\tLCK_HUMAN\nLCK\tLCK_MOUSE\nLCK\tLCK_RAT\nLCK\tLCK_SAISC\nLCK\tM3YUZ1_MUSPF\nLCK\tQ1RLY5_DANRE\nLCK\tQ3TLX4_MOUSE\nLCK\tQ3ZCM0_BOVIN\nLCK\tQ4FZR6_RAT\nLCK\tQ573B4_HUMAN\nLCK\tQ6PBC5_XENTR\nLCK\tQ6TPQ4_DANRE\nLCK\tQ8QGJ9_TAKRU\nLCK\tQ95M32_9PRIM\nLCK\tR4HHW9_LARCR\nLCK\tU3J2H2_ANAPP\nLCK\tU3JD00_FICAL\nLCK\tU3JD01_FICAL\nLCK\tU6DAS6_NEOVI\nLCK\tV5IZL0_MESAU\nLCK\tV9KRC9_CALMI\nLCK\tV9NED9_GINCI\nLCK\tW5QII1_SHEEP\nLCK\tW5QII2_SHEEP\nLCK\tW5U6C9_ICTPU\nLCK\tW8C9S5_CERCA\n','From\tTo\nABL2\tA0A087WQB7_MOUSE\nABL2\tA0A096MQ99_PAPAN\nABL2\tA0A096MZF3_PAPAN\nABL2\tA0A0D9RI41_CHLSB\nABL2\tA0A0R4IYB8_DANRE\nABL2\tA0A0S7HDR2_9TELE\nABL2\tA0A0S7HE26_9TELE\nABL2\tA0A0S7I3M1_9TELE\nABL2\tA0A0S7I6X8_9TELE\nABL2\tA0A0S7I6Z5_9TELE\nABL2\tA0A0S7I836_9TELE\nABL2\tA0A0S7M653_9TELE\nABL2\tA0A0S7MGJ6_9TELE\nABL2\tA0A0S7MH80_9TELE\nABL2\tA0A0S7MHD7_9TELE\nABL2\tA0A1A7WHM6_9TELE\nABL2\tA0A1A7YE56_9TELE\nABL2\tA0A1A7YS31_9TELE\nABL2\tA0A1A8A430_NOTFU\nABL2\tA0A1A8AHV9_NOTFU\nABL2\tA0A1A8C3I6_9TELE\nABL2\tA0A1A8CK76_9TELE\nABL2\tA0A1A8CMZ8_9TELE\nABL2\tA0A1A8EBU6_9TELE\nABL2\tA0A1A8FAV0_9TELE\nABL2\tA0A1A8G5L7_9TELE\nABL2\tA0A1A8GJ84_9TELE\nABL2\tA0A1A8IEE9_NOTKU\nABL2\tA0A1A8JDZ2_NOTKU\nABL2\tA0A1A8KIS9_NOTKU\nABL2\tA0A1A8KJC6_NOTKU\nABL2\tA0A1A8L4K0_9TELE\nABL2\tA0A1A8N3E8_9TELE\nABL2\tA0A1A8NUD7_9TELE\nABL2\tA0A1A8NVC4_9TELE\nABL2\tA0A1A8Q076_9TELE\nABL2\tA0A1A8QBC5_9TELE\nABL2\tA0A1A8R3T6_9TELE\nABL2\tA0A1A8SA18_9TELE\nABL2\tA0A1A8U1Y8_NOTFU\nABL2\tA0A1A8V873_NOTFU\nABL2\tA0A1D5PIY0_CHICK\nABL2\tA0A1D5PJY8_CHICK\nABL2\tA0A1D5PLI1_CHICK\nABL2\tA0A1D5PPL3_CHICK\nABL2\tA0A1L8GGA5_XENLA\nABL2\tA0A1S3AJS8_ERIEU\nABL2\tA0A1S3F4F0_DIPOR\nABL2\tA0A1S3T6A9_SALSA\nABL2\tA0A1S3T6C1_SALSA\nABL2\tA0A1S3T6C3_SALSA\nABL2\tA0A1S3T6C5_SALSA\nABL2\tA0A1S3T6C7_SALSA\nABL2\tA0A1S3T6C8_SALSA\nABL2\tA0A1U7SM91_ALLSI\nABL2\tA0A1U7TTI8_TARSY\nABL2\tA0A1U7TZR0_TARSY\nABL2\tA0A1U7TZR6_TARSY\nABL2\tA0A1U7U3L0_TARSY\nABL2\tA0A1U7UBR5_TARSY\nABL2\tA0A1U7UBS0_TARSY\nABL2\tA0A1U7UDI1_TARSY\nABL2\tA0A1U8C216_MESAU\nABL2\tA0A1V4JXB5_PATFA\nABL2\tA0A1V4JYW4_PATFA\nABL2\tA0A218V4Y8_9PASE\nABL2\tA0A250Y729_CASCN\nABL2\tA0A286Y1Z5_CAVPO\nABL2\tA0A286ZN83_PIG\nABL2\tA0A286ZP60_PIG\nABL2\tA0A287AQK4_PIG\nABL2\tA0A287AUX3_PIG\nABL2\tA0A287CTA2_ICTTR\nABL2\tA0A287D0F2_ICTTR\nABL2\tA0A2D0R0J3_ICTPU\nABL2\tA0A2D0R0J6_ICTPU\nABL2\tA0A2D0R2A1_ICTPU\nABL2\tA0A2I0MBR5_COLLI\nABL2\tA0A2I2V0U4_FELCA\nABL2\tA0A2I2V291_FELCA\nABL2\tA0A2I3FRA9_NOMLE\nABL2\tA0A2I3GSM6_NOMLE\nABL2\tA0A2I3GX06_NOMLE\nABL2\tA0A2I3HR95_NOMLE\nABL2\tA0A2I3HZX9_NOMLE\nABL2\tA0A2I3LLL1_PAPAN\nABL2\tA0A2I3MIM0_PAPAN\nABL2\tA0A2I3MK09_PAPAN\nABL2\tA0A2I3MNM5_PAPAN\nABL2\tA0A2I3SHL7_PANTR\nABL2\tA0A2I3TH30_PANTR\nABL2\tA0A2I3TNB1_PANTR\nABL2\tA0A2I3TWE6_PANTR\nABL2\tA0A2I4ALD9_9TELE\nABL2\tA0A2I4ALG0_9TELE\nABL2\tA0A2I4ALG5_9TELE\nABL2\tA0A2I4ALG9_9TELE\nABL2\tA0A2I4ALH3_9TELE\nABL2\tA0A2I4ALH9_9TELE\nABL2\tA0A2K5F8D6_AOTNA\nABL2\tA0A2K5F8H4_AOTNA\nABL2\tA0A2K5F8I2_AOTNA\nABL2\tA0A2K5F8N4_AOTNA\nABL2\tA0A2K5JSF0_COLAP\nABL2\tA0A2K5MC99_CERAT\nABL2\tA0A2K5MCB1_CERAT\nABL2\tA0A2K5MCC1_CERAT\nABL2\tA0A2K5MCE9_CERAT\nABL2\tA0A2K5MCF4_CERAT\nABL2\tA0A2K5MCG3_CERAT\nABL2\tA0A2K5QFQ9_CEBCA\nABL2\tA0A2K5QFS4_CEBCA\nABL2\tA0A2K5QFU3_CEBCA\nABL2\tA0A2K5QFX0_CEBCA\nABL2\tA0A2K5TT41_MACFA\nABL2\tA0A2K5TT44_MACFA\nABL2\tA0A2K5TT54_MACFA\nABL2\tA0A2K5TT62_MACFA\nABL2\tA0A2K5TT63_MACFA\nABL2\tA0A2K5XUT1_MANLE\nABL2\tA0A2K5XUV1_MANLE\nABL2\tA0A2K5XUV6_MANLE\nABL2\tA0A2K5XUV9_MANLE\nABL2\tA0A2K6CR67_MACNE\nABL2\tA0A2K6CR83_MACNE\nABL2\tA0A2K6CR95_MACNE\nABL2\tA0A2K6CRB2_MACNE\nABL2\tA0A2K6CRB8_MACNE\nABL2\tA0A2K6FW54_PROCO\nABL2\tA0A2K6FW62_PROCO\nABL2\tA0A2K6FW63_PROCO\nABL2\tA0A2K6FWA5_PROCO\nABL2\tA0A2K6KIK6_RHIBE\nABL2\tA0A2K6KIL1_RHIBE\nABL2\tA0A2K6KIL6_RHIBE\nABL2\tA0A2K6KIL7_RHIBE\nABL2\tA0A2K6RX24_RHIRO\nABL2\tA0A2K6RX33_RHIRO\nABL2\tA0A2K6RX39_RHIRO\nABL2\tA0A2K6UBS1_SAIBB\nABL2\tA0A2K6UBV1_SAIBB\nABL2\tA0A2K6UBV6_SAIBB\nABL2\tA0A2K6UBV7_SAIBB\nABL2\tA0A2K6UBW1_SAIBB\nABL2\tA0A2R8MAK2_CALJA\nABL2\tA0A2R8MYN3_CALJA\nABL2\tA0A2R9CKH6_PANPA\nABL2\tA0A2R9CMY8_PANPA\nABL2\tA0A2R9CMZ1_PANPA\nABL2\tA0A2U3VS97_ODORO\nABL2\tA0A2U3VSC1_ODORO\nABL2\tA0A2U3VSC3_ODORO\nABL2\tA0A2U3VSC7_ODORO\nABL2\tA0A2U3VSD0_ODORO\nABL2\tA0A2U3VSL4_ODORO\nABL2\tA0A2U3VSM0_ODORO\nABL2\tA0A2U4CQR9_TURTR\nABL2\tA0A2U4CQS1_TURTR\nABL2\tA0A2U4CQS2_TURTR\nABL2\tA0A2U4CR22_TURTR\nABL2\tA0A2Y9GH84_NEOSC\nABL2\tA0A2Y9LRE2_DELLE\nABL2\tA0A2Y9LSJ7_DELLE\nABL2\tA0A2Y9LUB3_DELLE\nABL2\tA0A2Y9LZ58_DELLE\nABL2\tA0A2Y9LZ63_DELLE\nABL2\tA0A2Y9M4L9_DELLE\nABL2\tA0A2Y9TER7_PHYMC\nABL2\tA0A2Y9TGT4_PHYMC\nABL2\tA0A2Y9TIH5_PHYMC\nABL2\tA0A2Y9TJX0_PHYMC\nABL2\tA0A2Y9TKB8_PHYMC\nABL2\tA0A337S4G5_FELCA\nABL2\tA0A340WNJ8_LIPVE\nABL2\tA0A341CIV9_NEOAA\nABL2\tA0A341CJC0_NEOAA\nABL2\tA0A384AMB6_BALAS\nABL2\tA0A384CJV3_URSMA\nABL2\tA0A3Q1MGX4_BOVIN\nABL2\tA0A3Q2IKY7_HORSE\nABL2\tA0A3Q7PIF4_CALUR\nABL2\tA0A3Q7PIM8_CALUR\nABL2\tA0A3Q7PU68_CALUR\nABL2\tA0A3Q7QZU6_CALUR\nABL2\tA0A3Q7T8J4_VULVU\nABL2\tA0A3Q7TPC0_VULVU\nABL2\tA0A3Q7VVD4_URSAR\nABL2\tA0A3Q7WJ16_URSAR\nABL2\tA0A3Q7WTG6_URSAR\nABL2\tA0A3Q7XF54_URSAR\nABL2\tA0A3Q7XQ69_URSAR\nABL2\tA0A3Q7XQ73_URSAR\nABL2\tA0A452ED85_CAPHI\nABL2\tA0A452EDG5_CAPHI\nABL2\tA0A452EDR5_CAPHI\nABL2\tA0A452EDT7_CAPHI\nABL2\tA0A452HRL8_9SAUR\nABL2\tA0A452QKF4_URSAM\nABL2\tA0A452QKH3_URSAM\nABL2\tA0A452V4B3_URSMA\nABL2\tA0A452V4B8_URSMA\nABL2\tA0A452V4K4_URSMA\nABL2\tA0A455BKJ1_PHYMC\nABL2\tA0A493T3Z9_ANAPP\nABL2\tA0A4W2CAY5_BOBOX\nABL2\tA0A4W2CRY7_BOBOX\nABL2\tA0A4W2E0E0_BOBOX\nABL2\tA0A4W2EI09_BOBOX\nABL2\tA0A4W2HDI2_BOBOX\nABL2\tA0A4W3JBR6_CALMI\nABL2\tA0A4W3JBS1_CALMI\nABL2\tA0A4W3JBS8_CALMI\nABL2\tA0A4W3JJS7_CALMI\nABL2\tA0A4W3JNL8_CALMI\nABL2\tA0A4W3JNN0_CALMI\nABL2\tA0A4W6G2J4_LATCA\nABL2\tA0A4W6G2S6_LATCA\nABL2\tA0A4W6G3W0_LATCA\nABL2\tA0A4W6G4L5_LATCA\nABL2\tA0A4X1U5Y5_PIG\nABL2\tA0A4X1U822_PIG\nABL2\tA0A4X1U827_PIG\nABL2\tA0A4X1U8J1_PIG\nABL2\tA0A4X1U8M4_PIG\nABL2\tA0A4X2KGG3_VOMUR\nABL2\tA0A4Z2JG71_9TELE\nABL2\tA0A5F4DM54_CANLF\nABL2\tA0A5F5PJT7_HORSE\nABL2\tA0A5F5PM28_HORSE\nABL2\tABL2_HUMAN\nABL2\tABL2_LEPMJ\nABL2\tABL2_MOUSE\nABL2\tB0UXN6_DANRE\nABL2\tB0UXN7_DANRE\nABL2\tB2RQ57_MOUSE\nABL2\tB7ZMZ2_MOUSE\nABL2\tB7ZUF9_XENTR\nABL2\tD1H0Y8_SCHMA\nABL2\tD2H3L6_AILME\nABL2\tF1M0N1_RAT\nABL2\tF1MWH8_BOVIN\nABL2\tF1PPK2_CANLF\nABL2\tF6YHH1_ORNAN\nABL2\tF7AU70_MACMU\nABL2\tF7B6Q9_HORSE\nABL2\tF7BKF3_XENTR\nABL2\tF7DE40_CALJA\nABL2\tF7DET4_CALJA\nABL2\tF7FED8_MONDO\nABL2\tF7HFJ4_CALJA\nABL2\tF8VQH0_MOUSE\nABL2\tG1MUH1_MELGA\nABL2\tG1P8R6_MYOLU\nABL2\tG1S0A6_NOMLE\nABL2\tG1SVS3_RABIT\nABL2\tG1U0I1_RABIT\nABL2\tG3SW23_LOXAF\nABL2\tG3TPS9_LOXAF\nABL2\tG3UWQ2_MOUSE\nABL2\tG3WWW2_SARHA\nABL2\tH0VN71_CAVPO\nABL2\tH0X7N4_OTOGA\nABL2\tH2MYS0_ORYLA\nABL2\tH2N4J6_PONAB\nABL2\tH2R8D7_PANTR\nABL2\tH2SQR1_TAKRU\nABL2\tH2SQR3_TAKRU\nABL2\tH3A9Q6_LATCH\nABL2\tH9F666_MACMU\nABL2\tH9FKK0_MACMU\nABL2\tH9GIP7_ANOCA\nABL2\tI3LP68_PIG\nABL2\tI3N8T1_ICTTR\nABL2\tI4ENX1_ECHMU\nABL2\tK7CL85_PANTR\nABL2\tK7CUY5_PANTR\nABL2\tK7FX74_PELSI\nABL2\tK7FX86_PELSI\nABL2\tM3W3U6_FELCA\nABL2\tM3Y1H1_MUSPF\nABL2\tQ61055_MOUSE\nABL2\tQ6P282_XENLA\nABL2\tQ99P28_MOUSE\nABL2\tU3J0U7_ANAPP\nABL2\tU3K0H4_FICAL\nABL2\tW5Q539_SHEEP\n','From\tTo\nERBB2\tA0A060PY73_CANLF\nERBB2\tA0A096P5C9_PAPAN\nERBB2\tA0A0A1XSA4_ZEUCU\nERBB2\tA0A0A9X587_LYGHE\nERBB2\tA0A0B5JQ80_DANRE\nERBB2\tA0A0D4CYJ6_PIG\nERBB2\tA0A0D9S388_CHLSB\nERBB2\tA0A0G2JT61_RAT\nERBB2\tA0A0R9RWK2_HUMAN\nERBB2\tA0A0S7JJ49_9TELE\nERBB2\tA0A0S7JKV7_9TELE\nERBB2\tA0A0S7JLM8_9TELE\nERBB2\tA0A0S7M0Z6_9TELE\nERBB2\tA0A142FHB9_9PASE\nERBB2\tA0A151P598_ALLMI\nERBB2\tA0A151P5B6_ALLMI\nERBB2\tA0A1A7WIK4_9TELE\nERBB2\tA0A1A7Z5G4_9TELE\nERBB2\tA0A1A8AQU3_NOTFU\nERBB2\tA0A1A8CEG2_9TELE\nERBB2\tA0A1A8CEI0_9TELE\nERBB2\tA0A1A8FXQ1_9TELE\nERBB2\tA0A1A8G147_9TELE\nERBB2\tA0A1A8GT24_9TELE\nERBB2\tA0A1A8GVI4_9TELE\nERBB2\tA0A1A8GX26_9TELE\nERBB2\tA0A1A8HXG1_NOTKU\nERBB2\tA0A1A8KM70_NOTKU\nERBB2\tA0A1A8MZ95_9TELE\nERBB2\tA0A1A8QQA9_9TELE\nERBB2\tA0A1A8QRN4_9TELE\nERBB2\tA0A1A8RTM6_9TELE\nERBB2\tA0A1A8SA88_9TELE\nERBB2\tA0A1A8UG98_NOTFU\nERBB2\tA0A1A8UIH4_NOTFU\nERBB2\tA0A1D5NWM4_CHICK\nERBB2\tA0A1D5QYY4_MACMU\nERBB2\tA0A1L5BWY6_CASFI\nERBB2\tA0A1L8ES20_XENLA\nERBB2\tA0A1S3A8L5_ERIEU\nERBB2\tA0A1S3GZZ1_DIPOR\nERBB2\tA0A1S3NFA0_SALSA\nERBB2\tA0A1U7Q225_MESAU\nERBB2\tA0A1U7RYQ2_ALLSI\nERBB2\tA0A1V4JQL6_PATFA\nERBB2\tA0A1W5BG94_CIOIN\nERBB2\tA0A1W5BG97_CIOIN\nERBB2\tA0A1W5BG99_CIOIN\nERBB2\tA0A1W5BK85_CIOIN\nERBB2\tA0A1W5BK90_CIOIN\nERBB2\tA0A1W5BLJ6_CIOIN\nERBB2\tA0A1W5BME2_CIOIN\nERBB2\tA0A218UMP6_9PASE\nERBB2\tA0A250Y7L2_CASCN\nERBB2\tA0A2I0LL36_COLLI\nERBB2\tA0A2I3HZH4_NOMLE\nERBB2\tA0A2I3ME41_PAPAN\nERBB2\tA0A2I3MJQ8_PAPAN\nERBB2\tA0A2I3MNK2_PAPAN\nERBB2\tA0A2I3RUM6_PANTR\nERBB2\tA0A2I3TJ87_PANTR\nERBB2\tA0A2I4BNR2_9TELE\nERBB2\tA0A2K5D0C7_AOTNA\nERBB2\tA0A2K5D0C9_AOTNA\nERBB2\tA0A2K5D0D5_AOTNA\nERBB2\tA0A2K5D0D8_AOTNA\nERBB2\tA0A2K5D0E7_AOTNA\nERBB2\tA0A2K5D0F9_AOTNA\nERBB2\tA0A2K5HUX9_COLAP\nERBB2\tA0A2K5HV12_COLAP\nERBB2\tA0A2K5HV16_COLAP\nERBB2\tA0A2K5HV44_COLAP\nERBB2\tA0A2K5HV61_COLAP\nERBB2\tA0A2K5L4X8_CERAT\nERBB2\tA0A2K5L507_CERAT\nERBB2\tA0A2K5L521_CERAT\nERBB2\tA0A2K5L547_CERAT\nERBB2\tA0A2K5L548_CERAT\nERBB2\tA0A2K5L560_CERAT\nERBB2\tA0A2K5L570_CERAT\nERBB2\tA0A2K5RNN2_CEBCA\nERBB2\tA0A2K5RNQ0_CEBCA\nERBB2\tA0A2K5RNQ6_CEBCA\nERBB2\tA0A2K5WUC3_MACFA\nERBB2\tA0A2K5WUE3_MACFA\nERBB2\tA0A2K5WUF7_MACFA\nERBB2\tA0A2K5WUL9_MACFA\nERBB2\tA0A2K5WUT7_MACFA\nERBB2\tA0A2K5YM49_MANLE\nERBB2\tA0A2K5YM50_MANLE\nERBB2\tA0A2K5YM85_MANLE\nERBB2\tA0A2K6EBS0_MACNE\nERBB2\tA0A2K6EBT1_MACNE\nERBB2\tA0A2K6EBT5_MACNE\nERBB2\tA0A2K6EBT9_MACNE\nERBB2\tA0A2K6GPN7_PROCO\nERBB2\tA0A2K6GPQ0_PROCO\nERBB2\tA0A2K6GPR4_PROCO\nERBB2\tA0A2K6M8Q2_RHIBE\nERBB2\tA0A2K6M8Q6_RHIBE\nERBB2\tA0A2K6M8Q8_RHIBE\nERBB2\tA0A2K6M8Q9_RHIBE\nERBB2\tA0A2K6M8R4_RHIBE\nERBB2\tA0A2K6RT29_RHIRO\nERBB2\tA0A2K6RT31_RHIRO\nERBB2\tA0A2K6RT32_RHIRO\nERBB2\tA0A2K6RT44_RHIRO\nERBB2\tA0A2K6RT56_RHIRO\nERBB2\tA0A2K6U1I8_SAIBB\nERBB2\tA0A2K6U1K5_SAIBB\nERBB2\tA0A2K6U1L1_SAIBB\nERBB2\tA0A2K6U1M7_SAIBB\nERBB2\tA0A2K6U1Q2_SAIBB\nERBB2\tA0A2K6U1Q7_SAIBB\nERBB2\tA0A2R9C656_PANPA\nERBB2\tA0A2R9C661_PANPA\nERBB2\tA0A2R9C907_PANPA\nERBB2\tA0A2R9CGT8_PANPA\nERBB2\tA0A2U3VJM7_ODORO\nERBB2\tA0A2U3VJM9_ODORO\nERBB2\tA0A2U4BNX7_TURTR\nERBB2\tA0A2Y9F099_PHYMC\nERBB2\tA0A2Y9I878_NEOSC\nERBB2\tA0A2Y9M2S5_DELLE\nERBB2\tA0A337S1T7_FELCA\nERBB2\tA0A340XV51_LIPVE\nERBB2\tA0A341B960_NEOAA\nERBB2\tA0A383ZR87_BALAS\nERBB2\tA0A384BZA6_URSMA\nERBB2\tA0A3Q0DQC5_TARSY\nERBB2\tA0A3Q0GGC3_ALLSI\nERBB2\tA0A3Q2H4E6_HORSE\nERBB2\tA0A3Q7PH14_CALUR\nERBB2\tA0A3Q7RXN7_VULVU\nERBB2\tA0A3Q7S6N7_VULVU\nERBB2\tA0A3Q7W6I7_URSAR\nERBB2\tA0A3Q7WUJ2_URSAR\nERBB2\tA0A3Q7XYU8_URSAR\nERBB2\tA0A452G592_CAPHI\nERBB2\tA0A452G593_CAPHI\nERBB2\tA0A452G5A4_CAPHI\nERBB2\tA0A452IAV1_9SAUR\nERBB2\tA0A452QZZ9_URSAM\nERBB2\tA0A452R082_URSAM\nERBB2\tA0A452USD7_URSMA\nERBB2\tA0A455AUH3_PHYMC\nERBB2\tA0A4W2BXI4_BOBOX\nERBB2\tA0A4W2C037_BOBOX\nERBB2\tA0A4W2C0R6_BOBOX\nERBB2\tA0A4W2CFI0_BOBOX\nERBB2\tA0A4W2CFL9_BOBOX\nERBB2\tA0A4W2DTD5_BOBOX\nERBB2\tA0A4W2DYP3_BOBOX\nERBB2\tA0A4W2E0I4_BOBOX\nERBB2\tA0A4W2ECU0_BOBOX\nERBB2\tA0A4W2ILP6_BOBOX\nERBB2\tA0A4W2ILR4_BOBOX\nERBB2\tA0A4W2IPJ8_BOBOX\nERBB2\tA0A4W2IPM9_BOBOX\nERBB2\tA0A4W2IPS9_BOBOX\nERBB2\tA0A4W2IQH6_BOBOX\nERBB2\tA0A4W6CEX3_LATCA\nERBB2\tA0A4W6CJY2_LATCA\nERBB2\tA0A4W6CKM2_LATCA\nERBB2\tA0A4W6CKS8_LATCA\nERBB2\tA0A4W6CKX9_LATCA\nERBB2\tA0A4W6CL77_LATCA\nERBB2\tA0A4W6CLC1_LATCA\nERBB2\tA0A4W6CLE6_LATCA\nERBB2\tA0A4W6CLN9_LATCA\nERBB2\tA0A4W6CLS4_LATCA\nERBB2\tA0A4X1UHM8_PIG\nERBB2\tA0A4X2K6K2_VOMUR\nERBB2\tA0A5F4CMD5_CANLF\nERBB2\tA0A5F4DGC1_CANLF\nERBB2\tA0A5F5PRG3_HORSE\nERBB2\tA0A5F5Q053_HORSE\nERBB2\tA0A5F7ZIA6_MACMU\nERBB2\tA0A5F8GRK3_MONDO\nERBB2\tA0A5F9C2F2_RABIT\nERBB2\tA0A5F9DDN8_RABIT\nERBB2\tA0A5G2Q7P5_PIG\nERBB2\tA0A5G2QHP2_PIG\nERBB2\tA0A5G2QIW1_PIG\nERBB2\tA0A5G2R0J1_PIG\nERBB2\tB4DTR1_HUMAN\nERBB2\tE0Z5T2_FELCA\nERBB2\tE2FGA1_FELCA\nERBB2\tE2I6F8_HUMAN\nERBB2\tE6ZH69_DICLA\nERBB2\tERBB2_CANLF\nERBB2\tERBB2_HUMAN\nERBB2\tERBB2_MESAU\nERBB2\tERBB2_MOUSE\nERBB2\tERBB2_RAT\nERBB2\tF1LRR9_RAT\nERBB2\tF1MCQ7_BOVIN\nERBB2\tF1PIQ9_CANLF\nERBB2\tF1R5S1_DANRE\nERBB2\tF1RWM5_PIG\nERBB2\tF5H1T4_HUMAN\nERBB2\tF6VNY4_HORSE\nERBB2\tF6W9F1_CALJA\nERBB2\tF6WAF2_CALJA\nERBB2\tF7C962_MONDO\nERBB2\tF7CBG7_CALJA\nERBB2\tF7D6W0_MACMU\nERBB2\tF7D6W7_MACMU\nERBB2\tF8T7F3_MESAU\nERBB2\tG1KC71_ANOCA\nERBB2\tG1LTV6_AILME\nERBB2\tG1PRB8_MYOLU\nERBB2\tG1QK40_NOMLE\nERBB2\tG1SZL0_RABIT\nERBB2\tG3SLF2_LOXAF\nERBB2\tG3WQQ2_SARHA\nERBB2\tH0V6A0_CAVPO\nERBB2\tH0X3G4_OTOGA\nERBB2\tH2NU95_PONAB\nERBB2\tH2QCV1_PANTR\nERBB2\tH2SCZ8_TAKRU\nERBB2\tH3B4Z9_LATCH\nERBB2\tH9BB15_FELCA\nERBB2\tH9BNW8_URSAM\nERBB2\tH9F2X5_MACMU\nERBB2\tH9FGU6_MACMU\nERBB2\tH9FHS2_MACMU\nERBB2\tH9FK97_MACMU\nERBB2\tH9FK98_MACMU\nERBB2\tH9H0G6_MELGA\nERBB2\tH9NEL5_FELCA\nERBB2\tI0FM22_MACMU\nERBB2\tI3M2X8_ICTTR\nERBB2\tJ3KRI9_HUMAN\nERBB2\tJ3KTI5_HUMAN\nERBB2\tJ3QL83_HUMAN\nERBB2\tJ3QLU9_HUMAN\nERBB2\tJ3QLV2_HUMAN\nERBB2\tJ3QRJ7_HUMAN\nERBB2\tJ3QRX1_HUMAN\nERBB2\tK7DJM0_PANTR\nERBB2\tK7FH73_PELSI\nERBB2\tK7GS43_PIG\nERBB2\tL8E8G2_HUMAN\nERBB2\tM3X794_FELCA\nERBB2\tM3XK26_LATCH\nERBB2\tM3YU62_MUSPF\nERBB2\tQ153D8_XENLA\nERBB2\tQ2EJ72_CHICK\nERBB2\tQ30DU0_SHEEP\nERBB2\tQ4H1F1_HUMAN\nERBB2\tQ4H1F2_HUMAN\nERBB2\tQ4H3N3_CIOIN\nERBB2\tQ5EBY4_DANRE\nERBB2\tQ7SY19_DANRE\nERBB2\tQ80Y89_MOUSE\nERBB2\tQ8C0E7_MOUSE\nERBB2\tQ8K3F9_RAT\nERBB2\tQ8R2X1_MOUSE\nERBB2\tQ9NP09_HUMAN\nERBB2\tQ9UK79_HUMAN\nERBB2\tU3JCP1_FICAL\nERBB2\tU3JCP2_FICAL\nERBB2\tU6DKP6_NEOVI\nERBB2\tV8PE85_OPHHA\nERBB2\tW5PQJ8_SHEEP\nERBB2\tW5U6S9_ICTPU\nERBB2\tX5DNK3_HUMAN\nERBB2\tX5DQX2_HUMAN\n','From\tTo\nFLT1\tA0A096MWZ2_PAPAN\nFLT1\tA0A096N1J5_PAPAN\nFLT1\tA0A0D9RZF0_CHLSB\nFLT1\tA0A0R4J0A4_MOUSE\nFLT1\tA0A0V0SD02_9BILA\nFLT1\tA0A0V0UJ72_9BILA\nFLT1\tA0A0V0W8A1_9BILA\nFLT1\tA0A0V0YMV9_TRIPS\nFLT1\tA0A0V1A9D3_9BILA\nFLT1\tA0A0V1BY71_TRISP\nFLT1\tA0A0V1ETM4_TRIPS\nFLT1\tA0A0V1G0X8_TRIPS\nFLT1\tA0A0V1J2U6_TRIPS\nFLT1\tA0A0V1LJG4_9BILA\nFLT1\tA0A0V1MYN4_9BILA\nFLT1\tA0A0V1PFH6_9BILA\nFLT1\tA0A151M7A2_ALLMI\nFLT1\tA0A1A7XUQ9_9TELE\nFLT1\tA0A1A7YEK6_9TELE\nFLT1\tA0A1A8A1Y7_NOTFU\nFLT1\tA0A1A8CLL2_9TELE\nFLT1\tA0A1A8DNS7_9TELE\nFLT1\tA0A1A8F9E8_9TELE\nFLT1\tA0A1A8H2W7_9TELE\nFLT1\tA0A1A8IVE6_NOTKU\nFLT1\tA0A1A8K6L8_NOTKU\nFLT1\tA0A1A8MAF5_9TELE\nFLT1\tA0A1A8MB71_9TELE\nFLT1\tA0A1A8N4K6_9TELE\nFLT1\tA0A1A8PXM3_9TELE\nFLT1\tA0A1A8SAY4_9TELE\nFLT1\tA0A1A8U0A2_NOTFU\nFLT1\tA0A1D5R1D8_MACMU\nFLT1\tA0A1D5R5Q1_MACMU\nFLT1\tA0A1S2ZP90_ERIEU\nFLT1\tA0A1S3G1R9_DIPOR\nFLT1\tA0A1S3G3Y0_DIPOR\nFLT1\tA0A1U7RAQ1_MESAU\nFLT1\tA0A1V4K9R6_PATFA\nFLT1\tA0A1W2PNW4_HUMAN\nFLT1\tA0A218V6F4_9PASE\nFLT1\tA0A250Y7F3_CASCN\nFLT1\tA0A2D0PQN9_ICTPU\nFLT1\tA0A2D0PS90_ICTPU\nFLT1\tA0A2D0PUW7_ICTPU\nFLT1\tA0A2I0MCT9_COLLI\nFLT1\tA0A2I0MCU5_COLLI\nFLT1\tA0A2I2U4Q4_FELCA\nFLT1\tA0A2I3GQ91_NOMLE\nFLT1\tA0A2I3GTJ3_NOMLE\nFLT1\tA0A2I3H5S8_NOMLE\nFLT1\tA0A2I3H901_NOMLE\nFLT1\tA0A2I3HPF9_NOMLE\nFLT1\tA0A2I3LNA5_PAPAN\nFLT1\tA0A2I3MSP4_PAPAN\nFLT1\tA0A2I3N7C9_PAPAN\nFLT1\tA0A2I3RY59_PANTR\nFLT1\tA0A2I3RYL3_PANTR\nFLT1\tA0A2I3SKG4_PANTR\nFLT1\tA0A2I3TIG8_PANTR\nFLT1\tA0A2I3TTE8_PANTR\nFLT1\tA0A2I4AQE2_9TELE\nFLT1\tA0A2K5DET6_AOTNA\nFLT1\tA0A2K5DEU0_AOTNA\nFLT1\tA0A2K5DEU1_AOTNA\nFLT1\tA0A2K5DEV9_AOTNA\nFLT1\tA0A2K5DEX0_AOTNA\nFLT1\tA0A2K5JLE9_COLAP\nFLT1\tA0A2K5JLF7_COLAP\nFLT1\tA0A2K5JLH7_COLAP\nFLT1\tA0A2K5JLJ1_COLAP\nFLT1\tA0A2K5M964_CERAT\nFLT1\tA0A2K5M967_CERAT\nFLT1\tA0A2K5M969_CERAT\nFLT1\tA0A2K5M993_CERAT\nFLT1\tA0A2K5M9B0_CERAT\nFLT1\tA0A2K5S5Z9_CEBCA\nFLT1\tA0A2K5S604_CEBCA\nFLT1\tA0A2K5S607_CEBCA\nFLT1\tA0A2K5S610_CEBCA\nFLT1\tA0A2K5S621_CEBCA\nFLT1\tA0A2K5VAF2_MACFA\nFLT1\tA0A2K5VAI0_MACFA\nFLT1\tA0A2K5VAI4_MACFA\nFLT1\tA0A2K5VAI6_MACFA\nFLT1\tA0A2K5YXL1_MANLE\nFLT1\tA0A2K5YXM3_MANLE\nFLT1\tA0A2K5YXM7_MANLE\nFLT1\tA0A2K5YXN3_MANLE\nFLT1\tA0A2K5YXP9_MANLE\nFLT1\tA0A2K6BY85_MACNE\nFLT1\tA0A2K6BY90_MACNE\nFLT1\tA0A2K6BY97_MACNE\nFLT1\tA0A2K6BYA0_MACNE\nFLT1\tA0A2K6BYA8_MACNE\nFLT1\tA0A2K6FS75_PROCO\nFLT1\tA0A2K6FS96_PROCO\nFLT1\tA0A2K6FS98_PROCO\nFLT1\tA0A2K6LRG5_RHIBE\nFLT1\tA0A2K6LRJ4_RHIBE\nFLT1\tA0A2K6LRK6_RHIBE\nFLT1\tA0A2K6LRL3_RHIBE\nFLT1\tA0A2K6LRL8_RHIBE\nFLT1\tA0A2K6LRQ2_RHIBE\nFLT1\tA0A2K6RLG1_RHIRO\nFLT1\tA0A2K6RLH1_RHIRO\nFLT1\tA0A2K6RLH4_RHIRO\nFLT1\tA0A2K6RLJ0_RHIRO\nFLT1\tA0A2K6RLJ5_RHIRO\nFLT1\tA0A2K6RLJ9_RHIRO\nFLT1\tA0A2K6S741_SAIBB\nFLT1\tA0A2K6S776_SAIBB\nFLT1\tA0A2K6S798_SAIBB\nFLT1\tA0A2K6S7A6_SAIBB\nFLT1\tA0A2K6S7C1_SAIBB\nFLT1\tA0A2K6S7D6_SAIBB\nFLT1\tA0A2R8N2U5_CALJA\nFLT1\tA0A2R8NAM6_CALJA\nFLT1\tA0A2S2NUB1_SCHGA\nFLT1\tA0A2S2QFV2_9HEMI\nFLT1\tA0A2U3WYN2_ODORO\nFLT1\tA0A2U4BGM2_TURTR\nFLT1\tA0A2U4BGQ9_TURTR\nFLT1\tA0A2Y9FN39_PHYMC\nFLT1\tA0A2Y9G484_NEOSC\nFLT1\tA0A2Y9N7N5_DELLE\nFLT1\tA0A2Y9NBG9_DELLE\nFLT1\tA0A2Y9SSI8_PHYMC\nFLT1\tA0A2Y9STY9_PHYMC\nFLT1\tA0A2Y9T0K7_PHYMC\nFLT1\tA0A337SQN5_FELCA\nFLT1\tA0A340YCU6_LIPVE\nFLT1\tA0A340YJ83_LIPVE\nFLT1\tA0A341C8V3_NEOAA\nFLT1\tA0A383YN38_BALAS\nFLT1\tA0A384C6R8_URSMA\nFLT1\tA0A3B3I0U9_ORYLA\nFLT1\tA0A3Q0GXK8_ALLSI\nFLT1\tA0A3Q1MF20_BOVIN\nFLT1\tA0A3Q7PAT2_CALUR\nFLT1\tA0A3Q7S292_VULVU\nFLT1\tA0A3Q7W2C8_URSAR\nFLT1\tA0A452FA74_CAPHI\nFLT1\tA0A452FAA7_CAPHI\nFLT1\tA0A452FAB3_CAPHI\nFLT1\tA0A452H3X8_9SAUR\nFLT1\tA0A452H3Y9_9SAUR\nFLT1\tA0A452SD05_URSAM\nFLT1\tA0A452SD24_URSAM\nFLT1\tA0A452SD28_URSAM\nFLT1\tA0A452SD34_URSAM\nFLT1\tA0A452SD71_URSAM\nFLT1\tA0A452SD73_URSAM\nFLT1\tA0A452SD77_URSAM\nFLT1\tA0A452SD78_URSAM\nFLT1\tA0A455BXF6_PHYMC\nFLT1\tA0A493T307_ANAPP\nFLT1\tA0A4C1SNH7_9NEOP\nFLT1\tA0A4C1ZJE2_9NEOP\nFLT1\tA0A4W2BNB5_BOBOX\nFLT1\tA0A4W2BNC4_BOBOX\nFLT1\tA0A4W2DJY8_BOBOX\nFLT1\tA0A4W2FF14_BOBOX\nFLT1\tA0A4W2FHP4_BOBOX\nFLT1\tA0A4W6DQN2_LATCA\nFLT1\tA0A4W6DQX7_LATCA\nFLT1\tA0A4W6DRA7_LATCA\nFLT1\tA0A4W6DRC4_LATCA\nFLT1\tA0A4X1THK0_PIG\nFLT1\tA0A4X1TJC0_PIG\nFLT1\tA0A4X1TJH2_PIG\nFLT1\tA0A4X1TLW6_PIG\nFLT1\tA0A4X1TLX8_PIG\nFLT1\tA0A4X1TLZ9_PIG\nFLT1\tA0A4X1TM05_PIG\nFLT1\tA0A4X2LRL9_VOMUR\nFLT1\tA0A4X2LZ44_VOMUR\nFLT1\tA0A4X2LZ69_VOMUR\nFLT1\tA0A4X2M0E5_VOMUR\nFLT1\tA0A4Y2BNJ6_ARAVE\nFLT1\tA0A564S777_DANRE\nFLT1\tA0A5F4BXC2_CANLF\nFLT1\tA0A5F4CHQ3_CANLF\nFLT1\tA0A5F5PFZ5_HORSE\nFLT1\tA0A5F5PIL4_HORSE\nFLT1\tA0A5F7ZIZ0_MACMU\nFLT1\tA0A5F7ZNG6_MACMU\nFLT1\tA0A5F8AH06_MACMU\nFLT1\tA0A5F8G6K5_MONDO\nFLT1\tA0A5F8G7A2_MONDO\nFLT1\tA0A5F9CVA8_RABIT\nFLT1\tA0A5F9CWB7_RABIT\nFLT1\tA0A5F9CXT9_RABIT\nFLT1\tA0A5G2QFF1_PIG\nFLT1\tA0A5G3JJQ5_XENTR\nFLT1\tA4QNT7_DANRE\nFLT1\tB1AC84_HUMAN\nFLT1\tB2L481_XENBO\nFLT1\tB3DKP1_DANRE\nFLT1\tB8XRY2_MACMU\nFLT1\tE6ZIS0_DICLA\nFLT1\tF1MDD9_BOVIN\nFLT1\tF1P382_CHICK\nFLT1\tF1PKV5_CANLF\nFLT1\tF1QBD0_DANRE\nFLT1\tF1RSU5_PIG\nFLT1\tF6T133_HORSE\nFLT1\tF6Z2U8_CALJA\nFLT1\tF7DSF6_XENTR\nFLT1\tF7EY52_MONDO\nFLT1\tF7FAY2_MACMU\nFLT1\tF7GDD2_ORNAN\nFLT1\tF7HE27_CALJA\nFLT1\tF7HJ92_CALJA\nFLT1\tF7HJI3_CALJA\nFLT1\tG1KJ54_ANOCA\nFLT1\tG1L6D4_AILME\nFLT1\tG1L6D7_AILME\nFLT1\tG1NQC6_MELGA\nFLT1\tG1P289_MYOLU\nFLT1\tG1QIB2_NOMLE\nFLT1\tG1TJG0_RABIT\nFLT1\tG3TFJ0_LOXAF\nFLT1\tG3V633_RAT\nFLT1\tG3X1E1_SARHA\nFLT1\tG3X1E2_SARHA\nFLT1\tG7NJV3_MACMU\nFLT1\tH0VNJ4_CAVPO\nFLT1\tH0XBK8_OTOGA\nFLT1\tH0ZN34_TAEGU\nFLT1\tH2LXU8_ORYLA\nFLT1\tH2NJI3_PONAB\nFLT1\tH2Q7C8_PANTR\nFLT1\tH2SRM1_TAKRU\nFLT1\tH3BD36_LATCH\nFLT1\tH9EWU7_MACMU\nFLT1\tH9EZF6_MACMU\nFLT1\tH9F7C5_MACMU\nFLT1\tH9N1E7_HUMAN\nFLT1\tH9N1E8_HUMAN\nFLT1\tI3K9I3_ORENI\nFLT1\tI3K9I4_ORENI\nFLT1\tI3N109_ICTTR\nFLT1\tK7AVV7_PANTR\nFLT1\tK7G2W2_PELSI\nFLT1\tK7G2X4_PELSI\nFLT1\tL7RSL3_HUMAN\nFLT1\tM3WIL9_FELCA\nFLT1\tM3Y011_MUSPF\nFLT1\tO55095_MOUSE\nFLT1\tQ3TJC1_MOUSE\nFLT1\tQ3Y6U3_MUSPR\nFLT1\tQ4JDD5_DANRE\nFLT1\tQ5GIT3_DANRE\nFLT1\tQ78ZF4_MOUSE\nFLT1\tQ7T0V5_XENLA\nFLT1\tQ7ZTN4_XENLA\nFLT1\tQ8K1H4_MOUSE\nFLT1\tQ9CSC8_MOUSE\nFLT1\tQ9CW58_MOUSE\nFLT1\tQ9DAV2_MOUSE\nFLT1\tU3IGW5_ANAPP\nFLT1\tU3K1K4_FICAL\nFLT1\tV8N7J8_OPHHA\nFLT1\tV8NCF7_OPHHA\nFLT1\tV9Q458_HUMAN\nFLT1\tV9Q5H8_HUMAN\nFLT1\tVGFR1_CHICK\nFLT1\tVGFR1_HUMAN\nFLT1\tVGFR1_MOUSE\nFLT1\tVGFR1_RAT\nFLT1\tW5PRI4_SHEEP\nFLT1\tW5PRI5_SHEEP\nFLT1\tW8VR92_PHOSU\nFLT1\tX2C8T1_HUMAN\nFLT1\tX2CBY1_HUMAN\nFLT1\tX2QEJ0_HUMAN\nFLT1\tX2QIK8_HUMAN\n','From\tTo\nSTK25\tA0A024R4B2_HUMAN\nSTK25\tA0A087WQY6_MOUSE\nSTK25\tA0A096MVN0_PAPAN\nSTK25\tA0A096MZ40_PAPAN\nSTK25\tA0A0A9XQZ9_LYGHE\nSTK25\tA0A0B2VV71_TOXCA\nSTK25\tA0A0C9QI98_9HYME\nSTK25\tA0A0D9R0D2_CHLSB\nSTK25\tA0A0P6JCN8_HETGA\nSTK25\tA0A0S2Z4Y2_HUMAN\nSTK25\tA0A0S2Z4Y9_HUMAN\nSTK25\tA0A0S2Z503_HUMAN\nSTK25\tA0A0S7KMA6_9TELE\nSTK25\tA0A0S7LAP7_9TELE\nSTK25\tA0A0S7LBP4_9TELE\nSTK25\tA0A0V0TR53_9BILA\nSTK25\tA0A0V1A615_9BILA\nSTK25\tA0A0V1C2K7_TRISP\nSTK25\tA0A0V1CIH0_TRIBR\nSTK25\tA0A0V1E2K3_TRIPS\nSTK25\tA0A0V1FM35_TRIPS\nSTK25\tA0A0V1H621_9BILA\nSTK25\tA0A0V1L251_9BILA\nSTK25\tA0A0V1P016_9BILA\nSTK25\tA0A0X3P908_SCHSO\nSTK25\tA0A146LJ33_LYGHE\nSTK25\tA0A151MK27_ALLMI\nSTK25\tA0A1D5PPH4_CHICK\nSTK25\tA0A1D5Q1Q1_MACMU\nSTK25\tA0A1D5R3R2_MACMU\nSTK25\tA0A1D5RIW1_MACMU\nSTK25\tA0A1S3GLL5_DIPOR\nSTK25\tA0A1S3R0D4_SALSA\nSTK25\tA0A1S3WMH5_ERIEU\nSTK25\tA0A1U7QMV8_MESAU\nSTK25\tA0A1U7SND0_ALLSI\nSTK25\tA0A1U7SQV5_ALLSI\nSTK25\tA0A1U7SV51_TARSY\nSTK25\tA0A1V4J4X1_PATFA\nSTK25\tA0A1Y1S645_9MICR\nSTK25\tA0A218UBX0_9PASE\nSTK25\tA0A250Y0N9_CASCN\nSTK25\tA0A286ZVI3_PIG\nSTK25\tA0A287B1F1_PIG\nSTK25\tA0A2C5WUX8_9PEZI\nSTK25\tA0A2I0LMG0_COLLI\nSTK25\tA0A2I3GX21_NOMLE\nSTK25\tA0A2I3M7Y0_PAPAN\nSTK25\tA0A2I3RE17_PANTR\nSTK25\tA0A2I3RWJ1_PANTR\nSTK25\tA0A2I3SFB9_PANTR\nSTK25\tA0A2I3SYH9_PANTR\nSTK25\tA0A2I4CJ01_9TELE\nSTK25\tA0A2J7QPF0_9NEOP\nSTK25\tA0A2J8IV21_PANTR\nSTK25\tA0A2J8RPQ2_PONAB\nSTK25\tA0A2K5ETH8_AOTNA\nSTK25\tA0A2K5ETP4_AOTNA\nSTK25\tA0A2K5I8P0_COLAP\nSTK25\tA0A2K5P447_CERAT\nSTK25\tA0A2K5P457_CERAT\nSTK25\tA0A2K5PUW5_CEBCA\nSTK25\tA0A2K5PUX2_CEBCA\nSTK25\tA0A2K5VLZ2_MACFA\nSTK25\tA0A2K5VM28_MACFA\nSTK25\tA0A2K5VM74_MACFA\nSTK25\tA0A2K6AI92_MANLE\nSTK25\tA0A2K6AM82_MACNE\nSTK25\tA0A2K6AM96_MACNE\nSTK25\tA0A2K6AMC4_MACNE\nSTK25\tA0A2K6G7U2_PROCO\nSTK25\tA0A2K6G7U4_PROCO\nSTK25\tA0A2K6LM83_RHIBE\nSTK25\tA0A2K6LMC3_RHIBE\nSTK25\tA0A2K6QXI2_RHIRO\nSTK25\tA0A2K6QXJ5_RHIRO\nSTK25\tA0A2K6QXK3_RHIRO\nSTK25\tA0A2K6TN11_SAIBB\nSTK25\tA0A2K6TN49_SAIBB\nSTK25\tA0A2P8XI94_BLAGE\nSTK25\tA0A2R8ZP31_PANPA\nSTK25\tA0A2R8ZP42_PANPA\nSTK25\tA0A2R8ZS15_PANPA\nSTK25\tA0A2S2N8F6_SCHGA\nSTK25\tA0A2S2QU84_9HEMI\nSTK25\tA0A2U3WTP7_ODORO\nSTK25\tA0A2U3ZTA2_ODORO\nSTK25\tA0A2U3ZU88_ODORO\nSTK25\tA0A2Y9GAC4_NEOSC\nSTK25\tA0A2Y9MPP0_DELLE\nSTK25\tA0A2Y9SSW6_PHYMC\nSTK25\tA0A337SBS1_FELCA\nSTK25\tA0A340YAC0_LIPVE\nSTK25\tA0A340YC47_LIPVE\nSTK25\tA0A340YDH9_LIPVE\nSTK25\tA0A341C9R7_NEOAA\nSTK25\tA0A346B538_CTEID\nSTK25\tA0A384AC36_BALAS\nSTK25\tA0A384C2V0_URSMA\nSTK25\tA0A3Q0DMD6_TARSY\nSTK25\tA0A3Q0DQ42_TARSY\nSTK25\tA0A3Q2I8R0_HORSE\nSTK25\tA0A3Q7QEH3_CALUR\nSTK25\tA0A3Q7RML6_VULVU\nSTK25\tA0A3Q7RUB6_CALUR\nSTK25\tA0A3Q7UEE9_URSAR\nSTK25\tA0A419QEJ3_CLOSI\nSTK25\tA0A452FKF2_CAPHI\nSTK25\tA0A452FKQ3_CAPHI\nSTK25\tA0A452GUQ9_9SAUR\nSTK25\tA0A452QX13_URSAM\nSTK25\tA0A452U102_URSMA\nSTK25\tA0A455AS02_PHYMC\nSTK25\tA0A4W2EUZ9_BOBOX\nSTK25\tA0A4W2G8F5_BOBOX\nSTK25\tA0A4W2G969_BOBOX\nSTK25\tA0A4W2I003_BOBOX\nSTK25\tA0A4W3ITH0_CALMI\nSTK25\tA0A4W3ITH7_CALMI\nSTK25\tA0A4W3J3B2_CALMI\nSTK25\tA0A4W3J3B7_CALMI\nSTK25\tA0A4W3JKQ8_CALMI\nSTK25\tA0A4X1SX80_PIG\nSTK25\tA0A4X1SY29_PIG\nSTK25\tA0A4X1SZA7_PIG\nSTK25\tA0A4X2M1Z8_VOMUR\nSTK25\tA0A4Y2DK10_ARAVE\nSTK25\tA0A4Z2HVY5_9TELE\nSTK25\tA0A507QXM4_MONPU\nSTK25\tA0A5B7F5R7_PORTR\nSTK25\tA0A5F4BVZ2_CANLF\nSTK25\tA0A5F4C2W3_CANLF\nSTK25\tA0A5F4VRI6_CALJA\nSTK25\tA0A5G3IEX1_XENTR\nSTK25\tA0A5G3KLW1_XENTR\nSTK25\tA0A5K1UU98_MACMU\nSTK25\tB5X484_SALSA\nSTK25\tC1LFW7_SCHJA\nSTK25\tC9J232_HUMAN\nSTK25\tC9J6L2_HUMAN\nSTK25\tC9J8E8_HUMAN\nSTK25\tC9JCC0_HUMAN\nSTK25\tC9JDH9_HUMAN\nSTK25\tC9JFA1_HUMAN\nSTK25\tC9JIH8_HUMAN\nSTK25\tC9JJV0_HUMAN\nSTK25\tC9JN58_HUMAN\nSTK25\tC9K0M6_HUMAN\nSTK25\tD3Z359_MOUSE\nSTK25\tE3TEI7_ICTPU\nSTK25\tE7EM58_HUMAN\nSTK25\tF1PTW5_CANLF\nSTK25\tF1SIN8_PIG\nSTK25\tF6R393_HORSE\nSTK25\tF6T1J5_ORNAN\nSTK25\tF7CTK5_XENTR\nSTK25\tF7CZ65_XENTR\nSTK25\tF7EP46_XENTR\nSTK25\tF7I1T0_CALJA\nSTK25\tG1K945_ANOCA\nSTK25\tG1MDK8_AILME\nSTK25\tG1MXQ1_MELGA\nSTK25\tG1P8M1_MYOLU\nSTK25\tG1RUT6_NOMLE\nSTK25\tG3UE93_LOXAF\nSTK25\tG3US19_MELGA\nSTK25\tG3WKP7_SARHA\nSTK25\tH0WCN8_CAVPO\nSTK25\tH0WYQ0_OTOGA\nSTK25\tH0Z6U3_TAEGU\nSTK25\tH2LCC3_ORYLA\nSTK25\tH2LJ57_ORYLA\nSTK25\tH2RLF0_TAKRU\nSTK25\tH2RLF1_TAKRU\nSTK25\tH3ARF4_LATCH\nSTK25\tH7C279_HUMAN\nSTK25\tH9ELY8_MACMU\nSTK25\tI3JWU5_ORENI\nSTK25\tI3MUI1_ICTTR\nSTK25\tK7APK1_PANTR\nSTK25\tK7B6E3_PANTR\nSTK25\tK7EU27_PONAB\nSTK25\tK7FJN7_PELSI\nSTK25\tM3VZ60_FELCA\nSTK25\tM3XM73_MUSPF\nSTK25\tQ1JPC4_BOVIN\nSTK25\tQ3U6Y0_MOUSE\nSTK25\tQ4FZE2_XENTR\nSTK25\tQ5HZD3_XENTR\nSTK25\tQ6NRY7_XENLA\nSTK25\tQ6V9V9_RAT\nSTK25\tQ7TPV1_MOUSE\nSTK25\tQ7ZVG8_DANRE\nSTK25\tSTK25_BOVIN\nSTK25\tSTK25_HUMAN\nSTK25\tSTK25_MOUSE\nSTK25\tT2MIU6_HYDVU\nSTK25\tU3DAV0_CALJA\nSTK25\tU3E0T8_CALJA\nSTK25\tU3F9D7_CALJA\nSTK25\tU3FDK8_CALJA\nSTK25\tU3JLA5_FICAL\nSTK25\tU6DUS0_NEOVI\nSTK25\tV8NK81_OPHHA\nSTK25\tW5Q8B9_SHEEP\nSTK25\tW5U8L6_ICTPU\n','From\tTo\nCSF1R\tA0A034VGN7_BACDO\nCSF1R\tA0A087XDG3_POEFO\nCSF1R\tA0A096ND57_PAPAN\nCSF1R\tA0A096P010_PAPAN\nCSF1R\tA0A0B5CV26_XENLA\nCSF1R\tA0A0D9RGZ5_CHLSB\nCSF1R\tA0A0G2K3Y5_RAT\nCSF1R\tA0A0G2KBC4_RAT\nCSF1R\tA0A0H3WMK7_CTEID\nCSF1R\tA0A162BS36_DICLA\nCSF1R\tA0A172J2F2_PLEAT\nCSF1R\tA0A1D5QZE0_MACMU\nCSF1R\tA0A1S2ZAN2_ERIEU\nCSF1R\tA0A1S3FY95_DIPOR\nCSF1R\tA0A1U7QU48_MESAU\nCSF1R\tA0A1U7QWR1_MESAU\nCSF1R\tA0A1U7TZZ3_TARSY\nCSF1R\tA0A1U8D968_ALLSI\nCSF1R\tA0A1V4KUF4_PATFA\nCSF1R\tA0A218V0L2_9PASE\nCSF1R\tA0A250YLR5_CASCN\nCSF1R\tA0A287CUR7_ICTTR\nCSF1R\tA0A2D0SCK2_ICTPU\nCSF1R\tA0A2D0SDY1_ICTPU\nCSF1R\tA0A2I0M9Z8_COLLI\nCSF1R\tA0A2I2UX09_FELCA\nCSF1R\tA0A2I3HCD9_NOMLE\nCSF1R\tA0A2I3SWC5_PANTR\nCSF1R\tA0A2I3TM41_PANTR\nCSF1R\tA0A2I4B7B8_9TELE\nCSF1R\tA0A2K5DHS2_AOTNA\nCSF1R\tA0A2K5DHT7_AOTNA\nCSF1R\tA0A2K5IJ08_COLAP\nCSF1R\tA0A2K5IJ14_COLAP\nCSF1R\tA0A2K5IJ96_COLAP\nCSF1R\tA0A2K5KLP5_CERAT\nCSF1R\tA0A2K5KLR8_CERAT\nCSF1R\tA0A2K5RTL9_CEBCA\nCSF1R\tA0A2K5RTM7_CEBCA\nCSF1R\tA0A2K5WG90_MACFA\nCSF1R\tA0A2K5WG91_MACFA\nCSF1R\tA0A2K5WG93_MACFA\nCSF1R\tA0A2K5ZNF8_MANLE\nCSF1R\tA0A2K5ZNI2_MANLE\nCSF1R\tA0A2K5ZNI7_MANLE\nCSF1R\tA0A2K6DJG1_MACNE\nCSF1R\tA0A2K6DJH8_MACNE\nCSF1R\tA0A2K6DJI5_MACNE\nCSF1R\tA0A2K6DJJ4_MACNE\nCSF1R\tA0A2K6GJS1_PROCO\nCSF1R\tA0A2K6KT22_RHIBE\nCSF1R\tA0A2K6KT23_RHIBE\nCSF1R\tA0A2K6KT35_RHIBE\nCSF1R\tA0A2K6ND50_RHIRO\nCSF1R\tA0A2K6ND74_RHIRO\nCSF1R\tA0A2K6ND84_RHIRO\nCSF1R\tA0A2K6T497_SAIBB\nCSF1R\tA0A2K6T4A3_SAIBB\nCSF1R\tA0A2R9C7W3_PANPA\nCSF1R\tA0A2R9C866_PANPA\nCSF1R\tA0A2U3W492_ODORO\nCSF1R\tA0A2U3YD79_LEPWE\nCSF1R\tA0A2Y9HXX0_NEOSC\nCSF1R\tA0A2Y9PSB0_DELLE\nCSF1R\tA0A340WDB7_LIPVE\nCSF1R\tA0A341AU54_NEOAA\nCSF1R\tA0A384AN03_BALAS\nCSF1R\tA0A384C8D9_URSMA\nCSF1R\tA0A384C8M7_URSMA\nCSF1R\tA0A3B3X4S9_9TELE\nCSF1R\tA0A3B3YD18_9TELE\nCSF1R\tA0A3Q0GAU0_ALLSI\nCSF1R\tA0A3Q0GBG6_ALLSI\nCSF1R\tA0A3Q0GEB7_ALLSI\nCSF1R\tA0A3Q7NXV4_CALUR\nCSF1R\tA0A3Q7T5A9_VULVU\nCSF1R\tA0A3Q7UAR2_VULVU\nCSF1R\tA0A3Q7W022_URSAR\nCSF1R\tA0A3Q7XTU3_URSAR\nCSF1R\tA0A3Q9B2L2_9TELE\nCSF1R\tA0A3R7C241_CLOSI\nCSF1R\tA0A452G8V1_CAPHI\nCSF1R\tA0A452SIC6_URSAM\nCSF1R\tA0A455BN09_PHYMC\nCSF1R\tA0A494BA23_MOUSE\nCSF1R\tA0A494BBK8_MOUSE\nCSF1R\tA0A4C1T3F0_9NEOP\nCSF1R\tA0A4W2DKW0_BOBOX\nCSF1R\tA0A4W2F1H9_BOBOX\nCSF1R\tA0A4W2GF08_BOBOX\nCSF1R\tA0A4W2GI33_BOBOX\nCSF1R\tA0A4W2GI50_BOBOX\nCSF1R\tA0A4X1T8I8_PIG\nCSF1R\tA0A4X1T9D8_PIG\nCSF1R\tA0A4X1TA18_PIG\nCSF1R\tA0A4X2KFG0_VOMUR\nCSF1R\tA0A4Z2JGS7_9TELE\nCSF1R\tA0A5F4CGY4_CANLF\nCSF1R\tA0A5F4CMG7_CANLF\nCSF1R\tA0A5F4WJV8_CALJA\nCSF1R\tA0A5F5PL64_HORSE\nCSF1R\tA0A5F5XD98_FELCA\nCSF1R\tA0A5F5XJW6_FELCA\nCSF1R\tA0A5F5XQG4_FELCA\nCSF1R\tA0A5F8AB69_MACMU\nCSF1R\tA0A5F9CM04_RABIT\nCSF1R\tA0A5F9CUU0_RABIT\nCSF1R\tA0A5F9CZ43_RABIT\nCSF1R\tA0A5F9D8B6_RABIT\nCSF1R\tA0A5F9DHG0_RABIT\nCSF1R\tA0A5F9DLX2_RABIT\nCSF1R\tA0A5F9DVQ4_RABIT\nCSF1R\tA0A5G2QG32_PIG\nCSF1R\tA0A5G2QIU0_PIG\nCSF1R\tA0A5G2R619_PIG\nCSF1R\tA0A5G3HI00_XENTR\nCSF1R\tA0A5G3HIA2_XENTR\nCSF1R\tA0A5G3HJE2_XENTR\nCSF1R\tA0A5G3HQN9_XENTR\nCSF1R\tA0A5G3HR97_XENTR\nCSF1R\tA0A5G3HRC2_XENTR\nCSF1R\tA0A5G3J6C2_XENTR\nCSF1R\tA0A5G3J6E5_XENTR\nCSF1R\tA0A5G3J6U0_XENTR\nCSF1R\tA0A5G3JF81_XENTR\nCSF1R\tA0A5G3JFT7_XENTR\nCSF1R\tA0A5G3JG78_XENTR\nCSF1R\tA2VDG3_HUMAN\nCSF1R\tA4JYH8_DANRE\nCSF1R\tA7Z067_BOVIN\nCSF1R\tB3DHL0_DANRE\nCSF1R\tB5UB81_PAROL\nCSF1R\tCSF1R_DANRE\nCSF1R\tCSF1R_FELCA\nCSF1R\tCSF1R_HUMAN\nCSF1R\tCSF1R_MOUSE\nCSF1R\tCSF1R_RAT\nCSF1R\tD3KYQ4_PAROL\nCSF1R\tD4ACA7_RAT\nCSF1R\tD6RGW1_HUMAN\nCSF1R\tD8UWE3_TAEGU\nCSF1R\tE2RJV2_CANLF\nCSF1R\tE9PEK4_HUMAN\nCSF1R\tF1N869_CHICK\nCSF1R\tF1RL77_PIG\nCSF1R\tF6RTV0_XENTR\nCSF1R\tF6TNL5_MACMU\nCSF1R\tF6UHZ8_XENTR\nCSF1R\tF6UIA0_XENTR\nCSF1R\tF6VSR1_XENTR\nCSF1R\tF6VTH8_XENTR\nCSF1R\tF6WT52_MONDO\nCSF1R\tF6YXS1_HORSE\nCSF1R\tF6Z3F8_XENTR\nCSF1R\tF7AB14_XENTR\nCSF1R\tF7BTH9_MACMU\nCSF1R\tF7FF76_CALJA\nCSF1R\tF7FU59_CALJA\nCSF1R\tG1KRD5_ANOCA\nCSF1R\tG1LEV7_AILME\nCSF1R\tG1N542_MELGA\nCSF1R\tG1PMB9_MYOLU\nCSF1R\tG1RGX9_NOMLE\nCSF1R\tG1TTE0_RABIT\nCSF1R\tG3THM7_LOXAF\nCSF1R\tG3WX04_SARHA\nCSF1R\tG3WX05_SARHA\nCSF1R\tH0VVM7_CAVPO\nCSF1R\tH0WQ88_OTOGA\nCSF1R\tH0YAL5_HUMAN\nCSF1R\tH0YRT7_TAEGU\nCSF1R\tH2PH24_PONAB\nCSF1R\tH2QRS7_PANTR\nCSF1R\tH3B2Z5_LATCH\nCSF1R\tH9FQ91_MACMU\nCSF1R\tI3K5N3_ORENI\nCSF1R\tI3K5N4_ORENI\nCSF1R\tI3LJV6_PIG\nCSF1R\tI3MYD4_ICTTR\nCSF1R\tK7G321_PELSI\nCSF1R\tK9IVS4_PIG\nCSF1R\tM3W0P5_FELCA\nCSF1R\tM3XGP0_LATCH\nCSF1R\tM3YRD5_MUSPF\nCSF1R\tQ0P635_MOUSE\nCSF1R\tQ14085_HUMAN\nCSF1R\tQ1JP72_BOVIN\nCSF1R\tQ3B7T5_RAT\nCSF1R\tQ3TAZ7_MOUSE\nCSF1R\tQ3TBH6_MOUSE\nCSF1R\tQ3U1X4_MOUSE\nCSF1R\tQ3U1Y3_MOUSE\nCSF1R\tQ3U210_MOUSE\nCSF1R\tQ3U3W0_MOUSE\nCSF1R\tQ3U4N8_MOUSE\nCSF1R\tQ3UKC6_MOUSE\nCSF1R\tQ4GWZ1_SPAAU\nCSF1R\tQ4W447_HORSE\nCSF1R\tQ5ND91_SPAAU\nCSF1R\tQ640T5_XENTR\nCSF1R\tQ6NXV8_MOUSE\nCSF1R\tU3IZY8_ANAPP\nCSF1R\tU3JZS3_FICAL\nCSF1R\tU6D1L4_NEOVI\nCSF1R\tW5P8R4_SHEEP\nCSF1R\tW5UK60_ICTPU\n','From\tTo\nSTK24\tA0A0D9RY58_CHLSB\nSTK24\tA0A0P6JYF2_HETGA\nSTK24\tA0A0S7GPQ1_9TELE\nSTK24\tA0A0S7GQ98_9TELE\nSTK24\tA0A0S7GQJ3_9TELE\nSTK24\tA0A0S7GTQ8_9TELE\nSTK24\tA0A0S7LXJ0_9TELE\nSTK24\tA0A0S7M519_9TELE\nSTK24\tA0A1L8H969_XENLA\nSTK24\tA0A1S2ZN98_ERIEU\nSTK24\tA0A1S3FRG8_DIPOR\nSTK24\tA0A1S3PMC8_SALSA\nSTK24\tA0A1U7QUK5_MESAU\nSTK24\tA0A1U7S8M2_ALLSI\nSTK24\tA0A1U7V0B1_TARSY\nSTK24\tA0A1V4J3X3_PATFA\nSTK24\tA0A1V4J430_PATFA\nSTK24\tA0A1V4J437_PATFA\nSTK24\tA0A218VAP2_9PASE\nSTK24\tA0A250Y0S8_CASCN\nSTK24\tA0A286XE58_CAVPO\nSTK24\tA0A286XJD4_CAVPO\nSTK24\tA0A287B0C4_PIG\nSTK24\tA0A287B723_PIG\nSTK24\tA0A287DDY9_ICTTR\nSTK24\tA0A2B4RJ21_STYPI\nSTK24\tA0A2D0R4N4_ICTPU\nSTK24\tA0A2D0R4R3_ICTPU\nSTK24\tA0A2I0MT81_COLLI\nSTK24\tA0A2I2UQ39_FELCA\nSTK24\tA0A2I2V5B4_FELCA\nSTK24\tA0A2I3G6H2_NOMLE\nSTK24\tA0A2I3GY64_NOMLE\nSTK24\tA0A2I3HQV2_NOMLE\nSTK24\tA0A2I3RCD0_PANTR\nSTK24\tA0A2I3RLH5_PANTR\nSTK24\tA0A2I3RTU2_PANTR\nSTK24\tA0A2K5CRV4_AOTNA\nSTK24\tA0A2K5CRX1_AOTNA\nSTK24\tA0A2K5CRX6_AOTNA\nSTK24\tA0A2K5CRY0_AOTNA\nSTK24\tA0A2K5CRY2_AOTNA\nSTK24\tA0A2K5IFP7_COLAP\nSTK24\tA0A2K5IFS2_COLAP\nSTK24\tA0A2K5IG48_COLAP\nSTK24\tA0A2K5L813_CERAT\nSTK24\tA0A2K5L820_CERAT\nSTK24\tA0A2K5S002_CEBCA\nSTK24\tA0A2K5S019_CEBCA\nSTK24\tA0A2K5S024_CEBCA\nSTK24\tA0A2K5S030_CEBCA\nSTK24\tA0A2K5S033_CEBCA\nSTK24\tA0A2K5TTV3_MACFA\nSTK24\tA0A2K5TTW9_MACFA\nSTK24\tA0A2K5TTY2_MACFA\nSTK24\tA0A2K5TU16_MACFA\nSTK24\tA0A2K5TU21_MACFA\nSTK24\tA0A2K5ZVF7_MANLE\nSTK24\tA0A2K5ZVG4_MANLE\nSTK24\tA0A2K5ZVH1_MANLE\nSTK24\tA0A2K5ZVH9_MANLE\nSTK24\tA0A2K6E4I9_MACNE\nSTK24\tA0A2K6E4K2_MACNE\nSTK24\tA0A2K6E4K3_MACNE\nSTK24\tA0A2K6E4L1_MACNE\nSTK24\tA0A2K6E4M2_MACNE\nSTK24\tA0A2K6G7F8_PROCO\nSTK24\tA0A2K6G7G2_PROCO\nSTK24\tA0A2K6G7H5_PROCO\nSTK24\tA0A2K6G7H8_PROCO\nSTK24\tA0A2K6LA06_RHIBE\nSTK24\tA0A2K6LA09_RHIBE\nSTK24\tA0A2K6LA14_RHIBE\nSTK24\tA0A2K6LA16_RHIBE\nSTK24\tA0A2K6LA18_RHIBE\nSTK24\tA0A2K6LA23_RHIBE\nSTK24\tA0A2K6LA29_RHIBE\nSTK24\tA0A2K6QFX6_RHIRO\nSTK24\tA0A2K6QFY3_RHIRO\nSTK24\tA0A2K6QFZ4_RHIRO\nSTK24\tA0A2K6QFZ7_RHIRO\nSTK24\tA0A2K6ST43_SAIBB\nSTK24\tA0A2K6ST51_SAIBB\nSTK24\tA0A2K6ST71_SAIBB\nSTK24\tA0A2K6ST78_SAIBB\nSTK24\tA0A2K6ST82_SAIBB\nSTK24\tA0A2R8MPQ5_CALJA\nSTK24\tA0A2R8PEY8_CALJA\nSTK24\tA0A2R8ZDV1_PANPA\nSTK24\tA0A2R8ZDX1_PANPA\nSTK24\tA0A2R8ZH52_PANPA\nSTK24\tA0A2R8ZK29_PANPA\nSTK24\tA0A2U3W9M0_ODORO\nSTK24\tA0A2U3W9N0_ODORO\nSTK24\tA0A2U3ZK78_ODORO\nSTK24\tA0A2U4A6T9_TURTR\nSTK24\tA0A2Y9HJ66_NEOSC\nSTK24\tA0A2Y9NEU9_DELLE\nSTK24\tA0A337SJH5_FELCA\nSTK24\tA0A337SWJ9_FELCA\nSTK24\tA0A340Y1W5_LIPVE\nSTK24\tA0A341B0C1_NEOAA\nSTK24\tA0A341B0D9_NEOAA\nSTK24\tA0A341B1M5_NEOAA\nSTK24\tA0A346B537_CTEID\nSTK24\tA0A384BB29_BALAS\nSTK24\tA0A384BKT9_URSMA\nSTK24\tA0A384BLR5_URSMA\nSTK24\tA0A384BM77_URSMA\nSTK24\tA0A3B3D5C9_ORYME\nSTK24\tA0A3B4CVQ2_PYGNA\nSTK24\tA0A3B4CXT8_PYGNA\nSTK24\tA0A3P8Z871_ESOLU\nSTK24\tA0A3Q0GXD1_ALLSI\nSTK24\tA0A3Q0GXD7_ALLSI\nSTK24\tA0A3Q0GY76_ALLSI\nSTK24\tA0A3Q0H1Y2_ALLSI\nSTK24\tA0A3Q0H1Y9_ALLSI\nSTK24\tA0A3Q0SRN3_AMPCI\nSTK24\tA0A3Q1M0C2_BOVIN\nSTK24\tA0A3Q2HH95_HORSE\nSTK24\tA0A3Q2HQI5_HORSE\nSTK24\tA0A3Q2P122_FUNHE\nSTK24\tA0A3Q2P161_FUNHE\nSTK24\tA0A3Q2P1A0_FUNHE\nSTK24\tA0A3Q2SY34_FUNHE\nSTK24\tA0A3Q2U4Z7_FUNHE\nSTK24\tA0A3Q2XHZ9_HIPCM\nSTK24\tA0A3Q3FSM1_9LABR\nSTK24\tA0A3Q7PA71_CALUR\nSTK24\tA0A3Q7SFD4_VULVU\nSTK24\tA0A3Q7SRR7_VULVU\nSTK24\tA0A3Q7VJS2_URSAR\nSTK24\tA0A3Q7WG15_URSAR\nSTK24\tA0A438BSZ0_VITVI\nSTK24\tA0A452ELZ9_CAPHI\nSTK24\tA0A452EM41_CAPHI\nSTK24\tA0A452EMQ4_CAPHI\nSTK24\tA0A452HBJ1_9SAUR\nSTK24\tA0A452SV79_URSAM\nSTK24\tA0A452SVB9_URSAM\nSTK24\tA0A455AXK0_PHYMC\nSTK24\tA0A455AYG3_PHYMC\nSTK24\tA0A455AZH3_PHYMC\nSTK24\tA0A455AZH9_PHYMC\nSTK24\tA0A455BBB7_PHYMC\nSTK24\tA0A493TB35_ANAPP\nSTK24\tA0A4W2CSK0_BOBOX\nSTK24\tA0A4W4GC97_ELEEL\nSTK24\tA0A4W4GG67_ELEEL\nSTK24\tA0A4W5P3M4_9TELE\nSTK24\tA0A4W5P766_9TELE\nSTK24\tA0A4W5PAM5_9TELE\nSTK24\tA0A4W5PAQ1_9TELE\nSTK24\tA0A4W5PC02_9TELE\nSTK24\tA0A4W5PC88_9TELE\nSTK24\tA0A4W5PEB4_9TELE\nSTK24\tA0A4W5PEC3_9TELE\nSTK24\tA0A4W6G4Z0_LATCA\nSTK24\tA0A4W6G5H9_LATCA\nSTK24\tA0A4W6G5I4_LATCA\nSTK24\tA0A4W6G6Y4_LATCA\nSTK24\tA0A4W6G7T6_LATCA\nSTK24\tA0A4W6G7U7_LATCA\nSTK24\tA0A4X1UQW8_PIG\nSTK24\tA0A4X1UQY3_PIG\nSTK24\tA0A4X1USG8_PIG\nSTK24\tA0A4X1UUR2_PIG\nSTK24\tA0A4X2M5G3_VOMUR\nSTK24\tA0A5F4CCI6_CANLF\nSTK24\tA0A5F4CUV2_CANLF\nSTK24\tA0A5F5PJU9_HORSE\nSTK24\tA0A5F5PXP7_HORSE\nSTK24\tA0A5F8GX80_MONDO\nSTK24\tA0A5F9C1C8_RABIT\nSTK24\tA0A5F9C1Z7_RABIT\nSTK24\tA0A5F9C310_RABIT\nSTK24\tA0A5F9CIN6_RABIT\nSTK24\tA0A5F9D2R7_RABIT\nSTK24\tA0A5F9DA19_RABIT\nSTK24\tA0A5F9DPG4_RABIT\nSTK24\tA0A5F9DQG6_RABIT\nSTK24\tA0A5G2QR71_PIG\nSTK24\tA0A5G2RG17_PIG\nSTK24\tA0A5K1UYR7_CALJA\nSTK24\tA2VE07_BOVIN\nSTK24\tB3DL35_XENTR\nSTK24\tB4DR80_HUMAN\nSTK24\tF1N5S6_BOVIN\nSTK24\tF1PTW4_CANLF\nSTK24\tF1PTW6_CANLF\nSTK24\tF1RP42_PIG\nSTK24\tF6QUQ9_CALJA\nSTK24\tF6XBH7_XENTR\nSTK24\tF7BC69_ORNAN\nSTK24\tF7BN05_MACMU\nSTK24\tF7CDT9_MONDO\nSTK24\tF7E0W2_HORSE\nSTK24\tF7H3J5_MACMU\nSTK24\tF7H3K3_MACMU\nSTK24\tG1KSP2_ANOCA\nSTK24\tG1LBE2_AILME\nSTK24\tG1NPW0_MELGA\nSTK24\tG1PLI2_MYOLU\nSTK24\tG1R8M9_NOMLE\nSTK24\tG1U840_RABIT\nSTK24\tG3NK11_GASAC\nSTK24\tG3NK18_GASAC\nSTK24\tG3SVV1_LOXAF\nSTK24\tG3VTJ7_SARHA\nSTK24\tH0V866_CAVPO\nSTK24\tH0WWL9_OTOGA\nSTK24\tH0Y630_HUMAN\nSTK24\tH0ZLC5_TAEGU\nSTK24\tH2LCC4_ORYLA\nSTK24\tH2NK70_PONAB\nSTK24\tH2RA15_PANTR\nSTK24\tH3ATW7_LATCH\nSTK24\tH9KVF3_RAT\nSTK24\tI3M8B7_ICTTR\nSTK24\tK7AL57_PANTR\nSTK24\tK7F5J1_PELSI\nSTK24\tL8E7H5_HUMAN\nSTK24\tM3WLG2_FELCA\nSTK24\tM3YJI8_MUSPF\nSTK24\tQ3U335_MOUSE\nSTK24\tQ3U3T6_MOUSE\nSTK24\tQ5F375_CHICK\nSTK24\tQ5JV98_HUMAN\nSTK24\tQ5JV99_HUMAN\nSTK24\tQ5U0E6_HUMAN\nSTK24\tQ6FG81_HUMAN\nSTK24\tQ6GP83_XENLA\nSTK24\tQ6P0Y1_HUMAN\nSTK24\tQ7SYN3_DANRE\nSTK24\tSTK24_HUMAN\nSTK24\tSTK24_MOUSE\nSTK24\tSTK24_RAT\nSTK24\tU3E6I1_CALJA\nSTK24\tU3I5U5_ANAPP\nSTK24\tU3JP75_FICAL\nSTK24\tU6DW94_NEOVI\nSTK24\tV8NZ79_OPHHA\nSTK24\tW5KLQ6_ASTMX\nSTK24\tW5M6X6_LEPOC\nSTK24\tW5M6Z2_LEPOC\nSTK24\tW5NZ04_SHEEP\nSTK24\tW5NZ06_SHEEP\nSTK24\tW8BHJ7_CERCA\nSTK24\tW8BUV7_CERCA\n','From\tTo\nPAK7\tA0A0G2KNJ7_DANRE\nPAK7\tA0A0S7ERG4_9TELE\nPAK7\tA0A0S7J1I9_9TELE\nPAK7\tA0A0S7M8W6_9TELE\nPAK7\tA0A0V0SAC5_9BILA\nPAK7\tA0A0V0SAC8_9BILA\nPAK7\tA0A0V0SAH2_9BILA\nPAK7\tA0A0V0SBD6_9BILA\nPAK7\tA0A151MGV2_ALLMI\nPAK7\tA0A151MGV3_ALLMI\nPAK7\tA0A1A7W6X1_9TELE\nPAK7\tA0A1A7WMJ3_9TELE\nPAK7\tA0A1A8AF37_NOTFU\nPAK7\tA0A1A8AYH3_NOTFU\nPAK7\tA0A1A8BD02_9TELE\nPAK7\tA0A1A8BSP6_9TELE\nPAK7\tA0A1A8CSM2_9TELE\nPAK7\tA0A1A8E903_9TELE\nPAK7\tA0A1A8FLH8_9TELE\nPAK7\tA0A1A8G1T9_9TELE\nPAK7\tA0A1A8GIP8_9TELE\nPAK7\tA0A1A8I1J6_NOTKU\nPAK7\tA0A1A8IGE1_NOTKU\nPAK7\tA0A1A8JAN6_NOTKU\nPAK7\tA0A1A8JKK9_NOTKU\nPAK7\tA0A1A8JMT1_NOTKU\nPAK7\tA0A1A8LTX1_9TELE\nPAK7\tA0A1A8MWC6_9TELE\nPAK7\tA0A1A8NK69_9TELE\nPAK7\tA0A1A8PDH6_9TELE\nPAK7\tA0A1A8QSF2_9TELE\nPAK7\tA0A1A8SDU9_9TELE\nPAK7\tA0A1A8SEQ9_9TELE\nPAK7\tA0A1A8UPG8_NOTFU\nPAK7\tA0A1A8UQR9_NOTFU\nPAK7\tA0A1S3AS98_ERIEU\nPAK7\tA0A1S3ASA4_ERIEU\nPAK7\tA0A1S3EQL1_DIPOR\nPAK7\tA0A1V4IH55_PATFA\nPAK7\tA0A218V884_9PASE\nPAK7\tA0A2I0M9E3_COLLI\nPAK7\tA0A2U3WYV8_ODORO\nPAK7\tA0A2U3XNM0_LEPWE\nPAK7\tA0A340WYC1_LIPVE\nPAK7\tA0A340WZ25_LIPVE\nPAK7\tA0A384D3B5_URSMA\nPAK7\tA0A4U6X857_9PEZI\nPAK7\tA0A4Z2I0L3_9TELE\nPAK7\tB0JYZ1_XENTR\nPAK7\tB5DE30_DANRE\nPAK7\tF7DBS7_CALJA\nPAK7\tF7FKL1_MACMU\nPAK7\tI3ITF8_DANRE\nPAK7\tPAK5_HUMAN\nPAK7\tPAK5_MOUSE\nPAK7\tPAK5_RAT\nPAK7\tQ3V2Z3_MOUSE\nPAK7\tQ3V442_MOUSE\nPAK7\tQ4KL59_MOUSE\nPAK7\tQ5PQ75_XENLA\nPAK7\tQ6NVJ4_DANRE\nPAK7\tQ8BVB0_MOUSE\nPAK7\tT2M4L1_HYDVU\nPAK7\tU6CUD9_NEOVI\nPAK7\tV8PEI1_OPHHA\nPAK7\tW5UFP7_ICTPU\n','From\tTo\nJAK3\tA0A024R7M7_HUMAN\nJAK3\tA0A0A0MVD8_PAPAN\nJAK3\tA0A0B7FY92_THACB\nJAK3\tA0A0B8RZF9_PIG\nJAK3\tA0A0D9QZJ5_CHLSB\nJAK3\tA0A0G2K636_RAT\nJAK3\tA0A0R4J0R7_MOUSE\nJAK3\tA0A0S2Z485_HUMAN\nJAK3\tA0A0S2Z4R7_HUMAN\nJAK3\tA0A0S7HFF2_9TELE\nJAK3\tA0A0S7HJ06_9TELE\nJAK3\tA0A0S7K565_9TELE\nJAK3\tA0A0U2PCH5_TACFU\nJAK3\tA0A1A7WLG4_9TELE\nJAK3\tA0A1A7Z1X5_9TELE\nJAK3\tA0A1A8AYG1_NOTFU\nJAK3\tA0A1A8BA30_9TELE\nJAK3\tA0A1A8E945_9TELE\nJAK3\tA0A1A8FYS3_9TELE\nJAK3\tA0A1A8G2G8_9TELE\nJAK3\tA0A1A8HXA6_NOTKU\nJAK3\tA0A1A8I0N0_NOTKU\nJAK3\tA0A1A8I2A7_NOTKU\nJAK3\tA0A1A8KM50_NOTKU\nJAK3\tA0A1A8MBD6_9TELE\nJAK3\tA0A1A8MBG3_9TELE\nJAK3\tA0A1A8MC69_9TELE\nJAK3\tA0A1A8NP30_9TELE\nJAK3\tA0A1A8RPM3_9TELE\nJAK3\tA0A1A8V4B1_NOTFU\nJAK3\tA0A1S3ADN5_ERIEU\nJAK3\tA0A1S3G4P2_DIPOR\nJAK3\tA0A1S3MPK9_SALSA\nJAK3\tA0A1V4KQA1_PATFA\nJAK3\tA0A1Z1B8K9_9GOBI\nJAK3\tA0A218U7X2_9PASE\nJAK3\tA0A250YM00_CASCN\nJAK3\tA0A286XLG2_CAVPO\nJAK3\tA0A2D0R0L4_ICTPU\nJAK3\tA0A2I3T122_PANTR\nJAK3\tA0A2I3TMU4_PANTR\nJAK3\tA0A2I4BES6_9TELE\nJAK3\tA0A2J8LTH4_PANTR\nJAK3\tA0A2K5BXU8_AOTNA\nJAK3\tA0A2K5BXX9_AOTNA\nJAK3\tA0A2K5H7M5_COLAP\nJAK3\tA0A2K5H7P1_COLAP\nJAK3\tA0A2K5L3H2_CERAT\nJAK3\tA0A2K5L3K1_CERAT\nJAK3\tA0A2K5RM58_CEBCA\nJAK3\tA0A2K5RM66_CEBCA\nJAK3\tA0A2K5WFY6_MACFA\nJAK3\tA0A2K5WFZ1_MACFA\nJAK3\tA0A2K5YYM6_MANLE\nJAK3\tA0A2K5YYN4_MANLE\nJAK3\tA0A2K6DGB5_MACNE\nJAK3\tA0A2K6EQQ8_PROCO\nJAK3\tA0A2K6LW12_RHIBE\nJAK3\tA0A2K6LW13_RHIBE\nJAK3\tA0A2K6NJX6_RHIRO\nJAK3\tA0A2K6NJZ6_RHIRO\nJAK3\tA0A2K6NJZ8_RHIRO\nJAK3\tA0A2K6NK05_RHIRO\nJAK3\tA0A2K6U8W2_SAIBB\nJAK3\tA0A2K6U928_SAIBB\nJAK3\tA0A2R8MSL1_CALJA\nJAK3\tA0A2R8P2H7_CALJA\nJAK3\tA0A2R8RWH2_DANRE\nJAK3\tA0A2R9C372_PANPA\nJAK3\tA0A2U3W8S8_ODORO\nJAK3\tA0A2U3Z0Y0_LEPWE\nJAK3\tA0A2U4B295_TURTR\nJAK3\tA0A2Y9F7X6_PHYMC\nJAK3\tA0A2Y9GN84_NEOSC\nJAK3\tA0A2Y9M4G7_DELLE\nJAK3\tA0A340XLT5_LIPVE\nJAK3\tA0A341D8D9_NEOAA\nJAK3\tA0A384B744_BALAS\nJAK3\tA0A384D6N1_URSMA\nJAK3\tA0A3Q0CF12_MESAU\nJAK3\tA0A3Q0FYA4_ALLSI\nJAK3\tA0A3Q2LLF4_HORSE\nJAK3\tA0A3Q7Q4A1_CALUR\nJAK3\tA0A3Q7Q4E3_CALUR\nJAK3\tA0A3Q7Q4F0_CALUR\nJAK3\tA0A3Q7QEX2_CALUR\nJAK3\tA0A3Q7RJQ4_CALUR\nJAK3\tA0A3Q7STG4_VULVU\nJAK3\tA0A3Q7V4V1_URSAR\nJAK3\tA0A452DZ12_CAPHI\nJAK3\tA0A452DZ23_CAPHI\nJAK3\tA0A452HWJ7_9SAUR\nJAK3\tA0A452HXA0_9SAUR\nJAK3\tA0A452HXQ7_9SAUR\nJAK3\tA0A452HXS2_9SAUR\nJAK3\tA0A452HXW3_9SAUR\nJAK3\tA0A452RVA1_URSAM\nJAK3\tA0A452RVG9_URSAM\nJAK3\tA0A452RVH2_URSAM\nJAK3\tA0A452U085_URSMA\nJAK3\tA0A452U1B0_URSMA\nJAK3\tA0A452U1E2_URSMA\nJAK3\tA0A452U1H6_URSMA\nJAK3\tA0A452U1I9_URSMA\nJAK3\tA0A452U1J2_URSMA\nJAK3\tA0A452U1J3_URSMA\nJAK3\tA0A452U1P9_URSMA\nJAK3\tA0A452U1T1_URSMA\nJAK3\tA0A4W2DC00_BOBOX\nJAK3\tA0A4W2EXR7_BOBOX\nJAK3\tA0A4W2HMI1_BOBOX\nJAK3\tA0A4W4GEL3_ELEEL\nJAK3\tA0A4W4GEM0_ELEEL\nJAK3\tA0A4W4GEN8_ELEEL\nJAK3\tA0A4W4GEQ0_ELEEL\nJAK3\tA0A4W4GFV2_ELEEL\nJAK3\tA0A4W4GFW2_ELEEL\nJAK3\tA0A4W4GFZ3_ELEEL\nJAK3\tA0A4W4GJ36_ELEEL\nJAK3\tA0A4W4GJ75_ELEEL\nJAK3\tA0A4W4GJH3_ELEEL\nJAK3\tA0A4X1UAQ3_PIG\nJAK3\tA0A4X2L5P7_VOMUR\nJAK3\tA0A5F8AN52_MACMU\nJAK3\tA0A5F8G8A0_MONDO\nJAK3\tA0A5F8GX05_MONDO\nJAK3\tA0A5F8HLF7_MONDO\nJAK3\tA0A5G3K9J1_XENTR\nJAK3\tA0A5G3KAL4_XENTR\nJAK3\tA0A5G3KB09_XENTR\nJAK3\tA9UMJ6_XENTR\nJAK3\tB0V237_DANRE\nJAK3\tB2GUC9_XENTR\nJAK3\tB7ZU18_XENTR\nJAK3\tD2GV17_AILME\nJAK3\tE1BEL4_BOVIN\nJAK3\tE2RG40_CANLF\nJAK3\tF1LR79_RAT\nJAK3\tF6TDI7_XENTR\nJAK3\tF6X8E5_ORNAN\nJAK3\tF6X8L2_XENTR\nJAK3\tF6Y1F8_XENTR\nJAK3\tF6YR07_XENTR\nJAK3\tF7B2N9_HORSE\nJAK3\tF7BWK3_XENTR\nJAK3\tF7F934_MONDO\nJAK3\tF7GS58_MACMU\nJAK3\tG1P7K0_MYOLU\nJAK3\tG1R1P8_NOMLE\nJAK3\tG3TF05_LOXAF\nJAK3\tG3VR92_SARHA\nJAK3\tH0VSL1_CAVPO\nJAK3\tH0X195_OTOGA\nJAK3\tH1A3H7_TAEGU\nJAK3\tH2NY19_PONAB\nJAK3\tH2QFR9_PANTR\nJAK3\tH8YSG0_DICLA\nJAK3\tH9H0M3_MELGA\nJAK3\tI3MAE0_ICTTR\nJAK3\tJ9P762_CANLF\nJAK3\tJAK3_HUMAN\nJAK3\tJAK3_MOUSE\nJAK3\tJAK3_RAT\nJAK3\tK7FHA5_PELSI\nJAK3\tK7GSE8_PIG\nJAK3\tK9J8B1_XENTR\nJAK3\tM3W7U9_FELCA\nJAK3\tM3YWH9_MUSPF\nJAK3\tP97423_MOUSE\nJAK3\tQ28FM9_XENTR\nJAK3\tQ6JDV4_PIG\nJAK3\tQ8HYM5_FELCA\nJAK3\tQ9PWD0_TETFL\nJAK3\tU6CX16_NEOVI\nJAK3\tW5PZ17_SHEEP\nJAK3\tX1WHN6_DANRE\n','From\tTo\nABL1\tA0A024R8E2_HUMAN\nABL1\tA0A096P0F7_PAPAN\nABL1\tA0A0A6YVY6_MOUSE\nABL1\tA0A0A6YXS9_MOUSE\nABL1\tA0A0A6YY65_MOUSE\nABL1\tA0A0D9RQJ3_CHLSB\nABL1\tA0A0K8UQS0_BACLA\nABL1\tA0A0P6JE62_HETGA\nABL1\tA0A0S7F9E6_9TELE\nABL1\tA0A0S7F9I1_9TELE\nABL1\tA0A0S7GD24_9TELE\nABL1\tA0A0S7GD26_9TELE\nABL1\tA0A0S7GD53_9TELE\nABL1\tA0A0S7GGU8_9TELE\nABL1\tA0A0S7GIY7_9TELE\nABL1\tA0A0U4EGU4_HUMAN\nABL1\tA0A151MIR6_ALLMI\nABL1\tA0A1A7XE44_9TELE\nABL1\tA0A1A7ZIX5_NOTFU\nABL1\tA0A1A8D084_9TELE\nABL1\tA0A1A8D9A0_9TELE\nABL1\tA0A1A8ES32_9TELE\nABL1\tA0A1A8J985_NOTKU\nABL1\tA0A1A8JTQ9_NOTKU\nABL1\tA0A1A8JVT0_NOTKU\nABL1\tA0A1A8MRZ4_9TELE\nABL1\tA0A1A8NQF6_9TELE\nABL1\tA0A1A8NUQ8_9TELE\nABL1\tA0A1A8SN95_9TELE\nABL1\tA0A1A8UC72_NOTFU\nABL1\tA0A1D1YX71_9ARAE\nABL1\tA0A1D5R032_MACMU\nABL1\tA0A1L8F1E7_XENLA\nABL1\tA0A1S3A6Y3_ERIEU\nABL1\tA0A1S3A6Z6_ERIEU\nABL1\tA0A1S3FJV8_DIPOR\nABL1\tA0A1U7R1K5_MESAU\nABL1\tA0A1U7RC08_MESAU\nABL1\tA0A1U7RV03_ALLSI\nABL1\tA0A1U7S4W8_ALLSI\nABL1\tA0A1U7TEK2_TARSY\nABL1\tA0A1V4J6A0_PATFA\nABL1\tA0A1V4J6M5_PATFA\nABL1\tA0A218V5K2_9PASE\nABL1\tA0A250Y715_CASCN\nABL1\tA0A286XHL0_CAVPO\nABL1\tA0A286XXH5_CAVPO\nABL1\tA0A287BB27_PIG\nABL1\tA0A287D3G7_ICTTR\nABL1\tA0A287DBB5_ICTTR\nABL1\tA0A2B4SKV9_STYPI\nABL1\tA0A2D0Q9I1_ICTPU\nABL1\tA0A2D0QAM3_ICTPU\nABL1\tA0A2D0QAN0_ICTPU\nABL1\tA0A2I0M413_COLLI\nABL1\tA0A2I2UT66_FELCA\nABL1\tA0A2I3MEN2_PAPAN\nABL1\tA0A2I3S7G9_PANTR\nABL1\tA0A2I3SL43_PANTR\nABL1\tA0A2I3TNG5_PANTR\nABL1\tA0A2I4CVV6_9TELE\nABL1\tA0A2I7M856_9CHAR\nABL1\tA0A2K5DU74_AOTNA\nABL1\tA0A2K5DU86_AOTNA\nABL1\tA0A2K5DU95_AOTNA\nABL1\tA0A2K5DUB2_AOTNA\nABL1\tA0A2K5JWZ8_COLAP\nABL1\tA0A2K5JX04_COLAP\nABL1\tA0A2K5JX05_COLAP\nABL1\tA0A2K5JX56_COLAP\nABL1\tA0A2K5LX66_CERAT\nABL1\tA0A2K5LX70_CERAT\nABL1\tA0A2K5LX78_CERAT\nABL1\tA0A2K5LX89_CERAT\nABL1\tA0A2K5QAT2_CEBCA\nABL1\tA0A2K5QAW2_CEBCA\nABL1\tA0A2K5QAX0_CEBCA\nABL1\tA0A2K5QB42_CEBCA\nABL1\tA0A2K5X6F2_MACFA\nABL1\tA0A2K5X6I8_MACFA\nABL1\tA0A2K5XNS9_MANLE\nABL1\tA0A2K5XNT4_MANLE\nABL1\tA0A2K5XNU8_MANLE\nABL1\tA0A2K5XNW4_MANLE\nABL1\tA0A2K6B7I9_MACNE\nABL1\tA0A2K6B7J3_MACNE\nABL1\tA0A2K6B7M5_MACNE\nABL1\tA0A2K6H0C8_PROCO\nABL1\tA0A2K6H0H0_PROCO\nABL1\tA0A2K6K9J3_RHIBE\nABL1\tA0A2K6K9L1_RHIBE\nABL1\tA0A2K6K9Q2_RHIBE\nABL1\tA0A2K6PIM2_RHIRO\nABL1\tA0A2K6PIN7_RHIRO\nABL1\tA0A2K6PIP4_RHIRO\nABL1\tA0A2K6PIW5_RHIRO\nABL1\tA0A2K6U2T1_SAIBB\nABL1\tA0A2K6U2U1_SAIBB\nABL1\tA0A2K6U2V9_SAIBB\nABL1\tA0A2R8P5P1_CALJA\nABL1\tA0A2R9BA11_PANPA\nABL1\tA0A2R9BA34_PANPA\nABL1\tA0A2R9BBG8_PANPA\nABL1\tA0A2U3VD43_ODORO\nABL1\tA0A2U3VDJ9_ODORO\nABL1\tA0A2U3YZ15_LEPWE\nABL1\tA0A2U4A456_TURTR\nABL1\tA0A2U4A468_TURTR\nABL1\tA0A2U4A4Z4_TURTR\nABL1\tA0A2Y9F5M5_PHYMC\nABL1\tA0A2Y9F5R7_PHYMC\nABL1\tA0A2Y9F6C6_PHYMC\nABL1\tA0A2Y9F7C4_PHYMC\nABL1\tA0A2Y9H5N5_NEOSC\nABL1\tA0A2Y9H5R0_NEOSC\nABL1\tA0A2Y9NGU8_DELLE\nABL1\tA0A2Y9NX24_DELLE\nABL1\tA0A2Y9SXG4_PHYMC\nABL1\tA0A340WVZ5_LIPVE\nABL1\tA0A341CTZ6_NEOAA\nABL1\tA0A341CWA7_NEOAA\nABL1\tA0A384B4B0_BALAS\nABL1\tA0A384B4J2_BALAS\nABL1\tA0A384B4S0_BALAS\nABL1\tA0A384B5J5_BALAS\nABL1\tA0A384CRH9_URSMA\nABL1\tA0A3B3H733_ORYLA\nABL1\tA0A3B3IBN8_ORYLA\nABL1\tA0A3Q1LI98_BOVIN\nABL1\tA0A3Q1MN77_BOVIN\nABL1\tA0A3Q2HBN5_HORSE\nABL1\tA0A3Q2HQ21_HORSE\nABL1\tA0A3Q2I376_HORSE\nABL1\tA0A3Q7QP71_CALUR\nABL1\tA0A3Q7RTP4_CALUR\nABL1\tA0A3Q7SR65_VULVU\nABL1\tA0A3Q7UCY2_VULVU\nABL1\tA0A3Q7WAB6_URSAR\nABL1\tA0A3Q7Y170_URSAR\nABL1\tA0A452FJM7_CAPHI\nABL1\tA0A452FJX1_CAPHI\nABL1\tA0A452FKI1_CAPHI\nABL1\tA0A452IMB5_9SAUR\nABL1\tA0A452IMF7_9SAUR\nABL1\tA0A452QVJ4_URSAM\nABL1\tA0A452QVJ5_URSAM\nABL1\tA0A452QVJ6_URSAM\nABL1\tA0A452UXJ0_URSMA\nABL1\tA0A455BW56_PHYMC\nABL1\tA0A493TAY4_ANAPP\nABL1\tA0A4W2CTP7_BOBOX\nABL1\tA0A4W2EA68_BOBOX\nABL1\tA0A4W2FNN3_BOBOX\nABL1\tA0A4W2HQG7_BOBOX\nABL1\tA0A4W3J683_CALMI\nABL1\tA0A4W3J8Z9_CALMI\nABL1\tA0A4W3J924_CALMI\nABL1\tA0A4W3J933_CALMI\nABL1\tA0A4W3J940_CALMI\nABL1\tA0A4W3JGY6_CALMI\nABL1\tA0A4W3JKT6_CALMI\nABL1\tA0A4W6CFJ9_LATCA\nABL1\tA0A4W6CFL6_LATCA\nABL1\tA0A4W6CJ98_LATCA\nABL1\tA0A4W6CJD3_LATCA\nABL1\tA0A4W6CJD7_LATCA\nABL1\tA0A4W6CJE3_LATCA\nABL1\tA0A4W6CJF8_LATCA\nABL1\tA0A4X1TCE2_PIG\nABL1\tA0A4X1TD72_PIG\nABL1\tA0A4X2KWI6_VOMUR\nABL1\tA0A4X2KY57_VOMUR\nABL1\tA0A4Z2ITP1_9TELE\nABL1\tA0A5F4C2P3_CANLF\nABL1\tA0A5F7ZUS6_MACMU\nABL1\tA0A5F8GAW2_MONDO\nABL1\tA0A5F8GBZ2_MONDO\nABL1\tA0A5F9CZ86_RABIT\nABL1\tA0A5G3HZG0_XENTR\nABL1\tA0A5G3HZQ9_XENTR\nABL1\tA0A5G3HZW8_XENTR\nABL1\tA0A5G3ILK7_XENTR\nABL1\tA0A5G3JQ89_XENTR\nABL1\tA0A5G3JS72_XENTR\nABL1\tA0A5G3JS88_XENTR\nABL1\tA2AV23_MOUSE\nABL1\tA8JNU2_DROME\nABL1\tABL1_HUMAN\nABL1\tABL1_LEPMJ\nABL1\tABL1_MOUSE\nABL1\tBZP46_ORYSJ\nABL1\tD1H0Y7_SCHMA\nABL1\tD2H177_AILME\nABL1\tD3ZGM3_RAT\nABL1\tE9PT20_RAT\nABL1\tF1MXG8_BOVIN\nABL1\tF1NCD9_CHICK\nABL1\tF1PEX6_CANLF\nABL1\tF1PEX7_CANLF\nABL1\tF1S0X4_PIG\nABL1\tF6QCG1_ORNAN\nABL1\tF6QCG8_ORNAN\nABL1\tF7BN18_XENTR\nABL1\tF7DR40_HORSE\nABL1\tF7EH78_MACMU\nABL1\tF7FU48_MONDO\nABL1\tF7I4W0_CALJA\nABL1\tF8W2F9_DANRE\nABL1\tF8W5Q8_DANRE\nABL1\tG1N1M4_MELGA\nABL1\tG1PC86_MYOLU\nABL1\tG1RPU2_NOMLE\nABL1\tG1SSF7_RABIT\nABL1\tG3SZ92_LOXAF\nABL1\tG3UM32_LOXAF\nABL1\tG3WV25_SARHA\nABL1\tH0V6K4_CAVPO\nABL1\tH0WWK4_OTOGA\nABL1\tH0Z2V6_TAEGU\nABL1\tH2LT87_ORYLA\nABL1\tH2PU80_PONAB\nABL1\tH2QY21_PANTR\nABL1\tH2ULL5_TAKRU\nABL1\tH3B794_LATCH\nABL1\tH3B795_LATCH\nABL1\tH9ES70_MACMU\nABL1\tH9F7I9_MACMU\nABL1\tH9GNS0_ANOCA\nABL1\tH9YUX8_MACMU\nABL1\tI0FQI2_MACMU\nABL1\tI3K9B9_ORENI\nABL1\tI3K9C0_ORENI\nABL1\tI3NH11_ICTTR\nABL1\tI4ENX0_ECHMU\nABL1\tK7B7C5_PANTR\nABL1\tK7CD11_PANTR\nABL1\tK7CNV9_PANTR\nABL1\tK7DKC3_PANTR\nABL1\tK7FHZ2_PELSI\nABL1\tM3W3H7_FELCA\nABL1\tM3XH30_LATCH\nABL1\tM3YMM7_MUSPF\nABL1\tM9PCT8_DROME\nABL1\tM9PCU1_DROME\nABL1\tM9PFI5_DROME\nABL1\tM9PFI8_DROME\nABL1\tM9PFS1_DROME\nABL1\tM9PFX0_DROME\nABL1\tM9PI83_DROME\nABL1\tQ13688_HUMAN\nABL1\tQ13689_HUMAN\nABL1\tQ13691_HUMAN\nABL1\tQ13914_HUMAN\nABL1\tQ13915_HUMAN\nABL1\tQ3B836_HUMAN\nABL1\tQ3B837_HUMAN\nABL1\tQ3SYK5_MOUSE\nABL1\tQ3TM13_MOUSE\nABL1\tQ3V2W1_MOUSE\nABL1\tQ61262_MOUSE\nABL1\tQ86Y36_HUMAN\nABL1\tQ8CAT4_MOUSE\nABL1\tR4GRW0_HUMAN\nABL1\tU3BH45_CALJA\nABL1\tU3CVZ7_CALJA\nABL1\tU3CWS4_CALJA\nABL1\tU3JFF5_FICAL\nABL1\tU6D2E7_NEOVI\nABL1\tV5NT95_XENLA\nABL1\tW5PBZ8_SHEEP\nABL1\tW5UM98_ICTPU\nABL1\tX2JGR4_DROME\n','From\tTo\nSTK10\tA0A096N5S6_PAPAN\nSTK10\tA0A0A9WF54_LYGHE\nSTK10\tA0A0B2UQ63_TOXCA\nSTK10\tA0A0B8RVG3_PIG\nSTK10\tA0A0D9RB93_CHLSB\nSTK10\tA0A0G2K7Z9_RAT\nSTK10\tA0A0H2UKI6_DANRE\nSTK10\tA0A0P6K5K3_HETGA\nSTK10\tA0A0S7FW62_9TELE\nSTK10\tA0A0S7FW70_9TELE\nSTK10\tA0A0S7FW80_9TELE\nSTK10\tA0A0S7FW91_9TELE\nSTK10\tA0A0S7FW96_9TELE\nSTK10\tA0A0S7FWA4_9TELE\nSTK10\tA0A0S7FX35_9TELE\nSTK10\tA0A0S7FXY9_9TELE\nSTK10\tA0A0S7FZB2_9TELE\nSTK10\tA0A0S7G0X8_9TELE\nSTK10\tA0A0S7G1C0_9TELE\nSTK10\tA0A0S7G2H5_9TELE\nSTK10\tA0A0S7G398_9TELE\nSTK10\tA0A0S7G3S0_9TELE\nSTK10\tA0A0S7G535_9TELE\nSTK10\tA0A0V0SGY9_9BILA\nSTK10\tA0A0V0UGT0_9BILA\nSTK10\tA0A0V0XBY5_9BILA\nSTK10\tA0A0V0XC19_9BILA\nSTK10\tA0A0V0XC38_9BILA\nSTK10\tA0A0V0YMC2_TRIPS\nSTK10\tA0A0V1AEJ5_9BILA\nSTK10\tA0A0V1AFD8_9BILA\nSTK10\tA0A0V1AFN4_9BILA\nSTK10\tA0A0V1AFV2_9BILA\nSTK10\tA0A0V1BKA3_TRISP\nSTK10\tA0A0V1D7M8_TRIBR\nSTK10\tA0A0V1FY12_TRIPS\nSTK10\tA0A0V1HGW2_9BILA\nSTK10\tA0A0V1HHP9_9BILA\nSTK10\tA0A0V1HHU2_9BILA\nSTK10\tA0A0V1HJS4_9BILA\nSTK10\tA0A0V1LPA9_9BILA\nSTK10\tA0A0V1LQI0_9BILA\nSTK10\tA0A0V1PMW1_9BILA\nSTK10\tA0A151M6P7_ALLMI\nSTK10\tA0A1A7X5U6_9TELE\nSTK10\tA0A1A7YA82_9TELE\nSTK10\tA0A1A8BQ00_9TELE\nSTK10\tA0A1A8E9M2_9TELE\nSTK10\tA0A1A8F6G9_9TELE\nSTK10\tA0A1A8HB16_9TELE\nSTK10\tA0A1A8HDB8_9TELE\nSTK10\tA0A1A8IKX0_NOTKU\nSTK10\tA0A1A8IKZ0_NOTKU\nSTK10\tA0A1A8LIN4_9TELE\nSTK10\tA0A1A8LLE1_9TELE\nSTK10\tA0A1A8P352_9TELE\nSTK10\tA0A1A8P3C8_9TELE\nSTK10\tA0A1A8Q6Y5_9TELE\nSTK10\tA0A1A8VEI0_NOTFU\nSTK10\tA0A1D5QDI4_MACMU\nSTK10\tA0A1L8GXM9_XENLA\nSTK10\tA0A1S3G5W1_DIPOR\nSTK10\tA0A1S3W8G0_ERIEU\nSTK10\tA0A1U7RN56_ALLSI\nSTK10\tA0A1V4KU24_PATFA\nSTK10\tA0A218UN17_9PASE\nSTK10\tA0A250YLT1_CASCN\nSTK10\tA0A287BDH4_PIG\nSTK10\tA0A2I0LST1_COLLI\nSTK10\tA0A2I4C5I1_9TELE\nSTK10\tA0A2K5DFN6_AOTNA\nSTK10\tA0A2K5J697_COLAP\nSTK10\tA0A2K5MLR2_CERAT\nSTK10\tA0A2K5MM21_CERAT\nSTK10\tA0A2K5PVC2_CEBCA\nSTK10\tA0A2K5PVC8_CEBCA\nSTK10\tA0A2K5U587_MACFA\nSTK10\tA0A2K6A1V8_MANLE\nSTK10\tA0A2K6BDL5_MACNE\nSTK10\tA0A2K6BDN7_MACNE\nSTK10\tA0A2K6FLF8_PROCO\nSTK10\tA0A2K6LWI8_RHIBE\nSTK10\tA0A2K6RJL4_RHIRO\nSTK10\tA0A2K6RJM0_RHIRO\nSTK10\tA0A2K6URC5_SAIBB\nSTK10\tA0A2R8ZGP9_PANPA\nSTK10\tA0A2U3VCF1_ODORO\nSTK10\tA0A2U3Y0G5_LEPWE\nSTK10\tA0A2U4C7J7_TURTR\nSTK10\tA0A2U4C7S6_TURTR\nSTK10\tA0A2Y9HQI0_NEOSC\nSTK10\tA0A2Y9PVC3_DELLE\nSTK10\tA0A2Y9S0C6_PHYMC\nSTK10\tA0A2Y9S7D0_PHYMC\nSTK10\tA0A340Y2F4_LIPVE\nSTK10\tA0A341C5D0_NEOAA\nSTK10\tA0A367IS05_RHIAZ\nSTK10\tA0A383Z8Q4_BALAS\nSTK10\tA0A384C7W2_URSMA\nSTK10\tA0A3Q0CUT3_MESAU\nSTK10\tA0A3Q1M334_BOVIN\nSTK10\tA0A3Q7NME1_CALUR\nSTK10\tA0A3Q7Q3G4_CALUR\nSTK10\tA0A3Q7U6C7_VULVU\nSTK10\tA0A3Q7XJW7_URSAR\nSTK10\tA0A3R7FN86_CLOSI\nSTK10\tA0A452FMS7_CAPHI\nSTK10\tA0A452IJI3_9SAUR\nSTK10\tA0A452IJM0_9SAUR\nSTK10\tA0A452IJP8_9SAUR\nSTK10\tA0A452IJR8_9SAUR\nSTK10\tA0A452IJW8_9SAUR\nSTK10\tA0A452IK08_9SAUR\nSTK10\tA0A452IK21_9SAUR\nSTK10\tA0A452IK46_9SAUR\nSTK10\tA0A452IK55_9SAUR\nSTK10\tA0A452SES9_URSAM\nSTK10\tA0A452SF43_URSAM\nSTK10\tA0A452SF88_URSAM\nSTK10\tA0A452SF95_URSAM\nSTK10\tA0A452SFA1_URSAM\nSTK10\tA0A452SFD7_URSAM\nSTK10\tA0A452SFE9_URSAM\nSTK10\tA0A452SFH9_URSAM\nSTK10\tA0A452SFK9_URSAM\nSTK10\tA0A452VKA6_URSMA\nSTK10\tA0A452VKB7_URSMA\nSTK10\tA0A452VKF9_URSMA\nSTK10\tA0A452VKI6_URSMA\nSTK10\tA0A452VKJ7_URSMA\nSTK10\tA0A452VKK0_URSMA\nSTK10\tA0A452VKN4_URSMA\nSTK10\tA0A455BW30_PHYMC\nSTK10\tA0A455C9F0_PHYMC\nSTK10\tA0A493SUL9_ANAPP\nSTK10\tA0A493TPE5_ANAPP\nSTK10\tA0A4W2E971_BOBOX\nSTK10\tA0A4W2EIK4_BOBOX\nSTK10\tA0A4W2GAI7_BOBOX\nSTK10\tA0A4X1U792_PIG\nSTK10\tA0A4X1U798_PIG\nSTK10\tA0A4X2JWC7_VOMUR\nSTK10\tA0A4Y2BYS2_ARAVE\nSTK10\tA0A5F4C1T2_CANLF\nSTK10\tA0A5F4DFH9_CANLF\nSTK10\tA0A5F5PRD3_HORSE\nSTK10\tA0A5F5Y114_FELCA\nSTK10\tA0A5F8A052_MACMU\nSTK10\tA0A5G2Q7M0_PIG\nSTK10\tA0A5G3K9J6_XENTR\nSTK10\tA0A5G3K9K0_XENTR\nSTK10\tA0A5G3KA96_XENTR\nSTK10\tA0A5G3KAD4_XENTR\nSTK10\tA0A5G3KBD4_XENTR\nSTK10\tA0A5G3KBJ4_XENTR\nSTK10\tA0A5G3KBJ6_XENTR\nSTK10\tA1A553_MOUSE\nSTK10\tF1PBJ2_CANLF\nSTK10\tF1RRZ3_PIG\nSTK10\tF6R5W7_XENTR\nSTK10\tF6SJH0_MONDO\nSTK10\tF6UUL1_XENTR\nSTK10\tF6XZF0_XENTR\nSTK10\tF6YHJ7_HORSE\nSTK10\tF7D137_HORSE\nSTK10\tF7G2G0_MONDO\nSTK10\tF7GZF5_MACMU\nSTK10\tF7HI69_CALJA\nSTK10\tG1KR20_ANOCA\nSTK10\tG1LXH0_AILME\nSTK10\tG1MWP9_MELGA\nSTK10\tG1P6R8_MYOLU\nSTK10\tG1Q746_MYOLU\nSTK10\tG1QHR0_NOMLE\nSTK10\tG1SN76_RABIT\nSTK10\tG3WHY4_SARHA\nSTK10\tH0V7X6_CAVPO\nSTK10\tH0X5X9_OTOGA\nSTK10\tH0YB71_HUMAN\nSTK10\tH0YPU8_TAEGU\nSTK10\tH2PHD0_PONAB\nSTK10\tH2RSN7_TAKRU\nSTK10\tH2XJF0_BOVIN\nSTK10\tH3APZ9_LATCH\nSTK10\tH9F7S3_MACMU\nSTK10\tH9FH91_MACMU\nSTK10\tH9KVF6_RAT\nSTK10\tI0FI40_MACMU\nSTK10\tI3LBD5_PIG\nSTK10\tI3N5I4_ICTTR\nSTK10\tK7B1G0_PANTR\nSTK10\tK7C2M9_PANTR\nSTK10\tK7FQ69_PELSI\nSTK10\tM3VUR5_FELCA\nSTK10\tM3YMV9_MUSPF\nSTK10\tQ3UG53_MOUSE\nSTK10\tQ5DTS2_MOUSE\nSTK10\tSTK10_BOVIN\nSTK10\tSTK10_CHICK\nSTK10\tSTK10_DANRE\nSTK10\tSTK10_HUMAN\nSTK10\tSTK10_MOUSE\nSTK10\tSTK10_RAT\nSTK10\tSTK10_XENTR\nSTK10\tT2MIH2_HYDVU\nSTK10\tU3BJV7_CALJA\nSTK10\tU3FME7_CALJA\nSTK10\tU3I695_ANAPP\nSTK10\tU3K2G0_FICAL\nSTK10\tU6DHP6_NEOVI\nSTK10\tV8PIQ8_OPHHA\nSTK10\tW5P0P0_SHEEP\nSTK10\tW5P0P3_SHEEP\nSTK10\tW5U7Q0_ICTPU\n','From\tTo\nPAK4\tA0A024R0J1_HUMAN\nPAK4\tA0A024R0L8_HUMAN\nPAK4\tA0A0A0MVY2_PAPAN\nPAK4\tA0A0A0MW16_PAPAN\nPAK4\tA0A0A9X9U8_LYGHE\nPAK4\tA0A0D9QV87_CHLSB\nPAK4\tA0A0P6JDX6_HETGA\nPAK4\tA0A0S7F490_9TELE\nPAK4\tA0A0S7F873_9TELE\nPAK4\tA0A1A7Y5C9_9TELE\nPAK4\tA0A1A7Z7E0_9TELE\nPAK4\tA0A1A7ZQ42_NOTFU\nPAK4\tA0A1A8AC76_NOTFU\nPAK4\tA0A1A8B5W5_NOTFU\nPAK4\tA0A1A8BIH9_9TELE\nPAK4\tA0A1A8CXR2_9TELE\nPAK4\tA0A1A8DGN8_9TELE\nPAK4\tA0A1A8FDT9_9TELE\nPAK4\tA0A1A8GC23_9TELE\nPAK4\tA0A1A8HBR1_9TELE\nPAK4\tA0A1A8IN09_NOTKU\nPAK4\tA0A1A8JTG7_NOTKU\nPAK4\tA0A1A8N7I1_9TELE\nPAK4\tA0A1A8P2K7_9TELE\nPAK4\tA0A1A8PGB8_9TELE\nPAK4\tA0A1A8SKR0_9TELE\nPAK4\tA0A1A8U4U8_NOTFU\nPAK4\tA0A1D5P8Y3_CHICK\nPAK4\tA0A1D5QPF3_MACMU\nPAK4\tA0A1S3FZ19_DIPOR\nPAK4\tA0A1S3FZJ1_DIPOR\nPAK4\tA0A1S3WC89_ERIEU\nPAK4\tA0A1U7UK21_TARSY\nPAK4\tA0A1U8CN30_MESAU\nPAK4\tA0A218UBR7_9PASE\nPAK4\tA0A250Y3B7_CASCN\nPAK4\tA0A2D0SWF0_ICTPU\nPAK4\tA0A2I3HHJ7_NOMLE\nPAK4\tA0A2J8QFD4_PANTR\nPAK4\tA0A2K5EGD3_AOTNA\nPAK4\tA0A2K5EGD7_AOTNA\nPAK4\tA0A2K5HVI7_COLAP\nPAK4\tA0A2K5HVL3_COLAP\nPAK4\tA0A2K5NMP4_CERAT\nPAK4\tA0A2K5NMW1_CERAT\nPAK4\tA0A2K5RZ07_CEBCA\nPAK4\tA0A2K5RZ23_CEBCA\nPAK4\tA0A2K5UKI9_MACFA\nPAK4\tA0A2K5UKJ7_MACFA\nPAK4\tA0A2K6A5V8_MANLE\nPAK4\tA0A2K6A5W0_MANLE\nPAK4\tA0A2K6C0T3_MACNE\nPAK4\tA0A2K6C0V3_MACNE\nPAK4\tA0A2K6C0V4_MACNE\nPAK4\tA0A2K6EWA9_PROCO\nPAK4\tA0A2K6EWB0_PROCO\nPAK4\tA0A2K6JZG6_RHIBE\nPAK4\tA0A2K6JZJ3_RHIBE\nPAK4\tA0A2K6JZJ9_RHIBE\nPAK4\tA0A2K6QYV0_RHIRO\nPAK4\tA0A2K6QYW0_RHIRO\nPAK4\tA0A2K6U8Q4_SAIBB\nPAK4\tA0A2K6U8R0_SAIBB\nPAK4\tA0A2R9BEW7_PANPA\nPAK4\tA0A2R9BNH7_PANPA\nPAK4\tA0A2R9BNI4_PANPA\nPAK4\tA0A2U3VN23_ODORO\nPAK4\tA0A2U3VNB6_ODORO\nPAK4\tA0A2U3Y6X0_LEPWE\nPAK4\tA0A2Y9HY20_NEOSC\nPAK4\tA0A2Y9I4H0_NEOSC\nPAK4\tA0A2Y9PFK0_DELLE\nPAK4\tA0A337SMX5_FELCA\nPAK4\tA0A340XU08_LIPVE\nPAK4\tA0A340XZM7_LIPVE\nPAK4\tA0A341DBS8_NEOAA\nPAK4\tA0A383ZYB3_BALAS\nPAK4\tA0A384BXU1_URSMA\nPAK4\tA0A3Q0FPH8_ALLSI\nPAK4\tA0A3Q1MSM5_BOVIN\nPAK4\tA0A3Q2HY69_HORSE\nPAK4\tA0A3Q2KQ25_HORSE\nPAK4\tA0A3Q7MRZ2_CALUR\nPAK4\tA0A3Q7MTH1_CALUR\nPAK4\tA0A3Q7T8N2_VULVU\nPAK4\tA0A3Q7TPV9_URSAR\nPAK4\tA0A3Q7UEL5_VULVU\nPAK4\tA0A452EPN4_CAPHI\nPAK4\tA0A452HCZ6_9SAUR\nPAK4\tA0A452HD04_9SAUR\nPAK4\tA0A452HD06_9SAUR\nPAK4\tA0A452QDL2_URSAM\nPAK4\tA0A455AVB1_PHYMC\nPAK4\tA0A455AXH2_PHYMC\nPAK4\tA0A480YBA9_PIG\nPAK4\tA0A493SU83_ANAPP\nPAK4\tA0A4W2CXX6_BOBOX\nPAK4\tA0A4W2E386_BOBOX\nPAK4\tA0A4W2FTQ9_BOBOX\nPAK4\tA0A4W2FWX3_BOBOX\nPAK4\tA0A4W2HLB8_BOBOX\nPAK4\tA0A4W6FSG5_LATCA\nPAK4\tA0A4W6FTA1_LATCA\nPAK4\tA0A4X1SXD8_PIG\nPAK4\tA0A4X1SZ68_PIG\nPAK4\tA0A4X2LY56_VOMUR\nPAK4\tA0A4X2MA01_VOMUR\nPAK4\tA0A5F4CZ89_CANLF\nPAK4\tA0A5F8G216_MONDO\nPAK4\tA0A5F8HC93_MONDO\nPAK4\tA0A5F9C5H3_RABIT\nPAK4\tA0A5F9CDS1_RABIT\nPAK4\tA0A5F9DDJ6_RABIT\nPAK4\tA0A5F9DDL2_RABIT\nPAK4\tA0A5G3HB79_XENTR\nPAK4\tA0JP99_XENTR\nPAK4\tB4DUG0_HUMAN\nPAK4\tB5DF62_RAT\nPAK4\tE2RE16_CANLF\nPAK4\tF1QZR2_DANRE\nPAK4\tF6PS80_XENTR\nPAK4\tF6Z9H7_MACMU\nPAK4\tF6Z9I4_MACMU\nPAK4\tF6ZWD5_HORSE\nPAK4\tF7AWJ1_CALJA\nPAK4\tF7BEF8_CALJA\nPAK4\tF7G9B2_MONDO\nPAK4\tG1LQ05_AILME\nPAK4\tG1NX77_MYOLU\nPAK4\tG1RWR4_NOMLE\nPAK4\tG1U468_RABIT\nPAK4\tG3R3A8_GORGO\nPAK4\tG3SAG8_GORGO\nPAK4\tG3SVV4_LOXAF\nPAK4\tG9NDI9_HYPVG\nPAK4\tG9P8P5_HYPAI\nPAK4\tH0W783_CAVPO\nPAK4\tH0X6C6_OTOGA\nPAK4\tH2L7P5_ORYLA\nPAK4\tH2L7P8_ORYLA\nPAK4\tH2NYR3_PONAB\nPAK4\tH2T824_TAKRU\nPAK4\tH3BGE6_LATCH\nPAK4\tH9EZH4_MACMU\nPAK4\tH9GI94_ANOCA\nPAK4\tI2CWB1_MACMU\nPAK4\tI3LSG7_PIG\nPAK4\tK7C9R2_PANTR\nPAK4\tK7GBN4_PELSI\nPAK4\tM0R0L9_HUMAN\nPAK4\tM0R1R1_HUMAN\nPAK4\tM0R2X4_HUMAN\nPAK4\tM0R3G6_HUMAN\nPAK4\tM3WSV7_FELCA\nPAK4\tM3XIR3_LATCH\nPAK4\tM3Z221_MUSPF\nPAK4\tPAK4_HUMAN\nPAK4\tPAK4_MOUSE\nPAK4\tQ17QZ0_BOVIN\nPAK4\tQ6GMI1_DANRE\nPAK4\tQ8K0U2_MOUSE\nPAK4\tU6D2D7_NEOVI\nPAK4\tV8NII4_OPHHA\nPAK4\tW5P7K5_SHEEP\n','From\tTo\nMST4\tA0A034V499_BACDO\nMST4\tA0A0C9PHI3_9HYME\nMST4\tA0A0P6JCW5_HETGA\nMST4\tA0A0S7KTC2_9TELE\nMST4\tA0A0S7KTC4_9TELE\nMST4\tA0A0S7ME95_9TELE\nMST4\tA0A1A7WU69_9TELE\nMST4\tA0A1A7YX41_9TELE\nMST4\tA0A1A8AXG8_NOTFU\nMST4\tA0A1A8E8N1_9TELE\nMST4\tA0A1A8FX90_9TELE\nMST4\tA0A1A8GKI2_9TELE\nMST4\tA0A1A8HYW9_NOTKU\nMST4\tA0A1A8KRM2_NOTKU\nMST4\tA0A1A8Q190_9TELE\nMST4\tA0A1A8RQA7_9TELE\nMST4\tA0A343CWX4_PENVA\nMST4\tA0A484FEB8_COLOR\nMST4\tB1WB98_DANRE\nMST4\tB4F6N4_XENTR\nMST4\tB5X318_SALSA\nMST4\tB5X4M0_SALSA\nMST4\tF1LXV3_RAT\nMST4\tG3EHN3_RHIIR\nMST4\tH9Z0G0_MACMU\nMST4\tK7CAE8_PANTR\nMST4\tMST4_ORYSJ\nMST4\tQ08DU5_BOVIN\nMST4\tQ499L9_HUMAN\nMST4\tQ6B4G6_MAIZE\nMST4\tQ6GM92_XENLA\nMST4\tQ8GU08_VISAL\nMST4\tSTK26_HUMAN\nMST4\tSTK26_MOUSE\nMST4\tU3E4W1_CALJA\nMST4\tV8PC58_OPHHA\n','From\tTo\nMAP2K4\tA0A096P2Y5_PAPAN\nMAP2K4\tA0A0A1WJ64_ZEUCU\nMAP2K4\tA0A0B2UIC1_TOXCA\nMAP2K4\tA0A0B8RVQ6_PIG\nMAP2K4\tA0A0D9R9T4_CHLSB\nMAP2K4\tA0A0V0SD27_9BILA\nMAP2K4\tA0A0V0UIK9_9BILA\nMAP2K4\tA0A0V0W8C5_9BILA\nMAP2K4\tA0A0V0YN59_TRIPS\nMAP2K4\tA0A0V0YNR9_TRIPS\nMAP2K4\tA0A0V0YNX8_TRIPS\nMAP2K4\tA0A0V1A7Z6_9BILA\nMAP2K4\tA0A0V1DC79_TRIBR\nMAP2K4\tA0A0V1ETI0_TRIPS\nMAP2K4\tA0A0V1G0N1_TRIPS\nMAP2K4\tA0A0V1J311_TRIPS\nMAP2K4\tA0A0V1K591_TRIPS\nMAP2K4\tA0A0V1K5B3_TRIPS\nMAP2K4\tA0A0V1LJL5_9BILA\nMAP2K4\tA0A0V1PF13_9BILA\nMAP2K4\tA0A151P7E5_ALLMI\nMAP2K4\tA0A151P7J4_ALLMI\nMAP2K4\tA0A1D5NVP0_CHICK\nMAP2K4\tA0A1D5P2I9_CHICK\nMAP2K4\tA0A1D5QAU9_MACMU\nMAP2K4\tA0A1S2ZQ60_ERIEU\nMAP2K4\tA0A1S3EXC0_DIPOR\nMAP2K4\tA0A1S3S3C8_SALSA\nMAP2K4\tA0A1S3S3D1_SALSA\nMAP2K4\tA0A1S3S3D2_SALSA\nMAP2K4\tA0A1S3S3D6_SALSA\nMAP2K4\tA0A1S3S3F0_SALSA\nMAP2K4\tA0A1S3S3T2_SALSA\nMAP2K4\tA0A1S3S3T5_SALSA\nMAP2K4\tA0A1U7UD68_TARSY\nMAP2K4\tA0A1V4J1W0_PATFA\nMAP2K4\tA0A250YEC9_CASCN\nMAP2K4\tA0A286X7C1_CAVPO\nMAP2K4\tA0A287B4Y9_PIG\nMAP2K4\tA0A287CXW2_ICTTR\nMAP2K4\tA0A2B4SS28_STYPI\nMAP2K4\tA0A2D0S3T6_ICTPU\nMAP2K4\tA0A2D0S3T9_ICTPU\nMAP2K4\tA0A2D0S4Q1_ICTPU\nMAP2K4\tA0A2D0S5B4_ICTPU\nMAP2K4\tA0A2D0S5B7_ICTPU\nMAP2K4\tA0A2I0LP75_COLLI\nMAP2K4\tA0A2I2UHW9_FELCA\nMAP2K4\tA0A2I2UJA2_FELCA\nMAP2K4\tA0A2I3BQK8_MOUSE\nMAP2K4\tA0A2I3G100_NOMLE\nMAP2K4\tA0A2I3G2G1_NOMLE\nMAP2K4\tA0A2I3GYS3_NOMLE\nMAP2K4\tA0A2I3HXB2_NOMLE\nMAP2K4\tA0A2I3LK59_PAPAN\nMAP2K4\tA0A2I3LPH7_PAPAN\nMAP2K4\tA0A2I3MQM3_PAPAN\nMAP2K4\tA0A2I3N3I0_PAPAN\nMAP2K4\tA0A2I3S5T8_PANTR\nMAP2K4\tA0A2I3S7V5_PANTR\nMAP2K4\tA0A2I3SCZ0_PANTR\nMAP2K4\tA0A2K5EY41_AOTNA\nMAP2K4\tA0A2K5EY51_AOTNA\nMAP2K4\tA0A2K5EY93_AOTNA\nMAP2K4\tA0A2K5IZZ4_COLAP\nMAP2K4\tA0A2K5J0A0_COLAP\nMAP2K4\tA0A2K5J0F8_COLAP\nMAP2K4\tA0A2K5NJ62_CERAT\nMAP2K4\tA0A2K5NJ66_CERAT\nMAP2K4\tA0A2K5NJ70_CERAT\nMAP2K4\tA0A2K5NJQ4_CERAT\nMAP2K4\tA0A2K5NJQ9_CERAT\nMAP2K4\tA0A2K5NJR4_CERAT\nMAP2K4\tA0A2K5WT50_MACFA\nMAP2K4\tA0A2K5WT66_MACFA\nMAP2K4\tA0A2K5WTM3_MACFA\nMAP2K4\tA0A2K5WTN3_MACFA\nMAP2K4\tA0A2K5WTN8_MACFA\nMAP2K4\tA0A2K5ZZA1_MANLE\nMAP2K4\tA0A2K5ZZB1_MANLE\nMAP2K4\tA0A2K5ZZC9_MANLE\nMAP2K4\tA0A2K5ZZE0_MANLE\nMAP2K4\tA0A2K6CK52_MACNE\nMAP2K4\tA0A2K6CK60_MACNE\nMAP2K4\tA0A2K6CK95_MACNE\nMAP2K4\tA0A2K6CKA6_MACNE\nMAP2K4\tA0A2K6M2M4_RHIBE\nMAP2K4\tA0A2K6M2N1_RHIBE\nMAP2K4\tA0A2K6M2N3_RHIBE\nMAP2K4\tA0A2K6M2N5_RHIBE\nMAP2K4\tA0A2K6M2P2_RHIBE\nMAP2K4\tA0A2K6PJ79_RHIRO\nMAP2K4\tA0A2K6PJ81_RHIRO\nMAP2K4\tA0A2K6PJ85_RHIRO\nMAP2K4\tA0A2K6PJ93_RHIRO\nMAP2K4\tA0A2K6PJ99_RHIRO\nMAP2K4\tA0A2K6PJF3_RHIRO\nMAP2K4\tA0A2K6UU95_SAIBB\nMAP2K4\tA0A2K6UUA2_SAIBB\nMAP2K4\tA0A2K6UUB2_SAIBB\nMAP2K4\tA0A2K6UUG8_SAIBB\nMAP2K4\tA0A2K6UUH8_SAIBB\nMAP2K4\tA0A2R8MD99_CALJA\nMAP2K4\tA0A2R8MIZ6_CALJA\nMAP2K4\tA0A2R8MSZ0_CALJA\nMAP2K4\tA0A2R8ZXE8_PANPA\nMAP2K4\tA0A2R9A2Y6_PANPA\nMAP2K4\tA0A2U3X381_ODORO\nMAP2K4\tA0A2U3ZX97_ODORO\nMAP2K4\tA0A2U4C2R1_TURTR\nMAP2K4\tA0A2U4C2S6_TURTR\nMAP2K4\tA0A2U4C2V0_TURTR\nMAP2K4\tA0A2Y9HL57_NEOSC\nMAP2K4\tA0A2Y9PYR4_DELLE\nMAP2K4\tA0A2Y9Q357_DELLE\nMAP2K4\tA0A2Y9Q8W6_DELLE\nMAP2K4\tA0A2Y9Q8Z8_DELLE\nMAP2K4\tA0A2Y9SXK3_PHYMC\nMAP2K4\tA0A2Y9T1J9_PHYMC\nMAP2K4\tA0A340X191_LIPVE\nMAP2K4\tA0A340X517_LIPVE\nMAP2K4\tA0A340X7M8_LIPVE\nMAP2K4\tA0A341C221_NEOAA\nMAP2K4\tA0A383ZJM4_BALAS\nMAP2K4\tA0A383ZJT0_BALAS\nMAP2K4\tA0A384BVT3_URSMA\nMAP2K4\tA0A3B3C8J9_ORYME\nMAP2K4\tA0A3B3DMV5_ORYME\nMAP2K4\tA0A3B3HCX2_ORYLA\nMAP2K4\tA0A3B3I1T7_ORYLA\nMAP2K4\tA0A3B4FXV6_9CICH\nMAP2K4\tA0A3B4FXX3_9CICH\nMAP2K4\tA0A3B4VMD8_SERDU\nMAP2K4\tA0A3B4VNQ6_SERDU\nMAP2K4\tA0A3B4VQY7_SERDU\nMAP2K4\tA0A3B4WRZ8_SERLL\nMAP2K4\tA0A3B4X2L6_SERLL\nMAP2K4\tA0A3B4ZER7_9TELE\nMAP2K4\tA0A3B5A0H0_9TELE\nMAP2K4\tA0A3P8TN73_AMPPE\nMAP2K4\tA0A3P8TWN6_AMPPE\nMAP2K4\tA0A3P9AB55_ESOLU\nMAP2K4\tA0A3P9ACT6_ESOLU\nMAP2K4\tA0A3P9J3R5_ORYLA\nMAP2K4\tA0A3Q0CL64_MESAU\nMAP2K4\tA0A3Q0CLL3_MESAU\nMAP2K4\tA0A3Q0DIK4_TARSY\nMAP2K4\tA0A3Q0FV42_ALLSI\nMAP2K4\tA0A3Q1BYJ5_AMPOC\nMAP2K4\tA0A3Q1EB59_9TELE\nMAP2K4\tA0A3Q1EEF9_9TELE\nMAP2K4\tA0A3Q1LTC6_BOVIN\nMAP2K4\tA0A3Q1N7I1_BOVIN\nMAP2K4\tA0A3Q2H2Q5_HORSE\nMAP2K4\tA0A3Q2QRL1_FUNHE\nMAP2K4\tA0A3Q2TE07_FUNHE\nMAP2K4\tA0A3Q2WG69_HAPBU\nMAP2K4\tA0A3Q7NGF4_CALUR\nMAP2K4\tA0A3Q7PVC1_CALUR\nMAP2K4\tA0A3Q7SIT6_VULVU\nMAP2K4\tA0A3Q7XPU9_URSAR\nMAP2K4\tA0A452G3K6_CAPHI\nMAP2K4\tA0A452G3N3_CAPHI\nMAP2K4\tA0A452IUX1_9SAUR\nMAP2K4\tA0A452IV22_9SAUR\nMAP2K4\tA0A452IV58_9SAUR\nMAP2K4\tA0A452IV80_9SAUR\nMAP2K4\tA0A452IVK3_9SAUR\nMAP2K4\tA0A452SKZ8_URSAM\nMAP2K4\tA0A452SL04_URSAM\nMAP2K4\tA0A452SL09_URSAM\nMAP2K4\tA0A452SL26_URSAM\nMAP2K4\tA0A452SL49_URSAM\nMAP2K4\tA0A452SL54_URSAM\nMAP2K4\tA0A452SL67_URSAM\nMAP2K4\tA0A452V7B9_URSMA\nMAP2K4\tA0A452V7E5_URSMA\nMAP2K4\tA0A452V7F0_URSMA\nMAP2K4\tA0A452V7F8_URSMA\nMAP2K4\tA0A452V7G3_URSMA\nMAP2K4\tA0A452V7G6_URSMA\nMAP2K4\tA0A452V7K2_URSMA\nMAP2K4\tA0A455CA87_PHYMC\nMAP2K4\tA0A493T0V3_ANAPP\nMAP2K4\tA0A4W2C3P1_BOBOX\nMAP2K4\tA0A4W2EG83_BOBOX\nMAP2K4\tA0A4W2HVL0_BOBOX\nMAP2K4\tA0A4W2HZ68_BOBOX\nMAP2K4\tA0A4W6E635_LATCA\nMAP2K4\tA0A4W6E678_LATCA\nMAP2K4\tA0A4W6E8W7_LATCA\nMAP2K4\tA0A4W6E912_LATCA\nMAP2K4\tA0A4W6EA05_LATCA\nMAP2K4\tA0A4W6EAG8_LATCA\nMAP2K4\tA0A4X1SMR4_PIG\nMAP2K4\tA0A4X1SMU6_PIG\nMAP2K4\tA0A4X1SNA9_PIG\nMAP2K4\tA0A4X1SNB9_PIG\nMAP2K4\tA0A4X2L6Z5_VOMUR\nMAP2K4\tA0A4X2LFV1_VOMUR\nMAP2K4\tA0A4X2LI71_VOMUR\nMAP2K4\tA0A4Y2F6B1_ARAVE\nMAP2K4\tA0A5F4C504_CANLF\nMAP2K4\tA0A5F5PRU9_HORSE\nMAP2K4\tA0A5F9CPJ2_RABIT\nMAP2K4\tA0A5F9DLU5_RABIT\nMAP2K4\tA0A5G2QJE6_PIG\nMAP2K4\tA0A5G2QZR5_PIG\nMAP2K4\tA0JMU4_XENLA\nMAP2K4\tA4FVH5_DANRE\nMAP2K4\tA5PJP8_BOVIN\nMAP2K4\tA9UM81_XENTR\nMAP2K4\tB1ATL6_MOUSE\nMAP2K4\tB1NA68_DANRE\nMAP2K4\tD6RRB5_DANRE\nMAP2K4\tE5S8L7_TRISP\nMAP2K4\tF1LP57_RAT\nMAP2K4\tF1PBB6_CANLF\nMAP2K4\tF1SS51_PIG\nMAP2K4\tF6SMU6_HORSE\nMAP2K4\tF7BGB2_XENTR\nMAP2K4\tF7FU91_MACMU\nMAP2K4\tG1KTZ0_ANOCA\nMAP2K4\tG1M3F8_AILME\nMAP2K4\tG1MTA2_MELGA\nMAP2K4\tG1NWW3_MYOLU\nMAP2K4\tG1RWT9_NOMLE\nMAP2K4\tG1SYK3_RABIT\nMAP2K4\tG3PN99_GASAC\nMAP2K4\tG3PNA1_GASAC\nMAP2K4\tG3PNA4_GASAC\nMAP2K4\tG3TBC1_LOXAF\nMAP2K4\tG3VC26_SARHA\nMAP2K4\tG3VC27_SARHA\nMAP2K4\tH0VR36_CAVPO\nMAP2K4\tH0XFS0_OTOGA\nMAP2K4\tH0Z626_TAEGU\nMAP2K4\tH2L641_ORYLA\nMAP2K4\tH2NSS3_PONAB\nMAP2K4\tH2QCA7_PANTR\nMAP2K4\tH2S727_TAKRU\nMAP2K4\tH2S728_TAKRU\nMAP2K4\tH2ZZ74_LATCH\nMAP2K4\tH9FBU8_MACMU\nMAP2K4\tI3JT50_ORENI\nMAP2K4\tI3N200_ICTTR\nMAP2K4\tJ3QLE2_HUMAN\nMAP2K4\tK7APE5_PANTR\nMAP2K4\tK7EP58_HUMAN\nMAP2K4\tK7ESW5_PONAB\nMAP2K4\tK7G452_PELSI\nMAP2K4\tK7G484_PELSI\nMAP2K4\tM0QW52_MOUSE\nMAP2K4\tM0QWG5_MOUSE\nMAP2K4\tM0QWR0_MOUSE\nMAP2K4\tM3WG54_FELCA\nMAP2K4\tM3XKY0_LATCH\nMAP2K4\tM3XX35_MUSPF\nMAP2K4\tMP2K4_HUMAN\nMAP2K4\tMP2K4_MOUSE\nMAP2K4\tQ1JPX6_DANRE\nMAP2K4\tQ4KSH6_RAT\nMAP2K4\tQ4S9U5_TETNG\nMAP2K4\tQ543X6_MOUSE\nMAP2K4\tQ568X2_DANRE\nMAP2K4\tQ8K2U0_MOUSE\nMAP2K4\tQ9DGR7_DANRE\nMAP2K4\tR4GN37_HUMAN\nMAP2K4\tR4GN68_HUMAN\nMAP2K4\tS4VP54_RAT\nMAP2K4\tT2MG98_HYDVU\nMAP2K4\tU3J072_ANAPP\nMAP2K4\tU3K9J8_FICAL\nMAP2K4\tV8PI19_OPHHA\nMAP2K4\tW5Q1D9_SHEEP\nMAP2K4\tW5UKG6_ICTPU\n','From\tTo\nMAP2K2\tA0A0A0MWB2_PAPAN\nMAP2K2\tA0A0B2UUA8_TOXCA\nMAP2K2\tA0A0D9REF2_CHLSB\nMAP2K2\tA0A0F6T521_CLABA\nMAP2K2\tA0A0P6J4K0_HETGA\nMAP2K2\tA0A141GRT3_HETFO\nMAP2K2\tA0A1D5QW55_MACMU\nMAP2K2\tA0A1S2ZUU3_ERIEU\nMAP2K2\tA0A1S3FP56_DIPOR\nMAP2K2\tA0A1U7R0S2_MESAU\nMAP2K2\tA0A1U7RCT8_ALLSI\nMAP2K2\tA0A1V4KQ20_PATFA\nMAP2K2\tA0A1W2P769_MOUSE\nMAP2K2\tA0A1W2P7V9_MOUSE\nMAP2K2\tA0A218UCC3_9PASE\nMAP2K2\tA0A250YHA8_CASCN\nMAP2K2\tA0A2I0LPS5_COLLI\nMAP2K2\tA0A2I2UB94_FELCA\nMAP2K2\tA0A2I3H9X5_NOMLE\nMAP2K2\tA0A2I3MTU5_PAPAN\nMAP2K2\tA0A2I3MZQ8_PAPAN\nMAP2K2\tA0A2I3RZM0_PANTR\nMAP2K2\tA0A2I3ST63_PANTR\nMAP2K2\tA0A2I4AX47_9TELE\nMAP2K2\tA0A2K5EVY2_AOTNA\nMAP2K2\tA0A2K5EVZ4_AOTNA\nMAP2K2\tA0A2K5EW47_AOTNA\nMAP2K2\tA0A2K5I8A0_COLAP\nMAP2K2\tA0A2K5I8A1_COLAP\nMAP2K2\tA0A2K5I8B1_COLAP\nMAP2K2\tA0A2K5LLJ7_CERAT\nMAP2K2\tA0A2K5LLK1_CERAT\nMAP2K2\tA0A2K5PN16_CEBCA\nMAP2K2\tA0A2K5X2M7_MACFA\nMAP2K2\tA0A2K5X2N8_MACFA\nMAP2K2\tA0A2K5X2P8_MACFA\nMAP2K2\tA0A2K5YI82_MANLE\nMAP2K2\tA0A2K6E3Q7_MACNE\nMAP2K2\tA0A2K6E3R3_MACNE\nMAP2K2\tA0A2K6E3R4_MACNE\nMAP2K2\tA0A2K6PDB8_RHIRO\nMAP2K2\tA0A2K6PDG5_RHIRO\nMAP2K2\tA0A2K6UT24_SAIBB\nMAP2K2\tA0A2K6UT63_SAIBB\nMAP2K2\tA0A2P8XJR2_BLAGE\nMAP2K2\tA0A2R9A5D9_PANPA\nMAP2K2\tA0A2R9A9J1_PANPA\nMAP2K2\tA0A2S2NGA6_SCHGA\nMAP2K2\tA0A2U3VKE8_ODORO\nMAP2K2\tA0A2U3Y596_LEPWE\nMAP2K2\tA0A2U4CB63_TURTR\nMAP2K2\tA0A2Y9I9W3_NEOSC\nMAP2K2\tA0A2Y9LU19_DELLE\nMAP2K2\tA0A2Y9LVV0_DELLE\nMAP2K2\tA0A2Y9TIC9_PHYMC\nMAP2K2\tA0A340XKD9_LIPVE\nMAP2K2\tA0A341BIK6_NEOAA\nMAP2K2\tA0A383Z2X0_BALAS\nMAP2K2\tA0A384DRM3_URSMA\nMAP2K2\tA0A3B5K537_TAKRU\nMAP2K2\tA0A3Q7NAJ5_CALUR\nMAP2K2\tA0A3Q7TZB8_VULVU\nMAP2K2\tA0A3Q7VG39_URSAR\nMAP2K2\tA0A3R7GWD9_CLOSI\nMAP2K2\tA0A3S5ZPX3_BOVIN\nMAP2K2\tA0A452EN53_CAPHI\nMAP2K2\tA0A452EN98_CAPHI\nMAP2K2\tA0A452ENE3_CAPHI\nMAP2K2\tA0A452TP17_URSMA\nMAP2K2\tA0A452TP56_URSMA\nMAP2K2\tA0A452TP75_URSMA\nMAP2K2\tA0A455AVP2_PHYMC\nMAP2K2\tA0A4W2D5U5_BOBOX\nMAP2K2\tA0A4W2D5W5_BOBOX\nMAP2K2\tA0A4W2FV68_BOBOX\nMAP2K2\tA0A4W3JPH8_CALMI\nMAP2K2\tA0A4W3JPI3_CALMI\nMAP2K2\tA0A4W3JPJ1_CALMI\nMAP2K2\tA0A4W3K239_CALMI\nMAP2K2\tA0A4W3K242_CALMI\nMAP2K2\tA0A4W3K245_CALMI\nMAP2K2\tA0A4W3K3K8_CALMI\nMAP2K2\tA0A4W3KFN9_CALMI\nMAP2K2\tA0A4X1VM93_PIG\nMAP2K2\tA0A4X1VRD9_PIG\nMAP2K2\tA0A4X1VTZ6_PIG\nMAP2K2\tA0A4X2ME47_VOMUR\nMAP2K2\tA0A4Z2ICE2_9TELE\nMAP2K2\tA0A5F4BP50_CANLF\nMAP2K2\tA0A5F4BZH7_CANLF\nMAP2K2\tA0A5F4C1T3_CANLF\nMAP2K2\tA0A5F4W2I3_CALJA\nMAP2K2\tA0A5F7ZYR0_MACMU\nMAP2K2\tA0A5F8GLW3_MONDO\nMAP2K2\tA0A5F8H6I9_MONDO\nMAP2K2\tA0A5G2QEB7_PIG\nMAP2K2\tA0A5G2RHW3_PIG\nMAP2K2\tB3KS97_HUMAN\nMAP2K2\tD3Z446_MOUSE\nMAP2K2\tF1NYZ2_CHICK\nMAP2K2\tF6S8W1_MONDO\nMAP2K2\tF6S9S9_MONDO\nMAP2K2\tF6YUU0_HORSE\nMAP2K2\tF7BMT0_MACMU\nMAP2K2\tF7GNM2_CALJA\nMAP2K2\tF7GNS9_CALJA\nMAP2K2\tG1QS02_NOMLE\nMAP2K2\tG5E9C7_HUMAN\nMAP2K2\tH0VU84_CAVPO\nMAP2K2\tH0WJ32_OTOGA\nMAP2K2\tH2LJN3_ORYLA\nMAP2K2\tH2V9F3_TAKRU\nMAP2K2\tH3AN70_LATCH\nMAP2K2\tH9EUI5_MACMU\nMAP2K2\tH9G3P4_ANOCA\nMAP2K2\tH9YUB5_MACMU\nMAP2K2\tI3L5D4_PIG\nMAP2K2\tI3M097_ICTTR\nMAP2K2\tK7BYT0_PANTR\nMAP2K2\tK7GPS0_PIG\nMAP2K2\tM0QWN2_MOUSE\nMAP2K2\tM0R1B6_HUMAN\nMAP2K2\tM3WK49_FELCA\nMAP2K2\tMP2K2_CANLF\nMAP2K2\tMP2K2_CHICK\nMAP2K2\tMP2K2_CYPCA\nMAP2K2\tMP2K2_HUMAN\nMAP2K2\tMP2K2_MOUSE\nMAP2K2\tMP2K2_RAT\nMAP2K2\tMP2K2_XENLA\nMAP2K2\tQ17QH2_BOVIN\nMAP2K2\tQ1LY62_DANRE\nMAP2K2\tQ32Q50_DANRE\nMAP2K2\tQ3UPM3_MOUSE\nMAP2K2\tQ3USU3_MOUSE\nMAP2K2\tQ8CB46_MOUSE\nMAP2K2\tQ91YS7_MOUSE\nMAP2K2\tU3F340_CALJA\nMAP2K2\tW5PM87_SHEEP\n','From\tTo\nTTK\tA0A034V3C8_BACDO\nTTK\tA0A068Z6S2_9GAMM\nTTK\tA0A072GTB3_VIBPH\nTTK\tA0A077MYS3_XENBV\nTTK\tA0A077N1V4_XENBV\nTTK\tA0A077NTM1_XENBV\nTTK\tA0A077P4D7_XENBV\nTTK\tA0A077P8V6_XENBV\nTTK\tA0A077PX06_XENBV\nTTK\tA0A077QGT1_XENBV\nTTK\tA0A087FLS1_KLEVA\nTTK\tA0A087WPS8_MOUSE\nTTK\tA0A096NW85_PAPAN\nTTK\tA0A0A8P1F4_XENNE\nTTK\tA0A0B2VAD3_TOXCA\nTTK\tA0A0B4JD39_DROME\nTTK\tA0A0B4JDG1_DROME\nTTK\tA0A0B7J7C1_PHOPO\nTTK\tA0A0D6I355_SALTM\nTTK\tA0A0D7LZE7_CITFR\nTTK\tA0A0D9RXV7_CHLSB\nTTK\tA0A0F5BBX0_SALER\nTTK\tA0A0G2K2S5_RAT\nTTK\tA0A0G3KAN4_ECOLX\nTTK\tA0A0G3QGX3_9GAMM\nTTK\tA0A0H3KJP9_BURM1\nTTK\tA0A0I4LLK1_SHISO\nTTK\tA0A0K0HFH7_SALBC\nTTK\tA0A0K2T0V0_LEPSM\nTTK\tA0A0K8UWZ9_BACLA\nTTK\tA0A0M4KPM0_9BURK\nTTK\tA0A0P6JE68_HETGA\nTTK\tA0A0Q4M885_9GAMM\nTTK\tA0A0Q4NH88_9GAMM\nTTK\tA0A0Q4NHP8_9GAMM\nTTK\tA0A0R4CVY4_XENNE\nTTK\tA0A0S7HKS8_9TELE\nTTK\tA0A0S7HL69_9TELE\nTTK\tA0A0S7HLC3_9TELE\nTTK\tA0A0S7HLD5_9TELE\nTTK\tA0A0S7HLZ7_9TELE\nTTK\tA0A0S7HPJ8_9TELE\nTTK\tA0A0S7HQ03_9TELE\nTTK\tA0A0T7RU93_SALET\nTTK\tA0A0U3TKJ3_9GAMM\nTTK\tA0A0V0J854_SCHSO\nTTK\tA0A0V9JKB9_9ENTR\nTTK\tA0A0W2I825_9ENTR\nTTK\tA0A0X3NVK4_SCHSO\nTTK\tA0A0X3P106_SCHSO\nTTK\tA0A0X3Q784_SCHSO\nTTK\tA0A151MU83_ALLMI\nTTK\tA0A1A7X3F0_9TELE\nTTK\tA0A1A7YIE7_9TELE\nTTK\tA0A1A8AG61_NOTFU\nTTK\tA0A1A8C698_9TELE\nTTK\tA0A1A8C8K3_9TELE\nTTK\tA0A1A8F1G0_9TELE\nTTK\tA0A1A8H4H4_9TELE\nTTK\tA0A1A8ITF6_NOTKU\nTTK\tA0A1A8JUB4_NOTKU\nTTK\tA0A1A8MPR5_9TELE\nTTK\tA0A1A8N764_9TELE\nTTK\tA0A1A8N7R2_9TELE\nTTK\tA0A1A8RT28_9TELE\nTTK\tA0A1A8V4L4_NOTFU\nTTK\tA0A1L8G354_XENLA\nTTK\tA0A1Q8MU28_SHIBO\nTTK\tA0A1S3AKN9_ERIEU\nTTK\tA0A1S3GFS9_DIPOR\nTTK\tA0A1S3GFY7_DIPOR\nTTK\tA0A1S3GG88_DIPOR\nTTK\tA0A1S3GI81_DIPOR\nTTK\tA0A1U7U8E3_TARSY\nTTK\tA0A1V4JNQ5_PATFA\nTTK\tA0A1W5DT31_9GAMM\nTTK\tA0A1W9F1U8_9GAMM\nTTK\tA0A1X1MTL0_9VIBR\nTTK\tA0A212I5I7_9ENTR\nTTK\tA0A218UEF9_9PASE\nTTK\tA0A285BBM9_9ENTR\nTTK\tA0A286ZQJ7_PIG\nTTK\tA0A2B4SAZ5_STYPI\nTTK\tA0A2B4SCD8_STYPI\nTTK\tA0A2D0S2J0_ICTPU\nTTK\tA0A2D0S2M3_ICTPU\nTTK\tA0A2I0MTR9_COLLI\nTTK\tA0A2I3HAE2_NOMLE\nTTK\tA0A2I3MM14_PAPAN\nTTK\tA0A2I3NAZ9_PAPAN\nTTK\tA0A2I4C742_9TELE\nTTK\tA0A2I5HMR4_SALDZ\nTTK\tA0A2J8PVP5_PANTR\nTTK\tA0A2J8PVP7_PANTR\nTTK\tA0A2K4XCN1_PSEVC\nTTK\tA0A2K5E880_AOTNA\nTTK\tA0A2K5E894_AOTNA\nTTK\tA0A2K5K2C7_COLAP\nTTK\tA0A2K5K2C8_COLAP\nTTK\tA0A2K5LGI9_CERAT\nTTK\tA0A2K5LGL9_CERAT\nTTK\tA0A2K5LGP7_CERAT\nTTK\tA0A2K5RD99_CEBCA\nTTK\tA0A2K5RDA1_CEBCA\nTTK\tA0A2K5V506_MACFA\nTTK\tA0A2K5V541_MACFA\nTTK\tA0A2K5V552_MACFA\nTTK\tA0A2K5YB36_MANLE\nTTK\tA0A2K5YB42_MANLE\nTTK\tA0A2K5YB58_MANLE\nTTK\tA0A2K6E6V3_MACNE\nTTK\tA0A2K6E6V6_MACNE\nTTK\tA0A2K6E6V9_MACNE\nTTK\tA0A2K6E6W7_MACNE\nTTK\tA0A2K6FT28_PROCO\nTTK\tA0A2K6MJA3_RHIBE\nTTK\tA0A2K6MJA9_RHIBE\nTTK\tA0A2K6MJB7_RHIBE\nTTK\tA0A2K6NLI6_RHIRO\nTTK\tA0A2K6NLJ2_RHIRO\nTTK\tA0A2K6S095_SAIBB\nTTK\tA0A2K6S0B1_SAIBB\nTTK\tA0A2N8ZGN0_9VIBR\nTTK\tA0A2P8ZLU2_BLAGE\nTTK\tA0A2R7X4Q5_ONCFA\nTTK\tA0A2R8MHM6_CALJA\nTTK\tA0A2R9CFF4_PANPA\nTTK\tA0A2S2PTQ9_SCHGA\nTTK\tA0A2S8E7U5_SHIDY\nTTK\tA0A2U0MYL1_KLEPN\nTTK\tA0A2U2V9Q5_ECOLX\nTTK\tA0A2U3VX47_ODORO\nTTK\tA0A2U3ZF99_ODORO\nTTK\tA0A2U4AVB0_TURTR\nTTK\tA0A2U4AVB4_TURTR\nTTK\tA0A2U4AVB8_TURTR\nTTK\tA0A2U4AVR9_TURTR\nTTK\tA0A2X4W6U4_SALER\nTTK\tA0A2Y2SP99_SHIFL\nTTK\tA0A2Y9EM89_PHYMC\nTTK\tA0A2Y9EN14_PHYMC\nTTK\tA0A2Y9EN97_PHYMC\nTTK\tA0A2Y9HBN2_NEOSC\nTTK\tA0A2Y9PGH3_DELLE\nTTK\tA0A2Y9PSW5_DELLE\nTTK\tA0A2Y9TD05_PHYMC\nTTK\tA0A330LRN6_9GAMM\nTTK\tA0A330LX82_9GAMM\nTTK\tA0A337RY86_FELCA\nTTK\tA0A340XBW6_LIPVE\nTTK\tA0A340XCD3_LIPVE\nTTK\tA0A341CEP4_NEOAA\nTTK\tA0A341CF22_NEOAA\nTTK\tA0A379PUD8_SALER\nTTK\tA0A379XIW4_SALER\nTTK\tA0A383YYS3_BALAS\nTTK\tA0A384BKH4_URSMA\nTTK\tA0A384BLC1_URSMA\nTTK\tA0A3A2HTI1_9VIBR\nTTK\tA0A3A6RFS1_ECOLX\nTTK\tA0A3Q0CR35_MESAU\nTTK\tA0A3Q0E7B0_TARSY\nTTK\tA0A3Q0HNW6_ALLSI\nTTK\tA0A3Q2LCD8_HORSE\nTTK\tA0A3Q2U8H9_CHICK\nTTK\tA0A3Q2UNU4_CHICK\nTTK\tA0A3Q7T1B3_VULVU\nTTK\tA0A3Q7WMN0_URSAR\nTTK\tA0A3Q7XJR8_URSAR\nTTK\tA0A3S4FXB0_SALER\nTTK\tA0A3S5DAZ6_SALET\nTTK\tA0A3S5YP22_SALER\nTTK\tA0A3S6F0S0_YERET\nTTK\tA0A3T7XIY6_SALET\nTTK\tA0A3U2F2J8_SHIFL\nTTK\tA0A3V9HJL9_SALET\nTTK\tA0A403DR36_SHIFL\nTTK\tA0A419PJ04_CLOSI\nTTK\tA0A422XA18_9ENTR\nTTK\tA0A422ZZL2_KLEPN\nTTK\tA0A443IGL1_9GAMM\nTTK\tA0A447JA72_SALET\nTTK\tA0A452E2L3_CAPHI\nTTK\tA0A452GFW4_9SAUR\nTTK\tA0A452QTK5_URSAM\nTTK\tA0A452QTL7_URSAM\nTTK\tA0A452QTM2_URSAM\nTTK\tA0A4C1TUA7_9NEOP\nTTK\tA0A4C1UA75_9NEOP\nTTK\tA0A4C1UAP4_9NEOP\nTTK\tA0A4C1X2C9_9NEOP\nTTK\tA0A4C1Z3D1_9NEOP\nTTK\tA0A4W2CD85_BOBOX\nTTK\tA0A4W2E9M3_BOBOX\nTTK\tA0A4W2FU14_BOBOX\nTTK\tA0A4W2FXA0_BOBOX\nTTK\tA0A4W6E699_LATCA\nTTK\tA0A4W6E907_LATCA\nTTK\tA0A4W6E924_LATCA\nTTK\tA0A4W6E9Z2_LATCA\nTTK\tA0A4X1UZ17_PIG\nTTK\tA0A4X1V396_PIG\nTTK\tA0A4X1V4B2_PIG\nTTK\tA0A4X2K2F1_VOMUR\nTTK\tA0A4X2K2G3_VOMUR\nTTK\tA0A4Y2K3H2_ARAVE\nTTK\tA0A4Z2H712_9TELE\nTTK\tA0A509AW88_9ENTR\nTTK\tA0A509B3Z5_9ENTR\nTTK\tA0A5F4CFD9_CANLF\nTTK\tA0A5F8A1I8_MACMU\nTTK\tA0A5F8AER4_MACMU\nTTK\tA0A5F8GDQ6_MONDO\nTTK\tA0A5F8GM94_MONDO\nTTK\tA0A5F9D691_RABIT\nTTK\tA0A5G2QAH1_PIG\nTTK\tA0A5G2QVM7_PIG\nTTK\tA0A5G2R0J7_PIG\nTTK\tA0A5G3HTW0_XENTR\nTTK\tA0A5G3JIW2_XENTR\nTTK\tA0A5K1V5J2_SALER\nTTK\tB4QT85_DROSI\nTTK\tB6BVU8_9PROT\nTTK\tC3SMC2_ECOLX\nTTK\tC8UJ28_ECO1A\nTTK\tD2TBZ3_ERWP6\nTTK\tD2TIW0_CITRI\nTTK\tD3H291_ECO44\nTTK\tD3V738_XENBS\nTTK\tD3VGY2_XENNA\nTTK\tD4A4S7_RAT\nTTK\tD4HU92_ERWAC\nTTK\tD6REX1_HUMAN\nTTK\tD6RF82_HUMAN\nTTK\tD6RFY1_HUMAN\nTTK\tD6RIC6_HUMAN\nTTK\tD8LFJ7_ECTSI\nTTK\tD8LFK1_ECTSI\nTTK\tD8MKD3_ERWBE\nTTK\tE0SLM2_DICD3\nTTK\tE1BPP1_BOVIN\nTTK\tE1BQR2_CHICK\nTTK\tE1SIT9_PANVC\nTTK\tE2QGZ6_ECOLX\nTTK\tE2QYE4_CANLF\nTTK\tE5B086_ERWAM\nTTK\tE9QBU7_DANRE\nTTK\tF1R3D0_DANRE\nTTK\tF1R3H9_DANRE\nTTK\tF1RQL2_PIG\nTTK\tF6Q0S3_XENTR\nTTK\tF6UR58_XENTR\nTTK\tF6VA00_MACMU\nTTK\tF6WLR2_HORSE\nTTK\tF6XUX9_MONDO\nTTK\tF7A5J0_XENTR\nTTK\tF7DXE1_XENTR\nTTK\tF7G2V9_ORNAN\nTTK\tF7GAQ5_CALJA\nTTK\tG0AE47_COLFT\nTTK\tG1KTV1_ANOCA\nTTK\tG1M2B8_AILME\nTTK\tG1NLX2_MELGA\nTTK\tG1P648_MYOLU\nTTK\tG1RPD1_NOMLE\nTTK\tG1SSE5_RABIT\nTTK\tG3UI82_LOXAF\nTTK\tG3UJ38_LOXAF\nTTK\tG3WSP2_SARHA\nTTK\tH0X3W1_OTOGA\nTTK\tH2N2D7_ORYLA\nTTK\tH2PJN3_PONAB\nTTK\tH2S275_TAKRU\nTTK\tH2S276_TAKRU\nTTK\tH2S278_TAKRU\nTTK\tH3BA86_LATCH\nTTK\tH4IHY2_ECOLX\nTTK\tH4IYW4_ECOLX\nTTK\tH4KNS5_ECOLX\nTTK\tH4LIZ5_ECOLX\nTTK\tH8W6P5_MARHS\nTTK\tH9ZD28_MACMU\nTTK\tI2BEI8_SHIBC\nTTK\tI3ITY1_ORENI\nTTK\tI3ITY2_ORENI\nTTK\tJ7RYB3_ECOLX\nTTK\tJ9NVE1_CANLF\nTTK\tK0C7F3_ALCDB\nTTK\tK0WQ57_SHIFL\nTTK\tK5TGW5_VIBHA\nTTK\tK5V9L7_9VIBR\nTTK\tK7DTY6_PANTR\nTTK\tK7FS58_PELSI\nTTK\tM3WYI3_FELCA\nTTK\tM3XH76_LATCH\nTTK\tM3Z322_MUSPF\nTTK\tO61527_DROSL\nTTK\tQ05B23_XENTR\nTTK\tQ0I0Y2_HAES1\nTTK\tQ0VT63_ALCBS\nTTK\tQ1LS48_CUPMC\nTTK\tQ1R4V1_ECOUT\nTTK\tQ31UY5_SHIBS\nTTK\tQ329L6_SHIDS\nTTK\tQ3IFE3_PSEHT\nTTK\tQ3JXB1_BURP1\nTTK\tQ3TPW2_MOUSE\nTTK\tQ3YW03_SHISS\nTTK\tQ6PD96_MOUSE\nTTK\tQ8BX21_MOUSE\nTTK\tQ8BY97_MOUSE\nTTK\tQ8T3M3_DROME\nTTK\tQ98T92_XENLA\nTTK\tR0MMK8_NOSB1\nTTK\tSLMA_ECOLI\nTTK\tT2MG79_HYDVU\nTTK\tTTKA_DROME\nTTK\tTTKB_DROME\nTTK\tTTK_DANRE\nTTK\tTTK_HUMAN\nTTK\tTTK_MACFA\nTTK\tTTK_MOUSE\nTTK\tU1I9M5_9PAST\nTTK\tU3IEM5_ANAPP\nTTK\tU3JJ28_FICAL\nTTK\tU6DV95_NEOVI\nTTK\tV5Z363_9GAMM\nTTK\tV8NFV6_OPHHA\nTTK\tV9KC47_CALMI\nTTK\tW0BU60_9ENTR\nTTK\tW5PB56_SHEEP\n','From\tTo\nSTK16\tA0A034WTQ4_BACDO\nSTK16\tA0A087WQ12_MOUSE\nSTK16\tA0A087WQT0_MOUSE\nSTK16\tA0A096MX79_PAPAN\nSTK16\tA0A0A1XD46_ZEUCU\nSTK16\tA0A0A9W8K3_LYGHE\nSTK16\tA0A0D9R6Y1_CHLSB\nSTK16\tA0A0K8UEV4_BACLA\nSTK16\tA0A0S7GXJ8_9TELE\nSTK16\tA0A0S7GYL5_9TELE\nSTK16\tA0A0S7H1M6_9TELE\nSTK16\tA0A0V0SHJ6_9BILA\nSTK16\tA0A0V0UDS1_9BILA\nSTK16\tA0A0V0X5N8_9BILA\nSTK16\tA0A0V0X5T1_9BILA\nSTK16\tA0A0V0YNB9_TRIPS\nSTK16\tA0A0V1ACJ9_9BILA\nSTK16\tA0A0V1BUL9_TRISP\nSTK16\tA0A0V1BUR0_TRISP\nSTK16\tA0A0V1BVD4_TRISP\nSTK16\tA0A0V1BVJ7_TRISP\nSTK16\tA0A0V1BWU9_TRISP\nSTK16\tA0A0V1DJF0_TRIBR\nSTK16\tA0A0V1EYT7_TRIPS\nSTK16\tA0A0V1EYU3_TRIPS\nSTK16\tA0A0V1EZ55_TRIPS\nSTK16\tA0A0V1EZ96_TRIPS\nSTK16\tA0A0V1EZM8_TRIPS\nSTK16\tA0A0V1G6M9_TRIPS\nSTK16\tA0A0V1I0N7_9BILA\nSTK16\tA0A0V1JH70_TRIPS\nSTK16\tA0A0V1JH72_TRIPS\nSTK16\tA0A0V1JHC9_TRIPS\nSTK16\tA0A0V1JHD6_TRIPS\nSTK16\tA0A0V1JHI2_TRIPS\nSTK16\tA0A0V1JHJ1_TRIPS\nSTK16\tA0A0V1KEH7_TRIPS\nSTK16\tA0A0V1LVB0_9BILA\nSTK16\tA0A0V1MV79_9BILA\nSTK16\tA0A0V1MVD4_9BILA\nSTK16\tA0A0V1MVM1_9BILA\nSTK16\tA0A0V1PP51_9BILA\nSTK16\tA0A0X3NIY6_SCHSO\nSTK16\tA0A0X3P7M0_SCHSO\nSTK16\tA0A0X3PPQ8_SCHSO\nSTK16\tA0A0X3PWW2_SCHSO\nSTK16\tA0A151LY83_ALLMI\nSTK16\tA0A151LY84_ALLMI\nSTK16\tA0A1A7Y6A1_9TELE\nSTK16\tA0A1A7ZYY2_NOTFU\nSTK16\tA0A1A8E485_9TELE\nSTK16\tA0A1A8H8Z1_9TELE\nSTK16\tA0A1A8K3S9_NOTKU\nSTK16\tA0A1A8N4E6_9TELE\nSTK16\tA0A1A8NBQ7_9TELE\nSTK16\tA0A1Q9D386_SYMMI\nSTK16\tA0A1S2ZIB0_ERIEU\nSTK16\tA0A1S3GBZ2_DIPOR\nSTK16\tA0A1S3GF01_DIPOR\nSTK16\tA0A1U7QBS0_MESAU\nSTK16\tA0A1U7TAJ7_TARSY\nSTK16\tA0A1V4K6E0_PATFA\nSTK16\tA0A218VBB1_9PASE\nSTK16\tA0A250YF73_CASCN\nSTK16\tA0A287A437_PIG\nSTK16\tA0A287B9I3_PIG\nSTK16\tA0A2B4SKR5_STYPI\nSTK16\tA0A2I3GEA1_NOMLE\nSTK16\tA0A2I3GJE8_NOMLE\nSTK16\tA0A2I3GVZ4_NOMLE\nSTK16\tA0A2I3LG74_PAPAN\nSTK16\tA0A2I3MUF1_PAPAN\nSTK16\tA0A2I3N9K2_PAPAN\nSTK16\tA0A2I3SA74_PANTR\nSTK16\tA0A2I3SAB9_PANTR\nSTK16\tA0A2I3STR9_PANTR\nSTK16\tA0A2I4ASI4_9TELE\nSTK16\tA0A2J7RE03_9NEOP\nSTK16\tA0A2J8TUI3_PONAB\nSTK16\tA0A2K5EZL2_AOTNA\nSTK16\tA0A2K5EZM4_AOTNA\nSTK16\tA0A2K5EZN0_AOTNA\nSTK16\tA0A2K5Q9L2_CEBCA\nSTK16\tA0A2K5Q9N9_CEBCA\nSTK16\tA0A2K5Q9P1_CEBCA\nSTK16\tA0A2K5U4R0_MACFA\nSTK16\tA0A2K5U4S8_MACFA\nSTK16\tA0A2K5U4T2_MACFA\nSTK16\tA0A2K5U4X3_MACFA\nSTK16\tA0A2K5YDM5_MANLE\nSTK16\tA0A2K5YDR2_MANLE\nSTK16\tA0A2K5YDS1_MANLE\nSTK16\tA0A2K5YDU0_MANLE\nSTK16\tA0A2K6E511_MACNE\nSTK16\tA0A2K6E521_MACNE\nSTK16\tA0A2K6E522_MACNE\nSTK16\tA0A2K6E534_MACNE\nSTK16\tA0A2K6GWX8_PROCO\nSTK16\tA0A2K6GX16_PROCO\nSTK16\tA0A2K6GX20_PROCO\nSTK16\tA0A2K6GX71_PROCO\nSTK16\tA0A2K6QPJ5_RHIRO\nSTK16\tA0A2K6QPK4_RHIRO\nSTK16\tA0A2K6QPN6_RHIRO\nSTK16\tA0A2K6QPP2_RHIRO\nSTK16\tA0A2K6UZ29_SAIBB\nSTK16\tA0A2K6UZ41_SAIBB\nSTK16\tA0A2K6UZ67_SAIBB\nSTK16\tA0A2R9ADE9_PANPA\nSTK16\tA0A2R9ADF3_PANPA\nSTK16\tA0A2R9AI81_PANPA\nSTK16\tA0A2R9AJQ9_PANPA\nSTK16\tA0A2S2P6E3_SCHGA\nSTK16\tA0A2S2QP04_9HEMI\nSTK16\tA0A2U3YGV3_LEPWE\nSTK16\tA0A2U3ZI62_ODORO\nSTK16\tA0A2Y9FGJ8_PHYMC\nSTK16\tA0A2Y9FIQ0_PHYMC\nSTK16\tA0A2Y9I3U7_NEOSC\nSTK16\tA0A2Y9MMF2_DELLE\nSTK16\tA0A2Y9MWN7_DELLE\nSTK16\tA0A337S8S3_FELCA\nSTK16\tA0A337SGY1_FELCA\nSTK16\tA0A340W8Y8_LIPVE\nSTK16\tA0A341CVY1_NEOAA\nSTK16\tA0A341CYC1_NEOAA\nSTK16\tA0A371E6C7_MUCPR\nSTK16\tA0A384AKY6_BALAS\nSTK16\tA0A384AL15_BALAS\nSTK16\tA0A384C1E9_URSMA\nSTK16\tA0A384C2K2_URSMA\nSTK16\tA0A3B3I3R2_ORYLA\nSTK16\tA0A3B5KH64_TAKRU\nSTK16\tA0A3Q0E363_TARSY\nSTK16\tA0A3Q2I4A1_HORSE\nSTK16\tA0A3Q2L1U8_HORSE\nSTK16\tA0A3Q7QKS7_CALUR\nSTK16\tA0A3Q7SMV3_VULVU\nSTK16\tA0A3Q7TRN9_VULVU\nSTK16\tA0A3Q7V6E4_URSAR\nSTK16\tA0A3Q7WBG2_URSAR\nSTK16\tA0A419PWW4_CLOSI\nSTK16\tA0A452E2Z7_CAPHI\nSTK16\tA0A452HIB8_9SAUR\nSTK16\tA0A452QZ38_URSAM\nSTK16\tA0A452UX75_URSMA\nSTK16\tA0A455AYZ7_PHYMC\nSTK16\tA0A4D8YBY1_SALSN\nSTK16\tA0A4D8YMG9_SALSN\nSTK16\tA0A4D8ZT97_SALSN\nSTK16\tA0A4D9C5V5_SALSN\nSTK16\tA0A4W2CRM3_BOBOX\nSTK16\tA0A4W2E8V1_BOBOX\nSTK16\tA0A4W2E9Y4_BOBOX\nSTK16\tA0A4W2EH31_BOBOX\nSTK16\tA0A4W2GGZ7_BOBOX\nSTK16\tA0A4W2GJ15_BOBOX\nSTK16\tA0A4W2GJ61_BOBOX\nSTK16\tA0A4W4GZY2_ELEEL\nSTK16\tA0A4X1UDN6_PIG\nSTK16\tA0A4X1UFG6_PIG\nSTK16\tA0A4X1UG72_PIG\nSTK16\tA0A4X1UG92_PIG\nSTK16\tA0A4X2JLT3_VOMUR\nSTK16\tA0A4X2JM80_VOMUR\nSTK16\tA0A4X2JT23_VOMUR\nSTK16\tA0A4Z2I8U6_9TELE\nSTK16\tA0A5B7GCL7_PORTR\nSTK16\tA0A5F4BRS6_CANLF\nSTK16\tA0A5F4CLM9_CANLF\nSTK16\tA0A5F4DG08_CANLF\nSTK16\tA0A5F5PZT7_HORSE\nSTK16\tA0A5F8GD59_MONDO\nSTK16\tA0A5F8HB10_MONDO\nSTK16\tA0A5F9C2P8_RABIT\nSTK16\tA0A5F9CKA0_RABIT\nSTK16\tA0A5G3L1L5_XENTR\nSTK16\tA2CEW6_DANRE\nSTK16\tA2CEW7_DANRE\nSTK16\tB0KWI4_CALJA\nSTK16\tB4DPS1_HUMAN\nSTK16\tB5X278_SALSA\nSTK16\tB8ZZI5_HUMAN\nSTK16\tB8ZZN3_HUMAN\nSTK16\tC1BPG6_CALRO\nSTK16\tC1C2F2_CALCM\nSTK16\tC1EBL8_MICCC\nSTK16\tC3KHU3_ANOFI\nSTK16\tD3YXM1_MOUSE\nSTK16\tE2RC53_CANLF\nSTK16\tE5SEQ8_TRISP\nSTK16\tF1SR81_PIG\nSTK16\tF6PJD9_CALJA\nSTK16\tF6T6L5_CALJA\nSTK16\tF6TMQ9_MONDO\nSTK16\tF6TSU0_CALJA\nSTK16\tF7A9H7_XENTR\nSTK16\tF7B0W4_MACMU\nSTK16\tF7B0X8_MACMU\nSTK16\tF7BGI1_HORSE\nSTK16\tG1DGH7_CAPHI\nSTK16\tG1KNQ2_ANOCA\nSTK16\tG1L6B1_AILME\nSTK16\tG1L6B7_AILME\nSTK16\tG1PAP0_MYOLU\nSTK16\tG1RDP2_NOMLE\nSTK16\tG1TH34_RABIT\nSTK16\tG1TX06_RABIT\nSTK16\tG3T301_LOXAF\nSTK16\tG3WJX6_SARHA\nSTK16\tG5AKJ9_HETGA\nSTK16\tG7N8Z5_MACMU\nSTK16\tH0V3I4_CAVPO\nSTK16\tH0XDI9_OTOGA\nSTK16\tH0Z6A1_TAEGU\nSTK16\tH2LFF3_ORYLA\nSTK16\tH2LFF4_ORYLA\nSTK16\tH2R3E2_PANTR\nSTK16\tH3AJ63_LATCH\nSTK16\tI0FJS0_MACMU\nSTK16\tI3J5U8_ORENI\nSTK16\tI3MZX5_ICTTR\nSTK16\tK7FG06_PELSI\nSTK16\tM3X5R7_FELCA\nSTK16\tM3YYT9_MUSPF\nSTK16\tQ05DP1_MOUSE\nSTK16\tQ2KI50_BOVIN\nSTK16\tQ566X9_DANRE\nSTK16\tQ641G0_XENLA\nSTK16\tQ6GLG5_XENTR\nSTK16\tQ8BSV9_MOUSE\nSTK16\tQ8CCS9_MOUSE\nSTK16\tSTK16_HUMAN\nSTK16\tSTK16_MOUSE\nSTK16\tSTK16_RAT\nSTK16\tT2MHK6_HYDVU\nSTK16\tU3IS16_ANAPP\nSTK16\tU3JNS6_FICAL\nSTK16\tV5GVQ1_ANOGL\nSTK16\tV8P833_OPHHA\nSTK16\tW5QFJ5_SHEEP\nSTK16\tW5U6U4_ICTPU\nSTK16\tW8C531_CERCA\n','From\tTo\nEGFR\tA0A034V3L8_BACDO\nEGFR\tA0A034V485_BACDO\nEGFR\tA0A059PB47_CRAGI\nEGFR\tA0A060CTT3_PINIB\nEGFR\tA0A067YL82_LIMDI\nEGFR\tA0A068JCC9_HUMAN\nEGFR\tA0A068JD88_HUMAN\nEGFR\tA0A096MVJ9_PAPAN\nEGFR\tA0A0D6DQG3_BLAGE\nEGFR\tA0A0D9RU37_CHLSB\nEGFR\tA0A0E3H1W4_CAMFO\nEGFR\tA0A0P0GH12_FELCA\nEGFR\tA0A0S7EJH6_9TELE\nEGFR\tA0A0S7JYE0_9TELE\nEGFR\tA0A0S7JYF2_9TELE\nEGFR\tA0A0S7JYH1_9TELE\nEGFR\tA0A0S7JYS5_9TELE\nEGFR\tA0A0V0S3N5_9BILA\nEGFR\tA0A0V0S3S2_9BILA\nEGFR\tA0A0V0S3X6_9BILA\nEGFR\tA0A0V0U7F9_9BILA\nEGFR\tA0A0V0U7H9_9BILA\nEGFR\tA0A0V0U822_9BILA\nEGFR\tA0A0V0U826_9BILA\nEGFR\tA0A0V0WZJ9_9BILA\nEGFR\tA0A0V0WZX5_9BILA\nEGFR\tA0A0V0WZX9_9BILA\nEGFR\tA0A0V0WZY6_9BILA\nEGFR\tA0A0V0WZZ7_9BILA\nEGFR\tA0A0V0X0Q1_9BILA\nEGFR\tA0A0V0XQS2_TRIPS\nEGFR\tA0A0V0XQU1_TRIPS\nEGFR\tA0A0V0XQU9_TRIPS\nEGFR\tA0A0V0XQX5_TRIPS\nEGFR\tA0A0V0XR13_TRIPS\nEGFR\tA0A0V0XR50_TRIPS\nEGFR\tA0A0V0XRJ1_TRIPS\nEGFR\tA0A0V1A433_9BILA\nEGFR\tA0A0V1A508_9BILA\nEGFR\tA0A0V1A545_9BILA\nEGFR\tA0A0V1A584_9BILA\nEGFR\tA0A0V1BBQ3_TRISP\nEGFR\tA0A0V1D949_TRIBR\nEGFR\tA0A0V1D977_TRIBR\nEGFR\tA0A0V1D9B5_TRIBR\nEGFR\tA0A0V1D9G0_TRIBR\nEGFR\tA0A0V1D9Q2_TRIBR\nEGFR\tA0A0V1D9R0_TRIBR\nEGFR\tA0A0V1D9R6_TRIBR\nEGFR\tA0A0V1DAM4_TRIBR\nEGFR\tA0A0V1DAW6_TRIBR\nEGFR\tA0A0V1E8F1_TRIPS\nEGFR\tA0A0V1FHM9_TRIPS\nEGFR\tA0A0V1FHX6_TRIPS\nEGFR\tA0A0V1HEC5_9BILA\nEGFR\tA0A0V1HEW6_9BILA\nEGFR\tA0A0V1HF75_9BILA\nEGFR\tA0A0V1HFL1_9BILA\nEGFR\tA0A0V1HFW0_9BILA\nEGFR\tA0A0V1HG55_9BILA\nEGFR\tA0A0V1HG78_9BILA\nEGFR\tA0A0V1HGV1_9BILA\nEGFR\tA0A0V1J535_TRIPS\nEGFR\tA0A0V1J554_TRIPS\nEGFR\tA0A0V1J558_TRIPS\nEGFR\tA0A0V1J583_TRIPS\nEGFR\tA0A0V1JYJ9_TRIPS\nEGFR\tA0A0V1JYL9_TRIPS\nEGFR\tA0A0V1JYM5_TRIPS\nEGFR\tA0A0V1JYQ1_TRIPS\nEGFR\tA0A0V1JYT1_TRIPS\nEGFR\tA0A0V1JYV3_TRIPS\nEGFR\tA0A0V1JYX9_TRIPS\nEGFR\tA0A0V1JZV1_TRIPS\nEGFR\tA0A0V1LSX2_9BILA\nEGFR\tA0A0V1LSX9_9BILA\nEGFR\tA0A0V1LU32_9BILA\nEGFR\tA0A0V1P0D6_9BILA\nEGFR\tA0A0V1P0D7_9BILA\nEGFR\tA0A0V1P0G9_9BILA\nEGFR\tA0A0V1P0L7_9BILA\nEGFR\tA0A0V1P0N3_9BILA\nEGFR\tA0A0V1P0R0_9BILA\nEGFR\tA0A0X3Q0Y8_SCHSO\nEGFR\tA0A1A7WP18_9TELE\nEGFR\tA0A1A7ZC18_NOTFU\nEGFR\tA0A1A8CPT5_9TELE\nEGFR\tA0A1A8EA64_9TELE\nEGFR\tA0A1A8EMY8_9TELE\nEGFR\tA0A1A8G3U8_9TELE\nEGFR\tA0A1A8GHE0_9TELE\nEGFR\tA0A1A8HIW0_9TELE\nEGFR\tA0A1A8HXZ2_NOTKU\nEGFR\tA0A1A8NUQ6_9TELE\nEGFR\tA0A1A8PJN8_9TELE\nEGFR\tA0A1D5NZB4_CHICK\nEGFR\tA0A1D5PKH0_CHICK\nEGFR\tA0A1D5QDK1_MACMU\nEGFR\tA0A1D5QLH2_MACMU\nEGFR\tA0A1D5RE09_MACMU\nEGFR\tA0A1L7RNH1_BLAGE\nEGFR\tA0A1L8FRS8_XENLA\nEGFR\tA0A1L8FWP8_XENLA\nEGFR\tA0A1Q2SJP9_9SCAR\nEGFR\tA0A1S3AHV3_ERIEU\nEGFR\tA0A1S3GRI1_DIPOR\nEGFR\tA0A1U7QE91_MESAU\nEGFR\tA0A1U7U194_TARSY\nEGFR\tA0A1U8BSJ9_MESAU\nEGFR\tA0A1V4JV60_PATFA\nEGFR\tA0A1V4JVA3_PATFA\nEGFR\tA0A1W2PRR9_HUMAN\nEGFR\tA0A1Y0DMK7_HUMAN\nEGFR\tA0A218UPF3_9PASE\nEGFR\tA0A286ZV23_PIG\nEGFR\tA0A2B4RP77_STYPI\nEGFR\tA0A2D0T8L6_ICTPU\nEGFR\tA0A2I0M4T2_COLLI\nEGFR\tA0A2I3GG42_NOMLE\nEGFR\tA0A2I3GK86_NOMLE\nEGFR\tA0A2I3GL38_NOMLE\nEGFR\tA0A2I3GX32_NOMLE\nEGFR\tA0A2I3HF60_NOMLE\nEGFR\tA0A2I3ML37_PAPAN\nEGFR\tA0A2I3MLV6_PAPAN\nEGFR\tA0A2I3N177_PAPAN\nEGFR\tA0A2I3N2U0_PAPAN\nEGFR\tA0A2I3RN53_PANTR\nEGFR\tA0A2I3RYF9_PANTR\nEGFR\tA0A2I3SN01_PANTR\nEGFR\tA0A2I3T5Z5_PANTR\nEGFR\tA0A2I3T899_PANTR\nEGFR\tA0A2I3TRV3_PANTR\nEGFR\tA0A2I4CXD8_9TELE\nEGFR\tA0A2K5EV34_AOTNA\nEGFR\tA0A2K5EV42_AOTNA\nEGFR\tA0A2K5EV45_AOTNA\nEGFR\tA0A2K5IMZ3_COLAP\nEGFR\tA0A2K5IN68_COLAP\nEGFR\tA0A2K5INH6_COLAP\nEGFR\tA0A2K5INW9_COLAP\nEGFR\tA0A2K5INZ0_COLAP\nEGFR\tA0A2K5INZ5_COLAP\nEGFR\tA0A2K5N7J8_CERAT\nEGFR\tA0A2K5N7M7_CERAT\nEGFR\tA0A2K5N7N5_CERAT\nEGFR\tA0A2K5N7P0_CERAT\nEGFR\tA0A2K5N7U3_CERAT\nEGFR\tA0A2K5Q8X8_CEBCA\nEGFR\tA0A2K5Q8Y2_CEBCA\nEGFR\tA0A2K5Q906_CEBCA\nEGFR\tA0A2K5Q956_CEBCA\nEGFR\tA0A2K5WK39_MACFA\nEGFR\tA0A2K5WK74_MACFA\nEGFR\tA0A2K5WKB8_MACFA\nEGFR\tA0A2K5WKC1_MACFA\nEGFR\tA0A2K5WKD8_MACFA\nEGFR\tA0A2K5WKE2_MACFA\nEGFR\tA0A2K5WKK2_MACFA\nEGFR\tA0A2K5Z884_MANLE\nEGFR\tA0A2K5Z8B7_MANLE\nEGFR\tA0A2K5Z8C2_MANLE\nEGFR\tA0A2K5Z8D4_MANLE\nEGFR\tA0A2K5Z8L4_MANLE\nEGFR\tA0A2K6B1K2_MACNE\nEGFR\tA0A2K6B1P0_MACNE\nEGFR\tA0A2K6B1P2_MACNE\nEGFR\tA0A2K6B1R0_MACNE\nEGFR\tA0A2K6B1R5_MACNE\nEGFR\tA0A2K6B1R9_MACNE\nEGFR\tA0A2K6FWF3_PROCO\nEGFR\tA0A2K6FWF5_PROCO\nEGFR\tA0A2K6FWG5_PROCO\nEGFR\tA0A2K6FWI4_PROCO\nEGFR\tA0A2K6MNK3_RHIBE\nEGFR\tA0A2K6MNL0_RHIBE\nEGFR\tA0A2K6MNL6_RHIBE\nEGFR\tA0A2K6MNL7_RHIBE\nEGFR\tA0A2K6MNN4_RHIBE\nEGFR\tA0A2K6MNS1_RHIBE\nEGFR\tA0A2K6MNV0_RHIBE\nEGFR\tA0A2K6R5V6_RHIRO\nEGFR\tA0A2K6R6K2_RHIRO\nEGFR\tA0A2K6R6K3_RHIRO\nEGFR\tA0A2K6R6K4_RHIRO\nEGFR\tA0A2K6R6K5_RHIRO\nEGFR\tA0A2K6R6L6_RHIRO\nEGFR\tA0A2K6R6M1_RHIRO\nEGFR\tA0A2K6R6M7_RHIRO\nEGFR\tA0A2K6R6M8_RHIRO\nEGFR\tA0A2K6TT16_SAIBB\nEGFR\tA0A2K6TT37_SAIBB\nEGFR\tA0A2K6TT47_SAIBB\nEGFR\tA0A2K6TT55_SAIBB\nEGFR\tA0A2K6TTC6_SAIBB\nEGFR\tA0A2P8YVW7_BLAGE\nEGFR\tA0A2R9CME7_PANPA\nEGFR\tA0A2R9CMF2_PANPA\nEGFR\tA0A2R9CMH6_PANPA\nEGFR\tA0A2R9CMH7_PANPA\nEGFR\tA0A2R9CMI1_PANPA\nEGFR\tA0A2R9CP96_PANPA\nEGFR\tA0A2R9CTL6_PANPA\nEGFR\tA0A2U3X2H8_ODORO\nEGFR\tA0A2U3ZXZ4_ODORO\nEGFR\tA0A2U4BQ29_TURTR\nEGFR\tA0A2Y9F1C6_PHYMC\nEGFR\tA0A2Y9I608_NEOSC\nEGFR\tA0A2Y9I6V3_NEOSC\nEGFR\tA0A2Y9PPT8_DELLE\nEGFR\tA0A2Y9PTK8_DELLE\nEGFR\tA0A2Y9Q0F2_DELLE\nEGFR\tA0A2Y9S4T3_PHYMC\nEGFR\tA0A330J061_GLOMR\nEGFR\tA0A337S424_FELCA\nEGFR\tA0A337S7U7_FELCA\nEGFR\tA0A340XQY3_LIPVE\nEGFR\tA0A340XUB4_LIPVE\nEGFR\tA0A341CZE6_NEOAA\nEGFR\tA0A341D235_NEOAA\nEGFR\tA0A384AK28_BALAS\nEGFR\tA0A384AK81_BALAS\nEGFR\tA0A384AKI5_BALAS\nEGFR\tA0A384D413_URSMA\nEGFR\tA0A3G1S231_SINCO\nEGFR\tA0A3G1S238_SINCO\nEGFR\tA0A3P9P0M2_POERE\nEGFR\tA0A3Q0CK49_MESAU\nEGFR\tA0A3Q0GES5_ALLSI\nEGFR\tA0A3Q0GIC6_ALLSI\nEGFR\tA0A3Q1M731_BOVIN\nEGFR\tA0A3Q1MHB0_BOVIN\nEGFR\tA0A3Q2I5Q7_HORSE\nEGFR\tA0A3Q2L870_HORSE\nEGFR\tA0A3Q7MPB6_CALUR\nEGFR\tA0A3Q7MPC8_CALUR\nEGFR\tA0A3Q7P017_CALUR\nEGFR\tA0A3Q7SKW1_VULVU\nEGFR\tA0A3Q7VXF8_URSAR\nEGFR\tA0A3Q7VXG1_URSAR\nEGFR\tA0A419PU83_CLOSI\nEGFR\tA0A452FEC6_CAPHI\nEGFR\tA0A452FED4_CAPHI\nEGFR\tA0A452FEK3_CAPHI\nEGFR\tA0A452FET4_CAPHI\nEGFR\tA0A452TUH9_URSMA\nEGFR\tA0A452TUJ8_URSMA\nEGFR\tA0A452TV51_URSMA\nEGFR\tA0A452TVB9_URSMA\nEGFR\tA0A452TVE7_URSMA\nEGFR\tA0A452TVF5_URSMA\nEGFR\tA0A452TVJ1_URSMA\nEGFR\tA0A452TVU4_URSMA\nEGFR\tA0A452TVY8_URSMA\nEGFR\tA0A452TVZ4_URSMA\nEGFR\tA0A455BBC9_PHYMC\nEGFR\tA0A455BDS0_PHYMC\nEGFR\tA0A481NXK1_CTEID\nEGFR\tA0A493TAW2_ANAPP\nEGFR\tA0A4C1SUQ0_9NEOP\nEGFR\tA0A4C1SWH8_9NEOP\nEGFR\tA0A4W2BT12_BOBOX\nEGFR\tA0A4W2CA87_BOBOX\nEGFR\tA0A4W2G1Z4_BOBOX\nEGFR\tA0A4W2G676_BOBOX\nEGFR\tA0A4W2G697_BOBOX\nEGFR\tA0A4W6BYA2_LATCA\nEGFR\tA0A4W6BYU7_LATCA\nEGFR\tA0A4W6BYW2_LATCA\nEGFR\tA0A4W6BYW8_LATCA\nEGFR\tA0A4W6C3F8_LATCA\nEGFR\tA0A4W6C3J7_LATCA\nEGFR\tA0A4W6C3Y0_LATCA\nEGFR\tA0A4W6C400_LATCA\nEGFR\tA0A4W6C5J2_LATCA\nEGFR\tA0A4W6C778_LATCA\nEGFR\tA0A4X1T7H2_PIG\nEGFR\tA0A4X1T808_PIG\nEGFR\tA0A4X1T853_PIG\nEGFR\tA0A4X1T8M2_PIG\nEGFR\tA0A4X2L871_VOMUR\nEGFR\tA0A4X2LA42_VOMUR\nEGFR\tA0A4Z2HPH1_9TELE\nEGFR\tA0A5B7ELR7_PORTR\nEGFR\tA0A5F4D104_CANLF\nEGFR\tA0A5F4WAF1_CALJA\nEGFR\tA0A5F8HG27_MONDO\nEGFR\tA0A5F9C1H3_RABIT\nEGFR\tA0A5G2QBP1_PIG\nEGFR\tA0A5G2QTG3_PIG\nEGFR\tA0A5G2R2I8_PIG\nEGFR\tA0A5G2RCJ6_PIG\nEGFR\tA0A5G3HM67_XENTR\nEGFR\tA0A5G3HP24_XENTR\nEGFR\tA0A5G3JC20_XENTR\nEGFR\tA0A5G3JEC1_XENTR\nEGFR\tA0A5G3JRJ2_XENTR\nEGFR\tA2VCQ7_HUMAN\nEGFR\tA5A753_PIG\nEGFR\tA7VMV4_HUMAN\nEGFR\tA7VN05_HUMAN\nEGFR\tA7VN06_HUMAN\nEGFR\tA7VN07_HUMAN\nEGFR\tA8IP97_RAT\nEGFR\tB2KKR3_LYMST\nEGFR\tB3DK54_DANRE\nEGFR\tB3XXJ8_GRYBI\nEGFR\tB5KLP6_CAPHI\nEGFR\tB5KLP7_CAPHI\nEGFR\tB6EYY0_MACFA\nEGFR\tB6EZ04_MACMU\nEGFR\tB6RC65_HUMAN\nEGFR\tB6RC66_HUMAN\nEGFR\tB6VPZ6_ONCMY\nEGFR\tB6VPZ7_CYPCA\nEGFR\tB6VPZ8_CYPCA\nEGFR\tB7ZDJ1_CUPSA\nEGFR\tC0KZ13_TRICA\nEGFR\tC7TYH4_SCHJA\nEGFR\tC9JYS6_HUMAN\nEGFR\tE0AE18_BOVIN\nEGFR\tE2I6F5_HUMAN\nEGFR\tE2JE31_FELCA\nEGFR\tE7BSV0_HUMAN\nEGFR\tE7CXR8_RAT\nEGFR\tE7CXR9_RAT\nEGFR\tE9PFD7_HUMAN\nEGFR\tEGFR_APIME\nEGFR\tEGFR_CHICK\nEGFR\tEGFR_DROME\nEGFR\tEGFR_HUMAN\nEGFR\tEGFR_MACMU\nEGFR\tEGFR_MOUSE\nEGFR\tF1JTL6_HUMAN\nEGFR\tF1N7F3_BOVIN\nEGFR\tF1PF03_CANLF\nEGFR\tF6T945_ORNAN\nEGFR\tF6TBS4_XENTR\nEGFR\tF6UFM4_XENTR\nEGFR\tF6X5R9_XENTR\nEGFR\tF6XRP2_CALJA\nEGFR\tF6XRZ8_CALJA\nEGFR\tF6XYW7_XENTR\nEGFR\tF7B2I8_MONDO\nEGFR\tF7BPU8_XENTR\nEGFR\tF7BRE7_XENTR\nEGFR\tF7C8B9_XENTR\nEGFR\tF7CBE6_CALJA\nEGFR\tG1KTY1_ANOCA\nEGFR\tG1NIB9_MELGA\nEGFR\tG1RB22_NOMLE\nEGFR\tG1SZE4_RABIT\nEGFR\tG3TD59_LOXAF\nEGFR\tG3U4U1_LOXAF\nEGFR\tG3UNV2_MELGA\nEGFR\tG3UQM0_MELGA\nEGFR\tG3V6K6_RAT\nEGFR\tG3W2J0_SARHA\nEGFR\tG9MC26_HUMAN\nEGFR\tG9MC81_HUMAN\nEGFR\tH0UT18_CAVPO\nEGFR\tH0WZV6_OTOGA\nEGFR\tH0YU95_TAEGU\nEGFR\tH2MTN5_ORYLA\nEGFR\tH2MTN7_ORYLA\nEGFR\tH2PM35_PONAB\nEGFR\tH2QUL1_PANTR\nEGFR\tH2TYZ8_TAKRU\nEGFR\tH2TYZ9_TAKRU\nEGFR\tH9F140_MACMU\nEGFR\tH9FJ74_MACMU\nEGFR\tH9FK76_MACMU\nEGFR\tI3LMY0_PIG\nEGFR\tI3M8G8_ICTTR\nEGFR\tI3WA68_HUMAN\nEGFR\tI3WA72_HUMAN\nEGFR\tI3WA73_HUMAN\nEGFR\tI6SZ00_HALDV\nEGFR\tK7F9S9_PELSI\nEGFR\tK7F9T7_PELSI\nEGFR\tL0R6G1_HUMAN\nEGFR\tM3WMN1_FELCA\nEGFR\tM3Z380_MUSPF\nEGFR\tN1IRL3_MERUN\nEGFR\tQ147T7_HUMAN\nEGFR\tQ153D7_XENLA\nEGFR\tQ1RQH0_HORSE\nEGFR\tQ1RQH1_HORSE\nEGFR\tQ20C69_DROVI\nEGFR\tQ30BI0_HUMAN\nEGFR\tQ30DU1_SHEEP\nEGFR\tQ3TQS6_MOUSE\nEGFR\tQ4H1F3_HUMAN\nEGFR\tQ504U8_HUMAN\nEGFR\tQ5SVE7_MOUSE\nEGFR\tQ68GS6_HUMAN\nEGFR\tQ69AY2_DROSI\nEGFR\tQ69AY3_DROSI\nEGFR\tQ69AY4_DROSI\nEGFR\tQ69AY5_DROSE\nEGFR\tQ6LEH5_DROME\nEGFR\tQ6QMR6_DANRE\nEGFR\tQ6QMR7_DANRE\nEGFR\tQ6QMR8_DANRE\nEGFR\tQ6QT04_CANLF\nEGFR\tQ6RXL6_BOVIN\nEGFR\tQ6SA99_DROME\nEGFR\tQ6SAA0_DROME\nEGFR\tQ6SAA1_DROME\nEGFR\tQ6SAA2_DROME\nEGFR\tQ6SAA3_DROME\nEGFR\tQ6SAA4_DROME\nEGFR\tQ6SAA5_DROME\nEGFR\tQ6SAA7_DROME\nEGFR\tQ6SAA8_DROME\nEGFR\tQ6SAA9_DROME\nEGFR\tQ6SAB0_DROME\nEGFR\tQ6SAB3_DROME\nEGFR\tQ6SAB4_DROME\nEGFR\tQ6SAB5_DROME\nEGFR\tQ6SAB6_DROME\nEGFR\tQ6SAB8_DROME\nEGFR\tQ6SAB9_DROME\nEGFR\tQ6SAC0_DROME\nEGFR\tQ6SAC1_DROME\nEGFR\tQ6SAC2_DROME\nEGFR\tQ6SAC4_DROME\nEGFR\tQ6SAC6_DROME\nEGFR\tQ6SAC7_DROME\nEGFR\tQ6SAC8_DROME\nEGFR\tQ6SAC9_DROME\nEGFR\tQ6SAD0_DROME\nEGFR\tQ6SAD1_DROME\nEGFR\tQ6SAD3_DROME\nEGFR\tQ6SAD5_DROME\nEGFR\tQ6SAD7_DROME\nEGFR\tQ6SAD8_DROME\nEGFR\tQ6SAD9_DROME\nEGFR\tQ6SAE0_DROME\nEGFR\tQ6SAE1_DROME\nEGFR\tQ6SAE2_DROME\nEGFR\tQ6SAE3_DROME\nEGFR\tQ6SAE4_DROME\nEGFR\tQ6SAE6_DROME\nEGFR\tQ6SAE8_DROME\nEGFR\tQ6SAE9_DROME\nEGFR\tQ6SAF1_DROME\nEGFR\tQ6SAF2_DROME\nEGFR\tQ6SAF4_DROME\nEGFR\tQ6SAF5_DROME\nEGFR\tQ6SAF6_DROME\nEGFR\tQ6SAF7_DROME\nEGFR\tQ6SAG0_DROME\nEGFR\tQ6SAG1_DROME\nEGFR\tQ6SAG2_DROME\nEGFR\tQ6SAG3_DROME\nEGFR\tQ6SAG5_DROME\nEGFR\tQ6SAG6_DROME\nEGFR\tQ6SAG7_DROME\nEGFR\tQ6SAG8_DROME\nEGFR\tQ6SAG9_DROME\nEGFR\tQ6SAH0_DROME\nEGFR\tQ6SAH1_DROME\nEGFR\tQ6SAH2_DROME\nEGFR\tQ6SAH3_DROME\nEGFR\tQ6SAH4_DROME\nEGFR\tQ6SAH5_DROME\nEGFR\tQ6SAH6_DROME\nEGFR\tQ6SAH7_DROME\nEGFR\tQ6SAH8_DROME\nEGFR\tQ6SAH9_DROME\nEGFR\tQ6SAI0_DROME\nEGFR\tQ6SAI2_DROME\nEGFR\tQ6SAI4_DROME\nEGFR\tQ6SAI5_DROME\nEGFR\tQ6SAI6_DROME\nEGFR\tQ6SAI7_DROME\nEGFR\tQ6SAI8_DROME\nEGFR\tQ6SAI9_DROME\nEGFR\tQ6SAJ0_DROME\nEGFR\tQ6SAJ1_DROME\nEGFR\tQ6SAJ2_DROME\nEGFR\tQ6SAJ3_DROME\nEGFR\tQ6SAJ5_DROME\nEGFR\tQ6SAJ6_DROME\nEGFR\tQ6SAJ7_DROME\nEGFR\tQ6SAJ8_DROME\nEGFR\tQ6SAJ9_DROME\nEGFR\tQ6SAK0_DROME\nEGFR\tQ6SAK1_DROME\nEGFR\tQ6SAK2_DROME\nEGFR\tQ6SAK3_DROME\nEGFR\tQ6SAK4_DROME\nEGFR\tQ6SAK5_DROME\nEGFR\tQ6SAK6_DROME\nEGFR\tQ6SAK7_DROME\nEGFR\tQ6SAK8_DROME\nEGFR\tQ6SAK9_DROME\nEGFR\tQ6SAL0_DROME\nEGFR\tQ6SAL2_DROME\nEGFR\tQ6SAL3_DROME\nEGFR\tQ6SAL4_DROME\nEGFR\tQ6SAL5_DROME\nEGFR\tQ6SAL6_DROME\nEGFR\tQ6SAL7_DROME\nEGFR\tQ6SAL8_DROME\nEGFR\tQ6SAL9_DROME\nEGFR\tQ6SAM0_DROME\nEGFR\tQ6SAM1_DROME\nEGFR\tQ6SAM2_DROME\nEGFR\tQ6SAM3_DROME\nEGFR\tQ6SAM5_DROME\nEGFR\tQ6SAM6_DROME\nEGFR\tQ6SAM7_DROME\nEGFR\tQ6SAM9_DROME\nEGFR\tQ6SAN0_DROME\nEGFR\tQ6SAN1_DROME\nEGFR\tQ6SAN2_DROME\nEGFR\tQ6SAN6_DROME\nEGFR\tQ6SAN7_DROME\nEGFR\tQ6SAN8_DROME\nEGFR\tQ6SAP0_DROME\nEGFR\tQ6SAP3_DROME\nEGFR\tQ6SAP4_DROME\nEGFR\tQ6SAP7_DROME\nEGFR\tQ6SAQ3_DROME\nEGFR\tQ6SAQ5_DROME\nEGFR\tQ6SAQ7_DROME\nEGFR\tQ6SAQ8_DROME\nEGFR\tQ6SAQ9_DROME\nEGFR\tQ6SAR2_DROME\nEGFR\tQ6SAR4_DROME\nEGFR\tQ6SAR7_DROME\nEGFR\tQ6SAR9_DROME\nEGFR\tQ6SAS0_DROME\nEGFR\tQ6SAS1_DROME\nEGFR\tQ6SAS2_DROME\nEGFR\tQ6SAS3_DROME\nEGFR\tQ6SAS4_DROME\nEGFR\tQ6SAS5_DROME\nEGFR\tQ6SAS6_DROME\nEGFR\tQ6SAS7_DROME\nEGFR\tQ6SAS8_DROME\nEGFR\tQ6SAS9_DROME\nEGFR\tQ6SAT0_DROME\nEGFR\tQ6SAT1_DROME\nEGFR\tQ6SAT2_DROME\nEGFR\tQ6SAT3_DROME\nEGFR\tQ6SAT4_DROME\nEGFR\tQ6SAT5_DROME\nEGFR\tQ6SAT6_DROME\nEGFR\tQ6SAT7_DROME\nEGFR\tQ6SAT8_DROME\nEGFR\tQ6SAT9_DROME\nEGFR\tQ6SAU0_DROME\nEGFR\tQ6SAU1_DROME\nEGFR\tQ6SAU2_DROME\nEGFR\tQ6SAU3_DROME\nEGFR\tQ6SAU4_DROME\nEGFR\tQ6SAU5_DROME\nEGFR\tQ6SAU6_DROME\nEGFR\tQ6SAU7_DROME\nEGFR\tQ6SAU8_DROME\nEGFR\tQ6SAU9_DROME\nEGFR\tQ6SAV0_DROME\nEGFR\tQ6SAV1_DROME\nEGFR\tQ6SAV2_DROME\nEGFR\tQ6SAV3_DROME\nEGFR\tQ6SAV4_DROME\nEGFR\tQ6SAV5_DROME\nEGFR\tQ6SAV6_DROME\nEGFR\tQ6SAV7_DROME\nEGFR\tQ6SAV8_DROME\nEGFR\tQ6SAV9_DROME\nEGFR\tQ6SAW0_DROME\nEGFR\tQ6SAW1_DROME\nEGFR\tQ6SAW2_DROME\nEGFR\tQ6SAW3_DROME\nEGFR\tQ6SAW4_DROME\nEGFR\tQ6SAW5_DROME\nEGFR\tQ6SAW6_DROME\nEGFR\tQ6SAW7_DROME\nEGFR\tQ6SAW8_DROME\nEGFR\tQ6SAW9_DROME\nEGFR\tQ6SAX0_DROME\nEGFR\tQ6SAX1_DROME\nEGFR\tQ6SAX2_DROME\nEGFR\tQ6SAX3_DROME\nEGFR\tQ6SAX4_DROME\nEGFR\tQ6SAX5_DROME\nEGFR\tQ6SAX6_DROME\nEGFR\tQ6SAX7_DROME\nEGFR\tQ6SAX8_DROME\nEGFR\tQ6SAY0_DROME\nEGFR\tQ6SAY1_DROME\nEGFR\tQ6SAY2_DROME\nEGFR\tQ6SAY3_DROME\nEGFR\tQ6SAY4_DROME\nEGFR\tQ6SAY6_DROME\nEGFR\tQ6SAY7_DROME\nEGFR\tQ6SAY8_DROME\nEGFR\tQ6SAY9_DROME\nEGFR\tQ6SAZ0_DROME\nEGFR\tQ6SAZ1_DROME\nEGFR\tQ6SAZ2_DROME\nEGFR\tQ6SAZ3_DROME\nEGFR\tQ6SAZ4_DROME\nEGFR\tQ6SAZ5_DROME\nEGFR\tQ6SAZ6_DROME\nEGFR\tQ6SAZ7_DROME\nEGFR\tQ6SAZ8_DROME\nEGFR\tQ6SAZ9_DROME\nEGFR\tQ6SB00_DROME\nEGFR\tQ6SB01_DROME\nEGFR\tQ6SB03_DROME\nEGFR\tQ6SB04_DROME\nEGFR\tQ6SB05_DROME\nEGFR\tQ6SB11_DROME\nEGFR\tQ6SB14_DROME\nEGFR\tQ6SB15_DROME\nEGFR\tQ6SB25_DROME\nEGFR\tQ6SB26_DROME\nEGFR\tQ6SB28_DROME\nEGFR\tQ6SB30_DROME\nEGFR\tQ6SB48_DROME\nEGFR\tQ6SB52_DROME\nEGFR\tQ6SB53_DROME\nEGFR\tQ6SB62_DROME\nEGFR\tQ6SB67_DROME\nEGFR\tQ6SB80_DROME\nEGFR\tQ6SBA8_DROME\nEGFR\tQ6SBB1_DROME\nEGFR\tQ6SBB3_DROME\nEGFR\tQ6SBB6_DROME\nEGFR\tQ6SBC0_DROME\nEGFR\tQ6SBC1_DROME\nEGFR\tQ6SBC2_DROME\nEGFR\tQ6SBC3_DROME\nEGFR\tQ6SBC4_DROME\nEGFR\tQ6SBD2_DROME\nEGFR\tQ6SBD4_DROME\nEGFR\tQ6SBE0_DROME\nEGFR\tQ6SBE1_DROME\nEGFR\tQ6SBE5_DROME\nEGFR\tQ6SBE7_DROME\nEGFR\tQ6SBE9_DROME\nEGFR\tQ6SBF1_DROME\nEGFR\tQ6SBF4_DROME\nEGFR\tQ6SBF5_DROME\nEGFR\tQ6SBF7_DROME\nEGFR\tQ6SBF8_DROME\nEGFR\tQ6SBG5_DROME\nEGFR\tQ6SBH9_DROME\nEGFR\tQ6SBJ6_DROME\nEGFR\tQ6SBU8_DROME\nEGFR\tQ6SBV4_DROME\nEGFR\tQ6SBV5_DROME\nEGFR\tQ6SBW4_DROME\nEGFR\tQ6SBX2_DROME\nEGFR\tQ6SBY5_DROME\nEGFR\tQ6SBY6_DROME\nEGFR\tQ6SBZ9_DROME\nEGFR\tQ6SC05_DROME\nEGFR\tQ6SC18_DROME\nEGFR\tQ6SC19_DROME\nEGFR\tQ6SC32_DROME\nEGFR\tQ6SC35_DROME\nEGFR\tQ6SC37_DROME\nEGFR\tQ6VQA3_DANRE\nEGFR\tQ75MF2_HUMAN\nEGFR\tQ7PPN5_ANOGA\nEGFR\tQ7SZF7_DANRE\nEGFR\tQ86NZ2_DROME\nEGFR\tQ8HY60_PIG\nEGFR\tQ8MLW0_DROME\nEGFR\tQ8NDU8_HUMAN\nEGFR\tQ8SZW1_DROME\nEGFR\tQ8WZA5_HUMAN\nEGFR\tQ99PK1_MOUSE\nEGFR\tQ9BIH9_ANOGA\nEGFR\tQ9EP98_MOUSE\nEGFR\tQ9ERV6_MOUSE\nEGFR\tQ9H3C8_HUMAN\nEGFR\tQ9H3D0_HUMAN\nEGFR\tQ9QX70_RAT\nEGFR\tQ9WTS1_RAT\nEGFR\tQ9WVF5_MOUSE\nEGFR\tU3CYI9_CALJA\nEGFR\tU3CZW4_CALJA\nEGFR\tU3I5P3_ANAPP\nEGFR\tU3JQX4_FICAL\nEGFR\tU6DJG6_NEOVI\nEGFR\tV5I974_ANOGL\nEGFR\tV5I995_ANOGL\nEGFR\tV8NFR6_OPHHA\nEGFR\tW5PWC5_SHEEP\nEGFR\tW8ADF8_CERCA\nEGFR\tW8B8G8_CERCA\n','From\tTo\nGSG2\tA0A0A9Y4N7_LYGHE\nGSG2\tA0A0B2VHR2_TOXCA\nGSG2\tA0A0B2VW41_TOXCA\nGSG2\tA0A0B7G5U2_THACB\nGSG2\tA0A0R4J0P2_MOUSE\nGSG2\tA0A0V0S6A7_9BILA\nGSG2\tA0A0V0YHV2_TRIPS\nGSG2\tA0A0V1EMM4_TRIPS\nGSG2\tA0A0V1EMU5_TRIPS\nGSG2\tA0A0V1G5N1_TRIPS\nGSG2\tA0A0V1I638_9BILA\nGSG2\tA0A0V1J7Z4_TRIPS\nGSG2\tA0A0V1K857_TRIPS\nGSG2\tA0A0V1N2C3_9BILA\nGSG2\tA0A0V1NKN6_9BILA\nGSG2\tA0A1A8A3A6_NOTFU\nGSG2\tA0A1A8CLQ8_9TELE\nGSG2\tA0A1A8CNS3_9TELE\nGSG2\tA0A1A8F9A8_9TELE\nGSG2\tA0A1A8G1R0_9TELE\nGSG2\tA0A1A8HUH3_NOTKU\nGSG2\tA0A1A8IQJ9_NOTKU\nGSG2\tA0A1A8KML7_NOTKU\nGSG2\tA0A1A8L943_9TELE\nGSG2\tA0A1A8ME20_9TELE\nGSG2\tA0A1A8N2G3_9TELE\nGSG2\tA0A1A8Q431_9TELE\nGSG2\tA0A1A8RTX3_9TELE\nGSG2\tA0A1A8SAD9_9TELE\nGSG2\tA0A1A8U2T4_NOTFU\nGSG2\tA0A1C7MID3_GRIFR\nGSG2\tA0A1S2ZL66_ERIEU\nGSG2\tA0A1S3EX76_DIPOR\nGSG2\tA0A1V4K261_PATFA\nGSG2\tA0A2B4RP98_STYPI\nGSG2\tA0A2D0QGZ4_ICTPU\nGSG2\tA0A2D0QHU0_ICTPU\nGSG2\tA0A2I4BLQ9_9TELE\nGSG2\tA0A2I4BLR1_9TELE\nGSG2\tA0A2N6NPQ7_BEABA\nGSG2\tA0A2U3X1R7_ODORO\nGSG2\tA0A2U3YRU7_LEPWE\nGSG2\tA0A2U4AN91_TURTR\nGSG2\tA0A3G2S3U3_9BASI\nGSG2\tA0A419Q5I6_CLOSI\nGSG2\tA0A419QI05_CLOSI\nGSG2\tA0A4D9ABU0_SALSN\nGSG2\tA0A4D9C115_SALSN\nGSG2\tA0A4Z2HUV7_9TELE\nGSG2\tA0PJ70_HUMAN\nGSG2\tE5SGF2_TRISP\nGSG2\tHASP_BOVIN\nGSG2\tHASP_HUMAN\nGSG2\tHASP_MOUSE\nGSG2\tM0RBT0_RAT\nGSG2\tQ2KHL1_MOUSE\nGSG2\tQ3UU60_MOUSE\nGSG2\tQ7JMV5_ANOGA\nGSG2\tT2M873_HYDVU\n','From\tTo\nEIF2AK2\tA0A096NH56_PAPAN\nEIF2AK2\tA0A0D9RI08_CHLSB\nEIF2AK2\tA0A0P6K2W0_HETGA\nEIF2AK2\tA0A0R4IHF6_DANRE\nEIF2AK2\tA0A0U2UDT4_SHEEP\nEIF2AK2\tA0A0U2UZZ5_CRIGR\nEIF2AK2\tA0A151MHB2_ALLMI\nEIF2AK2\tA0A1A7WPS0_9TELE\nEIF2AK2\tA0A1A7Y0J0_9TELE\nEIF2AK2\tA0A1A7YIF1_9TELE\nEIF2AK2\tA0A1A7ZJ65_NOTFU\nEIF2AK2\tA0A1A7ZUD1_NOTFU\nEIF2AK2\tA0A1A8CBG3_9TELE\nEIF2AK2\tA0A1A8CV57_9TELE\nEIF2AK2\tA0A1A8DQJ3_9TELE\nEIF2AK2\tA0A1A8ENT7_9TELE\nEIF2AK2\tA0A1A8FD92_9TELE\nEIF2AK2\tA0A1A8H4S4_9TELE\nEIF2AK2\tA0A1A8HZF6_NOTKU\nEIF2AK2\tA0A1A8I5H7_NOTKU\nEIF2AK2\tA0A1A8JPV7_NOTKU\nEIF2AK2\tA0A1A8K7A8_NOTKU\nEIF2AK2\tA0A1A8L677_9TELE\nEIF2AK2\tA0A1A8Q1Z7_9TELE\nEIF2AK2\tA0A1A8QYC5_9TELE\nEIF2AK2\tA0A1A8U8B1_NOTFU\nEIF2AK2\tA0A1A8UDD9_NOTFU\nEIF2AK2\tA0A1D5QPP2_MACMU\nEIF2AK2\tA0A1Q9DTA4_SYMMI\nEIF2AK2\tA0A1S3EXJ5_DIPOR\nEIF2AK2\tA0A1S3F081_DIPOR\nEIF2AK2\tA0A1S3ND83_SALSA\nEIF2AK2\tA0A1S3NDP8_SALSA\nEIF2AK2\tA0A1S3W8T7_ERIEU\nEIF2AK2\tA0A1U7U4Z5_TARSY\nEIF2AK2\tA0A1U7UB40_TARSY\nEIF2AK2\tA0A1U8CXQ8_ALLSI\nEIF2AK2\tA0A1V4JK69_PATFA\nEIF2AK2\tA0A218URU6_9PASE\nEIF2AK2\tA0A250Y2K9_CASCN\nEIF2AK2\tA0A287ABT2_PIG\nEIF2AK2\tA0A287BKF1_PIG\nEIF2AK2\tA0A2D0QLB0_ICTPU\nEIF2AK2\tA0A2D0QLL2_ICTPU\nEIF2AK2\tA0A2D0QPA9_ICTPU\nEIF2AK2\tA0A2I0LY98_COLLI\nEIF2AK2\tA0A2I2V073_FELCA\nEIF2AK2\tA0A2I3HGY3_NOMLE\nEIF2AK2\tA0A2I3N054_PAPAN\nEIF2AK2\tA0A2I3TQZ8_PANTR\nEIF2AK2\tA0A2I4B0V4_9TELE\nEIF2AK2\tA0A2I4B0W6_9TELE\nEIF2AK2\tA0A2K5EQW3_AOTNA\nEIF2AK2\tA0A2K5EQX4_AOTNA\nEIF2AK2\tA0A2K5EQZ4_AOTNA\nEIF2AK2\tA0A2K5MN56_CERAT\nEIF2AK2\tA0A2K5MN76_CERAT\nEIF2AK2\tA0A2K5MNI1_CERAT\nEIF2AK2\tA0A2K5SI06_CEBCA\nEIF2AK2\tA0A2K5SI09_CEBCA\nEIF2AK2\tA0A2K5UQW6_MACFA\nEIF2AK2\tA0A2K5UR00_MACFA\nEIF2AK2\tA0A2K5YSY9_MANLE\nEIF2AK2\tA0A2K5YT04_MANLE\nEIF2AK2\tA0A2K5YT19_MANLE\nEIF2AK2\tA0A2K6DCT1_MACNE\nEIF2AK2\tA0A2K6DCU6_MACNE\nEIF2AK2\tA0A2K6EWE4_PROCO\nEIF2AK2\tA0A2K6EWH4_PROCO\nEIF2AK2\tA0A2K6LBX3_RHIBE\nEIF2AK2\tA0A2K6LBY5_RHIBE\nEIF2AK2\tA0A2K6QW90_RHIRO\nEIF2AK2\tA0A2K6QW92_RHIRO\nEIF2AK2\tA0A2K6QW94_RHIRO\nEIF2AK2\tA0A2K6TKV6_SAIBB\nEIF2AK2\tA0A2K6TKW9_SAIBB\nEIF2AK2\tA0A2S1M5M7_RANCL\nEIF2AK2\tA0A2U3WM64_ODORO\nEIF2AK2\tA0A2U3Y1D2_LEPWE\nEIF2AK2\tA0A2U4BZL1_TURTR\nEIF2AK2\tA0A2U4BZL3_TURTR\nEIF2AK2\tA0A2U4BZL8_TURTR\nEIF2AK2\tA0A2U4BZQ7_TURTR\nEIF2AK2\tA0A2U4BZT9_TURTR\nEIF2AK2\tA0A2Y9HN76_NEOSC\nEIF2AK2\tA0A2Y9NA31_DELLE\nEIF2AK2\tA0A2Y9NE94_DELLE\nEIF2AK2\tA0A2Y9NKF8_DELLE\nEIF2AK2\tA0A340Y9F1_LIPVE\nEIF2AK2\tA0A341C428_NEOAA\nEIF2AK2\tA0A341C4H2_NEOAA\nEIF2AK2\tA0A343QZ10_DESRO\nEIF2AK2\tA0A346QRN3_DANRE\nEIF2AK2\tA0A383YWE8_BALAS\nEIF2AK2\tA0A384CUB6_URSMA\nEIF2AK2\tA0A385HBS2_ROUAE\nEIF2AK2\tA0A3Q2HA94_HORSE\nEIF2AK2\tA0A3Q7NQB8_CALUR\nEIF2AK2\tA0A3Q7NR85_CALUR\nEIF2AK2\tA0A3Q7NRF5_CALUR\nEIF2AK2\tA0A3Q7P089_CALUR\nEIF2AK2\tA0A3Q7TC52_VULVU\nEIF2AK2\tA0A3Q7VA95_URSAR\nEIF2AK2\tA0A3S8RDB4_CRIMI\nEIF2AK2\tA0A3S8RDB7_MESBR\nEIF2AK2\tA0A452EV25_CAPHI\nEIF2AK2\tA0A452EV64_CAPHI\nEIF2AK2\tA0A452GLH5_9SAUR\nEIF2AK2\tA0A452GLS3_9SAUR\nEIF2AK2\tA0A455BVH4_PHYMC\nEIF2AK2\tA0A493TDG7_ANAPP\nEIF2AK2\tA0A4W2CKC8_BOBOX\nEIF2AK2\tA0A4W2CP11_BOBOX\nEIF2AK2\tA0A4W2H0X6_BOBOX\nEIF2AK2\tA0A4W2H3L8_BOBOX\nEIF2AK2\tA0A4X2LCH2_VOMUR\nEIF2AK2\tA0A4X2LHS6_VOMUR\nEIF2AK2\tA0A5F8ASK9_MACMU\nEIF2AK2\tA0A5F8G754_MONDO\nEIF2AK2\tA0A5F8GAS5_MONDO\nEIF2AK2\tA0A5F8HEP3_MONDO\nEIF2AK2\tA0A5F9CPE1_RABIT\nEIF2AK2\tA0A5F9CPI6_RABIT\nEIF2AK2\tA0A5F9CPW7_RABIT\nEIF2AK2\tA0A5F9CSV0_RABIT\nEIF2AK2\tA0A5F9CW81_RABIT\nEIF2AK2\tA0A5G2QJU0_PIG\nEIF2AK2\tA0A5G2QYH9_PIG\nEIF2AK2\tA0JNE6_BOVIN\nEIF2AK2\tA1KR30_XENLA\nEIF2AK2\tA4LAA1_MACMU\nEIF2AK2\tA4LAA2_MACMU\nEIF2AK2\tA9ULW3_XENLA\nEIF2AK2\tA9YT50_DANRE\nEIF2AK2\tB0B1T7_DANRE\nEIF2AK2\tB7ZKK7_HUMAN\nEIF2AK2\tB9UKH5_CHLAE\nEIF2AK2\tB9UKH6_PANPA\nEIF2AK2\tB9UKH7_PANTR\nEIF2AK2\tB9UKH8_NOMLE\nEIF2AK2\tB9UKH9_9PRIM\nEIF2AK2\tB9UKI0_PONPY\nEIF2AK2\tB9UKI1_ERYPA\nEIF2AK2\tB9UKI2_MACMU\nEIF2AK2\tB9UKI3_CERAT\nEIF2AK2\tB9UKI4_ATEGE\nEIF2AK2\tB9UKI5_SAGLB\nEIF2AK2\tB9UKI6_PLEMO\nEIF2AK2\tB9UKI7_LAGLA\nEIF2AK2\tB9UKI8_COLGU\nEIF2AK2\tB9UKI9_TRAFR\nEIF2AK2\tB9UKJ0_MIOTA\nEIF2AK2\tB9UKJ1_9PRIM\nEIF2AK2\tB9UKJ2_SYMSY\nEIF2AK2\tB9V257_AOTTR\nEIF2AK2\tC9JZT2_HUMAN\nEIF2AK2\tE2AK2_HUMAN\nEIF2AK2\tE2AK2_MOUSE\nEIF2AK2\tE2AK2_RAT\nEIF2AK2\tF1NLD7_CHICK\nEIF2AK2\tF6MF66_MESAU\nEIF2AK2\tF6U1R9_ORNAN\nEIF2AK2\tF6ZWY6_HORSE\nEIF2AK2\tF7CM70_MONDO\nEIF2AK2\tF7GV96_CALJA\nEIF2AK2\tF7GVA8_CALJA\nEIF2AK2\tF8WBH4_HUMAN\nEIF2AK2\tG1KHR6_ANOCA\nEIF2AK2\tG1MGS0_AILME\nEIF2AK2\tG1NF96_MELGA\nEIF2AK2\tG1S2C0_NOMLE\nEIF2AK2\tG1SSL4_RABIT\nEIF2AK2\tG3TL54_LOXAF\nEIF2AK2\tG3VEM0_SARHA\nEIF2AK2\tG3VEM1_SARHA\nEIF2AK2\tH0VKW6_CAVPO\nEIF2AK2\tH0WL03_OTOGA\nEIF2AK2\tH0ZEN8_TAEGU\nEIF2AK2\tH2P6H7_PONAB\nEIF2AK2\tH2QHR3_PANTR\nEIF2AK2\tH9ZBA6_MACMU\nEIF2AK2\tI3MLV8_ICTTR\nEIF2AK2\tK7GJD9_PELSI\nEIF2AK2\tM0RDJ3_RAT\nEIF2AK2\tM3W5Q7_FELCA\nEIF2AK2\tM3YER1_MUSPF\nEIF2AK2\tQ05CP4_HUMAN\nEIF2AK2\tQ0P4I0_XENTR\nEIF2AK2\tQ0P4K2_XENTR\nEIF2AK2\tQ153F4_MESAU\nEIF2AK2\tQ287T5_RABIT\nEIF2AK2\tQ2KM13_CANLF\nEIF2AK2\tQ2TL61_HORSE\nEIF2AK2\tQ6PK38_HUMAN\nEIF2AK2\tQ78E06_MOUSE\nEIF2AK2\tQ8IW76_HUMAN\nEIF2AK2\tQ91YN2_MOUSE\nEIF2AK2\tU3KBL8_FICAL\nEIF2AK2\tW5PJ40_SHEEP\n','From\tTo\nBUB1\tA0A034V0G8_BACDO\nBUB1\tA0A034VKZ1_BACDO\nBUB1\tA0A061S2V5_9CHLO\nBUB1\tA0A096NUE0_PAPAN\nBUB1\tA0A0A1XEG1_ZEUCU\nBUB1\tA0A0B2VP08_TOXCA\nBUB1\tA0A0B7FF52_THACB\nBUB1\tA0A0C9S012_9HYME\nBUB1\tA0A0D9RWF5_CHLSB\nBUB1\tA0A0F8B1I8_CERFI\nBUB1\tA0A0S7IZM4_9TELE\nBUB1\tA0A0V0J5A4_SCHSO\nBUB1\tA0A0V0J9L7_SCHSO\nBUB1\tA0A0V0JAQ4_SCHSO\nBUB1\tA0A0V0SHR0_9BILA\nBUB1\tA0A0V0UC72_9BILA\nBUB1\tA0A0V0UCK1_9BILA\nBUB1\tA0A0V0UD38_9BILA\nBUB1\tA0A0V0XC98_9BILA\nBUB1\tA0A0V0XCJ4_9BILA\nBUB1\tA0A0V0XCU4_9BILA\nBUB1\tA0A0V0YMM4_TRIPS\nBUB1\tA0A0V1AF91_9BILA\nBUB1\tA0A0V1AGP9_9BILA\nBUB1\tA0A0V1BJ79_TRISP\nBUB1\tA0A0V1D792_TRIBR\nBUB1\tA0A0V1D7D6_TRIBR\nBUB1\tA0A0V1D7E5_TRIBR\nBUB1\tA0A0V1FYC8_TRIPS\nBUB1\tA0A0V1HX08_9BILA\nBUB1\tA0A0V1IWF2_TRIPS\nBUB1\tA0A0V1K6Y6_TRIPS\nBUB1\tA0A0V1LPT5_9BILA\nBUB1\tA0A0V1N1C9_9BILA\nBUB1\tA0A0V1PNJ2_9BILA\nBUB1\tA0A0V1PNP3_9BILA\nBUB1\tA0A0X3NXP0_SCHSO\nBUB1\tA0A0X3PMU8_SCHSO\nBUB1\tA0A146KPK8_LYGHE\nBUB1\tA0A151M1M0_ALLMI\nBUB1\tA0A167DVS0_9ASCO\nBUB1\tA0A1A7WCZ7_9TELE\nBUB1\tA0A1A7XFU9_9TELE\nBUB1\tA0A1A7YK53_9TELE\nBUB1\tA0A1A7ZJP2_NOTFU\nBUB1\tA0A1A8AAH9_NOTFU\nBUB1\tA0A1A8BYV2_9TELE\nBUB1\tA0A1A8ET20_9TELE\nBUB1\tA0A1A8FGH3_9TELE\nBUB1\tA0A1A8IDW1_NOTKU\nBUB1\tA0A1A8II84_NOTKU\nBUB1\tA0A1A8MUZ5_9TELE\nBUB1\tA0A1A8QEU9_9TELE\nBUB1\tA0A1A8QJY6_9TELE\nBUB1\tA0A1A8QUJ1_9TELE\nBUB1\tA0A1A8R007_9TELE\nBUB1\tA0A1A8VER2_NOTFU\nBUB1\tA0A1B2JAR0_PICPA\nBUB1\tA0A1C7M8F5_GRIFR\nBUB1\tA0A1C7NF66_9FUNG\nBUB1\tA0A1D5QMZ4_MACMU\nBUB1\tA0A1D8PLR1_CANAL\nBUB1\tA0A1J6I5E1_NICAT\nBUB1\tA0A1S3AML6_ERIEU\nBUB1\tA0A1S3FGX5_DIPOR\nBUB1\tA0A1U7SZJ9_TARSY\nBUB1\tA0A1U7T647_TARSY\nBUB1\tA0A1U7TA44_TARSY\nBUB1\tA0A1V4IHF5_PATFA\nBUB1\tA0A2C5X5F7_9PEZI\nBUB1\tA0A2D0PI97_ICTPU\nBUB1\tA0A2D0PJR6_ICTPU\nBUB1\tA0A2D0PMC8_ICTPU\nBUB1\tA0A2H8TVQ9_9HEMI\nBUB1\tA0A2I0M9H7_COLLI\nBUB1\tA0A2I3H614_NOMLE\nBUB1\tA0A2I3LE98_PAPAN\nBUB1\tA0A2I3LIE4_PAPAN\nBUB1\tA0A2I3MVN0_PAPAN\nBUB1\tA0A2I3RV03_PANTR\nBUB1\tA0A2I3SD22_PANTR\nBUB1\tA0A2I4C074_9TELE\nBUB1\tA0A2K5CDB2_AOTNA\nBUB1\tA0A2K5CDR3_AOTNA\nBUB1\tA0A2K5CDS0_AOTNA\nBUB1\tA0A2K5K196_COLAP\nBUB1\tA0A2K5K198_COLAP\nBUB1\tA0A2K5K1A6_COLAP\nBUB1\tA0A2K5K1A8_COLAP\nBUB1\tA0A2K5KHT4_CERAT\nBUB1\tA0A2K5KHU4_CERAT\nBUB1\tA0A2K5KHV0_CERAT\nBUB1\tA0A2K5KHV2_CERAT\nBUB1\tA0A2K5RSJ0_CEBCA\nBUB1\tA0A2K5RSM8_CEBCA\nBUB1\tA0A2K5RSQ8_CEBCA\nBUB1\tA0A2K5RSR3_CEBCA\nBUB1\tA0A2K5W7F5_MACFA\nBUB1\tA0A2K5W7I7_MACFA\nBUB1\tA0A2K5W7J1_MACFA\nBUB1\tA0A2K5XUI3_MANLE\nBUB1\tA0A2K5XUM9_MANLE\nBUB1\tA0A2K5XUN2_MANLE\nBUB1\tA0A2K5XUP4_MANLE\nBUB1\tA0A2K6BUR3_MACNE\nBUB1\tA0A2K6BUT5_MACNE\nBUB1\tA0A2K6BUU4_MACNE\nBUB1\tA0A2K6BUV1_MACNE\nBUB1\tA0A2K6GK77_PROCO\nBUB1\tA0A2K6GKB1_PROCO\nBUB1\tA0A2K6GKB6_PROCO\nBUB1\tA0A2K6KEI2_RHIBE\nBUB1\tA0A2K6KEL2_RHIBE\nBUB1\tA0A2K6KEL4_RHIBE\nBUB1\tA0A2K6KEL6_RHIBE\nBUB1\tA0A2K6KEN0_RHIBE\nBUB1\tA0A2K6Q275_RHIRO\nBUB1\tA0A2K6Q276_RHIRO\nBUB1\tA0A2K6Q277_RHIRO\nBUB1\tA0A2K6Q282_RHIRO\nBUB1\tA0A2K6V156_SAIBB\nBUB1\tA0A2K6V161_SAIBB\nBUB1\tA0A2K6V162_SAIBB\nBUB1\tA0A2N6NGW8_BEABA\nBUB1\tA0A2R8MD69_CALJA\nBUB1\tA0A2R8MQ62_CALJA\nBUB1\tA0A2R8QQZ9_DANRE\nBUB1\tA0A2R9BSZ5_PANPA\nBUB1\tA0A2R9BT00_PANPA\nBUB1\tA0A2R9BWG3_PANPA\nBUB1\tA0A2R9C266_PANPA\nBUB1\tA0A2S2PMX0_SCHGA\nBUB1\tA0A2U3VWP6_ODORO\nBUB1\tA0A2U3ZF64_ODORO\nBUB1\tA0A2U3ZF65_ODORO\nBUB1\tA0A2U4B098_TURTR\nBUB1\tA0A2U4B0C6_TURTR\nBUB1\tA0A2Y9FCE9_PHYMC\nBUB1\tA0A2Y9HVK2_NEOSC\nBUB1\tA0A2Y9Q7A1_DELLE\nBUB1\tA0A2Y9TA17_PHYMC\nBUB1\tA0A340Y9D5_LIPVE\nBUB1\tA0A340YB32_LIPVE\nBUB1\tA0A340YCW0_LIPVE\nBUB1\tA0A340YFP4_LIPVE\nBUB1\tA0A341BDS2_NEOAA\nBUB1\tA0A369JUX8_HYPMA\nBUB1\tA0A371FVU7_MUCPR\nBUB1\tA0A384BCI5_BALAS\nBUB1\tA0A384BCK6_BALAS\nBUB1\tA0A384BD96_BALAS\nBUB1\tA0A384CU38_URSMA\nBUB1\tA0A3B3HNE7_ORYLA\nBUB1\tA0A3G2S1L9_9BASI\nBUB1\tA0A3L6E6K3_MAIZE\nBUB1\tA0A3M9Y2P4_9PEZI\nBUB1\tA0A3Q0CN53_MESAU\nBUB1\tA0A3Q0GTN4_ALLSI\nBUB1\tA0A3Q7NRL1_CALUR\nBUB1\tA0A3Q7S3V0_VULVU\nBUB1\tA0A3Q7UZF5_URSAR\nBUB1\tA0A452EAQ1_CAPHI\nBUB1\tA0A452EAT1_CAPHI\nBUB1\tA0A452EB17_CAPHI\nBUB1\tA0A452HRC7_9SAUR\nBUB1\tA0A452R3V5_URSAM\nBUB1\tA0A452U5M7_URSMA\nBUB1\tA0A452U5N7_URSMA\nBUB1\tA0A484G0J6_COLOR\nBUB1\tA0A4R8PSB8_9PEZI\nBUB1\tA0A4R8RS11_COLTR\nBUB1\tA0A4R8TEM6_9PEZI\nBUB1\tA0A4U6XD45_9PEZI\nBUB1\tA0A4W2DHP2_BOBOX\nBUB1\tA0A4W3J5S1_CALMI\nBUB1\tA0A4W6EPG0_LATCA\nBUB1\tA0A4W6EQK6_LATCA\nBUB1\tA0A4X1VPB3_PIG\nBUB1\tA0A4X1VU78_PIG\nBUB1\tA0A4X1VUU8_PIG\nBUB1\tA0A4X2LI37_VOMUR\nBUB1\tA0A4Z2H724_9TELE\nBUB1\tA0A559L7Q9_FUSOX\nBUB1\tA0A559MBN4_9HELO\nBUB1\tA0A590UJ99_HUMAN\nBUB1\tA0A590UK50_HUMAN\nBUB1\tA0A5F4D4C2_CANLF\nBUB1\tA0A5F5XJX9_FELCA\nBUB1\tA0A5F5XMW7_FELCA\nBUB1\tA0A5F9C5V7_RABIT\nBUB1\tA0A5F9CEG6_RABIT\nBUB1\tA0A5G2QME5_PIG\nBUB1\tA0A5G2QRE9_PIG\nBUB1\tA0A5G2RJM3_PIG\nBUB1\tA0A5G3JDW2_XENTR\nBUB1\tA0JLU2_HUMAN\nBUB1\tA1Z6I7_DROME\nBUB1\tA2APR8_MOUSE\nBUB1\tA6QLW0_BOVIN\nBUB1\tA6ZUJ9_YEAS7\nBUB1\tB0VJZ8_DANRE\nBUB1\tBUB1_ARATH\nBUB1\tBUB1_DICDI\nBUB1\tBUB1_HUMAN\nBUB1\tBUB1_MOUSE\nBUB1\tBUB1_SCHPO\nBUB1\tBUB1_YEAST\nBUB1\tC0H9A9_SALSA\nBUB1\tC7GVV3_YEAS2\nBUB1\tC9IYH4_HUMAN\nBUB1\tC9JQA4_HUMAN\nBUB1\tC9JRC7_HUMAN\nBUB1\tD3AZV9_POLPP\nBUB1\tD4A5D3_RAT\nBUB1\tE2RAD4_CANLF\nBUB1\tF1NDM5_CHICK\nBUB1\tF1QNS6_DANRE\nBUB1\tF2QT36_KOMPC\nBUB1\tF4Q3S0_CAVFA\nBUB1\tF6R3D0_XENTR\nBUB1\tF6U7Z4_CALJA\nBUB1\tF6UI40_MACMU\nBUB1\tF6VJF3_XENTR\nBUB1\tF6WNU4_HORSE\nBUB1\tF6YWN3_MONDO\nBUB1\tF7FTL8_ORNAN\nBUB1\tG1LSM0_AILME\nBUB1\tG1MV50_MELGA\nBUB1\tG1PDD0_MYOLU\nBUB1\tG1T6K1_RABIT\nBUB1\tG3T227_LOXAF\nBUB1\tG3URG7_MELGA\nBUB1\tG3VMC0_SARHA\nBUB1\tG3VMC1_SARHA\nBUB1\tG7PMY0_MACFA\nBUB1\tH0VN36_CAVPO\nBUB1\tH0WX62_OTOGA\nBUB1\tH2LZL1_ORYLA\nBUB1\tH2P5E0_PONAB\nBUB1\tH2QIJ3_PANTR\nBUB1\tH2UKJ2_TAKRU\nBUB1\tH2UKJ4_TAKRU\nBUB1\tH3BC50_LATCH\nBUB1\tH9GJN0_ANOCA\nBUB1\tH9H9X6_NOMLE\nBUB1\tH9Z276_MACMU\nBUB1\tI3KR85_ORENI\nBUB1\tI3KR86_ORENI\nBUB1\tI3LRF1_PIG\nBUB1\tJ9NX78_CANLF\nBUB1\tK7D3R8_PANTR\nBUB1\tK7GI69_PELSI\nBUB1\tK8YUP2_NANGC\nBUB1\tM3WLM0_FELCA\nBUB1\tM3XGI7_LATCH\nBUB1\tM5BQF9_THACB\nBUB1\tO76755_DROME\nBUB1\tQ0P4V9_XENTR\nBUB1\tQ6GPC9_XENLA\nBUB1\tQ7KN56_DROME\nBUB1\tQ8C4D0_MOUSE\nBUB1\tQ8K1K8_MOUSE\nBUB1\tQ98UH3_XENLA\nBUB1\tQ9BXX5_HUMAN\nBUB1\tQ9CS10_MOUSE\nBUB1\tQ9VMS5_DROME\nBUB1\tS4RXY0_PETMA\nBUB1\tT2MIK8_HYDVU\nBUB1\tU3IWU9_ANAPP\nBUB1\tU3KAD5_FICAL\nBUB1\tV5GA69_ANOGL\nBUB1\tV8NE90_OPHHA\nBUB1\tW0TBC9_KLUMD\nBUB1\tW5PY25_SHEEP\nBUB1\tW8BQK8_CERCA\nBUB1\tW8BRV8_CERCA\n','From\tTo\nSRPK1\tA0A024RCU9_HUMAN\nSRPK1\tA0A060RSL0_PLARE\nSRPK1\tA0A0C9RZB9_9HYME\nSRPK1\tA0A0Q3UQG7_AMAAE\nSRPK1\tA0A0S7EFQ6_9TELE\nSRPK1\tA0A0S7GJ86_9TELE\nSRPK1\tA0A0S7GNX9_9TELE\nSRPK1\tA0A0S7H811_9TELE\nSRPK1\tA0A0S7K0Z4_9TELE\nSRPK1\tA0A0S7K113_9TELE\nSRPK1\tA0A0S7K124_9TELE\nSRPK1\tA0A0S7K130_9TELE\nSRPK1\tA0A0S7K143_9TELE\nSRPK1\tA0A0S7K1B5_9TELE\nSRPK1\tA0A0S7K1J6_9TELE\nSRPK1\tA0A0S7K1L1_9TELE\nSRPK1\tA0A0S7K1M8_9TELE\nSRPK1\tA0A0S7K1R2_9TELE\nSRPK1\tA0A0S7K1T9_9TELE\nSRPK1\tA0A0S7K246_9TELE\nSRPK1\tA0A0S7K2G1_9TELE\nSRPK1\tA0A0S7K2N9_9TELE\nSRPK1\tA0A0S7K302_9TELE\nSRPK1\tA0A0S7K341_9TELE\nSRPK1\tA0A0S7K4B1_9TELE\nSRPK1\tA0A0U1RRU0_RAT\nSRPK1\tA0A0V0S9X9_9BILA\nSRPK1\tA0A0V0SA14_9BILA\nSRPK1\tA0A0V0TJL6_9BILA\nSRPK1\tA0A0V0TJM7_9BILA\nSRPK1\tA0A0V0TJV8_9BILA\nSRPK1\tA0A0V0TJY0_9BILA\nSRPK1\tA0A0V0WKJ2_9BILA\nSRPK1\tA0A0V0WKJ5_9BILA\nSRPK1\tA0A0V0WKR9_9BILA\nSRPK1\tA0A0V0WL01_9BILA\nSRPK1\tA0A0V0XVK0_TRIPS\nSRPK1\tA0A0V0ZF28_9BILA\nSRPK1\tA0A0V0ZF67_9BILA\nSRPK1\tA0A0V1BEK3_TRISP\nSRPK1\tA0A0V1BEP7_TRISP\nSRPK1\tA0A0V1BEX6_TRISP\nSRPK1\tA0A0V1D383_TRIBR\nSRPK1\tA0A0V1D3D9_TRIBR\nSRPK1\tA0A0V1D3J3_TRIBR\nSRPK1\tA0A0V1EGT2_TRIPS\nSRPK1\tA0A0V1EH75_TRIPS\nSRPK1\tA0A0V1FSK6_TRIPS\nSRPK1\tA0A0V1HAW5_9BILA\nSRPK1\tA0A0V1HCG9_9BILA\nSRPK1\tA0A0V1HCK3_9BILA\nSRPK1\tA0A0V1IZ97_TRIPS\nSRPK1\tA0A0V1LIZ8_9BILA\nSRPK1\tA0A0V1LJ02_9BILA\nSRPK1\tA0A0V1LJ32_9BILA\nSRPK1\tA0A0V1LJY5_9BILA\nSRPK1\tA0A0V1M8A1_9BILA\nSRPK1\tA0A0V1M8I7_9BILA\nSRPK1\tA0A0V1M995_9BILA\nSRPK1\tA0A0V1P7P2_9BILA\nSRPK1\tA0A0V1P7P4_9BILA\nSRPK1\tA0A0Y9U8V9_PLABE\nSRPK1\tA0A146LMA3_LYGHE\nSRPK1\tA0A1C3KH40_9APIC\nSRPK1\tA0A1C3KYX5_PLAMA\nSRPK1\tA0A1C6WQF9_PLABE\nSRPK1\tA0A1C6X4K9_PLACH\nSRPK1\tA0A1C6XCA1_PLACH\nSRPK1\tA0A1D3LBI7_PLACH\nSRPK1\tA0A1D3LFT8_PLACH\nSRPK1\tA0A1D3PBW9_PLAMA\nSRPK1\tA0A1D3THF5_9APIC\nSRPK1\tA0A1D5PFV9_CHICK\nSRPK1\tA0A1J1GS45_PLAGA\nSRPK1\tA0A1J1H4R6_PLARL\nSRPK1\tA0A1Q9CRU5_SYMMI\nSRPK1\tA0A1Q9DA47_SYMMI\nSRPK1\tA0A1S3AFI6_ERIEU\nSRPK1\tA0A1S3G896_DIPOR\nSRPK1\tA0A1U7R3F1_MESAU\nSRPK1\tA0A1U7S6Q4_ALLSI\nSRPK1\tA0A1U7TS69_TARSY\nSRPK1\tA0A1U7U1S4_TARSY\nSRPK1\tA0A1V4KWH1_PATFA\nSRPK1\tA0A218UJ12_9PASE\nSRPK1\tA0A250YKI9_CASCN\nSRPK1\tA0A286ZTD5_PIG\nSRPK1\tA0A286ZWV2_PIG\nSRPK1\tA0A287A409_PIG\nSRPK1\tA0A287A656_PIG\nSRPK1\tA0A287AX92_PIG\nSRPK1\tA0A287BP05_PIG\nSRPK1\tA0A2D0QWW4_ICTPU\nSRPK1\tA0A2I0LKP6_COLLI\nSRPK1\tA0A2I3HP93_NOMLE\nSRPK1\tA0A2I3HRM0_NOMLE\nSRPK1\tA0A2I3RKY5_PANTR\nSRPK1\tA0A2I3TC35_PANTR\nSRPK1\tA0A2I4C5E5_9TELE\nSRPK1\tA0A2I4C5E6_9TELE\nSRPK1\tA0A2I4C5G5_9TELE\nSRPK1\tA0A2K5CDN7_AOTNA\nSRPK1\tA0A2K5CDV5_AOTNA\nSRPK1\tA0A2K5RNW8_CEBCA\nSRPK1\tA0A2K5RNX3_CEBCA\nSRPK1\tA0A2K6EFK4_PROCO\nSRPK1\tA0A2K6UWR6_SAIBB\nSRPK1\tA0A2K6UWS4_SAIBB\nSRPK1\tA0A2K6UWT8_SAIBB\nSRPK1\tA0A2N6NDL8_BEABA\nSRPK1\tA0A2P8Y9P7_BLAGE\nSRPK1\tA0A2R9A803_PANPA\nSRPK1\tA0A2R9A809_PANPA\nSRPK1\tA0A2R9A909_PANPA\nSRPK1\tA0A2U3WGI5_ODORO\nSRPK1\tA0A2U3Y642_LEPWE\nSRPK1\tA0A2U4CID8_TURTR\nSRPK1\tA0A2U4CIE1_TURTR\nSRPK1\tA0A2U4CIE3_TURTR\nSRPK1\tA0A2Y9GM79_NEOSC\nSRPK1\tA0A2Y9GSX4_NEOSC\nSRPK1\tA0A2Y9P6Q5_DELLE\nSRPK1\tA0A2Y9SFQ9_PHYMC\nSRPK1\tA0A2Y9SJH7_PHYMC\nSRPK1\tA0A2Y9SMC5_PHYMC\nSRPK1\tA0A340Y6H3_LIPVE\nSRPK1\tA0A341D474_NEOAA\nSRPK1\tA0A384BE69_BALAS\nSRPK1\tA0A384C9V7_URSMA\nSRPK1\tA0A3B2W430_MOUSE\nSRPK1\tA0A3B2W4B9_MOUSE\nSRPK1\tA0A3B2W7I6_MOUSE\nSRPK1\tA0A3B2W883_MOUSE\nSRPK1\tA0A3B3HIW7_ORYLA\nSRPK1\tA0A3B3HR04_ORYLA\nSRPK1\tA0A3B3IL12_ORYLA\nSRPK1\tA0A3B3S504_9TELE\nSRPK1\tA0A3Q1M8Z5_BOVIN\nSRPK1\tA0A3Q1NM78_BOVIN\nSRPK1\tA0A3Q2KXQ9_HORSE\nSRPK1\tA0A3Q7NCI4_CALUR\nSRPK1\tA0A3Q7RN15_VULVU\nSRPK1\tA0A3Q7V6I5_URSAR\nSRPK1\tA0A419Q2W8_CLOSI\nSRPK1\tA0A452EJJ5_CAPHI\nSRPK1\tA0A452HAY5_9SAUR\nSRPK1\tA0A452HAZ4_9SAUR\nSRPK1\tA0A452HAZ9_9SAUR\nSRPK1\tA0A452R0M1_URSAM\nSRPK1\tA0A452TZY5_URSMA\nSRPK1\tA0A455AE34_PHYMC\nSRPK1\tA0A455AFA3_PHYMC\nSRPK1\tA0A493SWF1_ANAPP\nSRPK1\tA0A493SZG0_ANAPP\nSRPK1\tA0A493TFW4_ANAPP\nSRPK1\tA0A493TGP1_ANAPP\nSRPK1\tA0A4W2BZK7_BOBOX\nSRPK1\tA0A4W2DU41_BOBOX\nSRPK1\tA0A4W2FKL5_BOBOX\nSRPK1\tA0A4W4EW23_ELEEL\nSRPK1\tA0A4W4EW74_ELEEL\nSRPK1\tA0A4W4EWF1_ELEEL\nSRPK1\tA0A4W4EWS1_ELEEL\nSRPK1\tA0A4W4EXG7_ELEEL\nSRPK1\tA0A4W4EY65_ELEEL\nSRPK1\tA0A4W4EZ36_ELEEL\nSRPK1\tA0A4W4EZ54_ELEEL\nSRPK1\tA0A4W6CPR2_LATCA\nSRPK1\tA0A4W6CPR8_LATCA\nSRPK1\tA0A4W6CQ07_LATCA\nSRPK1\tA0A4W6CQ12_LATCA\nSRPK1\tA0A4W6CQ59_LATCA\nSRPK1\tA0A4X1SW41_PIG\nSRPK1\tA0A4X1SWL5_PIG\nSRPK1\tA0A4X1SWP1_PIG\nSRPK1\tA0A4X1SWT2_PIG\nSRPK1\tA0A4X1SXS0_PIG\nSRPK1\tA0A4X1SXZ9_PIG\nSRPK1\tA0A4X1SYA2_PIG\nSRPK1\tA0A4X1SYF2_PIG\nSRPK1\tA0A4X2LIU1_VOMUR\nSRPK1\tA0A4X2LIV3_VOMUR\nSRPK1\tA0A4X2LVB0_VOMUR\nSRPK1\tA0A4Z2J0B0_9TELE\nSRPK1\tA0A5B7DU98_PORTR\nSRPK1\tA0A5F4BUU6_CANLF\nSRPK1\tA0A5F4BXR6_CANLF\nSRPK1\tA0A5F4C8C5_CANLF\nSRPK1\tA0A5F4D2Z9_CANLF\nSRPK1\tA0A5F4D6E7_CANLF\nSRPK1\tA0A5F4DLX7_CANLF\nSRPK1\tA0A5F8G4A2_MONDO\nSRPK1\tA0A5F8GIC1_MONDO\nSRPK1\tA0A5F9CJU4_RABIT\nSRPK1\tA0A5F9CP00_RABIT\nSRPK1\tA0A5F9D9S2_RABIT\nSRPK1\tA0A5G3H2I6_XENTR\nSRPK1\tA0A5G3I9R3_XENTR\nSRPK1\tB2KF32_MOUSE\nSRPK1\tB7UCT2_PIG\nSRPK1\tCIPK7_ARATH\nSRPK1\tD5KJI7_PIG\nSRPK1\tD5KJI8_PIG\nSRPK1\tD5KJI9_PIG\nSRPK1\tD6RB98_HUMAN\nSRPK1\tD6RBF8_HUMAN\nSRPK1\tD6RBM8_HUMAN\nSRPK1\tD6RDZ3_HUMAN\nSRPK1\tE1BNV4_BOVIN\nSRPK1\tE2RL63_CANLF\nSRPK1\tE9PTN4_RAT\nSRPK1\tF6PW60_PIG\nSRPK1\tF6X887_MONDO\nSRPK1\tF6Z8F7_XENTR\nSRPK1\tF6ZD95_XENTR\nSRPK1\tF7C1S0_HORSE\nSRPK1\tF7IEZ5_CALJA\nSRPK1\tF8WGJ7_MOUSE\nSRPK1\tG1L120_AILME\nSRPK1\tG1MPY2_MELGA\nSRPK1\tG1PH88_MYOLU\nSRPK1\tG1RBZ7_NOMLE\nSRPK1\tG1SVM0_RABIT\nSRPK1\tG3RCK6_GORGO\nSRPK1\tG3SL39_LOXAF\nSRPK1\tG3U3L3_LOXAF\nSRPK1\tH0V2J5_CAVPO\nSRPK1\tH0X169_OTOGA\nSRPK1\tH0Y932_HUMAN\nSRPK1\tH0YTU3_TAEGU\nSRPK1\tH2LPV7_ORYLA\nSRPK1\tH2PIU3_PONAB\nSRPK1\tH2QV71_PANTR\nSRPK1\tH2UXG4_TAKRU\nSRPK1\tH3AXH7_LATCH\nSRPK1\tH3BLV9_HUMAN\nSRPK1\tH9FYW2_MACMU\nSRPK1\tH9GNX6_ANOCA\nSRPK1\tI3K1A3_ORENI\nSRPK1\tI3MLK8_ICTTR\nSRPK1\tK7FGC4_PELSI\nSRPK1\tM3XD27_FELCA\nSRPK1\tM3XXI8_MUSPF\nSRPK1\tM9MS50_DROME\nSRPK1\tM9NDF9_DROME\nSRPK1\tM9NDZ1_DROME\nSRPK1\tM9NEI4_DROME\nSRPK1\tM9NGC6_DROME\nSRPK1\tM9PDA8_DROME\nSRPK1\tM9PIK8_DROME\nSRPK1\tQ28DU4_XENTR\nSRPK1\tQ3TTF9_MOUSE\nSRPK1\tQ3TTI6_MOUSE\nSRPK1\tQ3UB06_MOUSE\nSRPK1\tQ3URR1_MOUSE\nSRPK1\tQ3US65_MOUSE\nSRPK1\tQ4KLN3_RAT\nSRPK1\tQ6INS4_XENLA\nSRPK1\tQ6PGU8_DANRE\nSRPK1\tQ8S3K5_ELAOL\nSRPK1\tQ8S9N4_ELAOL\nSRPK1\tQ8T3S1_DROME\nSRPK1\tQ8VHL3_CRILO\nSRPK1\tQ9FYB4_ARATH\nSRPK1\tQ9VNV0_DROME\nSRPK1\tSRPK1_HUMAN\nSRPK1\tSRPK1_MOUSE\nSRPK1\tSRPK1_PONAB\nSRPK1\tT2M7G3_HYDVU\nSRPK1\tU3CTI8_CALJA\nSRPK1\tU3I6K9_ANAPP\nSRPK1\tU3JLT0_FICAL\nSRPK1\tU6D917_NEOVI\nSRPK1\tW5PSH6_SHEEP\nSRPK1\tW5UJQ8_ICTPU\nSRPK1\tW5USM6_ICTPU\n','From\tTo\nMAPK11\tA0A096NL08_PAPAN\nMAPK11\tA0A0D9QY14_CHLSB\nMAPK11\tA0A0G2KB06_RAT\nMAPK11\tA0A1A7X297_9TELE\nMAPK11\tA0A1A7Z7B0_9TELE\nMAPK11\tA0A1A7ZJ89_NOTFU\nMAPK11\tA0A1A8C9R3_9TELE\nMAPK11\tA0A1A8ENJ6_9TELE\nMAPK11\tA0A1A8F513_9TELE\nMAPK11\tA0A1A8H8G8_9TELE\nMAPK11\tA0A1A8I944_NOTKU\nMAPK11\tA0A1A8JTT2_NOTKU\nMAPK11\tA0A1A8N196_9TELE\nMAPK11\tA0A1A8NAF7_9TELE\nMAPK11\tA0A1A8NUT4_9TELE\nMAPK11\tA0A1A8Q0N8_9TELE\nMAPK11\tA0A1A8RNV4_9TELE\nMAPK11\tA0A1D5PRV8_CHICK\nMAPK11\tA0A1S2YQI6_CICAR\nMAPK11\tA0A1S2ZHG0_ERIEU\nMAPK11\tA0A1S3W6S1_ERIEU\nMAPK11\tA0A1S3W7V4_ERIEU\nMAPK11\tA0A1U7SQ23_TARSY\nMAPK11\tA0A1V4K363_PATFA\nMAPK11\tA0A218UXF3_9PASE\nMAPK11\tA0A286XFB6_CAVPO\nMAPK11\tA0A286ZJZ3_PIG\nMAPK11\tA0A2I3FWE7_NOMLE\nMAPK11\tA0A2K5EQA7_AOTNA\nMAPK11\tA0A2K5HAU1_COLAP\nMAPK11\tA0A2K5L9X2_CERAT\nMAPK11\tA0A2K5L9Y6_CERAT\nMAPK11\tA0A2K5SJ57_CEBCA\nMAPK11\tA0A2K5SJ58_CEBCA\nMAPK11\tA0A2K5U0Z6_MACFA\nMAPK11\tA0A2K5U104_MACFA\nMAPK11\tA0A2K5Z5R0_MANLE\nMAPK11\tA0A2K6BKF4_MACNE\nMAPK11\tA0A2K6BKI6_MACNE\nMAPK11\tA0A2K6FLD3_PROCO\nMAPK11\tA0A2K6FLF2_PROCO\nMAPK11\tA0A2K6MF86_RHIBE\nMAPK11\tA0A2K6MF90_RHIBE\nMAPK11\tA0A2K6RQR4_RHIRO\nMAPK11\tA0A2K6V0I5_SAIBB\nMAPK11\tA0A2R9B8G6_PANPA\nMAPK11\tA0A2U3WLA9_ODORO\nMAPK11\tA0A2U3Z7B5_LEPWE\nMAPK11\tA0A2U3ZQ17_ODORO\nMAPK11\tA0A2U4B639_TURTR\nMAPK11\tA0A2U4B687_TURTR\nMAPK11\tA0A2Y9H0E6_NEOSC\nMAPK11\tA0A2Y9Q4N3_DELLE\nMAPK11\tA0A2Y9QAX2_DELLE\nMAPK11\tA0A2Y9T159_PHYMC\nMAPK11\tA0A2Y9T558_PHYMC\nMAPK11\tA0A340WXR5_LIPVE\nMAPK11\tA0A341AVN3_NEOAA\nMAPK11\tA0A345DKR4_9TELE\nMAPK11\tA0A384APM9_BALAS\nMAPK11\tA0A384D1Y9_URSMA\nMAPK11\tA0A3G2LZD0_CTEID\nMAPK11\tA0A3Q0CEW1_MESAU\nMAPK11\tA0A3Q0HHX5_ALLSI\nMAPK11\tA0A3Q7PUF8_CALUR\nMAPK11\tA0A3Q7PWF1_CALUR\nMAPK11\tA0A3Q7PX38_CALUR\nMAPK11\tA0A3Q7Q7R1_CALUR\nMAPK11\tA0A3Q7RAL4_CALUR\nMAPK11\tA0A3Q7RCD4_CALUR\nMAPK11\tA0A3Q7RCE0_CALUR\nMAPK11\tA0A3Q7U0V6_VULVU\nMAPK11\tA0A3Q7U4D1_URSAR\nMAPK11\tA0A452F5K8_CAPHI\nMAPK11\tA0A452H9Q6_9SAUR\nMAPK11\tA0A452HA00_9SAUR\nMAPK11\tA0A452REQ4_URSAM\nMAPK11\tA0A452RES7_URSAM\nMAPK11\tA0A4W2C012_BOBOX\nMAPK11\tA0A4W3J2I3_CALMI\nMAPK11\tA0A4W3J2I8_CALMI\nMAPK11\tA0A4W3J2J2_CALMI\nMAPK11\tA0A4W3J596_CALMI\nMAPK11\tA0A4W3JCH5_CALMI\nMAPK11\tA0A4W3JCI0_CALMI\nMAPK11\tA0A4W3JCI5_CALMI\nMAPK11\tA0A4W3JGW1_CALMI\nMAPK11\tA0A4W3JZX8_CALMI\nMAPK11\tA0A4X1ULS6_PIG\nMAPK11\tA0A4X1UNE8_PIG\nMAPK11\tA0A4X2JZ05_VOMUR\nMAPK11\tA0A5F4D7W6_CANLF\nMAPK11\tA0A5G3L917_XENTR\nMAPK11\tA0A5G3L920_XENTR\nMAPK11\tA0A5G3L9P4_XENTR\nMAPK11\tA0A5G3LAU2_XENTR\nMAPK11\tA0A5G3LAU5_XENTR\nMAPK11\tA0A5G3LAZ2_XENTR\nMAPK11\tA1A4H8_BOVIN\nMAPK11\tA7MC61_XENTR\nMAPK11\tD2CIV6_CRAAR\nMAPK11\tD4A3U7_RAT\nMAPK11\tF6ZWY8_XENTR\nMAPK11\tF6ZX56_XENTR\nMAPK11\tF6ZX65_XENTR\nMAPK11\tF7A4R4_XENTR\nMAPK11\tF7A9Z3_HORSE\nMAPK11\tF7C798_MACMU\nMAPK11\tF7FBR0_ORNAN\nMAPK11\tF7FBR4_ORNAN\nMAPK11\tF8UFC1_SOLLC\nMAPK11\tF8WDP4_HUMAN\nMAPK11\tG1K9Y9_ANOCA\nMAPK11\tG1L6U9_AILME\nMAPK11\tG1N7V2_MELGA\nMAPK11\tG1QII7_NOMLE\nMAPK11\tG3W7G2_SARHA\nMAPK11\tG3X793_BOVIN\nMAPK11\tH0WEK3_DANRE\nMAPK11\tH0XAN7_OTOGA\nMAPK11\tH0Z0Y9_TAEGU\nMAPK11\tH3B7L5_LATCH\nMAPK11\tH9FX69_MACMU\nMAPK11\tI3MPH0_ICTTR\nMAPK11\tI3VB57_DANRE\nMAPK11\tI3VB58_DANRE\nMAPK11\tK6Z8H5_PANTR\nMAPK11\tK7G5X7_PELSI\nMAPK11\tM3WG00_FELCA\nMAPK11\tM3XT48_MUSPF\nMAPK11\tMK11_HUMAN\nMAPK11\tMK11_MOUSE\nMAPK11\tQ3TT60_MOUSE\nMAPK11\tQ640H9_XENLA\nMAPK11\tQ6IQ84_DANRE\nMAPK11\tQ6P243_MOUSE\nMAPK11\tQ6PG57_MOUSE\nMAPK11\tU3BS38_CALJA\nMAPK11\tU3FHR1_CALJA\nMAPK11\tU3I9V4_ANAPP\nMAPK11\tU3K560_FICAL\nMAPK11\tV8NGJ1_OPHHA\nMAPK11\tV9KS90_CALMI\nMAPK11\tW5UIG5_ICTPU\n','From\tTo\nMAPK12\tA0A096NL07_PAPAN\nMAPK12\tA0A0D9QY18_CHLSB\nMAPK12\tA0A0F6QMN6_CLABA\nMAPK12\tA0A141GRT4_HETFO\nMAPK12\tA0A151PEP0_ALLMI\nMAPK12\tA0A178VX75_ARATH\nMAPK12\tA0A1D5Q2M1_MACMU\nMAPK12\tA0A1S2YVV6_CICAR\nMAPK12\tA0A1S2ZHF8_ERIEU\nMAPK12\tA0A1S2ZHH4_ERIEU\nMAPK12\tA0A1U7RAQ9_ALLSI\nMAPK12\tA0A1V4K3A2_PATFA\nMAPK12\tA0A218UYY3_9PASE\nMAPK12\tA0A250XVX5_CASCN\nMAPK12\tA0A287ARU5_PIG\nMAPK12\tA0A287B826_PIG\nMAPK12\tA0A287BQ93_PIG\nMAPK12\tA0A2I0MH92_COLLI\nMAPK12\tA0A2I2UFM0_FELCA\nMAPK12\tA0A2I3FZL9_NOMLE\nMAPK12\tA0A2I3GP09_NOMLE\nMAPK12\tA0A2I3HGA0_NOMLE\nMAPK12\tA0A2I3LLD4_PAPAN\nMAPK12\tA0A2I3LMT3_PAPAN\nMAPK12\tA0A2I3N0G1_PAPAN\nMAPK12\tA0A2I3S582_PANTR\nMAPK12\tA0A2I3SAT1_PANTR\nMAPK12\tA0A2K5EJE1_AOTNA\nMAPK12\tA0A2K5EJG4_AOTNA\nMAPK12\tA0A2K5EJJ7_AOTNA\nMAPK12\tA0A2K5EJK1_AOTNA\nMAPK12\tA0A2K5EJK3_AOTNA\nMAPK12\tA0A2K5IAX9_COLAP\nMAPK12\tA0A2K5IAY5_COLAP\nMAPK12\tA0A2K5M6B7_CERAT\nMAPK12\tA0A2K5M6C4_CERAT\nMAPK12\tA0A2K5M6D2_CERAT\nMAPK12\tA0A2K5M6D8_CERAT\nMAPK12\tA0A2K5RHH5_CEBCA\nMAPK12\tA0A2K5RHI8_CEBCA\nMAPK12\tA0A2K5RHK2_CEBCA\nMAPK12\tA0A2K5UBS3_MACFA\nMAPK12\tA0A2K5UBS5_MACFA\nMAPK12\tA0A2K5UBS7_MACFA\nMAPK12\tA0A2K5ZHE5_MANLE\nMAPK12\tA0A2K6DU65_MACNE\nMAPK12\tA0A2K6DU76_MACNE\nMAPK12\tA0A2K6DU79_MACNE\nMAPK12\tA0A2K6DU80_MACNE\nMAPK12\tA0A2K6FJU2_PROCO\nMAPK12\tA0A2K6FJV2_PROCO\nMAPK12\tA0A2K6FJX0_PROCO\nMAPK12\tA0A2K6KAI8_RHIBE\nMAPK12\tA0A2K6KAM8_RHIBE\nMAPK12\tA0A2K6KAR4_RHIBE\nMAPK12\tA0A2K6QN14_RHIRO\nMAPK12\tA0A2K6QN30_RHIRO\nMAPK12\tA0A2K6QN52_RHIRO\nMAPK12\tA0A2K6QN73_RHIRO\nMAPK12\tA0A2K6SM64_SAIBB\nMAPK12\tA0A2K6SM66_SAIBB\nMAPK12\tA0A2K6SM93_SAIBB\nMAPK12\tA0A2R8MZ69_CALJA\nMAPK12\tA0A2R9A7E0_PANPA\nMAPK12\tA0A2R9ACI8_PANPA\nMAPK12\tA0A2U3WLB3_ODORO\nMAPK12\tA0A2U3XVF8_LEPWE\nMAPK12\tA0A2Y9GWC5_NEOSC\nMAPK12\tA0A2Y9H291_NEOSC\nMAPK12\tA0A2Y9Q0H4_DELLE\nMAPK12\tA0A2Y9Q4M9_DELLE\nMAPK12\tA0A2Y9Q525_DELLE\nMAPK12\tA0A2Y9QAT3_DELLE\nMAPK12\tA0A2Y9QAW6_DELLE\nMAPK12\tA0A2Y9T303_PHYMC\nMAPK12\tA0A340X472_LIPVE\nMAPK12\tA0A341ARK2_NEOAA\nMAPK12\tA0A341ATG5_NEOAA\nMAPK12\tA0A341AUG6_NEOAA\nMAPK12\tA0A341AVB6_NEOAA\nMAPK12\tA0A345DKR5_9TELE\nMAPK12\tA0A384D2A0_URSMA\nMAPK12\tA0A3Q0CK56_MESAU\nMAPK12\tA0A3Q0DLH7_TARSY\nMAPK12\tA0A3Q0HHV7_ALLSI\nMAPK12\tA0A3Q0HHW5_ALLSI\nMAPK12\tA0A3Q1LJZ7_BOVIN\nMAPK12\tA0A3Q1LKQ5_BOVIN\nMAPK12\tA0A3Q1MF91_BOVIN\nMAPK12\tA0A3Q2HKN3_HORSE\nMAPK12\tA0A3Q7RAP8_CALUR\nMAPK12\tA0A3Q7TGR0_URSAR\nMAPK12\tA0A3Q7UQU3_VULVU\nMAPK12\tA0A452CM09_BALAS\nMAPK12\tA0A452EVI4_CAPHI\nMAPK12\tA0A452H9R9_9SAUR\nMAPK12\tA0A452RDR5_URSAM\nMAPK12\tA0A452RDS6_URSAM\nMAPK12\tA0A452UWC0_URSMA\nMAPK12\tA0A452UWX1_URSMA\nMAPK12\tA0A455BCB5_PHYMC\nMAPK12\tA0A455BCY9_PHYMC\nMAPK12\tA0A455BFC0_PHYMC\nMAPK12\tA0A4W2CEP3_BOBOX\nMAPK12\tA0A4W2DSD8_BOBOX\nMAPK12\tA0A4W2FFW7_BOBOX\nMAPK12\tA0A4W2HBZ0_BOBOX\nMAPK12\tA0A4W3JHC1_CALMI\nMAPK12\tA0A4X2JRT7_VOMUR\nMAPK12\tA0A4X2JWE8_VOMUR\nMAPK12\tA0A4X2JZ94_VOMUR\nMAPK12\tA0A4Z2FDK1_9TELE\nMAPK12\tA0A4Z2G0V1_9TELE\nMAPK12\tA0A5F4VUE9_CALJA\nMAPK12\tA0A5F4W6Z3_CALJA\nMAPK12\tA0A5F4WF39_CALJA\nMAPK12\tA0A5F7ZRT1_MACMU\nMAPK12\tA0A5F8G1N9_MONDO\nMAPK12\tA0A5F8GEW6_MONDO\nMAPK12\tA0A5F8H2N4_MONDO\nMAPK12\tA0A5F9CL79_RABIT\nMAPK12\tA0A5G3HYR3_XENTR\nMAPK12\tA0A5G3HZ57_XENTR\nMAPK12\tA0A5G3JRJ5_XENTR\nMAPK12\tA0JPR2_RAT\nMAPK12\tA5PJL3_BOVIN\nMAPK12\tA8MY48_HUMAN\nMAPK12\tB5MDL5_HUMAN\nMAPK12\tE2RSB0_CANLF\nMAPK12\tF1NLU9_CHICK\nMAPK12\tF4IH84_ARATH\nMAPK12\tF6SLA9_XENTR\nMAPK12\tF6SX39_HORSE\nMAPK12\tF6ZTS0_MACMU\nMAPK12\tF7A4Q7_XENTR\nMAPK12\tF7BWS0_MONDO\nMAPK12\tF7CEX8_MONDO\nMAPK12\tF7GWH8_CALJA\nMAPK12\tF7H6W8_CALJA\nMAPK12\tF8UFC2_SOLLC\nMAPK12\tG1KA34_ANOCA\nMAPK12\tG1L6R3_AILME\nMAPK12\tG1N7P4_MELGA\nMAPK12\tG1NX89_MYOLU\nMAPK12\tG1QII1_NOMLE\nMAPK12\tG1STA2_RABIT\nMAPK12\tG3THA9_LOXAF\nMAPK12\tG3W6V4_SARHA\nMAPK12\tH0VS06_CAVPO\nMAPK12\tH0XAN2_OTOGA\nMAPK12\tH0Z0V2_TAEGU\nMAPK12\tH2P4V8_PONAB\nMAPK12\tH2RBF0_PANTR\nMAPK12\tH2SIN3_TAKRU\nMAPK12\tH9F710_MACMU\nMAPK12\tI3MPG4_ICTTR\nMAPK12\tI7GJA1_MACFA\nMAPK12\tJ9NSF6_CANLF\nMAPK12\tK7A0D7_PANTR\nMAPK12\tK7G794_PELSI\nMAPK12\tM3WFZ9_FELCA\nMAPK12\tM3XT99_MUSPF\nMAPK12\tMK12_DANRE\nMAPK12\tMK12_HUMAN\nMAPK12\tMK12_MOUSE\nMAPK12\tMK12_RAT\nMAPK12\tQ1LVN7_DANRE\nMAPK12\tQ28G95_XENTR\nMAPK12\tQ5U3S2_DANRE\nMAPK12\tQ66HA3_RAT\nMAPK12\tQ7ZZX6_XENLA\nMAPK12\tQ99M09_MOUSE\nMAPK12\tU3FUI3_CALJA\nMAPK12\tU3I8V7_ANAPP\nMAPK12\tU3K537_FICAL\nMAPK12\tV9L0F6_CALMI\nMAPK12\tW5QEW6_SHEEP\n','From\tTo\nMAPK9\tA0A096NDC4_PAPAN\nMAPK9\tA0A0B2V2U3_TOXCA\nMAPK9\tA0A0B2VN06_TOXCA\nMAPK9\tA0A0B2VXS8_TOXCA\nMAPK9\tA0A0B8RZD8_PIG\nMAPK9\tA0A0D9R7D3_CHLSB\nMAPK9\tA0A0G2JYS4_RAT\nMAPK9\tA0A0P0M032_EPICO\nMAPK9\tA0A151M748_ALLMI\nMAPK9\tA0A151M778_ALLMI\nMAPK9\tA0A1A7WNQ4_9TELE\nMAPK9\tA0A1A7YP94_9TELE\nMAPK9\tA0A1A8AUZ2_NOTFU\nMAPK9\tA0A1A8CXJ7_9TELE\nMAPK9\tA0A1A8FQT1_9TELE\nMAPK9\tA0A1A8GIF3_9TELE\nMAPK9\tA0A1A8IA22_NOTKU\nMAPK9\tA0A1A8Q3A6_9TELE\nMAPK9\tA0A1A8R1B7_9TELE\nMAPK9\tA0A1A8SCL2_9TELE\nMAPK9\tA0A1D5R2Q5_MACMU\nMAPK9\tA0A1S2YKC2_CICAR\nMAPK9\tA0A1S2ZZX2_ERIEU\nMAPK9\tA0A1S2ZZX8_ERIEU\nMAPK9\tA0A1S2ZZY2_ERIEU\nMAPK9\tA0A1S2ZZY9_ERIEU\nMAPK9\tA0A1S3GC59_DIPOR\nMAPK9\tA0A1S3GC81_DIPOR\nMAPK9\tA0A1S3GDV3_DIPOR\nMAPK9\tA0A1S3GEJ9_DIPOR\nMAPK9\tA0A1S3WGR3_ERIEU\nMAPK9\tA0A1S3WGV8_ERIEU\nMAPK9\tA0A1U7Q7K6_MESAU\nMAPK9\tA0A1U7QGR1_MESAU\nMAPK9\tA0A1U7S7G4_ALLSI\nMAPK9\tA0A1U7SMK8_TARSY\nMAPK9\tA0A1U7SU97_TARSY\nMAPK9\tA0A1U7SY66_TARSY\nMAPK9\tA0A1U7T5G7_TARSY\nMAPK9\tA0A1U7T827_TARSY\nMAPK9\tA0A1U8C3R5_MESAU\nMAPK9\tA0A1V4KUH3_PATFA\nMAPK9\tA0A1V4KUP0_PATFA\nMAPK9\tA0A218V0Z0_9PASE\nMAPK9\tA0A250YI26_CASCN\nMAPK9\tA0A286XIA1_CAVPO\nMAPK9\tA0A287APM5_PIG\nMAPK9\tA0A287ASQ5_PIG\nMAPK9\tA0A287DAT2_ICTTR\nMAPK9\tA0A2D0SY20_ICTPU\nMAPK9\tA0A2D0SYG3_ICTPU\nMAPK9\tA0A2I0M9W6_COLLI\nMAPK9\tA0A2I0M9X3_COLLI\nMAPK9\tA0A2I2UNM7_FELCA\nMAPK9\tA0A2I3GJE4_NOMLE\nMAPK9\tA0A2I3GN85_NOMLE\nMAPK9\tA0A2I3HJX0_NOMLE\nMAPK9\tA0A2I3HMT7_NOMLE\nMAPK9\tA0A2I3LWB5_PAPAN\nMAPK9\tA0A2I3M4C8_PAPAN\nMAPK9\tA0A2I3M9T9_PAPAN\nMAPK9\tA0A2I3MW42_PAPAN\nMAPK9\tA0A2I3MX46_PAPAN\nMAPK9\tA0A2I3NB53_PAPAN\nMAPK9\tA0A2I3RB16_PANTR\nMAPK9\tA0A2I3RT61_PANTR\nMAPK9\tA0A2I3S908_PANTR\nMAPK9\tA0A2I3SA64_PANTR\nMAPK9\tA0A2I3SV25_PANTR\nMAPK9\tA0A2I3SYI5_PANTR\nMAPK9\tA0A2I3T5Q4_PANTR\nMAPK9\tA0A2J8JTD9_PANTR\nMAPK9\tA0A2K5CWP5_AOTNA\nMAPK9\tA0A2K5CWQ3_AOTNA\nMAPK9\tA0A2K5CWQ5_AOTNA\nMAPK9\tA0A2K5CWQ9_AOTNA\nMAPK9\tA0A2K5CWR9_AOTNA\nMAPK9\tA0A2K5CWS3_AOTNA\nMAPK9\tA0A2K5I1W0_COLAP\nMAPK9\tA0A2K5I230_COLAP\nMAPK9\tA0A2K5I250_COLAP\nMAPK9\tA0A2K5I264_COLAP\nMAPK9\tA0A2K5I274_COLAP\nMAPK9\tA0A2K5I277_COLAP\nMAPK9\tA0A2K5I278_COLAP\nMAPK9\tA0A2K5I2G9_COLAP\nMAPK9\tA0A2K5M3Q2_CERAT\nMAPK9\tA0A2K5M3T4_CERAT\nMAPK9\tA0A2K5M3W9_CERAT\nMAPK9\tA0A2K5M3Y0_CERAT\nMAPK9\tA0A2K5M3Y2_CERAT\nMAPK9\tA0A2K5M3Y8_CERAT\nMAPK9\tA0A2K5SJC8_CEBCA\nMAPK9\tA0A2K5SJC9_CEBCA\nMAPK9\tA0A2K5SJD0_CEBCA\nMAPK9\tA0A2K5SJD1_CEBCA\nMAPK9\tA0A2K5SJD3_CEBCA\nMAPK9\tA0A2K5SJD6_CEBCA\nMAPK9\tA0A2K5W148_MACFA\nMAPK9\tA0A2K5W155_MACFA\nMAPK9\tA0A2K5W1A8_MACFA\nMAPK9\tA0A2K5W1C8_MACFA\nMAPK9\tA0A2K5W1G6_MACFA\nMAPK9\tA0A2K5W1I5_MACFA\nMAPK9\tA0A2K5XW36_MANLE\nMAPK9\tA0A2K5XW41_MANLE\nMAPK9\tA0A2K5XW83_MANLE\nMAPK9\tA0A2K5XW97_MANLE\nMAPK9\tA0A2K5XW99_MANLE\nMAPK9\tA0A2K5XWA0_MANLE\nMAPK9\tA0A2K5XWA4_MANLE\nMAPK9\tA0A2K5XWB7_MANLE\nMAPK9\tA0A2K6DL14_MACNE\nMAPK9\tA0A2K6DL31_MACNE\nMAPK9\tA0A2K6DL64_MACNE\nMAPK9\tA0A2K6DL74_MACNE\nMAPK9\tA0A2K6EJJ3_PROCO\nMAPK9\tA0A2K6EJK2_PROCO\nMAPK9\tA0A2K6EJK5_PROCO\nMAPK9\tA0A2K6EJK7_PROCO\nMAPK9\tA0A2K6EJK8_PROCO\nMAPK9\tA0A2K6EJL9_PROCO\nMAPK9\tA0A2K6K6Z3_RHIBE\nMAPK9\tA0A2K6K716_RHIBE\nMAPK9\tA0A2K6K721_RHIBE\nMAPK9\tA0A2K6K730_RHIBE\nMAPK9\tA0A2K6K741_RHIBE\nMAPK9\tA0A2K6SE50_SAIBB\nMAPK9\tA0A2K6SEB5_SAIBB\nMAPK9\tA0A2K6SED1_SAIBB\nMAPK9\tA0A2K6SEF6_SAIBB\nMAPK9\tA0A2K6SEI9_SAIBB\nMAPK9\tA0A2K6SEJ6_SAIBB\nMAPK9\tA0A2R8MHE7_CALJA\nMAPK9\tA0A2R9BKG9_PANPA\nMAPK9\tA0A2R9BKH2_PANPA\nMAPK9\tA0A2R9BKL4_PANPA\nMAPK9\tA0A2R9BKL9_PANPA\nMAPK9\tA0A2R9BL65_PANPA\nMAPK9\tA0A2R9BLH2_PANPA\nMAPK9\tA0A2R9BUL2_PANPA\nMAPK9\tA0A2U3WTJ7_ODORO\nMAPK9\tA0A2U3WTK9_ODORO\nMAPK9\tA0A2U3WTU8_ODORO\nMAPK9\tA0A2U3ZT75_ODORO\nMAPK9\tA0A2U3ZT76_ODORO\nMAPK9\tA0A2U3ZT88_ODORO\nMAPK9\tA0A2U3ZU70_ODORO\nMAPK9\tA0A2U4BIF8_TURTR\nMAPK9\tA0A2U4BIH5_TURTR\nMAPK9\tA0A2U4BIL6_TURTR\nMAPK9\tA0A2Y9H207_NEOSC\nMAPK9\tA0A2Y9MA10_DELLE\nMAPK9\tA0A2Y9MBC4_DELLE\nMAPK9\tA0A2Y9MJ04_DELLE\nMAPK9\tA0A2Y9MPD4_DELLE\nMAPK9\tA0A2Y9S0B3_PHYMC\nMAPK9\tA0A2Y9S3T8_PHYMC\nMAPK9\tA0A337S9M5_FELCA\nMAPK9\tA0A337SMN0_FELCA\nMAPK9\tA0A337SNI8_FELCA\nMAPK9\tA0A340XW31_LIPVE\nMAPK9\tA0A340XWJ6_LIPVE\nMAPK9\tA0A340XZA4_LIPVE\nMAPK9\tA0A340Y1Q2_LIPVE\nMAPK9\tA0A340Y2D2_LIPVE\nMAPK9\tA0A341D1C4_NEOAA\nMAPK9\tA0A341D3Z9_NEOAA\nMAPK9\tA0A345DKR2_9TELE\nMAPK9\tA0A383Z422_BALAS\nMAPK9\tA0A383Z4A9_BALAS\nMAPK9\tA0A384DL00_URSMA\nMAPK9\tA0A384DL69_URSMA\nMAPK9\tA0A3P8YGW8_ESOLU\nMAPK9\tA0A3P8YH10_ESOLU\nMAPK9\tA0A3Q0CT47_MESAU\nMAPK9\tA0A3Q0DJL8_TARSY\nMAPK9\tA0A3Q0DNV8_TARSY\nMAPK9\tA0A3Q0DQF6_TARSY\nMAPK9\tA0A3Q0DRR5_TARSY\nMAPK9\tA0A3Q0DSA0_TARSY\nMAPK9\tA0A3Q0GW17_ALLSI\nMAPK9\tA0A3Q0GZM6_ALLSI\nMAPK9\tA0A3Q1LJU7_BOVIN\nMAPK9\tA0A3Q1LVH0_BOVIN\nMAPK9\tA0A3Q2HL84_HORSE\nMAPK9\tA0A3Q2UDZ8_CHICK\nMAPK9\tA0A3Q7ND36_CALUR\nMAPK9\tA0A3Q7NDZ5_CALUR\nMAPK9\tA0A3Q7NE00_CALUR\nMAPK9\tA0A3Q7NIW5_CALUR\nMAPK9\tA0A3Q7NIX1_CALUR\nMAPK9\tA0A3Q7PR19_CALUR\nMAPK9\tA0A3Q7RF89_VULVU\nMAPK9\tA0A3Q7RQ51_VULVU\nMAPK9\tA0A3Q7SQC5_VULVU\nMAPK9\tA0A3Q7SQC9_VULVU\nMAPK9\tA0A3Q7VQV8_URSAR\nMAPK9\tA0A3Q7WEW4_URSAR\nMAPK9\tA0A3Q7WPF3_URSAR\nMAPK9\tA0A3Q7XAM8_URSAR\nMAPK9\tA0A410HYR8_BOSMU\nMAPK9\tA0A452EJ55_CAPHI\nMAPK9\tA0A452EJ75_CAPHI\nMAPK9\tA0A452HMN4_9SAUR\nMAPK9\tA0A452HMU0_9SAUR\nMAPK9\tA0A452S957_URSAM\nMAPK9\tA0A452S991_URSAM\nMAPK9\tA0A452S9D9_URSAM\nMAPK9\tA0A452S9J3_URSAM\nMAPK9\tA0A452S9S1_URSAM\nMAPK9\tA0A452S9U5_URSAM\nMAPK9\tA0A452S9Z1_URSAM\nMAPK9\tA0A452SA98_URSAM\nMAPK9\tA0A452SAA7_URSAM\nMAPK9\tA0A452SAD0_URSAM\nMAPK9\tA0A452VKA4_URSMA\nMAPK9\tA0A455BXV6_PHYMC\nMAPK9\tA0A4W2EVJ1_BOBOX\nMAPK9\tA0A4W2F5L7_BOBOX\nMAPK9\tA0A4W2F5P8_BOBOX\nMAPK9\tA0A4W2F714_BOBOX\nMAPK9\tA0A4W2H719_BOBOX\nMAPK9\tA0A4W2HA79_BOBOX\nMAPK9\tA0A4W2HC85_BOBOX\nMAPK9\tA0A4W3HCS0_CALMI\nMAPK9\tA0A4W3HCS4_CALMI\nMAPK9\tA0A4W3HCX9_CALMI\nMAPK9\tA0A4W3HFS6_CALMI\nMAPK9\tA0A4W3HH68_CALMI\nMAPK9\tA0A4W3HH84_CALMI\nMAPK9\tA0A4W3HHL6_CALMI\nMAPK9\tA0A4W3HI48_CALMI\nMAPK9\tA0A4W3HI79_CALMI\nMAPK9\tA0A4W3HI97_CALMI\nMAPK9\tA0A4X1VQA9_PIG\nMAPK9\tA0A4X1VQB4_PIG\nMAPK9\tA0A4X1VUS0_PIG\nMAPK9\tA0A4X1VVX0_PIG\nMAPK9\tA0A4X1VWV3_PIG\nMAPK9\tA0A4X1VWV8_PIG\nMAPK9\tA0A4X2LPA9_VOMUR\nMAPK9\tA0A4X2LWF5_VOMUR\nMAPK9\tA0A4X2LXX3_VOMUR\nMAPK9\tA0A4Z2EGZ5_9TELE\nMAPK9\tA0A4Z2H075_9TELE\nMAPK9\tA0A5F4CSB9_CANLF\nMAPK9\tA0A5F4CU62_CANLF\nMAPK9\tA0A5F4D9F9_CANLF\nMAPK9\tA0A5F4DD76_CANLF\nMAPK9\tA0A5F4W486_CALJA\nMAPK9\tA0A5F5PRH2_HORSE\nMAPK9\tA0A5F5PZG1_HORSE\nMAPK9\tA0A5F5Q0Q3_HORSE\nMAPK9\tA0A5F8GMR1_MONDO\nMAPK9\tA0A5F8GYR6_MONDO\nMAPK9\tA0A5F8H3W9_MONDO\nMAPK9\tA0A5F9CC57_RABIT\nMAPK9\tA0A5F9CRG3_RABIT\nMAPK9\tA0A5F9DIR5_RABIT\nMAPK9\tA0A5F9DLU8_RABIT\nMAPK9\tA0A5F9DLY2_RABIT\nMAPK9\tA0A5G2QEQ3_PIG\nMAPK9\tA0A5G2QG50_PIG\nMAPK9\tA0A5G2RGU1_PIG\nMAPK9\tA0A5G3HYT8_XENTR\nMAPK9\tB5BUJ7_HUMAN\nMAPK9\tD4A5V8_RAT\nMAPK9\tD7R525_HUMAN\nMAPK9\tD7R526_HUMAN\nMAPK9\tD7R527_HUMAN\nMAPK9\tD7R528_HUMAN\nMAPK9\tE5RJ57_HUMAN\nMAPK9\tE9QE29_DANRE\nMAPK9\tF1P630_CANLF\nMAPK9\tF1S5Q3_PIG\nMAPK9\tF6PKE3_CALJA\nMAPK9\tF6T736_CALJA\nMAPK9\tF6TJP4_XENTR\nMAPK9\tF6WV79_MONDO\nMAPK9\tF6Y132_HORSE\nMAPK9\tF7B5L8_ORNAN\nMAPK9\tF7FHL7_CALJA\nMAPK9\tG1KSZ8_ANOCA\nMAPK9\tG1L207_AILME\nMAPK9\tG1N6G4_MELGA\nMAPK9\tG1P5V8_MYOLU\nMAPK9\tG1QNU5_NOMLE\nMAPK9\tG1T9N9_RABIT\nMAPK9\tG3TDF2_LOXAF\nMAPK9\tG3UFF2_LOXAF\nMAPK9\tG3VT63_SARHA\nMAPK9\tG5C7A6_HETGA\nMAPK9\tG7MUJ7_MACMU\nMAPK9\tH0VPX4_CAVPO\nMAPK9\tH0XGB5_OTOGA\nMAPK9\tH0YSC0_TAEGU\nMAPK9\tH2PHM5_PONAB\nMAPK9\tH2QS62_PANTR\nMAPK9\tH2RKF1_TAKRU\nMAPK9\tH2RKF3_TAKRU\nMAPK9\tH2ZYL7_LATCH\nMAPK9\tH9EMV4_MACMU\nMAPK9\tH9EMV5_MACMU\nMAPK9\tI0FTF4_MACMU\nMAPK9\tI3N0V5_ICTTR\nMAPK9\tJ3KNK1_HUMAN\nMAPK9\tK7ASB9_PANTR\nMAPK9\tK7B4X0_PANTR\nMAPK9\tK7CDN6_PANTR\nMAPK9\tK7GBS7_PELSI\nMAPK9\tK7GBT4_PELSI\nMAPK9\tK7GPL0_PIG\nMAPK9\tM3X688_FELCA\nMAPK9\tM3XND7_MUSPF\nMAPK9\tM3ZCQ6_ICTTR\nMAPK9\tMK09_CHICK\nMAPK9\tMK09_HUMAN\nMAPK9\tMK09_MOUSE\nMAPK9\tMK09_RAT\nMAPK9\tQ2KI72_BOVIN\nMAPK9\tQ5NCK8_MOUSE\nMAPK9\tQ6P727_RAT\nMAPK9\tQ8C094_MOUSE\nMAPK9\tU3CSF4_CALJA\nMAPK9\tU3E7L6_CALJA\nMAPK9\tU3EWQ9_CALJA\nMAPK9\tU3F1Z7_CALJA\nMAPK9\tU3J1C8_ANAPP\nMAPK9\tU3JZ19_FICAL\nMAPK9\tV8P2J9_OPHHA\nMAPK9\tW5NU92_SHEEP\nMAPK9\tW5UDK5_ICTPU\nMAPK9\tW9RFI2_9ROSA\n','From\tTo\nCLK3\tA0A060RUA2_PLARE\nCLK3\tA0A096NMR1_PAPAN\nCLK3\tA0A0D9RP31_CHLSB\nCLK3\tA0A0Y9WRR6_PLABE\nCLK3\tA0A151MAG7_ALLMI\nCLK3\tA0A151MAH1_ALLMI\nCLK3\tA0A1C3KSA4_9APIC\nCLK3\tA0A1C6YCA1_PLACH\nCLK3\tA0A1C6YFC2_PLACH\nCLK3\tA0A1D3RVH9_PLACH\nCLK3\tA0A1D3RVM6_PLACH\nCLK3\tA0A1D3THU6_9APIC\nCLK3\tA0A1J1GUP2_PLAGA\nCLK3\tA0A1J1H5Q1_PLARL\nCLK3\tA0A1L1SQT5_MOUSE\nCLK3\tA0A1L1SQZ6_MOUSE\nCLK3\tA0A1L1SST3_MOUSE\nCLK3\tA0A1L1STP6_MOUSE\nCLK3\tA0A1S3ALF5_ERIEU\nCLK3\tA0A1S3EVG5_DIPOR\nCLK3\tA0A1S3WTN7_ERIEU\nCLK3\tA0A1S3WTT7_ERIEU\nCLK3\tA0A1U7QN84_MESAU\nCLK3\tA0A1U7UF36_TARSY\nCLK3\tA0A1U8BYD3_MESAU\nCLK3\tA0A1V4JDE8_PATFA\nCLK3\tA0A1Y3DP25_PLAKN\nCLK3\tA0A218UA81_9PASE\nCLK3\tA0A250YE36_CASCN\nCLK3\tA0A287B9K6_PIG\nCLK3\tA0A287BLN0_PIG\nCLK3\tA0A287D1Z2_ICTTR\nCLK3\tA0A2I0M6K7_COLLI\nCLK3\tA0A2I2UMG1_FELCA\nCLK3\tA0A2K5DIQ6_AOTNA\nCLK3\tA0A2K5HXX1_COLAP\nCLK3\tA0A2K5HY59_COLAP\nCLK3\tA0A2K5LL45_CERAT\nCLK3\tA0A2K5PT33_CEBCA\nCLK3\tA0A2K5VU65_MACFA\nCLK3\tA0A2K5YNY4_MANLE\nCLK3\tA0A2K6BHJ7_MACNE\nCLK3\tA0A2K6G966_PROCO\nCLK3\tA0A2K6K677_RHIBE\nCLK3\tA0A2K6K6A9_RHIBE\nCLK3\tA0A2K6PLQ0_RHIRO\nCLK3\tA0A2K6PLW5_RHIRO\nCLK3\tA0A2K6SHU4_SAIBB\nCLK3\tA0A2K6SHU7_SAIBB\nCLK3\tA0A2R9BSP0_PANPA\nCLK3\tA0A2U3WGL9_ODORO\nCLK3\tA0A2U3Z105_LEPWE\nCLK3\tA0A2U4CME3_TURTR\nCLK3\tA0A2Y9F2C8_PHYMC\nCLK3\tA0A2Y9F450_PHYMC\nCLK3\tA0A2Y9HCS1_NEOSC\nCLK3\tA0A2Y9HJE4_NEOSC\nCLK3\tA0A2Y9NKU4_DELLE\nCLK3\tA0A2Y9NNY0_DELLE\nCLK3\tA0A2Y9NX74_DELLE\nCLK3\tA0A2Y9P0U8_DELLE\nCLK3\tA0A340WDS1_LIPVE\nCLK3\tA0A341CRK3_NEOAA\nCLK3\tA0A341CUE4_NEOAA\nCLK3\tA0A384BE96_BALAS\nCLK3\tA0A384DIP0_URSMA\nCLK3\tA0A3Q0EI62_TARSY\nCLK3\tA0A3Q0G7P5_ALLSI\nCLK3\tA0A3Q2GYV9_HORSE\nCLK3\tA0A3Q2HDZ8_HORSE\nCLK3\tA0A3Q7Q1X4_CALUR\nCLK3\tA0A3Q7QCU0_CALUR\nCLK3\tA0A3Q7RCX9_VULVU\nCLK3\tA0A3Q7U238_URSAR\nCLK3\tA0A452C6E3_BALAS\nCLK3\tA0A452EKS8_CAPHI\nCLK3\tA0A452EL37_CAPHI\nCLK3\tA0A452EL49_CAPHI\nCLK3\tA0A452IXS2_9SAUR\nCLK3\tA0A452SQS6_URSAM\nCLK3\tA0A452TEM2_URSMA\nCLK3\tA0A452TET0_URSMA\nCLK3\tA0A455BPG2_PHYMC\nCLK3\tA0A455BQ44_PHYMC\nCLK3\tA0A4W2CEI4_BOBOX\nCLK3\tA0A4W2FFG9_BOBOX\nCLK3\tA0A4W2HF42_BOBOX\nCLK3\tA0A4X1TKM3_PIG\nCLK3\tA0A4X1TMN5_PIG\nCLK3\tA0A4X2KUD1_VOMUR\nCLK3\tA0A559M6J1_9HELO\nCLK3\tA0A5F9CD58_RABIT\nCLK3\tA0A5F9DR67_RABIT\nCLK3\tA0A5G3ICV1_XENTR\nCLK3\tA5PKQ3_XENLA\nCLK3\tB3KTV2_HUMAN\nCLK3\tCLK3_BOVIN\nCLK3\tCLK3_HUMAN\nCLK3\tCLK3_MOUSE\nCLK3\tCLK3_RAT\nCLK3\tE2RGH3_CANLF\nCLK3\tF1NLR7_CHICK\nCLK3\tF6UNR0_XENTR\nCLK3\tF7A6Z9_MACMU\nCLK3\tF7B8B3_HORSE\nCLK3\tF7FWT9_CALJA\nCLK3\tF7GKK5_MACMU\nCLK3\tG1LQY8_AILME\nCLK3\tG1MV00_MELGA\nCLK3\tG1PMF2_MYOLU\nCLK3\tG1RWU6_NOMLE\nCLK3\tG1TEB6_RABIT\nCLK3\tG3SRI1_LOXAF\nCLK3\tG3US95_MELGA\nCLK3\tG3WQI1_SARHA\nCLK3\tH0W2B7_CAVPO\nCLK3\tH0XB79_OTOGA\nCLK3\tH2Q9S8_PANTR\nCLK3\tH3B476_LATCH\nCLK3\tH3BNC8_HUMAN\nCLK3\tH3BNQ5_HUMAN\nCLK3\tH3BQG1_HUMAN\nCLK3\tH3BQR7_HUMAN\nCLK3\tH3BRE4_HUMAN\nCLK3\tH3BRK0_HUMAN\nCLK3\tH3BRT8_HUMAN\nCLK3\tH3BRW2_HUMAN\nCLK3\tH3BTW9_HUMAN\nCLK3\tH3BUL5_HUMAN\nCLK3\tH3BV35_HUMAN\nCLK3\tH3BVF8_HUMAN\nCLK3\tH9F9H2_MACMU\nCLK3\tH9GLY2_ANOCA\nCLK3\tI3MLY7_ICTTR\nCLK3\tJ9NTZ5_CANLF\nCLK3\tK7FP38_PELSI\nCLK3\tM3Z0R9_MUSPF\nCLK3\tQ3UWB8_MOUSE\nCLK3\tQ5F445_CHICK\nCLK3\tQ5I092_XENTR\nCLK3\tQ9QXE3_MOUSE\nCLK3\tU3JPS0_FICAL\nCLK3\tU6DKV4_NEOVI\nCLK3\tV8NBT0_OPHHA\nCLK3\tV8P313_OPHHA\nCLK3\tW5P030_SHEEP\n','From\tTo\nCSNK2A1\tA0A087WY74_HUMAN\nCSNK2A1\tA0A0B2UL36_TOXCA\nCSNK2A1\tA0A0B2V5Q9_TOXCA\nCSNK2A1\tA0A0V0SDZ5_9BILA\nCSNK2A1\tA0A0V0U6Q7_9BILA\nCSNK2A1\tA0A0V0VWI8_9BILA\nCSNK2A1\tA0A0V0VWP4_9BILA\nCSNK2A1\tA0A0V0WMT0_9BILA\nCSNK2A1\tA0A0V0WMT7_9BILA\nCSNK2A1\tA0A0V0YEY1_TRIPS\nCSNK2A1\tA0A0V0YF28_TRIPS\nCSNK2A1\tA0A0V0ZT31_9BILA\nCSNK2A1\tA0A0V1BRB9_TRISP\nCSNK2A1\tA0A0V1D5K2_TRIBR\nCSNK2A1\tA0A0V1D5N2_TRIBR\nCSNK2A1\tA0A0V1I5T1_9BILA\nCSNK2A1\tA0A0V1LL41_9BILA\nCSNK2A1\tA0A0V1LLG4_9BILA\nCSNK2A1\tA0A0V1MK36_9BILA\nCSNK2A1\tA0A0V1P8P5_9BILA\nCSNK2A1\tA0A151NX96_ALLMI\nCSNK2A1\tA0A1A7WLT0_9TELE\nCSNK2A1\tA0A1A8D8B3_9TELE\nCSNK2A1\tA0A1A8EQ25_9TELE\nCSNK2A1\tA0A1A8HVM2_NOTKU\nCSNK2A1\tA0A1A8L2S2_9TELE\nCSNK2A1\tA0A1A8PVK4_9TELE\nCSNK2A1\tA0A1A8U7C1_NOTFU\nCSNK2A1\tA0A1D5RL05_MACMU\nCSNK2A1\tA0A1L8EM01_XENLA\nCSNK2A1\tA0A1S2ZQS3_ERIEU\nCSNK2A1\tA0A1S2ZQS6_ERIEU\nCSNK2A1\tA0A1S3EQY8_DIPOR\nCSNK2A1\tA0A1U7RWQ0_ALLSI\nCSNK2A1\tA0A1U7S3L1_ALLSI\nCSNK2A1\tA0A1U7THT6_TARSY\nCSNK2A1\tA0A1V4KKK6_PATFA\nCSNK2A1\tA0A218US37_9PASE\nCSNK2A1\tA0A250YHD6_CASCN\nCSNK2A1\tA0A287B9A8_PIG\nCSNK2A1\tA0A2B4S8L1_STYPI\nCSNK2A1\tA0A2I0LIS4_COLLI\nCSNK2A1\tA0A2I2Z5L5_GORGO\nCSNK2A1\tA0A2I3HY49_NOMLE\nCSNK2A1\tA0A2J8VIA6_PONAB\nCSNK2A1\tA0A2R8PF08_CALJA\nCSNK2A1\tA0A2R8QH30_DANRE\nCSNK2A1\tA0A2R8Y3W6_HUMAN\nCSNK2A1\tA0A2R8Y4D6_HUMAN\nCSNK2A1\tA0A2R8Y4H0_HUMAN\nCSNK2A1\tA0A2R8Y5A0_HUMAN\nCSNK2A1\tA0A2R8Y797_HUMAN\nCSNK2A1\tA0A2R8Y7T1_HUMAN\nCSNK2A1\tA0A2R8YCC9_HUMAN\nCSNK2A1\tA0A2R8YCK2_HUMAN\nCSNK2A1\tA0A2R8YD58_HUMAN\nCSNK2A1\tA0A2R8YDP2_HUMAN\nCSNK2A1\tA0A2R8YDY7_HUMAN\nCSNK2A1\tA0A2R8YEL7_HUMAN\nCSNK2A1\tA0A2R8YEW1_HUMAN\nCSNK2A1\tA0A2R8YF43_HUMAN\nCSNK2A1\tA0A2R8YF47_HUMAN\nCSNK2A1\tA0A2R8YFU2_HUMAN\nCSNK2A1\tA0A2R9AMW1_PANPA\nCSNK2A1\tA0A2R9AMX1_PANPA\nCSNK2A1\tA0A2R9ASF6_PANPA\nCSNK2A1\tA0A2R9AUS7_PANPA\nCSNK2A1\tA0A2U3VEH2_ODORO\nCSNK2A1\tA0A2U3VEH9_ODORO\nCSNK2A1\tA0A2U3ZN53_ODORO\nCSNK2A1\tA0A2U3ZN57_ODORO\nCSNK2A1\tA0A2U4CLU6_TURTR\nCSNK2A1\tA0A2Y9FJJ8_PHYMC\nCSNK2A1\tA0A2Y9GZJ1_NEOSC\nCSNK2A1\tA0A2Y9PZA0_DELLE\nCSNK2A1\tA0A337SK77_FELCA\nCSNK2A1\tA0A340WIK6_LIPVE\nCSNK2A1\tA0A340WIV4_LIPVE\nCSNK2A1\tA0A341B7D0_NEOAA\nCSNK2A1\tA0A384B285_BALAS\nCSNK2A1\tA0A384D347_URSMA\nCSNK2A1\tA0A3Q0CNT5_MESAU\nCSNK2A1\tA0A3Q1LSG3_BOVIN\nCSNK2A1\tA0A3Q2I8V6_HORSE\nCSNK2A1\tA0A3Q7R8S0_VULVU\nCSNK2A1\tA0A3Q7VCZ8_URSAR\nCSNK2A1\tA0A452FP18_CAPHI\nCSNK2A1\tA0A452FP67_CAPHI\nCSNK2A1\tA0A452T074_URSMA\nCSNK2A1\tA0A480PD02_PIG\nCSNK2A1\tA0A4W2CTU6_BOBOX\nCSNK2A1\tA0A4W2CTV4_BOBOX\nCSNK2A1\tA0A4W2CXS5_BOBOX\nCSNK2A1\tA0A4W2CXU5_BOBOX\nCSNK2A1\tA0A4W2CY09_BOBOX\nCSNK2A1\tA0A4W2DDA6_BOBOX\nCSNK2A1\tA0A4W2EDP6_BOBOX\nCSNK2A1\tA0A4W2EDP8_BOBOX\nCSNK2A1\tA0A4W2GSN8_BOBOX\nCSNK2A1\tA0A4W2ICG9_BOBOX\nCSNK2A1\tA0A4W3GNS2_CALMI\nCSNK2A1\tA0A4W3GNT0_CALMI\nCSNK2A1\tA0A4W3GPV4_CALMI\nCSNK2A1\tA0A4W3GPW0_CALMI\nCSNK2A1\tA0A4W3GPW4_CALMI\nCSNK2A1\tA0A4W3GR98_CALMI\nCSNK2A1\tA0A4W3H6I6_CALMI\nCSNK2A1\tA0A4W3H6K3_CALMI\nCSNK2A1\tA0A4W3H6L1_CALMI\nCSNK2A1\tA0A4X1SN68_PIG\nCSNK2A1\tA0A4X1SN72_PIG\nCSNK2A1\tA0A4X1SNX5_PIG\nCSNK2A1\tA0A4X1SPI6_PIG\nCSNK2A1\tA0A4X1SPM6_PIG\nCSNK2A1\tA0A4X1SPN3_PIG\nCSNK2A1\tA0A4X1SPU8_PIG\nCSNK2A1\tA0A4X1SPV6_PIG\nCSNK2A1\tA0A4Z2HBJ6_9TELE\nCSNK2A1\tA0A4Z2JDE5_9TELE\nCSNK2A1\tA0A5F4C2C4_CANLF\nCSNK2A1\tA0A5F4CFN2_CANLF\nCSNK2A1\tA0A5F4CKD6_CANLF\nCSNK2A1\tA0A5F4D0I1_CANLF\nCSNK2A1\tA0A5F4W580_CALJA\nCSNK2A1\tA0A5F4W8H3_CALJA\nCSNK2A1\tA0A5F4WA47_CALJA\nCSNK2A1\tA0A5F5XFC2_FELCA\nCSNK2A1\tA0A5F5XV15_FELCA\nCSNK2A1\tA0A5F5Y5U4_FELCA\nCSNK2A1\tA0A5F9CDI0_RABIT\nCSNK2A1\tA0A5F9CE99_RABIT\nCSNK2A1\tA0A5F9D4P0_RABIT\nCSNK2A1\tA0A5F9DBL1_RABIT\nCSNK2A1\tA0A5G2QHC2_PIG\nCSNK2A1\tA0A5G2QMN7_PIG\nCSNK2A1\tA0A5G2QPT4_PIG\nCSNK2A1\tA0A5G2QZG9_PIG\nCSNK2A1\tA0A5G2R4G4_PIG\nCSNK2A1\tA0A5G2R5H4_PIG\nCSNK2A1\tA0A5G2RLX3_PIG\nCSNK2A1\tA0A5G3KB47_XENTR\nCSNK2A1\tA2ANR6_MOUSE\nCSNK2A1\tB5BUH5_HUMAN\nCSNK2A1\tCSK21_BOVIN\nCSNK2A1\tCSK21_CHICK\nCSNK2A1\tCSK21_HUMAN\nCSNK2A1\tCSK21_MOUSE\nCSNK2A1\tCSK21_RABIT\nCSNK2A1\tCSK21_RAT\nCSNK2A1\tCSK21_XENLA\nCSNK2A1\tE2RHZ1_CANLF\nCSNK2A1\tE7EU96_HUMAN\nCSNK2A1\tE9QCX0_DANRE\nCSNK2A1\tE9QHH7_DANRE\nCSNK2A1\tE9QIP1_DANRE\nCSNK2A1\tF2Z5I5_PIG\nCSNK2A1\tF7BXZ1_HORSE\nCSNK2A1\tF7BZR6_XENTR\nCSNK2A1\tF7CKT2_MACMU\nCSNK2A1\tF7I6N8_CALJA\nCSNK2A1\tF7I6P0_CALJA\nCSNK2A1\tF7I6P4_CALJA\nCSNK2A1\tF7I6P7_CALJA\nCSNK2A1\tG1N6J4_MELGA\nCSNK2A1\tG1PF23_MYOLU\nCSNK2A1\tG1RE63_NOMLE\nCSNK2A1\tG1TQ31_RABIT\nCSNK2A1\tG3R0V5_GORGO\nCSNK2A1\tG3RVT9_GORGO\nCSNK2A1\tG3SRS0_LOXAF\nCSNK2A1\tG3VX45_SARHA\nCSNK2A1\tH0V659_CAVPO\nCSNK2A1\tH0Z8S4_TAEGU\nCSNK2A1\tH9GHI1_ANOCA\nCSNK2A1\tI3MC13_ICTTR\nCSNK2A1\tK7D8P4_PANTR\nCSNK2A1\tK9K2S9_HORSE\nCSNK2A1\tM3W025_FELCA\nCSNK2A1\tQ5U5J2_HUMAN\nCSNK2A1\tQ6INV5_XENLA\nCSNK2A1\tQ6NSS6_MOUSE\nCSNK2A1\tQ803N7_DANRE\nCSNK2A1\tQ91397_DANRE\nCSNK2A1\tT2MEH3_HYDVU\nCSNK2A1\tU3JT45_FICAL\nCSNK2A1\tV9GY80_HUMAN\nCSNK2A1\tV9GYA2_HUMAN\nCSNK2A1\tV9GYW6_HUMAN\nCSNK2A1\tV9KVR3_CALMI\nCSNK2A1\tW5QCL5_SHEEP\nCSNK2A1\tW5QCL6_SHEEP\n','From\tTo\nCSNK2A2\tA0A087XZ89_POEFO\nCSNK2A2\tA0A096NJ03_PAPAN\nCSNK2A2\tA0A096NJ04_PAPAN\nCSNK2A2\tA0A0B2VHW2_TOXCA\nCSNK2A2\tA0A0D9QXL1_CHLSB\nCSNK2A2\tA0A0P6J1S2_HETGA\nCSNK2A2\tA0A1D5RLE4_MOUSE\nCSNK2A2\tA0A1D5RLQ2_MOUSE\nCSNK2A2\tA0A1D5RM55_MOUSE\nCSNK2A2\tA0A1D5RM74_MOUSE\nCSNK2A2\tA0A1S2ZJ52_ERIEU\nCSNK2A2\tA0A1S3EWQ3_DIPOR\nCSNK2A2\tA0A1S3KML5_SALSA\nCSNK2A2\tA0A1S3KNE3_SALSA\nCSNK2A2\tA0A1U7U1S6_TARSY\nCSNK2A2\tA0A1U8C630_MESAU\nCSNK2A2\tA0A1U8D7X0_ALLSI\nCSNK2A2\tA0A1U8D9G9_ALLSI\nCSNK2A2\tA0A1V4K0M0_PATFA\nCSNK2A2\tA0A250YG90_CASCN\nCSNK2A2\tA0A286Y1G3_CAVPO\nCSNK2A2\tA0A287B496_PIG\nCSNK2A2\tA0A287BBE8_PIG\nCSNK2A2\tA0A287DCK1_ICTTR\nCSNK2A2\tA0A2D0RIU7_ICTPU\nCSNK2A2\tA0A2D0RJY5_ICTPU\nCSNK2A2\tA0A2H4NBS8_TOXLE\nCSNK2A2\tA0A2H4NBU4_TOXRE\nCSNK2A2\tA0A2H4NBV2_TOXCR\nCSNK2A2\tA0A2I0M7Q9_COLLI\nCSNK2A2\tA0A2I0M7R1_COLLI\nCSNK2A2\tA0A2I2YPY9_GORGO\nCSNK2A2\tA0A2I2ZUG9_GORGO\nCSNK2A2\tA0A2K5F095_AOTNA\nCSNK2A2\tA0A2K5F0B1_AOTNA\nCSNK2A2\tA0A2K5F0C5_AOTNA\nCSNK2A2\tA0A2K5IXE4_COLAP\nCSNK2A2\tA0A2K5IXI9_COLAP\nCSNK2A2\tA0A2K5L3Z6_CERAT\nCSNK2A2\tA0A2K5L425_CERAT\nCSNK2A2\tA0A2K5QNJ1_CEBCA\nCSNK2A2\tA0A2K5QNK5_CEBCA\nCSNK2A2\tA0A2K5VDS2_MACFA\nCSNK2A2\tA0A2K5XTS2_MANLE\nCSNK2A2\tA0A2K5XTX7_MANLE\nCSNK2A2\tA0A2K6B6H9_MACNE\nCSNK2A2\tA0A2K6B6P0_MACNE\nCSNK2A2\tA0A2K6B6P4_MACNE\nCSNK2A2\tA0A2K6B6U5_MACNE\nCSNK2A2\tA0A2K6GFB2_PROCO\nCSNK2A2\tA0A2K6GFC5_PROCO\nCSNK2A2\tA0A2K6KLR6_RHIBE\nCSNK2A2\tA0A2K6PG97_RHIRO\nCSNK2A2\tA0A2K6PGA1_RHIRO\nCSNK2A2\tA0A2K6PGI2_RHIRO\nCSNK2A2\tA0A2K6TJ25_SAIBB\nCSNK2A2\tA0A2K6TJ34_SAIBB\nCSNK2A2\tA0A2R9BZ77_PANPA\nCSNK2A2\tA0A2U3VLT7_ODORO\nCSNK2A2\tA0A2U4BDK6_TURTR\nCSNK2A2\tA0A2U4BDY8_TURTR\nCSNK2A2\tA0A2Y9IG88_NEOSC\nCSNK2A2\tA0A2Y9LQ77_DELLE\nCSNK2A2\tA0A2Y9STA9_PHYMC\nCSNK2A2\tA0A340Y3S4_LIPVE\nCSNK2A2\tA0A341BN58_NEOAA\nCSNK2A2\tA0A383YY34_BALAS\nCSNK2A2\tA0A384D1B8_URSMA\nCSNK2A2\tA0A3B3DGT6_ORYME\nCSNK2A2\tA0A3B3DGU1_ORYME\nCSNK2A2\tA0A3B3I7K1_ORYLA\nCSNK2A2\tA0A3B3U3K8_9TELE\nCSNK2A2\tA0A3B3U3M5_9TELE\nCSNK2A2\tA0A3B3WQM0_9TELE\nCSNK2A2\tA0A3B3WQS1_9TELE\nCSNK2A2\tA0A3B4AYT2_9GOBI\nCSNK2A2\tA0A3B5KHZ1_TAKRU\nCSNK2A2\tA0A3B5MV58_9TELE\nCSNK2A2\tA0A3B5QML5_XIPMA\nCSNK2A2\tA0A3B5RB80_XIPMA\nCSNK2A2\tA0A3P8W2E2_CYNSE\nCSNK2A2\tA0A3P9MWD4_POERE\nCSNK2A2\tA0A3Q0ECB7_TARSY\nCSNK2A2\tA0A3Q0G779_ALLSI\nCSNK2A2\tA0A3Q0GAR2_ALLSI\nCSNK2A2\tA0A3Q1M4Z6_BOVIN\nCSNK2A2\tA0A3Q1M760_BOVIN\nCSNK2A2\tA0A3Q2CQ58_CYPVA\nCSNK2A2\tA0A3Q2CQ65_CYPVA\nCSNK2A2\tA0A3Q2U5L2_CHICK\nCSNK2A2\tA0A3Q7TCV4_VULVU\nCSNK2A2\tA0A3Q7UPX2_URSAR\nCSNK2A2\tA0A3Q7VEV2_URSAR\nCSNK2A2\tA0A3Q7VNS1_URSAR\nCSNK2A2\tA0A452FFM4_CAPHI\nCSNK2A2\tA0A452ITR5_9SAUR\nCSNK2A2\tA0A452ITR8_9SAUR\nCSNK2A2\tA0A452RQ40_URSAM\nCSNK2A2\tA0A452RQ45_URSAM\nCSNK2A2\tA0A452RQ56_URSAM\nCSNK2A2\tA0A452RQ64_URSAM\nCSNK2A2\tA0A452V7E0_URSMA\nCSNK2A2\tA0A452V7F5_URSMA\nCSNK2A2\tA0A452V7F7_URSMA\nCSNK2A2\tA0A493U2H8_ANAPP\nCSNK2A2\tA0A4W2C9K6_BOBOX\nCSNK2A2\tA0A4W2CCA7_BOBOX\nCSNK2A2\tA0A4W2GCR6_BOBOX\nCSNK2A2\tA0A4W2HZ52_BOBOX\nCSNK2A2\tA0A4W2HZ62_BOBOX\nCSNK2A2\tA0A4W3H5N7_CALMI\nCSNK2A2\tA0A4W3H5S3_CALMI\nCSNK2A2\tA0A4W3H9V6_CALMI\nCSNK2A2\tA0A4W3H9X0_CALMI\nCSNK2A2\tA0A4W3H9X5_CALMI\nCSNK2A2\tA0A4W3HAG9_CALMI\nCSNK2A2\tA0A4W3HAH5_CALMI\nCSNK2A2\tA0A4W3HSK5_CALMI\nCSNK2A2\tA0A4W4GAB2_ELEEL\nCSNK2A2\tA0A4W4GAB7_ELEEL\nCSNK2A2\tA0A4W4GBL3_ELEEL\nCSNK2A2\tA0A4W4GEC7_ELEEL\nCSNK2A2\tA0A4W4GED2_ELEEL\nCSNK2A2\tA0A4W4GEZ0_ELEEL\nCSNK2A2\tA0A4W6EG26_LATCA\nCSNK2A2\tA0A4W6EG64_LATCA\nCSNK2A2\tA0A4W6EIC8_LATCA\nCSNK2A2\tA0A4W6EJE3_LATCA\nCSNK2A2\tA0A4W6EJI4_LATCA\nCSNK2A2\tA0A4W6EK54_LATCA\nCSNK2A2\tA0A4X1UU40_PIG\nCSNK2A2\tA0A4X1UVZ5_PIG\nCSNK2A2\tA0A4X1UW00_PIG\nCSNK2A2\tA0A4X1UYZ3_PIG\nCSNK2A2\tA0A4X2M906_VOMUR\nCSNK2A2\tA0A4X2MDK6_VOMUR\nCSNK2A2\tA0A4X2MF35_VOMUR\nCSNK2A2\tA0A4X2MFX4_VOMUR\nCSNK2A2\tA0A4Z2GN09_9TELE\nCSNK2A2\tA0A5F4C0P0_CANLF\nCSNK2A2\tA0A5F4CP97_CANLF\nCSNK2A2\tA0A5F4CY05_CANLF\nCSNK2A2\tA0A5F4D444_CANLF\nCSNK2A2\tA0A5F5PK10_HORSE\nCSNK2A2\tA0A5F5PNM4_HORSE\nCSNK2A2\tA0A5F8G4H7_MONDO\nCSNK2A2\tA0A5F8G710_MONDO\nCSNK2A2\tA0A5F8G8X3_MONDO\nCSNK2A2\tA0A5F8H3Z5_MONDO\nCSNK2A2\tA0A5F8HHF6_MONDO\nCSNK2A2\tA0A5G3H9K9_XENTR\nCSNK2A2\tA0A5G3H9P5_XENTR\nCSNK2A2\tA0A5G3ITL6_XENTR\nCSNK2A2\tA0A5G3JXS1_XENTR\nCSNK2A2\tB4F7A9_RAT\nCSNK2A2\tCSK22_BOVIN\nCSNK2A2\tCSK22_HUMAN\nCSNK2A2\tCSK22_MOUSE\nCSNK2A2\tF1MJ55_BOVIN\nCSNK2A2\tF1P666_CANLF\nCSNK2A2\tF1RFS8_PIG\nCSNK2A2\tF6R0E2_CALJA\nCSNK2A2\tF6R511_MONDO\nCSNK2A2\tF6RN85_XENTR\nCSNK2A2\tF6RPY1_XENTR\nCSNK2A2\tF6ZR68_HORSE\nCSNK2A2\tF7AE43_XENTR\nCSNK2A2\tF7AE52_XENTR\nCSNK2A2\tF7CDU8_XENTR\nCSNK2A2\tF7F9Q9_ORNAN\nCSNK2A2\tG1MX47_MELGA\nCSNK2A2\tG1NSY2_MYOLU\nCSNK2A2\tG1QT51_NOMLE\nCSNK2A2\tG1SN56_RABIT\nCSNK2A2\tG3RVD0_GORGO\nCSNK2A2\tG3TBH0_LOXAF\nCSNK2A2\tG3VL09_SARHA\nCSNK2A2\tG3VL10_SARHA\nCSNK2A2\tH0VCD1_CAVPO\nCSNK2A2\tH0WMG3_OTOGA\nCSNK2A2\tH0Z2U6_TAEGU\nCSNK2A2\tH2MAQ1_ORYLA\nCSNK2A2\tH2NR24_PONAB\nCSNK2A2\tH2T3W7_TAKRU\nCSNK2A2\tH3A2F8_LATCH\nCSNK2A2\tH3A2F9_LATCH\nCSNK2A2\tH3BNI9_HUMAN\nCSNK2A2\tH3BSA1_HUMAN\nCSNK2A2\tH3BV19_HUMAN\nCSNK2A2\tH9ENR6_MACMU\nCSNK2A2\tH9GDW6_ANOCA\nCSNK2A2\tI3JYH5_ORENI\nCSNK2A2\tI3M2G9_ICTTR\nCSNK2A2\tK4H9K7_9SAUR\nCSNK2A2\tK4HAZ9_9SAUR\nCSNK2A2\tK4HDC5_9SAUR\nCSNK2A2\tK4HFF4_MALTE\nCSNK2A2\tK7CVS9_PANTR\nCSNK2A2\tK7EVT1_PONAB\nCSNK2A2\tK7F9Q4_PELSI\nCSNK2A2\tK9IVE3_PIG\nCSNK2A2\tM3WZB7_FELCA\nCSNK2A2\tM3Y025_MUSPF\nCSNK2A2\tQ28BY4_XENTR\nCSNK2A2\tQ545V8_MOUSE\nCSNK2A2\tQ66JC2_XENTR\nCSNK2A2\tQ6INF7_XENLA\nCSNK2A2\tQ8BPF9_MOUSE\nCSNK2A2\tQ8CDH5_MOUSE\nCSNK2A2\tQ9TVB6_BOVIN\nCSNK2A2\tQ9XSK3_BOVIN\nCSNK2A2\tU3J9H3_ANAPP\nCSNK2A2\tU3JWL2_FICAL\nCSNK2A2\tU3JWL6_FICAL\nCSNK2A2\tW5NT21_SHEEP\nCSNK2A2\tW5UA04_ICTPU\nCSNK2A2\tW5UL57_ICTPU\n','From\tTo\nMAPK8\tA0A076V475_EPICO\nMAPK8\tA0A096P4B3_PAPAN\nMAPK8\tA0A0B2UXN5_TOXCA\nMAPK8\tA0A0D9RCP6_CHLSB\nMAPK8\tA0A0F6QLD1_CLABA\nMAPK8\tA0A0G2KA63_RAT\nMAPK8\tA0A0P6K1T2_HETGA\nMAPK8\tA0A1D5PYH8_CHICK\nMAPK8\tA0A1L8FFC8_XENLA\nMAPK8\tA0A1L8FKT1_XENLA\nMAPK8\tA0A1S2YI47_CICAR\nMAPK8\tA0A1S2ZYD9_ERIEU\nMAPK8\tA0A1S2ZYE2_ERIEU\nMAPK8\tA0A1S2ZYE3_ERIEU\nMAPK8\tA0A1S2ZYF0_ERIEU\nMAPK8\tA0A1S3GJZ0_DIPOR\nMAPK8\tA0A1S3GJZ4_DIPOR\nMAPK8\tA0A1S3GK70_DIPOR\nMAPK8\tA0A1S3GMJ3_DIPOR\nMAPK8\tA0A1S3RPL2_SALSA\nMAPK8\tA0A1S3RPM6_SALSA\nMAPK8\tA0A1S3RR11_SALSA\nMAPK8\tA0A1U7QZQ1_ALLSI\nMAPK8\tA0A1U7R7K6_ALLSI\nMAPK8\tA0A1U7RB51_ALLSI\nMAPK8\tA0A1U7UES5_TARSY\nMAPK8\tA0A1U7URY8_TARSY\nMAPK8\tA0A1U7URZ3_TARSY\nMAPK8\tA0A1U7UTD7_TARSY\nMAPK8\tA0A1U8C4B3_MESAU\nMAPK8\tA0A1U8CEP0_MESAU\nMAPK8\tA0A1V4KIX6_PATFA\nMAPK8\tA0A1V4KJ16_PATFA\nMAPK8\tA0A1V4KJ66_PATFA\nMAPK8\tA0A1V4KJ95_PATFA\nMAPK8\tA0A218UWN8_9PASE\nMAPK8\tA0A250XZV0_CASCN\nMAPK8\tA0A286XG70_CAVPO\nMAPK8\tA0A286Y3Q4_CAVPO\nMAPK8\tA0A287BES1_PIG\nMAPK8\tA0A287D610_ICTTR\nMAPK8\tA0A287DDY1_ICTTR\nMAPK8\tA0A2I0LY33_COLLI\nMAPK8\tA0A2I0LY34_COLLI\nMAPK8\tA0A2I0LY37_COLLI\nMAPK8\tA0A2I2UA34_FELCA\nMAPK8\tA0A2I2V255_FELCA\nMAPK8\tA0A2I3BPL3_MOUSE\nMAPK8\tA0A2I3G2W1_NOMLE\nMAPK8\tA0A2I3GE11_NOMLE\nMAPK8\tA0A2I3H7H0_NOMLE\nMAPK8\tA0A2I3HR66_NOMLE\nMAPK8\tA0A2I3M2E7_PAPAN\nMAPK8\tA0A2I3MDC5_PAPAN\nMAPK8\tA0A2I3NG67_PAPAN\nMAPK8\tA0A2I3SY17_PANTR\nMAPK8\tA0A2I3TK81_PANTR\nMAPK8\tA0A2I3TKI6_PANTR\nMAPK8\tA0A2I4C2D3_9TELE\nMAPK8\tA0A2K5C6G5_AOTNA\nMAPK8\tA0A2K5C6I9_AOTNA\nMAPK8\tA0A2K5C6M8_AOTNA\nMAPK8\tA0A2K5C6N8_AOTNA\nMAPK8\tA0A2K5IUP1_COLAP\nMAPK8\tA0A2K5IUP2_COLAP\nMAPK8\tA0A2K5IV20_COLAP\nMAPK8\tA0A2K5IV51_COLAP\nMAPK8\tA0A2K5KNP2_CERAT\nMAPK8\tA0A2K5KNR7_CERAT\nMAPK8\tA0A2K5KNU6_CERAT\nMAPK8\tA0A2K5KNV5_CERAT\nMAPK8\tA0A2K5S2U3_CEBCA\nMAPK8\tA0A2K5S2Y5_CEBCA\nMAPK8\tA0A2K5S2Y6_CEBCA\nMAPK8\tA0A2K5S309_CEBCA\nMAPK8\tA0A2K5S312_CEBCA\nMAPK8\tA0A2K5WL44_MACFA\nMAPK8\tA0A2K5WLB6_MACFA\nMAPK8\tA0A2K5WLC2_MACFA\nMAPK8\tA0A2K5YJB3_MANLE\nMAPK8\tA0A2K5YJC6_MANLE\nMAPK8\tA0A2K5YJE5_MANLE\nMAPK8\tA0A2K5YJI5_MANLE\nMAPK8\tA0A2K6BNI6_MACNE\nMAPK8\tA0A2K6BNK1_MACNE\nMAPK8\tA0A2K6BNN9_MACNE\nMAPK8\tA0A2K6BNU2_MACNE\nMAPK8\tA0A2K6GQ10_PROCO\nMAPK8\tA0A2K6GQ24_PROCO\nMAPK8\tA0A2K6GQ28_PROCO\nMAPK8\tA0A2K6GQ33_PROCO\nMAPK8\tA0A2K6M6S5_RHIBE\nMAPK8\tA0A2K6M6S7_RHIBE\nMAPK8\tA0A2K6M6S9_RHIBE\nMAPK8\tA0A2K6NJF2_RHIRO\nMAPK8\tA0A2K6NJG3_RHIRO\nMAPK8\tA0A2K6NJG4_RHIRO\nMAPK8\tA0A2K6NJG9_RHIRO\nMAPK8\tA0A2K6V0A6_SAIBB\nMAPK8\tA0A2K6V0D3_SAIBB\nMAPK8\tA0A2K6V0F8_SAIBB\nMAPK8\tA0A2K6V0H4_SAIBB\nMAPK8\tA0A2R8MUK9_CALJA\nMAPK8\tA0A2R8Z8K5_PANPA\nMAPK8\tA0A2R8Z8K6_PANPA\nMAPK8\tA0A2R8Z8R8_PANPA\nMAPK8\tA0A2R8ZCM1_PANPA\nMAPK8\tA0A2U3WIK7_ODORO\nMAPK8\tA0A2U3WIN8_ODORO\nMAPK8\tA0A2U3Y3J1_LEPWE\nMAPK8\tA0A2U3Y3J3_LEPWE\nMAPK8\tA0A2U3Y3K3_LEPWE\nMAPK8\tA0A2U3Y3V4_LEPWE\nMAPK8\tA0A2U3ZP63_ODORO\nMAPK8\tA0A2U3ZP64_ODORO\nMAPK8\tA0A2U3ZQ99_ODORO\nMAPK8\tA0A2U4AJW4_TURTR\nMAPK8\tA0A2U4AK28_TURTR\nMAPK8\tA0A2U4AKF0_TURTR\nMAPK8\tA0A2Y9EPS4_PHYMC\nMAPK8\tA0A2Y9HWG9_NEOSC\nMAPK8\tA0A2Y9MDB2_DELLE\nMAPK8\tA0A2Y9MEH5_DELLE\nMAPK8\tA0A2Y9MMF5_DELLE\nMAPK8\tA0A2Y9MSU8_DELLE\nMAPK8\tA0A337S0H7_FELCA\nMAPK8\tA0A340WA22_LIPVE\nMAPK8\tA0A340WCL1_LIPVE\nMAPK8\tA0A340WFE4_LIPVE\nMAPK8\tA0A340WGJ4_LIPVE\nMAPK8\tA0A340WGX8_LIPVE\nMAPK8\tA0A341AWF8_NEOAA\nMAPK8\tA0A341AWG0_NEOAA\nMAPK8\tA0A345DKR1_9TELE\nMAPK8\tA0A383ZUI6_BALAS\nMAPK8\tA0A383ZUI9_BALAS\nMAPK8\tA0A383ZUK3_BALAS\nMAPK8\tA0A383ZUQ4_BALAS\nMAPK8\tA0A384CHN5_URSMA\nMAPK8\tA0A384CHV2_URSMA\nMAPK8\tA0A384CIS2_URSMA\nMAPK8\tA0A3B3IRW7_HUMAN\nMAPK8\tA0A3B5KDV2_TAKRU\nMAPK8\tA0A3G2LZC1_CTEID\nMAPK8\tA0A3Q0CYE2_MESAU\nMAPK8\tA0A3Q0EE82_TARSY\nMAPK8\tA0A3Q0EE88_TARSY\nMAPK8\tA0A3Q0EFR7_TARSY\nMAPK8\tA0A3Q0EHH7_TARSY\nMAPK8\tA0A3Q0EJH3_TARSY\nMAPK8\tA0A3Q0EJY2_TARSY\nMAPK8\tA0A3Q0EJY7_TARSY\nMAPK8\tA0A3Q1LWJ8_BOVIN\nMAPK8\tA0A3Q1MG18_BOVIN\nMAPK8\tA0A3Q1NK45_BOVIN\nMAPK8\tA0A3Q2IJQ1_HORSE\nMAPK8\tA0A3Q2TY19_CHICK\nMAPK8\tA0A3Q7PSI1_CALUR\nMAPK8\tA0A3Q7PT26_CALUR\nMAPK8\tA0A3Q7Q3H8_CALUR\nMAPK8\tA0A3Q7RI57_VULVU\nMAPK8\tA0A3Q7RTM1_VULVU\nMAPK8\tA0A3Q7ST67_VULVU\nMAPK8\tA0A3Q7TM44_URSAR\nMAPK8\tA0A3Q7UMF3_URSAR\nMAPK8\tA0A3Q7VHC3_URSAR\nMAPK8\tA0A410HYN2_BOSMU\nMAPK8\tA0A452G3A1_CAPHI\nMAPK8\tA0A452G3D6_CAPHI\nMAPK8\tA0A452IW41_9SAUR\nMAPK8\tA0A452IW60_9SAUR\nMAPK8\tA0A452QUI3_URSAM\nMAPK8\tA0A452QUJ4_URSAM\nMAPK8\tA0A452QUQ1_URSAM\nMAPK8\tA0A452QUQ5_URSAM\nMAPK8\tA0A452QUR7_URSAM\nMAPK8\tA0A452QUS9_URSAM\nMAPK8\tA0A452QUV4_URSAM\nMAPK8\tA0A452QUW2_URSAM\nMAPK8\tA0A452QUX6_URSAM\nMAPK8\tA0A452UKB8_URSMA\nMAPK8\tA0A455AMA8_PHYMC\nMAPK8\tA0A455AP52_PHYMC\nMAPK8\tA0A493SY79_ANAPP\nMAPK8\tA0A4W2D0C3_BOBOX\nMAPK8\tA0A4W2D0G8_BOBOX\nMAPK8\tA0A4W2D4V0_BOBOX\nMAPK8\tA0A4W2EQI0_BOBOX\nMAPK8\tA0A4W2IQ39_BOBOX\nMAPK8\tA0A4W3GU13_CALMI\nMAPK8\tA0A4W3GVP0_CALMI\nMAPK8\tA0A4W3GX98_CALMI\nMAPK8\tA0A4W3GXA2_CALMI\nMAPK8\tA0A4W3GXF0_CALMI\nMAPK8\tA0A4W3GXJ0_CALMI\nMAPK8\tA0A4W3GXM5_CALMI\nMAPK8\tA0A4W3HCY6_CALMI\nMAPK8\tA0A4W3HD85_CALMI\nMAPK8\tA0A4W3HD89_CALMI\nMAPK8\tA0A4W6C1Y5_LATCA\nMAPK8\tA0A4W6C228_LATCA\nMAPK8\tA0A4W6C261_LATCA\nMAPK8\tA0A4W6C2L4_LATCA\nMAPK8\tA0A4W6C7R4_LATCA\nMAPK8\tA0A4W6C7T5_LATCA\nMAPK8\tA0A4W6C7U3_LATCA\nMAPK8\tA0A4W6C7Z8_LATCA\nMAPK8\tA0A4W6C990_LATCA\nMAPK8\tA0A4W6C9B2_LATCA\nMAPK8\tA0A4X1VFH8_PIG\nMAPK8\tA0A4X1VFI4_PIG\nMAPK8\tA0A4X1VFI9_PIG\nMAPK8\tA0A4X1VFJ9_PIG\nMAPK8\tA0A4X1VGK1_PIG\nMAPK8\tA0A4X2L7R1_VOMUR\nMAPK8\tA0A4X2LAA2_VOMUR\nMAPK8\tA0A5B7FP95_PORTR\nMAPK8\tA0A5B7HXD2_PORTR\nMAPK8\tA0A5F4C144_CANLF\nMAPK8\tA0A5F4CIV7_CANLF\nMAPK8\tA0A5F4D520_CANLF\nMAPK8\tA0A5F5Q3Z9_HORSE\nMAPK8\tA0A5F8GRP2_MONDO\nMAPK8\tA0A5F9CJJ4_RABIT\nMAPK8\tA0A5F9CLK5_RABIT\nMAPK8\tA0A5F9CXP6_RABIT\nMAPK8\tA0A5F9DFZ3_RABIT\nMAPK8\tA0A5G2Q9X6_PIG\nMAPK8\tA0A5G2QHK3_PIG\nMAPK8\tA0A5G3KHD7_XENTR\nMAPK8\tA0A5G3KHW5_XENTR\nMAPK8\tA0A5G3KI50_XENTR\nMAPK8\tA0A5G3KIF0_XENTR\nMAPK8\tA0A5G3KIZ1_XENTR\nMAPK8\tA0A5G3KJ70_XENTR\nMAPK8\tA0A5K1UJA7_CALJA\nMAPK8\tA1L4K2_HUMAN\nMAPK8\tA3KBF5_MOUSE\nMAPK8\tA6P3E4_MOUSE\nMAPK8\tB3DL24_XENTR\nMAPK8\tB5BUB8_HUMAN\nMAPK8\tC9J762_HUMAN\nMAPK8\tC9JWQ4_HUMAN\nMAPK8\tD2HR13_AILME\nMAPK8\tD3Z1Z4_MOUSE\nMAPK8\tD8U9G8_VOLCA\nMAPK8\tD9IV43_SOLLC\nMAPK8\tE1C1H4_CHICK\nMAPK8\tF1LP66_RAT\nMAPK8\tF1MM73_BOVIN\nMAPK8\tF1SEL1_PIG\nMAPK8\tF6Q7Z1_CALJA\nMAPK8\tF6TKL3_XENTR\nMAPK8\tF6UBN1_HORSE\nMAPK8\tF6W6A7_XENTR\nMAPK8\tF6Y4G8_CANLF\nMAPK8\tF6ZKB3_MACMU\nMAPK8\tF6ZKC0_MACMU\nMAPK8\tF7A6Z8_XENTR\nMAPK8\tF7A736_XENTR\nMAPK8\tF7AJT6_CALJA\nMAPK8\tF7AK86_CALJA\nMAPK8\tF7BH28_XENTR\nMAPK8\tF7GE32_MONDO\nMAPK8\tG1DG63_CAPHI\nMAPK8\tG1DG64_CAPHI\nMAPK8\tG1N7L8_MELGA\nMAPK8\tG1PQ88_MYOLU\nMAPK8\tG1TPJ7_RABIT\nMAPK8\tG3U9Y7_LOXAF\nMAPK8\tG3UEK5_LOXAF\nMAPK8\tG3WZ65_SARHA\nMAPK8\tG3WZ66_SARHA\nMAPK8\tG3X8U9_MOUSE\nMAPK8\tG7PEU4_MACFA\nMAPK8\tH0VTN4_CAVPO\nMAPK8\tH0WXV4_OTOGA\nMAPK8\tH2NAB1_PONAB\nMAPK8\tH2R0G7_PANTR\nMAPK8\tH3BGY1_LATCH\nMAPK8\tH3BGY2_LATCH\nMAPK8\tH9EQK5_MACMU\nMAPK8\tH9GHV5_ANOCA\nMAPK8\tI3MJI4_ICTTR\nMAPK8\tK7CZW7_PANTR\nMAPK8\tK7GCT4_PELSI\nMAPK8\tK7GCT9_PELSI\nMAPK8\tK7GRS5_PIG\nMAPK8\tK9J8C8_XENTR\nMAPK8\tM3W062_FELCA\nMAPK8\tM3YCJ7_MUSPF\nMAPK8\tMK08_DANRE\nMAPK8\tMK08_HUMAN\nMAPK8\tMK08_MOUSE\nMAPK8\tMK08_RAT\nMAPK8\tMK08_XENLA\nMAPK8\tQ502A0_DANRE\nMAPK8\tQ544A0_MOUSE\nMAPK8\tQ7TSJ7_MOUSE\nMAPK8\tQ9FXT7_CHLRE\nMAPK8\tU3FKG6_CALJA\nMAPK8\tU3JA23_ANAPP\nMAPK8\tU3JYE8_FICAL\nMAPK8\tV8NPJ3_OPHHA\nMAPK8\tW5NTN9_SHEEP\nMAPK8\tW8EB54_9ROSA\n','From\tTo\nCLK2\tA0A096MLB0_PAPAN\nCLK2\tA0A0B4K6R9_DROME\nCLK2\tA0A0B4K7Z3_DROME\nCLK2\tA0A0C4DHG9_DROME\nCLK2\tA0A0D9S577_CHLSB\nCLK2\tA0A0G2K5Q8_RAT\nCLK2\tA0A0S7KLJ2_9TELE\nCLK2\tA0A0S7KLL6_9TELE\nCLK2\tA0A0S7KLR8_9TELE\nCLK2\tA0A0S7KNM3_9TELE\nCLK2\tA0A0V1FJ52_TRIPS\nCLK2\tA0A0V1HF28_9BILA\nCLK2\tA0A0V1HFQ9_9BILA\nCLK2\tA0A0V1HFV0_9BILA\nCLK2\tA0A0V1HG13_9BILA\nCLK2\tA0A0V1J621_TRIPS\nCLK2\tA0A0V1K1P3_TRIPS\nCLK2\tA0A0V1N4I8_9BILA\nCLK2\tA0A0V1N4J5_9BILA\nCLK2\tA0A0V1N506_9BILA\nCLK2\tA0A0V1P414_9BILA\nCLK2\tA0A0V1P4A4_9BILA\nCLK2\tA0A0V1P4H6_9BILA\nCLK2\tA0A0V1P4K0_9BILA\nCLK2\tA0A0X3NG75_SCHSO\nCLK2\tA0A0X3NIQ5_SCHSO\nCLK2\tA0A0X3PHS2_SCHSO\nCLK2\tA0A0X3PWK4_SCHSO\nCLK2\tA0A0X3PXS1_SCHSO\nCLK2\tA0A151MTF1_ALLMI\nCLK2\tA0A151MTG2_ALLMI\nCLK2\tA0A151MTI7_ALLMI\nCLK2\tA0A1D5P8C4_CHICK\nCLK2\tA0A1J1HAF8_PLARL\nCLK2\tA0A1L8FDG0_XENLA\nCLK2\tA0A1S2ZVS5_ERIEU\nCLK2\tA0A1S2ZVT3_ERIEU\nCLK2\tA0A1S3F244_DIPOR\nCLK2\tA0A1S3F377_DIPOR\nCLK2\tA0A1S3F3J5_DIPOR\nCLK2\tA0A1S3F4E0_DIPOR\nCLK2\tA0A1S3WE96_ERIEU\nCLK2\tA0A1S3WE97_ERIEU\nCLK2\tA0A1S3WEE3_ERIEU\nCLK2\tA0A1S3WEJ8_ERIEU\nCLK2\tA0A1U7R3D1_MESAU\nCLK2\tA0A1U7TXQ8_TARSY\nCLK2\tA0A1U7TZH3_TARSY\nCLK2\tA0A1U8DC51_ALLSI\nCLK2\tA0A1V4KD83_PATFA\nCLK2\tA0A1V4KDG9_PATFA\nCLK2\tA0A218UNC7_9PASE\nCLK2\tA0A250Y202_CASCN\nCLK2\tA0A287AIW2_PIG\nCLK2\tA0A2B4S310_STYPI\nCLK2\tA0A2D0S486_ICTPU\nCLK2\tA0A2I0LM53_COLLI\nCLK2\tA0A2I2UHV8_FELCA\nCLK2\tA0A2I3HUG4_NOMLE\nCLK2\tA0A2I3LKY1_PAPAN\nCLK2\tA0A2I3RIN5_PANTR\nCLK2\tA0A2I3TEU5_PANTR\nCLK2\tA0A2I4BLH8_9TELE\nCLK2\tA0A2K5CA45_AOTNA\nCLK2\tA0A2K5CA67_AOTNA\nCLK2\tA0A2K5CAI8_AOTNA\nCLK2\tA0A2K5CAK0_AOTNA\nCLK2\tA0A2K5CAK6_AOTNA\nCLK2\tA0A2K5IN46_COLAP\nCLK2\tA0A2K5INJ7_COLAP\nCLK2\tA0A2K5LV20_CERAT\nCLK2\tA0A2K5LV44_CERAT\nCLK2\tA0A2K5LV50_CERAT\nCLK2\tA0A2K5SIB5_CEBCA\nCLK2\tA0A2K5SIB8_CEBCA\nCLK2\tA0A2K5SIB9_CEBCA\nCLK2\tA0A2K5SIC2_CEBCA\nCLK2\tA0A2K5V3A1_MACFA\nCLK2\tA0A2K5V3C0_MACFA\nCLK2\tA0A2K5V3C2_MACFA\nCLK2\tA0A2K5XBS3_MANLE\nCLK2\tA0A2K5XBU3_MANLE\nCLK2\tA0A2K6BR12_MACNE\nCLK2\tA0A2K6BR39_MACNE\nCLK2\tA0A2K6GTE8_PROCO\nCLK2\tA0A2K6LIN0_RHIBE\nCLK2\tA0A2K6LIQ3_RHIBE\nCLK2\tA0A2K6Q0S2_RHIRO\nCLK2\tA0A2K6Q0T6_RHIRO\nCLK2\tA0A2K6TP00_SAIBB\nCLK2\tA0A2R9CJL9_PANPA\nCLK2\tA0A2R9CJM8_PANPA\nCLK2\tA0A2U3W4H6_ODORO\nCLK2\tA0A2U3XTQ8_LEPWE\nCLK2\tA0A2U3ZIB5_ODORO\nCLK2\tA0A2U3ZIC0_ODORO\nCLK2\tA0A2U4B7N2_TURTR\nCLK2\tA0A2U4B7N7_TURTR\nCLK2\tA0A2U4B7R1_TURTR\nCLK2\tA0A2U4B7U0_TURTR\nCLK2\tA0A2U4B7W0_TURTR\nCLK2\tA0A2U4B815_TURTR\nCLK2\tA0A2U4B824_TURTR\nCLK2\tA0A2Y9GEU8_NEOSC\nCLK2\tA0A2Y9GF11_NEOSC\nCLK2\tA0A2Y9GI55_NEOSC\nCLK2\tA0A2Y9LZ57_DELLE\nCLK2\tA0A2Y9LZY4_DELLE\nCLK2\tA0A2Y9M1T0_DELLE\nCLK2\tA0A2Y9M1T4_DELLE\nCLK2\tA0A2Y9M7B9_DELLE\nCLK2\tA0A2Y9M7C4_DELLE\nCLK2\tA0A2Y9MC68_DELLE\nCLK2\tA0A2Y9RZJ5_PHYMC\nCLK2\tA0A2Y9S336_PHYMC\nCLK2\tA0A2Y9S7J9_PHYMC\nCLK2\tA0A2Y9S7K4_PHYMC\nCLK2\tA0A340X8U8_LIPVE\nCLK2\tA0A341BME8_NEOAA\nCLK2\tA0A341BMJ7_NEOAA\nCLK2\tA0A341BNW9_NEOAA\nCLK2\tA0A341BQ42_NEOAA\nCLK2\tA0A383Z9Y9_BALAS\nCLK2\tA0A383ZAF1_BALAS\nCLK2\tA0A383ZAT5_BALAS\nCLK2\tA0A384CIW0_URSMA\nCLK2\tA0A384CJ10_URSMA\nCLK2\tA0A3Q0D2X9_MESAU\nCLK2\tA0A3Q0D6Z9_MESAU\nCLK2\tA0A3Q0D810_MESAU\nCLK2\tA0A3Q0E562_TARSY\nCLK2\tA0A3Q7N0P4_CALUR\nCLK2\tA0A3Q7N3J4_CALUR\nCLK2\tA0A3Q7N3Y8_CALUR\nCLK2\tA0A3Q7PF65_CALUR\nCLK2\tA0A3Q7S363_VULVU\nCLK2\tA0A3Q7SAG2_VULVU\nCLK2\tA0A3Q7SD24_VULVU\nCLK2\tA0A3Q7SHQ0_VULVU\nCLK2\tA0A3Q7TCP5_VULVU\nCLK2\tA0A3Q7TYF5_URSAR\nCLK2\tA0A3Q7ULH0_URSAR\nCLK2\tA0A3Q7UWL0_URSAR\nCLK2\tA0A3Q7VDL7_URSAR\nCLK2\tA0A419QEK8_CLOSI\nCLK2\tA0A452C9W0_BALAS\nCLK2\tA0A452G0W6_CAPHI\nCLK2\tA0A452G186_CAPHI\nCLK2\tA0A452ICF0_9SAUR\nCLK2\tA0A452S3A1_URSAM\nCLK2\tA0A452S3B0_URSAM\nCLK2\tA0A452V017_URSMA\nCLK2\tA0A455ARP3_PHYMC\nCLK2\tA0A455ARW9_PHYMC\nCLK2\tA0A455AT72_PHYMC\nCLK2\tA0A455ATU7_PHYMC\nCLK2\tA0A4W2EZK9_BOBOX\nCLK2\tA0A4W2FNF4_BOBOX\nCLK2\tA0A4W4ET20_ELEEL\nCLK2\tA0A4W4ETG0_ELEEL\nCLK2\tA0A4W4EU44_ELEEL\nCLK2\tA0A4W4EU80_ELEEL\nCLK2\tA0A4W4EU95_ELEEL\nCLK2\tA0A4W4EUR6_ELEEL\nCLK2\tA0A4W4EUV4_ELEEL\nCLK2\tA0A4W4EVT1_ELEEL\nCLK2\tA0A4X1VZF1_PIG\nCLK2\tA0A5F5PSX5_HORSE\nCLK2\tA0A5F5PVD8_HORSE\nCLK2\tA0A5F5XIZ6_FELCA\nCLK2\tA0A5F8A6Z7_MACMU\nCLK2\tA0A5F8AB98_MACMU\nCLK2\tA0A5G3L778_XENTR\nCLK2\tA4V3J9_DROME\nCLK2\tA8JRF7_DROME\nCLK2\tA8K7I0_HUMAN\nCLK2\tB1AVT0_HUMAN\nCLK2\tB7Z0R6_DROME\nCLK2\tB7Z8N6_HUMAN\nCLK2\tCLK2_HUMAN\nCLK2\tCLK2_MOUSE\nCLK2\tD3Z4P7_MOUSE\nCLK2\tD3Z7L4_MOUSE\nCLK2\tD6RG48_MOUSE\nCLK2\tE1B9Q2_BOVIN\nCLK2\tE9Q5Y1_MOUSE\nCLK2\tF1P919_CANLF\nCLK2\tF6YYR1_HORSE\nCLK2\tF7B9C5_XENTR\nCLK2\tF7CP69_HORSE\nCLK2\tF7DDQ7_XENTR\nCLK2\tF7H9U4_CALJA\nCLK2\tF8WHX0_MOUSE\nCLK2\tG1KZM7_AILME\nCLK2\tG1MPY5_MELGA\nCLK2\tG1PMR7_MYOLU\nCLK2\tG1RN41_NOMLE\nCLK2\tG1SD50_RABIT\nCLK2\tG3TQE9_LOXAF\nCLK2\tG3UCC0_LOXAF\nCLK2\tG3URH5_MELGA\nCLK2\tG3W2I0_SARHA\nCLK2\tG8F6B7_MACFA\nCLK2\tH0V093_CAVPO\nCLK2\tH0X0V7_OTOGA\nCLK2\tH1A1Z7_TAEGU\nCLK2\tH2N5H3_PONAB\nCLK2\tH2R8K9_PANTR\nCLK2\tH3AY68_LATCH\nCLK2\tH9FPQ9_MACMU\nCLK2\tH9G6I9_ANOCA\nCLK2\tI0FP48_MACMU\nCLK2\tI3MCN8_ICTTR\nCLK2\tK7FVU9_PELSI\nCLK2\tM3Y2F1_MUSPF\nCLK2\tQ0KI03_DROME\nCLK2\tQ3KPX3_XENLA\nCLK2\tQ3UFP4_MOUSE\nCLK2\tQ5M7P2_XENTR\nCLK2\tQ5XI98_RAT\nCLK2\tQ7Z0P9_CRAGI\nCLK2\tQ7ZY59_XENLA\nCLK2\tQ86B72_DROME\nCLK2\tQ86B73_DROME\nCLK2\tQ8IML9_DROME\nCLK2\tQ91YR2_MOUSE\nCLK2\tQ9BRG8_HUMAN\nCLK2\tT2MFW5_HYDVU\nCLK2\tU3IXM3_ANAPP\nCLK2\tU3JJQ0_FICAL\nCLK2\tV8NG29_OPHHA\nCLK2\tW5P1T5_SHEEP\nCLK2\tW5ULD9_ICTPU\n','From\tTo\nCDK6\tA0A034WGA6_BACDO\nCDK6\tA0A096NF27_PAPAN\nCDK6\tA0A0C9R6W6_9HYME\nCDK6\tA0A0D9RJK6_CHLSB\nCDK6\tA0A0G2JEJ6_MOUSE\nCDK6\tA0A0G2JGA8_MOUSE\nCDK6\tA0A0G2JGH2_MOUSE\nCDK6\tA0A0K8SG52_LYGHE\nCDK6\tA0A0S7EF98_9TELE\nCDK6\tA0A0S7EZ39_9TELE\nCDK6\tA0A0S7F968_9TELE\nCDK6\tA0A0S7IGC8_9TELE\nCDK6\tA0A0S7KSH9_9TELE\nCDK6\tA0A151MQI8_ALLMI\nCDK6\tA0A1A7WEQ4_9TELE\nCDK6\tA0A1A8ALH8_NOTFU\nCDK6\tA0A1A8C075_9TELE\nCDK6\tA0A1A8E5F0_9TELE\nCDK6\tA0A1A8FUT0_9TELE\nCDK6\tA0A1A8GP36_9TELE\nCDK6\tA0A1A8I7W3_NOTKU\nCDK6\tA0A1A8LHT2_9TELE\nCDK6\tA0A1A8QXY2_9TELE\nCDK6\tA0A1A8R042_9TELE\nCDK6\tA0A1A8RAM9_9TELE\nCDK6\tA0A1A8V9E1_NOTFU\nCDK6\tA0A1S2ZPH9_ERIEU\nCDK6\tA0A1S3G063_DIPOR\nCDK6\tA0A1U7QYD4_MESAU\nCDK6\tA0A1U7S5B1_ALLSI\nCDK6\tA0A1V4KBT2_PATFA\nCDK6\tA0A218V102_9PASE\nCDK6\tA0A250YFK2_CASCN\nCDK6\tA0A286XDZ7_CAVPO\nCDK6\tA0A286Y0A9_CAVPO\nCDK6\tA0A287D0P3_ICTTR\nCDK6\tA0A2B4R6J5_STYPI\nCDK6\tA0A2B4SXB2_STYPI\nCDK6\tA0A2I0MRA7_COLLI\nCDK6\tA0A2I3H5H6_NOMLE\nCDK6\tA0A2I3MS43_PAPAN\nCDK6\tA0A2I3SYX9_PANTR\nCDK6\tA0A2J8WJX0_PONAB\nCDK6\tA0A2K5CZL8_AOTNA\nCDK6\tA0A2K5K1L5_COLAP\nCDK6\tA0A2K5K1N0_COLAP\nCDK6\tA0A2K5MIA8_CERAT\nCDK6\tA0A2K5MIB3_CERAT\nCDK6\tA0A2K5RP05_CEBCA\nCDK6\tA0A2K5U072_MACFA\nCDK6\tA0A2K5ZCP0_MANLE\nCDK6\tA0A2K5ZCQ4_MANLE\nCDK6\tA0A2K6DGP9_MACNE\nCDK6\tA0A2K6DGQ0_MACNE\nCDK6\tA0A2K6GJI6_PROCO\nCDK6\tA0A2K6N2K9_RHIBE\nCDK6\tA0A2K6N2S8_RHIBE\nCDK6\tA0A2K6P4F4_RHIRO\nCDK6\tA0A2K6P4H7_RHIRO\nCDK6\tA0A2K6V0N9_SAIBB\nCDK6\tA0A2P8YTR4_BLAGE\nCDK6\tA0A2S2N6M6_SCHGA\nCDK6\tA0A2S2P8K5_SCHGA\nCDK6\tA0A2S2QHF1_9HEMI\nCDK6\tA0A2U3VH71_ODORO\nCDK6\tA0A2U4APP7_TURTR\nCDK6\tA0A2U9D0C8_SCOMX\nCDK6\tA0A2Y9H122_NEOSC\nCDK6\tA0A2Y9P3B0_DELLE\nCDK6\tA0A2Y9PG09_DELLE\nCDK6\tA0A2Y9TEJ3_PHYMC\nCDK6\tA0A337ST42_FELCA\nCDK6\tA0A340XKI5_LIPVE\nCDK6\tA0A341D7E0_NEOAA\nCDK6\tA0A369JM21_HYPMA\nCDK6\tA0A384B982_BALAS\nCDK6\tA0A384BPK8_URSMA\nCDK6\tA0A3Q0FH10_ALLSI\nCDK6\tA0A3Q0FHX5_ALLSI\nCDK6\tA0A3Q0FI10_ALLSI\nCDK6\tA0A3Q0FL24_ALLSI\nCDK6\tA0A3Q7P3W1_CALUR\nCDK6\tA0A3Q7P4W5_CALUR\nCDK6\tA0A3Q7SS30_VULVU\nCDK6\tA0A3Q7WII5_URSAR\nCDK6\tA0A419PH55_CLOSI\nCDK6\tA0A452E8F3_CAPHI\nCDK6\tA0A452HZ31_9SAUR\nCDK6\tA0A452QSE9_URSAM\nCDK6\tA0A452UYM4_URSMA\nCDK6\tA0A452UYN0_URSMA\nCDK6\tA0A493T652_ANAPP\nCDK6\tA0A4W2FPR8_BOBOX\nCDK6\tA0A4W4GXR1_ELEEL\nCDK6\tA0A4X2M7T9_VOMUR\nCDK6\tA0A4Y2EZJ7_ARAVE\nCDK6\tA0A4Y2TU26_ARAVE\nCDK6\tA0A4Z2H8U5_9TELE\nCDK6\tA0A4Z2J8T2_9TELE\nCDK6\tA0A559MGL0_9HELO\nCDK6\tA0A5F5PE88_HORSE\nCDK6\tA0A5F5PXH8_HORSE\nCDK6\tA0A5F8ATC9_MACMU\nCDK6\tA0A5F8GNW4_MONDO\nCDK6\tA0A5F9D0G6_RABIT\nCDK6\tB7UCB0_DIOKA\nCDK6\tB9EQ22_SALSA\nCDK6\tCDK6_HUMAN\nCDK6\tCDK6_MOUSE\nCDK6\tE1BC36_BOVIN\nCDK6\tE2RLM4_CANLF\nCDK6\tE4XX71_OIKDI\nCDK6\tE9QFH2_DANRE\nCDK6\tF1MA87_RAT\nCDK6\tF6NR87_DANRE\nCDK6\tF6XCT7_HORSE\nCDK6\tF6YF92_CALJA\nCDK6\tF6YRJ4_XENTR\nCDK6\tF7CH15_XENTR\nCDK6\tF7CIV1_MONDO\nCDK6\tF7E5V8_MACMU\nCDK6\tF7F9J8_ORNAN\nCDK6\tG1KQY0_ANOCA\nCDK6\tG1MHH7_AILME\nCDK6\tG1N9I3_MELGA\nCDK6\tG1PJ96_MYOLU\nCDK6\tG1RZD7_NOMLE\nCDK6\tG1T0K2_RABIT\nCDK6\tG3TL01_LOXAF\nCDK6\tG3VKY4_SARHA\nCDK6\tG7P1D7_MACFA\nCDK6\tH0XQI5_OTOGA\nCDK6\tH0YTD0_TAEGU\nCDK6\tH2LGM4_ORYLA\nCDK6\tH2QUX4_PANTR\nCDK6\tH2UNY1_TAKRU\nCDK6\tH3A4P1_LATCH\nCDK6\tH9EMN3_MACMU\nCDK6\tI3IT74_DANRE\nCDK6\tI3JXN1_ORENI\nCDK6\tI3MM51_ICTTR\nCDK6\tI3NHL0_ICTTR\nCDK6\tI6LL49_APICC\nCDK6\tK7FTK2_PELSI\nCDK6\tK7GT60_PIG\nCDK6\tM3YAA8_MUSPF\nCDK6\tQ0GMK5_RABIT\nCDK6\tQ0VBK8_MOUSE\nCDK6\tQ3U4J9_MOUSE\nCDK6\tQ5HZQ2_XENLA\nCDK6\tQ90771_CHICK\nCDK6\tQ99MD0_RAT\nCDK6\tT2MEA4_HYDVU\nCDK6\tU3IPA2_ANAPP\nCDK6\tU3K755_FICAL\nCDK6\tU6DUM6_NEOVI\nCDK6\tV8NGA2_OPHHA\nCDK6\tV8PE37_OPHHA\nCDK6\tW5QB43_SHEEP\nCDK6\tW5U7I7_ICTPU\n','From\tTo\nCDK7\tA0A034V527_BACDO\nCDK7\tA0A061SDP9_9CHLO\nCDK7\tA0A0B2VPA1_TOXCA\nCDK7\tA0A0D9RTG8_CHLSB\nCDK7\tA0A0K8V215_BACLA\nCDK7\tA0A0P6K2P0_HETGA\nCDK7\tA0A0R4IXP6_DANRE\nCDK7\tA0A0S2Z3F9_HUMAN\nCDK7\tA0A0S7KT73_9TELE\nCDK7\tA0A0S7KT88_9TELE\nCDK7\tA0A0S7KT93_9TELE\nCDK7\tA0A0S7KTB1_9TELE\nCDK7\tA0A0S7KTD3_9TELE\nCDK7\tA0A0S7KTI0_9TELE\nCDK7\tA0A0S7KTW9_9TELE\nCDK7\tA0A0S7KU42_9TELE\nCDK7\tA0A0S7KU70_9TELE\nCDK7\tA0A0S7KV38_9TELE\nCDK7\tA0A0V0SJ71_9BILA\nCDK7\tA0A0V0TG32_9BILA\nCDK7\tA0A0V0W8U6_9BILA\nCDK7\tA0A0V0YAW3_TRIPS\nCDK7\tA0A0V1ADI4_9BILA\nCDK7\tA0A0V1BRW8_TRISP\nCDK7\tA0A0V1D3X2_TRIBR\nCDK7\tA0A0V1FG43_TRIPS\nCDK7\tA0A0V1I5Q7_9BILA\nCDK7\tA0A0V1I5R7_9BILA\nCDK7\tA0A0V1I5R8_9BILA\nCDK7\tA0A0V1I5S0_9BILA\nCDK7\tA0A0V1K9Z6_TRIPS\nCDK7\tA0A0V1KAU7_TRIPS\nCDK7\tA0A0V1LFW6_9BILA\nCDK7\tA0A0V1N995_9BILA\nCDK7\tA0A0V1NJ33_9BILA\nCDK7\tA0A0X3NWX2_SCHSO\nCDK7\tA0A0X3P554_SCHSO\nCDK7\tA0A0Y0BLM8_PORTR\nCDK7\tA0A151PHL6_ALLMI\nCDK7\tA0A1A7YFC7_9TELE\nCDK7\tA0A1A8C2P2_9TELE\nCDK7\tA0A1A8CMU5_9TELE\nCDK7\tA0A1A8H285_9TELE\nCDK7\tA0A1A8IEK2_NOTKU\nCDK7\tA0A1A8IR58_NOTKU\nCDK7\tA0A1A8MD05_9TELE\nCDK7\tA0A1A8MVA3_9TELE\nCDK7\tA0A1A8N3I2_9TELE\nCDK7\tA0A1A8Q0K7_9TELE\nCDK7\tA0A1A8R0V6_9TELE\nCDK7\tA0A1A8SBM1_9TELE\nCDK7\tA0A1A8TZ56_NOTFU\nCDK7\tA0A1Q9D671_SYMMI\nCDK7\tA0A1S3ANJ2_ERIEU\nCDK7\tA0A1S3F3R6_DIPOR\nCDK7\tA0A1S3PFR0_SALSA\nCDK7\tA0A1U7RK24_ALLSI\nCDK7\tA0A1U7SNQ3_TARSY\nCDK7\tA0A1V4L298_PATFA\nCDK7\tA0A218UNV1_9PASE\nCDK7\tA0A250YGD2_CASCN\nCDK7\tA0A286YDC0_MOUSE\nCDK7\tA0A287BKU6_PIG\nCDK7\tA0A2B4T0A1_STYPI\nCDK7\tA0A2I0M1A4_COLLI\nCDK7\tA0A2I3GGC4_NOMLE\nCDK7\tA0A2I3H4U2_NOMLE\nCDK7\tA0A2I3H873_NOMLE\nCDK7\tA0A2I3LUL7_PAPAN\nCDK7\tA0A2I3M1G3_PAPAN\nCDK7\tA0A2I3M5Y5_PAPAN\nCDK7\tA0A2I3S9X0_PANTR\nCDK7\tA0A2I4D2I3_9TELE\nCDK7\tA0A2J7PNW7_9NEOP\nCDK7\tA0A2J7PNX1_9NEOP\nCDK7\tA0A2J7PNY0_9NEOP\nCDK7\tA0A2J7PNY6_9NEOP\nCDK7\tA0A2J7PP04_9NEOP\nCDK7\tA0A2J8L111_PANTR\nCDK7\tA0A2J8L115_PANTR\nCDK7\tA0A2K5DLY2_AOTNA\nCDK7\tA0A2K5DLZ9_AOTNA\nCDK7\tA0A2K5DM08_AOTNA\nCDK7\tA0A2K5DM72_AOTNA\nCDK7\tA0A2K5DM81_AOTNA\nCDK7\tA0A2K5DM89_AOTNA\nCDK7\tA0A2K5J079_COLAP\nCDK7\tA0A2K5J083_COLAP\nCDK7\tA0A2K5J087_COLAP\nCDK7\tA0A2K5J088_COLAP\nCDK7\tA0A2K5MI65_CERAT\nCDK7\tA0A2K5MI76_CERAT\nCDK7\tA0A2K5MI77_CERAT\nCDK7\tA0A2K5MIF4_CERAT\nCDK7\tA0A2K5R755_CEBCA\nCDK7\tA0A2K5R790_CEBCA\nCDK7\tA0A2K5R793_CEBCA\nCDK7\tA0A2K5R7A1_CEBCA\nCDK7\tA0A2K5R7D3_CEBCA\nCDK7\tA0A2K5TTK3_MACFA\nCDK7\tA0A2K5TTK9_MACFA\nCDK7\tA0A2K5TTL6_MACFA\nCDK7\tA0A2K5TTM0_MACFA\nCDK7\tA0A2K5TTM9_MACFA\nCDK7\tA0A2K5YSG6_MANLE\nCDK7\tA0A2K5YSH4_MANLE\nCDK7\tA0A2K5YSI0_MANLE\nCDK7\tA0A2K5YSI8_MANLE\nCDK7\tA0A2K6B4C5_MACNE\nCDK7\tA0A2K6B4E1_MACNE\nCDK7\tA0A2K6B4E9_MACNE\nCDK7\tA0A2K6B4F3_MACNE\nCDK7\tA0A2K6B4H0_MACNE\nCDK7\tA0A2K6KIJ6_RHIBE\nCDK7\tA0A2K6KIN5_RHIBE\nCDK7\tA0A2K6KIP2_RHIBE\nCDK7\tA0A2K6KIQ7_RHIBE\nCDK7\tA0A2K6QZC6_RHIRO\nCDK7\tA0A2K6QZD8_RHIRO\nCDK7\tA0A2K6QZE4_RHIRO\nCDK7\tA0A2K6QZI0_RHIRO\nCDK7\tA0A2L0CAR4_9CUCU\nCDK7\tA0A2L0CAR9_9CUCU\nCDK7\tA0A2L0CAS1_9CUCU\nCDK7\tA0A2L0CAS7_9CUCU\nCDK7\tA0A2L0CAS9_9CUCU\nCDK7\tA0A2L0CAT1_9COLE\nCDK7\tA0A2L0CAT3_9CUCU\nCDK7\tA0A2L0CAT7_9CUCU\nCDK7\tA0A2L0CAT9_9CUCU\nCDK7\tA0A2L0CAU2_9COLE\nCDK7\tA0A2L0CAU3_9CUCU\nCDK7\tA0A2L0CAU6_9CUCU\nCDK7\tA0A2L0CAV0_9CUCU\nCDK7\tA0A2L0CAV1_9CUCU\nCDK7\tA0A2L0CAV2_9CUCU\nCDK7\tA0A2L0CAV3_9CUCU\nCDK7\tA0A2L0CAV4_9COLE\nCDK7\tA0A2L0CAV8_9CUCU\nCDK7\tA0A2L0CAV9_9CUCU\nCDK7\tA0A2L0CAW1_9COLE\nCDK7\tA0A2L0CAW2_9COLE\nCDK7\tA0A2L0CAW3_9COLE\nCDK7\tA0A2L0CAW4_9CUCU\nCDK7\tA0A2L0CAW8_9CUCU\nCDK7\tA0A2L0CAW9_9CUCU\nCDK7\tA0A2L0CAX0_9COLE\nCDK7\tA0A2L0CAX1_9CUCU\nCDK7\tA0A2L0CAX3_9COLE\nCDK7\tA0A2L0CAX4_9SCAR\nCDK7\tA0A2L0CAX8_9CUCU\nCDK7\tA0A2L0CAX9_9CUCU\nCDK7\tA0A2L0CAY0_9CUCU\nCDK7\tA0A2L0CAY1_9CUCU\nCDK7\tA0A2L0CAY2_9COLE\nCDK7\tA0A2L0CAY4_9CUCU\nCDK7\tA0A2L0CAY8_9CUCU\nCDK7\tA0A2L0CAY9_9COLE\nCDK7\tA0A2L0CAZ0_9CUCU\nCDK7\tA0A2L0CAZ1_9CUCU\nCDK7\tA0A2L0CAZ2_9CUCU\nCDK7\tA0A2L0CAZ4_9CUCU\nCDK7\tA0A2L0CAZ7_9CUCU\nCDK7\tA0A2L0CAZ9_9CUCU\nCDK7\tA0A2L0CB01_9COLE\nCDK7\tA0A2L0CB02_9CUCU\nCDK7\tA0A2L0CB03_9COLE\nCDK7\tA0A2L0CB05_9CUCU\nCDK7\tA0A2L0CB07_9CUCU\nCDK7\tA0A2L0CB09_9COLE\nCDK7\tA0A2L0CB11_9CUCU\nCDK7\tA0A2L0CB12_9CUCU\nCDK7\tA0A2L0CB13_9COLE\nCDK7\tA0A2L0CB14_9CUCU\nCDK7\tA0A2L0CB15_9CUCU\nCDK7\tA0A2L0CB17_9CUCU\nCDK7\tA0A2L0CB19_9COLE\nCDK7\tA0A2L0CB21_9CUCU\nCDK7\tA0A2L0CB22_9CUCU\nCDK7\tA0A2L0CB23_9COLE\nCDK7\tA0A2L0CB24_9CUCU\nCDK7\tA0A2L0CB25_9COLE\nCDK7\tA0A2L0CB26_9CUCU\nCDK7\tA0A2L0CB27_9COLE\nCDK7\tA0A2L0CB29_9CUCU\nCDK7\tA0A2L0CB31_9COLE\nCDK7\tA0A2L0CB32_9CUCU\nCDK7\tA0A2L0CB33_9COLE\nCDK7\tA0A2L0CB34_9CUCU\nCDK7\tA0A2L0CB35_9COLE\nCDK7\tA0A2L0CB36_9CUCU\nCDK7\tA0A2L0CB37_9COLE\nCDK7\tA0A2L0CB38_9CUCU\nCDK7\tA0A2L0CB39_9CUCU\nCDK7\tA0A2L0CB41_9CUCU\nCDK7\tA0A2L0CB42_9COLE\nCDK7\tA0A2L0CB43_9CUCU\nCDK7\tA0A2L0CB44_9CUCU\nCDK7\tA0A2L0CB45_9CUCU\nCDK7\tA0A2L0CB46_9CUCU\nCDK7\tA0A2L0CB47_9SCAR\nCDK7\tA0A2L0CB48_9CUCU\nCDK7\tA0A2L0CB50_9CUCU\nCDK7\tA0A2L0CB51_9COLE\nCDK7\tA0A2L0CB52_9CUCU\nCDK7\tA0A2L0CB53_9SCAR\nCDK7\tA0A2L0CB54_9COLE\nCDK7\tA0A2L0CB55_9CUCU\nCDK7\tA0A2L0CB56_9COLE\nCDK7\tA0A2L0CB57_9CUCU\nCDK7\tA0A2L0CB58_9COLE\nCDK7\tA0A2L0CB59_9CUCU\nCDK7\tA0A2L0CB60_9SCAR\nCDK7\tA0A2L0CB62_9CUCU\nCDK7\tA0A2L0CB63_9CUCU\nCDK7\tA0A2L0CB64_9CUCU\nCDK7\tA0A2L0CB65_9COLE\nCDK7\tA0A2L0CB66_9SCAR\nCDK7\tA0A2L0CB67_9CUCU\nCDK7\tA0A2L0CB68_9CUCU\nCDK7\tA0A2L0CB69_9CUCU\nCDK7\tA0A2L0CB70_9CUCU\nCDK7\tA0A2L0CB72_9CUCU\nCDK7\tA0A2L0CB73_9CUCU\nCDK7\tA0A2L0CB74_9CUCU\nCDK7\tA0A2L0CB75_9COLE\nCDK7\tA0A2L0CB76_9CUCU\nCDK7\tA0A2L0CB77_9CUCU\nCDK7\tA0A2L0CB78_9CUCU\nCDK7\tA0A2L0CB79_9COLE\nCDK7\tA0A2L0CB80_9CUCU\nCDK7\tA0A2L0CB81_9CUCU\nCDK7\tA0A2L0CB83_9CUCU\nCDK7\tA0A2L0CB84_9COLE\nCDK7\tA0A2L0CB85_9COLE\nCDK7\tA0A2L0CB86_9CUCU\nCDK7\tA0A2L0CB87_9CUCU\nCDK7\tA0A2L0CB88_9SCAR\nCDK7\tA0A2L0CB89_9COLE\nCDK7\tA0A2L0CB90_9CUCU\nCDK7\tA0A2L0CB91_9SCAR\nCDK7\tA0A2L0CB93_9CUCU\nCDK7\tA0A2L0CB94_9CUCU\nCDK7\tA0A2L0CB95_9CUCU\nCDK7\tA0A2L0CB96_9CUCU\nCDK7\tA0A2L0CB97_9CUCU\nCDK7\tA0A2L0CB98_9CUCU\nCDK7\tA0A2L0CB99_9CUCU\nCDK7\tA0A2L0CBA0_9CUCU\nCDK7\tA0A2L0CBA1_9COLE\nCDK7\tA0A2L0CBA2_9CUCU\nCDK7\tA0A2L0CBA3_9CUCU\nCDK7\tA0A2L0CBA4_9NEOP\nCDK7\tA0A2L0CBA5_CHRPP\nCDK7\tA0A2L0CBA6_9COLE\nCDK7\tA0A2L0CBA7_9COLE\nCDK7\tA0A2L0CBA8_9COLE\nCDK7\tA0A2L0CBA9_9CUCU\nCDK7\tA0A2L0CBB0_9COLE\nCDK7\tA0A2L0CBB1_9COLE\nCDK7\tA0A2L0CBB2_9CUCU\nCDK7\tA0A2L0CBB3_9COLE\nCDK7\tA0A2L0CBB4_9CUCU\nCDK7\tA0A2L0CBB5_9CUCU\nCDK7\tA0A2L0CBB6_9CUCU\nCDK7\tA0A2L0CBB7_9CUCU\nCDK7\tA0A2L0CBB8_9CUCU\nCDK7\tA0A2L0CBB9_9CUCU\nCDK7\tA0A2L0CBC0_9COLE\nCDK7\tA0A2L0CBC1_9CUCU\nCDK7\tA0A2L0CBC2_9CUCU\nCDK7\tA0A2L0CBC3_9COLE\nCDK7\tA0A2L0CBC4_9COLE\nCDK7\tA0A2L0CBC5_9CUCU\nCDK7\tA0A2L0CBC6_9CUCU\nCDK7\tA0A2L0CBC7_9CUCU\nCDK7\tA0A2L0CBC8_9COLE\nCDK7\tA0A2L0CBC9_9CUCU\nCDK7\tA0A2L0CBD0_9COLE\nCDK7\tA0A2L0CBD1_9COLE\nCDK7\tA0A2L0CBD2_9CUCU\nCDK7\tA0A2L0CBD3_9COLE\nCDK7\tA0A2L0CBD4_9CUCU\nCDK7\tA0A2L0CBD5_9CUCU\nCDK7\tA0A2L0CBD6_9CUCU\nCDK7\tA0A2L0CBD7_9SCAR\nCDK7\tA0A2L0CBD8_9COLE\nCDK7\tA0A2L0CBD9_9CUCU\nCDK7\tA0A2L0CBE0_9COLE\nCDK7\tA0A2L0CBE1_9CUCU\nCDK7\tA0A2L0CBE2_9CUCU\nCDK7\tA0A2L0CBE3_NECLI\nCDK7\tA0A2L0CBE4_9CUCU\nCDK7\tA0A2L0CBE5_9CUCU\nCDK7\tA0A2L0CBE6_9CUCU\nCDK7\tA0A2L0CBE7_9COLE\nCDK7\tA0A2L0CBE8_9COLE\nCDK7\tA0A2L0CBE9_9CUCU\nCDK7\tA0A2L0CBF0_9CUCU\nCDK7\tA0A2L0CBF1_9CUCU\nCDK7\tA0A2L0CBF2_9COLE\nCDK7\tA0A2L0CBF3_9COLE\nCDK7\tA0A2L0CBF4_9SCAR\nCDK7\tA0A2L0CBF5_9CUCU\nCDK7\tA0A2L0CBF6_9COLE\nCDK7\tA0A2L0CBF7_9COLE\nCDK7\tA0A2L0CBF8_9CUCU\nCDK7\tA0A2L0CBF9_9CUCU\nCDK7\tA0A2L0CBG0_9CUCU\nCDK7\tA0A2L0CBG1_9CUCU\nCDK7\tA0A2L0CBG2_9CUCU\nCDK7\tA0A2L0CBG3_9SCAR\nCDK7\tA0A2L0CBG4_9CUCU\nCDK7\tA0A2L0CBG5_9SCAR\nCDK7\tA0A2L0CBG6_9CUCU\nCDK7\tA0A2L0CBG7_9CUCU\nCDK7\tA0A2L0CBG8_9CUCU\nCDK7\tA0A2L0CBG9_9CUCU\nCDK7\tA0A2L0CBH0_9COLE\nCDK7\tA0A2L0CBH1_9CUCU\nCDK7\tA0A2L0CBH2_9COLE\nCDK7\tA0A2L0CBH3_9CUCU\nCDK7\tA0A2L0CBH4_9CUCU\nCDK7\tA0A2L0CBH5_9CUCU\nCDK7\tA0A2L0CBH6_9COLE\nCDK7\tA0A2L0CBH7_9CUCU\nCDK7\tA0A2L0CBH8_9COLE\nCDK7\tA0A2L0CBH9_9COLE\nCDK7\tA0A2L0CBI0_9SCAR\nCDK7\tA0A2L0CBI1_9CUCU\nCDK7\tA0A2L0CBI2_9COLE\nCDK7\tA0A2L0CBI3_9CUCU\nCDK7\tA0A2L0CBI4_9COLE\nCDK7\tA0A2L0CBI5_9CUCU\nCDK7\tA0A2L0CBI6_9COLE\nCDK7\tA0A2L0CBI7_9COLE\nCDK7\tA0A2L0CBI8_9COLE\nCDK7\tA0A2L0CBI9_9CUCU\nCDK7\tA0A2L0CBJ0_9CUCU\nCDK7\tA0A2L0CBJ1_9CUCU\nCDK7\tA0A2L0CBJ2_9CUCU\nCDK7\tA0A2L0CBJ3_9SCAR\nCDK7\tA0A2L0CBJ4_9CUCU\nCDK7\tA0A2L0CBJ5_NICNE\nCDK7\tA0A2L0CBJ6_9SCAR\nCDK7\tA0A2L0CBJ7_9CARA\nCDK7\tA0A2L0CBJ8_9COLE\nCDK7\tA0A2L0CBJ9_9COLE\nCDK7\tA0A2L0CBK0_9CUCU\nCDK7\tA0A2L0CBK1_9SCAR\nCDK7\tA0A2L0CBK2_9SCAR\nCDK7\tA0A2L0CBK3_9COLE\nCDK7\tA0A2L0CBK4_9CUCU\nCDK7\tA0A2L0CBK5_9COLE\nCDK7\tA0A2L0CBK6_9COLE\nCDK7\tA0A2L0CBK7_9COLE\nCDK7\tA0A2L0CBK8_9CUCU\nCDK7\tA0A2L0CBK9_9CUCU\nCDK7\tA0A2L0CBL0_9SCAR\nCDK7\tA0A2L0CBL1_9COLE\nCDK7\tA0A2L0CBL2_9COLE\nCDK7\tA0A2L0CBL3_9CARA\nCDK7\tA0A2L0CBL4_9CUCU\nCDK7\tA0A2L0CBL5_9CUCU\nCDK7\tA0A2L0CBL6_9CUCU\nCDK7\tA0A2L0CBL7_9COLE\nCDK7\tA0A2L0CBL8_9CUCU\nCDK7\tA0A2L0CBL9_9CUCU\nCDK7\tA0A2L0CBM0_9COLE\nCDK7\tA0A2L0CBM1_9COLE\nCDK7\tA0A2L0CBM2_9CUCU\nCDK7\tA0A2L0CBM3_9CUCU\nCDK7\tA0A2L0CBM4_9CUCU\nCDK7\tA0A2L0CBM5_9CUCU\nCDK7\tA0A2L0CBM6_9SCAR\nCDK7\tA0A2L0CBM7_9CUCU\nCDK7\tA0A2L0CBM8_9CUCU\nCDK7\tA0A2L0CBM9_TRYDI\nCDK7\tA0A2L0CBN0_9CUCU\nCDK7\tA0A2L0CBN1_9CUCU\nCDK7\tA0A2L0CBN2_9CUCU\nCDK7\tA0A2L0CBN3_9CUCU\nCDK7\tA0A2L0CBN4_9COLE\nCDK7\tA0A2L0CBN5_9CUCU\nCDK7\tA0A2L0CBN6_9COLE\nCDK7\tA0A2L0CBN7_9COLE\nCDK7\tA0A2L0CBN8_9COLE\nCDK7\tA0A2L0CBN9_9CUCU\nCDK7\tA0A2L0CBP0_9CUCU\nCDK7\tA0A2L0CBP1_9CUCU\nCDK7\tA0A2L0CBP2_9COLE\nCDK7\tA0A2L0CBP3_9COLE\nCDK7\tA0A2L0CBP4_9COLE\nCDK7\tA0A2L0CBP5_9COLE\nCDK7\tA0A2L0CBP6_9CUCU\nCDK7\tA0A2L0CBP7_9CUCU\nCDK7\tA0A2L0CBP8_9COLE\nCDK7\tA0A2L0CBP9_9COLE\nCDK7\tA0A2L0CBQ0_9COLE\nCDK7\tA0A2L0CBQ1_9CUCU\nCDK7\tA0A2L0CBQ2_9COLE\nCDK7\tA0A2L0CBQ3_9COLE\nCDK7\tA0A2L0CBQ5_9CUCU\nCDK7\tA0A2L0CBQ6_9SCAR\nCDK7\tA0A2L0CBQ7_9COLE\nCDK7\tA0A2L0CBQ8_9CUCU\nCDK7\tA0A2L0CBQ9_9COLE\nCDK7\tA0A2L0CBR1_9COLE\nCDK7\tA0A2L0CBR4_9SCAR\nCDK7\tA0A2L0CBR5_9COLE\nCDK7\tA0A2L0CBR6_9CUCU\nCDK7\tA0A2L0CBR7_9COLE\nCDK7\tA0A2L0CBR8_9COLE\nCDK7\tA0A2L0CBS1_9CUCU\nCDK7\tA0A2L0CBS3_9COLE\nCDK7\tA0A2L0CBS5_9SCAR\nCDK7\tA0A2L0CBS6_9COLE\nCDK7\tA0A2L0CBS7_9COLE\nCDK7\tA0A2L0CBS8_9COLE\nCDK7\tA0A2L0CBT1_9DYTI\nCDK7\tA0A2L0CBT3_9CUCU\nCDK7\tA0A2L0CBT5_9COLE\nCDK7\tA0A2L0CBT7_9COLE\nCDK7\tA0A2L0CBU1_9COLE\nCDK7\tA0A2L0CBU3_9SCAR\nCDK7\tA0A2L0CBU6_9COLE\nCDK7\tA0A2L0CBU7_9CUCU\nCDK7\tA0A2L0CBV0_9CUCU\nCDK7\tA0A2L0CBV2_9COLE\nCDK7\tA0A2L0CBV6_9CUCU\nCDK7\tA0A2L0CBV8_9COLE\nCDK7\tA0A2L0CBW0_9COLE\nCDK7\tA0A2L0CBW3_9COLE\nCDK7\tA0A2L0CBW6_9COLE\nCDK7\tA0A2L0CBW7_9CUCU\nCDK7\tA0A2L0CBX1_9CUCU\nCDK7\tA0A2L0CBX3_9COLE\nCDK7\tA0A2L0CBY0_9CUCU\nCDK7\tA0A2L0CBY1_9COLE\nCDK7\tA0A2L0CBY2_9CUCU\nCDK7\tA0A2L0CBY7_9COLE\nCDK7\tA0A2L0CBZ0_9CUCU\nCDK7\tA0A2L0CBZ1_9COLE\nCDK7\tA0A2L0CBZ3_9COLE\nCDK7\tA0A2L0CBZ4_9COLE\nCDK7\tA0A2L0CBZ5_9CUCU\nCDK7\tA0A2L0CC00_9COLE\nCDK7\tA0A2L0CC05_9CUCU\nCDK7\tA0A2L0CC10_9CUCU\nCDK7\tA0A2L0CC17_9COLE\nCDK7\tA0A2L0CC20_9CUCU\nCDK7\tA0A2L0CC29_9COLE\nCDK7\tA0A2L0CC40_9COLE\nCDK7\tA0A2L0CC52_9CUCU\nCDK7\tA0A2L0CC61_9CUCU\nCDK7\tA0A2L0CC67_9CUCU\nCDK7\tA0A2L0CC78_9COLE\nCDK7\tA0A2P8YDL8_BLAGE\nCDK7\tA0A2R8MDA1_CALJA\nCDK7\tA0A2R8MYQ4_CALJA\nCDK7\tA0A2R8PCX4_CALJA\nCDK7\tA0A2R8ZYS1_PANPA\nCDK7\tA0A2R8ZZV4_PANPA\nCDK7\tA0A2R9A312_PANPA\nCDK7\tA0A2R9A4C5_PANPA\nCDK7\tA0A2S2NGT8_SCHGA\nCDK7\tA0A2S2QEZ6_9HEMI\nCDK7\tA0A2U3W5S2_ODORO\nCDK7\tA0A2U4AWX4_TURTR\nCDK7\tA0A2U4AXF6_TURTR\nCDK7\tA0A2Y9FSY3_PHYMC\nCDK7\tA0A2Y9I1K3_NEOSC\nCDK7\tA0A2Y9I1L2_NEOSC\nCDK7\tA0A2Y9I1X2_NEOSC\nCDK7\tA0A2Y9I243_NEOSC\nCDK7\tA0A2Y9I8M3_NEOSC\nCDK7\tA0A2Y9PY83_DELLE\nCDK7\tA0A2Y9PYA2_DELLE\nCDK7\tA0A2Y9SKN9_PHYMC\nCDK7\tA0A337SJ00_FELCA\nCDK7\tA0A337SMG4_FELCA\nCDK7\tA0A340X6F6_LIPVE\nCDK7\tA0A340XD58_LIPVE\nCDK7\tA0A341CB32_NEOAA\nCDK7\tA0A341CB70_NEOAA\nCDK7\tA0A341CBM2_NEOAA\nCDK7\tA0A367KXS4_RHIST\nCDK7\tA0A383ZLS3_BALAS\nCDK7\tA0A383ZLS4_BALAS\nCDK7\tA0A384C3X3_URSMA\nCDK7\tA0A384C3Z1_URSMA\nCDK7\tA0A384C544_URSMA\nCDK7\tA0A3B3I852_ORYLA\nCDK7\tA0A3Q0CDC2_MESAU\nCDK7\tA0A3Q0DK82_TARSY\nCDK7\tA0A3Q0DSZ9_TARSY\nCDK7\tA0A3Q0GBY2_ALLSI\nCDK7\tA0A3Q0GG82_ALLSI\nCDK7\tA0A3Q2GRV6_HORSE\nCDK7\tA0A3Q2HJ72_HORSE\nCDK7\tA0A3Q2KSU1_HORSE\nCDK7\tA0A3Q7QX05_VULVU\nCDK7\tA0A3Q7R6Q7_VULVU\nCDK7\tA0A3Q7S8W3_VULVU\nCDK7\tA0A3Q7UCG6_VULVU\nCDK7\tA0A3Q7UQW6_VULVU\nCDK7\tA0A3Q7V036_URSAR\nCDK7\tA0A3Q7VQX9_URSAR\nCDK7\tA0A3Q7W0D1_URSAR\nCDK7\tA0A3Q7WW39_URSAR\nCDK7\tA0A419PVN8_CLOSI\nCDK7\tA0A452CCL9_BALAS\nCDK7\tA0A452FV66_CAPHI\nCDK7\tA0A452FV92_CAPHI\nCDK7\tA0A452FVY6_CAPHI\nCDK7\tA0A452GRI0_9SAUR\nCDK7\tA0A452RWI2_URSAM\nCDK7\tA0A493STZ5_ANAPP\nCDK7\tA0A493TGT5_ANAPP\nCDK7\tA0A4D8Y042_SALSN\nCDK7\tA0A4D8Y2P1_SALSN\nCDK7\tA0A4D8Y5H4_SALSN\nCDK7\tA0A4D8Y6I3_SALSN\nCDK7\tA0A4D8YTX9_SALSN\nCDK7\tA0A4D8Z814_SALSN\nCDK7\tA0A4D8ZUA1_SALSN\nCDK7\tA0A4D9APF0_SALSN\nCDK7\tA0A4D9BZY6_SALSN\nCDK7\tA0A4D9C3H2_SALSN\nCDK7\tA0A4D9C5M2_SALSN\nCDK7\tA0A4W2G296_BOBOX\nCDK7\tA0A4W2G2B2_BOBOX\nCDK7\tA0A4W2HXB6_BOBOX\nCDK7\tA0A4W3JPN1_CALMI\nCDK7\tA0A4W3JRY3_CALMI\nCDK7\tA0A4W3K265_CALMI\nCDK7\tA0A4W3K3P2_CALMI\nCDK7\tA0A4W3KFQ1_CALMI\nCDK7\tA0A4W3KFQ5_CALMI\nCDK7\tA0A4W4FRZ9_ELEEL\nCDK7\tA0A4W4FS02_ELEEL\nCDK7\tA0A4W4FUT6_ELEEL\nCDK7\tA0A4W6G971_LATCA\nCDK7\tA0A4W6GA31_LATCA\nCDK7\tA0A4W6GAJ7_LATCA\nCDK7\tA0A4X1UFA9_PIG\nCDK7\tA0A4X1UJB4_PIG\nCDK7\tA0A4X1UK08_PIG\nCDK7\tA0A4X1UKP5_PIG\nCDK7\tA0A4X2LV67_VOMUR\nCDK7\tA0A4Y2LTZ3_ARAVE\nCDK7\tA0A4Z2E7U5_9TELE\nCDK7\tA0A4Z2FEA8_9TELE\nCDK7\tA0A5B6UCD7_9ROSI\nCDK7\tA0A5B6UD06_9ROSI\nCDK7\tA0A5B6UEK3_9ROSI\nCDK7\tA0A5B7EUW1_PORTR\nCDK7\tA0A5F4C2E9_CANLF\nCDK7\tA0A5F4C5N9_CANLF\nCDK7\tA0A5F5PQU5_HORSE\nCDK7\tA0A5F5PRX4_HORSE\nCDK7\tA0A5F8GIZ2_MONDO\nCDK7\tA0A5F9C2L3_RABIT\nCDK7\tA0A5F9DBW9_RABIT\nCDK7\tA0A5F9DGS9_RABIT\nCDK7\tA0A5F9DVC2_RABIT\nCDK7\tA0A5G2QDA2_PIG\nCDK7\tA0A5G2QH32_PIG\nCDK7\tA0A5G2QNN8_PIG\nCDK7\tA0A5G2QQ17_PIG\nCDK7\tA0A5G2R6U9_PIG\nCDK7\tA0A5G3K1Z7_XENTR\nCDK7\tA4VAK7_DANRE\nCDK7\tA9VB95_MONBE\nCDK7\tB8Y497_DIOKA\nCDK7\tC9K505_PIG\nCDK7\tCDK7_CARAU\nCDK7\tCDK7_DICDI\nCDK7\tCDK7_HUMAN\nCDK7\tCDK7_MOUSE\nCDK7\tCDK7_RAT\nCDK7\tCDK7_XENLA\nCDK7\tD3BAT3_POLPP\nCDK7\tD6R9G1_HUMAN\nCDK7\tD6R9Z2_HUMAN\nCDK7\tD6RAD4_HUMAN\nCDK7\tD6REC6_HUMAN\nCDK7\tD6RF14_HUMAN\nCDK7\tD6RFL0_HUMAN\nCDK7\tD6RGG9_HUMAN\nCDK7\tD6RI01_HUMAN\nCDK7\tD6RIG9_HUMAN\nCDK7\tE1C8L2_CHICK\nCDK7\tE2R8R2_CANLF\nCDK7\tE2R8R3_CANLF\nCDK7\tE3TCH8_ICTFU\nCDK7\tE3TG08_ICTPU\nCDK7\tE4XWK2_OIKDI\nCDK7\tF1LQC8_RAT\nCDK7\tF4PTY5_CAVFA\nCDK7\tF6Q6M6_PIG\nCDK7\tF6RN55_XENTR\nCDK7\tF6TC82_MONDO\nCDK7\tF6XZK0_MACMU\nCDK7\tF7D1B6_ORNAN\nCDK7\tF7DTX6_HORSE\nCDK7\tF7EN34_XENTR\nCDK7\tF7ERK3_XENTR\nCDK7\tF7ERT1_XENTR\nCDK7\tF7ERV1_XENTR\nCDK7\tF7IRE4_CALJA\nCDK7\tG1KSN0_ANOCA\nCDK7\tG1L901_AILME\nCDK7\tG1MZF7_MELGA\nCDK7\tG1NZ25_MYOLU\nCDK7\tG1ST86_RABIT\nCDK7\tG3T8Q9_LOXAF\nCDK7\tG3W2M2_SARHA\nCDK7\tG9MEH0_HYPVG\nCDK7\tG9NF18_HYPAI\nCDK7\tH0VRK4_CAVPO\nCDK7\tH0X3D0_OTOGA\nCDK7\tH0ZVE4_TAEGU\nCDK7\tH2LY76_ORYLA\nCDK7\tH2PFR4_PONAB\nCDK7\tH2QR08_PANTR\nCDK7\tH2TM72_TAKRU\nCDK7\tH2TM74_TAKRU\nCDK7\tH3B8P3_LATCH\nCDK7\tI3K805_ORENI\nCDK7\tI3MYX5_ICTTR\nCDK7\tK7FT73_PELSI\nCDK7\tM3W2X7_FELCA\nCDK7\tM3YT41_MUSPF\nCDK7\tM3ZAW4_NOMLE\nCDK7\tM3ZBG1_NOMLE\nCDK7\tM4SIY7_9BILA\nCDK7\tM4SZJ3_9BILA\nCDK7\tQ08DX5_BOVIN\nCDK7\tQ24216_DROME\nCDK7\tQ25463_MARGL\nCDK7\tQ28II8_XENTR\nCDK7\tQ2V6H0_DANRE\nCDK7\tQ3THG5_MOUSE\nCDK7\tQ3UJT4_MOUSE\nCDK7\tQ6NWC5_DANRE\nCDK7\tQ8CAC4_MOUSE\nCDK7\tT2MFX7_HYDVU\nCDK7\tU3IDN5_ANAPP\nCDK7\tU3K5J1_FICAL\nCDK7\tU6DFM2_NEOVI\nCDK7\tV8P638_OPHHA\nCDK7\tV9KYU0_CALMI\nCDK7\tW5P6S6_SHEEP\nCDK7\tW8AGV1_CERCA\n','From\tTo\nVRK1\tA0A096NJP0_PAPAN\nVRK1\tA0A0A9W2F3_LYGHE\nVRK1\tA0A0C9R2K3_9HYME\nVRK1\tA0A0D9RDN3_CHLSB\nVRK1\tA0A0P6K766_HETGA\nVRK1\tA0A0S7I1D1_9TELE\nVRK1\tA0A0S7I3J9_9TELE\nVRK1\tA0A0S7LCX5_9TELE\nVRK1\tA0A0V0J5M1_SCHSO\nVRK1\tA0A0X3NK51_SCHSO\nVRK1\tA0A0X3P9N5_SCHSO\nVRK1\tA0A146KY46_LYGHE\nVRK1\tA0A151P8M2_ALLMI\nVRK1\tA0A151P8T2_ALLMI\nVRK1\tA0A1A7X6I3_9TELE\nVRK1\tA0A1A7Z9I2_NOTFU\nVRK1\tA0A1A8CV90_9TELE\nVRK1\tA0A1A8HKV9_9TELE\nVRK1\tA0A1A8JJQ1_NOTKU\nVRK1\tA0A1A8MZ56_9TELE\nVRK1\tA0A1A8N8K6_9TELE\nVRK1\tA0A1A8SGX8_9TELE\nVRK1\tA0A1A8ULA6_NOTFU\nVRK1\tA0A1D5PSB8_CHICK\nVRK1\tA0A1L8F9N0_XENLA\nVRK1\tA0A1S2ZF29_ERIEU\nVRK1\tA0A1S3F935_DIPOR\nVRK1\tA0A1U7Q960_MESAU\nVRK1\tA0A1U7S0D9_ALLSI\nVRK1\tA0A1U7TZ39_TARSY\nVRK1\tA0A1U7U584_TARSY\nVRK1\tA0A1U7UH99_TARSY\nVRK1\tA0A1U7UIW1_TARSY\nVRK1\tA0A1U8BJ73_MESAU\nVRK1\tA0A1U8BMX9_MESAU\nVRK1\tA0A1V4KAK1_PATFA\nVRK1\tA0A1Y7VLL5_MOUSE\nVRK1\tA0A1Y7VMH3_MOUSE\nVRK1\tA0A218V3I1_9PASE\nVRK1\tA0A250YH89_CASCN\nVRK1\tA0A287BJB2_PIG\nVRK1\tA0A2B4R6Q1_STYPI\nVRK1\tA0A2B4RMM0_STYPI\nVRK1\tA0A2D0RPS8_ICTPU\nVRK1\tA0A2I0MP12_COLLI\nVRK1\tA0A2I2V0C2_FELCA\nVRK1\tA0A2I3MV34_PAPAN\nVRK1\tA0A2I3TT10_PANTR\nVRK1\tA0A2I4BHW4_9TELE\nVRK1\tA0A2J7Q4G3_9NEOP\nVRK1\tA0A2K5HYW7_COLAP\nVRK1\tA0A2K5HZ58_COLAP\nVRK1\tA0A2K5HZ74_COLAP\nVRK1\tA0A2K5MS34_CERAT\nVRK1\tA0A2K5MS47_CERAT\nVRK1\tA0A2K5XS81_MANLE\nVRK1\tA0A2K6B135_MACNE\nVRK1\tA0A2K6B170_MACNE\nVRK1\tA0A2K6B182_MACNE\nVRK1\tA0A2K6G4R6_PROCO\nVRK1\tA0A2K6JWI3_RHIBE\nVRK1\tA0A2K6JWJ2_RHIBE\nVRK1\tA0A2K6PNA1_RHIRO\nVRK1\tA0A2K6PNE5_RHIRO\nVRK1\tA0A2R9CAS2_PANPA\nVRK1\tA0A2R9CAX1_PANPA\nVRK1\tA0A2R9CDG1_PANPA\nVRK1\tA0A2U3X4L8_ODORO\nVRK1\tA0A2U3ZXR9_ODORO\nVRK1\tA0A2U4CRQ6_TURTR\nVRK1\tA0A2U4CRS6_TURTR\nVRK1\tA0A2U4CRU0_TURTR\nVRK1\tA0A2Y9G8P8_NEOSC\nVRK1\tA0A2Y9G8U8_NEOSC\nVRK1\tA0A2Y9NAH6_DELLE\nVRK1\tA0A2Y9NCG8_DELLE\nVRK1\tA0A2Y9NR69_DELLE\nVRK1\tA0A2Y9T7E1_PHYMC\nVRK1\tA0A2Y9TDD6_PHYMC\nVRK1\tA0A2Y9TFB3_PHYMC\nVRK1\tA0A341CK83_NEOAA\nVRK1\tA0A341CLS8_NEOAA\nVRK1\tA0A383ZUY0_BALAS\nVRK1\tA0A383ZUZ4_BALAS\nVRK1\tA0A383ZV55_BALAS\nVRK1\tA0A384CMT1_URSMA\nVRK1\tA0A384CMV2_URSMA\nVRK1\tA0A384CNS5_URSMA\nVRK1\tA0A3B5KKX4_TAKRU\nVRK1\tA0A3P9ATG5_9CICH\nVRK1\tA0A3P9AUD2_9CICH\nVRK1\tA0A3P9AUE2_9CICH\nVRK1\tA0A3P9AUG0_9CICH\nVRK1\tA0A3Q0CH22_MESAU\nVRK1\tA0A3Q0CM95_MESAU\nVRK1\tA0A3Q0CM99_MESAU\nVRK1\tA0A3Q0CMQ9_MESAU\nVRK1\tA0A3Q0CPA7_MESAU\nVRK1\tA0A3Q0EB07_TARSY\nVRK1\tA0A3Q0GF81_ALLSI\nVRK1\tA0A3Q1N7W2_BOVIN\nVRK1\tA0A3Q2HE58_HORSE\nVRK1\tA0A3Q2UCI0_CHICK\nVRK1\tA0A3Q7NUA1_CALUR\nVRK1\tA0A3Q7NUW2_CALUR\nVRK1\tA0A3Q7P4A8_CALUR\nVRK1\tA0A3Q7QAE8_CALUR\nVRK1\tA0A3Q7TF10_VULVU\nVRK1\tA0A3Q7U4N1_VULVU\nVRK1\tA0A3Q7X9R4_URSAR\nVRK1\tA0A3Q7XVP9_URSAR\nVRK1\tA0A452FQP4_CAPHI\nVRK1\tA0A452FQS1_CAPHI\nVRK1\tA0A452FR19_CAPHI\nVRK1\tA0A452IJY8_9SAUR\nVRK1\tA0A452RZ24_URSAM\nVRK1\tA0A452T483_URSMA\nVRK1\tA0A452T4F1_URSMA\nVRK1\tA0A493TLM7_ANAPP\nVRK1\tA0A493TZH0_ANAPP\nVRK1\tA0A4W2CPY0_BOBOX\nVRK1\tA0A4W2DYI4_BOBOX\nVRK1\tA0A4W2E6X7_BOBOX\nVRK1\tA0A4W2HSG7_BOBOX\nVRK1\tA0A4W4H668_ELEEL\nVRK1\tA0A4W4H673_ELEEL\nVRK1\tA0A4W4HAD8_ELEEL\nVRK1\tA0A4W4HDB7_ELEEL\nVRK1\tA0A4X2JU86_VOMUR\nVRK1\tA0A5B7DYM1_PORTR\nVRK1\tA0A5F4CHC1_CANLF\nVRK1\tA0A5F4DDS0_CANLF\nVRK1\tA0A5F4W0E2_CALJA\nVRK1\tA0A5F4W549_CALJA\nVRK1\tA0A5F4WGY1_CALJA\nVRK1\tA0A5F5XH82_FELCA\nVRK1\tA0A5F5XHT9_FELCA\nVRK1\tA0A5F5XWE1_FELCA\nVRK1\tA0A5F5XYZ6_FELCA\nVRK1\tA0A5F7ZIF6_MACMU\nVRK1\tA0A5F8A3Q7_MACMU\nVRK1\tA0A5F8H9C0_MONDO\nVRK1\tA0A5F9CRL9_RABIT\nVRK1\tA0A5F9CX00_RABIT\nVRK1\tA0A5F9D3K9_RABIT\nVRK1\tA0A5F9DBH5_RABIT\nVRK1\tA0A5F9DEB2_RABIT\nVRK1\tA0A5F9DH81_RABIT\nVRK1\tA0A5G3I3M7_XENTR\nVRK1\tA0A5G3I3R0_XENTR\nVRK1\tA0A5G3I500_XENTR\nVRK1\tA0A5G3JYK3_XENTR\nVRK1\tA0A5K1VGK3_CALJA\nVRK1\tA7MCR4_DANRE\nVRK1\tB5X850_SALSA\nVRK1\tB5XCK9_SALSA\nVRK1\tE2RCN2_CANLF\nVRK1\tF1NDK1_CHICK\nVRK1\tF1QA55_DANRE\nVRK1\tF1SAQ6_PIG\nVRK1\tF6Q7P3_XENTR\nVRK1\tF6QBG2_XENTR\nVRK1\tF6QF79_XENTR\nVRK1\tF6QLF8_MONDO\nVRK1\tF7C9Q3_XENTR\nVRK1\tF7CKX3_HORSE\nVRK1\tF7DW75_ORNAN\nVRK1\tF7F549_MACMU\nVRK1\tF7H395_CALJA\nVRK1\tG1KP35_ANOCA\nVRK1\tG1M6K9_AILME\nVRK1\tG1NKM2_MELGA\nVRK1\tG1PVY8_MYOLU\nVRK1\tG1QIR9_NOMLE\nVRK1\tG1TFM7_RABIT\nVRK1\tG3T2Q4_LOXAF\nVRK1\tG3UBU3_LOXAF\nVRK1\tG3WJQ2_SARHA\nVRK1\tG7PBF9_MACFA\nVRK1\tH0UX03_CAVPO\nVRK1\tH0XG50_OTOGA\nVRK1\tH0YJ50_HUMAN\nVRK1\tH0YJF7_HUMAN\nVRK1\tH0YJJ9_HUMAN\nVRK1\tH0ZR37_TAEGU\nVRK1\tH2M5B0_ORYLA\nVRK1\tH2Q8W5_PANTR\nVRK1\tH2T488_TAKRU\nVRK1\tH3BEY9_LATCH\nVRK1\tH3BEZ0_LATCH\nVRK1\tH9ZD73_MACMU\nVRK1\tI3MHG0_ICTTR\nVRK1\tK7EVI0_PONAB\nVRK1\tK7G2S1_PELSI\nVRK1\tL7MRU0_HORSE\nVRK1\tL7N2V5_XENTR\nVRK1\tM3W988_FELCA\nVRK1\tM3XVB8_MUSPF\nVRK1\tQ28H45_XENTR\nVRK1\tQ3TL56_MOUSE\nVRK1\tQ3UWH3_MOUSE\nVRK1\tQ4R439_MACFA\nVRK1\tQ6AYA2_RAT\nVRK1\tQ6P8D6_XENTR\nVRK1\tQ7SYZ0_XENLA\nVRK1\tQ8CA60_MOUSE\nVRK1\tT2M6K9_HYDVU\nVRK1\tU3E880_CALJA\nVRK1\tU3IFA9_ANAPP\nVRK1\tU3JIR1_FICAL\nVRK1\tU6CVT3_NEOVI\nVRK1\tV8P2Q4_OPHHA\nVRK1\tVRK1_BOVIN\nVRK1\tVRK1_DANRE\nVRK1\tVRK1_HUMAN\nVRK1\tVRK1_MOUSE\nVRK1\tW5NPJ7_SHEEP\nVRK1\tW5UL53_ICTPU\n','From\tTo\nMAPK10\tA0A096NBS6_PAPAN\nMAPK10\tA0A0D9QWE9_CHLSB\nMAPK10\tA0A0G2JEJ8_MOUSE\nMAPK10\tA0A0G2JG69_MOUSE\nMAPK10\tA0A0G2JGL2_MOUSE\nMAPK10\tA0A0R4I9H8_DANRE\nMAPK10\tA0A0R4IB91_DANRE\nMAPK10\tA0A0R4IJM3_DANRE\nMAPK10\tA0A0R4IKK9_DANRE\nMAPK10\tA0A0R4IN30_DANRE\nMAPK10\tA0A0U1RRS7_RAT\nMAPK10\tA0A0U1RRU3_RAT\nMAPK10\tA0A0U1RVI2_RAT\nMAPK10\tA0A0Y0DBV4_EPICO\nMAPK10\tA0A109QMH6_EPICO\nMAPK10\tA0A1A7X050_9TELE\nMAPK10\tA0A1A7XXM0_9TELE\nMAPK10\tA0A1A8AWQ2_NOTFU\nMAPK10\tA0A1A8C8S2_9TELE\nMAPK10\tA0A1A8DV07_9TELE\nMAPK10\tA0A1A8FCF8_9TELE\nMAPK10\tA0A1A8H4H3_9TELE\nMAPK10\tA0A1A8IXW1_NOTKU\nMAPK10\tA0A1A8KN22_NOTKU\nMAPK10\tA0A1A8L093_9TELE\nMAPK10\tA0A1A8RC32_9TELE\nMAPK10\tA0A1A8RQS1_9TELE\nMAPK10\tA0A1D5PXC6_CHICK\nMAPK10\tA0A1D5QMC7_MACMU\nMAPK10\tA0A1D5QS39_MACMU\nMAPK10\tA0A1L8FFC8_XENLA\nMAPK10\tA0A1L8HVU2_XENLA\nMAPK10\tA0A1P0B7D2_HUMAN\nMAPK10\tA0A1S2ZV64_ERIEU\nMAPK10\tA0A1S2ZV69_ERIEU\nMAPK10\tA0A1S2ZV73_ERIEU\nMAPK10\tA0A1S3FG53_DIPOR\nMAPK10\tA0A1S3FH08_DIPOR\nMAPK10\tA0A1S3FHR9_DIPOR\nMAPK10\tA0A1S3FI48_DIPOR\nMAPK10\tA0A1S3LTS0_SALSA\nMAPK10\tA0A1S3LTV9_SALSA\nMAPK10\tA0A1S3LU38_SALSA\nMAPK10\tA0A1S3LU44_SALSA\nMAPK10\tA0A1S3LU76_SALSA\nMAPK10\tA0A1S3LU81_SALSA\nMAPK10\tA0A1S3LUY0_SALSA\nMAPK10\tA0A1S3WDS9_ERIEU\nMAPK10\tA0A1S3WE21_ERIEU\nMAPK10\tA0A1U7SUE8_TARSY\nMAPK10\tA0A1U7T112_TARSY\nMAPK10\tA0A1U7T522_TARSY\nMAPK10\tA0A1V4JUT6_PATFA\nMAPK10\tA0A1V4JUX0_PATFA\nMAPK10\tA0A1W2PNF5_HUMAN\nMAPK10\tA0A1W2PPM4_HUMAN\nMAPK10\tA0A1W2PPW1_HUMAN\nMAPK10\tA0A1W2PPZ3_HUMAN\nMAPK10\tA0A1W2PQ03_HUMAN\nMAPK10\tA0A1W2PQX4_HUMAN\nMAPK10\tA0A1W2PRF2_HUMAN\nMAPK10\tA0A1W2PRG9_HUMAN\nMAPK10\tA0A1W2PRZ2_HUMAN\nMAPK10\tA0A218V301_9PASE\nMAPK10\tA0A286XDG8_CAVPO\nMAPK10\tA0A286XL72_CAVPO\nMAPK10\tA0A286Y591_CAVPO\nMAPK10\tA0A286YEN5_HUMAN\nMAPK10\tA0A286YEQ0_HUMAN\nMAPK10\tA0A286YEQ7_HUMAN\nMAPK10\tA0A286YES0_HUMAN\nMAPK10\tA0A286YES8_HUMAN\nMAPK10\tA0A286YES9_HUMAN\nMAPK10\tA0A286YEV3_HUMAN\nMAPK10\tA0A286YEV5_HUMAN\nMAPK10\tA0A286YEV8_HUMAN\nMAPK10\tA0A286YEW4_HUMAN\nMAPK10\tA0A286YEW9_HUMAN\nMAPK10\tA0A286YEX7_HUMAN\nMAPK10\tA0A286YEY0_HUMAN\nMAPK10\tA0A286YEY2_HUMAN\nMAPK10\tA0A286YEZ1_HUMAN\nMAPK10\tA0A286YF02_HUMAN\nMAPK10\tA0A286YF04_HUMAN\nMAPK10\tA0A286YF19_HUMAN\nMAPK10\tA0A286YF35_HUMAN\nMAPK10\tA0A286YF43_HUMAN\nMAPK10\tA0A286YF53_HUMAN\nMAPK10\tA0A286YF62_HUMAN\nMAPK10\tA0A286YF80_HUMAN\nMAPK10\tA0A286YF83_HUMAN\nMAPK10\tA0A286YF85_HUMAN\nMAPK10\tA0A286YF95_HUMAN\nMAPK10\tA0A286YF97_HUMAN\nMAPK10\tA0A286YFA3_HUMAN\nMAPK10\tA0A286YFA6_HUMAN\nMAPK10\tA0A286YFA7_HUMAN\nMAPK10\tA0A286YFB6_HUMAN\nMAPK10\tA0A286YFB7_HUMAN\nMAPK10\tA0A286YFC0_HUMAN\nMAPK10\tA0A286YFC3_HUMAN\nMAPK10\tA0A286YFD7_HUMAN\nMAPK10\tA0A286YFD9_HUMAN\nMAPK10\tA0A286YFE2_HUMAN\nMAPK10\tA0A286YFE9_HUMAN\nMAPK10\tA0A286YFH4_HUMAN\nMAPK10\tA0A286YFI3_HUMAN\nMAPK10\tA0A286YFJ0_HUMAN\nMAPK10\tA0A286YFJ4_HUMAN\nMAPK10\tA0A286YFJ6_HUMAN\nMAPK10\tA0A286YFK3_HUMAN\nMAPK10\tA0A286YFM6_HUMAN\nMAPK10\tA0A286YFN2_HUMAN\nMAPK10\tA0A287D549_ICTTR\nMAPK10\tA0A2I0MGR0_COLLI\nMAPK10\tA0A2I0MGR4_COLLI\nMAPK10\tA0A2I0MGT8_COLLI\nMAPK10\tA0A2I2U9C9_FELCA\nMAPK10\tA0A2I2UVJ2_FELCA\nMAPK10\tA0A2I2UZ15_FELCA\nMAPK10\tA0A2I3G256_NOMLE\nMAPK10\tA0A2I3G2P3_NOMLE\nMAPK10\tA0A2I3GA23_NOMLE\nMAPK10\tA0A2I3GGY8_NOMLE\nMAPK10\tA0A2I3GHY9_NOMLE\nMAPK10\tA0A2I3H930_NOMLE\nMAPK10\tA0A2I3HTR1_NOMLE\nMAPK10\tA0A2I3HXF1_NOMLE\nMAPK10\tA0A2I3LEY1_PAPAN\nMAPK10\tA0A2I3LFZ4_PAPAN\nMAPK10\tA0A2I3MB78_PAPAN\nMAPK10\tA0A2I3MKV8_PAPAN\nMAPK10\tA0A2I3MNI1_PAPAN\nMAPK10\tA0A2I3MSL8_PAPAN\nMAPK10\tA0A2I3RHF1_PANTR\nMAPK10\tA0A2I3RKW7_PANTR\nMAPK10\tA0A2I3RXF8_PANTR\nMAPK10\tA0A2I3RZ31_PANTR\nMAPK10\tA0A2I3SUE2_PANTR\nMAPK10\tA0A2I3SVN1_PANTR\nMAPK10\tA0A2I3TGM9_PANTR\nMAPK10\tA0A2I4B2E2_9TELE\nMAPK10\tA0A2I4B2H7_9TELE\nMAPK10\tA0A2I4B2H8_9TELE\nMAPK10\tA0A2I4B2I1_9TELE\nMAPK10\tA0A2I4B2I3_9TELE\nMAPK10\tA0A2J8L2M2_PANTR\nMAPK10\tA0A2J8L2V4_PANTR\nMAPK10\tA0A2J8V538_PONAB\nMAPK10\tA0A2K5C655_AOTNA\nMAPK10\tA0A2K5C673_AOTNA\nMAPK10\tA0A2K5C681_AOTNA\nMAPK10\tA0A2K5C6A6_AOTNA\nMAPK10\tA0A2K5C6C7_AOTNA\nMAPK10\tA0A2K5C6D2_AOTNA\nMAPK10\tA0A2K5C6E4_AOTNA\nMAPK10\tA0A2K5C6G3_AOTNA\nMAPK10\tA0A2K5IQP8_COLAP\nMAPK10\tA0A2K5IQT9_COLAP\nMAPK10\tA0A2K5IQZ5_COLAP\nMAPK10\tA0A2K5IR00_COLAP\nMAPK10\tA0A2K5IS01_COLAP\nMAPK10\tA0A2K5IS19_COLAP\nMAPK10\tA0A2K5IS24_COLAP\nMAPK10\tA0A2K5IS41_COLAP\nMAPK10\tA0A2K5NXB9_CERAT\nMAPK10\tA0A2K5NXF3_CERAT\nMAPK10\tA0A2K5NXM1_CERAT\nMAPK10\tA0A2K5NXM9_CERAT\nMAPK10\tA0A2K5NXN9_CERAT\nMAPK10\tA0A2K5NY19_CERAT\nMAPK10\tA0A2K5NY24_CERAT\nMAPK10\tA0A2K5PSL6_CEBCA\nMAPK10\tA0A2K5PST2_CEBCA\nMAPK10\tA0A2K5PST9_CEBCA\nMAPK10\tA0A2K5PTD7_CEBCA\nMAPK10\tA0A2K5PTF0_CEBCA\nMAPK10\tA0A2K5PTP2_CEBCA\nMAPK10\tA0A2K5PTP6_CEBCA\nMAPK10\tA0A2K5PTP8_CEBCA\nMAPK10\tA0A2K5PTQ7_CEBCA\nMAPK10\tA0A2K5TUK5_MACFA\nMAPK10\tA0A2K5TUX5_MACFA\nMAPK10\tA0A2K5TV02_MACFA\nMAPK10\tA0A2K5TVI5_MACFA\nMAPK10\tA0A2K5TVK6_MACFA\nMAPK10\tA0A2K5TVL4_MACFA\nMAPK10\tA0A2K5XSA7_MANLE\nMAPK10\tA0A2K5XSI5_MANLE\nMAPK10\tA0A2K5XSJ6_MANLE\nMAPK10\tA0A2K5XSP6_MANLE\nMAPK10\tA0A2K5XSQ9_MANLE\nMAPK10\tA0A2K5XSS9_MANLE\nMAPK10\tA0A2K5XST4_MANLE\nMAPK10\tA0A2K6DXZ7_MACNE\nMAPK10\tA0A2K6DY34_MACNE\nMAPK10\tA0A2K6DY41_MACNE\nMAPK10\tA0A2K6DY56_MACNE\nMAPK10\tA0A2K6DYC7_MACNE\nMAPK10\tA0A2K6DYF6_MACNE\nMAPK10\tA0A2K6DYG8_MACNE\nMAPK10\tA0A2K6DYH9_MACNE\nMAPK10\tA0A2K6G6Y6_PROCO\nMAPK10\tA0A2K6G6Y7_PROCO\nMAPK10\tA0A2K6G6Z7_PROCO\nMAPK10\tA0A2K6G706_PROCO\nMAPK10\tA0A2K6G712_PROCO\nMAPK10\tA0A2K6G715_PROCO\nMAPK10\tA0A2K6G723_PROCO\nMAPK10\tA0A2K6G724_PROCO\nMAPK10\tA0A2K6JVI7_RHIBE\nMAPK10\tA0A2K6JW84_RHIBE\nMAPK10\tA0A2K6JW89_RHIBE\nMAPK10\tA0A2K6JWB7_RHIBE\nMAPK10\tA0A2K6JWB9_RHIBE\nMAPK10\tA0A2K6JWC0_RHIBE\nMAPK10\tA0A2K6JWF4_RHIBE\nMAPK10\tA0A2K6JWI5_RHIBE\nMAPK10\tA0A2K6QJ59_RHIRO\nMAPK10\tA0A2K6QJ68_RHIRO\nMAPK10\tA0A2K6QJ79_RHIRO\nMAPK10\tA0A2K6QJ80_RHIRO\nMAPK10\tA0A2K6QJA7_RHIRO\nMAPK10\tA0A2K6QJB3_RHIRO\nMAPK10\tA0A2K6QJB9_RHIRO\nMAPK10\tA0A2K6QJC0_RHIRO\nMAPK10\tA0A2K6U1D6_SAIBB\nMAPK10\tA0A2K6U1X9_SAIBB\nMAPK10\tA0A2K6U1Z3_SAIBB\nMAPK10\tA0A2K6U2B1_SAIBB\nMAPK10\tA0A2K6U2C7_SAIBB\nMAPK10\tA0A2K6U2D8_SAIBB\nMAPK10\tA0A2K6U2E7_SAIBB\nMAPK10\tA0A2R8M774_CALJA\nMAPK10\tA0A2R8NBQ6_CALJA\nMAPK10\tA0A2R8PIA6_CALJA\nMAPK10\tA0A2R8PJL3_CALJA\nMAPK10\tA0A2R9CES1_PANPA\nMAPK10\tA0A2R9CEV4_PANPA\nMAPK10\tA0A2R9CEZ5_PANPA\nMAPK10\tA0A2R9CF30_PANPA\nMAPK10\tA0A2R9CF54_PANPA\nMAPK10\tA0A2R9CF85_PANPA\nMAPK10\tA0A2R9CHP0_PANPA\nMAPK10\tA0A2R9CHQ6_PANPA\nMAPK10\tA0A2R9CHR8_PANPA\nMAPK10\tA0A2U3VAW8_ODORO\nMAPK10\tA0A2U3VAX4_ODORO\nMAPK10\tA0A2U3XE93_LEPWE\nMAPK10\tA0A2U3XE94_LEPWE\nMAPK10\tA0A2U3XE98_LEPWE\nMAPK10\tA0A2U3Z9L9_ODORO\nMAPK10\tA0A2U4AMN9_TURTR\nMAPK10\tA0A2U4AMP1_TURTR\nMAPK10\tA0A2U4AMP6_TURTR\nMAPK10\tA0A2U4AMQ1_TURTR\nMAPK10\tA0A2U4AMR9_TURTR\nMAPK10\tA0A2U4AMS7_TURTR\nMAPK10\tA0A2U4AN31_TURTR\nMAPK10\tA0A2Y9M5L4_DELLE\nMAPK10\tA0A2Y9M6Q3_DELLE\nMAPK10\tA0A2Y9MEB4_DELLE\nMAPK10\tA0A2Y9MEB8_DELLE\nMAPK10\tA0A2Y9S213_PHYMC\nMAPK10\tA0A2Y9S232_PHYMC\nMAPK10\tA0A2Y9S8P8_PHYMC\nMAPK10\tA0A2Y9SA12_PHYMC\nMAPK10\tA0A337S2P6_FELCA\nMAPK10\tA0A337S5B1_FELCA\nMAPK10\tA0A337SLW1_FELCA\nMAPK10\tA0A337ST22_FELCA\nMAPK10\tA0A340XWQ9_LIPVE\nMAPK10\tA0A340XWR2_LIPVE\nMAPK10\tA0A340XZU0_LIPVE\nMAPK10\tA0A340XZU5_LIPVE\nMAPK10\tA0A340Y2C5_LIPVE\nMAPK10\tA0A340Y2X9_LIPVE\nMAPK10\tA0A341CYV1_NEOAA\nMAPK10\tA0A341CZ03_NEOAA\nMAPK10\tA0A341CZ09_NEOAA\nMAPK10\tA0A341CZW0_NEOAA\nMAPK10\tA0A341CZW4_NEOAA\nMAPK10\tA0A341D1L2_NEOAA\nMAPK10\tA0A345DKR3_9TELE\nMAPK10\tA0A383ZQL2_BALAS\nMAPK10\tA0A383ZQL7_BALAS\nMAPK10\tA0A383ZR66_BALAS\nMAPK10\tA0A384DBH6_URSMA\nMAPK10\tA0A384DBI1_URSMA\nMAPK10\tA0A384DBI8_URSMA\nMAPK10\tA0A384DC57_URSMA\nMAPK10\tA0A384DC84_URSMA\nMAPK10\tA0A3Q0D0T6_MESAU\nMAPK10\tA0A3Q0DTA9_TARSY\nMAPK10\tA0A3Q0DUT1_TARSY\nMAPK10\tA0A3Q1H2P6_9TELE\nMAPK10\tA0A3Q1LRM6_BOVIN\nMAPK10\tA0A3Q1LXD7_BOVIN\nMAPK10\tA0A3Q2H8T6_HORSE\nMAPK10\tA0A3Q2HAA3_HORSE\nMAPK10\tA0A3Q2HWU5_HORSE\nMAPK10\tA0A3Q2ICB6_HORSE\nMAPK10\tA0A3Q7Q867_CALUR\nMAPK10\tA0A3Q7QB22_CALUR\nMAPK10\tA0A3Q7QB32_CALUR\nMAPK10\tA0A3Q7QL17_CALUR\nMAPK10\tA0A3Q7RQM1_CALUR\nMAPK10\tA0A3Q7S4W6_VULVU\nMAPK10\tA0A3Q7T1X8_VULVU\nMAPK10\tA0A3Q7TFE6_VULVU\nMAPK10\tA0A3Q7TFF2_VULVU\nMAPK10\tA0A3Q7VX18_URSAR\nMAPK10\tA0A3Q7WKE9_URSAR\nMAPK10\tA0A3Q7WUV0_URSAR\nMAPK10\tA0A3Q7WUV5_URSAR\nMAPK10\tA0A3Q7XRH6_URSAR\nMAPK10\tA0A3Q7XRI0_URSAR\nMAPK10\tA0A419PMJ8_CLOSI\nMAPK10\tA0A452CDI5_BALAS\nMAPK10\tA0A452EBW1_CAPHI\nMAPK10\tA0A452ECR9_CAPHI\nMAPK10\tA0A452GUB2_9SAUR\nMAPK10\tA0A452GUD6_9SAUR\nMAPK10\tA0A452RTM8_URSAM\nMAPK10\tA0A452RTN8_URSAM\nMAPK10\tA0A452VLI0_URSMA\nMAPK10\tA0A452VLL5_URSMA\nMAPK10\tA0A452VLP0_URSMA\nMAPK10\tA0A499QM61_9TELE\nMAPK10\tA0A4W2D533_BOBOX\nMAPK10\tA0A4W2D554_BOBOX\nMAPK10\tA0A4W2ETA1_BOBOX\nMAPK10\tA0A4W2G5P0_BOBOX\nMAPK10\tA0A4W2G630_BOBOX\nMAPK10\tA0A4W2G9I4_BOBOX\nMAPK10\tA0A4W2I2A8_BOBOX\nMAPK10\tA0A4W2I2B3_BOBOX\nMAPK10\tA0A4W2I2C6_BOBOX\nMAPK10\tA0A4W4GAU9_ELEEL\nMAPK10\tA0A4W6C5U8_LATCA\nMAPK10\tA0A4W6C5Y0_LATCA\nMAPK10\tA0A4W6C6K6_LATCA\nMAPK10\tA0A4W6CC44_LATCA\nMAPK10\tA0A4W6CC89_LATCA\nMAPK10\tA0A4W6CCB4_LATCA\nMAPK10\tA0A4W6CD13_LATCA\nMAPK10\tA0A4W6CEH1_LATCA\nMAPK10\tA0A4W6CEW1_LATCA\nMAPK10\tA0A4W6CEX2_LATCA\nMAPK10\tA0A4X1TYC3_PIG\nMAPK10\tA0A4X1U0H3_PIG\nMAPK10\tA0A4X1U0N6_PIG\nMAPK10\tA0A4X1U0Z9_PIG\nMAPK10\tA0A4X1U2N7_PIG\nMAPK10\tA0A4X1U386_PIG\nMAPK10\tA0A4X1U3P3_PIG\nMAPK10\tA0A4X1U3Q5_PIG\nMAPK10\tA0A4X2LYT7_VOMUR\nMAPK10\tA0A4X2M5E3_VOMUR\nMAPK10\tA0A4X2M5J3_VOMUR\nMAPK10\tA0A4X2M6X7_VOMUR\nMAPK10\tA0A4X2M6X8_VOMUR\nMAPK10\tA0A4X2M6Z0_VOMUR\nMAPK10\tA0A4X2M727_VOMUR\nMAPK10\tA0A4X2M728_VOMUR\nMAPK10\tA0A4X2M736_VOMUR\nMAPK10\tA0A4Z2GKQ1_9TELE\nMAPK10\tA0A5B7I1T0_PORTR\nMAPK10\tA0A5F4C1B5_CANLF\nMAPK10\tA0A5F4CGN3_CANLF\nMAPK10\tA0A5F4CQY6_CANLF\nMAPK10\tA0A5F4D9T6_CANLF\nMAPK10\tA0A5F4DCF5_CANLF\nMAPK10\tA0A5F4DFQ7_CANLF\nMAPK10\tA0A5F5PFS9_HORSE\nMAPK10\tA0A5F5PGT7_HORSE\nMAPK10\tA0A5F5PKH3_HORSE\nMAPK10\tA0A5F5PSA5_HORSE\nMAPK10\tA0A5F5PTR8_HORSE\nMAPK10\tA0A5F8AKY4_MACMU\nMAPK10\tA0A5F8G214_MONDO\nMAPK10\tA0A5F8GE43_MONDO\nMAPK10\tA0A5F8MP75_MOUSE\nMAPK10\tA0A5F9C9X2_RABIT\nMAPK10\tA0A5F9CM49_RABIT\nMAPK10\tA0A5F9D907_RABIT\nMAPK10\tA0A5F9DEC5_RABIT\nMAPK10\tA0A5F9DFM8_RABIT\nMAPK10\tA0A5F9DG77_RABIT\nMAPK10\tA0A5G2QSE9_PIG\nMAPK10\tA0A5G2QTP1_PIG\nMAPK10\tA0A5G2QV03_PIG\nMAPK10\tA0A5G2R1U3_PIG\nMAPK10\tA0A5G2RGH7_PIG\nMAPK10\tA0A5G3H851_XENTR\nMAPK10\tA0A5G3HAQ5_XENTR\nMAPK10\tA0A5G3IUE3_XENTR\nMAPK10\tA4FV00_BOVIN\nMAPK10\tA4QP40_DANRE\nMAPK10\tA7MC39_XENTR\nMAPK10\tA8MPV1_HUMAN\nMAPK10\tD6R9C1_HUMAN\nMAPK10\tD6RAJ0_HUMAN\nMAPK10\tD6RAU3_HUMAN\nMAPK10\tD6RBH2_HUMAN\nMAPK10\tD6RC26_HUMAN\nMAPK10\tD6RCB1_HUMAN\nMAPK10\tD6RDG1_HUMAN\nMAPK10\tD6RFX8_HUMAN\nMAPK10\tD6RFZ7_HUMAN\nMAPK10\tD6RJF9_HUMAN\nMAPK10\tE2R9W9_CANLF\nMAPK10\tE9QN59_MOUSE\nMAPK10\tF1PMN2_CANLF\nMAPK10\tF1RW16_PIG\nMAPK10\tF6R294_HORSE\nMAPK10\tF6VWB1_ORNAN\nMAPK10\tF6ZA47_MONDO\nMAPK10\tF6ZRD6_XENTR\nMAPK10\tF7BTM6_CALJA\nMAPK10\tF7CYE3_XENTR\nMAPK10\tF7GN93_MACMU\nMAPK10\tF7HNH3_CALJA\nMAPK10\tF7HRW7_MACMU\nMAPK10\tF7HRW9_MACMU\nMAPK10\tF7HU18_CALJA\nMAPK10\tF7HWT3_CALJA\nMAPK10\tF7HZ50_CALJA\nMAPK10\tF8UFC0_SOLLC\nMAPK10\tF8W9R5_HUMAN\nMAPK10\tG1KLT0_ANOCA\nMAPK10\tG1LA16_AILME\nMAPK10\tG1N752_MELGA\nMAPK10\tG1PQV9_MYOLU\nMAPK10\tG1RDP5_NOMLE\nMAPK10\tG1SZR5_RABIT\nMAPK10\tG3T8R6_LOXAF\nMAPK10\tG3UNL0_LOXAF\nMAPK10\tG3VGH8_SARHA\nMAPK10\tG3VGH9_SARHA\nMAPK10\tG5C950_HETGA\nMAPK10\tG7P5U7_MACFA\nMAPK10\tH0VP43_CAVPO\nMAPK10\tH0XC62_OTOGA\nMAPK10\tH0YX11_TAEGU\nMAPK10\tH2U861_TAKRU\nMAPK10\tH3AKD9_LATCH\nMAPK10\tI3JLB1_ORENI\nMAPK10\tI3JLB2_ORENI\nMAPK10\tI3LZ97_ICTTR\nMAPK10\tI3NEN4_ICTTR\nMAPK10\tI6L9F3_HUMAN\nMAPK10\tK7FC81_PELSI\nMAPK10\tK7FC83_PELSI\nMAPK10\tK7GP94_PIG\nMAPK10\tK7GT04_PIG\nMAPK10\tM3VY50_FELCA\nMAPK10\tM3XT66_MUSPF\nMAPK10\tMK10_HUMAN\nMAPK10\tMK10_MOUSE\nMAPK10\tMK10_RAT\nMAPK10\tQ05B47_BOVIN\nMAPK10\tQ0VFK8_XENTR\nMAPK10\tQ32LV0_DANRE\nMAPK10\tQ3TQZ7_MOUSE\nMAPK10\tQ499Y8_HUMAN\nMAPK10\tQ78GB8_MOUSE\nMAPK10\tQ80W80_MOUSE\nMAPK10\tQ80W81_MOUSE\nMAPK10\tQ80W82_MOUSE\nMAPK10\tQ8BQV0_MOUSE\nMAPK10\tQ8C9D4_MOUSE\nMAPK10\tT2MF53_HYDVU\nMAPK10\tU3D021_CALJA\nMAPK10\tU3IDX6_ANAPP\nMAPK10\tU3JSR1_FICAL\nMAPK10\tV8PB87_OPHHA\nMAPK10\tW5P155_SHEEP\nMAPK10\tW9RWK8_9ROSA\n','From\tTo\nCSNK1G1\tA0A096NMB7_PAPAN\nCSNK1G1\tA0A0B2VBV5_TOXCA\nCSNK1G1\tA0A0D9RJ80_CHLSB\nCSNK1G1\tA0A0G2K7C4_RAT\nCSNK1G1\tA0A0K8TZ38_BACLA\nCSNK1G1\tA0A0R4IUQ5_DANRE\nCSNK1G1\tA0A0U1RNW0_MOUSE\nCSNK1G1\tA0A0U1RNX9_MOUSE\nCSNK1G1\tA0A0U1RP94_MOUSE\nCSNK1G1\tA0A0U1RPD7_MOUSE\nCSNK1G1\tA0A0U1RPL1_MOUSE\nCSNK1G1\tA0A0U1RPM8_MOUSE\nCSNK1G1\tA0A0U1RQ81_MOUSE\nCSNK1G1\tA0A0U1RQG9_HUMAN\nCSNK1G1\tA0A0U1RQH9_HUMAN\nCSNK1G1\tA0A0U1RQK2_HUMAN\nCSNK1G1\tA0A0U1RQK6_HUMAN\nCSNK1G1\tA0A0U1RQN5_HUMAN\nCSNK1G1\tA0A0U1RQT7_HUMAN\nCSNK1G1\tA0A0U1RQU4_HUMAN\nCSNK1G1\tA0A0U1RQY7_HUMAN\nCSNK1G1\tA0A0U1RR14_HUMAN\nCSNK1G1\tA0A0U1RR31_HUMAN\nCSNK1G1\tA0A0U1RRH5_HUMAN\nCSNK1G1\tA0A0V0WUJ2_9BILA\nCSNK1G1\tA0A0V0WUK2_9BILA\nCSNK1G1\tA0A0V1BGY9_TRISP\nCSNK1G1\tA0A0V1BHL6_TRISP\nCSNK1G1\tA0A0V1CPP7_TRIBR\nCSNK1G1\tA0A0V1PCF0_9BILA\nCSNK1G1\tA0A140LHR2_MOUSE\nCSNK1G1\tA0A140LIJ6_MOUSE\nCSNK1G1\tA0A151NGR7_ALLMI\nCSNK1G1\tA0A1A7WNS1_9TELE\nCSNK1G1\tA0A1A7XZU8_9TELE\nCSNK1G1\tA0A1A7ZSY6_NOTFU\nCSNK1G1\tA0A1A8CBX7_9TELE\nCSNK1G1\tA0A1A8EMN7_9TELE\nCSNK1G1\tA0A1A8HLX0_9TELE\nCSNK1G1\tA0A1A8I2M0_NOTKU\nCSNK1G1\tA0A1A8L6Q9_9TELE\nCSNK1G1\tA0A1A8PRR6_9TELE\nCSNK1G1\tA0A1A8Q2Q1_9TELE\nCSNK1G1\tA0A1A8U9R7_NOTFU\nCSNK1G1\tA0A1B8Y0F9_XENTR\nCSNK1G1\tA0A1D5PZ25_CHICK\nCSNK1G1\tA0A1D5QA83_MACMU\nCSNK1G1\tA0A1L8GT69_XENLA\nCSNK1G1\tA0A1S3EMR9_DIPOR\nCSNK1G1\tA0A1S3WNL0_ERIEU\nCSNK1G1\tA0A1U7QVI2_MESAU\nCSNK1G1\tA0A1U8C3C7_MESAU\nCSNK1G1\tA0A1U8CAS7_MESAU\nCSNK1G1\tA0A1U8CBE7_MESAU\nCSNK1G1\tA0A1U8CXR0_ALLSI\nCSNK1G1\tA0A218UYX6_9PASE\nCSNK1G1\tA0A250Y1A7_CASCN\nCSNK1G1\tA0A286XM20_CAVPO\nCSNK1G1\tA0A286XNN5_CAVPO\nCSNK1G1\tA0A286Y592_CAVPO\nCSNK1G1\tA0A286ZQ62_PIG\nCSNK1G1\tA0A287A139_PIG\nCSNK1G1\tA0A287B6C7_PIG\nCSNK1G1\tA0A287CSQ2_ICTTR\nCSNK1G1\tA0A287D126_ICTTR\nCSNK1G1\tA0A287D6G3_ICTTR\nCSNK1G1\tA0A287D7I2_ICTTR\nCSNK1G1\tA0A287DGG2_ICTTR\nCSNK1G1\tA0A2I0LLP3_COLLI\nCSNK1G1\tA0A2I2USW7_FELCA\nCSNK1G1\tA0A2I2UZ19_FELCA\nCSNK1G1\tA0A2I3GA22_NOMLE\nCSNK1G1\tA0A2I3GGP1_NOMLE\nCSNK1G1\tA0A2I3GZT9_NOMLE\nCSNK1G1\tA0A2I3H2S4_NOMLE\nCSNK1G1\tA0A2I3H6C4_NOMLE\nCSNK1G1\tA0A2I3HDH8_NOMLE\nCSNK1G1\tA0A2I3HEP9_NOMLE\nCSNK1G1\tA0A2I3HV41_NOMLE\nCSNK1G1\tA0A2I3LHC6_PAPAN\nCSNK1G1\tA0A2I3M3V1_PAPAN\nCSNK1G1\tA0A2I3M9I7_PAPAN\nCSNK1G1\tA0A2I3MCG5_PAPAN\nCSNK1G1\tA0A2I3MF58_PAPAN\nCSNK1G1\tA0A2I3MLH0_PAPAN\nCSNK1G1\tA0A2I3MMG1_PAPAN\nCSNK1G1\tA0A2I3MYY4_PAPAN\nCSNK1G1\tA0A2I3N1D5_PAPAN\nCSNK1G1\tA0A2I3RW33_PANTR\nCSNK1G1\tA0A2I3S2X6_PANTR\nCSNK1G1\tA0A2I3SU59_PANTR\nCSNK1G1\tA0A2I3T0G2_PANTR\nCSNK1G1\tA0A2I3T5B7_PANTR\nCSNK1G1\tA0A2I3TL02_PANTR\nCSNK1G1\tA0A2I3TUD5_PANTR\nCSNK1G1\tA0A2I4ANI0_9TELE\nCSNK1G1\tA0A2I4ANJ2_9TELE\nCSNK1G1\tA0A2I4ANK2_9TELE\nCSNK1G1\tA0A2I4ANM6_9TELE\nCSNK1G1\tA0A2J8NRL2_PANTR\nCSNK1G1\tA0A2K5EMQ1_AOTNA\nCSNK1G1\tA0A2K5EMQ4_AOTNA\nCSNK1G1\tA0A2K5EMQ5_AOTNA\nCSNK1G1\tA0A2K5EMR5_AOTNA\nCSNK1G1\tA0A2K5EMS4_AOTNA\nCSNK1G1\tA0A2K5EMS7_AOTNA\nCSNK1G1\tA0A2K5EMT0_AOTNA\nCSNK1G1\tA0A2K5EMW7_AOTNA\nCSNK1G1\tA0A2K5K6W9_COLAP\nCSNK1G1\tA0A2K5K6X1_COLAP\nCSNK1G1\tA0A2K5K6X8_COLAP\nCSNK1G1\tA0A2K5K6Z8_COLAP\nCSNK1G1\tA0A2K5K700_COLAP\nCSNK1G1\tA0A2K5K711_COLAP\nCSNK1G1\tA0A2K5NVX5_CERAT\nCSNK1G1\tA0A2K5NVZ7_CERAT\nCSNK1G1\tA0A2K5NW05_CERAT\nCSNK1G1\tA0A2K5NW10_CERAT\nCSNK1G1\tA0A2K5NWC2_CERAT\nCSNK1G1\tA0A2K5NWF4_CERAT\nCSNK1G1\tA0A2K5NWH0_CERAT\nCSNK1G1\tA0A2K5NWH8_CERAT\nCSNK1G1\tA0A2K5NWI4_CERAT\nCSNK1G1\tA0A2K5QYR2_CEBCA\nCSNK1G1\tA0A2K5QYS2_CEBCA\nCSNK1G1\tA0A2K5QYT9_CEBCA\nCSNK1G1\tA0A2K5QYU2_CEBCA\nCSNK1G1\tA0A2K5QYU5_CEBCA\nCSNK1G1\tA0A2K5QYW3_CEBCA\nCSNK1G1\tA0A2K5QYX6_CEBCA\nCSNK1G1\tA0A2K5QYX9_CEBCA\nCSNK1G1\tA0A2K5QZ16_CEBCA\nCSNK1G1\tA0A2K5TZG2_MACFA\nCSNK1G1\tA0A2K5TZH0_MACFA\nCSNK1G1\tA0A2K5TZI2_MACFA\nCSNK1G1\tA0A2K5TZJ6_MACFA\nCSNK1G1\tA0A2K5TZK2_MACFA\nCSNK1G1\tA0A2K5TZL5_MACFA\nCSNK1G1\tA0A2K5TZN3_MACFA\nCSNK1G1\tA0A2K5TZQ1_MACFA\nCSNK1G1\tA0A2K5YF96_MANLE\nCSNK1G1\tA0A2K5YFA0_MANLE\nCSNK1G1\tA0A2K5YFA8_MANLE\nCSNK1G1\tA0A2K5YFC8_MANLE\nCSNK1G1\tA0A2K5YFD1_MANLE\nCSNK1G1\tA0A2K5YFD8_MANLE\nCSNK1G1\tA0A2K5YFE1_MANLE\nCSNK1G1\tA0A2K5YFF1_MANLE\nCSNK1G1\tA0A2K5YFF4_MANLE\nCSNK1G1\tA0A2K5YFG2_MANLE\nCSNK1G1\tA0A2K6E8G9_MACNE\nCSNK1G1\tA0A2K6E8J1_MACNE\nCSNK1G1\tA0A2K6E8J3_MACNE\nCSNK1G1\tA0A2K6E8J6_MACNE\nCSNK1G1\tA0A2K6E8J9_MACNE\nCSNK1G1\tA0A2K6E8L0_MACNE\nCSNK1G1\tA0A2K6E8L1_MACNE\nCSNK1G1\tA0A2K6E8L3_MACNE\nCSNK1G1\tA0A2K6E8L8_MACNE\nCSNK1G1\tA0A2K6E8M8_MACNE\nCSNK1G1\tA0A2K6GGB9_PROCO\nCSNK1G1\tA0A2K6GGC3_PROCO\nCSNK1G1\tA0A2K6GGC8_PROCO\nCSNK1G1\tA0A2K6GGD2_PROCO\nCSNK1G1\tA0A2K6GGD6_PROCO\nCSNK1G1\tA0A2K6GGD9_PROCO\nCSNK1G1\tA0A2K6GGE4_PROCO\nCSNK1G1\tA0A2K6GGE7_PROCO\nCSNK1G1\tA0A2K6MU65_RHIBE\nCSNK1G1\tA0A2K6MU70_RHIBE\nCSNK1G1\tA0A2K6MU73_RHIBE\nCSNK1G1\tA0A2K6MU81_RHIBE\nCSNK1G1\tA0A2K6MU83_RHIBE\nCSNK1G1\tA0A2K6QLT5_RHIRO\nCSNK1G1\tA0A2K6QLU7_RHIRO\nCSNK1G1\tA0A2K6QLU8_RHIRO\nCSNK1G1\tA0A2K6QLV4_RHIRO\nCSNK1G1\tA0A2K6QLV6_RHIRO\nCSNK1G1\tA0A2K6QLW1_RHIRO\nCSNK1G1\tA0A2K6QLW4_RHIRO\nCSNK1G1\tA0A2K6QLW8_RHIRO\nCSNK1G1\tA0A2K6QLX7_RHIRO\nCSNK1G1\tA0A2K6TML3_SAIBB\nCSNK1G1\tA0A2K6TMP0_SAIBB\nCSNK1G1\tA0A2K6TMP6_SAIBB\nCSNK1G1\tA0A2K6TMQ2_SAIBB\nCSNK1G1\tA0A2K6TMS6_SAIBB\nCSNK1G1\tA0A2K6TMT1_SAIBB\nCSNK1G1\tA0A2K6TMV0_SAIBB\nCSNK1G1\tA0A2R8MI70_CALJA\nCSNK1G1\tA0A2R8MT47_CALJA\nCSNK1G1\tA0A2R8NDG0_CALJA\nCSNK1G1\tA0A2R8Q753_DANRE\nCSNK1G1\tA0A2R9BBQ7_PANPA\nCSNK1G1\tA0A2R9BBR2_PANPA\nCSNK1G1\tA0A2R9BBS5_PANPA\nCSNK1G1\tA0A2R9BFG2_PANPA\nCSNK1G1\tA0A2R9BFH2_PANPA\nCSNK1G1\tA0A2R9BFK8_PANPA\nCSNK1G1\tA0A2R9BJY8_PANPA\nCSNK1G1\tA0A2R9BJZ9_PANPA\nCSNK1G1\tA0A2R9BK04_PANPA\nCSNK1G1\tA0A2U3WKQ4_ODORO\nCSNK1G1\tA0A2U3WKQ8_ODORO\nCSNK1G1\tA0A2U3WKV5_ODORO\nCSNK1G1\tA0A2U3XTZ1_LEPWE\nCSNK1G1\tA0A2U3ZQY4_ODORO\nCSNK1G1\tA0A2U4CDU6_TURTR\nCSNK1G1\tA0A2U4CDZ0_TURTR\nCSNK1G1\tA0A2U4CE04_TURTR\nCSNK1G1\tA0A2U4CE20_TURTR\nCSNK1G1\tA0A2Y9HJ34_NEOSC\nCSNK1G1\tA0A2Y9NK63_DELLE\nCSNK1G1\tA0A2Y9NN66_DELLE\nCSNK1G1\tA0A2Y9P030_DELLE\nCSNK1G1\tA0A337SD00_FELCA\nCSNK1G1\tA0A337SK94_FELCA\nCSNK1G1\tA0A340WMK5_LIPVE\nCSNK1G1\tA0A340WPR8_LIPVE\nCSNK1G1\tA0A340WUN7_LIPVE\nCSNK1G1\tA0A340WV94_LIPVE\nCSNK1G1\tA0A341AJQ8_NEOAA\nCSNK1G1\tA0A341AJZ6_NEOAA\nCSNK1G1\tA0A341AMZ1_NEOAA\nCSNK1G1\tA0A383YT44_BALAS\nCSNK1G1\tA0A384DIF8_URSMA\nCSNK1G1\tA0A384DIJ4_URSMA\nCSNK1G1\tA0A3B1IXD9_ASTMX\nCSNK1G1\tA0A3B1JTD5_ASTMX\nCSNK1G1\tA0A3B1K624_ASTMX\nCSNK1G1\tA0A3B3HY36_ORYLA\nCSNK1G1\tA0A3B3IC24_ORYLA\nCSNK1G1\tA0A3B4D8G0_PYGNA\nCSNK1G1\tA0A3B4EL68_PYGNA\nCSNK1G1\tA0A3B4EL73_PYGNA\nCSNK1G1\tA0A3B5K9C2_TAKRU\nCSNK1G1\tA0A3M9XXB6_9PEZI\nCSNK1G1\tA0A3Q0CYY2_MESAU\nCSNK1G1\tA0A3Q0CZM4_MESAU\nCSNK1G1\tA0A3Q0E1M0_TARSY\nCSNK1G1\tA0A3Q0G7J6_ALLSI\nCSNK1G1\tA0A3Q0G7K5_ALLSI\nCSNK1G1\tA0A3Q0G865_ALLSI\nCSNK1G1\tA0A3Q0GB29_ALLSI\nCSNK1G1\tA0A3Q0GBT7_ALLSI\nCSNK1G1\tA0A3Q0GBU2_ALLSI\nCSNK1G1\tA0A3Q1LYU9_BOVIN\nCSNK1G1\tA0A3Q1M2N3_BOVIN\nCSNK1G1\tA0A3Q1M652_BOVIN\nCSNK1G1\tA0A3Q1MTN4_BOVIN\nCSNK1G1\tA0A3Q2I7A5_HORSE\nCSNK1G1\tA0A3Q2IBB2_HORSE\nCSNK1G1\tA0A3Q2KT86_HORSE\nCSNK1G1\tA0A3Q3AWM4_CHICK\nCSNK1G1\tA0A3Q7Q2F4_CALUR\nCSNK1G1\tA0A3Q7Q2K2_CALUR\nCSNK1G1\tA0A3Q7Q2K8_CALUR\nCSNK1G1\tA0A3Q7Q2S5_CALUR\nCSNK1G1\tA0A3Q7Q2T1_CALUR\nCSNK1G1\tA0A3Q7QDE6_CALUR\nCSNK1G1\tA0A3Q7RI95_CALUR\nCSNK1G1\tA0A3Q7T032_VULVU\nCSNK1G1\tA0A3Q7T0G7_VULVU\nCSNK1G1\tA0A3Q7TEF4_URSAR\nCSNK1G1\tA0A3Q7TEF9_URSAR\nCSNK1G1\tA0A3Q7TKA4_VULVU\nCSNK1G1\tA0A3Q7TKQ6_VULVU\nCSNK1G1\tA0A3Q7U1H1_URSAR\nCSNK1G1\tA0A3Q7U1H6_URSAR\nCSNK1G1\tA0A3Q7U3U8_VULVU\nCSNK1G1\tA0A3Q7UD24_URSAR\nCSNK1G1\tA0A3Q7URI3_URSAR\nCSNK1G1\tA0A3Q7URI9_URSAR\nCSNK1G1\tA0A3Q7V894_URSAR\nCSNK1G1\tA0A3R7CPQ3_CLOSI\nCSNK1G1\tA0A452E782_CAPHI\nCSNK1G1\tA0A452E787_CAPHI\nCSNK1G1\tA0A452SBR3_URSAM\nCSNK1G1\tA0A452SBR5_URSAM\nCSNK1G1\tA0A452SBS8_URSAM\nCSNK1G1\tA0A452SBV1_URSAM\nCSNK1G1\tA0A452SBV5_URSAM\nCSNK1G1\tA0A452SBV7_URSAM\nCSNK1G1\tA0A452SBV9_URSAM\nCSNK1G1\tA0A452SBW2_URSAM\nCSNK1G1\tA0A452SBW4_URSAM\nCSNK1G1\tA0A455BSR0_PHYMC\nCSNK1G1\tA0A4C1V802_9NEOP\nCSNK1G1\tA0A4C1V8Y9_9NEOP\nCSNK1G1\tA0A4W2CSV6_BOBOX\nCSNK1G1\tA0A4W2CSW4_BOBOX\nCSNK1G1\tA0A4W2ED17_BOBOX\nCSNK1G1\tA0A4W2GL52_BOBOX\nCSNK1G1\tA0A4W2GM06_BOBOX\nCSNK1G1\tA0A4W2ICG2_BOBOX\nCSNK1G1\tA0A4W5QI44_9TELE\nCSNK1G1\tA0A4W5QPC2_9TELE\nCSNK1G1\tA0A4W6EQ03_LATCA\nCSNK1G1\tA0A4W6ESK1_LATCA\nCSNK1G1\tA0A4W6ETE5_LATCA\nCSNK1G1\tA0A4W6ETG8_LATCA\nCSNK1G1\tA0A4W6EUC3_LATCA\nCSNK1G1\tA0A4W6EUD9_LATCA\nCSNK1G1\tA0A4W6EVJ9_LATCA\nCSNK1G1\tA0A4W6EVM6_LATCA\nCSNK1G1\tA0A4X1TY97_PIG\nCSNK1G1\tA0A4X1U0W2_PIG\nCSNK1G1\tA0A4X1U0Y8_PIG\nCSNK1G1\tA0A4X1U3S4_PIG\nCSNK1G1\tA0A4X1U3T3_PIG\nCSNK1G1\tA0A4X2LGZ4_VOMUR\nCSNK1G1\tA0A4Z2GI78_9TELE\nCSNK1G1\tA0A4Z2GK45_9TELE\nCSNK1G1\tA0A5B7DTZ3_PORTR\nCSNK1G1\tA0A5F4CIZ2_CANLF\nCSNK1G1\tA0A5F4CRM5_CANLF\nCSNK1G1\tA0A5F4CUB9_CANLF\nCSNK1G1\tA0A5F4D2Z7_CANLF\nCSNK1G1\tA0A5F5PJS2_HORSE\nCSNK1G1\tA0A5F5PL37_HORSE\nCSNK1G1\tA0A5F5Q0A8_HORSE\nCSNK1G1\tA0A5F5XV19_FELCA\nCSNK1G1\tA0A5F8HH72_MONDO\nCSNK1G1\tA0A5F8HLP7_MONDO\nCSNK1G1\tA0A5F9C1N6_RABIT\nCSNK1G1\tA0A5F9C8S4_RABIT\nCSNK1G1\tA0A5F9D7J4_RABIT\nCSNK1G1\tA0A5F9DBJ7_RABIT\nCSNK1G1\tA0A5F9DH79_RABIT\nCSNK1G1\tA5YBM6_HORSE\nCSNK1G1\tA5YBM7_HORSE\nCSNK1G1\tD3Z2A2_MOUSE\nCSNK1G1\tE1BNW0_BOVIN\nCSNK1G1\tE2QU92_CANLF\nCSNK1G1\tE2QUB1_CANLF\nCSNK1G1\tE2QUB4_CANLF\nCSNK1G1\tF1PLV2_CANLF\nCSNK1G1\tF1S0A1_PIG\nCSNK1G1\tF6ZCQ7_XENTR\nCSNK1G1\tF7B3B2_XENTR\nCSNK1G1\tF7BZR5_MONDO\nCSNK1G1\tF7CMI0_HORSE\nCSNK1G1\tF7CMP4_HORSE\nCSNK1G1\tF7EGZ0_CALJA\nCSNK1G1\tF7H417_MACMU\nCSNK1G1\tF7HDF6_MACMU\nCSNK1G1\tF7HDF8_MACMU\nCSNK1G1\tF7I3V3_CALJA\nCSNK1G1\tF7I6C0_CALJA\nCSNK1G1\tG1L735_AILME\nCSNK1G1\tG1MR83_MELGA\nCSNK1G1\tG1MRA7_MELGA\nCSNK1G1\tG1PTL2_MYOLU\nCSNK1G1\tG1RPE4_NOMLE\nCSNK1G1\tG1T2F6_RABIT\nCSNK1G1\tG3TLB8_LOXAF\nCSNK1G1\tG3UUG5_MELGA\nCSNK1G1\tG3X2P9_SARHA\nCSNK1G1\tH0UTI9_CAVPO\nCSNK1G1\tH0XGB9_OTOGA\nCSNK1G1\tH0Z2K8_TAEGU\nCSNK1G1\tH2L7T9_ORYLA\nCSNK1G1\tH2NNG8_PONAB\nCSNK1G1\tH2Q9K7_PANTR\nCSNK1G1\tH2UZS9_TAKRU\nCSNK1G1\tH3B5G8_LATCH\nCSNK1G1\tH3B5G9_LATCH\nCSNK1G1\tH9ENM1_MACMU\nCSNK1G1\tH9GC86_ANOCA\nCSNK1G1\tI3JEG1_ORENI\nCSNK1G1\tI3JEG2_ORENI\nCSNK1G1\tI3M9T8_ICTTR\nCSNK1G1\tK7BCK9_PANTR\nCSNK1G1\tK7FSN8_PELSI\nCSNK1G1\tK7FSP7_PELSI\nCSNK1G1\tKC1G1_BOVIN\nCSNK1G1\tKC1G1_CHICK\nCSNK1G1\tKC1G1_DANRE\nCSNK1G1\tKC1G1_HUMAN\nCSNK1G1\tKC1G1_MACFA\nCSNK1G1\tKC1G1_MOUSE\nCSNK1G1\tKC1G1_RAT\nCSNK1G1\tKC1G1_XENLA\nCSNK1G1\tM3W1Q7_FELCA\nCSNK1G1\tM3XNB5_MUSPF\nCSNK1G1\tQ6GL54_XENTR\nCSNK1G1\tQ6P2B2_MOUSE\nCSNK1G1\tQ8IXA3_HUMAN\nCSNK1G1\tU3DAC9_CALJA\nCSNK1G1\tU3EF14_CALJA\nCSNK1G1\tU3I9V0_ANAPP\nCSNK1G1\tU3K5C8_FICAL\nCSNK1G1\tU3KQA5_HUMAN\nCSNK1G1\tU3KQB3_HUMAN\nCSNK1G1\tV8NXA5_OPHHA\nCSNK1G1\tV8NY24_OPHHA\nCSNK1G1\tW5L4W5_ASTMX\nCSNK1G1\tW5QHU6_SHEEP\n','From\tTo\nSTK11\tA0A034WKP5_BACDO\nSTK11\tA0A087WT72_HUMAN\nSTK11\tA0A0A0MWD6_PAPAN\nSTK11\tA0A0A1WRE4_ZEUCU\nSTK11\tA0A0A9XWP5_LYGHE\nSTK11\tA0A0D9RIP5_CHLSB\nSTK11\tA0A0H2UI02_RAT\nSTK11\tA0A0K2TWJ6_LEPSM\nSTK11\tA0A0K8WFP8_BACLA\nSTK11\tA0A0P6K5H0_HETGA\nSTK11\tA0A0S2Z4C7_HUMAN\nSTK11\tA0A0S2Z4D1_HUMAN\nSTK11\tA0A0S7JRI9_9TELE\nSTK11\tA0A0S7JRJ6_9TELE\nSTK11\tA0A0S7JRM8_9TELE\nSTK11\tA0A0S7JRV5_9TELE\nSTK11\tA0A0V0RNL7_9BILA\nSTK11\tA0A0V0TTC3_9BILA\nSTK11\tA0A0V0WN55_9BILA\nSTK11\tA0A0V0ZEI2_9BILA\nSTK11\tA0A0V1AZR9_TRISP\nSTK11\tA0A0V1CK93_TRIBR\nSTK11\tA0A0V1EWS9_TRIPS\nSTK11\tA0A0V1G059_TRIPS\nSTK11\tA0A0V1HW14_9BILA\nSTK11\tA0A0V1J4K3_TRIPS\nSTK11\tA0A0V1KBG5_TRIPS\nSTK11\tA0A0V1L627_9BILA\nSTK11\tA0A0V1N1S8_9BILA\nSTK11\tA0A0V1PKD1_9BILA\nSTK11\tA0A151P0S1_ALLMI\nSTK11\tA0A1A7Z680_9TELE\nSTK11\tA0A1A8CA05_9TELE\nSTK11\tA0A1A8DVK0_9TELE\nSTK11\tA0A1A8GWB1_9TELE\nSTK11\tA0A1A8I0M3_NOTKU\nSTK11\tA0A1A8KHS4_NOTKU\nSTK11\tA0A1A8QH31_9TELE\nSTK11\tA0A1A8RIT1_9TELE\nSTK11\tA0A1A8UJ30_NOTFU\nSTK11\tA0A1B0GUW2_HUMAN\nSTK11\tA0A1J4JB73_9EUKA\nSTK11\tA0A1J4JM38_9EUKA\nSTK11\tA0A1J4JQS9_9EUKA\nSTK11\tA0A1L8HNA4_XENLA\nSTK11\tA0A1L8HWM5_XENLA\nSTK11\tA0A1S2ZUX0_ERIEU\nSTK11\tA0A1S3FNY7_DIPOR\nSTK11\tA0A1S3FPI7_DIPOR\nSTK11\tA0A1U7RGA4_ALLSI\nSTK11\tA0A1U7RR66_ALLSI\nSTK11\tA0A1U7UNM0_TARSY\nSTK11\tA0A1U8CFZ5_MESAU\nSTK11\tA0A1U8CIM0_MESAU\nSTK11\tA0A1U8CSD6_MESAU\nSTK11\tA0A1V4KQA9_PATFA\nSTK11\tA0A218UB66_9PASE\nSTK11\tA0A250YIF5_CASCN\nSTK11\tA0A2B4RDI5_STYPI\nSTK11\tA0A2D0TA11_ICTPU\nSTK11\tA0A2I0LJP6_COLLI\nSTK11\tA0A2I3MWV9_PAPAN\nSTK11\tA0A2I4C9V1_9TELE\nSTK11\tA0A2K5F9M1_AOTNA\nSTK11\tA0A2K5F9M6_AOTNA\nSTK11\tA0A2K5F9P8_AOTNA\nSTK11\tA0A2K5JDT7_COLAP\nSTK11\tA0A2K5NH94_CERAT\nSTK11\tA0A2K5NHJ4_CERAT\nSTK11\tA0A2K5PFW7_CEBCA\nSTK11\tA0A2K5PFX0_CEBCA\nSTK11\tA0A2K5VHM0_MACFA\nSTK11\tA0A2K5VHS2_MACFA\nSTK11\tA0A2K5ZHT6_MANLE\nSTK11\tA0A2K5ZHZ2_MANLE\nSTK11\tA0A2K6BFK3_MACNE\nSTK11\tA0A2K6BFL2_MACNE\nSTK11\tA0A2K6EPR4_PROCO\nSTK11\tA0A2K6EPS9_PROCO\nSTK11\tA0A2K6K695_RHIBE\nSTK11\tA0A2K6K6B3_RHIBE\nSTK11\tA0A2K6K6B7_RHIBE\nSTK11\tA0A2K6NMW4_RHIRO\nSTK11\tA0A2K6SDZ3_SAIBB\nSTK11\tA0A2L1DGD8_MOUSE\nSTK11\tA0A2P8Z3J3_BLAGE\nSTK11\tA0A2R8M716_CALJA\nSTK11\tA0A2R9CM33_PANPA\nSTK11\tA0A2R9CM47_PANPA\nSTK11\tA0A2R9CP11_PANPA\nSTK11\tA0A2S2NL28_SCHGA\nSTK11\tA0A2S2QL85_9HEMI\nSTK11\tA0A2U3VKM4_ODORO\nSTK11\tA0A2U4BQK9_TURTR\nSTK11\tA0A2U4BQL8_TURTR\nSTK11\tA0A2U4BQM5_TURTR\nSTK11\tA0A2U4BQX7_TURTR\nSTK11\tA0A2Y9FTB2_PHYMC\nSTK11\tA0A2Y9IAX9_NEOSC\nSTK11\tA0A2Y9IB14_NEOSC\nSTK11\tA0A2Y9MGU1_DELLE\nSTK11\tA0A2Y9MGU7_DELLE\nSTK11\tA0A2Y9MHI5_DELLE\nSTK11\tA0A2Y9MK85_DELLE\nSTK11\tA0A2Y9MQW2_DELLE\nSTK11\tA0A2Y9MW56_DELLE\nSTK11\tA0A2Y9MW61_DELLE\nSTK11\tA0A340XN58_LIPVE\nSTK11\tA0A341AZ50_NEOAA\nSTK11\tA0A341AZ91_NEOAA\nSTK11\tA0A341B1V4_NEOAA\nSTK11\tA0A341B2K6_NEOAA\nSTK11\tA0A383ZNP4_BALAS\nSTK11\tA0A384DS39_URSMA\nSTK11\tA0A3Q7N9S9_CALUR\nSTK11\tA0A3Q7N9T4_CALUR\nSTK11\tA0A3Q7NAE9_CALUR\nSTK11\tA0A3Q7NE83_CALUR\nSTK11\tA0A3Q7U8T5_URSAR\nSTK11\tA0A3Q7UAL2_VULVU\nSTK11\tA0A3Q7UKK0_URSAR\nSTK11\tA0A3Q7UP23_VULVU\nSTK11\tA0A3Q7V194_URSAR\nSTK11\tA0A3Q7V5T7_VULVU\nSTK11\tA0A419PD12_CLOSI\nSTK11\tA0A452FJN7_CAPHI\nSTK11\tA0A452FJW6_CAPHI\nSTK11\tA0A452J5U7_9SAUR\nSTK11\tA0A452J879_CHICK\nSTK11\tA0A452TH73_URSMA\nSTK11\tA0A452TH78_URSMA\nSTK11\tA0A452THF7_URSMA\nSTK11\tA0A452THR4_URSMA\nSTK11\tA0A455C6M8_PHYMC\nSTK11\tA0A455C8Z4_PHYMC\nSTK11\tA0A455CBN3_PHYMC\nSTK11\tA0A4W2BZT9_BOBOX\nSTK11\tA0A4W2IPC4_BOBOX\nSTK11\tA0A4X1VW23_PIG\nSTK11\tA0A4X2LDL9_VOMUR\nSTK11\tA0A4Z2IKS2_9TELE\nSTK11\tA0A5B7DXS5_PORTR\nSTK11\tA0A5F4CFR9_CANLF\nSTK11\tA0A5F4WHL5_CALJA\nSTK11\tA0A5F5XJN1_FELCA\nSTK11\tA0A5F7ZZ39_MACMU\nSTK11\tA0A5G3H7L6_XENTR\nSTK11\tB9ZZS0_MESAU\nSTK11\tC1BWP1_ESOLU\nSTK11\tD4P556_HUMAN\nSTK11\tD6RGT4_MOUSE\nSTK11\tE1BCU9_BOVIN\nSTK11\tE9Q811_MOUSE\nSTK11\tF1R050_DANRE\nSTK11\tF1S6Q9_PIG\nSTK11\tF6TBE8_XENTR\nSTK11\tF6WNS2_MOUSE\nSTK11\tF6XGD4_CANLF\nSTK11\tF7CWP1_HORSE\nSTK11\tF7DR75_XENTR\nSTK11\tF7ERL9_MACMU\nSTK11\tG1L944_AILME\nSTK11\tG1MXQ2_MELGA\nSTK11\tG1PR95_MYOLU\nSTK11\tG1QL69_NOMLE\nSTK11\tG3VYM4_SARHA\nSTK11\tH0VPR3_CAVPO\nSTK11\tH0WVJ2_OTOGA\nSTK11\tH0YRL5_TAEGU\nSTK11\tH2MKS9_ORYLA\nSTK11\tH2NWT0_PONAB\nSTK11\tH2QET9_PANTR\nSTK11\tH2TJI9_TAKRU\nSTK11\tH3ATN2_LATCH\nSTK11\tH9ETP1_MACMU\nSTK11\tH9GKK5_ANOCA\nSTK11\tI3M0R8_ICTTR\nSTK11\tK7CK92_PANTR\nSTK11\tK7E2U8_MONDO\nSTK11\tK7EMR0_HUMAN\nSTK11\tK7EP59_HUMAN\nSTK11\tK7EQN8_HUMAN\nSTK11\tK7F4G2_PELSI\nSTK11\tM3WJD3_FELCA\nSTK11\tM3YB83_MUSPF\nSTK11\tQ28CV3_XENTR\nSTK11\tQ3V4A1_MOUSE\nSTK11\tQ568P5_DANRE\nSTK11\tQ5ZEZ1_HUMAN\nSTK11\tQ8T9L5_DROME\nSTK11\tQ9NS52_HUMAN\nSTK11\tSTK11_CHICK\nSTK11\tSTK11_HUMAN\nSTK11\tSTK11_MOUSE\nSTK11\tSTK11_RAT\nSTK11\tSTK11_XENLA\nSTK11\tT2MDA0_HYDVU\nSTK11\tU3D9S0_CALJA\nSTK11\tU3IHA0_ANAPP\nSTK11\tU3KHU8_FICAL\nSTK11\tU6DA25_NEOVI\nSTK11\tV5GQ02_ANOGL\nSTK11\tV8NEE7_OPHHA\nSTK11\tV8NFL0_OPHHA\nSTK11\tW5PNM2_SHEEP\nSTK11\tW8CAC6_CERCA\n','From\tTo\nPIM2\tA0A024QYW7_HUMAN\nPIM2\tA0A096N047_PAPAN\nPIM2\tA0A0B2V459_TOXCA\nPIM2\tA0A0D9RL32_CHLSB\nPIM2\tA0A0S7M6A0_9TELE\nPIM2\tA0A1A7X876_9TELE\nPIM2\tA0A1A7ZAQ4_NOTFU\nPIM2\tA0A1A8D5F5_9TELE\nPIM2\tA0A1A8EMM1_9TELE\nPIM2\tA0A1A8JGQ3_NOTKU\nPIM2\tA0A1A8N1L5_9TELE\nPIM2\tA0A1A8NAT2_9TELE\nPIM2\tA0A1A8SFR2_9TELE\nPIM2\tA0A1Q9DH67_SYMMI\nPIM2\tA0A1S3A522_ERIEU\nPIM2\tA0A1S3GGA9_DIPOR\nPIM2\tA0A1U7RBA8_MESAU\nPIM2\tA0A1U7U333_TARSY\nPIM2\tA0A250YFC1_CASCN\nPIM2\tA0A2B4SE58_STYPI\nPIM2\tA0A2B4SEX9_STYPI\nPIM2\tA0A2K5D3G7_AOTNA\nPIM2\tA0A2K5D3I9_AOTNA\nPIM2\tA0A2K5I7N6_COLAP\nPIM2\tA0A2K5MZQ8_CERAT\nPIM2\tA0A2K5QAX9_CEBCA\nPIM2\tA0A2K5QB21_CEBCA\nPIM2\tA0A2K5Y2W8_MANLE\nPIM2\tA0A2K6CRW2_MACNE\nPIM2\tA0A2K6F9Z7_PROCO\nPIM2\tA0A2K6MVG9_RHIBE\nPIM2\tA0A2K6Q2U4_RHIRO\nPIM2\tA0A2K6S5G9_SAIBB\nPIM2\tA0A2K6S5H2_SAIBB\nPIM2\tA0A2R8MQS9_CALJA\nPIM2\tA0A2R9C4I5_PANPA\nPIM2\tA0A2U3VPA9_ODORO\nPIM2\tA0A2U3Y3Y7_LEPWE\nPIM2\tA0A2U4C198_TURTR\nPIM2\tA0A2Y9FEJ8_PHYMC\nPIM2\tA0A2Y9G9Q8_NEOSC\nPIM2\tA0A2Y9MCP0_DELLE\nPIM2\tA0A2Y9MEV3_DELLE\nPIM2\tA0A2Y9SA27_PHYMC\nPIM2\tA0A340WQ66_LIPVE\nPIM2\tA0A341DDR1_NEOAA\nPIM2\tA0A341DDY0_NEOAA\nPIM2\tA0A369J496_HYPMA\nPIM2\tA0A383ZZ83_BALAS\nPIM2\tA0A384CJE3_URSMA\nPIM2\tA0A3Q7PBF7_CALUR\nPIM2\tA0A3Q7QYJ6_VULVU\nPIM2\tA0A3Q7W7U5_URSAR\nPIM2\tA0A3Q7XZN5_URSAR\nPIM2\tA0A452E4W4_CAPHI\nPIM2\tA0A452E4Y2_CAPHI\nPIM2\tA0A452GKL4_9SAUR\nPIM2\tA0A452R5P3_URSAM\nPIM2\tA0A4W2BKT6_BOBOX\nPIM2\tA0A4W6FYM8_LATCA\nPIM2\tA0A4X1VJM5_PIG\nPIM2\tA0A4X1VJX1_PIG\nPIM2\tA0A4X2KPF9_VOMUR\nPIM2\tA7YW25_BOVIN\nPIM2\tC9J8V9_HUMAN\nPIM2\tF1M419_RAT\nPIM2\tF1P8K8_CANLF\nPIM2\tF6RP02_BOVIN\nPIM2\tF6S4X1_MACMU\nPIM2\tF6TUF2_MONDO\nPIM2\tF6WGR1_MOUSE\nPIM2\tF7C4B8_CALJA\nPIM2\tF7CKA2_HORSE\nPIM2\tG1M182_AILME\nPIM2\tG1PD38_MYOLU\nPIM2\tG1R8Y1_NOMLE\nPIM2\tG1TA52_RABIT\nPIM2\tG3T074_LOXAF\nPIM2\tG3VGN8_SARHA\nPIM2\tG3VGN9_SARHA\nPIM2\tG5BJI8_HETGA\nPIM2\tH0V718_CAVPO\nPIM2\tH0X0X4_OTOGA\nPIM2\tH2PVK4_PONAB\nPIM2\tH2QYK5_PANTR\nPIM2\tH3AAB4_LATCH\nPIM2\tH3AAB5_LATCH\nPIM2\tH9F7L5_MACMU\nPIM2\tH9GCX5_ANOCA\nPIM2\tI3L8X4_PIG\nPIM2\tI3MMG2_ICTTR\nPIM2\tM3X0B0_FELCA\nPIM2\tM3YQL9_MUSPF\nPIM2\tPIM2_DANRE\nPIM2\tPIM2_HUMAN\nPIM2\tPIM2_MOUSE\nPIM2\tQ8R1Z0_MOUSE\nPIM2\tU6D3G4_NEOVI\nPIM2\tW5PPD2_SHEEP\nPIM2\tW5UTW6_ICTPU\nPIM2\tZ4YHY2_DANRE\n','From\tTo\nPASK\tA0A096NEQ0_PAPAN\nPASK\tA0A0B2VCX1_TOXCA\nPASK\tA0A0B2VD33_TOXCA\nPASK\tA0A0D9R0L2_CHLSB\nPASK\tA0A0K6FL13_9AGAM\nPASK\tA0A0S7J3D0_9TELE\nPASK\tA0A0S7LDZ8_9TELE\nPASK\tA0A0S7LE01_9TELE\nPASK\tA0A0S7LE03_9TELE\nPASK\tA0A0S7LE30_9TELE\nPASK\tA0A0S7LE77_9TELE\nPASK\tA0A0S7M2Z9_9TELE\nPASK\tA0A151MKA4_ALLMI\nPASK\tA0A1A7Z0Q7_9TELE\nPASK\tA0A1A7Z0Z6_9TELE\nPASK\tA0A1A8BCI0_9TELE\nPASK\tA0A1A8E841_9TELE\nPASK\tA0A1A8E9L8_9TELE\nPASK\tA0A1A8EAQ2_9TELE\nPASK\tA0A1A8EC22_9TELE\nPASK\tA0A1A8FZP1_9TELE\nPASK\tA0A1A8G1V8_9TELE\nPASK\tA0A1A8HZQ4_NOTKU\nPASK\tA0A1A8KL07_NOTKU\nPASK\tA0A1A8L5U1_9TELE\nPASK\tA0A1A8L6S7_9TELE\nPASK\tA0A1A8L8G7_9TELE\nPASK\tA0A1A8NRU1_9TELE\nPASK\tA0A1A8NSS5_9TELE\nPASK\tA0A1A8RQ81_9TELE\nPASK\tA0A1A8V410_NOTFU\nPASK\tA0A1C7MPB4_9FUNG\nPASK\tA0A1C7N3W4_9FUNG\nPASK\tA0A1D5Q1Y3_MACMU\nPASK\tA0A1S3AAR2_ERIEU\nPASK\tA0A1S3GK96_DIPOR\nPASK\tA0A1S3M0P4_SALSA\nPASK\tA0A1S3M0Z0_SALSA\nPASK\tA0A1S3M1F9_SALSA\nPASK\tA0A1U7SFW0_ALLSI\nPASK\tA0A1U7SR92_TARSY\nPASK\tA0A1V4J4Z2_PATFA\nPASK\tA0A1V4J550_PATFA\nPASK\tA0A218UA52_9PASE\nPASK\tA0A286ZR02_PIG\nPASK\tA0A286ZU04_PIG\nPASK\tA0A2B4RR45_STYPI\nPASK\tA0A2D0TA41_ICTPU\nPASK\tA0A2D0TA43_ICTPU\nPASK\tA0A2I0LMF6_COLLI\nPASK\tA0A2I3G6C8_NOMLE\nPASK\tA0A2I3H1T2_NOMLE\nPASK\tA0A2I3H5Q1_NOMLE\nPASK\tA0A2I3HNM7_NOMLE\nPASK\tA0A2I3M1X7_PAPAN\nPASK\tA0A2I3N3W4_PAPAN\nPASK\tA0A2J8IUU6_PANTR\nPASK\tA0A2J8IUW6_PANTR\nPASK\tA0A2K5CYI4_AOTNA\nPASK\tA0A2K5CYJ0_AOTNA\nPASK\tA0A2K5J5Y0_COLAP\nPASK\tA0A2K5J668_COLAP\nPASK\tA0A2K5J690_COLAP\nPASK\tA0A2K5N4Z1_CERAT\nPASK\tA0A2K5N4Z3_CERAT\nPASK\tA0A2K5N568_CERAT\nPASK\tA0A2K5QMA1_CEBCA\nPASK\tA0A2K5QMA9_CEBCA\nPASK\tA0A2K5WIL5_MACFA\nPASK\tA0A2K5WIN7_MACFA\nPASK\tA0A2K5WIR3_MACFA\nPASK\tA0A2K5WIZ7_MACFA\nPASK\tA0A2K6A899_MANLE\nPASK\tA0A2K6A8A5_MANLE\nPASK\tA0A2K6A8B4_MANLE\nPASK\tA0A2K6BKY8_MACNE\nPASK\tA0A2K6BL43_MACNE\nPASK\tA0A2K6BL74_MACNE\nPASK\tA0A2K6BL92_MACNE\nPASK\tA0A2K6LA88_RHIBE\nPASK\tA0A2K6LA95_RHIBE\nPASK\tA0A2K6LAA9_RHIBE\nPASK\tA0A2K6PBD7_RHIRO\nPASK\tA0A2K6PBK9_RHIRO\nPASK\tA0A2K6U4X1_SAIBB\nPASK\tA0A2K6U520_SAIBB\nPASK\tA0A2R8RNQ8_DANRE\nPASK\tA0A2R9B473_PANPA\nPASK\tA0A2R9B476_PANPA\nPASK\tA0A2U3ZT96_ODORO\nPASK\tA0A2U4BB40_TURTR\nPASK\tA0A2Y9G725_NEOSC\nPASK\tA0A2Y9MXX7_DELLE\nPASK\tA0A2Y9SPT5_PHYMC\nPASK\tA0A340Y8N1_LIPVE\nPASK\tA0A341CH45_NEOAA\nPASK\tA0A384AC98_BALAS\nPASK\tA0A384C0X8_URSMA\nPASK\tA0A3B3I225_ORYLA\nPASK\tA0A3Q0D227_MESAU\nPASK\tA0A3Q0HQH3_ALLSI\nPASK\tA0A3Q2KQB2_HORSE\nPASK\tA0A3Q2UPY4_CHICK\nPASK\tA0A3Q7QF28_CALUR\nPASK\tA0A3Q7SN50_VULVU\nPASK\tA0A3Q7UER4_URSAR\nPASK\tA0A452DTR6_CAPHI\nPASK\tA0A452DTW1_CAPHI\nPASK\tA0A452GT55_9SAUR\nPASK\tA0A452GT60_9SAUR\nPASK\tA0A452U1L8_URSMA\nPASK\tA0A452U1T6_URSMA\nPASK\tA0A452U1U5_URSMA\nPASK\tA0A452U1V9_URSMA\nPASK\tA0A455ARR6_PHYMC\nPASK\tA0A455ATN7_PHYMC\nPASK\tA0A4W2D761_BOBOX\nPASK\tA0A4W2D766_BOBOX\nPASK\tA0A4W2D7A3_BOBOX\nPASK\tA0A4W2GAP7_BOBOX\nPASK\tA0A4W2GBL9_BOBOX\nPASK\tA0A4W2GCR2_BOBOX\nPASK\tA0A4W2GCR7_BOBOX\nPASK\tA0A4W6E8S7_LATCA\nPASK\tA0A4X2M2K5_VOMUR\nPASK\tA0A4Z2F6F3_9TELE\nPASK\tA0A5F4C005_CANLF\nPASK\tA0A5F4CWD6_CANLF\nPASK\tA0A5F5XHS6_FELCA\nPASK\tA0A5F9CMP5_RABIT\nPASK\tA0A5G2QI62_PIG\nPASK\tA0A5G2QQS8_PIG\nPASK\tA0A5G3KT28_XENTR\nPASK\tA0A5G3KT51_XENTR\nPASK\tB0S5Z9_DANRE\nPASK\tB9EKC1_MOUSE\nPASK\tC9J1Z3_HUMAN\nPASK\tC9J4R3_HUMAN\nPASK\tD3YUI8_MOUSE\nPASK\tE1BJM5_BOVIN\nPASK\tE2RCE4_CANLF\nPASK\tF1N9K6_CHICK\nPASK\tF1SI05_PIG\nPASK\tF1SI07_PIG\nPASK\tF6U335_HORSE\nPASK\tF7EYX0_MONDO\nPASK\tF7FPL3_MACMU\nPASK\tF7FPL8_MACMU\nPASK\tF8WEB4_HUMAN\nPASK\tG1KA32_ANOCA\nPASK\tG1ME21_AILME\nPASK\tG1MX25_MELGA\nPASK\tG1P549_MYOLU\nPASK\tG1RTV9_NOMLE\nPASK\tG1TSK1_RABIT\nPASK\tG3SSZ7_LOXAF\nPASK\tH0V7F2_CAVPO\nPASK\tH0WH29_OTOGA\nPASK\tH2MM68_ORYLA\nPASK\tH2P964_PONAB\nPASK\tH2UG59_TAKRU\nPASK\tH7C0L7_HUMAN\nPASK\tH9FRT7_MACMU\nPASK\tK7AIV6_PANTR\nPASK\tK7GB19_PELSI\nPASK\tK7GB36_PELSI\nPASK\tM3VVQ7_FELCA\nPASK\tM3XM55_MUSPF\nPASK\tPASK_HUMAN\nPASK\tPASK_MOUSE\nPASK\tQ5PQT0_RAT\nPASK\tQ95SH0_DROME\nPASK\tSTK39_RAT\nPASK\tT2M7I6_HYDVU\nPASK\tU3C5F2_CALJA\nPASK\tU3D691_CALJA\nPASK\tU3FQK5_CALJA\nPASK\tU3JLE5_FICAL\nPASK\tV8PFR8_OPHHA\nPASK\tW5Q9F0_SHEEP\nPASK\tX1WEV2_DANRE\nPASK\tX1WF79_DANRE\nPASK\tX1WFR8_DANRE\n','From\tTo\nMKNK2\tA0A0D9RHI0_CHLSB\nMKNK2\tA0A0G2JDJ0_MOUSE\nMKNK2\tA0A0G2JGQ2_MOUSE\nMKNK2\tA0A0G2JGS6_MOUSE\nMKNK2\tA0A0P6J0B9_HETGA\nMKNK2\tA0A0R4IZX7_MOUSE\nMKNK2\tA0A0S7G7E7_9TELE\nMKNK2\tA0A0S7KEK1_9TELE\nMKNK2\tA0A0V0RKB2_9BILA\nMKNK2\tA0A0V0RKV9_9BILA\nMKNK2\tA0A0V0TLH7_9BILA\nMKNK2\tA0A0V0UVI3_9BILA\nMKNK2\tA0A0V1B182_TRISP\nMKNK2\tA0A0V1B1H7_TRISP\nMKNK2\tA0A0V1B1I8_TRISP\nMKNK2\tA0A0V1D4V0_TRIBR\nMKNK2\tA0A0V1D4Y7_TRIBR\nMKNK2\tA0A0V1L093_9BILA\nMKNK2\tA0A0V1L138_9BILA\nMKNK2\tA0A0V1P3G1_9BILA\nMKNK2\tA0A0V1P3K7_9BILA\nMKNK2\tA0A1A7XZ62_9TELE\nMKNK2\tA0A1A7ZT85_NOTFU\nMKNK2\tA0A1A8DZC6_9TELE\nMKNK2\tA0A1A8EYH6_9TELE\nMKNK2\tA0A1A8J9U7_NOTKU\nMKNK2\tA0A1A8L2Q2_9TELE\nMKNK2\tA0A1A8NSP6_9TELE\nMKNK2\tA0A1L8HWS4_XENLA\nMKNK2\tA0A1S2ZUT5_ERIEU\nMKNK2\tA0A1S3FNW3_DIPOR\nMKNK2\tA0A1S3FNX3_DIPOR\nMKNK2\tA0A1S3FQG4_DIPOR\nMKNK2\tA0A1S3FQZ3_DIPOR\nMKNK2\tA0A1U7RAX2_MESAU\nMKNK2\tA0A218UR69_9PASE\nMKNK2\tA0A250YII1_CASCN\nMKNK2\tA0A286ZTA3_PIG\nMKNK2\tA0A2B4SQX0_STYPI\nMKNK2\tA0A2C5XDR9_9PEZI\nMKNK2\tA0A2D0QW17_ICTPU\nMKNK2\tA0A2D0QYB6_ICTPU\nMKNK2\tA0A2I0LPC1_COLLI\nMKNK2\tA0A2I3M0I4_PAPAN\nMKNK2\tA0A2I3MGS1_PAPAN\nMKNK2\tA0A2I3MWQ5_PAPAN\nMKNK2\tA0A2I3ND17_PAPAN\nMKNK2\tA0A2I3S701_PANTR\nMKNK2\tA0A2I4B4Z7_9TELE\nMKNK2\tA0A2K5F5J5_AOTNA\nMKNK2\tA0A2K5F5K1_AOTNA\nMKNK2\tA0A2K5F5P6_AOTNA\nMKNK2\tA0A2K5F5V7_AOTNA\nMKNK2\tA0A2K5K9I5_COLAP\nMKNK2\tA0A2K5K9J1_COLAP\nMKNK2\tA0A2K5NPT7_CERAT\nMKNK2\tA0A2K5NQE8_CERAT\nMKNK2\tA0A2K5NQF4_CERAT\nMKNK2\tA0A2K5QCW1_CEBCA\nMKNK2\tA0A2K5QD02_CEBCA\nMKNK2\tA0A2K5QD35_CEBCA\nMKNK2\tA0A2K5QD64_CEBCA\nMKNK2\tA0A2K5W4W7_MACFA\nMKNK2\tA0A2K5W511_MACFA\nMKNK2\tA0A2K5YXJ4_MANLE\nMKNK2\tA0A2K5YXK2_MANLE\nMKNK2\tA0A2K6APV8_MACNE\nMKNK2\tA0A2K6G3E6_PROCO\nMKNK2\tA0A2K6G3E8_PROCO\nMKNK2\tA0A2K6G3G0_PROCO\nMKNK2\tA0A2K6LUY6_RHIBE\nMKNK2\tA0A2K6P997_RHIRO\nMKNK2\tA0A2K6P9A0_RHIRO\nMKNK2\tA0A2K6P9A8_RHIRO\nMKNK2\tA0A2K6V5D9_SAIBB\nMKNK2\tA0A2K6V5F1_SAIBB\nMKNK2\tA0A2R9C7I7_PANPA\nMKNK2\tA0A2R9C7J3_PANPA\nMKNK2\tA0A2U3VKK1_ODORO\nMKNK2\tA0A2U3VKK2_ODORO\nMKNK2\tA0A2U3YJJ8_LEPWE\nMKNK2\tA0A2Y9EP15_PHYMC\nMKNK2\tA0A2Y9EPU1_PHYMC\nMKNK2\tA0A2Y9I9W9_NEOSC\nMKNK2\tA0A2Y9I9Z5_NEOSC\nMKNK2\tA0A2Y9IA00_NEOSC\nMKNK2\tA0A2Y9IAK3_NEOSC\nMKNK2\tA0A2Y9LTK7_DELLE\nMKNK2\tA0A340XG10_LIPVE\nMKNK2\tA0A340XN30_LIPVE\nMKNK2\tA0A341AZ85_NEOAA\nMKNK2\tA0A383ZNE3_BALAS\nMKNK2\tA0A383ZNH2_BALAS\nMKNK2\tA0A384DRB2_URSMA\nMKNK2\tA0A3B3HHB8_ORYLA\nMKNK2\tA0A3Q2H312_HORSE\nMKNK2\tA0A3Q7N6D2_CALUR\nMKNK2\tA0A3Q7NAG1_CALUR\nMKNK2\tA0A3Q7NEA0_CALUR\nMKNK2\tA0A3Q7UKT7_URSAR\nMKNK2\tA0A3Q7V5Z8_VULVU\nMKNK2\tA0A452G8X3_CAPHI\nMKNK2\tA0A452G8Z5_CAPHI\nMKNK2\tA0A452H568_9SAUR\nMKNK2\tA0A452SR55_URSAM\nMKNK2\tA0A452TIU0_URSMA\nMKNK2\tA0A452TJD6_URSMA\nMKNK2\tA0A455AYG9_PHYMC\nMKNK2\tA0A455B0F5_PHYMC\nMKNK2\tA0A455BC72_PHYMC\nMKNK2\tA0A493TC39_ANAPP\nMKNK2\tA0A4W2G3Y3_BOBOX\nMKNK2\tA0A4W2G489_BOBOX\nMKNK2\tA0A4W2G6X1_BOBOX\nMKNK2\tA0A4W2HS14_BOBOX\nMKNK2\tA0A4W3GSJ8_CALMI\nMKNK2\tA0A4W3H7N7_CALMI\nMKNK2\tA0A4W4GG08_ELEEL\nMKNK2\tA0A4W4GHJ0_ELEEL\nMKNK2\tA0A4W4GHN6_ELEEL\nMKNK2\tA0A4W4GHP1_ELEEL\nMKNK2\tA0A4W4GL59_ELEEL\nMKNK2\tA0A4W4GLA9_ELEEL\nMKNK2\tA0A4X1VU70_PIG\nMKNK2\tA0A4X2LHI8_VOMUR\nMKNK2\tA0A4Z2GYL0_9TELE\nMKNK2\tA0A5F7ZHR6_MACMU\nMKNK2\tA0JPI2_XENLA\nMKNK2\tA7E2R7_MOUSE\nMKNK2\tC1BND6_CALRO\nMKNK2\tE1BCH7_BOVIN\nMKNK2\tE2QXV2_CANLF\nMKNK2\tF1NCK3_CHICK\nMKNK2\tF6R569_MONDO\nMKNK2\tF6WLW2_XENTR\nMKNK2\tF6WP09_MACMU\nMKNK2\tG1K3G1_XENTR\nMKNK2\tG1LFA3_AILME\nMKNK2\tG1MTH3_MELGA\nMKNK2\tG1PBR9_MYOLU\nMKNK2\tG1QN79_NOMLE\nMKNK2\tG1QN84_NOMLE\nMKNK2\tG3SQR9_LOXAF\nMKNK2\tG3U4X3_LOXAF\nMKNK2\tG3U5B1_LOXAF\nMKNK2\tG3W1W2_SARHA\nMKNK2\tH0VS21_CAVPO\nMKNK2\tH0WXK7_OTOGA\nMKNK2\tH0YPT4_TAEGU\nMKNK2\tH2LYV0_ORYLA\nMKNK2\tH2NWV9_PONAB\nMKNK2\tH2SLE4_TAKRU\nMKNK2\tH3ATW8_LATCH\nMKNK2\tH9EP11_MACMU\nMKNK2\tH9G3L0_ANOCA\nMKNK2\tH9Z801_MACMU\nMKNK2\tI2CTS4_MACMU\nMKNK2\tI3JUY9_ORENI\nMKNK2\tI3MJ57_ICTTR\nMKNK2\tK7A6E5_PANTR\nMKNK2\tK7AW93_PANTR\nMKNK2\tK7C336_PANTR\nMKNK2\tK7EIN7_HUMAN\nMKNK2\tK7EJ98_HUMAN\nMKNK2\tK7EK27_HUMAN\nMKNK2\tK7ELG3_HUMAN\nMKNK2\tK7ELN0_HUMAN\nMKNK2\tK7EMP5_HUMAN\nMKNK2\tK7GAC5_PELSI\nMKNK2\tM3WGB2_FELCA\nMKNK2\tM3Y6P1_MUSPF\nMKNK2\tMKNK2_HUMAN\nMKNK2\tMKNK2_MOUSE\nMKNK2\tMKNK2_RAT\nMKNK2\tMKNK2_XENLA\nMKNK2\tMKNK2_XENTR\nMKNK2\tQ3TPM2_MOUSE\nMKNK2\tQ9NV89_HUMAN\nMKNK2\tT2M533_HYDVU\nMKNK2\tU3DL15_CALJA\nMKNK2\tU3E4C4_CALJA\nMKNK2\tU3KH46_FICAL\nMKNK2\tU6DLW1_NEOVI\nMKNK2\tV8NZH1_OPHHA\nMKNK2\tV8P024_OPHHA\nMKNK2\tV8P164_OPHHA\nMKNK2\tV8P1E5_OPHHA\nMKNK2\tV9KII9_CALMI\nMKNK2\tW5PZ04_SHEEP\nMKNK2\tW6CR13_MOUSE\n','From\tTo\nMAPKAPK3\tA0A024R2W7_HUMAN\nMAPKAPK3\tA0A060KQ01_PANTA\nMAPKAPK3\tA0A087XFE9_POEFO\nMAPKAPK3\tA0A096N543_PAPAN\nMAPKAPK3\tA0A0B2VJ29_TOXCA\nMAPKAPK3\tA0A0V0RKL8_9BILA\nMAPKAPK3\tA0A0V0TU23_9BILA\nMAPKAPK3\tA0A0V0TU57_9BILA\nMAPKAPK3\tA0A0V0TVA9_9BILA\nMAPKAPK3\tA0A0V0WFV6_9BILA\nMAPKAPK3\tA0A0V0WGD6_9BILA\nMAPKAPK3\tA0A0V0XZF1_TRIPS\nMAPKAPK3\tA0A0V0ZK96_9BILA\nMAPKAPK3\tA0A0V1B137_TRISP\nMAPKAPK3\tA0A0V1CJV3_TRIBR\nMAPKAPK3\tA0A0V1E504_TRIPS\nMAPKAPK3\tA0A0V1I4T6_9BILA\nMAPKAPK3\tA0A0V1JKG3_TRIPS\nMAPKAPK3\tA0A0V1L603_9BILA\nMAPKAPK3\tA0A0V1M7S7_9BILA\nMAPKAPK3\tA0A0V1M847_9BILA\nMAPKAPK3\tA0A0V1NMI0_9BILA\nMAPKAPK3\tA0A151MAP8_ALLMI\nMAPKAPK3\tA0A1L8GHD7_XENLA\nMAPKAPK3\tA0A1S3G5Z5_DIPOR\nMAPKAPK3\tA0A1S3WGQ9_ERIEU\nMAPKAPK3\tA0A1S3WHG2_ERIEU\nMAPKAPK3\tA0A1U7QIY8_MESAU\nMAPKAPK3\tA0A1U7S169_ALLSI\nMAPKAPK3\tA0A1U7S3L8_ALLSI\nMAPKAPK3\tA0A1U7TCT7_TARSY\nMAPKAPK3\tA0A1V4K8W5_PATFA\nMAPKAPK3\tA0A1V4KAD3_PATFA\nMAPKAPK3\tA0A218UQV4_9PASE\nMAPKAPK3\tA0A250YEF9_CASCN\nMAPKAPK3\tA0A286X7R8_CAVPO\nMAPKAPK3\tA0A287CRU3_ICTTR\nMAPKAPK3\tA0A2I0LRA9_COLLI\nMAPKAPK3\tA0A2J8TH56_PONAB\nMAPKAPK3\tA0A2K5F2R6_AOTNA\nMAPKAPK3\tA0A2K5JB68_COLAP\nMAPKAPK3\tA0A2K5M295_CERAT\nMAPKAPK3\tA0A2K5M297_CERAT\nMAPKAPK3\tA0A2K5Q9F9_CEBCA\nMAPKAPK3\tA0A2K5Q9L3_CEBCA\nMAPKAPK3\tA0A2K5ZQJ5_MANLE\nMAPKAPK3\tA0A2K6BTL0_MACNE\nMAPKAPK3\tA0A2K6GI44_PROCO\nMAPKAPK3\tA0A2K6KLG8_RHIBE\nMAPKAPK3\tA0A2K6KLH5_RHIBE\nMAPKAPK3\tA0A2K6PPM5_RHIRO\nMAPKAPK3\tA0A2K6PPP9_RHIRO\nMAPKAPK3\tA0A2K6UNB7_SAIBB\nMAPKAPK3\tA0A2N6NTZ2_BEABA\nMAPKAPK3\tA0A2R8QPN8_DANRE\nMAPKAPK3\tA0A2R9A884_PANPA\nMAPKAPK3\tA0A2U3VVA3_ODORO\nMAPKAPK3\tA0A2U4CCI5_TURTR\nMAPKAPK3\tA0A2U4CCI7_TURTR\nMAPKAPK3\tA0A2Y9EZM8_PHYMC\nMAPKAPK3\tA0A2Y9GTE0_NEOSC\nMAPKAPK3\tA0A2Y9GTI7_NEOSC\nMAPKAPK3\tA0A2Y9GZ73_NEOSC\nMAPKAPK3\tA0A2Y9NBA1_DELLE\nMAPKAPK3\tA0A340WXR4_LIPVE\nMAPKAPK3\tA0A341BIE4_NEOAA\nMAPKAPK3\tA0A341BJS8_NEOAA\nMAPKAPK3\tA0A341BJU9_NEOAA\nMAPKAPK3\tA0A341BL57_NEOAA\nMAPKAPK3\tA0A341BL71_NEOAA\nMAPKAPK3\tA0A383Z0S0_BALAS\nMAPKAPK3\tA0A383Z0Y8_BALAS\nMAPKAPK3\tA0A384D4F4_URSMA\nMAPKAPK3\tA0A384D583_URSMA\nMAPKAPK3\tA0A3B3DC30_ORYME\nMAPKAPK3\tA0A3B3DDW0_ORYME\nMAPKAPK3\tA0A3B3QW04_9TELE\nMAPKAPK3\tA0A3B3QXC1_9TELE\nMAPKAPK3\tA0A3B3V2M3_9TELE\nMAPKAPK3\tA0A3B3XUH5_9TELE\nMAPKAPK3\tA0A3B4FB32_9CICH\nMAPKAPK3\tA0A3B4TZJ8_SERDU\nMAPKAPK3\tA0A3B4YPG1_SERLL\nMAPKAPK3\tA0A3B4YPH8_SERLL\nMAPKAPK3\tA0A3B4YPJ1_SERLL\nMAPKAPK3\tA0A3B5B7G4_9TELE\nMAPKAPK3\tA0A3B5MQI4_9TELE\nMAPKAPK3\tA0A3B5MSB2_9TELE\nMAPKAPK3\tA0A3B5MSB6_9TELE\nMAPKAPK3\tA0A3B5MT54_9TELE\nMAPKAPK3\tA0A3B5MT59_9TELE\nMAPKAPK3\tA0A3B5MY00_9TELE\nMAPKAPK3\tA0A3B5N0J6_9TELE\nMAPKAPK3\tA0A3P8R3B3_ASTCA\nMAPKAPK3\tA0A3P8TF97_AMPPE\nMAPKAPK3\tA0A3P8UQ44_CYNSE\nMAPKAPK3\tA0A3P8UUS1_CYNSE\nMAPKAPK3\tA0A3P8UUS6_CYNSE\nMAPKAPK3\tA0A3P8UUT5_CYNSE\nMAPKAPK3\tA0A3P9BRV3_9CICH\nMAPKAPK3\tA0A3P9P7B4_POERE\nMAPKAPK3\tA0A3P9P7B7_POERE\nMAPKAPK3\tA0A3P9P7J5_POERE\nMAPKAPK3\tA0A3P9P7L2_POERE\nMAPKAPK3\tA0A3P9P7N3_POERE\nMAPKAPK3\tA0A3P9P7T5_POERE\nMAPKAPK3\tA0A3Q0SU82_AMPCI\nMAPKAPK3\tA0A3Q0SUD5_AMPCI\nMAPKAPK3\tA0A3Q0SXT7_AMPCI\nMAPKAPK3\tA0A3Q0SXX9_AMPCI\nMAPKAPK3\tA0A3Q0T3C4_AMPCI\nMAPKAPK3\tA0A3Q0T8T6_AMPCI\nMAPKAPK3\tA0A3Q1BSV6_AMPOC\nMAPKAPK3\tA0A3Q1GK81_9TELE\nMAPKAPK3\tA0A3Q1IJA0_ANATE\nMAPKAPK3\tA0A3Q1M9S4_BOVIN\nMAPKAPK3\tA0A3Q2CGH2_CYPVA\nMAPKAPK3\tA0A3Q2H7M4_HORSE\nMAPKAPK3\tA0A3Q2IC87_HORSE\nMAPKAPK3\tA0A3Q2L283_HORSE\nMAPKAPK3\tA0A3Q2P2T4_FUNHE\nMAPKAPK3\tA0A3Q2U1E9_CHICK\nMAPKAPK3\tA0A3Q2WLY0_HAPBU\nMAPKAPK3\tA0A3Q2YZT8_HIPCM\nMAPKAPK3\tA0A3Q3BCB7_KRYMA\nMAPKAPK3\tA0A3Q3FR28_9LABR\nMAPKAPK3\tA0A3Q3JU11_MONAL\nMAPKAPK3\tA0A3Q3NHM1_9TELE\nMAPKAPK3\tA0A3Q3XCT8_MOLML\nMAPKAPK3\tA0A3Q3XIB1_MOLML\nMAPKAPK3\tA0A3Q3XIB4_MOLML\nMAPKAPK3\tA0A3Q3XN29_MOLML\nMAPKAPK3\tA0A3Q3XN33_MOLML\nMAPKAPK3\tA0A3Q3XSF1_MOLML\nMAPKAPK3\tA0A3Q4C114_MOLML\nMAPKAPK3\tA0A3Q7Q4P1_CALUR\nMAPKAPK3\tA0A3Q7RC78_VULVU\nMAPKAPK3\tA0A3Q7V655_URSAR\nMAPKAPK3\tA0A3Q7W602_URSAR\nMAPKAPK3\tA0A3Q7WRB3_URSAR\nMAPKAPK3\tA0A3Q7X278_URSAR\nMAPKAPK3\tA0A452EIV8_CAPHI\nMAPKAPK3\tA0A452GQX6_9SAUR\nMAPKAPK3\tA0A452RF65_URSAM\nMAPKAPK3\tA0A452V9B3_URSMA\nMAPKAPK3\tA0A455AUU6_PHYMC\nMAPKAPK3\tA0A455AUV1_PHYMC\nMAPKAPK3\tA0A455AV51_PHYMC\nMAPKAPK3\tA0A455AW58_PHYMC\nMAPKAPK3\tA0A455AW63_PHYMC\nMAPKAPK3\tA0A455AX18_PHYMC\nMAPKAPK3\tA0A455B8W1_PHYMC\nMAPKAPK3\tA0A455B8W5_PHYMC\nMAPKAPK3\tA0A493T5Z0_ANAPP\nMAPKAPK3\tA0A4W2BZA7_BOBOX\nMAPKAPK3\tA0A4W2HJD2_BOBOX\nMAPKAPK3\tA0A4W4GCF3_ELEEL\nMAPKAPK3\tA0A4W4GCI3_ELEEL\nMAPKAPK3\tA0A4W4GF84_ELEEL\nMAPKAPK3\tA0A4W4GF98_ELEEL\nMAPKAPK3\tA0A4W4GFW1_ELEEL\nMAPKAPK3\tA0A4W4GFW6_ELEEL\nMAPKAPK3\tA0A4W4GFX0_ELEEL\nMAPKAPK3\tA0A4W4GFZ0_ELEEL\nMAPKAPK3\tA0A4W4GG03_ELEEL\nMAPKAPK3\tA0A4W4GG09_ELEEL\nMAPKAPK3\tA0A4W5QQK0_9TELE\nMAPKAPK3\tA0A4W5QV91_9TELE\nMAPKAPK3\tA0A4W6EJW5_LATCA\nMAPKAPK3\tA0A4W6EJZ7_LATCA\nMAPKAPK3\tA0A4W6EP97_LATCA\nMAPKAPK3\tA0A4W6EQ92_LATCA\nMAPKAPK3\tA0A4W6EQA6_LATCA\nMAPKAPK3\tA0A4W6EQB2_LATCA\nMAPKAPK3\tA0A4W6EQG7_LATCA\nMAPKAPK3\tA0A4W6EQI7_LATCA\nMAPKAPK3\tA0A4W6EQJ6_LATCA\nMAPKAPK3\tA0A4X1SIW7_PIG\nMAPKAPK3\tA0A4X2LGN7_VOMUR\nMAPKAPK3\tA0A4Z2G7W0_9TELE\nMAPKAPK3\tA0A4Z2H3S9_9TELE\nMAPKAPK3\tA0A5F4W4Q1_CALJA\nMAPKAPK3\tA0A5F4WLE1_CALJA\nMAPKAPK3\tA0A5F5XY34_FELCA\nMAPKAPK3\tA0A5F5Y3C3_FELCA\nMAPKAPK3\tA0A5F8AEA1_MACMU\nMAPKAPK3\tA0A5G3IC31_XENTR\nMAPKAPK3\tA0A5G3IC65_XENTR\nMAPKAPK3\tA0A5G3ICB4_XENTR\nMAPKAPK3\tA0A5G3ICZ1_XENTR\nMAPKAPK3\tA0A5G3IJ30_XENTR\nMAPKAPK3\tA0A5G3IJA2_XENTR\nMAPKAPK3\tA0A5G3IJL5_XENTR\nMAPKAPK3\tA0A5G3IJN4_XENTR\nMAPKAPK3\tA0A5G3IJS3_XENTR\nMAPKAPK3\tA0A5K1UJP6_PIG\nMAPKAPK3\tA1A5X1_DANRE\nMAPKAPK3\tB0QZU6_MOUSE\nMAPKAPK3\tB2L4F5_XENBO\nMAPKAPK3\tB5BUJ8_HUMAN\nMAPKAPK3\tB8XSJ7_PIG\nMAPKAPK3\tC9J8E1_HUMAN\nMAPKAPK3\tC9JPW3_HUMAN\nMAPKAPK3\tD2HER6_AILME\nMAPKAPK3\tE1BT35_CHICK\nMAPKAPK3\tF1PU32_CANLF\nMAPKAPK3\tF6UK47_HORSE\nMAPKAPK3\tF7BFL2_MACMU\nMAPKAPK3\tF7C1G4_XENTR\nMAPKAPK3\tF7CUT8_MONDO\nMAPKAPK3\tF7F6V7_CALJA\nMAPKAPK3\tF7H7S3_MACMU\nMAPKAPK3\tG1KGA4_ANOCA\nMAPKAPK3\tG1MTA1_MELGA\nMAPKAPK3\tG1PEW0_MYOLU\nMAPKAPK3\tG1R5T7_NOMLE\nMAPKAPK3\tG1SNN9_RABIT\nMAPKAPK3\tG3SZA0_LOXAF\nMAPKAPK3\tG3UEE3_LOXAF\nMAPKAPK3\tG5EA63_SARHA\nMAPKAPK3\tG5EA64_SARHA\nMAPKAPK3\tG7NZY5_MACFA\nMAPKAPK3\tH0UX75_CAVPO\nMAPKAPK3\tH0WJN8_OTOGA\nMAPKAPK3\tH0Z2U9_TAEGU\nMAPKAPK3\tH2QMP5_PANTR\nMAPKAPK3\tH2TFV3_TAKRU\nMAPKAPK3\tH3BE73_LATCH\nMAPKAPK3\tH7C0G6_HUMAN\nMAPKAPK3\tH9ESE5_MACMU\nMAPKAPK3\tI3KS63_ORENI\nMAPKAPK3\tI3MJ10_ICTTR\nMAPKAPK3\tK7GCY8_PELSI\nMAPKAPK3\tM3W6Y1_FELCA\nMAPKAPK3\tM3YNF9_MUSPF\nMAPKAPK3\tM3ZHV6_XIPMA\nMAPKAPK3\tMAPK3_BOVIN\nMAPKAPK3\tMAPK3_HUMAN\nMAPKAPK3\tMAPK3_MOUSE\nMAPKAPK3\tMAPK3_RAT\nMAPKAPK3\tQ09HN3_MOUSE\nMAPKAPK3\tQ0IHG8_XENLA\nMAPKAPK3\tQ0IHL7_XENTR\nMAPKAPK3\tQ8BVC2_MOUSE\nMAPKAPK3\tU3K2Q0_FICAL\nMAPKAPK3\tV8P901_OPHHA\nMAPKAPK3\tW5PFH4_SHEEP\n','From\tTo\nPDPK1\tA0A034VB77_BACDO\nPDPK1\tA0A034VDH7_BACDO\nPDPK1\tA0A034VFJ2_BACDO\nPDPK1\tA0A061R5N4_9CHLO\nPDPK1\tA0A061RUV1_9CHLO\nPDPK1\tA0A096MU06_PAPAN\nPDPK1\tA0A0B2V1V6_TOXCA\nPDPK1\tA0A0B2VVT3_TOXCA\nPDPK1\tA0A0D9S0A5_CHLSB\nPDPK1\tA0A0P6K8V7_HETGA\nPDPK1\tA0A0S7FHM5_9TELE\nPDPK1\tA0A0S7FJR3_9TELE\nPDPK1\tA0A0S7FQK3_9TELE\nPDPK1\tA0A0V0RFK9_9BILA\nPDPK1\tA0A0V0RFL4_9BILA\nPDPK1\tA0A0V0TTP2_9BILA\nPDPK1\tA0A0V0TTQ8_9BILA\nPDPK1\tA0A0V0TV09_9BILA\nPDPK1\tA0A0V0WAG7_9BILA\nPDPK1\tA0A0V0YJY2_TRIPS\nPDPK1\tA0A0V0YJZ3_TRIPS\nPDPK1\tA0A0V0YKM5_TRIPS\nPDPK1\tA0A0V0YKP9_TRIPS\nPDPK1\tA0A0V0YKS9_TRIPS\nPDPK1\tA0A0V0YLH6_TRIPS\nPDPK1\tA0A0V0ZBW3_9BILA\nPDPK1\tA0A0V0ZC34_9BILA\nPDPK1\tA0A0V1AWY6_TRISP\nPDPK1\tA0A0V1AX22_TRISP\nPDPK1\tA0A0V1CFQ4_TRIBR\nPDPK1\tA0A0V1DZR8_TRIPS\nPDPK1\tA0A0V1DZZ0_TRIPS\nPDPK1\tA0A0V1FAX0_TRIPS\nPDPK1\tA0A0V1GVF8_9BILA\nPDPK1\tA0A0V1GVN4_9BILA\nPDPK1\tA0A0V1GVU8_9BILA\nPDPK1\tA0A0V1GVV5_9BILA\nPDPK1\tA0A0V1GWA5_9BILA\nPDPK1\tA0A0V1J5R6_TRIPS\nPDPK1\tA0A0V1LU07_9BILA\nPDPK1\tA0A0V1MUD5_9BILA\nPDPK1\tA0A0V1MUT2_9BILA\nPDPK1\tA0A0V1NX71_9BILA\nPDPK1\tA0A0X3PNW6_SCHSO\nPDPK1\tA0A151MHZ4_ALLMI\nPDPK1\tA0A1A7YYA9_9TELE\nPDPK1\tA0A1A8CIV3_9TELE\nPDPK1\tA0A1A8GPX6_9TELE\nPDPK1\tA0A1A8HPE3_NOTKU\nPDPK1\tA0A1A8MQX1_9TELE\nPDPK1\tA0A1A8R5D7_9TELE\nPDPK1\tA0A1A8RTS0_9TELE\nPDPK1\tA0A1A8UMQ3_NOTFU\nPDPK1\tA0A1C7LRE6_GRIFR\nPDPK1\tA0A1D5PJA6_CHICK\nPDPK1\tA0A1D5RKG7_MACMU\nPDPK1\tA0A1J4JFT1_9EUKA\nPDPK1\tA0A1L8EQJ5_XENLA\nPDPK1\tA0A1Q9C0D5_SYMMI\nPDPK1\tA0A1Q9DNK8_SYMMI\nPDPK1\tA0A1S3A212_ERIEU\nPDPK1\tA0A1S3ET17_DIPOR\nPDPK1\tA0A1S3EUI2_DIPOR\nPDPK1\tA0A1U7R619_MESAU\nPDPK1\tA0A1U7SSI9_ALLSI\nPDPK1\tA0A1U7TN28_TARSY\nPDPK1\tA0A1V4KUF1_PATFA\nPDPK1\tA0A218UCT1_9PASE\nPDPK1\tA0A250Y2U6_CASCN\nPDPK1\tA0A251T2F0_HELAN\nPDPK1\tA0A286XI52_CAVPO\nPDPK1\tA0A286XYA5_CAVPO\nPDPK1\tA0A286XZI4_CAVPO\nPDPK1\tA0A286ZV67_PIG\nPDPK1\tA0A287AST6_PIG\nPDPK1\tA0A287CT36_ICTTR\nPDPK1\tA0A287D1S3_ICTTR\nPDPK1\tA0A287D354_ICTTR\nPDPK1\tA0A287DC39_ICTTR\nPDPK1\tA0A2B4SIS8_STYPI\nPDPK1\tA0A2I0A850_9ASPA\nPDPK1\tA0A2I0LWT9_COLLI\nPDPK1\tA0A2I0X589_9ASPA\nPDPK1\tA0A2I3SNM8_PANTR\nPDPK1\tA0A2I3T4G1_PANTR\nPDPK1\tA0A2I3TN41_PANTR\nPDPK1\tA0A2I4B7A6_9TELE\nPDPK1\tA0A2I4B7A7_9TELE\nPDPK1\tA0A2I4B7A8_9TELE\nPDPK1\tA0A2K5C8A7_AOTNA\nPDPK1\tA0A2K5C8B2_AOTNA\nPDPK1\tA0A2K5C8C8_AOTNA\nPDPK1\tA0A2K5C8E5_AOTNA\nPDPK1\tA0A2K5J2S4_COLAP\nPDPK1\tA0A2K5LZE9_CERAT\nPDPK1\tA0A2K5LZF3_CERAT\nPDPK1\tA0A2K5LZG2_CERAT\nPDPK1\tA0A2K5SDL7_CEBCA\nPDPK1\tA0A2K5SDL9_CEBCA\nPDPK1\tA0A2K5SDM8_CEBCA\nPDPK1\tA0A2K5SDM9_CEBCA\nPDPK1\tA0A2K5SDN1_CEBCA\nPDPK1\tA0A2K5US74_MACFA\nPDPK1\tA0A2K5US83_MACFA\nPDPK1\tA0A2K5Y7Z4_MANLE\nPDPK1\tA0A2K5Y836_MANLE\nPDPK1\tA0A2K6EBU2_MACNE\nPDPK1\tA0A2K6EBU8_MACNE\nPDPK1\tA0A2K6EBV5_MACNE\nPDPK1\tA0A2K6ESZ6_PROCO\nPDPK1\tA0A2K6ET06_PROCO\nPDPK1\tA0A2K6ET13_PROCO\nPDPK1\tA0A2K6KU65_RHIBE\nPDPK1\tA0A2K6NFC0_RHIRO\nPDPK1\tA0A2K6T8Q1_SAIBB\nPDPK1\tA0A2K6T8S6_SAIBB\nPDPK1\tA0A2K6T8T1_SAIBB\nPDPK1\tA0A2K6T8T4_SAIBB\nPDPK1\tA0A2P0XJ60_CTEID\nPDPK1\tA0A2P0XJ68_PAROL\nPDPK1\tA0A2R8MSC3_CALJA\nPDPK1\tA0A2R8NFH1_CALJA\nPDPK1\tA0A2U3W585_ODORO\nPDPK1\tA0A2U3W588_ODORO\nPDPK1\tA0A2U4AL17_TURTR\nPDPK1\tA0A2U4AL69_TURTR\nPDPK1\tA0A2U4ALE1_TURTR\nPDPK1\tA0A2U4ALP0_TURTR\nPDPK1\tA0A2Y9HPW2_NEOSC\nPDPK1\tA0A2Y9PXT6_DELLE\nPDPK1\tA0A2Y9PYS4_DELLE\nPDPK1\tA0A2Y9Q418_DELLE\nPDPK1\tA0A2Y9Q4F2_DELLE\nPDPK1\tA0A2Y9Q4F6_DELLE\nPDPK1\tA0A337S312_FELCA\nPDPK1\tA0A337SBQ1_FELCA\nPDPK1\tA0A337SQU1_FELCA\nPDPK1\tA0A340X2S2_LIPVE\nPDPK1\tA0A340X354_LIPVE\nPDPK1\tA0A341CK09_NEOAA\nPDPK1\tA0A341CKI6_NEOAA\nPDPK1\tA0A384A2U4_BALAS\nPDPK1\tA0A384CJG8_URSMA\nPDPK1\tA0A384CKG3_URSMA\nPDPK1\tA0A3G1T1J4_GALME\nPDPK1\tA0A3Q0D9E2_MESAU\nPDPK1\tA0A3Q0FNZ0_ALLSI\nPDPK1\tA0A3Q2LQE3_HORSE\nPDPK1\tA0A3Q7R5H2_VULVU\nPDPK1\tA0A3Q7R5Y0_VULVU\nPDPK1\tA0A3Q7UME7_URSAR\nPDPK1\tA0A3R7FB84_CLOSI\nPDPK1\tA0A452CGL1_BALAS\nPDPK1\tA0A452EQ00_CAPHI\nPDPK1\tA0A452EQ64_CAPHI\nPDPK1\tA0A452IP84_9SAUR\nPDPK1\tA0A452IPG2_9SAUR\nPDPK1\tA0A452RCI6_URSAM\nPDPK1\tA0A455BXP6_PHYMC\nPDPK1\tA0A455BXP9_PHYMC\nPDPK1\tA0A493TRX2_ANAPP\nPDPK1\tA0A4C1V9Y1_9NEOP\nPDPK1\tA0A4W2BL21_BOBOX\nPDPK1\tA0A4W2BL22_BOBOX\nPDPK1\tA0A4W2BNG9_BOBOX\nPDPK1\tA0A4W2BP72_BOBOX\nPDPK1\tA0A4W2DJT5_BOBOX\nPDPK1\tA0A4W2F891_BOBOX\nPDPK1\tA0A4W2F9N7_BOBOX\nPDPK1\tA0A4X1VP61_PIG\nPDPK1\tA0A4X1VTI8_PIG\nPDPK1\tA0A4X1VU11_PIG\nPDPK1\tA0A4X1VU16_PIG\nPDPK1\tA0A4X1VUK9_PIG\nPDPK1\tA0A4X1VUL4_PIG\nPDPK1\tA0A4Y2K5Y0_ARAVE\nPDPK1\tA0A4Z2FEU7_9TELE\nPDPK1\tA0A507R289_MONPU\nPDPK1\tA0A5F4CJD1_CANLF\nPDPK1\tA0A5F4CRF8_CANLF\nPDPK1\tA0A5F5PJM4_HORSE\nPDPK1\tA0A5F5PSW0_HORSE\nPDPK1\tA0A5F8HH07_MONDO\nPDPK1\tA0A5G2QA35_PIG\nPDPK1\tA0A5G2QD73_PIG\nPDPK1\tA0A5G2QIP3_PIG\nPDPK1\tA0A5G2QJE8_PIG\nPDPK1\tA0A5G2QWA6_PIG\nPDPK1\tA0A5G2R5E7_PIG\nPDPK1\tA0A5G3J1B8_XENTR\nPDPK1\tA0A5G3J1E7_XENTR\nPDPK1\tA0A5G3J250_XENTR\nPDPK1\tA0A5G3J2G3_XENTR\nPDPK1\tA4V164_DROME\nPDPK1\tA4V166_DROME\nPDPK1\tC0PUQ1_SALSA\nPDPK1\tE6ZGW1_DICLA\nPDPK1\tE9PER6_HUMAN\nPDPK1\tF1MQJ7_BOVIN\nPDPK1\tF1Q2Y3_CANLF\nPDPK1\tF1SUH3_PIG\nPDPK1\tF2Z3V6_MOUSE\nPDPK1\tF2Z3X6_MOUSE\nPDPK1\tF2Z3X9_MOUSE\nPDPK1\tF2Z3Z9_MOUSE\nPDPK1\tF2Z400_MOUSE\nPDPK1\tF2Z4A5_MOUSE\nPDPK1\tF6VYU8_HORSE\nPDPK1\tF6YJE5_XENTR\nPDPK1\tF6ZD74_MONDO\nPDPK1\tF6ZJI9_XENTR\nPDPK1\tF6ZJK5_XENTR\nPDPK1\tF7B364_MACMU\nPDPK1\tF7DEM4_CALJA\nPDPK1\tF7HA88_CALJA\nPDPK1\tG1MI23_AILME\nPDPK1\tG1N4V9_MELGA\nPDPK1\tG1P2R5_MYOLU\nPDPK1\tG1SGW8_RABIT\nPDPK1\tG3SPQ1_LOXAF\nPDPK1\tG3TYY8_LOXAF\nPDPK1\tG3USR5_MELGA\nPDPK1\tH0V144_CAVPO\nPDPK1\tH0WRF5_OTOGA\nPDPK1\tH0ZGY3_TAEGU\nPDPK1\tH2LUD7_ORYLA\nPDPK1\tH2LUD8_ORYLA\nPDPK1\tH2NPV0_PONAB\nPDPK1\tH2R0W1_PANTR\nPDPK1\tH2USE9_TAKRU\nPDPK1\tH3A0R9_LATCH\nPDPK1\tH3BNV5_HUMAN\nPDPK1\tH3BPR5_HUMAN\nPDPK1\tH3BPW1_HUMAN\nPDPK1\tH3BQ10_HUMAN\nPDPK1\tH3BQA3_HUMAN\nPDPK1\tH5V8B9_DROME\nPDPK1\tH9ESX9_MACMU\nPDPK1\tH9G7Q7_ANOCA\nPDPK1\tI3KVJ6_ORENI\nPDPK1\tI3MK70_ICTTR\nPDPK1\tK7B926_PANTR\nPDPK1\tK7FW78_PELSI\nPDPK1\tK7FW85_PELSI\nPDPK1\tL7N2J7_XENTR\nPDPK1\tM3VXM6_FELCA\nPDPK1\tM3XLP8_LATCH\nPDPK1\tM3YV38_MUSPF\nPDPK1\tM5FMV3_BOVIN\nPDPK1\tPDPK1_ARATH\nPDPK1\tPDPK1_HUMAN\nPDPK1\tPDPK1_MOUSE\nPDPK1\tPDPK1_RAT\nPDPK1\tQ3TRL2_MOUSE\nPDPK1\tQ3UEW8_MOUSE\nPDPK1\tQ3UGN6_MOUSE\nPDPK1\tQ3UHZ0_MOUSE\nPDPK1\tQ5F3U4_CHICK\nPDPK1\tQ6GM85_XENLA\nPDPK1\tQ8K3L3_MOUSE\nPDPK1\tT2MIS8_HYDVU\nPDPK1\tU3J7W0_ANAPP\nPDPK1\tU3KI32_FICAL\nPDPK1\tU3KI33_FICAL\nPDPK1\tU6CWC8_NEOVI\nPDPK1\tV5FT64_ANOGL\nPDPK1\tW5QBA7_SHEEP\nPDPK1\tW8B117_CERCA\nPDPK1\tW8B3L4_CERCA\nPDPK1\tW8BIR5_CERCA\n','From\tTo\nMAPKAPK2\tA0A087WSN7_MOUSE\nMAPKAPK2\tA0A096MPQ4_PAPAN\nMAPKAPK2\tA0A096NF58_PAPAN\nMAPKAPK2\tA0A0B2V130_TOXCA\nMAPKAPK2\tA0A0B2V9K6_TOXCA\nMAPKAPK2\tA0A0B2VHM9_TOXCA\nMAPKAPK2\tA0A0D9RR16_CHLSB\nMAPKAPK2\tA0A1D5QDZ0_MACMU\nMAPKAPK2\tA0A1S2ZQX9_ERIEU\nMAPKAPK2\tA0A1S3F939_DIPOR\nMAPKAPK2\tA0A1U7QVC3_MESAU\nMAPKAPK2\tA0A1U7RVW1_ALLSI\nMAPKAPK2\tA0A1U7T1D6_TARSY\nMAPKAPK2\tA0A1V4KX88_PATFA\nMAPKAPK2\tA0A218UKH3_9PASE\nMAPKAPK2\tA0A250YF24_CASCN\nMAPKAPK2\tA0A286XQ19_CAVPO\nMAPKAPK2\tA0A2B4SU17_STYPI\nMAPKAPK2\tA0A2D0PJW2_ICTPU\nMAPKAPK2\tA0A2D0PLJ6_ICTPU\nMAPKAPK2\tA0A2I0LZX9_COLLI\nMAPKAPK2\tA0A2I2V3A8_FELCA\nMAPKAPK2\tA0A2I3FVD4_NOMLE\nMAPKAPK2\tA0A2I3LMN6_PAPAN\nMAPKAPK2\tA0A2I4APX6_9TELE\nMAPKAPK2\tA0A2K5D8K7_AOTNA\nMAPKAPK2\tA0A2K5D8L7_AOTNA\nMAPKAPK2\tA0A2K5JYM9_COLAP\nMAPKAPK2\tA0A2K5JYP1_COLAP\nMAPKAPK2\tA0A2K5JYP6_COLAP\nMAPKAPK2\tA0A2K5N5P6_CERAT\nMAPKAPK2\tA0A2K5N5Q4_CERAT\nMAPKAPK2\tA0A2K5N5Q7_CERAT\nMAPKAPK2\tA0A2K5QV21_CEBCA\nMAPKAPK2\tA0A2K5QV34_CEBCA\nMAPKAPK2\tA0A2K5QV41_CEBCA\nMAPKAPK2\tA0A2K5V8U8_MACFA\nMAPKAPK2\tA0A2K5V8V7_MACFA\nMAPKAPK2\tA0A2K5V8X7_MACFA\nMAPKAPK2\tA0A2K5XQV6_MANLE\nMAPKAPK2\tA0A2K5XQW1_MANLE\nMAPKAPK2\tA0A2K5XQX6_MANLE\nMAPKAPK2\tA0A2K6E4S0_MACNE\nMAPKAPK2\tA0A2K6E4T0_MACNE\nMAPKAPK2\tA0A2K6E4T1_MACNE\nMAPKAPK2\tA0A2K6EWN5_PROCO\nMAPKAPK2\tA0A2K6EWN7_PROCO\nMAPKAPK2\tA0A2K6MEZ3_RHIBE\nMAPKAPK2\tA0A2K6QK79_RHIRO\nMAPKAPK2\tA0A2K6U003_SAIBB\nMAPKAPK2\tA0A2K6U038_SAIBB\nMAPKAPK2\tA0A2K6U053_SAIBB\nMAPKAPK2\tA0A2R8NFJ7_CALJA\nMAPKAPK2\tA0A2U3X5H5_ODORO\nMAPKAPK2\tA0A2U3ZYH1_ODORO\nMAPKAPK2\tA0A2U3ZYH9_ODORO\nMAPKAPK2\tA0A2Y9GK99_NEOSC\nMAPKAPK2\tA0A2Y9GLY8_NEOSC\nMAPKAPK2\tA0A2Y9PVR7_DELLE\nMAPKAPK2\tA0A340YCL2_LIPVE\nMAPKAPK2\tA0A341BVE4_NEOAA\nMAPKAPK2\tA0A383Z965_BALAS\nMAPKAPK2\tA0A384BMM7_URSMA\nMAPKAPK2\tA0A3B4TA54_SERDU\nMAPKAPK2\tA0A3B4X6J7_SERLL\nMAPKAPK2\tA0A3B5A1B8_9TELE\nMAPKAPK2\tA0A3B5KPU1_TAKRU\nMAPKAPK2\tA0A3P8SBW3_AMPPE\nMAPKAPK2\tA0A3P8SD14_AMPPE\nMAPKAPK2\tA0A3Q1BA27_AMPOC\nMAPKAPK2\tA0A3Q1C930_AMPOC\nMAPKAPK2\tA0A3Q1M2S5_BOVIN\nMAPKAPK2\tA0A3Q1M614_BOVIN\nMAPKAPK2\tA0A3Q1M8E4_BOVIN\nMAPKAPK2\tA0A3Q1MET0_BOVIN\nMAPKAPK2\tA0A3Q3FB63_9LABR\nMAPKAPK2\tA0A3Q7MSU8_CALUR\nMAPKAPK2\tA0A3Q7MUM1_CALUR\nMAPKAPK2\tA0A3Q7MUQ2_CALUR\nMAPKAPK2\tA0A3Q7RAW2_VULVU\nMAPKAPK2\tA0A3Q7V0R6_URSAR\nMAPKAPK2\tA0A3R7CHB3_CLOSI\nMAPKAPK2\tA0A452GA94_CAPHI\nMAPKAPK2\tA0A452GAA6_CAPHI\nMAPKAPK2\tA0A452HDM0_9SAUR\nMAPKAPK2\tA0A452HE01_9SAUR\nMAPKAPK2\tA0A452HE05_9SAUR\nMAPKAPK2\tA0A452HE48_9SAUR\nMAPKAPK2\tA0A452HE54_9SAUR\nMAPKAPK2\tA0A452HE82_9SAUR\nMAPKAPK2\tA0A452Q7X1_URSAM\nMAPKAPK2\tA0A452VNI6_URSMA\nMAPKAPK2\tA0A455B8K5_PHYMC\nMAPKAPK2\tA0A4P3APB9_MOUSE\nMAPKAPK2\tA0A4W2CZX1_BOBOX\nMAPKAPK2\tA0A4W2FGU4_BOBOX\nMAPKAPK2\tA0A4W2FI01_BOBOX\nMAPKAPK2\tA0A4W2HGR5_BOBOX\nMAPKAPK2\tA0A4W4GIJ1_ELEEL\nMAPKAPK2\tA0A4W4GK90_ELEEL\nMAPKAPK2\tA0A4W4GKB1_ELEEL\nMAPKAPK2\tA0A4W4GN94_ELEEL\nMAPKAPK2\tA0A4W4GNY1_ELEEL\nMAPKAPK2\tA0A4W6DME0_LATCA\nMAPKAPK2\tA0A4W6DMF8_LATCA\nMAPKAPK2\tA0A4W6DMJ0_LATCA\nMAPKAPK2\tA0A4W6DPF3_LATCA\nMAPKAPK2\tA0A4W6DPJ2_LATCA\nMAPKAPK2\tA0A4W6DPV2_LATCA\nMAPKAPK2\tA0A4W6DPX7_LATCA\nMAPKAPK2\tA0A4X1US57_PIG\nMAPKAPK2\tA0A4X2L1D1_VOMUR\nMAPKAPK2\tA0A4Z2G575_9TELE\nMAPKAPK2\tA0A5F4W6H7_CALJA\nMAPKAPK2\tA0A5F4WBK2_CALJA\nMAPKAPK2\tA0A5F5PGM0_HORSE\nMAPKAPK2\tF1M9C0_RAT\nMAPKAPK2\tF1MCX5_BOVIN\nMAPKAPK2\tF1NCY7_CHICK\nMAPKAPK2\tF1SEZ4_PIG\nMAPKAPK2\tF6PX17_ORNAN\nMAPKAPK2\tF6UZ64_MONDO\nMAPKAPK2\tF6X3P7_HORSE\nMAPKAPK2\tF7CQI0_XENTR\nMAPKAPK2\tF7HHY8_MACMU\nMAPKAPK2\tF7IKB1_CALJA\nMAPKAPK2\tG1MGL7_AILME\nMAPKAPK2\tG1MXD6_MELGA\nMAPKAPK2\tG1NVK0_MYOLU\nMAPKAPK2\tG1QJ02_NOMLE\nMAPKAPK2\tG3N5D5_GASAC\nMAPKAPK2\tG3TLC6_LOXAF\nMAPKAPK2\tG3X0D0_SARHA\nMAPKAPK2\tH0VZ75_CAVPO\nMAPKAPK2\tH0X4J4_OTOGA\nMAPKAPK2\tH2MLY3_ORYLA\nMAPKAPK2\tH2N3X6_PONAB\nMAPKAPK2\tH2Q104_PANTR\nMAPKAPK2\tH2R1R9_PANTR\nMAPKAPK2\tH3ABJ2_LATCH\nMAPKAPK2\tH9FCK1_MACMU\nMAPKAPK2\tH9GD55_ANOCA\nMAPKAPK2\tI3KT30_ORENI\nMAPKAPK2\tI3M7T3_ICTTR\nMAPKAPK2\tK7D549_PANTR\nMAPKAPK2\tK7EIB9_RABIT\nMAPKAPK2\tK7F6T5_PELSI\nMAPKAPK2\tM3WIZ9_FELCA\nMAPKAPK2\tM3YEZ1_MUSPF\nMAPKAPK2\tMAPK2_CRILO\nMAPKAPK2\tMAPK2_HUMAN\nMAPKAPK2\tMAPK2_MOUSE\nMAPKAPK2\tMAPK2_RABIT\nMAPKAPK2\tQ28FG8_XENTR\nMAPKAPK2\tQ3U2P8_MOUSE\nMAPKAPK2\tQ6IRB4_XENLA\nMAPKAPK2\tQ80ZF4_RAT\nMAPKAPK2\tQ8R3U8_MOUSE\nMAPKAPK2\tT2MG35_HYDVU\nMAPKAPK2\tU3J6V1_ANAPP\nMAPKAPK2\tU3JCS7_FICAL\nMAPKAPK2\tV8NRV6_OPHHA\nMAPKAPK2\tW5P867_SHEEP\nMAPKAPK2\tW5UD67_ICTPU\n','From\tTo\nMAPK14\tA0A023NLJ0_MYAAR\nMAPK14\tA0A024RD15_HUMAN\nMAPK14\tA0A096NHR3_PAPAN\nMAPK14\tA0A0A7D769_ERISI\nMAPK14\tA0A0A9WHM8_LYGHE\nMAPK14\tA0A0D9RCS6_CHLSB\nMAPK14\tA0A0N8ETG7_HETGA\nMAPK14\tA0A0V0S4F3_9BILA\nMAPK14\tA0A0V0UF60_9BILA\nMAPK14\tA0A0V0X3C0_9BILA\nMAPK14\tA0A0V0XZ70_TRIPS\nMAPK14\tA0A0V0Y045_TRIPS\nMAPK14\tA0A0V0Y067_TRIPS\nMAPK14\tA0A0V1A8S9_9BILA\nMAPK14\tA0A0V1D7T9_TRIBR\nMAPK14\tA0A0V1EFP6_TRIPS\nMAPK14\tA0A0V1FQ84_TRIPS\nMAPK14\tA0A0V1JFA4_TRIPS\nMAPK14\tA0A0V1JFA5_TRIPS\nMAPK14\tA0A0V1KZH4_9BILA\nMAPK14\tA0A0V1MGR5_9BILA\nMAPK14\tA0A0V1NSE2_9BILA\nMAPK14\tA0A151N8L6_ALLMI\nMAPK14\tA0A1A7WEJ0_9TELE\nMAPK14\tA0A1A7X5M3_9TELE\nMAPK14\tA0A1A7YCL9_9TELE\nMAPK14\tA0A1A7YYD9_9TELE\nMAPK14\tA0A1A7Z9G4_NOTFU\nMAPK14\tA0A1A7ZNK0_NOTFU\nMAPK14\tA0A1A8AAJ3_NOTFU\nMAPK14\tA0A1A8BGX6_9TELE\nMAPK14\tA0A1A8BJI1_9TELE\nMAPK14\tA0A1A8CBK4_9TELE\nMAPK14\tA0A1A8CGJ5_9TELE\nMAPK14\tA0A1A8EAM4_9TELE\nMAPK14\tA0A1A8EE08_9TELE\nMAPK14\tA0A1A8EJY0_9TELE\nMAPK14\tA0A1A8EVD0_9TELE\nMAPK14\tA0A1A8FTK6_9TELE\nMAPK14\tA0A1A8GFV2_9TELE\nMAPK14\tA0A1A8I4J6_NOTKU\nMAPK14\tA0A1A8IIS6_NOTKU\nMAPK14\tA0A1A8JIU0_NOTKU\nMAPK14\tA0A1A8KCM4_NOTKU\nMAPK14\tA0A1A8LPB3_9TELE\nMAPK14\tA0A1A8LSS0_9TELE\nMAPK14\tA0A1A8MXB7_9TELE\nMAPK14\tA0A1A8N570_9TELE\nMAPK14\tA0A1A8NKY8_9TELE\nMAPK14\tA0A1A8P5U4_9TELE\nMAPK14\tA0A1A8PMD2_9TELE\nMAPK14\tA0A1A8PN21_9TELE\nMAPK14\tA0A1A8QNE7_9TELE\nMAPK14\tA0A1A8RDS0_9TELE\nMAPK14\tA0A1A8UDM9_NOTFU\nMAPK14\tA0A1A8V5A5_NOTFU\nMAPK14\tA0A1A8VGK0_NOTFU\nMAPK14\tA0A1D5PIQ5_CHICK\nMAPK14\tA0A1S2Z087_CICAR\nMAPK14\tA0A1S3AFP5_ERIEU\nMAPK14\tA0A1S3AFT5_ERIEU\nMAPK14\tA0A1S3AFU2_ERIEU\nMAPK14\tA0A1S3AFZ2_ERIEU\nMAPK14\tA0A1S3EFT9_CICAR\nMAPK14\tA0A1S3G8B7_DIPOR\nMAPK14\tA0A1S3G8Q3_DIPOR\nMAPK14\tA0A1S3G9Z2_DIPOR\nMAPK14\tA0A1S3WQ55_ERIEU\nMAPK14\tA0A1U7QS76_MESAU\nMAPK14\tA0A1U7R0J2_MESAU\nMAPK14\tA0A1U7RWW4_ALLSI\nMAPK14\tA0A1U7S6Q9_ALLSI\nMAPK14\tA0A1U8DFZ6_ALLSI\nMAPK14\tA0A1V4KWL2_PATFA\nMAPK14\tA0A1V4KWS3_PATFA\nMAPK14\tA0A1V4KWZ5_PATFA\nMAPK14\tA0A218UJ94_9PASE\nMAPK14\tA0A250XXX4_CASCN\nMAPK14\tA0A286XAN4_CAVPO\nMAPK14\tA0A286XPX6_CAVPO\nMAPK14\tA0A286Y3G5_CAVPO\nMAPK14\tA0A287AX01_PIG\nMAPK14\tA0A287BHS8_PIG\nMAPK14\tA0A287CTH9_ICTTR\nMAPK14\tA0A287D4U0_ICTTR\nMAPK14\tA0A2B4SN45_STYPI\nMAPK14\tA0A2I3FPL0_NOMLE\nMAPK14\tA0A2I3GMD6_NOMLE\nMAPK14\tA0A2I3GMJ2_NOMLE\nMAPK14\tA0A2I3LNN6_PAPAN\nMAPK14\tA0A2I3M138_PAPAN\nMAPK14\tA0A2I3MUV8_PAPAN\nMAPK14\tA0A2I3N9L4_PAPAN\nMAPK14\tA0A2I3NHI7_PAPAN\nMAPK14\tA0A2I3RMP6_PANTR\nMAPK14\tA0A2I3S4B9_PANTR\nMAPK14\tA0A2I3SDR9_PANTR\nMAPK14\tA0A2J8NZ39_PANTR\nMAPK14\tA0A2K5F7V1_AOTNA\nMAPK14\tA0A2K5F7V8_AOTNA\nMAPK14\tA0A2K5F7X1_AOTNA\nMAPK14\tA0A2K5F7X7_AOTNA\nMAPK14\tA0A2K5F7X9_AOTNA\nMAPK14\tA0A2K5F838_AOTNA\nMAPK14\tA0A2K5KET1_COLAP\nMAPK14\tA0A2K5KET4_COLAP\nMAPK14\tA0A2K5KET6_COLAP\nMAPK14\tA0A2K5KET9_COLAP\nMAPK14\tA0A2K5KEU1_COLAP\nMAPK14\tA0A2K5KEU2_COLAP\nMAPK14\tA0A2K5NM43_CERAT\nMAPK14\tA0A2K5NM46_CERAT\nMAPK14\tA0A2K5NM48_CERAT\nMAPK14\tA0A2K5NMA9_CERAT\nMAPK14\tA0A2K5NMB4_CERAT\nMAPK14\tA0A2K5NMC4_CERAT\nMAPK14\tA0A2K5RF45_CEBCA\nMAPK14\tA0A2K5RF58_CEBCA\nMAPK14\tA0A2K5RF91_CEBCA\nMAPK14\tA0A2K5RFA1_CEBCA\nMAPK14\tA0A2K5RFA9_CEBCA\nMAPK14\tA0A2K5RFB5_CEBCA\nMAPK14\tA0A2K5RFB9_CEBCA\nMAPK14\tA0A2K5WJV8_MACFA\nMAPK14\tA0A2K5WJW8_MACFA\nMAPK14\tA0A2K5WJY5_MACFA\nMAPK14\tA0A2K5WK12_MACFA\nMAPK14\tA0A2K6AEL6_MANLE\nMAPK14\tA0A2K6AEM4_MANLE\nMAPK14\tA0A2K6AEN4_MANLE\nMAPK14\tA0A2K6AEQ9_MANLE\nMAPK14\tA0A2K6EB32_MACNE\nMAPK14\tA0A2K6EB35_MACNE\nMAPK14\tA0A2K6EB37_MACNE\nMAPK14\tA0A2K6EB38_MACNE\nMAPK14\tA0A2K6EB39_MACNE\nMAPK14\tA0A2K6EB43_MACNE\nMAPK14\tA0A2K6FTR2_PROCO\nMAPK14\tA0A2K6FTS3_PROCO\nMAPK14\tA0A2K6LWL1_RHIBE\nMAPK14\tA0A2K6LWM6_RHIBE\nMAPK14\tA0A2K6LWN1_RHIBE\nMAPK14\tA0A2K6LWN5_RHIBE\nMAPK14\tA0A2K6LWQ1_RHIBE\nMAPK14\tA0A2K6QA59_RHIRO\nMAPK14\tA0A2K6QA65_RHIRO\nMAPK14\tA0A2K6QA67_RHIRO\nMAPK14\tA0A2K6QA72_RHIRO\nMAPK14\tA0A2K6QA80_RHIRO\nMAPK14\tA0A2K6QA81_RHIRO\nMAPK14\tA0A2K6TTF9_SAIBB\nMAPK14\tA0A2K6TTH9_SAIBB\nMAPK14\tA0A2K6TTI1_SAIBB\nMAPK14\tA0A2K6TTQ4_SAIBB\nMAPK14\tA0A2K6TTQ9_SAIBB\nMAPK14\tA0A2K6TTS0_SAIBB\nMAPK14\tA0A2R9C8T7_PANPA\nMAPK14\tA0A2R9C8U0_PANPA\nMAPK14\tA0A2R9CBM8_PANPA\nMAPK14\tA0A2R9CJF3_PANPA\nMAPK14\tA0A2R9CJI7_PANPA\nMAPK14\tA0A2U3WLM5_ODORO\nMAPK14\tA0A2U3WLM8_ODORO\nMAPK14\tA0A2U3WLS2_ODORO\nMAPK14\tA0A2U3ZQ58_ODORO\nMAPK14\tA0A2U4CIE8_TURTR\nMAPK14\tA0A2U4CIF1_TURTR\nMAPK14\tA0A2U4CIF3_TURTR\nMAPK14\tA0A2U4CIF4_TURTR\nMAPK14\tA0A2U4CIG0_TURTR\nMAPK14\tA0A2U4CIG1_TURTR\nMAPK14\tA0A2U7QVI4_TACFU\nMAPK14\tA0A2Y9GMC7_NEOSC\nMAPK14\tA0A2Y9GMI8_NEOSC\nMAPK14\tA0A2Y9GR77_NEOSC\nMAPK14\tA0A2Y9NQP5_DELLE\nMAPK14\tA0A2Y9NTT2_DELLE\nMAPK14\tA0A2Y9P5K9_DELLE\nMAPK14\tA0A2Y9SFQ4_PHYMC\nMAPK14\tA0A2Y9SMC1_PHYMC\nMAPK14\tA0A2Y9SP66_PHYMC\nMAPK14\tA0A337S6R4_FELCA\nMAPK14\tA0A340Y132_LIPVE\nMAPK14\tA0A340Y4M3_LIPVE\nMAPK14\tA0A340Y744_LIPVE\nMAPK14\tA0A341D3N7_NEOAA\nMAPK14\tA0A341D3U4_NEOAA\nMAPK14\tA0A341D6G6_NEOAA\nMAPK14\tA0A345DKR7_9TELE\nMAPK14\tA0A384BEB8_BALAS\nMAPK14\tA0A384BFI2_BALAS\nMAPK14\tA0A384C9Y1_URSMA\nMAPK14\tA0A3B2WAZ7_MOUSE\nMAPK14\tA0A3B2WB60_MOUSE\nMAPK14\tA0A3B3QTM4_9TELE\nMAPK14\tA0A3B3QUD5_9TELE\nMAPK14\tA0A3G2LZC0_CTEID\nMAPK14\tA0A3Q0DZV5_TARSY\nMAPK14\tA0A3Q0GPS8_ALLSI\nMAPK14\tA0A3Q0GQX1_ALLSI\nMAPK14\tA0A3Q0GUJ3_ALLSI\nMAPK14\tA0A3Q1LTT9_BOVIN\nMAPK14\tA0A3Q1M517_BOVIN\nMAPK14\tA0A3Q1M8H8_BOVIN\nMAPK14\tA0A3Q2U670_CHICK\nMAPK14\tA0A3Q7N970_CALUR\nMAPK14\tA0A3Q7NDB6_CALUR\nMAPK14\tA0A3Q7NI37_CALUR\nMAPK14\tA0A3Q7PQ83_CALUR\nMAPK14\tA0A3Q7RLA9_VULVU\nMAPK14\tA0A3Q7RWX3_VULVU\nMAPK14\tA0A3Q7SW21_VULVU\nMAPK14\tA0A3Q7TR11_URSAR\nMAPK14\tA0A3Q7UE84_URSAR\nMAPK14\tA0A3Q7UQV4_URSAR\nMAPK14\tA0A3Q7V6I8_URSAR\nMAPK14\tA0A3Q7VKM9_URSAR\nMAPK14\tA0A3R7F6Z4_CLOSI\nMAPK14\tA0A419PEA5_CLOSI\nMAPK14\tA0A452C6H0_BALAS\nMAPK14\tA0A452DM03_CAPHI\nMAPK14\tA0A452DM18_CAPHI\nMAPK14\tA0A452HB01_9SAUR\nMAPK14\tA0A452HB03_9SAUR\nMAPK14\tA0A452SIF9_URSAM\nMAPK14\tA0A452TZK7_URSMA\nMAPK14\tA0A452TZR3_URSMA\nMAPK14\tA0A493T1U9_ANAPP\nMAPK14\tA0A493TQ25_ANAPP\nMAPK14\tA0A4W2BZG5_BOBOX\nMAPK14\tA0A4W2BZG9_BOBOX\nMAPK14\tA0A4W2DU54_BOBOX\nMAPK14\tA0A4W2DUS0_BOBOX\nMAPK14\tA0A4W2FJE1_BOBOX\nMAPK14\tA0A4W2FJF2_BOBOX\nMAPK14\tA0A4W2HIH0_BOBOX\nMAPK14\tA0A4W2HJ64_BOBOX\nMAPK14\tA0A4W2HLL8_BOBOX\nMAPK14\tA0A4W2HMD9_BOBOX\nMAPK14\tA0A4W2HMM1_BOBOX\nMAPK14\tA0A4W3GI32_CALMI\nMAPK14\tA0A4W3GIJ0_CALMI\nMAPK14\tA0A4W3GIJ5_CALMI\nMAPK14\tA0A4W3GIS4_CALMI\nMAPK14\tA0A4W3GK53_CALMI\nMAPK14\tA0A4X1SVG2_PIG\nMAPK14\tA0A4X1SVL1_PIG\nMAPK14\tA0A4X1SWQ7_PIG\nMAPK14\tA0A4X1SWW5_PIG\nMAPK14\tA0A4X1SX27_PIG\nMAPK14\tA0A4X1SXA0_PIG\nMAPK14\tA0A4X2LU17_VOMUR\nMAPK14\tA0A4X2LUF0_VOMUR\nMAPK14\tA0A5F4BRL5_CANLF\nMAPK14\tA0A5F5PIA4_HORSE\nMAPK14\tA0A5F5Q2X4_HORSE\nMAPK14\tA0A5F8GJU8_MONDO\nMAPK14\tA0A5G2QD04_PIG\nMAPK14\tA0A5G2QSG6_PIG\nMAPK14\tA0A5G2R9K2_PIG\nMAPK14\tA0A5G3H2J6_XENTR\nMAPK14\tA6QLR9_BOVIN\nMAPK14\tB2KF34_MOUSE\nMAPK14\tB4E0K5_HUMAN\nMAPK14\tB5TY33_HUMAN\nMAPK14\tB7TJ15_SHEEP\nMAPK14\tD2KW21_XENLA\nMAPK14\tD3K4J3_CRIGR\nMAPK14\tE7EX54_HUMAN\nMAPK14\tF1RYA1_PIG\nMAPK14\tF6PU21_BOVIN\nMAPK14\tF6V370_MACMU\nMAPK14\tF6YAB9_MONDO\nMAPK14\tF7CWM1_HORSE\nMAPK14\tF7DNE5_HORSE\nMAPK14\tF8UFC4_SOLLC\nMAPK14\tG1L0U5_AILME\nMAPK14\tG1MQ37_MELGA\nMAPK14\tG1P5Z7_MYOLU\nMAPK14\tG1UHC5_PENJP\nMAPK14\tG3SVM3_LOXAF\nMAPK14\tG3U799_LOXAF\nMAPK14\tG3V617_RAT\nMAPK14\tG3VFI7_SARHA\nMAPK14\tG7MRU1_MACMU\nMAPK14\tG7P3B2_MACFA\nMAPK14\tH0VS33_CAVPO\nMAPK14\tH0WLI7_OTOGA\nMAPK14\tH2BEZ3_POPBA\nMAPK14\tH2BEZ4_POPDE\nMAPK14\tH2BEZ5_9ROSI\nMAPK14\tH2BEZ6_POPNI\nMAPK14\tH2PIU7_PONAB\nMAPK14\tH2ZWK3_LATCH\nMAPK14\tH3CIZ0_TETNG\nMAPK14\tH7C4E2_HUMAN\nMAPK14\tH9GNU5_ANOCA\nMAPK14\tI0FG08_MACMU\nMAPK14\tI0FG09_MACMU\nMAPK14\tI3MB86_ICTTR\nMAPK14\tI3NFA8_ICTTR\nMAPK14\tK7AZV2_PANTR\nMAPK14\tK7BNX5_PANTR\nMAPK14\tK7FGX7_PELSI\nMAPK14\tL0B9G5_POPTN\nMAPK14\tL0B9J3_POPMA\nMAPK14\tL0BBD4_POPAL\nMAPK14\tL0BBP0_POPTR\nMAPK14\tL7RSM2_HUMAN\nMAPK14\tL7X0J9_PENVA\nMAPK14\tM3X6W3_FELCA\nMAPK14\tM3XAW7_FELCA\nMAPK14\tM3XXU5_MUSPF\nMAPK14\tMK14A_DANRE\nMAPK14\tMK14B_DANRE\nMAPK14\tMK14_CANLF\nMAPK14\tMK14_HUMAN\nMAPK14\tMK14_MOUSE\nMAPK14\tMK14_PANTR\nMAPK14\tMK14_RAT\nMAPK14\tMK14_XENLA\nMAPK14\tQ28F23_XENTR\nMAPK14\tQ56A33_RAT\nMAPK14\tQ5U421_MOUSE\nMAPK14\tQ6DJ17_XENTR\nMAPK14\tQ6TNT1_DANRE\nMAPK14\tT2MD92_HYDVU\nMAPK14\tU3DHS6_CALJA\nMAPK14\tU3EP06_CALJA\nMAPK14\tU3I4B4_ANAPP\nMAPK14\tU3JLS8_FICAL\nMAPK14\tV9KMA8_CALMI\n']

In [41]:
# Split up the search results into a list of lists

results2=[]

for n,i in enumerate(results):
    results2.append(i.split())

In [42]:
# Make a dictionary of proteins

proteindict={}

for i in results2: # i = one kinase and all of its possible translations
    if(len(i)) > 2: # ignore empty lists
        humans = 0
        for e in range(3,len(i),2): # For every other item in the list (i.e. a potentially correct ID)
            if "_HUMAN" in str(i[e]): # check how many "human" options there are
                 humans += 1 
        if humans > 1: # If there are multiple "human" options
            for j in range(3,len(i),2):
                if str(i[j]) == str(i[2]) + "_HUMAN": # keep the one that equates to the original ID with
                    # suffix "_HUMAN"
                    proteindict[str(i[2])] = str(i[j])
        if humans == 1: # If there is only one "human" option, choose it
            for h in range(3,len(i),2):
                if "_HUMAN" in str(i[h]):
                    proteindict[str(i[2])] = str(i[h])

In [43]:
# Translate the kinases in the data frame and insert into new column

data = []

for n,i in enumerate(inhibitors_df.Kinase): # Iterate over original data frame's IDs
    if i in proteindict.keys():
        kinase = proteindict.get(i)
        data.append(kinase)
    else:
        data.append(str(i) + "_NOT_FOUND_IN_UNIPROT")

data = pd.Series(data)
inhibitors_df = inhibitors_df.assign(HUMAN_KINASE = data)

In [1]:
### Using UniProt ID dictionary obtained previously, translate the kinases 
# in the data frame and insert into new column

data = []

for n,i in enumerate(inhibitors_df.Kinase): # Iterate over original data frame's IDs
    if i in kin_dict.keys():
        kinase = kin_dict.get(i)
        data.append(kinase)
    else:
        data.append(str(i) + "_UNIPROT_ID_UNAVAILABLE")

data = pd.Series(data)
inhibitors_df = inhibitors_df.assign(ACC_ID = data)

Make a temporary column combining the inhibitor and kinase names, to check for duplicates

In [45]:
unique = []

for n,i in enumerate(inhibitors_df.Inhibitor):
    uniq = str(i)+str(inhibitors_df.Kinase[n])
    unique.append(uniq)

unique = pd.Series(unique)
inhibitors_df = inhibitors_df.assign(UNIQUE = unique)

Drop any duplicate kinase-inhibitor pairs and reset the indices

In [46]:
inhibitors_df = inhibitors_df.drop_duplicates( subset = "UNIQUE" )
inhibitors_df = inhibitors_df.reset_index( drop = True )

Remove any columns not required for the web app

In [47]:
inhibitors_df = inhibitors_df.drop(["Index", "To_Remove", "Partial_Img_URL", "Link", "UNIQUE"], axis = 1)

Make data frame of kinase-inhibitor pairs

In [48]:
inhib_kin_df = inhibitors_df[['Kinase', 'Inhibitor', 'HUMAN_KINASE']]

Make data frame of inhibitors

In [49]:
inhibitors_df = inhibitors_df.drop_duplicates(subset = "Inhibitor")
inhibitors_df = inhibitors_df.drop(["Kinase", "HUMAN_KINASE", "ACC_ID"], axis = 1)
inhibitors_df = inhibitors_df.reset_index(drop = True)

Make a column of primary keys

In [50]:
prim_key = []

count = 1

for i in inhibitors_df.Inhibitor:
    key = "IN"+"{:07d}".format(count)
    prim_key.append(key)
    count += 1

prim_key = pd.Series(prim_key)

inhibitors_df = inhibitors_df.assign(ID_IN = prim_key)

In [51]:
prim_key = []

count = 1

for i in inhib_kin_df.Inhibitor:
    key = "KI"+"{:07d}".format(count)
    prim_key.append(key)
    count += 1

prim_key = pd.Series(prim_key)

inhib_kin_df = inhib_kin_df.assign(ID_KI = prim_key)

Write to csv

In [52]:
inhib_kin_df.to_csv("inhib_kin.csv", index = False)

In [53]:
inhibitors_df.to_csv("inhibitors.csv", index = False)